# 7. Analiza sentimenta

In [23]:
import os
import math

from test_v07 import *

DATA_PATH = './data/imdb1/'
STOPWORD_FILENAME = './data/english.stop'


## 7.1 Izgradnja modela dokumenata

Funkcija **make_datasets()** ima sljedeće ulazne i izlazne parametre
* ulaz:
    * **data_path** putanja do datoteka
* izlaz:
    * lista oblika  

<pre>
0: train: pos: [filename, filename, ...]
          neg: [filename, filename, ...]
   test:  pos:  [filename, filename, ...]  
          neg:  [filename, filename, ...]  
1: train: pos: [filename, filename, ...]  
          neg: [filename, filename, ...]  
   test:  pos:  [filename, filename, ...]  
          neg:  [filename, filename, ...]  
...  
...
</pre>

Svaki element liste se zove "preklop" (engl. fold) i sastoji se od dokumenata za treniranje i testiranje. 
Preklop je rječnik sa dva ključa "train" i "test" čije su vrijednosti rječnik s dva ključa "pos" i "neg" čije su vrijednosti lista datoteka koje su "pos" ili "neg" kritike.

In [24]:
def make_datasets(data_path, numfolds = 10):
    """
    ulaz:
    -data_path: putanja do datoteka
    izlaz:
    -lista oblika
    0: train: pos: [filename, filename, ...]
              neg: [filename, filename, ...]
       test: pos:  [filename, filename, ...]
             neg:  [filename, filename, ...]
    1: train: pos: [filename, filename, ...]
              neg: [filename, filename, ...]
       test: pos:  [filename, filename, ...]
             neg:  [filename, filename, ...]
    ...
    ...
    Svaki element liste je rječnik sa dva ključa "train" i "test"
    čije su vrijednosti rječnik s dva ključa "pos" i "neg" čije su
    vrijednosti lista datoteka koje su "pos" ili "neg" kritike
    """
    klasses = [klass for klass in os.listdir(data_path) if not klass.startswith('.')]
    datasets = []
    filenames = {klass: sorted(os.listdir(data_path + klass + '/')) for klass in klasses}
    for fold in range(numfolds):
        trains = {klass: [] for klass in klasses}
        tests = {klass: [] for klass in klasses}
        for klass in klasses:
            for filename in filenames[klass]:
                if filename[2] == str(fold):
                    tests[klass].append(data_path + klass + '/' + filename)
                else:
                    trains[klass].append(data_path + klass + '/' + filename)
        datasets.append({'train': trains, 'test': tests})
    return datasets

datasets = make_datasets(DATA_PATH)
datasets

[{'test': {'neg': ['./data/imdb1/neg/cv000_29416.txt',
    './data/imdb1/neg/cv001_19502.txt',
    './data/imdb1/neg/cv002_17424.txt',
    './data/imdb1/neg/cv003_12683.txt',
    './data/imdb1/neg/cv004_12641.txt',
    './data/imdb1/neg/cv005_29357.txt',
    './data/imdb1/neg/cv006_17022.txt',
    './data/imdb1/neg/cv007_4992.txt',
    './data/imdb1/neg/cv008_29326.txt',
    './data/imdb1/neg/cv009_29417.txt',
    './data/imdb1/neg/cv010_29063.txt',
    './data/imdb1/neg/cv011_13044.txt',
    './data/imdb1/neg/cv012_29411.txt',
    './data/imdb1/neg/cv013_10494.txt',
    './data/imdb1/neg/cv014_15600.txt',
    './data/imdb1/neg/cv015_29356.txt',
    './data/imdb1/neg/cv016_4348.txt',
    './data/imdb1/neg/cv017_23487.txt',
    './data/imdb1/neg/cv018_21672.txt',
    './data/imdb1/neg/cv019_16117.txt',
    './data/imdb1/neg/cv020_9234.txt',
    './data/imdb1/neg/cv021_17313.txt',
    './data/imdb1/neg/cv022_14227.txt',
    './data/imdb1/neg/cv023_13847.txt',
    './data/imdb1/neg/cv024_

## 7.1 Učitavanje stop riječi i dokumenata

Napravi funkciju **read_stopwords()** po sljedećim ulaznim i izlaznim podacima
* ulaz:
    * **filename** datoteka
* izlaz:
    * skup stop riječi iz datoteke

In [25]:
def read_stopwords(filename):
    """
    ulaz:
    -filename: datoteka

    izlaz:
    -skup stop riječi iz datoteke
    """
    f = open(filename, 'r')
    stopwords = set()
    for word in f:
        stopwords.add(word.strip())
    f.close()
    return stopwords

testname(read_stopwords)
stopwords = test(read_stopwords, (STOPWORD_FILENAME,), {'nine', 'willing', 'inc', 'because', 'edu', 'whenever', 'well', 'hence', 'may', 'seven', 'sometime', 'becomes', 'among', 'had', 'lest', 'containing', 'yourself', 'of', 'theirs', "i'll", 'said', 'down', 'p', 'hereby', 'sub', 'then', 'there', 'qv', 'nothing', 'is', 'tries', 'new', 'x', 'everywhere', 'otherwise', 'com', 'presumably', 'amongst', 'become', 'were', 'they', 'her', 'also', 'hers', 'second', 'tried', 'gone', 'two', 'wants', 'above', 'although', 'much', 'rd', 'f', 'was', 'have', 'has', 'just', 'thorough', 'took', 'particular', 'together', 'up', 'became', 'mean', 'anybody', "they're", 'former', 'only', 'follows', 'right', 'think', 'especially', 'we', 'not', 'concerning', 'which', 'despite', 'tends', 'course', "aren't", 'used', 'cant', 'formerly', 'anyways', 'usually', 'your', 'goes', 'having', 'here', 'nor', "what's", "c's", 'reasonably', 'a', 'see', 'theres', 'meanwhile', "we'll", 'how', "you'd", 'third', 'contain', 'brief', 'becoming', 'asking', 'r', 'liked', 'novel', 'me', 'for', 'its', 'unfortunately', 'elsewhere', 'uses', 'are', 'whereupon', 'overall', 'know', 'consider', 'help', 'uucp', 'furthermore', 'enough', 'according', 'anyway', 'latter', 'far', 'truly', "we've", 'definitely', 'keep', 'yet', 'mostly', 'behind', 'un', 'serious', 'doing', 'probably', 'though', 'what', 'must', 'our', 'say', 'contains', 'upon', 'provides', 'more', "don't", 'per', "t's", 'thereupon', 'unlikely', 'going', 'always', 'anything', 'currently', 'still', 'associated', 'obviously', 'and', 'very', 'described', 'better', 'do', "you'll", 'him', 'looks', 'hardly', "haven't", 'k', "they've", 'seriously', 'thereafter', 'nd', 'particularly', 'be', 'whom', 'out', 'like', 'welcome', 'able', 'go', 'gives', 'seeming', 'trying', 'anyone', 'each', 'inner', 'namely', 'he', 'to', 'thank', 'beyond', 'herein', 'm', 'useful', 'various', 'wherever', 'thus', 'l', 'ask', 'within', 'merely', "shouldn't", "that's", 'sometimes', "they'll", 'wherein', "where's", "ain't", 'gotten', 'somehow', 'ignored', 'therein', 'ever', 'yours', 'other', 'let', 'certain', 'changes', 'whereby', 'hither', 'allow', 'could', 'respectively', 'inward', 'wonder', 'under', 'someone', 'therefore', 'necessary', 'etc', 'in', 'least', 'awfully', 'relatively', 'shall', 'corresponding', 'should', 'appear', 'need', 'whither', 'non', 'looking', 'get', 'v', 'sorry', "wouldn't", 'whereas', 'lately', 'n', 'says', 'wish', 'soon', 'six', 'himself', 'entirely', 'got', "i'd", 'even', 'example', 'e', 'self', 'vs', 'several', 'considering', 'however', 'such', 'why', 'getting', 'happens', 'okay', 'some', "there's", 'towards', "hasn't", 'sent', 'th', "we're", 'few', 'seemed', 'believe', 'four', 'b', 'whereafter', 'three', 'who', 'aside', 'does', 'or', 'actually', 'placed', "doesn't", 'further', 'latterly', 'once', 'maybe', 'one', 'herself', 'indicate', 'normally', 'regardless', 'ones', 'already', 'name', 'eight', 'so', 'their', 'ex', 'after', 'while', 'u', "you've", 'available', 'g', 'keeps', 'unto', 'afterwards', 'near', 'later', 'those', 'certainly', 'j', 'o', 'went', 'since', 'plus', 'whole', 'less', 'consequently', 'now', 'd', 'known', 'thanx', 'w', 'yes', 'oh', 'alone', 'my', 'us', 'neither', 'below', 'taken', 'hereafter', 'nowhere', 'whence', 'off', 'where', 'ourselves', 'regarding', 'given', 'by', 'except', 'immediate', "isn't", 'forth', 'over', 'else', 'try', 'everything', 'via', 'inasmuch', 'ours', 'toward', 'often', 'y', 'until', 'using', 'perhaps', 'them', 'about', 'seems', 'gets', 'specified', 'knows', 'seen', 'without', 'cannot', 'appropriate', 'every', 'clearly', 'both', 'downwards', "a's", 'everybody', 'accordingly', 'cause', 'old', 'with', 'tell', 'i', 'anyhow', 'besides', "who's", 'that', 'following', 'can', 'itself', 'q', 'thru', 'these', 'little', 'the', 'viz', 'hi', 'first', 'against', 'somewhat', "weren't", 'next', 'kept', 'thanks', 'sup', 'themselves', "wasn't", 'sure', "he's", 'done', 'allows', 'fifth', 'twice', 'across', 'apart', 'own', 'before', 'at', 'nearly', 'during', 'ltd', 'noone', 'causes', 'from', 'followed', 'away', "couldn't", 'best', 'last', 'most', 'nobody', 'almost', "won't", 'either', 'you', "let's", 'many', 'thence', 'hereupon', 'exactly', 'somewhere', 'please', 'anywhere', 'another', 'insofar', 'quite', 'way', 'appreciate', 'ok', 'regards', 'outside', 'five', 'between', 'ought', 'whoever', 'specify', 'would', 'throughout', 'instead', "here's", 'no', 'z', 'through', 'c', 'secondly', "c'mon", 'she', 'really', "hadn't", 'into', 'if', 'ie', 'want', 'same', 'whether', 'all', 'any', 'indicated', "i'm", 'zero', 's', 'will', 'thats', 'myself', 'possible', 'sensible', 'co', 'on', 'but', 'an', 'again', 'take', 'value', 'et', 'h', 'mainly', 'specifying', 'beforehand', 'as', 'around', 'thoroughly', 'beside', 'been', 'moreover', 'different', 'somebody', 'might', "we'd", 'likely', 'indeed', 'eg', "they'd", 'rather', 'nevertheless', 'others', "can't", "you're", 'selves', 'howbeit', 'hopefully', 'did', 'whose', 'when', 'came', 'his', 'never', 'whatever', 'along', "it'll", 'seeing', 'this', 'it', "it'd", 'am', 'come', 'unless', 'everyone', "i've", 'something', 'than', 'use', 're', 'hello', 'que', 'thereby', 'indicates', 'none', 't', 'seem', 'needs', 'yourselves', "didn't", 'onto', 'greetings', 'saying', 'comes', "it's", 'being', 'saw', 'look', 'too'})


READ_STOPWORDS
############################################################
OK	read_stopwords('./data/english.stop',) 
	=> {'here', "haven't", 'thats', "doesn't", 'inc', 'relatively', "couldn't", 'also', 'sub', 'ie', '...'}
	== {'here', "haven't", 'thats', "doesn't", 'inc', 'relatively', "couldn't", 'also', 'sub', 'ie', '...'}



### Dokument

Funkcija **read_document()** ima sljedeće ulazne i izlazne parametre
* ulaz:
    * **filename** datoteka dokumenta
    * stopwords: skup stop riječi
* izlaz:
    * skup riječi iz datoteke koje nisu stop riječi
    
Ova funkcija će odvojiti riječi iz teksta datoteke po praznim redovima i razmacima (koristiti `split()` metodu stringa)

In [39]:
def read_document(filename, stopwords = set()):
    """
    ulaz:
    -filename: datoteka dokumenta
    -stopwords: skup stop riječi

    izlaz:
    -skup riječi iz datoteke koje nisu u stopwords
    """
    txt = open(filename, 'r', encoding="utf-8").read().strip()
    words = {word for word in txt.split() if word and word not in stopwords}
    return words

testname(read_document)
_ = test(read_document, (datasets[0]['train']['pos'][0], ), set(['james', 'l', '.', 'brooks', ',', 'one', 'of', 'the', 'developers', 'of', 'the', 'simpsons', 'and', 'director', 'of', 'broadcast', 'news', ',', 'returns', 'to', 'the', 'big', 'screen', 'with', 'this', 'entertaining', ',', 'if', 'slightly', 'flawed', 'comedy', '.', 'nicholson', 'plays', 'melvin', 'udall', ',', 'probably', 'the', 'most', 'horrible', 'person', 'ever', 'on', 'the', 'screen', '.', "he's", 'racist', ',', 'homophobic', ',', 'and', 'never', 'has', 'a', 'good', 'word', 'to', 'say', 'to', 'anyone', '.', 'so', ',', 'nobody', 'talks', 'to', 'him', ',', 'except', 'waitress', 'carol', 'conelly', '(', 't', '.', 'v', 'sitcom', 'star', 'hunt', ',', 'who', 'was', 'last', 'seen', 'in', 'twister', ',', '1996', ')', '.', 'naturally', ',', 'udall', ',', 'conelly', 'and', 'gay', 'neighbor', 'simon', 'bishop', '(', 'kinnear', ')', 'who', 'nicholson', 'hates', ',', 'all', 'hit', 'it', 'off', 'in', 'the', 'end', '.', 'like', 'good', 'will', 'hunting', '(', '1997', ')', 'and', 'titanic', '(', '1997', ')', ',', 'even', 'though', 'the', 'outcome', 'is', 'completely', 'obvious', ',', 'as', 'good', 'as', 'it', 'gets', 'is', 'an', 'enjoyable', ',', 'funny', 'and', 'warm', 'comedy', '.', 'nicholson', 'is', 'hilarious', 'as', 'melvin', ',', 'churning', 'out', 'insults', 'with', 'superb', 'relish', '.', 'only', 'nicholson', 'could', 'get', 'away', 'with', 'the', 'lines', 'that', 'melvin', 'delivers', '.', 'hunt', 'is', 'also', 'good', 'as', 'waitress', 'carol', ',', 'and', 'easily', 'rises', 'to', 'the', 'challenge', 'of', 'nicholson', '.', "there's", 'also', '(', 'thankfully', ')', 'a', 'bit', 'of', 'chemistry', 'between', 'them', '.', 'kinnear', ',', 'as', 'the', 'gay', 'neighbor', ',', 'seems', 'to', 'have', 'a', 'slightly', 'underwritten', 'role', ',', "he's", 'more', 'of', 'a', 'plot', 'convience', 'than', 'a', 'character', '.', 'although', 'his', 'performance', 'is', 'good', ',', 'his', 'character', 'just', 'seems', 'to', 'exist', 'to', 'help', 'melvin', 'and', 'carol', 'come', 'together', '.', 'in', 'fact', ',', 'the', 'scene', 'stealer', 'is', "simon's", 'dog', ',', 'who', 'is', 'funnier', 'than', 'nicholson', '.', 'but', 'then', 'again', ',', 'pets', 'are', 'always', 'cute', 'on', 'screen', '.', 'providing', 'solid', 'support', 'is', 'cuba', 'gooding', ',', 'jnr', '(', 'jerry', 'maguire', ',', '1996', ')', 'and', 'yeardly', 'smith', '(', 'who', 'is', 'the', 'voice', 'of', 'lisa', 'simpsons', 'in', 'the', 'simpsons', ')', 'although', 'gooding', "isn't", 'as', 'good', 'as', 'is', 'character', 'in', 'maguire', ',', 'he', 'is', 'still', 'fun', '.', 'he', 'overacts', 'a', 'little', ',', 'but', 'not', 'so', 'much', 'as', 'to', 'be', 'annoying', '.', 'smith', 'is', 'also', 'good', ',', 'although', 'she', 'has', 'a', 'fairly', 'small', 'role', '.', 'even', 'director', 'lawrence', 'kasdan', '(', 'body', 'heat', ',', '1981', ')', 'makes', 'an', 'appearance', 'as', 'a', 'doctor', '.', 'but', 'this', 'is', 'primarily', 'nicholsons', 'film', ',', 'and', 'every', 'scene', "he's", 'in', ',', "he's", 'steals', 'it', '.', "he's", 'character', 'is', 'so', 'hateful', ',', 'though', ',', "it's", 'amazing', 'that', 'anyone', 'talks', 'to', 'him', 'at', 'all', ',', 'especially', 'carol', '.', 'and', 'this', 'is', 'the', 'films', 'main', 'problem', '.', "it's", 'totally', 'unbelievable', 'that', 'carol', 'would', 'ever', 'consider', 'liking', 'melvin', '.', 'she', "doesn't", 'fall', 'in', 'love', 'with', 'him', 'naturally', ',', 'the', 'film', 'forces', 'her', 'to', 'fall', 'in', 'love', 'with', 'him', '.', 'also', ',', 'melvins', 'character', 'seems', 'to', 'go', 'too', 'nice', ',', 'too', 'quickly', '.', 'i', 'would', 'doubt', 'anyone', 'with', 'a', 'character', 'like', 'melvins', 'would', 'be', 'able', 'to', 'turn', 'back', 'to', 'a', 'nice', ',', 'loving', 'person', '.', 'it', 'would', 'take', 'a', 'helluva', 'long', 'time', ',', 'much', 'longer', 'than', 'this', 'film', 'would', 'like', 'to', 'make', 'out', '.', 'brooks', 'direction', 'is', 'good', ',', 'though', ',', 'if', 'a', 'bit', 'average', ',', 'but', 'he', 'usually', 'manages', 'to', 'get', 'an', 'emotion', 'out', 'of', 'the', 'audience', '.', 'he', 'handles', 'the', 'comedy', 'scenes', 'better', 'than', 'the', 'sentimental', 'ones', '(', 'he', 'tends', 'to', 'pile', 'on', 'to', 'much', 'schmaltz', ')', 'but', 'generally', "he's", 'good', '.', "there's", 'also', 'a', 'nice', 'soundtrack', 'by', 'veteran', 'composer', 'hans', 'zimmer', '.', 'but', ',', 'generally', ',', 'as', 'good', 'as', 'it', 'gets', 'achieves', 'what', 'it', 'sets', 'out', 'to', 'do', ',', 'which', 'is', 'to', 'make', 'the', 'audience', 'feel', 'good', 'by', 'the', 'end', 'of', 'the', 'movie', '.', 'the', 'movie', 'is', 'a', 'bit', 'overlong', ',', 'but', 'nicholson', 'is', 'such', 'good', 'fun', 'that', 'the', 'running', 'time', 'passes', 'by', 'pretty', 'quickly', '.', 'overall', ',', 'as', 'good', 'as', 'it', 'gets', 'is', 'a', 'fun', 'movie', ',', 'even', 'though', 'it', 'may', 'be', 'unbelivable', ',', 'and', 'certainly', 'worth', 'seeing', '(', 'if', 'just', 'for', 'jack', 'nicholsons', 'performance', '.', ')', 'not', 'quite', 'as', 'good', 'as', 'it', 'gets', '(', 'pardon', 'the', 'bad', 'joke', ')', ',', 'but', 'still', 'good', 'fun', '.']))
_ = test(read_document, (datasets[0]['train']['pos'][0], stopwords), set(['james', '.', 'brooks', ',', 'developers', 'simpsons', 'director', 'broadcast', 'news', ',', 'returns', 'big', 'screen', 'entertaining', ',', 'slightly', 'flawed', 'comedy', '.', 'nicholson', 'plays', 'melvin', 'udall', ',', 'horrible', 'person', 'screen', '.', 'racist', ',', 'homophobic', ',', 'good', 'word', '.', ',', 'talks', ',', 'waitress', 'carol', 'conelly', '(', '.', 'sitcom', 'star', 'hunt', ',', 'twister', ',', '1996', ')', '.', 'naturally', ',', 'udall', ',', 'conelly', 'gay', 'neighbor', 'simon', 'bishop', '(', 'kinnear', ')', 'nicholson', 'hates', ',', 'hit', 'end', '.', 'good', 'hunting', '(', '1997', ')', 'titanic', '(', '1997', ')', ',', 'outcome', 'completely', 'obvious', ',', 'good', 'enjoyable', ',', 'funny', 'warm', 'comedy', '.', 'nicholson', 'hilarious', 'melvin', ',', 'churning', 'insults', 'superb', 'relish', '.', 'nicholson', 'lines', 'melvin', 'delivers', '.', 'hunt', 'good', 'waitress', 'carol', ',', 'easily', 'rises', 'challenge', 'nicholson', '.', '(', 'thankfully', ')', 'bit', 'chemistry', '.', 'kinnear', ',', 'gay', 'neighbor', ',', 'slightly', 'underwritten', 'role', ',', 'plot', 'convience', 'character', '.', 'performance', 'good', ',', 'character', 'exist', 'melvin', 'carol', '.', 'fact', ',', 'scene', 'stealer', "simon's", 'dog', ',', 'funnier', 'nicholson', '.', ',', 'pets', 'cute', 'screen', '.', 'providing', 'solid', 'support', 'cuba', 'gooding', ',', 'jnr', '(', 'jerry', 'maguire', ',', '1996', ')', 'yeardly', 'smith', '(', 'voice', 'lisa', 'simpsons', 'simpsons', ')', 'gooding', 'good', 'character', 'maguire', ',', 'fun', '.', 'overacts', ',', 'annoying', '.', 'smith', 'good', ',', 'fairly', 'small', 'role', '.', 'director', 'lawrence', 'kasdan', '(', 'body', 'heat', ',', '1981', ')', 'makes', 'appearance', 'doctor', '.', 'primarily', 'nicholsons', 'film', ',', 'scene', ',', 'steals', '.', 'character', 'hateful', ',', ',', 'amazing', 'talks', ',', 'carol', '.', 'films', 'main', 'problem', '.', 'totally', 'unbelievable', 'carol', 'liking', 'melvin', '.', 'fall', 'love', 'naturally', ',', 'film', 'forces', 'fall', 'love', '.', ',', 'melvins', 'character', 'nice', ',', 'quickly', '.', 'doubt', 'character', 'melvins', 'turn', 'back', 'nice', ',', 'loving', 'person', '.', 'helluva', 'long', 'time', ',', 'longer', 'film', 'make', '.', 'brooks', 'direction', 'good', ',', ',', 'bit', 'average', ',', 'manages', 'emotion', 'audience', '.', 'handles', 'comedy', 'scenes', 'sentimental', '(', 'pile', 'schmaltz', ')', 'generally', 'good', '.', 'nice', 'soundtrack', 'veteran', 'composer', 'hans', 'zimmer', '.', ',', 'generally', ',', 'good', 'achieves', 'sets', ',', 'make', 'audience', 'feel', 'good', 'end', 'movie', '.', 'movie', 'bit', 'overlong', ',', 'nicholson', 'good', 'fun', 'running', 'time', 'passes', 'pretty', 'quickly', '.', ',', 'good', 'fun', 'movie', ',', 'unbelivable', ',', 'worth', '(', 'jack', 'nicholsons', 'performance', '.', ')', 'good', '(', 'pardon', 'bad', 'joke', ')', ',', 'good', 'fun', '.']))



READ_DOCUMENT
############################################################
OK	read_document('./data/imdb1/pos/cv100_11528.txt',) 
	=> {'long', 'small', 'tends', 'screen', 'hit', "doesn't", 'film', 'james', 'running', 'also', '...'}
	== {'long', 'small', 'tends', 'screen', 'hit', "doesn't", 'film', 'james', 'running', 'also', '...'}

OK	read_document('./data/imdb1/pos/cv100_11528.txt', {'here', "haven't", 'thats', "doesn't", 'in... 
	=> {'long', 'unbelievable', 'small', 'hit', 'screen', 'film', 'james', 'running', 'person', 'jack', '...'}
	== {'long', 'unbelievable', 'small', 'hit', 'screen', 'film', 'james', 'running', 'person', 'jack', '...'}



## 7.2 Učenje klasifikaora

Napravi funkciju **train()** po sljedećim ulaznim i izlaznim podacima
* ulaz:
    * **testset** rječnik čiji ključevi su klase, a vrijednosti putanje do datoteka
    * **stopwords** skup stop-riječi
* izlaz:
    * **prior** je rječnik čiji su ključevi klase, a vrijednosti prior vjerojatnosti za svaku klasu
    * **megadoc** je rječnik čiji su ključevi klase, a vrijednosti unigram s frekvencijama riječi
    * **vocabulary** je skup svih riječi iz svih megadokumenata


$$P(c) = \frac{Nc}{N}$$

* $c$- klasa  
* $Nc$ - broj dokumenata klase c  
* $N$ - broj svih dokumenata


In [51]:
def train(trainset, stopwords):
    prior = {c: len(filenames) for c, filenames in trainset.items()}
    megadoc = {}
    vocabulary = set()
    for c, filenames in trainset.items():
        megadoc[c] = {}
        for fn in filenames:
            for w in read_document(fn, stopwords):
                vocabulary.add(w)
                megadoc[c][w] = megadoc[c].get(w, 0) + 1
    return prior, megadoc, vocabulary
        
testname(train)
model = test(train, (datasets[1]["train"], stopwords), ({'neg': 900, 'pos': 900}, {'neg': {'dank': 2, 'provocative': 5, 'newcomers': 4, 'concierge': 1, 'forensics': 1, "i'm-whining-because-i-can't-get-a-girl-i-want": 1, 'dolittle': 3, 'punisher': 1, 'goth': 5, 'blacked': 1, '2=awful': 1, 'digested': 1, "kai's": 1, 'lout': 2, 'slides': 1, 'violate': 1, 'clothes': 23, 'dampened': 1, 'dinner': 23, 'impregnates': 2, "director's": 23, 'motion': 59, "panther's": 1, 'in-depth': 2, 'loosely': 12, '57': 3, 'fall-out': 1, 'smokejumpers': 1, 'microbombs': 2, 'super-soldier': 1, 'demoliton': 1, 'illustrates': 4, 'detractors': 1, 're-establish': 2, 'review': 112, 'aging': 12, 'infatuated': 2, 'city-boy': 1, 'origami': 1, 'larsen': 1, 'mir': 4, 'decorated': 5, 'baseketball': 1, '15-foot': 1, 'abetting': 2, 'emotion-': 1, 'laser': 5, 'scared': 13, 'lurches': 3, 'bergman': 2, 'flowers': 5, 'rambunctous': 1, "laugh'": 1, "'clean": 1, 'packed': 18, 'manderley': 1, 'dreaming': 1, 'straight-': 1, '1/50th': 1, 'fianc': 4, 'aura': 1, 'crises': 4, 'protects': 1, 'hummana-hummana-hummana': 1, 'clock': 5, 'embroils': 1, 'trunk': 10, 'iggy': 1, 'cityscape': 1, 'violin-maker': 1, 'serendipity': 1, 'scatology': 1, 'boyishly': 1, "hatcher's": 2, 'mural': 1, 'cleavage': 9, 'letting': 16, 'joon': 2, 'wrong': 153, 'one-tone': 1, 'doonesbury': 1, 'mouth-breathing': 1, 'afflicted': 3, 'startling': 7, 'prank': 8, 'bandits': 1, 'resolutely': 2, 'reconsider': 1, 'onatopp': 1, 'tv-conversion-movies': 1, 'crackdown': 1, 'grin': 3, 'prelude': 4, 'clear-thinking': 1, 'nerve-addled': 1, 'skit-length': 1, 'imagining': 3, 'producer/writer': 1, 'pratfall': 1, 'stress': 5, 'pat': 10, 'mint': 1, 'noraruth@aol': 1, 'adrian': 2, 'do-it-yourselfer': 1, 'bowl': 4, 'blunders': 2, 'truism': 1, '`one': 2, 'quicker': 6, 'warmed': 1, 'guarantee': 11, 'mischief': 1, "tom's": 1, '180': 2, 'clever-way': 1, 'characteristics': 4, 'fluffy': 2, 'gauntlet': 2, 'hilarious': 45, 'winger': 1, 'susanne': 1, 'daryl': 4, 'threatned': 1, 'manor': 2, 'next-to-next-of-kin': 1, 'reformed': 4, 'deathbed': 1, 'helena': 5, 'hazardous': 1, 'make-out': 2, 'moland': 1, 'aaron': 6, 'blissful': 2, 'confirming': 1, 'boomy': 1, 'forcing': 13, "viewer's": 12, 'muster': 9, 'under-fire': 1, 'ponytail': 2, 'scumball': 1, 'wen': 1, 'sleep-walking': 1, 'teaser': 1, 'summer': 84, 'upper-crust': 1, 'strategic': 3, 'mar': 1, "hilly's": 1, 'domestic': 6, 'mosey': 1, 'depths': 7, 'pulled': 25, 'six-stories': 1, 'exacerbates': 1, 'klingons': 2, 'fancy': 17, 'satan': 8, 'erin': 2, 'horne': 2, 'question': 111, 'marveling': 2, 'ploys': 1, 'non-violent': 2, 'talkshow': 1, 'arsenal': 1, 'reservoir': 3, 'oppressors': 1, 'kessler': 1, 'pussy': 1, 'too-obvious': 2, 'solvable': 1, 'intelligently': 5, 'unsworth': 1, 'intellectually': 7, "photographer's": 2, 'exorcist': 11, 'eighth': 1, 'tinged': 1, 'civilized': 6, 'ca': 1, 'interligator': 1, 'kristofferson': 2, 'andrea': 2, 'drainage': 1, 'spiritualist': 2, 'dismemberment': 1, "f$&#in'": 1, 'danny': 22, 'godfried': 1, 'unsavory': 1, 'snowman': 1, 'in-your-ear': 1, 'breckin': 3, 'ice-cube': 1, "'60s/'70s": 1, 'funny-movies': 1, 'tammi': 1, 'abandoned': 18, 'lieh': 1, 'janet': 5, 'fuelled': 1, 'severed': 1, 'largely': 22, 'eloquent': 2, 'luc': 9, 'modernize': 1, 'takers': 1, 'greg': 20, 'diddley': 1, 'slavery': 4, "source'": 1, 'perdy': 1, 'intend': 6, 'skateboarder': 1, 'pepper': 8, 'tv-star': 1, 'nick': 30, 'navarro': 1, 'oil-driller': 3, 'andie': 3, 'snappy': 4, "butthead's": 1, '_whole_': 1, 'classroom': 2, "'do": 1, 'co-workers': 5, 'brad': 16, 'conversations': 14, 'listening': 8, 'unknown': 23, 'claudia': 3, 'loose': 18, 'intellectuals': 4, "jill's": 3, 'courtney': 10, 'favorite': 59, 'thief': 19, 'kathy': 7, 'perkus': 1, 'jfk': 3, 'wordplay': 2, 'diminished': 4, 'ty': 1, "jimmy's": 3, 'northern': 1, 'rent-a-wreck': 1, 'provoked': 2, 'establish': 9, 'shacking': 1, 'cantonese': 1, 'dvd-rom': 1, 'obviousness': 4, 'lobotomizing': 1, 'dramas': 4, 'compelled': 3, 'family-friendly': 2, 'haggardly': 1, 'brittany': 3, 'milla': 3, 'ceremonies': 1, 'fodder': 3, 'largest': 4, 'feeds': 3, 'motorists': 1, 'priestess': 1, 'bullock': 7, 'railroad': 6, 'plunging': 1, 'amusements': 2, 'electronic': 5, 'psychotics': 2, 'crumbs': 3, 'leonetti': 1, 'ticky': 1, 'sprinkled': 6, 'krippendorf': 3, 'brags': 3, 'sebastiano': 1, 'blades': 4, 'managed': 35, 'improvement': 13, 'mass-market': 1, 'excluded': 1, 'alley': 7, 'castigates': 1, 'anti-climatic': 1, 'nitwits': 1, 'thirties': 3, 'teleprompted': 1, 'range': 18, 'sanitises': 1, 'wrongs': 1, 'satellite': 3, 'back-up': 2, 'ava': 1, 'montreal': 3, 'entranced': 1, 'afterword': 1, 'rheinhold': 1, 'waterbeds': 1, "die'": 2, 'unnoticeable': 1, 'multi-armed': 1, 'scream-mask': 1, 'usual': 63, 'confines': 6, 'pics': 6, 'floundering': 1, 'barrier': 6, 'inevitability': 2, 'directionless': 1, 'carmen': 3, '-her': 1, 'voice-over': 9, 'hudreds': 1, 'springboard': 1, 'infuriating': 5, 'mildly': 27, 'fury': 6, 'pictured': 1, 'black-market': 1, 'crash-testing': 1, "jackson's": 5, '_roxbury_': 1, 'arthur': 13, 'reckoned': 1, 'mcnamara': 1, 'outbreak': 1, 'decreasing': 1, 'culminate': 1, 'train': 31, 'herek': 2, "quinn's": 3, "mgm's": 1, 'sorority': 2, 'second-rate': 8, 'rap': 6, 'reactions': 6, 'witnessing': 4, 'cymbals': 1, 'innuendo': 5, 'ambushes': 1, 'six-hundred-years-old': 1, 'good-buy': 1, 'tarveling': 1, 'lennox': 2, 're-make': 2, 'transfer': 4, 'permission': 3, 'ze': 1, 'dragonheart': 2, 'ripe': 3, 'blue-screen': 1, 'chi': 4, 'dragged': 12, 'innovatively': 1, 'topless': 8, 'gedrick': 2, 'immortals': 1, 'roland': 6, 'clueless': 21, 'improvisational': 1, 'subway': 10, 'economic': 3, 'syd': 1, 'nomi': 2, 'redoing': 1, 'defines': 5, 'natch': 1, 'seams': 4, 'gayheart': 9, 'practical': 4, 'creations': 7, 'surprise': 73, 'copied': 1, 'shadowy': 4, 'faa': 1, 'stapled': 1, 'tampered': 1, 'concealing': 3, 'slaughter': 3, 'ace': 9, 'volkswagen': 1, 'finishes': 7, 'achingly': 1, 'septic': 1, 'book': 95, 'effeminate': 5, 'heart': 74, 'journalistic': 1, 'patillo': 1, 'donated': 1, "giant'": 1, 'cleverly-constructed': 1, '-ridden': 3, 'rancorously': 1, 'go-go': 1, '19': 5, 'short-lived': 2, 'edna': 1, 'genius': 26, 'coolio': 2, 'protesters': 1, 'major': 105, "1900's": 1, 'high-school': 5, 'preston': 9, 'viewed': 12, 'ethan': 12, 'taking': 82, 'crust': 1, 'clapet': 1, 'frederick': 3, 'players': 39, "that'll": 3, 'hugged': 1, 'someplace': 2, 'volume': 5, 'lining': 2, 're-write': 3, 'draped': 1, 'detox': 1, 'cop/fbi': 1, 'wim': 1, 'infamous': 22, 'join': 34, 'costume': 21, '10/10': 4, 'arquette': 13, 'followings': 1, 'rationality': 3, 'doubting': 2, 'persuades': 2, 'dictionary': 1, 'choices--married': 1, 'wedlock': 2, 'flair': 14, 'pitches': 1, 'watching': 206, 'birkin': 1, 'undress': 1, 'partitions': 1, 'unqualified': 1, 'menancing': 1, 'ig': 1, 'minnie': 4, 'thom': 1, 'fairy-tale': 1, 'feather': 2, 'room': 85, 'humorous': 36, 'pilots': 4, '$7': 3, 'oj': 1, 'rock': 48, 'counseling': 1, 'outs': 1, 'msnbc': 2, "another's": 1, '95': 4, 'whomever': 1, 'saccharine': 6, 'action-verb': 1, 'reflex': 2, "america's": 8, 'outta': 2, 'electrocution': 1, 'judged': 2, 'day-style': 1, 'censors': 1, 'accelerates': 1, 'cleverness': 7, 'coolly': 1, 'found': 139, 'reiter': 1, 'cinematic': 58, "foster's": 1, 'airheads': 1, 'doors': 16, 'winslow': 1, 'based': 126, 'force': 46, 'attempting': 23, 'insects': 6, 'emmanuelle': 2, 'pout': 6, 'tre': 1, 'urge': 7, 'dewy-eyed': 1, 'bedfellows': 1, 'villainy': 3, "stamp's": 1, 'birdwatcher': 1, 'animal-rights': 1, "'showgirls'": 1, 'cued': 1, 'millisecond': 1, 'dancer': 12, 'memorabilia': 1, 'lang': 1, 'envisioned': 2, 'daqughter': 1, "element'": 1, 'excursion': 4, "mel's": 3, 'fixed': 2, "donner's": 2, 'reappearance': 1, 'potted': 1, '33': 2, 'occupants': 1, "-it's": 1, "robbie's": 2, 'intergalactic': 3, 'louisiana': 3, 'pooh-bah': 1, 'offer': 38, '[stick': 1, 'sensibility': 4, 'friend': 132, "kate's": 1, 'obsessions': 1, 'should--and': 1, 'jams': 2, 'osmet': 1, 'mith': 1, 'ballistic': 4, 'lowlifes': 2, 'madman': 5, 'urgent': 1, 'hoosiers': 1, 'racing': 11, 'keiko': 2, 'animosity': 1, 'yank': 1, 'affay': 1, 'celia': 1, 'slop': 3, 'obsenities': 1, 'schlictmann': 1, 'marrying': 2, "o'hara": 5, 'expose': 5, 'wonderment': 1, 'sparking': 1, 'fervor': 3, "faculty'": 1, 'baa-ram-ewe': 1, 'flash-backs': 1, 'lunges': 1, 'instalment': 1, "'bicentennial": 1, 'world-renowned': 2, 'tiny': 15, 'cheetah': 1, 'three-time': 1, 'consciousness': 5, 'guests': 10, "'snuffed'": 1, 'diablo': 1, 'visitation': 1, 'anatomical': 1, 'pillage': 1, 'seagal/mickey': 1, "malcolm's": 1, 'bay-style': 1, 'tracer': 1, 'abused': 3, "lawerence's": 1, '127': 1, 'set-ups': 3, 'hopkins': 10, 'foolhardiness': 1, '$11': 1, 'undeserved': 2, 'bordering': 2, 'mired': 3, 'fevered': 1, 'isley': 1, 'a-quarter': 1, 'vulnerability': 4, 'liddy': 1, 'geriatric': 3, 'pasted': 1, 'readers': 5, 'steadicam': 3, 'still-operational': 1, 'articulate': 6, 'heavenly': 4, 'lambasted': 1, "company's": 4, 'entrenched': 1, 'doom': 9, 'plusses': 1, 'copying': 4, 'smidgeon': 1, 'wiccan': 2, 'articles': 3, 'ghost--make': 1, 'runnings': 2, 'affections': 2, 'unexplained': 5, 'cellar': 2, "witherspoon's": 1, 'howewer': 1, 'deep-space': 1, '1980': 4, 'cloud-choked': 1, 'cid': 1, 'gallows': 2, 'books': 19, 'cinematogrophy': 1, 'helpfully': 1, 'led': 39, 'ridley': 2, "villains'": 4, 'avery': 3, 'jail': 24, 'almod': 1, "moore's": 5, 'caron': 1, 'illiteracy': 1, 'estella': 2, 'pronounced': 5, 'massacre': 6, 'enjoyed': 25, 'free-associating': 1, 'durning': 1, 'stalks': 7, 'lucked': 3, "helgeland's": 2, 'grabbed': 2, 'watered-down': 2, 'rude': 3, 'zingers': 3, 'genitals': 1, 'commonly': 3, 'stage': 28, 'delegates': 1, 'gradient': 1, 'pre-fab': 1, 'bening': 3, 'sleeps': 9, 'distributor': 3, 'tools': 2, 'gathering': 3, 'scheme': 20, "rachel's": 3, 'biological': 5, 'i-know-what-you-like': 1, 'ungainly': 1, 'wizardry': 2, 'mispaired': 1, 'resides': 5, 'square-jawed': 2, 'fabio': 2, 'tsai': 1, "brother'": 1, "could've": 12, 'headliner': 1, 'mcgrath': 1, "swingers'": 1, 'plights': 2, 'demi': 6, "nolte's": 1, 'howls': 1, 'quotable': 1, 'dzundza': 1, 'ponders': 2, 'relates': 4, 'beads': 2, 'weep': 5, 'craves': 1, 'stubble': 1, '4/4': 1, 'ny': 3, 'flurry': 2, 'synchronized': 1, 'carlton': 1, 'vining': 1, 'split-screens': 1, '52': 1, 'guardian': 2, 'half-win': 1, 'initials': 1, 'miranda': 2, 'amateurishly': 4, 'factor': 20, 'dumbo': 1, 'setback': 1, 'prerequisite': 2, 'switchback': 1, 'mancini': 1, 'church-': 1, 'wagner': 3, 'grenades': 2, "`snatch-the-paycheck-and-run'": 1, 'chaykin': 2, 'zellweger': 7, 'negligible': 4, 'swimming': 8, 'outdoors': 2, 'weirdest': 1, 'terraformed': 1, '_not_': 3, 'unimaginatve': 1, 'singular': 3, 'asses': 4, 'cher': 2, "labute's": 1, 'offence': 3, 'sleepwalks': 3, 'green-light': 2, 'tong': 2, "england'": 1, 'tourniquet': 1, 'antwerp': 2, 'lips': 11, 'predicted': 3, 'excuse': 44, 'stavro': 1, 'free': 42, 'underage': 2, 'yet-to-be-released': 1, 'damaged': 6, 'lighted': 1, 'ocean': 22, 'radiator': 1, 'leni': 1, 'improvized': 1, 'burgess': 1, 'jittery': 4, 'woolly': 1, "plot's": 5, 'dutifully': 2, 'collector': 2, 'claus': 1, 'nsa': 3, 'runs': 73, 'erric': 1, 'pollo': 1, 'hipsters': 1, 'besieged': 1, 'puff-piece': 1, 'madonna': 7, 'presents': 23, 'magnificent': 4, 'superdemon': 1, 'diametrically': 1, 'ornament': 1, '85': 7, 'debating': 1, 'morphs': 1, 'nostalgic': 8, 'fervour': 1, 'intercourse': 2, 'pistols': 1, 'beart': 1, 'morose': 1, 'oscar': 38, 'incited': 1, "eye-full's": 1, 'sumo': 2, 'elvis': 4, 'direct': 24, 'maddeningly-long': 1, 'fatherly': 2, 'mailing': 1, "directors'": 2, 'operations': 6, 'continually': 14, 'next-door': 1, 'writer/first-time': 1, 'sanctimonious': 1, 'dolce': 1, 'damnit': 2, 'cker': 1, 'load': 12, 'amidala': 2, 'charles': 23, "leo's": 2, 'revolt': 2, 'exponential': 1, 'search': 31, 'soft-core': 2, "conrad's": 1, 'beaming': 2, 'sex-romp': 1, 'palpably': 1, 'modifier': 1, 'wife--and': 1, 'ostott': 1, 'frazzled': 1, "luis'": 1, 'errupts': 1, 'portuguese': 1, 'reined': 1, 'bloodstream': 1, 'uninterestingly': 1, 'autumn': 1, 'mysterious-boyfriend': 1, 'passable': 12, 'disowned': 1, 'attendant': 3, 'pathologically': 1, 'neo-realism': 1, 'aggressivelly': 1, 'hyperjump': 1, 'leukemia': 1, 'frenetic': 4, 'thievery': 1, 'fx--such': 1, 'sw': 1, 'perfs': 1, 'convinces': 14, 'suzy': 2, '_the_quest_': 1, 'hermaphrodite': 1, 'upham': 1, 'unflattering': 1, 'albeit': 23, 'shape': 11, 'mud': 3, "country's": 2, 'inducing': 2, 'guffman': 1, 'dourdan': 1, 'daughter': 88, 'fairytale': 1, 'film-writing': 1, 'sanitized': 2, 'goo-iffy': 1, 'holed': 1, 'abortive': 1, 'restricted': 2, 'pouty-lips': 1, 'shrill': 2, 'nice-guy': 1, 'cellmate': 1, 'kris': 2, 'skipper': 1, 'tooling': 1, 'shemp': 1, 'self-referencing': 1, "goyer's": 1, 'negatively': 1, 'toned': 5, 'diminuitive': 1, 'all-too-brief': 3, "caulder's": 2, 'homer': 1, 'recourse': 2, 'punk-in-the-making': 1, "d'oevre": 1, 'marrow': 1, 'extravaganzas': 1, 'bombastic': 4, '`yeah': 1, 'page': 14, 'seamless': 1, 'alien-possessed': 1, "dicillo's": 2, 'clich': 46, 'lengthy': 9, 'unemotional': 1, "haddaway's": 2, 'marco': 1, 'characterless': 1, 'picking': 9, 'deanna': 3, 'analysis': 6, 'sympathy': 15, 'uneventful': 3, 'tag-team': 3, 'laced': 3, "gee's": 1, 'subconscious': 1, 'predecessors': 5, 'manufactured': 6, "psychlos'": 1, 'qualities': 21, 'linda': 8, 'calitri': 1, 'unjustly': 2, 'explanation': 32, 'erica': 1, 'incapacitated': 1, 'dominic': 3, 'arte': 1, 'mortensen': 4, 'civilians': 3, 'johnston': 1, '>from': 10, 'roy': 5, 'brood': 2, 'prefontaine': 1, 'humankind': 1, 'threatening': 27, 'point': 251, 'instant-gratification': 1, 'kleenex': 1, 'egomaniacal': 1, 'poor': 111, 'anthro-zoologists': 1, 'milo': 1, 'enjoyment': 13, 'gaffe': 1, 'adage': 2, 'unmasking': 1, 'whiskey': 2, 'overzealous': 1, 'sensitivities': 1, 'uppity': 1, 'naughty': 4, 'arises': 6, 'sterling': 2, 'tracking': 11, 'torrid': 1, 'hasidic': 2, 'fear': 44, 'patented': 3, 'off-the-cuff': 1, 'ripped-off': 1, 'briefcase': 2, 'flop': 13, 'objected': 1, 'rates': 3, 'repulsive': 5, 'pistone': 1, 'kinski': 2, 'glee': 4, 'milbauer': 1, 'goofily': 3, 'toys': 11, 'gait': 1, 'ax': 1, 'garden': 9, 'curl': 2, 'killer-like': 1, 'controlling': 3, 'whispers': 1, 'paquin': 1, '_brazil_': 1, 'performances': 119, 'quirkily': 2, 'theatrics': 2, 'corp': 1, 'newsman': 1, 'interspersed': 5, '31': 4, 'blaxploitation': 2, "clyde'": 1, 'motley': 1, 'invested': 7, 'madness': 10, 'millennium': 8, 'police': 81, 'bogus': 3, 'ho': 5, 'debacles': 1, 'building': 41, 'heist': 10, 'gate': 5, 'classy': 2, 'toucher-uppers': 1, 'cockney': 2, 'signify': 3, 'k9': 1, 'underdogs': 1, 'devolves': 5, 'dangerous': 30, 'ofcs': 1, "'animal'": 1, 'snarf': 1, 'focker': 1, 'serials': 1, 'antichrist': 3, 'slugfest': 1, 'dispensed': 3, 'fights': 30, 'niceties': 1, 'noxzema': 1, 'pucker': 1, 'proffessor': 2, 'kreskin': 1, 'sickness': 3, 'recover': 8, 'bongos': 1, 'lorry': 2, 'snob': 3, 'eats': 15, 'luhrmann': 1, 'sorceror': 1, 'responds': 6, 'dee': 2, 'gaunt': 1, 'hand-held': 2, "didn't-we-dress-funny-back-then": 1, 'dewey': 2, 'exceed': 1, 'warheads': 1, 'necking': 1, 'earth-': 1, 'folk': 7, 'howerd': 1, 'larger-than-life': 5, 'oddity': 1, 'dirt': 9, "dumas'": 1, 'handy': 3, '1885': 1, 'quarantined': 1, 'you\x12re': 2, 'preceding': 5, 'se': 3, 'muscle': 6, 'cans': 3, 'deaths': 12, 'juke': 1, "reiner's": 1, 'tight': 15, 'tights': 2, 'glory': 13, 'langenkamp': 2, 'melodramatic': 18, 'virtually': 37, 'extra-strength': 1, 'dysfunctional': 5, 'beeyatch': 1, 'obssessed': 1, 'trump': 1, 'quiet': 18, 'embarrass': 3, 'amphetimenes': 1, 'goodbye': 3, 'ex-new': 1, 'no-budget': 1, "nielsen's": 1, 'pipes': 3, 'preaching': 3, 'mother-': 1, 'inversion': 1, 'handphones': 1, 'foreman': 3, 'sturdy': 1, 'penelope': 2, 'people--would': 1, 'tango': 4, '\x16': 2, 'accompanies': 1, 'romane': 1, 'fahey': 3, 'frustrate': 1, 'gibson-inspired': 1, 'standup': 1, 'robotics': 1, 'sub-culture': 1, 'panoramic': 3, "dillon's": 2, 'unscathed': 4, 'enslave': 1, 'predicting': 1, "priest's": 1, 'ordinary': 8, 'jessalyn': 1, 'best-supporting': 1, 'vignettes': 2, 'passing': 20, "action'": 1, 'wayans': 5, 'revive': 7, 'relate': 16, "goldman's": 1, 'armed': 15, 'zeta': 4, 'wink-a': 1, 'tying': 4, 'jimmi': 1, 'dutch': 5, 'byrnes': 2, 'lift-off': 1, 'feigns': 1, 'released': 61, 'response': 15, 'concept': 39, 'interfering': 1, 'delpy': 1, 'mccoy': 2, 'copier': 1, 'hyper-drive': 1, 'organizer': 1, 'stashed': 2, 'beans': 4, 'tapeheads': 1, 'macho-misfits': 1, 'maudlin': 5, 'snake': 20, 'malaysia': 1, 'vision': 16, 'enigma': 2, 'heavyweight': 3, 'bronze': 1, 'zigged': 1, 'crew': 56, 'watchable': 20, 'homosexuality': 2, 'stubbornly': 3, 'tearful': 1, 'matte': 2, 'truant': 1, '_is_': 6, 'visualization': 4, 'sylvia': 2, 'uncommunicative': 1, "1981's": 1, "o'shea": 4, "top's": 1, "comedian's": 1, '1/4': 1, 'bullet-cam': 1, 'dukakis': 1, 'embry': 5, "keeble's": 1, "chelsom's": 1, 'esque': 2, 'caricatures': 9, 'mega-city': 1, "borgnine's": 1, 'guinea': 7, 'macarena': 2, 'seize': 1, 'closeup': 1, 'sweets': 1, 'customers': 5, 'springer': 1, 'cameron': 14, 'troubles': 5, 'k-obsessed': 1, 'intervals': 2, 'off-again': 1, 'effort': 64, 'lustily': 1, 'numero': 1, 'knifepoint': 1, 'matriarchal': 1, 'fidelity': 2, 'vomits': 5, 'lorre': 1, 'honing': 1, 'cyber-travelling': 1, 'matter': 113, 'well-formed': 1, 'sex-starved': 1, 'crotch-biting': 1, 'freebie': 1, 'veers': 4, 'skeet': 3, 'glutton': 1, 'erupts': 2, 'fuddy-duddy': 1, 'likes': 41, 'studied': 2, 'candle': 2, 'gia': 2, 'kraftwerk': 1, 'novelization': 1, 'snow-covered': 1, 'deposits': 1, 'kinnear': 7, 'appaling': 1, 'clerks': 4, 'untouchables': 1, 'outdoes': 1, 'winning': 28, 'settings': 7, 'strived': 1, 'dna': 5, 'realized': 35, 'mixture': 5, 'danza': 1, 'niger': 1, 'groceries': 1, 'earn': 10, 'clunker': 5, 'sadness': 4, 'schedules': 1, '+blah': 1, 'chekhov': 1, 'scripted': 6, 'flintstones': 2, 'mission': 58, 'proyas': 3, 'japan': 5, 'disapproves': 3, 'tenfold': 1, 'parted': 2, "'round": 1, 'grinder': 1, 'wabbit': 1, 'half-hokey': 1, 'outpost': 1, 'whimpering': 1, 'mongrel': 1, 'grating': 7, 'inane': 26, 'chemically-induced': 1, 'cobbles': 1, 'assembly-line': 1, "executives'": 1, 'definately': 6, 'ahead': 30, 'aiding': 2, "london's": 1, 'smuggle': 1, "maker's": 1, 'brauva': 1, 'humiliated': 4, 'one_': 1, 'elfont': 2, 'timberland-dweller': 1, 'silverstone': 9, 'relic': 4, 'agreeing': 2, 'breakout': 2, 'clapton': 2, 'meatballs': 2, 'exploited': 1, 'chews': 5, 'families': 14, 'drums': 1, '#': 1, 'verite': 1, 'curvier': 1, 'rebellious': 1, 'krikes': 1, 'plumb': 1, 'exceptional': 5, 'uhhm': 1, 'imagery': 13, 'petersen': 1, 'lori': 2, 'romano': 1, "fugees'": 1, 'bodyguards': 2, 'fantasise': 1, 'unaware': 7, 'sharp-edged': 1, 'cinematical': 1, 'written': 134, '5': 36, 'title': 117, 'executives': 4, "thing's": 2, 'astronaut': 11, 'uplift': 1, "_dawson's_creek_": 1, 'cristoffer': 1, 'malevolent': 5, 'henri': 1, 'france/usa': 1, 'favoring': 1, 'melvin': 3, 'bar-owner': 1, 'limit': 4, 'cod': 4, 'prance': 2, 'injury': 13, 'paternal': 3, 'offers': 52, 'grasshopper': 1, 'requisite': 8, 'pint-size': 1, 'incredulous': 4, 'kicking': 13, 'deflated': 1, 'candies': 1, 'debuting': 1, 'promisingly': 4, 'genre-last': 1, 'goodfellas': 3, 'sacrifice': 6, 'subscription': 1, 'lisping': 1, 'sanctuary': 2, 'killed': 82, "neighbors'": 1, 'promotion': 10, 'jonnie': 3, 'shoe': 5, 'fricking': 1, 'maginnis': 1, '`bio': 1, '_looks_': 1, 'barked': 1, 'potholes': 2, "sadie's": 1, 'waging': 1, 'tornadoes': 1, 'stahl': 3, 'presenter': 1, 'fluctuates': 2, 'thoughts': 25, 'ses': 1, 'fukienese': 1, 'highest': 7, 'commit': 8, 'reunions': 1, 'poisoned': 2, 'brothers-style': 1, 'strongly': 7, 'astounded': 2, 'neuwirth': 1, 'practices': 2, 'kegger': 1, 'brest': 1, 'reel': 17, 'half-dozen': 3, 'ideal': 8, 'intended': 45, 'knock-off': 1, 'udesky': 1, 'overextended': 1, 'banner': 2, 'quarter': 10, 'torkel': 1, "prison's": 1, 'non-nude': 1, 'sapling': 1, "glamorise'": 1, 'catholicism': 1, 'directions': 6, 'yeas': 1, 'church': 18, 'prophecy': 4, 'immobilise': 1, 'defects': 1, 'disliking': 4, '_don+t': 1, 'comedian': 12, 'policemen': 5, 'amsterdam': 1, 'absolutely': 79, 'awwww': 1, 'long-running': 3, 'minorities': 1, 'gardens': 1, 'caan/bruce': 1, 'decease': 1, 'lassic': 1, 'budweiser': 1, 'homage': 13, 'ex-universal': 1, 'sins': 2, 'mrs': 29, 'belonging': 5, 'ex-con': 3, 'westerns': 3, "coppola's": 1, "devito's": 2, 'natascha': 2, 'kevin': 84, '`2001': 1, 'undervalued': 1, 'lynch': 12, 'ridiculous': 96, 'contests': 1, 'reviewers': 6, 'trashy': 7, 'unrepentantly': 1, 'mean-looking': 1, 'studio-imposed': 1, 'crackerjack': 1, 'hasten': 2, 'biker-chick': 1, '1916': 1, 'woman-abusing': 1, 'extremel': 1, 'murphy': 18, 'prosaic': 3, 'blubbering': 1, 'mouseketeer': 1, 'stillman': 1, 'michelle': 14, 'lincoln': 5, 'unfriendly': 1, 'deficit': 1, 'rights': 12, 'grammatical': 1, 'burly': 4, 'loudness': 1, 'notorious': 10, 'racquel': 1, 'cross-dresser': 2, 'worth-paying-to-see': 1, "sant's": 3, 'slain': 5, 'jansen': 1, 'paper-thin': 6, 'reunite': 4, 'lasts': 6, 'tiresomely': 1, 'accomplished-but-stiff': 1, 'independent-minded': 2, 'saim': 1, 'olds': 7, 'screening': 23, 'lochlyn': 1, 'sutherland': 13, 'crappy': 9, 'unsettling': 10, 'man-meets-alien': 1, 'single-handed': 1, 'caps': 2, 'auster': 1, 'anytown': 1, 'raids': 3, 'relentlessy': 1, 'he/she/they': 1, 'them--': 1, 'constable': 1, 'extra-terrestrial': 1, 'by-the-book': 1, 'mccleod': 1, 'terrifyingly': 1, 'terl': 5, 'kilgore': 1, 'un-funniest': 1, 'outfit': 5, 'glasses': 10, 'imagines': 1, 'urgency': 8, 'rise': 25, 'laxatives': 1, 'exec': 8, 'dion': 1, 'cycle': 3, 'throne': 6, 'avant-garde': 1, 'pesudo-pseudo-pseudo-character': 1, 'malick': 2, "supernova's": 1, "_schindler's": 1, 'artist': 20, 'beantown': 1, 'hauteurs': 1, 'codes': 3, 'goody-two-shoes': 1, 'porns': 1, 'marcus': 2, 'greystoke': 3, 'line-up': 1, 'shelby': 1, 'flic': 1, "`6'": 1, 'despondent': 2, 'fiftieth': 1, 'bubble-headed': 1, 'screenings': 6, 'initially': 13, "screenplay's": 2, 'cold-hearted': 3, '5%': 2, 'keyboard': 2, 'cares/if': 1, 'too-good-to-be-true': 1, '_a_night_at_the_roxbury_': 1, 'archival': 1, 'countryish': 1, 'blasting': 4, 'lebanese-born': 1, 'fifteen-minute': 1, 'reminiscent': 21, 'percentages': 1, 'deprophesized': 1, 'wires': 7, '_does_': 2, 'blackly': 1, 'artwork': 1, 'barcalow': 1, 'mistreatment': 2, 'fascist': 2, 'remakes-cum-bastardizations': 1, 'hammers': 1, 'significance': 12, 'screened': 5, 'grocery': 4, 'scream': 51, '--michael': 2, 'ten-plus': 1, 'ryan_': 2, 'merchant': 1, 'closes': 4, 'timely': 3, 'attribute': 3, 'scientific': 4, 'trappings': 7, 'unrelentingly': 2, 'shadix': 1, 'taller': 2, "d'onofrio's": 1, 'joan': 14, 'fencing': 2, 'ranting': 1, "slashers'": 1, 're-appear': 1, 'clinic': 4, 'bochner': 1, 'monstrous': 4, 'juices': 2, 'tedium': 9, 'screenshots': 1, "`she's": 2, 'vikings': 4, "'shemp'": 1, 'delany': 2, 'titty': 1, '91-minute': 1, 'once-ever-5000-years': 1, "ricky's": 2, 'careless': 3, 'thora': 1, 'unfortunatly': 1, "oldman's": 1, "eflman's": 1, 'timm': 1, 'missed': 34, 'bad-asses': 1, 'fortresses': 1, 'rapist': 2, 'floppy': 1, 'cardboard': 18, 'elder': 1, 'fun-loving': 1, 'criticizes': 3, 'pansexual': 1, "god'": 1, 'hooked': 6, "god's": 4, 'bullying': 3, 'tolerated': 1, 'professional': 27, 'demographic': 3, 'levison': 1, 'imitations': 1, 'drivers': 2, 'gravity': 8, 'theresas': 1, 'afro-americans': 1, 'enticingly': 1, 'off-base': 1, 'retorts': 1, 'purchase': 2, 'discounted': 1, 'thang': 1, 'shave': 3, 'nation': 7, 'jealous': 10, 'magician': 2, "boneheaded'": 1, "anderson's": 4, 'savannah': 1, 'happy': 67, 'oversleeps': 1, 'compensating': 1, 'drones': 2, 'monk': 3, 'bonanza': 1, 'well-the': 1, 'friedklin': 1, '[insert': 1, 'musicians': 2, 'coaxes': 1, 'dietl': 1, 'verlaine': 1, 'first-time': 8, 'bidder': 3, 'hitch-': 1, 'fiona': 1, "life's": 4, 'churned': 3, 'spying': 2, 'runaway': 9, 'passover': 1, 'bloodied': 2, 'cheng': 1, "rameses'": 1, 'schemer': 1, 'participate': 7, 'anakins': 1, 'nether-realm': 1, 'hubbard': 4, 'projectioner': 1, 'pedro': 4, 'wheedon': 1, 'soppy': 3, 'recreational': 1, 'storyline': 39, 'slapping': 3, 'pompous': 4, '24-hour': 2, 'ginger': 4, 'anally': 1, 'off-limits': 1, 'belive': 1, 'novelty': 8, 'bask': 1, 'metaphor': 7, 'unattractive': 4, 'typecast': 2, 'embarrassing': 36, 'ad-libbed': 2, 'steroid': 4, 'decipherer': 1, 'offscreen': 2, 'bloodflow': 1, 'instigate': 2, 'cinemax': 5, 'sustain': 15, 'spectacles': 2, 'loneliness': 4, 'midwestern': 2, 'potentially': 24, 'mohiam': 1, 'super-duper': 1, 'non-ape-speaking': 1, 'maury': 1, 'suffer': 16, "somebody's": 1, 'helpless': 7, 'respective': 16, 'fleetingly': 1, "1999's": 8, 're-writes': 1, 'taiwan': 3, 'hooligans': 1, 'mitchum': 2, 'flinstone': 1, 'emoting': 2, 'nice': 108, 'mediating': 1, 'townspeople': 3, 'member': 51, "blossom's": 1, 'radically': 1, 'nm': 1, "harkonnen's": 1, 'disuse': 1, 'seedier': 1, 'swell': 3, 'marine': 7, 'hours': 85, "brynner's": 1, 'amblyn': 1, 'peppy': 1, 'composed': 9, 'escaped': 9, 'excises': 1, 'sharpening': 1, 'demonic': 4, 'underperforming': 1, 'invade': 3, 'bystanders': 1, 'inner-workings': 2, 'hangers': 1, 'flickering': 2, 'mid-life': 1, 'root': 19, 'heartwarmers': 1, 'pretty-boy': 2, 'companion': 8, 'filling': 3, 'prestigious': 10, 'deadpan': 4, 'commandments': 1, 'urkel': 2, "ann's": 1, 'truman': 9, 'shea': 1, "nobody's": 3, 'physicist': 2, 'cheif': 1, 'acute': 2, 'tardio': 1, 'horizon': 12, 'joes': 1, 'acrobatic': 2, 'dismissed': 1, 'tsui': 5, 'talker': 1, 'wasn\x12t': 1, 'grades': 2, 'smiles': 5, "nossiter's": 1, 'german-': 1, 'skanky': 1, 'knell': 1, 'mackenzie': 1, "'sci-fi'": 1, 'drying': 2, 'polarized': 1, 'gentile': 1, 'rigeur': 2, 'wider': 2, 'charisma-free': 1, 'stupids': 1, 'encased': 3, 'schmoozy': 1, 'hormonally-advantaged': 1, 'hospitalizes': 1, 'situations': 58, 'anecdotal': 1, 'gainfully': 1, 'syrup': 2, 'handy-dandy': 1, 'cages': 1, 'retribution/fight': 1, 'ganz': 1, 'butterfly': 2, 'keith': 2, '_many_': 1, 'mistreats': 1, 'grown-worthy': 1, 'cassandra': 1, 'theorists': 1, 'assination': 1, 'twisted': 21, 'teaming': 1, "novalee's": 1, "silver's": 1, 'kristin': 2, 'overacted': 3, '_here_': 1, 'monks': 4, 'hypodermic': 1, '000--paltry': 1, 'fabric': 1, 'gif': 1, 'presentation': 11, 'sissy': 2, 'lawnmowers': 1, "period's": 1, 'apparantly': 1, 'cake-walk': 1, 'requesting': 2, 'evaporates': 1, 'lawerence': 1, '45%': 1, 'gaerity': 1, 'patience': 11, 'orphanage': 1, 'awry': 10, 'veronica': 6, 'over/underwritten': 1, 'protect': 15, 'chopped': 2, 'unfailingly': 2, 'under-written': 1, 'leather-clad': 4, 'holmes': 9, 'riders': 1, 'vice-president': 1, 'mentos': 1, 'recognized': 5, 'ambience': 2, 'catwalks': 1, 'bloodsucking': 1, 'profusely': 1, 'softly': 1, 'partisanship': 1, 'rienzo': 1, 'conducted': 3, 'masterful': 7, "wilson's": 1, 'oft-played': 1, 'ostertag': 1, 'stargate': 3, 'clarification': 1, 'whaley': 1, 'machete': 2, 'atkins': 2, 'adequate': 16, 'monarch': 1, "1998's": 8, 'heed': 4, 'concede': 2, 'authority': 13, 'warfield': 1, 'incredible': 29, 'nuptial': 1, 'oscar-winner': 3, 'out-and-out': 1, "audiences'": 1, 'recounting': 2, 'doubt': 64, 'krueger': 3, 'slam-bang': 1, 'blemish-free': 1, 'prophet': 1, 'unmistakeable': 1, 'psychlos': 4, 'clich-heckerling': 1, 'tigers': 1, 'shiva': 1, 'clout': 5, 'opening': 104, 'redirecting': 1, 'peachy': 1, "all-in-a-day's-work": 1, 'cookies': 3, 'malicious': 4, 'disown': 1, 'burn': 8, 'park': 42, 'artillery': 1, 'immortal': 3, 'reversal': 2, 'yeller-style': 1, 'bribery': 1, 'drawn': 24, '10-day': 1, "master's": 1, 'differentiated': 2, 'machine-like': 1, 'jaded': 6, "dragonheart's": 1, 'mining/slave': 1, 'mess': 106, 'curiosities': 1, "end'": 1, 'mac': 2, 'treasured': 2, 'shapely': 1, 'camp': 36, "brazil's": 1, 'battling': 5, 'tabasco': 1, 'by-the-numbers': 8, 'desmond': 1, 'pears': 1, 'souffle': 1, 'refined': 2, 'originally': 26, 'detailed': 6, 'innovate': 1, 'bar': 29, 'proud': 13, 'threaten': 4, 'jonny': 1, 'housewife': 3, 'portland': 2, 'todde': 1, 'bumbling': 13, 'disco-': 1, 'beware': 7, 'abril': 1, 'flight': 10, 'bugs--how': 1, 'hooey': 1, 'sundance': 3, 'negligence': 1, 'relaxed': 2, 'motive': 11, 'plunder': 1, 'expel': 2, 'monosyllabic': 1, 'turn-of-the-century': 1, 'ariyan': 1, 'hunters': 4, 'intimidating': 1, 'grindhouse': 1, 'smokejumper': 1, 'placards': 1, 'mitchell': 7, 'gardening': 1, 'dismiss': 4, 'waldo': 2, "christopher's": 1, 'films-run': 1, 'spock': 2, 'highest-ranking': 1, 'crafts': 1, 'weekend': 24, 'efficiently': 3, 'shameful': 2, 'messiness': 1, 'powered': 2, 'camaraderie': 1, 'man-ethnicity': 1, 'sheila': 2, 'reginald': 1, 'ironically': 21, 'scary': 52, 'windows': 3, 'quick-tempered': 1, 'noteworthy': 9, 'alaska': 3, "croft's": 1, 'bladder': 1, 'rancid': 1, 'hi-jinks': 1, 'forgets': 11, 'womantic': 1, 'marilyn': 8, 'graced': 3, 'remade': 3, 'tit-shot': 1, 'merges': 2, 'wannabes': 5, 'zen-like': 1, 'niagra': 1, 'satisfactory': 8, 'alongside': 5, 'dumbass': 1, 'viking': 3, 'menzies': 2, 'sporadically': 8, 'gish': 2, 'richmond': 2, 'bluescreen': 1, 'devise': 3, 'com/~canran': 1, 'test': 32, 'patois': 1, 'asks': 42, 'doubtless': 1, 'lindenlaub': 1, 'elisabeth': 8, 'departed': 1, 'paranoid': 12, 'mankiewicz': 1, 'ignorant': 6, 'devestating': 1, 'caper-esque': 1, 'klumps': 1, 'batteries': 2, 'plague': 6, 'edification': 1, "nation's": 4, 'anti-gay': 1, 'rosenberg': 2, 'essence': 10, 'mourning': 1, 'soderbergh': 2, 'carolina': 6, 'drab': 5, 'i-think-i-actually-have-': 1, 'licensed': 1, 'foil': 5, 'eugene': 11, 'tagline': 9, 'chords': 4, 'experiencing': 4, 'splish': 1, 'ladder': 7, 'spook': 1, 'trailed': 1, 'water': 61, 'sell': 19, 'companions': 4, 'sting': 5, 'air': 58, 'bitchiness': 1, 'boris': 3, 'incronguously': 1, "players'": 1, 'self-help': 1, 'prolific': 2, 'billowing': 1, 'stoners': 3, "friedkin's": 1, 'hairdresser': 3, 'unacceptable': 4, 'jaw-dropper': 1, 'gasps': 1, 'reassess': 1, 'warm': 15, 'reclaim': 4, 'legends': 8, 'prizefighter': 1, 'silvie': 1, 'bigelow': 1, 'baby-sitter': 1, 'studio': 52, 'magnificently': 1, 'depression': 7, 'gun-': 1, 'unbroken': 1, 'quits': 2, 'lures': 1, 'festival': 13, 'barbarity': 1, 'vicinity': 1, 'bat-loathing': 1, 'provided': 29, 'pitcher': 4, 'laid': 11, 'fully-animated': 1, 'shouts': 9, 'veil': 1, 'squish': 1, 'gripping': 3, 'moira': 1, 'unison': 2, 'spending': 21, 'countered': 1, 'ancy': 1, 'sputtering': 4, 'amigos': 1, 'hurray': 1, 'kidding': 8, 'thoughtfulness': 1, 'diverts': 1, 'generalized': 1, 'iranians': 1, 'severe': 8, 'do--become': 1, 'flatten': 1, 'pedestrian': 8, 'stagnant': 2, 'avaricious': 1, 'commission': 1, 'shouldn+t': 1, 'propell': 1, 'dill': 1, 'glib': 5, 'antonio': 11, 'full-blown': 2, 'transform': 6, 'fascinating': 21, 'keyzer': 1, 'total': 47, 'ut': 1, 'numbingly': 1, 'ho-sun': 1, 'sarris': 1, 'legionnaire': 1, 'shaking': 8, 'motorist': 1, 'clocking': 1, 'henchmen': 12, 'pursuing': 2, 'competence': 2, 'revisit': 2, 'shatner': 4, 'snatchers': 4, 'connotations': 1, 'rage': 11, 'blindly': 2, "messenger'": 1, 'pertinent': 1, 'aptly-titled': 1, 'inside-out': 1, 'rooney': 2, 'sex-film': 1, 'perverse': 5, 'spader': 7, 'jerky': 7, 'keeping': 28, 'take-no-prisoners': 1, 'stolen': 18, 'helping': 16, 'doubtful': 3, 'heros': 1, 'choice': 38, 'explosive': 6, 'not-so-engrossing': 1, 'knit': 1, "grace's": 5, 'exquisitely': 1, 'technically': 12, 'wins': 16, 'vh1': 3, 'chrysler': 3, "baby's": 3, 'overlooking': 4, 'rowlands': 4, '999': 3, 'covert': 1, 'relaxation': 2, "jeffrey's": 1, 'moviegoers': 9, 'small-time': 1, 'trilian': 1, 'blueprints': 2, 'libby': 3, 'heartstopping': 1, 'mish-mash': 1, 'karate': 1, 'laptop': 2, 'golly': 5, 'groundhog': 1, 'uncomfortable': 13, 'two-minute': 2, 'eatery': 1, "lumet's": 4, 'leo': 5, 'planted': 2, 'skewering': 1, 'roses': 1, 'threateningly': 1, 'settle': 21, 'ugly': 35, 'ineptitude': 8, 'bad': 457, "deadpan'": 1, 'represent': 4, 'legends--is': 1, 'bounces': 3, 'hypothetical': 1, "three's": 3, 'visions': 11, 'appendages': 1, 'self-righteous': 3, 'saddens': 1, 'filmmaker': 26, 'didja': 1, 'other--': 1, 'ronald': 2, 'morass': 1, 'hindsight': 1, 'ghost-like': 2, 'transfixed': 2, "drugs'": 1, 'stormtroopers': 1, 'passage': 1, 'prophetic': 2, 'aces': 2, 'seamlessly': 1, 'babe-magnet': 1, 'small-scale': 1, 'skewed': 2, 'unengaging': 2, 'over-directing': 1, 'well-incorporated': 1, 'potion': 1, 'suckers': 2, 'outings': 1, "sunday'": 1, 'debut': 34, 'needless': 16, 'spotting': 3, '123': 1, 'secluded': 3, 'suggest': 23, 'antoine': 1, 'eternity': 5, 'desintegration': 1, 'landfill': 3, 'crassly-calculated': 1, 'flashily': 1, '000-foot': 1, 'microscopic': 1, 'cho': 1, "series'": 2, 'united': 20, 'weber': 1, 'outrageously': 3, "'hollow": 1, 'dreamer': 2, 'teen-flicks': 1, 'formulaic': 19, '1967': 3, 'fellowship': 1, 'lamentable': 1, 'honks': 2, 'francisco': 7, 'ikwydls': 1, 'italian': 9, 'arraigned': 1, 'suicides': 3, 'college-bound': 1, "willis's": 1, 'musically': 1, 'profess': 1, "`demons'": 1, 'idle': 6, 'housed': 2, 'favored': 1, 'looming': 2, 'writing/directing': 1, 'transfers': 1, 'good-guy': 1, 'iguana': 2, 'perspective': 14, 'us-mexico': 1, 'aficionados': 1, 'excelled': 1, 'gladiators': 2, 'milano': 1, 'pr': 3, "girlfriend's": 5, 'flits': 1, 'automatically': 8, 'prepubescent': 1, 'tree': 13, 'blonde': 14, 'hemingwayesque': 1, 'implores': 1, '81-minute': 1, 'condemned': 2, 'dreadfully': 4, 'agutter': 1, 'intriguingly': 1, 'false': 13, 'sneakers': 2, 'liberty': 3, 'mcmillan': 2, 'banished': 4, 'ranging': 7, 'hypothetically': 2, 'cruelty': 4, 'cut': 66, 'genes': 2, 'hardest': 2, 'weslely': 1, "ishioka's": 1, 'buliwyf': 1, 'jonathon': 1, 'borrow': 6, 'cuban/miami': 1, 'focus': 50, 'thrilling': 10, 'dullness': 4, 'heh': 1, 'rood': 1, 'ahmed': 5, 'wrongdoing': 1, 'patricia': 5, 'deplorable': 1, 'wichita': 1, 'heights': 3, 'recommends': 1, 'edison': 2, 'eric': 27, 'favorably': 1, 'vowed': 2, 'college-set': 1, 'ryuichi': 1, 'trogolodistes': 1, 'prejudice': 1, 'evolutionary': 1, 'mehki': 1, 'sexually-ripe': 1, 'delight': 9, 'beds': 4, 'mccall': 1, 'shortage': 1, 'jimmy': 18, 'plummets': 2, 'delectable': 1, 'credited': 15, 'drill': 6, "branaugh's": 1, 'terrify': 2, 'magritte': 1, 'translates': 2, 'doesn': 1, 'thriller/suspense': 1, 'wormer': 2, 'changed': 32, 'belays': 1, 'meanie': 1, 'maliciously': 1, 'aw': 3, 'lams': 1, 'minutiae': 2, "scientist's": 1, 'anticipate': 2, 'introduces': 19, 'rico': 2, 'yacht': 2, 'aussie': 1, 'depthless': 1, 'koren': 1, 'landslide': 2, 'dollar': 24, 'emannuelle': 1, 'oversentimental': 1, 'step': 29, 'battery-charging': 1, 'townie': 2, 'stuttering': 4, 'ke-': 1, 'ant': 3, 'ex-presidents': 1, 'backside': 1, 'holden': 1, 'confronted': 4, 'empathizes': 1, 'malibu': 1, 'troopers': 14, 'wh': 1, 'brownlee': 2, 'uttering': 4, 'instructions': 2, 'self-destruct': 2, 'cards': 10, 'speedily': 2, "boulle's": 1, 'twelve-year-old': 1, 'touching': 21, 'seattle-based': 1, 'misconceived': 1, 'chemical': 5, 'accompanying': 8, "payne's": 1, 'pelvis': 1, 'investor': 2, 'sexier': 2, 'translated': 2, 'firepower': 1, 'collapses': 5, 'zoo': 3, 'humanly': 3, 'rand': 1, 'car-sex': 2, 'cost': 12, "most'": 1, 'apple': 8, 'uprising': 1, 'skewered': 2, "schindler's": 7, 'copy': 18, 'ogled': 3, 'captivated': 2, 'potty-mouth': 2, 'supernatural': 15, 'industrialist': 1, 'munchie': 1, "paramount's": 1, 'linearity': 1, 'vamp': 3, 'curiously': 8, 'silly': 69, 'friendship': 9, 'sporty': 4, 'chestnut': 2, 'rooftop': 1, 'stands': 38, 'pryor': 2, 'often-acclaimed': 1, 'supplying': 2, 'press': 25, 'showgirls': 15, 'patrolled': 1, 'sphere': 5, 'reason': 203, "person's": 6, 'pining': 3, 'dedication': 1, 'southern': 12, 'duped': 1, 'two-gunned': 1, 'driller': 3, 'oscar-contender': 1, 'dennis': 43, 'grey': 2, 'nighshift': 1, 'sender': 1, 'telephone': 3, 'personalities': 16, 'whack': 3, 'crock': 3, 'unheralded': 1, 'hunted': 4, 'port': 1, 'antithesis': 1, 'eggo': 1, 'dench': 2, 'fires': 6, 'cameraman': 6, 'waxes': 2, 'prentice': 1, 'charging': 2, 'pros': 2, 'circa-10th': 1, 'no-hoper': 1, 'tacked': 4, 'ghost-lady': 1, "bats'": 1, 'cooked': 2, 'talbot': 1, 'whatever-the-f': 1, 'persons': 5, 'departs': 1, 'scaring': 3, 'leigh': 7, "puppeteer's": 1, 'peterson': 3, '51': 2, 'magwitch': 1, 'front': 58, 'sniffs': 1, 'contingent': 1, 'johnson': 9, 'levels': 13, 'montoya': 3, 'enamored': 2, 'electro-green': 1, 'banderas': 11, 'arm-chair': 1, 'bulletproof': 2, 'dancers': 3, 'bits': 26, 'civil': 11, 'as--of': 1, 'pov': 2, 'considerable': 12, 'holm': 3, 'shoot': 40, 'bankrupt': 5, '1964': 4, 'sections': 2, 'ingrains': 1, 'told': 70, 'serial-killer': 2, 'deep': 70, 'mousse': 1, 'japanimation': 1, "version's": 1, 'brauvara': 1, 'spate': 1, 'winner': 15, 'ikea': 1, 'leaning': 3, 'fools': 5, 'recovers': 3, 'supergirl': 1, "'funny": 1, 'previous': 59, "schutze's": 1, 'plexiglas': 1, 'loner': 3, 'paste': 1, 'astoundingly': 3, 'hard-to-find': 1, 'days': 97, 'area': 16, 'renovation': 1, 'ranges': 2, 'kikujiro': 1, "1989's": 4, 'primitive': 7, 'lange': 7, '-drenched': 1, 'seinfeld': 7, "show's": 3, 'downers': 1, 'weasely': 1, '95%': 2, 'fess': 2, 'academic': 3, 'clowns': 2, 'conspicuously': 2, 'jennifer': 45, 'schwarzeneggar': 1, 'sonnenfelds': 1, 'bludgeoned': 1, 'piss-poor': 1, 'producing': 11, 'mi2': 1, 'tricia': 1, 'rispoli': 1, 'reviews': 29, 'juan': 3, 'line-': 2, 'dweebish': 1, 'shifts': 12, 'cliched': 37, 'pineapple': 1, 'hand-drawn': 1, 'unmatched': 1, 'tolkan': 1, 'casino/hotel': 1, 'damian': 1, 'clippings': 1, 'hard-sell': 1, 'swallow': 11, 'nuanced': 3, 'stills--heather': 1, 'levis': 1, 'ticked': 2, 'rancor': 1, 'vote': 9, 'breech': 1, 'herniated': 1, 'enlivens': 1, 'snuffing': 2, 'laundry-impaired': 1, 'half': 143, 'audiotape': 1, 'slow': 57, 'heenan': 1, 'winston': 2, 'marred': 3, '1930s': 1, 'gobbledy-gook': 1, 'yale': 2, 'disconcertingly': 1, 'misdemeanors': 1, '1928': 2, 'dispenses': 2, 'ice-cream': 1, 'decadent': 2, 'slooooow': 1, 'climbing': 6, 'courting': 1, 'vampire': 15, 'eraserhead': 2, 'meander': 1, 'angular': 1, 'analyzed': 2, 'intertwined': 4, "3's": 1, 'insinuate': 2, 'emperor': 4, 'gossip': 1, 'colt': 1, 'mistakenly': 4, 'equal': 13, 'factions': 1, 'misery': 8, 'emax': 1, 'well-placed': 1, 'pay': 64, 'veritably': 1, 'mugshots': 1, 'chops': 4, 'rusty': 4, 'slowly': 34, "officer's": 2, 'thy': 1, 'beast': 24, 'elicits': 1, 'angrier': 1, 'redecorating': 1, 'anxious': 1, 'reiner': 2, 'hui-kung': 1, 'alan': 26, 'broadly-drawn': 1, 'flirtatiously': 1, 'unanswerable': 1, 'modern-day': 3, "vic's": 1, 'rade': 3, 'weasted': 1, 'rough': 10, 'crs': 1, 'innovations': 1, 'eduardo': 1, 'infrequent': 2, 'dozen': 23, 'spacesuit': 1, 'tunnel': 5, "hodge's": 1, '23-year-old': 1, 'smartly': 2, 'controversy': 3, 'unremittingly': 1, 'pun': 9, 'hiroshi': 1, 'railroads': 1, 'artifically': 1, 'boatman': 1, 'epic': 16, 'overwritten': 1, 'squint': 1, 'wince': 4, 'connects': 1, 'borders': 10, 'killer/horror': 1, 'backwards': 6, 'future': 59, 'bungler': 1, "ryan's": 5, 'poorly-done': 1, 'knowledgeable': 2, 'importance': 10, 'opposing': 4, 'deepak': 1, 'conceivably': 2, '3000-level': 1, 'turgeon': 3, 'awsome': 1, 'maniacal': 2, 'unanswered': 7, 'clapet-the': 1, 'storyboards': 1, 'molesting': 1, 'groundbreaking': 4, "general's": 8, 'negativity': 4, '--you': 1, 'island': 39, 'taunt': 4, 'recorded': 4, 'cabinet': 3, 'masturbatory': 2, 'composing': 1, 'rumoured': 2, 'honeymooning': 1, "dobie's": 1, 'drop': 24, '1990s': 9, 'abortion': 4, "april's": 1, 'film-going': 2, 'addressed': 4, 'utter': 18, 'oeuvre': 3, 'faults': 7, 'examinations': 1, 'awwwww': 1, '82-minutes': 1, 'full-page': 1, 'smirk': 5, 'wonderbra': 4, 'ski': 3, 'farm': 10, 'conclusions': 3, 'initiation': 2, 'touches': 11, 'timoney': 1, "talon's": 1, 'libido': 4, 'alienated': 2, 'ellicited': 1, 'effective': 39, 'caan': 10, 'guess': 117, 'boogie': 11, "travis'": 1, 'ladybugs': 1, 'succeeds': 18, 'avenge': 6, 'honorable': 3, 'smokers': 1, 'trekker': 1, 'wrote': 53, 'nom': 1, 'shutting': 1, 'summing': 1, 'unending': 1, 'trim': 1, 'blackout': 2, "salt's": 1, 'bejesus': 1, 'quasi-': 1, 'multi-faceted': 1, 'romeo+juliet': 1, 'pulls': 18, 'assigns': 2, 'visibly': 1, 'white-haired': 1, 'breath': 10, 'schomberg': 1, 'adornments': 1, 'gerald': 4, 'interrogation': 2, 'republican': 4, 'self-referential': 1, 'bernard': 3, 'one-and-only': 1, 'monstrously': 2, 'common': 45, 'faulted': 2, 'ontop': 1, "nice-guys-who-don't-deserve-to-be-in-prison": 1, 'man-god': 1, 'immigrants': 3, 'workplace': 2, 'stamp': 9, 'mirkine': 1, 'far-fetched': 8, 'protecting': 6, 'downtrodden': 2, 'henpecked': 1, "hbo's": 1, 'collars': 1, 'rainbow': 3, 'eruption': 3, 'yojimbo': 1, 'sensational': 2, "'quack": 1, 'wowzers': 1, 'definition': 12, 'authentic': 8, 'beckinsale': 3, 'hell': 87, 'recommending': 4, 'chains': 4, 'slug': 3, 'armored': 5, 'theatre-shaking': 1, 'misfits': 4, 'whistle': 2, 'natasha': 18, 'shelmickedmu': 1, 'flood': 4, 'meat-hooks': 1, 'undecided': 3, 'praise': 5, 'peril': 6, 'trevor': 3, 'f/x': 3, 'ventricle': 1, 'suspend': 5, 'in-jokey': 1, 'woodside': 1, 'improbabilities': 1, 'shaping': 1, 'foreplay': 4, 'informercial': 1, '_54_--so': 1, 'bombing': 1, 'puppets': 4, 'na': 3, "girl's": 11, 'accumulated': 1, 'sickening': 8, 'pimple-faced': 1, 'complacent': 1, 'cola': 1, 'win': 41, 'algae': 1, 'packaged': 3, 'wordy': 1, 'hundred-million-dollar': 1, 'ruthless': 11, 'blame': 48, 'isaak': 1, 'moving': 40, 'complicates': 1, "bros'": 1, 'pastry': 1, 'paul': 79, 'attend': 5, 'productive': 1, 'deserved': 11, 'finding': 29, 'newsreporter': 1, 'sideshow': 1, 'mastered': 1, 'samoan': 1, 'mikael': 3, 'entertained': 14, 'brainchild': 1, 'honoring': 2, 'pandemonium': 2, 'pteranodons': 1, 'timed': 1, 'dart': 1, "employer's": 1, 'docks': 1, 'simian': 2, 'diatribe': 2, 'receives': 13, 'internment': 1, 'psychiatrist': 14, 'slutty': 5, 'uberhack': 1, 'hastily': 5, 'brethren': 2, 'argentina': 1, 'sums': 9, 'dorky': 2, 'sandworms': 1, 'four-letter': 2, "carpenter's": 10, "max's": 4, 'pitillo': 5, 'paramount': 7, 'geronimo': 1, 'memorizing': 1, 'manufacturing': 2, 'rooftops': 4, 'breadwinner': 1, "_america's_funniest_home_videos_": 1, 'radha': 1, 'ruffians': 1, 'infuses': 1, 'ended': 28, 'nightingale': 2, 'yuelin': 1, 'stirrings': 1, 'automated': 1, 'writing': 81, 'screenplay-and': 1, 'psychobabble': 1, 'oxymoron': 1, 'tatooine': 1, 'fascistic': 1, "prince's": 2, 'pulsing': 2, 'serpents': 1, 'well-connected': 1, 'temperature': 2, 'obsessive': 4, 'unsurprisingly': 1, 'lapse': 3, 'claymation': 1, 'illusions': 1, 'nympho': 1, 'half-buttoned': 1, 'catatonically': 2, 'recounts': 2, 'preoccupation': 2, 'frolicking': 2, 'brown-it': 1, 'motormouth': 2, 'hilariosly': 1, 'dead-on': 3, 'shells': 2, 'direction': 98, 'feelings': 30, 'accelerating': 1, 'down--to': 1, 'yoshi': 1, 'subject': 52, 'cringe-inducing': 3, 'relocates': 1, "mtv'ish": 1, 'disguise': 15, 'occurrence': 4, 'minority': 2, 'takeshi': 1, 'bridget': 5, 'removes': 1, 'dove': 4, 'parents': 54, 'anti-shark': 1, 'scoopfuls': 1, 'digress': 5, "four's": 1, 'snobs': 1, 'overt': 4, 'scratches': 2, 'exists': 15, 'mira': 3, 'expound': 1, 'matted': 1, 'ethic': 1, 'suffocating': 1, 'hide': 26, 'cog': 1, 'zeus': 1, 'spiritless': 2, 'rapper': 5, 'cuckoo-bird': 1, 'caressing': 1, 'mekanek': 1, 'attacking': 7, 'titilation': 1, 'ex-flame': 1, 'mind-fuck': 1, 'crusaders': 1, 'frightened': 11, 'bruce': 42, 'risqueness--mostly': 1, 'teens': 21, 'contempt': 5, "50's": 2, 'stylelessly': 1, 'cripple': 1, 'sneaky-smart': 1, "cinematographer's": 1, 'harridans': 1, 'foxy': 2, 'howlingly': 1, 'dwarf': 2, 'diffuse': 1, 'sacrificial': 1, 'catsuit-clad': 1, 'rambles': 2, 'emmannuelle': 1, 'nonsense': 24, 'co-script': 1, 'standing': 28, 'hurtful': 1, 'crafty': 2, 'cold': 41, 'misha': 2, 'champion': 9, 'subsist': 1, 'burnett': 1, 'breathing': 10, 'extraordinaire': 2, 'pharmacological': 1, 'brothels': 1, 'serviceable': 3, 'eqypt': 1, 'toilets': 2, 'blender': 1, 'oaf': 1, 'brand': 16, 'juvenile': 13, 'foul-humored': 1, 'best--an': 1, 'hormonally': 1, 'runner': 13, 'soused': 1, 'cacophonous': 1, "crazy's": 1, 'weighed': 5, 'mostel': 1, 'rosy': 1, 'claws': 3, 'supported': 3, '2d': 1, 'decomposing': 1, 'sunrisesucks': 1, "f'gawds": 1, 'roth': 11, 'temporary': 1, 'titus': 4, 'goofball': 2, '1900s': 1, 'remaining': 21, 'lemmon': 3, 'locale--to': 1, 'rebuts': 1, 'institution': 10, 'disgrace': 1, 'clear': 60, 'riding': 12, 'drama-packed': 1, 'reporters': 4, 'leatherface': 3, 'planning': 11, 'ego': 9, 'squalid': 1, 'danforth': 1, 'princeton': 1, 'synopses': 1, 'bows': 2, 'ferret': 2, 'cate': 3, "schumacher's": 5, 'transformers': 1, 'nicotine': 2, 'mujibur': 1, 'masterminded': 2, 'cora': 1, "'long-lost'": 1, 'honey-bunny': 1, 'juggles': 1, '800': 1, 'themes': 18, 'epitaph': 1, 'nauseam': 2, 'edited': 17, 'artist-speak': 1, 'quasi-artistic': 1, 'casting-wise': 1, 'serpico': 3, 'gots': 1, 'flatulence': 5, 'cleophus': 1, 'owning': 1, 'b-actress': 1, 'box': 54, 'scotch-induced': 1, 'roosevelt': 1, 'confused': 37, 'degenerative': 1, 'statements': 7, 'broadside': 1, 'womanizing': 2, 'weaponesque': 1, 'plotholia': 1, 'ingesting': 2, 'isles': 1, "amazing's": 1, "poem's": 1, 'books-about-movies': 1, 'm-16': 1, 'hiave': 1, 'jon': 27, 'aide': 3, 'micelli': 2, 'regional': 1, 'long-dead': 1, 'arouses': 2, 'applauded': 1, 'taste': 30, 'artist-directors': 1, 'hershey': 2, 'lawman': 2, 'dionysius': 1, 'hijinks': 2, 'terrific': 18, 'eyelids': 1, 'prancer': 1, 'christ': 3, 'prices': 2, 'procured': 1, 'klutzy': 3, 'well-off': 1, 'distinguish': 3, "hanks'": 2, 'evacuated': 3, 'chief': 32, 'multimillionaire': 1, 'sachs': 1, 'portos': 1, 'colliding': 2, 'rochester': 1, 'sea-sickness': 1, 'fooling': 2, 'deficiency': 2, 'curmudgeonly': 1, "libby's": 1, 'bello': 3, 'bantering': 1, 'rounding': 5, "hotel's": 1, 'compensation': 2, "duguay's": 1, 'nine-film': 1, 'unintelligently': 1, 'inquisitive': 2, "lucasfilm's": 1, 'hesse': 1, 'acknowledgement': 1, 'janssen': 8, 'magnifcent': 1, 'connect': 8, 'eliminating': 1, 'float': 6, 'net-surfing': 1, "firm'": 1, 'goofy-but-loveable': 1, 'proven': 13, 'confidante': 1, 'dourif': 4, 'ill-marketed': 1, 'marry': 20, 'intoxicated': 1, 'obeying': 1, "jeopardy'": 1, 'organs': 2, 'non-specificity': 1, 'trusted': 7, 'hydrogen': 1, "m&m's": 1, 'uv': 1, 'demented': 6, '1956': 1, 'lazily': 1, 'extraordinary': 12, 'revives': 1, 'misuse': 1, 'mistaken': 15, 'gold-diggers': 1, 'restrained': 4, 're-mixing': 1, 'long-': 1, 'henriksen': 2, "team's": 5, 'less-than-lackluster': 1, 'lhermitte': 1, 'security': 34, 'establishment': 4, 'quayle-ish': 1, 'not-so-': 1, 'daviau': 1, 'flattened': 1, 'doofus': 1, 'stakeout': 2, 'accolade': 1, 'cabbie': 1, 'blindingly': 1, 'classmate': 3, 'cape': 7, 'degenerates': 11, 'nosebleeds': 1, 'ranks': 7, "claudia's": 2, 'unquestionable': 1, '10': 61, 'literally': 51, 'petrified': 2, "'s": 32, 'pattern': 5, 'misgivings': 1, 'quests': 1, 'spacecraft': 6, 'glue-like': 1, 'rescuers': 1, 'full-to-bursting': 1, 'boo': 4, 'floorboards': 1, 'hammerstein': 2, 'sheryl': 3, 'angela': 7, 'slung': 1, 'screendom': 1, 'tediousness': 1, 'doozy': 2, 'soda': 3, 'reusing': 1, 'eighteen': 4, "woman's": 19, '_holy_man_': 1, 'manuscript': 1, 'reformer': 1, 'handling': 14, 'venerable': 1, 'pentagon': 2, 'baffling': 3, 'norse': 3, 'surgical': 1, 'eroticism': 5, '90-plus': 1, 'toils': 1, 'separated-at-births': 1, 'maelstrom': 2, 'bonham': 4, 'beckham': 1, "love's": 4, 'local': 69, 'waltzes': 1, 'proudly': 2, 'merry': 4, 'jubilantly': 1, 'pacino': 9, 'blossom': 3, 'replies': 8, 'richie': 3, 'congress': 1, 'exposing': 1, 'avoided': 6, 'antagonizing': 1, 'contagious': 1, "abraham's": 1, 'uk': 2, 'discussed': 1, 'citywide': 1, 'bolek': 1, 'beesley': 1, 'cartoon': 38, 'rubber-faced': 1, 'cornwall': 1, 'fallen': 24, 'creamy': 1, 'revenues': 2, 'scumbag': 2, 'graphically': 1, 'escape': 44, "chestnut's": 1, 'nebraska': 1, 'lessened': 1, 'bekim': 1, 'inhospitable': 1, 'voiceover': 3, 'frames': 4, 'dams': 1, 'didn\x12t': 1, 'evidence': 31, 'cynic': 1, 'skinnier': 1, 'bothersome': 3, 'marvelously': 3, 'villain/hero': 1, "shelia's": 1, 'haas': 1, 'admitting': 1, 'roller-coaster': 1, 'feeble': 12, 'charitable': 2, 'huntz': 1, 'worship': 1, 'scientists': 16, 'approved': 1, 'understandable': 11, 'confidence': 13, 'nicholas': 11, 'unmoved': 2, 'cursory': 1, 'starbucks': 1, 'disoriented': 1, "needy'": 1, 'unwelcome': 5, 'on-screen': 15, 'enhancers': 1, 'clinging': 2, 'undresses': 1, 'possessed': 14, 'mancuso': 1, 'coaster': 6, 'meaner': 1, 'vital': 7, 'clouds': 4, '`my': 1, 'extraneous': 6, 'egomania': 1, 'flirts': 3, 'mos': 1, 'pre-launch': 1, 'alter-ego': 1, 'oh-so-fabulous': 1, "goldberg's": 1, 'longer': 53, 'sways': 1, 'humanity--is': 1, 'tuning': 1, 'pithy': 1, 'upstaged': 3, 'dylan': 7, 'kihlstedt': 1, 'emphasizing': 1, 'slam': 3, 'husband-to-be': 1, 'classics': 15, 'half-human/half-alien': 1, 'seemingly-endless': 2, 'three-minute': 4, 'peeing': 2, 'fathers': 2, 'unimaginative': 10, 'depp-type': 1, 'build-up': 3, 'mid-eighties': 1, 'hustling': 2, "andrew's": 1, 'processors': 1, 'performed': 3, 'fourteen': 1, 'nyu': 1, 'quotes': 8, 'garrett': 3, 'principles': 3, 'charleton': 1, 'desensitized': 1, 'swanson': 4, 'childish': 7, 'geezer': 2, 'wolf': 6, 'dichotomy': 1, 'toothless': 2, 'waddle': 1, 'collecting': 6, 'horror-movie': 1, 'clubbed': 1, 'mid-teen': 1, 'charlize--surprise': 1, 'khan': 2, 'gump': 1, 'racer': 1, 'islam': 1, 'renounce': 1, 'royalties': 3, 'heroine--gets': 1, 'premises': 7, 'nowadays': 9, "adjuster'": 1, 'climate': 2, 'alt-rock': 1, 'senatorial': 1, 'valderrama': 1, 'old-self': 1, 'titillate': 2, 'mathew': 1, 'staterooms': 1, 'wisdom': 17, 'tobolowsky': 1, 'walnuts--except': 1, 'jump-roping': 2, 'contemporary': 11, 'considered': 31, 'ewwwww': 1, 'slinking': 1, 'usually-splendid': 1, 'idiocy': 6, 'glove': 1, 'wanna-see-how-fast-i-can-drives': 1, 'estevez': 4, 'whilst': 3, "bunk-you're": 1, 'lesseps': 1, 'sigh-inducing': 1, 'over-use': 2, 'feminist': 4, 'fairfax': 1, "brill's": 1, 'now-tired': 1, 'funnyman': 1, 'nunn': 1, 'highly-publicized': 1, 'budgets': 4, 'bountiful': 1, 'birthdays': 1, 'ryan/andy': 1, 'cassie': 1, 'egotisitical': 1, 'mykelti': 3, 'plain': 37, 'noticing': 4, 'made-up': 2, 'siamese': 1, 'bribes': 1, 'hands-down': 2, 'gut-wrenchingly': 1, 'cowardly': 1, 'fraught': 2, 'pulsating': 2, 'glimpsing': 1, 'personality': 39, '18+': 1, '1': 75, 'foreclosure': 1, 'all-important': 2, 'sorrier': 1, 'unfocused': 6, 'dopy': 1, 'neighborhood': 9, 'atheist': 2, 'buddying': 2, 'grater': 1, 'mccabe': 2, 'desensitization': 1, 'pheromones': 1, 'gunk': 1, '17%': 1, 'beast-men': 1, "bachelor's": 1, 'dredges': 1, 'boating': 2, 'impressed': 16, 'sumptuous': 3, "raison-d'etre": 1, 'big-shot': 2, 'rodney': 2, 'time': 546, 'hearing': 23, 'promotions': 1, 'brendan': 3, 'pays': 7, 'hops': 3, 'difilippo': 1, 'restlessly': 1, 'belonged': 2, 'bill': 43, 'one-hundred': 3, 'spoofing': 2, 'undertook': 1, 'masterminding': 1, 'vacillating': 1, 'accurate': 12, 'fudd': 1, 'pearl': 5, 'underlings': 1, 'surrogate': 5, 'nosy': 2, 'venereal': 1, 'performance': 216, 'sheen': 9, "band's": 4, 'outlandish': 5, 'subconsciously': 2, "shows'": 1, "isn''t": 1, 'meat': 11, 'glenwood': 1, 'branded': 1, 'headlines': 3, "approach'": 1, 'goodnight': 2, 'fotomat': 1, 'bon': 4, 'b-monster': 1, 'gonzo': 2, 'dexterity': 1, 'rubell': 4, 'collaborated': 4, 'veer': 2, 'daniels': 9, 'buffed-up': 1, 'psychotic': 18, 'board-room': 1, 'ups': 4, 'pre-sold': 2, 'etch': 1, 'cuz': 1, 'incorporate': 2, 'admittingly': 1, 'updating': 3, 'marker': 2, "drivin'": 1, 'comedy': 236, 'stillborn': 2, 'psychosis': 2, 'exacts': 1, 'instrumental': 3, 'image': 31, "he'd": 18, 'farrellys': 4, 'discotheque': 1, 'longshot': 1, 'lisp': 1, 'collective': 7, 'didn': 1, 'cynthia': 3, 'reeves': 21, 'omar': 8, 'childs': 1, 'tarantino': 10, 'sexes': 4, 'deceit': 1, 'julian': 4, 'shelves': 3, 'semi-remake': 1, 'double-lines': 1, "dunaway's": 1, 'songwriter': 2, 'neil': 15, 'toilet': 16, 'technobabble': 1, 'middle': 89, 'fancies': 1, 'nc-17': 5, 'macho': 16, 'mccoll': 1, 'creator-owned': 1, 'uninfected': 1, 'mysterty': 1, 'viola': 1, 'standpoint': 3, 'bartholomew': 1, 'mainframe': 1, 'association': 3, 'eye-turned': 1, 'nexus': 1, "professor's": 1, 'compulsive': 2, 'antidote': 5, 'sup-plots': 1, 'kama': 1, 'rerun': 1, "car's": 1, 'ripping': 4, 'hardball': 1, 'equipped': 6, 'faye': 4, 'directoral': 1, "dog's": 1, 'murmur': 1, 'batarangs': 1, 'vitamin': 1, "jerry's": 3, 'jails': 1, 'shimmying': 1, 'sweat': 7, 'softy': 2, 'social': 24, 'comprehensible': 1, "halloween's": 1, 'progressive': 1, 'pitt': 5, 'elliot': 1, 'spreading': 4, 'cox': 13, 'two-thirds': 3, 'shannen': 1, 'practice': 10, 'conventions': 6, 'extend': 3, 'picture': 155, 'elimination': 1, "corpses'": 2, '_dead_': 1, 'well-deserving': 1, 'inception': 1, 'amateur': 8, 'newspaper': 10, 'hurriedly': 1, 'word--it': 1, 'grace': 37, 'cia': 8, 'christiane': 1, 'navigators': 1, 'jabbering': 1, 'elusive': 7, 'action/crime': 1, '`apollo': 1, 'made-for-tv': 8, 'koopa': 1, 'http': 3, 'onlooker': 1, '\x13ripley': 1, 'planetarium': 1, 'abruptly': 9, 'bourke': 1, 'discussion': 11, 'second-to-last': 1, 'masterless': 1, 'epinions': 1, 'whiz-bang': 1, 'amount': 44, 'implies': 3, 'huh': 15, 'assistance': 9, 'annihilation': 3, 'celluloid': 11, 'evelyn': 3, 'friday': 14, 'darkness': 17, 'favor': 23, "candidate's": 1, "dauphin's": 1, 'tangled': 3, 'shakes': 6, 'exotica': 1, 'spiffy': 3, 'demi-god': 1, 'naive': 8, 'redeems': 1, 'freddie': 18, 'dismay': 4, 'spilt': 1, 'semen': 2, 'benecio': 1, 'resident': 4, 'congratuations': 1, "ho's": 1, 'alarmed': 2, 'uecker': 1, 'yawns': 3, 'come_': 1, 'paulina': 1, 'koteas': 1, 'turkey': 29, 'felix': 2, 'successors': 2, 'bulb': 3, 'energy': 45, 'headlong': 1, 'disappeared': 8, "sand's": 1, 'shackled': 2, 'olivia': 3, 'success': 72, 'lovitz': 5, 'damages': 1, 'waddington': 1, 'talke': 1, 'futility': 2, 'way-too-glossy': 1, 'gonzales': 1, 'built-in': 1, 'x-ray': 3, 'easier': 17, 'extraterrestrials': 1, 'vitti': 1, 'inaudacious': 1, 'cronenberg-gore': 1, 'emotionally': 17, 'yaps': 1, 'lost': 122, 'winona': 3, "deschanel's": 1, 'remainder': 5, 'santa': 4, 'knock-out': 1, 'not-so-glorious': 1, 'slams': 2, 'bailed': 1, 'c-note': 1, '`down': 1, 'puts': 49, 'tinam': 1, 'bratty': 1, 'north': 12, 'razor': 1, 'roams': 1, 'obstructing': 1, 'cemetary': 2, "woman'": 1, 'conscience': 14, 'mabe': 2, "'star'": 1, '1800s': 2, 'opposite': 32, 'schmoe': 1, 'typicalness': 1, 'thrillride': 2, 'pledged': 1, 'sprawling': 4, 'star-studded': 2, 'firmly': 5, 'flatlining': 1, 'insecurities': 6, 'furry': 6, 'kong': 25, 'spike': 3, '_no': 1, 'jeremiah': 3, "blues'": 2, "burden's": 1, "meerson's": 1, 'fonz': 1, 'sulfuric': 1, "'humour'": 1, 'minion': 1, 'aliens': 39, 'nefarious': 2, 'wandered': 4, 'earth': 66, 'mundane': 5, 'accomplishments': 3, 'hellish': 1, 'sneer': 2, 'hopes': 27, 'mugshot': 1, 'exited': 3, 'pig': 5, 'bakker': 1, 'credo': 1, 'focuses': 12, '_dead_man_on_campus_': 1, 'compensated': 1, 'tap': 2, 'unappealing': 11, 'costner': 5, 'negotiating': 1, 'nekhorvich': 1, 'bestselling': 1, "summer's": 9, 'lights': 11, 'ermey': 5, 'remastered': 2, 'gielgud': 2, 'berkoff': 1, 'devil': 20, 'mumbles': 2, 'element': 49, 'believability': 11, 'ex-porn-': 1, 'guillaume': 1, "charles'": 1, 'quickening': 1, 'child': 63, 'pulse': 8, "squad's": 1, 'credulity': 1, "aaliyah's": 1, 'ever-wise': 1, 'ellis': 3, 'numbers': 19, 'sketched': 1, 'shrieking': 3, 'arliss': 3, 'austrian': 2, 'toughs': 2, 'unjust': 2, 'meal': 8, 'yorker': 4, 'escapades': 3, 'inserting': 2, 'harold': 10, "moresco's": 1, 'sprinkler': 1, 'auspicious': 1, "ed's": 2, 'yowza': 1, 'weapon': 27, 'hearty': 2, 'franchises': 1, 'frakes': 1, 'icon': 6, "century's": 1, 'interludes': 4, 'beresford': 2, 'ex-olympic': 1, 'cooper': 9, 'corpses': 4, 'woke': 2, 'alternative': 9, 'gook': 1, 'revisited': 1, 'silvio': 2, 'depending': 6, 'shadowed': 1, 'draws': 7, "d'artagnon": 1, 'maid': 9, 'agonizing': 1, 'sadomasochism': 2, 'suggested': 9, 'intersperesed': 1, '22-5': 1, 'macaw': 1, 'cough': 2, 'cowrites': 1, 'returning': 10, 'harmless': 9, 'demille': 1, 'vibrancy': 2, 'penciller': 1, 'endangers': 2, 'peculiarly': 1, 'vessels': 1, 'mindful': 1, 'leash': 1, "brewster's": 1, 'crouching': 3, 'hehehe': 1, 'succumbing': 1, 'kombat': 10, 'quarterback': 5, 'touch': 36, 'pushed': 9, 'multiplexes': 2, 'cannibalism': 2, 'techniques': 8, 'presidency': 1, 'ohio': 1, 'fistful': 2, 'buton': 1, "'meaningful'": 1, 'viewpoints': 3, 'serving': 12, '[1996]': 1, 'bananas': 5, 'smalltown': 1, 'mismatched-buddy': 1, 'realises': 6, 'counterpart': 3, 'self-indulgence': 1, 'detests': 1, 'complements': 1, 'upset': 9, 'coote': 1, 'doctor': 43, "'predator'": 1, 'electrical': 6, 'july': 4, 'studio-financed': 1, "mtv's": 2, 'feurerstein': 1, 'therapists': 1, 'herculean': 1, 'mba': 1, '131': 1, 'concepts': 10, 'sic': 1, 'bemused': 1, 'drown': 5, 'expanded': 2, 'stilted': 6, 'susses': 1, 'begging': 4, 'tomlin': 2, 'melange': 3, 'casket': 1, 'loom': 1, 'self-defense': 2, '[read': 1, 'similarities': 5, 'soaring': 3, 'yearnings': 1, 'woken': 1, 'condone': 3, 'enduring': 5, 'lower': 14, 'nicky': 5, 'confessing': 2, 'yuh': 1, 'fillmore': 1, 'unamusing': 3, '_breakfast_': 1, 'humbly': 2, 'fries': 3, 'members': 58, 'penis': 12, "o'donaghue": 1, "'act'": 1, 'deluded': 1, 'spare': 14, "ritter's": 1, 'trade': 10, 'task': 41, 'adoptive': 2, 'whew': 3, 'inscrutability': 1, 'elects': 1, 'composer': 3, 'wafer': 1, 'october': 6, 'affinity': 1, 'blissfully': 5, 'pit': 3, 'cover': 35, 'buttocks': 3, 'semi-excusable': 1, 'inventions': 4, 'obeys': 1, 'tortoise': 1, 'it-': 1, 'pratically': 2, 'lightweight': 10, 'sleepwalk': 5, "animal's": 2, 'absolut': 1, 'coke-head': 1, 'farce': 10, 'fahrenheit': 1, 'sea-': 1, "brolin's": 1, 'discuss': 8, 'midgets': 3, 'bathing': 4, 'huggers': 1, 'hotels': 1, 'drain': 2, "everybody's": 2, 'agony': 1, '7=good': 1, 'dime': 2, 'geoff': 4, 'scenic': 2, 'fame': 22, 'heavy-on-fx': 1, '//filmfreakcentral': 1, 'holocaust': 5, "angel's": 1, 'staccato': 2, 'unexceptional': 2, 'idiosyncrasies': 1, 'sandra': 14, "demme's": 1, 'grain': 1, 'automobile': 6, 'solutions': 2, 'anthropologists': 1, 'lancing': 1, 'hinder': 1, 'geiger': 1, 'snatches': 3, 'bye': 2, 'hillbillies': 2, 'utah': 1, 'affecting': 2, 'profanity': 20, "hayes's": 1, 'xander': 2, 'well-paced': 1, 'fergus': 1, 'disgraced': 1, 'schnazzy': 1, 'equation': 6, 'forty': 6, 'launched': 3, 'newspapers': 2, 'sharif': 2, 'compulsion': 2, '2024': 1, 'evenly': 1, 'significant': 24, 'tip': 3, 'caught-with-his-pants-down': 1, 'ducts': 1, 'mercy': 5, 'pastures': 1, 'personality-challenged': 1, 'everett': 10, 'foch': 1, 'cowboys': 3, 'hurdle': 2, 'ernst': 1, 'rips': 7, 'hawaii': 3, 'all--and': 1, 'dungeons': 1, "anorexic's": 1, 'feeling': 74, 'ago': 77, 'nell': 3, 'failings': 2, 'halperin': 4, "foley's": 2, 'reconnect': 1, 'crass': 3, 'snowfall': 3, 'sculptress': 2, 'miscalculated': 2, 'lester': 1, 'empty': 33, 'lives--for': 1, 'joe/chris': 1, 'partment': 1, 'ass-cracks': 1, 'preacher': 3, 'respond': 6, 'decapitated': 3, 'thing--if': 1, 'concerns': 16, "'what's": 1, 'heartbreak': 5, 'oriental': 1, 'style': 107, 'wham': 2, 'sloppily': 3, 'mtv-influenced': 1, 'author=hicks': 1, 'skin': 20, 'investigations': 3, 'luncheon': 1, 'chalkboard': 1, "heyst's": 1, 'button': 8, 'disagree': 3, 'stylist': 2, 'cries': 10, 'cruttwell': 1, 'cuarsn': 1, 'coasts': 4, 'dilorenzo': 1, "chicago's": 1, 'ending': 107, "denzel's": 1, 'frey': 1, 'accuses': 3, 'ivory': 3, 'berserk': 3, 'petrie': 1, 'enthusiastically': 3, 'buxom': 4, 'willingness': 4, "perf's": 1, 'titles': 12, 'schmidt': 2, 'far/strength': 1, 'rip-off': 20, 'arizonians': 1, 'malfunction': 1, 'existenz': 2, 'orthodox': 1, 'dashed': 3, 'wrought': 5, 'protests': 3, 'crocodile': 3, 'overacting': 14, 'lobotomized': 1, 'blackwolf': 1, 'earl': 2, 'reprieve': 1, "byrne's": 1, 'intrusion': 2, 'morphing': 4, 'squirming': 2, 'herve': 1, 'breeds': 2, 'barrels': 3, 'framing': 3, 'alda': 1, 'behemoth': 2, 'emanating': 1, 'holzbog': 1, 'pencil': 1, 'hemispheres': 1, 'haul': 3, '1986': 3, 'platinum': 1, 'midgett': 1, "spillane's": 1, 'amazes': 1, 'bounty': 3, 'alienation': 6, 'deserve': 19, 'after-school': 2, 'wry': 4, "arkin's": 1, 'thought': 156, '29': 1, 'insubordinate': 1, 'marquee': 2, 'face': 116, 'rigors': 1, 'beverly': 13, 'unintentionally': 18, 'clemente': 1, 'choreography': 6, 'stepford': 1, "james'": 2, 'sheridan': 2, 'appeared': 23, 'insurance': 2, 'transsexual': 2, '--': 173, 'combo': 2, 'unknowingly': 6, '1989': 7, 'jive': 1, 'stickiest': 1, 'grouch': 1, 'chills': 6, 'sappiness': 1, 'befriends': 10, 'dorothy': 1, 'morricone': 2, 'hollywoods': 1, 'horatio': 1, "marcus's": 1, "train's": 2, 'well-publicised': 1, 'glorifies': 1, 'whitlow': 1, 'jedi': 4, 'disdain': 3, 'predictably': 13, 'collegue': 1, "becker's": 1, 'funari': 1, 'holley': 1, 'swanky': 1, 'wildest': 2, 'roddy': 3, 'precedence': 1, 'overuse': 4, 'definitive': 2, 'defenceless': 1, 'undoubtly': 1, 'chancellor': 1, 'daughter-in-law': 2, 'stoop': 3, 'cameras': 17, 'bokeem': 3, 'centers': 12, 'olivier': 1, 'fresh': 34, 'go-ahead': 1, '13': 10, "morricone's": 1, 'owing': 1, 'cutouts': 1, "'daddy": 1, 'withdrawing': 1, 'old-as-time': 1, 'mansion': 11, "wallace's": 1, 'pretending': 8, 'steal': 31, "alessa's": 1, 'versus': 2, 'devoted': 13, 'grinding': 3, 'beeline': 1, 'canadian': 7, 'institutions': 1, 'culmination': 1, 'crap': 46, "spears'": 1, 'complement': 2, 'foresight': 1, 'suffering': 17, 'panned': 5, 'northmen': 1, 'blotteth': 1, 'transplant': 5, 'yipes': 3, 'windbag': 1, 'twiddling': 1, 'over-70': 1, 'meanest': 1, 'sees': 38, 'buffs': 2, 'poisons': 1, 'cracks': 8, 'arabia': 1, 'highly-placed': 1, 'grounded': 4, 'void': 6, 'taciturn': 2, 'one-legged': 1, 'robinson': 6, 'minor': 44, 'names': 46, 'popped': 5, 'nurses': 1, 'roughed': 1, 'trigger': 4, 'insert': 9, "henstridge's": 2, 'elongated': 1, "g's": 1, 'shop': 16, '`double': 1, "harrelson's": 1, 'turtles': 1, 'dim-witted': 6, 'nypd': 4, 'distract': 6, "`friends'": 1, 'watts': 1, 'programs': 2, 'presented': 27, 'sterile': 4, 'fowl': 1, "'handling'": 1, 'treat': 20, 'goods': 8, 'mornay': 1, 'routine': 33, 'testimony': 2, 'out-of-water': 1, 'teammate': 2, 'hammett': 1, 'hanged': 2, 'mightiest': 1, 'leisurely': 4, 'cartoons-': 1, 'sluggishly': 1, 'biotech': 1, 'widow': 2, 'offending': 1, 'pleased': 5, 'imply': 2, 'toad': 1, 'anti-nukes': 1, 'ramble': 3, 'multiplied': 1, "54's": 2, 'weitz': 1, 'deneuve': 4, 'overstated': 1, 'ingrown': 1, 'attenuated': 3, 'reviewing': 4, 'curiosity': 8, "ephron's": 1, 'vivre': 1, "kombat_'s": 1, 'completely-is': 1, 'self-awareness': 1, 'coffin': 7, 'lackluster': 18, 'kattan': 5, 'multi-national': 1, 'one-and-a-half': 2, 'close-to-death': 1, 'joining': 9, 'xiong': 2, 'kurtwood': 1, 'stridently': 1, "up's": 2, 'objective': 3, 'idiots': 13, 'variations': 8, 'unappetizing': 2, 'choppy': 10, 'character--a': 1, 'eleanor': 1, 'chin': 2, 'conduct': 2, 'communication': 5, 'treville': 1, 'defensive': 1, 'double-crossed': 2, "bach's": 1, 'gravy': 2, 'double-crossing': 3, 'unwise': 1, 'liberties': 1, 'gregg': 6, 'performing': 14, 'martha': 11, 'celestial': 1, 'supple': 1, 'head-start': 1, 'score': 60, 'turturro': 2, 'hardworking': 2, 'enthusiasm': 8, 'dresses': 9, 'cylk': 1, "sewell's": 1, 'bearse': 1, 'krubavitch': 1, 'torturous': 5, 'phillippe': 6, 'unusual': 10, '96': 3, 'surrounding': 17, 'substances': 1, 'synonyms': 1, 'studi': 6, 'leonard': 7, 'veins': 3, 'zenith': 1, 'exhumed': 1, 'yelling': 14, 'impossible--witness': 1, 'divines': 1, 'sudan': 1, 'lame': 70, 'dress-up': 1, "franchise's": 1, 'operates': 1, 'betrayal': 6, 'depicts': 3, 'liaison': 4, 'cuts': 27, '13-year-old': 1, 'un-technical': 1, 'bloodless': 2, 'marc': 10, 'mightily': 2, 'achilles': 1, 'unwind': 1, "swartzenegger's": 1, "bowie's": 1, 'struggle': 19, 'lugia': 1, 'paychecks': 4, "script's": 7, 'full-on': 2, 'moreso': 1, 'sodium': 1, "railroad's": 2, 'rumors': 4, 'helper-': 1, 'manipulated': 7, 'declares': 5, 'mack': 4, 'cancel': 3, 'assembly': 2, 'stored': 2, 'confession': 6, 'handcuffs': 2, 'migrant': 1, 'antisocial': 1, 'grime': 2, "inventor's": 1, 'necks': 3, 'wresting': 1, 'desk': 4, 'extravagance': 1, 'immerse': 1, 'tosses': 3, 'pennyworth': 1, 'frankie': 4, 'onw': 1, 'wears': 28, 'unleashes': 1, 'perrineau': 1, 'differently': 5, "ferrera's": 1, 'bedhopping': 1, 'soar': 1, 'assante': 2, 'elizabethan': 1, 'assert': 2, 'machines': 7, 'defeated': 4, 'overabundant': 1, 'tandem': 1, 'mowed': 1, 'verify': 1, 'mahurin': 1, 'cheerleader': 6, "sargent's": 1, 'ultra': 1, 'rapid-fire': 3, 'pianist': 1, 'peace-loving': 1, 'dionna': 1, 'voices': 14, 'masquerading': 2, 'grainy': 9, 'triangles': 1, '1985': 13, 'bathrobe': 1, 'passionate': 5, 'victorious': 1, 'chester': 3, 'pointlessly': 2, 'jiang': 1, 'headquarters': 4, "`fatty'": 1, 'item': 6, 'faster-than-gravity': 1, 'sermon': 2, 'patent': 1, '24-year-old': 1, 'duplicitous': 2, "nerds'": 1, 'whimper': 1, 'exam': 1, 'instinctively': 1, 'groomed': 1, 'all-around': 2, 'wheelchair-bound': 2, 'imagine': 53, 'jax': 3, 'predicated': 1, "hell/i'll": 1, 'comedy-drama': 2, 'warns': 1, 'texture': 3, 'storyboarded': 1, 'cents': 2, 'mainland': 2, 'small-to-big-screen': 1, 'designer': 6, 'duplicity': 2, 'big-screen': 8, 'unscrupulous': 2, 'rooms-into': 1, "spacey's": 1, 'newly-gestated': 1, "lara's": 2, 'luscious': 2, 'sleep': 27, 'crashed': 5, 'finer': 1, 'resurrect': 1, 'los': 17, '2/10': 2, 'bellow': 2, 'remarkably': 12, 'atom': 3, 'superman': 7, 'anguished': 2, 'tamala': 2, 'momentary': 2, 'rasputin': 1, 'eight-and-a-half': 1, 'raspy-voiced': 1, 'greta': 1, 'yawning': 2, 'ripley': 2, 'prequels': 1, 'bischoff': 2, 'lingers': 4, 'carrol': 1, 'sweeney': 4, 'crooks': 5, 'surveillance': 8, "boss'": 1, 'reforming': 1, 'won\x12t': 1, 'scientologist': 2, 'pencil-thin': 1, 'alienbusting': 1, 'demonstrations': 2, 'tatum': 2, 'sniff': 3, 'oneness': 1, "mignatti's": 1, 'projected': 5, 'dentures': 1, 'lack-of-speed': 1, 'lend': 7, 'tilt-o-whirl': 1, 'quickly': 91, "arc's": 1, 'lyons': 1, 'sucked': 16, 'hensons': 1, 'laurie': 2, 'youngsters': 3, 'nicely': 24, '`mystery': 1, 'hippo': 1, 'bankroll': 1, '666': 4, 'culminating': 6, "capshaw's": 1, 'sprockets': 1, 'freshest': 1, 'chevy': 9, 'lumet': 3, 'bladed': 1, 'safe': 25, 'biggie': 1, 'patriotic': 3, 'marlon': 4, 'nth': 1, 'ming-na': 1, 'hackford': 2, 'talked': 6, 'mcdormand': 1, 'emote': 1, 'sprinter': 1, 'ahmet': 1, 'rewatched': 1, 'underestimates': 1, 'splices': 1, 'seats': 7, 'defeats': 3, 'afterthought': 9, 'there--two': 1, 'cigar': 5, 'screwy': 1, 'wrench': 1, 'paced': 12, "hell's": 2, 'ex-round': 1, "c'est": 1, '97': 2, 'woodsboro': 1, 'overcoat': 2, 'priests': 8, 'wittily': 1, 'whazoo': 1, 'jemima': 1, 'ennio': 3, 'pleasure': 25, 'boyfriends': 2, 'redundantly': 1, 'orth': 3, 'gaze': 3, 'drippy': 1, "'unusual": 1, 'submarine': 2, 'smacked': 1, 'high-tech': 6, 'broyles': 1, 'wrap-up': 1, 'kaisa': 1, 'axe': 4, 'lehman': 3, 'strenuously': 1, 'tempted': 7, 'hammerhead': 2, 'ultra-hip': 1, 'prison-matron': 1, '8': 13, 'n-word': 1, 'wahlberg': 10, 'drum': 4, 'outcome': 6, 'capable': 31, 'uneasy': 7, 'frosty': 1, 'tune': 16, 'gabrielle': 2, 'earful': 1, "bobby's": 1, 'drawbridges': 1, 'suction': 1, 'pan': 7, 'abrahams': 3, 'run': 117, "santa's": 1, 'stranding': 1, 'theater-goers': 2, 'dirtied': 1, 'mocks': 1, 'prescient': 1, 'merian': 1, 'rough-yet-debonair': 1, 'perceptions': 1, 'digest': 4, 'upholding': 2, 'hours--or': 1, 'trish': 2, 'schwarzenegger': 20, 'nothingness': 2, "fun'": 1, 'fobbed': 1, 'magnetic': 4, 'ritter': 3, 'generations': 7, 'full-fledged': 4, 'berenger': 4, 'gorillas/live': 1, 'camps': 2, 'salvage': 9, 'fortunately': 13, 'contrasts': 1, 'dinners': 1, 'brolin': 4, 'elegance': 1, 'his/her': 3, "boys'": 5, 'dignam': 1, 'chiseled': 3, 'forces': 30, 'influx': 1, 'shyer': 1, 'pixie-faced': 1, 'allergic': 1, 'mucked': 1, 'builder': 1, 'bone': 13, 'winningham': 1, 'championship': 8, 'assistsant': 1, 'tattoos': 4, 'staying': 16, 'violently': 5, 'reviewed': 10, "frasier's": 1, 'x-': 3, 'parole': 9, "scorces'": 1, "piano's": 1, 'fares': 14, 'cleaned-up': 1, "minister's": 1, 'deadeningly': 1, 'graft': 2, 'not-too-distant': 1, 'foothold': 2, 'comely': 1, 'monomanical': 1, 'headache-inducing': 4, 'bereft': 3, 'flatly': 2, 'relieves': 1, 'samba': 1, 'den': 3, 'desaster': 1, 'symbols': 1, 'sleaze': 5, 'courageously': 1, 'compatible': 1, 'adherence': 4, 'thoughtlessly': 1, 'genteel': 1, "bloomingdale's": 1, 'cousins': 2, 'unbelievable': 32, 'semi-developed': 1, "i's": 1, 'ducks': 2, 'wobbly': 1, 'squeak': 1, 'remotely': 31, 'organized': 5, 'roger': 35, 'unfortunate': 24, 'juxtaposing': 1, 'adorableness': 1, 'improve': 6, 'off-the-wall': 4, 'natives': 6, 'terrier': 1, 'dependant': 1, 'proclaimed': 4, 'encouraging': 5, 'santanico': 1, 'fishes': 1, 'god': 70, 'carries': 12, 'awarded': 3, 'barta': 1, 'membership': 2, 'chisolm': 1, 'segment': 14, 'end': 336, 'carving': 3, 'crane': 2, 'jerome': 1, 'engineered': 4, 'tabbed': 1, 'comparitive': 1, 'posturing': 5, 'gutted': 2, 'gilliard': 1, 'sale': 2, 'attachment': 2, 'beamed': 1, 'tidied': 1, 'jab': 3, 'repent': 1, 'unispiring': 1, 'provocation': 1, 'whoopee': 1, 'vulnerable': 4, 'unisol': 1, 'blotter': 1, 'jeb': 2, 'elijah': 2, 'obtain': 1, '5-cent': 1, 'utilized': 3, 'moby': 1, "erin's": 1, 'belushi': 4, 'promoter': 3, 'unnamed': 3, 'first-tier': 1, 'selling': 21, 'attacks': 13, 'homcoming': 1, "calitri's": 1, 'drug-kingpins': 1, 'soothing': 2, 'vanna': 1, 'grew': 10, 'athlete-actors': 1, 'snails': 6, 'coordinates': 1, 'connecting': 2, 'curled': 1, 'overlap': 1, "raymond's": 1, 'super-encrypted': 1, 'ex-husband': 3, 'haunted': 19, 'banged': 3, "whispery-voice'd": 1, 'disturbed': 9, 'entendres': 4, 'puppy': 11, 'pilion': 1, 'sanitarium': 1, 'lost--literally--on': 1, 'raleigh': 1, 'reconstructive': 1, 'agitate': 1, 'plently': 1, 'leadership': 3, 'whacked-out': 2, 'responding': 1, 'grade': 20, 'phnah': 1, '11': 8, 'hee-haw': 1, 'mutilation': 3, 'shoulda-been-nominated': 1, 'shayne': 2, 'delicacy': 2, 'gizzards': 1, 'pyrotechnic': 2, 'bakkers': 1, 'jet': 14, 'goat': 3, "all's": 3, 'mccormack': 3, 'falters': 2, 'debilitating': 1, 'personable': 1, 'sword': 10, 'edmunds': 1, 'labeled': 1, 'club-singer': 1, '50%': 1, 'domineering': 1, 'scene-stealing': 2, 'killer': 67, 'chauvinistic': 2, 'fan': 61, 'bulging': 3, 'pirhanna': 1, 'pinchot': 2, 'movie/tv': 1, 'locally': 2, 'top': 107, 'meaning': 30, 'pawn': 1, 'tafkap': 1, 'angst-horror': 1, 'leaf': 3, 'love-hewitt': 2, 'adviser': 2, 'penmen': 1, 'sleazeball': 1, 'pint-sized': 1, 'ventriloquist': 1, 'unashamed': 1, "shakespeare's": 5, 'pristine': 3, 'aissa': 1, 'reelection': 2, "comet's": 1, 'polley': 1, 'confronts': 8, 'automaton': 2, 'newswoman': 1, 'steam': 10, 'snack': 3, 'bars': 6, 'underplays': 1, 'roswell': 1, 'unidentified': 1, 'louison': 1, 'fluff': 10, 'hat-trick': 1, 'mutilated': 3, 'irony': 15, "catch's": 1, 'chabat': 1, 'metal': 18, 'paymer': 1, 'machinist': 1, 'exception--is': 1, 'haste': 1, 'percentage': 2, 'impersonating': 1, 'christian': 17, 'performs': 9, 'imminent': 3, 'cheaper': 5, 'out-takes': 2, "flanders'": 1, 'uzi-firing': 1, 'fryer': 1, 'easy-to-': 1, 'rapes': 1, 'trail': 9, 'congratulates': 1, 'disappoints': 3, "lombardo's": 1, 'barnfield': 1, 'reccomend': 1, 'kung': 6, 'extortion': 1, 'baroque': 1, 'ding': 1, 'dictated': 1, 'alternating': 2, 'renowned': 6, "hooker's": 1, 'orton': 1, 'flailing': 2, 'post-apocalyptic': 2, 'thirds': 1, 'pefect': 1, 'spacing': 1, 'dominance': 3, 'quizzing': 1, 'lunkheads': 1, 'altogether': 14, 'u-haul': 1, "stupids'": 1, '$130': 1, 'recruited': 4, 'metres': 1, 'donnie': 3, 'wood': 19, 'red/blue': 1, 'actor/waiter': 1, 'plotting': 19, 'serenaded': 1, 'fakes': 3, 'resembles': 17, 'tinker': 1, 'word': 80, 'american-ized': 1, 'yoo': 1, 'slashers': 2, 'decade': 27, 'wyat': 1, 'frankly': 38, 'pauses': 7, 'submarines': 1, 'perishes': 1, "dane's": 1, 'discovered': 23, 'reworks': 1, 'tolerant': 1, '3/4': 1, 'byrne': 7, 'vincenzo': 1, 'unpromising': 1, 'bloom': 1, 'telephone-touters': 1, 'sizable': 2, 'chimes': 2, 'phone': 37, 'bio-lab': 1, 'realistically': 3, 'enlistees': 1, 'ball-peen': 1, 'trusting': 1, 'notion': 20, 'zoom-outs': 1, 'jasmine': 2, 'network': 17, 'sized': 1, 'revs': 1, 'thirteen': 2, 'leaking': 3, 'wynn': 3, 'existed': 1, 'telegram': 2, 'woman--another': 1, 'discrimination': 1, 'marketing': 14, 'marrieds': 1, 'business-owner': 1, 'mechanized': 1, 'tatoulis': 1, 'homogenized': 1, 'actress-turned-comedy': 1, 'aberdeen': 1, 'down-and-out': 3, 'merchandise': 4, 'infects': 2, 'bucks': 14, 'guest-starred': 1, 'straight-laced': 4, 'badham': 1, 'helming': 1, 'highlights': 6, 'babbling': 3, 'unoriginal': 15, 'rare': 24, 'bard': 1, "'dark": 1, 'growing': 28, 'sense--and': 1, 'drug-addled': 1, 'checkers': 1, 'family--father': 1, 'preferably': 4, '24': 2, 'flap': 1, 'sergio': 2, 'infantile': 2, 'swiftly': 1, 'spake': 1, "oz's": 1, 'writer/producer': 1, 'prehistoric': 1, 'marsh': 1, 'depended': 2, 'instructional': 1, 'dachshund': 1, 'inhabit': 4, "theron's": 3, 'con-air': 1, 'battled': 1, "fincher's": 1, 'dammit': 1, 'piglia': 1, 'faithless': 1, 'slog': 2, 'bodhi': 1, 'hauled': 1, 'jewelry-sporting': 1, 'smoke': 11, '`chef': 1, 'counterattack': 1, 'landlord': 2, 'decoys': 1, 'repeating': 2, 'ironside': 3, 'caller': 1, 'hungry': 7, 'marx': 2, 'hums': 3, 'lover': 28, 'prejudices': 1, 'species--a': 1, 'ella': 2, 'party': 50, 'spars': 1, 'prominent': 9, 'digs': 5, 'abysmally': 2, 'smelly': 3, 'displays': 9, 'maladjusted': 1, 'growls': 2, "30's": 2, 'shriek': 1, 'psychopaths': 1, 'primo': 1, 'tightly': 9, 'um': 5, "assignment'": 1, 'raw': 5, 'rimbaud': 1, '`take': 1, 'rioting': 1, 'low-life': 1, 'pent-up': 1, 'puritanical': 1, 'american': 108, 'striving': 7, 'inhabited': 1, 'dispirited': 1, 'celine': 3, "schwimmer's": 1, 'multi-ethnic/multi-national': 1, 'squaring': 2, "kessler's": 1, 'find': 246, 'urine': 2, 'lawford': 1, 'zaftig': 1, 'emphasize': 3, 'brutally': 13, 'plant': 15, 'combining': 6, 'foul-mouthed': 5, 'product-of-choice': 1, 'simcha': 1, 'potent': 13, 'sociological': 3, 'disaster': 45, 'asia': 2, 'lollies': 1, 'pierre': 4, 'baio': 1, 'mick': 1, 'sebastian': 3, 'backtracks': 1, 'explore': 6, 'suitcase': 2, 'garry': 5, 'approval': 1, 'bilingual': 1, 'beaker': 1, 'stakes': 3, 'fischer': 3, 'souza': 4, 'conjure': 3, 'explode': 7, 'heston/gardner/bujold': 1, 'unlikable': 13, 'faces': 31, 'down--way': 1, 'superficiality': 1, 'stoicism': 1, 'check-out': 1, 'tv2': 6, "dragon's": 1, 'excite': 2, 'selick': 1, 'flatter': 2, 'woods': 23, 'repelled': 1, 'lied': 2, 'aboslutely': 1, 'defy': 2, 'interpretations': 3, 'mini-series': 3, 'forbid': 5, 'spartacus': 3, 'magoo-style': 1, 'tad': 3, 'k-red': 1, 'miraculous': 2, 'gases': 2, 'stuntman': 1, 'distances': 3, 'greens': 1, 'longings': 1, "swift's": 1, 'hodgepodge': 8, 'whine': 3, "spain's": 1, 'smears': 2, 'mayo': 4, 'nachos': 1, 'run-down': 3, 'tumbles': 1, 'ustinov': 2, 'clunes': 1, 'subsequent': 7, 'beastiality': 1, 'vitriol': 1, 'bible': 5, 'miscasting': 3, 'orders': 12, 'woburn': 1, 'out-performs': 1, 'well-paid': 2, 'degrading': 2, 'raquel': 2, '_21_jump_street_': 1, 'recharge': 1, 'mcclurg': 2, 'visuals': 20, 'effaced': 2, 'lizard-stomps-manhattan': 1, 'this-obviously-cost-a-lot-so-you-know-': 1, 'spiked': 2, 'sprightly': 1, 'stylish': 15, 'play-by-': 1, 'overweight': 5, 'parasail': 1, 'wounding': 1, 'hitchhiked': 1, 'pictures': 40, 'maddy': 3, 'showcasing': 2, "--it's": 1, 'rupert': 8, 'incompleteness': 1, 'tumbling': 1, 'descend': 2, 'schoolteacher': 3, 'vaccine': 1, 'attentions': 3, 'non-animated': 1, 'suffuse': 1, 'poking': 12, 'resuscitated': 1, 'luthor': 1, 'arm': 15, 'marketed': 7, 'capitalise': 2, 'throttle': 2, 'involve': 12, 'unworkable': 2, '1000': 4, 'cocks': 1, 'scam': 5, 'onwards': 3, 'glave': 2, 'flex': 3, 'comraderie': 1, 'underneath': 6, 'reacquaints': 1, 'charbanic': 1, '`review': 1, 'ground': 48, 'videotaping': 1, 'cinephiles': 1, 'booted': 2, 'harried': 1, 'duquette': 1, 'ecclesiastes': 1, 'wander': 4, 'anonymous': 6, 'been-there': 1, 'scampering': 1, 'inuits': 1, 'ta': 1, 'getty': 1, 'singer_': 1, '#2': 2, 'carrying': 16, 'tanker': 3, 'sunny': 1, 'priesthood': 1, 'spine': 3, 'tgif': 2, 'dances': 5, 'desperately': 31, 'innacurate': 1, "goldsman's": 2, 'purchased': 1, 'skipped': 2, 'wooden': 21, 'probgram': 1, 'climbs': 1, 'suspected': 5, 'cartoons': 4, 'reminder': 1, 'unrecognizable': 2, 'legal': 4, 'gunton': 6, 'coburn': 3, 'bull-headed': 1, 'scooping': 1, 'adapters': 1, 'understated': 6, 'home-shopping': 1, 'torches': 2, 'hopeful': 8, 'brandy': 6, 'mario': 9, "basinger's": 1, 'sequence': 113, 'clashing': 2, "remington's": 1, 'flipper': 2, "d'amato": 1, 'seague': 1, 'springtime': 1, 'alienates': 1, 'swish-swish-zzzzzzz': 1, 'penile': 1, 'bites': 7, 'ridiculousness': 4, 'dairies': 1, 're-unification': 1, 'mousy': 3, 'receive': 11, 'tobacco': 2, 'hand': 124, 'typecasted': 1, 'billion': 4, 'flynt': 1, 'later-in-the-year': 1, 'overtake': 1, 'baywatch': 4, 'brain': 41, 'cheeseball': 1, 'strasser': 1, "die's": 1, 'positions': 2, 'truckload': 2, "marie's": 1, 'slugging': 1, 'breather': 1, 'cheapens': 1, 'nervously': 1, 'planting': 1, 'burke': 4, 'allergy-prone': 1, 'fluent': 2, 'session': 6, 'womanfriend': 1, 'regrettably-neglected': 1, 'talent-impaired': 1, 'shadowloo': 1, 'coed': 2, 'robicheaux': 2, 'radiated': 1, "`bats'": 1, 'treachery': 1, 'osborne': 3, 'dreyfuss': 3, 'corruptor': 1, "counselor's": 1, "hurley's": 1, "perkins'": 1, 'scare': 21, 'nauseum': 2, 'stepfather': 1, 'depresses': 1, 'contract': 5, 'voters': 1, 'blasts': 2, 'escher': 1, 'confirms': 2, 'extracting': 3, 'deviants': 1, 'corman-grade': 1, 'patronize': 1, 'deteriorate': 1, 'babes': 6, 'reforms': 1, 'defanged': 1, 'high-': 1, "producer's": 1, 'pride': 7, 'originality': 27, 'preschoolers': 2, 'belligerent': 1, 'strong-worded': 1, "gus'": 2, 'jigsaw': 2, 'darwin': 1, 'boldly': 1, 'rune-like': 1, 'bancroft': 2, 'database': 3, 'cliffhanger': 6, 'nanny': 1, 'arrows': 1, 'boozed-out': 1, 'souls': 11, 'wincott': 3, 'compromise': 1, 'thou': 3, 'drawbridge': 1, "griswold's": 1, 'interviewing': 1, 'stagnated': 1, 'appearances': 15, 'gina': 8, 'dealings': 2, 'consummated': 1, 'flyboys': 1, 'piercing': 2, 'admittedly': 19, "birdy's": 1, 'quintet--': 1, 'overheard': 1, "cod's": 1, 'non-classics': 1, "cardinal's": 2, 'vicent': 1, 'red-headed': 1, 'masterminds': 1, 'insolent': 1, 'hannibal': 3, 'valletta': 1, 'card-flipping': 1, 'bistro': 1, 'sides': 12, 'spit': 1, 'rope': 7, 'computer-aided': 1, 'expecting': 33, 'mischievous': 2, 'film-goers': 1, 'superintendent': 1, 'laura': 10, 'ideologies': 1, 'imperioli': 1, 'frontier': 2, "milano's": 1, 'askew': 3, 'academy-award': 1, 'errands': 1, 'pardon': 4, 'self-indulgent': 2, 'anti-gun': 1, 'remarks': 5, 'hunkish': 1, 'sharp-eyed': 1, 'whining': 10, 'intact': 1, 'rockies': 2, 'shares': 13, 'decadence': 2, "ben's": 1, 'replicants': 1, '--and': 2, 'sympathetically': 1, 'newmar': 1, 'elite': 7, 'masochistic': 4, 'buckman': 1, 'grossie': 1, 'tucci': 2, 'smallest': 1, 'makes': 295, "millius'": 1, "mornay's": 1, 'marching': 1, 'kink': 2, 'deborah': 5, 'sleight-of-hand': 2, 'girth': 1, 'grosser': 1, 'plods': 7, 'gandolfini': 3, 'batters': 2, 'hole--symbol': 1, 'sunhill': 3, 'contemptuously': 1, 'kernels': 1, 'gripe': 2, 'con-': 1, "snake's": 2, 'fiendish': 3, 'wigand': 1, 'faired': 1, 'radius': 2, 'overextension': 1, 'mating': 8, 'randomly': 5, 'ghost': 13, 'poems': 1, 'howser': 2, "rabin's": 1, 'android': 2, 'curly': 3, 'no-nonsense': 4, "mary'": 1, 'pothead': 2, 'ton': 9, 'emerging': 4, 'capability': 2, 'arid': 1, 'rudy': 2, 'jordan': 6, 'one-way': 1, 'ex-alchoholic': 1, 'snail': 1, 'abigail': 1, 'artistic': 14, 'silhouette': 1, 'memo': 1, "brother's": 15, 'inquisition': 1, 'juicy': 3, 'subtle': 25, 'viewpoint': 3, 'simulated': 3, 'patches': 1, 'unfamiliar': 5, 'hefnerism': 1, 'chaplin': 1, 'momentous': 5, 'quintano': 4, 'discarded': 2, 'polynice': 1, "granny's": 1, 'incumbent': 1, 'hard-to-swallow': 1, 'sledgehammers': 1, 'nazi': 5, 'waving': 4, 'pooh': 1, "haunting's": 1, 'disinterested': 3, 'unabomber': 1, "niro's": 1, 'hogget': 1, 'salvation': 5, 'fay': 1, 'gun': 53, "'00s": 1, 'accepts': 16, 'loren': 1, "congo's": 1, 'pratfalls': 4, "1991's": 2, 'portray': 7, '1953': 1, 'gwar': 2, 'subverts': 1, 'variation': 6, 'shakycam': 1, 'md': 2, 'quandary': 2, 'antipathetic': 1, 'wrongly': 2, 'absolute': 13, "'war": 1, 'ambush': 3, 'craig': 7, 'humiliates': 1, 'survive': 35, 'lopez': 8, 'regrettable': 1, 'divert': 1, 'occasionaly': 1, 'hanky-panky': 2, 'carhart': 1, 'sequence/flashback': 1, 'hal': 7, 'drivel': 10, 'awkwardness': 1, 'intervention': 2, 'englishman': 1, 'perfect': 66, 'engineers': 2, 'fairy-tale-in-the-': 1, "`georgia'": 1, "spring's": 1, 'challenge': 17, "twit'": 1, 'leno': 1, 'stowaway': 1, 'still-burning': 1, 'retrieve': 12, 'sitcom-style': 1, 'michel': 1, 'attuned': 2, 'wheaton': 1, 'lambert': 6, "o'callaghan": 1, 'inadvertent': 3, 'tatooed': 1, 'melanie': 8, 'travers': 1, "mulder's": 1, 'sonnenberg': 1, 'vaguest': 1, 'restores': 1, 'iffy': 3, 'crew-cut': 1, 'projection': 1, 'outlaws': 3, 'mouthpieces': 2, 'ferreted': 1, 'expendable': 2, 'pedagogical': 1, 'woodwinds': 1, 'pillow': 2, 'entire--and': 1, "boat's": 2, 'twirling': 1, 'leathers': 2, 'followup': 2, 'discerning': 3, 'walkouts': 1, 'suarez': 1, 'keynote': 1, 'goldeneye': 3, '_vampires_': 1, 'warehouse': 4, 'profane': 2, 'motorboat': 1, 'stunts/special': 1, 'differences': 12, 'generously': 1, 'football': 17, 'moore-esque': 1, 'trillions': 1, 'tragic': 18, 'initiated': 1, 'timex': 1, 'incubus': 1, 'joystick': 2, 'dogg': 1, 'inflate': 1, 'loses': 32, 'continental': 1, 'populate': 2, 'today': 29, 'sherlock': 2, 'smug': 3, 'quips': 3, 'welcomed': 1, 'ignorantly': 1, 'mumbled': 3, 'crocs': 1, 'genetically': 7, 'josie': 3, 'miramax/constellation/ugc/hatchette': 1, 'simplifies': 1, 'tnt': 1, 'copping': 2, 'owes': 3, 'drifts': 2, 'cranium': 2, 'different--black': 1, 'arrgh': 1, 'friendly': 17, 'unexpressed': 1, 'equivalent': 11, "'feel": 1, 'commissioned': 3, 'movie\x05': 1, 'uncaring': 6, 'schmuck': 2, 'alevey': 1, 'conglomeration': 1, 'screwball': 4, 'overeacting': 1, 'coincidences': 10, 'foolishness': 3, 'satisfy': 13, 'takes': 204, 'pitchers': 2, 'mercies': 1, 'scant': 4, 'brag': 4, 'half-interesting': 2, 'abandon': 9, 'andersons': 1, 'dates': 8, 'lin': 3, 'twists': 30, 'june': 4, 'top-rated': 1, 'bending': 1, 'well-received': 1, 'translation': 8, '95-minute': 1, 'guarantees': 3, 'stepped': 6, 'resoundingly': 1, 'seeps': 1, 'incidental': 2, 'eden': 2, 'misty': 1, 'leverage-using': 1, 'artistically': 3, 'babaloo': 1, 'earns': 6, 'cathrine': 2, 'mitsubishi': 1, 'grasshoppers': 1, 'he/she/it': 1, 'on-board': 1, 'nemesis': 10, 'davidson': 7, 'ingredient': 4, 'digit': 1, 'deck': 7, 'doctored': 1, 'billy': 36, "badalamenti's": 1, 'freewheeling': 1, 'compounding': 1, 'served': 14, 'user': 1, 'dianne': 1, 'handcuffed': 3, 'deleted': 4, "secretary's": 1, 'savini': 1, 'beefy': 1, 'humor': 134, 'gasp': 2, '-give': 2, 'wes': 18, 'modernizing': 2, 'vying': 2, 'relief': 39, 'trinity': 2, 'attach': 2, 'fridge': 2, 'frequently': 30, 'obscures': 1, 'sourabaya': 1, 'combatants': 1, 'easy-going': 1, 'jaws': 10, 'comet': 3, 'blossoming': 1, 'regressive': 1, "dickinson's": 1, 'harm': 6, 'convict': 6, 'muslim': 1, 'fortress': 3, 'raffo': 1, 'smacks': 3, "altman's": 2, 'tire': 3, "months'": 1, 'herman': 4, 'anchors': 1, 'life-forms': 1, 'unrelated': 6, 'augmented': 1, 'autopilot': 4, 'sinclair': 3, 'nineteenth': 1, 'durable': 1, 'kidney': 2, 'chalderns': 1, 'seconds': 39, 'transparent': 6, 'possessors': 1, 'condemn': 2, 'blouse': 1, 'estrogen': 1, 'still-heavily-accented': 1, 'underrated': 10, 'wheelchair': 2, 'confounding': 2, 'horks': 1, 'uncredited--can': 1, 'hunka': 1, '`basic': 1, 'letdown': 7, 'upheaval': 1, 'tourist': 7, "o'toole": 1, 'jetpacks': 1, 'sporting': 8, 'privately': 1, 'janey': 1, 'paradigm': 1, 'most-powerful': 1, 'xeroxed': 1, 'seventeenth': 1, 'unconvincingly': 2, 'absurdity': 8, 'se7en': 5, 'overpriced': 1, 'incompetent': 16, 'harrelson': 11, 'wakeup': 1, 'jima': 1, 'infurating': 1, 'parking': 6, 'undetected': 1, 'gaffes': 1, 'guardia': 1, 'confronting': 3, 'fumbling': 3, "wives'": 1, 'gentleman-at-large': 1, "hayek's": 1, 'tropic': 1, 'bumps': 4, 'crib': 2, 'reon': 1, 'rescued': 11, 'barrie': 1, 'expectation': 4, "'em": 7, 'cash-milking': 1, 'bore': 33, 'delved': 1, '======': 1, 'cocaine': 10, 'poses': 2, 'filtered': 2, '_lot_': 1, 'all-round': 1, 'big-undersea-serpent': 1, "batman's": 2, 'jackman': 1, 'action/sci-fi': 1, 'disagreed': 1, 'friends': 102, 'aloofness': 1, 'dippe': 1, 'kamen': 1, "crazy'": 1, 'perform': 19, 'de-glamed': 1, 'punk': 5, "bond's": 2, 'han': 4, 'screw': 9, 'undeniable': 4, 'chalk-full': 1, 'imbues': 2, 'grafitti': 1, 'mail': 9, "reynold's": 1, 'killers': 18, "damsel's": 1, 'shakily': 1, 'citizens': 8, 'movie/road': 1, 'samples': 2, 'affability': 1, 'asians': 1, 'adaption': 2, 'sandler-annoying': 1, 'denouement': 6, 'judgmental': 1, 'bombarded': 1, 'kickoff': 1, 'is--gasp': 1, 'dostoevski': 1, 'packet': 1, 'mckean': 1, 'glares': 1, 'slates': 1, 'jeep': 6, 'unlovable': 1, 'brighten': 2, 'discovery': 10, "directorate's": 1, 'pirates': 3, 'dispatched': 4, 'typing': 1, 'ninety-seven': 1, 'dysfunction': 1, 'abating': 1, 'happy-go-lucky': 4, 'occasionally': 45, 'liquid': 3, 'co-stars': 9, 'violation': 1, "cup's": 1, 'contemptable': 1, 'hallucinates': 1, 'outshining': 1, 'proposal': 4, 'pfeiffer': 1, 'lovers': 20, 'post-twin': 1, 'journalist': 7, 'baffels': 1, 'lineup': 3, "industry's": 2, 'leland': 3, 'unhealthy': 6, "dafoe's": 2, 'thief-the': 1, 'worse--stick': 1, 'sinead': 1, 'foibles': 2, 'deft': 1, 'francine': 2, 'inimitable': 2, 'shoving': 4, 'tail': 2, 'sinise': 11, 'full-length': 6, 'shaped': 3, 'vols': 1, 'venue': 2, 'christine': 5, 'add': 82, 'watchability': 1, 'glamorous': 7, 'mystically': 1, 'awful': 94, 'poignant': 5, 'big-breasted': 1, 'childhood': 14, 'flashy': 18, 'severely': 8, 'price': 25, 'oppression': 1, "regan's": 1, 'pivot': 1, 'petter': 1, 'flint': 1, 'piece': 83, 'triumphs': 3, 'peripheral': 4, 'tack': 1, 'effortless': 2, 'investigator': 11, 'rock-concert': 1, 'insensitive': 1, 'long-abandoned': 1, 'resurrected': 3, 'insinuates': 1, 'illustration': 2, 'dog': 66, 'fits': 9, 'ranted': 1, 'accomplishes': 2, 'bawls': 1, "parillaud's": 1, 'immersed': 1, 'confront': 8, "'film": 1, 'divisions': 1, 'traces': 6, 'understands': 3, 'hideaway': 4, "odyssey'": 1, 'part': 230, 'sexual': 71, 'helium': 2, 'jonah': 1, 'prophets': 1, 'romanceless': 1, 'afloat': 3, 'chasms': 1, 'liam': 7, 'andre': 2, 'barkeep': 1, 'thick-headed': 1, 'pelts': 1, 'bel-air': 1, 'frank': 25, 'rearrange': 1, 'crud': 1, 'tunnels': 3, 'reassuring': 3, 'lowest-common-denominator': 1, 'hag': 1, "li's": 4, 'undying': 2, 'consanguinity': 1, 'demotion': 1, 'downfall': 2, 'mid-70s': 1, 'weaponry': 6, 'cutlery-flinging': 1, "film's": 173, 'minute': 61, 'poorly-integrated': 1, 'bresslaw': 1, 'byways': 1, 'mctiernan': 5, 'radition': 1, 'delve': 2, 'roper': 2, 'dump': 8, 'fluid': 3, 'shouting': 13, 'sneers': 1, 'fetching': 2, 'thunder': 3, 'cadre': 1, 'blitzstein': 1, "spawn's": 4, 'devon': 1, 'posed': 4, 'tribunal': 1, 'soon-to-be': 2, 'ballisitic': 1, 'womanhood': 1, 'charizard': 1, 'observe': 6, 'soaks': 1, 'betty': 10, 'crusoe': 2, 'stealing': 16, 'testosterone-overdosed': 1, 'havilland': 1, 'kelley': 4, 'stiles': 3, 'squishing': 2, 'partial': 2, 'cokehead': 1, 'pulse-emitting': 1, 'archetypes': 2, 'mated': 1, 'sp': 1, 'treasure': 3, 'street': 53, 'credibility': 15, 'styrofoam': 1, 'one-tenth': 1, 'crisply': 1, 'oven': 2, 'ray': 20, 'breakdancing': 1, 'swat': 3, 'boyle': 3, 'manipulating': 2, 'manners': 1, 'frighten': 3, 'mocked': 4, 'pseudo-saintliness': 1, 'sleeping': 14, 'quotient': 2, 'selfish': 9, '_last': 1, 'backtrack': 1, 'tambor': 2, 'sketches': 3, 'horrified': 6, 'dynamite': 4, 'newsgroups': 2, 'opt': 2, 'joblo': 10, 'ceos': 1, 'nausuem': 1, 'hard-to-believe': 1, 'vacation': 16, 'denominator': 2, 'favourites': 1, 'exude': 1, 'banishes': 1, 'rescue': 42, 'holiday': 16, 'life_': 1, 'food-fight': 1, 'heartwarming-nuzzly-follow-your-dreams': 1, 'sliders': 1, "facility's": 1, 'zimmer': 1, 'transitory': 1, 'joint': 6, 'tribes': 2, 'support': 20, 'made-for-video': 2, 'smile': 34, 'bolstered': 1, 'inappropriate': 9, 'streamed': 1, 'active': 4, 'seasick': 1, 'flattering': 1, 'mankind': 6, "day's": 5, 'usaf': 1, "constanza's": 1, 'ex-circus': 1, 'bedpost': 1, 'harrassing': 2, 'kato': 1, 'sarone': 2, 'creationism': 1, 'film--': 1, 'delores': 1, 'danson': 3, 'deposited': 1, 'fretting': 1, 'structure': 16, 'treaty': 2, 'miserably': 22, 'characterized': 3, 'casual': 5, 'teenager': 9, 'bested': 1, 'diverting': 7, 'miraculously': 12, "explorers'": 1, 'star-torturer': 1, 'rohypnol': 1, 'salary': 1, 'sex-for-pay': 1, 'bracco': 1, 'decks': 1, 'manicured': 1, 'engrossment': 1, 'touts': 1, 'quotation': 1, 'methodology': 1, 'rip-roaring': 2, 'resemble': 12, 'cut-out': 1, 'boiling': 6, 'smirnov': 1, 'drunkard': 1, 'discloses': 1, 'convicts': 2, 'coworker': 1, 'shallow': 37, 'summarized': 4, 'squirmy': 1, 'over-wrought': 2, 'foot': 19, 'skeletons': 3, 'waxing': 2, 'gabfest': 1, 'muggers': 2, 'brink': 6, 'this-could-be-you': 1, 'asimov': 1, 'distorted': 2, 'crystal': 11, 'abdomen': 2, 'tibet': 2, '--tough': 1, '66': 2, 'family-oriented': 1, 'dalmations': 1, 'bont': 7, 'cheung': 1, "valentine's": 3, 'ilm': 2, 'scriptwriting': 1, 'butcher': 4, 'handedly': 1, 'unfairly': 2, 'clumsily': 5, 'penetrated': 2, 'homicidal': 8, 'rock-musician-turned-pavement-artist': 1, 'corner': 24, 'astounding': 1, 'placement': 5, 'olek': 1, 'unanimously': 1, 'thirteenth': 4, 'wesson': 2, 'faced': 9, 'obstetrician': 2, 'adapting': 2, 'recipient': 2, 'witch': 27, '`ready': 1, 'slurpy': 1, 'limb': 2, 'two-by-four': 1, 'heir': 1, 'provoking': 7, 'mcqueen': 2, 'placements': 3, 'rich-': 1, "cat's": 1, 'coerce': 2, 'manuals': 1, 'fleabag': 1, 'pullman': 4, 'bonehead': 1, 'bazookas': 1, 'wd-40': 2, 'kidnapped--she': 1, 'potency': 1, 'guest': 12, 'waspy': 2, 'still-alive': 1, 'atcheson': 1, 'yearning': 2, "city's": 2, 'amok': 2, 'rawhide/stand': 1, 'accusations': 2, 'aviators': 1, 'everyday': 4, 'wedding': 34, 'disarm': 1, 'turnaround': 2, 'pea': 1, 'igniting': 2, 'leaked': 1, 'half-heartedly': 1, 'weave': 4, 'recent': 81, "festival's": 1, 'respectability': 1, 'tee-shirt': 1, "jasper's": 1, 'urges': 3, 'formality': 2, 'morale': 2, '1987': 6, 'chockfull': 2, 'bookseller': 1, 'toothpick': 1, 'leading': 47, 'neutron': 1, 'zane': 5, 'scrumptious': 1, 'director/actor': 1, 'shoot-first-then-shoot-some-more': 1, 'low-powered': 1, '$12': 1, 'grit': 2, 'sheedy': 2, 'misspelled': 2, 'shifting': 5, 'heartening': 1, 'lesson': 10, 'voodoo': 3, 'judi': 2, 'audio': 5, 'show-stopping': 2, 'head': 132, 'sympathise': 1, 'dull': 84, 'abraham': 2, 'art': 51, 'family': 114, 'sidewalk': 3, "graynamore's": 1, 'ferland': 1, 'marginal': 8, 'hart': 5, 'doubly-cast': 1, 'ugliness': 2, 'drive-ins': 1, 'closing': 17, 'argument': 4, 'lamb': 2, 'manipulative': 14, 'radiates': 1, 'job-title': 1, 'kull': 1, "payback's": 1, 'ma': 4, 'worships': 1, 'jerry': 33, 'confined': 3, 'pizazz': 1, 'menstruation': 1, 'flower': 1, 'inconsistently': 1, 'spokeswoman': 1, 'forgot': 24, 'sweethearts': 2, 'alphabet': 1, 'mekhi': 2, 'billionth': 2, 'bags': 2, 'camera-work': 1, 'miata': 1, 'fake': 41, 'karnstein': 1, 'panache': 5, 'broke': 13, 'taint': 1, 'obvious': 134, 'disease': 14, 'agatha': 1, 'allusion': 1, 'mentally': 12, 'once--and': 1, "theater's": 2, 'retched': 1, 'third-string': 1, 'contend': 3, 'show-yet': 1, 'womb': 1, 'eclipsed': 1, 'plumbing': 2, 'portrayed': 14, 'epectations': 1, 'coleman': 3, 'big-underground-worm': 1, 'packs': 4, "client's": 2, 'outpaced': 1, 'tampon': 1, 'moment--': 1, 'howling': 3, 'squirtle': 1, 'laudable': 1, 'uppedity': 1, 'assistants': 1, 'elaborating': 1, 'kulich': 1, 'distractingly': 2, 'lust': 6, 'perps': 1, "'carry": 1, 'tickets': 8, 'rotary': 1, 'churns': 2, 'prochnow': 2, 'connie': 8, '20s': 1, "hopper's": 1, 'rambunctious': 1, 'premature': 1, 'misbegotten': 2, 'mekum': 1, 'absorb': 4, 'ramirez': 1, "1993's": 6, 'correlate': 1, 'pitted': 1, 'fact--but': 1, 'perkins': 5, 'lazard': 3, 'mandingo': 1, 'aplenty': 2, 'formal': 4, 'iii-woefully': 1, 'snowdad': 1, 'well-conceived': 1, 'delivery': 16, 'design': 24, 'disagreement': 1, "hodges'": 1, 'wilmington-as-detroit': 1, 'consistent': 11, 'amateurish': 11, 'adapt': 1, 'revisits': 1, '11-year': 1, "so-generic-and-predictable-it's-beyond-ridiculous": 1, 'preaches': 2, 'bachelor': 3, '`as': 1, 'climatic': 2, 'storybook': 1, 'ciphers': 2, 'mein': 1, 'whipping': 2, 'hyperdrive': 1, 'hinted': 4, '007': 3, 'seven-year-old': 1, 'kitty-faced': 1, 'foes': 1, 'waning': 1, 'thousand': 18, 'stinkers': 4, 'flat-out': 2, 'ebonics': 1, 'rabies': 1, "crock's": 1, 'shagadellic': 1, 'reeled': 1, 'lukas': 1, 'yup': 5, 're-enactment': 1, 'vindication': 1, 'strip-mining': 1, '-jack': 1, 'lifetime': 6, 'rapid': 7, 'taks': 1, 'retirement': 6, 'co-writer': 4, 'cringing-': 1, "killer's": 9, 'jao': 1, 'kip': 2, 'physics': 3, 'furniture': 10, "stern's": 1, 'english': 42, "deer-in-the-headlights'": 1, 'hip': 18, 'stole': 6, 'dark': 78, 'prologue': 10, "'poking": 1, 'irritates': 2, 'sucking-out': 1, 'repackaging': 1, 'quirky': 20, 'stepping': 4, 'stunt-butts': 1, 'entails': 2, 'liberation': 2, 'costar': 1, 'noodle-headed': 2, 'neverland': 1, 'heroine': 17, 'spanish': 5, 'easily': 65, 'unwatchable': 13, 'sandefur': 1, 'knot': 2, 'immortality': 1, 'relationships': 25, 'impassive': 1, 'gardner': 1, 'quartet': 4, 'art-house': 3, 'ensnared': 1, 'monday': 5, 'lesbian': 10, 'changing': 8, 'theft': 5, 'dignified': 2, 'hostile': 4, 'vexatious': 1, 'radicals': 1, 'ridicule': 3, 'co-starring': 2, 'avail': 2, 'kinder': 1, 'carroll': 2, 'bastardizing': 1, 'bruskotter': 1, 'ounce': 10, "robertson's": 1, 'circle': 6, 'weakness': 2, 'registered': 1, 'sub-mediocrity': 2, 'madame': 3, 'choosing': 10, 'adjusting': 1, 'one-dimensionally': 1, 'antics': 21, 'off-shoot': 1, 'notions': 1, 'borderlines': 1, 'animal-men': 1, 'high-gadget': 1, 'ebert': 15, 'translator': 1, 'gazing': 1, 'temporarily': 2, 'extremes': 1, 'mariner': 1, 'adding': 12, 'bark': 1, 'panaro': 1, '_scream_2_': 1, 'nutshell': 2, 'weighs': 5, 'countries': 5, 'forbidden': 4, 'notes': 18, 'flippant': 1, 'brunt': 3, '--------------------------------------------------------------': 1, 'analyze': 7, 'sighting': 2, 'instructs': 1, 'high-rise': 1, 'divided': 3, 'sheriff': 18, 'livelihoods': 1, 'pox': 1, 'jets': 2, 'humiliating': 3, 'gunshots': 1, 'hazard': 2, 'pisspoor': 1, 'westworld': 2, 'brightening': 1, 'instantly': 15, 'subjects': 13, 'beers': 1, 'vail': 1, 'losers': 10, 'november': 4, 'complicated': 17, 'collosal': 1, 'cringe': 11, 'skittery': 1, 'jacobi': 1, 'distance': 12, 'mechanics': 3, 'bossman': 1, 'simmer': 2, "presence's": 1, 'deconstruction': 1, 'contrived': 33, 'compound': 4, 'royally': 1, 'eccentricities': 1, 'spew': 2, 'cryogenically': 4, 'smiling': 4, 'produces': 5, "howard's": 1, 'categories': 3, 'liking': 12, 'computer-generated': 16, 'grandson': 3, 'razzie': 5, 'insubordination': 1, 'happy-for-all-parties': 1, 'path': 16, 'arnold': 20, 'botched': 5, 'aretha': 2, 'sexed': 1, "70's": 7, 'soul-searching': 1, 'mini-sub': 1, "to'": 1, 'well-written': 4, 'uber-rich': 1, 'hankie': 1, 'swarm': 5, "matchmaker's": 1, 'nntphub': 1, 'highly-anticipated': 1, 'traditionally': 2, "carey's": 1, 'mark-paul': 1, 'silk': 4, 'consumer': 1, 'fee': 3, 'nails': 6, 'nostalgia': 13, '76': 2, 'domestically': 1, 'attracting': 3, 'tirade': 2, 'convulses': 1, 'repelling': 1, 'ogden': 1, 'dousing': 2, 'represented': 7, 'pounding': 1, 'longstreet': 1, 'outlined': 3, 'gopher': 1, 'sorvino': 10, 'flashbacks': 20, 'reputation': 12, 'saturn': 1, 'commences': 2, 'blunted': 1, 't-shirts': 3, 'sourpuss': 1, 'mini-plot': 1, 'conservationist': 1, 'finely-tuned': 1, 'layla': 1, 'storm': 23, 'guerilla': 1, 'neck': 11, 'rahman': 1, 'chemistry': 34, 'dissembles': 1, 'leaks': 1, 'clockwatchers': 1, 'gleason': 1, 'recycles': 7, 'bonnier': 1, "breadwinner's": 1, 'neglecting': 1, 'lois': 2, 'policewomen': 1, 'pathetic-looking': 1, 'diametric': 1, 'knucklehead': 1, 'joyce': 1, 'absences': 3, 'lukanic': 1, 'andrzej': 3, 'arrange': 1, 'thrice': 1, 'humorously': 1, 'solmenly': 1, 'lackadasically': 1, 'outline': 7, 'fatalities': 1, 'ronny': 1, 'purveyor': 1, 'sprayed': 1, 'blow-ups': 1, 'traceable': 1, 'back-to-back': 1, "dumas's": 2, 'misadventures': 6, 'barroom': 2, 'immersive': 1, 'breathe': 2, 'bay': 9, 'grittier': 2, 'schoolwork': 1, 'loony': 1, 'armi': 1, 'targeting': 3, 'va-va-va-voom': 1, 'goldsman': 3, 'asked': 45, 'ekland': 1, 'leers': 2, 'parts': 76, 'slobbering': 1, 'meet': 73, 'arm-mounted': 1, 'tracing': 1, 'produce': 19, "1983's": 1, 'father-daughter': 1, 'betrayed': 3, 'infuriated': 2, 'uninspiring': 2, 'milk': 9, 'rapier': 1, 'undisclosed': 1, 'colorless': 1, 'disfigured': 2, 'babysitter': 5, 'ably': 2, 'hordes': 2, 'paulie': 3, 'eruption/fire/earthquake/explosion/tsunami/tornado/meteorite': 1, 'annabel': 1, 'holland': 4, 'looping': 1, "chris'": 1, 'abundant': 4, 'separation': 1, 'bloated': 6, 'reminders': 1, 'streetwise': 2, 'infomercial': 1, 'consistency': 5, '3465': 1, 'danny--here': 1, "treill's": 1, "dome'": 1, 'angst-ridden': 1, 'offing': 1, 'labor-inducing': 1, 'dierdre': 2, 'industry': 25, "interrupted'": 1, 'composers': 2, 'meteor': 3, 'vallick': 1, 'beyer': 1, 'abel': 3, 'paw': 1, 'flirtatious': 1, 'posturings': 1, 'daydreams': 1, "'acting'": 1, 'sweepstakes': 2, 'hone': 1, 'sons': 6, "helena's": 1, 'midway': 7, 'miniscule': 2, 'marital': 8, 'grossly': 1, 'warm-n-fuzzy': 1, 'engage': 13, 'collections': 1, 'receptionists': 1, 'sommers': 3, "mcgregor's": 1, 'hayek': 8, 'stream-of-consciousness': 1, 'bostonians': 1, 'contrast': 12, 'chart': 2, "debney's": 1, 'towns': 1, 'c-': 1, 'serena': 2, 'succumb': 2, 'graphic': 29, 'conditions': 4, 'vocal': 3, 'shortens': 1, 'knockers': 1, 'convince': 20, 'cusack': 7, 'bodes': 1, 'irishman': 2, 'experience': 74, 'spy/romance': 1, "poseidon's": 1, 'spiked-haired': 1, "johnny's": 2, 'intentionally': 5, 'sheffer': 1, 'entitled': 5, 'hardest-hitting': 1, 'linc': 3, 'penny': 9, 'hair': 40, 'eponymous': 2, 'knocking': 8, 'doorstep': 1, 'bs': 2, 'magical': 18, 'kirshner': 1, '5000': 1, 'vibrate': 1, 'emma': 11, 'koch': 1, 'war-torn': 1, 'ulcers': 1, 'regulations': 4, 'society': 32, 'concoctions': 1, 'soundman': 1, 'clown-for-hire': 1, 'parador': 1, 'attire': 2, 'sewage': 1, 'reiterated': 1, 'internet': 11, 'prettiest': 1, "dante's": 8, 'benny': 5, 'relies': 18, "norris'": 1, 'kalvert': 1, 'charade': 1, 'denis': 5, 'polo': 1, 'bouncy': 1, 'boomers': 1, 'strictly': 20, 'matchmaking': 1, 'obligingly': 1, 'minimalist': 1, 'espoused': 1, 'not-so-happily': 1, 'over-earnestness': 1, 'larger': 14, 'terrified': 5, '30ish': 1, "owens'": 1, 'non-traditional': 1, '[kjv]': 2, 'panoramas': 1, 'dailies': 2, "young's": 1, 'hitchhike': 1, 'goddamn': 4, 'sour': 10, 'ennui': 2, 'hotel': 25, 'cakes': 2, 'cons': 5, 'femme': 12, 'eilbacher': 1, 'cretin': 1, 'mcelhone': 2, 'erupting': 1, 'positives': 1, 'patrol': 3, 'hampered': 5, 'kang-sheng': 2, 'throws': 25, 'mercifully': 4, 'wisecracking': 10, "bizet's": 1, "mellis's": 1, 'dour': 2, 'nerd': 5, 'near-constant': 1, 'prehistory': 1, 'prize': 11, 'liao': 1, 'stocks': 1, 'assumption': 5, 'losses': 4, '`plot': 1, 'reigning': 1, '2000': 12, 'innocence': 8, 'spencer': 1, "seagal's": 4, 'jane': 15, 'necesary': 1, 'corrupt': 13, "schlondorff's": 1, 'jb': 1, 'breakaway': 1, 'kasalivich': 1, 'vain': 6, 'carrefour': 1, 'enhance': 3, 'unenergetically': 1, 'seed': 2, 'spontaneity': 4, 'deceased': 9, 'prime': 14, 'hex': 1, "whistler's": 1, 'telekinesis': 2, 'scrooged': 1, 'svenwara': 1, 'engaging': 28, 'renee': 8, 'stalk': 3, "bimbo's": 1, 'jakob': 2, "michele's": 1, 'leftovers': 2, '_the_last_days_of_disco_': 1, "walker's": 2, 'touchstone': 3, 'four-armed': 1, 'classified': 4, 'assured': 9, 'decent': 88, 'carey': 3, 'widmark': 2, 'dalmatian': 2, 'fend': 4, 'beating': 17, 'robes': 2, 'cast--all': 1, '16x9': 1, "ironside's": 1, 'hiss': 3, 'leather-wearing': 1, "videotape's": 1, 'stake': 5, 'remembering': 2, 'century-fox': 1, "hogget's": 1, 'pastel-colored': 1, 'remakes': 4, 'occured': 1, 'purports': 2, 'manslaughterer': 1, 'solace': 3, 'interplay': 3, 'tutelage': 1, 'valentine': 2, 'auel': 1, 'aspirins': 1, 'armful': 1, 'dilbert': 1, 'coke': 2, 'selleck': 2, 'fuller': 2, 'futile': 5, "reese's": 1, 'host': 16, 'myra': 3, 'crank': 1, "hoover's": 1, 'embarked': 2, 'bawdy': 1, 'parable': 2, 'establishments': 1, 'lament': 1, 'ira': 3, 'graveyard': 3, 'creators': 4, 'incredulity': 1, 'berry': 2, 'however--these': 1, 'sacrificing': 1, 'disney-made': 1, 'now-ritual': 1, 'rewarded': 3, 'flourish': 3, "endo's": 1, 'assasination': 1, 'advise': 2, 'impressively': 3, 'businesslike': 1, 'luciano': 1, 'penthouse': 3, 'complications': 5, 'hypocritical': 3, 'movies--like': 1, 'hauff': 1, "timer'": 1, 'optimism': 1, 'singer/alcoholic': 1, "jasmine's": 1, 'factors': 4, 'animatronic': 3, 'foundering': 1, 'arguing': 5, 'lusts': 1, "aiello's": 1, 'mawkish': 2, "wcw's": 2, 'chats': 1, 'revelation/twist': 1, 'passed': 22, 'assistant': 27, 'decision': 30, 'conti': 1, 'moronic-looking': 1, 'oral-sex/prostitution': 1, 'class-a': 1, 'municipality': 1, 'aficionado': 2, 'gargoyles--perhaps': 1, 'putty': 1, 'tax': 5, 'proximity': 2, 'global': 2, 'unsigned': 1, 'misfired': 3, 'de-force': 1, "y'all": 2, 'profound': 9, "morse's": 1, 'liane': 1, 'nigel': 2, 'isaacs': 1, 'push': 12, 'february': 6, 'godzilla': 18, 'examined': 4, 'shaving': 1, 'hoobler': 1, "cortino's": 1, 'man': 314, 'releasing': 6, 'binges': 2, 'durden': 1, 'roritor': 1, 'well-made': 5, "lords'": 1, 'labute': 2, 'aspects': 22, 'creek': 5, 'barbarians': 1, 'marky': 2, 'unsatisfying': 8, 'newborn': 2, 'rumble': 8, 'brick': 2, "viewers'": 2, 'functionary': 1, 'studly': 1, 'coaxing': 3, 'cam': 3, 'mars/venus': 1, 'presumed': 3, 'issue--there': 1, 'intimidated': 1, 'toller': 3, 'inseminates': 1, 'reins': 2, 'westerner-in-peril': 1, 'discernable': 2, 'faints': 2, 'cybill': 1, 'invulnerability': 1, 'priorities': 1, 'americanization': 1, 'humour': 21, 'advice': 26, 'hipness': 1, 'vengeful': 2, 'giamatti': 5, 'wands': 1, 'keen': 2, 'fahdlan': 4, 'mismash': 1, 'rediscovered': 1, 'impartial': 1, 'debutante': 1, 'eternal': 6, 'right--frighteningly': 1, 'unfreeze': 1, 'dupes': 1, 'much-publicized': 1, 'multimillion': 1, 'mao': 1, 'thorougly': 1, 'chagrin': 1, 'workaholics': 1, 'pleasurable': 2, 'caruso': 2, 'distinguished': 3, 'isn\x12t': 1, 'ladybug': 1, 'overturned': 2, 'one-liner': 4, 'hinges': 2, 'centennial': 1, 'subconscience': 2, '25-year': 1, 'honey': 3, 'wager': 3, 'disintegrate': 1, 'gates': 5, 'firefighting': 1, 'testicles': 2, 'brooms': 1, 'kick-off': 1, 'relative': 10, "astronaut's": 1, 'coerced': 1, 'blast': 6, 'hounds': 1, 'inhabitants': 4, 'unlucky': 1, "he'll": 23, 'no-brainer': 1, 'depraved': 4, 'kyra': 1, 'king': 53, 'extremely': 70, 'suburbia': 3, "kudrow's": 3, 'habit': 9, 'burdened': 3, 'grand': 23, 'slayers': 2, 'jinn': 2, 'snow-capped': 1, 'vicious': 12, 'merhi': 1, 'appreciated': 7, 'lawns': 1, 'purpose': 66, '$20': 2, 'shipped': 2, 'sneering': 4, 'cruder': 1, 'mount': 3, 'past': 111, 'friendships': 1, 'understanding': 13, 'birdcage': 2, 'godard': 1, '`something': 1, 'traits': 4, 'twenty-five': 2, 'xenia': 1, 'hamming': 4, 'ontario': 1, 'duplicate': 6, 'a-trois': 1, 'harbors': 3, 'expositions': 1, 'appalls': 1, 'uninitiated': 2, 'anabelle': 1, 'misdeed': 1, 'possesion': 1, 'scares': 13, 'warmer': 1, 'outlining': 1, 'out-of-print': 1, 'swaggering': 1, 'double-crosses': 2, 'spelled': 1, 'toghether': 1, 'soons': 1, 'shel': 1, 'hala': 1, 'exacting': 1, 'identity': 32, 'manager': 14, 'developments': 15, 'new-age': 1, 'newer-model': 1, 'rolling': 24, 'amadeus': 1, 'glitter': 1, 'prostitutes': 4, 'seals': 1, 'forged': 2, 'premier': 1, 'blend': 5, "cholodenko's": 1, 'scene-stealer': 1, 'truthful': 4, 'reagan-loving': 1, 'examine': 3, 'propensity': 3, 'drug-crazed': 1, 'compacts': 1, 'clearer': 1, 'accurately': 5, 'tongue-and-cheek': 1, 'vega': 1, 'obi': 3, 'unsatisfied': 3, 'champ': 3, 'camped': 2, 'cat-and-mouse': 1, 'participated': 1, 'karl': 3, 'omission': 1, 'tournament': 4, 'skeletor': 1, '\x14': 2, 'enchanted': 1, 'noted': 7, '50-year': 1, 'illusionists': 1, 'reader': 4, 'gleam': 1, 'staked': 1, 'aki': 1, 'jewelry': 1, 'platoons': 1, 'run-in': 2, 'retired': 8, 'furious': 4, 'plaything': 3, "sarah's": 2, 'summons': 2, 'antionio': 1, 'rusnak': 1, 'masturbation': 4, 'left': 161, 'sneaky': 2, 'cop-turned-private': 1, 'herz': 1, 'thematic': 1, 'elmore': 1, 'bot': 1, 'lumbering': 2, 'actionless': 1, 'limitations': 2, 'flanked': 1, 'munchausen': 1, 'spell': 7, 'scripts': 13, 'myers': 12, 'has-beens': 1, 'gunshot': 3, 'facilitated': 1, 'mcclements': 1, 'foolish': 8, 'squarely': 4, 'transforms': 4, '6/10': 4, 'large-scale': 1, 'equally': 35, '20-year': 1, 'nannies': 1, "aliens'": 1, 'other-worldly': 1, 'ronna': 1, 'verse': 3, 'hyper-capitalism': 1, 'sparse': 2, 'watchers': 1, 'wastes': 7, 'borrows': 7, 'rockwell': 2, "must've": 2, 'name&quot': 1, 'goliath': 2, 'addled': 1, "silverstone's": 3, 'rameses': 1, 'imparting': 1, 'come-on': 1, '#05425': 1, 'telemarketer': 1, "fasano's": 1, "guy'": 2, 'crusher': 1, 'out-of-touch': 1, '87': 4, 'berkley': 5, 'german': 7, 'typewriters': 1, 'artist/action': 1, 'graziano': 1, 'close-cropped': 1, 'client': 9, 'center-fielder': 1, 'president': 21, 'channing': 2, 'way-too-convenient': 1, 'umbrella': 2, 'ferociously': 1, 'non-fans': 2, 'experimental': 4, 'beatings': 2, 'outback': 2, 'sly': 5, 'blunderheaded': 1, 'number': 75, 'prositute': 1, 'source': 18, 'billed': 6, 'reject': 5, 'degrees': 7, 'aborted': 1, 'queued': 1, 'skeeter': 1, 'banks': 8, 'overtly': 5, 'cinematically': 2, 'nagging': 5, "cage's": 5, 'huffs': 1, "tupac's": 1, 'admits': 2, 'questioned': 5, 'emile': 1, 'lighting': 20, 'splenetik': 1, 'junior': 6, 'bared': 2, 'trailer': 52, 'seige': 1, 'considerably': 10, 'hurl': 1, "'bad'": 1, 'likens': 1, 'mishap': 1, 'homosexual': 5, 'customary': 1, 'housekeeper': 1, 'one-dimensional': 23, 'murdered': 25, 'nutter': 2, 'toff': 1, 'transcends': 4, 'sidious': 1, 'rivera': 1, 'william': 70, 'attracted': 16, 'affected': 10, 'psychomagically': 1, 'sexuality': 13, 'fund': 4, 'ballooning': 1, 'acquit': 2, 'tienne': 1, "'tremors": 1, 'cassidy': 1, 'all-encompassing': 1, 'imperialist': 1, 'stupidly': 7, 'megastar': 1, 'flare': 4, 'fleshes': 1, "crew's": 3, 'wearing': 27, 'shearer': 3, 'nightclub': 6, "scaramanga's": 1, 'alway': 1, 'infiltrated': 1, 'props': 6, 'gojira': 3, 'weight': 19, 'self-conscious': 2, "blockbuster'": 1, 'cronies': 1, 'dangle': 1, 'traumatized': 1, 'wreaked': 1, 'cornered': 2, "lee's": 6, 'grownup': 1, 'well-run': 1, 'ex-fan': 1, 'model-wife': 1, 'expressions': 12, 'd-day': 1, 'momumentally': 1, 'xtdl': 1, 'strutting': 2, 'interrelated': 1, 'maintaining': 4, 'haim': 1, 'etienne': 1, 'challenged': 4, 'productivity': 1, 'tale': 57, 'resist': 6, 'blooming': 1, 'mullan': 1, 'sassy': 2, 'quasi-incestuous': 1, 'circuitry': 2, 'abound': 4, 'hogan': 1, 'howled': 2, 'harmful': 1, 'bacall': 1, 'totopoulos': 1, 'cheaply-made': 1, 'exuberantly': 1, '300': 2, 'jawbreaker': 5, "paltrow's": 1, 'out--': 1, 'select': 7, 'crime': 54, 't&a': 3, 'wraps': 4, "satan's": 5, '_home': 1, 'marsden': 2, 'biochemist': 2, 'bess': 1, 'allure': 3, 'mold': 8, 'pejorative': 2, 'downpour': 2, 'roomm': 1, 'tediously': 4, 'ineffective': 10, 'erasing': 1, "heather's": 1, 'detriment': 3, 'pragmatic': 1, 'weaver': 5, 'pseudo-depth': 1, 'woodenly': 2, 'tickle': 2, 'divorce': 13, 'annie/she+s': 1, 'jean-pierre': 1, '.': 900, 'contrivance': 3, 'winding': 4, 'assume': 23, 'approaching': 7, 'recycled': 17, 'photographed': 8, 'fizzle': 2, 'juncture': 2, 'half-man': 1, "mackintosh's": 1, 'unfaithful': 3, 'priest': 20, 'sigourney': 4, 'saturated': 3, 'nonsensical': 8, "'script'": 1, 'employ': 5, 'fleischer': 1, 'newer': 4, 'koontz': 1, 'exciting': 58, 'substituted': 1, 'knights': 3, 'transferring': 2, 'crews': 1, 'superfluous': 9, "gwen's": 2, 'capture': 25, 'boner': 1, "lillard's": 2, 'encompassed': 1, 'trot': 1, 'ac/dc': 1, 'baker': 13, 'ping-ying': 1, '5=average': 1, 'times': 185, 'vivid': 6, 'cult-ish': 1, "marlowe's": 1, 'dance': 28, 'dullest': 2, 'starships': 1, 'fresher': 1, 'gimmicks': 7, 'kurtzman': 1, 'bell': 10, 'resurrecting': 4, 'heading': 7, 'instruct': 1, 'voluntarily': 1, 'needing': 2, 'meathooks': 1, 'offender': 5, 'lecturing': 1, 'flames': 5, 'mean-spirited': 8, 'believes': 22, 'vah-vah-voom': 1, 'electrocuted': 2, 'new-self': 1, 'visas': 1, 'sant': 1, 'stiff-as-a-board': 1, 'universe': 14, 'candles': 4, 'candleshoe': 1, 'recovering': 2, 'meng-hua': 1, 'forbes': 1, '12-minute': 1, 'king-type': 1, 'hospitable': 2, 'lends': 9, 'unfolding': 6, 'entrepenaur': 1, 'accepting': 4, 'martial-arts-star': 1, 'caligula': 2, "proyas's": 1, 'crudeness': 1, 'cleaver': 2, 'steppes': 1, 'sabotaging': 1, 'hath': 1, 'points': 62, 'wexler': 1, 'versatility': 1, 'recorder': 3, 'booming': 2, 'instigated': 2, 'scowls': 2, 'space': 62, "baigelman's": 1, 'heroic': 9, 'earthworms': 2, 'ps': 1, 'achieving': 2, 'pastiches': 1, 'mountainous': 1, 'medley': 2, 'mosaic': 1, 'surfboards': 1, 'swearing': 5, 'thriving': 1, 'charlie': 22, 'subplot': 56, 'bugs': 10, 'screenwriterese': 1, 'headfirst': 1, 'unproven': 1, 'bum': 2, 'places': 38, 'fierce': 3, 'seafaring': 2, 'horn-rimmed': 1, 'news': 43, 'amid': 3, 'obserable': 1, "verhoeven's": 5, 'female': 70, 'bisexual': 4, 'sarandon': 5, "zach's": 1, 'side-effect': 1, 'trembling': 1, 'ford': 10, 'banned': 1, "cgi's": 1, "mchale's": 3, 'second-half': 1, "korda's": 1, 'corrected': 1, 'mechoso': 1, 'horta': 2, 'robbed': 3, 'jillian': 1, 'randi': 1, 'futures': 1, 'resource-strapped': 1, 'hierarchy': 1, 'show-and-tell': 1, 'hosted': 3, 'highways': 1, 'triumph': 7, 'till': 12, 'cocoon': 1, 'benches': 1, 'semi-serious': 2, 'high-wire': 2, 'fingered': 1, 'self-assured': 1, 'helps': 20, 'breathes': 2, 'dinosaur-like': 1, "knott's": 2, 'cheap-looking': 3, 'con-girl': 1, 'jews': 3, 'incredulously': 1, 'bootstraps': 1, 'reckless': 7, 'lamp': 1, 'peels': 2, 'previews': 13, '54': 4, 'scales': 3, 'tbn': 1, 'big-shots': 1, "'pod'": 1, 'wardrobe': 9, 'reach': 31, 'simmons': 5, 'rhett': 1, 'self-involved': 1, 'unpaid': 1, 'needed': 52, 'arthritis': 1, 'things': 265, 'present': 64, 'bails': 1, 'obstetrics': 1, 'stories': 47, 'understudy': 1, "o'connell": 7, 'chinlund': 1, 'zeal': 2, 'botulism': 1, 'walk': 42, "o'herlihy": 1, 'strap': 1, 'metamorphosizes': 1, 'imprisonment': 2, 'pocket': 3, 'skulls': 1, 'dead-in-the-water': 1, 'interstellar': 2, 'insults': 8, 'relocated': 3, 'down-on-his-luck': 1, 'bubbling': 4, 'invisible': 12, 'tina': 3, 'faithfulness': 1, 'kissy-face': 1, 'excreting': 1, 'globe': 4, 'eh-dan': 1, '113': 2, 'tourists': 2, 'stomachs': 1, 'overloaded': 3, 'developer': 1, 'contracts': 1, 'saddles': 3, 'secure': 3, 'remember': 73, 'shootout': 1, 'quantify': 2, 'distasteful': 4, 'adele': 1, "charlie's": 8, 'assassinated': 2, 'payback': 7, 'chabert': 1, 'lear': 1, 'double': 31, 'eagerness': 1, 'latex': 5, 'sniggering': 1, "sciora's": 1, 'secretive': 2, 'vagrant': 1, 'interact': 7, 'fernwell': 1, 'elton': 3, 'scribe': 7, 'gargantuan': 2, 'hitmen': 4, "buzzcocks'": 1, 'arise': 6, 'strapless': 1, 'indiglo': 1, 'tucker': 3, 'miasma': 1, 'tea': 11, 'lombardo': 3, 'promising': 30, 'levinson/dustin': 1, 'slightest': 19, 'daft': 1, 'stern': 6, "schnitzler's": 1, 'confesses': 5, 'rosebudd': 1, 'berg': 3, 'lesbos': 1, 'lose': 42, 'timecop': 2, 'laws': 11, 'imposes': 1, 'bold': 4, 'harrison': 6, 'rebellion': 2, 'multi-colored': 1, 'dunno': 4, 'swashing': 1, 'character--': 1, 'jillion': 1, 'breaches': 1, 'and--again--you': 1, 'reconsiders': 1, 'marlowe': 2, 'incessantly': 9, 'metropolitan': 1, 'sparks': 9, 'near-impossible': 1, 'e1': 1, 'subvert': 2, 'straightened': 1, 'stiff': 10, 'lamest': 5, 'four--credited': 1, 'stream': 7, 'photo': 8, 'bunton': 3, 'plow': 2, 'bathes': 1, 'increases': 2, "'f": 1, 'road': 52, 'international': 21, 'anchor': 3, 'sexton': 2, 'superviolent': 1, 'blow-off': 1, '15-year': 3, 'whoo': 1, 'predictability': 8, 'miramax': 7, 'sipping': 1, 'speak': 40, 'running-like-the-wind-prey': 1, 'single-handedly': 4, 'lurid': 6, 'thieves': 9, 'bones': 13, 'devout': 4, 'nyah': 2, 'overrun': 2, 'celebs': 1, 'microwaves': 1, 'endemic': 2, 'trillian': 1, "sphere's": 1, 'power-hungry': 2, "instinct'": 2, 'mayersberg': 1, 'flamboyance': 1, "earth'": 1, 'garofalo': 8, 'split-level': 1, 'supposes': 1, 'officers': 7, 'day': 179, 'slogan': 2, 'atkinson': 1, 'gutlessness': 1, "headley's": 1, 'rard': 1, 'rehearsel': 1, 'placid': 3, 'ambassador': 9, 'paradox': 2, 'swim': 2, 'expected': 65, 'dodge': 1, "elliot's": 1, 'much-talked-about': 1, '607': 2, 'alfre': 1, 'defies': 8, 'episodic': 2, 'charges': 7, 'marisol': 2, 'chicken': 9, 'acknowledges': 2, 'meowth': 1, 'irrelevant': 15, 'godfathers': 1, 'jadzia-bolek': 1, 'responsibilities': 4, 'norma': 1, 'dora': 3, 'substantial': 6, 'initech': 1, 'nerve': 5, 'progresses': 15, 'recoup': 1, 'epa': 1, 'skim': 1, 'aims': 3, 'decisively': 1, 'detectives': 6, 'computer-animated': 3, 'spider-man': 1, 'plunge': 2, 'indelible': 1, 'backstory': 3, 'duked': 1, 'visionary': 3, "mca's": 1, 'tattooed': 3, "decade's": 1, 'whispering': 2, 'cross-section': 1, 'gradually': 9, 'hides': 5, 'butt': 18, 'smooch': 2, 'swoons': 1, 'saved': 34, 'husbands': 3, 'accustomed': 5, 'thierry': 3, "filmmakers'": 1, 'flexible': 2, 'multi-billion': 1, 'apparent': 49, 'plausability': 1, 'geller': 3, 'america': 36, 'drearily': 2, 'vci': 1, 'jo-jo': 1, 'unfilmable': 2, 'shining': 13, 'bandages': 1, 'cohesive': 3, "'lingerie": 1, 'half-open': 1, 'guileless': 1, 'proves': 43, 'over-extended': 2, "love'": 1, 'bankruptcy': 1, 'ned': 3, 'wails': 3, 'slim': 3, 'arbitrarily-titled': 1, 'emitted': 1, 'memorably': 1, 'nurse': 9, '_soldier_': 1, 'leary': 7, 'widows': 1, 'play': 150, 'machinations': 6, 'unthrilling': 1, "father's": 14, 'soft-boiled': 1, 'loudmouth': 1, 'muppet': 2, 'dougnac': 1, 'absorbed': 3, 'bumpy': 3, 'fantastical': 3, 'ashton': 3, 'bathroom': 24, 'anne-marie': 1, 'boozed': 1, 'nudge': 1, 'mic': 2, 'undermining': 1, 'defences': 1, 'shocks': 3, 'ferocious': 1, 'baldwin': 22, 'unarmed': 5, 'tuva': 1, 'mythical': 5, 'slogans': 3, 'ayers': 1, 'ballard': 1, 'sperm': 1, 'well-explored': 1, 'redeeming': 25, 'rank': 2, 'downstairs': 1, 'joyless': 4, 'christabella': 1, 'moranis': 2, 'twilight': 10, 'digging': 8, 'distress': 10, 'mexico--what': 1, 'holdup': 2, 'easily-angered': 1, 'paragons': 1, 'zombie': 7, 'investigate': 22, 'helen': 15, 'instruments': 4, 'keg': 1, 'locating': 4, 'gunslingers': 1, 'pretension': 1, 'bruise': 1, 'nodded': 1, 'statistics': 1, 'splendor': 2, 'craft': 11, 'hamper': 2, 'looses': 3, 'mined': 2, "everyone's-going-to-go": 1, 'grown': 17, '1999': 30, 'sisters': 7, 'wherewithal': 2, 'seen--little': 1, 'tow-truck': 1, 'sook-yin': 1, 'grenade': 1, "batologist's": 1, 'hypnotically': 1, 'paraglider': 1, 'tube': 1, 'shanghai': 2, 'rimmer': 1, 'hefner': 1, 'rosary': 1, 'pseudo-father': 1, 'concoct': 1, 'counts': 6, 'male': 35, 'neo-slasher': 1, 'country-wide': 1, '31-year-old': 1, 'pre-release': 3, 'incorporating': 1, 'promotes': 1, 'disregards': 1, 'eighty': 2, 'sand-twister': 1, 'ewan': 4, 'raeeyain': 1, "ashley's": 1, 'thumping': 1, 'distraction': 12, 'stupor': 3, 'goal': 25, "dumber'": 1, 'cigarette-smoking': 1, 'overheated': 2, 'options': 4, 'decapitating': 4, 'colo': 1, 'lefty': 1, 'commanding': 5, 'anew': 1, "'how": 1, 'wuhrer': 2, 'proved': 12, 'glacier': 1, 'johns': 1, 'voila': 1, 'policy': 1, 'spraypaints': 1, 'straight-out': 1, 'whimsical': 3, "maugham's": 1, 'bonds': 2, 'visited': 1, 'shagwell': 1, 'squall': 1, 'bios': 2, 'drips': 2, 'archeologist/grave-robber': 1, "'exciting'": 1, 'stiller': 4, 'actualy': 1, 'fluttering': 1, 'crossed': 5, 'flaw': 23, 'interstitial': 1, 'intoxicating': 1, 'funniest': 26, 'anti-gravity': 1, 'jam': 1, 'nightlife': 2, 'graham': 8, 'unraveled': 1, 'worm-eaten': 1, 'psychodramas': 1, 'sydow': 3, 'exclusivity': 1, 'schreiber': 6, 'moriarty': 1, 'only--reason': 1, 'azaria': 13, 'chong': 2, 'spaced': 2, 'louie': 2, 'smothered': 1, 'turgid': 3, 'celebrities': 6, 'acquires': 1, 'barbecue': 3, 'blandly': 2, 'cul-de-sac': 1, 'demoralised': 1, 'terrorists': 8, 'strives': 5, 'welch': 2, 'spoon_': 1, '000-acre': 1, 'bottles': 3, 'incarcerated': 2, 'turn': 122, 'realistic': 24, 'corporation': 5, 'stylewise': 1, 'plausible': 9, 'girlish': 1, 'triangle': 6, 'poem': 4, 'ioan': 1, 'orginal': 2, 'astonishing': 7, '_jumanji_': 1, 'shirley': 5, 'lowering': 1, '1896': 1, 'fewest': 1, 'brain-child': 1, 'locales': 2, 'nauseatingly': 2, 'level': 99, 'mismatched': 3, 'lanky': 1, 'guile': 1, 'glaring': 13, 'coarsely': 1, 'ulterior': 3, 'famke': 9, 'all-time': 4, 'holler': 1, 'location': 24, 'momentum': 8, 'career': 98, "granger's": 7, 'collateral': 2, "teen's": 1, 'dumber': 12, "larson's": 1, 'throwing': 24, 'cgi': 20, 'ry': 1, 'nightspot': 1, 'predated': 1, 'jason': 38, "rubin's": 1, 'zappa': 1, 'nicole': 6, 'neglected': 2, 'occurs': 24, 'namesake': 1, 'scattered': 5, 'golf': 3, 'extravertedness': 1, 'giant': 55, 'staring': 13, '_home_alone_': 1, 'edith': 1, 'gosselaar': 1, "rustler's": 1, 'speculate': 3, 'permeates': 1, 'jingle': 5, 'damme/wesley': 1, 'yearns': 1, 'ninety-five': 1, 'shatter': 1, 'douse': 1, 'snippets': 3, 'triggered': 2, 'slicker': 5, 'cheesy': 40, 'frequent': 10, 'near-future': 1, 'fields': 9, 'forty-five': 6, 'raise': 15, 'manipulator': 1, 'chamber': 2, 'vegas': 23, 'shock-therapy': 1, 'borrowed': 5, 'attempts': 109, 'unblinking': 1, 'foppish': 1, 'katana': 2, 'duds': 7, 'chest-beating': 2, 'bitten': 5, 'grosse': 4, 'eiffel': 2, 'overwhelmed': 1, 'i-2': 1, 'ms': 21, "neighbor's": 1, 'interior': 5, 'pour': 3, 'bit': 173, "robin'": 1, 'pregnant': 19, '400': 2, "loser's": 2, '`homage': 1, 'preschool': 1, 'ostensibly': 7, 'corporations': 1, 'savory': 1, 'sensuality': 1, 'spoons': 1, 'mouths': 6, 'lizzy': 1, 'strive': 2, 'flute-and-wind': 1, 'politics': 9, 'merest': 1, 'planets': 8, 'reporter': 16, 'onstage': 1, 'baz': 1, 'evolves': 1, 'sob': 1, 'voiceovers': 1, 'ad2am': 1, 'swing': 5, 'varying': 3, 'mixing': 2, 'desparate': 1, 'wigs': 2, 'recap': 1, 'lunatic': 6, 'bureaucratic': 2, 'kel': 2, 'concert': 7, 'farts': 3, 'blood': 56, 'genuine': 32, 'interferred': 1, 'avrech': 1, 'hour-long': 2, 'deeply': 12, 'mild': 13, 'bi-sexual': 3, 'lounging': 1, 'inmate': 4, 'bauer': 3, 'sagemiller': 2, '$1': 3, 'highlighting': 2, 'parked': 2, 'skyjacked': 1, 'amulet': 1, 'angst': 7, 'snipped': 1, 'librarian': 1, 'loaf': 3, 'cousin': 13, 'defeatist': 1, 'inhabits': 2, 'geological': 1, 'believe--in': 1, 'bimbos': 2, 'mark': 55, 'sultans': 1, 'koster': 1, 'coldly': 1, 'named': 118, 'intense': 22, 'recall': 19, "hutton's": 1, 'half-way': 3, 'enabled': 1, 'retains': 5, 'snipe': 1, 'eyes': 91, 'integral': 5, 'de-virginize': 1, 'himalayas': 2, 'comaprison': 1, 'shitheads': 1, 'loft': 2, 'siren': 1, "singer's": 1, 'sung': 1, 'decorates': 2, 'wwf': 2, 'chopping': 4, 'afraid': 19, 'uncle': 6, 'korsmo': 2, 'acrimonious': 1, 'pauly': 4, 'well-staged': 1, 'gander': 1, 'defends': 3, 'brit': 2, 'replace': 11, 'perched': 1, 'odious': 2, 'le': 2, 'kisses': 5, 'dictators': 1, 'illogically': 1, 'pg-movie': 1, 'satisfying': 13, 'inconsiderate': 1, 'narrowing': 1, 'rob': 22, 'charisma': 17, 'overplaying': 2, 'spooky': 7, 'arm-in-arm': 1, 'shoes': 7, 'chasing': 13, 'chance': 85, 'diffusing': 1, 'schmaltz': 1, 'scholar': 4, 'colony': 8, 'star-crossed': 1, 'showdown': 14, 'walther': 1, 'footprints': 5, 'onlookers': 1, 'lives--allows': 1, 'extreme': 25, 'whiney': 1, 'mini-plots': 1, 'snipes': 7, 'bewilder': 1, 'boil': 1, 'shrillness': 1, 'singing': 26, 'stunned': 3, 'prospects': 3, 'pukes': 1, 'scantily-clad': 3, 'spoonfuls': 1, 'torture': 12, 'non-existant': 1, 'inspector': 11, 'stanley': 11, 'macnee': 2, 'nullifies': 1, 'whoaaaaaa': 1, "laura's": 2, 'seventh': 2, 'man--namely': 1, 'squirrel': 1, '_john': 1, 'enthusiasts': 2, "'tis": 1, 'impregnated': 1, 'announcer': 2, 'conjectures': 1, 'baffled': 1, 'unlikeable': 5, 'orange': 6, 'tripping': 2, 'scratched': 2, 'antiseptic': 1, 'fast-forward': 5, 'laissez-faire': 1, 'fantastic': 12, 'wendt': 4, 'cinque': 1, 'strangelove': 3, 'verbally': 5, 'townsfolk': 5, 'willson': 1, '1969': 3, 'geniuses': 2, 'hard-core': 2, 'french-canadian': 2, 'machine-gun': 1, 'groundwork': 1, 'bamboozled': 1, 'alluring': 5, 'timothy': 8, 'vets': 1, 'ritchie': 1, 'realised': 2, "o'connor": 5, 'gloom': 4, 'vague': 9, 'filmy': 2, "mathilde's": 1, '132': 2, 'devious': 4, 'bronson': 8, 'half-witted': 1, 'true-to-life': 1, 'patsy': 4, 'tribulations': 1, 'exclaimed': 1, 'solomon': 1, 'seder': 1, 'transition': 9, 'before---and': 1, 'bath': 3, 'nostalgics': 1, 'medium': 7, 'kinetic': 4, 'chen': 3, 'golan': 1, 'scores': 10, 'backpacker': 1, 'road-trip': 1, '_real_': 3, 'protection': 2, 'tiepin': 1, 'flips': 1, 'landry': 1, 'payoff': 11, 'reportedly': 5, 'dwi': 1, 'adrenaline': 7, 'ventura-style': 1, 'though--wahlberg': 1, "countries'": 1, 'established': 23, 'spewing': 1, 'crunchily': 1, 'wickedly': 2, 'rosenbaum': 1, 'thirty-foot': 1, 'forrest': 3, 'synonym': 1, 'duchovny': 5, 'tide': 1, 'shouted': 3, 'numbered': 1, 'mystics': 1, 'cannes': 2, 'raged': 1, 'unsophisticated': 1, 'presumable': 1, 'sketching': 2, 'execution': 20, 'writhing': 1, 'larter': 5, 'sitcom-bred': 1, 'roles--': 1, 'bondage': 3, 'subdued': 8, 'suspiciously': 5, 'scandal': 2, 'smuggles': 1, 'bitch-queen': 1, '$172': 1, 'spaces': 2, 'clarion': 1, 'beowolf': 1, '90%': 5, 'inundates': 1, 'goo': 7, 'in-person': 1, 'lizards': 3, 'lollipops': 1, 'sneak': 9, 'hollywood-ized': 1, 'handled': 22, 'goodhearted': 1, 'record-breaking': 1, 'meier': 1, 'blankets': 1, 'lukewarm': 4, 'ordered': 1, 'demonstration': 2, 'bo': 5, 'aimlessly': 7, 'spared': 5, 'speech-impaired': 1, 'require': 16, 'hiself': 1, "kick's": 1, '[r]': 11, 'reruns': 1, 'irritatingly': 3, 'bevery': 1, 'share': 27, 'meaninglessness': 1, 'jury': 4, 'refuse': 8, 'delivering': 15, 'munching': 2, 'ravenscroft': 1, 'sense': 160, 'mercury': 5, 'unfolds': 6, 'dirty': 25, 'art-form': 1, 'expression': 17, 'reflection': 3, "tatopoulos'": 1, 'terrain': 4, 'sore': 5, 'deception': 5, 'turf': 3, "mom's": 1, 'goodman': 11, 'excels': 1, 'obssess': 1, 'trap': 15, 'detect': 3, 'zilch': 2, 'ventura': 5, "man'": 6, 'objection': 2, 'heartland': 1, 'blessed': 6, 'occult': 2, 'fastest': 1, 'mandrake': 1, 'work--while': 1, 'legacy': 2, 'conceptually': 1, 'brave': 10, "president's": 3, 'greed': 7, 'anthology-style': 1, 'mile': 14, 'jeunet': 1, 'misled': 1, 'colombia': 1, 'dien': 9, 'sarah': 17, 'temptress': 3, 'boy-next-door': 1, 'slimeball': 1, 'discord': 1, 'jekyll': 1, 'colombian': 1, 'tombstone': 2, 'murata': 1, 'rock-opera-turned-major': 1, 'savoy': 2, 'sanitary': 1, 'genetic': 2, 're-interpretation': 1, 'bradford': 1, 'several-hundred': 1, 'michaels': 1, 'mentality': 4, 'arse': 1, 'wrinkles': 2, 'percussive': 1, 'poison-': 1, 'horn': 2, 'recognizing': 3, 'funicello': 1, 'exterminator': 1, 'jimi': 1, 'lessening': 1, 'massuh': 1, 'fondly': 2, 'lughead': 1, 'appelation': 1, 'mop': 1, 'dissipates': 2, 'barbie': 1, 'industrial': 4, 'highness': 1, 'monotony': 2, 'laments': 3, 'snuffed': 1, 'doppelganger': 2, '_48_hrs': 1, 'alpha': 1, 'thematically': 1, 'megalomaniac': 2, 'bickering': 12, 'dedicates': 2, 'vein': 8, 's&m': 4, 'sprinklings': 1, 'coddling': 1, 'coherence': 5, 'ruby': 1, 'overruns': 1, 'player-turned-owner': 1, 'applicable': 2, 'dude': 19, 'nosed': 1, 'straw': 4, 'axed': 1, 'let-down': 2, 'crack': 22, 'computer-animation': 1, 'congratulations': 3, 'bubba': 1, 'sasha': 3, 'stylized': 3, 'mojorino': 1, 'mentors': 1, 'panting': 1, '3-hour': 1, 'imitation': 7, "dierdre's": 1, 'presaged': 1, 'heterosexual': 4, "2000's": 2, 'achievement': 14, 'i-do-my-job-whether-they-are-innocent-or-guilty': 1, "david's": 3, 'ferrara': 2, 'gigs': 1, 'jawed': 2, 'rubble-strewn': 1, 'raptors': 4, "o'shae's": 1, 'famous': 36, 'connery': 12, 'excruciating': 7, 'munson': 2, 'dim-bulb': 1, 'remix': 1, 'homey': 1, 'beal': 2, 'unattended': 1, 'sab': 1, 'freddy': 6, 'purchasing': 1, '105': 1, 'aa': 1, 'super-agents': 1, 'indecency': 1, 'futurama': 1, 'rocking': 1, 'xing': 1, 'flushing': 1, "rambaldi's": 1, 'lascivious': 1, 'prints': 2, 'splattering': 2, 'melt': 2, 'damsels': 1, 'demonstrating': 2, 'downplays': 1, 'max-like': 1, 'kensington': 1, 'overdue': 3, 'childlike': 1, 'sayings': 1, 'regret--the': 1, 'danger/more': 1, 'frightening': 22, 'isabelle': 1, 'flaws': 26, 'unreachable': 1, 'realise': 3, 'schemers': 1, 'cullen': 1, "'girl": 1, 'jody': 1, 'knife-wielding': 1, 'mild-mannered': 4, "5'10": 1, 'weaker': 2, 'subservient': 2, 'rouge': 1, 'experimenting': 3, 'negative': 12, 'switches': 8, "pimp's": 1, 'launches': 4, '_fear_and_loathing_in_las_vegas_': 1, 'freeway': 3, 'sufficiently': 3, 'gwen': 3, "programme's": 1, 'witnessed': 8, 'ooooooo': 1, 'freshness': 3, "babysitter's": 1, 'je': 1, 'breaching': 1, 'babies': 4, 'high-budget': 1, 'carol': 6, '11-year-old': 1, 'hard-ass': 1, 'core': 26, "gale's": 1, 'regression': 1, 'primed': 2, 'prehensile': 1, 'pie': 15, '4/29/00': 1, 'quibble': 1, 'tiresome': 23, 'voiced-over': 1, 'landon': 1, 'scene-and': 1, 'throat': 9, 'cavern': 1, 'producer': 41, 'excavating': 1, 'seeker': 1, 'silliness': 9, 'nic': 3, 'stagy': 1, 'bizarrely': 3, 'arching': 1, 'prescott': 2, 'complaints': 3, 'bowler': 5, 'life': 279, 'incest': 5, 'jejune': 1, "forlani's": 2, 'goddam': 1, 'lusted': 1, 'deduce': 1, 'vanilla': 1, 'scriptiing': 1, 'socialist': 1, 'unromantic': 1, 'regroup': 2, 'annual': 1, 'solution': 8, 'mouth': 32, 'appropriately': 11, 'eery': 1, 'dramatize': 1, 'smugly': 2, 'pickup': 2, 'bases': 1, 'negated': 4, 'undertaking': 1, 'schizophrenic': 3, 'mystical--perhaps': 1, "julie's": 3, "jung's": 1, 'machismo': 1, 'losely': 1, 'promoted': 5, "zellweger's": 3, 'inventiveness': 1, 'back-stabber': 1, 'flock': 1, 'train-': 1, 'paraded': 1, 'paraphrasing': 2, 'brushing': 1, 'charted': 1, 'expressionism': 1, 'guild': 2, 'gent': 2, 'blurry': 2, 'string-based': 1, 'point-of-view': 3, 'slash': 2, 'cultural': 7, "hittin'": 1, 'ants': 3, 'whisker': 1, 'pen': 10, 'speculator': 1, 'scream3': 1, 'amputation': 1, 'lifted': 12, "riss'": 1, 'fave': 1, 'overdirecting': 1, 'powdered': 1, 'pointing': 10, 'soften': 1, "allen's": 6, 'felatio': 2, 'allman': 1, 'chunky': 1, 'insiders': 1, 'inanity': 5, 'shae': 1, 'foiled': 2, 'figuratively': 2, 'virginie': 1, '8=excellent': 1, 'linn': 1, 'wrestle': 4, 'rooting': 10, 'circles': 12, 'julie': 29, 'beckons': 1, 'possessing': 8, 'conspiracies': 2, 'pronounce': 5, 'well-acted': 1, 'victory': 7, 'prowess': 4, 'marie': 6, 'normal-sized': 1, 'face/off': 6, 'upper-class': 2, 'computer-creation': 1, 'pre-schoolers': 1, 'blown': 22, 'filmic': 1, 'ribs': 1, "rodman's": 1, 'hk': 1, 'uneffective': 1, 'neo-noir': 1, 'orbit': 5, 'magnolia': 4, 'dinsmoor': 1, 'pollution': 1, 'croaker': 1, 'knee': 4, 'boulder': 2, 'conglomerates--have': 1, 'humor-free': 2, 'heightened': 2, 'flash-forward': 1, "`dawson's": 1, 'millions-esque': 1, 'propose': 3, 'music': 126, 'jet-powered': 1, 'red': 54, 'arrakis': 1, 'spunky': 7, 'lilianna': 1, 'jagged': 4, 'emerald': 1, 'auto-pilot': 4, 'chopin': 1, 'cassettes': 1, 'smart': 49, 'despicable': 10, 'whiz': 4, 'pull': 59, 'coo': 1, 'crawling': 2, '1958': 2, "firm's": 1, 'pissed': 8, 'volatile': 3, 'similar': 59, 'rebuild': 1, 'tug': 4, 'proclivity': 1, 'gee': 10, '1990': 5, 'enjoy': 76, "girls'": 5, 'greatly': 11, 'dorm': 2, 'freezing': 2, 'rented': 6, 'tank/prison': 1, 'grilling': 1, 'now-ralph': 1, 'moon': 9, 'shalit': 1, 'arcs': 1, 'frustration': 6, 'tasty': 2, 'castaldi': 1, 'ultra-cool': 2, 'roman': 6, 'exaggeration': 4, 'perceptible': 2, 'define': 5, 'farrell': 2, 'shining/event': 1, 'lump': 2, 'brokedown': 1, 'pampered': 2, 'closest': 12, 'prematurely': 1, 'sprite': 1, '_mortal': 1, 'isaac': 2, 'disturbing': 25, 'hans': 3, 'flashing': 3, 'viewers': 53, 'tempting': 2, 'shoulders': 12, "penn's": 2, 'ravine': 1, 'amateurish-foregrounds': 1, 'mute': 5, 'rocks': 16, '_still_': 1, 'acres': 1, 'fur': 1, 'crevasse': 1, 'dead-set': 1, 'populist': 1, 'genres': 6, 'succeeded': 7, 'heidi': 1, 'head-chopping': 1, 'fortunes': 2, 'intricately': 1, 'bribing': 1, 'kettler': 1, 'persuasion': 1, 'multi-dimensional': 1, 'frightfully': 2, 'flashback': 21, 'fishermen': 1, 'jovi': 4, 'genxers': 1, 'set-pieces--': 1, 'located': 3, 'asian-american': 1, "massee's": 1, 'thrust': 5, 'reteamed': 1, "plenty's": 1, 'bombastically': 1, 'trademarks': 2, 'plent': 1, 'belongs': 14, 'captivity': 1, 'oddly-timed': 1, "huang's": 1, 'red-dyed': 1, 'ghostbusters': 5, 'homework': 3, "korman's": 1, 'screech': 1, '-type': 2, "1977's": 1, 'eyelashes': 1, 'unforgivably': 1, 'pollak': 3, 'highwaymen': 2, 'wrecking': 2, 'mopes': 4, 'rosie': 7, 'gere\x12s': 1, 'yau': 1, 'belle': 1, 'viruses': 1, 'pap': 1, 'vicars': 1, 'excalibur': 3, 'meditations': 1, "lambert's": 2, 'brief-clad': 1, 'dealt': 15, 'limits': 8, 'overpowers': 1, 'misfit': 4, 'donations': 1, 'versatile': 1, 'pseudo-hip': 1, 'thyme': 1, "mama's": 1, 'nosing': 1, 'epic-sized': 1, 'nightmares': 13, 'afro': 1, 'spinal': 1, 'eliza': 2, 'retailers': 1, 'framework': 7, 'snapping': 2, '17': 8, 'peebles': 1, 'loudest': 1, 'congratulate': 1, 'conquest': 4, 'whatsoever': 32, 'money-making': 3, 'cycles': 1, 'undertones': 1, 'wage': 1, 'helgenberger': 3, 'defeating': 1, 'doorway': 1, "bacon's": 2, 'rufus': 5, 'altruistic': 1, 'closely': 10, 'truck': 13, 'ballad': 1, 'cargo': 4, 'grass': 2, 'ill-synched': 1, 'slack-jawed': 1, 'accord': 1, 'icky': 6, 'plantations': 1, 'dogs-could': 1, 'joking': 2, "zeik's": 1, 'accentuated': 1, 'tricky': 6, 'so-so': 6, 'madsen': 7, 'meager': 1, 'salvageable': 2, 'rewrites': 3, 'expertly': 1, 'trend': 14, 'amusingly': 7, "schmidt's": 2, 'soundtrack': 41, 'randall': 3, 'tags': 4, 'rollers': 1, 'lure': 7, 'uber-buff': 1, 'caverns': 1, 'vampires_': 1, 'leaguers': 1, 'refund': 2, 'plunked': 1, 'knicks': 1, 'unborn': 2, 'truck-chase': 1, 'disadvantageous': 1, 'digest/gumpian': 1, 'stick-handles': 1, 'thusly': 1, 'hutton': 3, 'disconnect': 2, 'laugh-o-meter': 2, 'kroon': 1, 'wailing': 1, 'vi': 2, 'co-starred': 2, "elizabeth's": 3, 'washington--one': 1, 'send-off': 3, 'beautifully': 5, 'campiness': 6, 'hip-hop': 1, '000': 38, "'no": 2, 'option': 5, "'breaker'": 1, 'bound': 18, 'skull': 9, 'shapeless': 4, 'lists': 1, 'all--but': 1, 'snake-gaiter': 1, "amistad's": 1, '$400': 2, 'nests': 1, 'shady': 12, 'non-monkey': 1, 'schow': 1, 'pointed': 8, "'hooked'": 1, 'bloodfilled': 1, 'listing': 3, 'narrating': 2, 'pleasures': 9, 'anymore': 13, 'fretful': 1, 'superweapon': 1, 'stephen': 27, "vikings'": 1, "gellar's": 2, 'sample': 5, 'jodie': 1, 'indigestible': 1, 'elbow': 2, 'stinky': 1, 'incarnations': 2, 'overflows': 1, 'honor': 13, 'exchanges': 7, 'scan': 2, 'under-developed': 1, 'blankety-blank': 1, 'brancato': 2, 'betrays': 4, 'critic-proof': 2, 'oise': 1, 'lopped': 1, 'boggs': 4, 'born': 21, 'again-the': 1, "protagonists'": 1, 'odds': 15, 'yorkers': 2, 'shove': 1, 'leverage': 3, 'magnified': 1, 'cut-outs': 5, 'left-field': 1, 'self-centered': 1, 'groaning': 3, 'advertisement': 1, 'law-abiding': 1, 'mib': 2, 'affects': 4, 'arrrrrrrghhhh': 1, 'pleading': 2, 'scripter': 1, 'prosthetic': 2, 'maggots': 2, 'underestimate': 3, 'ra': 1, 'moaning': 3, 'uninvolving': 19, 'captive': 2, 'dam': 2, 'surrenders': 1, 'holdups': 1, "ashton's": 1, 'imo': 1, 'preparations': 2, "ne'er-do-well": 1, 'movements': 8, 'writer': 74, 'living': 76, 'knock': 14, 'basically': 85, 'fortune-telling': 1, 'prized': 1, 'christmas-themed': 1, 'boosting': 1, 'hudlin': 1, "xiii's": 1, 'johnathon': 3, 'peddle': 1, 'big-underwater-snake': 1, 'ventured': 1, 'aaa': 2, 'wunderkind': 5, 'discourse': 1, 'floor': 28, 'low-quality': 1, 'sentences': 2, 'crenna': 1, '`varsity': 2, 'refilmed': 1, 'interloper': 1, 'glassy': 1, 'telemovies': 1, 'sibling': 4, 'overload': 1, 'yuks': 3, '1952': 1, 'answers': 17, 'webb': 3, 'virgin/whore': 1, 'tackyana': 1, 'fibers': 1, 'innocents': 5, 'souped': 1, "ratt's": 1, 'screentime': 2, 'valentines': 1, 'obsolete': 2, 'arraki': 1, 'morgan': 10, 'compassion': 6, 'scenery': 22, 'erratic': 5, 'winged': 1, 'rebound': 3, 'succession': 5, 'inklings': 1, 'pregnancy': 8, 'comprised': 11, 'at--unaccountably': 1, 'billions': 5, 'scriptwriters': 1, 'trivialize': 1, 'sustainably': 1, 'palette': 2, 'annabeth': 2, 'misunderstood': 1, 'direct-to-video': 6, 'couple': 135, 'table': 20, 'silver': 14, 'volcanologist': 1, 'lesabre': 1, 'legends--those': 1, 'rang': 1, 'manhunter': 2, 'mgm': 2, 'harvie': 1, 'capote': 1, 'governing': 1, 'welcoming': 1, 'slurry': 1, 'long-term': 2, "priestley's": 1, 'rarity': 2, 'moron': 5, 'gerrard': 1, 'loews': 1, 'retained': 1, 'perversion': 1, 'jeffrey': 12, 'locks': 6, 'bumming': 1, 'laff-o-meter': 1, "carrie's": 1, 'loveable': 3, 'snags': 1, 'world': 205, 'non-believers': 1, 'dependent': 4, 'gonna': 19, '7': 9, 'well-earned': 1, 'stays': 14, 'males': 10, 'awe-inspiring': 2, "'til": 2, 'greatest': 31, 'ishtar': 3, 'blofeld': 1, 'conspiracy/martial-arts': 1, 'siam': 1, 'mayor': 11, 'spungen': 1, 'slither': 1, 'blinded': 2, "sonny's": 1, 'handily': 1, 'trite': 25, 'apologize': 6, 'illustrator': 1, 'proposals': 1, 'medal': 1, 'blucher': 1, 'unfilmable--the': 1, 'blond': 12, 'twirl': 2, 'elias': 1, 'kissy-faces': 1, 'nightly': 2, 'omnipotent': 1, 'blood-spattered': 1, 'mcclane': 1, 'filmed': 37, "black'": 1, 'specliast': 1, 'opulent': 4, 'mind--': 1, 'bundled': 1, "manipulations'": 1, 'vendela': 2, 'hot-shot-young-no-experience-never-killed-a-man': 1, 'chorus': 1, "''": 3, 'denied': 4, 'intuitive': 1, 'hamill': 1, 'sidetracks': 2, 'elastic': 1, 'surfaced': 1, 'entrepreneur': 3, 'drugged-out': 1, 'failing': 13, 'towne': 2, "farrelly's": 1, 'critical': 21, 'karina': 1, 'low-caliber': 1, 'stomped': 1, 'clenched': 1, 'rehash': 13, 'foray': 4, 'intruded': 1, 'donna': 7, 'imprisoned': 7, 'novella': 1, 'stares': 4, 'videocassette': 1, 'overlong': 12, 'recruitment': 2, 'wallpaperer': 1, 'disney': 28, 'hallie': 1, 'avoiding': 6, 'croc-hunting': 1, 'cutout': 1, 'preserved': 3, 'precipices': 1, 'nether': 2, 'retina': 1, 'harelik': 1, "seth's": 2, 'technological': 3, 'resent': 4, 'cinderella': 2, 'virginia': 1, 'overkill': 6, 'gentry': 1, 'ultimate': 30, 'mime': 1, 'unauthentic': 1, 'it92s': 1, 'wigger': 1, 'rent': 47, 'representing': 2, 'adrenalin': 2, 'tunes': 10, 'hacked-up': 1, 'ringwald': 3, 'hurry': 3, 'nimble': 1, 'son': 84, 'neptune': 1, 'good-work': 1, 'terminators': 1, 'patterson': 2, 'f--k': 1, 'caleb': 3, 'jose': 2, 'nature-loving': 1, 'unwatchably': 1, 'swiped': 1, 'smith': 37, 'man/bag': 1, 'good-bye': 1, 'mediums': 1, 'variant': 2, 'interacting': 2, 'dismissive': 1, 'generates': 5, 'filmgoer': 1, 'freelance': 1, 'numb': 4, 'snell': 1, '-clever': 1, 'animality': 1, 'pumped': 3, 'scholarly': 1, 'incorrect': 5, "earth's": 8, 'harrigan': 1, 'disconcerting': 5, 'thinking': 87, 'mobbed': 1, 'humanoids': 1, 'dmv': 1, 'acquitted': 1, 'rambo': 3, 'twenty-cuts-per-minute': 1, 'burying': 2, 'portrayals': 4, 'add-ins': 1, 'shuffle': 1, 'incipient': 1, 'suspensor': 1, 'club-hopping': 3, 'ashamed': 6, 'verbatim': 1, 'dare': 7, 'explicit': 8, 'adlib': 1, 'quinn': 5, 'spain': 3, 'activist': 2, 'embarrassingly': 5, 'machina': 2, 'bestow': 2, 'rochelle': 1, 'dalliance': 1, 'nfl': 3, "rimbaud's": 1, 'ill-composed': 1, 'noun': 1, 'egotist': 1, 'erotically': 1, 'time-defying': 1, 'not-so-brilliantly': 1, 'mansions': 2, 'pooches': 1, 'no-no': 2, 'superhero/action': 1, 'tropics': 1, 'on-line': 2, 'randle': 2, 'rhyme': 2, 'contentedly': 2, '101': 10, 'dredge': 1, 'signed': 11, 'hid': 3, 'traveled': 2, 'whereabouts': 2, 'closed-down': 1, 'foggiest': 2, 'francoise': 1, 'jamaican': 2, 'regulate': 1, 'balinski': 1, 'unadulterated': 3, 'turtletaub': 1, 'strategically-placed': 1, 'revere': 1, 'expert': 25, 'skirt': 2, 'estelle': 1, 'vomiting': 3, 'facade': 3, 'petri': 1, 'ripoff': 5, 'flu': 1, 'robbery': 11, 'fullest': 2, 'miss': 44, 'forze': 1, 'chaja': 1, 'overwrought': 10, 'nastiest': 1, 'peng-jung': 1, '`this': 2, 'raul': 1, 'crybaby': 1, 'axel': 2, 'passion': 18, 'indoctrinated': 2, 'shipping': 2, 'exposure': 6, 'materializes': 1, 'wide-eyed': 2, 'froehlich': 1, 'paperwork': 1, 'heard': 59, 'gunfight': 1, 'campy': 18, 'vista': 1, 'ambitious': 8, "nomi's": 1, "jackal's": 1, 'blandy': 1, 'saints': 1, 'hayden': 1, 'shower': 17, 'mobsters': 4, 'frann': 1, 'lasted': 3, 'shocked': 8, 'devito': 3, 'waterworld': 1, 'deliriously': 2, 'scratching': 6, 'loss': 19, 'advertise': 1, 'akroyd': 2, 'unintentional': 19, 'chain': 13, 'flyover': 1, 'demon': 5, 'telescopes': 1, 'lenny': 1, 'reports': 7, 'contest': 9, 'buddies': 23, 'curveballs': 2, 'beguiling': 2, 'synch': 2, 'leather-face': 1, 'includes': 47, 'buckets': 5, 'comprehendable': 1, 'boston': 8, 'weld': 2, 'interrogates': 1, 'economy': 6, "jolie's": 7, 'could/but': 1, 'painstaking': 3, 'commentator': 1, 'bumpers': 1, 'insufferable': 3, "deliver'": 1, 'picturesque': 2, 'col-': 1, 'monsters': 12, 'his/her/their': 1, 'ape]': 1, 'johto': 1, 'takings--and': 1, 'pittsburgh': 1, 'similarly-themed': 2, 'bathed': 1, 'microwave': 1, 'wandering': 10, 'madeline': 2, 'motherly': 2, 'drooling': 3, 'filters': 1, 'bay/jerry': 1, 'phelps': 1, 'mumford': 1, 'upper': 5, 'upscale': 1, 'volker': 2, "d'abo": 2, 'essay': 1, 'modus': 1, 'gyllenhall': 1, 'alarms--the': 1, 'one-week-old-blue-cheese-smelling': 1, 'indecipherable': 4, 'sin': 7, 'scraping': 1, 'guinness': 2, 'malik': 1, 'fish-and-game': 1, 'accumulation': 1, 'toadie': 1, 'crewman': 2, "cameron's": 6, 'bumbles': 1, 'prospective': 1, 'overwhelmingly': 2, 'roots': 9, 'annette': 6, 'loathe': 2, 'palmer': 4, 'romania': 2, 'vicar': 1, "universal's": 1, 'mercury-encrypted': 1, 'destructive': 2, "major'": 1, 'bringer': 1, 'in-crowd': 2, 'muck': 5, 'tyrannical': 1, 'horrible': 49, 'queries': 1, 'mock-ups': 1, 'shoveler': 2, 'thereof': 6, 'movie-bedpost': 1, 'frantically': 6, 'community': 19, 'impostors': 1, "leon's": 1, 'flat': 68, 'dharma': 1, "armand's": 1, 'counterfeit': 4, 'slime': 2, 'swapping': 1, 'countryside': 3, 'wachowski': 1, 'ice-cold': 1, '49': 1, 'unmet': 1, 'ecl@mtcts1': 1, "kidnappers'": 1, 'pay-per-view': 4, 'shipboard': 1, 'precursor': 2, 'damed': 1, 'slips': 7, 'screenplay---or': 1, 'karaman': 1, 'energies': 1, "wife's": 5, 'sufferings': 1, 'loops': 2, 'devastating': 3, 'lustful': 1, 'suitors': 1, 'craving': 4, 'cheapest': 1, 'hard-pressed': 3, 'danica': 1, 'kirk': 5, 'brie': 1, 'lathered': 1, '`patch': 1, 'serra': 3, 'focusing': 5, 'marla': 1, "alone'": 1, 'overdrive': 2, "garret's": 1, 'crushing': 5, 'chan': 18, 'culturally': 2, 'bulbs': 1, 'virulent': 1, "rasslin'": 1, 'however---not': 1, 'torturing': 1, 'soars': 1, 'homing': 1, 'gmt': 1, 'sullen': 3, 'cavity': 1, 'megalomania': 1, 'token': 19, "letter's": 1, 'infect': 1, 'underground': 14, 'keough': 1, 'consensual': 1, 'heyst': 1, 'hands-why': 1, 'seuss': 1, 'impulsively': 2, 'vigilante': 4, 'exacerbated': 1, 'kilner': 1, "spielberg's": 2, 'ethel': 1, 'allen-': 1, 'guessing': 13, '_remains': 1, 'dealing': 30, 'lessons': 7, 'resolve': 8, 'improv': 1, 'stewart-esque': 1, 'superstar': 3, 'specializes': 3, '?': 618, 'messiah': 1, 'puked': 1, 'indentity': 1, 'uncanny': 5, 'compromised': 2, 'mentally-disturbed': 1, 'retitle': 1, 'affectionately': 1, 'cordial': 1, 'retelling': 5, "stillman's": 1, 'genuinely': 26, "turner's": 2, 'spurgin': 1, 'enlightening': 3, 'fims': 1, 'slyly': 1, 'incentive': 1, 'surprises': 20, 'necrophilia': 2, 'inexorably': 1, "carlito's": 1, 'blues/rock': 1, 'optimistic': 4, 'conducts': 1, 'ramblings': 1, "'97": 2, 'brutal': 15, "canine's": 1, 'bobo': 1, 'awe': 8, "`darlene's": 1, 'nadir': 2, 'pimps': 1, 'sherbedgia': 1, 'krupa': 1, 'oscar-worthy': 1, 'crucial': 16, 'stereotypically': 2, 'atlantic': 4, 'binge-drinking': 1, 'amply-sized': 1, 'affection': 9, 'trapeze': 1, 'damn': 37, 're-cut': 2, 'dehumanization': 3, "breillat's": 1, 'cozart': 1, "fox's": 1, 'skeptic': 2, 'arguement': 1, 'hurls': 2, 'consisted': 1, 'co-horts': 1, 'pale': 11, 'whobilation': 1, 'leprechaun': 2, 'buddy-action': 1, 'ingenious': 3, 'point-blank': 1, 'artificial': 16, 'profits': 4, 'eyeballs': 4, "`daylight'": 1, "seattle'": 1, 'marion': 1, 'claims': 21, 'lovebirds': 1, 'insanity': 6, 'matiko': 1, "adams'": 2, 'produced': 46, 'informative': 2, 'coach': 20, 'brightens': 1, 'hairball': 1, 'tackle': 3, 'overlooked': 5, 'noxzeema': 1, 'executive': 18, 'salkind': 1, 'dolls': 6, "francis'": 1, 'rye': 2, 'coltrane': 1, 'tight-knit': 1, 'joe-wreaking': 1, 'moron-proof': 1, 'well-shot': 1, 'street-wise': 3, 'indoors': 1, 'houses': 10, 'hagerty': 1, "instinct's": 3, 'niggling': 2, 'enemy': 17, 'misteps': 1, 'handsome-looking': 1, 'await': 2, 'dalton': 3, 'buster': 3, 'gobs': 1, 'dano': 1, "audrey's": 1, 'deal': 70, 'injection': 3, 'overhears': 1, 'jokey': 4, 'kellner': 1, 'key': 37, 'cleans': 3, 'organizations': 1, 'pawnbroker': 1, 'helicopters': 4, '--made': 1, 'ridiculously': 21, "febre's": 1, 'zephyr': 1, 'song': 34, 'gazed': 1, "tarzan's": 1, 'sieber': 1, 'homoerotic': 1, 'accommodate': 1, 'director/writer': 4, 'suffice': 13, 'first-hand': 2, 'abnormally': 1, 'trip': 46, 'scams': 3, 'admiring': 1, 'flown': 1, 'chronological': 1, 'puzo': 1, 'responsible': 28, 'rugged': 3, 'haughtiness': 1, 'full': 117, 'rescues': 6, 'hundred': 19, 'formidable': 4, 'mandel': 1, 'semi': 1, 'lope': 1, 'parent': 6, '6': 18, 'ambitions': 3, 'doesn9t': 1, 'sweep': 1, 'evans': 5, 'additive': 1, 'disagreeable': 1, 'engrossing': 5, 'outfielders': 1, 'ensuring': 2, 'distinguishes': 3, 'audi': 1, 'photographs': 3, 'exhausted': 4, 'unwillingly': 2, 'mallrats': 3, 'nicest': 1, 'wizard': 5, 'ocmic': 1, 'displayed': 13, 'nerves': 9, 'leap': 9, 'buys': 6, 'inconvenient': 1, "potter's": 2, "niece's": 1, 'portorican': 1, "helgenberger's": 1, 'pained': 3, 'lumbergh': 1, 'exorcise': 2, 'timeless': 2, 'endlessly': 4, 'assumes': 7, 'idea': 161, 'plundering': 1, 'magma': 2, 'ripple': 1, 'callow': 1, '[spoiler': 1, 'spotted': 5, 'hush': 4, 'ordinarily': 2, 'initial': 18, 'slate': 2, 'backlash': 1, 'movie-within-a-movie': 1, 'walters': 1, 'soak': 1, 'constructs': 1, 'justified': 6, 'ya': 5, 'collegiate': 1, 'bigfoot-sized': 1, 'fascism': 3, 'haunting': 19, 'mcdowall': 3, 'duct': 1, 'endings': 11, 'unmade': 1, 'creates': 28, 'fractions': 1, 'surrounds': 3, 'fornicators': 1, 'fire-ball': 1, 'inexcusable': 4, 'mod': 7, 'feel-good': 3, 'vandercave': 1, 'peek-a-boo': 1, 'disbelief': 18, 'underscoring': 1, 'department': 23, "'never": 1, 'buckles': 1, 'alvin': 1, 'poker': 3, 'break-danced': 1, 'half-black': 1, 'wan': 6, 'bobby': 11, 'gabby': 2, 'inspect': 2, 'swallowed': 1, 'languidly': 1, 'chauffeur-driven': 1, 'martial': 20, 'andrew': 21, 'months': 35, 'reconciling': 1, "2'8''": 1, 'copulating': 1, 'practically': 24, 'derived': 7, "rudolph's": 1, 'duties': 6, 'kernel': 1, 'else--i': 1, 'trails': 1, 'capshaw': 1, 'dissection': 2, 'lampoon': 1, 'broughton': 1, "avengers'": 1, "[spawn's]": 1, 'shallowness': 4, 'preparing': 10, 'needles': 1, '`': 2, 'tatyana': 1, 'drang': 1, 'perpetual': 4, 'multiplex': 5, 'post-revolutionary': 1, 'optimum': 1, 'tarp': 1, 'underplayed': 1, 'nukes': 1, 'phenomenal': 2, 'wire-': 1, 'bland': 53, 'insight': 21, "preacher's": 1, 'prop': 3, 'portion': 8, 'anatomically': 1, 'historians': 1, 'bellows': 2, 'nose': 14, "cane's": 1, 'uhh': 1, 'superweapons': 1, "organization's": 1, 'lowest': 7, 'governor': 1, 'ditto': 10, 'launcher': 1, 'crawford': 5, 'plea': 3, 'sales': 4, 'convention': 12, 'iq': 12, 'vaudeville': 1, 'demon-fighting': 1, 'payed': 2, 'chuck': 7, 'keital': 1, 'merit': 12, 'employment': 2, 'innocuously': 1, 'dagger': 1, 'solve': 13, 'observing': 1, 'tantrum': 1, 'buck-naked': 1, 'nahon': 1, 'knowles': 1, 'getup': 1, "he'": 1, 'dominating': 3, 'rushmore': 1, "suvari's": 2, 'carpenter': 10, 'cleavon': 1, 'benigni': 1, 'corvette': 1, 'irving': 3, 'schwimmer': 4, 'bakalian': 1, 'woos': 1, 'peek': 2, "butler's": 1, 'lovely': 20, 'dropping': 12, 'brandon-like': 1, 'collage': 1, 'mori': 1, 'horseshoes': 1, "modine's": 1, 'wowed': 1, 'tarantino/robert': 1, 'videodrome': 1, 'conquer': 4, 'captains': 1, 'co-production': 1, 'nichols': 4, "slow-mo's": 1, 'breakfast': 8, 'dzunza': 1, 'wit': 37, 'retreats': 1, 'richly': 1, 'retrospect': 2, 'belaboured': 1, 'ppreciate': 1, 'insecure': 6, 'noseworthy': 1, 'phoebe': 3, 'hoffman': 14, 'migrate': 1, 'greenpeace-friendly': 1, 'presiding': 1, 'yerzy': 1, 'promptly': 7, 'ingest': 1, 'lorraine': 1, 'lecture': 7, 'newsleak': 1, 'declare': 1, 'kline': 7, 'lumbers': 1, 'abductions': 1, 'transcripts': 1, 'predicable': 1, 'penultimate': 2, '30-minute': 1, 'acclaimed': 6, 'case': 151, 'outrunning': 1, 'worse': 125, 'elicit': 7, 'nobel': 1, 'henchman': 7, "drag-it's": 1, 'dependable': 3, 'interference': 3, 'performer': 16, 'improbable': 4, 'importantly': 19, 'reccemond': 1, '4-year-old': 1, 'barest': 1, 'wizards': 1, 'puking': 2, 'estrangement': 1, 'rant': 5, 'gadgets': 5, 'precisely': 10, 'two-headed': 1, "judah's": 1, 'newfoundland': 2, '150': 1, 'anders': 1, 'cannery': 1, 'cultured': 1, 'gleaned': 1, 'silverware': 2, ']': 7, 'defined': 7, 'riddles': 2, 'invitation': 4, "kitano's": 1, 'bicycle': 1, "returns'": 1, 'ever-antsy': 1, "gwenie's": 1, 'fiber': 1, 'schweppervescent': 1, 'front-line': 1, 'labyrinth': 4, 'messias': 1, "o'donnell": 16, 'violence/gore': 1, 'sack': 4, 'nearest': 1, 'achievements': 3, 'insightful': 6, 'afi': 1, 'ranged': 1, 'macnamara': 1, 'fans': 76, 'demonically': 1, 'sewell': 4, 'sincere': 4, "lange's": 2, "--there's": 1, "judd's": 3, 'crying': 13, 'insane': 13, 'overdoser': 1, 'phew': 2, 'boob': 2, 'inspiration': 10, 'gambling': 9, 'bloodstains': 1, "marc's": 1, 'nostalgically': 1, 'overpaid': 3, 'small': 119, 'farther': 1, 'sunsets': 2, '20': 34, 'pagniacci': 1, 'restroom': 1, 'scrapbook': 2, '15th': 2, 'tagawa': 2, 'pulp': 14, 'adventurer': 1, 'spread': 12, 'nocturnal': 1, 'nosedives': 1, 'noah': 1, 'copulate': 1, 'morbid': 3, 'sito': 1, 'writeup': 1, 'graves': 2, 'angle': 19, 'grows': 27, 'differentiates': 1, 'exploration': 7, 'excellently': 1, 'generalizations': 1, 'shears': 1, 'destroyed': 21, 'ridiculing': 1, 'appetizer': 1, 'self-consciously': 2, 'palm': 2, 'dividing': 1, 'tease-a-thon': 1, 'malebolgia': 1, 'insect': 2, 'dir-ect-or': 1, 'hidden': 19, 'prepare': 7, 'heinous': 1, 'weary': 5, "dick's": 1, 'inordinate': 1, "men's": 3, 'low-budget': 10, 'dissapointing': 2, 'assume--everything': 1, 'shadyac': 1, 'unspools': 1, 'pasts': 1, 'accountant': 2, 'antz': 3, 'businessmen': 4, 'long-lived': 3, 'pronouncing': 2, 'palms': 1, 'tricking': 2, "phillip's": 1, 'trysha': 1, 'liberate': 2, 'slapped': 1, 'conflict': 30, 'passengers': 8, '$1000': 1, "dga's": 1, 'prequel': 3, 'chambers': 9, 'races': 8, 'novels': 6, "paul's": 3, 'pittance': 1, 'creeping': 1, 'involved': 90, 'elevate': 2, 'stash': 3, 'distinctly': 4, 'forthcoming': 2, 'circumcision-giving': 1, 'latest': 74, 'burg': 1, 'spectre': 1, 'globe-trotting': 1, "space'": 1, 'exorcised': 1, 'noticable': 1, 'tolerate': 4, 'dudley': 2, 'peeping': 2, 'onboard': 3, 'toe-stub': 1, 'manages': 94, 'chaney': 1, 'imported': 3, 'interestingly': 8, '92-minute': 1, 'hope-horror': 1, 'trivia': 3, "barbara's": 1, 'pansy': 1, 'flambeeing': 1, 'turned': 77, 'seductive': 5, 'wetsuits': 2, 'good': 518, 'pokemon': 2, 'procedure': 5, 'rigorous': 1, 'loser': 22, 'informs': 9, "there'd": 1, "scinto's": 1, ')': 856, 'uh-uh': 1, 'northwest': 2, 'disability': 1, 'one-word': 1, 'burned': 9, 'odyssey': 6, '50s/early': 1, 'serafine': 1, 'bruised': 3, 'fold': 2, "'vanity": 1, 'ebola': 1, 'finch': 2, 'purity': 2, 'dishwasher': 1, 'reiser': 1, 'miscegenation': 1, 'pseudo-': 1, 'true/three': 1, 'angered': 2, 'hard-earned': 7, "'wooden": 1, 'tri-star': 1, 'stemmed': 1, 'opportunistic': 1, 'questionable': 8, 'solemnly': 1, 'scene-chewing': 1, 'renamed': 2, 'opportunity': 54, 'crossbreed': 1, 'self-destructed': 1, 'ordering': 4, 'tornatore': 1, 'illinois': 2, 'intruding': 1, "movie's": 89, 'erupt': 1, 'arms-merchant-cum-safari-host-': 1, 'whiz-quiz-kid': 1, 'dress': 17, 'smoked': 2, 'applicants': 1, 'finds': 96, 'harbour': 2, 'now-escaped': 1, 'marsha': 2, 'competent': 12, 'thiefs': 1, 'epics': 1, 'artistry': 3, 'crawls': 1, 'broadcast': 14, 'incidently': 1, 'croissant': 1, 'remains': 36, 'succulent': 1, 'unlike': 48, 'titan': 1, 'camerawork': 8, 'arnez': 1, 'canister': 1, 'nuances': 2, 'misadventure': 2, 'two-tone': 1, "should've": 4, 'cottage': 1, 'beattie': 1, 'projectionist': 1, 'shtick': 7, 'nomination': 10, "minelli's": 1, 'content': 29, 'discredit': 3, 'full-frontal': 1, 'level--it': 1, 'pepto': 1, 'megan': 3, 'action-comedy': 2, "mimura's": 1, 'defoliates': 1, 'anij': 1, "'94": 4, 'persecution': 1, 'abuses': 3, 'spoofs': 6, 'coos': 3, 'bullies': 3, 'connelly': 2, 'refuge': 4, 'insure': 2, 'features': 70, 'enchilada': 1, 'arye': 1, 'cadillac-cruising': 1, 'threshold': 2, 'steiger': 1, 'thandie': 2, '^': 1, 'assemblage': 1, 'addicted': 8, 'creed-miles': 1, 'gwyneth': 10, 'anne': 17, 'gains': 2, 'off-kilter': 4, "betty's": 1, 'overgrown': 2, 'drug-op': 1, 'dog-napping': 1, 'adjectives': 4, 'majority': 13, 'jackie': 20, "mccoy's": 1, 'pretends': 12, 'forty-nine': 1, 'incense': 1, 'nite': 2, 'rosario': 1, 'hospitals': 2, 'mouthing': 3, 'eminently': 2, 'magnitude': 3, 'accordance': 2, 'comical': 7, 'digital': 15, 'hangover': 2, "macnee's": 1, 'taunting': 2, 'spiritedly': 1, 'blood-soaked': 3, 'cassavetes': 1, 'protoplasmic': 1, "colony's": 1, "red's": 2, 'noticeably': 6, 'cloning': 2, 'lesra': 1, 'kidnapper': 2, 'sidequel': 1, 'goodness': 1, 'stella': 2, 'mechanism': 1, 'nuzzling': 1, 'cop-corruption': 1, 'tannen': 1, 'slipup': 1, 'cheddar': 1, 'wholly': 5, 'stunt': 16, 'cooder': 1, 'fashioning': 1, 'bides': 1, 'whines': 2, 'gymnasts': 1, 'garnered': 2, 'hams': 5, "club's": 1, 'dilapidated': 2, 'backwords': 1, 'logjams': 1, 'heritage': 2, 'recognition': 9, 'waterboy': 2, 'commendably': 1, 'histrionics': 2, '1973': 4, "tv's": 23, 'aerosmith': 4, 'gobbling': 1, 'tidal': 3, 'parasitic': 1, 'ritzy': 1, 'device--you': 1, 'purses': 1, 'great': 260, 'sworn': 4, 'durham': 6, 'good-natured': 5, 'transvestitism': 1, 'pornography--ie': 1, 'reassembled': 1, 'raped': 10, 'victoria': 6, "aniston's": 1, "mandoki's": 2, 'wistfully': 1, 'vanishing': 4, "intentions'": 1, 'regard': 1, 'thwarts': 2, 'directive': 1, 'copland': 1, "west's": 6, 'butt-head': 1, 'kayley': 1, 'kids': 89, 'woo-like': 1, 'writer/director/hack': 1, 'self-sacrificing': 1, 'hsiao-hsien': 1, 'mickey': 16, 'background': 45, 'director': 380, 'hold-up': 1, 'poorly': 56, 'pritchett': 1, 'scorn': 1, 'elfman': 11, 'successful': 58, 'manipulation': 2, 'restauranteur': 1, 'unforgettable': 4, 'dashiell': 1, 'limb-twisting': 1, "'leaving": 1, 'lensed': 1, 'some--not': 1, 'slept': 3, 'sells': 5, 'undeserving': 3, 'kid': 53, 'coroner': 1, 'breakups': 1, 'puppet': 4, 'goldberg-type': 1, 'eye-catching': 4, 'villain': 65, 'sampling': 1, "levy's": 2, 'swayze': 1, 'connor': 3, 'spetters': 1, 'parrot': 3, 'liu': 7, 'deed': 7, 'unchristian': 1, "lehmann's": 1, 'sandy': 4, 'vanishes': 2, 'pig-headed': 1, 'average': 33, 'insatiable': 1, 'mockery': 1, 'ghostly': 1, 'bumping': 3, 'piper': 4, 'reliant': 1, 'senile': 2, 'maltin': 2, 'fu': 7, 'strung-out': 1, 'large': 56, 'nab': 1, 'partly': 6, 'unworthy': 4, 'beek': 2, 'flubber': 7, 'acceptable': 17, 'councilmembers': 1, 'commitment': 6, 'old-sitcoms-to-movie': 1, 'anyhoo': 1, 'seizure': 1, 'declared': 3, 'runner-esque': 1, 'heck': 23, 'map': 7, 'thoughtful': 11, "vanities'": 1, 'apfelschnitt': 1, 'apex': 2, 'compliment': 10, 'dissapointment': 2, 'cutaway': 1, "finnegan's": 2, 'bedmates': 1, 'example--but': 1, 'grumpier': 2, 'story': 452, 'strindberg': 1, 'swamps': 1, 'scene': 335, 'decently': 2, 'game-experience': 1, 'rollerblades': 1, 'schooler': 1, 'mistake': 32, 'eszterhas': 8, "sarone's": 1, 'trap-ish': 1, 'hess': 2, 'comfort': 7, 'animal-skin': 1, 'mythological': 2, 'chewed': 1, 'cram': 2, 'comprehend': 7, 'fresh-from-jersey': 1, 'hodges': 1, 'mumbo-jumbo': 5, 'read': 78, 'transgressions': 2, 'expects': 10, 'showmanship': 2, 'humiliate': 4, 'children': 72, 'wannabe-zany': 1, 'escalates': 2, '84': 2, 'milked': 3, '1948': 2, 'terminator': 18, 'coping': 2, 'verne': 1, 'jenna': 8, 'hand-jobs': 1, 'up-and-comer': 1, 'boost': 4, 'spilled': 5, 'fourth': 16, 'singed': 1, "adjani's": 1, 'scribblings': 2, 'wanda': 6, "fichtner's": 2, '22-minute': 2, 'salesmen': 1, 'nerds': 8, 'facelessness': 1, 'codename': 1, 'emergency': 6, 'headless': 3, 'suspicion': 7, 'blew': 5, '4=disappointing': 1, 'rape/murder': 1, "gilliam's": 2, 'potboilers': 1, 'ragtop': 1, 'venkman': 1, 'embarassing': 6, '$20s': 1, 'fulfills': 1, 'degeneres': 3, 'inadvertenty': 1, 'kidnapping': 16, 'chameleon': 2, "fulci's": 1, 'knockoff': 1, 'male/female': 1, 'pockets': 4, 'enhancing': 1, 'defusal': 1, 'send': 24, 'unentertaining': 4, 'crash]': 1, 'becky': 1, 'soft': 14, 'terrorized': 3, "scattershot'": 1, '25': 9, 'newark': 2, "bastard's": 4, "off'": 1, 'drugs': 25, 'raider': 3, 'whacks': 2, 'attraction': 19, 'clarify': 1, 'violinist': 1, 'gentlemen': 6, "kasdan's": 2, 'earthworm': 1, 'george': 53, 'klingon-looking': 1, 'moseys': 1, 'co-owner': 1, 'stallone/stone': 1, 'baptist': 1, 'bordello': 1, 'earshot': 1, 'stable': 4, 'uncompelling': 1, 'chazz': 5, 'vaudevillian': 1, 'minds': 25, 'aiello': 4, 'sir': 9, 'sanderson': 1, 'examples': 17, 'backing': 2, "recall'": 2, 'fx': 5, 'qu': 1, 'philosophical': 8, "africa's": 1, 'dealer': 13, 'turner': 7, 'resists': 1, 'golfballs': 1, 'coloured': 1, 'accelerator': 1, 'saharan': 1, 'straight-faced': 1, 'masters': 8, 'avoids': 1, 'getaway': 2, 'egoyan': 3, 'premieres': 1, 'sabotage': 3, 'constricted': 1, 'petite': 1, 'muddle': 2, 'columbine': 2, 'boast': 7, 'goldsmith': 2, '1988': 9, 'flesh': 18, 'gunman': 3, 'manipulates': 3, 'manual': 3, 'plentiful': 4, 'robinson-blackmore': 1, 'blethyn': 1, 'profs': 1, 'toenails': 1, 'scowling': 1, 'speeds': 2, 'serve': 28, 'rosanna': 2, 'juries': 2, 'toning': 1, 'tossing': 1, 'hispanic': 1, 'langella': 3, 'acknowledged': 2, 'adventures': 17, 'assassins': 5, 'even-': 1, 'convenience': 9, 'intentioned': 2, 'threats': 5, 'unreceptive': 1, 'properties': 1, 'post-shower': 1, "russell's": 2, 'unfit': 4, 'jada': 2, 'absence': 14, 'semi-saving': 1, 'mufti': 1, 'timekiller': 1, 'recieve': 1, 'wandering-but-not-going-anywhere': 1, 'fairness': 10, 'caves': 5, 'scorn-ful': 1, 'composite': 1, 'heat': 13, 'shorthand': 1, 'jessica': 10, 'covertly': 1, 'luke-skywalker-like': 1, 'opportunists': 1, 'margaret': 2, 'disappear': 8, 'falzone': 1, '8-minute': 1, 'designed': 27, 'figs': 1, 'huggable': 1, 'federico': 1, 'greenblatt': 1, 'neckbraces': 1, 'mantle': 1, 'snarl': 1, 'ogres': 1, 'motherf': 1, 'plenty': 43, 'acceptance': 2, 'woodman': 1, 'pinpoint': 2, 'looooot': 1, 'much-needed': 6, 'enmeshed': 1, 'danika': 1, 'launch': 3, 'se7en-ish': 1, 'bolster': 1, 'skimp': 1, 'visceral': 6, 'juergen': 1, 'over-acting': 2, '200': 9, 'learning': 13, 'improved': 12, 'helmer': 2, 'reappear': 2, 'acupuncture': 1, 'chewing': 8, 'helplessly': 1, 'dolph': 4, 'presume': 5, 'sewer': 4, 'yu': 1, 'tested': 2, 'alien-rip-off': 1, 'golino': 1, 'mishaps': 1, 'yang': 3, 'fastened': 1, 'adrien': 1, 'occurances': 1, 'rowell': 1, 'zeist': 1, 'revving': 1, 'rural': 3, 'twitching': 2, 'lass': 1, 'bodyguard': 4, 'cube': 6, 'joe': 47, 'colors': 6, 'mullally': 1, 'polished': 7, "madsen's": 1, 'joanou': 2, 'nauseated': 1, 'thrillseekers': 1, 'ephrons': 1, 'inuit': 1, "mcgowan's": 2, 'heartily': 3, 'badges': 1, 'operator': 6, 'assortment': 6, 'aid': 13, 'ageing': 1, 'mugging': 7, 'indifference': 2, 'disco-themed': 1, 'explosions': 27, 'cessation': 1, 'travesty': 5, 'shalhoub---but': 1, 'consumated': 1, 'super-orgasmic': 1, 'radar': 5, 'once-simple': 1, 'burlesque': 1, 'box-office': 13, 'anticlimactic': 4, 'pearls': 1, 'beautiful': 75, 'darkest': 3, 'philosophers': 1, 'menage-': 1, 'surface': 24, 'electrician': 1, 'koufax': 2, 'afred': 1, 'pieced': 1, '27': 2, 'paradoxical': 1, 'victoriously': 1, 'harbor': 5, 'dragon': 9, 'blandness': 2, 'shrewd': 2, 'crucible': 2, 'replica': 4, 'neck-': 1, 'diamond': 18, 'morally': 1, 'hannigan': 2, 'drag': 19, 'conned': 2, 'digressions': 1, 'investing': 2, 'buffoons': 5, 'admirably': 4, 'nursery': 2, 'impersonators': 1, 'regretfully': 2, 'teacher': 21, 'coagulate': 1, 'foxx': 4, 'godless': 1, 'trifle': 1, 'processed': 1, 'brow': 2, 'slashed': 3, "martha's": 3, 'minor-league': 1, 'cabaret': 3, 'light-hearted': 5, 'slaves': 6, 'coattails': 2, 'mover-and-shaker': 1, 'yen': 1, 'bothering': 2, 'frailty': 1, 'unitentionally': 1, 'leads': 80, 'salute': 1, "petersen's": 1, 'beg': 2, 'congenial': 1, 'transportation': 4, 'wreaking': 4, 'fabiani': 1, 'animal': 18, 'animators': 2, 'miserable': 17, 'convoluted': 19, 'haircut': 3, 'competing': 1, 'raised': 11, 'rituals': 3, 'knee-jerk': 1, 'post': 14, 'relay': 1, 'expressed': 4, 'illogical': 11, 'kitten': 2, 'payday': 1, 'illustrations': 1, 'bridges': 6, 'aircraft': 3, 'lynskey': 1, 'terrorism': 1, "valerie's": 2, 'ryu': 1, 'cuckolded': 1, 'backfire': 3, 'gigantic': 8, 'hive': 1, 'mocking': 7, 'ends': 110, 'plot-driving': 1, '05': 2, 'dawkins': 3, 'perry': 7, 'reelviews': 1, 'gene': 23, 'sired': 1, 'ginty': 1, 'bilking': 1, 'welfare': 2, 'modine': 5, 'imports': 2, 'pastorelli': 1, 'demonstrates': 12, 'cobblers': 1, 'bride': 17, 'dismember': 1, "jake's": 1, 'mouthful': 1, 'thugs': 13, 'actress': 59, 'haze': 4, 'nixes': 2, 'nicer': 5, 'helpings': 1, 'looker': 1, 'hmm': 3, 'purely': 12, 'snobbery': 1, "lambs-he's": 1, 'animals': 26, 'father-in-law': 1, 'landis': 3, 'incongruities': 1, 'co-scripted': 1, 'superglued': 1, 'senator': 4, 'kinky': 7, "nobody'll": 1, 'overshadows': 1, 'archer': 2, 'catastrophe': 8, 'emptiness': 3, 'scotsman': 2, 'wallpaper': 2, 'sword-play': 1, 'saucers': 1, 'max-sized': 1, 'sub-': 1, 'misguided': 14, 'yard': 3, 'mimic': 6, 'prinze': 17, 'spoiler': 5, "darryl's": 1, 'gradual': 1, 'gregarious': 1, 'laborious': 5, 'tiger': 8, 'mannered': 2, 'dinos': 3, 'legitimate': 4, 'jive-talking': 2, 'affect': 5, 'frosting': 1, 'welder': 1, 'four-minute': 1, 'comedic': 34, 'angeles': 16, 'giggle': 6, "cisco's": 2, 'longest-seeming': 1, 'fancy-schmancy': 1, 'nauseous': 2, 'streets': 21, 'bitchiest': 1, 'howl': 1, 'crumbling': 3, 'imagination': 25, 'rival': 28, 'downsizing': 1, 'sioux': 1, 'pier': 1, 'unanimous': 1, 'concerned': 19, 'stereoscopic': 1, 'vibrant': 5, 'urban-legendary': 1, 'manner': 43, 'muddled': 15, 'gebrecht': 1, 'romances': 3, 'pantry': 1, 'sliced-off': 1, 'yammering': 1, 'self-healing': 1, 'turrets': 1, 'slugs': 1, 'monumental': 2, '1942': 1, 'glitzy': 3, 'hapless': 18, 'repetitious': 2, 'madwoman': 1, 'rigid': 4, 'tipped': 4, 'dislike': 3, 'superheroes': 3, 'wise': 28, 'intensive': 1, 'clint': 5, 'guy--a': 1, 'sunshiny': 1, 'schoolchildren': 1, 'sal': 1, 'illegitimate': 3, 'junk-bonds': 1, 'utterly-pointless': 1, 'diago': 1, 'narration': 14, 'bent': 7, 'catered': 1, 'fronts': 4, 'gross-out': 5, 'travelers': 2, 'sands': 2, 'exiled': 2, 'stink': 4, 'supermodel': 8, 'galapagos': 1, 'marshalls': 1, 'patchwork': 1, 'off-putting': 1, 'bartusiak': 1, 'contenders': 3, 'hoenig': 1, 'geometry': 1, 'impostor': 1, 'lapd': 3, 'incompetently': 2, 'spark': 16, 'oppressing': 1, "'lost": 1, 'networks': 4, 'liverpool': 1, 'lung': 1, 'salacious': 1, 'contradiction': 2, 'psychlo': 5, 'dressed': 28, 'gore': 35, 'sector': 1, 'cherubic': 2, 'describe': 30, 'spooketeria': 2, 'terrors': 1, 'buildings--a': 1, 'zwick': 1, 'barney': 2, 'enjoyable': 49, 'not-so-darkest': 1, 'bearable': 5, 'chiefly': 1, 'movietone': 1, 'real-time': 1, 'ho-hummer': 1, 'theresa': 9, 'passenger': 8, 'doubled': 1, 'mcdonald': 6, 'pallbearer': 1, 'whitman': 2, 'pizza': 2, 'clicked': 1, 'official': 17, 'gruffudd': 1, 'culture': 17, "space's": 2, 'tank-': 1, "`v'": 1, 'psychically': 2, "cube's": 1, "batter's": 1, 'playthings': 1, 'starters': 3, 'substory': 1, 'baird': 1, 'mckinney': 2, "bunz's": 1, 'exiting': 2, 'tod': 2, 'rogaine-nightmare': 1, 'low': 49, "'production'": 1, 'teenagers--though': 1, 'titillatingly': 1, 'med': 3, 'un-funny': 1, 'recieved': 1, 'wired': 2, 'inclusion': 7, '89-minutes': 1, 'newest': 7, 'bartender': 11, 'newfound': 5, 'transcended': 2, 'leveling': 1, 'never-before-seen': 1, 'optioned': 1, 'harrier': 2, "lesra's": 1, 'imogen': 1, 'all-but-screaming': 1, 'loan-shark': 1, 'kinka': 1, 'gecko': 1, 'comrade': 2, 'strengths': 2, 'cesarean': 1, 'pretext': 3, 'education': 6, 'loni': 1, 'fetishes': 2, 'pimp': 2, '1600': 3, "patric's": 1, 'storyteller': 1, 'bonus': 4, 'thwarting': 1, 'illogic': 2, "virus'": 1, 'catching': 8, 'uninspired': 27, 'oooh': 1, 'cinemtography': 1, 'amputated': 1, 'douriff': 1, 'irene': 7, 'earning': 1, 'miffed': 1, 'kevil': 1, 'caned': 1, 'gidget': 1, 'eno': 1, 'unfunny': 57, 'maggie': 7, 'factories': 1, 'tyra': 1, 'steadfast': 1, 'spotlight': 4, 'boundaries': 5, 'booby': 3, 'oedipus': 1, 'stuntwoman': 1, "'t": 2, 'searched': 2, 'minute-long': 2, 'accidents': 1, 'apocalypse': 5, 'witt': 4, 'bear': 18, 'anti-semitism': 1, 'dragged-out': 1, 'kilt': 1, 'radiantly': 1, 'polito': 1, 'pre-credit': 1, 'scene-by-scene': 1, 'motaro': 1, 'abrasive': 3, 'lackey': 2, "titus'": 1, 'silhouetted': 1, 'sixteenth': 2, 'fanboy': 1, 'frills': 1, "williams'": 5, "prentice's": 1, 'androids': 1, 'belinda': 1, 'volvo': 1, 'anser': 1, 'nob': 1, 'inexplicably': 16, 'terrible--mr': 1, 'hoenicker': 1, 'drastic': 3, 'touting': 2, 'over-the-top': 27, 'samuel': 10, 'mindnumbingly': 3, 'romancing': 1, 'presumption': 1, 'sacrificed': 1, "bont's": 4, 'thinnest': 1, 'multi-week': 1, "mcconaughey's": 1, 'receiver': 1, 'pummeling': 1, 'post-break-up': 1, 'neuromancer': 1, 'angry': 26, "friedman's": 1, 'week-old': 1, 'jaunts': 1, 'e-mails': 3, 'stabbed': 6, 'bloods': 1, 'france': 11, 'briefly': 10, 'invasion': 4, 'chastising': 1, 'worrying': 6, 'ear': 10, 'evil-blackmailing': 1, 'transcend': 2, 'mireniamu': 1, '_halloween': 1, "superman's": 1, 'shootouts': 1, 'venturing': 1, 'lady': 36, 'picnic': 1, 'grab-bag': 1, 'discard': 1, 'mtv-style': 1, 'converts': 1, 'ballroom': 2, 'caligari': 1, 'chair': 12, '$35': 1, 'hot-to-trot': 2, 'aalyah': 1, "night's": 4, 'pointe': 4, 'uniformly': 5, 'tangling': 1, 'illustrious': 1, 'appalled': 2, 'soran': 1, 'sleepless': 1, 'vii': 2, 'reflecting': 2, 'taped': 1, 'shines': 7, 'squeeze': 7, 'endangered': 4, "lion's": 1, 'demonstrable': 1, 'simplistic': 7, "dellaplane's": 1, 'savant': 1, 'unofficial': 3, 'hipster': 1, 'decrepit': 1, '`humans': 1, 'murky': 11, 'r-rated': 7, 'pertaining': 1, 'walkers': 1, 'freely': 3, 'authoritarians': 1, "louise's": 1, 'cadavers': 1, 'perkiness': 2, "simon's": 4, 'hitwoman': 1, '4/10': 4, 'post-marriage': 1, 'geeky': 8, 'hype': 19, 'paratroop': 1, 'updated': 5, 'opulence': 1, "devil's": 12, 'laxative': 1, 'ever-smoldering': 1, 'laleh': 1, 'super-adorable': 1, 'men': 120, 'hiding': 21, 'narrowed': 1, 'nightmarish': 3, 'bonifant': 2, 'glaringly': 2, 'hokey': 19, 'christof': 1, 'hurley': 2, 'disobedience': 1, 'shue': 7, 'onset': 7, 'meddling': 2, 'closets': 1, 'alcott': 2, "jr's": 1, 'bloodthirsty': 2, 'swiping': 1, 'four-day': 1, 'griswolds': 2, 'mentioned': 39, 'quantity': 4, 'shelter/satanist': 1, 'marshal': 2, "t'was": 1, 'straitlaced': 1, 'orwell': 1, 'nathan': 3, 'rubbell': 1, 'dud': 13, 'blocked': 3, 'jude': 1, 'toque': 1, "policeman's": 1, 'kube': 1, 'graduate': 9, 'compassionate': 1, "'funny'": 3, 'wary': 2, 'rears': 2, 'function': 9, 'goldblum': 10, 'freeze': 9, 'formally': 2, 'ribald': 1, 'uhhhhhm': 1, 'cotton': 2, 'wazzoo': 1, 'counter-attack': 1, 'conducting': 2, 'ladyhawke': 1, 'homicide': 7, 'technician': 1, 'drawings': 3, 'outraged': 2, 'new--other': 1, 'laughfests': 1, 'sincerely': 3, 'owners': 4, 'slogs': 1, "sober'": 1, 'half-paid': 1, "television's": 1, "cindy's": 2, 'deviant': 1, 'preoccupied': 1, 'nia': 2, 'accentuate': 1, 'informant': 1, 'stubbornness': 1, 'mnemonic': 3, 'subscribe': 1, 'series': 150, 'champagne': 2, 'hanky': 1, 'low-tech': 2, 'deference': 1, 'chomp': 1, 'magnate': 1, 'camoes': 1, 'surrealism': 3, 'photos': 4, "headmaster's": 1, 'lila': 1, "'80's": 1, 'ankh': 1, "dad's": 2, 'bust': 8, 'babycakes': 1, 'cheech': 4, 'denouncement': 1, 'buildings': 7, 'unmasked': 1, 'brimming': 2, 'reprise': 2, 'polar': 3, 'summed': 6, 'is/are': 1, 'suzie': 4, 'warchus': 1, 'freakiest': 1, "field'": 1, 'carter': 17, 'i+ll': 1, 'bingo': 1, 'convenient': 15, 'prevent': 14, 'over-populated': 1, 'unpredictable': 10, "jim's": 3, "packin'": 1, 'profession': 10, 'hunting': 16, 'speaker': 3, 'persuade': 2, 'riffs': 2, 'hairy': 1, 'dozier': 1, 'mca': 1, "senator's": 1, 'biceps': 2, 'ultra-right': 1, "baldwin's": 3, 'deveu': 1, 'minutes--': 1, 'conquered': 2, 'eggs': 3, 'residue': 2, 'carla': 2, "piller's": 1, 'agree': 30, 'vibrations': 1, "sutherland's": 1, 'obsessively': 4, 'blather': 3, 'character-the': 2, 'pummel': 1, '10%': 2, 'sluggish': 6, 'busty': 4, 'dafoe': 5, 'sprays': 1, "mcmahon's": 2, 'surprising': 25, 'pagan': 1, 'natured': 1, 'craftiness': 1, 'judas': 3, 'killings': 6, 'down-and-dirty': 1, 'tanning': 2, 'rastafarian': 1, 'coupled': 6, 'screenplay': 108, 'referred': 9, 'distressingly': 1, 'clocked': 1, 'protruding': 2, 'master': 30, "classmate's": 1, 'self-aware': 3, 'cadence': 1, "d'angelo": 3, '_offscreen_': 1, 'recycle': 1, 'thoughtless': 2, 'unbridled': 1, 'applause': 2, 'desperado': 1, 'bangs': 2, 'grenkowitz': 1, 'watery': 3, 'schell': 2, 'suggests': 19, 'jokes--otherwise': 1, 'depress': 1, 'bursting': 4, 'grimy': 3, 'mercenaries': 2, "howie's": 1, 'slayer': 4, 'emery': 1, 'patrons': 9, 'juice': 6, 'quintessential': 2, 'reception': 1, 'hide-out': 1, 'chan-film': 1, 'magua': 1, 'mostow': 1, 'spoils': 1, 'regulars': 5, 'blackmailing': 1, 'caddy': 1, 'plan': 67, 'hoodlums': 3, '&#137': 2, "moment's": 2, 'clark': 9, 'affable': 2, 'mass': 10, 'overly-long': 1, 'boards': 1, 'hairstyles': 2, '_twice_': 1, 'gaby': 3, 'holder': 1, 'doormen': 1, "play's": 1, 'bela': 2, 'montana': 2, 'norton': 7, 'unsuitable': 1, 'actor/director': 2, "rocque's": 1, 'slick': 12, 'predestined': 2, 'challenger': 1, 'save-the-world-from-aliens-or-environment-': 1, '94': 2, 'excusing': 1, 'rocketman': 1, "labour's": 2, 'excited': 6, 'un-watchable': 1, 'fleeting': 4, '_before_': 1, 'picard': 3, 'yankovich': 1, 'construction': 3, 'mutants': 1, 'voice-overs': 2, "apple's": 1, 'munchkin': 1, 'blunts': 1, 'tours': 1, 'mentions': 4, '$5': 3, '17th': 5, "how's": 3, 'pesky': 4, 'leaping': 2, 'juggling': 4, 'wonderland': 2, 'good-luck': 1, 'divorces': 3, 'limping': 1, 'rantings': 1, 'falsies': 1, 'bird': 4, 'a+': 2, 'die': 72, 'info': 1, 'daphne': 1, 'nitrite': 1, 'karen': 8, 'defend': 8, 'annoys': 2, 'sunrise': 3, 'neurological': 1, 'airhead': 1, 'kiddie-porn': 1, "yang's": 1, 'benj': 1, 'fatales': 1, 'express': 14, 'ham-fisted': 1, "'usual'": 1, 'vowing': 1, '$45': 1, 'deciding': 7, 'book-skimming': 1, 'victims': 25, 'gunfighter': 1, 'glam-rock': 1, 'discretion': 1, 'australian': 6, 'deadlier': 1, 'daena': 1, 'numbed': 1, 'jefferson': 1, 'belt': 10, 'swayed': 1, 'mildred': 1, 'geyser': 1, 'l-u-v': 1, 'operating': 3, 'mesmerizing': 3, 'redhead': 3, 'bad-ass': 2, 'matheson': 1, 'unifying': 1, 'locals': 6, 'blazing': 6, 'preserves': 1, 'boston-based': 1, '1692': 1, 'treatment': 21, 'investigation': 25, 'over-conceived': 1, 'hairdo': 2, 'fletcher': 2, 'slippery': 1, "impossible'": 1, 'saloon': 1, 'venture': 10, 'stunk': 1, 'boaz': 1, 'conflicted': 1, 'gloriously': 3, 'rocco': 2, 'yulin': 2, 'arguable': 2, 'washes': 1, 'mathilde': 1, '_dirty_work_': 1, 'contributions': 1, 'fireplace': 3, 'separated': 10, 'vomity': 1, 'macabre': 5, 'celebrity': 13, 'buisness': 1, 'drug-dealer': 1, 'jah': 1, 'recording': 1, 'superbly': 4, "christine's": 2, 'purgatory': 2, 'gross': 20, '_hard_ware': 1, 'twang': 1, "chan's": 5, 'lucio': 1, 'luridly': 1, 'overconfident': 1, 'weirdly': 5, 'matilda': 1, 'aaaaaaaaah': 1, 'survivors': 13, 'obscenities': 2, 'well-modulated': 1, 'package': 10, 'lou': 13, 'disbelieved': 1, 'restraint': 9, 'quentin': 7, 'duke': 6, 'dramedy': 2, 'kirstin': 1, 'haddad': 1, 'sparked': 1, 'outdated': 2, 'doggy-style': 1, 'self-deprecation': 2, 'stronger': 14, 'theory': 18, 'superstitions': 1, '3-5': 1, 'fatality': 1, 'macdowell': 5, 'hartman': 3, 'co-written': 4, 'brenner': 6, 'spacey': 4, 'swords': 3, 'relaxing': 2, 'snuff': 7, "hewitt's": 1, 'resort': 5, 'stansfield': 1, 'ricardo': 2, 'shoulder': 7, "assante's": 1, 'adore': 2, 'drifters': 1, 'rating=': 5, '1869': 2, '90': 35, 'airlock': 2, 'pierced': 1, 'titanics': 1, "ribisi's": 2, 'cutesy': 5, 'scuzzy': 2, 'panties': 1, 'tight-lipped': 1, 'halves': 3, 'piggy': 1, 'wagon-train': 1, 'penning': 1, 'free-spirited': 2, '97%': 1, 'methodologies': 1, 'intellect': 4, 'overtaking': 1, 'intersect': 2, 'half-brother': 3, "lanier's": 1, 'cats': 5, 'swashbuckers': 1, 'tch': 1, 'granting': 1, "cover's": 1, 'shane': 8, 'laziness': 2, 'shawnee': 1, 'creatively': 6, 'teammates': 4, 'paddles': 1, 'roundish': 1, 'b-movies': 4, 'coughed': 1, 'device': 34, 'mcshane': 1, 'cheezy': 5, 'wayyyy': 1, 'putting': 25, 'not-for-mail': 1, 'keywords': 1, 'sunshine': 2, 'paperthin': 1, 'tyrell': 3, '1700s': 1, 'hazel': 1, 'tailspin': 1, 'weapons': 21, 'descendent': 1, 'fraction': 4, "delpy's": 1, 'bahia': 1, 'condor': 1, 'joanna': 5, 'troubled': 12, "mancini's": 1, 'supertechnology': 1, 'pickens': 1, 'graded': 1, 'snatch': 1, 'smart-mouthed': 2, 'polson': 1, 'fillings': 1, 'murdoch': 2, 'adventurous': 3, 'florida': 6, 'self-respecting': 3, 'toyed': 1, "leoni's": 1, 'heroize': 1, 'weigh': 2, 'jerod': 2, 'measureable': 1, 'alec': 9, 'molestation': 1, 'globus': 1, 'stud-reporter': 1, 'cleavage-busting': 1, 'judgement': 7, 'machine': 42, 'scapegoat': 1, 'imcomprehinsable': 2, 'oblivious': 5, 'unerotic': 1, 'sank': 2, 'dodges': 1, 'gayness': 1, 'batter': 1, 'robards': 1, 'meetings': 2, 'mini-elwood': 1, 'celeste': 1, 'institute': 2, 'big-league': 1, 'vary': 1, 'go-round': 2, 'bedroom': 5, 'mastermind': 5, 'umpteen': 1, 'achieved': 8, 'penalty': 1, "ships'": 1, 'crow': 10, 'ancient': 12, 'lu': 1, 'snots': 1, 'powerhouses': 1, 'swinging': 6, 'huntingburg': 1, 'double-cross': 1, 'soliloquies': 1, "college's": 1, 'pleasantly': 5, 'elks': 1, 'minivan': 1, 'pris': 1, "danson's": 1, 'light': 54, 'agrees': 23, 'mid-1980s': 1, 'ape-like': 1, 'balthazar': 1, 'backers': 3, 'strait-laced': 1, 'lamppost': 1, 'called': 146, 'molasses': 2, 'firorina': 1, 'maneuver': 4, 'punish': 3, 'priestley': 1, 'women-only': 1, "hearts'": 1, 'brained': 1, 'ensure': 7, 'maureen': 4, 'un-involving': 1, 'keys': 1, 'goatee': 3, 'nancy': 2, 'cynically': 1, 'sinful': 1, 'hacked': 6, 'develop': 23, '1998s': 1, 'suites': 1, 'oppressively': 2, 'successive': 3, 'in-place': 1, "hubby's": 1, 'foyle': 1, 'capped': 1, 'similarly': 7, 'foreheads': 1, 'b-rated': 1, 'niro': 5, 'spends': 47, 'mars': 20, 'futile-attempt': 2, 'marshmallow': 1, 'good-hearted': 5, 'doctorate': 2, 'offend': 5, 'staple': 8, 'inability': 15, 'ceases': 2, 'reliving': 3, 'solo': 6, 'housewives--that': 1, 'cheesiness': 3, 'ditz': 1, 'odgen': 1, "hammer's": 1, 'ultra-expensive': 1, 'looney': 3, 'slackers': 2, 'impressionist': 1, 'approaches': 9, "beatles'": 2, 'mannequins': 4, 'mergers': 1, 'horde': 1, 'hardness': 1, 'lowlives': 1, 'ponytails': 1, 'adored': 1, 'predicament': 2, 'type': 67, 'snobbish': 1, 'britt': 1, 'maguire': 3, 'villa': 1, 'shalt': 1, 'psychopath': 5, 'protocol': 1, 'commitments': 1, "badgers'": 1, 'charred': 1, 'irresponsible': 1, 'hermit': 1, "1960's": 4, 'cooperation': 2, 'eiko': 1, 'coke-addled': 1, "snipes'": 2, 'supports': 1, 'classier': 1, 'drix': 2, 'sucks': 21, 'weekly': 4, 'benton': 1, 'respect': 33, 'kirbys': 1, "whut're": 1, 'uh-huh': 3, 'edson': 1, "em'": 1, "`there's": 1, 'isuro': 1, 'call': 82, 'suspicious': 16, 'beavis': 2, 'brynner': 1, 'masks': 7, "double-d's": 1, 'drugged': 4, 'drawl': 1, 'anguishes': 1, "diner'": 1, '_monster_movie_': 1, 'offed': 1, 'excuse-someone': 1, 'gps': 1, 'toyko': 1, 'attracts': 3, 'palate': 2, 'that--energy': 1, 'dallas': 8, "nun's": 1, 'drains': 1, 'publisher': 2, 'swagger': 1, 'brenda': 5, "ulee's": 1, "movies-i've": 1, 'blinking': 1, 'sub-theme': 1, 'bujold': 2, 'alfred': 11, 'theatres': 7, 'pocahontas': 2, 'soto': 1, 'macgowan': 1, 'mpd': 1, 'actor/director/producer': 1, 'spitting': 1, 'dateline': 1, 'olde': 1, '1hr': 1, 'audacity': 2, 'popularity': 15, 'paralyzed': 2, 'liven': 5, 'gut-busting': 1, "iliff's": 1, 'sidelines': 1, 'badgers': 1, 'eroticize': 1, 'headed': 25, 'hunky': 6, "cox's": 3, 'iii--though': 1, 'biting': 4, 'meditation': 2, 'fast-moving': 1, 'destruction': 14, "cabbie's": 1, 'koenig': 1, 'intent': 14, 'year': 167, 'assures': 2, 'kilo': 1, 'pre-pubescent': 1, 'remorse': 2, 'clinical': 2, 'cherot': 1, 'qualified': 1, 'palpable': 2, 'squeaky': 2, "joshua's": 2, 'fonda': 6, 'alma': 1, 'exclaiming': 2, 'chechik': 2, 'aspiring': 13, 'post-production': 4, 'problem': 180, 'weisz': 2, 'segue': 1, 'constraints': 2, 'shy': 9, 'aformentioned': 1, "australia's": 2, 'swedish': 1, 'corridor': 1, 'somerset': 1, 'appetizing': 1, 'premeditated': 1, 'alcatraz': 1, 'wristwatch': 3, 'temptations': 2, 'under-appreciated': 1, 'perils': 1, 'bloodlines': 1, 'open-armed': 1, "park's": 1, 'ker': 1, 'hana': 1, 'falseness': 1, 'house-ghost': 1, '_saved_by_the_bell_': 1, 'teresa': 3, 'perks': 2, 'intimidation': 1, 'kickstarting': 1, 'extent': 19, 'loquacious': 1, 'selection': 6, 'assasins': 1, 'mattes--leave': 1, 'embraced': 3, 'moralistic': 1, 'havoc': 12, "nelkan's": 1, 'hired': 27, 'published': 1, 'stifles': 1, 'videographer': 1, 'repressed': 3, 'aussies': 1, 'assaulted': 3, 'out-of-shape': 1, 'fusing': 1, 'it--i': 1, 'redheads': 1, 'block': 14, 'exist': 31, 'crimes': 8, 'sounding': 12, 'decides': 73, 'grants': 1, 'problematic': 7, 'beaten': 13, 'ohlmyer': 4, 'judson': 2, 'still-lively': 1, 'residence': 4, 'dummy': 1, 'brawls': 1, 'and/or': 14, 'page-turner': 1, 'mescaline': 1, 'barber': 6, 'white-bread': 2, 'admitted': 4, 'baron': 3, 'synthetic': 4, 'penalosa': 1, "ba'ku": 2, 'romps': 1, 'x-philes': 2, 'body': 90, 'biologist': 8, "monster's": 1, 'kar-wai': 1, 'manse': 1, 'believably': 1, 'discover': 44, 'guise': 4, 'processes': 1, 'foreshadows': 1, 'amidst': 9, 'vows': 8, "fall's": 1, 'oneida': 1, 'stumbles': 21, "code'": 1, "`sabo'": 1, 'long-missing': 1, 'alessandro': 2, 'postman': 5, 'tomes': 1, 'rubbing': 1, 'little-seen': 2, 'wet': 10, 'soft-pedaled': 1, 'underweight': 1, 'tastes': 6, 'recapture': 5, 'underside': 1, 'duvall': 6, 'cage': 28, 'choudhury': 1, 'yea': 5, 'rhea': 4, 'ex-brooklynite': 1, 'report': 10, 'stooped': 1, 'critters': 1, 'vessel': 8, 'abysmal': 9, 'launching': 3, "'serve": 1, 'break-into-the-building': 1, 'then-colleagues': 1, 'uncover': 10, 'advisor': 2, 'reteaming': 1, 'mcnugent': 1, "'bots": 1, 'nine-feet': 1, 'malcom': 1, 'hood': 12, 'over-done': 1, 'silvers': 2, 'rumba': 1, 'pat-on-the-back': 1, 'partake': 1, 'glob': 2, 'tradition': 16, 'jaw-': 1, 'differentiate': 3, 'high-technology': 1, 'nescafe': 1, 'hollywood': 169, 'what--': 1, "spicer's": 1, 'flights': 2, 'flea': 1, 'excepting': 1, 'eat': 23, 'sneaking': 4, '9-year': 1, 'then-patrick': 1, 'extra': 21, 'pyrotechnics': 1, '_pick_chucky_up_': 1, 'arcadia': 1, 'jovial': 1, 'verges': 1, 'straight-to-video': 4, 'ubiquitous': 1, 'ferocity': 3, 'dalle': 1, 'stanton': 3, 'jen': 1, 'lakes': 2, "jjaks'": 1, 'mogul': 5, 'pseudonym': 7, 'descends': 3, 'experiments': 6, 'revolting': 2, 'jack': 59, "homage'": 1, 'spray': 5, '1979': 4, 'chad': 1, 'tensions': 3, 'bathgate': 1, 'nookie': 1, 'singles': 3, "brilliance--baker's": 1, 'tpm': 1, 'dracula': 5, 'corridors': 5, 'anticipated': 6, 'larson': 3, 'lighter': 2, 'undefined': 1, 'spiers': 4, 'vetter': 1, 'things--a': 1, 'do-whatever-it-takes': 1, '15-week': 1, 'motif': 2, 'multitudes': 1, 'fitzpatrick': 1, 'mysogny': 1, 'blockade': 1, 'pyroclastic': 1, 'hatosy': 3, 'gore-drenched': 1, 'zesty': 1, 'sizemore': 2, 'litter': 1, 'roles--dedicated': 1, 'carefree': 2, 'jackets': 2, 'movie-star': 1, 'ove': 1, 'invaded': 2, 'confusion': 17, 'undefeatable': 1, 'balk': 5, 'graph': 1, 'watered': 1, 'pizzazz': 1, 'unconnected': 1, 'power-mad': 1, 'milos': 1, 'flirtation': 1, 'distorts': 1, 'big-': 1, 'correct': 11, 'beaudoin': 1, 'uma': 6, 'deputy': 2, 'witchcraft': 3, 'novikov&#137': 4, "ermey's": 1, 'comic-book': 1, 'defenses': 1, '16-year-old': 1, 'dread': 1, 'prominently': 8, 'hep': 1, 'calculations': 1, 'bait': 6, 'celebrated': 1, 'sitcom-airiness': 1, 'highlight': 20, 'alexandre': 2, 'suppressed': 1, 'buffoonish': 2, 'simpatico': 1, 'inventive': 15, "losers'": 1, 'dilate': 1, 'etiquette': 1, 'dumbest': 7, 'heroically': 1, 'colored': 2, 'speed-reading': 1, 'spoke': 3, 'crazed': 13, 'ebay': 1, 'fatally': 4, 'fidget': 1, "brown's": 2, 'labeling': 3, 'non-acceptance': 1, 'at--revenge': 1, 'shabby': 5, 'abandoning': 1, 'garner': 5, "richelieu's": 1, 'manila': 1, 'imitating': 4, "kelley's": 1, 'postulated': 1, 'rigged': 2, 'dorian': 1, 'counterfeit-looking': 1, 'teddy': 5, 'moods': 2, 'romanian': 1, 'restraining': 1, 'cabel': 2, 'corpse-rotting': 1, 'creed': 1, 'romy': 3, 'low-key': 3, 'jilted': 1, "blawp's": 1, '8-year-old': 1, 'snatching': 1, 'split-second': 2, 'merrick': 1, 'perish': 1, 'action': 266, 'external': 3, 'arquettes': 1, 'crashes': 8, 'participant': 1, 'expectations': 28, 'intensively': 1, 'ensemble': 15, 'ex-': 2, 'adult': 29, 'weena': 1, 'inspirational': 3, 'dogma': 8, 'thick-headedness': 1, 'lip-reading': 1, 'aimless': 4, 'don': 22, 'environmental': 1, 'cineplexes': 1, 'oppressive': 4, 'colleague': 6, 'treats': 14, 'lighthearted': 3, "story's": 8, 'retake': 1, 'cancer': 8, 'neighbor': 9, 'introductory': 2, 'coalesce': 1, 'wednesdays': 1, '36-hour': 1, 'slowed': 4, "2000'": 1, 'out-terrorize': 1, 'fraker': 3, 'stalking': 6, 'bastion': 1, 'listens': 4, 'spiteful': 3, "gibson's": 3, 'protracted': 3, 'authored': 2, 'toting': 3, 'concussion': 1, 'adorned': 1, '$2': 3, 'lame-brained': 4, 'conjunctions': 1, 'movies--a': 1, 'com/moviefan983/moviereviewcentral': 1, 'bierko': 1, 'fails': 89, "herbert's": 1, 'strapping': 3, 'tones': 4, "'collection": 1, 'elizabeth': 21, 'returned': 14, 'randy': 22, 'jnr': 1, 'chew': 3, 'dearly': 2, 'fever': 6, 'dupr': 1, 'merits': 7, 'uncrowned': 1, 'ferrari': 1, 'sit': 64, 'making': 190, 'sheep-pig': 1, 'brainiac': 1, 'choderlos': 1, 'faded': 2, 'religiously': 1, 'cue': 12, 'silent': 21, 'embittered': 2, 'flannery': 3, 'harboring': 2, 'storm-swept': 1, 'payne': 5, 'tinto': 1, 'interpret': 1, 'hugging': 1, 'pastiche': 1, 'rewound': 1, 'calming': 1, 'solondz': 1, 'blip': 1, 'three-star': 1, 'chimpanzee': 4, 'firm': 9, 'permissive': 1, 'law': 40, "rubik's": 1, 'pills': 2, 'sheep': 5, 'depiction': 4, 'stuntwork': 1, 'jamaica': 2, 'choreographer': 3, 'outages': 1, 'burden': 2, 'volunteer': 2, 'boing': 1, 'playful': 4, 'life-threatening': 2, 'repairs': 1, 'ariel': 1, 'dotted': 1, 'takehiro': 1, 'inflames': 1, 'west/cpw': 1, 'friendliness': 1, 'exercises': 6, 'naughton': 1, 'baldly': 1, 'hybrid': 2, 'netherworld': 1, 'shyster': 1, 'gigabytes': 1, 'shale': 1, 'shifty': 1, 'canna': 1, 'paleontology': 1, 'emerged': 2, 'valerie': 5, 'birthed': 2, 'outnumber': 1, 'arc': 10, 'vhs': 3, 'hamster': 2, 'aesthetic': 2, 'endeavor': 1, 'mower': 2, 'burglary': 4, '10s': 1, 'writers--': 1, 'voyager': 1, 'phosphorus': 1, 'under-age': 1, "d'onofrio": 4, "tourette's": 1, 'reindeer': 5, 'tapioca': 1, 'tokyo': 4, '_full_house_': 1, 'eventual': 4, 'pearce': 1, 'refers': 9, 'hurrah': 1, 'first-person': 1, 'caped': 2, 'pettiford': 1, 'packing': 1, 'docu-style': 1, 'bleak': 15, 'resources': 10, 'has-been': 1, 'spent': 56, 'pimlico': 1, 'cataloguing': 1, 'contrastingly': 1, 'bimbo': 7, 'prodigal': 1, "cooper's": 3, 'messily': 1, 'comprises': 1, 'steele': 2, 'communities': 1, 'screwing': 7, "hope's": 1, 'laps': 1, 'negotiator': 3, 'wavers': 1, 'percolating': 1, 'decor': 1, 'applegate': 2, 'controlled': 11, 'stimulate': 1, '_clueless_': 1, 'mazar': 1, 'monster': 35, 'long-suffering': 1, 'afford': 2, 'wimmin': 1, 'drawing': 9, "beaver_'s": 1, 'julio': 1, 'theatre': 26, 'pacific': 10, 'degraded': 1, 'anxiously': 1, 'hounsou': 1, 'expo': 1, 'charasmatic': 1, 'penchant': 8, 'manchu': 1, 'lifelessly': 1, 'pathos': 4, 'album': 4, 'hard-edged': 1, "gaby's": 1, 'peoples': 5, 'we\x12ll': 1, 'relied': 2, "crawford's": 2, 'empathise': 1, 'appropriately-named': 1, 'wasteful': 3, 'alligator': 1, 'queen': 30, 'spoilers': 13, 'complain': 5, 'bounce': 7, 'pre-fabricated': 1, 'honeymoon': 4, 'f-ing': 1, 'never-ending': 1, 'steer': 3, 'disclosure': 2, "naville's": 1, 'stocked': 1, 'milhoan': 1, 'stoppard': 1, 'quick': 58, 'sparring': 1, 'deferrential': 1, 'wild-and-crazy': 1, 'trawls': 1, 'criticize': 3, "catherine's": 1, 'leia': 2, 'executed': 24, 'fiercer': 1, 'western/sci-fi': 1, "halicki's": 1, 'wasteland': 7, 'posting': 1, 'penned': 11, 'believed': 13, "original's": 4, 'outbursts': 2, 'vertigo': 2, 'carrie': 11, "'deep": 1, 'gio': 1, 'snaggleteeth': 1, 'ensuing': 6, 'console': 1, 'buckaroo': 1, 'shirt': 6, 'indifferently': 1, 'theroux': 1, 'tough-as-nails': 1, 'er': 10, 'skeptical': 4, 'evan': 2, 'folktale': 1, 'transpired': 3, 'franchise': 20, 'hustler': 2, 'beat-a-dead-horse-into-glue': 1, '\\': 2, 'odette': 4, 'dunces': 1, 'arts-type': 1, 'uuuhhmmm': 1, 'daylights': 1, 'political': 22, 'company': 57, 'glorious': 6, 'lister': 1, 'popular': 71, 'poirrier-wallace': 1, 'match': 30, 'recognizeable': 1, 'boyfriend': 31, 'tracks': 15, 'jaffa': 1, 'ticks': 1, 'scrupulous': 1, 'pastels': 2, 'sweater': 1, 'elevator': 4, 'jazzed': 1, 'crook': 6, 'reunion': 7, 'swarmed': 1, "beltrami's": 1, 'law-enforcement': 1, 'deadening': 1, 'midst': 11, 'youths': 3, 'million': 57, 'elaborated': 1, 'waddling': 1, 'jamey': 1, 'jarrell': 1, 'tusks': 1, 'gals': 3, 'hoisted': 1, "var's": 1, 'poolman': 1, 'inkpot': 6, 'palma': 6, 'retread': 5, 'saint': 4, "'98": 2, 'ore': 1, 'rains': 3, 'disappointments': 6, 'misogynist': 1, 'unstoppably': 1, 'surfacing': 3, "katana's": 1, "vonnegut's": 1, 'academy': 26, 'charmed': 4, 'barnes': 10, 'deathblows': 1, 'identital': 1, 'rightfully': 1, 'inches-from-death': 1, 'unsuspecting': 10, 'glimcher': 1, 'aniston-as-love-interest': 1, "movie'": 1, 'berliner': 1, 'dresser': 1, 'food-fate': 1, 'streetfighter': 1, '83': 2, 'teamwork': 1, "dora's": 1, 'obsesses': 1, 'cornell': 2, 'steelworks': 1, 'incriminates': 1, 'impromptu': 2, 'equates': 2, 'dies': 41, 'iconic': 2, 'shuts': 1, "kilmer's": 3, 'reservations': 3, 'burger': 4, 'berlin': 1, 'disagrees': 2, 'yaz': 1, "`pump-bitch'": 1, 'cares': 40, 'typically': 8, 'proft': 2, 'deepest': 2, 'interactions': 5, 'chun': 1, '230': 1, "dawn't": 1, 'insulators': 1, 'meg': 13, 'dohlen': 1, '_double_team_': 1, 'commercials': 17, 'brunette': 3, 'shadowing': 1, 'targetted': 1, 'girl/aspiring': 1, 'poster': 10, 'somber': 3, 'uncomfortably': 2, 'forsythe': 4, 'royce': 1, 'co-producing': 1, 'saddled': 11, 'chang': 2, 'line': 147, 'enrolls': 1, 'shot-for-shot': 2, 'spacewalk': 1, 'competitors': 1, 'yards': 5, 'geared-down': 1, 'deceitful': 1, 'overlook': 6, 'resolute': 1, 'hubby': 2, 'conceal': 3, 'southerner': 1, 'undeveloped': 5, 'divvy': 1, 'skinner': 2, "juliet'": 2, 'fright-free': 1, 'cheeks': 2, 'enhances': 1, 'yer': 1, 'twin': 16, 'shipmate': 1, 'heartbreaking': 3, 'ardor': 1, 'preppies': 2, 'paralells': 2, 'holloway': 1, 'erection': 1, '1923': 1, 'trashiness': 2, 'sean': 34, 'outtakes': 3, 'hints': 11, 'woodbine': 3, 'mistimed': 1, 'frisky': 3, 'goon': 1, 'cavemen': 4, 'peet': 5, 'nouvelle': 1, 'spattering': 1, '58': 1, "`disappointing'": 1, 'tuesday': 3, 'vanessa': 7, "manager's": 1, '_saving': 2, 'baseball': 14, 'avid': 4, 'clan': 5, 'deconstruct': 1, 'impotence': 1, 'rubrics': 1, 'mpaa': 2, 'dramatic': 58, "heaven's": 5, '-style': 1, 'superglues': 1, 'wiggles': 1, 'treating': 6, 'mechanisms': 1, 'hendrix': 1, 'manned': 3, "mother's": 6, 'nicolo': 1, 'bateman': 1, 'luminous': 3, 'papers': 3, 'fiennes': 6, "pacino's": 2, 'prying': 1, "davidson's": 1, 'rattlesnakes': 1, 'embarks': 2, 'nearly-inconsequential': 1, 'spectral': 1, 'bumper': 1, 'callous': 1, 'havisham': 1, 'self-proclaimed': 2, 'uh': 8, 'behaviour': 1, 'bevy': 1, 'telltale': 2, 'correctness': 1, 'pilleggi': 1, 'torments': 1, 'neatness': 1, 'earnestness': 3, 'west': 34, 'porsche': 1, "banderas's": 1, 'x-rated': 1, 'knockin': 1, 'cad': 3, 'pink': 5, 'receipts': 1, 'stricken': 3, 'tasted': 1, 'snyder': 1, "gregio's": 1, 'tenure': 1, 'audible': 6, 'beheaded': 1, 'egyptian': 1, 'admirer--': 1, 'vicellous': 1, 'studs': 1, 'enterprises': 1, 'headstrong': 1, 'xavier': 1, '5-million': 1, 'swindlers': 1, 'loop': 2, 'stylishness': 1, 'heldenberger': 1, 'stalk-and-slash': 1, 'eve': 24, 'applying': 1, 'peeling': 2, 'frowns': 1, 'kim': 19, "lenz's": 1, 'dowling': 2, 'warnings': 4, 'semi-subplot': 1, 'marked': 10, 'codger': 1, "state's": 1, 'cuckoos': 1, 'black-tie': 1, 'tidbit': 1, 'cliche-ridden': 2, 'preciously': 1, 'collins': 2, 'period': 23, 'donahue': 1, 'jobs': 19, 'sports': 31, 'ilsa': 1, 'out-doing': 1, 'identified': 3, 'disallows': 2, 'bio': 1, 'waist': 2, 'greeted': 2, "runner's": 1, 'colonists': 2, 'rose': 17, "hero's": 4, 'zooms': 6, 'spector': 1, 'wise-cracking': 3, 'dull--': 1, 'thorugh': 1, 'oblivion': 7, 'vanity': 7, 'partner-in-crime': 2, 'realists': 1, "13'": 1, 'maniacs': 1, 'hard-as-nails': 1, 'trades': 2, 'racked': 2, 'liable': 3, 'screenplays': 6, 'farewell': 1, 'crimefighting': 1, 'tho': 2, 'whoops': 5, 'wrestler': 5, "clerk's": 1, 'transvestite/medium': 1, 'bernt': 1, 'fran': 1, 'dueling': 1, 'tasks': 3, 'seduce': 5, 'bafta': 1, "each's": 1, 'yellow': 7, 'cover-up': 6, 'horner': 2, 'showing': 57, "moxxon's": 1, 'delicately': 3, 'daddy': 11, 'leaps': 12, "ancestor's": 1, 'actress-type': 1, 'rug': 3, '10-foot': 1, 'aniston': 4, 'hi-def': 1, 'redux': 1, 'big': 298, 'hommage': 1, 'saw--at': 1, 'typos': 1, "lefty's": 1, 'hirsch': 2, 'war_': 1, 'cheri': 1, 'textbooks': 1, 'matrimony': 1, 'uninformed': 1, 'auctioned': 1, 'ambiguously': 1, '48th': 1, 'chinese': 19, 'fulfilling': 6, 'steeped': 1, 'purrs': 1, 'doyle--at': 1, 'riot': 8, 'aloof': 4, 'backlit': 1, 'greeting': 4, 'makers': 25, 'capabilities': 3, 'cavernous': 3, 'befuddled': 6, 'tenement': 1, '1977': 4, 'indulgence': 1, 'clean': 24, 'vow': 4, 'uprooting': 1, 'abounding': 1, 'jeff': 27, 'goldie': 4, 'quarrelling': 1, 'chariots': 1, 'ex-cop': 6, 'reloading': 1, 'memoirs': 1, 'resentment': 2, 'chronicling': 2, 'university': 13, 'reverting': 1, 'journeys': 2, '--but': 2, 'punched-up': 1, 'programmers': 1, 'facetiousness': 1, 'exteriors': 2, 'warlord': 1, 'heald': 4, 'williams': 43, 'burnt': 1, "friggin'": 2, 'blatantly-do': 1, 'gangly': 1, 'omniscient': 2, 'mama]': 1, 'eckhart': 2, 'rethought': 1, "sean's": 1, 'redheaded': 1, 'lively': 10, 'precedent': 1, 'dazzled': 1, 'filtered-light': 1, 'combine': 4, 'faring': 2, 'caper': 6, 'follow': 52, 'liner/haunted': 1, 'surrealistic': 3, 'voting': 2, 'turbulence': 2, 'putrid': 6, 'fat': 30, 'obstacles': 7, 'khondji': 3, 'thirst': 2, 'faultless': 1, 'fleeing': 7, 'basis': 24, 'simon': 21, '$4': 1, 'rushed': 7, 'livingrooms': 1, 'spite': 8, 'player': 26, 'soliloquy': 1, 'acquire': 1, 'pubs': 1, '_more_': 1, 'dahlings': 1, 'suspenseless': 2, 'delusions': 2, 'flatliners': 1, 'unheard': 1, 'chapelle': 2, 'convincing': 50, 'chinatown': 8, 'meantime': 10, 'landed': 3, "banderas'": 1, 'mainstream': 17, 'sustaining': 2, 'physician': 3, 'thrax': 2, 'alias': 2, 'populace': 1, 'poppins': 2, 'phoniest': 1, 'schulman': 1, 'assigned': 10, 'belch': 1, 'adulterer': 1, 'understatement': 3, 'swan': 2, 'skimps': 1, 'feminine': 2, 'language': 31, 'understandably': 3, 'genevieve': 1, 'records': 6, 'neckties': 1, 'poorly-motivated': 1, 'casualty': 1, 'sketchily': 1, 'dish': 4, 'formless': 1, 'ex-cellent': 1, 'jordon': 1, 'harvest_': 1, 'shandling': 3, 'fatal': 18, "randle's": 1, 'not--she': 1, 'died': 30, 'honours': 1, 'operandi': 1, 'hookers': 3, 'hyde': 6, 'pigeon': 2, 'ignite': 2, 'rafe': 1, 'impart': 1, 'tests': 8, 'unsubtle': 2, 'fad': 1, 'destitute': 2, 'tightened': 1, 'retaining': 1, 'trainload': 1, 'zany': 13, 'bumfuck': 1, 'supersecret': 1, "other's": 8, 'intensely': 4, 'numbing': 1, 'blocking': 1, "shandling's": 2, 'enlist': 5, 'awed': 1, 'tries--then': 1, 'inheriting': 1, 'events': 67, 'threw': 3, "yakin's": 1, 'attain': 2, 'insuring': 1, "fred's": 1, "creek'": 1, "hart's": 1, "linklater's": 1, 'hedaya': 4, 'town': 71, 'benevolent': 5, 'deconstructing': 2, 'grunting': 6, 'hammerbottom': 1, 'theodore': 1, 'dissolve': 1, 'marriage': 21, 'daytime': 2, 'warn': 7, 'assuredly': 1, 'curfew': 2, 'arena': 3, 'focal': 1, 'mccallum': 1, 'racks': 2, "residents'": 1, 'downbeat': 3, 'cara': 1, 'full-scale': 1, 'roach': 1, 'libero': 1, 'mcglone': 2, 'suprisingly': 2, 'spoiling': 2, 'implants': 1, 'standards': 14, 'wait': 70, 'treill': 1, 'nature': 50, 'knothole': 1, 'vaughn': 3, 'country': 37, 'whitney': 1, 'evening': 17, 'confirmation': 2, "haven'": 1, 'canadians': 1, 'guilty': 35, 'waltz': 3, 'goofier': 2, 'yakov': 1, '#19': 1, '`let': 2, 'continual': 2, 'always-broke': 1, 'slide': 4, "marshall's": 2, 'hathaway': 1, 'what+s': 1, 'sacha': 1, 'em': 3, 'svengali-like': 1, "adele's": 1, 'holidays': 2, 'stott': 2, 'dough': 4, "spacek's": 1, 'nods': 2, 'nostril': 1, 'admired': 1, 'mumps': 1, 'declined': 3, "coleman's": 1, 'capturing': 6, 'mounts': 2, "helen's": 3, 'dealers': 6, 'romantic-comedy-action': 1, 'mcdiarmid': 1, 'sub-standard': 1, 'pre-occupied': 1, 'vegetables': 1, "unabomber's": 1, 'taut': 1, 'moments--prologues': 1, 'sutra': 1, 'mk2': 1, 'consumption': 1, 'dropouts': 1, 'housewives': 1, 'resurrection': 4, 'nailing': 2, "lynch's": 2, 'full-time': 1, 'manipulate': 4, 'leisure': 3, 'differs': 4, 'shucks': 1, 'aww': 1, 'swishes': 1, 'syllables': 1, 'judith': 5, 'budgie': 1, 'limey': 2, 'foreboding': 2, 'binoche': 1, 'von': 4, 'spawn': 9, 'lunches': 1, 'imperative': 2, 'converge': 1, 'overalls': 1, 'conehead': 1, 'daredevils': 1, 'beacon': 3, 'elegiac': 1, 'evils': 2, "1997's": 14, 'slot': 2, 'unapologetic': 1, 'druggie': 1, 'non-spoiler': 1, 'time-to-time': 3, 'glossy': 5, 'outre': 1, 'alleged': 4, 'performaces': 1, 'inaccurately': 1, 'tape': 16, 'identifying': 3, 'blurb': 1, 'ensue': 5, 'debt': 4, 'allison': 1, 'trashiest': 1, 'snag': 1, 'yokel': 1, 'pharmacist': 1, 'penetration': 1, 'curtis-hall': 1, 'relation-': 1, 'hugs': 1, '40-year-old': 1, 'spirited': 3, 'teary': 2, 'escape-as-quickly-as-you-can': 1, 'cauldron': 2, 'firestorm': 3, 'york': 70, 'nuptials--to': 1, 'trolley': 1, 'prove': 39, 'expresses': 1, 'yucked': 1, 'depravation': 1, 'nora': 7, 'brotherhood': 1, 'lame-ass': 1, 'chicks': 3, 'preventing': 4, 'crutch': 1, 'troma': 1, 'attendent': 1, 'hoped': 9, "gere's": 2, 'unequivocally': 1, 'pacify': 1, 'salesman': 6, 'consecutive': 6, 'excites': 2, 'katherine': 5, 'clocks': 2, "guy's": 8, 'good-': 1, 'wholeheartedly': 4, 'rko': 1, 'butch': 2, 'transforming': 2, 'headings': 3, 'return--': 1, 'meticulously': 3, 'deteriorates': 3, "`blissfully-confused'": 1, 'gallup': 1, 'faked': 2, 'memories': 23, 'sunset': 1, 'slut': 4, 'beth': 4, 'pointy': 1, 'double-entendres': 1, 'bog': 2, 'slaughtered': 7, 'determine': 12, 'complaint': 5, 'idiosyncratic': 1, 'murdering': 8, 'josiane': 1, 'mimi': 5, 'witty': 23, 'feud': 2, 'maze': 1, 'partridge': 1, 'reifsnyder': 2, 'resonance': 6, 'damme': 13, "lima's": 1, 'dunaway': 2, 'appreciation': 1, 'marcelli': 1, 'gto': 1, 'jean': 17, "leone's": 1, 'only--and': 1, 'wiseacre': 2, 'chestnuts': 1, 'chushingura': 1, 'else--perfect': 1, 'imaginations': 1, 'cafeteria': 3, 'brightly': 1, 'plotholes': 1, 'protester': 1, 'savvy': 3, 'annabella': 2, "reader's": 2, 'psychic': 9, 'unrightfully': 1, 'sliced': 3, 'entrepreneurship': 1, 'vitro': 1, 'mushy': 1, 'kasdan': 2, 'passive': 5, 'quick-flash': 1, "woods'": 2, 'violence': 90, 'fisherman-victim': 1, 'correctly': 9, 'kurusawa': 1, 'pushups': 1, 'distinction': 8, 'volunteering': 1, 'benson': 2, 'giddy': 3, 'repulse': 1, 'director/screenwriter': 2, 'reliable': 13, 'beatles': 1, "loggia's": 1, 'killlers': 1, 'motifs': 1, 'meatloaf': 1, "chucky's": 1, 'disorganised': 2, 'critically-acclaimed': 2, 'winches': 1, "kubrick's": 4, "1950's": 3, 'enormous': 18, 'gambler': 4, 'paradiso': 1, 'vexatiousness': 1, 'thankless': 7, 'realising': 3, 'plunges': 3, 'immortalized': 1, 'strategically': 1, "lawman's": 1, 'plausibility': 2, 'lookalike': 1, 'reneges': 1, 'yun': 1, 'sivero': 1, 'fuels': 1, 'nod': 6, "owner's": 1, 'beauteous': 1, 'scriptwriter': 3, 'confuse': 4, '_i_know': 1, 'un-led-up': 1, 'missiles': 3, 'surfaces': 2, 'gypsy': 2, 'down-on-her-luck': 1, 'bondish': 1, 'one-sided': 4, 'boulle': 2, 'saxon': 2, 'libraries': 1, 'pedigree': 2, 'mole': 1, 'fired': 17, 'inevitable': 23, 'thrill': 13, 'mountain': 17, '1981': 14, 'frederic': 1, 'embark': 1, 'unthinkable': 1, 'perfume-drenched': 1, 'adorn': 1, 'impaled': 1, 'shudder-inducing': 1, 'dummies': 3, 'serbia': 1, 'ian': 15, 'cell-phone-guy': 1, 'unashamedly': 2, 'mccarthy': 2, "dern's": 1, 'close-ups': 17, 'head-to-head': 2, 'knuckleheaded': 1, 'true-crime': 1, 'lil': 2, 'apon': 1, 'preist': 1, 'extract': 2, 'sensibilities': 5, 'spells': 1, 'pericles': 1, 'chips': 1, 'wallowing': 2, 'sphinx': 1, 'unspectacular': 1, 'supply': 10, 'rhetoric': 2, 'helmets': 2, 'newly': 5, 'stifling': 1, 'conception': 2, '200+': 1, 'requirement': 2, 'wreckage': 4, "ebert's": 5, 'judd': 9, 'inebriation': 1, 'latitude': 1, 'squares': 1, 'ass': 41, 'posh': 5, 'don+t': 1, 'set-piece': 2, 'obviously-inserted': 1, 'resulting': 15, 'sentiment': 6, 'propel': 7, 'grunted': 1, 'mandatory': 4, 'destroying': 13, 'loves': 27, 'dust': 7, 'gratingly': 1, 'squealing': 1, 'rugrats': 2, 'connections': 5, 'likable': 30, 'ivins': 1, 'oteri': 1, 'twitch': 1, 'accomplice': 4, 'arguably': 10, 'katharine': 1, 'undersea': 1, 'carrey': 13, 'sub-genre': 3, 'arabella': 1, 'ranch': 3, 'winnebago': 1, 'verbal': 4, 'still-living': 3, 'foundation': 3, 'congressional': 1, 'rubbery': 1, 'clumsiness': 2, 'olympics': 1, 'ticking': 3, "richards'": 1, 'fontenot': 1, 'faction': 1, 'plimpton': 4, "'kidnapped'": 1, 'belittling': 1, "tis'": 1, 'children_': 1, "finale's": 1, 'whisked': 1, 'broker': 1, 'veritable': 5, 'junkies': 3, 'plod': 3, 'unused': 1, 'copper': 1, 'glenda': 3, 'detracting': 1, 'teams': 9, '2029': 2, 'devouring': 1, 'parodies': 5, 'lane': 8, 'sewers': 1, 'highlander': 6, 'secretaries': 1, "can'": 1, 'stayed': 13, 'fabled': 3, 'un-noteworthy': 1, "sonia's": 1, 'bernie': 1, 'ricochet': 1, 'butchered': 3, 'iron-on': 1, 'orlando': 3, 'schroeder': 1, 'girlie': 1, 'histories': 2, 'holy': 9, "schoolboy's": 1, 'makeover': 3, 'shattered': 4, 'high-power': 1, 'un-flossed': 1, 'smiley': 1, 'rienfenstal': 1, 'alerted': 1, 'showers': 3, 'restrain': 4, 'scenery-chewing': 1, 'sorcerer': 4, 'ears': 6, 'cary': 2, 'mid-week': 2, 'responded': 2, 'elected': 1, '____': 1, 'entire': 132, 'stupefying': 3, 'obliterating': 1, 'camo': 1, 'cobra': 1, 'relationships-from-hell': 1, 'individually': 1, 'unreasonable': 4, 'shaved': 2, 'unconcious': 1, 'crippling': 1, 'cds': 3, 'newlyweds': 1, 'estate': 5, 'shortages': 1, "died'": 1, 'relegated': 8, 'tactic': 3, 'coitus': 3, 'orgasmically': 1, 'poisoning': 1, 'handsomely': 1, 'comatose': 2, 'barbara': 9, 'elwes': 3, 'spout': 1, 'maintains': 2, 'quasi-utopian': 1, '_2001_': 1, 'bomb': 32, 'racoon': 2, 'spotlighted': 1, 'uptight': 7, 'destroys': 12, 'singapore': 1, 'finnegan': 4, 'effects-filled': 1, 'rolf': 2, 'paragliding': 1, 'incapacitate': 1, 'justification': 5, 'guerrillas': 1, 'resolution': 9, 'model': 21, 'learns': 40, 'hoisting': 1, 'authorities': 8, 'lens': 3, 'illegal': 13, 'mid': 4, 'dwelling': 4, 'id4': 3, 'animator': 2, 'parks': 3, 'intrude': 2, 'fumes': 1, 'thing': 286, 'offset': 1, 'mena': 10, 'advancing': 2, 'stardust': 1, 'lace': 1, 'praying': 5, 'ruber': 1, 'detector': 1, 'lucent': 1, 'pakula': 2, 'club/act': 1, 'self-contained': 1, 'advancements': 1, 'basquiat': 1, 'gutenberg': 1, 'suicide--bad': 1, 'career-minded': 2, 'serial': 20, 'commencement': 1, 'edges': 1, 'banker': 1, 'sacked': 2, 'slightly': 48, 'porthos': 3, 'knicked': 1, 'potty': 2, 'loosest': 1, 'bonfire': 2, "prostitute's": 1, 'three-quarters': 4, 'mite': 1, 'illuminati': 1, 'borscht-belt': 1, 'umpire': 1, 'selections': 1, 'drama': 87, 'submission': 1, 'age': 48, 'watcher': 2, 'oozing': 2, 'grandchild': 2, 'fingers': 12, 'dunst': 2, 'romp': 7, 'one-act': 1, 'staged': 14, 'tyke': 1, "'men": 1, "acquaintances-i'd": 2, 'airline': 3, 'vintage': 3, 'bickerman': 1, 'norwood': 2, 'hospital': 21, 'vesco': 1, "simmons'": 2, 'leather': 9, "wenders'": 1, 'sixteen': 4, 'entertains': 1, 'archeologist': 1, 'elated': 1, 'heaving': 1, 'time-bomb': 1, 'brute': 1, 'monologues': 3, "amy's": 1, 'utilising': 1, 'stylings': 1, 'detachment': 2, 'bike': 2, 'easter': 1, 'occurrences': 5, '70': 5, 'nintendo': 1, 'semi-naked': 2, 'oliver': 19, 'disinterest': 3, 'depends': 10, 'mitch': 3, "hudson's": 2, 'thespian': 4, 'impenetrable': 2, 'slightly-neurotic': 1, 'studying': 12, 'experiences': 9, "moreu's": 1, 'laundry': 2, 'monstrosity': 1, 'slays': 1, 'chat': 2, 'raking': 1, 'assignment': 17, 'illusion': 10, 'samurai_': 1, 'raping': 3, 'die-hard': 10, 'greys': 1, 'yemen': 1, 'awhile': 10, 'polar-opposite': 1, 'caterer': 1, 'sentiments': 4, 'difficult': 67, 'workmen': 1, 'devine': 4, 'magically': 5, '$3': 1, 'incriminating': 2, 'skyrocket': 3, 'aidan': 2, 'transporting': 4, 'revolve': 4, 'amazement': 1, 'disposal': 5, 'jibe': 1, 'directed': 116, 'poof': 2, 'sara': 4, 'stand': 47, 'diary': 4, "edwards'": 1, 'ties': 11, 'evil': 112, 'growl': 2, 'evita': 2, 'upright': 1, 'alain': 1, 'richardson': 8, "cumming's": 1, 'last-place': 1, 'baranski': 1, "pongo's": 1, 'boozing': 1, 'twentysometyhings': 1, 'joy': 14, 'decay': 2, 'posture': 1, 'langton': 1, "'thing": 1, 'stamped': 6, 'penetrate': 2, 'canning': 1, 'class-conscious': 1, 'goofs': 1, 'atwood': 1, 'illegally': 1, 'identically': 1, 'envision': 2, 'pre-teen': 3, 'transported': 8, 'script': 293, 'counselor': 6, 'presidential': 3, 'condensed': 3, 'participation': 3, 'renaldi': 1, 'hu': 1, "'con": 1, 'dork': 3, 'dial': 3, 'silently': 3, '98%': 1, "firestorm's": 1, 'not-so-bright': 2, 'streetwalkers': 1, 'heckerling': 5, '$$$': 1, 'chronicles': 4, 'masterpiece': 19, 'thade': 2, 'sloppiest': 1, 'jobson': 1, 'b-films': 2, 'olive': 1, 'investigative': 2, "_can't_": 1, 'repellant': 1, 'river': 13, 'stung': 2, 'abilities': 24, 'punished': 4, 'chances': 8, 'ii': 32, 'slice': 5, "shirase's": 1, 'strangest': 1, 'ofa': 1, 'steadily': 3, 'ninja': 3, 'one-note': 15, 'won': 20, 'passionately': 1, 'boy-friend': 1, 'dave': 9, 'coated': 4, 'aptitude': 2, 'overstaying': 1, 'orchestrating': 1, 'supplies': 4, 'applauding': 2, 'default': 2, 'sotomejor': 1, 'attention-deficit-disorder': 1, 'lugging': 1, 'outfits--and': 1, 'transport': 5, 'embezzlement': 1, 'suit-': 1, 'vomit': 6, 'action-movie': 1, 'underlit': 1, 'thorn': 1, 'paints': 7, "picard's": 1, 'bullet': 7, 'werewolves': 2, '``insectopia': 1, 'editing': 45, 'belabored': 1, 'bewilderingly': 1, 'special': 156, 'monkeys': 5, 'keanur': 1, 'haynes': 1, 'ren': 1, 'serious-sounding': 1, 'ivey': 1, 'gazes': 1, 'innocent-at-heart': 1, 'drawer': 3, 'doogie': 3, 'status': 18, 'plot': 453, 'complicity': 1, 'time-honored': 1, "violator's": 1, 'artfully': 1, 'uninteresting': 36, 'waiter': 1, "bava's": 1, 'mailheart': 1, 'moviegoing': 2, 'fashion-conscious': 1, 'shiny': 7, 'crucifix': 1, 'physics-': 1, 'spat': 1, 'felon': 1, 'shack': 3, 'sec': 2, 'lugubrious': 2, 'forest': 14, 'funny': 239, 'cow-moo': 1, 'comaraderie': 1, 'egg': 10, 'documentaries': 3, 'volunteers': 1, 'sil': 2, 'wilcock': 7, 'madam': 3, 'kicker': 3, '_last_': 1, 'indicator': 1, 'hiatus': 3, 'strike': 23, 'extension': 1, 'joseph': 10, 'claustrophobia': 1, 'alberto': 1, 'sage': 2, "mary's": 1, 'midwife': 1, 'inkling': 2, "`surprise'": 1, 'second-in-command': 1, 'high-priced': 1, 'detonate': 1, 'co-screenwriter': 2, 'groaned': 2, 'melding': 2, "android's": 1, 'population': 15, 'subtitle': 1, 'exoticism': 1, 'shore': 6, 'trashily': 2, 'cardinal': 8, 'metaphorically': 1, 'uninterrupted': 1, "'original'": 1, "rumor's": 1, 'diversionary': 1, 'periodically': 1, 'seattle': 5, 'boots': 4, 'bounds': 4, 'mashed': 2, 'worked': 54, 'avenues': 1, 'rapaport': 3, "group's": 3, 'conrad': 5, 'undone': 1, 'product': 42, "nail's": 1, 'interwoven': 1, 'lobotomies': 1, 'trainman': 1, 'translations': 1, 'unlimited': 2, "pandora's": 2, 'prediction': 3, 'stereo': 2, 'spin': 16, 'communicate': 12, 'disrobe': 2, 'near-total': 1, "successor's": 1, 'vents': 1, 'unconvincing': 20, 'reinvigorated': 1, 'mcneill': 1, 'menage-a-trois': 1, 'stiletto': 2, 'audio-recorded': 1, 'inhabiting': 1, 'idly': 1, 'york_': 1, 'irked': 1, 'flawed': 10, 'earthling': 1, 'flies': 8, 'goofiest': 1, 'fellows': 1, 'pieces': 36, 'quarrels': 1, 'padding': 3, '1709': 1, 'slushee': 1, "dawson's": 5, 'mistaking': 1, 'gibson': 11, 'syncing': 1, 'appreciates': 1, '@': 1, 'vigorous': 2, 'vacant': 5, 'sickly': 4, 'screenwriter': 59, 'exploring': 11, 'dispondent': 1, 'stylistic': 7, 'frenchman': 5, 'sweet': 39, 'mans': 2, 'differnt': 1, 'inspire': 9, 'mixes': 3, 'insider': 3, 'keeble': 1, 'perkuny': 1, 'indicating': 3, 'debonair': 3, 'casper': 8, 'colour': 2, 'opinions': 7, 'sarajevo-like': 1, 'constructed': 8, 'leitch': 1, 'linder': 1, 'botched-robbery': 1, 'carousing': 1, 'counter': 4, 'fun--oh': 1, 'marg': 3, '1/25th': 2, 'hated': 15, 'stinkor': 1, 'ajax': 1, 'cold-blooded': 1, 'bendel': 1, 'reviewer': 7, 'edell': 1, 'omit': 3, "then-he's": 1, "hopkins'": 1, 'particulars': 1, 'arch-rival': 1, 'lenz': 1, 'mumbo': 2, "richardson's": 1, 'charity': 3, 'clarity': 3, 'crisp': 6, 'evasion': 2, 'jungle': 22, 'virgin': 6, 'blaze': 3, "leary's": 1, 'malcolm-as-momma': 1, 'dunne': 8, 'expletive': 2, 'budget': 58, 'payroll': 3, 'valek': 4, 'gagging': 2, 'prokofiev': 1, "ladies'": 1, 'winslet': 1, 'inferiority': 1, 'hotties': 2, 'clarence': 4, 'concurrence': 1, 'intrusive': 4, 'schools': 1, 'miscast': 20, 'sprucing': 1, 'rebecca': 16, 'amanda': 17, 'soft-tossed': 2, "aykroyd's": 1, 'centrifuge': 1, 'palminteri': 6, 'real': 254, 'sf': 3, 'mid-way': 1, 'emily': 10, 'confining': 2, "waterworld's": 1, 'generating': 1, 'inexpensive': 1, 'looooooong': 1, 'symbolism': 7, 'superscript': 1, 'grandeur': 2, 'cut-away': 1, 'carelessly': 3, 'slow-moving': 5, 'head-shaving': 1, "bison's": 1, 'administration': 2, 'nordoff-hall': 1, 'spiner': 3, 'politically': 6, 'hard-boiled': 4, 'bouncer': 1, 'litten': 1, 'combing': 1, 'mst3ked': 1, 'gellar': 5, 'wise-ass': 3, 'requested': 2, 'unnatural': 2, 'structured': 5, 'infiltrates': 2, 'marina': 1, 'advertiser': 1, "'straight'": 1, "danza's": 1, 'tombs': 3, 'platform': 2, 'ingerman': 1, 'mercenary': 8, 'heap': 6, 'impeccably': 2, 'sunnybrook': 1, 'estimate': 1, "hill's": 3, 'rises': 9, 'paris': 19, "humanity's": 1, 'sitcom': 22, 'walsh': 4, 'smoother': 2, 'befitting': 1, 'whisperer': 1, 'wondering': 33, '1945': 1, 'betters': 2, 'sideswipes': 2, 'bazooka': 2, 'operation': 11, 'narcissistic': 1, 'skulking': 2, 'dlose': 1, 'bramon': 1, 'absurdly': 5, 'influential': 1, 'eastern': 6, 'human': 108, 'cartoonish': 11, 'fight-picking': 1, 'wisecracks': 3, 'ha': 8, 'roster': 1, 'method': 8, 'voiced': 13, 'similarity': 1, 'approachable': 2, 'targets': 4, 'renounces': 1, 'artful': 1, 'mummy': 4, 'yeah': 44, 'automats': 1, 'greater': 15, 'comment': 13, 'clarke': 7, 'abouts': 1, 'lucy': 8, 'taye': 4, 'doomsday': 1, 'voice': 59, 'gawk': 1, 'small-screen': 1, 'heyday': 2, 'co-directors': 1, 'travolta': 22, 'interview': 7, 'drills': 2, 'rudimentary': 1, 'flummoxing': 1, 'pomp': 1, 'kosher': 1, 'ravens': 1, 'designated': 1, 'shampoo': 1, 'political-': 1, 'inflicting': 3, 'apocalyptic': 6, 'chandler': 2, 'subliminal': 1, 'adroit': 1, 'video-esque': 1, 'ego-withering': 1, 'scotland': 4, 'quickie': 3, 'keeslar': 1, 'overhaul': 1, 'hohh': 1, 'spunk': 2, '500k': 1, 'acclaim': 3, 'bowden': 1, 'wrong-perhaps': 2, 'drowsy-voiced': 1, 'thing_': 1, 'dwindled': 1, 'emotionally-charged': 1, 'nighttime': 3, 'kissed': 5, 'arthouse': 1, 'trains': 5, 'historically': 3, "pistone's": 1, "nitro'": 1, 'stoli': 1, 'naoki': 1, 'carrey-annoying': 1, 'araki': 1, 'dignity': 11, 'engrossed': 1, 'erickson': 1, 'by-play': 1, 'apaches': 1, 'menstruating': 1, 'biographies': 1, 'fitting': 4, 'scumbags': 1, 'no-holds-barred': 1, 'quoi': 1, 'contingency': 1, 'flopped': 4, 'mt': 1, 'debris': 1, 'abbott': 1, 'details': 28, 'hoppe': 2, 'launderer': 1, "'one": 1, 'hmmmm': 3, 'time-filler': 1, 'pc': 2, 'bubbles': 2, 'carbon': 5, 'darkness_': 1, 'gazon': 1, 'sidney': 6, 'rehashes': 4, 'commanders': 2, "'partner'": 1, 'well-intentioned': 4, 'philadelphia': 6, "yourself'": 1, '40mins': 1, 'murderous': 12, 'smeared': 4, 'fanciful': 3, 'inept': 36, 'vice-presidential': 1, 'kiernan': 1, 'potentially-interesting': 1, 'undisturbed': 1, 'probability': 1, 'hmos': 1, 'shame': 42, 'faux-mysticism': 1, 'rot': 2, "planet'": 1, 'mcbride': 3, 'jokes': 102, 'pathetically': 9, 'colonels': 1, 'twinkling': 1, 'bead-adorned': 1, 'awaits': 1, 'campaign': 17, 'stray': 6, 'dryer': 1, 'highlighted': 3, 'premiere': 7, 'gilligan': 2, 'grabs': 3, 'mustang': 1, "latter's": 2, 'rammed': 1, 'ann': 10, 'cagney': 1, 'unexpectedly': 7, "o'keefe": 1, 'boasting': 1, 'gorgeous': 15, 'storage': 2, 'ludicrously': 3, 'unholy': 4, 'probide': 1, 'three-pitch': 1, 'explanations': 4, 'undemanding': 4, 'anacondas': 2, 'casually': 3, 'lauper': 1, 'fit': 29, "'10": 1, 'prevalent': 2, 'under-10': 1, 'xref': 1, 'mirror-imaged': 1, 'gadget-filled': 1, 'senility': 1, "chiba's": 1, 'battle': 69, "captain's": 1, 'motown': 1, 'prospect': 6, 'video-game': 1, "'threatening": 1, 'bears': 12, 'resisting': 1, 'kid-friendly': 1, 'crude': 11, 'filmmakers': 97, 'cuckoo': 1, 'simultaneously': 8, 'nobodies': 2, 'organ': 3, 'vexing': 1, 'length': 32, 'criminally': 4, 'one-stardom': 1, "bueller's": 1, 'impediment': 1, 'lampooning': 1, 'mcg': 1, 'member-gone-bad': 1, 'evolved': 3, 'kieren': 1, 'hawk': 5, 'specifically': 13, 'enact': 2, 'blindness': 1, 'mesmerized': 1, 'steam-controlled': 1, 'hangout': 2, "up'": 1, 'corbin': 3, 'cables': 1, 'pop-psychology': 1, 'bauchau': 1, 'bald': 7, '>': 3, 'ginter': 1, 'break-up': 1, 'ye': 2, 'would-be-ahabs': 2, 'collapsing': 2, 'coiffed': 1, 'pronto': 1, 'disappearing': 3, '700': 2, 'uncompleted': 1, 'africa': 12, 'unspeakably': 1, 'utterance': 1, 'ruin': 17, 'prissy': 2, 'olden': 1, 'aperitif': 1, 'prosperous': 1, 'meyers': 1, '80-year': 1, 'slaughtering': 1, 'weaken': 2, '1984': 6, 'co-coaching': 1, 'hot-shot': 2, 'hil-ari': 1, 'ws': 1, 'cd': 3, 'quirks': 6, "butabis'": 1, 'culkin': 3, 'calls': 47, 'miracles': 2, "jakob's": 2, 'cave': 8, 'dangerously': 3, 'tripped-out': 1, 'lynching': 1, "sinise's": 2, 'bugs-but': 1, "fightin'": 1, 'tummy': 1, 'sumptuous-looking': 1, 'spire': 1, 'chucky': 1, 'premonition': 2, 'rendezvous': 1, 'reinforcing': 1, 'detonation': 1, 'brass': 3, 'gurian': 1, "frank's": 1, 'made-for-the-sci-fi-channel': 1, 'wretched': 5, 'impregnating': 2, 'tee': 1, 'bankability': 1, 'sidesplitting': 1, 'crazy': 34, 'immigrant': 1, 'expense': 3, 'ex-gangbanger': 1, "ned's": 1, 'elmo': 1, 'murder-': 1, 'housemaid': 1, '1900': 3, 'assaulting': 2, 'disguised': 5, 'midsection': 1, "'uncle": 1, 'kane': 5, "junger's": 1, 'addictions': 1, 'herring': 4, 'caine': 5, 'bearing': 4, 'bravado': 1, 'midpoint': 1, "inspector's": 1, 'forays': 1, 'consultant': 5, 'jacki': 1, 'beasly': 1, 'slasher-flick': 1, 'klutzes': 1, 'reverend': 2, 'ming-liang': 1, 'energetically': 2, 'phillip': 1, 'midsummer': 1, 'optimists': 1, 'dissect': 1, 'possesses': 15, 'eloquently': 1, 'nude': 19, 'henry': 24, 'drek': 1, 'wilt': 1, 'sessions': 4, 'myth': 5, 'reason--somehow': 1, 'faux-psycho': 1, 'minks': 1, 'zones': 2, 'doesn\x12t': 2, 'dabney': 2, 'heft': 1, "'broads'": 1, 'cubicled': 1, 'pontificating': 1, 'oh-so-cool': 1, "baker's": 4, 'well-choreographed': 3, 'parillaud': 1, 'moran': 3, 'last-minute': 3, 'actions': 29, 'reptile': 1, 'poland': 1, 'sawa': 1, 'retrieved': 1, 'mcgregor': 5, 'dragging': 4, 'pfferpot': 1, 'disobey': 1, 'superfreak': 1, 'keeper': 3, '00': 4, 'ex-girlfriend': 6, 'delighted': 3, 'inconsequential': 2, 'programmed': 3, 'quaint': 3, 'rea': 4, 'tubing': 1, 'heady': 1, 'expertise': 5, 'palatial': 1, 'sadsack': 1, 'flashes': 6, 'super-senses': 1, 'emotions': 26, 'tornado': 5, 'sutton': 1, 'steed': 4, 'ol': 1, 'grodin': 1, 'loggia': 5, 'confuses': 1, 'zimmerly': 1, 'recalling': 1, 'three-year-old': 1, 'matter-of-': 1, 'work': 315, '35': 6, 'frights': 4, 'gifts': 1, 'saves': 18, 'introduced': 42, 'accessorize': 1, 'intrigue': 7, 'creep': 3, 'rosemary': 1, 'cheek': 4, 'one-star': 1, 'threat': 11, 'erase': 2, 'rebuilding': 1, 'plucky': 1, 'hospitality': 2, 'comic': 112, 'creatures': 20, 'craze': 6, "charm's": 1, 'burn-out': 1, 'tragically': 4, 'robe': 1, 'infantrymen': 1, 'overdose': 1, 'inhibit': 1, 'sans': 12, 'relating': 9, '75%': 2, '25%': 2, 'thankful': 2, 'newsgroup': 2, 'affluent': 2, 'coma': 8, 'self-serving': 1, 'pees-ahn': 1, 'pressed': 5, 'aided': 4, 'einstein': 1, 'creeps': 3, 'reworked': 1, 'atmospheric': 5, 'argonauticus': 1, 'emanates': 1, 'implications': 3, 'bureaucrat': 1, "topol's": 1, 'overplay': 1, '`a': 1, "pericles'": 1, 'indecent': 1, 'metropolis': 2, 'stewing': 1, "cusack's": 1, 'bang': 10, "cabot's": 1, 'eaten': 3, "prudential's": 1, "franklin's": 2, 'specifics': 4, 'intricacies': 2, 'thanking': 2, 'smutty': 1, 'grant/matthew': 1, '$20-million': 1, 'feline': 1, 'begin': 83, 'krzysztof': 1, 'mono': 2, "beach--yang's": 1, 'additionally': 5, 'howie': 3, 'gigolo': 2, 'commenting': 3, 'weeds': 1, 'reported': 1, 'wide': 28, 'aka': 4, 'funded': 3, 'valley': 9, '_huge_': 1, 'hallways': 2, 'belches': 2, 'flaming': 2, 'dog-loving': 1, 'suggestively': 1, 'earnestly': 1, 'gordan': 1, 'chainsmoking': 1, 'put-upon': 1, 'dunwich': 1, 'semi-graphic': 1, 'croc': 2, 'infectious': 2, 'lasso': 1, 'one-two': 2, "'three's": 1, 'electronically': 2, 'smug-looking': 1, 'quick-e': 1, 'spanned': 1, 'seven-foot': 1, 'prince': 17, 'risking': 2, 'taboo': 1, 'certificate': 2, 'punch': 21, 'brothers': 53, 'love-smitten': 1, 'barry': 26, 'flamenco': 1, 'auteur': 5, 're-created': 1, 'inscrutable': 1, 'davidovich': 1, 'salvati': 1, 'luppi': 1, 'splat': 3, 'exonerating': 1, 'testosterone': 2, 'arouse': 1, 'ex-newspaper': 1, 'ex-boyfriend': 5, 'footnotes': 1, "hartley's": 1, 'triangular': 1, 'geezers': 1, 'mcconaughey': 3, 'depite': 1, 'thrill-seeker': 1, 'cornucopia': 3, 'departure': 6, 'challenging': 9, 'chapters': 2, 'uuuhmm': 1, 'bewildered': 4, 'jammed': 3, "gerald's": 1, 'attempted': 17, 'marijuana': 5, 'tweed': 2, 'kaige': 1, 'whould': 2, 'ricci': 8, "cast's": 1, 'imperial': 2, "`what's": 1, 'first-': 1, 'fades': 8, 'frighteningly': 2, 'outlanders': 1, 'kaleidoscope': 1, 'jell-o': 2, 'loathsome': 2, 'quart': 2, 'courier': 3, 'bakersfield': 1, 'regularity': 1, 'prey': 4, 'deterioration': 1, 'long-haired': 3, 'aim': 7, 'kenny': 5, "dinosaurs'": 1, 'jewel': 13, 'foreshadowing': 6, 'ex-model': 1, 'environments': 1, 'originals': 3, 'study': 14, 'pathological': 1, 'macy': 6, 'ego-driven': 1, 'acid': 9, 'eyed': 2, 'foo': 2, 'processions': 1, 'lumps': 1, 'moderately-successful': 1, 'roles': 102, 'microchip-sized': 1, 'chins': 1, "stewart's": 1, 'streams': 1, "mosley's": 1, 'fund-raiser': 1, 'cornfield': 1, 'illicit': 2, 'braced': 1, "delia's": 1, 'opted': 5, 'faithful': 9, 'latent': 2, 'story--or': 1, 'semi-robed': 1, 'biel': 3, 'disintigrated': 1, 'wing': 10, 'verhoeven': 9, 'well-done': 3, 'texas': 18, 'conceit': 6, 'kurtz': 1, 'leans': 1, 'squander': 1, 'super-mushy': 1, 'foulmouthed': 1, 'hepburn': 1, 'horrific': 7, 'kubrick': 3, 'oahu': 1, 'cheers': 3, 'grape': 2, 'gapes': 2, 'weakly': 2, 'cribbed': 1, 'to-back': 1, 'not-so-talented': 1, 'sealy': 1, 'metaphorical': 1, 'busybody': 1, 'sian': 1, 'db': 1, 'kirstie': 1, 'tiff': 2, 'sort': 120, 'honesty': 7, 'un-interrupted': 1, 'spores': 1, 'zip': 2, 'handcuff': 1, 'hunk': 5, 'dissatisfying': 2, 'electricity': 1, 'overdubbed': 1, 'paolo': 2, 'games': 23, 'irreverence': 1, 'wives': 5, 'stomps': 1, 'osment': 1, 'sciorra': 1, '39': 1, 'dropped': 23, 'cg': 1, 'untamed': 2, "houdini's": 1, 'bass': 3, 'sleuthing': 2, 'juxtaposition': 1, 'a-flailing': 1, 'artificially': 4, 'patterns': 3, 'website': 8, '1=a': 1, "nature's": 1, 'strut': 2, 'toronto': 7, 'foundered': 1, 'eomann': 1, 'amendments': 1, 'eye': 47, 'tangents': 4, 'offended': 7, 'tv-movie-ish': 1, 'elle': 4, 'trifecta': 1, 'mixon': 2, 'cantrell': 2, 'semi-': 1, 'fraud': 2, 'rejoin': 1, 'romanovs': 1, 'disorder': 3, 'resonate': 1, 'fireball': 2, 'clown': 9, 'compliments': 1, 'washed-up': 3, 'nooooo': 2, 'reasons': 44, 'stairwells': 1, 'soft-spoken': 5, 'giger': 1, 'occurred': 12, 'joke---that': 1, 'drug-dealing': 1, 'eighteenth': 1, 'porn': 19, 'summarizes': 1, "robin's": 3, 'molest': 1, 'brunettes': 1, 'brocade-draped': 1, 'annoyances': 1, 'quarry': 1, 'tsavo': 1, 'ham-handed': 2, 'workload': 1, 'assassinate': 2, 'elephant': 4, 'play-write': 1, 'hear': 52, 'sherman': 3, 'skater': 2, "lazard's": 1, 'canton': 2, 'vastly': 2, 'sandwich': 2, 'fitted': 1, 'childers': 1, 'influenced': 4, 'phillipie': 1, 'rhythms': 4, '`what': 1, 'otherwise--but': 1, "patillo's": 1, 'darting': 1, 'spills': 2, 'tribeca': 1, 'unaddressed': 1, 'intriguing': 35, 'happiness': 7, 'plodding': 8, 'wanders': 17, 'person': 104, 'city': 99, 'sunrises': 2, 'leeringly': 1, 'conscientious': 1, 'specials': 1, 'ophiophile': 1, 'toole': 1, 'continued': 11, '$115': 1, 'physically': 10, 'amboy': 1, 'wrenched': 1, 'considers': 5, 'splendidly': 3, 'stickers': 1, 'kapner': 1, 'myths': 1, 'aids': 4, 'escadapes': 1, 'awaken': 2, "chase's": 1, 'braintree': 1, 'actors': 224, 'returns': 38, 'delroy': 6, 'myrtle': 1, 'brooklyn-bound': 1, 'denizens': 1, 'uplifting': 3, 'sober': 6, 'cheaply': 1, 'intrigued': 3, 'religion': 4, 'smits': 1, 'sheepishly': 1, 'campus': 3, 'alger': 1, 'teri': 6, 'breach': 1, 'consigned': 2, 'persistence': 1, "parents'": 5, 'journeyed': 2, 'quieter': 1, 'breached': 1, 'paltrow': 11, 'hooks': 7, 'consumers': 1, 'miley': 1, 'desperation': 8, "cinema's": 1, 'majestic': 2, 'scissorshands': 1, 'sibling-like': 1, 'agenda': 6, '107': 2, 'slave': 12, 'sweetie': 1, 'wheezing': 1, 'purple': 1, 'simpler': 2, 'male-bonding': 2, 'advantages': 1, 'halls': 3, 'cary-hiroyuki': 2, 'nest': 9, "jermaine's": 1, 'gossipy': 1, 'sub-inspired': 1, 'falling': 33, 'chicago': 19, 'attempt': 130, 'frontals': 1, 'pronouns': 1, 'paragraph': 9, 'colleagues': 8, 'respected': 11, 'rats': 3, 'distrusts': 1, 'prince-like': 1, 'kenobi': 2, 'consultants': 1, 'ingmar': 1, 'souped-up': 2, 'potatoes': 2, 'dialog': 24, 'asshole': 5, 'gave': 54, 'junk': 17, 'arresting': 3, 'fleming': 3, 'calloway': 2, "finch's": 1, 'overhead': 4, 'victor': 7, 'voted': 3, 'farrow': 1, 'inject': 15, 'sassiness': 1, 'hap': 1, 'pill-induced': 1, 'heighten': 1, 'swordsman': 2, '600': 2, 'friedberg': 1, "daughter's": 5, 'mermaid': 4, 'profundity': 2, "jones's": 1, 'shoot-em-ups': 1, 'brightness': 1, 'everclear': 1, 'bedazzled': 3, 'hippyish': 1, 'maddened': 1, 'consumerism': 1, 'remar': 2, 'houser': 1, "today's": 10, 'directly': 29, 'overcoming': 1, 'bronx': 5, 'overseen': 2, 'hues': 1, 'serina': 1, 'warranted': 5, 'teases': 3, 'ooze': 1, 'scratchy': 1, 'discussing': 10, "andy's": 1, 'snapshot': 1, 'tune-meister': 1, 'trouble': 62, 'holes': 29, 'goodies': 1, 'gaining': 2, 'cast-member': 1, 'higher': 17, 'brazil': 8, 'costs': 9, 'calculatedly': 1, 'liberated': 1, 'big-town': 1, 'darren': 7, 'inadequacies': 1, 'recite': 5, "samantha's": 1, 'writer/director': 22, 'pontoon': 1, 'woodard': 1, 'semi-bright': 1, 'utility': 1, 'enervation': 1, 'old-style': 1, 'apted': 1, 'lawn': 6, 'tents': 1, 'genzel': 1, 'flowed': 2, 'hungarian': 1, 'junkyard': 2, 'kick--willis': 1, 'acquittal': 2, 'ivy': 8, 're-shoot': 1, 'remarking': 1, 'cancellation': 1, 'roundtree': 2, 'jumanji': 1, 'neutral': 4, 'movies--': 1, '1913': 1, 'disbanded': 1, 'domination': 4, "'having": 1, 'barren': 3, 'visiting': 5, 'snazzy': 2, 'prisoners': 7, 'cellulars': 1, 'nuke': 2, 'spotlights': 1, 'diehard': 1, 'gen-xers': 1, 'professions': 2, 'spiders': 1, 'expressionless': 1, "`batman'": 1, 'leder': 2, 'menahem': 1, 'suceeds': 1, 'pompeii': 1, 'schuster': 1, 'trigonometry': 1, "wood's": 2, 'high-camp': 1, 'coinciding': 2, '`snake': 1, "o'shae": 1, 'sociopath': 2, 'speed': 32, 'increase': 3, "'99": 1, 'hyper-color': 1, 'lapses': 2, 'angie': 1, 'jocks': 3, 'inherited': 2, '@#$^%': 1, 'commerce': 1, 'referenced': 1, 'speaking': 36, 'riley': 2, "judge's": 2, 'labor': 8, 'stating': 3, 'gold-tinted': 1, 'tonya': 1, 'improvises': 1, 'williamson-esque': 1, 'proxy': 3, 'freedom': 10, 'admit': 47, 'collide': 3, 'oliveira': 1, 'heinlein': 3, 'lounge': 6, "martin's": 4, 'ills': 1, 'incognito': 1, 'inheritance': 4, 'awkward': 20, 'encourage': 7, 'one-line-at-a-time': 1, 'lightsome': 1, 'wrinkled': 2, 'sela': 2, "bookseller's": 1, 'toxic': 1, 'bassett': 2, 'turtleneck': 1, 'logan': 1, 'ode': 1, 'x-men': 1, 'marks': 18, 'remeber': 1, 'volcanic': 1, 'abandons': 3, 'dissuade': 2, 'breaking': 24, 'amusement': 12, 'cady': 1, 'kashiwabara': 1, 'huntington': 1, "hughley's": 1, 'hooters': 2, 'in-habit': 1, 'raunch': 2, 'amends': 1, 'vault': 5, 'endear': 1, 'bonk': 1, 'fixing': 2, "'in": 1, 'upside': 9, 'gotcha': 11, 'angeles/las': 1, 'edits': 7, 'hilly': 1, "bosses'": 1, 'coifs': 1, 'christianson': 1, 'stagelights': 1, 'ant-eaters': 1, 'non-presence': 1, 'shawn': 5, 'unflushed': 1, 'unblemished': 1, 'month': 15, 'tediously-and': 1, 'eye-candy': 1, 'choses': 1, 'deserves': 23, 'woo': 21, 'confusingly': 3, '40-plus': 1, 'vogue': 4, 'grandma': 4, 'kumble': 1, 'featherbrained': 1, '7/10': 8, 'heisting': 1, 'warren': 15, 'batarang': 1, 'bikinis': 1, 'equipment': 8, 'depressed': 15, 'pantyhose': 1, 'rings': 11, 'cutest': 1, 'salty': 1, 'cadets': 1, 'blair': 29, 'cheerfully': 1, 'experimentation': 1, 'squirt': 1, 'renewed': 1, 'techno': 2, 'spacek': 1, 'ovation': 2, 'that+s': 1, 'mintoro': 1, 'flame': 8, 'germans': 2, 'stalkers': 1, 'floats': 5, 'lavish': 6, 'ascends': 1, "slugs'": 1, "everett's": 1, 'talking': 74, 'out-smarted': 1, 'skin-tight': 2, 'hat-wearing': 1, 'poirot': 1, 'selecting': 2, 'antagonists-as-protagonists': 1, 'barbeque': 1, 'eleven': 4, 'workweek': 1, "reichle's": 1, 'fast-food': 2, 'exceptions': 6, 'second-class': 1, 'master-slave': 1, 'mugger': 1, "power's": 1, '[sic]': 1, 'patrolman': 1, 'encountered': 4, 'olsen': 1, 'educated': 3, 'bangy': 1, 'intro': 1, "hyams'": 1, 'piousness': 1, 'piano': 4, "terl's": 1, "rhames'": 1, 'obligations': 1, 'everglades': 1, 'impacted': 1, 'across-the-board': 1, 'photographer': 6, 'lee': 70, 'wallow': 3, 'bucket': 1, 'universal': 8, 'jill': 5, 'fashion': 37, 'leelee': 4, 'unabashedly': 3, 'technique': 14, "driver's": 1, 'venice': 1, 'perpetuation': 1, 'inhuman': 2, 'crackles': 1, '2007': 1, 'fabricated': 3, 'scope': 6, 'christening': 1, "schreiber's": 1, 'gun-crazy': 1, 'dublin': 1, 'omnivorous': 1, 'laughed': 29, 'scroll': 2, 'fished': 1, 'thickens': 4, 'secretly': 10, 'movement': 10, 'overblown': 9, 'brandi': 1, 'class-divided': 1, 'headache': 12, 'archetype': 2, 'hair-removal': 1, 'humourless': 3, 'comeuppance': 4, 'chimera': 1, 'erratically': 1, 'anti-terrorist': 2, 'toe': 3, 'reborn': 2, 'misogynistic': 5, 'force-feeding': 1, 'compulsory': 1, 'marin': 2, 'lines': 140, 'morsels': 1, 'observational': 1, 'hawthorne': 1, 'sgt': 5, 'note-for-note': 1, 'nerve-jangling': 1, 'generous': 7, 'secretary': 12, 'harden': 3, 'klutz': 3, "target's": 2, 'dotson': 1, 'branch': 1, 'effectively': 23, 'yell': 10, 'lipstick': 3, 'repulsively': 1, 'bellyaching': 1, 'auto': 8, 'spins': 1, 'ghastly': 3, 'exotic': 7, 're-runs': 1, 'zoologist': 2, "speilberg's": 1, "diedre's": 1, 'hedonistic': 1, 'uninviting': 1, 'romping': 1, 'first--and': 1, 'investigated': 1, 'consequence': 2, "freddy's": 1, 'tilt': 1, 'israeli': 2, 'kennedy': 7, 'labored': 6, 'beastuality': 1, 'carwash': 1, 'reads': 21, 'phi': 1, 'component': 3, 'cinema': 56, 'color': 15, 'dodging': 3, 'falk': 1, 'artifact': 1, 'hair-raising': 1, 'sexualize': 1, 'firstly': 3, "'stab": 1, 'paint-by-numbers': 4, '779': 1, 'date-unsound': 1, 'thick-bladed': 1, 'shill': 1, 'cannom': 1, 'gas': 7, 'petersson': 1, 'cornfields': 1, 'alignment': 2, 'withering': 2, 'repugnant': 1, 'parsley': 1, 'misunderstanding': 1, 'mafia': 14, 'moves': 55, 'innovation': 2, 'introduction': 7, "hemming's": 1, 'kicked': 8, 'assuring': 2, 'tanks': 3, 'teensy': 1, '_hope': 1, 'raining': 3, 'third-rate': 3, 'comparable': 2, 'whiteness': 1, 'periscope': 1, 'undoubtedly': 10, 'prolonged': 4, 'casablanca': 4, 'townfolk': 1, 'frogs': 2, 'describes': 10, 'harlem': 4, 'extraction': 1, 'cycling': 1, "porter's": 2, 'jailed': 2, 'lobs': 1, "plane's": 1, 'westridge': 1, '13th': 12, 'elderly': 12, '2654': 1, 'intimidates': 1, 'psycho-in-the-rain': 1, 'all-star': 4, 'grave': 12, 'unimpressive': 6, 'forgivable': 3, 'oseransky': 1, 'seduction': 8, 'dumbfounded': 1, 'shrew': 1, 'exploded': 2, "why'd": 1, 'vader': 2, 'replacing': 3, 'transposes': 1, 'demises': 1, 'behest': 1, 'vanzant': 1, 'nesmith': 1, 'heirs': 2, "gal's": 1, 'indolent': 1, 'viciousness': 1, 'finest': 5, 'floats_': 1, 'recovery': 1, 'cherry-on-the-sundae': 1, 'stressed-out': 1, 'massive': 16, 'plastic': 9, 'stock-in-trade': 1, 'freshman': 5, 'reluctance': 1, 'carlyle': 4, 'biggest': 62, 'aged': 6, 'navy': 6, 'counterparts': 7, 'attention': 72, 'sold': 7, 'jerkish': 1, 'affront': 1, 'deserving': 6, 'anarchy': 1, 'singe': 1, 'wrap': 2, 'joshua': 5, 'peons': 1, 'unflinching': 1, 'respects': 8, 'escapist': 1, 'festively': 1, 'ment': 1, 'corpse': 8, 'distancing': 3, 'hichock': 2, '_do_': 1, 'earthly': 2, 'enthralling': 4, 'branson': 1, 'woman': 163, 'moguls': 1, 'screenwriterly': 1, 'brinks': 1, 'firearms': 2, 'coutroom': 1, 'pondered': 1, 'canceled': 1, 'work-a-day': 1, 'connoisseurs': 2, 'johnny': 23, 'unanticipated': 1, 'lastly': 6, 'swindles': 1, 'objectifying': 1, 'tens': 2, 'anti-male': 1, 'insult': 28, 'inconsistency': 4, 'pounds': 5, 'lured': 3, 'filmmaking': 30, 'shift': 16, 'teamed': 9, 'mianders': 1, "macpherson's": 1, 'booty': 3, 'mettle': 1, 'delaney': 5, 'godfather': 8, 'winds': 14, 'maxim': 4, 'carrier': 2, 'somethine': 1, 'seedy': 8, 'by-the-number': 1, 'turboman': 1, "troyer's": 1, 'sci-fi/horror': 4, 'integrate': 1, 'torn-up': 1, 'painted--you': 1, 'uhhhm': 1, 'banshee': 2, "rodgers'": 1, 'self-esteem': 2, 'harumph': 1, 'tv-movie-of-the-week': 1, 'familial': 3, 'thundering': 1, 'compiled': 2, 'navigator': 2, 'ooooo': 1, 'tampering': 1, 'dismantling': 1, "'anywhere": 1, 'sesame': 1, 'harmonica': 1, 'overseas': 2, 'jacott': 1, 'anger': 5, 'tough-talking': 1, 'sabbato': 1, 'prevents': 9, 'spheres': 2, 'everyman': 2, 'bigwig': 1, 'matronly': 1, 'destination': 9, "'modernization'": 1, 'tech': 4, "bennett's": 1, 'earthy': 3, 'emsemble': 1, 'malcolm': 7, 'thurmand': 1, 'monkey': 10, "out'": 1, 'machineguns': 1, 'liners': 4, 'salome': 1, 'realize': 58, 'everytime': 1, 'suspense': 73, 'sabihy': 1, 'shamed': 1, 'brent': 3, 'shakespearean': 7, 'engenders': 1, 'disperses': 1, "odette's": 1, "antonio's": 1, 'bio-genetics': 1, 'that--despite': 1, "anti-hero's": 1, 'tick': 1, "ustinov's": 2, 'emigrating': 1, 'tailed': 1, 'corniest': 1, 'skill': 15, 'squelch': 1, 'strand': 2, 'apparatuses': 1, 'butterworth': 1, 'suave-yet-unsuave': 1, 'schintzius': 1, 'viability': 1, 'figuring': 6, 'healed': 1, 'satirize': 3, '1983': 3, 'league': 16, 'tycoons': 1, 'perf': 1, 'brew': 1, 'rifkin': 2, 'alcoholics': 3, 'im': 2, 'maugham': 1, 'dipped': 1, 'allocated': 1, 'palace': 4, 'policies': 1, 'cord': 3, 'infantry': 3, 'falwells': 1, 'sables': 1, 'cop-buddy': 1, 'movie-you': 1, 'hurt': 21, "witness's": 1, 'danielle': 3, 'alana': 1, 'poorly-developed': 1, 'overused': 7, 'disarming': 3, 'seance': 1, 'business': 58, "'romeo": 2, 'cementing': 2, 'charming': 31, 'never-heard-of': 1, 'downpours': 1, 'thunderously': 1, 'movies--this': 1, 'benoni': 1, 'rushon': 1, 'videoshelves': 1, 'fluish': 1, 'galileo': 1, 'vomit-inducing': 1, 'seku': 1, "garfunkel's": 2, 'occurring': 3, 'dallied': 1, 'comedy--and': 1, 'tetsuo-the': 1, "tennant's": 1, 're-issued': 1, 'blowout': 1, 'examination': 10, 'banking': 1, 'cleopatra': 1, "december's": 1, 'sway': 2, 'young': 184, 'mirrors': 6, 'demeanor': 8, 'trapped': 26, 'regarded': 2, 'snow': 6, 'sequences-they': 2, 'constant': 31, "genre's": 1, 'threeway': 1, 'beaudine': 1, 'meteorological': 1, 'batcave': 3, 'minors': 2, "shaiman's": 2, 'cyan': 1, 'institutionalize': 1, 'amyl': 1, 'whocares': 1, 'grease': 4, 'bout': 1, 'implodes': 1, "felix's": 1, "lacrosse's": 1, 'kids-intense': 1, 'export': 1, 'made-people': 1, 'hailed': 1, 'destroy': 28, 'pau': 1, 'appallingly': 2, 'throw-together': 1, 'always-likable': 1, 'gods': 4, 'cruise': 20, 'contractual': 1, 'wound': 8, 'amazingly': 7, 'shadow': 14, 'attractively': 1, 'pamela': 4, '-creator': 1, 'rifle': 4, 'chemicals': 2, 'poison': 8, 'rodriguez': 5, 'tidy': 3, 'burping': 1, 'administrator': 1, 'subtly': 1, 'partners': 6, 'fist': 5, 'colorado': 2, 'selects': 2, 'visor': 1, 'he/she': 1, 'conceited': 2, "norman's": 1, 'theatrically': 1, 'frizzi': 1, 'footprint': 2, 'mail-order': 1, 'confidently': 1, 'bonus--': 1, 'shelley': 2, 'research': 18, 'squared': 1, '_american_beauty_': 1, 'benefits': 3, 'prodigy': 3, 'peron': 1, 'apologizes': 2, 'overfills': 1, 'britton': 2, 'first-rate': 5, 'best-': 1, 'timid': 3, 'proclaim': 3, 'thrown': 54, "channel's": 3, 'waylon': 1, 'pitchman': 1, "thief'": 1, 'disk': 1, 'skates': 1, '[car': 1, 'mel': 16, 'misinterpreted': 1, 'fireside': 1, 'wicked': 12, 'gordon': 12, 'symbolist': 1, 'demeans': 1, 'manifesting': 1, 'proficient': 3, 'projects': 20, 'choreographing': 2, 'endorsements': 1, 'chirping': 1, 'skateboards': 1, 'setups': 7, 'medicore': 1, 'caitlyn': 1, "zwick's": 2, 'pigeons': 2, "geronimo's": 1, 'eszterhas-ish': 1, '747s': 1, "chimp's": 1, 'teaspoons': 1, 'sofia': 1, 'chambermaid': 2, "martha'": 1, 'dialogues': 1, '$60': 1, 'egocentric': 1, 'emmerich': 7, 'one-hour': 1, 'entailing': 1, 'seth': 8, 'skirmishes': 1, 'breakup': 4, 'arguments': 8, 'boyum': 1, 'drifter-cum-postman': 1, 'compass': 2, 'cdc': 1, 'insincere': 2, 'ecstasy': 1, 'back-': 1, 'tended': 1, 'hospitalized': 1, 'on-again': 1, 'planchet': 1, 'muted': 3, 'rottingham': 1, 'absent-minded': 1, 'thrower': 1, 'straightforward': 8, 'detection': 1, 'mortgage': 2, 'robber': 6, "cruise's": 1, 'browbeats': 1, 'candy': 11, 'discourage': 2, 'sham': 3, 'wine': 4, 'radiate': 1, 'fewer': 5, 'suaku': 1, 'supervisor': 2, 'cooler': 3, 'bentley': 2, 'vampire-films': 1, 'austin': 10, 'madison': 8, 'enojyed': 1, 'come-uppance': 1, 'implement': 1, 'postapocalyptic': 1, "'judas'": 1, 'stipend': 1, 'memento': 1, 'hard-working': 2, 'efforts': 23, 'installments': 2, 'withheld': 1, 'bobcat': 1, "conscience'": 1, 'soundmix': 2, 'raffin': 1, 'sympathies': 1, "'his'": 1, 'irons-like': 1, "hyams's": 1, 'kelsey': 1, 'jr': 34, 'self-mocking': 1, 'wendkos': 1, 'charlton': 9, 'one-': 2, 'perth': 1, 'acquaintances': 2, 'isla': 1, 'dragons': 3, 'lawyer': 29, 'farm/estate': 1, 'uploading': 1, "foods'": 1, 'freakish': 1, "richard's": 1, 'eleven-o-clock': 1, 'muddies': 1, 'bio-dome': 1, 'unfortunatetely': 1, "'he": 1, 'steroids': 2, 'sharp': 16, 'broader': 1, "you's": 1, 'rate': 30, 'shivers': 1, 'echo': 5, 'college': 44, 'to-the-point': 1, 'layer': 5, 'gum': 4, 'linz': 2, 'matalin': 1, '37': 6, 'bunny': 4, "harm's": 1, 'pity': 10, 'spoon-feeding': 1, "stacey's": 1, 'imbeciles': 2, 'lesley': 2, 'greatness': 9, 'son--is': 1, "aalyah's": 1, 'listened': 3, 'gertz': 1, 'watchabe': 1, 'predominantly': 4, 'esposito': 4, 'curing': 1, 'underscored': 2, 'seafood': 1, 'finger': 14, 'obsess': 1, 'anthony': 24, 'simulation': 1, 'favorable': 1, 'fishtank': 1, 'interesting': 221, 'loved': 25, "smarter-than-you'd": 1, 'inflicted': 2, 'buzz': 9, 'nik-nak': 1, 'broadcaster': 1, 'easy--look': 1, "donny's": 1, 'interpretation': 6, 'vest': 2, 'jeans': 5, 'sinks': 9, 'near-pornographic': 1, 'yearbook': 1, 'cigarettes': 7, 'harford': 1, "kent's": 1, '8/10': 10, '[theater': 1, 'sunderland': 1, 'smash': 4, "whatever's": 1, 'katrina': 2, 'maxwell': 5, 'francesco': 1, 'remedy': 2, 'pm': 2, 'lucifer': 1, 'wowing': 1, "it's-so-bad-it's-good": 1, 'six-pack-wielding': 1, 'digitized': 1, 'squints': 1, 'wench': 1, 'belittled': 1, 'robertses': 1, 'treated': 17, 'gogolack': 1, 'scalp': 1, 'send-ups': 1, 'sydney': 5, 'speech': 30, 'occupation': 1, 'harkonnen': 1, 'germany': 3, 'four-': 1, 'uncharacteristically': 2, 'reanimator': 1, 'acting/english': 1, 'breillat': 1, 'flying': 41, 'knowledge': 21, 'columbia': 6, 'derailed': 1, 'moronic': 13, 'reporting': 1, 'tendencies': 6, 'paraphrase': 2, 'gray': 8, 'pickles': 2, '189': 1, 'exceptionlly': 1, "starbuck's": 1, 'flyboy': 1, 'stevenson': 1, 'owed': 1, 'mst3k': 1, 'marsters': 1, 'straightman': 1, 'sardonic': 3, 'actor-wise': 1, "page's": 1, 'attaining': 1, 'vile': 3, 'wile': 1, 'noirs': 1, 'binds': 1, 'dead': 131, 'unshaven': 1, 'co-produced': 2, 'annoying': 92, 'haired': 1, 'dashing': 6, 'characers': 1, 'loin': 2, 'knives': 7, "1400's": 1, 'self-titled': 1, 'professing': 1, 'cliche': 28, 'brandis': 1, 'subtract': 1, 'formed': 8, 'leon': 5, 'detract': 1, 'caro/jean-pierre': 1, 'mcgaw': 1, 'shits': 2, 'raja': 5, 'early-teen': 1, 'indoor': 1, 'robotic': 6, "stifler's": 2, 'resonates': 1, '18-foot-high': 1, 'patchy': 1, 'abusive': 7, 'winless': 1, 'edt': 1, 'halt': 6, 'crates': 1, 'swatch': 1, 'hour-and-a-half': 1, 'who\x12s': 1, 'justice': 15, 'darnell': 1, 'troop': 1, 'vince': 9, 'needlessly': 6, "danes'": 1, 'demolition': 2, 'self-reflexivity': 1, 'consult': 1, 'cherish': 2, 'title-card': 1, 'ironing': 1, 'arriving': 4, 'cross-burners': 1, 'positively': 3, 'gobbledegook': 1, 'quirkyness': 1, 'effects': 157, 'gland': 1, 'shen': 1, 'casinos': 1, 'accompany': 6, "bruce's": 2, 'obsessed': 26, 'van-damme': 2, 'mano-e-mano': 1, 'froth': 1, 'puke': 1, 'oversee': 2, 'van': 43, '_bad_': 1, 'opar': 1, 'otherworldly': 4, 'mythology': 3, 'mironoff': 1, 'glimmer': 3, 'pre-': 1, 'three-dimensional': 3, 'dewayne': 1, 'sextette': 1, 'hottest': 5, 'nil': 5, 'man-eating': 2, "kingsley's": 2, 'cuteness': 6, 'empathy': 8, 'chases': 24, 'horniness': 1, 'answering': 2, 'puncture': 1, 'sulka': 1, 'semi-interesting': 1, 'expenses': 3, 'film--especially': 1, 'sane': 5, 'dmx': 2, 'steers': 1, 'strips': 1, 'downer': 3, 'fellatio': 2, 'tuxedo-clad': 1, 'african': 15, 'jade': 2, 'ranger': 2, 'worse-than-stereotyped': 1, 'maw': 1, 'mushroom': 2, 'g-rated': 1, "kip's": 1, 'long-time': 1, 'dislikable': 2, 'execept': 1, 'steinberg': 1, "cotton's": 1, 'mush': 5, 're-recorded': 1, 'smalltime': 1, 'lull': 1, 'cummings': 2, 'violations': 1, 'hewitt': 10, 'leoni': 3, 'lawernce': 1, 'marienbad_': 1, "`enriched'": 1, 'saddest': 1, 'searching': 10, 'videocasette': 1, 'robots': 6, 'peep': 1, 'handwaving': 1, 'posts': 1, 'adaptation': 31, 'locomotive': 1, 'admirers': 1, 'refuting': 1, "mystery's": 1, 'electro-': 1, "godard's": 1, 'himselfs': 1, 'indefatigable': 1, 'autistic': 3, 'bigtime': 1, 'baggy': 1, 'exasperation': 2, 'low-brow': 3, 'flabbergasted': 2, 'stripes': 1, 'opponents': 4, 'don92t': 1, 'dynamo': 1, 'ione': 1, 'hilariously': 7, 'disaster-movie': 1, 'eachother': 1, 'molded': 1, 'chainsaw': 6, 'missable': 1, 'choosen': 1, 'spurts': 2, 'flunked': 1, 'leaving': 63, 'pebble': 1, 'simple': 76, 'ready': 44, 'falls': 106, 'deems': 5, 'repessed': 1, 'year-long': 1, "carrey's": 7, 'torn': 12, 'urban': 21, 'comedy-thriller': 1, 'athletes': 2, 'featured': 23, 'fiddling': 3, 'lacey': 1, 'anxiety': 3, 'fours': 1, 'vitarelli': 1, 'searchers': 1, 'you+ve': 1, 'al': 17, 'reaction': 22, 'minister': 3, 'here]': 1, 'mongo': 2, "stavros'": 1, 'positronic': 1, "penny's": 1, 'wrenching': 1, 'oh-so-carefully': 1, 'shake': 7, 'wattage': 1, 'palpitation': 1, 'it--and': 1, 'uninsured': 1, 'downturn': 1, 'walton': 1, 'barbarino': 1, 'lock': 3, "cbs's": 1, 'bloc': 1, 'socialite': 1, 'immigrated': 1, 'newly-pumped': 1, 'biker': 3, 'minute-and': 2, '1939': 4, 'high-flying': 2, 'ribisi': 6, "tornado'": 1, 'spaghetti-strapped': 1, 'belief': 7, 'fused': 1, 'overboard': 3, 'niece': 6, 'gere': 4, "nancy's": 1, '_knock_off_': 1, 'endless': 26, 'mountains': 10, 'continuous': 5, 'dropout': 1, 'surgery': 4, 'along--and': 1, 'letters': 10, "alice's": 2, 'makeshift': 2, 'cruz': 5, 'heavy-metal': 1, 'banana': 5, "paxton's": 1, 'whimsy': 3, 'rootless': 1, 'eluded': 2, 'illustrate': 5, 'anti-sexual': 1, 'gulia': 1, 'dempsey': 1, 'story-telling': 1, 'cohesion': 3, 'sorna': 1, "scenario'": 1, 'program': 17, "critic's": 1, 'basil': 2, 'putzulu': 1, '42': 1, 'meanings': 1, 'sheesh': 2, 'anti-depressant': 1, "halperin's": 1, 'invents': 3, 'pickett': 1, 'refreshing': 6, 'contraption': 5, 'equating': 1, 'cosmos': 1, 'involving': 90, 'panicking': 1, 're-telling': 2, 'firefighter': 2, 'three-day': 2, 'wear': 27, 'clancy': 2, "hurlyburly's": 1, '$200': 3, 'combs': 3, 'damping': 1, 'yarn': 6, 'shaye': 2, "network's": 2, 'limo': 5, 'utterly': 30, 'retardation': 1, 'actioner': 2, 'outgrossed': 1, 'napolean': 1, 'tooth': 2, 'devoid': 13, 'courted': 1, 'rabid': 8, 'loretta': 3, 'bigalow': 1, 'unrealistic-looking': 1, 'lapage': 1, 'spurned-psychos-getting-their-revenge': 1, 'ill-conceived': 7, 'journalist-turned-screenwriter': 1, 'nightclubs': 1, 'outlands': 1, 'stereotyped': 6, 'robed': 2, 'codenamed': 1, 'moralist': 1, 'processing': 1, 'gofer': 1, 'lip-synchs': 1, 'nishi': 2, 'survivor': 6, 'dangerfield': 2, 'cardiac': 1, 'stout': 2, 'double-chin': 1, '234': 1, 'tailored': 1, 'cambodia': 1, 'expressionists': 1, 'precise': 5, 'schlubby': 1, 'piled': 2, 'postlethwaite': 1, 'terry': 15, "o'fallon": 1, '_that_': 1, "morritz's": 1, 'minions': 3, "stalked's": 1, 'carton': 1, 'alcoholic': 12, '!': 341, 'sloshed': 1, "'trainspotting'": 1, 'hugh': 10, 'mountainside': 2, 'slurping': 1, '1996': 15, 'lee-zerd': 1, 'detracted': 1, 'no-one': 3, 'ditching': 1, 'redeemed': 4, 'gross-fest': 1, 'valid': 2, 'cumming': 1, 'riotous': 2, 're-cuts': 1, 'subgenre': 2, 'par': 13, '_____': 2, 'loud-mouthed': 1, 'reed': 4, 'many--alyson': 1, 'legendary': 15, 'undergoes': 3, 'lifetimes': 1, 'cheerful': 3, 'items': 3, 'cautionary': 4, 'bluth': 1, 'dressing': 7, 'morph': 1, 'latch': 3, 'catholic': 14, 'distracting': 14, 'lesser': 7, 'mothballs': 1, 'vc': 1, 'irrational': 2, 'canine': 1, 'farbissina': 1, 'brussels': 3, 'adolescence': 2, 'rivals': 6, 'construed': 2, 'coup': 4, 'strolls': 4, "governor's": 1, 'conspirators': 3, 'rollins': 4, 'porno': 7, 'short-i': 1, 'overdoes': 3, 'riiiiight': 1, 'stumble': 10, 'reboux': 1, '80s': 7, 'emmett': 1, 'sits': 14, 'haphazard': 3, 'zach': 2, 'club': 41, 'wiping': 1, 'flops': 1, 'speechless': 5, 'stairs': 2, 'envelope': 6, 'plasma': 1, 'canny': 1, 'tells': 86, 'incoherence': 2, 'hammiest': 1, 'biloxi': 1, 'nicknamed': 3, "knight's": 1, 'acceleration': 1, 'spleen': 1, '22': 4, 'unambitious': 1, 'jewel-thief': 1, 'skogland': 1, 'toughness': 1, 'blatantly': 16, 'un-produced': 1, 'hefty': 3, 'nine-year-old': 3, 'plotlessness': 1, 'mega-movie': 1, 'prof': 1, 'strands': 2, 'disturbingly': 1, 'martyr-figure': 1, '$140': 1, 'doves': 1, 'particulary': 1, 'paxton': 5, "kevin's": 3, 'blairwitch': 1, 'dimming': 1, 'savagely': 2, 'tougher': 4, 'affirms': 1, 'martian': 9, 'evolving': 2, 'testing': 5, 'sneak-outs': 1, 'mournful': 3, 'roasting': 1, 'invigorate': 1, 'trey': 2, 'mccracken': 2, 'stop': 98, 'spastic': 2, 'rainy': 3, 'detects': 1, 'hilt': 1, 'pillsbury': 1, 'civilised': 1, 'match-ups': 1, 'promise': 45, 'mud-wrestling': 1, "macdonald's": 3, 'norris': 1, 'jung': 2, "liotta's": 1, 'chauffeur': 2, 'despise': 3, 'sena': 1, 'disappears': 25, '`dead': 1, "month's": 1, 'mathematical': 1, 'belong': 9, 'agreeable': 2, 'completed': 7, 'out-perform': 1, 'fight': 89, 'everton': 1, 'thomspon': 1, 'woulda': 1, 'erstwhile': 3, "_roxbury_'s": 1, 'mentor': 9, 'know-how': 1, 'hunks': 1, 'wig': 11, 'industries': 2, 'rail': 2, 'fatale': 5, 'backbone': 2, 'anamoly': 1, 'maclachlan': 4, 'mysterious': 44, 'existance': 1, 'alumni': 1, 'partner': 49, 'extinct': 2, 'shortsighted': 1, 'curtis': 8, 'worthier': 1, 'lined': 3, 'oil-guy-cum-astronaut': 2, 'land': 34, 'subjected': 11, 'much-decorated': 1, 'disburse': 1, 'knock-knock': 1, 'coyote': 9, 'maul': 3, 'angus': 4, "miramax's": 1, 'i-iii': 1, 'pikachu': 1, 'classical': 4, 'tentacled': 2, 'transparently': 2, 'wiseguy': 2, 'cues': 2, 'new-kid-on-the-block': 1, 'abbreviated': 1, "peet's": 1, 'employee': 9, 'robbing': 7, 'helmet': 1, 'fiery': 6, 'wiretaps': 1, 'flounder': 1, 'pretentiousness': 1, 'signifiers': 1, 'tudeski': 2, 'tell-tale': 1, 'karla': 6, 'intends': 6, 'charismatic': 13, 'over-18s': 1, 'negatives': 3, 'doctors': 9, 'ex-girlfriends': 2, "get's": 1, '@#%': 1, 'clayton': 1, 'jaunt': 1, 'immensely': 11, 'greeks': 1, 'excising': 1, 'gallery': 4, 'unwanted': 1, 'raves': 6, "gadget's": 2, 'boasts': 10, 'wages': 1, 'steps': 17, 'zelweggar': 1, "mohr's": 1, 'stifle': 2, 'furlong': 1, 'mother': 89, 'tarantino-ish': 1, '1938': 1, 'facile': 2, 'layers': 2, 'goofiness': 7, 're-occurring': 1, 'retching': 1, 'sims': 2, 'georgia': 7, 'drugstore': 1, 'aliens/titanic': 1, 'attacker': 1, 'untalented': 4, 'object': 14, 'ever-reliable': 1, 'trinket': 1, 'ard': 1, 'mcgruder': 1, 'herrier': 1, 'excellence': 1, 'nntp-posting-host': 1, 'premiered': 4, 'aphrodite': 1, 'alterior': 1, 'michelangelo': 1, 'expressionistic': 1, 'crimefighters': 1, 'basement': 8, 'drug-infested': 1, 'dumping': 4, 'best-ever': 1, 'comprehendible': 1, 'spiceworld': 1, 'squeezes': 2, 'lava': 4, 'luggage': 5, 'emitting': 1, 'puppeteer': 1, 'so-called': 22, 'outstanding': 5, 'whovier': 1, 'man/half': 1, 'worldview': 1, 'hand-in-hand': 1, 'arlo': 1, 'knight': 10, 'reels': 4, 'throughout-erase': 2, 'fiberglass': 1, 'preposterously': 2, 'neighbour-socking': 1, 'true-life': 1, 'wardrobes': 4, 'salvadorian': 1, 'kayla': 1, 'squads': 1, 'eminem': 1, 'colonel': 9, 'proverbial': 11, "derek's": 1, "creator's": 1, 'skills': 28, 'choreographed': 13, 'straighten': 2, 'sexually': 11, 'sings': 3, 'humanities': 2, 'cassavettes': 1, 'humming': 3, '_h20_': 1, 'knowingly': 2, 'not-too-bright': 1, 'halfway': 33, 'braided': 1, 'chris': 56, 'olympic': 3, 'sloppy': 27, 'midnight': 9, 'fledgling': 4, 'manicures': 1, 'incident': 9, 'skyrocketed': 2, 'plants': 6, 'vylette': 2, 'half-hour': 13, 'incomprehensible': 15, 'territory': 12, 'self-treatment': 1, 'inscribe': 1, 'accept': 27, 'sci-fi': 41, 'creature': 27, 'devils': 1, 'hard-he': 1, 'picked': 18, 'submit': 3, 'market': 14, 'recycling': 5, 'fer': 1, "antonioni's": 1, 'hair-brained': 1, "capote's": 1, 'seatbelts': 1, 'gross-wise': 1, 'hurling': 2, 'kudos': 3, 'litmus': 1, 'points--just': 1, 'converging': 1, 'send-up': 2, 'hates': 15, 'abrams': 2, 'speilberg': 1, 'restless': 7, 'contractions': 1, 'b-movie': 18, 'assail': 1, 'rack': 3, 'treatise': 1, "feel'": 1, 'winnings': 2, 'kkk': 2, 'break-out': 1, '7-up': 1, 'cracked': 5, 'acquired': 1, 'evilo': 1, 'motivational': 2, 'thankfully': 21, "fries'": 1, 'fast-talker': 1, 'fiance': 7, 'consumed': 1, 'clavell': 1, 'tarnish': 2, 'dumfounding': 1, 'weinstein': 2, 'lowlife': 2, 'self-vanity': 1, 'rollerblading': 1, "letterman's": 1, 'facinelli': 4, 'interactive': 2, 'pantaeva': 1, 'climax--which': 1, 'quasi-fascist': 1, 'agility': 3, 'insecticide': 1, 'gyrating': 1, 'heathers': 4, 'hawaiian': 2, 'sil--is': 1, 'intently': 1, 'quake': 1, "go-get-'em-tiger-we're-behind-you": 1, 'moviemakers': 1, 'wcw': 5, 'geoffrey': 10, 'directors': 35, 'spontaneous': 5, "chances'": 1, 'aramis': 3, 'e-mail': 2, 'stringy': 1, 'blueprint': 2, 'droves': 2, 'stretches': 6, 'tailor': 4, 'ot': 1, 'idea--a': 1, 'boucher': 1, 'dessert': 1, 'moore': 26, 'awfulness': 5, 'skywalker': 4, 'conventionally': 1, 'pearlstein': 1, 'professionals': 2, 'provocations': 1, 'address': 7, 'implanted': 2, 'verbinski': 1, 'albino': 3, 'blowing': 6, "o'connor's": 1, 'uprorously': 1, 'lovestruck': 1, 'overstay': 1, 'natty': 1, 'bed-ridden': 1, 'gilliam': 4, 'heym': 1, 'judgments': 3, 'weird-looking': 2, 'sagas': 1, 'fastball': 1, 'hotly-contested': 1, 'jocularly': 1, 'sick-and-twisted': 1, "1984's": 4, 'headphones': 1, 'chaos': 9, 'momma': 2, "engagement'": 1, 'judicial': 1, 'running': 127, 'transports': 1, 'penance': 3, 'delayed': 2, 'mercaneries': 1, 'revealed': 22, 'palance': 1, 'phantom': 10, 'kenneth': 12, 'bravura': 1, 'inviting': 2, 'helped': 22, 'choo': 1, 'dwells': 1, 'grinning': 1, 'survey': 1, 'sleepwalking': 1, 'accent': 44, 'benz': 4, 'monster-instigated': 1, 's/m': 1, 'acts': 36, 'tear-jerking': 1, 'victories': 2, 'wookie': 1, 'cried': 4, 'colonize': 1, 'female-fearing': 1, 'message': 43, 'prisoner': 6, 'prozac': 2, 'faulkner': 1, 'telegraphed': 6, 'sunken': 1, 'boats': 2, "patrick's": 2, 'ufo-ology': 1, 'machinery': 2, 'rothwell': 1, "arnie's": 1, 'actosta': 1, 'abraded': 1, 'uglies': 1, 'tachometer': 1, 'endearing': 8, 'manchurian': 4, 'busts': 1, 'wards': 1, 'blathers': 1, 'lifelong': 6, 'contentiousness': 1, 'mulder': 3, 'lizard': 7, 'topnotch': 1, 'deny': 5, 'cheer': 10, 'injustice': 3, 'lashes': 1, 'hamburger': 1, 'snarfed': 1, 'gunpoint': 3, 'potato': 1, 'zest': 5, 'irons': 1, 'omen': 4, "pope's": 1, "`at&t'": 1, '3-7': 1, 'abundantly': 2, 'tenants': 1, 'faux-jingoistic': 1, 'struck': 8, 'newly-founded': 1, "'n'": 2, 'renaissance': 3, 'waters': 11, 'action-film': 1, 'expression-challenged': 1, "t&c's": 1, 'copperfield': 1, 'rigg': 2, "geoffrey's": 1, 'pretentiously': 2, 'routinely': 1, 'guestbook': 1, 'garage': 3, 'kicks': 20, 'encourages': 1, "slim's": 1, 'warriors': 9, 'maturing': 1, 'unwilling': 4, 'realism--in': 1, 'summarize': 2, 'tailing': 1, 'commendable': 2, 'yum': 1, 'criticisms': 2, 'cheapie': 2, 'refused': 4, 'wall-to-wall': 2, 'sunglasses': 4, 'acknowledge': 3, 'feminism': 2, 'vindicated': 1, 'meant': 47, 'spin-off': 2, 'initializes': 1, 'preposterous': 14, 'snoozer': 1, "handmaid's": 1, 'affectations': 1, 'dated': 8, 'unger': 4, 'hoo': 2, 'lying': 15, 'detonating': 1, 'proclaiming': 6, 'frail': 1, 'adopt': 2, 'point--and': 1, 'warrior': 14, 'banish': 1, 'half-star': 2, 'merienbad': 1, 'womanizer': 5, 'adjacent': 1, 'film-maker': 4, 'flashdancer': 1, 'memory': 21, "o'quinn": 1, 'hard': 172, 'finals': 3, 'all-girl': 1, 'conveniently': 10, 'narcissistically': 1, 'narrator': 10, 'nightie': 1, 'invite': 2, 'fresh-faced': 1, 'fueled': 2, 'stern-': 1, 'samantha': 4, 'precedes': 2, 'plotline': 9, 'truffaut': 1, 'militant': 2, 'dazed': 3, 'nominations': 5, "o'day": 1, 'pokes': 2, 'bai': 2, "'dog-in-danger'": 1, 'ole': 1, 'exploitation': 16, 'seeked': 1, 'compulsively': 1, 'nazi-style': 1, 'bricks': 1, 'gismos': 1, 'middle-aged': 6, 'contemplates': 1, 'night-vision': 1, 'releases': 12, 'ex-government': 1, 'craftors': 1, 'rationalization': 2, 'uncommercial': 1, 'cartoony': 2, 'omaha': 1, 'mounted': 1, 'keaton': 10, 'covers': 12, 'bubblegum-blond': 1, 'laurel': 1, 'sleeper': 3, "khondji's": 1, 'razzle-dazzle': 1, 'mook': 1, 'pact': 4, 'bummer': 2, 'heroin-chic': 1, 'invited': 9, 'winstone': 2, 'hip-to-only-themselves': 1, 'mug': 5, 'muscle-bound': 1, 'flimsy': 8, 'annie': 9, 'beholder': 3, 'moribund': 1, 'dubbing': 6, 'opens': 63, 'spy-': 1, 'sleepers': 1, 'karate-chopping': 2, 'enterprise': 10, 'effortlessly': 3, 'forgoes': 1, "robbins'": 3, 'horny': 16, 'lower-class': 1, 'heigl': 1, 'screenwriting-is-hell': 1, 'impossibilities': 1, 'grievous': 3, 'despondence': 1, "mid-80's": 1, '20-': 1, 'issues': 25, 'smoker': 1, 'da': 3, 'crestfallen': 1, 'log': 1, 'ushered': 2, 'kidlets': 1, 'gen-y': 1, 'protector': 2, '-esque': 3, 'misanthropy': 1, "clique's": 1, 'whisper': 2, 'indy': 5, 'maryland': 1, 'leslie': 12, 'engaged': 11, 'ravages': 1, 'breuer': 2, 'light-plane': 1, 'relinquish': 1, 'holidaygoers': 1, 'narrate': 1, 'debra': 1, 'meanders': 5, 'playmate': 3, "`messiah'": 1, 'niche': 1, 'sadistic': 12, 'coarser': 1, '24/12/99': 1, 'christie': 2, 'suppression': 1, 'farrelly': 7, 'yawn-provoking': 1, 'slow-going': 1, 'repress': 1, 'outworld': 3, 'outlaw': 3, 'deadline': 2, 'lags': 1, 'unable': 20, 'forgiving': 4, 'kindergarten': 3, 'swells': 2, 'drug': 36, 'benjamin': 2, 'little--': 1, 'hunts': 3, 'unpleasantness': 2, 'croft': 6, 'inflated': 1, 'movie-going': 3, 'kick': 17, 'dope-smoking': 1, 'erotic': 20, '12-': 1, 'mcginley': 4, 'hoop': 2, 'preparation': 3, 'acting': 261, 'rejects': 4, 'petey': 1, 'inhibitions': 1, 'clearing': 1, 'docked': 1, "iguana's": 1, 'vie': 1, 'chow': 8, 'isolation': 5, 'willy': 1, 'grapples': 2, 'ex-partner': 1, 'nico': 1, 'values': 21, 'restoration': 1, 'widest': 1, 'outlet': 1, 'fright': 9, 'failed': 48, 'screeching': 5, 'reduce': 4, 'pentathol': 1, 'blawp': 1, 'endearment': 1, 'replied': 1, 'critically': 5, 'baddie': 4, 'bunz': 1, 'target': 37, 'mattress': 1, 'fingernails': 2, 'hensley': 1, 'flocking': 1, 'heightens': 1, 'crapped': 1, 'suggesting': 3, '28-year-old': 2, 'campers': 1, 'arbiters': 1, 'interfere': 3, 'clinton': 2, 'gena': 2, 'deviously': 1, "ryder's": 1, 'grenier': 1, 'moody': 4, 'jolting': 2, 'infidelity': 2, 'cup': 10, '55': 1, 'perk': 1, 'wimpy': 1, 'segments': 4, 'douglas': 18, "penelope--it's": 1, 'b-grade': 6, 'stir': 6, 'put': 166, 'unfathomable': 1, 'miscues': 2, 'neat': 16, 'return': 70, 'mushrooms': 2, 'hendren': 1, 'scavenge': 1, 'bros': 11, "shopkeeper's": 1, "gittin'": 1, 'shocker': 1, 'terminal': 5, 'gallagher': 1, 'kitchen': 9, 'hard-hitting': 1, 'overtaken': 2, 'coating': 2, 'depp': 10, 'aspires': 1, 'star-power': 1, '_four_': 1, 'perfectly': 34, 'high-schoolers': 1, 'wiseass': 3, 'agile': 2, 'capriciously': 1, 'flashes-sideways': 1, 'ten-foot': 1, '102': 2, "freakin'": 2, 'trod': 1, "pitt's": 1, 'hatches': 1, "'comedy": 1, 'vinci': 1, 'hyperreal': 1, 'neighbourhood': 2, 'toy': 11, 'editor': 15, 'groans': 1, 'brides': 2, 'daniel': 18, 'climact': 1, "town's": 5, 'fully-armed': 1, 'instructor': 2, 'poopie': 1, 'ribbons': 2, 'fedora': 1, 'rush': 26, 'himbos': 1, 'ornate': 1, 'jeanne': 2, 'untimely': 5, 'chunnel': 1, 'uneven': 12, 'runtime': 6, 'gestures': 1, 'perches': 1, "olds'": 1, 'stunt-work': 1, 'kruger': 3, 'hunch': 2, 'schwarzneggar': 1, 'displeased': 1, 'piza': 1, 'poacher': 1, 'tabloid': 4, 'trick': 21, '2': 109, 'mahem': 1, 'technology-': 1, 'selected': 7, 'buckley': 2, 'self-depreciating': 1, 'updates': 2, 'perceptive': 1, 'alums': 1, 'ceiling': 4, 'suspension': 10, "fighter's": 1, 'brinkley': 1, 'counter-productive': 1, 'jaw': 1, 'seductress': 1, 'sparing': 1, 'hometown': 3, 'cheapened': 2, 'journeyman': 1, 'cool': 75, 'movie--': 2, 'thing__about': 1, 'sever': 1, 'confrontation': 13, 'move': 76, 'scaffold-like': 1, 'freaking': 2, "`starship'": 1, 'kahlberg': 1, 'descended': 2, 'trashing': 1, 'saboteur': 1, 'duty': 7, 'fremen': 1, 'gaffer': 1, 'tragedies': 1, 'deeds': 1, 'cry': 18, 'sais': 1, 'repectively': 1, 'jean-claude': 8, 'scaffolding': 1, "year's": 58, 'sound': 76, 'molina': 2, 'umpteenth': 2, 'swore': 2, 'sindel': 2, 'zelllweger': 1, 'sink': 10, 'bully': 6, 'roaches': 1, 'endangering': 2, 'cbs': 3, 'unusually': 4, 'clarisse': 1, 'overrides': 1, 'showtime': 3, "pharoah's": 1, 'cinematographer-turned-director': 1, 'fallacy': 1, 'unrighteous': 1, 'swings': 4, 'intermittent': 2, 'guitar': 3, 'nausea': 1, 'volumes': 1, 'ashley': 8, 'landlady': 2, "writer's": 6, 'peckinpah': 3, 'attractive': 30, 'beck': 1, 'techno-babble': 1, 'sun': 19, 'zen': 2, 'splashed': 1, "atkinson's": 1, 'convention-breakers': 1, 'faulty': 1, 'archery': 1, 'bungled': 1, 'punt': 1, 'indestructible': 1, '_pecker_': 1, 'baggage': 3, 'remembers': 2, 'exorbitantly': 1, 'suspect': 33, 'qui-gon': 2, 'williamson-less': 1, "mctiernan's": 2, 'indisputable': 1, 'skipping': 1, 'moses': 2, 'stuck': 43, 'leaden': 8, 'overachieving': 1, 'transpire': 1, 'school': 92, 'trys': 1, "think's": 1, 'wilderness': 2, 'paleontologist': 2, 'offense': 3, 're-generating': 1, 'parlay': 1, 'greenhouse': 1, 'restaurant': 17, 'costumes': 22, 'positive': 28, 'too-cheesy-to-be-accidental': 1, 'sez': 1, 'well-executed': 1, 'back': 307, 'noxious': 3, 'confessional': 1, "snipes's": 1, 'fecal': 1, 'luther': 1, "elwood's": 1, 'agonizingly': 2, 'dot': 3, 'cooperative': 2, 'attar': 1, 'requisitive': 1, 'rebuttal': 1, 'ado': 6, 'disgusted': 4, 'contagion': 2, 'detracts': 2, 'iceberg': 3, 'mimics': 2, 'forgiven': 6, 'yak-fest': 1, "manson's": 1, 'invulnerable': 1, 'filler': 9, 'fringe': 1, 'species': 24, 'dinky': 2, '_there_': 1, 'trickery': 1, 'posey': 3, 'kwik-e-mart': 1, 'reference': 14, 'paget': 1, 'mishears': 1, 'turner-owned': 1, 'astonished': 3, 'bundle': 3, 'outgoing': 1, 'titillating': 2, 'elasped': 1, 'feuds': 1, 'privilege': 4, 'six-astronaut': 1, 'rex': 1, 'theatrical': 17, 'explained': 35, 'matches': 2, 'dweeb': 1, "arnold's": 4, 'unsinkable': 1, 'procures': 1, 'dialouge': 1, 'midget': 3, 'whipped': 4, 'tarheels': 1, 'sub-section': 1, 'chums': 1, 'republics': 1, 'basing': 3, 'sets': 70, 'low-rent': 2, 'blending': 2, 'mind-blowing': 1, 'ab': 1, 'beat': 30, 'shamelessly': 6, 'imaginative': 5, 'drawn-out': 5, 'ecological': 1, 'one-armed': 3, 'soprano': 1, 'relentlessly': 6, 'avengers': 6, 'implausibility': 3, 'snakes': 5, 'yielded': 2, 'brunet': 1, 'vinnie': 4, 'inexplicable': 12, 'but-is-shot': 1, 'sensitive': 9, 'credit--': 1, 'stanze': 1, 'barn': 2, 'scowl': 1, 'ranft': 1, 'lungs': 2, 'brazilian': 5, 'concrete': 5, 'hysterical': 8, 'else--and': 1, 'expands': 1, '1/8': 1, 'character-setups': 1, 'danvers': 1, 'alot': 1, "cryin'": 2, 'removed': 18, 'santoro': 2, 'co-star': 14, 'soon-to-be-notorious': 1, 'well-realized': 1, 'arab': 5, "patch's": 2, 'gladiator': 1, 'symphony': 1, 'footsteps': 7, 'handsome': 14, 'scheming': 6, 'obscenity': 1, 'begins': 118, 'implicitly': 2, 'edgy': 7, 'grader': 1, 'disrespect': 1, 'cases': 16, 'exposition': 9, "rubick's": 1, 'korea': 1, 'innocuous-enough-on-the-surface': 1, "phillippe's": 1, 'icing': 1, 'bret': 2, 'fabrications': 1, 'dime-a-dozen': 1, '`dad': 1, 'allegiance': 1, 'hrs': 1, 'aggressive': 9, '98': 4, 'ex-junkie': 1, 'revenge-for-hire': 2, 'standouts': 1, 'fab': 2, 'above-mentioned': 1, 'talent-challenged': 1, 'socially': 3, 'christopher': 43, 'wrestling': 12, 'esophagus': 2, '1521': 1, 'farcical': 2, 'greenlit': 2, 'atypical': 1, 'post-': 3, 'parody': 30, 'persona': 19, 'painfully': 47, 'legend': 30, 'plopped': 1, "hughes's": 1, 'littered': 3, 'lavatorial': 1, 'react': 6, "`boob'": 1, 'drastically': 6, 'jungles': 2, 'psychos': 1, 'weariness': 1, 'fully-realized': 1, '`that': 1, 'ryan-nicolas': 1, 'overcrowded': 3, 'haplessly': 1, 'groom': 1, 'fantasizes': 2, 'shelter': 1, 'beret-wearing': 1, 'puzzle': 11, 'publically': 1, 'logo': 7, '1991': 6, 'obscene': 4, "alcott's": 1, 'buffoon': 2, 'heaps': 4, 'understandeably': 1, 'launchers': 1, 'earthlings': 1, 'mathou': 1, 'super-intelligence': 1, 'awakener': 1, 'types': 15, 'greene': 2, 'noxima': 1, 'also-ran': 1, 'shuffled': 1, 'kibbe': 1, 'beaudene': 1, 'sprint': 1, "sausages'": 1, 'rain': 19, "porky's": 1, 'process': 27, "underwood's": 2, 'inexcusably': 1, 'glass': 10, 'lush': 4, 'clue': 26, 'candidate': 9, 'captors': 2, 'sideways': 1, 'anchorman': 1, 'bangkok': 1, 'quiclky': 1, 'cloned': 2, 'unforgivable': 5, 'signatures': 1, 'writings': 3, 'frenzy': 5, 'unspoken': 3, 'prosecution': 1, '9mm': 1, 'kinda': 10, 'live-in': 4, 'commandeer': 1, 'bacchanal': 1, 'assessment': 3, 'philosophizing': 1, 'crooked': 7, 'mann': 5, 'believable': 43, 'lot': 220, 'premise': 82, 'slicing': 1, 'insomnia': 5, 'jumping': 17, 'nicknames': 1, 'automobiles': 5, 'humane': 1, 'sketchy': 2, 'cerrano': 2, 'cleaner': 3, 'auditions': 1, 'stingy': 1, 'well-timed': 2, 'hallucinations': 5, "o'": 2, 'headful': 1, 'aggie': 1, 'badies': 1, 'shepard': 1, 'derelict': 2, 'travails': 3, 'gregory': 3, 'disposable': 8, 'schrieber': 2, 'aback': 2, 'milton': 2, 'floorboard': 1, 'licence': 1, 'inflating': 1, 'cringes': 4, 'gyrations': 1, 'turnarounds': 1, 'pursues': 6, "duvall's": 1, 'thurral': 1, 'dickie': 1, 'dream': 39, 'panning': 3, "cuckoo's": 4, 'themed': 1, 'implant': 3, 'tara': 2, 'concludes': 5, 'fundamentally': 5, 'degredation': 1, 'unironic': 1, 'uploaded': 1, 'avika': 1, 'deem': 1, 'raved': 1, 'aberration': 1, 'sammy': 4, 'power': 66, 'ultimately': 53, 'daredevil': 3, 'stubborn': 2, 'noticably': 2, 'mecilli': 1, 'intones': 1, 'despair': 6, 'glimpses': 10, 'imbuing': 1, 'accuser': 1, 'umbrellas': 1, 'egg-headed': 1, 'wronged': 3, 'starves': 1, 'half-empty': 1, '(': 855, 'open-heart': 2, 'schematic': 1, 'indulgently': 1, 'expelled': 2, 'murderer': 12, 'assasinated': 1, 'stabile': 1, 'veterans': 7, 'izzard': 1, 'konner': 1, 'geekiest': 1, 'muscular': 3, 'lads': 2, 'dickens': 5, 'friskiness': 1, 'credits': 83, 'access': 6, 'unhappily': 2, 'outfits': 11, 'feel': 152, 'odoriferous': 1, 'officially': 2, 'benefit': 14, "60's": 4, 'fry': 2, 'planes': 7, 'makeovers': 1, 'crapper': 1, 'narrow-waisted': 1, 'village': 16, "legend's": 1, 'stud': 2, 'kickass': 1, 'newsradio': 1, 'gleeson': 2, 'backdrop': 7, 'reincarnation': 1, 'thuggish': 1, 'witherspoon': 4, 'skg': 2, 'pleasing': 9, 'remarked': 2, 'ridulous': 1, 'model-turned-actress': 1, 'sheets': 5, 'galloping': 1, "matty's": 1, 'predators': 1, 'scratch': 6, 'elm': 6, 'uli': 1, 'monumentally': 5, '_snl_': 1, 'esteem': 3, 'embodied': 1, 'leopard': 2, 'screechy': 1, 'nominees': 1, 'jagger': 1, 'strengthens': 1, 'ramis': 4, 'urinates': 1, 'immobile': 1, 'scions': 1, 'kate': 15, 'symptomatic': 1, 'laugh': 103, 'password': 1, 'innuendoes': 3, 'masturbates': 1, "grandson's": 1, 'renny': 2, 'reconcile': 2, 'sweetness': 4, 'nino': 1, 'mask': 18, 'kimble': 1, "chow's": 1, 'captured': 16, 'champs': 1, 'route': 10, 'frost': 4, 'miriam': 1, "'60s": 2, 'protege': 1, 'thomas': 34, 'memorial': 2, 'punishments': 1, 'rear': 7, 'kristen': 1, 'bothered': 9, 'strays': 1, 'rapped': 1, 'bomber': 2, 'angrily': 2, 'barking': 1, '#3': 1, 'barcelona': 2, 'eagles': 1, 'modestly': 1, 'james-who': 1, 'certifiable': 2, 'top-billed': 2, 'empathic': 1, 'graveness': 1, 'dre': 1, 'consoles': 1, 'gamble': 4, 'deciphering': 1, 'strong': 64, 'grandest': 1, 'sdds': 1, 'tedious': 29, 'wallace': 5, 'portrayal': 14, 'obi-wan': 2, 'simpering': 2, "butcher's": 2, 'frame': 26, 'laboured': 3, "feldman's": 1, 'actor/magician': 1, 'rename': 1, 'appropriateness': 1, 'churn': 1, 'effectiveness': 5, "good'": 1, 'injuries': 4, 'printed': 5, 'managing': 1, 'organize': 1, 'actor-turned-directors': 1, 'chance--unless': 1, 'chose': 19, 'varsity': 6, 'plissken': 1, '1993': 14, 'chop-chopped': 1, 'hoodoo': 1, "people's": 5, "wendy's": 1, 'indistinct': 1, 'cleaned': 1, 'injects': 1, 'neverending': 1, 'sex-crazed': 5, 'nivola': 2, 'promises': 14, 'docu-drama': 1, 'witted': 1, 'spooked': 1, 'favour': 3, 'suitcases': 2, 'film-school-grad': 1, 'helmut': 1, 'dahl': 1, 'well-meaning': 3, 'nina': 4, 'bouncing': 4, 'distressing': 1, 'heartless': 7, 'hitchhiking': 1, 'backlot': 1, 'hurricaine': 1, 'faculty': 4, 'stare': 12, 'newly-restored': 1, 'rowing': 1, 'low-down': 1, "forever'": 1, 'eventful': 1, 'forgives': 1, 'expressway': 1, 'bodies': 17, 'reliability': 1, "epps'": 1, 'gangster-flick': 1, 'bean': 7, "philip's": 2, 'airport': 10, 'liquor': 5, 'pike': 2, 'dog-half': 1, 'unown': 1, 'ambition': 7, '_jerry_maguire_': 1, 'held': 23, 'buddha': 1, 'movie': 717, 'unceremoniously': 1, 'slashfest': 1, 'wad': 1, 'prolong': 2, 'unscary': 3, 'joaquin': 5, 'overcast': 1, "jenny's": 1, 'tapes': 5, 'thal': 1, 'edge': 25, "fleder's": 1, 'scamper': 1, 'engages': 4, 'hullo': 2, 'concentrate': 5, 'writer/director/co-star': 1, 'perpetrated': 2, 'actuality': 4, 'bull-crap': 1, 'duplication': 2, 'professory': 1, 'buying': 7, 'farces': 1, 'recruits': 5, 'bind': 1, 'chicken-bashing': 1, "`jaws'": 1, 'create': 55, 'high-heaven': 1, 'kang': 4, 'cirque': 1, 'camouflage': 1, 'pace--was': 1, 'plates': 2, 'description': 11, "she'd": 10, "_polish_wedding_'s": 1, "'nuff": 2, 'slap': 5, 'straightens': 1, 'platitudes': 2, '-': 190, 'project': 86, 'jerk': 10, 'birth': 17, 'gang': 34, 'hunter': 18, 'lifelike': 2, 'traveller': 1, 'act': 95, 'marking': 3, 'drought': 1, 'fc': 1, 'emilio': 3, 'bane': 2, 'well-developed': 2, 'staggers': 1, 'deceptions': 1, 'prizes': 1, 'intrinsic': 2, 'cosmic': 2, 'in-concert': 1, 'assets': 3, 'explosives': 1, 'spokesman': 1, 'fine-': 1, 'retreating': 1, 'hamlet': 2, 'afar': 3, 'skipp': 1, 'cookie-cutter': 4, 'gripes': 1, 'roache': 1, "eve's": 5, 'ignore': 12, 'buildup': 4, 'forties': 2, 'wholesome': 3, 'wished': 8, 'damon': 3, "killer'": 1, 'ibenez': 1, 'hala-russell': 1, 'reshooting': 1, "alzheimer's": 3, 'soothsayer': 1, "children-it's": 1, 'emphasizes': 1, 'scrumptiously': 1, 'carnage': 13, "witness'": 1, 'nixon': 3, 'ham': 3, 'complicating': 1, "connie's": 1, 'petulant': 2, 'crappiness': 1, 'feels': 74, 'battered': 1, 'jeroen': 1, 'tired': 42, 'inferno': 6, 'ammunitions': 1, 'agency': 4, 'new-born': 2, 'leone': 2, 'welles': 11, 'strikeout': 3, 'tiger/land': 1, 'blessing': 3, 'athos': 4, 'actresses': 17, 'drape-hanging': 1, 'terrifically': 1, 'unknowns': 3, 'sacrifices': 3, 'boondocks': 1, "cool's": 1, '$50000': 1, "biebe's": 1, 'bounderby': 1, 'mature': 8, 'tyson': 1, 'gossamer-thin': 1, 'half-expect': 1, 'ape': 12, 'causing': 17, 'hodgman': 1, 'anthropologist': 6, 'spinoff': 1, 'vorld': 1, 'bulldozer': 2, 'builds': 9, 'hand-to-hand': 1, 'newly-created': 1, 'super': 19, 'ami': 1, 'two-foot': 1, 'immunity': 2, 'vieluf': 1, 'careful': 5, 'delivered': 23, "harding's": 1, 'jolts': 1, 'fornication': 1, 'advised': 7, 'remarrying': 1, 'cannibalize': 1, 'kung-fu': 4, 'self-pity': 1, 'pulleys': 1, 'gm': 1, 'assure': 2, 'biopic': 3, 'proposition': 6, 'harris': 9, 'goldberg/ted': 1, 'ship': 43, 'surrender': 2, 'sending': 11, 'vet': 2, 'catherine': 27, 'avenger': 2, 'nickolas': 2, 'unoriginality': 2, 'cower': 1, 'masterwork': 1, 'patients': 12, 'rhythm': 4, 'sorrow': 2, 'tis': 2, 'prayed': 1, 'wayne': 24, 'razzle': 3, 'gills': 1, 'gritty': 9, 'obscurities': 1, 'elaborate': 26, 'ayla': 1, "_double_team_'s": 1, 'big-headed': 1, 'coughlan': 2, 'cranky': 5, 'furs': 1, 'yamamoto': 1, 'co-sex': 1, 'crunchable': 1, 'wasps': 2, 'male-ego-stroking': 1, 'intelligent': 47, 'prime-time': 1, 'iv': 3, 'cranks': 1, 'sordidness': 1, 'yippee': 1, 'trait': 11, 'faint': 2, 'sanctimony': 1, 'unkind': 1, 'missles': 1, 'takeover': 1, 'granted': 24, 'satirizing': 4, 'remarkable': 14, 'decoy': 1, 'solving': 1, 'war-era': 1, 'speirs': 1, 'sideline': 1, "goodman's": 2, 'georgie': 1, '79': 1, 'horse': 21, 'australia': 4, 'pairing': 8, 'super-hero': 3, 'theories': 4, 'means': 62, 'slipups': 1, 'disparate': 2, 'disoreitating': 1, 'contemptuous': 2, 'keels': 1, 'hats': 1, 'uncharismatic': 2, 'dreariness': 1, 'kinks': 2, "fahey's": 1, 'recommitted': 1, 'mangled': 4, 'logs': 1, 'votes': 1, 'waller': 1, '40': 11, 'wrinkle': 1, 'machinas': 1, 'cigars': 1, 'nominated': 10, 'border': 8, 'pre-school': 1, 'french-accented': 1, 'snitch': 1, 'fashions': 3, 'pretty': 202, 'fort': 6, 'ridiculousy': 1, 'thermopolis': 1, 'kutcher': 1, 'rool': 1, 'firth': 2, "couple's": 2, 'brim': 2, 'burrows': 3, 'cross-cutting': 1, 'legion': 1, 'indigestion': 1, 'collect': 8, 'longingly': 1, 'cabalistic': 1, 'glop': 1, 'explores': 4, 'foreground': 2, 'demand': 7, 'interchange': 1, 'downs': 5, 'exclusively': 4, 'workhorse': 1, 'whistles': 2, "inmates'": 1, 'showiest': 1, 'iota': 4, "'n": 2, 'validates': 1, 'super-rich': 1, "'hit": 1, 'chain-smoking': 3, 'advances': 7, 'epiphanies': 1, 'dishonourable': 1, 'carried': 13, 'elements': 84, 'underused': 9, 'begotten': 1, "institution's": 1, 'gilmore': 4, 'novo': 1, 'sonja': 1, 'insistence': 5, "vehicle's": 1, 'tootsie': 2, 'oceanographic': 1, "paris'": 2, 'fictional': 5, 'kool': 1, "'only": 1, 'tylenol': 1, 'twiggish': 1, 'publishes': 1, 'throw': 46, 'shameless': 10, 'exterminate': 1, '80-minute': 1, 'bellowing': 2, 'functions': 2, 'mickelberry': 1, 'invariably': 4, '1970s': 8, 'infamy': 1, 'wasted': 81, 'tomato': 2, "'geniuses'": 1, 'suggestive': 2, 'devolve': 1, 'merriment': 1, 'marcel': 1, 'sophomore': 2, 'truely': 1, 'terminator-like': 1, 'chopra': 1, 'earmarks': 1, 'conversions': 2, 'echoes': 7, 'lent': 2, 'societies': 1, 'cruisers': 2, 'spouts': 4, "detorri's": 1, 'peters': 3, 'horrors': 6, 'fiercely': 1, 'flora': 1, 'hulk': 2, 'linked': 3, 'at-a-glance': 1, 'endured': 7, 'continues': 23, 'godfrey': 1, 'madeleine': 4, 'oops': 7, 'contribution': 2, 'scrutinize': 2, 'retail': 2, 'wickedness': 1, 'thug': 4, 'methods': 8, 'busboy': 2, 'adultrous': 1, "ferrara's": 1, 'apes': 10, 'clone': 15, 'transferred': 4, 'lifestyles': 1, 'federation': 4, 'shooting': 39, 'ed': 20, 'combination': 16, '_this_': 1, 'perlich': 1, 'tact': 1, 'surpasses': 1, 'grief': 6, 'uncharted': 1, 'poodle': 1, 'vortex': 2, 'bowen': 1, 'insists': 10, 'joely': 6, 'kidvid': 1, 'corruption': 7, 'rising': 22, 'shoddy': 9, 'vests': 1, 'up-and-coming': 1, 'compass--her': 1, 'red-handed': 1, 'ambivlaent': 1, 'unwinds': 1, 'studdly': 1, 'film\x12s': 1, 'marijuana-laced': 1, 'colton': 2, 'refuses': 19, "mamet's": 1, 'giuseppe': 2, 'evict': 1, 'insulated': 1, 'lurking': 3, 'alex': 15, 'fanatical': 2, 'dietz': 1, 'sapping': 1, 'five-year-old': 2, 'sadomasochists': 1, 'rectangles': 1, 'profitably': 1, 'deluge': 3, 'max': 27, 'balance': 7, '31st': 1, "hunter's": 1, 'voicework': 1, 'swans': 1, 'drew': 14, 'stockpiling': 1, 'transformation': 9, 'contradictory': 3, 'forte': 3, 'medical': 12, 'relations': 5, 'experts': 3, 'replaced': 22, 'devilish': 2, 'finesse': 2, 'jurgen': 1, 'entrance': 3, 'casualties': 2, 'feature': 71, 'sacking': 1, 'schl': 1, '117': 1, 'self-appointed': 2, 'errand': 1, 'baotian': 1, 'demise': 8, "i'll-convince-people-to-kill-each-other": 1, 'covering': 8, 'plummer': 4, 'motives': 13, 'roxanne': 1, 'troops': 12, 'paraplegic': 1, 'kennel': 1, 'replete': 5, 'longs': 4, 'dropper': 1, 'wounds': 6, 'job': 154, "husband's": 14, '100m': 1, 'amicable': 2, 'dastardly': 1, 'fully-clothed': 2, 'resembling': 10, 'roll': 22, 'guild-mandated': 1, 'writer/creator': 1, '4am': 1, 'non-linear': 1, 'offspring': 2, "harry's": 9, 'lifeform': 1, 'screamers': 1, 'enjoys': 11, 'tinkering': 3, 'amendment': 1, 'detroit': 5, 'vehicle': 33, 'herman-make': 1, 'appearing': 21, 'websites': 1, 'oversized': 3, 'opened': 11, 'self-loathing': 2, 'expect': 77, 'tenseness': 1, 'prue': 1, 'counsellor': 1, 'haunt': 9, 'vanquished': 1, 'forms': 11, 'spanking': 1, 'sleeves': 1, 'eisner': 1, 'encounters': 19, 'commemorate': 1, 'producers': 38, 'psyche': 6, 'zhivago': 1, 'jennings': 1, "fitchner's": 1, 'researchers': 2, '90s': 5, 'humanity': 27, 'bootleg': 1, 'raimi': 2, "workers'": 1, '2001': 17, 'afoot': 1, 'prefer': 6, 'graver': 1, 'blitz': 2, 'commend': 1, 'mconaughy': 1, 'interrupt': 2, 'occupied': 1, 'dollars': 34, 'turkies': 1, 'human-animal': 1, 'callousness': 1, 'speeder': 1, 'amalgamation': 3, 'grad': 2, "man's": 31, 'british': 38, "headey's": 1, "'if": 1, 'frampton': 1, 'failures': 6, 'inauspiciously': 1, 'induced': 3, 'prattle': 1, 'devotion': 7, 'ironies': 1, 'massey': 1, 'exaggerated': 3, 'dodie': 1, 'dig': 10, 'nbc': 1, 'brings': 59, 'exploitative': 6, 'diverse': 4, 'cheesecake': 1, 'scatter': 1, 'strobe': 5, "everyone's": 17, 'implode': 1, 'sliding': 2, 'prison-turned': 1, "leonardo's": 1, 'raunchy': 7, 'slow-paced': 1, 'get-go': 6, "'seven": 1, 'prison': 42, '_saturday_night_live_': 2, 'tantalizing': 1, 'sponsorship': 1, 'shout': 7, 'way-back': 1, 'convictions': 2, 'calibrated': 1, 'mary': 32, 'latter-day': 1, 'cruising': 1, 'hellbent': 1, 'hygiene': 2, 'dogs': 14, 'shirtless': 3, 'dorky-looking': 1, "ken's": 1, "author's": 2, 'qb': 1, 'tortuous': 2, "1978's": 2, 'plausibly': 1, 'mishmash': 3, 'dip': 2, 'demonstrate': 8, 'hum-drum': 1, 'capt': 4, 'candlelight': 2, 'moulin': 1, 'tribune': 2, 'boy': 91, 'wisely': 7, 'domain': 1, 'yuckity-yuck': 1, 'confidential': 4, 'shellulite': 1, 'robbie': 4, 'creaky': 12, 'husbands/boyfriends': 1, 'peaks': 6, 'duration': 5, 'employed': 9, 'mettler': 1, 'witnesses': 9, 'outer-city': 1, 'arrow': 9, 'glorified': 2, 'unconventional': 8, 'coincidence': 12, "ac/dc's": 1, "night'": 1, 'shudder': 5, 'audio/video': 1, 'huffing': 3, 'laughing': 46, "mia's": 1, "hark's": 1, 'erected': 1, 'shaken': 2, 'superimposed': 1, 'movie-making': 3, 'squiggly': 1, 'decorative': 1, 'stems': 1, "landis'": 2, 'summit': 1, 'huff': 2, 'high-profile': 5, 'couch': 6, 'epilogue': 5, 'hang-dog': 1, 'boring': 151, 'gassed': 1, 'nincompoops': 3, 'trough': 1, 'peaceful': 3, 'espn': 1, 'century': 36, 'extra-curricular': 1, '`princess': 1, 'glad': 16, 'construct': 3, 'shootings': 1, 'arrest': 9, 'judging': 16, 'greasy': 1, 'embrace': 8, "bird's-eye-view": 1, '1991-france': 1, 'flake': 1, 'prioritized': 1, 'imbue': 1, 'risk': 14, 'deflowered': 1, 'unmistakably': 1, 'awakes': 2, 'also-rans': 1, 'tirelessly': 1, 'finn': 4, 'energize': 1, 'timeline': 1, 'dissatisfied': 2, 'veggie': 1, '10-year-': 1, "coote's": 1, 'exterior': 1, 'muffle': 1, 'victim': 38, "public's": 1, 'joylessness': 1, 'devereaux': 1, "gorton's": 2, 'pulse-pounding': 3, 'virtual': 9, 'tectonic': 1, 'psycho-in-love': 1, 'fiancee': 9, 'glimpse': 12, 'bees': 1, 'south': 22, 'censorship': 2, 'flabby': 1, 'resigning': 1, 'controllers': 1, "mainstream's": 1, "nikki's": 1, 'shopkeeper': 1, 'patently': 6, 'undid': 1, 'tigerland': 1, 'flashiness': 2, 'gist': 1, 'text': 10, 'fooled': 7, "rock'n'roll": 1, 'reply': 5, 'larroquette': 1, 'cherokee': 2, 'venality': 1, 'loeb': 2, 'accuse': 1, 'exceeds': 2, 'peeled': 2, 'figures': 22, 'nipple': 2, 'rampage': 6, 'biebe': 1, 'less-than-stellar': 2, 'doherty': 1, 'decriminalization': 1, 'dominant': 3, 'inadvertently': 3, 'octopus': 2, 'assorted': 3, 'filmation': 1, 'advantage': 17, 'coliseum': 2, 'iced': 1, 'pot-obsessed': 1, 'hou': 1, 'hughes': 13, 'ambiguous': 8, 'traffic': 6, 'threesome': 2, 'terrifying': 9, 'chained': 1, 'zillion': 1, 'replicas': 1, 'natural': 20, 'activated': 1, 'kurt': 12, 'pounders': 1, 'scruffy': 4, 'maxims': 1, 'ages': 13, 'sigmoid': 1, 'sanitation': 1, 'worth': 113, 'cum-islamic-missionary': 1, 'cromwell': 12, 'finale': 37, 'halloween': 22, 'sketch-comedy': 1, 'brite': 1, 'disproportioned': 1, 'feat': 7, 'dollop': 1, 'organic': 2, 'mikel': 1, 'fetish': 5, 'febre': 4, "know's": 1, 'decisions': 12, 'mind': 132, 'drone': 1, 'personnel': 1, 'well-worn': 3, 'link': 12, 'homefront': 1, 'stupid-ass': 1, 'klendathu': 2, 'lagoon': 2, 'mad': 34, 'engine': 5, 'roz': 1, 'expirating': 1, 'vacationers': 1, 'compendium': 1, 'lax': 1, "mayor's": 1, 'super-intelligent': 2, 'netting': 1, 'snidley': 1, 'restrictions': 1, 'coughing': 2, 'magnet': 1, 'debacle': 6, "queens'": 1, 'distinguishable': 1, 'delivers': 36, 'telecast': 1, 'beckoned': 1, 'dinosaurs': 7, 'relatives': 6, 'vindictive': 3, "pepper's": 1, 'flash-forwards': 1, 'interrupted': 9, 'protectors': 1, 'bats': 6, 'eensy': 1, 'capra': 1, "rule'": 1, 'seduces': 2, 'sixty': 2, 'visit': 25, 'francesca': 3, 'anarchic': 2, 'flail': 1, 'earth-normal': 1, 'fish': 16, "brenda's": 2, 'stuffs': 1, "shelton's": 1, 'unnoticed': 3, 'navigated': 1, 'liveliness': 1, "jay's": 1, 'phenomenas': 1, 'discount': 4, 'ecl@mtcts2': 1, 'dubois': 2, 'dolenz': 1, 'warning': 22, "robbin's": 2, 'violins': 1, 'nuns': 4, '*': 119, 'horrendously': 2, 'gifted': 7, 'yahoo': 1, 'opning': 1, "belushi's": 1, 'joke--a': 1, 'trimming': 1, 'worthless': 13, 'port-o-potty': 1, 'che-kirk': 1, 'tinges': 1, 'unpardonable': 1, 'visually': 20, 'sex-driven': 1, 'mfm': 1, "`rumble'": 1, 'roads': 5, 'cheesefest': 1, "'speed'": 1, "dreyer's": 1, 'daylight': 5, 'reid': 2, 'protagonist': 37, 'electric': 3, 'cole--': 1, 'beebe': 1, 'cashier': 1, 'washout': 2, 'providence': 4, 'plots': 25, 'proxies': 1, 'dormies': 1, 'mastery': 3, 'election': 6, 'entirety': 8, 'incapable': 8, 'sublime': 1, 'siblings': 4, 'empty-headed': 4, 'deliciously': 7, 'chainsaw-massacre-type': 1, '04': 1, "grant's": 3, 'brace': 4, 'monotonous': 10, 'heretofore': 2, 'steely': 1, 'criticising': 1, 'cuban': 3, 'brightly-lit': 1, 'romantically': 3, 'substitute': 4, 'casted': 3, 'porn-film': 1, 'deniro': 4, 'officer': 37, 'boisterous': 1, 'january': 1, 'subkoff': 1, 'sigh': 5, 'primary': 14, 'fulfill': 5, 'undisciplined': 1, 'posters': 2, 'foils': 1, 'evokes': 2, 'evolve': 5, 'barring': 1, 'pin-up': 1, 'xin-xin': 2, 'sanctioned': 1, 'clean-up': 1, 'director-cinematographer': 1, 'duguay': 2, 'entrusted': 2, 'mariachi': 1, 'insistance': 1, '1940s': 1, "sandler's": 5, 'premise--kafka': 1, 'long-lost': 1, 'predictions': 2, 'moustache': 4, 'nonreligious': 1, 'pleasure-seeking': 1, 'in-command': 1, 'role-reversal': 1, 'tots': 1, 'toll': 2, 'handheld': 2, 'mobs': 1, 'warned': 18, 'crab': 4, 'cities': 7, 'sliver': 1, 'lawrence': 14, 'revolves': 23, 'retrospective': 1, 'heroin': 13, 'survives': 10, 'presidents': 2, 'whittaker': 2, 'blackboard': 1, 'hound': 1, 'fractured': 1, 'shorts': 3, 'derivative': 16, 'slathering': 1, 'obnoxious': 23, 'sub-plot': 17, 'exchanging': 1, 'tinkertoy': 1, 'aaaaaaaahhhh': 1, 'elucidate': 1, 'monastery': 3, 'implausable': 1, 'holly': 8, 'newsletter': 1, 'clay': 4, 'struggles': 10, 'promote': 2, 'clad': 6, '137': 1, 'feisty': 6, 'tremors': 5, 'dundee': 1, 'underbelly': 1, 'fantasyland': 1, 'escapes': 23, 'parallel': 9, 'abreast': 1, 'claw': 5, 'sherilyn': 2, 'wright': 7, 'pearly': 1, 'sockets': 4, 'everywoman': 1, 'detained': 1, 'oldman': 6, 'burton': 10, 'physical': 34, 'chides': 1, 'funnery': 1, 'dons': 5, "carolco's": 1, 'inherently': 7, 'kinds': 16, 'acquaintance': 1, 'bollocks': 2, "away's": 1, 'hysterically': 4, 'cyber-doctor': 1, 'opportunities': 13, "forsythe's": 1, 'jai': 3, 'coarse': 1, "peacemaker'": 1, 'decapitate': 1, 'bedrock': 1, "roberts's": 1, 'sliminess': 1, 'squabbles': 1, 'advanced': 8, 'prostitute': 11, 'reign': 6, 'cronenberg--is': 1, 'utilizes': 4, 'cook': 10, 'crumble': 2, 'rossi': 1, 'leftover': 2, 'overplays': 1, 'cow': 4, 'reunited': 3, 'expired': 1, 'thrillingly': 1, 'quickest': 1, 'cloaked': 1, 'isle': 1, 'perplexed': 1, 'stomp': 1, 'bash': 6, 'eldest': 3, 'terrorizes': 1, "willis'": 6, 'shadows': 10, 'fly': 31, 'boredom': 11, 'vibrating': 1, 'remind': 16, '1954': 3, 'grieco-esque': 1, 'religious': 19, 'pessimistic': 2, 'spoken': 9, 'enlightenment': 1, 'comedically': 1, 'whirlwind': 3, '\x13they': 1, 'isolated': 13, 'view': 39, 'set-pieces': 2, '30-second': 2, 'saturday-morning': 1, 'splendid': 2, 'dispite': 1, 'laserdisc': 3, "tingle's": 2, 'blam': 1, 'rationalize': 1, 'stoned': 5, 'copout': 1, 'godawful': 6, 'cutthroat': 2, 'brakes': 1, 'junkie-i': 1, 'hero-less': 1, 'conceptualization': 1, "'enough": 1, 'hayman': 3, 'sickens': 1, 'girl-type': 1, 'pseudoerotic': 1, 'unforgiveable': 3, 'remove': 10, 'pratfalling': 1, 'marceau': 2, 'home-video': 1, 'piles': 7, "creature's": 1, 'cuba': 10, 'leashes': 1, 'foursome': 3, 'erased': 2, 've': 4, 'mili': 1, 'carves': 1, 'hormonal': 3, 'biggs': 9, 'soft-': 1, 'skyscraper': 1, 'justifiably': 1, 'intention': 14, 'posterior': 1, 'wider-open': 1, 'kietal': 1, 'can-can': 1, 'accounted': 1, 'bottom': 41, 'unwillingness': 2, 'titanic-like': 1, 'prominant': 1, 'squinting': 1, '-rydain': 1, 'petty': 6, 'twentysomethings': 1, "'amusing": 1, 'tentacle': 1, 'rope-bridge': 1, "chester's": 1, "stu's": 1, "ricci's": 3, 'impress': 12, 'concentration': 2, 'liman': 1, 'malroux': 3, 'fs': 1, 'dollops': 1, 'bertolini': 1, 'harrisburg': 1, "alzhiemer's": 1, 'fuck': 5, 'extort': 1, 'made-for-disney': 1, 'subpar': 2, 'high-concept': 4, 'recitals': 1, 'landscape': 6, 'adam': 32, 'rescuing': 4, 'sevigny': 4, 'cohort': 3, 'interacted': 1, 'emotionless': 5, 'spirituality': 2, 'shrunk': 1, 'allegorically': 1, 'ticking-clock': 1, 'letup': 1, 'albinos': 2, 'bosley': 1, 'undiscerning': 1, 'fawning': 3, 'glider': 1, 'nazi-charged': 1, 'grunge': 1, 'celebrates': 2, 'rambo-style': 1, 'obstacles--such': 1, 'privates': 1, 'print': 9, 'vic': 4, 'up-and-comers': 2, 'elisa': 1, 'revelations': 3, 'yancy': 1, 'mozzell': 1, 'pabulum': 1, 'collaboration': 7, 'pot': 11, 'traci': 2, 'gruel': 1, 'captions': 1, 'abominable': 2, 'impervious': 1, 'snuggly': 1, 'shit': 8, 'rammed-down-your-throat': 1, 'lifeforms': 1, "koontz's": 1, 'airplane-style': 1, 'clutches': 5, "counterpart's": 1, 'memphis': 3, 'impersonator': 2, 'adrift': 1, 'comedienne': 1, "'rmd'": 1, 'phrase': 9, 'six-year-old': 1, 'wild-eyed': 2, '--stiff': 1, 'superhuman': 3, 'jumbo': 4, 'tearfully': 2, "'traffic": 1, 'modern': 40, 'bubblegum': 1, 'traumatised': 1, 'avital': 2, 'illness': 7, 'cheerleaders': 3, 'lisbon': 1, "attorney's": 1, 'wilmer': 1, 'melodrama': 20, 'pronounces': 3, 'wacky': 17, 'instincts': 9, 'women': 87, 'stand-ins': 1, 'oscar-nominated': 1, 'establishing': 6, 'gaps': 8, 'uber': 3, 'mostly-misfired': 1, 'good/there+s': 1, 'hacking': 1, 'version': 84, 'notable': 18, 'raines': 2, 'footsy': 1, 'xena': 2, 'cells': 4, 'bewilderment': 3, 'lux': 1, 'incorrectly': 3, 'cereal': 4, 'atomic': 4, 'haliwell': 1, 'enlighten': 4, 'obstructive': 1, 'arrived': 7, "[editor's": 1, 'humanitarian': 1, 'sixties': 5, 'enigmatic': 5, 'confederation': 1, "luger's": 1, 'reprint': 1, 'pretty-in-pink': 1, 'romantic-comedy': 1, 'completely': 167, 'spaghetti': 4, 'love-interest': 1, 'crispin': 1, 'constrictor': 1, 'earp-style': 1, 'fragile': 2, 'forgetable': 1, 'fought': 6, 'blasphemy': 2, "parson's": 1, 'robyn': 1, 'worthwhile': 20, 'played': 248, 'wittiness': 2, 'co-ordinator': 1, 'mcdonaldburger': 1, 'conservative': 1, 'trader': 4, 'fled': 4, 'ravaging': 1, 'world-class': 1, "esposito's": 1, 'macleane': 2, 'featherstone': 4, 'weathered': 1, 'sex-kitten': 1, 'progressing': 2, 'gekko': 1, 'watered-': 1, 'appointment': 2, 'man-animals': 1, 'balbricker': 1, 'genuineness': 1, 'hang-ups': 1, 'bishop': 1, 'completists': 1, "bogie's": 1, 'series-someone': 1, 'popsicle': 2, 'hoods': 1, 'workshop': 2, 'popping': 8, 'junky': 2, 'horseracing': 1, 'skateboarders': 1, 'narrative': 28, 'atrocious': 17, 'rewarding': 3, 'messages': 13, 'air-headed': 2, 'claptrap': 1, 'reentry': 1, "cass's": 1, 'icy': 4, 'mid-1960s': 1, 'phenomenon': 8, 'hatcher': 5, 'views': 7, 'wink': 3, 'rut': 2, 'earnest': 10, 'cured': 1, '`you': 3, 'third-billing': 1, 'bateer': 1, 'comments': 27, 'rejoice': 1, 'stranger': 19, 'skies': 3, 'dolby': 3, 'mother-daughter': 3, 'hesitate': 2, 'added': 30, 'five-hundred-pound': 1, 'entered': 6, 'craters': 1, "shane's": 3, 'keenen': 2, 'non-existent': 10, 'dodgers': 1, 'usurper': 1, 'cross-dressing': 3, 'smuggling': 1, 'revell': 1, 'digger': 1, 'asteroids': 2, 'must-be-improvised': 1, 'lifts': 2, 'voodoo-inspired': 1, 'shockingly': 9, 'scholarship': 6, 'oafish': 1, 'ring-wearing': 1, 'swelled': 2, 'czechoslovakia': 1, 'bluff': 1, 'levy': 3, 'injecting': 2, 'overlit': 1, 'truth': 40, 'kraft': 2, 'jodi': 1, 'constitute': 1, "george's": 2, 'houseboat': 4, 'chilling': 3, 'languid': 1, 'drinks': 5, 'invokes': 1, 'weddings': 6, 'allred': 1, 'cellular': 4, 'hating': 5, 'mother-in-law': 5, 'meyer': 14, 'anti-heroes': 1, 'margin': 2, 'nhl': 1, 'mohamed': 1, 'delgado': 1, 'crux': 1, 'elegant': 4, 'urging': 1, 'downhill': 17, 'colin': 4, 'blucas': 2, 'adds': 40, 'controls': 3, 'impose': 1, 'dispose': 2, 'lyonne': 5, 'trendily': 1, 'glycerine': 1, 'believing': 9, 'cinematographers': 5, "hawn's": 1, 'prude': 2, 'teeny-bopper': 1, 'complained': 2, 'unwritten': 2, 'gut': 4, 'shutterbug': 1, 'satanic': 1, 'accident': 27, 'rocketship': 1, 'fill': 30, 'fight-scenes': 1, 'unmenacing': 1, '104-minute': 1, 'cloris': 2, 'checking': 8, 'sweetheart': 4, 'mcarthur': 1, 'supposedly': 47, 'ringed-planet': 1, 'auctioneer': 1, 'gags': 49, "betsy's": 1, 'comically': 5, 'ridden': 4, 'awoke': 1, 'chandler/ross': 1, 'high-caliber': 2, '`any': 1, 'watergate': 2, 'ninety': 6, 'belching': 2, 'predict': 9, 'hydraulic': 1, 'yields': 2, "sinyor's": 1, 'rainforest': 1, '53-year-old': 1, 'globel': 1, 'slithering': 1, 'meanies': 1, 'coco': 1, 'pissing': 1, 'days--as': 1, 'sharply': 2, 'diminutive': 1, 'shunned': 2, 'suitor': 1, 'accessible': 1, 'tearing': 6, 'virtuoso': 1, 'throwback': 1, 'soured': 1, 'bowfinger': 2, 'relocate': 2, 'fantasies': 14, "amanda's": 3, 'villard': 1, 'kooky': 2, 'terrible-occasionally': 1, 'much-loathed': 1, 'ni-sen': 1, 'neilsen': 1, 'sit-com': 5, 'disposition': 1, 'wit--': 1, 'commissioner': 2, 'crossroads': 2, "loach's": 1, 'rainier': 1, 'victimized': 1, 'double-dealing': 1, 'hateful': 6, 'deciphered': 1, 'busy': 11, 'masked': 6, 'tyrone': 1, 'cookie': 7, 'unsound': 1, 'hockey': 7, 'whirry': 2, 'pitying': 1, 'limitless': 2, 'reciting': 5, 'hilarities': 1, '3=poor': 1, 'typed': 2, "hopkins's": 1, 'enabling': 2, 'confrontational': 1, 'issue': 21, 'pileup': 2, "n'": 2, 'westernized': 1, 'sorely': 8, 'sharing': 6, 'pursing': 1, 'rappaport': 2, 'ivan': 5, 'vartan': 1, 'potter': 4, 'ratings': 7, 'autobiography': 2, 'include': 57, 'dominates': 3, 'au': 1, 'phrases': 7, 'indiscernable': 1, 'password-protected': 2, 'harvest': 2, 'audrey': 8, 'grisoni': 1, 'bullshit': 2, 'silly-sounding': 1, 'demographics': 1, 'goldthwait': 1, 'crowbar': 1, 'unafraid': 2, 'faintest': 2, 'baghdad': 1, 'rewrite': 7, 'thin-looking': 1, 'fertilization': 1, 'pile': 21, "'hilarious'": 1, 'lolita': 4, 'drinking': 22, 'crater': 1, 'terrorizing': 4, 'economics': 2, 'advocate': 8, 'violent': 48, 'maintained': 6, 'stellan': 3, 'youthful': 1, 'subdues': 1, 'woody': 27, 'jana': 1, 'evidenced': 3, 'clashes': 2, 'occaisional': 1, 'phones': 8, 'rebel': 8, '10-minute': 1, 'gheorghe': 3, 'flashlight': 1, 'z-movies': 1, 'vampiric': 1, 'stritch': 1, 'almost-halfway-there': 1, 'branching': 1, 'coming': 102, 'era': 7, 'majorly': 1, 'potty-humor': 1, 'assuming': 8, 'openly': 4, 'mouse': 8, 'duck': 9, 'coolest': 5, 'scent': 2, 'yam': 1, 'after-hours': 1, 'pathetically-written': 1, 'compel': 1, 'depalma': 7, 'shenanigans': 3, 'verdict': 2, 'fair': 52, 'evaluation': 2, 'semi-stall': 1, 'contrasting': 3, 'creation': 6, 'actionfest': 1, 'manic': 14, 't-1000': 1, 'eulogy': 3, 'humerous': 1, 'frankenstein': 5, 'warhol': 1, 'awake': 14, 'laila': 1, 'stuff--an': 1, 'sip': 1, 'roof-top': 1, 'messes': 3, 'dials': 1, 'godzila': 1, 'murderers': 2, 'straights': 1, "nathan's": 1, 'late': 101, 'inanimate': 2, 'genetics': 1, 'event': 30, 'contributes': 4, 'arts': 19, 'evacuates': 1, 'supportive': 1, 'meschach': 1, 'phrygia': 1, 'tattoo': 2, 'corner\x12s': 1, 'costumed': 5, 'stripper': 8, 'hike': 2, 'gift': 16, 'one-liners': 27, 'homages': 3, 'mortally': 1, 'referee': 1, 'ppk': 1, 'documents': 3, 'boobies': 1, 'herrings': 3, 'overlapping': 1, 'yikes': 1, 'crusty': 2, 'climate-controlling': 1, 'sock-em': 1, 'undirected': 1, '1865': 1, 'bother': 34, 'brainless': 9, 'fbi': 25, 'transposing': 1, 'reprising': 3, "britain's": 1, 'supercop': 2, 'ancestors': 1, 'cotner': 1, 'humid': 1, 'clarkson': 3, 'galore': 3, 'unrealistic': 17, 'sighs': 1, 'woe': 2, "brasco's": 1, "spazio'": 1, 'forests': 2, 'journey': 21, 'laboratory': 1, "elfman's": 2, 'thwarted': 3, 'richelieu': 4, 'delinquents': 1, 'stages': 8, 'seymour': 1, 'astonishingly': 3, "fargo's": 1, 'beats': 13, 'spreads': 2, 'halliwell': 2, 'clubbing': 2, '`dumb': 1, 'jazzy': 2, 'paired': 5, 'bookend': 1, 'songs': 29, "palma's": 3, 'disillusioned': 6, 'prevention': 1, 'tripplehorn': 1, 'nigger': 1, 'eager': 10, 'lapaglia': 1, 'inadequate': 4, 'supposedly-intellectual': 1, 'goings-on': 6, 'extensive': 7, 'ultra-orthodox': 1, 'ravell': 1, 'summation': 1, 'personal': 45, 'ticker': 1, 'deep-voiced': 2, 'screws': 6, 'infinity': 1, 'amber': 1, 'parallax': 1, 'ooooooooh': 1, 'fast': 61, 'matured': 1, 'exemplifies': 3, 'required': 21, 'canet': 1, 'amc': 1, 'credence': 2, 'splashing': 1, 'literal': 3, 'lightly': 6, 'liv': 5, 'ex-special': 1, '12-year-old': 1, 'buckwheat': 2, 'conflicts': 8, 'rednecks': 4, 'rub': 1, 'breathy': 1, 'feces': 2, 'garlington': 1, 'observer': 2, 'matchmaker': 3, 'rave': 4, 'scored': 4, 'musketeers': 6, 'hiker': 1, 'phenomena': 3, 'normandy': 1, 'sputter': 1, 'damning': 2, 'possibility': 15, 'terrio': 1, 'amuse': 1, 'electro-magnetic': 1, 'uniforms': 5, 'crept': 1, 'idaho': 1, 'blood-suckers': 1, 'situations--into': 1, 'nun': 5, 'filming': 18, "'batman": 1, 'ha-ha': 1, 'trucking': 1, 'evident': 18, 'vultures': 3, 'mechanic/musician': 1, 'child-like': 1, 'houston': 2, 'screenwriting': 13, 'tycoon': 4, 'overscored': 2, 'knack': 5, 'playboy': 7, "oprah's": 1, 'unclothed': 1, 'moustaches': 1, 'explodes': 6, 'scans': 1, 'goddammit': 1, 'rya': 1, 'preliminary': 1, 'lambskin': 1, 'round-robin': 1, 'pestering': 1, 'northam': 1, 'fourth-graders': 1, 'bliss': 3, 'danner': 4, 'apropos': 1, 'fiend': 1, 'weasel': 1, 'fanatasies': 1, 'babysitting': 2, 'ransom': 13, "douglas'": 1, 'ambitiously': 2, 'mention': 53, 'autopsy': 1, 'flustered': 3, 'jeremy': 7, 'center': 27, 'mile-a-minute': 1, 'pre': 1, 'junkets': 1, 'philippe': 1, 'high-fiving': 1, 'confiding': 1, 'delves': 3, 'transitions': 3, 'golden': 16, 'straight': 82, 'sunglass-wearing': 1, 'semester': 4, 'inbred': 4, 'dopey': 5, "herrmann's": 1, 'motorcade': 1, 'in-between': 2, 'accompanied': 10, 'astray': 1, 'nope': 7, 'subtitling': 1, 'evaporated': 2, 'gentleman': 7, 'virtues': 4, 'sepia-colored': 1, 'rifles': 4, 'irritated': 3, 'battlefield': 10, 'scrubbed': 1, 'smurfs': 1, 'careers': 14, 'triggers': 1, 'meek': 3, 'sole': 34, 'stick-to-what-you-do-best': 1, "sequel's": 3, 'rid': 10, 'silliest': 4, 'unfazed': 1, 'tore': 2, 'make-shift': 1, 'glue': 1, "bilko's": 2, 'witness': 25, 'barrymore': 11, 'doggy': 1, "coyle's": 1, 'poets': 2, 'hick': 3, 'chloe': 7, 'lead-faced': 1, 'zelig': 1, 'austen': 1, 'working': 81, 'listless': 4, 'obtained': 3, 'optic': 1, 'occupy': 2, 'lustfully': 1, 'momentarily': 5, 'quit': 9, 'long-long': 1, 'traced': 5, 'spaceballs': 1, 'shread': 1, 'newt': 1, 'doyle': 4, 'mohawk': 1, 'nincompoop': 1, 'attending': 7, 'resistible': 1, 'permanently': 2, 'plummeting': 1, 'les': 4, 'companies': 4, "swingin'": 1, 'gaudy': 2, 'transmission': 1, 'vacuum': 2, 'dangers': 4, 'cheated': 9, 'st': 6, 'reminisces': 1, 'snowstorm': 2, 'unparalleled': 1, 'gester': 1, 'grimace': 2, 'eagerly': 5, 'jimmie': 2, '_and_': 3, 'perennial': 3, 'bat': 13, 'flat-top': 1, 'irina': 1, 'evacuate': 5, 'waitresses': 1, 'unskilled': 1, 'flawlessness': 1, 'blazingly': 1, 'overdosing': 1, 'alter': 2, 'worthiness': 1, 'playing': 130, 'dredging': 1, 'vessey': 1, 'elwood': 2, 'fondle': 1, 'black-and-white': 1, 'swept': 5, 'publicity': 8, 'addressing': 1, 'cease': 3, "godzilla's": 2, 'umm': 2, 'vargas': 1, 'buscemi': 9, 'perverted': 4, 'farmers': 1, 'confusing': 45, 'spliced': 3, 'jaye': 1, 'sexy': 28, 'field': 24, 'orser': 1, 'wizard-of-oz-fashion': 1, 'impresses': 1, 'fulfilment': 1, 'prig': 1, 'snidely': 1, 'whatnot': 3, 'killer-taunting-his-victim-on-the-phone': 1, 'buggery': 1, 'architect': 5, 'righteous': 2, "pistols'": 1, 'terrestrial': 1, 'pack': 6, 'osemt': 1, 'self-absorption': 1, 'dory': 1, 'staff': 8, 'faux-cute': 1, 'brilliant': 36, 'advertisements': 4, 'featuring': 25, 'abe': 2, 'yore': 1, 'relatively-brief': 1, 'presenting': 6, 'falling-out': 1, 'housekeeping': 1, 'distinguishing': 2, 'favreau': 3, "soldier's": 1, 'elektra': 1, 'carvey': 1, '21st': 1, 'barzoon': 1, 'doubles': 1, 'polly': 1, 'marvellous': 3, 'hillary': 1, 'sustained': 2, 'distracted': 9, 'ratio': 10, 'easy-bake': 1, 'kale': 1, 'armies': 2, 'campbell': 18, "she's": 131, 'bandaras': 1, '_leave': 1, 'affectionate': 2, 'subtexts': 1, 'autism': 3, 'errors': 5, "`total'": 1, 'sh--': 1, 'opener': 3, 'materialistic': 2, 'shoot-out': 2, 'go-carts': 1, 'gears': 3, 'subsequently': 7, 'pools': 2, 'interest': 99, 'razzmatazz': 1, 'celebrating': 3, 'clip': 7, 'swoosie': 1, 'beam': 2, 'wrecked': 1, 'moisture': 1, 'worf': 2, 'advocating': 1, 'solitude': 1, 'recut': 4, 'intelligence': 55, 'catalog': 1, 'substance': 40, 'students': 19, 'bitter': 18, 'dang': 1, 'masterson': 3, 'skye': 2, 'disprove': 2, 'pants': 12, 'near-flawless': 1, 'overly-familiar': 1, 'amounting': 1, 'crippled': 3, 'russkies': 1, 'waiting': 49, 'melodramas': 2, 'suicide': 22, 'researcher': 1, 'laforge': 1, 'minimal': 9, 'inventor': 4, 'purr': 3, 'inconsistent': 9, 'dole': 2, 'underutilized': 2, "hoffman's": 3, 'martinis--and': 1, 'swap': 1, "jia-li's": 1, 'avenging': 2, 'good--but': 1, 'lofty': 2, 'unintelligble': 1, "sayles'": 1, 'blindfolded': 3, "luc's": 1, 'diaper': 1, 'nodding': 1, 'fast-paced': 9, 'difficulties': 3, 'slickster': 1, 'notch': 7, 'suave': 6, 'lookalikes': 2, 'strip': 22, 'iran': 1, 'cliffs': 1, 'greenway': 1, 'happy--yet': 1, 'modicum': 2, 'glitch': 1, 'flanery': 1, 'zucker': 4, 'lack': 99, 'darabont': 1, 'nasal': 1, "verhoven's": 2, 'half-baked': 4, "i'm-going-to-stare-you-down": 1, 'instinct': 15, 'dripping': 5, 'postmodernism': 1, 'hovering': 1, 'hammy': 7, "officer'": 1, 'header': 2, 'side': 92, 'ketchum': 2, 'revulsion': 1, 'impersonated': 1, 'spot-on': 1, 'founding': 2, 'bonding': 3, 'eclectic': 5, 'incarnation': 9, 'superficial': 16, 'administering': 1, 'remained': 8, 'fortunate': 1, 'sprung': 1, 'jiro': 2, 'filimg': 1, 'barnacles': 1, 'note': 77, 'goofy-looking': 1, 'quasi-ominously': 1, "1992's": 1, 'phoenix': 7, 'appealing': 31, 'england': 16, 'snippet': 2, "1994's": 7, '106': 2, 'boxed': 1, 'two-year': 2, 'furiously': 3, 'happen': 89, 'awakened': 3, "campbell's": 2, 'food-eating': 1, 'message-id': 1, 'infancy': 1, 'lianna': 1, 'reply-to': 1, 'rows': 2, 'altar': 8, 'roldan': 4, 'lifelessness': 1, "deming's": 1, 'distributing': 1, 'priscilla': 4, "mike's": 1, 'dj': 3, 'seaman': 1, 'vera': 4, 'high-security': 1, 'horndogs': 1, 'brannagh': 1, 'televisions': 1, 'prior': 10, 'jumbled': 9, 'recklessly': 1, 'addict': 8, 'vivacious': 1, 'clever': 63, 'letter-opener': 1, 'super-smart': 2, "1980's": 5, 'relishes': 3, 'oncoming': 2, 'luis': 4, 'recognise': 1, 'hourly': 1, 'challenges': 5, 'mega-buck': 1, 'cords': 2, 'hyper-erratic': 1, 'flotsam': 1, 'wars': 20, 'blade': 19, "evil'": 1, 'lela': 5, 'name-brand': 1, 'hopelessly': 14, 'semi-autobiographical': 1, 'shores': 4, 'riddler': 2, 'curtolo': 1, 'funneled': 1, 'botch': 3, 'gleaming': 2, 'storyboard': 1, "'zoot'": 1, "hurricane's": 1, 'chuckle': 18, 'nudnik': 1, 'jonathan': 23, 'overtones': 5, 'spence': 1, 'roxane': 1, 'scarring': 1, 'holier': 1, "o'barr": 1, 'departments': 1, 'webster': 1, "'neck'": 1, 'for--in': 1, 'buddy-type': 1, 'rome': 3, 'flik': 1, 'once-revered': 1, 'yanks': 1, 'soil': 3, 'pee-wee': 1, 'rocked': 1, 'zack': 2, 'sufficient': 5, 'escaping': 5, 'hallmark': 8, 'sidekicks': 3, 'kidnapped': 17, 'drool': 3, 'lending': 2, 'super-light': 1, 'possum': 1, 'provokes': 4, 'preferences': 1, '16': 6, 'overpopulation': 3, 'harding': 1, '/': 12, 'precision': 4, 'donner': 3, 'stupidest': 6, 'partygoers': 1, 'kava': 1, 'bottom-of-the-barrel': 5, "elisabeth's": 2, 'pfieffer': 1, 'absurd': 30, 'cherry': 1, 'banzai': 2, 'dwindling': 1, 'grisly': 2, 'jett': 1, 'wartime': 3, 'tread': 1, 'billie': 3, 'compilation': 1, 'humor--as': 1, 'sunday': 7, 'stand-up': 6, 'rita': 3, 'scientology': 5, 'switched': 1, 'flores': 4, 'democratic': 1, 'volcanoes': 2, 'reheal': 1, 'moo-moo': 1, 'vamping': 1, 'linking': 3, "sly's": 1, "bachelor'": 2, 'ex-army': 1, 'complexly': 1, 'stride': 2, 'receiving': 4, 'simulate': 2, 'vidal': 1, 'ricky': 6, 'puffing': 4, '12-hour': 1, 'poked': 3, 'dumb': 70, 'birthday': 16, 'marcelo': 1, 'urbanite': 1, "krippendorf's": 4, 'state-of-the-art': 3, 'brighter': 3, 'poet/songwriter/': 1, 'plug': 1, 'volley': 1, 'cabbies': 1, 'pygmalion': 2, 'tub': 5, 'effete': 1, 'basinger': 5, 'heal': 1, 'brat': 4, 'freshly': 2, 'ringing': 1, 'instructed': 3, 'worker': 8, 'dwarfs': 4, 'touched': 4, 'checkout': 1, 'spoonful': 1, 'robert--or': 1, "serina's": 1, 'animalities': 1, 'armoire': 1, 'jobeth': 2, 'psycho': 19, 'thoughtfully': 2, "gwaaaa's": 1, 'fellas': 3, 'reflect': 3, 'whim': 5, 'cutherton-smyth': 1, 'one-joke': 8, 'anomaly': 1, 'documentary': 26, 'repetitiously': 1, 'beethoven': 1, 'donaldson': 1, 'photography': 23, 'examining': 3, 'shagged': 2, 'rascally': 1, 'claudio': 1, 'burrito': 1, 'ominously': 1, "february's": 1, 'vietnam': 6, 'pop-culture': 1, 'umu': 1, 'accommodating': 2, 'self-': 1, 'frisson': 1, 'previously': 28, "danielle's": 1, 'shreds': 3, 'davison': 2, 'wanna': 6, 'cursed': 2, 'eccentric': 21, 'diffused': 1, 'evangelist': 1, 'alight': 1, 'roberts': 15, 'films-': 1, 'hold': 54, 'darker': 2, 'cagey': 1, 'strung': 5, 'morse': 4, 'hooky': 1, 'related': 14, 'stench': 1, 'he-': 1, '60s': 6, 'thier': 1, 'sexploitation': 2, 'assault': 11, 'unrelenting': 3, 'classless': 1, 'traveling': 6, 'poorer': 2, 'hostages': 2, 'soft-porn': 2, 'unsuccessful': 7, 'new-improved': 1, 'shrewdly': 1, 'record': 24, 'scuzzy-looking': 1, 'unrealized': 1, 'imaginable': 7, 'backed': 3, 'sunning': 1, 'torment': 1, 'directorial': 21, 'overshadow': 1, 'increasingly-': 1, 'hbo': 6, 'edie': 2, 'auteurs': 1, 'tits': 3, 'cosmo': 1, 'insipid': 15, 'ex-spouse': 1, "1982's": 1, 'depressing': 25, 'lena': 4, 'overlooks': 1, 'two--then': 1, "`showgirls'": 1, 'lindo': 8, 'weirdoes': 1, 'acerbity': 1, 'malone': 6, 'explored': 18, "air'": 1, 'eventually': 73, 'messing': 1, 'disappointment': 37, 'honking': 1, 'limited-circulation': 1, 'blood-and-gore': 1, "schneider's": 2, 'clothed': 1, 'nifty': 14, 'figured': 14, 'eye-popping': 3, 'pondering': 2, 'duncan': 7, 'outburst': 1, 'clutter': 3, 'aires': 3, 'flip-of-the-coin': 1, 'abounds': 1, 'brightest': 3, 'subtext': 3, 'paull': 1, 'reset': 1, 'well-documented': 1, '_but': 1, '16-year-olds': 2, 'ruggedly': 1, 'simplified': 3, 'cristofer': 3, 'ovitz': 2, 'betray': 1, 'grungy': 3, 'superhero': 9, 'vinny': 4, 'guys/bad': 1, 'siege': 3, 'protected': 5, 'kent': 2, 'whiny': 6, 'dumfounded': 1, 'firing': 10, 'infected': 4, 'fishy': 1, 'yost': 2, 'solves': 2, 'piers': 1, "arraki's": 1, 'ex-secretary': 1, 'duo': 15, 'werewolfism': 1, 'sectors': 1, 'resorts': 6, 'country-western': 2, 'spurned-psycho-lover-gets-her-revenge': 1, 'improving': 2, 'jettisons': 1, 'earnest-to-a-fault': 1, 'occupies': 3, 'remarry': 1, 'shambles': 1, 'depth': 43, "data's": 1, 'matchsticks': 1, 'setting': 41, 'booze-filled': 1, 'caustic': 1, 'behave': 7, 'levinson-directed': 1, 'dissected': 1, 'basket': 2, 'rhames': 4, 'riots': 1, 'att': 1, 'grossness': 2, 'pulitzer': 2, 'synthesized': 1, 'cunning': 1, 'mailed-in': 1, 'sweeps': 1, 'ball-picking': 1, "doogie's": 1, 'indulgent': 2, 'normal': 23, 'singlehandedly': 1, 'renditions': 1, 'inform': 4, 'opera': 16, 'confirmed': 5, 'tongue': 8, 'tommy': 25, 'mayer': 1, 'self-talk': 1, 'retiring': 2, 'whores': 1, 'hurlyburly': 2, '--dolls': 1, 'personified': 1, 'relentless': 5, 'laflour': 1, 'standard': 51, "subway's": 1, 'pare': 1, 'kersey': 2, 'dishonest': 1, 'whittled': 2, 'minus': 8, 'reduced': 13, 'grasp': 10, 'dougray': 3, 'adults': 28, 'murray': 14, 'bring': 85, 'confessionals': 1, 'dating': 8, 'handicap': 1, 'dishonor': 1, 'reubens': 3, 'lionel': 2, 'deviate': 3, 'frat': 3, 'something-or-the-other': 1, 'modeling': 3, 'draping': 1, "'exorcised'": 1, 'tuckers': 1, 'owens': 2, 'scharzenegger': 1, 'shoveling': 1, 'dismemberments': 1, "nympho's": 1, 'gap': 3, 'trans': 1, 'historian': 1, 'immediately': 59, 'goddess': 3, 'scotty': 2, "jack's": 4, 'lay': 4, 'urinary': 1, 'enmity': 1, 'cloud': 4, 'becomming': 1, 'coordinated': 1, 'sorted': 1, "danny's": 2, 'fading': 2, "kline's": 3, 'deafened': 1, 'unsatisfactory': 4, 'undeniably': 4, 'miscarriage': 2, 'street-smart': 1, 'piet': 1, "caine's": 1, 'portobello': 1, 'soviets': 1, 'drawing-room': 1, 'farms': 1, 'spectrum': 1, 'alliance': 3, 'montgomery': 1, '35-year-old': 1, "characters'": 9, 'crowned': 1, 'tracts': 1, 'hideout': 1, 'reside': 1, 'cheeky': 1, 'bugger': 1, 'indifferent': 4, 'context': 16, 'chuckles': 13, 'stroking': 1, "spike's": 1, 'guttenberg': 3, 'herzfeld': 3, 'money': 169, 'schulz': 1, 'de-': 1, 'turbulent': 2, 'boy-meets-fish-and-saves-environment': 1, 'chumming-up': 1, 'collectibles': 1, 'productions': 6, 'achieves': 4, "pee-wee's": 1, 'rekindle': 2, 'converted': 3, 'ladder-fight': 1, 'desirable': 2, 'roof': 6, 'caliber': 11, 'owen': 7, 'virus': 13, 'symbolize': 1, 'peephole': 1, 'utopia-gone-awry': 1, 'rubs': 1, "planet's": 1, 'goddaughter': 2, 'congresswoman': 1, 'suggestion': 8, 'cleaning': 7, 'instinctive': 1, 'lacked': 4, 'gross-outs': 1, 'harland': 2, "chambers'": 1, 'libel': 1, "hollywood's": 7, 'latifa': 2, 'suri': 1, 'jazz': 5, '_waiting_to_exhale_': 1, 'polite': 3, '102-minute': 2, 'haystack': 1, 'vanish': 4, 'livened': 2, 'boiled': 3, 'miles': 10, 'kudrow': 8, 'on-edge': 1, 'dimensional': 5, 'minimally': 2, 'pretentious': 19, 'hideous': 12, 'minstrel': 1, 'splits': 1, '+': 8, "parr's": 1, 'kneecap': 1, 'assasin': 2, 'solar': 3, 'dialouge/short': 1, 'aimed': 15, 'brooklyn': 9, 'screwed-up': 1, 'drives': 14, 'toss': 5, 'faze': 1, "o'brien": 1, 'turned-on': 1, "else's": 3, 'ndorff': 1, 'peacemaker': 1, 'sid': 3, 'jersey': 11, 'elizondo': 1, 'extraordinarily': 6, 'oracles': 1, 'planing': 1, 'fielding': 4, 'excerpt': 2, 'hoping': 46, "emile's": 1, 'cassette': 1, 'ruined': 17, 'brody': 2, 'right--': 1, 'georges': 1, 'dumas': 2, 'miserables': 1, 'vicarious': 3, 'acknowledgment': 2, '90-minute': 4, 'pals': 2, 'spruced': 1, 'eschews': 1, 'signifying': 2, 'leyland': 1, 'immaculate': 1, 'auction': 3, '[her': 1, 'interviewees': 1, 'spruce': 1, 'hinky': 1, 'terror': 18, 'horns': 1, 'pseudo': 1, 'willis': 26, 'walt': 5, 'gangster': 15, 'messed': 1, 'wannabe': 16, 'facemask': 1, 'precautions': 1, 'audaciously': 1, "arthur's": 1, "siam's": 1, 'x-files': 9, 'interest/chambermaid': 1, 'caulder': 2, "jam's": 2, 'telegraphing': 1, 'victorian': 2, 'sayles': 1, 'post-war': 1, 'alyssa': 1, 'fumbled': 1, 'bulworth': 2, 'thicker': 1, 'pringles': 1, 'off-screen': 5, 'rating': 42, 'dean': 14, 'discarding': 1, 'usage': 6, 'then-wants-to-pay-for-sex-': 1, 'ruminations': 2, 'selfless': 1, 'harsh': 11, 'super-computer': 1, 'hingle': 2, '157': 1, 'designing': 2, 'talkative': 1, 'shrieks': 1, 'financially': 7, 'shot---as': 1, 'dismissing': 2, 'corey': 3, "madonna'": 1, 'ailments': 1, 'revered': 5, 'raccoons': 1, 'character-heavy': 1, 'miniatures': 2, 'selma': 3, 'shred': 5, 'screens': 7, '1/10': 2, 'bazooms': 1, 'dubs': 2, 'brock': 2, 'schlock': 7, 'intentional': 8, 'rollicking': 2, 'plows': 1, 'mispronouncing': 1, 'serves': 26, 'edwards': 3, 'busting': 1, 'modest': 5, '1971': 5, 'affair': 32, 'hazy': 2, 'german/west': 1, 'salem': 1, 'square': 8, 'summary': 4, 'eddie': 26, 'heals': 3, 'tied': 7, 'sexist': 11, 'excessive': 15, 'update': 11, 'bigger': 32, "blondie's": 1, "zardoz's": 1, 'honeywell': 1, 'over-achiever': 1, 'iguanas': 1, '20th': 11, 'cleanup': 1, 'portman': 6, 'caftan': 1, 'conscious': 5, "bug's": 2, 'accomplished': 14, 'leagues': 5, 'baser': 1, 'cubs': 1, 'bootlegging': 1, 'mantra': 5, 'unpleasant': 20, 'lambs': 5, 'vander': 1, 'self-irony': 1, 'knew': 55, 'word-of-mouth-proof': 1, 'schumaccer': 1, 'lightning-lit': 1, 'night-sticking': 1, 'representation': 1, 'typewriter': 1, '_fisherman': 1, 'oddly--who': 1, 'real-life': 15, 'non-baseball': 1, 'fairly': 44, 'chuckster': 1, 'bigwigs': 1, 'rationale': 3, 'tragedy': 15, 'knocks': 5, 'suck': 12, 'asylum': 2, 'bolts': 1, 'consist': 2, 'display--there': 1, 'underdog': 5, 'passports': 1, 'absent-mindedness': 2, 'dimwit-with-a-shady-past': 1, '1925': 1, 'conscience-stricken': 1, 'elves': 2, 'tendency': 7, 'aplomb': 1, 'increasing': 3, '_entertainment_weekly_': 1, 'accused': 17, 'longing': 4, 'bullet-shaped': 1, 'balasko': 1, 'afterlife': 2, 'live_': 1, 'gladstone': 1, 'old-guy': 1, 'cranking': 2, 'mysore': 1, 'dubious': 17, 'parachutes': 1, 'race': 35, 'frau': 1, 'useless': 18, 'singh': 1, 'exuberant': 2, 'braindead': 3, 'decades': 8, 'pimple': 2, 'dulled': 1, 'bludgeoning': 1, 'bribe': 2, 'hara-kiri': 1, 'plot-line': 2, 'izabella': 1, 'heart-broken': 1, 'overlord': 1, 'diedre': 1, "ex-wife's": 1, 'checked': 4, "jaws'": 2, 'criticizing': 1, 'pandering': 1, 'tramp': 2, 'breaks': 26, 'millionaire/crocodile': 1, 'profiler': 1, 'clumsy': 16, 'prevails': 1, 'rocque': 1, 'pretense': 1, 'sport': 6, 'alone_': 1, 'rastafarians': 1, '1960s': 6, 'trese': 1, 'arsenic': 1, 'dead/and': 1, 'ninth-hand': 1, 'hardee-har-har': 1, 'dan': 15, 'stereotypes': 26, 'alternatively': 1, '-inspired': 1, 'blue-eyed': 1, 'uhm': 1, 'mode': 8, 'jubilation': 1, 'josef': 2, '2000-pound': 1, '100%': 3, 'anatomizes': 1, 'predator--all': 1, 'stu': 2, 'mel-brooks-produced': 1, 'hadley': 1, 'beach': 19, 'garbed': 1, 'reveal': 22, '3-year': 1, 'resonating': 1, 'arabian': 1, 'highs': 4, 'aware': 20, 'comparison': 27, 'oscar-caliber': 5, 'rockefeller': 1, 'bosoms': 1, 'printing': 1, 'ro': 1, 'eyeball': 2, 'tree-covered': 1, 'quarters': 1, 'video': 125, 'excuses': 3, 'expressing': 4, 'speeches': 9, 'multiples': 1, 'misfires': 4, 'loopy': 1, 'overflowing': 1, 'humored': 3, 'wares': 3, "comics'": 1, 'activity': 9, 'brawn': 1, 'virtue': 8, 'conceived': 8, 'man-animal': 2, 'warner': 22, 'takeoff': 2, "callaway's": 1, 'daisies': 2, 'finance': 2, 'bikini-': 1, 'mohr': 9, "someone's-out-to-get-me": 1, 'pantomine': 1, 'do-gooders': 1, 'love-triangle': 1, 'recuperate': 1, 'hell-bent': 3, 'dolphin': 1, "base's": 1, 'meticulous': 5, 'misrepresenting': 2, 'susan': 14, 'ardour': 1, "kong's": 2, 'recipe': 5, 'flirt': 5, 'haven': 4, 'endurance': 2, 'gulp': 2, 'endo': 1, 'wesley': 6, 'glow': 2, 'squabble': 5, 'cahoots': 2, 'scouting': 1, 'levritt': 1, 'routines': 2, 'snarling': 1, 'disc': 5, 'fictious': 1, 'concurrently': 1, 'hitters': 2, 'shaft': 4, "tomorrow'": 1, "matthau's": 1, 'fittingly': 2, 'contort': 2, 'go-around': 1, 'peaches': 1, 'playstation': 1, 'fisher': 9, 'detmer': 3, 'slathers': 1, 'progress': 13, 'potions': 2, 'zed': 2, 'founded': 1, 'sight-gag': 1, 'vicky': 1, 'often-adapted': 1, 'scribbling': 1, 'orignal': 1, 'nathaniel': 2, 'by-the-books': 1, 'goriest': 1, 'hellraiser': 3, 'pallisades': 1, 'bombs': 4, 'kowalski': 1, 'russian': 18, 'hears': 9, 'forgotten': 30, 'recognizes': 4, 'equiped': 1, 'adulation': 1, 'vegas-based': 1, 'uncovered': 3, 'fargo': 6, "chompin'": 1, 'bee': 2, 'guys': 107, 'foaming-mouth': 1, 'recombination': 1, 'dune-esque': 1, 'deedle': 3, 'nyc': 4, 'spicoli': 1, 'depend': 4, 'cuddly': 2, 'balloons': 2, 'encyclopedia': 1, 'there--during': 1, 'recruit': 3, 'teen': 44, 'gwennie': 1, 'illustrated': 3, 'nominee': 1, 'bailey': 1, 'reprintable': 1, 'coolidge': 1, 'believers': 1, "victoria's": 1, "world'": 1, 'tramps': 2, 'tracy': 2, 'pursuit': 10, 'cormack': 1, 'heat-16': 1, 'wags': 1, 'diagnosis': 1, 'desired': 8, 'jaleel': 1, 'stretched': 14, 'chokes': 1, 'smells': 3, "momma's": 2, 'power-mongering': 1, 'paid': 20, 'milking': 1, 'col': 4, 'trio': 19, 'fortitude': 1, 'well-groomed': 1, 'b/w': 1, "ii's": 1, 'alternated': 1, 'shattering': 1, 'withdrawal': 2, 'matters': 32, '38th': 1, 'listen': 13, 'rigs': 1, 'blah': 6, 'action-hero': 1, 'crammed': 5, 'reflections': 1, 'jeffries': 3, 'bro': 1, 'spielberg': 13, 'astride': 1, 'bossy': 1, 'genieveve': 1, '23': 4, 'helmed': 8, 'eaters': 5, 'oscar-': 1, 'margulies': 1, 'ribbon': 1, "emmerich's": 2, 'ace-in-the-hole': 1, 'nba': 3, 'betraying': 2, 'renders': 3, '_disturbing_behavior_': 1, 'drive-in': 1, 'couple-ness': 1, 'mascot': 1, 'clarified': 1, 'indicative': 2, 'strapped': 6, 'salma': 7, 'award-winner': 1, 'hitmen--that': 1, 'jun': 1, 'yessssss': 1, "schrader's": 2, 'declaring': 1, 'bed': 26, 'timmonds': 2, 'billiard': 1, 'baby-boomers': 1, 'giving': 92, 'detail': 24, 'midland': 1, 'cracklings': 1, 'evolve-they': 1, 'mccartney': 1, 'regails': 1, 'soul': 25, 'rookie': 5, 'organization': 10, 'moviegoer': 2, 'restrict': 1, 'mtv2': 1, "rick's": 1, 'tomas': 1, 'quizzical': 1, 'warrant': 11, 'finished': 18, "director'": 1, 'propellers': 2, 'combating': 1, 'ingenuous': 1, "son'a": 2, 'hushed': 4, 'commercialize': 1, 'theron': 11, 'trout': 2, 'latin': 2, 'crave': 1, 'used-up': 1, 'anti-fans': 1, 'jeni': 1, 'brains': 15, 'lease': 2, 'license': 4, '$75': 2, 'exasperating': 2, 'attack': 41, 'foregone': 1, 'rarely': 37, 'trajectory': 2, "belly'": 1, 'loudly': 5, 'non-thrilling': 1, 'co-producer': 1, 'bigger-than-life': 1, 'titanic': 12, 'bahns': 1, 'korman': 1, 'filmography': 2, 'perpetrator': 5, 'menacing': 14, 'high-ranking': 2, 'exerts': 1, 'essentials': 2, 'peri': 1, 'brain-zapped': 1, "face'": 1, 'gap_': 1, 'returnee': 2, 'marginally': 9, 'groan-inducing': 2, 'violet': 1, 'stones': 4, 'hugo': 2, 'criterion': 1, 'talia': 1, 'less-arrogant-than-usual': 1, 'epilogues': 1, 'pornographic': 4, "boorman's": 3, 'reich': 1, 'strawberry': 1, 'scandals': 1, 'wings': 6, 'coastal': 1, 'beatty': 4, 'photo-copied': 1, 'sharpe': 1, 'dea': 2, 'wilder': 1, 'faltered': 1, 'suit': 27, 'resume': 10, 'droney': 1, 'best-selling': 3, 'blech': 1, 'skyscrapers': 1, "alma's": 1, 'kiefer': 3, "'70s": 5, 'assassinates': 1, "alan's": 1, 'asteroid': 3, "hoenicker's": 1, 'miami': 10, 'grosses': 2, 'hamstrung': 2, 'gadgetry': 1, 'passport': 4, 'strikes': 17, 'reward': 9, '_scream_': 1, 'benefactor': 3, 'chariot': 2, "blair's": 1, 'williamson--to': 1, 'campfire': 3, "pipstone's": 1, 'primarily': 15, 'reservation': 2, 'compare': 14, "eyes'": 1, 'delinquent': 1, 'tissue': 3, 'wrists': 1, 'gorillas': 4, 'aris': 1, 'waaaayyyyyy': 1, 'sonatine': 1, 'hash': 2, 'injected': 4, 'isaiah': 2, 'falco': 1, 'enlists': 3, 'hippie': 6, "'psycho'": 1, 'tycoon/villain': 1, '`jeez': 1, 'casey': 6, 'chip': 2, 'psychotherapy': 2, 'engines': 2, 'kfc': 1, 'nauseating': 8, 'semblance': 8, 'lameness': 1, '103': 2, 'offensive': 39, 'comic-relief': 1, 'dobie': 1, 'aa-meetings': 1, 'rowdy': 1, "might've": 7, 'stations': 5, 'top-billers': 1, 'riff': 2, 'unresolved': 2, 'bashing': 1, 'repairman': 1, 'non-sensical': 2, '3': 59, 'ick': 1, 'outdoor': 2, 'aunt': 5, 'crafted': 8, 'gums': 1, "sandra's": 1, '`ends': 1, 'posted': 2, 'preteen': 2, 'ragsdale': 1, "reviewer's": 1, 'clunky': 5, 'loincloth': 1, 'wreak': 5, 'loli': 1, 'nirto-est': 1, 'pushing': 10, 'foods': 3, "`praise'": 1, 'jones': 37, 'colored-glass': 1, 'maddeningly': 3, 'film--and': 2, 'katt': 5, 'whips': 2, 'mics': 1, 'kethcum': 1, 'unschooled': 1, 'over-emotes': 1, 'bergl': 2, "drew's": 1, 'bumpkin': 1, 'quotations': 1, 'butt-kicking': 1, 'electro-pop': 1, 'substitutes': 1, 'borg': 2, 'warning--spontaneously': 1, 'susceptible': 3, 'panama': 2, 'tactful': 1, 'taunts': 1, 'lankier': 1, 'coached': 1, 'policed': 1, 'hot-wires': 1, 'pernilla': 1, 'cake': 9, 'bender': 1, 'halfheartedly': 1, 'sinyor': 1, 'levying': 2, 'forehead': 2, '007-esque': 1, 'doses': 4, 'ways---and': 1, 'rousing': 2, 'callisto': 1, 'noirish': 1, 'cringed': 1, 'sharkskin': 1, 'grant': 26, 'great-': 1, 'platoon': 3, 'notably': 14, 'waterfront': 1, 'compensate': 10, 'union': 7, 'matinee': 3, 'internal': 9, 'barely-funny': 1, 'agent/assassin': 1, "scorsese's": 4, 'cinergi': 1, 'bussotti': 1, 'entertaining': 95, 'efficiency': 3, 'yogi': 1, 'criteria': 1, 'chastity': 2, 'kilronan': 2, 'sonny': 5, 'subjective': 3, 'joel': 23, 'sing': 15, 'connors': 1, 'unfeeling': 1, 'forever': 29, 'hodge-podge': 1, 'non-cartoon': 1, 'buff': 7, 'occur': 16, 'yourself-but': 1, 'middle-class': 2, 'de': 43, 'rocky': 11, 'ishibashi': 2, 'oldest': 3, 'purproses': 1, "peploe's": 1, 'zinger': 1, 'type-casting': 1, 'raymond': 9, 'disasterous': 2, 'retreat': 4, 'tough-guy': 2, '1968': 3, 'functioned': 1, "myer's": 1, "'investigation'": 1, 'hotshot': 8, 'thirty-five': 1, "darren's": 2, 'brazen': 3, 'marches': 3, 'meals': 3, 'diarrhea': 2, 'gord': 1, 'oxide': 2, "'should": 1, 'bafflingly': 1, "shoot'em": 1, 'restore': 4, 'hiller': 2, "i'm-having-a-mid-life-crisis": 1, 'will--mere': 1, 'two-man': 1, 'reflects': 6, "gaynor's": 1, "'snuff": 1, 'santaro': 1, 'periods': 2, "luigi's": 1, 'mesh': 4, 'enquirer': 2, 'richards': 8, 'credible': 14, 'measurements': 1, 'blurting': 1, 'palotti': 1, 'authenticity': 3, 'accuracy': 4, 'branches': 2, 'heated': 4, 'mayhem': 11, 'roan': 1, 'angel': 18, 'intolerable': 2, 'emotional': 60, 'ericson': 1, 'innocent': 34, 'caught': 42, 'smokealot': 1, 'disasterously': 1, 'puttingly': 1, 'bursts': 9, 'master-mind': 1, 'hot-shots': 1, 'billionaire': 3, "alana's": 1, 'probably-never-will-bes': 1, 'thick': 17, 'neighbors': 7, 'film-making': 4, 'cattle': 2, 'hein': 1, 'scenes': 342, 'instrument': 5, 'barely-in-control': 1, 'pads': 1, 'har': 2, 'necessarily': 22, 'innate': 2, 'implausibilities': 4, "edgar's": 1, 'risible': 2, 'roast': 1, 'hays': 1, 'clients': 5, 'noir': 11, 'dt': 1, 'skimpy': 8, 'formula': 43, 'phifer': 3, 'unsure': 7, 'cio': 1, 'classic': 78, "era's": 1, 'sherri': 2, 'robinson-like': 1, 'underpinnings': 1, 'tie': 8, 'craziness': 1, 'raid': 2, 'vigil': 1, 'parallels': 5, 'take-off': 2, 'gambon': 3, 'materials': 6, 'would-be': 24, 'draven': 2, 'burstyn': 1, 'geri': 3, 'adolescent': 11, 'moonraker': 1, 'gymnastics': 1, 'tip-top': 1, 'schlondorff': 1, 'spears': 1, 'hooking': 1, 'flop-house': 1, 'fight--': 1, 'dickson': 1, 'affordable': 1, 'squandered': 5, 'shorter': 6, 'rhino': 2, 'counterbalanced': 1, 'covered': 26, 'multiply': 2, 'ultra-conservative': 1, 'flighty': 1, 'deformed': 1, "`cooks'": 1, 'nubla': 1, 'mustard': 1, 'limps': 2, 'fabulous': 7, 'skinny-dip': 1, 'culminates': 1, 'keister': 1, 'miscalculation': 2, "sunhill's": 1, 'stinker': 5, 'vidnovic': 1, 'jellylike': 1, 'flanders': 1, 'pourkarim': 1, 'well-to-do': 3, 'patriot': 3, "karla's": 3, 'bmw': 2, 'cute': 43, 'imbecilic': 1, 'symbolically': 1, "browning's": 1, "villard's": 1, 'herbert': 1, 'branagh': 6, 'hanover': 2, 'handgun': 2, 'macaulay': 2, 'michigan': 1, 'joins': 18, 'commodity': 8, 'reknown': 1, 'wishing': 12, 'stopped': 24, 'woman-child': 1, 'brainard': 2, 'hooker': 6, 'pg-13': 12, 'come-back': 1, 'preferred': 6, 'walken': 7, 'stolz': 1, "michelle's": 1, 'lavatory': 1, 'summers': 5, 'imbalances': 1, 'pray': 13, 'hudsucker': 2, 'fissure': 1, 'zipper': 1, 'demornay': 1, 'jaw-dropping': 2, 'wail': 2, 'fullyloaded': 1, 'syllable': 1, 'ear-shattering': 1, 'tumbled': 1, 'drunk': 30, 'maxine': 2, 'welshman': 2, 'slang': 4, 'capitalizing': 2, 'puns': 8, 'idealistic': 3, 'envoke': 1, 'headphones/tiara': 1, 'outward': 1, 'fratboy': 1, 'gotta': 7, 'goo-goo': 3, 'cadaver': 1, 'wynt': 1, 'mindlessly': 1, 'bleach': 1, "something's": 2, 'shrink': 6, 'amorality': 1, 'vcrs': 1, 'humbug': 1, 'sats': 1, 'maniacle': 1, 'sacrilegious': 2, 'role-shifting': 2, 'cryogenic': 2, 'punches': 7, 'navigate': 2, 'blunt': 3, 'chisholm': 2, 'divisive': 1, 'pieter': 1, 'registers': 3, 'gosh': 4, 'exposed': 11, 'boy-loses-girl': 1, 'pre-production': 1, 'convert': 5, 'red-hot': 1, "teresa's": 1, 'dynamic': 9, 'eye-opening': 1, 'colleen': 1, 'seann': 4, 'cozy': 2, 'locket': 1, 'conquistador': 1, 'cut-from-nc17': 1, 'infallibility': 1, 'interaction': 21, 'overdone': 15, 'safer': 1, 'traversing': 1, 'coin': 1, "ramsey's": 1, 'profit': 2, 'offices': 1, 'revenge': 37, 'gibbons': 1, "exec's": 1, 'miner': 3, '115': 3, 'liaisons': 3, '_never': 1, 'sayinig': 1, 'zoom-ins': 1, 'rampaging': 2, 'tortured': 5, 'contrary': 10, 'brought': 50, 'demeaning': 5, 'swordfish': 1, 'dimwit': 2, 'commencing': 1, 'emotion': 33, 'sucker': 5, 'homestead': 1, 'installment': 16, 'unnecessary': 33, 'defuse': 1, 'reversed': 1, 'hammer': 5, 'journal': 4, 'lex': 2, 'sandar': 1, 'carved': 5, "'that's": 1, "casey's": 1, 'underwood': 6, '1898': 1, 'aquanet': 1, 'shrimp': 1, 'above-the-title': 1, 'needling': 1, 'anguish': 3, 'proportionate': 1, "week's": 5, 'cockamamie': 1, 'siskel': 5, 'zaniness': 3, 'splice': 1, 'weed': 3, 'forster': 2, 'renew': 1, 'airplane': 14, 'jailbreak': 2, 'lovejoy': 1, 'goals': 4, 'guerrilla': 3, 'doc': 3, 'classes': 9, 'bouts': 1, 'guinee': 1, 'backyard': 4, 'christmas': 24, 'paths': 5, 'appears': 109, 'concentrated': 2, 'contribute': 3, '1998': 35, 'black': 123, "'i'm": 1, 'thurgood': 1, 'jesus': 6, 'reddish': 1, 'sahara': 1, 'writing-directing': 1, 'solved': 5, 'multi': 1, 'face-painting': 1, 'cameo': 50, 'filmakers': 1, 'leak': 4, '6=better': 1, 'aristocrat': 3, 'inherit': 2, 'rally': 2, 'kai': 1, 'radical': 2, 'hug': 3, 'spiel': 1, 'sillier': 1, 'motorcycle': 9, 'slo-mo': 4, 'demands': 20, 'unhappy': 8, 'pre-credits': 1, 'karma': 1, 'cutting-edge': 1, 'operative': 9, 'delectably': 1, 'tree-in': 1, 'undiscovered': 5, 'roundtable': 1, 'voyeuristically': 1, 'prairie': 2, 'slaughterhouse': 1, 'macfadyen': 1, 'empathetic': 3, 'pawns': 5, 'ninjas': 1, 'xusia': 1, 'banal': 12, 'scientifically': 2, 'rounded': 1, 'swoop': 2, 'troweled': 1, 'terra-forming': 1, 'biehn': 1, 'computerized': 3, "louie's": 1, 'washed': 4, 'live-action': 17, 'lowbrow': 3, 'implied': 3, 'attitudes': 1, 'drenched': 1, 'allied': 1, 'walls': 14, 'mish-mosh': 2, 'hudson': 13, 'betti': 1, 'gilfedder': 1, 'menacingly': 1, 'lision': 1, 'bus--but': 1, "taiwan's": 1, 'adapted': 15, 'bicker-': 1, 'individuals': 18, "childhood's": 1, 'rio': 1, 'suck-ups': 1, 'wanted': 63, 'mirror': 12, 'yesterday': 1, 'rv': 1, 'cribbing': 2, 'drifter': 3, '3/10': 3, 'cliques': 2, 'debuted': 1, "partners'": 1, "season's": 1, 'cain': 1, "this'": 1, 'turns': 127, 'trucks': 6, "parent's": 2, 'militiamen': 1, 'revels': 3, 'singularly': 6, 'b-': 1, 'pied': 1, 'howler': 3, 'vr': 4, "soran's": 1, 'neo-fascist': 1, 'energetic': 9, 'yatf': 1, 'grungiest': 1, 'grieco': 2, 'vivian': 1, "book's": 1, 'ditzy': 6, 'bellboy': 1, 'scoring': 2, 'prone': 2, 'binge': 1, 'feverish': 2, 'kalman': 1, 'cornball': 3, 'shrug-of-the-shoulders': 1, 'quashed': 1, 'arqua': 1, 'meaningful': 15, 'disposes': 1, 'draw': 27, 'seesaws': 1, 'second-': 2, 'oral': 3, 'kingdom': 4, 'prima': 1, 'tops': 8, 'tv': 113, 'san': 11, '`real': 1, 'hoover': 1, 'pose': 5, 'buried': 12, 'hatred': 1, 'teachers': 6, 'looters': 1, 'genre-parody': 1, 'moist': 1, 'svelte': 1, 'hammond': 3, 'debts': 2, 'mistreated': 1, 'sophistication': 4, 'dad': 32, 'exonerated': 1, 'solid': 40, 'dormant': 3, 'drinker': 1, 'involves': 56, 'windswept': 2, 'autos': 1, 'shackles': 1, 'mudhole': 1, 'spilling': 1, 'indignity': 2, 'cringeworthy': 2, 'desire': 18, "eddie's": 2, 'carefully': 12, 'tony': 16, 'mama': 2, "belong'": 1, 'hire': 13, 'silly-looking': 1, 'huston': 2, '___': 1, 'repair': 1, "gardenia's": 1, 'poetry': 6, 'stooge': 2, 'huevelman': 1, 'squid': 3, 'creativeness': 1, 'breast': 5, 'smorgasbord': 1, 'fitzgerald': 1, 'inspires': 5, 'mcdowell': 3, 'orphaned': 4, 'run-through': 1, 'duval': 3, "draven's": 1, 'yuor': 1, 'rumbles': 3, 'geneticist': 2, 'defender': 1, 'makings': 5, 'brasco': 1, 'barrel': 3, 'processor': 2, 'lesbo': 2, 'fundamental': 9, 'ultra-religious': 1, 'tomei': 3, "'unhip'": 1, 'plotless': 2, 'scaramanga': 1, 'unfair': 7, 'deranged': 8, 'tin': 5, 'shrubs': 1, 'shear': 1, 'crowd': 33, 'hampton': 1, 'vistas': 2, 'mommy': 3, 'chef': 7, 'perfuctory': 1, 'good-looking': 5, 'kirby': 3, "1987's": 1, 'scrambling': 1, "england's": 2, 'boggingly': 1, 'exhibits': 3, 'indestructable': 1, 'freak': 12, 'indefinitely': 1, 'jonas': 1, 'ladies': 12, 'mini-mr': 1, 'girl': 133, 'trash': 33, 'radio': 20, 'rapidity': 1, 'derring-do': 2, 'bennett': 1, 'half-drunk': 1, 'sex': 124, 'cart': 2, 'bayou': 3, 'wade': 1, 'mil': 1, 'charms': 4, 'camera': 122, 'pseudo-feminist': 1, 'tables': 11, 'ligaments': 1, 'resorting': 2, 'ingrid': 1, 'outer': 11, 'pornos': 1, 'glengarry': 2, 'shockwave': 1, 'bands': 1, "hitch's": 1, 'working-class': 2, 'hi-tech': 1, 'suppress': 3, 'hysterics': 1, 'eastwick': 1, "sobel's": 1, "richie's": 1, 'tempt': 1, 'hideously': 6, 'anastasia': 2, 'truthfully': 1, 'oh-so-wise': 1, 'pledges': 1, 'refer': 8, '--schumacher': 1, 'victimisation': 1, "m'kay": 1, 'superior': 53, 'rehearses': 1, 'hick-boy': 1, 'goldin': 1, 'invent': 4, 'garret': 1, 'wrestlers': 5, 'fade': 1, "'rules": 1, 'revel': 1, 'americans': 9, "'rome": 1, 'tut': 2, 'gender-bending': 1, "boy's": 3, '86': 1, 'unorthodox': 1, 'friction': 3, 'bail': 4, 'undermines': 8, 'fault': 40, 'richard': 52, 'on-target': 1, 'karaszewski': 1, 'textbook': 3, 'elses': 1, 'cole': 9, 'concealed': 1, 'blow': 29, 'sitcomish': 1, 'recreates': 1, 'rube': 2, 'grow': 20, 'stardom': 5, 'souk': 1, '_andre_': 1, 'pilgrims': 1, 'treasures': 2, 'hogs': 1, 'skillful': 4, 'spill': 1, 'confucius': 2, 'moptop': 1, 'vocation': 1, 'slow-mo': 1, 'eludes': 1, 'folklore': 2, 'won92t': 1, 'indiana': 9, 'driving': 25, 'chanting': 1, "amblyn's": 1, 'hitter': 3, 'snipers': 2, 'soccer': 2, 'cows': 1, 'shelly': 4, 'imitate': 1, 'facility': 6, 'insights': 6, 'dandy': 4, 'aboard': 10, 'brosnan': 4, 'stewardess': 1, 'dysart': 1, "o'neal": 2, 'opting': 1, 'disobeying': 1, 'honed': 2, 'vickie': 2, 'populating': 2, 'take-off--': 1, '1972': 4, 'unequipped': 1, 'strongpoint': 1, 'half-mast': 1, 'break-in': 4, 'wore': 6, 'existence': 22, 'ferguson': 2, 'prototype': 3, 'distributed': 1, 'ernest': 3, 'paces': 3, 'war': 53, 'dead-weight': 1, 'lawsuit': 3, 'kotter': 1, 'fuentes': 1, 'targeted': 7, 'strength': 17, 'beautifully-rendered': 1, 'peculiar': 9, 'spews': 1, 'cafe': 2, 'uninsightful': 1, 'cable': 23, 're-situating': 1, 'ne': 1, 'strategy': 7, 'exhale': 2, 'bewitching': 1, 'beals': 2, 'pas': 2, 'entei': 1, "sisters'": 2, 'titled': 10, 'gunned': 3, 'plumets': 1, 'queasy': 1, 'cheapo': 1, 'bludgeons': 2, 'glimmers': 2, 'stopwatch': 1, 'fetal': 1, 'werewolf': 7, 'assaults': 5, 'flakes': 2, 'vice-versa': 1, 'buyer': 3, 'cloth': 4, "elena's": 1, 'cheryl': 1, 'scintilla': 1, 'frumpy': 2, 'roofs': 1, 'asserted': 1, "microphone's": 1, 'shite': 4, 'developing': 14, 'saddened': 1, 'redundant': 5, 'ezio': 1, 'anakin': 2, 'fortune': 12, 'interpreter': 3, 'weebo': 2, 'make': 457, 'bartkowiak': 3, 'def': 2, 'commander': 11, 'overact': 1, 'circular': 1, 'permutation': 1, 'hors': 1, 'baigelman': 1, 'strozzi': 1, 'a-team': 1, 'healing': 2, 'animals--': 1, 'implicates': 1, 'pairs': 6, 'nebbish': 4, 'potheads': 2, 'relieved': 1, 'who-and': 1, 'pre-review': 1, 'd-grade': 1, 'daring': 7, 'hubris': 1, 'brushstrokes': 1, 'ringmaster': 1, 'grip': 5, "emily's": 1, 'sleepy': 7, 'hairless': 1, 'picture]': 1, 'painting': 5, 'fiction': 38, 'implementing': 1, 'stove': 2, 'backpacks': 1, 'assassination': 7, 'janitor': 6, 'bloodshed': 3, 'throats': 3, "macy's": 1, 'martians': 3, 'materialize': 1, 'sticker': 1, 'me-he': 1, 'historic': 2, 'collided': 1, 'vertiginous': 1, "huntingburg's": 1, 'babbles--she': 1, 'scenery-': 2, 'double-zero': 1, 'throghout': 1, 'mike': 25, 'gotham': 5, 'basics': 6, 'admire': 11, 'bad-guys': 1, 'kermit': 1, 'hasn': 1, 'casserole': 1, 'damage': 13, 'flow': 6, 'plot-related': 1, '50': 11, 'scatological': 3, 'bushes': 1, 'heaven': 12, 'post-party': 1, 'bison': 1, 'tend': 14, 'mind-twisting': 1, 'night': 136, 'phillipe': 4, 'ally': 8, "wolf's": 1, 'vocally': 3, 'carrera': 1, 'suits': 20, 'flaccid': 2, 'booth': 5, 'action-music-video': 1, "rourke's": 1, 'hodge': 1, 'action-thriller': 3, '18-wheeler': 1, 'wal-mart': 2, 'testament': 4, 'identities': 3, 'ten-minutes': 1, 'f-word': 3, 'wright-penn': 1, '/cinematographer/editor': 1, 'actor/stunt': 1, 'oftentimes': 8, 'mysteriously': 6, 'slamming': 6, 'full-frame': 1, 'davies': 3, 'acme': 1, 'whispered': 1, 'gussies': 1, 'gobbledygook': 2, 'summoned': 3, 'factly': 1, "grinch's": 1, 'inaccurate': 1, 'krajeski': 1, 'improvisation': 1, 'thriller': 96, 'counsel': 5, 'confirm': 4, 'reflective': 1, 'chimpanzees': 1, 'amazonas': 1, 'disembodied': 1, 'paradise': 6, 'surly': 1, "gedrick's": 1, 'unhip': 1, 'situated': 1, 'spillane': 1, 'suvari': 9, '[': 1, 'three-piece': 1, 'albert': 10, 'sacramento': 1, 'headly': 2, 'bile': 1, 'ice-t': 4, 'comedies': 34, 'ramboesque': 1, 'attacked': 9, 'marie-sylvie': 1, 'stateside': 2, 'kahn': 3, 'barnett': 1, 'melts': 3, 'symbol': 1, 'embarrassments': 1, 'alluded': 3, 'stint': 5, 'anton': 1, 'talented': 63, 'prim': 1, 'mortified': 1, 'eyro': 1, 'motivator': 1, 'mademoiselle': 1, 'boomer': 1, 'exterminated': 1, 'underscores': 2, 'thrillerism': 1, 'bogart': 1, '[he': 1, 'justin': 11, 'chocolate': 7, 'best-friend': 1, 'neurotic': 5, 'leviathan': 2, 'dorks': 1, 'mysterians': 1, 'trotted': 1, 'revisionist': 4, 'implicit': 1, 'besson': 6, 'astronauts': 7, 'hobby': 4, 'fuhrer': 1, 'two-hours': 1, 'download': 1, 'martin': 41, 'twenty-somethings': 1, 'bare-chested': 1, 'enlivened': 2, 'overpower': 1, '99-cent': 1, 'story--it': 1, 'softball': 1, 'grossing': 3, 'exchanged': 1, 'mercer': 1, 'enables': 2, 'bastards': 2, 'tussles': 1, 'elections': 1, 'mine': 26, '`demons': 1, 'politician': 2, 'diligence': 1, 'employing': 4, 'leggings': 1, 'risqu': 2, 'badly': 48, 'clude': 1, 'exaggerate': 1, 'offshoot': 1, 'wistful': 4, 'enlisted': 1, "rock's": 2, 'vondie': 2, 'crash': 26, 'cowboy': 13, 'electromagnetic': 1, "wigand's": 1, 'beaman': 1, 'fucking': 6, 'museum': 3, 'tuition': 1, 'interruptus': 1, 'estranged': 6, 'undercuts': 3, 'up-and-up': 1, 'bilk': 1, 'toddlers': 1, 'loud': 58, 'wounded': 5, 'drowned': 7, 'weaving': 3, 'slayings': 1, 'declining': 1, 'imposed': 1, 'boulevard': 2, 'blackface': 1, 'ease': 8, 'all-american': 1, 'leaves': 70, 'ah': 9, "sena's": 1, "five's": 1, "hour's": 1, 'stolen-from-no-less-than-three-movies': 1, 'nothing--least': 1, 'blander-than-bland': 1, "bronson's": 2, 'fortune-hunter': 1, 'mendes': 1, 'plainly': 3, 'oversees': 1, "guccione's": 1, 'heightening': 1, 'boom': 6, 'throes': 3, 'deadly': 27, 'super-hyped': 1, 'heather': 9, 'zak': 3, 'ten': 75, 'fully-': 1, 'mismatch': 1, 'derailing': 1, 'dustin': 8, 'save-the-earth': 1, '20-minute': 1, 'noticed': 7, 'co-lead': 1, '=======================': 1, 'tales': 11, 'addictive': 1, 'then-diana': 1, 'loomis': 1, 'unappreciated': 1, "abigail's": 1, 'educate': 5, 'intercut': 4, 'prodigious': 1, 'afer': 1, 'baking': 1, 'correctional': 1, "'secret'": 1, 'sockful': 1, 'plead': 1, 'lifeless': 32, 'unbelievably': 9, 'antihero': 1, 'conning': 1, '300-plus-years': 1, "authors-i've": 2, 'overcooked': 2, 'terrence': 5, 'hitchhiker': 2, 'prematurely--not': 1, 'radiation-rich': 1, 'boban': 1, 'dune': 4, 'time-space': 1, 'slows': 5, 'ketchup': 3, 'system': 35, 'insurmountable': 2, 'godmother': 2, 'recently-deceased': 1, 'hop': 1, 'maria': 9, '7-13': 1, 'denies': 2, 'wheeler': 1, 'skits': 4, 'sweeping': 9, 'advises': 3, 'ho-hum': 6, 'colourful': 2, 'cloths': 1, 'ann-margret': 1, 'drove': 4, 'enforcer': 3, 'pelt': 1, '`when': 1, 'paradoxes': 1, 'acted': 30, 'there\x12s': 1, 'telekinetic': 1, 'muttered': 1, 'sublimated': 1, 'leeper': 1, 'snively': 1, 'disgusting': 12, 'covenant': 1, '80-foot': 3, 'engine-boy': 1, 'mirren': 4, 'dished': 1, 'mill': 3, 'bottoms': 1, 'campery': 1, 'schellenberg': 1, "caulfield's": 1, 'surprised': 33, 'unthinking': 1, 'hurtling': 2, 'sweating': 2, 'capabilties': 1, 'regurgitating': 1, 'nielsen': 13, 'deftness': 1, 'owe': 3, 'eminent': 1, 'lanced': 1, 'quaid': 13, 'mutation': 1, 'hard-nosed': 3, 'difference-': 1, 'ridgemont': 4, 'white-faced': 1, 'fucks': 1, 'unravel': 1, 'girl-poor': 1, 'hennings': 1, 'deloreans': 1, 'drowning': 8, 'corny': 20, 'ladders': 2, 'keanu': 20, 'pair': 47, 'promised': 14, 'deploy': 1, 'frenchmen': 1, '[on': 1, 'locations': 11, 'easely': 1, 'explosion': 14, 'on-stage': 1, 'policia': 1, 'afternoon': 7, 'role': 247, 'sadly': 56, 'rambling': 2, 'lowenstein': 1, 'ramifications': 2, 'ref': 1, 'harlin': 2, 'tomorrow': 5, 'adolf': 1, 'leticia': 1, 'mother-to-be': 1, '1-800-call-att': 1, 'fyfe': 1, "proctor's": 1, 'strikingly': 1, 'figueras': 1, 'nasa': 10, 'woven': 2, 'evil-doer': 1, "billy's": 1, 'boxing': 8, "`i'm": 2, 'dixie': 1, 'catharsis': 2, 'retort': 1, 'canada': 4, 'legs': 17, 'cello': 1, 'disneyfied': 1, 'everyone--you': 1, 'inter-woven': 1, 'photogenic': 2, 'limited_': 1, 'upstanding': 1, 'counterpoint--or': 1, 'examines': 2, "stanley's": 1, 'schoolers': 3, 'ventures': 4, 'hemisphere': 2, 'aquarium': 1, 'loophole': 1, 'panel-sized': 1, 'co-hosting': 1, 'fluff-piece': 1, 'reacts': 3, 'brown-haired': 1, 'even-handed': 1, 'yawn': 11, 'misadvertised': 1, 'situation': 56, 'queens': 3, 'set': 181, 'actor': 167, 'allegedly': 8, 'fanatics': 4, 'spouses': 2, 'davidovitch': 1, "hedwig's": 1, 'erotic-thriller-cinemax-style': 1, 'overexaggerant': 1, 'impregnation': 1, 'snickered': 1, 'abrupt': 5, "silverstein's": 1, 'moodier': 1, 'date': 32, 'vaults': 2, 'lacklustre': 3, 'remembered': 11, 'boxes': 1, 'bedridden': 3, 'hmmm': 7, 'stipulate': 1, 'peers': 4, 'co-conspirator': 2, 'pimping': 1, 'goose': 4, 'deceive': 1, 'remnants': 2, 'rolled': 6, 'underhanded': 3, "80's": 12, "bernie's": 1, 'blythe': 4, 'similarly-named': 1, 'telling': 52, 'ramones': 1, 'important': 63, 'over-dramatic': 2, 'kickboxer': 1, 'missile': 4, "1986's": 1, 'mental': 27, 'quest': 21, "strangelove'": 1, 'jells': 2, 'black-wannabe': 1, 'yo': 3, 'indulges': 2, "sexton's": 1, 'ape-trader': 1, 'credibly': 2, 'emblazoned': 3, "'28": 1, 'previously-used': 1, 'pa': 2, 'balls': 10, 'julia': 22, 'sub-titled': 1, 'zahn': 3, 'bran': 1, 'fizzles': 2, '30th': 2, 'sand': 8, "gio's": 1, 'whodunit': 3, 'celebration': 3, 'commotion': 1, 'videotapes': 1, 'tomb': 3, "consumer's": 1, 'girlfriend': 75, 'decorum': 2, "game'": 1, 'worldwide': 3, 'forewarned': 5, 'waterfall': 5, 'locklear': 1, 'dim': 5, "spanner's": 1, 'doubly': 4, 'lodge': 1, 'hails': 1, 'reinspired': 1, 'piling': 1, 'stephan': 2, 'molester': 2, 'howard': 9, 'fellow': 40, 'admiration': 7, 'six-million': 1, 'cell': 20, 'tripe': 8, "'dude'": 1, 'charm': 51, 'ming': 2, 'ling': 2, 'knockout': 1, 'intimacy': 2, 'deejay': 2, 'richest': 1, "trek's": 1, 'trots': 1, 'heeere': 1, 'port-o-potties': 1, 'delusional': 1, "'summer": 1, 'crafting': 1, 'wrapper': 2, 'indulge': 7, 'plane': 21, 'cambell': 1, 'travel': 22, 'unite': 1, 'oh-so-cleverly': 2, 'zoe': 1, 'freaks': 6, 'midly': 1, 'depictions': 1, 'royal': 8, "stahl's": 1, 'self-described': 1, 'non-acting': 1, 'revamped': 2, 'double-jeopardy': 1, 'eighies': 1, 'rapists': 1, 'overshadowed': 7, 'blithe': 1, 'reunites': 3, 'motivated': 1, 'homeless': 4, 'emergence': 1, 'near-blind': 1, 'androginous': 1, 'blistering': 1, 'kubrickesque': 1, 'possibilities': 12, 'oz': 8, 'quantum': 1, 'ozone': 1, 'fabricate': 4, 'butabi': 2, 'villainess': 1, 'punchlines': 4, "anyone's": 5, 'government-controlled': 1, 'plotlines': 1, 'kirsten': 2, 'libidos': 1, 'fathom': 2, 'claude': 3, 'bashed': 2, 'guitarist': 1, 'decoration': 4, 'pussy/carlos': 1, 'alludes': 1, 'maximum-security': 1, 'kitano': 1, 'antique': 1, 'impaired': 1, 'pupils': 2, 'unsubstantial': 1, 'bravest': 1, 'scenarios': 4, 'nervous': 9, 'mishandled': 3, 'clamp': 1, "ld's": 1, '_scarface_': 1, 'pick': 43, 'elaine': 1, 'juliet': 3, "d'artangnan": 1, 'heavily-sponsored': 1, 'politicians': 3, 'yuri': 1, 'balding': 1, 'nastier': 1, 'guts': 14, 'sixth': 8, 'payrolls': 1, 'hand-job': 1, "hughes'": 1, 'mirrored': 1, 'diaz': 7, 'stick': 31, 'dimension': 13, 'pre-teens': 1, 'time-': 1, 'climb': 11, 'astute': 2, 'snider': 1, 'fish-out-water': 1, 'aversion': 2, 'richness': 2, 'house': 102, 'widower': 4, 'cheeziness': 2, 'unbilled': 2, 'charged': 11, 'afoul': 2, 'wallet': 3, 'yasmin': 1, 'inroad': 1, 'angstdom': 1, 'friggin-nobi': 1, '57th': 1, 'oriented': 1, 'drunken': 12, 'italicize': 1, 'maher': 1, 'sleek': 1, 'courageous': 2, 'bridge': 12, 'mia': 8, 'allergies': 1, 'laszlo': 1, 'regurgitated': 3, 'affleck': 12, 'cryptic': 3, 'accusation': 2, 'benign': 3, "jovivich's": 1, 'inn': 2, 'break': 54, 'masterpieces': 4, 'gruff': 6, 'loading': 2, "shadyac's": 1, 'heroism': 4, 'fog': 6, 'speedy': 2, 'tritt': 1, 'groan-worthy': 1, 'rudiments': 1, 'dern': 3, 'reasoning': 10, 'tolan': 3, "`flaunt'": 1, 'mutha': 1, 'badu': 1, 'yells': 11, 'spaceship': 7, 'mutter': 1, 'unstructured': 1, "affleck's": 2, 'hanania': 1, 'pines': 4, 'baring': 1, 'olmstead': 1, 'concoction': 2, 'psychology': 1, 'trailers': 19, "dino's": 1, 'goodall': 2, 'renting': 10, 'cleared': 1, 'bucketloads': 1, 'scummy': 6, 'punk-noir': 1, 'far--arguably': 1, 'houseman': 2, 'triple-bladed': 1, 'levity': 2, 'smuggled': 1, 'sinfully': 2, 'slaughters': 2, 'bruckheimer': 8, "mandell's": 1, 'pointlessness': 1, 'divorced': 3, 'indie': 10, 'manhattan': 7, 'tributary': 1, 'saber': 1, '50s': 4, 'investigating': 15, 'in-game': 1, 'supernova': 2, 'lynch-esque': 1, '--much': 1, 'repulsed': 1, 'attention-getter': 1, 'cheered': 4, 'compared': 25, 'chunk': 4, 'kidnap': 12, "d'lyn": 1, 'selective': 2, 'chalk': 3, 'subsidiary': 1, 'one-piece': 1, 'angelina': 11, 'perfects': 1, 'katz': 1, '1926': 1, 'tightfitting': 1, 'spurs': 1, 'mannerisms': 4, 'smoggy': 1, "arc'": 1, 'arrive': 16, "nest'": 1, "bob's": 3, "heinlein's": 2, 'chelsom': 3, 'ross': 9, '[pg]': 3, 'started': 38, 'ex-fiance': 1, 'drowns': 3, 'joyride': 1, 'russell': 24, 'newell': 4, 'isabella': 1, '1/2': 4, 'suburban': 4, 'winkler': 3, 'disengaging': 1, 'recylcled': 1, 'swordplay': 4, 'flashdance': 1, 'parka': 2, 'devastated': 4, 'phantastic': 1, 'kramer': 3, 'settlers': 3, 'garnering': 2, 'naming': 2, 'divulge': 1, "_scream_'s": 1, 'vodka': 2, "`tremors'": 1, 'conquering': 1, 'hectic-but-overall-uneventful': 1, 'slotted': 1, 'psycho-playing': 1, 'bene': 1, 'mice': 2, 'glide': 1, 'safety': 12, 'drunkenly': 2, 'unexploited': 1, 'history': 49, 'implausible': 19, 'self-reference': 1, 'forgetting': 4, "aim's": 1, 'sky': 16, "hemingway's": 1, 'collapse': 2, 'greased': 1, 'numerous': 37, 'cradle': 5, 'id': 3, "`ultra-stylish'": 1, 'geez': 1, 'descending': 1, 'stone-faced': 1, 'farina': 3, 'peer': 4, 'ill-advised': 5, 'startled': 2, 'fingertips': 3, 'leonowens': 1, 'haunts': 1, 'actress/singer': 1, 'restaurants': 2, 'lodged': 2, "reeves'": 4, 'comfortably': 1, 'unraveling': 1, 'tipsy': 3, 'greece': 1, 'feedback': 1, 'mesquida': 1, 'ex-cons': 1, 'sobieski--watch': 1, 'last-ditch': 1, 'attains': 2, 'pared': 1, 'chock': 3, 'admonition': 1, 'nonetheless': 16, 'retrograding': 1, "quack'": 1, 'terri': 10, 'copyright': 8, 'squadron': 1, 'buffalo': 2, 'advil': 1, '10th': 1, 'readied': 1, 'self-surgery': 1, 'states': 32, 'complicate': 1, 'true': 96, 'emerson': 2, 'mopey': 1, 'technicians': 3, 'employer': 3, "warren's": 2, '21': 2, 'dyer': 1, 'sophomoric': 5, 'unpopped': 1, 'caddyshack': 3, 'entangled': 1, "walken's": 1, 'eerily': 4, 'unimpressed': 1, 'houyhnhnms': 1, 'sit-coms': 2, 'climax': 49, 'mass-produced': 1, 'fictionalized': 1, 'trespassing': 1, 'deflecting': 1, 'extols': 1, "'stand": 1, 'forebearing': 1, 'automatic-pilot': 1, 'tv-series': 1, 'fret': 2, 'beginning': 101, 'proclivities': 1, 'giggling': 2, 'laugh-free': 2, 'proprietor': 3, 'distracts': 3, 'skateboard': 1, 'lotto': 1, 'pounce': 1, 'goro': 1, 'homilies': 1, 'registering': 1, "o'clock": 2, 'nibble': 1, 'including': 112, 'accents': 7, 'guamo': 1, 'naught': 1, 'phone-sex': 1, "dugan's": 1, 'timer': 2, 'ellen': 8, 'squished': 1, 'handfuls': 1, 'un-safe': 1, 'hill': 26, 'someting': 1, 'lucrative': 5, 'mule': 1, 'wells': 3, 'action-packed': 2, 'tour-de-force': 1, 'pierce': 8, 'contrasted': 1, 'nit-picky': 2, 'self-importance': 1, 'berkowitz': 1, 'unreleased': 1, 'linear': 3, 'accommodates': 1, 'duffle': 1, "thewlis'": 1, 'pope': 3, 'decidedly': 11, 'comeback': 4, 'introductions': 3, 'craven-directed': 1, 'wife': 144, 'unequivocal': 2, 'kellogg': 2, '_breakfast_of_champions_': 1, 'paves': 1, 'infinitum': 2, 'cruel': 11, 'downshifts': 2, 'alien/human': 1, 'satisfied': 8, 'enola': 1, 'hassidic': 1, 'steaming': 2, 'showed': 26, '_wartime': 1, 'reef': 2, 'unsatisying': 1, 'tiberius': 1, 'reaching': 5, 'clooney': 12, 'cross': 23, 'scraggly': 1, 'robbers': 3, "grandparents'": 1, 'personification': 1, 'measures': 3, 'itchy': 2, 'technical': 28, 'airplanes': 2, '1-dimensional': 1, 'sudden': 25, 'observation': 5, 'widening': 1, 'junkie': 5, 'twenty-four': 1, 'painter': 5, 're-visited': 1, 'excrucitating': 1, 'sluttier': 1, 'kitana': 1, 'authorized': 1, 'bode': 1, 'outrage': 2, 'monologue': 7, 'granger': 7, 'hook-hand': 1, 'mutations': 1, 'pd': 1, 'downcast': 1, 'sat': 27, 'brubaker': 1, 'legalized': 1, 'bought': 11, 'repetitively': 1, 'inside': 52, 'directorship': 1, 'titular': 10, 'shin-high': 1, "caan's": 1, 'amiss': 2, 'frustrated': 11, 'ravich': 1, 'asner': 2, 'circumstance': 1, '-rated': 1, 'incoherent': 11, 'kissed_': 1, 'borgnine': 1, 'sure-fire': 2, 'fried': 3, 'hour': 114, 'denueve': 1, 'chaotic': 5, "tiffany's": 1, 'beginners': 1, 'federal': 6, 'conventional': 11, 'beleive': 1, 'kalecki': 1, 'brawl': 5, 'recreated': 2, 'excusable': 2, 'nitrous': 2, 'spy': 17, 'forming': 3, 'marty': 3, 'even-tempered': 2, 'langer': 1, 'scrambled': 2, 'dribble': 2, 'aquatic': 1, 'disclosed': 2, 'prevalence': 1, 'flourishes': 3, 'caro': 1, 'benicio': 4, 'ali': 7, 'passes': 20, 'entry': 23, 'baffle': 1, 'shepherd': 1, 'marketable': 3, 'ritz': 2, 'meter': 4, '[pg-13]': 5, 'saws': 2, 'broken': 36, 'doll': 6, 'graff': 2, 'brother-in-law': 3, 'attended': 12, 'athletic': 3, 'paranormal': 2, 'earliest': 3, 'trooper': 3, 'ying': 1, 'fleshing': 1, 'simplicity': 2, 'punishable': 2, 'motivations': 11, 'bizarre': 45, 'tactical': 1, 'self-discoveries': 1, 'phil': 18, "gloria's": 1, 'matt': 18, 'adaptations': 8, 'popcorn': 10, "lawrence's": 3, 'nada': 1, 'impression': 46, 'ragtag': 3, 'care': 127, 'venomous': 1, 'hawn': 4, 'underling': 2, 'mexican': 6, 'apparitions': 2, 'frees': 1, 'taylor': 28, "family's": 8, 'pitch-black': 1, 'pudding': 1, 'moview': 2, 'pleasant': 13, 'demure': 1, 'zone': 17, 'arsenio': 1, 'hand-wringing': 1, 'cronenberg': 1, 'g-friend': 1, 'incongruous': 2, 'emotion-charged': 1, 'epitome': 5, 'noel': 2, 't2': 2, 'phlegmming': 1, 'curvaceous': 1, 'sould': 1, 'nurturing': 2, 'dunn': 5, 'generation': 31, 'tele-marketer': 1, 'ultra-violence': 1, 'coca-cola': 1, 'vansihes': 1, 'appeals': 3, 'referencing': 1, 'indomitable': 1, 'newsmagazine': 2, "tim's": 1, 'walks': 27, 'allowed': 29, 'henson': 1, 'graeme': 1, 'in-your-face': 4, 'years': 230, 'alert': 6, '14': 8, 'second-chance': 1, 'action-adventure': 2, 'laying': 3, 'entanglements': 1, 'reverse': 7, 'amphetamines': 1, 'fell': 19, 'conspiracy': 24, 'swift': 5, 'jurek': 1, 'reilly': 5, 'vcr': 2, 'bowls': 2, 'trendy': 1, 'altman': 2, 'hf': 1, 'none-too-subtle': 1, 'griswold': 2, 'thesis': 3, 'cropduster': 1, 'scantily': 3, 'chord': 2, 'saga': 6, "al's": 2, 'kremp': 3, 'dully': 1, 'impressions': 2, "jason's": 1, 'accidentally': 31, "`mission'": 1, 'lundy': 1, "burton's": 7, 'suffered': 13, 'scenes--grocery': 1, 'grudge': 2, 'gentle': 10, 'rehashed': 5, 'fact': 258, 'towners': 1, 'phantoms': 1, 'divinity': 1, 'cking': 1, 'benzali': 2, 'pruit': 1, 'reverential': 1, 'non-fiction': 2, 'antagonist': 4, 'cellist': 2, 'famously': 1, 'eighties': 4, 'plot-by-numbers': 1, 'po': 1, 'cannell': 1, 'playback': 1, '1949': 2, 'beloved': 14, "goldblum's": 1, 'bureau': 2, 'swooping': 1, 'retalliation': 1, 'split': 10, 'tumor': 2, 'chortled': 1, "keaton's": 2, 'hunchback': 1, 'beastmaster': 1, 'snap-brim': 1, 'ignoring': 4, 'tyler': 15, 'lip-synch': 1, '9=borderline': 1, 'recreations': 1, 'binks': 2, 'constipated': 2, 'grinds': 3, 'sonia': 1, '15': 22, 'aurore': 1, 'individuality': 3, 'streaking': 1, 'rainfall': 1, 'harvests': 1, 'lobotomy': 2, '_doctor_dolittle_': 1, 'terms': 29, 'waft': 1, 'burns': 7, 'spielbergization': 1, 'granny': 3, 'decapitations': 2, 'waturu': 1, 'olympia': 1, 'throbbing': 1, 'chaps': 1, 'collap-': 1, 'ingredients': 10, 'conqueror': 1, 'professional/leon': 1, 'sake': 26, 'boobs': 3, 'intercom': 1, 'symbolic': 7, 'disjointedness': 1, 'swiss': 2, 'hot': 49, 'gathered': 2, "clang's": 1, 'elmaloglou': 1, 'briggs': 2, 'consequences': 9, 'eradicated': 2, "musketeer's": 1, 'manufacture': 2, 'consenting': 1, 'rocks-actually': 2, 'scientests': 1, 'replenish': 1, 'road-comedy': 1, 'portal': 1, 'beams': 4, '-the': 1, "'r'": 1, 'angelic--in': 1, '4': 42, 'specter': 2, '44': 1, 'copulation': 2, 'predates': 1, 'hind-ends': 1, 'stoic': 5, 'kops': 1, 'one-two-punch': 1, 'seduced': 5, 'hawking': 2, 'dejectedly': 1, 'stepsister': 1, 'slasher': 27, 'taboo-smashing': 1, 'denzel': 4, "russel's": 1, 'dazzling': 4, 'schultz': 1, 'browning': 2, 'hypothalamuses': 1, 'jacqeline': 1, 'slow-motion': 8, 'videophones': 1, 'slater': 7, 'lesbianism': 2, 'swear': 11, 'sabotages': 1, 'highly': 31, 'shoot-em': 1, "leder's": 1, 'gilles': 1, 'emphasis': 11, 'painkiller': 1, 'amrriage': 1, 'tears': 13, 'evaporating': 1, 'stay': 69, 'chain-weed-smoking': 1, 'gottfried': 1, 'naville': 1, 'cinematography': 41, 'patronized': 1, 'junket': 2, "'90's": 1, 'unauthorized': 1, 'stevens': 2, "early-'90s": 1, 'jeter': 2, 'management': 1, 'woman-bashes': 1, 'caption': 5, 'scumbagginess': 1, 'possessive': 1, 'reel-to-reel': 1, "skeet's": 1, 'copybook': 1, 'girl-friends': 1, 'kind-hearted': 1, 'guillotine': 1, 'zellwegger': 1, 'broods': 1, 'ridiculity': 1, 'diversion': 2, 'part-time': 2, 'ordinary--premise': 1, 'hole\x14': 1, 'lazy': 20, 'characters--but': 1, 'bamboo': 1, 'approximately': 10, "pfieffer's": 1, 'kennif': 1, "dwayne's": 1, 'do-gooder': 2, 'demarco': 1, 'misfire': 9, 'idiotically': 1, 'unbearable': 14, 'dissimilar': 3, 'horror/comedy': 1, 'cabin': 5, 'soporific': 1, 'movement--without': 1, 'inconclusive': 1, 'scheduled': 5, 'emperors': 1, 'chants': 1, 'dominique': 1, "`colorful'": 1, '_really_': 4, 'defied': 2, 'lon': 1, 'sh': 2, 'incompetence': 6, 'segal-like': 1, 'stupidity': 25, 'lieu': 2, 'deflect': 1, 'kgb': 1, 'scorecard': 1, "that'd": 1, 'horror/schlock': 1, "heckerling's": 3, 'donald': 15, 'caution': 3, "`zulu'": 1, 'perfected': 3, 'pinon': 1, 'sophie': 3, 't-rex-i': 1, 'clubs': 3, 'personage': 1, 'crummy': 2, 'tight-jawed': 1, 'reaper': 1, "fleischer's": 1, 're-claim': 1, "austin's": 1, 'invincible': 2, 'barf': 2, 'tuptim': 1, 'mart': 1, 'suppose': 55, 'bread': 2, 'liar': 4, 'amaze': 3, 'monthly': 2, 'recognize': 21, 'cast--': 1, 'freaky': 4, 'nominal': 1, "washington's": 1, 'movie-ness': 1, 'films': 343, 'redd': 1, 'neglects': 2, 'in-': 2, 'perfect--you': 1, 'digestions': 1, 'heroes': 31, 'merciless': 3, 'ignites': 1, 'spaceships': 1, 'vienna': 1, "ghoul's": 1, 'pixie-hairdo': 1, 'thursday': 1, 'distressed': 1, 'stunted': 1, 'synergy': 1, 'understood': 11, 'brewing': 2, 'moviestar': 1, 'direct-to-cable': 1, 'dateless': 1, "lynn's": 2, 'tryst': 1, 'tv-conversions': 1, 'nicol': 1, 'munches': 1, 'rumor': 2, 'atms': 1, 'disdainful': 1, 'speaks': 20, 'pleasingly': 1, 'unpredictability': 2, 'mega-producer': 1, 'torpedoes': 4, 'proverbs': 1, 'trent': 2, 'snobby': 1, 'selfishness': 2, 'bins': 1, 'atm': 1, 'eaton': 1, "don'tcha": 1, 'savior': 5, "jacobi's": 1, 'ninth': 4, 'composition': 2, 'short-on-substance': 1, 'eloi': 1, 'unobjective': 1, 'tyranny': 1, 'brutality': 5, 'impossible': 67, 'propels': 2, 'delightfully': 8, '50s-ish': 1, 'scientist': 26, 'arch-enemy': 1, 'mania': 3, 'guillermo': 3, 'throw-away': 1, 'kilrathi': 2, 'arrrghhh': 1, 'schrader': 2, 'cheering': 2, 'goonies': 1, 'evaluate': 1, 'surprisingly': 60, "mars'": 2, 'octopussy': 1, 'paraphernalia': 1, 'replacements': 2, 'sadie': 1, 'skin-diving': 1, 'station-giveaway': 1, 'chimp': 3, 'rated': 39, 'evoke': 8, 'skateboard-related': 1, 'damned': 13, 'seducing': 1, 'coastline': 1, 'kirsebom': 1, 'dana': 7, 'obligatory': 26, 'half-a-million': 1, 'focused': 7, 'harbours': 1, 'oberon': 1, 'stated': 10, 'yawner': 2, 'savages': 5, 'arcand': 1, 'ahern': 2, "roy's": 2, 'delicious': 4, 'involuntarily': 1, 'gatlin': 1, 'outing': 14, 'accidently': 2, 'pig-pig-pig-pig-pig': 1, "'well": 1, 'amish-mocking': 1, 'stopping': 6, 'pollutes': 1, 'careen': 1, 'pirate': 4, 'slight': 12, 'drugging': 1, 'double-exposure': 1, 'tough': 35, 'reprehensible': 4, 'lowry': 1, 'weak': 50, 'fireballs': 2, 'nuts': 9, 'ill-timed': 2, 'con-man': 1, 'holbrook': 1, "violation'": 1, 'unfinished': 1, 'eclairs': 1, "bening's": 2, 'roadster': 1, 'flew': 5, 'grips': 1, 'tricky--': 1, 'tarantino-look': 1, 'peter': 80, 'filmgoers': 1, 'typist': 1, 'schemes': 9, 'fighters': 6, 'vignette': 2, 'smarmy': 8, "'basic": 1, 'absurdist': 1, 'shoestring': 1, "berkley's": 1, 'hipper': 1, 'pork': 2, 'interests': 6, 'intruder': 1, 'amor': 1, 'standby': 3, 'incredibly': 48, "melvin's": 2, 'eliminated': 8, 'guard': 23, 'uninterested': 4, "carl's": 1, 'gnaw': 1, 'plush': 3, 'smitten': 1, 'willpower': 1, 'unnecessarily': 6, 'in-heat': 1, 'foreign-guy-who-mispronounces-english': 1, 'penniless': 2, 'outruns': 1, 'goers': 1, 'scantly': 1, 'intereference': 1, 'rubin-vega': 1, 'psychiatrists': 1, 'office': 61, 'half-assedness': 1, 'preview': 11, 'currency': 2, 'central': 45, 'cracking': 3, 'poverty': 1, 'strained': 8, 'annoyingly': 8, 'quimby': 2, 'bungle': 1, 'marooned': 1, 'destabilize': 1, 'regains': 1, 'darryl': 4, 'roomie': 1, 'steve': 56, 'together--feuding': 1, 'flamethrowers': 1, 'shrugs': 2, 'beckon': 1, 'undoubtably': 1, 'turteltaub': 1, 'boling': 1, 'narratives': 2, 'telefixated': 1, 'above-par': 1, 'posing': 9, 'peck': 1, 'inmates': 5, 'generically': 1, 'mining': 14, 'insisting': 2, 'oppertunity': 1, 'brusque': 1, 'strayed': 1, 'harry': 35, 'weatherman': 1, 'snap': 6, 'horrid': 9, 'suite': 2, 'lassez': 2, 'recipients': 1, '3-d': 2, "1996's": 12, 'antiquated': 1, 'stuff': 91, 'dilema': 1, '`hanging': 1, 'man-whore': 1, 'fishing': 14, 'insurrection': 3, "one's": 21, 'downloading': 1, 'humanness': 1, 'fearful': 2, 'miracle': 5, "horse's": 1, 'misses': 16, 'planetary': 1, 'dragon-like': 1, 'broadcasting': 4, 'punks': 4, 'pseudo-documentary': 2, 'bashfully': 1, 'orbach': 1, 'abstract': 2, 'heavens': 1, 'notches': 3, 'kewpie': 1, 'streak': 5, 'lyric': 4, 'matty': 2, 'hitting': 13, 'a--': 1, 'parsons': 4, '5-year-old': 1, 'chase': 58, 'gevedon': 1, 'follow-up': 9, 'burger-flipper': 1, 'beta': 2, 'foreign': 24, 'mailman': 2, 'dizzying': 7, 'retarded': 1, 'housewarming': 1, 'notebook': 2, 'vocabulary': 4, 'hacky': 1, 'deutch': 1, 'live': 110, "ling's": 1, 'cornish': 1, "student's": 3, 'mutters': 2, 'graduated': 3, 'ucla': 1, 'moment-for-moment': 1, 'stomach-churning': 1, 'bouchez': 1, 'silverado': 2, '-my': 1, 'relish': 3, 'euphoric': 1, '1982': 6, 'dolly': 1, 'tippi': 1, 'unreasonably': 1, 'emotional-radiance': 1, 'stone': 35, 'hotel_': 1, 'pongo': 1, 'placate': 1, 'environmentally': 1, 'married': 42, 'dreadful': 17, 'revival': 7, 'politely': 2, 'worshippers': 1, "duchovny's": 1, 'announce': 2, 'five-finger': 1, 'ascetic': 1, 'aspirations': 6, 'loomed': 1, 'full-force': 1, 'extramarital': 1, 'spooks': 1, 'band': 35, 'stalked': 6, 'faux': 6, "heroes'": 3, 'heartthrob': 2, 'zealous': 1, 'fun-yet-dumb': 1, 'conversation': 15, 'narratively': 1, 'stormy': 2, 'rusted-out': 1, "barrymore's": 2, 'racking': 1, 'deer': 1, 'soggy': 2, 'revitalized': 1, 'outlines': 1, 'norman': 11, 'six-year': 3, 'mcmurray': 1, 's$19': 1, 'nazis': 3, 'suddeth': 1, 'steenburgen': 1, 'sinner': 1, 'party-pooper': 1, 'ogre': 4, 'melting': 3, 'bastard': 8, 'chick': 18, 'nihilistic': 2, 'borrowing': 4, 'racially': 1, 'abby': 1, 'sidetracked': 4, 'golfing': 1, 'ehrin': 1, 'stogie': 1, 'effect': 60, "crystal's": 2, '`they': 1, 'dangling': 10, 'serous': 1, 'henrikson': 1, 'scrape': 1, 'store': 41, 'seated': 6, 'fred': 11, 'mutates': 1, 'baseless': 3, 'funnier': 16, 'invades': 3, 'swede': 1, 'gloria': 5, 'raccoon': 2, 'man-in-black': 1, 'polti': 1, 'scaling': 1, 'cloony': 1, 'deane': 1, 'overjoyed': 1, 'actors---washington': 1, "debbie's": 1, 'robert': 75, 'delude': 1, 'gen-x': 1, 'staging': 5, "winner's": 1, 'transaction': 1, 'notre': 1, "tee--it's": 1, 'donnas': 1, 'displaying': 3, 'denver': 4, 'rhode': 2, '21-year': 1, 'ammo': 2, 'dalmatians': 5, "gabriel's": 2, 'gadget': 5, 'jew': 4, 'planet': 62, 'oomph': 1, 'knowing': 18, 'oooooo': 1, 'fern': 4, "columbus'": 2, 'ungodly': 2, 'themself': 1, 'differing': 1, 'fincher': 3, 'matthau': 8, 'coverage': 2, 'schwartzenegger': 1, 'hanging': 21, 'ethnic': 3, 'shelling': 1, 'blue-screens': 1, 'wring': 2, 'skyline': 1, 'submits': 1, "stephie's": 1, 'goldman': 5, "courtney's": 2, '_exactly_': 1, 'half-interested': 1, 'shuttles': 3, 'contentment': 1, 'cop': 57, 'facepaint': 1, 'thrilled': 3, 'mix': 33, 'exhibited': 1, 'creeped': 1, 'lemons': 1, 'writers': 63, 'lapsed': 1, 'hot-enough-to-melt-rubber': 1, 'dumbed': 6, 'received': 18, 'supervised': 1, "dien's": 1, 'shaved-headed': 1, 'html': 1, 'disrespectful': 1, 'trois': 1, 'prevert': 1, 'zakharov': 1, 'grabbing': 3, "besson's": 2, 'bringing': 22, 'unruly': 3, 'margerithe': 1, 'blatancy': 1, 'council': 1, 'adores': 2, 'edit': 2, 'medic': 1, 'broth': 1, 'pisani': 1, 'god-like': 2, 'anticipation': 9, 'rowe': 1, 'mechanical': 13, 'dante': 1, 'chic': 3, 'trials': 1, 'babe-a-licious': 1, 'landmarks': 2, 'slimes': 1, 'connotes': 1, 'egypt': 4, 'scariest': 3, 'rushing': 4, 'soon-to-be-committed': 1, 'allotted': 1, 'possiblities': 1, 'laded': 1, 'gender': 6, 'morant': 1, 'spindly': 1, 'beatrice': 2, 'elena': 3, 'miller-ey': 1, 'foodstuffs': 1, "knockin'": 1, 'pay-day': 1, 'barb': 2, 'withers': 1, 'duh': 4, 'neon': 3, 'whizzing': 1, 'septien': 1, 'unmistakable': 3, 'infuse': 2, 'absorbs': 1, "protagonist's": 2, 'analogy': 3, 'strokes': 1, 'deja': 3, 'heche': 4, 'wednesday': 1, 'pete': 3, 'clifton': 1, 'outcasts': 1, 'pathetic': 52, 'satisfaction': 4, 'indebted': 1, 'redford': 1, 'solitary': 5, 'consume': 1, 'knocked': 10, 'comics': 6, 'flunky': 1, 'physique': 1, 'temple': 4, 'poop': 1, 'bubble': 5, 'voyage': 9, 'comprehendably': 1, 'mcmullen': 1, "freddie's": 1, 'wide-angle': 2, 'namesakes': 1, "`diddly'": 1, "terribly-'90s": 1, 'infrared': 2, 'ralph': 5, 'watches': 13, 'cellulite': 1, 'cursing': 2, 'hardships': 1, 'valeria': 1, 'ammunition': 2, 'hopeless': 11, 'gaghan': 1, 'offerings': 5, 'improbably': 3, 'ronin': 5, 'videos': 12, 'terrrible': 1, 'cultures': 3, 'initally': 1, 'poignantly': 1, 'recovered': 1, 'character-': 2, 'brazenly': 3, 'organised': 1, 'nesting': 1, 'on-': 1, 'outsmarted': 2, 'roommates': 5, 'cerebral': 1, 'prances': 2, 'kuei-mei': 2, 'terminating': 1, 'interviewer': 1, "broderick's": 4, 'diva': 5, 'brown': 27, 'invest': 7, 'molesters': 1, 'disciplined': 1, 'surrounded': 10, 'out-of-place': 1, 'pesence-challenged': 1, 'forfeited': 1, 'truckers': 2, 'und': 1, 'bad-enough-to-be-good': 1, 'pressure': 10, 'scoundrel': 1, 'bushy': 1, 'voight': 11, 'hyper': 2, 'grisham': 5, 'consistently': 12, 'interviewed': 5, '_require_': 1, 'patiently': 2, 'atop': 4, 'notwithstanding': 1, 'reproduces': 1, 'cohen': 4, 'spy-girl-for-the-bad-guy': 1, 'porcine': 1, 'laurene': 1, 'award-winning': 4, 'applesauce': 1, 'merchant-ivory': 1, 'cheeze': 1, 'lusty': 2, 'personal--but': 1, "ferguson's": 1, 'thee': 2, "cross's": 1, 'wwii': 1, 'entertain': 22, 'symbolised': 1, 'net': 5, 'sticks': 14, 'foisted': 1, 'electronica': 1, 'cross-country': 4, 'cranked': 2, 'rousingly': 1, 'ostensible': 1, 'sfx': 1, 'guide': 14, 'whore-chasing': 1, 'high-powered': 2, 'handguns': 1, 'mtcts2': 1, 'fate': 23, 'perception': 2, 'wu': 1, 'progressed': 4, 'involvement': 13, 'cannibalistic': 1, 'outcast': 4, 'arbogast': 3, 'talent': 111, "krishnamma's": 1, 'stoops': 1, 'lived': 19, 'due': 61, 'aromas': 1, 'dared': 2, "stein's": 1, 'paratrooper': 1, 'korner': 1, 'insanely': 4, 'noho': 1, 'emcee': 1, 'scanner': 1, 'moxin': 1, "segal's": 1, 'marisa': 5, 'misplaced': 4, 'mostly-silent': 1, "murphy's": 3, 'movie--a': 1, 'wilma': 1, 'chooses': 15, 'with--which': 1, 'critic': 25, 'gizmos': 2, 'unmotivated': 2, "ellin's": 1, 'athleticism': 1, 'venora': 2, 'spinning': 8, 'finally': 120, 'ponderously': 1, 'vy': 1, 'greats': 1, 'cb': 1, 'angels-style': 1, 'waikiki': 1, 'alexander': 8, 'we-lei': 1, 'lynn': 4, 'court': 14, "students'": 1, 'butler': 7, 'lycanthropy': 1, 'zsigmond': 1, 'supporting': 91, 'showings': 1, 'farmer': 1, 'improvised': 4, 'true-to-heart': 1, 'extricate': 2, "abc's": 1, 'meld': 1, 'allegory': 4, 'bilko': 4, 'epps': 5, 'underwritten': 8, 'smooth': 5, 'fragmented': 3, 'autographs': 1, 'bowery': 1, 'creepiness': 2, 'piller': 1, '32': 2, 'breaths': 1, 'starlet': 3, 'crystal-clear': 1, 'assness': 1, 'laxative-induced': 1, 'perpetually': 7, 'henstridge': 10, 'notice': 35, 'donny': 1, 'heavy-handed': 10, 'dreary': 15, 'sadler': 4, 'elementary': 3, 'nonexistent': 4, 'taplitz': 1, 'knockoffs': 2, 'patron': 2, 'unendurably': 1, 'ridiculed': 3, 'trained': 8, 'goofy': 27, 'kohn': 1, 'peasant': 2, 'glazed': 1, 'ungloriously': 1, "'face": 1, 'unapologetically': 2, 'hattie': 2, "someone's-in-the-house": 1, 'roadside': 1, "kaisa's": 1, 'movie-plot-ridiculity': 1, '$100': 10, 'birdy': 1, 'barker': 2, 'waiter/struggling': 1, 'pg': 11, 'hedley': 1, 'excitement': 23, 'restatement': 1, "keitel's": 1, 'maids': 1, 'numbness': 1, 'disguises': 6, 'unfulfilled': 2, 'shawshank': 3, 'technicolor': 2, 'stacey': 2, 'yimou': 1, 'comparing': 6, 'stabbing': 1, 'qualifies': 3, 'stirring': 3, 'cheap': 63, 'tcheky': 4, 'kelsch': 1, 'eclipse': 1, 'varied': 2, 'fedex': 1, 'altercations': 1, 'sniper': 3, 'mutually': 1, 'tanaka': 2, "munchkin's": 1, 'outstandingly': 1, 'reputable': 4, 'charts': 4, 'gavin': 3, 'guarding': 1, 'passingly': 2, '`total': 1, 'argued': 4, 'drift': 5, 'mosquito': 1, 'distraught': 4, 'uncovers': 4, '-kilter': 1, 'sprite--this': 1, 'fare': 30, 'spheeris': 1, 'delapidated': 1, 'earthquake': 6, 'valiant': 1, 'nerdy': 4, 'top-secret': 3, 'crippen': 1, 'harmed': 1, 'landowners': 1, 'slumber': 2, 'classmates': 2, "6'1": 1, 'trainer/consultant': 1, 'quick-cut': 1, 'ever-popular': 2, 'heavy': 50, "shue's": 3, 'kidnaps': 7, 'competition': 7, 'serling-ish': 1, 'winds-up': 1, 'widowed': 3, 'fraser': 2, 'vehicular': 1, 'retro': 2, 'curtain': 1, 'mineral': 2, 'nemeses--shaken': 1, 'hillhouse': 1, 'chore': 10, 'marie-laure': 1, 'suburbs': 2, 'losing': 20, 'transmit': 3, 'tugs': 1, 'unexploitive': 1, 'co-wrote': 5, 'matter-of-factly': 1, 'guiteau': 1, 'blasted': 2, 'contradicting': 1, 'charlotte': 7, 'baxter': 1, 'fling': 2, 'attatched': 1, 'moral': 21, 'fetched': 1, 'savage': 11, 'chapel': 1, 'digitally': 1, 'sophisticated': 12, 'sitter': 1, 'first-timer': 2, 'rustin': 1, 'four--yes': 1, 'paroled': 2, 'musetta': 1, "screen--arkin's": 1, 'lives': 100, 'well-trained': 1, 'teenage': 48, 'advisors': 1, 'cubes': 1, 'nelken': 2, 'necessities': 2, 'periodic': 1, 'cordless': 1, 'honors': 2, 'refusing': 4, 'exceptionally': 9, 'julliana': 1, 'baked': 3, 'circling': 2, 'studios': 16, 'ballhaus': 1, 'seriously--god': 1, 'stack': 3, 'implanting': 1, 'bandaged': 1, 'newhart': 2, 'natalie': 8, 'zhang': 1, 'caretaker': 2, 'skinheaded': 1, 'neo-reality': 1, 'stamina': 1, 'freight': 1, 'handstand': 1, 'establishes': 7, 'ever-partying': 1, 'huddle': 2, '"': 697, 'questioning': 3, 'pass': 45, 'seminude': 1, 'reading': 24, 'assumptions': 1, "'ghostbusters'": 1, 'father-son': 2, 'voyeurs': 1, 'stoner': 6, 'dumber--and': 1, 'chaired': 2, 'ambiguity': 2, 'idiotic': 30, 'distant': 10, 'manage': 40, 'hytner': 2, 'intellectualism': 1, 'sputters': 4, 'familiar': 65, 'sub-snl': 1, 'save': 128, 'alice': 7, 'frenzied': 4, 'hacker': 3, 'coffee': 16, '60': 10, 'scarborough': 1, 'cujo': 1, 'birds': 1, 'by-standers': 1, 'originator': 2, 'mckenna': 1, 'sorts': 18, 'thx': 1, 'nutty': 10, '12-year': 2, 'cinematographer': 19, 'stores': 5, '1/3': 1, 'interim': 1, 'climaxes': 2, 'ailing': 3, 'overemotional': 2, 'writer-director': 8, "montgomery's": 1, 'proffered': 1, 'exposes': 1, 'moments': 136, 'crassness': 1, 'sequel': 74, 'rides': 3, 'engelman': 1, 'furthering': 1, 'mustered': 1, 'bruno': 5, 'wall': 29, 'driscoll': 1, 'facilitate': 1, 'tightens': 1, 'impassioned': 2, 'firelight': 1, "'mr": 1, 'rabbi': 2, 'backfires': 2, 'footage': 31, 'stop-animation': 1, 'stunts': 23, 'francois': 2, 'two-bit': 1, 'uninvolivng': 1, 'barges': 2, 'tweaking': 1, 'avenue': 1, 'rightful': 1, 'half-assed': 5, 'catches': 18, 'roughneck': 1, 'contemplation': 1, 'stater': 1, 'confusions': 1, 'finely': 1, 'pursuer': 1, 'shield': 4, 'hearted': 8, 'valued': 1, 'steals': 16, 'glossed': 1, 'vivica': 5, 'pantolinao': 1, 'maura': 1, 'gambit': 2, 'burbano': 1, 'lindy': 1, 'klebold-harris': 1, 'breasted': 2, 'dis': 1, 'rochon': 6, 'jackpot': 2, 'imgen': 1, 'babe': 13, 'owns': 7, 'chcaracters': 1, 'insinuating': 1, 'brother': 64, '70s': 9, 'understand': 61, 'jointly': 1, 'sequel-the': 1, 'alarming': 2, 'bull': 9, 'meritorious': 1, "host's": 1, 'rebbe': 1, 'taling': 1, 'cover-ups': 1, '1947': 2, 'senior': 7, 'dangereuses': 1, 'expansive': 1, 'first-run': 2, 'mounting': 2, 'panic': 4, 'learn': 57, 'halted': 1, 'carnal': 1, 'majors': 3, 'stinking': 1, 'exquisite': 2, 'restoring': 1, 'distanced': 3, 'formation': 3, 'robo': 1, "financee's": 1, 'smother': 2, 'not-quite-half-baked': 1, 'clinched': 1, 'pleaser': 3, 'entrapment': 1, 'muscles': 4, 'hush-hushes': 1, 'jewess': 1, 'limply': 2, 'sooo': 1, 'wyatt': 4, 'flabbergastingly': 1, "speechless's": 1, 'deduction': 1, 'briar': 1, 'wringing': 1, 'woodies': 1, 'rapidly': 12, 'cemented': 1, 'bryan': 3, 'klein': 7, 'badness': 7, "'kong": 1, 'vincent': 13, 'stargher': 1, 'phony': 12, 'bloodbath': 3, 'bombarding': 1, 'tensy': 1, 'puerto': 2, 'medic-alert': 1, 'masses': 2, 'scrawled': 1, 'deguerin': 1, "zoom-in's": 1, 'negate': 1, 'disappearance': 1, 'up-to-date': 1, 'shook': 1, 'circuit': 3, 'cider': 1, 'single-digit': 1, "huff's": 1, "macdowell's": 1, 'flipping': 3, 'abyssinian': 1, 'wallets': 1, 'clifford': 4, 'madoka': 1, 'noble': 12, '`captain': 1, 'off-post': 1, 'shies': 1, 'feats': 2, 'slave-talk': 1, 'stance': 2, 'continue': 28, 'skeletal': 3, 'deficient': 3, 'starred': 14, 'trying-to-be-hip': 1, "'thriller": 1, 'structurally': 2, 'comforting': 1, 'hovertank': 1, 'out-of-body': 1, 'slickest': 1, 'riveting': 4, 'squid-like': 1, 'bet': 13, 'untidy': 1, 'bourgeoisie': 1, 'tropical': 4, 'actioners': 2, 'home-made': 1, 'set-up': 17, 'holding': 18, "peoples'": 1, 'debates': 1, 'tonight': 6, 'yoda-esque': 1, 'rockers': 2, 'groupie': 1, 'example-': 1, 'unfulfilling': 2, 'conspirator': 1, 'reworking': 2, 'grieve': 1, 'non-': 1, 'drafted': 3, 'slammed': 2, "friends'": 2, 'tellingly': 2, 'depressingly': 2, 'woeful': 3, 'heaping': 1, 'geared': 5, 'season': 21, 'laughter-packed-moments': 1, 'secret': 62, 'producer/lover': 1, 'flavor': 8, 'assimilating': 2, 'frantic': 6, 'meets': 68, 'prompts': 1, 'whos': 2, 'carradine': 2, 'letterboxed': 2, 'blackman': 1, 'caldicott': 1, "assistant's": 1, "novelist's": 1, 'broadway': 6, 'bores': 1, 'berkeley--er': 1, 'doubtfire': 4, 'flatulating': 2, 'overcame': 2, 'viable': 1, 'novikov': 2, "lucas'": 1, 'snuggles': 1, 'sags': 2, 'vonnegut': 2, 'shows': 120, 'confrontations': 4, 'englund': 3, 'sound-bite': 1, 'tawdry': 1, 'fleshed': 3, 'blocks': 5, 'knits': 1, 'relationship': 99, '`eye': 1, 'advance': 18, 'unbalanced': 2, 'humanized': 1, 'silences': 1, 'starting': 29, 'eye_': 1, 'massacusetts': 1, 'could--run': 1, 'anticipants': 1, 'mouthed': 1, 'mustache': 5, 'fighter': 16, 'moonlighting': 1, 'nitro': 5, 'stifled': 3, 'detached': 5, 'channels': 2, 'material--a': 1, 'bulk': 10, 'areas': 15, '9': 18, 'toho': 1, 'baseball-related': 1, 'lone': 7, 'none-too-pleased': 1, 'hell-liner': 1, 'getting-even': 1, 'step-father': 1, 'accounts': 5, 'embodies': 2, 'keystone': 1, 'signing': 3, 'enrolling': 1, 'crisis': 11, 'trends': 3, 'district': 2, 'erbe': 1, 'f-18': 1, 'vitality': 2, 'top-notch': 2, 'overwight': 1, 'lawerences': 1, 'reprogramming': 1, 'fearest': 1, 'loosing': 1, 'separates': 1, 'assassin/operative': 1, 'characteristic': 4, 'cnn': 1, 'chest': 12, 'aerial': 1, 'infested': 1, 'aisles': 4, "dracula'": 1, 'macks': 1, '4-d': 1, 'proceed': 8, 'portions': 8, 'flick': 79, 'ape-man': 1, 'free-wheeling': 1, 'endorsement': 1, 'immolates': 1, 'foretold': 1, 'surely': 22, 'penguin': 4, 'renound': 1, 'stinkiest': 1, 'gardener': 1, "'perfect'": 1, 'tex': 1, 'ramona': 1, 'carrie-anne': 1, 'co-writed': 1, "france's": 2, "kersey's": 2, 'discernible': 4, "animals'": 1, 'overburdened': 1, 'b-----d': 1, 'mock': 6, 'swerve': 1, 'dense': 3, 'intelligent--and': 1, 'hissing': 1, 'josh': 11, 'beetle': 1, 'distended': 1, 'warring': 4, 'hampshire': 1, 'harvey': 5, 'bargain': 7, 'heel': 1, 'demanding': 10, 'schitck': 2, 'trial': 18, "mao's": 1, 'nears': 2, "monica's": 3, 'head-scratcher': 2, '$50': 5, 'cruella': 3, 'unavoidable': 3, 'doze': 1, 'frosted': 2, 'intentions': 11, '108': 2, 'dalva': 1, 'cry-baby': 1, 'hexagons': 1, 'bayard': 1, 'vip': 2, 'half-machine': 2, "secret'": 1, 'topples': 3, 'dice': 7, '26th': 2, 'clerk': 16, 'julius': 1, 'radiation': 3, 'reaches': 20, 'shots': 76, 'glued': 2, 'repeat': 9, 'anthropological': 1, "brenner's": 2, 'trailer-a': 2, 'finesses': 1, 'lunatics': 1, 'bahamas': 5, "hall's": 1, 'chatham': 1, 'newcomer': 12, 'centered': 6, 'jowls': 1, "when's": 1, 'detective': 36, 'sharper': 1, 'reitman': 4, "mistress'": 1, '1975': 1, 'diminish': 1, "'lucky'": 1, ':': 570, 'slimy': 9, 'flawlessly': 2, 'sprinklers': 1, 'inhereit': 1, 'manhunt': 3, 'b-horror': 1, 'hesitant': 3, 'utmost': 3, 'crown': 8, 'oprah': 1, 'longed': 2, 'nikki': 1, 'suddenly': 57, 'equalizer': 1, 'prancing': 1, 'pruitt': 2, 'disinvited': 1, 'guitar-playing': 1, 'peace': 9, 'testy': 1, 'shroud': 1, 'eight-': 1, 'earth-shattering': 3, 'dosmo': 1, 'evidently': 2, 'software': 2, "jewison's": 1, 'unpleasantly': 1, 'irritating': 32, 'angles': 22, 'gesture': 4, 'side-effects': 2, 'sex-obsessed': 1, '360': 1, 'jim': 51, 'analect': 1, 'theater': 74, 'psychoanalyst': 1, 'creepy': 31, 'favorites': 7, 'shielded': 2, 'flooded': 6, 'maverick': 5, 'gestate': 1, 'moment': 122, 'post-w': 1, 'tenley': 3, 'personally': 13, 'checks': 5, 'ever-mutating': 1, "adam's": 1, 'ritually': 1, 'conjugal': 1, 'grieving': 3, 'trumps': 1, 'leick': 1, 'fence': 3, 'runners': 1, 'nearly-': 1, '99': 5, 'gooder': 1, 'inauspicious': 2, 'milieu': 2, "rudy's": 1, "spice's": 1, 'series--was': 1, "lemmon's": 2, 'arbitrary': 5, "spader's": 3, 'teeth': 20, 'tugboat': 2, 'satellites': 3, 'dane': 2, 'ivana': 1, 'rumpled': 1, 'naked': 43, 'sooner': 10, 'nephew': 5, "commandant's": 1, 'ambivalent': 1, 'bosses': 2, 'palatable': 3, 'appalling': 9, 'ric': 1, 'theaters': 35, 'desperate': 46, "azaria's": 1, "fern's": 1, 'bright': 44, 'rene': 3, 'passer': 1, 'realm': 12, 'rich': 52, "there're": 1, "movie-he's": 1, 'cretins': 1, 'hole': 18, 'melinda': 1, 'entity': 3, 'cgi-enhanced': 1, 'remote': 10, 'veteran': 22, 'neglect': 3, 'doug': 6, 'awesome': 8, 'melted': 1, 'lisa': 17, 'proof': 11, 'jurassic': 15, 'fish-': 2, 'underlying': 12, 'sweetest': 2, 'near-sighted': 1, 'input': 1, 'heroics': 1, 'make-up': 16, 'jester': 1, 'badder': 1, 'tainted': 2, 'bedded': 1, 'dissappointed': 1, 'vail\x05where': 1, 'horror': 98, 'bare-knuckle': 1, 'gilpin': 1, 'equality': 1, 'car-chase': 1, 'gems': 2, 'vilmos': 1, 'ponderous': 2, "army's": 3, "araki's": 1, "'financial": 1, 'whore': 7, 'argues': 2, 'flaky': 1, 'travellers': 1, 'mamet-style': 1, 'frozen': 8, 'fairuza': 2, 'cia-like': 1, 'molly': 12, 'stacks': 2, 'jumped': 4, 'run-shrieking-from-the-theater': 1, 'hawtrey': 1, 'tried-and-true': 2, 'headey': 1, 'lad': 2, 'trager': 1, 'poppa': 1, 'lindner': 1, 'thirty-year-old': 2, 'pitiful': 6, 'gershon': 6, 'telepathic': 1, 'crushed': 4, 'inherent': 7, 'trees': 7, 'one-time': 3, 'efficient': 6, 'therapist': 4, 'post-psychedelia': 1, 'salinger': 3, 'sniveling': 3, 'sonnenfeld': 6, 'ahem': 4, 'floating': 8, 'impunity': 1, 'perez': 2, "lil'": 1, 'buggers': 1, "'ani'": 1, 'slimeballs': 1, 'mumble': 1, 'analyzes': 1, 'arkin': 6, 'lost-ray': 1, 'www': 3, 'bank': 18, 'lucas': 7, 'calling': 15, "women's": 5, 'believeable': 1, 'transcendental': 1, 'touted': 4, 'not-so-stunning': 1, 'hormone': 2, 'broken-down': 2, 'skinny': 4, 'yared': 1, 'visitors': 3, 'eviction': 2, 'lettering': 1, 'movie]': 1, 'disasters': 5, 'exchange': 23, 'dubya': 1, 'rotting': 3, 'fictions': 1, 'type-cast': 1, 'announces': 5, 'mason': 4, 'slowing': 1, 'instability': 1, 'manifest': 2, 'devlin': 4, 'repeatedly': 12, 'camping': 2, '$90': 5, 'oversexed': 2, 'trusty': 6, "steiger's": 1, 'checkbook': 1, 'belly-button': 1, 'chamberlain': 1, 'regaining': 1, 'hsueh-wei': 1, 'english/drama': 1, '129': 2, 'ladles': 1, 'whisperer_': 1, 'resistibility': 1, 'finishing': 5, 'uniquely': 1, 'puzzling': 4, 'tripper': 1, 'fleder': 1, "company'": 1, 'jadzia': 1, 'extracted': 1, 'tripp': 2, 'indians': 4, 'pancakes': 1, 'belongings': 2, 'remaking': 3, 'stellar': 9, 'threatened': 4, 'goluboff': 1, 'midshipman': 1, 'grunt': 3, "'amateur'": 1, 'oppisites': 1, 'sneaked': 1, 'scandal-': 1, 'bare': 17, 'randian': 1, 'amboys': 1, 'gold-toned': 1, 'stylistically': 1, 'karloff': 2, 'tons': 8, 'recognizable': 5, 'antonioni': 1, 'deceptively': 1, 'mcconnell': 4, 'sensuous': 4, "bullock's": 2, 'pacing': 28, 'titshots': 1, 'material': 111, 'tingle': 2, 'settling': 1, 'damaging': 2, "change-i'll": 1, 'one-man': 1, 'shine': 4, "this's": 1, 'operatic': 1, 'choose': 16, '`mirabella': 1, "avery's": 1, 'virus-like': 1, 'loafer': 1, 'keifer': 2, 'balki': 1, 'stepsons': 1, 'prohibition-era': 1, 'double-bill': 1, "kattan's": 3, 'mains': 1, 'brutalized': 1, 'migraine': 1, 'allergy': 1, 'frightful': 2, 'softhearted': 1, 'action-horror': 1, 'sturm': 1, 'narcotic': 2, 'fairies': 1, 'visible': 2, 'coppola': 1, 'lauded': 3, "hank's": 2, 'rascal': 1, 'mcbain': 1, 'lewis': 16, 'gasoline': 2, 'gerbil': 1, 'dream-boat': 1, 'hasta': 1, 'tarantula': 5, 'color-by-numbers': 1, 'pee': 5, 'oppressed': 1, 'diaphragm': 2, 'littleton': 1, 'confides': 1, "1949's": 1, 'thong': 1, 'foe': 4, 'winners': 3, 'compelling': 23, 'wealthy': 23, 'sully': 1, "stallone's": 4, 'bidding': 1, 'convicted': 7, 'tilly': 1, 'starship': 12, 'kiss': 31, 'wavering': 2, "nichols'": 1, 'publicist': 1, 'girlfriends': 5, "folk's": 1, 'shopping': 11, 'alligators': 2, 'ern': 1, 'bootcamp': 1, 'flaunt': 1, 'obscure': 5, 'muddy': 2, 'talents': 46, 'corks': 1, 'oneliners': 1, 'happened': 64, "nicky's": 1, 'dining': 1, 'whoopi': 4, 'committed': 14, 'show-unrelated': 1, 'informed': 7, 'grim': 8, 'ample': 4, 'lamarr': 1, 'bikers': 1, "stone's": 8, 'sheds': 2, '1976': 7, 'nuff': 2, "jacob's": 3, 'tristen': 1, 'jurisdiction': 1, 'loftier': 1, 'obtuse': 1, 'dina': 7, 'delro': 1, 'godforsaken': 1, 'no-frills': 1, 'chiller': 1, 'ummm': 1, 'romance': 51, 'gloss': 2, 'gala': 1, 'franklin': 8, 'tremendous': 8, "'jurassic": 1, 'screen': 216, 'covetous': 1, 'hobo': 1, 'depicted': 2, 'youngers': 1, 'ranking': 2, 'booze': 4, 'flirting': 5, "star's": 2, 'minutes': 248, 'batgirl': 5, 'bakshi': 1, 'yanked': 2, 'inflections': 1, 'crosses': 3, 'comebacks': 2, 'arent': 1, 'marvelous': 3, 'knife': 13, 'zero-gravity': 2, 'dives': 1, 'linchpin': 1, 'muppets': 3, "'30s-style": 1, 'disturb': 1, 'satiric': 3, 'film--topping': 1, 'hardy': 2, 'francis': 3, "mind's": 1, 'koepp': 2, 'rightly': 3, 'inexperience': 2, 'galeweathers': 1, 'neumeier': 1, 'mcconaughay': 1, 'independence': 18, 'scanners': 3, 'cineplex': 1, 'raiden': 1, 'overabundance': 1, 'wee': 7, 'one-named': 1, 'ahh': 3, 'microphage': 1, '1997': 22, 'opposed': 6, 'advancement': 2, "sakamoto's": 1, 'kurgan': 1, 'payoffs': 3, 'big-name': 4, 'snort': 1, 'matures': 1, 'unwittingly': 3, 'pzoniaks': 1, 'align': 2, 'nastassja': 2, 'geocities': 1, 'uncertain': 7, 'infer-red': 1, 'viewing': 31, 'pretentions': 1, 'big-time': 5, 'two-day': 1, 'spirit': 24, 'freeman': 10, 'bonnie': 2, 'frankenstein-style': 1, 'overdo': 2, 'it+s': 1, 'dupe': 1, 'allusions': 2, "scream's": 2, 'sinbad': 1, 'carribean': 1, 'revolutionary': 2, 'lomax': 2, 'jealousy': 5, 'deserted': 9, 'festivities': 2, 'fervently': 2, 'unctuous': 1, 'lite': 1, 'sopranos': 1, "hoskins'": 1, 'enraging': 1, 'pensive': 1, 'wasting': 11, 'regina': 1, 'avalon': 1, 'frain': 1, 'salt': 4, "miner's": 1, 'coast': 7, 'superiority': 1, 'grade-school': 1, "crichton's": 4, 'attributed': 4, 'jericho': 6, 'amoral': 3, 'robocop': 4, 'typcast': 1, 'melora': 1, 'prompting': 2, '_dead_man_': 1, "lovin'": 1, 'radiation-induced': 2, 'co-ed': 1, 'self-absorbed': 5, 'intrepid': 4, 'parentless': 1, 'collision': 5, 'worldly': 4, 'whitey': 3, 'recount': 2, 'agent': 55, 'unsupervised': 1, 'jan': 12, 'nearby': 10, 'unprofessional': 1, 'schneider': 7, "roro's": 1, 'lands': 12, 'distilled': 1, 'native': 7, 'vehicles': 14, 'whoa': 6, 'deaf/mute': 1, 'cede': 1, 'ron': 15, 'suggestions': 2, 'answer': 50, 'mazursky': 1, 'gale': 3, 'prophecies': 3, 'lampooned': 1, 'thermians': 1, 'cavanaugh': 2, 'possessions': 2, 'hoskins': 1, 'lectures': 5, "herc's": 1, 'reserves': 1, 'prepared': 8, 'surreal': 11, '1903': 1, 'reclusive': 1, 'jeweller': 1, 'inveterate': 1, 'forced': 72, 'masquerade': 1, 'cheating': 11, 'seventy-eight': 1, 'parochial': 1, 'schumacher': 14, 'boat': 27, 'fedoras': 1, 'redirection': 1, 'offered': 16, 'wag': 2, 'angels': 15, 'bombast': 1, 'half-digested': 3, 'di$ney': 1, 'install': 1, '&': 59, 'housebound': 1, 'sticking': 8, 'consciously': 3, 'outdone': 3, 'prayer': 2, 'patched': 1, 'shoot-': 1, 'group': 121, 'scott': 59, 'catcher': 5, 'homecoming': 3, 'luxurious': 1, 'portrays': 10, 'stiers': 2, "brainard's": 1, 'ellsworth': 1, 'jettisoned': 3, 'requires': 22, 'redemptive': 1, 'tone': 54, 'better---so': 1, 'magic': 23, 'gould': 1, 'blows': 19, 'dogged': 4, 'cleansed': 1, 'stall': 6, 'lala': 1, 'di': 6, 'uuuuuuggggggglllllllyyyyy': 1, 'copious': 4, 'wyle': 1, "shaw's": 1, 'ehren': 3, 'glanced': 1, 'scooby': 2, "'ghetto": 1, 'orchestrated': 1, 'ultra-serious': 1, 'facet': 2, 'hayes': 2, '1974': 4, 'overwhelming': 6, 'sailing': 8, 'shallows': 1, 'strums': 1, "greg's": 1, "fiennes'": 1, 'concern': 13, 'intricate': 8, 'nonstop': 5, 'tray': 1, 'forgettable': 16, 'ornery': 1, 'findings': 2, 'built': 24, 'zeta-jones': 6, "dosen't": 3, 'snarky': 2, '`oh': 1, 'unncessary': 1, 'distinctions': 1, 'pogue': 2, 'appearance': 50, 'leaders': 4, 'curling': 1, 'pinkerton': 2, 'criticise': 1, 'spun': 2, 'pitched': 3, 'bruel': 1, 'effects-extravaganza': 2, 'torpor': 1, 'homosexuals': 1, 'bottle': 11, 'koven': 1, 'zegers': 1, "myers's": 1, 'wolfgang': 1, "_54_'s": 1, 'temper': 3, 'todd': 14, 'dragnet': 3, 'neatly': 3, 'thesps': 1, 'prepares': 5, 'staros': 1, 'bathe': 1, 'kamil': 1, 'imperfections': 1, 'waking': 5, 'resemblance': 9, 'regurgitation': 2, 'skirts': 1, "one-he's": 1, 'advertising': 3, 'subtitles': 4, 'pool': 14, 'dans': 3, 'high-level': 2, 'crotchety': 2, 'space-trucker-turned-impromptu-survivalist': 1, 'film-makers': 5, 'non-horror': 1, 'mulholland': 1, "kurosawa's": 3, 'half-hearted': 4, 'mascara': 1, 'dearest': 1, 'liner': 7, "dogs'": 1, "dove's": 1, 'biblical': 5, "days'": 2, 'joyous': 1, 'reviled': 1, 'pricey': 1, 'giulianni': 1, 'specific': 13, 'motor': 4, 'distractions': 2, 'existing': 4, 'crackle': 2, 'schooling': 1, "molly's": 1, 'paddle': 1, 'feds': 1, '0/10': 1, 'stalker': 2, 'outsmarts': 1, 'error': 6, 'git': 1, 'burnt-out': 1, 'snooze': 1, 'joker': 7, 'surviving': 9, 'monica': 11, 'artie': 4, '=20': 1, 'fighter-bombers': 1, 'embracing': 1, 'over-active': 1, 'slapped-together': 2, 'nineteen': 1, 'dishing': 1, 'good-and': 1, 'scrapped': 1, 'spotty': 2, 'airy': 2, 'liotta': 3, 'associate': 8, 'shoved': 4, 'attends': 1, 'tricked': 3, 'ledoyen': 1, 'biography': 2, 'monotone': 6, 'mallet': 1, 'skirt-chasing': 1, 'linkage': 1, 'guards': 9, 'designers': 3, 'romanticism': 1, 'twelfth': 1, 'cocktail': 3, 'inch': 5, 'grievously': 1, 'projections': 3, 'defying': 4, 'disorienting': 3, 'gen': 1, 'behaves': 3, 'bulgaria': 1, 'orchestral': 1, '$19': 1, 'dental': 3, 'horror/crime': 1, 'mega-advertising': 1, 'declaration': 4, 'seeds': 1, '29-year-old': 1, 'amphibian-based': 1, 'freeing': 5, 'sparkle': 2, 'entrepeneurs': 1, 'resurgence': 1, 'sydni': 1, 'herger': 1, 'hollow': 23, 'occupying': 2, 'parting': 1, 'blanchett': 2, "'blue": 1, 'commended': 2, '//www': 2, 'til': 1, 'pastoral': 1, 'undercut': 5, "neil's": 1, 'shortcomings': 4, 'hat': 15, 'crams': 1, '2/6/99': 1, 'moon-crawler': 2, "claire's": 1, 'walking': 32, 'donors': 1, 'brush': 2, 'speakers': 2, 'orangey': 1, 'swill': 4, 'headlining': 1, 'drained': 3, 'qualify': 7, 'supplied': 2, 'fehmiu': 1, 'testimonies': 1, 'nights': 20, 'dominio': 1, 'grate': 5, "trilogy's": 1, "langenkamp's": 1, 'bush': 3, "'twas": 1, 'ingenius': 1, 'counterargument': 1, 'narrated': 3, 'laugh-inducting': 1, "elliott's": 1, 'notices': 6, 'jackass': 3, 'butchers': 1, 'statues': 1, 'betterment': 1, "zwigoff's": 1, 'funding': 4, 'darn': 7, 'apparently': 97, 'wrong--and': 1, 'falcone': 1, 'crusader': 2, 'phillips': 9, 'cameos': 22, 'mood': 21, 'humoring': 1, 'syndrome': 9, 'cleverest': 1, 'dorn': 3, '2176': 2, 'thisclosefromdeath': 1, 'dreaded': 8, 'bowels': 5, 'rankin': 1, 'newsreels': 1, 'neighbour': 3, 'boarded': 1, 'sykes': 1, 'christened': 1, '101%': 1, 'awards': 12, 'nuzzles': 1, 'jarring': 6, 'interchangeable': 1, 'soldiers': 19, 'tv-idea': 1, 'flaps': 1, 'stupider': 1, 'scum': 1, 'functional': 1, 'explitives': 1, 'waves': 9, 'tips': 4, 'rabidly': 2, 'tight-fitting': 1, 'profitable': 4, 'iscove': 1, "sanders'": 1, 'amen': 1, 'assassin': 9, "'laughter'": 1, 'garcia': 5, 'fuse': 1, 'culinary': 1, 'purportedly': 1, 'precious': 13, "mann's": 3, 'spots': 17, "dicaprio's": 3, "eszterhas'": 2, 're-experiencing': 1, 'crusading': 1, 'stunningly': 3, 'alcoholism': 3, "blanchett's": 1, 'ulimate': 1, 'confessions': 1, 'business-as-usual': 1, 'pains': 4, 'unreal': 4, 'foreseeable': 1, 'inspired': 27, 'broiled': 1, 'tactics': 5, 'dellaplane': 1, 'spelling': 2, 'satirized': 1, 'justifying': 1, 'lawyers': 3, 'half-human': 1, 'dead-zone': 1, 'oversight': 1, 'uganda': 1, '=====================': 1, 'fish-out-of-water': 4, 'reagan': 8, 'shuttle': 4, 'volcano': 7, 'ben': 31, 'misstep': 3, 'glumly': 1, 'expunged': 1, 'peppers': 2, 'leonardo': 9, 'racism': 6, "casino's": 1, 'east': 11, 'blob': 3, 'bar/whorehouse': 1, 'get-up': 2, 'argue': 13, 'roared': 1, 'slacker': 5, 'prizefighted': 1, 'unemployment': 1, 'begrudgingly': 1, "booze-drinkin'": 1, 'akiva': 5, 'creativity': 9, 'recedes': 1, 'shovel': 3, 'mane': 2, 'wondrous': 3, 'overdramaticized': 1, 'isacsson': 1, 'simple-minded': 1, 'tarantino-esque': 1, 'hsu': 1, 'characters': 422, 'condoms': 3, 'cowering': 2, '`mission': 1, 'breezy': 2, 'eliot': 1, 'hilarity': 3, 'faggots': 1, 'esteemed': 3, 'officials': 4, 'gorefests': 1, '1963': 3, 'ibn': 4, 'cuffs': 1, 'powers-that-be': 1, 'satire': 25, 'merchandising': 4, '59': 1, 'whitaker': 3, 'orangutan': 2, 'check': 39, 'reconstructed': 1, 'siouxsie': 1, 'stallone': 12, 'sharks': 5, 'cheesiest': 1, 'submerged': 2, 'rubbish': 3, 'transpires': 2, 'height': 4, 'dejection': 1, 'over-written': 1, 'snatched': 1, 'dismayed': 1, 'mtv': 15, 'pulman': 1, 'contact': 21, 'panes': 1, 'ability': 54, 'invaluable': 1, 'diseases': 1, 'shatters': 1, 'undermine': 2, '1600s': 1, 'big-guy': 1, 'hieroglyphics': 1, 'antony': 1, 'harmony': 2, 'sister-in-law': 1, 'jacques': 3, 'orchestra': 2, 'boundary': 1, 'toninho': 1, 'immolation': 1, 'riotously': 2, "oteri's": 1, "that'": 1, 'maddening': 3, 'incompetents': 2, 'lacks': 50, 'apologizing': 1, "orion's": 1, 'embers': 1, 'asleep': 18, 'buenos': 3, 'pregnancy/child': 1, 'contributed': 7, 'long-standing': 1, 'sheeva': 2, 'cringing': 2, 'cope': 5, 'studying-abroad': 1, 'brandon': 4, 'kaela': 1, "'ghetto'": 1, 'triumphant': 1, 'cliches': 41, 'non-stop': 8, 'sonya': 4, "screenwriters'": 1, 'weissmuller': 1, 'family-man': 1, 'hc': 1, 'naaaaah': 1, "jeweler's": 1, 'harriet': 1, 'plaster': 1, 'dispatches': 1, 'filthy': 2, 'dr': 64, 'unbearably': 4, 'fig': 1, 'authoress': 1, "`armageddon'": 1, 'schuemacher': 1, 'exception': 41, 'brainy': 3, 'wargames': 1, 'severance': 1, 'soulless': 5, 'strangling': 2, 'wrapped': 14, 'noxema': 1, 'titanic-fever': 2, 'fulci': 1, 'insistent': 1, 'irate': 1, 'willard': 1, 'oblige': 2, 'dancing': 21, 'shut': 20, 'continuation': 3, 'partnership': 1, 'hector': 3, 'lit': 6, 'depict': 5, 'stained-glass': 2, 'coherently': 1, 'hypnotic': 4, 'laughter': 29, 'menu': 3, 'trucksploitation': 1, 'distributors': 1, 'developed': 36, 'services': 4, 'tiring': 4, 'metaphysical': 2, 'neat-o': 1, "sherman's": 1, 'tentacles': 3, 'broadly': 2, 'foley': 6, 'vendetta': 2, 'high-class': 1, 'aztec': 1, 'francisco_': 1, 'ex-football': 2, 'corinthians': 1, 'seclusion': 1, 'naval': 2, 'jolt': 3, 'tangible': 1, "criminals'": 1, 'ditched': 3, 'determination': 4, 'falwell': 1, 'gimme': 1, 'double-spaced': 1, 'perusing': 1, 'smarts': 3, 'evers': 1, 'china': 13, 'jasper': 2, 'schtick': 3, 'luster': 1, 'redneck': 3, 'applaud': 3, 'gailard': 1, 'drive': 42, 'jamie': 13, '83-minute': 2, 'gooding': 6, 'risky': 7, 'tool': 2, 'library': 2, 'misspelling': 2, 'windsor': 1, 'action-crime': 1, "`battlefield'": 1, 'starved': 1, 'rhetoric-spouting': 1, 'necessity': 3, "darnell's": 1, 'unconscious': 5, 'guidelines': 2, "pixar's": 1, 'conciousness': 1, "university's": 2, 'all-too': 3, 'mentor-turned-evil': 1, 'repetitive': 25, 'landing': 7, "`accidentally'": 1, 'quick-draw': 1, 'franz': 1, 'eyesight': 1, 'spurred': 1, 'fetishized': 1, 'less-than-subtle': 1, 'medak': 3, 'dogstar': 1, 'russel': 2, 'commercial': 22, 'mississippi': 2, 'foreigners': 1, 'created': 55, 'holliday': 2, 'narrates': 1, 'ceremonial': 2, 'irritation': 1, 'fast-talking': 3, 'noises': 7, 'klump': 1, "alfred's": 2, 'barriers': 2, 'smaller': 8, 'penzance': 1, 'cloudy': 1, "bauchau's": 1, 'much-discussed': 1, 'orleans': 2, 'stump': 2, "farley's": 1, 'vota': 1, 'unauthorised': 1, 'utopia': 2, 'carl': 10, 'driver': 20, 'half-expecting': 1, 'moderate': 4, 'snake-eyes': 1, 'spurting': 1, 'linger': 3, 'partition': 1, 'amoeba': 1, 'owned': 5, 'cream': 10, 'thrift': 1, 'fluttered': 1, 'locker': 3, 'foresee': 2, 'high-strung': 1, 'agitated': 1, "pirate's": 1, 'resounding': 5, 'wainwright': 1, 'laurence': 4, 'drilled': 1, 'single-minded': 1, 'advertised': 10, 'daughters': 5, 'resolved': 7, 'counter-culture': 2, 'fictitious': 4, 'statement': 21, 'wonderful': 45, 'equals': 3, 'ethnicities': 1, 'intonation': 1, 'pseudo-epic': 1, 'mcteer': 1, 'horses': 8, 'deals': 28, 'abject': 1, 'guided': 1, 'enticing': 4, 'weeks': 21, 'feet': 32, 'awkwardly': 8, 'cinema--or': 1, 'hairdos': 2, "1995's": 4, 'indios': 1, 'aspect': 43, 'observed': 3, '_gag': 1, 'top-form': 1, 'realities': 5, 'flee': 6, 'outfield': 1, 'dominate': 4, 'skillfully': 3, 'moxxon': 1, 'miniseries': 4, 'innovative': 6, 'fluke': 5, 'bleeding': 3, 'warburton': 1, 'whala': 1, 'seat': 20, 'impossibly': 4, 'ruins': 9, 'licks': 3, 'chortle': 1, 'inclined': 5, 'bums': 2, 'unstoppable': 6, 'embarrassment': 14, 'plans': 35, 'practiced': 2, 'persuasively': 1, 'helpful': 4, 'ongoing': 7, 'dung': 3, 'pecker': 2, 'resourceful': 6, '$80': 1, 'likened': 2, 'leg': 9, 'animated': 31, 'addicts': 1, 'greener': 1, 'identical': 9, 'colon': 1, 'amistad': 2, 'insincerity': 2, 'patriotism': 1, 'reached': 12, 'verbose': 1, 'expedition': 6, 'joann': 1, 'experienced': 14, "buddy's": 1, 'caused': 14, 'gooden': 1, 'journals': 1, 'berman': 2, 'brilliance': 5, 'section': 13, 'draft': 6, 'atoll': 1, 'ludicrous': 36, 'over-the-': 1, 'renegade': 9, 'psycho-sexual': 1, "mcculloch's": 1, 'dreamy': 2, 'jalla': 1, 'roused': 1, 'adolescents': 2, 'lows': 2, 'team': 85, 'viciously': 1, 'dose': 12, 'costars': 1, 'jackson': 27, 'adjani': 1, 'peformances': 1, 'hush-hush': 1, 'problems': 109, "game's": 2, 'eva': 2, 'skill--thus': 1, 'manifests': 2, 'painkillers': 2, 'blot': 3, "review's": 1, 'mistakes': 12, 'soles': 1, 'sweden': 1, 'superlative': 2, 'futz': 1, "tatopoulos's": 2, 'embarrased': 1, '15-minutes': 1, 'hairpiece': 1, 'fugitive-like': 1, 'publicly': 3, 'inquires': 1, 'veiny': 3, 'mojo': 3, 'damsel': 2, 'ephron': 7, 'disco': 12, 'dominions': 1, 'unmemorable': 1, 'crowded': 1, 'worry': 16, 'kara': 2, 'remain': 19, 'dozing': 1, 'dusk': 2, 'cashing': 3, 'hedwig': 1, 'ghosts': 6, 'sentimentality': 8, 'raver': 1, 'character--but': 1, 'replacement': 8, 'marlboro': 1, 'hitchhikers': 1, 'diplomat': 1, 'kilpatric': 1, "tyson's": 1, 'accusing': 3, 'taco': 4, 'dislikes': 2, 'warmth': 8, 'apt': 3, 'rake': 4, 'thailand': 3, 'nationalist': 1, 'ditch': 1, 'troy': 2, 'kristy': 2, 'percent': 7, 'grunts': 3, 'talon': 1, 'malice': 2, 'starman': 1, "madonna's": 3, 'works': 105, 'seeking': 6, 'markets': 1, '36': 2, 'splash': 2, "fishes's": 1, 'canoe': 2, 'granddaughter': 1, 'princess': 19, 'secondary': 9, 'bag': 19, 'wonderfully': 8, 'wishes': 14, 'lindsey': 3, 'coure': 1, 'hoopla': 1, 'ashes': 3, 'cluttered': 3, 'state': 44, 'loaded': 13, 'motel': 8, 'atrociously': 1, 'over-indulgence': 1, 'lieutenant': 3, 'blatz': 1, 'fistfight': 3, 'geek': 9, 'hitmen--cisco': 1, 'enable': 1, 'agreed': 5, 'pilot': 21, 'asides': 4, 'donning': 2, 'decide': 49, 'elise': 2, 'stew': 3, 'dauphin': 1, 'victim--her': 1, 'depaul': 2, "henson's": 2, 'mis-guided': 1, 'starving': 1, 'byron': 1, 'pours': 3, '10=a': 1, 'charlize': 10, "francis's": 1, 'feign': 1, 'best-forgotten': 1, 'rueland': 1, 'discordant': 1, 'samurai': 2, 'cash--so': 1, 'fine-looking': 1, 'bio-ports': 1, 'seltzer': 1, 'crocodiles': 2, 'approached': 7, 'telepathically': 1, 'denial': 1, 'demonic--power': 1, 'faltering': 1, 'streisand': 1, 'fritter': 1, 'violator': 1, 'herd': 1, 'goldberg': 8, 'humping': 1, 'regales': 1, 'attorney': 5, 'bleed': 2, 'pelted': 1, 'stranglers': 1, 'intersection': 2, 'tending': 1, 'thunk': 1, 'secrets': 7, 'sentient': 2, 'steel': 7, 'impulses': 1, 'icons': 3, 'fable': 4, 'protective': 2, 'frightens': 1, 'enormously': 2, 'puppetry': 2, "gets'": 1, 'educating': 1, 'remorse--the': 1, 'leachman': 2, 'disable': 1, 'caliber--and': 1, 'crest': 2, 'money-wise': 1, 'flounders': 1, 'nunez': 1, 'movies': 329, 'funeral': 15, 'wisecrack': 3, 'hansel': 1, 'non-human': 3, 'militia': 2, 'catchy': 5, 'ing': 4, 'lollipop': 1, 'surf': 2, 'spot': 26, 'prowls': 1, 'ari': 3, 'couldn': 2, 'grunt-like': 1, 'paper': 21, 'joey': 5, 'punishing': 1, 'diluted': 1, 'jollies': 1, 'urn': 1, 'sanders': 2, 'co-eds': 1, 'london': 14, 'mulan': 1, '_what': 1, 'consternation': 1, 'alcohol': 12, 'idiot': 16, 'downward': 3, 'reconciliation': 7, 'thomspons': 1, 'regenerated': 1, 'decline': 4, 'hardware': 1, 'malcontents': 1, 'regret': 4, 'reenlists': 1, 'seville': 1, 'rican': 2, 'all-out': 5, 'animes': 1, 'hang': 18, 'shards': 1, 'critically-savaged': 1, 'totally': 65, 'drilling': 3, 'murder': 71, 'zimmely': 1, 'devised': 1, 'laughs-to-jokes': 1, 'mede': 1, 'edtv': 1, 'hyams': 7, 'sunburned': 1, 'warfare': 5, 'one-eyed': 2, '-helmer': 1, 'execrable': 4, 'boo-fest': 1, 'psychologically': 4, 'chronic': 4, 'dethroning': 1, 'talentless': 2, "mancuso's": 1, 'european': 8, 'beast-people': 1, 'cronos': 3, "mandel's": 1, 'bravery': 2, 'daisy': 5, 'gym': 3, 'possession': 5, 'failure': 45, 'insipidly': 1, 'hooded': 2, 'de-clawed': 1, 'befriend': 1, 'collected': 4, 'high-speed': 1, "powers'": 3, 'eggert': 1, 'interiors': 4, 'watering': 1, 'pseudo-intellectual': 2, 'bullets': 15, 'guam': 1, 'over-decorated': 2, "arquette's": 2, 'seek': 13, 'appreciable': 2, 'wedged': 1, 'brady': 4, 'links': 2, 'wise-cracks': 1, 'stereotypical': 25, 'snore': 3, 'charmingly': 1, 'cates': 2, 'stuffed': 10, 'motions': 18, 'august': 12, "maurice's": 1, "snail's": 1, 'drags': 21, 'cop-out': 1, 'stimulating': 2, 'unturned': 1, 'powder': 6, 'dusted': 1, 'repertoire': 3, "rd's": 2, 'negates': 1, 'cackles': 1, 'bungles': 1, 'novak': 1, 'pups': 1, 'jawa': 1, 'discipline': 1, "dylan's": 2, 'arch': 5, 'wiseass-is': 2, 'substantially': 1, 'atreides': 1, 'ringleader': 1, 'springfield': 1, 'training': 14, 'drifting': 6, 'boyish': 2, 'older': 38, 'single-mindedly': 1, 'exams': 1, 'splattered': 4, 'mikey': 1, 'anderson': 21, 'blaster': 2, 'recommended': 5, 'mind-numbingly': 3, 'rodent': 2, 'investment': 5, 'romanctic': 1, 'hot-and-bothered': 1, 'feather-hatted': 1, "mirren's": 1, 'admission': 11, 'demotic': 1, 'schneid': 1, 'real-looking': 1, "craven's": 3, "thompson's": 3, 'making-of': 1, 'ghetto': 6, 'good-idea-turned-bad': 1, 'babble': 2, "estella's": 1, 'attests': 1, 'sweat-inducing': 1, 'homogeneity': 1, 'herald': 1, 'jake': 12, 'sam': 29, 'gunfights': 1, 'facial': 13, "everything's": 1, 'vatican': 4, 'thwart': 4, '==========================': 1, 'glowers': 1, 'cleansing': 2, 'respectable': 11, 'hero': 84, 'interminably': 2, "song's": 1, 'elation': 1, 'mayoral': 1, 'monty': 6, 'jojo': 1, 'overcome': 21, 'contemplating': 8, 'painted': 6, 'tongue-in-cheek': 6, 'kathyrn': 1, 'cockroaches': 1, 'soup': 5, 'tamara': 1, 'violates': 1, '1950s': 2, 'full-of-tension': 1, 'enliven': 3, 'acumen': 1, 'brokerage': 1, 'hoggett': 1, 'hatchett': 1, 'lockers': 1, 'sloppiness': 2, 'toenail': 1, 'trashes': 1, 'bullcrap': 1, 'john': 203, 'double-decker': 1, 'trademark': 18, 'morosely': 1, 'strangle': 3, 'switching': 3, 'fantasy': 25, 'blues': 10, 'trumpet': 2, 'chapter': 8, 'unredeemable': 2, 'sends': 27, "predecessors'": 1, 'boss': 47, 'glitz': 3, 'adequately': 7, 'reverses': 1, 'nations': 1, 'pal': 7, "wainwright's": 1, 'replay': 1, '4th': 5, "joel's": 1, 'pressures': 3, 'der': 2, 'mey': 1, 'mo': 2, 'amish': 3, 'blur': 1, 'self-admittedly': 1, 'hasidics': 1, 'ingenue': 5, 'runner_': 1, 'scathing': 3, 'mystical': 7, 'solely': 21, 'prom': 6, 'rendered': 7, "bluebirds'": 1, 'ward': 9, "disney's": 13, 'conclude': 4, '51st': 1, '_all_': 1, 'ring': 14, 'procedures': 2, 'elude': 2, 'responsibility': 4, 'certified': 4, 'rarest': 1, 'windup': 1, 'nitroglycerine': 1, 'yellowstone': 2, 'tagged': 1, 'europe': 4, 're-invent': 1, 'plot-wise': 1, 'undetectable': 1, 'unchecked': 1, 'grossed': 3, 'custodians': 1, 'cute--try': 1, 'deliberate': 2, 'santas': 1, 'one-by-one': 4, 'terminally': 7, 'truths': 2, 'stale': 21, 'bluescreening': 1, 'downed': 1, 'strangeness': 3, 'fernando': 1, 'cabs': 3, "lover's": 2, 'brother-brother': 1, 'dimly': 1, 'batmans': 1, "bugs'": 1, 'sordid': 8, 'reused': 1, 'mobster': 5, 'casino': 9, 'stirs': 1, '`bonnie': 1, 'letter': 11, 'raison': 1, 'steakley': 1, 'bull-heaed': 1, 'commands': 5, 'too-over-the-top': 1, 'griffin': 6, 'unobserved': 1, 'supplier': 1, 'semi-intriguing': 1, 'random': 30, 'courtship': 4, 'tossed': 14, 'zardoz': 1, 'reshoot': 1, 'maynard': 1, 'comedians': 3, 'amateurism': 1, 'raucously': 1, 'ethics': 5, '24th': 1, 'huckleberry': 1, 'glen': 6, 'viewings': 4, "trooper's": 1, "kelly's": 4, 'conductor': 2, 'minimize': 1, 'whit': 4, 'yearn': 4, 'well-supported': 1, 'menial': 1, 'ortega': 1, 'movie-goers': 7, 'spend': 64, "caprenter's": 1, 'shields': 3, 'fetale': 1, 'always-': 1, 'i\x12m': 1, 'achieve': 15, 'blank-from-hell': 1, "hunt's": 2, 'roxbury': 2, '_54_': 1, 'cowers': 1, 'dome': 1, 'reek': 1, 'blue-lighted': 1, 'not-very-good': 1, 'indulged': 2, 'effervescent': 1, 'mr': 74, 'newman': 6, 'military': 31, 'garners': 1, 'lance': 6, 'lundgren': 3, 'dottie': 1, 'thrusts': 1, 'foster': 6, 'stereotype': 11, "2001's": 1, 'greenlight': 2, 'knifed': 1, 'objects': 10, 'buffy': 8, "'macho": 1, 'down-to-earth': 1, 'half-speed': 1, 'trusts': 2, 'collides': 2, 'invention': 4, 'jascha': 1, 'signal': 3, 'merrier': 2, 'martial-arts': 1, 'incongrous': 1, 'gun-and-plane': 1, 'swallows': 3, 'burglar': 3, 'cheekily': 1, 'rediculous': 1, '2017': 2, 'offensively': 2, 'dushku': 2, 'impersonal': 1, 'underway': 2, 'convulsing': 1, 'drector': 1, 'corporate': 6, "brothers'": 5, 'heartfelt': 4, 'likewise': 5, 'angry-but-inept': 1, 'figeroa': 1, 'twist-ties': 1, 'cliche-spewing': 1, 'array': 8, 'flavorful': 1, 'carin': 2, 'outsiders': 1, 'davis': 12, 'too-forgiving': 2, 'brooke': 5, 'doa': 2, 'obscured': 3, 'dispense': 2, 'maudit': 1, 'sunk': 9, 'good-heartedfrom': 1, 'wasp': 2, 'outacts': 1, 'reinvigorate': 1, 'nicholson': 7, 'willem': 4, 'figure': 85, 'difficulty': 7, '128': 1, 'philosophy': 7, 'graces': 4, 'hollywoodization': 1, 'circulation': 1, 'non-story': 1, 'brian': 27, 'super-policeman': 1, 'reputedly': 1, 'trivialised': 1, 'cruddy': 1, 'collate': 1, '2-2': 1, 'ingrained': 1, 'mommie': 1, 'instances': 13, 'copycat': 2, 'nozaki': 1, 'ex-cia': 1, 'inventory': 1, '1980s': 11, '_escape': 1, 'pans': 7, 'gregor': 5, 'sitcoms': 1, 'miniature': 3, 'lawrence/fat': 1, 'mintues': 1, 'display': 26, 'redoubtable': 1, 'habits': 5, 'co-': 1, 'favors': 7, 'leave': 76, 'unfathomably': 1, 'sequel-crazy': 1, 'cash': 39, 'often-times': 1, 'dilemma': 9, 'questions': 56, 'profoundly': 7, 'barbed': 2, 'storylines': 3, 'infinitely': 7, "'snuff'": 1, 'reminds': 17, 'eliminate': 8, 'soylent': 1, 'sendup': 1, 'dispels': 1, 'disbeliefs': 1, 'theology': 1, 'imbecile': 1, "pill-poppin'": 1, 'overacts': 5, 'chia': 1, 'perspectives': 2, 'sweats': 1, 'slaps': 5, 'self-parodizing': 1, 'licking': 3, 'inning': 1, 'stooges': 3, 'teleconferencing': 1, 'sentinels': 1, 'nosferatu': 1, 'roderick': 1, 'jacket': 8, 'mainstays': 1, '_reach_the_rock_': 1, 'unnecessarily--the': 1, 'apple-cheek': 1, 'nineties': 5, 'skis': 1, 'rests': 5, 'introduce': 10, 'intestines': 1, 'throwaway': 10, 'bibb': 1, "titanic's": 1, 'grabbag': 1, 'gutsy': 1, 'tunneled': 1, 'limburgher': 1, 'hospitalised': 1, 'sarge-type': 1, 'taps': 1, 'japanese': 21, 'tower': 7, 'vegetable': 1, 'a-list': 4, 'judgment': 5, 'cydonia': 1, 'monolith': 1, 'consuming': 1, 'infer': 1, 'base': 16, "'when": 1, 'foreshadow': 1, 'backstage': 1, 'beliefs': 2, 'cut-and-paste': 1, 'endure': 10, 'veterinarian': 1, 'howington': 1, 'slew': 9, 'aswell': 4, 'pander': 2, 'near-blizzard': 1, 'metaphors': 2, 'hendra': 1, 'hyperactive': 6, 'spectaculars': 1, "joan's": 2, 'clause': 3, 'mandoki': 1, 'kindest': 1, 'nordern': 1, 'dollhouse': 1, 'adjective': 4, '_must_': 1, 'obsessive/compulsive': 1, 'rogers': 6, 'siegfried': 1, 'cements': 1, "movies'": 3, 'gizmo': 1, 'trivializes': 1, 'ponder': 5, 'recreate': 4, 'awakening': 2, 'kissinger': 1, 'preceded': 4, 'amusing': 57, "austen's": 2, 'explaining': 16, 'computer': 49, 'surley': 1, 'iron-clad': 1, 'landings': 1, 'enchanting': 3, 'fowler': 2, 'blue-screening': 1, 'boatload': 3, 'surpassed': 3, 'stupefied': 1, "rock'": 1, 'unveil': 2, 'harring': 1, 'detected': 1, 'tuned': 4, 'rays': 2, 'es': 1, 'proctor': 1, 'diabolique': 2, 'cooling': 1, 'chris+': 1, 'aykroyd': 5, 'hits': 26, 'mushroom-cloud': 1, 'messenger': 6, 'stylishly': 1, 'lemme': 1, 'self-respect': 2, 'nail-biters': 1, 'harebrained': 1, 'jungian': 1, 'hundreds': 11, '16mm': 1, 'sadsacks': 1, "birds'": 1, 'attributes': 10, 'objectified': 1, 'butchery': 1, 'wises': 1, 'stroke': 3, 'tasting': 1, 'mae': 1, 'sherry': 1, 'gawking': 1, 'virginity': 1, 'hills': 13, 'describing': 9, 'regeneration': 1, 'jargon': 1, 'reese': 6, 'cache': 2, 'mercilessly': 4, 'lyrics': 10, 'horticulturist': 1, 'sleepy-eyed': 1, 'intuition': 2, 'sillas': 1, 'national': 21, 'welcomes': 1, 'budge': 1, 'diversions': 3, 'fearless': 2, "wyatt's": 1, "society's": 1, 'hitchcock': 12, 'wanker': 1, 'goop': 2, 'squad': 10, 'carte': 1, 'disgruntled': 4, 'rained': 2, 'lola': 1, '43': 2, 'gently': 2, 'article': 6, 'affairs': 9, 'neeson': 7, 'razor-sharp': 1, 'pre-med': 2, 'irs': 3, 'grandpa': 3, 'travelled': 1, 'citizen': 4, 'respite': 1, 'announcement': 5, "moreau's": 1, 'poignancy': 3, 'hard-unearned': 1, 'fifties': 4, 'grammar': 2, 'luftwaffe': 2, 'overestimated': 1, '1962': 2, 'kristel': 1, 'ordeal': 6, 'profile': 4, 'seeks': 7, 'feature--is': 1, "disappoint'": 1, 'u-turn': 1, 'fanfare': 2, 'scariness': 1, "job-it's": 1, 'sympathetic': 27, 'rourke/jean-claude': 1, 'edged': 1, 'resting': 1, 'pin': 2, 'fantastically': 4, 'supervision': 3, 'shielding': 1, 'carry': 38, 'infamously': 1, 'zealously': 1, 'loud-noise-jump-scare': 1, 'guaranteed': 10, 'engelbrecht': 1, 'spoon': 2, "id's": 1, 'bikini-ready': 1, 'fondled': 1, 'ominous': 8, 'ply': 1, 'pais': 1, 'non-discriminating': 1, 'deemed': 4, 'chess': 1, '_babe_': 2, 'kindly': 5, 'two-hour': 10, 'motorbike': 2, 'soaking': 2, 'coincidentally': 8, 'lots': 60, 'fuel': 8, 'clientele': 2, 'unsuccessfully': 4, 'catwoman': 1, 'guilifoyle': 1, "gangster's": 1, 'five-minute': 3, 'toolbox': 1, 'candlewax': 1, 'single-mindedness': 1, 'villagers': 3, 'reappears': 4, 'tic': 1, 'side-kick': 2, 'hairs': 1, 'levinson': 4, 'bloodthirstiness': 1, 'components': 2, 'beauty': 29, 'priced': 1, 'heartstrings': 6, 'surfer': 3, 'increasingly': 17, 'muresan': 3, 'weirdos': 1, 'remake': 41, "longo's": 1, 'assembles': 2, 'hugely': 6, 'easily-corrupted': 1, 'see-through': 3, "fiance's": 1, 'not--graphics': 1, 'hearts': 6, 'bogged': 4, 'overplayed': 6, 'grimly': 2, '10-year-olds': 1, 'divest': 1, 'dampens': 1, 'old-fashioned': 4, 'dodger': 1, 'hijinx': 3, 'gloves': 3, 'is--and': 1, "bill's": 1, 'habanera': 1, 'invoked': 1, 'time-frame': 1, 'nap': 2, 'invading': 1, 'embassy': 3, 'well-publicized': 1, 'laughs': 82, 'godly': 1, "palmer's": 2, 'sangfroid': 1, 'squinty-eyed': 1, 'roxburgh': 1, 'literacy': 1, 'exorcism': 1, 'glance': 7, 'kurosawa': 1, 'powerless': 4, 'pyjamas': 1, 'conjures': 3, 'pseudonyms': 1, 'caricatured': 1, 'amtrak': 1, 'wanting': 18, 'lingering': 5, 'reserve': 1, 'lions': 5, 'romanticising': 1, 'eighth-assedness': 1, 'backs': 4, 'television': 66, 'broad': 12, 'terrorize': 4, 'rejection': 1, 'on-key': 1, 'staggering': 3, 'snook': 1, 'potrayed': 1, 'over-protective': 1, 'devices': 23, 'sea': 19, 'groaner': 2, 'clairvoyant': 1, 'stoddard': 2, 'word--of': 1, 'amarillo': 1, 'revise': 1, 'relief--to': 1, 'hypothalamii': 1, 'announcing': 2, 'click': 4, 'clash': 5, 'immense': 4, 'greedy': 6, 'burt': 6, 'wildly': 17, 'raising': 7, '64': 2, 'yuppie': 7, 'forlani': 5, 'arduous': 2, 'trumpeter': 1, 'barks': 2, 'again-except': 1, 'onscreen': 19, 'wanna-be': 2, 'handlebar': 1, 'weather': 6, 'basketball': 16, 'zombie-like': 2, "user's": 2, 'misappropriating': 1, 'manson': 6, 'hopper': 12, 'courtroom': 15, 'smithees': 1, 'callously': 1, 'gooey': 4, "bloodsuckers'": 1, 'document': 4, 'arli$$': 1, 'detonated': 1, 'nipples': 3, 'corn': 4, '78': 2, 'garbled': 2, 'tank': 7, 'sentimental': 12, 'penny-dreadful': 1, 'experiment': 19, 'plump': 1, 'anime': 3, 'sync': 3, '45': 4, 'negligees': 1, 'twenties': 3, 'realizes': 17, 'substandard': 3, 'incessant': 4, 'overwhelm': 1, 'nikita': 4, 'open': 57, 'release': 62, "rydain's": 1, 'dc': 1, "dads'": 1, 'fond': 7, 'programming': 4, 'pellegrino': 1, 'rivalling': 1, 'trackers': 2, "magoo'": 1, 'graynamore': 1, '&nbsp': 1, "audience's": 15, 'yuen': 2, 'altered': 9, 'estranging': 1, 'glam': 1, 'assigning': 2, 'foul': 8, 'guccione': 1, 'ireland': 5, 'brando': 4, 'frustrating': 14, 'dramatizes': 1, 'defining': 7, 'format': 6, 'hard-luck': 1, 'accepted': 6, 'tutors': 1, 'transit': 2, 'giancarlo': 2, 'kits': 2, 'armani': 1, 'procreation': 1, 'astrophysicist': 3, "hooper's": 1, 'topics': 3, 'ticketed': 1, 'pseudo-sensitive': 1, 'hoot': 2, 'shorty': 3, 'dross': 2, 'electing': 1, 'demonstrated': 8, 'through-the-sight': 1, 'farley': 7, 'loving': 22, 'sometimes-sick': 1, 'interchangable': 1, 'churning': 3, 'jean-luke': 1, 'portraying': 9, 'roaring': 2, 'kitty': 3, 'decrease': 2, 'exact': 30, 'y2k': 3, 'weller': 2, 'unprepared': 2, 'favourite': 6, 'counselors': 2, 'surrealist': 1, 'opinion': 36, 'oscar-winning': 2, 'inborn': 1, 'mob': 19, 'vamires': 1, 'ooooh': 3, 'souring': 1, 'hook': 15, 'wads': 1, 'sartain': 1, 'seidler': 1, 'centuri': 1, 'successor': 1, 'shaggy': 2, 'outfitted': 1, 'title--it': 1, 'lyrical': 1, 'slinky': 3, "leguizamo's": 2, 'perrier': 1, 'cheese': 14, 'film--the': 1, 'squirm': 3, 'equivalents': 1, "sinclair's": 1, 'tasteless': 9, 'mumbling': 5, 'plotted': 3, 'storyboard-to-scene': 1, 'nearing': 2, 'revived': 2, 'pitting': 1, 'edging': 1, 'contained': 14, 'exhilarated': 1, 'she-ra': 1, 'assertive': 2, 'stemming': 1, 'evolution': 6, 'trachtenberg': 2, 'puff': 1, 'perfomances': 1, 'desolation': 2, 'teeters': 1, 'nello': 1, 'riches': 1, 'spinosauraus': 1, 'derives': 1, 're-animator': 1, 'ogle': 1, 'stature': 5, 'amuck': 1, "actor's": 6, 'california': 14, 'effects-laden': 2, 'hitman': 8, 'counterpoint': 3, 'liberating': 2, 'untouched': 2, 'bluegrass': 1, 'rental': 6, '20thcentury': 1, 'hallmarks': 1, 'orphan': 5, 'pi': 1, 'audience-unfriendly': 1, "'the": 8, 'films=92': 1, 'dreyfus': 1, 'sailboat': 1, 'objectivity': 2, 'compartment': 1, 'hardheaded': 1, 'scorned': 1, 'household': 7, 'stragely': 1, 'hypocrite': 1, 'week': 24, 'adventure': 28, 'mona': 4, 'showering': 2, 'tatopoulos': 4, 'cyr': 1, "lot's": 1, 'dicillo': 2, 'flash': 19, 'lithium': 2, 'invoke': 2, 'raging': 7, 'evelyne': 1, 'guard/chet': 1, "murray's": 1, 'picks': 22, 'bidets': 1, "landon's": 1, 'rule-the-world': 1, 'terrible': 83, 'thompsons': 1, 'wyoming': 4, 'removing': 4, "mcfarlane's": 1, 'switzerland': 1, 'attic': 2, 'terribly-plotted': 1, 'term': 19, 'aspired': 1, 'deteriorating': 2, 'guidance': 7, 'schlocky': 2, 'southwest': 3, 'tender': 3, 'spoiled': 17, 'best-seller': 3, 'ineptly': 3, 'courage': 5, 'size': 19, 'thread': 10, "ray'": 1, 'thames': 1, 'ooh': 4, '45-minute': 1, 'lara': 6, 'jailer': 1, 'circa-1983': 1, 'maximilian': 1, 'influence': 11, 'quicksand': 1, 'bond': 36, '47': 2, 'valiantly': 1, 'interlude': 1, 'properly': 14, "lomax's": 1, 'celebrate': 3, 'mentorship': 1, '114': 1, 'overhearing': 3, "right'": 1, 'hangar': 1, 'pursuers': 2, 'several-scene': 1, 'fruition': 1, 'tacky': 6, 'fills': 6, 'levers': 1, 'fondness': 1, 'loot': 3, 'apprehended': 1, 'service': 19, 'pick-me-up': 1, 'toppled': 1, 'signy': 1, 'data': 9, 'jean-marc': 1, 'radmar': 1, 'reformers': 1, "rosenberg's": 1, 'olin': 2, 'recommend': 42, 'beeper': 1, 'castle': 4, 'originated': 1, "mi2's": 1, 'unearthed': 2, 'hurts': 4, 'teaching': 11, 'dynamics': 4, 'reassessed': 1, 'aptly': 9, 'plantation': 1, 'platter': 1, 'anytime': 1, 'rice': 3, 'lucie': 1, 'puddy': 1, 'factory': 8, 'wierd': 1, 'stupendously': 1, 'intercuts': 1, 'phenomenally': 3, 'slickers': 2, '1933': 1, "up-i've": 1, 'consulting': 1, 'fray': 1, 'immuno-deficient': 1, 'frothing': 1, 'marries': 4, 'novelist': 2, 'jarringly': 1, 'current-films': 1, 'jogging': 3, 'class': 36, '-with-a-passion': 1, 'abort': 1, "1950s'": 1, 'one-on-one': 1, 'mind-boggeling': 1, "_dragon_'s": 1, 'zagged': 1, 'adheres': 2, 'quatermass': 1, 'symbolise': 1, 'i-wanna-please-dada': 1, 'deaf': 4, 'poll': 1, 'successfully': 25, 'soaked': 3, "00'": 1, 'kay': 3, 'cannibals': 1, 'urinating': 1, 'savour': 1, 'nary': 6, 'little-to-no': 1, 'talkfest': 1, '90210': 5, 'momsen': 1, 'ala': 4, 'maturity': 5, 'sorcery': 3, 'rebs': 1, 'jar': 5, 'cruises': 1, 'mechanic': 5, 'pursue': 5, 'bismol': 1, 'cyberdog': 1, 'titillation': 1, 'ruler': 3, 'guadalcanal': 1, 'beat-up': 1, 'pretensions': 3, 'smuggler': 1, 'needle': 2, 'spoliers': 1, 'speeding': 6, 'exaggerations': 1, 'multi-multi-plex': 1, 'gangsters': 6, 'patinkin': 1, 'oddball': 2, 'off-camera': 1, 'canaan': 1, 'sails': 1, 'rehabilitates': 1, 'toaster': 2, 'edel': 1, 'caviar': 1, 'committing': 5, 'patch': 14, 'jabs': 4, 'abuzz': 1, 'promoting': 3, 'sounds': 64, "'home": 1, 'barbet': 1, 'snaps': 3, 'gugino': 2, 'refrain': 3, 'gratuitous': 23, 'generate': 17, 'lewinsky': 1, 'co-penned': 1, 'sparkling': 1, 'achoo': 1, 'crudely': 1, 'string': 11, 'half-assedly': 1, 'beginnings': 2, 'likeable': 12, 'stumbled': 5, 'thigh-high': 1, 'wilson': 21, 'dead-end': 3, ',': 898, '1942/1986': 1, 'implacable': 1, 'spokesperson': 1, 'willie': 4, 'figurine': 1, "capra's": 1, 'leto': 6, 'crop': 4, 'dicks': 3, 'saviors': 2, 'arrogance': 2, 'splendour': 1, 'integration': 1, 'noon': 3, 'goodtimes': 1, 'insulted': 7, 'impotent': 2, 'raucous': 3, 'tear': 10, 'mafioso': 1, 'asian': 7, 'funny--it': 1, 'earp': 3, 'handicapped': 2, 'nicolas': 12, 'maiden': 5, "school's": 5, 'scruples': 2, 'lesbians': 2, 'recurring': 10, "'i": 1, 'occasions': 9, 'enslaved': 3, '1994': 10, 'disection': 1, 'chuckled': 5, 'tomfoolery': 2, 'taxes': 1, 'bursted': 1, 'occasional': 31, 'magazine': 18, 'stuffy': 2, 'roaming': 4, 'coddle': 1, 'abomination': 2, 'softening': 1, 'minded': 3, 'polarity': 1, 'defiant': 1, 'egotistical': 1, "newspaper's": 1, 'plusse': 1, "farrellys'": 2, 'chomps': 1, "bard's": 1, "later'": 1, 'conversational': 1, 'imitators': 1, 'bunch': 89, 'pound': 4, 'strange': 56, 'hosts': 2, 'paramour': 1, 'flipped': 3, 'redeem': 4, 'mothers': 5, 'sundae': 1, 'pond': 3, 'hyperviolent': 1, "cole's": 1, 'astronomer': 1, 'whale': 2, 'stammering': 2, 'geena': 1, 'frivolous': 3, "'end": 1, 'gibbs': 1, 'psychologist': 14, 'alumnus': 2, 'spring': 9, 'diana': 2, 'blares': 2, 'fella': 1, 'forks': 1, 'panicked': 1, 'genital': 1, 'stupidness': 1, 'thing__not': 1, 'sail': 3, 'superpower': 2, 'hate': 48, "salwen's": 1, 'ufo': 1, 'contentious': 1, 'overpraised': 1, 'crichton': 6, 'start': 131, 'chill': 5, "2's": 2, 'deacon': 2, 'pause': 2, 'citing': 2, 'giggly': 1, 'draco': 1, 'resurrects': 1, "corman's": 1, 'caulfield': 2, 'desiring': 1, 'assailant': 1, 'snowy': 1, 'churchill': 1, 'willingly': 3, 'couplehood': 1, 'wreck': 12, 'enraged': 1, 'heartbreakers': 1, 'mangle': 4, 'trousers': 1, 'insouciance': 2, 'gun-toting': 1, 'drama/comedy': 1, 'grier-worshiping': 1, 'cops': 34, 'forger': 1, 'pg-rated': 1, 'eccleston': 3, 'factored': 1, 'caffeine': 2, 'smashing': 8, "d'": 1, 'earlier': 51, 'vita': 1, 'religiously-oriented': 1, 'enjoyably': 3, 'tersely': 1, 'viggo': 4, 'great-looking': 1, 'casanova': 2, 'chlorine': 1, 'spurns': 1, 'uniform': 3, 'perfecting': 1, "would've": 7, 'memorable': 27, '88': 1, 'scientists\x12': 1, 'gratuitously': 1, "apes'": 2, 'uno': 1, 'sprinkling': 1, 'revelatory': 1, 'self-defeating': 1, '500': 3, 'moved': 25, 'procreate': 1, 'blockbuster': 22, 'pinch': 1, 'stuart': 10, 'non-kosher': 1, 'hardass': 2, "sarandon's": 1, 'cohn': 1, 'chi-chi': 1, 'emulating': 1, 'shark': 7, 'guy': 190, 'quebec': 1, "deniro's": 2, 'it--she': 1, 'karan': 1, 's-t-r-e-t-c-h': 1, 'successes': 7, 'trekkies': 4, 'catchphrases': 1, 'ill-defined': 1, 'tribute': 3, 'whichever': 3, 'plops': 1, 'tierney': 1, 'b&w--everything': 1, 'author': 20, 'overthrow': 5, 'potential': 81, 'proliferation': 1, 'traumatic': 1, 'unpopular': 2, 'multiple': 16, 'top-line': 1, 'lobbying': 1, 'giveaway': 3, 'stinks': 14, 'maintain': 8, 'stand-by': 1, 'bests': 1, "there'll": 1, 'giants': 5, 'yun-fat': 4, 'textual': 1, 'strychnine-laced': 1, 'maryam': 1, 'wreaks': 1, 'hashbrown': 1, 'mind-numbing': 7, 'furred': 1, 'slipping': 4, 'gregorian': 1, 'maine': 2, 'heresy': 2, 'coordinator': 2, 'kellerman': 1, 'contextual': 1, 'transpiring': 1, 'fistfights': 3, 'liberal': 2, 'neill': 5, 'disappointing': 47, "hyam's": 2, "doctor's": 4, 'husband': 52, 'ideally': 2, 'desert': 15, 'broderick': 12, "ripley's": 1, 'extraterrestrial': 3, 'claire': 15, 'seasonal': 2, 'trods': 1, 'sub-plots': 5, 'timing': 14, 'injures': 1, "'comedy'": 2, 'workman-like': 1, 'early': 119, 'defect': 1, 'dine': 1, 'bartkiwiak': 1, 'sad': 49, '93': 3, 'edition': 3, 'calm': 10, 'incongruously': 1, 'stand-in': 1, 'employees': 7, 'assurance': 1, 'fouls': 1, 'behalf': 1, "yamamoto's": 1, "man-animals'": 1, 'gerard': 2, 'laurentiis': 1, '`like': 1, 'spirals': 3, 'befouled': 1, "quaid's": 3, 'forman': 1, 'dreamworks': 2, 'conferences': 1, 'illuminate': 2, 'comedown': 1, 'abduction': 1, 'skateboarding': 2, 'youngest': 7, 'motorcycles': 5, 'hoffmann': 1, 'curmudgeons': 1, 'bacon': 10, 'determines': 1, "o'reilly": 2, 'batch': 3, "who've": 3, 'pows': 1, 'canned': 3, 'undistinguished': 3, "mystery-girl-who's-no-real-mystery": 1, 'subtitled': 2, 'thornton': 6, 'twinge': 1, 'reappearing': 2, "filmmaker's": 2, 'luck': 35, 'cheery': 1, 'vipers': 1, 'temptation': 6, 'corrupter': 2, 'characterizations': 9, 'salvaged': 1, "beresford's": 1, 'self-reflection': 1, 'salivate': 1, 'backseat': 2, 'boarding': 3, 'gumption': 2, 'trivial': 4, 'coupon': 1, 'hail': 2, 'soeur': 1, 'stepmother': 1, 'never-ceasing': 1, 'arcade': 1, 'carted': 1, 'lowell': 2, 'in-laws': 1, 'endures': 1, 'scrolled': 1, 'stephie': 2, 'people': 360, 'speculated': 2, 'screaming': 29, 'mcgowan': 12, 'business-like': 1, "'good'": 1, 'ablaze': 2, 'blames': 3, 'pretzel': 1, 'culture-clash': 1, 'sluizer': 1, '`alex': 1, 'sue': 3, 'defenders/video': 1, "travolta's": 8, 'lie': 15, 'figueroa': 1, 'inanely': 2, 'variety': 14, 'madcap': 1, 'baddies': 5, 'reissues': 1, 'hounded': 1, 'principal': 13, 'pies': 1, 'thor': 1, 'akira': 1, 'get-together': 1, 'rover': 2, 'georgina': 2, 'gramophone': 1, 'death-defying': 2, "rubell's": 2, 'aren': 1, 'subplots': 28, 'xxv': 1, 'crotchety20': 1, 'delia': 3, 'others--mostly': 1, 'excruciatingly': 11, "wells'": 2, 'rainslickered': 1, 'half-thought-through': 1, 'explorers': 1, 'semi-experienced': 1, 'unmagical': 1, 'senseless': 5, 'digresses': 1, 'existential': 1, 'product-placement': 1, 'aluminium': 1, "gunton's": 1, 'ethnically': 1, 'tamuera': 1, 'inhalants': 1, 'girls': 52, 'wealth': 7, '3654': 1, 'females': 4, 'caring': 16, 'denying': 4, 'ten-minute': 2, "ross's": 1, 'mate': 10, 'dilemmas': 2, 'mcgraw': 1, 'shalhoub': 3, 'leering': 3, 'costello': 1, 'issac': 2, 'gruesome': 11, 't-rex': 1, 'doe-eyed': 1, 'ham-on-wry': 1, 'impending': 9, 'wild': 56, 'mishandle': 2, '100-minute': 3, 'pos': 1, 'redolent': 1, 'fumbles': 1, 'barfing': 1, 'introvert-boy-falls-for-extrovert-girl': 1, 'utilize': 5, 'rangoon': 1, 'irwin': 3, 'professionalism': 6, 'auditorium': 1, 'gignac': 1, 'financiers': 1, 'pausing': 2, 'radiant': 4, 'ashe': 1, 'impulsive': 1, 'competitor': 1, 'overstays': 1, "buscemi's": 5, ';': 325, 'deus': 1, 'perky': 3, '2002': 1, 'bedside': 2, 'marshall': 8, "joe's": 4, 'partially': 9, 'sobieski': 4, 'outright': 2, 'alive': 40, 'predilection': 3, 'crabs': 2, 'squabbling': 1, 'almost-witty': 2, 'whitfeld': 1, 'overuses': 2, 'individual': 27, 'mini': 2, 'systematically': 3, 'gush': 2, 'explain': 53, "britt's": 1, "hytner's": 1, 'pursued': 3, 'one-celled': 1, 'blink': 5, 'excursions': 2, 'travelling': 4, 'gun-runner': 1, 'tramell': 1, 'locate': 6, 'broads': 1, 'disgust': 3, 'beleaguered': 1, 'arenas': 1, 'skarsg': 2, 'egos': 2, 'contaminated': 1, 'scurries': 1, 'ronkonkoma': 1, 'tolling': 1, 'haviland': 1, 'flashcards': 1, 'thrillers': 34, 'jere': 1, 'vulgarity': 2, '$500': 2, 'strolling': 1, 'introspective': 1, 'altman-esque': 1, "luke's": 1, 'passages': 2, 'heffron': 1, 'feebly': 1, 'danes': 5, 'pensively': 1, 'cogs': 2, 'misconduct': 2, 'ziegler': 1, 'darrell': 2, 'loads': 4, 'detour': 1, 'quietly': 5, 'blackmail': 3, 'harvard': 5, 'smirk--regardless': 1, '70ies': 1, 'trips': 6, "dietz's": 1, 'detonator': 1, 'quickly-made': 1, 'shannon': 14, 'lund': 1, 'reticent': 3, 'millenium': 4, 'trashed': 5, 'rubble': 2, 'morality': 8, 'encompasses': 1, 'groan': 8, 'depicting': 6, 'commentary': 12, 'tigher': 1, 'sixth-grade': 1, 'supervillain': 1, 'boy-meets-girl': 2, 'overlay': 1, 'borderline': 5, 'bothers': 4, 'french': 35, 'punchline': 6, 'lithuanian': 1, '1914': 2, 'preserve': 3, 'muffled': 1, 'non-event': 2, 'gladly': 1, "lucus'": 1, 'curse': 5, 'indulging': 4, 'multi-tiered': 2, 'teenagers': 29, 'disproves': 1, 'vernon': 2, 'earned': 16, 'refusal': 3, 'monahan': 1, 'proposes': 3, 'circulating': 1, 'marxist': 2, 'lectured': 1, 'customs': 2, "'all": 1, 'newton': 7, 'stirred': 3, 'morphed': 1, 'teck-oh': 1, 'exemplified': 1, 'crouched': 1, 'trophy': 4, 'fetishistic': 1, 'matthew': 41, 'gear': 4, 'mom': 21, 'backstabbing': 1, "generations'": 1, 'punts': 1, 'billing': 11, 'lap--a': 1, "schwarzenegger's": 6, 'janitorial': 1, 'africans': 1, 'near-crazy': 1, 'musical': 35, 'fanatic': 1, 'healthy': 8, "'total": 1, 'dolled': 1, 'creating': 31, 'ideas': 69, 'villains---and': 1, 'calvin': 5, 'hall': 20, 'deschanel': 1, 'twist--people': 1, 'busted': 5, 'publications': 1, 'combines': 6, "_don't_": 1, 'vacuous': 7, 'con': 20, 'leery': 2, 'directs': 9, "boyfriend's": 1, 'sights': 8, 'striptease': 3, 'lulls': 1, 'singers': 1, 'bebe': 1, 'binding': 1, 'escalate': 3, 'daffy': 2, 'nameless': 1, 'filing': 3, 'unleashed': 5, 'hit': 102, 'deliver': 38, 'injurious': 1, 'decipher': 4, "partner's": 2, 'beneath': 14, 'nosedive': 1, 'sod': 1, 'rockets': 1, 'whirs': 1, 'likelihood': 3, 'lefessier': 1, 'baileygaites': 2, 'limbo': 5, "white's": 3, 'squandering': 2, "jimmie's": 1, 'round-the-world': 1, 'kassovitz': 1, 'miscue': 2, 'enamor': 1, 'rejuvenate': 1, 'inconspicuously': 1, 'blurs': 2, 'playoffs': 1, 'give': 228, 'menaces': 1, 'goddard': 2, 'quinlan': 4, 'glover': 6, 'laughable': 47, 'ghandi': 1, '99%': 2, 'medfield': 1, 'matched': 3, 'edmonds': 1, 'fountain': 5, 'guarded': 4, 'results': 41, '18-year-old': 2, 'proposterous': 1, 'crucifies': 1, 'morritz': 1, 'foul-tempered': 1, 'adagio': 1, '--so': 1, 'suspenseful': 21, 'entertainment': 65, 'snorting': 2, 'smarter': 10, 'snowball': 1, 'transpose': 1, 'soulful': 1, 'engineer': 4, 'lucky': 25, 'arrives': 24, 'complete': 85, 'hazmat': 1, '_blade': 1, 'empathize': 1, 'berardinelli': 1, 'usa': 8, 'millionaires': 1, 'accidental': 4, 'missing': 46, 'poorly-staged': 1, 'nation-wide': 1, 'altering': 2, 'lugosi': 2, 'income': 4, 'whiffleball': 1, 'hyped-up': 1, 'overexposed': 2, 'jewish': 8, 'realms': 2, 'sarita': 1, 'announced': 4, 'unity': 2, 'wipes': 2, 'assemble': 3, 'soleil': 1, 'castrate': 1, 'mouth-to-mouth': 1, '1944': 1, 'tie-ins': 4, 'unheard-of': 1, 'executioner': 1, 'despised': 1, 'cupcake': 1, 'conclusion': 41, 'hallahan': 1, 'convicting': 1, 'diabetic': 1, "craig's": 2, 'apology': 2, "powder's": 1, 'geeks': 2, 'blas': 2, 'arrogant': 7, 'murder-and-cover-up': 1, 'screeches': 1, 'vu': 5, 'customer': 5, 'nominatored': 1, 'stood': 12, "a-ha's": 1, 'deeper': 16, 'colorful': 14, 'bemusement': 1, 'slump': 1, 'decency': 6, 'nursing': 2, 'label': 9, "karen's": 2, 'compact': 1, 'predecessor': 10, 'sensitivity': 2, "dollah'": 1, 'wendy': 3, 'perfunctory': 9, 'cap': 6, 'semi-accomplished': 1, 'lehmann': 2, "suspense'": 1, 'villainous': 9, 'chop-job': 1, 'repay': 1, 'cathartic': 2, 'facher': 1, 'vampire$': 1, 'hong': 23, 'prevented': 4, 'inches': 2, 'virtual-reality': 1, 'professionally': 4, 'single': 90, 'torch': 1, 'amourous': 1, 'math': 5, 'rose-tinted': 1, 'ball': 23, 'indian': 10, 'cobo': 1, 'convey': 9, 'execute': 2, 'tighter': 3, 'punctuates': 2, 'rotten': 4, 'unspeakable': 1, 'winded': 3, 'struts': 4, "frankie'": 1, 'conspicuous': 3, 'behaving': 2, "'innocent'": 1, 'coolness': 2, 'melancholic': 2, 'bookkeeper': 1, 'striking': 12, 'yoda': 2, 'ever-': 1, '80%': 1, "cab's": 1, 'matrix': 14, 'filmgoing': 1, 'selena': 1, 'mixed': 19, 'continous': 1, 'beeps': 1, 'on-par': 1, 'slices': 2, 'fake-looking': 2, 'borrr-ring': 1, 'resuscitation': 1, 're-focuses': 1, 'saluting': 1, 'brain-dead': 5, 'dear': 11, "casper's": 1, 'requirements': 6, "anton's": 1, 'gussied': 1, 'traditionalist': 1, 'parade': 14, '_halloween_': 1, 'indie-underground': 1, 'tempestuous': 1, 'crunch': 2, 'leered': 1, 'suited': 10, 'blunder': 1, 'balloon': 2, 'quadrangle': 1, 'chaser': 1, 'purge': 1, 'regularly': 5, 'second-tier': 1, 'xxx': 1, 'racial': 5, 'kilometres': 1, 'tucked': 2, 'custom': 1, 'seismic': 1, 'collaborator': 1, 'obstacle': 1, 'flip': 8, 'contradicts': 1, 'iago-like': 1, '2036': 1, 'ontkean': 1, "lampoon's": 3, 'detrimentally': 1, 'apply': 6, 'yanking': 1, 'scolds': 1, 'arabe': 1, 'kamikaze': 1, 'completion': 1, 'lurk': 2, 'boiler': 1, 'pointers': 2, 'trading': 3, 'ludicrousness': 1, 'world-view': 1, 'connick': 2, 'thirty-minute': 1, 'register': 10, 'fuses': 1, 'army': 30, 'polish': 4, 'meandering': 4, 'ownership': 2, 'fatigued': 1, 'hark': 4, 'dulles': 1, 'coen': 2, 'maker': 8, 'stigmata': 6, "vegas'": 1, 'mortgages': 1, 'grouper': 1, 'elmer': 1, 'underwhelming': 4, 'angelo': 2, 'flocked': 1, 'laguna': 1, 'tennant': 1, 'fingerprints': 1, 'filled': 61, 'enhanced': 8, 'draggy': 1, '_red': 1, 'belloq': 2, 'lowest-brow': 1, 'mantegna': 1, '`enemy': 1, 'nothiiiiiinggggggggggg': 1, 'bus': 25, 'plastered': 4, '2293': 1, 'toppling': 1, "asimov's": 1, 'racket': 1, 'ad-lib': 2, "scumball's": 1, 'louder': 2, 'danger': 23, 'provocatively': 1, 'sicken': 1, "garrett's": 2, '80': 13, '7th': 1, 'outrun': 3, 'praising': 3, 'long': 263, 'tapped': 1, 'penn': 9, 'orgasm': 2, 'proposed': 3, 'degree': 23, 'r&b': 3, 'closure': 2, 'primordial': 2, 'assembled': 10, 'galaxy': 8, '`but': 1, 'handyman': 2, 'standbys': 2, 'bowie': 2, 'nightmare': 18, 'rabbit': 4, 'mathematician': 5, 'dive': 4, 'aristocrats': 1, 'bless': 2, 'cindy': 9, 'writes': 10, 'combat': 8, 'constance': 1, 'spatter': 1, 'punctuated': 6, 'waits': 5, 'pouty': 2, 'springs': 6, 'shoot-outs': 3, 'pre-frontal': 1, 'foul-ups': 1, 'unkempt': 3, 'servicable': 3, 'astaire': 1, 'lovable': 6, 'out-of-nowhere': 1, 'underdeveloped': 15, 'paradoxically': 1, 'harpoons': 1, 'repeats': 5, 'irish': 9, 'obligation': 3, 'frankfurters': 1, '_in': 1, "raining--you'd": 1, 'poorly-paced': 1, 'raspy': 1, 'action/adventure': 1, 'spewed': 2, 'headline': 4, 'aggression': 3, 'half-stated': 1, 'imposing': 6, 'audition': 1, 'behavior': 20, 'picky': 1, 'emancipation': 1, 'liquefied': 1, 'peacefully': 3, 'dismal': 12, 'outsider': 4, 'li': 10, 'smell': 8, 'circa': 5, "depalma's": 4, 'shon': 1, 'complex': 32, 'teen-agers': 1, 'cold-cocks': 1, 'tomatoes': 1, 'fx-heavy': 1, '\x13earth': 1, 'defendant': 2, 'mon': 1, 'sneeze': 3, 'seasons': 2, 'juxtaposed': 1, 'consists': 18, 'boorish': 4, 'bates': 8, 'little-known': 1, '100': 18, "long's": 2, 'grouse': 1, 'harasses': 1, 'ugliest': 1, "thade's": 1, 'facetious': 3, 'anti-hype': 1, 'neon-lit': 1, 'aforementioned': 27, 'raft': 2, "`dragonheart'": 1, 'disengaged': 1, 'eastwood': 6, "liz's": 4, 'spurning': 1, 'mentionable': 1, 'postal': 2, 'hunt': 20, 'struggling': 24, 'heavy-handedly': 1, 'capital': 6, "you'": 1, 'gratifying': 1, 'drink': 14, 'subsided': 1, 'jukebox': 1, 'glenne': 3, 'sulk': 1, 'vending': 1, 'year-old': 2, 'fumble': 1, '61': 1, 'risa': 1, 'garofolo': 1, 'fiasco': 4, 'troi': 2, 'armageddon': 17, 'signature': 5, 'woo-type': 1, 'wroth': 1, 'drug-abuser': 1, 'closet': 7, 'dry': 22, 'well-': 1, 'tackles': 4, "time'": 1, 'poolboy': 1, 'seriousness': 6, 'marcia': 2, 'conferring': 1, 'simpsons': 1, 'automatic': 4, 'approve': 1, 'mind-bogglingly': 1, 'busying': 1, "'net": 1, 'fart': 13, 'meshes': 1, "berenger's": 1, 'innocuous': 7, 'szubanski': 2, 'lark': 2, 'pasty-faced': 1, 'identifiable': 2, 'literature': 8, 'condemns': 1, 'phaedra': 1, 'code': 11, 'constitutes': 1, 'forgo': 1, 'blips': 1, 'merteuil': 1, 'malloy': 1, 'stupefyingly': 3, "mandingo's": 1, 'airwolf': 2, 'requests': 1, 'parental': 3, 'side-show': 1, "denny's": 1, 'kwai': 2, 'faith': 15, 'emits': 1, 'chattering': 1, 'deliveries': 1, 'heart-warming': 2, 'cowboy/hick': 1, 'viard': 1, 'greased-up': 2, 'astound': 1, 'sportscaster': 3, 'comparisons': 10, 'reynolds': 3, 'clitoris': 1, 'kills--he': 1, 'replayed': 2, '`beverly': 1, 'yanne': 1, 'absorbing': 3, 'white-collar': 2, 'driven': 18, 'synopsis': 33, 'bumblings': 1, "demille's": 1, 'reigned': 2, 'model-friends': 1, 'missus': 2, 'gathers': 4, 'poor-taste': 1, 'lift': 4, 'saunters': 1, "kim's": 1, 'artificiality': 2, 'hysteria': 3, "makers'": 1, 'rectangle': 1, 'undermined': 4, 'nutcase': 1, 'mayfield': 1, 'reduces': 1, "ayla's": 1, "heffron's": 1, 'prouder': 2, 'fully': 22, "wanna-be's": 1, "fussell's": 1, 'simultaneous': 1, "replacements'": 1, 'politically-correct': 1, 'rip-offs': 3, 'trauma': 4, 'smokes': 7, 'jjaks': 1, 'tawdriness': 1, 'baby': 45, 'co-writing': 2, 'region': 4, 'chain-snatchers': 1, 'vapid': 7, "chuck's": 1, 'dawn': 5, "ivy's": 2, 'definitly': 1, 'one-character-decides-not-to-return': 1, 'science-fiction': 7, 'grappling': 1, 'pest': 1, 'swims': 3, 'stagecoach': 1, 'payment': 2, 'professor': 18, 'hackman': 9, "boyd's": 1, 'bud': 3, '_have_': 1, 'definable': 1, 'controversial': 6, 'shou': 3, 'makeup-laden': 1, 'visits': 10, 'lobster': 1, 'voyeur': 2, "ru'afo": 1, 'el': 3, 'fears': 12, 'unremarkable': 4, 'sylvie': 1, 'constantly': 47, 'priceless': 3, 'hendricks': 1, 'jungle2jungle': 2, 'patient': 15, 'passionless': 1, 'jeopardize': 2, 'sugary': 4, 'half-second': 1, 'already-thin': 1, 'slum': 1, 'takashi': 1, 'meeks': 1, 'obliterated': 2, 'frankenheimer': 5, 'janevski': 1, 'woud': 1, 'chortling': 2, 'functioning': 2, 'automatons': 1, "victim's": 5, 'cabana': 1, "dudley's": 1, 'revolted': 1, 'attackers': 1, 'actual': 51, 'african-americans': 2, 'apprentice': 2, 'reverberates': 1, 'flour': 1, 'well-known': 5, 'stem': 3, 'busey': 5, 'deviates': 1, 'tie-up': 1, 'neater': 1, 'archetypal': 1, 'virtuous': 1, "'penalty'": 1, 'quick-cuts': 1, 'suspensful': 1, 'unseen': 7, 'lacrosse': 1, 'sexiness': 2, 'taught': 6, 'perplexing': 2, 'introducing': 7, 'paltry': 1, 'kalifornia': 2, 'punctuate': 1, 'purse': 2, 'clandestine': 1, 'sexless': 1, 'addresses': 1, 'skating': 1, 'traveler': 2, 'downing': 1, 'waited': 8, 'marian': 3, 'rockhound': 2, 'blind': 18, 'adversary': 1, 'loyalty': 4, 'fireman': 1, 'attenborough': 1, 'torpedo-ed': 1, 'stowe': 5, 'gators': 1, "down's": 1, 'marvin': 2, 'one-down': 1, 'documented': 1, 'contents': 3, 'madly': 5, 'setbacks': 1, 'deep-core': 2, 'amy': 17, 'demo': 1, '$39': 1, "visconti's": 1, 'representatives': 1, 'discovering': 10, 'comply': 1, 'dominatrix': 1, 'aesthetically': 1, 'suffers': 30, 'pitch': 11, 'comanche': 1, 'hawkes': 2, 'torro': 1, "salomon's": 1, 'private': 36, 'bears_': 1, 'shuns': 1, 'phoned': 1, 'regan': 1, 'defense': 14, 'student': 36, 'adopts': 5, 'fisherman': 11, 'amplified': 1, 'baker-enhanced': 1, 'keener': 7, 'december': 5, 'showcase': 14, 'super-cypher': 1, 'doofy': 2, 'playfully': 2, 'caldwell': 1, 'spectacular': 23, "finney's": 1, 'damn-near': 1, 'amiable': 4, 'mcbeal': 5, 'improves': 2, 'stock': 31, 'careens': 1, 'game': 78, 'miko': 4, 'sign': 39, 'destined': 7, 'talk': 85, 'wormhole': 1, 'unclear': 8, 'beatng': 1, 'thought-transference': 1, 'ruth': 1, 'angelic': 1, "perdy's": 1, 'spontenaity': 1, 'cyndi': 1, 'transformed': 7, 'flynn': 2, 'johnnie': 1, 'searches': 4, 'muse': 3, 'careening': 4, 'russ': 3, 'chatting': 1, 'mortals': 3, 'frasier': 2, 'styles': 4, '161': 1, 'intimate': 3, 'catch': 53, 'critics': 38, 'surgeon': 3, 'docudrama': 1, 'velvet': 11, 'melrose': 2, "theresa's": 1, 'wehrmacht': 1, 'purred': 1, 'smelling': 3, 'thurl': 1, 'once-linked': 1, 'irritable': 1, 'shante': 1, 'dairy': 1, 'disorientation': 1, 'pace': 33, "verne's": 1, 'fest': 3, 'ostentatious': 1, 'unfold': 5, 'idyllic': 5, 'williamson': 15, 'younger': 35, 'blank': 17, "frame's": 1, 'cooperate': 1, 'pre-war': 1, "suzie's": 1, 'traeger': 1, 'sorderbergh': 1, 'mused': 1, 'conduit': 2, 'occasionally-witty': 1, '`biggest': 1, 'mystery': 65, 'corridor/tunnel/airduct': 1, 'prestige': 1, 'patrick': 22, 'robertson': 2, 'cheque': 1, "weir's": 1, 'on-ship': 1, 'gorilla': 5, 'radford': 1, 'headaches': 1, 'weirdness': 6, 'souvenir': 1, "carroll's": 1, 'robust': 2, 'salvadoran': 1, "beat'em": 1, 'race-related': 1, 'thirty-something': 1, 'pits': 3, 'admirable': 9, 'bump': 8, 'murilo': 1, 'lunch': 8, 'depressive': 2, 'chimps': 4, 'gehrig': 1, 'championing': 1, 'devotes': 2, 'heretic': 1, 'confederate': 1, 'lol': 3, 'rounds': 3, 'backus': 1, 'tannek': 1, 'mysteries': 4, 'infuriates': 2, 'wb': 4, "beautiful'": 1, 'slumming': 1, 'pistol': 6, 'therapy': 5, 'funky': 5, '`cruel': 1, 'malkovich': 4, 'overexposure': 1, 'entering': 6, 'woozy': 1, '`there': 1, 'crystal/hugh': 1, 'ash': 4, 'paintings': 4, 'pretend': 15, 'ghostface': 2, 'carnival': 4, 'niro/james': 1, 'distribution': 6, 'tabloids': 1, 'denounce': 1, '-2': 1, 'goodboy': 4, 'innately': 1, 'casseus': 1, 'sneak-ins': 1, 'low-class': 1, 'weird': 38, "rhino's": 1, 'adeptly': 1, 'winter': 1, 'un-virginal': 1, 'mini-skirt': 1, 'initiate': 1, 'sawyer': 1, 'hangs': 6, 'butter': 1, "daniels'": 1, 'osmosis': 2, 'hips': 1, 'ceremony': 3, "prinze's": 3, 'persnickety': 1, 'belated': 2, 'flaus': 1, 'months--due': 1, 'massachusetts': 4, 'rat': 5, 'lise': 1, "horner's": 1, 'overtime': 2, 'childdhood': 1, 'sagging': 2, 'situational': 1, 'roommate': 9, "catharine's": 1, 'overachieve': 1, 'gregson': 1, 'tuna': 2, 'brewster': 2, "skulls'": 1, 'erm': 1, 'beefcake': 1, 'minnesota': 3, 'loyal': 10, 'gary': 32, "edison's": 1, 'sum': 13, 'sexism': 1, 'unit': 10, 'eagle': 1, "june's": 1, 'logy': 1, 'chittering': 1, 'impressive': 69, 'coats': 1, 'bedrooms': 1, '2am': 1, 'wax': 1, 'g-string': 1, 'zorro': 2, 'moons': 1, 'propulsion': 1, 'clicks': 1, 'sucking': 4, 'gourmet': 1, 'heartwarming': 3, 'comrades': 5, 'zena': 1, 'cockpit': 1, 'gaping': 8, 'locates': 1, 'communications': 1, "thurman's": 1, 'inciting': 1, 'railway': 2, 'rambo-on-testosterone-therapy': 1, 'layout': 1, 'shotgun': 6, 'girlfriend/moll': 1, 'shakespeare': 8, 'bookie': 2, 'tease': 3, 'rerelease': 1, 'bologna': 1, 'tom': 55, 'assist': 5, 'signified': 2, 'sometimes-funny': 1, 'disappearances': 2, 'incongruent': 1, 'grier': 5, 'ishmael': 2, 'graphics': 6, 'begun': 6, 'turkish': 1, 'easiest': 2, 'peanut': 1, 'proportions': 12, '$@^@': 1, '-ed': 2, 'irks': 1, 'permeate': 1, 'oldham-designed': 1, 'dog-eared': 1, 'brouhaha': 1, 'bow': 1, 'aggravating': 4, 'honest': 21, 'operate': 5, 'genre--_i_know_what_you_did_last_summer_': 1, 'trouble-making': 1, 'veruca': 1, 'morris': 2, 'darken': 1, 'incalculable': 1, 'pedicure': 1, 'advent': 1, 'dangles': 2, 'gatins': 2, '12': 12, 'bedtimes': 1, 'complains': 8, 'enigmatical': 1, '_dancing_': 1, 'looms': 2, 'screams': 20, '>honk': 1, 'caricature': 8, 'displeasure': 2, 'neandrathal': 1, 'crowchild': 1, 'outset': 10, 'dope': 5, 'crescendos': 1, 'feast': 4, 'headlined': 2, 'lib': 3, 'arlington': 1, 'steep': 3, 'creepier': 3, 'lowered': 2, 'fauna': 1, 'canary': 1, 'flubber-powered': 1, 'finney': 2, 'vapors': 1, 'haysbert': 2, 'agents': 15, 'buttons': 1, '2099': 1, 'blatant': 12, 'ashby': 1, "rat's": 2, 'lombard': 1, 'logic': 33, "millionaire's": 1, 'jared': 5, 'stripping': 2, 'conceptual': 2, 'juggle': 3, 'teed': 1, 'badass': 3, 'devote': 2, "'bout": 1, 'yep': 4, 'half-bad': 1, 'fox': 26, 'amis': 2, 'systems': 1, 'indies': 1, 'sued': 1, 'dreamed': 5, 'two-and-a-half': 4, 'michael': 122, 'ballyhooed': 1, 'brand-new': 1, 'anti-climactic': 4, 'masculinity': 1, "'whatever'": 1, 'defusing': 1, 'upstairs': 3, 'forgetful': 2, 'jerks': 8, 'gail': 2, 'pumping': 5, 'sentence': 11, 'stock-aitken-waterman': 1, 'spade': 4, 'birch': 2, '106-minute': 1, 'ate': 1, 'polyester': 1, 'cale': 2, 'jessie': 2, 'shao': 3, 'affiliated': 1, 'roving': 3, 'growth': 6, 'noodles': 2, 'dudes': 8, 'whites': 3, 'cane': 6, 'measure': 15, 'schwartz': 3, 'wet-': 1, 'asexually': 1, 'imbedded': 1, 'stavros': 1, 'ramon': 1, '75': 1, "reno's": 1, 'improperly': 1, 'excessively': 4, 'commonplace': 2, 'sailors': 1, 'wills': 1, 'mugged': 1, 'unlock': 2, '_urban_legend_': 1, 'sirtis': 1, "'tough": 1, 'rain-soaked': 1, 'extravagant': 6, 'hey': 38, 'lonsdale': 1, 'nonchalant': 3, 'opus': 6, 'puppet-style': 1, 'audiences': 54, 'concorde': 1, 'submitted': 1, 'anna': 5, 'humphries': 1, "`cops'": 1, 'thin': 51, 'venom': 1, 'bandies': 1, 'settlement': 2, 'filleted': 1, 'dissolves': 8, "pucci's": 1, 'blue-collar': 2, 'bravely': 1, 'pole': 2, 'deedee': 1, 'belting': 2, 'death': 120, 'carol+ghostbusters=scrooged': 1, 'sexpot': 1, 'build': 22, 'bawitdaba': 1, 'chairman': 3, "presidents'": 1, 'chuckie': 1, 'bumble': 2, 'meeting': 26, 'withdrawn': 3, 'heels': 10, "sister's": 2, 'zooming': 2, 'hairpin': 1, 'complexities': 3, 'reasonable': 14, 'possess': 11, 'silverman': 4, 'triplets': 1, 'begining': 1, 'aaliyah': 1, 'unforced': 1, 'inexorable': 2, 'credit': 45, 'judy': 5, 'stalkings': 1, 'journalists': 2, 'game-pod': 1, 'burwell': 1, 'grimacing': 1, 'logically': 2, 'disaffected': 3, 'misunderstand': 2, 'row': 11, "1971's": 1, 'homeboys': 1, 'paws': 2, 'anti-hero': 1, 'videotape': 3, 'ghouls': 2, 'floors': 3, 'batting': 2, 'word-of-mouth': 1, 'scolex': 1, 'stark': 3, 'blandest': 2, 'kong-recipe': 1, 'burning': 10, "alicia's": 2, 'spans': 1, 'absent': 10, 'offends': 1, 'priority': 2, 'laptops': 2, 'poorly-shown': 1, 'crazed-looking': 1, 'connection': 22, 'caroline': 5, 'kidnapper/murderer': 1, 'cogliostro': 1, 'coked-up': 1, 'bosom': 3, 'extended': 26, 'belly': 10, 'morals': 4, 'disheartening': 3, 'embarrsingly': 1, 'david': 99, 'moreau': 3, "3'": 1, 'brennan': 3, 'salon': 2, "costner's": 3, 'withholding': 1, 'snicker': 2, "'cute": 1, 'hermit-like': 1, 'ex-beau': 1, 'cognac': 1, 'louis': 5, 'irrefutable': 1, 'pendleton': 1, 'nike': 1, 'sanguine': 1, 'frequents': 1, 'relevance': 3, 'recently': 60, "hitchcock's": 8, 'bungling': 1, "terry's": 1, 'dixon': 2, 'shivery': 1, 'face-to-face': 1, 'expensive': 21, 'bitchy': 5, 'singer': 24, "depardieu's": 1, 'dullard': 1, 'synchronicity': 1, 'rock-solid': 1, 'governmental': 1, 'mutual': 2, '_jerry_maguire_--will': 1, 'damp': 1, 'homogenize': 1, 'land-once': 2, 'suchet': 3, 'imaginary': 2, "ways-i'll": 1, 'rifle/camera': 1, '#$%': 1, 'schaech': 5, "dreamworks'": 1, 'spirits': 4, 'shortened': 3, "damme's": 4, 'conflicting': 2, 'whole-sentence': 1, 'obey': 1, 'graininess': 1, 'bills': 4, 'basic': 43, 'h2k': 1, 'decked': 3, 'lauren': 11, 'sluttish': 1, 'siberian': 1, 'branaugh': 1, "taylor's": 4, 'dialogue': 210, "'95": 1, 'misfortune': 1, 'empire': 8, 'low-flying': 1, "hounsou's": 1, 'conspire': 4, 'ingenuity': 5, 'chenoa': 1, 'ethical': 3, 'daunting': 1, 'catalogue': 1, 'ravishingly': 1, 'afore': 1, 'hotcakes': 1, 'supervirus': 1, 'enrich': 1, '3-why': 1, 'direcing': 1, 'legal-drug': 1, 'dumped': 11, 'felony-studded': 1, 'gather': 6, 'scarface': 2, 'herky-jerky': 2, "'three": 1, 'mobile': 2, 'lake': 9, 'terribly': 34, 'absolution': 1, 'passangers': 1, 'nondescript': 2, 'fabulously': 1, 'diverted': 2, "experimentation's": 1, 'vampires': 13, "yost's": 1, 'proceedings': 34, 'guys/girls': 1, 'health': 10, '-4': 18, 'sensing': 4, 'alison': 2, 'prostheses': 1, "method's": 1, 'smirky': 1, 'hokum': 5, 'attached': 14, 'boy-and-his-dog': 1, 'charmless': 6, 'behaved': 1, 'amazed': 10, 'reluctantly': 9, 'pego': 3, 'rewards': 4, 'opportune': 1, 'evens': 1, 'repeated': 15, "`we're": 2, 'senses': 11, 'researching': 3, 'skew': 1, 'washing': 1, 'strangled': 1, 'duarte': 1, 'intangible': 2, 'wading': 1, 'underwater': 7, 'represents': 15, 'crucifix-weapon': 1, 'pricked': 1, 'mare': 2, 'darius': 3, 'reproduce': 1, 'krabb': 1, 'shephard': 1, 'quadruple': 1, 'themselves--they': 1, 'goddamned': 1, 'unenjoyable': 1, '2-d': 2, 'prevue': 1, 'pain': 19, 'secondhand': 1, 'spontaneously': 5, 'vast': 7, 'mish-mashed': 1, 'rossellini': 1, 'doff': 1, 'hm': 1, 'shagging': 1, 'coquette': 1, 'landowner': 1, "_it's": 1, 'humbled': 1, 'cared': 8, 'lieutenants': 1, 'buck': 10, 'filmore': 1, 'scars': 1, 'genuis': 1, 'delving': 2, "peel's": 1, 'separately': 1, 'once-sharp': 1, 'populous': 1, '13-20': 1, 'mckenney': 1, 'confident': 6, 'juicing': 1, 'placing': 7, 'hensleigh': 1, 'chuckling': 3, 'plot-holes': 2, 'meaning-free': 1, 'bug': 18, 'guzzling': 1, 'young-whippersnapper': 1, 'sketch': 11, 'witch-related': 1, 'sunlight': 4, 'continuum': 1, 'operas': 4, 'imparted': 1, "clinton's": 2, 'bascially': 1, 'rollercoaster': 1, "green's": 1, 'lethal': 25, 'prefers': 7, 'speedboat': 2, 'doomed': 11, 'hiring': 3, 'danger/attempted': 1, 'zeit--get': 1, 'cab': 6, 'rumours': 3, 'bloodline': 1, 'second-run': 1, "maybury's": 1, 'analyst': 1, 'cremated': 1, 'dedicated': 14, 'gnosis': 1, 'sluts': 1, "disgusting-for-disgustingness'-sake": 1, 'special-effects': 1, 'kiddie': 2, 'peel': 6, 'ships': 10, 'slapstick': 22, "neither's": 1, 'integrity': 1, 'grab': 10, 'inconsistencies': 7, 'slater/johnny': 1, "continuity's": 1, "depp's": 2, 'posse': 2, 'undead': 3, 'acrobatics': 4, 'contrivances': 9, 'one-fourth': 1, 'preppie': 1, 'underachieving': 3, 'take-the-money-and-': 1, 'pollack': 7, 'simply': 159, 'kiki': 2, 'relax': 2, 'renfro': 1, 'founder': 5, 'seller': 1, 'tour-': 1, 'influences': 2, 'pay-per': 1, 'soldier': 12, 'poe': 1, "90s'": 1, 'rejuvenates': 1, "frankenstein's": 2, 'chingmy': 1, 'characterisation': 2, 'entities': 2, 'negotiate': 5, 'order': 95, 'grown-up': 6, 'pages': 9, 'magazines': 4, 'karaoke': 3, 'gem': 7, 'millions': 13, 'beer': 9, 'portrait': 9, 'annoy': 5, 'thingie': 2, 'carville': 1, 'jock': 6, 'subtlety': 18, "pitillo's": 1, 'pod': 5, 'insomniac': 2, "untouchables'": 1, 'bedsprings': 1, 'thirsty': 1, 'puppies': 2, 'centerpieces': 1, 'shelve': 1, 'photograph': 3, 'original': 187, 'rowan': 1, 'arlene': 1, 'put-on': 1, 'roller': 9, 'dispatching': 1, 'absurdism': 1, "o'barr's": 1, 'digestion': 2, 'spice': 19, 'skif': 1, 'promotional': 6, "allayah's": 1, 'drops': 13, 'slow-fade': 1, 'modeled': 4, 'sensually': 1, 'benoit': 1, 'supermarket': 2, 'out-bland': 1, "'er": 1, 'buggy': 1, 'near-sequel': 1, '28': 2, 'skip': 30, 'magda': 2, 'directing': 38, 'sitting': 48, 'bayouesque': 1, 'rash': 6, 'fatigues': 1, 'sinister-sounding': 1, 'enticed': 1, 'current': 30, '`the': 11, 'boring/confusing': 1, 'shelmikedmu': 2, 'suzanne': 1, 'guesses': 2, 'neccesity': 1, 'maximum': 8, "delaurentiis'": 1, "gulliver's": 1, 'lysterine': 1, 'manifestations': 2, '5/10': 10, 'globes': 1, 'botches': 2, 'randanzo': 1, 'fast-motion': 2, 'extentions': 1, 'locale': 2, 'gingerly': 1, 'literary': 8, 'faiths': 1, 'specialized': 1, 'echoed': 1, 'window': 31, 'luxury': 8, 'hydra': 1, 'nastiness': 3, 'laughless': 1, 'rangers': 5, 'rocker': 4, 'finite': 1, 'reinforce': 2, 'valleys': 1, 'self-congratulatory': 2, 'rehashing': 2, 'litefoot': 1, '_loathe_': 1, 'turning': 37, "'controversial'": 1, 'motels': 1, 'wrecks': 4, 'fed': 10, 'helgeland': 2, 'overprotective': 3, '3000': 15, 'adorable': 6, 'shot': 105, 'unprintable': 2, 'chit': 1, 'noting': 5, 'ferris': 4, 'ex-ranger': 1, 'amnesia': 4, 'presumed-but-never-proved': 1, 'epiphany': 3, 'swashbuckler': 1, 'burrow': 3, 'dog-walking': 1, "hippo's": 1, "1970's": 4, 'comprehension': 1, 'hilarous': 1, 'supreme': 1, 'commented': 1, 'traps': 4, 'round': 13, 'patriarchy': 1, 'retire': 5, 'defeat': 8, 'travis': 5, 'proclaims': 6, 'pammy': 1, 'sincerity': 3, 'close-up': 8, 'arthurian': 1, 'belts': 1, 'prayers': 1, 'smoothly': 3, 'b-list': 1, 'tension': 46, 'segal': 3, 'derek': 4, 'stability': 3, 'fujioka': 1, 'whacked': 3, 'gallant': 1, 'angsty': 2, 'crashing': 11, 'pricks': 1, 'routes': 1, 'scorsese': 2, "veronica's": 1, 'canvas': 1, "'evolution": 1, "'new'": 2, 'kindergartner': 1, 'tantrums': 1, 'resents': 3, 'elegaic': 1, 'tulip': 2, 'uh-oh': 1, "kidman's": 1, 'biz': 3, 'exercise': 31, 'uttered': 5, 'mcfadden': 1, "schaech's": 1, 'cloying': 2, 'metallic': 5, 'fuzzy': 6, 'sattler': 1, 'smacking': 2, 'crucified': 2, 'dusters': 1, 'counter-terrorist': 1, 'sequelitis': 1, 'disproving': 1, 'exterminating': 1, "desouza's": 1, 'afterward': 3, 'enforcement': 6, 'lot--bored': 1, 'raving': 4, 'la': 33, 'avoid': 43, 'mandingos': 1, 'mugs': 3, "`secret'": 1, 'asthma': 2, 'laughlin': 2, 'remarries': 2, 'compete': 2, 'committee': 4, '8mm': 8, 'oversimplification': 1, 'honchos': 1, 'take-no-crap': 1, 'much-ballyhooed': 1, 'cheats': 2, 'corman': 5, 'solidly': 1, 'trampoline': 1, 'screenwriters': 37, 'preferring': 1, 'accomplish': 17, 'horseman': 2, 'vote-pandering': 1, 'resilient': 1, 'barbaric': 1, 'alleviate': 1, 'rorschach': 1, 'stormtrooper': 1, 'seasoning': 1, 'couteney': 1, 'category': 24, 'lamely': 3, 'tepid': 6, 'boyfriend/soon-to-be': 1, 'unsexy': 1, 'white': 82, '1-1': 1, '1965': 1, "couples'": 1, 'regular': 29, 'fragments': 2, 'duets': 1, 'surround': 6, "warner's": 1, 'proper': 9, 'halting': 1, "king's": 9, 'patriarchal': 1, 'revealing': 14, '`have': 1, 'carlos': 2, 'relying': 5, "'psychologically": 1, 'intern': 1, 'jenny': 3, "sweethearts'": 1, 'landmark': 1, 'paddling': 1, 'garth': 4, 'handle': 29, "'minor-regulars'": 1, 'interminable': 9, 'nevermind': 1, 'python': 4, 'centaur': 1, 'battles': 11, "terel's": 1, "'bluer'": 1, 'haughty': 4, 'gain': 17, 'extinction-': 1, 'chainsaws': 1, "oregon's": 1, 'yanni': 1, 'kit': 5, 'drumroll': 1, 'incite': 3, 'chairs': 2, 'morneau': 3, 'straps': 1, 'veneer': 1, '_heathers_': 1, 'barr': 2, 'concluding': 3, '`drive': 1, 'airing': 1, 'impersonation': 2, 'nine-year-': 1, 'urged': 3, 'mortimer': 1, 'out-of-key': 1, 'scrutinizing': 1, 'orbiting': 2, 'escalated': 1, '20-30': 1, 'sticky': 3, 'mozell': 1, 'sedgwick': 1, 'formulas': 5, 'splitting': 1, 'pleasured': 1, 'camelot': 1, 'reactor': 1, 'lingerie': 2, 'armrest': 1, 'boxer': 3, 'derogatory': 2, 'luke': 9, 'gesserits': 1, "'old": 1, 'met': 22, 'paycheck': 9, 'whiff': 5, 'iliff': 1, 'norville': 3, "hubbard's": 2, 'exits': 1, 'telescope': 1, 'excess': 5, 'accompaniment': 3, 'frying': 1, 'casts': 6, 'spices': 1, 'del': 8, 'coaching': 3, 'griffith': 9, 'pratt': 1, 'home-cooked': 1, 'thulin': 1, 'goody-goody': 1, 'squeaky-clean': 1, 'tenth': 2, 'tinkling': 1, 'fleshy': 1, 'mills': 1, 'medicine': 9, 'planner': 2, 'smoking': 16, 'pharmaceutical': 2, 'serpentine': 3, 'whitfield': 1, 'develops': 12, 'tense': 13, 'reconciles': 1, 'organizes': 1, 'pine': 2, "inkpot's": 4, 'roro': 1, 'guaranteeing': 1, 'minimalistic': 1, 'property': 9, 'props-strategically-positioned-between-naked-actors-and-camera': 1, 'pantucci': 1, 'giggled': 1, 'bathtub': 10, 'grateful': 5, 'craftsmanship': 2, 'tenuous': 1, 'crawl': 8, 'cure': 8, "insurrection's": 1, "kay's": 1, 'pic': 4, 'spraying': 1, 'ime': 1, 'post-_scream_': 1, "'a": 2, '26': 2, 'romantic': 73, 'deposit': 2, 'oboe': 1, 'plagiarism': 1, 'refunds': 1, 'whoop-dee-doo': 1, 'hope': 101, '1995': 25, "sterling's": 1, 'elitism': 1, 'kissner': 1, 'counting': 12, 'lousy': 17, 'parodying': 2, 'sergio-leone-': 1, 'virgil': 1, 'downey': 1, 'open-mike': 1, 'cuesta': 1, 'electronics': 1, 'furnished': 1, 'partying': 1, 'tequila': 1, 'immune': 3, 'wiped': 5, 'limousine': 2, 'hisses': 1, 'propaganda': 6, 'giggles': 5, 'forgive': 8, 'confess': 3, 'movie-goer': 4, 'vaporized': 1, 'ticket': 11, "talkin'": 1, '--a': 2, 'uncontrollable': 1, 'yakuza': 2, 'ollie': 1, 'inexperienced': 4, "mcdowell's": 1, 'journalism': 1, 'innumerable': 3, 'low-level': 1, 'ineptness': 1, 'pesci': 1, "houser's": 1, "religion's": 1, 'moviemaking': 5, 'versions': 6, 'berger': 1, 'shit-terpiece': 1, 'impeachment': 1, 'assent': 1, 'powers': 22, 'quota': 2, 'sneaks': 5, 'hung': 3, '9/10': 5, 'croon': 2, 'excellent-but-so-': 1, 'capitalism': 2, 'carrey-ish': 1, 'secures': 2, 'fundamentals': 1, '_some_': 1, 'arranged': 2, 'twelve': 11, 'possibly': 52, 'beings': 7, 'jellyfish': 1, 'buddy': 26, 'abandonment': 1, 'rafael': 2, 'alfie': 1, 'iii': 19, 'pushes': 6, 'watched': 34, 'quoting': 2, 'mentally-challenged': 1, 'grammer': 1, 'mockingly': 1, 'swartzenegger': 1, 'luckily': 8, 'expensive-': 1, 'concentrates': 5, 'no-name': 2, 'lowly': 3, 'counsels': 1, 'misrepresentation': 1, 'embody': 1, 'co-creator': 1, 'padded': 2, 'rick': 18, 'rickman': 3, 'streetcar': 2, "kids'": 6, 'stunning': 19, 'aimee': 1, 'ignores': 3, "spices'": 1, 'mistress': 5, "'ugly's": 1, 'bob': 43, 'captivates': 1, 'obnoxiously': 1, 'celestrial': 1, 'dragging/salting': 1, 'tanked': 1, 'cyberkillers': 1, 'brill': 1, 'wielding': 7, 'yelled': 2, 'scottish': 6, 'mexico': 8, "90's": 9, 'dying': 25, 'mysticism': 3, 'rodmilla': 1, 'forcefully': 1, '1992': 9, 'novalee': 1, 'idol': 4, 'command': 9, 'growling': 3, 'nolan': 1, '1871': 3, 'ensues': 17, 'torrential': 2, 'teeming': 1, 'soundtracks': 4, 'jumping-off': 1, 'rendering': 3, 'form': 65, 'puberty': 4, 'backdraft': 1, 'overnight': 3, "portman's": 1, 'artsy': 2, 'marijo': 1, 'un-nerving': 1, 'diet': 2, 'hollywoodized': 1, 'daggers': 1, 'thorpe': 1, 'call-waiting': 1, 'rewatch': 2, 'greyhound': 1, 'homicides': 1, 'disgustingly': 4, 'absconded': 1, 'echoing': 1, 'matching': 1, 'constructed-harry': 2, "rising's": 2, 'story-line': 3, 'poitier': 1, 'musketeer': 5, 'fumblings': 1, '30-year-old': 1, "roberts'": 1, 'arisen': 1, 'collection': 24, 'circus': 6, 'nomadic': 1, '+4': 18, 'supermodel-level': 1, 'asap': 1, 'ill-tempered': 1, 'disappoint': 5, 'knox': 4, 'disenfranchised': 1, 'neo-': 1, 'snorks': 1, 'anglo': 1, "frears'": 1, 'stank': 1, 'sledgehammer': 4, 'gypsy-like': 1, 'ineffectual': 5, 'julianna': 1, 'maniacally': 3, 'bras': 2, 'tumultuous': 1, 'zellwegar': 1, 'maurice': 4, 'inferior': 10, 'invigorating': 5, 'documenting': 1, 'references': 18, 'alessa': 1, 'scripting': 4, 'upcoming': 18, 'stressed': 2, 'non-verbal': 1, 'extensively': 2, 'resistance': 4, 'nonmembers': 1, 'body-builders': 2, 'threads': 4, 'demons': 6, 'antsy': 2, "phil's": 1, 'heartbeat': 1, 'accomplices': 2, 'quoted': 2, 'grail': 1, 'rickles': 1, '555': 1, 'exports': 1, 'unimportant': 2, '$70': 6, 'niko': 1, 'coming-of-age': 4, 'teaches': 9, "johnson's": 2, 'accosted': 1, 'wind': 19, 'red-eyed': 1, 'flamboyant': 5, 'candidates': 2, "caesella's": 1, "lewis'": 2, 'home-schools': 1, 'duel': 3, 'wynter': 3, '500th': 1, 'skarsgard': 1, 'mis-casting': 1, "eye's": 1, 'blanche': 2, 'chummy': 1, 'small-town': 7, 'caged': 1, 'sponge': 2, 'familiar--one': 1, "unger's": 1, 'littering': 1, 'bogs': 1, 'medieval': 4, 'capacity': 7, 'cesars': 1, 'punched': 1, 'sizeable': 1, 'well-designed': 1, 'amundsen': 1, 'hobel': 1, 'parker': 9, 'dont': 4, 'faking': 3, 'comprehensive': 1, 'rock-em': 1, 'crossbows': 1, 'stan': 4, 'intelligible': 3, 'atrocity': 4, 'edgar': 1, 'cd-rom': 1, 'devises': 2, 'doubtlessly': 1, 'imdb': 2, 'sidesteps': 1, 'wash': 7, 'humans': 41, 'timbers': 1, 'coat': 7, 'congregation': 1, 'disastrously': 3, 'remington': 3, 'movie-make': 1, 'billowy': 2, 'life-and-death': 1, 'day-timer': 1, 'condom': 1, 'sidekick': 20, 'comfortable': 12, 'heads': 40, 'overreaching': 1, 'strict': 11, 'cleaner/slave': 1, 'korda': 2, 'high-quality': 2, 'cringe-': 1, 'reeks': 7, 'equitably': 1, 'span': 7, 'deliberately': 9, "ryan'": 1, 'rudd': 5, 'robs': 3, 'takaaki': 2, '1/1/1900': 1, 'eccentrics': 3, 'neanderthal': 2, 'makeup': 18, 'half-convincing': 1, 'glossing': 1, 'intaking': 1, 'rework': 1, 'modernization': 1, 'presence': 78, 'exploding': 5, 'opponent': 8, 'britney': 1, "stigmata's": 1, 'pallid': 1, 'sciorra+s': 1, 'hurried': 1, 'slower': 5, 'bernhard': 2, 'heavy-rotation': 1, "canines'": 1, 'excellent': 31, 'uproariously': 1, 'cackling': 2, 'bemoaning': 1, 'worlds': 6, 'reinforcement': 1, "tucker's": 1, 'schandling': 1, 'carrot': 3, 'column': 4, 'macpherson': 3, 'narcolepsy': 1, 'colorblind': 1, 'proving': 8, "york's": 3, 'transvestites': 2, 'mewes': 3, "1988's": 1, 'professors': 1, 'brackett': 2, 'wwii-era': 1, 'curtains': 3, 'kritic': 1, 'irrationalities': 1, 'starpower': 1, 'adams': 18, 'regurgitate': 1, 'waaaaay': 1, 'bacteria': 1, "project's": 1, 'bumcheeks': 1, 'workman': 1, 'rookies': 2, "bean's": 3, 'zeltser': 1, 'exceedingly': 3, 'pore': 1, 'hindi': 1, 'perfekt': 1, 'startlingly': 2, 'rape': 21, 'logistics': 2, 'happier': 1, 'dramatically': 11, 'pep': 1, 'greer': 3, 'strangulation': 1, 'not-too-precocious': 1, 'lackadaisical': 1, 'improvements': 1, 'sheet': 2, 'mottled': 1, 'hard-to-decipher': 1, 'ky': 1, 'woolrich': 1, 'brooding': 9, "ship's": 6, 'fists': 2, 'horribly': 24, 'exploit': 11, 'dutton': 2, "corky's": 1, 'deuce': 1, 'lily': 4, 'impersonates': 2, 'revisiting': 1, 'station': 25, 'gunmen': 1, 'retribution': 3, 'humors': 1, 'philosophies': 2, 'behind-the-scenes': 4, 'go-cart': 1, 'distinctive': 8, 'episode': 37, 'smack': 4, 'mystery-turns': 1, 'dimly-lit': 1, 'validity': 1, "sorvino's": 1, 'decrees': 1, 'fifteen': 25, 'conked': 1, 'tension-filled': 1, 'cleveland': 3, 'finish': 15, 'lightning': 11, "milton's": 1, 'pales': 1, 'excrutiatingly': 1, 'hannah': 7, 'embroiled': 1, '_somewhere_': 1, '175': 1, 'mortal': 15, 'jia-li': 1, 'slashing': 1, 'files': 4, 'supremely': 4, "alex's": 2, 'countdown': 2, 'urinal': 1, 'starkly': 1, 'refreshingly': 1, 'nightwolf': 1, 'lasting': 8, 'alterations': 1, 'woo-ping': 2, 'rifle-ly': 1, 'strom': 1, 'holgado': 1, 'newbie': 1, 'mist': 3, 'wally': 2, 'pulling': 17, 'stewart': 10, 'buddhist': 1, 'morning': 32, 'moldy': 1, 'twosome': 1, "damone's": 1, 'award': 23, 'included': 25, 'retain': 4, 'dual': 1, 'monument': 1, 'pharmacy': 1, "rochon's": 1, 'statue': 4, 'motivation': 10, 'concentrating': 5, 'perfection': 13, 'rolls': 9, 'poet': 3, 'conviction': 7, 'fulfillment': 1, "tarantino's": 3, 'tempo': 1, 'germ': 2, 'garb': 2, 'escapism': 2, 'fairy': 5, 'blurred': 2, 'non-sequiturs': 1, 'addiction': 5, 'needy': 1, 'roars': 1, 'know-it-all': 1, 'tiered': 1, 'nigh': 1, 'mcquarrie': 2, 'pyramid': 2, 'banality': 6, 'eeeewwwww': 1, 'alternate': 14, 'prostitues': 1, 'violated': 2, '105-minute': 2, "jackie's": 1, 'ken': 10, 'bungee': 4, 'planned': 11, 'weisberg': 3, 'wincer': 1, "outworld's": 1, 'polaroid': 1, 'civilization': 5, 'masaya': 1, 'self-professed': 2, 'weirder': 1, 'purposely': 7, 'green': 27, 'briefer': 1, 'alternately': 3, 'languages': 2, 'emotive': 1, 'thighs': 2, 'food': 31, 'fair/canticle': 1, 'loco': 1, 'gavan': 1, 'mammals': 2, 'torsos': 2, 'mammoth': 4, 'oddest': 1, 'anal': 2, 'participating': 2, 'ismael': 1, 'half-world': 1, 'miscarrying': 1, 'upbeat': 6, 'sterotypes': 1, 'mi': 2, 'uninvolved': 2, 'palitaliano': 2, 'palmetto': 6, 'rig': 4, 'aldys': 2, 'progression': 1, 'windy': 2, 'one-dimension': 1, 'grotesque': 9, 'ellie': 5, 'talky': 4, 'bosomy': 1, 'tremor': 1, 'schmaltzfest': 1, "'you": 1, 'authors': 5, 'impulse': 3, 'topheavy': 1, 'stills': 2, 'prudent': 1, 'difference': 29, 'vies': 2, 'nudity': 32, 'dearth': 1, 'blue-toned': 1, 'gascogne': 1, 'cigarette': 4, 'commercial-like': 1, 'occaisionally': 1, 'btw': 4, 'sex-': 2, 'wolves': 6, 'crusades': 2, 'ran': 18, 'blockbusters': 6, 'latifah': 1, 'twins': 10, 'inaccuracies': 1, '\x13if': 1, 'partaking': 1, 'merging': 1, 're-wire': 1, 'commanded': 2, 'climactic': 18, 'intrigues': 2, 'editors': 3, "woo's": 7, 'levar': 1, 'firearm': 1, 'philosophically': 1, 'scully': 3, "matilda's": 1, 'psychoanalyze': 1, 'scarcely': 4, 'bernsen': 2, 'it_': 1, 'beard': 10, 'mediocre': 35, 'cowled': 1, 'here-to-there': 1, 'pure': 25, 'fruit': 4, '110': 1, 'frances': 1, 'illiterate': 3, 'hull': 3, 'serling': 2, 'techno-gothic': 1, 'editted': 2, 're-think': 2, 'ups-and-downs': 1, 'rule': 31, 'increased': 3, "'taken": 1, 'commando': 1, 'ninth-consecutive': 1, 'epitomizes': 1, 'felicity': 1, 'web': 9, "bujold's": 1, 'homeworld': 1, 'platt': 9, 'huge': 69, "d'artagnan": 4, 'thriftily': 1, 'manufacturer': 3, 'over-alled': 1, 'worn': 8, 'larry': 15, 'emporer': 2, 'breakdown': 7, 'abberation': 1, 'explains': 25, 'brilliancy': 1, 'bets': 6, 'career-defining': 1, 'jump-start': 2, 'tilting': 1, 'nicholson-blown-down-the-street': 1, 'bowel': 2, 'pets': 3, 'lowe': 1, 'prostitution': 2, 'skyrocketing': 2, 'suitable': 9, 'joke': 70, 'gq': 1, 'sinister': 10, "government's": 1, 'out-of-context': 1, "carter's": 3, 'lucinda': 1, 'screw-up': 2, 'decided': 53, 'exclusive': 5, 'bootsie': 1, 'thrilled--': 1, 'wallows': 1, "v's": 1, 'wilde': 2, 'avatar': 1, 'allotting': 1, 'dredd': 1, 'civilisation': 1, 'notting': 2, 'gator': 1, 'drug-addicted': 1, "thomas'": 1, '1961': 4, 'handiwork': 1, 'circumventing': 1, 'realism': 13, 'logical': 11, '[to': 1, 'thompson': 11, 'aristocratic': 1, 'kingsley': 3, 'stonily': 1, 'predator': 10, 'observations': 5, 'summertime': 1, "tristen's": 1, 'knepper': 1, 'catatonic': 3, 'drug-addict': 2, 'plot-driven': 1, 'hicks': 1, 'aspire': 3, 'slack': 4, 'uppers': 1, 'flares': 1, 'barely': 51, 'lili': 6, 'foremost': 3, 'strong-willed': 3, 'words': 80, 'odd': 43, "davis'": 1, 'tenant': 2, '&quot': 1, 'superiors': 3, 'gravitates': 1, 'projectile': 1, 'name--': 1, 'amounts': 22, 'katie': 8, 'garp': 1, 'winwood': 1, 'semi-automatic': 1, 'terrifies': 1, 'spurned': 1, 'apartment': 32, 'pinkett': 2, 'unsettled': 1, 'beards': 1, "'95s": 1, 'reflectively': 1, 'hostility': 1, 'taxi': 2, 'ironic': 17, 'fichtner': 2, "actors'": 8, 'mcfarlane': 2, 'vadar': 1, 'result': 89, 'monkey-like': 1, 'karin': 1, 'helms': 1, 'dug': 5, 'guesswork': 1, 'wrights': 1, 'stormare': 6, 'centuries': 2, 'marlo': 2, 'black-he': 1, 'pleases': 2, 'eraser': 4, 'longest': 2, 'assisting': 1, 'ascent': 1, 'disappointingly': 4, 'fax': 2, 'ww2': 1, 'main': 146, 'gaius': 1, 'spurned-psycho-lover-who-wants-revenge': 1, 'encounter': 30, 'permanant': 1, 'psyched': 1, 'dolphins': 1, 'patton': 3, 'stretching': 6, 'loathing': 6, 'breath-taking': 1, 'self-discovery': 1, 'naturally': 23, "someone's": 11, 'one-if': 1, 'he-man': 2, 'freshen': 1, 'post-chasing': 1, 'jacksonville': 2, 'lewd': 1, 'deedles': 2, 'wiser': 3, 'handed': 3, 'resurface': 1, 'battle--going': 1, 'pre-arranged': 1, 'savoured': 1, 'lebanese': 1, 'handbasket': 1, 'fearing': 3, 'induces': 1, 'dazzle': 7, 'escorting': 1, '_the': 6, 'mtv-land': 1, 'mortars': 1, 'lipsynching': 1, 'animatedly': 1, 'none-too-successful': 1, 'sub-road': 1, 'marketplace': 6, 'anticipating': 2, "bogg's": 1, 'vigilant': 1, 'ilynea': 1, 'warbling': 2, "ol'": 8, 'apace': 1, 'occasion': 8, 'repetition': 3, 'bickers': 2, 'pecan': 2, 'attitude': 29, 'heiress': 2, 'worried': 10, 'hormones': 3, 'moth': 1, 'unleash': 2, 'time-travel': 1, 'illuminata': 1, 'retriever': 2, 'fraidy-cat': 1, 'binoculars': 1, 'images': 28, 'impregnate': 4, 'oozes': 3, 'pedestal': 2, 'pimply': 1, 'supercode': 1, 'vicki': 1, 'september': 3, 'love': 266, 'courtoom': 1, 'kind': 188, 'metamorphoses': 1, 'arkansas': 1, "nyc's": 1, 'scamp': 1, 'priestly': 1, 'warrants': 2, 'disruptions': 1, 'incidentally': 9, 'waded': 1, 'creditors': 1, 'courteney': 3, 'brainerd': 1, 'bend': 2, 'reserved': 7, 'unwarranted': 1, "xiong's": 1, 'seldes': 2, 'madio': 1, 'mueller-stahl': 3, "cavanaugh's": 1, "doens't": 1, 'mainstay': 1, 'profanities': 3, 'connected': 5, '-1': 3, 'figureheads': 1, 'harmon': 1, 'recreation': 5, 'infused': 2, 'undercurrent': 2, 'jimmies': 1, 'car': 114, 'ck': 5, 'velda': 1, 'howlers': 2, 'spousal': 1, "jennifer's": 1, 'motorcyclists': 1, 'meteors': 1, 'microchip': 1, 'rubin': 2, 'champions': 2, 'sellers': 2, 'backward': 1, 'criticised': 1, 'cackle': 1, '=': 10, "world's": 16, "house's": 1, 'origins': 3, 'engulf': 1, 'circumstances': 19, 'alright': 9, 'wet-wired': 1, 'below-average': 1, 'underestimating': 1, "schwartzenegger's": 1, 'import': 2, '1966': 4, 'evacuation': 1, 'ingested': 1, 'inveigles': 1, 'worries': 4, 'cult': 28, 'molecular': 1, 'slender': 1, 'skins': 1, 'landau': 4, 'aneeka': 1, 'elodie': 1, 'groping': 1, 'familiarity': 3, 'multidimensional': 1, 'brats': 1, 'ape-creature': 1, 'chopsticks': 1, 'djimon': 3, 'lets': 17, 'ruben': 2, 'trace': 6, 'coeds': 1, 'oscars': 4, 'spies': 5, 'awa': 1, "chan'": 1, 'mimicking': 2, 'marathon': 3, 'boorman': 3, 'ad-libbing': 1, 'rest': 177, 'beryl': 1, "toll's": 1, 'proficiency': 2, 'gained': 9, 'petra': 1, 'kingpin': 6, 'dreck': 6, 'magoo': 5, 'terminated': 2, 'guano': 1, 'beymer': 1, 'ingen': 1, 'dire': 2, 'nadia': 2, 'employs': 4, 'iwo': 1, 'cynics': 2, 'smartass': 1, 'horsing': 1, 'self-worth': 1, 'adopted': 3, 'lice': 1, 'public': 33, 'basic--and': 1, 'thrashing': 2, 'serenely': 1, 'romero': 2, 'puerile': 1, 'commercially': 2, 'neighboring': 3, 'slake': 1, 'honcho': 2, 'mattered': 2, 'saucy': 3, 'ichabod': 1, 'cutlery': 1, 'bone-headed': 1, 'reminded': 20, 'alicia': 14, '2hr': 1, 'dares': 5, 'leguizamo': 7, 'cynicism': 8, 'assess': 2, 'combined': 17, 'morons': 3, 'spider': 5, 'margiulles': 1, 'bulldog': 2, 'provoke': 2, 'towers': 1, 'rayden': 2, 'hughley': 3, 'cesar': 2, 'rawness': 1, 'conference': 4, 'brothel': 2, 'star-struck': 1, 'meaningless': 13, "raja's": 1, 'exterminators': 1, 'guru': 5, 'pissant': 1, 'grounds': 6, 'smattering': 3, 'schizophrenia': 2, 'reacting': 5, 'assimilate': 1, 'frears': 1, 'breakdown--that': 1, 'reiterate': 1, 'curtailed': 1, 'vengeance': 9, 'creator': 9, 'millers': 2, 'snoop': 3, 'break-ups': 1, 'subtleties': 2, '_long_': 1, 'preference': 1, 'humourous': 2, 'boyz': 1, 'noise': 8, 'succeed': 11, "webb's": 1, 'jolly': 2, 'awaited': 3, 'scriptures': 1, 'sampson': 1, "'video": 1, 'lt': 4, 'pungent': 2, 'ashore': 1, 'horesback': 1, 'coal': 1, 'knees': 2, 'hack': 11, 'command-line': 1, 'piloted': 1, 'hav': 1, 'stops': 21, 'drive-thru': 1, 'hallucination': 2, 'encouragement': 1, 'pornography': 8, 'brotherly': 1, 'manson-type': 1, 'reveals': 19, "spade's": 1, 'pharmaceuticals': 1, 'antagonists': 3, 'phoned-in': 1, "biel's": 1, 'signals': 3, 'sacred': 2, 'hijinks-addled': 1, 'dashboard': 1, 'lenses': 5, 'soldier-type': 1, 'inopportune': 1, 'environment': 15, 'slated': 3, '23rd': 1, "herzfeld's": 1, 'schedule': 2, 'killing': 61, 'consensus': 1, 'anniversary': 9, 'balancing': 1, 'bookish': 1, 'sleeve': 4, 'character-based': 1, 'frayn': 1, 'forget': 48, 'pushy': 4, 'tremendously': 3, 'gosnell': 3, 'snapshots': 1, 'athena': 1, 'kings': 2, 'kidman': 2, 'klingon': 3, 'pacula': 3, "glazer's": 1, "sammy's": 2, 'reno': 11, 'antler': 1, 'index': 2, 'royalty': 3, 'yoram': 1, 'half-million': 1, 'non-satirical': 1, 'clarinet-playing': 1, 'rollin': 1, 'twister': 7, 'sex-talk': 1, 'marv': 1, 'showcased': 2, 'prompted': 3, 'arrival': 13, 'korean': 2, 'presentations': 1, "degeneres'": 1, 'kit-kat': 1, 'quote': 11, 'wonders': 16, 'barbell': 1, 'mentioning': 10, 'suspense-thriller': 1, "disagree'": 1, 'stoltz': 3, 'post-operative': 1, '2020': 3, 'jaw-droppingly': 6, 'gleefully': 5, 'excesses': 2, 'substitutions': 1, 'pepsi': 2, 'bid': 3, 'unexistent': 1, 'rotating': 3, 'hardened': 1, 'kickboxing': 2, 'jack-in-the-box': 1, 'bra': 4, 'weir': 3, 'psychedelic': 1, 'diabolical': 10, 'nuclear': 16, 'corresponds': 1, 'dozens': 18, 'underworld': 10, 'thousands': 13, 'professes': 2, 'institutionalized': 1, 'eligibility': 1, "canada's": 1, 'mull': 1, 'sitters': 1, 'anthropology': 2, 'it--red': 1, 'overly-chatty': 1, 'thewlis': 4, "january's": 1, 'poised': 1, 'leblanc': 1, 'vocabularies': 1, 'skiing': 1, '`little': 1, 'soul-transferring': 1, 'armand': 3, 'daily': 7, 'thespians': 2, 'easy': 71, 'bloody': 30, 'enlightened': 2, 'saturday': 30, 'pigs': 4, 'rants': 2, 'giovanni': 5, "kid's": 5, 'spree': 3, 'big-budget': 12, 'tart': 4, 'joined': 12, 'counted': 8, 'wildlife': 3, 'snot': 1, "thulin's": 1, 'lo': 4, '30-40': 1, 'trickle': 1, 'shortly': 8, 'bluntly': 4, 'ancestor': 1, 'like-minded': 1, 'fix': 11, 'hip-hop-talking': 1, "beast's": 2, 'usurped': 1, 'artifacts': 2, 'sites': 2, 'restored': 4, 'delicatessen': 1, 'issued': 1, 'on-the-sidelines': 1, 'observant': 2, 'abuse': 12, 'hatch': 4, 'grins': 3, 'unrequited': 2, 'conveying': 5, "'staple": 1, 'remedial': 2, 'weepie-wannabe': 1, 'hitler': 4, 'obsession': 15, 'poetic': 4, 'overly-sensitive': 1, 'position': 26, 'almasy': 1, 'ramsey': 5, 'suckered': 4, 'query': 1, 'buzzwords': 1, 'torturously': 1, 'information': 34, 'munro': 1, 'limp': 8, 'regions': 2, 'batman': 43, 'biology': 1, 'beta-testing-cum-teaser': 1, 'james': 125, 'vanished': 5, 'insidious': 3, 'sleaziness': 1, 'crowe': 3, 'gangsta': 1, "heston's": 3, 'hose': 5, '19th': 5, 'tires': 1, 'meaty': 3, 'dryland': 1, 'troubling': 4, '-reviewed': 1, 'ping-pong': 1, 'fine': 94, 'besvater': 1, 'dvd': 11, '20-year-old': 2, 'workers': 6, '_': 2, 'half-formed': 1, 'alas': 33, 'answered': 10, 'mara': 1, 'western': 16, 'twist': 40, 'betsy': 2, 'catastrophic': 1, 'valuables': 1, 'pitt-esque': 1, 'unemployed': 3, 'ceaseless': 1, 'mandy': 2, "jordan's": 1, 'stag': 2, 'cicely': 1, 'velicorapters': 1, 'paying': 17, 'spectacular--almost': 1, 'conveys': 1, 'parachute': 2, 'accidentlly': 1, 'revolving': 5, "sue's": 1, 'schoolboy': 1, 'dicaprio': 7, 'dugan': 3, 'fortenberry': 1, 'therapeutic': 1, 'boxers': 1, 'fouled': 1, 'final': 114, 'outrageous': 18, 'african-american': 7, "finn's": 1, 'hackles': 1, "son's": 2, 'slammer': 1, 'mcnaughton': 3, 'hamburg': 1, 'oil': 8, 'brooks': 12, 'ripped': 13, 'lenient': 1, 'upstage': 1, 'anatomy': 3, 'h20': 6, 'performers': 24, 'vision-impaired': 1, 'stallion': 1, 'italy': 3, 'goggins': 2, 'buddy/cop/drug/sexy-witness': 1, 'triad': 1, 'bridgette': 3, 'pet': 15, 'overrated': 8, 'pubescent': 1, 'elevated': 2, 'kathryn': 2, 'burlier': 1, 'heston': 10, 'tidbits': 4, 'afternoons': 1, '1-2-': 1, 'three-year-olds': 1, 'melee': 1, 'heavily': 34, 'kell': 1, 'conceivable': 2, 'well-noted': 1, 'pisses': 2, 'identify': 9, 'laugh-out-loud': 3, 'feral': 2, 'diving': 1, 'diego': 1, 'allyson': 1, 'signs': 27, 'kiss-ups': 1, 'done-that': 1, 'iffiness': 1, 'greenwood': 5, 'enemies': 13, "cruncher'": 1, 'micromanaged': 1, 'media': 24, 'retro-comedy': 1, 'caravan': 1, "sidney's": 2, 'funhouse': 1, 'smithee': 4, 'broeck': 1, 'fatass': 1, 'dry-cleaned': 1, 'locked': 6, 'grinch': 1, 'standout': 7, 'glowing': 6, 'nigh-invulnerable': 1, 'afield': 2, 'hoodlum': 3, 'inlists': 1, 'generic': 28, 'proceeds': 13, 'gough': 4, 'dispatch': 2, 'meteorite': 2, 'helicopter': 4, 'outgrown': 2, 'drawls': 1, 'garbage': 29, 'warped': 6, 'baffles': 2, 'mini-wonderland': 1, 'not-so-innocent': 1, 'lonely': 12, 'prisons': 2, 'beasts': 1, 'assisted': 2, 'entitle': 1, 'pnly': 1, 'terence': 3, 'big-busted': 1, 'production': 89, "'comical'": 1, 'inish': 1, 'cupid': 1, 'proponents': 1, 'neuroses': 1, 'gunplay': 1, 'serbedzija': 1, 'hires': 8, 'deservedly': 2, 'responses': 4, 'jailbird': 1, 'faceless': 4, 'sought': 2, 'mediocrity': 6, 'spatula': 1, 'duckling': 1, 'sequels': 18, 'surmise': 1, 'opaque': 4, 'pops': 11, 'diamonds': 3, "matalin's": 1, 'gay': 24, 'mother-who-loves-you': 1, 'opposites': 1, 'noticeable': 11, "rem's": 1, 'downtime': 1, 'fm': 1, 'fashioned': 1, 'granite': 2, 'chum': 3, '43-foot-long': 1, 'ryan': 30, 'spawned': 6, 'maybury': 1, 'necroplasmic': 1, 'festering': 2, 'thumb-sucking': 1, 'self-indulged': 1, 'kerry': 2, 'powerhouse': 1, 'albertson': 1, 'breeding': 4, 'hemp': 2, 'henslowe': 1, 'tv-movie': 2, 'providing': 15, 'threatens': 9, 'ruining': 3, 'pilgrim': 3, 'conversion': 2, 'rote': 2, 'arne': 1, 'non-involving': 1, 'dub': 1, "nick's": 3, 'closer': 18, 'bluesman': 1, 'claustrophobic': 4, 'chuckle-out-loud': 1, 'precinct': 1, 'gays': 1, 'variants': 1, 'laxity': 1, 'ulrich': 3, 'explicitly': 3, 'old-time': 1, 'harries': 1, 'punching': 3, 'charge': 22, 'engagement': 5, 'sanity': 3, 'only-in-the-movies': 1, 'rushes': 4, 'beef': 2, 'shades': 8, "it'": 3, 'braeden': 1, 'hint': 27, 'crazymadinlove': 1, 'busses': 1, 'reduncancy': 1, 'ex-journalist': 1, 'lid': 2, 'bleached': 1, 'granddaddy': 2, 'futuristic': 7, 'movie-induced': 1, 'melissa': 5, 'reccover': 1, '5th': 1, 'conniving': 7, 'over-acts': 1, 'barracking': 1, 'saffron': 2, 'loud-mouth': 1, 'infertile': 1, 'film': 755, 'overbearing': 13, 'dinosaur': 5, 'awakens': 3, 'door': 47, 'rodgers': 3, 'humanize': 1, 'betting': 6, 'sony': 4, 'myriad': 5, 'graduates': 4, 'metro': 5, 'behaviors': 1, 'environs': 1, 'deformation': 1, 'applies': 6, 'courtesans': 1, 'ex-lover': 2, 'wearer': 1, 'stretch': 19, 'pertain': 1, 'near-unintelligble': 1, 'blustered': 1, 'holds': 22, 'dimitri': 1, 'competitiveness': 1, 'vamps': 4, 'laserdiscs': 1, 'heavily-armed': 1, 'dick': 10, 'agnieszka': 1, 'feature-length': 8, 'gordie': 3, 'continuously': 6, 'roche': 2, 'ledges': 1, 'big-city': 1, 'purposes': 4, 'stampede': 1, 'pious': 2, 'returing': 1, 'silverstein': 1, 'quirky-but-realistic': 1, 'non-friendly': 1, 'positioned': 2, 'brits': 1, 'rulebook': 1, 'sang': 2, 'place': 192, 'gloomy': 7, 'winces': 1, 'veterinary': 1, 'near-riot': 1, 'fixedly': 1, 'recommendation': 9, 'application': 1, 'lydia': 1, 'happenings': 2, 'donovan': 2, 'detailing': 6, 'run-ins': 2, 'spoof': 15, 're-election': 2, "'surprise'": 1, 'awesomely': 2, 'sked': 1, "u'tinni": 1, 'kathleen': 13, 'laclos': 1, 'millionaire': 13, 'jesse': 7, 'taxis': 1, '_blade_': 1, 'sounded': 9, 'vulgar': 5, 'plagued': 9, 'scar': 2, 'court-ordered': 1, 'workmanlike': 4, 'beelzebub': 1, 'made': 331, 'flowing': 8, 'penitentiary': 1, 'fateful': 3, 'oddly': 26, 'farting': 3, 'versa': 2, 'lillard': 5, 'intellectual': 12, 'apprehend': 2, 'woodland': 1, 'moniker': 6, 'unwisely': 1, "boston's": 1, 'val': 11, 'setup': 13, 'storywise': 1, 'lloyd': 12, 'prurient': 1, "uecker's": 1, 'fluently': 2, 'feed': 3, 'interred': 1, 'lillianna': 1, 'canyon': 4, 'parties': 7, 'character': 445, 'roberts--': 1, 'anti-glory': 1, 'fraternity': 1, 'pureed': 1, 'watson': 7, 'tent': 1, '_the_': 2, 'applied': 3, 'addams': 2, 'cardboard-cutout': 1, 'rococo': 1, 'buddy-buddy': 1, 'tightly-edited': 1, 'theme': 37, 'sheer': 24, 'garish': 6, 'explorer/mariner': 1, 'wrath': 2, 'flutters': 1, 'exclaims': 4, 'superb': 18, 'cilvilization': 1, 'essentially': 21, 'strictly-by-the-numbers': 1, 'primate': 2, 'burne': 1, 'kill': 112, 'chernobyl': 4, 'encompass': 1, 'glancing': 4, "rumble'": 1, 'seemingly': 44, '2%': 1, 'pop': 35, 'bewildering': 1, 'fire--': 1, 'poke': 5, 'bugged': 2, 'mad-dog': 1, '`sleepless': 1, 'reprises': 5, 'persuasions': 1, 'blond-haired': 1, 'potters': 1, 'veranda': 1, 'engineering': 1, 'brokering': 1, 'caribbean': 1, 'excused': 2, 'whopping': 1, 'filmdom': 1, 'teen-age': 2, 'jewison': 1, 'dissipating': 1, 'unbeknownst': 5, 'celebrity-type': 1, 'programmer': 2, 'spoil': 3, 'duplicates': 1, 'captivation': 1, 'terminology': 2, 'raises': 7, 'bavarian': 1, 'kwouk': 1, 'personas': 2, 'salwen': 1, '1978': 6, "tsui's": 1, 'relevent': 1, 'self-effacing': 1, 'murders': 21, 'flag': 3, 'computers': 2, 'missouri': 5, 'washington': 15, 'rudolph': 3, 'show': 208, 'dwayne': 3, 'breathtaking': 3, 'corners': 5, 'rehired': 2, 'teller': 1, 'troupe': 3, '2018': 1, 'heart-tugging': 1, 'armin': 3, 'fizzling': 1, 'pervasive': 1, 'worthy': 21, 'relevant': 7, 'honestly': 19, "karloff's": 1, 'philosopher': 1, 'warlock': 2, 'kick-boxing': 1, 'anjelica': 2, 'groups': 10, 'existent': 1, '0': 19, 'panther': 2, 'silence': 18, 'altruist': 1, 'course-a': 1, 'spits': 3, 'reminscent': 1, 'structural': 1, 'wackier': 1, 'tails': 2, 'associates': 4, 'bookends': 1, 'tribal': 3, "kimball's": 1, 'grimm': 1, 'rendition': 7, 'smut': 2, 'doohan': 1, 'performace': 1, 'leeches': 2, 'pretended': 1, 'exacted': 1, 'dishes': 2, 'ride': 41, 'dirtier': 1, 'near-indestructible': 1, 'blowjob': 1, "rosemary's": 4, 'blends': 2, 'chromium': 1, 'succumbs': 3, 'stewards': 1, 'grapple': 1, 'thorne-smith': 1, 'pill': 4, 'fair-skinned': 1, 'intensity': 14, 'technologically': 2, 'deadbeat': 1, 'non-intrusive': 1, 'rivalry': 3, 'berkeley': 1, 'buy': 31, 'tristar': 2, 'no-where': 1, 'pressing': 2, 'wald': 1, "body's": 1, 'banter': 6, '`10': 1, 'cranberries': 1, 'horror-comedy': 2, "julia's": 3, 'kinship': 1, 'science': 53, 'claustral': 1, 'towering': 5, 'parent-child': 1, "films'": 1, 'wheel': 10, 'lean': 4, 'sitcom-level': 1, 'parrish': 1, 're-visit': 1, 'blaring': 3, 'over-used': 3, 'furrowed': 1, 'ana': 1, 'sister': 32, 'eye-popper': 1, 'slug-like': 1, 'denise': 10, 'youth': 11, 'characterization': 22, 'americana': 1, 'jumps': 20, 'sponsor': 1, 'octane': 1, 'forward': 37, 'layered': 1, 'widely': 3, '_wayyyyy_': 1, 'toes': 3, 'huckster': 1, 'coffey': 1, 'tattered': 1, 'shariff': 1, 'charleville': 1, 'boy-drinks-entire-bottle-of-shampoo-and-may-or-may-not-get-girl-back': 1, 'ultra-secret': 1, 'delights': 2, "kieslowski's": 1, 'addition': 43, 'complexity': 8, 'railing': 1, 'wife-to-be': 1, "region's": 1, 'enters': 8, 'humdrum': 3, "primate's": 3, 'fakeouts': 1, 'goofy/embarrassing': 1, 'fishburne': 5, "actresses'": 1, 'designs': 4, 'friendliest': 1, 'gamely': 1, 'artsy-fartsy': 1, 'freshener': 1, 'descriptions': 4, "zeta-jones's": 1, 'macdonald': 7, 'recalls': 2, 'shoddier': 1, 'klux': 1, 'cancerogenic': 1, 'portable': 1, 'backdrops': 2, 'anticlimatic': 1, 'drabby': 2, 'fluidity': 1, 'succinct': 1, 'featurette': 1, 'blanks': 3, 'count': 41, 'hedonism': 1, 'hurricane': 5, 'throwaways': 1, 'sacks': 2, 'nielson': 2, 'bookstore': 4, 'ice': 25, 'man-made': 1, 'obssessive': 1, 'bells': 3, "arlene's": 1, 'realizing': 18, 'in-jokes': 6, 'newly-formed': 1, 'narrow': 2, 'balm': 1, 'choked': 2, 'wide-release': 1, 'puny': 2, 'secrecy': 3, 'reshoots': 4, 'tale--the': 1, 'min': 3, 'condemnation': 4, 'con-team': 1, 'fan-base': 1, 'dunois': 1, 'choices': 18, 'incidents': 2, 'chlo': 2, 'wont': 2, 'emphasized': 1, 'lurks': 1, 'autobiographical': 3, 'poorly-written': 1, "wc's": 1, 'nevada': 2, 'fathered': 1, 'undoings': 1, 'al-l-l-l-l-l-l-l-l-l-lrighty': 1, 'stutter': 1, 're-adaptation': 1, 'gusto': 5, 'spiral': 5, 'immaturity': 1, 'nelson': 7, 'twisting': 5, 'acerbic': 4, 'mutant': 6, 'errs': 1, 'dennehy': 1, 'cocky': 6, 'jermaine': 1, 'almost-subliminal': 1, 'rising-star': 1, 'unstable': 7, 'theo': 2, 'gruesomely': 4, 'homes': 7, 'highpoint': 1, 'followup-to': 1, 'utopian': 1, 'dentist': 3, 'helm': 9, 'hulke': 1, 'bumped': 5, 'clamoring': 1, 'harnessed': 2, "ferrell's": 1, "90210'": 1, 'carlisle': 1, 'rawls': 1, 'strangely': 19, "doer'": 1, 'grisham-like': 1, 'ill': 16, 'sicker': 1, 'upn': 3, 'interviews': 12, 'extinction': 1, 'constipation': 1, "marty's": 2, 'inserted': 4, 'softened': 1, 'dino': 6, 'artists': 7, 'shirishama': 1, 'amused': 4, 'captor': 2, 'discusses': 1, "lord's": 1, 'uncredited': 2, 'fail': 33, 'contender': 3, 'pavement': 1, 'evince': 2, 'unsupportive': 1, 'leader': 36, 'laughably': 17, 'irreversible': 2, 'treks': 1, 'insultingly': 3, "issac's": 2, 'traitor': 1, 'mindset': 6, 'reprised': 2, 'revulsed': 1, 'visual': 46, 'regrets': 2, 'marley': 3, 'dwell': 3, 'ceaselessly': 1, 'producer-director': 1, 'menace': 15, 'sequoia': 1, 'lint': 1, 'praises': 1, 'phrasing-too': 1, "nothin'": 2, 'hulking': 2, 'enriching': 1, 'gleeful': 1, 'bio-mechanical': 1, 'suspects': 19, 'predictable': 89, 'balaban': 2, 'womanized': 1, 'ope': 2, 'awaiting': 6, "'number": 1, 'steady': 4, '`i': 3, 'draconian': 1, 'triple': 6, 'surges': 2, 'painterly': 1, 'cartman': 1, '-like': 2, 'contradictions': 3, 'cherry-colored': 1, 'simplicities': 1, 'irritate': 2, 'cartwright': 2, 'someday': 5, 'informants': 1, 'jumpy': 1, 'managers': 1, 'klan': 1, 'quiz': 2, 'contortions': 1, 'horrendous': 8, 'boyd': 2, 'translucent': 1, 'zoot': 2, 'agreement': 2, 'nubile': 3, 'derive': 5, "`what-ever-happened-to'": 1, 'unbelieveable': 1, 'premere': 1, 'broder': 1, 'wielders': 1, 'delicate': 2, 'pleads': 2, 'berlinger': 1, "fulci'": 1, 'settles': 5, "vicky's": 1, 'wondered': 11, 'bite': 18, 'doubts': 2, 'march': 7, 'horrifying': 7, 'trafficking': 2, 'starring': 72, 'sodomy': 2, 'revolution': 3, 'dillon': 5, 'subculture': 1, 'eerie': 10, 'simpson': 5, 'miller': 16, 'shay': 1, 'powerful': 45, 'kinney': 1, '133': 1, "`m2m'": 1, 'veracity': 1, "charlotte's": 1, 'washed-out': 1, 'plugs': 4, 'tangential': 2, 'holograms': 1, 'janice': 1, 'happily': 18, 'freezes': 2, 'reluctant': 10, 'pelvic': 1, 'not-quite-kosher': 1, 'ku': 1, 'nehru': 1, 'and--': 1, 'lebrock': 1, "'most": 1, 'home': 165, 'asswhole': 2, 'rashomon': 2, '03': 1, 'hallucinatory': 2, 'filth': 3, "holm's": 1, 'strains': 5, 'bleachers': 2, 'evade': 2, "megan's": 1, 'workable': 1, 'forensic': 1, "falzone's": 1, 'topped': 1, 'perado': 1, 'wedding-needing': 1, 'truncated': 1, 'tim': 42, 'emerges': 4, 'genovia': 1, 'gown': 1, 'opts': 4, 'slob': 5, 'barrage': 11, 'supposed': 163, 'birdhouse': 2, 'thinly': 2, 'lovemaking': 4, 'augustus': 1, 'trainspotting': 1, 'caterpillar': 1, 'favours': 1, 'rubber': 10, 'lord': 17, 'teen-orientated': 1, 'annoyance': 3, 'sun-times': 1, 'tote': 1, 'raoul': 3, 'levine': 4, 'offbeat': 3, 'mutated': 6, 'narrow-sighted': 1, 'hanks': 5, "freeze's": 3, 'debello': 1, 'tormented': 5, 'naunton': 1, 'flooding': 2, 'michele': 1, 'convincingly': 11, 'sprouse': 3, 'perimeter': 1, 'foiling': 1, 'bernstein': 2, 'vocals': 1, 'pounced': 1, "toho's": 1, 'bankable': 1, 'validating': 1, 'organisations': 2, "kinnear's": 2, "nazi's": 1, 'ogling': 1, 'seekers': 1, 'lays': 7, 'abs': 2, "chad'z": 1, 'macmurray': 1, 'puritan': 1, 'felt': 61, 'ark': 7, 'shlock': 2, 'reigns': 1, 'naomi': 1, 'goodly': 1, 'harrowing': 5, 'elitist': 2, 'rip': 12, 'hyped': 5, 'chipmunk': 1, 'lunacy': 4, 'entourage': 1, 'tunney': 6, 'backgrounds': 2, 'eagerly-awaited': 1, 'lerner': 2, 'two-way': 1, 'pro': 7, 'stomping': 3, 'lineage': 1, 'internally': 1, 'blare': 1, 'cyberpunk': 1, 'ditty': 2, 'dann': 1, 'prakazrel': 1, 'practise': 1, 'demonstrators': 1, 'warden': 10, 'winks': 2, 'endanger': 2, 'diane': 11, 'lecter': 1, 'cleverly': 3, 'excrements': 1, 'buddy-cop': 1, 'lavishly-decorated': 1, 'sprinting': 1, 'ejogo': 1, "character's": 35, 'terel': 1, 'devirginized': 1, 'tweaked': 1, 'painful': 38, 'canal': 2, 'usefulness': 1, 'suspended': 3, 'smashed': 3, 'trivia--ironically': 1, 'exploiting': 4, 'starkness': 1, 'identification': 3, 'gracefully': 3, 'infinite': 4, 'stints': 1, 'inspection': 1, 'examiner': 1, 'debate': 3, 'straight-arrow': 1, 'blamed': 2, 'frontal': 3, 'positing': 1, 'f-u-n': 1, 'ville': 4, 'fabricates': 1, 'usher': 2, 'talks': 24, 'oregon': 1, 'plaza': 1, 'over-hyped': 1, 'expand': 5, "denise's": 1, 'rhapsody': 1, 'handles': 9, 'idea---a': 1, 'aftermath': 3, "frighteners'": 1, "wexler's": 1, 'parenthood': 1, 'riddled': 2, 'smirking': 2, 'unexciting': 8, 'dickerson': 1, "'cept": 1, 'zamora': 1, 'jaunty': 1, 'conaway': 1, 'cinemas': 5, 'asinine': 4, 'mid-august': 1, 'justifies': 1, 'tussle': 1, 'glances': 5, 'playbook': 1, 'cloaks': 1, 'kelly': 24, 'scrappy': 2, 'fulfilled': 2, 'philly': 1, 'corresponded': 1, 'hypothesis': 1, 'mosh': 1, 'paint': 11, 'raiders': 7, 'outposts': 1, '`parole': 1, 'twitchy': 1, 'carriage': 1, 'drug-related': 1, 'reeve': 1, 'atrice': 1, 'organisms': 1, 'marching/singing': 1, 'claim': 9, 'lawnmower': 5, 'c+': 1, 'napkin': 1, 'uplifted': 1, 'fire': 38, 'oh-so-ironic': 1, 'spouting': 11, 'wave': 15, 'handphone': 1, 'shared': 7, "perry's": 4, 'atrocities': 2, 'hargrove': 1, 'all-of-a-sudden-superhuman': 1, 'verge': 5, "nasa's": 1, "o'neill": 1, 'kari': 3, 'well-plotted': 1, 'overdress': 1, 'stepdaughter': 4, 'faster': 10, 'sporadic': 2, 'goalie-mask-wearing': 1, 'humiliation': 2, 'rocky-style': 2, "crow's": 1, 'harder': 14, 'strategies': 2, 'beau': 2, 'rearing': 1, 'screamed': 2, 'sympathize': 9, 'thaddeus': 2, 'life-saving': 1, 'legree': 1, 'braveheart': 7, "breeder's": 1, 'widescreen': 2, 'businesswoman': 2, 'hartnett': 3, 'sweat-literally': 1, 'plot-': 1, 'pitching': 2, 'stutters': 1, 'conveyed': 3, 'excitment': 1, 'humorless': 4, 'drop-out': 1, 'reacquaint': 1, 'foregin': 1, 'rappers': 1, 'collar': 4, 'eruting': 1, 'funds': 7, 'diaries': 4, 'bufford': 1, 'seniors': 3, 'valmont': 1, 'undoes': 1, 'leguizimo': 1, 'godzillas': 3, 'reimagines': 1, 'unforgiven': 1, 'exit': 9, 'ouch': 2, 'role-playing': 1, "valek's": 1, 'teach': 11, 'loxley': 1, 'incantation': 1, 'photographic': 2, 'rehab': 3, 'combative': 2, 'critiques': 1, "bush's": 1, 'peerless': 1, 'ever-bemused': 1, 'hardy-har-har': 1, 'evactuate': 1, 'sportscar': 1, 'notables': 1, 'topol': 1, 'vigor': 2, 'county': 4, '11th-hour': 2, 'learned': 21, 'kod': 1, 'owner': 37, "freeman's": 1, 'glen/glenda': 1, 'coworkers': 2, 'mcknight': 1, 'ignorance': 4, 'bombard': 1, 'framed': 18, 'kimsey': 1, 'plaudits': 1, 'argonautica': 2, 'tarzan': 5, 'antagonism': 2, 'undisputed': 2, 'paucity': 1, 'drafts': 1, '[fill': 1, 'cro-magnon': 1, 'slithers': 2, 'receptionist': 1, 'squeals': 2, 'crewmate': 1, 'nova': 1, 'instant': 7, 'professionals-miramax': 1, 'bluster': 1, 'rosewood': 1, 'shwarzenegger': 1, 'survived': 6, 'dissatisfaction': 3, 'thumb': 5, 'government': 36, 'imparts': 1, 'jacob': 2, 'coven': 2, 'degradation': 1, 'demanded': 5, 'bored': 44, 'furrier': 1, 'partyers': 1, 'nigga': 1, 'sinking': 10, 'ruptured': 1, 'overworked': 3, 'blooded': 1, 'dimwitted': 4, 'manero': 1, 'ving': 4, 'non-webb': 1, 'elaborately': 1, 'encountering': 1, 'outshines': 1, 'imperfect': 1, 'contracted': 2, '_wishmaster_': 1, 'exudes': 2, 'eyeballed': 1, 'countless': 27, 'colossal': 5, 'paean': 1, 'co-writer/director': 2, 'dewy': 1, 'novotny': 1, 'detestable': 3, 'lottery': 4, 'gargling': 1, 'mini-me': 1, 'agencies': 1, "airplane's": 1, 'karyo': 4, 'inappropriately': 1, 'make-believe': 1, 'endowed': 3, 'puffy': 2, 'resentful': 1, 'tainer': 1, 'longtime': 2, 'celibacy': 2, 'installations': 1, 'clique': 4, 'solemn': 1, 'adulthood': 1, 'stranded': 11, "mo'nique": 1, 'models': 7, 'verbalized': 1, 'less-than-honorable': 1, 'exuding': 2, 'casting': 49, 'dismally': 1, 'computer-masked': 1, 'coax': 2, 'eastwood-esque': 1, 'tour': 9, 'legitimacy': 1, '______': 1, 'smugness': 1, 'spooking': 1, 'genders': 1, "daniel's": 2, 'sergeants': 1, '`macho': 1, 'frequency': 2, 'seldom': 2, 'rachel': 13, 'glowering': 2, 'fetishizes': 1, 'amazing': 46, 'typical': 50, 'thumbs': 10, 'semi-clothed': 1, 'businessman': 9, 'nerdies': 1, 'blurts': 3, 'allan': 4, 'alarm': 4, 'ski-doos': 1, 'macbeth': 1, 'cutsie': 1, 'irrepressible': 2, "doyle's": 2, 'trysts': 2, 'james-younger': 1, 'pick-up': 2, 'breed': 7, 'lengths': 6, 'lonliness': 1, "woody's": 1, 'upmost': 1, 'mis-steps': 1, 'override': 2, 'genuinely-funny': 1, 'macht': 2, 'abolish': 1, 'independently': 1, 'impersonations': 1, 'mcracken': 1, 'filter': 3, 'lead': 99, 'downright': 20, 'fanatically': 1, '`sand': 1, 'medically': 1, 'transvestite': 2, 'shown': 48, 'magnetism': 2, 'woolworth': 1, '11-story': 1, 'two-dimensional': 3, 'siff': 1, 'petitions': 1, 'alien': 51, 'weirdo': 1, 'readily': 7, 'cheadle': 9, 'witless': 1, 'khachaturian': 1, 'rogue': 5, "finder's": 1, 'wando': 1, "gilligan's": 2, 'joiner': 1, 'portals': 2, '103-minute': 1, 'vendor': 1, 'rekindling': 1, 'self-parody': 3, 'reprisal': 1, 'lords': 1, 'girlina': 1, 'lina': 2, 'progressively': 5, 'beer-drinking': 1, 'fun': 171, 'pip': 1, "douglas's": 1, 'ecstatic': 2, 'melbourne': 2, 'sisterhood': 2, 'rec': 1, 'smirks': 2, 'container': 1, "florida's": 1, 'sexiest': 1, 'weighted': 1, 'fall': 93, 'requiem': 1, "reporter's": 1, '18th': 2, 'glorification': 1, 'hercules': 4, 'macomber': 1, 'hostage': 14, 'justify': 14, 'april': 5, 'permeated': 3, 'communist': 2, 'hanna': 1, 'alienate': 2, 'sound-alike': 1, 'boot': 13, 'parenting': 3, 'eisenberg': 2, 'steamy': 2, 'promo': 2, 'brattish': 1, 'lsd': 1, 'immersion': 1, 'highway': 10, 'rift': 1, 'high': 146, 'nervous-wreck': 1, 'sick': 29, 'averting': 1, 'charter': 2, 'discontents': 1, '_they': 1, 'docile': 1, 'jenkins': 1, '`unsung': 1, 'wide-open': 1, 'plight': 7, 'mccardie': 1, 'riffing': 3, "bumpy's": 1, 'dov': 1, "y'know": 1, "sadist's": 1, 'ghidrah': 1, 'flows': 4, 'voluptuous': 2, 'glenn': 15, 'unwitting': 1, '126': 1, 'adept': 4, 'chunks': 5, 'wrapping': 2, 'nirvana': 1, 'timidity': 1, 'jupiter': 1, 'chapin': 1, 'kills': 38, '14-year-old': 2, 'guiding': 5, '89': 1, 'begs': 7, "lundgren's": 1, 'helluva': 1, 'magyuver': 1, 'grumpy': 4, 'exhibit': 8, 'specialised': 2, 'digimon': 1, 'firefight': 1, 'skulduggery': 1, 'realizations': 1, 'matarazzo': 1, 'control': 58, 'philandering': 2, "ira's": 1, 'topic': 16, 'condescending': 5, 'polynesian': 2, 'gales': 1, 'commericials': 1, 'winters': 1, 'well-crafted': 2, 'watchful': 2, 'lip': 2, 'underwear': 3, 'brash': 4, '48': 2, "et'": 1, 'idolizes': 3, 'stampeding': 1, 'gimmicky': 4, 'jean-luc': 3, "'shouting": 1, 'guilt': 7, 'marshals': 3, "'cause": 5, 'servants': 3, 'homeys': 1, 'keyhole': 1, 'vaguely': 13, 'disjointed': 5, 'abusing': 1, 'apprehension': 1, 'breakthrough': 1, 'barter': 1, 'jeopardy': 8, 'radioactivity': 1, 'meddled': 1, 'abducted': 1, 'distinct': 9, '8th': 2, 'blue': 35, 'odereick': 1, 'rooms': 13, 'miss-directed': 1, 'loutish': 2, 'loveless': 5, 'rutheford': 1, 'complication': 2, 'praised': 7, 'laden': 3, 'explantion': 1, 'oderkerk': 1, "christina's": 1, 'self-evidently': 1, 'barred': 2, 'lorne': 1, 'sendoff': 1, 'zombie-stomping': 1, 'scale': 17, 'embedded': 2, "winston's": 1, 'products': 12, 'extras': 10, 'prefered': 1, 'preferable': 1, 'gillian': 2, 'cohorts': 3, 'tie-in': 1, 'one-upmanship': 2, 'turi': 1, 'multitude': 4, 'protagonists': 20, 'telecommunicative': 1, 'liars': 1, 'red-lit': 1, 'philip': 8, 'chicago-connected': 1, 'strangers': 7, 'pectorals': 1, 'ryder': 3, 'decoding': 1, "rex's": 1, 'rampling': 1, 'jiggling': 1, 'afore-mentioned': 2, 'dalmantions': 1, 'disappointed': 20, 'fuss': 4, 'wake': 15, 'inevitably': 18, 'outwitting': 1, 'suspicions': 2, 'ground--all': 1, 'gullible': 2, 'informing': 1, 'aired': 2, 'inspiring': 6, 'curious': 14, 'playwright': 2, 'terrace': 1, "'90s": 3, 'matewan': 1, 'jolie': 12, 'merchants': 1, 'reverts': 1, 'anti-establishment': 1, 'mountain/hill': 1, 'actualizing': 1, 'succinctly': 2, "wayne's": 1, 'travels': 17, 'lick': 3, 'before--sometimes': 1, 'luchino': 1, 'thrills': 24, 'mcmahon': 1, 'pointless': 64, 'cholera': 1, "'baby": 1, 'doo': 3, 'natacha': 1, 'montages': 2, 'frown': 3, 'infiltrate': 2, 'wow': 19, 'short': 108, 'chop': 6, 'diner': 6, 'kitsch': 2, 'ascended': 1, 'countryman': 1, 'hurry-up-and-wait': 1, 'telegraphs': 1, 'twenty-minute': 1, 'once-in-a-lifetime': 1, 'cathy': 1, 'illeana': 3, 'concocts': 1, 'poseidon': 2, 'humanoid': 1, 'cheat': 1, 'sleeplessness': 1, 'rampant': 8, 'cars': 22, 'potboiler': 3, "faltermeyer's": 1, "her's": 1, 'betcha': 2, 'emanated': 1, 'spikes': 1, 'settled': 2, 'crotch': 5, 'recollection': 1, 'redman': 7, 'poachers': 2, 'cattrall': 1, "geyser's": 1, 'waste': 95, 'kilmer': 10, 'abode': 3, 'principals': 5, 'sources': 4, 'egg-shaped': 1, 'awash': 1, 'incorporated': 1, 'subordinate': 1, '21-year-old': 1, 'batologist': 1, 'post-cold': 2, 'tag': 8, 'surplus': 2, 'sub-par': 7, 'flick-': 1, 'petitioned': 1, 'bicker': 4, "steven's": 2, 'bargained': 3, 'whip': 5, 'bigots': 1, 'adrien/marc': 1, 'arrangement': 2, 'lasser': 1, "beholder'": 1, "d'artagnan's": 1, 'absentee': 2, 'mulcahy': 1, 'anaconda': 10, 'lies': 34, 'god-damns': 1, 'jameses': 1, 'overscore': 1, 'raptor': 1, 'hectic': 1, 'pontificate': 3, 'clones': 4, 'specimens': 1, 'crabby': 1, 'lobotomise': 1, 'cliff': 12, 'mercedes': 4, 'marines': 3, "'terrore": 1, 'blow-out': 1, 'noooo': 1, 'rourke': 3, 'dizzy': 3, '======================': 1, 'meditating': 1, 'dubbed': 11, 'invisibility': 2, 'erykah': 1, 'jewels': 1, 'fleshed-out': 1, 'skids': 1, 'semi-mystical': 1, 'pumpkins': 1, 'lam': 1, 'succeeding': 3, '--it': 1, 'gold': 26, 'prevail': 1, 'late-night': 5, 'gus': 6, 'thought-provoking': 9, 'altmanesque': 1, 'propositions': 1, 'innocently': 2, 'mistress--but': 1, 'fbi/atf': 1, 'semler': 2, 'wades': 2, 'bartenders': 1, 'swingers': 4, 'metaphor-heavy': 1, 'arrested': 8, 'hots': 2, 'defending': 3, "smith's": 8, 'gurgle': 1, 'risks': 4, 'contributing': 2, 'costing': 1, 'tv-star-to-film': 1, 'objections': 1, "fleming's": 1, 'incurably': 1, 're-capture': 1, 'exhibition': 2, 'yucks': 1, 'allayah': 1, 'impeccable': 3, 'twentieth': 4, 'general': 51, 'caddies': 1, 'salesgirl': 1, "wang's": 1, 'sarcastic': 11, 'erotica': 3, 'americanized': 1, 'woodward': 1, 'daresay': 1, "agree'": 1, 'investigators': 5, 'thumbing': 1, 'kaplan': 2, 'deserter': 1, 'edginess': 1, 'greenaway': 1, "cassavettes'": 1, 'rebelling-against-daddy': 1, 'montmartre': 1, 'slipped': 3, 'cajun': 1, "clueless's": 2, 'artemus': 6, 'catharine': 1, 'attract': 4, 'gymnasium': 1, 'dried': 1, 'rider': 3, 'policeman': 7, 'cell-phones': 1, 'deflate': 1, 'dash': 5, 'disastrous': 8, 'tame': 11, 'engulfed': 2, 'rooming': 2, 'rocket': 5, 'cameramen': 1, 'meet-cute': 1, 'alleys': 1, 'adeptness': 1, 'ground-breaking': 1, '-but': 2, 'classify': 1, 'servant': 2, 'embarrasses': 2, 'wildfire': 3, 'braces': 2, 'moore\x12s': 1, "rudd's": 1, 'bigglesworth': 1, 'ghostbuster': 1, 'specialist': 4, 'swears': 3, "lawyer's": 1, 'exploitatively': 1, 'jackhammer': 1, 'beachwalkers': 1, "health's": 1, 'adopting': 2, '1-900': 1, 'thirty-six': 1, 'asbestos': 1, 'ritual': 9, 'warm-up': 1, 'card': 16, 'co-worker': 7, "ape's": 1, 'migrating': 2, 'maleness': 1, 'mcbainbridge': 1, 'thud': 2, 'rent-a-': 1, "families'": 1, "sims'": 1, 'reptilian': 2, 'theatrics-': 1, 'molds': 1, 'long-winded': 2, 'credits--who': 1, 'poopies': 1, 'refrained': 1, 'immature': 2, 'cross-cut': 1, 'sidewinder': 1, 'too-pleasant': 2, 'one-scene': 1, 'pcu': 1, 'smiled': 3, "'problem'": 1, 'fluoro': 1, 'flogging': 1, 'noisy': 4, 'adhere': 1, 'illegally-acquired': 1, 'oh-so': 2, 'stifler': 2, 'tavern': 2, 'divide': 2, 'turmoil': 8, "chaja's": 1, 'illustrating': 1, 'close': 82, 'midair': 1, 'bitch': 9, 'ice-pick': 1, 'stiffs': 2, 'islands': 7, 'sudddenly': 1, 'plunkett': 2, 'hazards': 1, '17-year': 2, 'brooder': 1, 'controled': 1, 'behold': 7, 'kafka': 1, 'handful': 24, 'protest': 4, 'capsule': 25, 'channel': 7, 'juliette': 5, 'blitzed': 1, 'liz': 7, 'followers': 3, 'only--attempt': 1, 'upstaging': 1, 'orville': 1, 'sap': 5, 'cemetery': 5, 'coveted': 2, 'short-changed': 1, 'maud': 1, 'troyer': 1, 'blurt': 2, 'coition': 1, 'graceful': 5, 'saps': 1, 'list': 48, 'communicating': 1, 'ahhh': 1, 'nolte': 5, 'ever-versatile': 1, 'misuses': 1, 'biohazard': 1, "todd's": 2, "`braveheart'": 1, 'un-related': 1, 'norm': 11, 'greet': 3, 'hackneyed': 11, 'aggressively': 2, 'injured': 8, 'siskel/ebert': 1, 'irresistible': 9, 'primal': 3, 'seizes': 2, 'ways': 61, 'rooted': 1, '70m': 1, 'maneuvers': 1, 'overplotted': 1, 'bah': 2, 'ladyship': 1, "american'": 1, 'lemonade': 1, 'valuable': 6, 'jackal': 7, 'bystander': 1, 'clap': 2, 'sacrilege': 1, 'persist': 1, 'irreverent': 2, 'exhibiting': 2, 'skilled': 5, 'blake': 8, "atrocity's": 1, 'animatronics': 1, 'fleiss': 1, 'wrist': 2, 'louise': 3, 'referring': 7, 'xfx': 1, 'cyber-courier': 1, 'unprofessionalism': 1, 'snooty': 2, 'drier': 1, 'suitably': 9, 'uncovering': 4, 'jeez': 3, 'gateway': 1, '1940s-style': 1, 'stumbling': 6, "studio's": 3, 'incantations': 1, 'tall': 16, 'orson': 7, 'choke': 1, "gosselaar's": 1, "so-corny-it's-funny": 1, 'subversive': 4, 'flush': 1, 'slam-dunks': 1, 'lookout': 1, 'realization': 9, 'shipwrecked': 1, 'episodes': 19, 'sulking': 1, "'back": 1, 'inter-office': 2, 'jovovich': 3, 'imitates': 2, "williamson's": 4, 'unsung': 1, 'annihilate': 1, '========================': 1, 'declines': 1, 'change': 57, '`dr': 1, 'orgasmic': 1, 'brinkford': 1, 'gag': 29, 'bandwagon': 4, 'preach': 2, 'weathers': 3, 'trek': 20, 'bradley': 2, 'complaining': 7, 'long-awaited': 2, 'strategist': 1, 'division': 4, 'flower-arranging--in': 1, 'populated': 6, 'mighty': 16, 'shell': 7, 'fakery': 1, 'swashbuckling': 3, 'ruled': 6, 'oddballs': 1, 'doused': 3, 'chriqui': 1, 'dramatics': 5, "genuises'": 1, 'hen-pecked': 1, 'desouza': 1, 'abyss': 12, 'purposefully': 5, 'stymied': 1, 'mariah': 2, 'racist': 7, 'joie': 1, 'norsemen': 2, 'gauge': 7, 'ancient-swords': 1, 'self-mutilation': 1, 'booby-trapped': 1, 'forehead-slapper': 1, 'soap': 19, 'moscow': 1, 'deliverance': 5, 'emerge': 5, 'fingerprint': 1, 'unexpected': 21, 'salutes': 1, 'calculated': 6, 'horn-dogs': 1, 'goons': 7, '-based': 1, 'sofa': 1, 'stationed': 2, 'precocious': 2, 'weakest': 3, 'terrorist': 14, 'budding': 5, 'spelenetik': 1, 'wisconsin': 1, 'leapfrog': 1, 'amounted': 1, "goldsmith's": 2, 'dosage': 1, 'sludge': 3, 'gordy': 1, 'lap': 5, "rafe's": 1, 'retro-future': 1, 'arty': 3, 'radioactive': 3, 'goggles': 4, 'dreadlocks': 1, 'stomach': 14, '$40': 3, 'cynical': 14, 'ramshackle': 1, 'pad': 8, "sam's": 6, 'moss': 1, 'siegel': 1, 'blundered': 1, 'cecile': 1, 'revelation': 12, "gandolfini's": 1, 'ejection': 1, 'tacked-on': 1, 'neve': 11, 'money-grubbing': 1, 'static': 7, 'della': 2, 'fool': 9, "anybody's": 3, 'kidnappers': 2, 'gilsig': 1, 'bikini': 5, 'shaky': 8, 'imprint': 1, 'remark': 3, 'narcotics': 2, 'bitterly': 1, 'dork-chick': 1, 'claiming': 8, 'scoop': 3, 'sciora': 1, 'darkened': 2, 'limited': 27, 'composure': 1, 'sadoski': 1, 'mothers-in-law': 1, 'nien-chen': 1, 'clips': 7, 'multi-million': 2, 'prepped': 1, 'stars': 145, 'enviable': 1, 'now-uma': 1, 'facing': 10, 'hank': 19, "vampire's": 1, 'undergoing': 2, 'marketwise': 1, 'shriker': 1, 'ever-contrary': 1, 'killathon': 1, 'withhold': 1, 'clues': 7, 'critique': 24, 'shelved': 2, "even's": 1, 'day\x12s': 1, "can't-miss": 1, 'rational': 4, 'morgue': 3, 'sharon': 14, 'eh': 11, 'gal': 5, 'bushel': 1, 'silky': 1, '1960': 2, 'provide': 43, 'violin': 5, '1997--the': 1, 'centerpiece': 3, 'marvel': 8, 'weighty': 4, 'carbon-copy': 1, 'mccole': 1, 'star': 178, 'coupling': 1, 'punishment': 9, 'dame': 4, 'curtailing': 1, 'twenty': 36, 'tiffany': 2, 'with]': 1, 'faddish': 1, 'jackee': 1, 'over-played': 1, 'sprouted': 1, 'rustic': 1, 'mimms': 1, 'clothing': 10, 'resumes': 6, 'expire': 1, 'rasp': 2, 'morton': 3, 'her/and': 1, 'pigeonholed': 1, 'mid-air': 3, 'cushions': 1, 'villages': 1, 'excel': 1, 'investigates': 1, "picture's": 3, 'sandler': 12, 'twenty-eight': 2, 'ringo': 1, 'rod': 4, 'ads': 16, 'hard-': 2, 'deflection': 1, 'essential': 7, 'plays': 262, 'kelogg': 1, 'bodily': 3, 'imagined': 8, 'abundance': 4, 'ips': 1, 'graduation': 3, 'down-at-the-heels': 1, 'reliance': 5, 'moreu': 1, 'shoots': 18, 'liebes': 1, 'completes': 1, 'repercussions': 2, 'stacked': 3, 'embarrassed': 16, 'toast': 2, 'haley': 3, 'salaries': 3, 'improvise': 3, 'wrongfully': 4, 'compartmented': 1, 'decrement-life-by-90-minutes': 1, 'persists': 1, 'cheesy-looking': 1, 'actors/actresses': 1, 'zaillian': 1, 'hurting': 2, 'plaguing': 2, 'malarkey': 1, 'wc': 1, 'relics': 1, 'isolate': 3, 'mall': 3, 'compose': 2, 'bury': 8, 'misleading': 3, 'taiwanese': 3, 'phase': 5, 'propelling': 2, 'circuits': 2, 'wets': 1, 'colleges': 1, 'mementos': 1, 'zombies': 4, "postman's": 1, 'twenty-something': 1, 'thirty': 31, 'impales': 1, 'permanent': 2, 'cadillac': 1, 'enviromentally': 1, 'funnymen': 1, 'semi-thorough': 1, 'moresco': 1, 'antagonistic': 1, 'incongruity': 3, 'traditional': 11, 'stacking': 1, 'retreads': 2, 'nadja': 1, '_his_': 1, 'play-by-play': 1, 'brennick': 1, 'edward': 18, 'brashness': 2, 'observes': 1, 'director/writer/cinematographer/editor': 1, 'quell': 1, 'father': 111, "gottfried's": 1, 'stabs': 5, 'mind-meld': 1, 'chanteuse': 1, 'courtesy': 11, 'firefighters': 2, 'dreams': 39, 'languid-how': 1, 'late-seventies': 1, 'recurrent': 2, 'shock': 35, 'kirkland': 2, 'grocers': 1, 'feared': 2, 'richer': 2, 'indication': 4, 'toro': 5, 'semi-nude': 1, 'deconstructed': 1, 'digestible': 2, "children's": 15, 'wilds': 2, 'fur-coated': 1, 'promiscuous': 2, "friend's": 6, "cop's": 2, 'attendants': 1, 'darlene': 1, 'humiliations': 1, "hitler's": 1, 'skirmish': 1, 'switch': 9, 'overly': 25, 'zamm': 1, 'rely': 14, 'thrill-a-minute': 1, 'cluelessness': 3, 'noggin': 2, 'sealed': 5, 'voyeurism': 3, 'lion': 6, "mankind's": 2, 'cambridge': 1, 'shoplift': 1, 'character-driven': 1, 'sequences': 123, 'modelling': 1, 'sundowns': 1, 'farrel': 1, 'letterman': 3, 'crossing': 8, 'e-mailed': 1, 'condition': 17, 'straightest': 2, 'habitat': 1, 'unprecedented': 2, 'mohicans': 2, 'concur': 1, 'conspires': 2, 'stumped': 1, 'britain': 2, 'hailing': 1, 'diggs': 4, 'phoniness': 1, 'moderately': 10, 'real-': 1, 'metaphasic': 1, 'now-estranged': 1, 'credentials': 4, 'copies': 5, 'surroundings': 3, 'marino': 1, 'gabriel': 16, 'extemely': 2, 'mid-january': 1, 'skeleton': 3, 'steven': 47, 'abhorrent': 3, 'craven': 6, 'sabrina': 3, 'regaled': 1, 'burst': 8, 'marcie': 3, 'fatuous': 2, "verlaine's": 1, "x-files's": 1, 'english-speaking': 1, 'allen': 23, 'boils': 10, 'americanizing': 1, 'shifted': 3, 'worn-out': 4, 'integrated': 2, 'bends': 1, 'narrowly': 4, 'roam': 1, 'dale': 1, 'emblems': 1, "_mind's": 1, 'smothering': 1, 'tribe': 12, 'finley': 1, 'mimicing': 1, 'tricks': 18, 'movieplex': 1, 'sergeant': 8, 'obituary': 1, 'slogging': 1, 'kissing': 9, "ex's": 1, "fishburne's": 1, 'jump': 29, 'chosen': 35, 'mythic': 3, 'pipe': 2, 'longo': 1, 'quirk': 6, 'overestimates': 1, 'consumes': 4, 'separating': 2, 'sight': 46, 'lamps': 2, 'norway': 1, 'unneccesary': 1, 'rentals': 1, 'intermittently': 1, 'orgy': 3, "meyer's": 2, 'hawke': 1, 'tackled': 1, 'disservice': 1, 'resigned': 2, 'sally': 5, 'co-protagonist': 1, 'sappy': 11, 'shocking': 20, "lola's": 1, 'three-hour': 1, 'genre-shifting': 1, 'snl': 3, "twins'": 3, 'robbins': 12, 'sleazy': 15, 'sabotaged': 4, 'seventeen': 2, 'greenwich': 1, 'chieftain': 1, 'determined': 28, 'poured': 2, 'infomercials': 1, 'crush': 13, 'wimp': 5, 'derisive': 3, 'disgraceful': 1, 'cut-rate': 2, 'well-filmed': 1, 'insulting': 23, 'tasked': 1, 'pounded': 2, 'embrassment': 1, 'issiah': 1, 'techie': 2, 'consideration': 8, 'tammy': 1, 'gel': 3, 'waster': 1, '30': 25, 'account': 10, 'robot': 12, 'heil': 1, 'tennille': 1, 'negotiation': 2, 'lore': 1, 'crusade': 5, 'kyle': 6, 'lagpacan': 1, 'complacency': 1, 'dices': 1, 'obscenely': 2, 'boys': 44, 'orpheus': 1, 'hades': 2, 'peeks': 3, 'definite': 9, 'aloud': 1, 'strongest': 2, 'pouting': 2, 'vice': 10, 'trainer': 4, "passenger's": 1, 'clockwork': 5, 'revisted': 1, 'prunella': 1, 'andy': 8, "k-she's-supposed-to-be": 1, 'lindinger': 1, 'scenario': 13, 'darkly': 3, 'instantaneously': 2, 'tapping': 1, '[the': 1, 'haphazardly': 2, 'slums': 1, "singin'": 1, 'goner': 1, 'kingsly': 1, 'insecurity': 2, 'superviser': 1, 'looked': 56, 'storh': 1, 'villains': 32, 'worth--le': 1, 'fire-taming': 1, 'walked': 16, 'teddies': 1, 'sarcasm': 3, 'threated': 1, 'showcases': 2, 'technology': 17, 'omega': 3, 'director/editor': 1, 'alternates': 4, 'agape': 1, 'seagal': 12, 'reminding': 3, 'cadaverous': 2, "sheen's": 4, 'fetch': 2, 'representative': 4, 'bowling': 6, "kruger's": 1, 'berkshire': 2, 'prefixed': 1, "miller's": 2, 'regrettably': 3, 'stab': 11, 'posessed': 1, 'sept': 1, 'deux': 1, 'shana': 1, 'reality': 41, 'ex-ballet': 1, 'redemption': 12, "_knock_off_'s": 1, 'peak': 8, 'capes': 1, 'puzzled': 3, 'dingy': 4, 'sorta': 3, 'alliances': 2, 'shimono': 1, 'trust': 40, '18': 9, 'honsou': 1, 'edmund': 2, 'russians': 4, 'continuity': 7, 'impatiently': 1, 'saxophone': 2, 'atlanta': 5, 'preening': 4, 'evocative': 2, 'folly': 1, 'lachman': 1, "'what": 1, 'nonchalantly': 3, 'uniqueness': 2, 'grandmother': 8, 'suspending': 1, 'encore': 1, 'overbeck': 1, 'typically-stimulating': 1, 'sensation': 2, 'site': 16, 'animation': 18, 'dialogue-laden': 1, 'gospel': 2, 'pulpy': 2, 'nuisance': 1, 'judge': 19, 'emulate': 1, 'curve': 5, 'projector': 3, 'dominated': 2, 'suicidal': 9, 'half-': 1, 'tolerable': 11, 'gilbert': 4, 'triple-digit': 2, 'consisting': 6, 'judah': 1, 'extracts': 2, 'saget': 1, '28up': 1, 'single-elimination': 1, 'semi-likable': 2, 'tight-budget': 1, 'eating': 23, 'janeane': 9, 'strain': 3, 'mid-1970s': 1, 'lobby': 2, 'burkittsville': 1, 'frog': 3, 'trivia--joel': 1, 'breasts': 28, 'chopper': 1, 'luger': 1, 'escobar': 1, 'absurdities': 1, 'limbs': 4, 'pledge': 1, 'stieger': 1, 'dissolved': 3, 're-enters': 1, 'typhoon': 3, 'pacifistic': 1, 'tormenting': 1, 'began': 29, 'lovell': 1, 'happening': 31, 'repulsion': 1, 'gunfire': 6, 'mastabatory': 1, 'putzes': 1, 'sweaty': 5, 'facts': 19, 'las': 23, 'belabors': 1, 'jay': 20, 'gravedigging': 1, 'girard': 1, 'stroll': 3, 'ditches': 2, 'overbaked': 1, 'arizona': 7, 'tile': 1, 'hollan': 1, 'boardroom': 1, 'quitting': 3, 'convinced': 25, 'groovy': 2, 'darkman': 3, 'upwards': 1, 'approach': 42, 'generated': 14, 'hands': 54, 'grrrl': 1, 'psychological-romance-thriller': 1, 'anemic': 5, 'ted': 17, 'fwahahahahahahahaha': 1, 'fixes': 3, 'hopefulness': 1, 'firms': 1, 'obtains': 1, 'muddling': 1, 'witch-doctor': 1, 'helpfulness': 1, 'satisfactorily': 1, 'bitches': 1, 'maestro': 1, 'stuck-up': 3, 'pouring': 6, 'weekened': 1, 'groucho': 2, 'gory': 14, 'gnawing': 1, 'pooch': 6, '---': 5, 'swamp': 3, 'ames': 1, 'ernie': 5, "garofalo's": 1, 'philanderer': 1, 'to-swallow': 1, 'assumed': 7, "`elvis'": 1, 'employers': 3, '710': 1, 'expressive': 8, 'ordeals': 1, 'hacks': 4, 'off-the-rack': 1, 'drunks': 1, 'kaye': 3, 'slaparound': 1, 'corky': 1, 'disenchanted': 3, 'franciscan': 1, 'dramatized': 2, 'antagonizes': 2, 'squader': 1, 'malls': 2, 'sirajul': 1, 'romeo': 10, 'pumped-up': 3, 'fiddle': 3, 'orgies': 2, 'stamper': 3, 'hildago': 1, 'newsweek': 1, 'full-feature': 1, 'skit': 8, 'depravity': 2, 'ottman': 1, 'jetsons-like': 1, 'stagebound': 1, 'naboo': 2, 'spouse': 4, 'hallway': 3, "angels'": 2, "connor's": 1, 'seaside': 4, 'pam': 8, 'wipe': 3, 'bueller': 1, 'self-assurance': 1, 'city-devastation': 1, 'glamour': 1, 'sobs': 1, 'rife': 3, 'ugly-ass': 1, 'scarred': 3, 'youngster': 3, 'lethargic': 1, 'weaknesses': 1, 'culprit': 3, 'suspeneful': 1, 'vela': 1, 'mah': 1, 'traction': 1, 'forsee': 1, 'attendance': 1, 'deficiencies': 4, 'breathless': 2, 'prescribed': 1, 'vat': 1, 'arnie': 4, 'thanksgiving': 4, 'low-cut': 2, 'yessuh': 1, 'entertainingly': 1, 'pudgy': 1, 'avoidance': 1, 'moping': 1, 'isiah': 1, 'freeze-dried': 1, 'unempathetic': 1, 'patric': 5, 'flees': 3, '10-year-old': 3, 'mindless': 18, 'benefited': 5, 'forgiveness': 2, 'slaying': 1, 'choir': 1, "kayley's": 1, 'dabbling': 1, 'publish': 1, 'anothergreat': 1, 'impact': 36, 'spicy': 3, 'wed': 1, 'insist': 2, 'rickety': 1, 'captain': 32, 'governed': 1, 'shelton': 4, 'columnist': 1, 'activities': 8, 'bussed': 1, 'self-consciousness': 1, 'residents': 11, "summer'": 1, 'pseudo-legendary': 1, 'believer': 3, 'sociopathic': 1, 'inserts': 3, 'akin': 5, 'toasts': 1, "maureen's": 2, 'tally': 2, 'enjoying': 24, 'discovers': 40, 'far-too-common': 1, 'paranoia': 6, 'euclidean': 1, 'piss': 3, "mouseketeer's": 1, 'chock-full': 4, 'pasadena': 1, "1963's": 1, 'santi': 1, 'bypassed': 1, 'walter': 21, 'swindling': 1, 'guns': 31, 'exhausting': 2, 'beales': 1, 'attained': 1, 'corralled': 1, 'storms': 6, 'armor': 3, 'board': 25, 'turkeys': 2, 'mapped': 1, 'scarier': 4, 'muttering': 2, 'aroused': 1, 'discomfort': 2, 'everlasting': 2, 'minimum': 9, 'blaming': 2, 'millimeter': 1, "'80s": 9, 'joke-machine': 1, 'dwight': 1, 'rag-tag': 1, 'diplomats': 1, 'censored': 1, 'strings': 7, 'two-hours-and-then-some': 1, 'spoiler-filled': 1, 'hotter': 2, 'var': 1, 'admiting': 1, 'circumcision': 2, '$8': 2, 'keitel': 2, 'honkey': 1, 'calculating': 1, 'entries': 3, '$31': 1, 'ribboners': 1, 'unsympathetic': 4, 'guessed': 15, '`battlefield': 1, 'asset': 11, "john's": 4, 'financial': 10, 'andrews': 3, 'mk': 2, 'velocity': 1, 'dykstra': 1, 'excommunicated': 1, "welles'": 2, 'overexcited': 1, 'moderation': 1, 'lether': 1, 'bolt': 3, 'pervades': 1, 'ambrose': 3, 'messy': 4, 'stupid': 137, 'ear-plugs': 1, 'degenerate': 3, 'annoyed': 13, 'gothic': 4, 'alfonso': 1, 'blacks': 7, 'barkeeps': 1, 'cheerily': 1, "devlin's": 1, 'commits': 8, 'flicks': 28, 'observatory': 1, 'cpl': 1, 'tan': 1, 'vladimir': 2, 'cat': 21, 'sentenced': 6, 'scofield': 1, 'good-film-making': 1, 'republic': 1, 'unbiased': 1, 'overseer': 1, 'here--': 2, 'non-titillating': 1, 'mines': 3, "horta's": 1, 'spiritual': 13, 'meanspirited': 1, 'nado': 1, 'laces': 1, 'maniac': 11, 'alba': 1, 'criminals': 19, 'woefully': 4, 'loan': 2, 'burr': 1, 'opposition': 2, '--are': 1, 'hedgetrimmers': 1, 'pow': 1, 'goth-style': 1, 'tarsem': 1, '2000-the': 1, 'sequal': 1, 'sylvester': 9, "death's": 1, 'inoffensive': 3, 'unplugging': 1, 'but-you-knew-along': 1, 'hopefuls': 1, 'virgo': 1, 'hoary': 1, 'early-19th': 1, 'unaccomplished': 1, 'playground': 1, 'utilizing': 4, 'waxen': 1, 'carlo': 1, 'cutting': 15, 'neo-nazis': 1, 'consonants': 1, 'buoyant': 1, 'honorary': 1, "mib's": 1, 'droids': 1, 'lab': 11, 'fugitive': 14, 'halle': 2, 'uncontrolled': 1, 'liken': 1, 'folks': 38, 'rosy-cheeked': 1, 'universally': 5, 'elevators': 1, 'whiplash': 3, 'triggering': 2, 'stripped': 5, 'burma': 2, "vierny's": 1, 'sillyness': 1, 'gimmick': 12, 'shacks': 1, 'sensual': 2, 'slo': 2, 'manifestation': 2, 'vulcan': 1, 'cast': 229, 'imf': 1, "close's": 1, 'regain': 3, 'rundown': 2, 'toto': 1, 'rhapsodic': 1, 'amorous': 1, 'fifty': 6, 'truer': 3, 'europop': 1, 'diners': 2, 'bearings': 1, 'cooking': 8, 'grandparents': 1, 'couples': 12, 'generally': 39, 'extends': 1, 'befriended': 5, "station's": 2, 'unique': 32, 'appeal': 46, 'exploits': 4, 'disliked': 3, 'porter': 4, 'captures': 2, 'vacationing': 2, 'potty-mouthed': 1, "jones'": 6, 'becker': 3, 'ugh': 11, 'rodman': 4, "lucy's": 1, 'tongue-lashing': 1, 'debauchery': 2, 'democrat': 1, 'predicts': 1, 'crouse': 1, 'repartee': 2, 'nonplussed': 1, 'always-woozy': 1, 'undercover': 19, 'consummating': 1, 'flexibility': 2, 'predominant': 2, 'deprived': 3, 'well-deserved': 3, 'flawless': 4, '60%': 1, 'delightful': 9, 'invented': 6, 'lauter': 1, 'musicals': 1, "dow's": 1, 'sugar': 3, 'pinnacle': 4, 'sonic': 1, 'prodded': 1, 'flaunting': 2, 'scout': 3, 'catholics': 1, 'shaw': 5, 'downtown': 3, 'julianne': 4, 'seventies': 6, 'walker': 9, 'lonergan': 1, 'cater': 1, "mcdonald's": 3, 'salomon': 1, 'coincidental': 1, 'alyson': 1, 'dew': 1, 'gatewood': 1, "she'll": 5, 'cutie': 1, 'gesserit': 1, 'audience': 283, 'satirical': 8, 'deus-ex-machina': 1, 'laid-back': 1, 'additions': 2, 'quality': 72, 'vancouver': 2, 'self-important': 4, 'hardcore': 5, 'incorporates': 1, 'shirts': 2, 'silicone': 3, 'gbn': 1, 'surpass': 1, 'storytelling': 15, 'hetero': 1, 'god-daughter': 1, 'ultimatum': 2, 'crowds': 2, 'widespread': 2, 'viewer': 59, 'gangs': 1, 'anita': 3, 'whoville': 1, 'onslaught': 3, 'converter': 1, 'heaped': 2, 'wong': 9, "o'byrne": 1, 'run-of-the-mill': 2, 'fates': 1, 'verhoven': 2, 'interupts': 1, 'nail': 4, 'non-threatening': 1, 'ship/ocean': 1, 'swirling': 2, 'alexis': 1, 'empath': 1, 'iliopulos': 1, 'rejected': 6, 'molasses-slow': 1, 'ranked': 1, 'pave': 1, 'matter-of-fact': 2, 'snake-oil': 1, 'iron': 4, 'cracker': 2, 'closed': 9, 'wakes': 7, 'survival': 6, 'psychopathic': 5, 'pryce': 2, 'telegraph': 1, 'olympian': 1, 'unintelligible': 4, 'criminal': 24, 'dispute': 1, "studios'": 1, 'christina': 13, 'envies': 1, 'theological': 1, 'historical': 10, 'independent': 11, 'trilogy': 10, "simpson's": 1, 'italians': 1, 'columbus': 2, '#1': 6, 'chase/goldie': 1, 'superpowers': 1, 'resignation': 2, 'criticism': 8, 'calmly': 6, 'tag-line': 1, 'preserving': 1, 'liev': 7, "fellini's": 1, '_polish_wedding_': 1, 'ill-fated': 5, 'arms': 16, 'creative': 25, 'sultry': 11, 'insides': 2, "'": 44, 'nasty': 26, "inadequate'": 1, 'captivating': 7, 'worm': 6, 'incomprehensibly': 2, 'ropes': 4, 'actively': 5, 'wyatts': 1, '30m': 1, 'robin': 61, 'bin': 3, 'melodramatic--but': 1, 'precariously': 2, 'seductively': 1, 'aiming': 5, 'offering': 6, 'livingston': 2, 'insignificant': 1, "d's": 2, 'kong-based': 1, 'youre': 1, "beatty's": 1, 'powell': 3, 'portait': 1, 'civic': 1, 'gassing': 1, 'depardieu': 1, 'airliner': 1, 'desires': 6, 'concluded': 3, 'plumber': 2, 'weak-willed': 1, 'twinkle': 1, "means'": 1, 'gaul': 1, 'meanness': 1, 'fighting': 44, "welle's": 1, 'request': 2, 'cotton-mouthed': 1, 'buddhism': 1, 'in-joke': 3, 'additional': 3, 'headset': 1, 'tvs': 1, 'discouraging': 1, 'sewed': 1, 'tilts': 1, 'ritalin': 1, 'tactfully': 1, 'compounded': 1, 'horndog': 1, 'fangs': 4, 'ex-wife': 5, 'three-person': 1, 'listed': 5, 'write': 62, 'significantly': 5, 'jumbles': 1, 'signaled': 1, 'wallenberg': 1, 'neal': 1, 'custody': 9, '===============': 1, "fort's": 1, 'falsely': 2, 'glencoe': 1, 'systematic': 1, 'famed': 10, "stiles'": 1, 'organisation': 2, 'ringwood': 1, 'business-oriented': 1, 'parisian': 1, "ghostface's": 1, 'principle': 4, 'all-business': 1, 'on-court': 1, 'sweet-natured': 2, 'out-of-control': 4, 'kewpies': 1, 'close-to-awful': 1, 'niall': 1, 'thurman': 10, 'spanner': 1, 'restart': 1, 'worst': 176, 'disregard': 4, 'enlightens': 1, 'rapids': 1, 'vans': 1, 'shelf': 7, 'unenthusiastic': 2, 'ruffian': 1, 'spectacle': 6, 'frappacino': 1, 'wire': 8, 'induce': 1, 'kitschy': 2, 'clutch': 1, 'enthusiastic': 7, 'apollo': 5, 'grandfather': 8, 'plagiarized': 2, 'starts': 112, 'sickeningly': 2, 'guffaws': 3, 'manservant': 1, 'interested': 54, '90-tooth': 1, 'ad': 26, 'alike': 2, 'world-weary': 2, "evil's": 2, 'witches': 5, 'rep': 1, 'seasoned': 7, 'screwed': 14, 'cisco': 3, 'counfound': 1, '``the': 2, 'lebowski': 1, 'jurrasic': 2, 'afforded': 3, 'pivotal': 5, 'vivona': 1, 'amazon': 3, 'storefront': 1, 'clarissa': 1, 'lifestyle': 8, 'separate': 12, 'mtv-': 1, 'lair': 3, 'wayne/batman': 1, 'accomplishment': 5, 'loitered': 1, 'comet-disaster': 1, 'chappelle': 3, "pbs'": 1, 'fascinated': 8, 'oily': 3, 'oh-so-promising': 1, 'suspends': 1, 'hackwork': 1, "magoo's": 1, 'pump': 2, 'ferrell': 3, 'unforgivingly': 1, 'billboard': 1, 'independece': 1, 'pear': 1, 'mere': 13, 'track': 38, 'rapport': 1, 'genre': 78, 'atmosphere': 44, 'legless': 2, 'swimsuit': 2, 'dawson': 2, 'justifiable': 1, 'waitress': 9, 'meathook': 1, 'splatter': 1, 'instance': 31, 'origin': 5, 'destiny': 6, "jet's": 1, 'grafted': 1, 'stefanson': 4, 'stein': 4, 'degenerated': 1, 'tortures': 3, 'claimed': 3, 'roughly': 5, 'retaliation': 2, 'brilliantly': 10, 'imax': 1, 'attired': 1, 'bearded': 3, 'cannon': 6, 'rosenthal': 1, "jericho's": 1, "ollie's": 1, 'darcy': 2, "burns'": 3, 'watch': 208, 'arrangements': 3, 'uncouth': 1, 'stickell': 1, 'spectacularly': 5, 'capitalize': 12, "shuckin'-and-jivin'": 1, 'rules': 28, 'swoops': 1, 'beetlejuice': 1, 'uncertainty': 2, 'musician': 8, 'kodak': 2, 'darth': 5, 'buffet': 1, 'bakula': 3, 'perking': 1, 'yakin': 1, 'shoud': 1, 'bellamy': 1, "child's": 6, "1985's": 1, 'sizes': 2, 'indien': 3, 'peccadilloes': 1, 'pastier': 1, "nicholson's": 4, 'bride-hopeful': 1, 'enter': 25, 'post-credit': 1, 'hassle': 1, 'utters': 8, 'allowing': 15, 'invites': 8, "investigator's": 1, 'binary': 1, 's-l-o-w': 1, 'agent/pop': 1, 'congo': 3, 'pantoliano': 2, 'spoon-fed': 1, 'scornful': 1, 'bregman': 1, 'spaniard': 1, 'thinks': 55, 'plot-twist': 1, 'recaptured': 1, 'saving': 35, 'snide': 2, 'unbreakable': 2, 'flavoring': 2, 'joys': 4, 'commerical': 1, 'grandiose': 3, 'submerging': 1, "bellboy's": 1, 'all-night': 2, 'camcorder': 4, "gyllenhall's": 1, 'slab': 1, 'lacking': 36, 'senstivie': 1, 'alignments': 1, 'clouseau': 1, 'dumps': 2, 'woof': 1, 'overhyped': 1, 'darned': 2, 'fenn': 2, 'coherent': 15, 'semi-offensive': 1, 'montage': 12, 'yemeni': 1, 'kidnaped': 1, "holmes'": 1, 'post-industrial': 1, 'zombified': 1, 'crawled': 1, 'fatboy': 1, 'revenge--and': 1, 'descent': 2, 'cop-who-sees-ashley-fleeing-an-accident-scene-and-': 1, 'visa': 1, 'tangiential': 1, 'morrison': 5, 'development': 60, 'chased': 13, 'buchman': 2, 'psychological': 21}, 'pos': {'dank': 3, 'maneuvering': 2, 'provocative': 8, 'newcomers': 4, 'synth-heavy': 1, 'a-': 3, 'white-knuckle': 2, 'vegetarian': 1, 'bergman': 7, 'winifred': 1, 'goth': 1, 'darwanism': 1, 'lout': 2, 'slides': 5, 'seas': 4, 'cuddles': 1, 'clothes': 23, 'dampened': 1, 'dinner': 24, 'zero-sum': 1, "director's": 21, 'alps': 1, 'outstripped': 1, 'ark-full': 1, 'motion': 68, 'down-played': 1, "colony's": 1, 'loosely': 10, '57': 3, 'egg-selling': 1, "gurgi's": 1, 'likeable': 24, 'illustrates': 8, 'kaufman': 5, 'detractors': 3, 'fergus': 1, 'review': 100, 'aging': 23, 'forgiving': 5, 'migrates': 1, 'larsen': 1, 'enslaved': 1, 'tried-and-true': 4, 'baseketball': 1, '2050': 1, 'experimenters': 1, 'funnel': 2, 'laser': 1, "nbc's": 1, 'scared': 22, 'davidzt': 1, 'beasley': 1, 'flowers': 10, '$110': 1, 'testings': 1, 'denuto': 1, 'packed': 16, 'roundabout': 1, 'back-street': 1, 'centering': 2, 'super-charged': 1, 'fianc': 9, 'stronghold': 1, 'steadiocam': 1, 'benjamin': 4, 'crises': 4, 'achingly': 3, '`pulp': 1, 'clock': 4, 'trunk': 7, 'iggy': 3, 'cityscape': 1, 'clung': 1, 'serendipity': 3, 'boyishly': 1, 'human/alien': 1, 'cleavage': 5, 'letting': 24, 'joon': 2, 'obsequious': 1, 'wrong': 122, 'none-too-obvious': 1, 'prank': 3, 'irregulars': 1, 'lang': 6, 'materialism': 1, 'kev': 1, 'soiree': 1, 'radios': 1, 'misconceptions': 1, 'grin': 10, 'capper': 1, 'gracie': 1, 'demolishes': 1, 'entry-level': 1, 'imagining': 2, 'stress': 11, 'pat': 21, 'entrenched': 3, 'rossellinia': 1, 'competitive': 3, '`call': 1, 'linden': 1, 'ent': 1, 'blunders': 1, 'quicker': 1, 'catastrophes': 3, 'tsai': 1, '_october': 1, 'stultifying': 1, "tom's": 8, 'ledger': 2, 'vannesa': 1, 'courtroom-certainly': 1, 'characteristics': 5, 'fluffy': 5, 'hilarious': 106, 'daryl': 7, 'sub-titles': 1, "lovers'": 1, 'skewer': 1, 'reformed': 1, 'helena': 6, 'cincinnati': 2, 'gossip': 2, 'edict': 1, 'hazardous': 1, "hewitt's": 1, "william's": 3, 'aaron': 9, 'blissful': 2, 'low-traffic': 1, 'forcing': 11, 'genetically': 6, 'siva': 1, 'muster': 8, 'lapsing': 1, 'pear': 2, 'hillarous': 2, 'wen': 7, 'critic-type': 1, 'geopolitics': 1, 'christians': 1, 'amphibious': 1, 'aardman': 1, 'summer': 88, 'upper-crust': 1, 'marrow': 3, 'mar': 2, 'domestic': 12, "fraser's": 2, 'unity': 1, 'depths': 12, 'pulled': 34, 'negotiates': 2, 'fancy': 12, 'satan': 8, 'beckel': 1, 'dinsdale': 1, 'question': 107, 'agonisingly': 1, 'marveling': 1, 'organism': 1, 'ploys': 1, 'in-depth': 2, 'yucky': 2, 'blaxploitation': 5, 'comparisons': 13, 'drop-off': 2, 'sufism': 1, 'ki': 1, 'reservoir': 10, 'oppressors': 1, 'kessler': 1, 'pussy': 1, 'rears': 2, 'intelligently': 7, 'intellectually': 9, 'girl/boy': 1, 'exorcist': 3, 'eighth': 2, 'function': 10, 'ca': 2, 'kristofferson': 1, 'andrea': 6, 'donovan': 6, 'back-and-forth': 1, 'mayan': 1, 'adrianne': 1, 'danny': 39, "capulet's": 2, 'unsavory': 2, 'elina': 1, 'annex': 1, 'breckin': 3, "greg'": 1, 'uplifiting': 1, 'scoffs': 1, 'withing': 1, 'abandoned': 23, 'janet': 8, 'devastatingly': 1, 'severed': 6, 'largely': 31, 'formally': 1, 'superstars': 1, 'bonked': 1, 'luc': 4, 'modernize': 2, 'male-oriented': 1, 'greg': 10, 'fritz': 3, 'lobbyist': 1, "dash'": 1, 'anti-christ': 1, 'casting-against-type': 1, 'intend': 6, 'pepper': 4, 'nick': 19, 'navarro': 1, '`death': 1, 'pulpy': 1, 'andie': 4, 'snappy': 5, 'man-whoring': 1, 'homer+s': 1, "'do": 1, 'co-workers': 13, 'brad': 23, 'conversations': 17, "hutter's": 2, 'skewed': 4, 'captivatingly': 1, 'unknown': 21, 'maidservant': 1, 'one-hundred-dollar': 1, 'claudia': 2, 'bingo': 1, 'eliciting': 5, 'latura': 1, 'courtney': 9, 'cong': 1, 'favorite': 63, 'thief': 14, 'classifying': 1, 'kathy': 14, "'big": 1, 'jfk': 5, 'tuning': 4, "jinn's": 1, 'biases': 1, 'unsettling': 10, 'establish': 7, 'shacking': 1, 'cantonese': 2, 'dvd-rom': 2, 'alien-like': 1, 'avant-garde': 4, 'dramas': 18, 'marcus': 4, 'jailbait': 1, 'horndog': 1, 'equip': 2, "niccol's": 3, 'foolishness': 2, 'no-good': 1, 'lulled': 1, 'ceremonies': 1, 'satisfy': 12, 'feeds': 5, 'asparagus': 1, 'bullock': 8, 'railroad': 2, "brosnan's": 2, 'powaqqatsi': 1, 'decorated': 2, 'electronic': 5, 'hooked': 3, 'leonetti': 1, 'eschew': 1, 'sami': 1, 'sprinkled': 4, 'krippendorf': 1, 'renier': 1, 'risen': 3, 'out-': 1, 'managed': 37, 'trumpeted': 1, 'improvement': 9, 'lifeboat': 1, 'excluded': 2, 'alley': 5, 'castigates': 1, 'anti-climatic': 1, 'sauntering': 1, 'thirties': 1, 'yelchin': 2, 'range': 38, 'namesake': 2, 'wrongs': 4, 'satellite': 7, 'anecdotal': 1, "floom's": 1, "doin'": 1, 'ava': 1, 'montreal': 1, '-all': 1, 'scant': 4, 'sanxay': 3, 'non-goateed': 1, 'unnoticeable': 1, 'id-fueled': 1, 'dastardly': 2, 'chivalry': 4, 'beeing': 1, 'confines': 6, 'zen-like': 1, 'barrier': 2, 'itself--what': 1, 'inevitability': 1, 'bedford': 2, '_dog': 1, 'carmen': 6, 'knowledge': 40, 'springboard': 1, 'infuriating': 2, 'aamir': 1, 'mildly': 10, 'oumph': 1, 'often-moving': 1, 'fury': 8, 'pictured': 1, "binoche's": 1, 'lin': 4, 'enlistee': 1, 'junkie': 5, 'mcnamara': 1, 'outbreak': 2, 'sleepy-voiced': 1, 'culminate': 1, 'train': 37, 'alcott': 1, "quinn's": 1, 'barondes': 1, 'sorority': 3, 'triton': 1, 'sell': 20, 'rick/ilsa': 1, "adult-helfgott's": 1, 'jordanna': 1, "mccabe's": 2, 'rap': 9, 'reactions': 15, 'witnessing': 7, 'songwriting': 1, 'envious': 2, 'dysfuntion': 1, 'well-photographed': 1, 'commission': 2, 'salad': 1, 'petals': 1, 're-make': 1, 'cell-mates': 1, 'transfer': 9, 'stammers': 1, 'ze': 1, 'dragonheart': 1, 'ripe': 9, 'unusable': 1, 'chi': 2, 'authorial': 1, '62': 1, 'dragged': 4, 'vernacular': 2, 'topless': 3, 'tremble': 1, 'roland': 4, 'clueless': 15, 'low-tech': 1, 'improvisational': 2, 'wilbur': 1, 'subway': 3, 'economic': 7, 'transistor': 1, 'nomi': 1, 'photographers': 1, 'defines': 7, 'wiest': 2, 'suspiria': 1, 'gayheart': 1, 'frederic': 1, 'immaculately-white': 1, 'tuned': 2, 'surprise': 89, 'prospers': 1, 'e-dreams': 1, 'stapled': 1, 'tampered': 1, 'argento': 4, 'slaughter': 9, 'ace': 6, 'marienbad/m': 1, "'matrix": 1, 'anticlimax': 1, 'veering': 2, 'book': 94, 'wachowskis': 1, 'heart': 115, 'reingold': 1, "warren's": 1, "bubby's": 1, 'subdued-and': 1, "a'": 2, 'mix-ups': 1, '_murder_': 1, 'go-go': 1, '19': 8, 'short-lived': 2, 'tearjerker': 5, 'genius': 36, 'protesters': 2, 'major': 114, "1900's": 3, 'high-school': 8, 'preston': 2, 'hinterlands': 1, 'ethan': 19, 'taking': 97, 'crust': 2, 'sophomoric': 3, 'frederick': 3, 'players': 28, "that'll": 3, 'sinewy': 1, 'red-herrings': 1, 'someplace': 1, 'volume': 1, 'streep': 6, 'advising': 1, '_shaft_in_africa_': 1, "spall's": 1, 'detox': 2, "garofalo's": 2, 'wim': 2, 'murderess': 1, 'infamous': 15, 'giants': 1, "don't-mess-with-my-life-and-i-won't-mess-with-yours": 1, 'join': 40, 'costume': 22, 'ignoramus': 1, '10/10': 7, 'arquette': 13, 'incidental': 2, 'barage': 1, 'rationality': 1, 'doubting': 1, 'persuades': 2, 'dictionary': 3, '1944': 1, 'writers': 30, 'flair': 16, 'healer': 1, 'pitches': 2, 'watching': 167, 'birkin': 1, 'undress': 2, 'talal': 2, 'minnie': 15, 'thom': 2, 'clinton': 7, 'problem-why': 1, 'room': 96, 'scales': 3, 'creedence': 1, 'humorous': 42, 'pilots': 2, 'gbatokai': 1, '$7': 1, 'hierarchies': 2, 'rock': 45, 'bladder': 1, 'outs': 1, 'msnbc': 1, "another's": 6, 'restrictive': 1, 'unpublished': 1, 'saccharine': 1, 'paralleling': 2, 'keating': 1, "america's": 18, 'big-be': 1, 'sightless': 1, 'cross-cultural': 1, 'sikh': 1, 'judged': 6, 'ex-partner': 1, 'censors': 3, 'hinds': 1, 'treads': 3, 'nation-state': 1, "act's": 1, "male's": 1, 'ultra-modern': 1, 'metered': 1, 'cheering': 4, 'juni': 1, 'rosen': 1, 'coolly': 1, 'father-knows-best': 1, 'found': 167, 'claymation': 1, "foster's": 4, 'twentieth-century': 1, 'doors': 19, 'on-board': 1, 'based': 162, 'force': 70, 'attempting': 22, 'insects': 6, 'emmanuelle': 2, 'hoses': 1, 'money-': 1, 'socially/culturally/politically/globally': 1, 'urge': 13, 'herding': 2, 'nitty-gritty': 1, 'viii': 1, 'villainy': 4, 'supported': 5, 'jolting': 2, 'teammates': 2, '_experience_': 1, "humans'": 1, 'complement': 1, 'dancer': 7, 'scalvaging': 1, 'lambs_': 1, 'memorabilia': 1, '2400': 1, 'overcoats': 2, 'helfgott': 2, 'envisioned': 6, 'excursion': 4, "mel's": 2, 'fixed': 4, 'ony': 1, '33': 5, 'occupants': 1, "-it's": 1, "robbie's": 2, 'intergalactic': 4, 'uvula': 1, 'gedde': 3, 'offer': 49, "jeopardy'": 1, 'sensibility': 11, 'friend': 142, "kate's": 2, 'obsessions': 2, 'fear--the': 1, 'cape': 5, 'divining': 2, 'lowlifes': 3, 'madman': 3, 'teaching': 10, 'imaging': 1, 'racing': 4, 'polar': 4, 'animosity': 4, "real-men'": 1, 'inaccurately': 1, 'slop': 1, 'less-than-original': 1, 'labour-intensive': 1, '[see': 1, 'as--': 1, 'unknowing': 2, 'upsetting': 4, 'proprietor': 3, 'marrying': 10, "o'hara": 6, 'expose': 8, 'ridings': 1, 'fervor': 5, 'gavras': 1, 'go-ahead': 1, 'fairytail': 1, "trapper's": 1, 'tiny': 27, 'cheetah': 1, 'criticized': 8, 'julia-louis': 1, 'vaulerbility': 1, 'guests': 6, "jar's": 1, 'saxophonist': 1, 'miscues': 1, 'obscuring': 1, 'visitation': 1, 'co-operating': 1, 'pillage': 2, "malcolm's": 1, "'fuck'": 1, 'ceasefire': 1, '_boom_': 1, 'abused': 5, 'katzenberg': 3, 'mansions': 2, 'hopkins': 17, 'takenaka': 2, 'demi': 1, "vincent/jerome's": 1, 'undeserved': 2, 'unformal': 1, 'bordering': 4, 'battler': 1, "'kung": 1, "standefer's": 1, 'fears': 24, 'jordans': 1, 'strode': 1, "car's": 1, 'revolutionaries': 3, 'congregate': 1, 'pikul': 1, 'non-cynics': 1, 'geriatric': 1, 'pasted': 1, 'readers': 6, 'extravaganza': 3, 'rags-to-riches': 1, 'tough-nosed': 1, 'articulate': 7, 'heavenly': 6, 'lambasted': 1, "mobutu's": 1, "company's": 6, 'remembrance': 1, 'doom': 17, "briggs'": 1, '`cheer': 1, 'arachnid-type': 1, 'bethany': 3, 'wiccan': 1, 'articles': 3, 'salutes': 1, 'andersen': 1, 'affections': 7, 'unexplained': 3, 'valor': 1, 'vaccinated': 1, '1980': 7, 'gallows': 2, 'patient': 24, 'raheem': 1, 'helpfully': 1, 'led': 62, 'ridley': 4, 'honest-politician-rare': 1, 'ziggy': 2, 'crossgates': 4, 'avery': 2, '_onegin_': 1, "strangelove's": 1, "moore's": 4, 'minisub': 1, 'belinda': 1, 'pronounced': 5, 'fistful': 2, 'enjoyed': 56, 'durning': 2, 'frontman': 1, 'choreograph': 1, 'lucked': 2, "helgeland's": 1, 'grabbed': 3, 'watered-down': 2, 'exaggerated--this': 1, 'rude': 9, 'zingers': 3, 'authoritive': 1, 'eight-year-old': 4, 'stage': 50, 'virility': 1, 'dispassionate': 2, 'bening': 9, 'memorable': 92, 'masochism': 1, 'plastics': 1, 'distributor': 6, 'tools': 4, 'gathering': 8, 'scheme': 23, 'clear-cut': 2, 'eaisly': 1, 'men-only': 1, 'meeks': 1, 'resides': 7, 'discrimination': 3, 'busby': 2, 'jacobs': 1, "cheatin'": 1, 'guarantee': 4, 'mandatory': 4, 'eater': 1, "could've": 6, 'alda': 2, 'mischief': 1, '1986': 7, 'mule': 1, 'manicurist': 1, "original'": 1, 'quotable': 1, "federation's": 1, 'ponders': 5, 'sus-pects': 1, 'asuma': 1, 'opportunies': 1, '1138': 1, 'craves': 2, 'engravings': 1, 'tattered': 1, 'metzler': 1, 'grinder': 2, 'perceptive': 6, 'howitt': 1, '52': 3, 'guardian': 9, 'insectopia': 1, "haynes'": 1, 'distract': 8, 'twinkies': 1, 'factor': 17, '1791': 1, 'castilian': 1, 'wagner': 5, '__________________________________________________________': 1, "woodward'": 1, 'recur': 2, 'spotless': 3, 'zellweger': 2, 'eggs': 4, 'negligible': 1, 'swimming': 5, 'clemens': 1, 'disintegrated': 2, 'weirdest': 2, 'cellophane': 1, '_not_': 3, 'singular': 5, 'asses': 2, 'ethnicity': 2, 'henreid': 1, 'rethinks': 1, 'rigormortis': 1, 'a-hole': 1, "minutes'": 1, 'spielberg\x12s': 1, 'antwerp': 1, 'lips': 6, 'predicted': 6, 'excuse': 20, 'smacks': 1, 'free': 54, 'fragment': 1, "eachother's": 1, 'damaged': 6, 'jonze': 4, 'lighted': 1, 'ocean': 19, 'agree': 27, 'chon': 1, 'mit': 5, 'burgess': 1, 'suprisingly': 2, 'jittery': 4, 'film-within-the-film': 1, 'vibrations': 2, 'collector': 6, 'splashes': 4, 'claus': 2, 'steel-factories': 1, 'absolute': 17, 'runs': 91, 'drug-lord': 1, 'gingrich': 1, 'besieged': 2, 'travers': 1, "`it's": 1, 'presents': 42, 'magnificent': 27, 'torrent': 2, 'diametrically': 1, 'manner': 47, 'to-hell-with-morality': 1, 'ornament': 1, "jimmy's": 2, 'obsessively': 3, 'nostalgic': 10, 'intercourse': 1, 'beart': 1, "room's": 2, 'morose': 3, 'spoiled-rich-girl': 1, 'oscar': 98, 'flatly': 2, 'elvis': 2, 'direct': 33, 'fast-emptying': 1, 'fatherly': 2, 'mailing': 1, "fingernail'": 1, 'operations': 8, 'continually': 8, 'acutely': 4, 'carving': 2, 'decidedly': 16, 'friend-friend': 1, 'blotter': 1, 'load': 9, 'amidala': 5, 'charles': 30, "leo's": 1, 'revolt': 1, 'highschool': 1, 'exponential': 1, 'search': 54, 'story-line': 4, "conrad's": 2, 'beaming': 3, 'two-thousand': 1, "peck's": 1, "luis'": 1, 'rock-solid': 2, 'nicolodi': 1, 'sappiest': 1, 'suspend': 4, 'passable': 2, 'dismays': 1, 'grease': 7, 'attendant': 8, "leuchter's": 1, 'mcglory': 1, 'integrating': 1, 'apollonia': 1, 'leukemia': 1, 'tempe': 1, 'frenetic': 4, "evars'": 1, 'gallifrey': 1, 'reville': 1, 'convinces': 15, 'infantile': 1, '`sleepy': 1, 'banishment': 1, "powell's": 1, 'handsomest': 1, 'unflattering': 3, 'albeit': 28, 'shape': 17, 'interrupting': 1, "country's": 5, 'classroom': 2, 'guffman': 2, 'alternate-1999': 1, 'ruiz': 1, 'daughter': 91, 'sanitized': 2, 'turnstyle': 1, 'chickens': 3, 'restricted': 4, 'easy-listening': 1, 'fatally': 1, 'cantarini': 4, 'pig-tails': 1, 'nice-guy': 3, "wong's": 1, 'kris': 2, 'predictament': 1, 'fibrosis': 1, 'overexplain': 1, 'aurelius': 3, 'superbly-staged': 1, 'stills': 1, 'mongolian': 1, 'principal': 14, 'homer': 6, 'recourse': 1, 'smartest': 3, 'strategic': 2, 'bombastic': 1, 'butt-numbing': 1, 'page': 3, 'seamless': 14, 'honorable': 2, 'rifkin': 3, 'lengthy': 11, 'unemotional': 2, 'tibbs': 1, "crow'": 1, 'admirers': 1, 'glimpsed': 1, 'delight--': 1, 'deanna': 1, "liberty's": 1, 'analysis': 8, 'forge': 2, 'sympathy': 24, 'uneventful': 1, 'steiger': 4, 'compromise': 8, 'epigraphs': 1, 'predecessors': 10, 'manufactured': 5, "neuve'": 1, 'qualities': 21, 'disatisfaction': 1, 'possiblity': 1, 'linda': 14, 'niflheim': 1, 'civilized': 1, 'explanation': 17, 'erica': 1, 'ex-president': 1, 'malaise': 1, 'tornado-caused': 1, 'illness--which': 1, 'creature': 25, 'mortensen': 2, 'civilians': 4, 'johnston': 2, '>from': 23, 'pennebaker': 1, 'brood': 1, 'malay': 1, 'euphoria': 2, 'explicate': 1, 'humankind': 4, 'zabriskie': 1, 'algeria': 1, 'threatening': 19, 'coens': 5, 'point': 198, 'transaction': 1, 'seargent': 1, 'tourist': 3, 'poor': 49, 'jeanne': 6, 'pheneomena': 1, 'bendix': 1, 'skating': 1, 'workings': 6, 'rallying': 2, 'adage': 3, 'guarantees': 2, 'whiskey': 6, 'overzealous': 3, 'dreading': 1, 'uppity': 1, 'naughty': 4, 'arises': 8, 'sterling': 2, 'tracking': 7, 'erin': 1, 'hasidic': 1, 'fear': 64, "perpetrator's": 1, 'briefcase': 4, 'flop': 5, "lopez's": 1, "tavoularis'": 1, 'kureishi': 1, 'hulce': 1, 'senseless': 5, 'loaf-sung': 1, 'anti-semitic': 2, 'repulsive': 3, 'pistone': 1, 'kinski': 4, 'glee': 3, 'goofily': 1, 'paige': 1, 'toys': 16, 'leone\x12s': 1, "hunting'": 1, 'gait': 1, 'ax': 2, 'garden': 8, "murderer'": 1, 'controlling': 3, 'elna': 1, 'whispers': 2, "'true": 1, 'paquin': 3, 'merry-go-round': 1, 'kong-style': 1, '_brazil_': 1, 'performances': 214, 'sheik': 3, 'plains': 1, 'corp': 1, 'newsman': 2, 'conduct': 1, 'dollies': 1, 'year--every': 1, 'squishing': 1, '31': 5, 'motley': 5, 'invested': 8, 'madness': 14, 'millennium': 8, 'police': 85, 'ramboids': 1, 'bogus': 3, 'ho': 1, "tyson's": 1, 'creams': 1, 'building': 46, 'heist': 7, 'gate': 4, 'misused': 1, 'classy': 8, 'havilland': 2, 'signify': 2, 'scrounges': 1, 'underdogs': 1, 'aisling': 1, 'persuaded': 3, 'jeffrey': 17, 'sakes': 1, 'semi-tear': 1, "`face/off'": 1, 'kitschiest': 1, 'serials': 2, 'ultra': 1, 'strenuous': 1, 'fights': 25, 'uhry': 1, 'ocean-going': 1, 'portraits': 4, 'migraines': 1, 'cyborg-ninja': 1, "saddam's": 1, 'alum': 2, 'sickness': 4, 'recover': 8, 'schmoe': 1, 'ink': 1, 'fandom': 2, 'snob': 3, 'downsizing': 1, '1-': 1, 'thirteenth': 6, 'luhrmann': 2, 'psychotically': 1, 'responds': 13, 'dee': 2, 'naturalness': 3, 'gaunt': 1, 'hand-held': 3, 'numeric': 1, 'blacker': 2, 'horror-fans': 1, 'schell': 2, 'unbuttoning': 1, 'exceed': 1, 'abraham': 4, 'sidewalk': 2, "beart's": 1, 'emptying': 1, 'prove': 53, 'folk': 12, 'larger-than-life': 5, 'oddity': 1, 'setbacks': 3, 'dirt': 10, 'irrevocably': 1, 'hanksian': 1, 'preceding': 6, 'vehemence': 1, "carlito's": 3, 'se': 5, 'muscle': 3, 'hironobu': 1, 'cans': 3, 'excitements': 1, 'deranged': 7, '-jack': 2, 'tight': 21, 'colonialists': 1, 'tights': 1, 'glory': 17, 'melodramatic': 17, 'virtually': 39, 'dysfunctional': 5, "birdsall's": 1, 'carpets': 1, 'spookiness': 1, 'orginality': 1, 'backstreet': 1, 'buckley': 1, 'quiet': 45, 'embarrass': 1, 'bursting': 4, 'goodbye': 8, 'worralls': 1, 'slurring': 1, 'unparallelled': 1, 'pipes': 2, 'preaching': 5, 'ipo': 1, 'province': 2, '19-year': 1, 'doppelgangers': 1, 'foreman': 6, 'lens': 3, 'penelope': 3, 'yitzak': 1, 'scrapping': 1, 'tango': 2, 'omits': 1, 'unimpressive': 1, 'accompanies': 5, 'immortalised': 1, 'castration': 1, 'standup': 3, 'disrespect': 1, 'sub-culture': 1, 'treason': 1, 'panoramic': 1, 'film-schoolish': 1, "dillon's": 2, 'mets-orioles': 1, 'monitored': 1, 'predicting': 3, "priest's": 2, 'ordinary': 32, 'permit': 3, 'vignettes': 9, 'passing': 15, 'uninspiring': 2, 'nut': 1, 'culp': 1, 'revive': 4, 'modify': 1, 'relate': 19, "goldman's": 1, 'revolutions': 1, 'armed': 16, 'zeta': 2, 'anti-classics': 1, 'contrast': 41, "kivilo's": 1, 'dutch': 3, 'byrnes': 2, 'film--is': 1, 'zinnia': 1, 'released': 86, 'suffers': 20, 'hen': 2, 'harp': 2, 'concept': 46, "dante's": 6, 'cameo-esque': 1, 'delpy': 1, "kidnapper's": 1, 'mccoy': 2, 'hyper-drive': 1, 'untold': 1, 'titular': 4, 'stashed': 2, "farmer's": 1, 'manly': 2, 'beans': 1, "`simpsons'": 1, 'walcott': 1, 'trophies': 1, 'maudlin': 2, 'deter': 4, "dunne's": 1, 'malaysia': 3, 'politicos': 1, 'vision': 34, 'enigma': 5, 'half-truth-telling': 1, 'heavyweight': 4, 'novelisation': 1, 'sound-wave': 1, 'crew': 62, 'watchable': 9, 'homosexuality': 6, 'stubbornly': 1, 'lionel': 1, 'matte': 1, 'promesse_': 1, 'guiltily': 2, 'suspensefully': 1, 'hometown': 12, 'tram': 1, 'senitmental': 1, 'sylvia': 6, "'cheezy'": 1, 'paradice': 1, 'hudsucker': 4, 'private': 58, 'mckellar': 3, 'taxi-cab': 1, '1/4': 1, 'cool': 59, 'shuns': 2, 'movie--': 1, 'caricatures': 10, 'look-at-me': 1, 'nosing': 1, 'manhoff': 1, 'orca': 1, 'bloodletting': 2, 'enraged': 3, 'piloted': 1, 'seize': 1, 'flesh-and-blood': 2, 'customers': 11, 'springer': 8, 'cameron': 38, 'troubles': 17, 'once-profitable': 1, 'transitioning': 2, 'off-again': 1, 'handlers': 1, 'effort': 61, 'guang': 1, 'basically-big-roach-type-bug': 1, "donaldson's": 1, 'student': 42, 'adoring': 1, 'fairytale': 2, 'xenophobe': 1, 'charme': 1, 'honing': 1, 'matter': 121, 'lukemia': 1, 'titmalau': 1, 'assiduously': 2, "matrix'": 1, 'veers': 3, 'fried': 5, 'erupts': 3, 'sugarish': 1, 'likes': 39, 'studied': 5, 'llewelyn': 2, 'candle': 2, 'formidable': 4, 'fences': 1, 'charades': 1, '[expectations': 1, 'snow-covered': 4, "moment's": 1, 'kinnear': 6, 'fantastical': 2, 'anti-intruder': 1, 'appaling': 1, 'techno-thriller': 1, 'clerks': 4, "jon's": 1, 'corrupted': 1, 'untouchables': 2, 'outdoes': 1, 'winning': 38, 'settings': 13, 'misunderstandings': 1, 'dna': 6, 'realized': 35, 'undeterred': 1, 'mixture': 12, 'danza': 1, "slade's": 1, 'instruct': 1, 'earn': 9, 'amelie': 1, 'visas': 1, 'impatient': 1, 'ez': 1, 'crudeness': 2, 'scripted': 10, 'jacobim': 1, 'proyas': 5, 'applying': 1, 'japan': 10, 'disapproves': 1, 'parted': 1, "'round": 1, 'blowed': 1, 'outpost': 1, 'jerk-ules': 1, 'benches': 1, "redford's": 4, 'inane': 8, 'repute': 1, 'self-confidence': 1, 'definately': 10, 'aiding': 2, "london's": 3, 'colorfully': 1, 'benichou': 1, 'humiliated': 4, 'forming': 2, 'descendant': 1, 'surround-sound': 1, 'irons': 2, 'relic': 1, 'agreeing': 2, 'invasive': 1, 'squarish': 2, 'arctic': 3, 'families': 17, 'verite': 2, 'standoffish': 2, 'rebellious': 9, 'incoming': 1, 'drills': 1, 'scrambled': 1, 'imagery': 22, 'petersen': 2, 'lori': 3, 'romano': 1, 'thierry': 1, 'uneasily': 1, 'shaolin': 3, 'unaware': 12, 'summoning': 3, 'sharp-edged': 1, 'cybil': 2, 'out-of-it': 1, 'slickness': 1, 'written': 149, '5': 24, 'admired': 13, 'increasingly-flustered': 1, "thing's": 1, 'astronaut': 8, 'hucksters': 1, 'disclosed': 2, 'malevolent': 6, 'henri': 1, 'favoring': 1, 'flourishes': 5, 'jawas': 1, 'limit': 5, 'chucklesome': 1, 'directional': 1, 'bonds': 5, 'injury': 3, 'paternal': 2, 'offers': 60, 'requisite': 9, 'incredulous': 1, 'airwaves': 1, 'kicking': 4, 'extending': 2, 'debuting': 2, 'challenger': 1, 'appetizers': 1, 'coen-heads': 1, 'goodfellas': 11, 'corroding': 1, 'sacrifice': 9, 'automatically': 14, 'killed': 72, 'promotion': 2, 'biggest': 59, 'gearing': 1, 'irrationality': 1, 'rhetoric': 4, 'asp': 2, "sheriff's": 1, 'passes': 17, "ones'": 1, 'suschitzky': 1, 'waging': 2, 'tornadoes': 1, 'stahl': 1, 'androgony': 1, 'han': 6, 'bimbos': 1, 'highest': 10, 'commit': 10, 'redefining': 1, 'economy': 2, 'strongly': 21, 'undeniable': 6, 'well-staged': 1, 'floris': 1, 'neuwirth': 2, 'practices': 3, 'long-suppressed': 1, 'brest': 1, 'reel': 7, 'imbues': 4, 'half-dozen': 1, 'huxley': 1, 'ideal': 24, 'intended': 37, 'knock-off': 1, 'pickets': 1, 'banner': 1, 'quarter': 5, '`if': 1, "'hot": 1, 'delorenzo': 1, 'volcanos': 1, 'grieg': 1, 'devotee': 1, 'sexbomb': 1, 'levites': 1, 'catholicism': 4, 'loose': 32, 'directions': 7, 'franka': 1, 'dragonflies': 1, 'harlequin': 1, 'zoom': 2, 'objectionable': 2, 'church': 21, 'prophecy': 4, 'wierdness': 1, 'wits': 7, 'vartan': 1, 'deception': 7, 'comedian': 13, 'excels': 7, 'absolutely': 67, 'uncompelling': 2, 'long-running': 1, 'nigger': 2, 'merman': 1, 'carjackers': 1, 'intents': 1, 'saddled': 2, 'antarctic': 1, 'googly': 1, 'sittings': 1, 'budweiser': 1, 'homage': 30, 'chemisty': 1, 'sins': 6, 'mrs': 21, 'belonging': 6, 'ex-con': 2, "chinatown's": 1, 'westerns': 5, 'natascha': 3, 'kevin': 70, 'creepers': 1, 'smashed-up': 1, 'lynch': 8, 'ridiculous': 20, 'dichotomous': 1, 'reviewers': 10, 'trashy': 4, 'angel-related': 1, 'stu': 1, 'uncover': 7, 'rican': 3, 'solipsism': 1, "kimble's": 1, 'chauffeuring': 1, 'milius': 1, 'murphy': 33, 'prosaic': 3, 'lake': 16, "2'": 2, 'unromantic': 1, 'griffiths': 4, 'irresistable': 2, 'enjoyability': 2, 'buy-more': 1, 'lincoln': 3, 'unfriendly': 4, 'pennsylvania': 2, 'deficit': 2, "scagnetti's": 1, 'rights': 17, 'swings': 3, 'notifying': 1, 'co-defendant': 1, 'cross-dresser': 2, "`scream'": 2, 'poslethwaite': 1, "sant's": 2, "times'": 1, 'whiz': 6, 'paper-thin': 2, 'reunite': 1, 'greatly': 20, "mirkin's": 1, 'mythos': 2, 'flaunts': 1, 'olds': 2, 'architecturally-unsound': 1, 'cocoons': 1, "food's": 1, 'white-man-black-man': 1, 'rediscover': 2, 'schreck': 2, 'chelcie': 1, 'caps': 4, 'letterboxing': 1, 'shiro': 1, 'dreams/fantasies': 1, 'raids': 2, 'antennae-like': 1, 'jeannie': 1, 'yuppies': 2, 'brawls': 1, 'constable': 1, 'cru': 1, 'extra-terrestrial': 4, 'misjudgment': 1, "fei-hong's": 1, 'terrifyingly': 1, 'slumping': 1, 'courses': 2, 'divulges': 1, 'outfit': 6, 'glasses': 4, 'urgency': 5, 'rise': 29, 'unidentifiable': 2, 'ack-acking': 1, 'exec': 3, "composer's": 1, 'cycle': 6, 'throne': 9, "dracula's": 1, 'begets': 2, 'malick': 3, 'renegades': 1, 'matures': 1, 'character-controlled': 1, 'fallow': 1, 'aryan': 1, 'sequestered': 1, 'overwhelmed': 9, 'cooled': 1, 'hecklers': 1, 'codes': 2, 'comprised': 5, 'compelled': 3, 'silver': 25, 'reccomending': 1, 'flic': 1, 'cattaneo': 2, 'rehearsal': 2, "guys'": 1, 'despondent': 1, 'garters': 1, 'mob-style': 1, 'lodgerley': 1, 'screenings': 2, 'bumpkinisms': 1, 'initially': 38, "screenplay's": 1, 'things--tea': 1, 'cold-hearted': 1, 'bow-wow': 1, 'fiction-type': 1, 'kaaya': 1, '8-year': 1, 'shaffer': 3, 'representing': 2, 'conejo': 1, 'blasting': 2, 'gangstas': 2, 'milla': 2, 'reminiscent': 36, 'pranks': 1, 'karen': 13, 're-enact': 1, "cinderella's": 1, 'monitoring': 3, 'vaulerble': 1, 'artwork': 4, 'fascist': 3, 'significance': 11, 'screened': 5, 'grocery': 8, 'coordinators': 1, 'scream': 32, '--michael': 2, 'merchant': 1, 'closes': 5, 'timely': 8, 'gravestown': 1, 'rutger': 1, 'harmonizing': 2, 'trappings': 4, 'unrelentingly': 3, 'perky/cute': 1, 'comprehensible': 1, 'touring': 2, 'joan': 20, 'fencing': 2, 'ranting': 3, 'clinic': 4, 'dog_': 1, 'monstrous': 6, 'nashel': 1, 'depraved': 1, "pocum's": 1, 'lena': 1, 'dirk': 3, 'risk-adverse': 1, 'titty': 1, 'troubled': 28, "ricky's": 2, 'optimistic': 3, "'97": 5, 'valenti': 1, "oldman's": 1, 'lacing': 1, 'missed': 29, 'ratzo': 1, 'plunging': 1, 'futuristic-looking': 1, 'care': 101, 'motown-singing': 1, 'floppy': 1, 'lunghi': 1, 'cardboard': 7, 'fun-loving': 3, 'id4': 8, "god'": 1, 'psychotics': 1, 'protigi': 1, "ali's": 1, 'generals': 1, 'tolerated': 2, 'professional': 28, 'demographic': 1, 'imitations': 4, 'unperturbed': 2, 'doo-doo': 2, 'gravity': 5, 'glamourous': 1, 'lutz': 2, 'fees': 3, 'swallowed': 3, 'purchase': 4, 'pace': 65, 'discounted': 1, 'leiter': 1, "jfk's": 1, 'shave': 4, 'nation': 14, 're-inforced': 1, 'pleased--in': 1, "enclave's": 1, 'magician': 4, "anderson's": 5, 'savannah': 3, 'happy': 92, 'denton': 1, 'fallible': 2, 'peek': 2, 'monk': 3, 'knife-wielding': 1, 'musicians': 10, 'pleasant': 26, 'bagger': 1, "pignon's": 1, 'venetian-looking': 1, 'extents': 1, 'trinidad': 1, 'fiona': 8, "life's": 7, 'churned': 1, 'achievements': 6, 'runaway': 2, 'outsmart': 2, 'bloodied': 3, 'cheng': 1, 'sunsets': 3, 'cronfronting': 1, 'gradisca': 1, 'schemer': 1, 'culture': 46, 'vegas-style': 1, 'zack': 4, 'epidemic': 1, 'seldon': 1, 'bridging': 1, 'facet': 1, 'soppy': 2, 'elevate': 2, 'above-average': 2, 'pompous': 7, 'pinter-esque': 1, 'quibbles': 3, 'lahore': 1, 'ginger': 3, 'mckay': 1, "horner's": 5, 'skinhead': 4, 'statuesque': 1, 'novelty': 3, 'bask': 1, 'metaphor': 11, 'forefront': 6, 'typecast': 4, 'embarrassing': 9, 'vatican-sponsored': 1, 'offscreen': 2, 'bleakness': 1, 'ancient-sounding': 1, 'merpeople': 1, 'sustain': 10, "1973's": 1, "fat's": 2, "pyle's": 1, 'spectacles': 3, 'loneliness': 11, 'midwestern': 3, 'arye': 1, 'maury': 4, 'mass': 17, "somebody's": 1, 'helpless': 11, 'respective': 13, 'dryburgh': 1, 'fleetingly': 2, "1999's": 5, 'hams': 2, 'mitchum': 2, 'emoting': 3, 'nice': 124, 'costumes': 33, 'member': 54, 'spurted': 1, 'cachet': 1, 'writers/directors': 2, 'radically': 3, 'sextet': 1, 'zi-yi': 1, 'swell': 3, 'marine': 6, 'huison': 1, 'hours': 89, 'peppy': 4, 'supervise': 1, 'composed': 18, 'jar-jar': 4, 'endeavors': 4, 'escaped': 10, 'disobey': 1, 'squeaker': 1, 'demonic': 3, 't2': 2, 'hagakure': 1, 'fusion': 3, 'bystanders': 2, 'sallah': 1, 'mid-life': 5, 'root': 25, 'pretty-boy': 1, 'companion': 12, 'filling': 11, 'venerated': 1, 'deadpan': 11, 'uk': 4, "mamet's": 4, "ann's": 1, 'truman': 17, 'massironi': 2, 'llamas': 1, "nobody's": 3, 'destinies': 1, '1959': 3, 'acute': 1, 'evil-eyed': 1, 'horizon': 9, 'borrowers': 1, 'go-between': 1, 'dismissed': 9, 'tsui': 2, 'talker': 1, 'dissapointly': 1, 'dunn': 3, 'recieve': 1, 'relation': 2, 'grades': 2, 'smiles': 9, 'knell': 1, 'cappuccino': 1, 'recognise': 1, 'polarized': 1, 'gentile': 1, 'farmer': 6, "africans'": 1, 'lucille': 1, 'lashings': 1, 'encased': 1, 'milquetoast': 2, 'phobic': 2, 'nationalism': 1, 'bordertown': 1, 'heart-breaking': 1, 'kite': 3, 'ganz': 4, 'bothering': 3, 'accentuates': 2, 'keith': 4, 'pasty': 1, 'f-18': 1, 'cassandra': 2, 'cool--and': 1, 'theorists': 1, 'perry': 3, 'crate': 1, 'twisted': 26, 'teaming': 6, "packer's": 1, "`plump'": 1, 'kristin': 3, 'overacted': 1, "who'd": 2, 'poll-takers': 1, 'hypodermic': 2, 'disney-animation': 1, 'hanoi': 2, 'fabric': 8, "episode's": 1, 'indiscretion': 1, 'presentation': 15, 'tiranny': 1, "period's": 1, 'intensive': 1, 'requesting': 1, "guinness's": 1, 'patience': 11, 'orphanage': 2, 'awry': 12, 'veronica': 6, 'gorilla': 5, 'protect': 15, 'chopped': 1, 'tack': 2, 'sleaze-fest': 1, 'leather-clad': 1, 'holmes': 9, 'riders': 1, 'vice-president': 2, 'recognized': 13, 'petersburg': 1, 'comedy--it': 1, 'rainman': 1, 'softly': 2, 'well-kept': 1, 'conducted': 6, 'maggie': 11, "wilson's": 1, 'thai-born': 1, 'stargate': 5, 'adequate': 17, 'monarch': 2, "1998's": 11, 'usual': 73, 'ineffective': 7, 'concede': 2, 'repaint': 1, 'authority': 17, 'miracurously': 1, 'incredible': 54, 'eponine': 1, "tidyman's": 1, 'out-and-out': 2, "audiences'": 3, 'recounting': 1, 'aborbed': 1, 'doubt': 77, 'constantine': 1, 'pics': 2, 'scrap-pile': 1, 'schizopolis': 1, 'in-your-face': 3, 'prophet': 2, 'sakari': 1, 'asexual': 2, 'costners': 1, 'tigers': 2, 'madyline': 1, 'weighing': 1, 'hurricanes': 1, 'opening': 118, 'piece': 103, 'saito': 2, 'malicious': 3, 'cloud-backed': 1, 'upgrades': 1, 'disown': 2, '1/2th': 1, 'mecca': 1, 'park': 47, 'render': 3, "goro's": 1, 'immortal': 7, 'reversal': 4, 'consumption-crazy': 1, 'warp': 3, 'soc': 1, 'venting': 3, "master's": 2, 'kamikazes': 1, 'bump': 2, 'jaded': 5, 'internet-like': 1, 'carefully-crafted': 1, 'hole-in-the-wall': 1, 'excited': 11, 'scion': 1, 'mac': 1, 'pakistan--they': 1, 'conflicted': 9, 'camp': 33, 'ding-dong': 1, 'stephanopolus-style': 1, 'penitentiary': 1, 'salesperson': 1, 'tabasco': 1, 'rowlf': 1, 'desmond': 6, 'refined': 4, 'mahoney': 3, 'matzo': 1, 'originally': 34, 'detailed': 24, 'raunchy': 1, 'swordsman': 1, 'long-abandoned': 1, 'proud': 18, 'threaten': 6, 'aurora': 1, 'jonny': 3, 'scripters': 1, 'housewife': 7, 'rocco': 1, 'portland': 1, 'bumbling': 16, 'hybridizes': 1, 'calrissian': 1, 'menacingly': 1, '$5000': 1, 'seam': 1, 'waif': 1, 'chaste': 2, 'beware': 1, 'hi-fi': 1, 'flight': 18, 'lugubriously': 1, 'sundance': 11, 'cavorting': 2, 'relaxed': 3, 'doped-out-eye-view': 1, 'plunder': 1, 'crackling': 1, 'degenerating': 1, 'fringe': 1, "boat's": 1, 'roy': 4, 'spoon-fed-entertainment': 1, 'hunters': 1, 'this--i': 1, 'deterent': 1, 'placards': 1, 'late-father': 1, 'mitchell': 14, 'brained': 1, 'manolo': 1, 'dismiss': 8, 'revelation--when': 1, 'spock': 1, 'crafts': 1, 'weekend': 20, 'secular': 1, 'efficiently': 3, 'fifths': 1, 'sufis': 1, 'drafts': 1, 'hindrance': 3, 'powered': 4, 'camaraderie': 3, 'lavender': 1, 'sheila': 2, 'trustworthy': 2, 'intensifies': 2, 'perpetually-changing': 1, 'clothesline': 1, 'ultraman': 1, 'ironically': 17, 'deemphasize': 1, 'scary': 44, 'windows': 7, "hirst's": 1, 'quick-tempered': 1, 'noteworthy': 8, 'remorse': 4, "namesake's": 1, 'adamson': 1, 'brain-over-brawn': 1, 'forgets': 9, 'longbaugh': 1, 'buisness': 1, 'marilyn': 6, 'selection': 7, 'embraced': 3, 'all-out': 3, 'writer/director/star': 1, 'copasetic': 1, 'specializing': 1, 'rendezvous': 2, 'meticulousness': 1, 'satisfactory': 4, 'alongside': 14, 'therapy': 9, 'tipped': 1, 'thereus': 1, 'eight-year': 1, 'gish': 2, 'muslims': 2, "dead'": 1, 'hortense': 1, 'skips': 2, 'torching': 1, 'devise': 1, 'test': 30, 'epigrams': 1, 'chop-socky': 2, 'doubtless': 1, "jessica's": 1, 'elisabeth': 4, 'feuds': 1, 'paranoid': 16, 'twotg': 1, 'cocaine-i': 1, '6%': 1, 'disillusionment': 2, 'ignorant': 7, 'ed-209': 1, 'batteries': 1, 'plague': 4, "short's": 1, 'tenant': 1, 'rosenberg': 1, 'belgium': 1, 'essence': 18, 'practical': 9, 'soderbergh': 3, 'carolina': 6, 'drab': 5, 'anti-slavery': 1, 'free-spirit': 1, 'attribute': 1, 'heart-sweeping': 1, 'grungers': 1, 'foil': 6, 'eugene': 14, 'tagline': 7, 'audience': 273, 'experiencing': 10, 'ladder': 6, 'spook': 2, 'trailed': 1, 'water': 55, 'second-rate': 3, 'complexion': 2, 'hard-rock': 1, 'sting': 3, 'air': 47, 'endora': 1, 'r2-d2': 7, 'windmill': 1, 'boris': 3, 'keyboard': 1, 'self-help': 2, 'preceeded': 1, 'idolizing': 2, 'cassanova-ness': 1, "friedkin's": 1, 'hairdresser': 2, 'ngo': 1, 'altough': 2, 'reassess': 1, 'liam': 15, 'flealands': 1, 'reclaim': 2, 'rimini': 1, "chad's": 1, 'overdirected': 1, 'bigelow': 1, 'distancing': 2, 'studio': 41, 'magnificently': 2, 'c-3p0': 1, 'monosyllables': 1, 'refreshed': 1, 'quits': 6, 'facto': 3, 'waitress': 13, 'festival': 26, 'third-class': 2, 'provided': 33, 'pitcher': 1, 'catchiness': 1, 'laid': 18, 'package': 10, 'shouts': 7, 'commercials': 9, 'gripping': 18, 'fullfill': 1, 'domingo': 1, 'unison': 3, 'spending': 22, 'withholds': 2, 'beats': 15, 'mpaa': 13, 'checkmate': 1, 'tone/mood': 1, 'diverts': 1, 'generalized': 2, 'severe': 8, 'gluttony': 1, 'hyper-silly': 1, 'shit': 13, 'pedestrian': 5, 'stagnant': 3, 'review-': 1, 'divinity': 1, 'eve': 9, 'replaying': 2, 'no-goodniks': 1, 'antonio': 6, 'full-blown': 4, 'transform': 5, "clayton's": 1, 'fronting': 1, 'skunk': 1, 'fascinating': 60, 'total': 28, 'paddock': 1, 'sects': 2, 'cylinders': 2, 'sarris': 1, "`pig-keeper'": 1, 'santostefano': 1, 'motorist': 1, 'clocking': 1, 'henchmen': 4, 'pursuing': 6, 'rosaleen': 1, 'competence': 1, 'revisit': 3, "'we": 1, 'man-hating': 1, 'tues': 1, 'connotations': 2, 'rage': 18, 'paralized': 1, 'protestants': 2, 'pertinent': 3, 'intros': 1, 'multiplots': 1, 'rooney': 1, 'sustaining': 2, 'intuitively': 2, 'perverse': 4, 'spader': 3, 'jerky': 6, 'keeping': 40, 'take-no-prisoners': 1, 'helping': 15, 'doubtful': 4, 'heros': 3, 'bounce': 2, 'choice': 64, 'explosive': 10, 'spa': 1, 'panteliano': 1, 'multi-form': 1, 'out-thinking': 1, 'tina': 1, "grace's": 4, 'western-like': 1, 'technically': 11, 'punxsutawney': 1, 'unprotected': 2, 'slide': 6, 'bartleby': 2, 'chrysler': 2, 'seal': 1, 'sure-handed': 1, 'rowlands': 3, 'behavious': 1, 'world-class': 3, 'covert': 2, 'relaxation': 3, 'moviegoers': 8, 'norrington': 1, 'perfectly-groomed': 1, 'alcoholism': 3, 'libby': 6, 'scavengers': 1, 'coalition': 1, 'ceremony': 7, 'karate': 4, 'non-british': 1, 'golly': 1, 'whitworth': 1, 'groundhog': 4, 'steaminess': 1, 'uncomfortable': 14, 'eatery': 1, "lumet's": 2, 'freedom--in': 1, 'leo': 6, '3-5': 1, 'lenser': 1, 'roses': 4, 'motifs': 1, 'fatality': 1, 'ugly': 22, 'paradiso': 1, 'cloke': 3, 'bad': 234, 'ooh': 1, 'whoah': 2, 'curt': 1, 'represent': 14, 'you=92re': 1, 'schwimmer': 2, 'bounces': 1, 'breakage': 1, "three's": 1, 'brightly-colored': 1, 'kapur': 1, 'visions': 13, 'self-righteous': 5, 'switch': 15, 'filmmaker': 28, 'geniuses': 4, 'whic': 1, 'ronald': 5, 'schellhardt': 1, 'beautician': 1, 'transfixed': 1, 'chung': 1, "kimmy's": 1, 'passage': 8, 'prophetic': 2, 'seamlessly': 6, "truman's": 6, 'small-scale': 1, 'mach': 1, "rockin'": 4, 'comicbook': 1, 'literati': 1, 'defendants': 1, 'hypersleep': 3, 'autumnal': 1, 'spate': 1, 'needless': 14, 'uneasiness': 2, 'fleugel': 1, 'edges': 5, 'two-hour-plus': 1, 'categorizing': 1, 'snuff': 4, 'suggest': 28, 'auxiliary': 1, 'antoine': 1, 'phyllida': 1, 'sixteen': 6, "dixon's": 1, 'adopter': 1, 'executives': 4, 'tulips': 2, 'seagrave': 1, 'greenlighted': 1, 'innuendos': 2, 'wink-wink': 1, 'circulate': 1, 'incriminating': 2, 'sleuthing': 1, 'weber': 1, 'outrageously': 4, 'formulaic': 14, '1967': 4, 'jackson/travolta': 1, 'lazy-as-always': 1, 'authoritarianism': 1, 'lamentable': 1, 'fracture': 2, "'maybe": 1, 'francisco': 7, 'chronicles': 16, 'chasers': 1, 'italian': 24, 'plate': 5, 'orwellian': 1, 'copper': 4, 'marshmallows': 1, 'instill': 1, '`mr': 1, 'impressing': 1, 'bmw': 1, 'cult-classics': 1, 'idle': 8, 'housed': 1, 'favored': 2, 'ricardo': 2, 'looming': 5, 'cleaned': 2, 'pricking': 1, 'friedman': 1, 'transfers': 2, 'hammond': 2, 'worked': 51, 'mountainsides': 1, 'aficionados': 5, "`tarzan'": 1, 'excelled': 1, 'awoken': 1, 'gladiators': 1, 'milano': 2, 'pr': 3, 'downside': 9, "salsa'd": 1, "girlfriend's": 2, 'porn-actress-turned-porn-producer': 1, "city'": 1, "baby's": 3, 'tree': 6, 'blonde': 11, 'creations': 7, '81-minute': 1, 'dreadfully': 3, 'wagnerian': 1, "crackin'": 1, 'jenson': 1, 'intriguingly': 2, "burrough's": 2, 'coincides': 3, 'false': 25, 'sneakers': 1, 'liberty': 9, 'scatalogical': 1, 'banished': 7, 'coal-mining': 1, '15': 22, 'cruelty': 12, 'cut': 67, 'lungs': 2, 'hardest': 3, 'shadowy': 8, 'minority': 4, 'imagination--and': 1, 'fifty-three': 1, 'jonathon': 1, 'borrow': 2, 'aguirresarobe': 1, 'focus': 69, 'thrilling': 28, 'dullness': 1, 'bakshi': 1, 'heh': 1, 'gingerly': 1, 'laugh-filled': 1, 'culture-whiz': 1, 'ahmed': 3, 'wrongdoing': 2, 'patricia': 8, 'checkpoint': 2, 'deplorable': 2, 'fad': 3, 'heights': 8, 'recommends': 1, 'edison': 2, 'eric': 15, 'blackmailed': 3, 'nonexistant': 2, 'vowed': 2, 'trumping': 2, 'prejudice': 5, "duty'": 1, 'fierstein': 4, 'unforgettable--perhaps': 1, 'delight': 30, 'beds': 2, 'jimmy': 21, 'plummets': 2, 'gerry': 3, 'contraptions': 1, 'delectable': 4, 'credited': 7, 'drill': 2, 'emmet': 1, "shrek's": 2, 'carrion': 1, 'cheesily': 1, 'finishes': 5, 'kurring': 1, 'honourable': 2, 'attention-grabbing': 1, 'legitimately': 1, 'possibility': 14, 'lestercorp': 1, "stravinsky's": 2, 'midsection': 1, "scientist's": 1, 'anticipate': 3, 'introduces': 24, 'melodramtic': 1, 'rico': 7, 'yacht': 1, 'aussie': 1, 'permanence': 1, 'koren': 1, 'luxuries': 1, 'scudding': 1, 'dollar': 13, 'proverbial': 4, 'demeaned': 1, 'sub-human': 1, 'carpet': 2, 'mykelti': 1, '800': 1, 'step': 35, 'idealism': 2, 'stuttering': 2, 'meir': 1, 'ant': 3, 'adafarasin': 1, 'backside': 1, 'holden': 5, "danna's": 1, 'confronted': 11, '153': 1, 'troopers': 14, 'wh': 1, 'cornelius': 1, 'uttering': 3, 'instructions': 3, 'punch': 27, 'iben': 1, 'oddness': 2, 'cards': 17, 'sitations': 1, 'prosecutes': 1, 'touching': 51, 'seattle-based': 1, 'joanne': 2, 'invites': 8, 'chemical': 2, 'accompanying': 8, "vacano's": 1, '48th': 1, 'translated': 1, 'staggeringly': 1, 'collapses': 4, 'zoo': 1, 'alamo': 1, 'brable': 1, 'cost': 10, 'g-string': 1, 'apple': 13, '4/23/99': 1, 'frixos': 1, 'offended': 13, 'disorder': 7, 'ogled': 1, 'rathbone': 1, 'captivated': 5, 'abrahams': 2, 'supernatural': 18, 'clinically': 1, 'donuts': 2, 'messageable': 1, "paramount's": 2, "aurelius'": 1, 'intellect': 3, 'carat': 1, 'curiously': 4, 'silly': 47, 'friendship': 37, 'sporty': 1, 'chestnut': 1, 'balderdash': 2, 'stands': 60, 'leibowitz': 1, 'supplying': 2, 'skitters': 1, 'press': 25, 'contemporizing': 1, 'none-too-subtle': 2, 'scariest-looking': 1, 'initiative': 2, 'roadblocks': 1, 'reason': 127, "person's": 11, 'pining': 1, 'jee-zus': 1, 'dedication': 8, 'southern': 19, 'viewed': 30, 'intersecting': 1, 'reinvented': 3, 'undoubtful': 1, 'driller': 1, 'geneva': 3, 'dennis': 26, 'bagpipes': 1, 'grey': 1, 'kahuna': 1, 'change-of-pace': 3, 'telephone': 5, 'pro-slavery': 1, 'personalities': 19, 'out-of-the-way': 1, 'nosedive': 2, 'erupting': 2, "bank's": 1, 'hunted': 3, 'port': 3, 'antithesis': 4, 'dench': 7, 'at-odds': 1, 'fires': 5, 'cameraman': 6, 'opted': 1, 'soon-to-be-classic': 1, 'charging': 1, 'pros': 2, 'perversely': 3, 'tacked': 7, 'fly-overs': 1, 'cheerfully': 5, 'cooked': 1, 'lambeau': 5, 'recruiter': 1, 'semi-predictable': 1, 'persons': 3, 'departs': 4, 'scaring': 4, 'performers': 21, "tatyana's": 1, 'leigh': 10, 'inseparable': 4, 'peterson': 2, "salonga's": 1, 'front': 55, "mikhail's": 1, 'droning': 1, 'yell': 2, 'johnson': 11, 'kuzco': 1, 'levels': 22, 'montoya': 2, 'doltish': 1, 'enamored': 4, 'symbolic': 9, '`where': 1, 'dead-ends': 1, 'dancers': 2, 'mocked': 3, 'hero/antihero': 1, 'civil': 16, 'lightheartedness': 1, 'inputs': 1, 'pov': 1, 'considerable': 11, 'holm': 8, 'shoot': 35, 'bankrupt': 1, 'paralyzingly': 2, "worker's": 1, 'angle': 20, 'told': 95, 'sustains': 1, 'deep': 87, 'flattered': 2, 'zar': 1, 'winner': 30, 'whales': 2, 'leaning': 2, 'fools': 3, 'recovers': 2, "'funny": 1, 'voices/accents': 1, 'personality-impaired': 1, 'complementary': 3, 'venetian': 1, 'arabs': 2, 'indeterminate': 3, 'flawlessly': 2, 'stipe': 1, 'tech': 4, 'intimidate': 2, 'groundlings': 1, 'selfish': 15, "renee's": 1, 'days': 114, 'eye-opener': 1, 'area': 20, 'gorefest': 2, 'milagro': 1, 'ranges': 1, 'twig-sculptures': 1, 'tease': 3, 'would-be-girlfriend': 1, 'demeanor': 14, 'testify': 1, 'resuce': 1, 'downers': 1, 'subtly': 7, "who're": 1, 'conspicuously': 1, 'thecatrical': 1, 'device': 34, 'addictive': 1, 'jennifer': 24, "robocop's": 1, 'aborginal': 1, 'piss-poor': 1, 'producing': 14, 'petstore': 1, 'callousness': 1, 'emmerich': 10, 'mandella': 1, 'walkin': 1, 'nusevic': 1, 'trickster': 1, 'holism': 1, 'truthful': 6, 'clam': 1, 'juan': 1, 'gaetan': 1, 'upteenth': 1, 'diggers': 1, 'love-': 1, 'back-to-': 1, 'slithery': 1, 'shifts': 16, 'cliched': 21, 'hand-drawn': 2, 'consequences': 22, 'esposito': 1, "renault's": 1, 'oohs': 1, "jeanne's": 1, 'stubborness': 1, 'swallow': 6, 'apologies': 3, "bilal's": 1, 'advisable': 2, 'perreault': 2, 'chivalrous': 1, 'vote': 6, 'shpadoinkle': 1, 'breech': 2, "'clerks'": 1, 'kentucky': 6, 'enlivens': 1, "l'auto": 1, "tanner's": 1, 'half': 126, 'slow': 70, 'tweeder': 1, 'blemheim': 1, 'marred': 4, '1930s': 3, "bernstein's": 1, 'fiascoes': 1, 'newborn': 1, 'sakaguchi': 1, 'african': 12, 'dispenses': 3, 'sighing': 1, 'decadent': 2, 'churchman': 1, 'dystopic': 1, 'courting': 2, 'vampire': 17, 'eraserhead': 3, 'meander': 3, 'angular': 1, 'analyzed': 2, 'intertwined': 4, "3's": 1, 'emperor': 13, 'al': 23, 'motherload': 1, 'mistakenly': 4, 'equal': 26, 'factions': 4, 'misery': 9, 'mountains': 10, 'pay': 54, 'variable': 3, 'family-run': 1, 'obscura': 1, 'after-effects': 1, 'chops': 1, 'shoot-em-up': 2, 'rusty': 1, 'huggan': 1, 'slowly': 71, 'slo': 1, 'beast': 16, 'elicits': 2, 'pastel': 2, 'weapons': 18, 'anxious': 4, 'descendent': 2, 'sprints': 2, 'reiner': 3, "1990's": 4, 'understand': 77, 'technology--the': 1, 'pauline': 1, 'modern-day': 12, 'statesmen': 1, 'slade/wild': 1, "tracks'": 1, 'rough': 15, "romeo's": 2, 'bread-and-butter': 1, "f-18's": 1, 'condor': 2, 'dozen': 22, "regiment's": 1, 'tunnel': 6, 'smartly': 4, 'controversy': 10, 'unremittingly': 1, 'pun': 13, "zeffirelli's": 1, 'loggins': 1, '3d': 1, 'glaze': 1, 'mace': 1, 'epic': 49, 'vasquez': 2, 'wince': 2, 'lector': 1, 'existentialist': 1, 'borders': 8, 'backwards': 3, 'future': 98, 'peyton': 1, "ulysses'": 1, 'posits': 1, 'ambivalence': 2, 'importance': 22, 'opposing': 5, 'fennyman': 1, 'wonderous': 1, 'loans': 1, '_death': 1, 'retrieving': 1, 'kennedyesque': 1, 'bigshot': 1, 'six-foot': 1, 'unanswered': 5, 'cholodenko': 1, 'molesting': 1, 'scrooge': 1, 'groundbreaking': 11, "general's": 2, 'ex-love': 1, 'island': 30, 'recorded': 8, 'blurb': 1, 'won=92t': 1, 'composing': 3, "'authenticity'": 1, 'nutrition': 1, 'graeme': 2, 'drop': 16, '1990s': 9, 'abortion': 4, 'calvinistic': 1, 'addressed': 3, 'utter': 18, 'support': 43, 'faults': 16, 'examinations': 2, 'compares': 3, 'cassel': 2, 'romanticise': 1, 'diarist': 1, "taxi's": 1, 'laserdisc': 5, 'smirk': 6, 'spaceport': 3, 'potentialities': 1, "gary's": 1, 'ski': 3, 'farm': 15, 'conclusions': 4, 'wanton': 2, 'touches': 34, 'lifesaving': 1, 'shoulder-length': 2, 'alienated': 4, 'effective': 113, 'sheree': 1, 'siding': 1, 'caan': 4, 'guess': 57, 'tasteful': 2, 'aliens-as-human': 1, 'sexes': 2, 'sofa': 1, 'holocaust-ravaged': 1, 'succeeds': 46, 'avenge': 2, 'passions': 6, 'vampire-fest': 1, 'sigel': 1, 'short-': 1, 'reaffirming': 1, 'wrote': 48, 'nom': 1, 'shutting': 1, 'summing': 1, '-school': 1, 'trim': 4, 'k-filled': 1, 'blackout': 1, 'chomped': 1, 'pant-wetter': 1, "f'words": 1, 'active': 8, 'multi-faceted': 1, 'pulls': 34, 'assigns': 2, "tango's": 1, 'breath': 14, 'trappers': 1, 'caveat': 2, 'journeyman': 1, 'gerald': 3, 'picketers': 1, 'sleezo': 1, 'republican': 1, 'paternalistic': 3, '65': 1, 'bernard': 9, 'blue-ish': 1, 'erroneous': 1, 'inseperable': 1, 'monstrously': 1, 'common': 61, 'over-zealous': 1, 'immigrants': 4, 'workplace': 1, 'stamp': 4, 'inventing': 2, 'shed': 6, 'netted': 2, 'far-fetched': 3, 'protecting': 4, 'chunnel': 1, 'unfocused': 2, 'silkwood': 1, 'trigger-happy': 2, 'rainbow': 2, 'scorese': 1, 'eruption': 1, 'horror-loving': 1, 'aux': 1, 'mothering': 1, 'straight-arrow': 1, 'definition': 7, 'authentic': 22, 'unsentimental': 1, 'beckinsale': 3, 'hell': 68, 'coifed': 1, 'chains': 3, 'hummingbird': 1, 'vllainous': 1, "knowin'": 1, 'misfits': 2, 'gross-out': 5, 'trade-in': 1, 'bonafide': 2, "`horror'": 1, 'alderaan': 1, 'consort': 1, 'praise': 19, 'peril': 2, 'trevor': 1, 'f/x': 3, "korben's": 1, 'concise': 1, 'metamorphosis': 2, 'mind-blowing': 3, 'near-by': 1, 'lumber': 1, 'contenda': 1, 'improbabilities': 1, 'shaping': 3, 'barn': 5, 'magnifying': 2, 'interrogate': 1, 'roadblock': 1, 'police-woman': 1, 'zeffirelli': 1, 'hamm': 3, 'backer': 2, 'bright-eyed': 1, 'na': 7, 'harvard-schooled': 1, "girl's": 5, 'monogamy': 2, 'accumulated': 3, 'whisking': 1, 'hessian': 1, "chareau's": 1, 'outta': 2, 'absorbant': 1, 'sled': 1, 'win': 54, 'danson': 1, 'buffoon': 2, 'ruthless': 15, 'dab': 1, 'puzzle': 8, 'smoothes': 3, 'moving': 91, "bogart's": 2, 'poseurs': 2, "guggenheim's": 1, 'war-crazy': 1, 'paul': 63, 'attend': 12, 'coloreds': 2, 'inhuman': 4, 'guitars': 1, 'idealist': 1, '37th': 1, 'contractually-obligated': 1, 'finding': 42, 'farms': 1, 'sideshow': 3, 'gregorio': 1, 'mastered': 4, 'samoan': 1, 'entertained': 13, 'columbian': 3, 'honoring': 2, 'intro': 1, 'lovestory': 1, 'diesl': 1, 'non-sexual': 1, 'nominees': 4, 'docks': 1, 'exorbitance': 1, 'sank': 3, 'receives': 15, 'internment': 1, 'psychiatrist': 6, 'miriam': 6, 'protege': 3, 'singers': 8, 'brethren': 2, 'affront': 1, 'sums': 8, 'dorky': 2, 'dam': 4, 'publishers': 1, "carpenter's": 2, 'mostly-unrelated-storywise': 1, "max's": 6, 'matthews': 1, 'paramount': 8, 'manufacturing': 1, 'rooftops': 1, 'decapitation': 1, 'longshank': 2, 'radha': 1, 'competiton': 1, 'infuses': 3, 'ended': 29, 'nightingale': 1, 'flanks': 1, 'marko': 1, 'automated': 1, 'unite': 5, 'feng': 1, 'anytime': 4, 'oxymoron': 2, 'tatooine': 7, "prince's": 1, 'pick-ups': 1, 'pulsing': 2, 'trask': 2, 'budgeted': 2, 'temperature': 3, 'walk-on': 2, 'bested': 1, 'unsurprisingly': 3, 'lapse': 1, 'thawed': 1, 'spiritualistic': 1, 'recounts': 1, 'preoccupation': 2, 'circus': 4, 'immeasurable': 1, 'motormouth': 1, 'rigour': 1, 'slattery': 1, 'dead-on': 8, 'shells': 3, 'direction': 135, 'downtrodden': 4, 'gang-bangers': 1, 'feelings': 39, 'kidnapers': 1, 'postino': 1, 'subject': 78, 'cringe-inducing': 1, 'papas': 1, "'moving'": 1, 'disguise': 5, 'innit': 1, 'occurrence': 5, 'fort': 3, 'bridget': 11, 'removes': 2, 'dove': 3, 'parents': 76, 'repetitiveness': 2, 'monopoly': 3, 'reemerging': 1, 'digress': 2, 'buckles': 2, 'zorg': 1, 'snobs': 1, 'overt': 3, '15th': 1, 'exists': 17, 'mira': 2, 'put--it': 1, 'ethic': 2, 'preachy': 5, 'stocky': 1, 'scrutinize': 2, 'zeus': 2, 'attacking': 8, 'keyed': 1, 'him/herself': 2, 'crusaders': 3, 'frightened': 16, 'bruce': 47, 'wrong--again': 1, 'teens': 12, 'gidley': 1, 'reap': 3, "50's": 5, 'romper': 1, 'cripple': 2, "cinematographer's": 1, 'valentine_': 1, 'councilmen': 1, 'foxy': 2, 'painstakingly': 4, 'sacrificial': 1, 'logic': 22, 'rambles': 1, 'nonsense': 3, 'specfically': 1, 'standing': 34, 'crafty': 2, 'cold': 70, 'misha': 1, 'champion': 12, 'latter-day': 1, 'breathing': 9, 'extraordinaire': 2, 'serviceable': 3, 'sethna': 1, 'action-comedies': 1, 'carr': 1, 'brand': 15, 'juvenile': 5, 'full-grown': 1, 'recklessness': 2, 'lu': 1, 'runner': 8, 'pleasantville': 6, 'text': 14, 'yagher': 1, 'hematologist': 1, 'nigeria': 1, 'apt': 10, 'rosy': 3, 'trailer': 27, 'ever-affable': 1, 'mine-cart': 1, "human's": 2, 'pulitzer-prize': 1, 'roth': 9, 'bassan': 1, 'mind': 177, 'titus': 1, 'goofball': 2, 'scathingly': 2, 'remaining': 24, 'lemmon': 5, 'love-interest': 2, 'disgrace': 3, 'clear': 87, 'riding': 12, 'soliloquies': 1, 'planning': 7, 'ego': 4, "gates'": 2, 'eye-popping': 2, 'musings': 1, 'election': 10, 'mogul/psychotic': 1, 'cate': 4, "schumacher's": 1, 'spiraling': 1, 'urinate': 3, 'brides': 1, 'cora': 1, 'adept': 7, 'juggles': 3, 'themes': 47, 'meadows': 1, 'lush': 14, 'edited': 13, 'love-it-or-hate-it': 1, 'coppolas': 1, 'sub-standardly': 1, 'preppy': 1, 'burial': 3, 'serpico': 1, 'pvt': 2, 'flatulence': 1, 'leftover': 3, 'wildflowers': 1, 'box': 55, 'confused': 42, 'degenerative': 1, "heroine's": 2, 'cutthroat': 4, 're-evaluated': 1, 'folding': 2, 'manipulations': 3, 'helene': 1, 'steamier': 1, 'treks': 2, '---agent': 1, 'piracy': 2, 'curb': 2, 'performed': 18, "amazing's": 1, 'screamingly': 1, 'spine-chilling': 2, "jock's": 2, '`taxi': 1, 'fawning': 1, 'amateurs': 1, 'jon': 21, 'aide': 4, 'micelli': 1, 'grouch': 1, 'mechanically': 1, 'long-dead': 1, 'vehemently': 1, 'double-sided': 2, 'applauded': 4, "gaz's": 1, 'grazes': 1, 'shallow': 29, 'taste': 28, 'anxiety-ridden': 1, 'bert': 1, "revolution's": 1, 'extremist': 2, 'terrific': 72, 'manh': 1, 'christ': 11, 'light': 93, 'bumbles': 1, 'well-off': 3, 'distinguish': 3, "hanks'": 3, 'chief': 21, 'kario': 1, 'multimillionaire': 1, 'toothy': 1, 'moralize': 1, 'sachs': 1, 'jerusalem': 3, 'colliding': 4, 'freewill': 1, 'truth': 61, 'constitute': 1, 'deficiency': 1, 'disintegration': 2, "libby's": 1, 'bello': 2, 'once-breeding': 1, 'bantering': 2, 'kindhearted': 3, 'rounding': 5, "hotel's": 1, 'newly-reunited': 1, 'compensation': 3, 'receptive': 1, 'early-warning': 1, "maximus'": 2, 'weakling': 2, "lucasfilm's": 1, 'gordon-levitt': 2, 'rais': 1, 'enforce': 2, 'anecdotes': 5, 'janssen': 4, 'garbageman': 1, 'ottorino': 2, 'eliminating': 1, 'louisiana': 4, 'backers': 1, 'float': 3, '`everyone': 1, 'on-again': 1, 'overnight': 3, 'purest': 2, 'openly': 8, 'un-hip': 1, 'laertes': 1, 'intoxicated': 3, 'somerset': 2, 'release--john': 1, 'trusted': 7, 'paz': 1, 'sympathizes': 1, 'welsh': 4, 'jerks': 2, 'brainless': 6, 'demented': 4, 'semi-successful': 1, 'extraordinary': 35, 'revives': 2, 'misuse': 1, 'mistaken': 11, 'restrained': 11, 'robby': 2, 'croatian': 1, 'prays': 3, 'henriksen': 5, 'pumping': 7, 'baths': 1, 'security': 24, 'arachnid-like': 1, 'jack/rose': 1, 'establishment': 2, 'tender--de': 1, 'all-seeing': 1, 'picket': 1, 'harumph': 1, 'virgins': 1, 'boasted': 3, 'cabbie': 1, 'blindingly': 1, 'classmate': 4, 'straddle': 1, 'glue': 5, 'lovey': 1, 'ranks': 15, "claudia's": 1, 'enthralled': 4, 'styling': 1, '10': 46, 'ballistic': 1, 'revengeful': 1, 'literally': 42, 'petrified': 1, "'s": 31, 'pattern': 12, 'dyed': 2, 'widens': 1, 'birch': 6, "jean's": 1, 'dubarry': 1, 'seats]': 2, 'rescuers': 2, 'sherlockian': 1, 'sheryl': 2, 'angela': 14, 'tear-jerking': 1, "schrader's": 3, 'tediousness': 1, 'pf': 1, 'doozy': 2, 'majestically': 1, 'pegged': 1, 'soda': 1, "carol's": 2, 'railed': 1, 'plumber-turned-private': 1, 'eighteen': 4, 'escapade': 2, "woman's": 7, 'edith': 2, 'manuscript': 1, "`fish-out-of-water'": 1, 'mehrjui': 1, 'handling': 8, 'slow-dancing': 1, 'cheung': 1, 'dispel': 1, 'fueling': 1, 'ziembicki': 1, 'surgical': 1, 'hones': 1, 'sizzles': 1, 'eroticism': 3, 'cleanest': 1, 'consciouness': 1, 'suture': 3, 'bonham': 3, "fa's": 1, 'bohemian': 1, 'minutes': 182, 'petievich': 1, 'fall-down': 1, "love's": 2, 'local': 82, 'pajamas': 1, 'waltzes': 2, 'proudly': 2, 'merry': 3, 'pacino': 9, 'rozema': 1, 'blossom': 5, 'replies': 10, '_gattaca_': 1, 'congress': 3, 'exposing': 3, 'cipher': 1, 'valencia': 1, 're-christening': 1, 'cuckold': 1, 'contagious': 2, "abraham's": 2, 'groundbreakingly': 1, 'discussed': 7, 'mud': 3, 'likeability': 4, 'delaughter': 1, "'special": 1, 'constructive': 1, 'pacific': 3, "hogan's": 3, 'cartoon': 19, "music's": 2, 'consequential': 2, 'somersets': 1, 'gudin': 1, 'moores': 1, 'tondo': 1, 'larceny': 1, 'outpace': 1, 'scumbag': 2, 'mone': 1, 'dershowitz': 1, 'graphically': 1, 'neo-noirs': 1, 'escape': 67, 'robotically': 1, 'fishy': 2, 'resorts': 7, 'brainwash': 1, 'auditions': 1, 'voiceover': 5, 'whoop': 1, 'frames': 6, 'evidence': 24, 'spear': 1, 'bothersome': 2, 'stirringly': 1, 'marvelously': 6, 'yearns': 3, 'unloaded': 1, 'haas': 1, 'roller-coaster': 1, 'historian': 2, 'charitable': 1, 'chump': 1, 'unanimously': 1, 'worship': 2, 'scientists': 16, 'approved': 1, 'understandable': 11, 'slowest': 2, 'nicholas': 11, 'ruling': 3, 'undistinguished': 1, 'cursory': 2, 'starts': 126, 'disoriented': 1, 'connecticut': 6, 'unwelcome': 3, 'on-screen': 16, 'clinging': 1, 'possessed': 11, 'mancuso': 1, 'rings': 9, '75': 2, 'coaster': 3, 'vital': 13, 'clouds': 6, '`my': 1, 'extraneous': 4, 'flirts': 8, 'mos': 2, 'robbi': 1, 'catania': 2, 'competent': 8, 'oversaturated': 1, "goldberg's": 1, 'longer': 55, 'compress': 1, 'joyride': 3, 'primeval': 1, 'upstaged': 3, 'dylan': 8, 'kihlstedt': 2, 'feingold': 1, 'emphasizing': 5, 'slam': 1, 'excessively': 4, "socialites'": 1, 'three-minute': 1, 'peeing': 1, 'striker': 1, 'higgins': 1, 'unimaginative': 1, 'scarce': 4, 'trivialities': 1, 'nevers': 1, 'softly-toned': 1, 'build-up': 5, 'mid-eighties': 3, 'ballentine': 1, 'activity': 7, 'unnerve': 1, "andrew's": 1, 'cornerstone': 2, 'bojan': 1, 'bondsmen': 1, 'fourteen': 5, 'undeterring': 1, 'country-club': 1, 'quotes': 5, 'medgar': 1, "'psycho'": 2, 'gd': 1, 'lindsay': 2, 'principles': 4, 'charleton': 2, 'boiled': 2, 'underdone': 1, 'swanson': 3, 'childish': 6, 'impediment': 3, 'gotti': 1, 'wolf': 8, 'dichotomy': 3, 'coated': 1, 'reticence': 1, 'provoking': 9, 'collecting': 3, 'analytical': 1, 'mid-teen': 1, 'romanticizes': 2, 'nba': 2, 'khan': 4, 'gump': 11, 'racer': 1, 'islam': 1, 'renounce': 1, 'premises': 2, 'consciousness': 4, 'nowadays': 13, 'true--a': 1, 'uhf': 1, 'c-for-charlie': 1, 'climate': 6, 'co-producer': 1, 'surprisinly': 1, 'titillate': 1, 'odder': 1, 'wisdom': 13, 'tobolowsky': 1, 'ginseng': 1, 'artistical': 1, 'contemporary': 30, 'ballot': 1, 'considered': 49, 'danna': 2, 'embarrassments': 1, 'granddaughters': 1, 'lassie': 1, 'idiocy': 1, 'glove': 1, 'endor': 1, 'forethought': 1, "'here'": 1, 'whilst': 7, 'charm-meter': 1, 'lesseps': 2, 'bloodsuckers': 2, 'feminist': 5, "'attila'": 1, 'winking': 1, 'shang': 5, 'apperance': 2, 'rallies': 2, 'heisted': 1, 'budgets': 2, 'hey': 20, 'someodd': 1, 'mordantly': 1, 'tia': 1, 'plain': 28, 'noticing': 6, 'jefferson': 3, 'bribes': 2, 'stratford': 2, 'lurve': 1, 'redeems': 3, 'gut-wrenchingly': 1, "wender's": 1, 'boni': 1, 'fraught': 4, 'pulsating': 1, 'glimpsing': 1, 'personality': 49, 'suppose': 27, '1': 54, 'schwinn': 1, 'amfibium': 1, "meir's": 1, '_rushmore_': 1, 'burstyn': 2, 'stepchildren': 1, 'superflous': 1, 'neighborhood': 7, 'atheist': 7, 'mccabe': 1, 'heavily-accented': 1, 'gunk': 1, 're-open': 1, 'pity': 12, 'plauged': 1, 'boating': 2, 'bonehead': 1, 'sumptuous': 10, 'big-shot': 1, 'rodney': 3, 'time': 570, 'hearing': 23, 'nationality': 1, "`final'": 1, 'brendan': 9, 'pays': 20, 'hops': 1, 'legislation': 1, 'inky': 2, 'goals': 15, 'on-form': 1, 'spoofing': 2, 'ex-hubby': 1, 'pedophile': 1, 'accurate': 19, 'pearl': 4, 'thin': 31, 'surrogate': 9, 'nosy': 2, 'potency': 5, 'performance': 315, 'sheen': 8, 'linklater': 1, 'outlandish': 11, 'subconsciously': 1, 'jars': 3, 'meat': 18, 'furter': 1, 'sacrificed': 2, 'meaningful': 7, 'goodnight': 2, 'generics': 1, 'bon': 1, 'gonzo': 3, 'dexterity': 1, '_moby': 1, 'rubell': 1, 'collaborated': 1, 'veer': 3, 'daniels': 6, 'beater': 1, 'psychotic': 15, 'ups': 10, 'settlement': 5, 'pleas': 1, 'cuz': 1, 'sabine': 1, 'admittingly': 5, 'updating': 1, 'ideologically': 1, 'hardness': 1, 'chatterbox': 1, 'jingles': 1, 'alek': 2, 'bigwig': 3, 'psychosis': 5, 'vacancy': 1, 'patti': 2, 'output': 3, 'image': 54, "he'd": 7, 'farrellys': 1, 'inelegantly': 1, 'redgrave': 5, 'top-selling': 1, 'collective': 7, 'didn': 1, 'incompatible': 1, 'copolla': 1, 'beaches': 1, 'battle-worn': 1, 'cynthia': 5, 'reeves': 11, 'treasury': 1, 'omar': 3, 'tarantino': 19, 'krumholtz': 1, 'penney': 1, 'deceit': 2, "sweeney's": 1, 'julian': 1, 'idisyncratic': 1, 'shelves': 8, 'assaults': 3, 'posthumous': 2, 'songwriter': 2, 'dagger-edged': 1, 'toilet': 11, 'technobabble': 1, 'middle': 69, 'fancies': 2, 'nc-17': 13, 'macho': 12, 'shackleton': 1, 'venier': 1, '_their_': 1, 'suggestiveness': 1, 'viola': 2, 'standpoint': 5, 'association': 6, 'regretted': 1, "professor's": 2, 'contemporaries': 1, 'shorten': 2, 'compulsive': 4, 'antidote': 2, 'inaccurate': 2, 'ripping': 2, 'equipped': 4, 'faye': 2, 'directoral': 2, "dog's": 2, "jerry's": 2, 'martin': 64, 'sweat': 2, 'blokes': 1, 'intrinsics': 1, 'social': 57, 'hee': 1, 'progressive': 1, 'pitt': 14, 'naivet': 2, 'divison': 1, 'cox': 13, 'two-thirds': 4, 'mcniell': 1, 'practice': 13, 'conventions': 16, 'egotism': 1, 'multi-national': 1, "jedi's": 1, 'extend': 1, 'picture': 194, 'elimination': 2, 'foolery': 1, 'trent': 3, 'inception': 1, 'amateur': 1, 'newspaper': 9, 'nihilists': 3, 'underlines': 1, 'grace': 31, 'cia': 8, 'unpure': 1, 'moviegoing': 1, 'elusive': 6, 'resonation': 1, 'made-for-tv': 4, 'glimpses': 6, '`music': 1, 'pressure-cooker': 1, 'ranger': 4, 'sadder': 1, "millie's": 1, 'insular': 3, 'abruptly': 2, 'discussion': 14, 'co-exist': 2, 'portrtayal': 1, 'constitution': 1, 'mozambique': 1, 'whiz-bang': 1, 'amount': 61, 'seldom': 6, 'huh': 4, 'publication': 1, 'assistance': 11, 'annihilation': 3, 'celluloid': 5, 'evelyn': 3, "sister's": 4, 'friday': 10, 'forewarned': 12, 'favor': 22, "russell's": 3, 'tangled': 2, 'shakes': 5, 'mountain-climber': 1, 'exotica': 4, 'fops': 1, 'eisenhower': 1, 'freddie': 2, 'quantum': 2, 'semen': 1, 'resident': 4, 'lockup': 1, 'divergent': 2, 'squint': 1, 'voyeuristic': 3, 'singers---they': 1, "ambrose's": 1, 'koteas': 3, 'turkey': 3, 'felix': 6, 'out-of-chronological-order': 1, 'dramaturgical': 1, 'begin': 79, 'bulb': 1, 'sure-deal': 1, 'energy': 47, 'headlong': 1, 'disappeared': 6, 'dogsit': 1, 'shackled': 1, 'weren=92t': 1, 'gil': 1, 'sprang': 2, 'success': 100, 'lovitz': 5, 'damages': 2, 'waddington': 1, 'complemented': 3, 'repulsed': 4, 'catholic-rattling': 1, 'tolls': 1, "seriously-it's": 1, 'easier': 21, '121': 1, 'conceiving': 1, 'meekness': 2, 'extraterrestrials': 3, 'snipping': 1, 'vitti': 2, 'leading': 54, 'emotionally': 41, 'oasis': 2, 'dey': 1, 'lost': 133, 'winona': 7, 'centenial': 1, 'copying': 1, 'martyr': 2, 'remainder': 8, 'santa': 4, 'lionized': 1, 'slams': 1, 'monetary': 6, 'closed-in': 1, 'distracts': 1, 'juzo': 2, 'captives': 1, 'puts': 60, 'urgencies': 1, "ken's": 1, 'north': 26, 'half-empty': 1, 'roams': 1, 'naivety': 1, 'ham-radio': 1, 'cemetary': 2, 'near-ruin': 1, 'diplomatic': 1, 'conscience': 11, 'hindering': 2, 'finely-realized': 2, 'military/cia': 1, 'noses': 1, 'opposite': 33, 'ex-prison': 1, 'stivaletti': 1, 'sprawling': 7, 'near-eden-like': 1, 'firmly': 9, 'tone-perfect': 1, 'insecurities': 4, 'snoots': 1, 'furry': 1, 'kong': 21, 'spike': 11, "'gomer": 1, 'porpoise': 1, 'reawaken': 1, "blues'": 1, "seconds'": 1, 'conscience-stricken': 1, 'scholarships': 2, 'princess-guarding': 1, 'leavins': 1, 'fonz': 1, 'analytic': 1, 'buns': 1, 'azazel': 1, 'baby-sitting': 1, 'minion': 2, 'aliens': 51, 'classifies': 1, 'wandered': 2, 'gila': 1, 'earth': 74, 'mundane': 12, 'accomplishments': 4, 'grimacing': 3, 'antonia': 1, 'hellish': 5, 'salle': 1, "oates'": 1, 'sneer': 3, 'sagebrush': 1, 'hopes': 37, 'exited': 1, 'pig': 13, 'proceeders': 1, 'credo': 1, 'focuses': 30, 'expediency': 1, 'tap': 5, 'beset': 3, 'unappealing': 4, 'costner': 6, 'negotiating': 2, 'bestselling': 2, "hrundi's": 1, "summer's": 7, 'lights': 24, 'ermey': 1, 'remastered': 6, 'gielgud': 2, 'temerity': 1, 'devil': 11, 'deliberation': 4, 'mumbles': 1, 'ear-piercing': 1, 'element': 57, 'believability': 6, '`fastest': 1, 'xenophon': 1, "charles'": 2, 'onshore': 1, 'roelfs': 2, 'child': 93, 'pulse': 5, 'credulity': 2, 'real-world': 2, 'heart-wrenching': 2, 'ellis': 6, 'numbers': 22, 'sketched': 1, 'shrieking': 3, 'arliss': 2, 'pushkin': 1, 'austrian': 2, "lem's": 1, 'un-creative': 1, 'meal': 2, 'yorker': 4, 'perpetuated': 1, 'escapades': 5, 'inserting': 2, 'harold': 11, 'ultraconservative': 1, 'near-': 1, 'bohemians': 1, 'ii_--it': 1, 'weapon': 15, '+3': 2, 'hearty': 4, 'franchises': 1, 'icon': 10, "century's": 1, 'interludes': 5, 'beresford': 2, "`family'": 1, 'sanches': 1, 'cooper': 10, 'outwitting': 1, 'woke': 2, 'alternative': 6, "lester's": 2, 'revisited': 4, 'choices-': 1, 'silvio': 2, 'depending': 4, 'shadowed': 2, 'draws': 13, 'officially': 8, 'mill-types': 1, 'maid': 8, 'suite': 8, 'whorehouses': 1, 'ronnie': 3, 'suggested': 4, 'chase': 48, 'malevolence': 1, 'housewarming': 1, 'returning': 14, 'harmless': 10, 'mccann': 1, 'recommeded': 1, 'demille': 1, 'endangers': 1, 'peculiarly': 1, 'vessels': 1, 'nicjolson': 1, 'transmitter': 2, 're-team': 1, 'caron': 1, 'delta': 2, 'pulsates': 1, 'underappreciated': 4, 'crouching': 3, 'sometimes-understated': 1, 'succumbing': 2, 'kombat': 1, 'quarterback': 4, 'touch': 56, 'poehler': 1, 'nicolette': 1, 'non-literal': 1, 'pushed': 9, 'renard': 1, "varley's": 1, 'techniques': 15, 'ohio': 1, 'massacre': 3, "cliche'": 1, 'tommorrow': 1, 'lanai': 2, 'elena': 2, 'serving': 16, 'funniest--and': 1, "dream-rosaleen's": 1, 'bananas': 3, 'sadomasochism': 1, 'poop': 3, 'counterpart': 4, 'complements': 2, 'upset': 16, 'doctor': 24, 'electrical': 2, 'july': 12, 'self-named': 1, "mtv's": 1, 'manipulative': 16, 'plunged': 4, 'rung': 1, 'therapists': 2, 'foisted': 1, 'vig': 1, 'simcoe': 1, 'concepts': 13, 'bemused': 1, 'drown': 2, 'ness': 1, 'stilted': 10, "hoblit's": 2, 'tomlin': 1, 'greats': 1, '2023': 1, 'ironsides': 1, 'self-defense': 1, 'untapped': 1, 'similarities': 17, 'lamenting': 2, '5671': 1, 'soaring': 2, 'englishmen': 1, 'yearnings': 1, 'condone': 3, 'enduring': 4, 'lower': 11, 'neglect': 6, 'defeaningly': 1, 'nicky': 4, 'confessing': 2, 'podrace': 1, 'harshness': 1, 'members': 66, 'penis': 3, 'defended': 4, 'anti-social': 3, 'tibetans': 2, 'deluded': 2, 'dumbed-down': 2, 'honey-mustard': 1, 'commonly': 2, 'trade': 21, 'task': 46, 'niebaum': 1, "soze's": 1, 'stamping': 1, 'composer': 10, 'advertising': 8, 'affinity': 5, 'blissfully': 1, 'pit': 3, 'cover': 42, 'sissy': 1, 'murdoch-esque': 1, 'inventions': 1, 'schwartznager': 1, 'theme/theory': 1, 'mega-budgeted': 1, 'terrain': 2, 'prodded': 1, 'firmer': 2, 'striptease-an': 1, 'wringing': 2, 'scrutiny': 8, 'airplay': 1, 'yevgeny': 1, 'yakking': 2, 'discuss': 16, 'dowdy': 1, 'fidel': 2, 'belly': 3, 'immorality': 2, 'aquire': 1, "'70's": 1, "everybody's": 3, 'truths': 7, 'dime': 2, 'recommened': 1, 'dulloc': 1, 'scenic': 1, 'hardened': 5, "atlantic's": 1, 'fame': 31, 'visage': 2, '//filmfreakcentral': 2, 'holocaust': 9, '`roswell': 1, 'unexceptional': 1, 'embodies': 13, 'laconic': 3, "demme's": 2, 'grain': 4, 'automobile': 4, 'solutions': 3, 'straps': 1, 'shiver': 1, 'captivate': 1, 'hinder': 1, "ovation'": 1, 'hillbillies': 2, 'invade': 7, 'errol': 2, 'affecting': 15, 'profanity': 10, 'shagless': 1, 'xander': 1, "aloise's": 1, 'implemented': 1, 'disgraced': 3, 'lionizing': 1, 'mistakenly-suspected': 1, 'equation': 3, 'forty': 11, 'launched': 7, 'spinella': 1, 'newspapers': 2, 'flinging': 2, 'clarify': 1, 'compulsion': 1, 'custom-tailored': 1, 'poles': 2, 'creepy': 37, 'significant': 21, 'councils': 1, "`closure'": 1, 'jug': 1, 'pastures': 1, 'rumpo': 1, 'everett': 7, 'cowboys': 2, 'tainsy': 1, 'in-cluding': 1, 'esteem': 1, 'sameness': 1, 'bystander-extras': 1, 'gems': 5, 'rips': 2, 'hawaii': 2, 'eaddress': 1, 'biological': 2, 'feeling': 100, 'ago': 86, 'nell': 2, "gould's": 2, 'drop-dead': 1, "michell's": 2, 'fake': 19, 'snowfall': 1, 'seducing': 1, 'herskovitz': 1, 'empty': 18, 'spera': 2, 'tearful': 1, 'giacomo': 1, 'ex-minister': 1, 'preacher': 5, 'respond': 4, 'decapitated': 4, 'enterprising': 3, 'concerns': 11, 'ing': 1, 'heartbreak': 3, 'mulroney': 1, 'style': 122, 'shinohara': 1, "cox's": 1, 'sterotypically': 1, 'good-horror': 1, 'investigations': 3, 'creeps-and': 1, 'newswoman': 1, 'agatha': 1, 'accumulate': 1, 'chalkboard': 1, "fernando's": 1, 'revitalise': 1, 'disagree': 7, 'azrael': 2, "merton's": 1, 'bantu': 1, 'lose': 44, 'genitalia': 1, 'goodnatured': 1, 'coasts': 1, 'affirmations': 1, 'arden': 1, 'ending': 160, 'square-jawed': 1, 'accuses': 5, 'cleanse': 1, 'petrie': 1, 'enthusiastically': 4, 'silverback': 1, 'buxom': 2, "deane's": 1, 'platform-soled': 1, 'willingness': 3, 'titles': 6, 'aggravation': 1, 'rip-off': 5, 'malfunction': 1, 'distilled': 1, 'orthodox': 2, 'goldenberg': 1, 'licked': 1, 'wrought': 4, 'labrynthine': 1, 'affirmation': 2, 'crocodile': 1, 'overacting': 5, 'dar': 1, 'earl': 8, 'reflexivity': 1, 'american-': 1, "shaw's": 2, 'bostwick': 2, "fiennes'": 2, 'groveling': 1, 'locane': 1, 'biner': 1, 'squirming': 1, 'breeds': 2, 'deepened': 2, 'framing': 4, 'short-hand': 1, 'calendar': 6, 'skirts': 2, 'shortcut': 2, 'pencil': 1, 'textured': 5, 'aligned': 1, 'amazes': 3, 'portrayed': 54, 'alienation': 5, 'deserve': 20, 'upheaval': 1, 'researched': 1, 'expensive': 18, 'thought--will': 1, 'password': 1, 'guiness': 2, 'self-enlightenment': 1, 'marquee': 1, 'palladino': 1, 'face': 118, 'rigors': 1, 'booker': 3, 'bush': 4, 'delicious': 9, 'overarching': 3, 'choreography': 8, "grandson's": 1, "james'": 2, 'sheridan': 2, 'appeared': 19, 'coleman': 1, 'kiffer': 1, 'link-between': 1, 'transsexual': 2, '--': 135, 'brassieres': 1, 'combo': 4, 'unknowingly': 4, 'recognisable': 2, '=====================': 1, 'horse-training': 1, 'bop': 1, 'caraccio': 1, 'relates': 6, 'ever-changing': 1, 'converges': 1, 'chills': 9, 'sappiness': 3, 'befriends': 15, 'dorothy': 4, 'morricone': 3, 'steak': 2, 'dismayed': 4, 'illness': 5, 'apprise': 1, 'weep': 2, 'palpable': 12, 'philip': 10, 'howling': 5, 'predictably': 7, 'preconceived': 3, 'samurai': 6, 'swanky': 1, 'apprehensive': 1, 'tai': 2, 'overuse': 2, 'incomplete': 3, 'definitive': 5, 'beanfield': 1, 'chancellor': 1, 'russo': 4, 'fancy_': 1, 'cameras': 14, 'centers': 13, 'olivier': 1, 'imperious': 2, 'fresh': 47, '13': 23, "morricone's": 1, 'owing': 2, 'ny': 4, "malone's": 1, 'charleston': 1, 'mansion': 11, 'broad-stroked': 1, "wallace's": 3, 'pretending': 5, 'steal': 26, 'lecter': 1, 'versus': 12, 'gatsby': 1, 'devoted': 18, 'grinding': 2, 'imaginary': 3, 'institutions': 2, 'culmination': 4, 'crap': 18, 'pioneers': 2, 'lows': 3, 'nobility': 6, 'foresight': 1, 'hero/heroine': 1, 'distractingly': 1, 'suffering': 30, 'panned': 6, 'resnais': 1, 'channeling': 1, '155': 2, 'transplant': 4, 'divergent---': 1, 'deaf-mute': 1, 'outtake': 2, 'cort': 2, 'meanest': 1, 'buddie': 1, 'sees': 75, 'long-vanished': 1, 'buffs': 8, 'counsellor': 1, 'hatched': 2, '250': 1, 'script-writer': 1, 'grounded': 5, 'void': 2, 'taciturn': 3, 'finfer': 1, 'minor': 73, 'names': 32, 'rained': 1, 'nurses': 2, 'caper': 6, 'joey': 9, 'insert': 4, "australia's": 1, 'neo-nazi': 3, 'hum-vee': 1, 'things--nielsen-boosting': 1, "g's": 1, 'shop': 21, 'requested': 1, "harrelson's": 3, 'turtles': 3, 'possession': 7, 'nypd': 3, 'police-station': 1, 'perceptively': 1, 'executes': 4, 'programs': 2, 'brs': 1, 'presented': 52, 'con-man': 2, 'fowl': 1, 'phenominal': 1, 'nailbiter': 1, 'treat': 39, 'goods': 8, "it's-a-holocaust-film": 1, "supernatural-entity-takes-over-boy's-body": 1, 'rin-tin-tin': 1, 'routine': 25, 'testimony': 5, 'oskar': 2, 'teammate': 1, 'khanna': 1, 'insinuations': 1, "flick's": 1, 'non-nudity': 1, 'johns': 1, 'hanged': 2, 'leisurely': 7, 'yang': 1, 'schrager': 1, 'mother-of-all-storms': 1, 'pleased': 11, 'imply': 7, 'toad': 1, 'info': 4, 'sadistically': 1, 'enlightened': 1, 'didactic': 3, 'weitz': 2, 'fish-eye': 1, 'overstated': 3, 'reviewing': 5, 'curiosity': 10, "ephron's": 1, "'nother": 1, 'erich': 1, 'martial-arts': 1, 'self-awareness': 4, 'coffin': 1, 'lackluster': 6, 'kattan': 2, "'blabbed": 1, 'robb': 1, 'joining': 8, "bening's": 1, 'objective': 8, 'branaugh': 1, 'variations': 4, 'chaykin': 3, 'bettany': 1, 'eleanor': 1, 'chin': 1, 'stooges': 2, 'communication': 11, 'cmdr': 1, "1993's": 2, 'defensive': 4, 'bookshop': 1, 'tower': 6, 'andrews': 1, 'mirroring': 1, 'iambic': 1, 'double-crossing': 4, 'unwise': 1, 'liberties': 4, 'coughs': 1, 'performing': 10, 'martha': 3, 'celestial': 5, 'chiouaoua': 2, 'score': 95, '_so': 1, 'turturro': 12, 'yosarian': 1, 'enthusiasm': 14, 'dresses': 4, 'trenchcoat': 1, 'satirizes': 1, 'southport': 1, 'self-respect': 3, 'bligh': 1, 'torturous': 1, 'lazard': 1, 'gungan': 3, 'cache': 4, "conservatives'": 1, 'unusual': 35, '96': 1, 'surrounding': 19, '1987/88': 1, 'grandpa': 1, 'leonard': 12, 'poignancy': 6, 'zenith': 1, "cummin'": 1, 'maximus': 3, 'yelling': 5, 'pie-selling': 1, 'figgis': 1, 'undiscernable': 1, 'lame': 15, 'verification': 1, 'fabricated': 1, 'operates': 6, 'betrayal': 13, 'depicts': 12, 'liaison': 4, 'cuts': 21, '`thelma': 1, 'ex-co-': 1, 'iniquitous': 1, 'bloodless': 2, "best'": 1, 'trotting': 1, 'marc': 8, 'mightily': 1, 'questioners': 1, 'filmgoers': 3, 'interpretations': 2, 'achilles': 1, 'unwind': 1, 'catalogue': 1, 'struggle': 41, 'arija': 1, '[superior': 1, 'delivery': 18, 'full-on': 1, 'delighted': 3, 'rumors': 8, 'manipulated': 6, 'declares': 8, 'mack': 2, 'cancel': 3, 'hotcakes': 1, 'contestants': 4, 'below-ground': 1, 'confession': 9, "fisherman's": 1, 'migrant': 2, 'antisocial': 1, "windsor's": 1, 'davidtz': 5, 'stendhal': 1, 'perlman': 3, 'antiheroes': 1, 'nts': 1, 'discussions': 7, 'nonfans': 1, 'argumentative': 1, 'immerse': 4, 'tosses': 4, 'iggy-esuqe': 1, "vampire'": 1, "murray's": 5, "fears'": 2, 'bram': 4, 'wears': 21, 'unleashes': 3, 'differently': 10, 'auschwitz': 1, 'oscar-hungry': 1, 'soar': 1, 'eriq': 1, 'assert': 2, 'machines': 10, 'teapot': 1, 'madmen': 1, 'achievable': 1, "`hot'": 1, 'firings': 1, 'verify': 3, "d'etre": 2, 'cheerleader': 5, 'beguilingly': 1, 'rapid-fire': 2, 'pianist': 4, 'voices': 22, 'trivialized': 2, 'masquerading': 6, 'grainy': 2, 'triangles': 1, '1985': 7, 'gather': 11, 'passionate': 13, 'victorious': 4, 'chester': 1, 'expectant': 2, 'headquarters': 5, 'nursery': 1, 'item': 6, 'macalaester': 1, 'rohde': 1, 'madden': 2, 'patent': 2, 'faylen': 1, '24-year-old': 2, 'infrequently': 2, 'suspended': 5, 'crowd-drawer': 1, 'exam': 2, 'agreements': 1, 'astronomer': 7, 'pork': 1, 'wheelchair-bound': 1, 'imagine': 52, 'realisation': 2, 'protocol': 1, 'comedy-drama': 1, 'frame': 32, 'texture': 3, 'world-wide': 2, 'goths': 1, 'cents': 3, 'sulu': 1, "'more": 1, 'nuclear-weapon': 1, 'convergence': 1, 'designer': 20, 'duplicity': 1, 'ultra-nationalist': 1, "susan's": 1, 'preservation--': 1, 'unscrupulous': 4, "spacey's": 3, "melvin's": 1, 'verna': 1, 'cop-hunting-killer': 1, 'luscious': 4, 'bachelor': 5, 'sleep': 21, 'crashed': 5, "wah's": 1, 'finer': 4, 'hook-laden': 1, 'labyrinthine': 1, 'los': 19, "'rumble": 1, "odds-and-ends'": 1, 'smalltown': 1, "day'": 2, 'remarkably': 19, 'atom': 5, 'superman': 5, 'anguished': 2, 'dishearteningly': 1, 'momentary': 1, 'rasputin': 1, 'orgasms': 1, 'greta': 2, 'yawning': 1, 'ripley': 8, 'explorers': 2, 'lingers': 4, 'waittress': 1, 'outweigh': 2, 'sweeney': 1, 'semi-religious': 1, 'crooks': 7, 'surveillance': 1, 'glen': 7, 'groundlessness': 1, 'kirkpatrick': 1, 'priestley': 1, 'stained': 2, 'journeying': 1, 'depsite': 1, 'tatum': 1, 'talent--samuel': 1, 'feebly': 1, 'projected': 5, 'hinted': 10, 'glitters': 1, 'intrusions': 1, 'plying': 1, 'quickly': 97, 'lyons': 1, 'sucked': 14, 'stygian': 1, 'musker': 1, 'youngsters': 9, 'health': 8, 'berzerk': 1, 'culminating': 3, 'writing-henry': 1, 'sounding': 7, 'asbury': 1, 'lumet': 1, 'changing-of-the-seasons': 1, 'safe': 29, 'cleary': 1, 'patriotic': 3, 'beauties': 1, 'quinlan': 2, 'tumor': 1, 'explorer': 3, 'ming-na': 6, 'husk': 1, 'hackford': 2, 'moss-covered': 1, 'talked': 8, 'resonated': 1, 'hoblit': 3, 'emote': 1, 'pubic': 1, 'joyously': 1, 'bussing': 1, 'melodies': 1, 'elicited': 5, 'boyhood': 2, 'cap': 6, 'alferd': 1, 'afterthought': 5, 'unspeakable': 2, 'expletitive': 1, 'screwy': 2, 'trampy': 1, 'paced': 20, "hell's": 2, 'bellows': 1, "reiner's": 1, 'madonna': 2, 'woodsboro': 3, 'overcoat': 1, 'markings': 1, "bowie's": 2, 'kilometres': 1, 'ennio': 3, 'new-found': 1, 'ice-bound': 1, 'pleasure': 42, 'boyfriends': 3, 'polish': 3, 'hark': 3, 'drippy': 1, 'submarine': 7, "metoclorian'": 1, 'fingerprints': 5, 'elloquently': 1, 'crime-gone-wrong': 1, 'conisderation': 1, 'wrap-up': 1, 'axe': 3, 'ultra-huge': 1, 'vallejo': 1, 'nine-episode': 1, 'deflects': 1, 'genevi': 1, 'barge': 1, 'tempted': 5, 'drop-offs': 1, '8': 29, 'yup': 4, 'drum': 4, 'outcome': 16, 'capable': 30, 'uneasy': 8, 'slap-sticky': 1, 'tune': 12, "tourist's": 1, 'strangle': 1, "toback's": 2, 'gabrielle': 5, 'contributors': 2, "bobby's": 3, 'limp-wristed': 1, 'pan': 4, 'well-': 4, 'run': 104, "santa's": 1, '85': 7, 'flushed': 1, 'snigger': 1, 'percy': 3, 'anti-rich': 1, 'dope': 2, 'perceptions': 2, 'digest': 2, 'absorbing': 11, "demille's": 1, 'double-pronged': 1, 'schwarzenegger': 7, 'well-oiled': 1, 'vito': 2, "'strangers'": 1, 'magnetic': 4, "jody's": 1, 'lightsaber': 2, 'mobilizes': 1, 'bachar': 1, 'full-fledged': 2, 'berenger': 2, 'camps': 6, 'fortunately': 28, 'contrasts': 5, 'eminent': 2, 'deviating': 1, 'visualize': 1, 'elegance': 3, 'scour': 2, 'escapee': 2, 'leonardi': 1, 'movingly--into': 1, 'concubines': 1, 'forces': 44, 'fassbinder': 1, 'vine': 2, 'spoons': 1, 'amplified': 1, 'december': 15, 'negating': 1, 'bone': 12, 'championship': 5, 'swinging-': 1, 'twosomes': 1, 'tattoos': 3, 'staying': 20, 'violently': 1, 'reviewed': 10, 'brancia': 1, 'parole': 6, 'fares': 2, 'computer-generate': 1, 'cleaned-up': 2, 'riffs': 1, "waxman's": 1, 'primae': 1, 'breathers': 1, 'comely': 1, 'scott-thomas': 1, 'feast': 8, 'bereft': 2, 'red-and-white': 1, 'combine': 11, "pullman's": 2, 'rebaptizes': 1, 'den': 3, 'meter': 1, 'symbols': 9, 'attendees': 2, 'sleaze': 1, 'courageously': 3, 'kusakari': 2, 'sugarland': 1, 'eyebrow': 3, 'adherence': 1, 'suspending': 3, 'genteel': 1, 'phenomenon-': 1, 'teriffic': 1, "menges'": 1, 'cousins': 1, 'unbelievable': 19, 'preconception': 1, 'startup': 1, 'wobbly': 3, 'remotely': 11, 'stripping': 3, 'roger': 39, 'unfortunate': 32, 'brassed': 1, 'fore': 1, 'improve': 5, 'off-the-wall': 3, 'pocohontas': 1, 'listened': 2, 'proclaimed': 3, 'therefor': 1, "1969's": 1, 'encouraging': 3, 'anh': 2, 'fishes': 1, 'god': 65, 'carries': 30, 'awarded': 6, 'tibet': 2, 'informed': 5, 'carousel': 1, 'chisolm': 1, 'nina': 7, 'segment': 17, 'end': 321, 'romero': 2, 'crane': 6, 'jerome': 5, 'comedy/mystery': 1, 'imrie': 1, 'roux': 1, 'posturing': 1, 'gutted': 1, 'candy-coated': 1, 'gilliard': 1, "hassler's": 1, 'sale': 8, 'attachment': 3, 'mischievously': 1, 'ethical': 6, 'repent': 1, 'kieran': 2, 'whoopee': 1, 'vulnerable': 17, 'christmastown': 1, 'elijah': 6, 'obtain': 8, 'copyright': 8, 'utilized': 2, 'moby': 1, "erin's": 1, 'millennia': 2, 'zakk': 1, 'nihilist': 3, 'promoter': 3, 'chihuahua': 2, 'unnamed': 5, 'selling': 14, 'break-up/make-up': 1, 'attacks': 30, 'grudges': 2, 'soothing': 2, 'lethal': 9, 'grew': 19, 'grotesqe': 1, 'self-centeredness': 1, 'slap-stick': 1, 'connecting': 6, 'straws': 1, 'simplistically': 2, 'ex-husband': 2, 'haunted': 14, 'disturbed': 11, '35mm': 2, 'entendres': 1, 'quadruple': 1, 'sanitarium': 1, 'supermarket': 3, 'reconstructive': 1, 'matter-of-factness': 1, 'leadership': 9, "worralls'": 1, 'responding': 1, 'grade': 9, 'high-calorie': 1, 'stupid-': 1, 'goddamned': 1, 'shot': 118, 'spin-doctoring': 1, 'bewitched': 1, 'delicacy': 1, 'pistella': 1, 'intimidates': 1, 'raimi/rob': 1, 'jet': 10, 'goat': 1, 'mccormack': 3, 'falters': 2, 'debilitating': 2, 'things--start': 1, 'miracuously': 1, 'sword': 14, 'blatherings': 1, 'aronov': 1, 'labeled': 10, 'potter': 4, '50%': 1, 'domineering': 5, 'scene-stealing': 2, 'killer': 66, 'massage-therapist': 1, 'prison-bound': 1, 'fan': 54, 'bathing': 1, 'bulging': 2, 'tact': 3, 'treacherous': 4, "roddenberry's": 2, 'forgives': 2, 'top': 112, "mantegna's": 1, 'gangs': 3, 'furrows': 1, 'meaning': 44, 'pawn': 5, "insurrection's": 1, 'former-angel': 1, 'artsier': 1, 'massage': 5, 'pint-sized': 2, 'oogie': 1, "shakespeare's": 10, "'debate'": 1, 'molds': 3, 'polley': 8, 'confronts': 11, 'frauds': 1, "furlong's": 1, 'glossy': 2, 'steam': 10, 'bars': 9, 'underplays': 2, 'roswell': 1, 'fluff': 10, 'irony': 19, 'wahlberg-former': 1, 'gushing': 1, 'metal': 13, 'half-hidden': 1, 'lipnicki': 1, 'paymer': 4, 'mentoring': 1, 'swinton': 4, 'haste': 2, 'julianna': 1, 'performs': 8, 'imminent': 4, 'threads': 5, 'fehr': 1, "lucinda's": 1, 'neighbouring': 2, "herskovitz's": 1, 'condemned': 3, "esquire's": 1, 'cottrell': 2, 'rapes': 2, 'trail': 15, 'littering': 1, 'not-so-cheap': 1, 'disappoints': 4, 'abetted': 1, "lombardo's": 1, 'delightfully': 9, 'reccomend': 1, 'kung': 9, 'theater--and': 1, 'muslim--are': 1, 'dictated': 1, 'rikiya': 1, 'renowned': 8, 'divesting': 1, 'deprivation/appreciation': 1, 'r-rating': 1, 'orton': 2, 'utopia': 4, 'post-apocalyptic': 1, 'pidgeonhole': 1, 'spacing': 1, 'dominance': 1, 'virtuosity': 2, 'snotty': 2, 'picket-fenced': 1, 'wastelands': 1, 'altogether': 18, 'iniquity': 1, 'kickback': 1, "frasier's": 1, 'recruited': 6, 'donnie': 8, 'wood': 18, 'waterway': 1, 'plotting': 14, 'serenaded': 2, 'fakes': 2, 'resembles': 14, 'germania': 3, "dershowitz's": 1, 'word': 86, '91': 2, "nelson's": 1, 'decade': 36, 'klaus': 1, 'frankly': 13, 'pauses': 3, 'theft': 2, 'submarines': 2, 'completist': 1, 'limelight': 2, 'rama': 1, 'discovered': 32, 'synthesier': 1, 'lengthens': 1, 'dignified': 4, 'byrne': 5, 'rhea': 2, 'unpromising': 1, 'clipboard': 1, 'harum': 1, "matilda's": 2, 'insoluble': 1, 'pitchfork': 1, "forster's": 1, 'realistically': 6, 'trusting': 2, 'notion': 15, "weyl's": 1, 'jasmine': 1, 'network': 14, 'sized': 1, 'straight-forwardly': 1, 'workmates': 2, 'prospecting': 1, 'cd': 2, 'mackenzie': 2, "'flood'": 1, 'thirteen': 8, 'leaking': 1, 'skit': 2, 'acore': 1, 'outraging': 1, 'marketing': 15, 'fax': 2, 'oscar--pitt': 1, 'creeper': 1, "mulqueen's": 1, 'inspections': 1, 'down-and-out': 1, 'merchandise': 3, 'buck': 9, 'copywriters': 1, 'bucks': 15, 'deceived': 1, "labors'": 1, 'preachiness': 1, 'vanquish': 1, 'helming': 4, 'highlights': 15, 'babbling': 1, 'tolkien': 2, 'indie-rock': 1, 'rare': 54, 'vestern': 1, 'correctional': 1, 'solicit': 1, 'drug-addled': 1, 'karaoke-based': 1, 'pedersen': 1, 'court': 32, 'preferably': 1, 'kira': 1, '24': 5, '6-year-old': 1, 'hub': 1, 'flap': 2, 'kerr': 5, 'kien': 1, 'swiftly': 4, 'alter-egos': 1, "oz's": 1, 'employing': 7, 'abstract': 7, 'marsh': 1, 'fashioned': 11, 'palme': 2, 'subtleties': 4, "theron's": 1, 'hyperspeed': 1, 'graduating': 1, "july's": 1, 'gut-wrenching': 3, 'faithless': 1, 'slog': 1, 'hauled': 2, 'smoke': 10, 'shooter': 1, 'disenfranchised': 2, 'landlord': 3, 'repeating': 4, 'applegate': 1, 'lyric': 2, "e'": 1, 'ascension': 1, '1919': 2, 'hungry': 9, 'twister-occurrences': 1, "brooks's": 1, 'toothpick': 2, 'inconveniences': 1, 'hums': 1, 'lover': 36, 'belittle': 1, 'rail-laying': 1, 'malcom': 1, "'barntill": 1, 'ella': 1, 'party': 49, 'prominent': 17, 'tee': 3, 'torry': 1, 'knowledgeable': 3, 'maladjusted': 1, 'inducing': 2, "30's": 4, '$250': 1, 'psychopaths': 2, 'common-': 1, 'seagull': 1, 'tightly': 5, 'um': 4, 'raw': 18, 'subtheme': 1, "'green": 1, "madman's": 1, 'rioting': 1, 'low-life': 2, 'pent-up': 2, 'puritanical': 1, 'american': 177, 'striving': 5, 'popeye': 1, 'inhabited': 3, 'send-ups': 1, 'celine': 1, 'morissette': 3, 'varieties': 1, 'manifesto': 1, 'find': 275, 'urine': 2, 'lawford': 1, 'brutally': 18, 'grungy': 1, 'plant': 4, 'rehabilitate': 1, 'combining': 6, 'cilauro': 1, 'punching': 4, 'half-vampire': 2, 'assignments': 1, 'disaster': 30, 'asia': 2, 'decay': 2, 'pierre': 2, 'holds': 44, 'embraces': 3, 'enveloped': 1, 'defacing': 1, 'sebastian': 2, 'lodger': 1, 'caviezel': 5, 'explore': 11, 'notice-rather': 1, 'garry': 2, 'approval': 1, 'pretender': 1, 'diarrhea': 1, 'lash': 1, 'stakes': 8, 'protects': 3, 'wrongheaded': 1, 'conjure': 5, 'dry-witted': 1, 'quoted': 4, 'monoliths': 1, 'renovated': 1, 'faces': 45, 'superficiality': 1, 'transformation--is': 1, 'locale': 2, 'un-noticeable': 1, 'foreign': 24, 'excite': 2, 'massacering': 1, 'auriol': 1, 'selick': 2, 'ende': 1, 'woods': 22, 'founder': 2, 'aoki': 2, 'lied': 2, 'boarder': 2, "'double": 1, 'contacting': 1, 'yzma': 1, 'action/thriller': 1, '1949': 1, 'maps': 2, 'pre-conceived': 1, 'snitches': 1, 'spartacus': 2, '500-like': 1, 'tad': 23, 'lilting': 1, 'rub': 2, 'condon': 1, 'glengarry': 2, 'miraculous': 5, 'stuntman': 1, 'scrabble': 1, 'distances': 2, 'greens': 2, 'notebook': 2, 'film-class': 1, 'one-notedom': 1, 'outgunned': 1, 'skipper': 1, 'whine': 2, 'sensitively': 3, 'swarm': 3, 'nachos': 3, 'run-down': 4, 'tumbles': 2, 'darlanne': 1, 'subsequent': 16, 'ius': 1, 'epiphany': 4, 'ruffians': 1, 'bible': 11, 'hounding': 2, 'sitter': 3, 'well-paid': 1, 'boyd': 2, 'raquel': 3, 'transporting': 2, 'recharge': 1, 'far-from-overpowering': 1, 'cosette': 3, 'visuals': 35, 'holiness': 2, 'spiked': 1, 'stylish': 20, 'overweight': 7, "jerusalem's": 1, 'weiss': 4, 'sections': 4, 'pictures': 52, '`the': 9, 'showcasing': 2, 'viewfinder': 1, 'rupert': 4, 'mutters': 2, 'tumbling': 1, 'descend': 1, 'pre-teen': 1, 'felliniesque': 1, 'ratner': 3, 'attentions': 2, 'non-animated': 1, 'trained': 9, 'blockubuster': 1, 'hobbes': 2, 'verges': 1, 'mis': 2, 'cesar': 1, 'marketed': 2, 'pores': 1, 'lithgow': 9, 'involve': 9, '1000': 4, 'glitter': 1, 'scam': 9, 'movie-maker': 1, 'glave': 1, 'flex': 1, 'soul-baring': 1, 'multi-character': 1, 'dawn': 15, 'underneath': 15, 'mist-esque': 1, 'jaoui': 1, 'contraception': 1, 'klass': 1, 'ground': 47, 'nitpicks': 1, 'unintrusive': 1, 'clara': 1, 'turn-on': 1, 'duquette': 3, 'wander': 7, 'anonymous': 8, 'downtown': 2, 'cast--goran': 1, 'bonnefroy': 1, 'impregnable': 1, 'hecuba': 1, 'independance': 2, 'stanton': 4, 'allan': 3, 'troublemaker': 2, "games'": 1, 'freed': 7, 'inferring': 1, 'priesthood': 1, 'spine': 2, 'walk-off': 1, "mitchell's": 1, 'dances': 8, 'desperately': 28, 'winsomely': 1, 'latrobe': 1, 'newbies': 1, 'skipped': 2, 'wooden': 20, 'climbs': 3, 'suspected': 8, 'cartoons': 9, 'reminder': 11, 'unrecognizable': 6, 'legal': 19, 'coburn': 4, "damon's": 3, 'understated': 12, 'moffat': 1, 'koppelman': 1, 'riddle': 1, 'brandy': 3, '_anything_': 1, 'clich': 16, 'sequence': 130, 'clashing': 3, 'editorializing': 1, 'wizened': 1, 'stargaze': 1, 'springtime': 1, 'semi-articulate': 1, 'bites': 4, 'ridiculousness': 2, 'dairies': 1, 'mousy': 3, 'receive': 17, 'tobacco': 1, 'hand': 125, 'billion': 6, 'overtake': 2, 'consummate': 3, 'regina': 1, 'destined': 28, 'full-price': 1, 'settled': 9, 'truckload': 3, 'upstart': 1, 'prescripted': 1, 'breather': 2, 'slawomir': 1, 'marco': 3, 'beers': 2, 'planting': 1, 'burke': 4, 'devon': 5, 'taming': 3, 'aol': 1, 'twentieth': 6, 'engulfed': 4, 'treachery': 3, 'osborne': 1, 'ferries': 1, 'dreyfuss': 5, 'articulately': 1, 'bouncy': 4, 'scare': 16, 'stepfather': 3, 'oleynik': 2, 'afar': 3, "measurement's": 1, 'fosdick': 1, 'hi-larious': 1, 'including': 147, 'confirms': 1, 'entomologist': 1, 'deviants': 1, 'inconsistencies': 4, 'menno': 1, "1968's": 1, 'technologically': 3, 'co-existed': 1, 'deteriorate': 1, 'babes': 2, 'in-valid': 1, 'amassed': 1, 'ibsen-esque': 1, 'propoganda': 1, "producer's": 1, "labute's": 3, 'semblence': 1, 'originality': 21, 'belligerent': 1, 'quirkier': 1, 'funerals': 1, 'rev': 2, 'boldly': 5, 'bancroft': 5, 'database': 3, 'laced': 5, 'nanny': 3, 'impulsive': 2, 'revelation': 20, 'souls': 12, 'wincott': 2, 'subconscious': 8, 'thou': 4, 'charmer': 6, 'tuxedo': 1, "shalhoub's": 1, 'bunyans': 1, 'appearances': 24, 'gina': 5, 'dealings': 3, 'ever-appealing': 1, 'piercing': 1, 'mechanics': 3, 'villian': 3, 'retro-style': 1, 'embarrassed': 7, 'masterminds': 3, 'simmer': 1, 'a--': 1, 'sides': 24, 'spit': 3, 'rope': 3, 'poisoning': 3, 'expecting': 35, 'mischievous': 5, 'horoscopes': 1, 'super-cop': 1, 'laura': 20, 'labs': 1, "jarmusch's": 1, 'frontier': 4, 'utters': 2, 'regimen': 2, 'donovan--it': 1, 'allies': 3, 'errands': 1, 'pardon': 2, 'victimizer': 1, 'remarks': 9, 'bubble-gum': 2, 'reinserted': 1, 'decalogue': 1, 'whining': 5, 'intact': 9, 'rockies': 1, 'shares': 18, 'severs': 1, 'toughly': 2, "ben's": 2, 'story-driven': 1, 'resembled': 1, "nookey's": 1, 'struck': 17, 'elite': 17, 'covering-up': 1, 'mineo': 1, 'smallest': 3, 'makes': 356, 'marching': 2, 'deborah': 4, 'evasively': 1, 'plods': 1, 'homework': 1, 'gandolfini': 2, "costanza's": 1, 'dramatized': 2, 'gripe': 5, "snake's": 1, 'wagered': 1, '_dragonheart_': 1, 'preparing': 8, 'pouring': 6, 'swamp': 4, 'veloz': 1, "channel'": 1, 'mating': 1, 'randomly': 2, 'ghost': 19, '_ghost': 1, 'howser': 4, 'stain': 2, 'android': 3, 'curly': 2, 'no-nonsense': 5, "mary'": 2, 'pothead': 2, 'ton': 2, 'thivisol': 1, 'emerging': 4, 'teasers': 1, 'capability': 3, 'arid': 2, 'rudy': 3, 'jordan': 9, "empire's": 1, 'one-way': 1, 'bootsy': 1, 'artistic': 27, 'silhouette': 1, "men's": 4, 'memo': 1, "brother's": 5, 'grandson': 3, 'narcoleptic': 1, 'juicy': 5, 'subtle': 85, 'labelling': 2, 'viewpoint': 6, 'simulated': 1, 'url': 1, 'ex-athlete': 1, 'momentous': 3, 'apparatus': 2, 'moniker': 3, 'discarded': 3, 'to--again': 1, 'incumbent': 3, 'republic': 9, 'sentimentalized': 1, 'nazi': 9, 'waving': 2, 'humor-wise': 1, 'flaquer': 1, 'mountaintops': 1, 'introversion': 1, "niro's": 2, 'henchlady': 1, 'salvation': 6, 'gun': 40, 'milliken': 1, 'accepts': 13, 'loren': 3, "congo's": 1, 'pratfalls': 1, 'bemusing': 1, "1991's": 4, 'portray': 19, '#15': 1, 'apathy': 2, 'sykes': 1, 'variation': 12, '+2': 5, 'underbelly': 2, "ethan's": 1, 'wrongly': 4, "hands'": 1, 'craig': 8, "1930's": 2, 'beautifully-constructed': 1, 'survive': 28, 'lopez': 8, 'cronenberg-esque': 1, 'regrettable': 2, 'paean': 1, 'temperatures': 1, 'violating': 1, '`munchings': 1, 'handyman': 1, 'groaning': 2, 'hal': 2, 'drivel': 4, 'awkwardness': 1, 'avenged': 1, 'stephane': 1, 'englishman': 3, 'perfect': 162, 'scarlet': 2, 'pipeline': 1, 'outlines': 1, 'engineers': 2, 'limousines': 1, "daisy's": 1, 'over-': 2, "spring's": 1, 'challenge': 28, 'moustached': 1, 'hindus': 2, 'niftiest': 1, 'retrieve': 10, 'fending': 1, 'swipes': 1, "desi's": 1, 'michel': 2, 'attuned': 2, 'lambert': 1, 'arabia': 3, 'tatooed': 1, 'melanie': 8, "mulder's": 2, 'cropped': 1, 'hippest': 1, 'iffy': 2, 'projection': 2, 'outlaws': 1, 'expendable': 2, 'weather': 2, 'reappearance': 2, 'pillow': 4, "1985's": 1, 'whistling': 1, 'desiree': 1, 'pornographer': 1, "skywalker's": 2, 'saddam\x12s': 1, 'discerning': 1, 'timelines': 1, "syndrome's": 1, 'goldeneye': 4, 'warehouse': 4, 'profane': 2, 'differences': 20, 'coconut': 1, 'football': 13, "zaire'": 1, 'old-': 1, 'tragic': 33, 'giosue': 2, 'winterbottom': 2, 'dogg': 1, 'ambling': 1, 'loses': 42, 'chirped': 1, 'populate': 1, 'dystopian': 1, 'bears': 12, 'today': 57, 'sherlock': 3, 'smug': 5, 'quips': 4, 'welcomed': 2, 'milos': 5, 'insulin': 1, 'exley': 2, "viewer's": 8, 'josie': 2, 'attilla-looking': 1, 'tnt': 1, 'copping': 2, 'owes': 16, 'welles': 2, 'parrots': 1, 'ultimo': 1, 'unveiling': 1, 'friendly': 18, 'bernheim': 1, 'equivalent': 11, 'gast': 1, 'commissioned': 1, 'first-hand': 2, 'uncaring': 3, 'schmuck': 1, 'conglomeration': 1, 'screwball': 3, 'bastard': 13, 'coincidences': 3, 'commence': 1, 'jansen': 2, 'fodder': 11, 'takes': 281, 'masochist': 1, 'doped-up': 1, 'mercies': 1, 'infuriatingly': 1, "pie'": 1, 'brag': 1, 'abandon': 12, 'andersons': 1, 'dates': 3, "jackson's": 4, "elliott's": 2, 'twists': 53, 'perceiving': 1, 'june': 7, 'scholars': 6, 'weeps': 1, 'bending': 1, 'well-received': 1, 'translation': 3, 'psychodrama': 1, 'jump-out-at-you-from-the-dark': 1, 'quint': 1, "sciorra's": 1, 'stepped': 3, 'resoundingly': 1, 'time-constrained': 1, 'larisa': 2, "nolan's": 1, 'eden': 2, 'artistically': 3, 'babaloo': 2, 'shipments': 1, 'earns': 9, 'dense': 3, 'scuffle': 1, 'winslow': 1, 'greets': 4, 'privileges': 2, 'furnishing': 1, 'quartered': 1, 'davidson': 2, 'ingredient': 1, 'donkey': 5, 'vibrancy': 1, 'cinematography/art': 1, 'billy': 39, 'distortion': 1, 'busboy': 1, 'freewheeling': 2, 'palmieri': 1, 'overlaid': 1, 'served': 16, 'nostalgia': 10, 'matrix_': 1, 'handcuffed': 2, 'deleted': 9, 'receptacle': 1, 'nolstalgic': 1, 'brace-sporting': 1, 'vulnerability': 12, 'humor': 124, 'gasp': 3, 'video-age': 1, '+s': 1, 'sniffing': 1, 'inflation': 1, 'wes': 14, 'modernizing': 2, 'vying': 2, 'relief': 43, 'trinity': 4, 'attach': 3, 'frequently': 28, 'strauss': 1, 'icily': 1, 'recipes': 1, 'ousting': 1, 'jaws': 9, 'istvan': 1, 'store': 47, 'blossoming': 3, 'perceived': 5, 'harm': 10, 'convict': 6, 'ape': 4, 're-energize': 1, 'mid-level': 3, "altman's": 5, 'tire': 2, 'piling': 2, 'unjustifiable': 1, 'herman': 6, 'anchors': 1, 'unrelated': 4, 'topple': 2, 'sinclair': 2, 'nineteenth': 2, '[mike': 1, 'intensively': 1, 'overly-officious': 1, 'seconds': 20, 'transparent': 2, 'thrills-a-minute': 1, 'condemn': 4, 'disintegrating': 1, 'blouse': 1, 'silverstone': 1, 'underrated': 15, 'boyfriend/cop/partner': 1, 'wheelchair': 5, 'landord': 1, 'schlictman': 1, 'regurgitates': 1, 'nits': 1, 'unjustly': 3, 'letdown': 5, 'tab': 1, 'opposes': 3, 'bandits': 5, 'kundun': 5, 'rates': 1, 'smart-alec': 1, "o'toole": 2, 'sporting': 5, 'lime-colored': 1, 'privately': 1, 'janey': 1, 'ravenous': 1, 'hand-to-hand': 2, 'post-salary': 1, 'cross-cuts': 1, 'ur-reality': 1, 'theologians': 1, 'recites': 2, 'absurdity': 4, 'se7en': 3, 'sing-along': 1, 'dewey': 8, 'flamingos': 3, 'harrelson': 7, 'piven': 1, 'scheider': 1, 'veiled': 2, 'sitcom-esque': 1, 'parking': 7, 'undetected': 1, 'gaffes': 2, 'yech': 1, 'confronting': 5, 'fumbling': 5, 'dah-dum': 1, "hayek's": 1, 'bumps': 7, 'inclination': 2, 'crib': 4, "detective's": 2, 'ciro': 1, 'twenty-two': 3, 'reon': 1, 'rescued': 6, 'deane': 1, 'expectation': 7, "kaaya's": 1, 'bore': 8, 'scratches': 1, 'cocaine': 6, 'poses': 4, 'luck': 18, 'kouf': 1, 'aaaahhhs': 1, 'scouts': 1, 'action/sci-fi': 1, 'thoughout': 1, 'panicky': 2, 'schmoozes': 1, 'friends': 147, 'obliterates': 3, 'dives': 4, 'theorizing': 1, 'detract': 12, 'multi-': 2, 'doesn=92t': 1, 'nine-year': 1, "shyamalan's": 2, 'kaleidoscopic': 1, 'punk': 5, 'dorado': 2, 'cauterize': 1, 'thoughts': 23, 'much-larger': 1, 'offensiveness': 1, 'negin': 1, 'refusal': 3, 'louisa': 2, "reynold's": 1, 'fasttalking': 1, 'killers': 17, "bill's": 1, 'herrington': 1, 'normandy': 3, 'fellini': 3, 'citizens': 19, 'samples': 4, 'asians': 1, "donager's": 1, 'life-sized': 1, 'adaption': 3, 'vocalists': 1, 'denouement': 5, 'unconsciously': 2, 'cancels': 1, 'bombarded': 2, 'crystallized': 1, 'grogan': 1, 'packet': 1, 'glares': 1, 'jeep': 3, 'brighten': 2, 'discovery': 14, 'pirates': 3, 'dispatched': 5, 'safina': 1, 'profoundness': 2, 'inteligence': 1, 'elitism': 1, 'dysfunction': 2, "'58": 1, 'happy-go-lucky': 1, 'occasionally': 52, 'liquid': 1, 'believed': 20, 'sow': 1, 'hallucinates': 1, 'proposal': 3, 'lovers': 22, 'lapdance': 1, 'roadside': 1, 'journalist': 9, 'nonsensical': 2, 'lineup': 3, 'urban': 18, 'charter': 1, 'sweaty': 7, 'romanov': 1, 'normal-looking': 1, 'potentially': 8, 'dalliances': 1, 'similarly-named': 1, 'foibles': 5, "paulie's": 1, 'acquisition': 1, 'deft': 11, 'shoving': 1, 'tail': 5, 'sinise': 3, "hadden's": 1, 'photosensitivity': 1, 'full-length': 4, 'prerelease': 1, 'seperation': 1, 'shaped': 6, 'wider': 3, 'venue': 1, 'christine': 8, 'add': 66, 'glamorous': 7, 'hanly': 1, 'berkley-like': 1, 'phylida': 1, "40's": 1, 'curmudgeons': 1, 'poignant': 26, 'edgefield': 1, 'childhood': 30, 'flashy': 19, 'amalgam': 1, 'severely': 6, 'price': 33, 'oppression': 6, 'women_': 1, 'pivot': 1, 'revised': 1, 'blanc': 1, 'numbingly': 2, 'triumphs': 8, 'mind-boggling': 1, 'steady': 13, 'effortless': 6, 'investigator': 10, 'rib-tickling': 1, 'carlito': 2, 'enterataining': 1, 'chameleonic': 1, "hershman's": 1, 'mnc': 1, 'dog': 42, 'fits': 30, 'wracked': 1, 'recite': 3, 'jolted': 1, 'telegram': 1, 'bawls': 1, 'immersed': 5, 'confront': 7, 'divisions': 1, 'traces': 5, 'depletion': 1, 'godfather-films': 1, 'understands': 10, '20-something': 1, 'kingpins': 1, 'occasionally-humorous': 1, 'part': 245, 'grump': 2, 'grasping': 3, 'onegin': 1, 'steely-eyed': 1, 'prophets': 3, 'afloat': 2, 'warm': 34, "bates'": 3, 'quincy': 5, 'andre': 5, 'directed': 135, "linda's": 1, 'charecter': 1, 'frank': 51, '--gluttony': 1, 'reassuring': 4, 'rectified': 1, 'heavily-hyped': 1, "white'": 1, 'gabby': 1, 'downfall': 7, 'mid-70s': 1, 'wins': 20, 'small-time': 7, 'minute': 35, 'hyper-tense': 1, 'priam': 1, 'thicket': 1, 'mctiernan': 1, 'methodically': 1, 'begbee': 1, 'roper': 2, 'dump': 5, 'fluid': 5, 'shouting': 11, 'tshombe': 1, 'schoolmates': 1, 'fetching': 1, 'thunder': 3, 'cadre': 2, 'open-minded': 1, 'pinter': 2, 'breakneck': 2, "luhrmann's": 5, 'vet': 5, 'ii--the': 1, 'womanhood': 1, 'proposes-demands-that': 1, 'observe': 7, "wolf's": 1, 'sheltered': 3, 'denigrates': 1, 'betty': 6, 'stealing': 15, 'kelley': 2, 'stiles': 4, 'overpowering': 3, 'partial': 3, 'empathizes': 1, 'unintended': 3, 'literary': 7, 'giddiness': 1, 'treasure': 12, 'tanner': 2, 'street': 42, 'credibility': 7, 'specialized': 1, 'echoed': 1, 'seen-it-all': 1, 'oven': 2, 'efects': 1, 'ray': 22, 'prosecute': 1, 'swat': 2, 'boyle': 9, 'manipulating': 3, 'manners': 4, "bishop's": 1, 'frighten': 2, 'dangerous': 29, 'pilkington': 1, 'sleeping': 8, 'quotient': 2, 'vegetative': 1, 'backtrack': 2, 'sketches': 2, 'horrified': 5, 'dynamite': 3, 'cower': 1, 'opt': 1, 'well-structured': 1, 'birnbaum': 1, 'c3-po': 1, 'joblo': 11, 'director-george': 1, 'speakeasy': 1, 'renoir': 1, 'vacation': 17, 'denominator': 3, 'favourites': 2, 'rombi': 1, 'exude': 2, 'banishes': 1, 'rescue': 37, 'holiday': 15, 'rolled': 8, 'accursed': 1, 'read-along': 1, 'suit-and-tie': 1, 'transitory': 1, 'joint': 4, 'tribes': 3, 'oeuvre': 2, 'made-for-video': 1, 'neo-feminist': 1, 'board': 33, 'smile': 34, 'bolstered': 2, 'inappropriate': 7, 'best-executed': 1, 'bejesus': 1, 'seasick': 1, 'reductionism': 1, 'flattering': 1, 'dalai': 3, "day's": 1, 'acrobatic': 1, 'seale': 2, 'armored': 2, 'pimp': 2, 'rotunno': 1, 'film--': 1, 'unjustifyably': 1, 'technicalities': 1, 'deposited': 2, 'nooks': 1, 'structure': 32, 'wordlessly': 3, 'treaty': 2, 'miserably': 5, 'gun-running': 2, 'ignited': 1, 'casual': 6, 'obsessive': 9, 'headdresses': 1, 'diverting': 1, 'master-servant': 1, 'closed': 11, "newt's": 1, 'dermot': 1, 'salary': 3, 'listings': 2, 'proffessor': 1, 'denounces': 1, 'boy-gets-killed-in-horrible-accident': 1, 'decks': 1, 'boolie': 1, 'rebelled': 2, 'dissolute': 1, "party's": 1, 'unites': 2, 'comediennes': 2, 'rip-roaring': 1, 'resemble': 8, 'handing': 2, 'rents': 1, 'boiling': 1, 'drunkard': 3, 'others--sikh': 1, 'on-the-job': 1, 'buff-o': 1, 'baddass': 1, 'received': 19, 'coworker': 1, 'wales': 3, 'lengths-no': 1, 'over-wrought': 1, 'julianne': 12, 'dogma': 4, 'skeletons': 2, 'waxing': 1, '3000': 1, 'step-away': 1, '#6': 1, 'brink': 11, 'hughton': 2, 'crystal': 8, 'aried': 1, '66': 2, 'family-oriented': 1, '1957': 4, 'unmarried': 1, 'bont': 3, 'over-processed': 1, "coufax's": 1, 'nottice': 1, "valentine's": 2, 'ilm': 4, 'pikser': 2, 'butcher': 5, 'vandalism': 1, 'clumsily': 2, 'livewire': 1, 'homicidal': 4, 'corner': 20, 'thomson': 1, 'roshomon-like': 1, 'astounding': 16, 'placement': 8, 'ooze': 3, 'eats': 6, 'franzoni': 1, 'faced': 29, 'gertrude': 2, 'obstetrician': 1, 'floyd': 2, 'adapting': 7, 'recipient': 3, 'witch': 14, 'rightist': 1, 'tinder': 1, 'nineties': 5, 'heir': 5, 'acting/music/effects': 1, 'decapitates': 1, "larter's": 1, "cat's": 1, 'democrats': 2, 'fleabag': 1, 'malnutrition': 1, 'cyrus': 1, 'victim/potential': 1, 'pullman': 6, 'disaster-slash-action': 1, 'impressed': 19, 'italian-like': 1, 'catagory': 1, 'blimp': 1, 'kafkaesque': 3, 'guest': 10, 'wallpaper': 1, 'indecisiveness': 1, 'equaled': 1, 'yearning': 5, "city's": 6, 'amok': 1, 'neil': 19, 'accusations': 3, 'everyday': 23, 'wedding': 26, 'disarm': 1, '_very_small_': 1, 'turnaround': 1, 'prime-time': 1, 'elliot': 14, 'non-communicative': 1, 'ducastel': 1, 'tbwp': 1, 'recent': 95, 'cranks': 1, 'tee-shirt': 1, 'intertwines': 1, 'urges': 5, 'carters': 2, 'striding': 1, 'underdevelopment': 1, 'morale': 3, '1987': 9, 'love-struck': 1, 'roughness': 1, 'hypnotises': 1, 'bookseller': 1, 'samantha': 2, 'torment': 3, 'chequered': 1, 'creaking': 1, 'belies': 2, 'zane': 9, 'scrumptious': 1, "street'": 1, 'obesity': 1, 'cuckor': 1, 'ma': 5, 'psychiatric': 2, 'grit': 2, 'sheedy': 2, 'emblematic': 2, 'lesson': 22, '54th': 1, 'plain-cheese': 1, 'voodoo': 2, 'judi': 7, 'audio': 5, 'show-stopping': 2, 'head': 134, 'sherriff': 1, 'dull': 20, 'nemeses': 1, 'art': 87, 'blandly': 2, 'jost': 1, 'surge': 3, 'marginal': 1, 'hart': 8, 'omnipresence': 1, 'ugliness': 1, 'heroin-addicted': 1, 'closing': 21, 'argument': 18, "heche's": 2, 'milk': 7, 'ninety-minute': 1, 'digitalized': 1, 'calculation': 1, 'southwest': 2, 'jerry': 31, 'entrepreneurial': 1, 'flower': 2, 'niagara': 1, 'assitance': 1, 'kitchy': 1, 'forgot': 5, 'sweethearts': 1, 'arcane': 1, 'characters--': 1, 'mekhi': 2, 'parsee': 1, 'bags': 6, "aniston's": 1, 'camera-work': 1, 'backwater': 1, 'crass': 3, 'panache': 5, 'broke': 14, 'obvious': 92, 'disease': 10, 'luncheon': 1, 'allusion': 1, 'mentally': 16, 'down-at-heels': 1, 'retched': 1, 'pual': 1, "shouldn't-for": 1, 'fundamentalist': 2, 'protests': 4, 'morphing': 1, "guaspari's": 1, 'plumbing': 2, 'formulates': 1, 'insurance': 14, 're-edit': 1, 'packs': 8, 'paus': 1, "roger's": 1, 'wounds': 11, 'huddleston': 5, '70s/early': 1, 'laudable': 1, 'shoeless': 1, "feeder's": 1, 'darndest': 1, 'assistants': 3, 'indie-minded': 1, 'scintillating': 1, 'lust': 13, 'aggravatingly': 1, 'tickets': 10, 'kuosmanen': 1, 'rotary': 1, 'truckloads': 1, 'connie': 5, 'note-': 1, "hopper's": 2, 'rambunctious': 2, 'non-matinee': 1, 'eisley': 2, 'arrangements': 4, 'absorb': 5, '===========': 2, "bastard's": 8, 'pitted': 3, 'broodwarrior': 1, 'perkins': 1, 'eldard': 3, 'tension': 63, 'freeze-frame': 2, 'formal': 2, 'gender': 5, "mcdonald's": 7, 'epigraph': 1, 'popular': 77, 'design': 43, 'egyptians': 2, "governer's": 1, 'dreamers': 1, 'disagreement': 1, 'swing-dancing': 1, 'daleks': 1, 'err': 1, 'trickier': 1, 'bubblebath': 1, 'consistent': 9, 'beatrice': 1, 'amateurish': 3, 'passe': 1, 'adapt': 3, 'revisits': 1, 'epicenter': 1, '11-year': 1, 'bereaved': 2, 'separation': 3, 'quarter-century': 1, 'ferrets': 1, 'swoon': 3, 'climatic': 3, 'ciphers': 1, 'mein': 1, 'whipping': 2, 'wasn+t': 1, 'exiles': 1, 'lend': 7, '007': 5, 'blacksmith': 1, 'waning': 2, 'thousand': 14, 'circulating': 2, 'flat-out': 5, 'pointy-breasted': 1, 'constrained': 2, 'deaths': 7, 'rabies': 1, 'streetwise': 1, 'gisbourne': 1, 'shagadellic': 1, 'reeled': 1, 'lukas': 1, 'wahlberg': 10, 're-enactment': 2, 'vindication': 2, "grusin's": 1, 'life-long': 3, 'lifetime': 15, 'rapid': 10, 'retirement': 8, 'co-writer': 7, 'unwraps': 1, "killer's": 6, "north's": 1, 'physics': 2, 'star': 182, 'furniture': 1, "stern's": 1, 'english': 57, 'wizardry': 2, 'identifies': 1, 'quasi-evil': 1, 'chow-yun': 1, 'dark': 114, 'prologue': 13, 'irritates': 2, 'pignon': 1, 'barclay': 1, 'quirky': 23, 'stepping': 5, 'extract': 1, "kinski's": 1, 'pip': 1, "bateman's": 1, 'excitiable': 1, 'heroine': 26, 'spanish': 12, 'easily': 98, 'unwatchable': 4, 'soze': 2, "'matron'": 1, 'narc': 1, 'darts': 2, '`wild': 1, 'logs': 1, 'anthology': 1, 'foreigner': 3, 'gang-bullseye': 1, 'tutor': 4, 'gardner': 1, 'ambiguousness': 1, 'quartet': 5, 'lesbian': 12, 'yield': 4, 'changing': 18, 'borealis': 1, 'fails': 34, 'afa': 1, 'attentive': 3, 'hostile': 5, 'immortalizing': 1, 'vegetable': 2, 'ridicule': 2, 'kinder': 1, 'carroll': 3, 'carp': 1, 'simulating': 1, 'coulda': 2, 'eyeroll': 1, 'circle': 13, 'trump': 1, 'sift': 1, 'creepy-looking': 1, 'compunction': 1, "insider's": 2, 'disclosing': 1, 'denker': 1, 'moat-filled': 1, 'adjusting': 1, 'antics': 14, 'ike': 1, 'notions': 8, "holland's": 1, 'space-faring': 1, 'ebert': 5, 'translator': 3, 'skinhead-dom': 1, 'abel': 2, 'temporarily': 5, 'joe/josephine': 1, 'extremes': 4, 'slow-starting': 1, 'randy': 9, 'adding': 20, 'overcloud': 1, 'taylor-gordon': 1, 'stand-in': 2, 'nutshell': 4, 'unbrewed': 1, 'countries': 13, 'forbidden': 10, 'notes': 29, 'flippant': 2, 'risa': 1, "ryan'": 1, 'analyze': 6, 'instructs': 2, 'jeesh': 1, 'headhunter': 1, 'getty': 1, 'divided': 12, 'sheriff': 13, 'livelihoods': 1, 'crowd-pleasing': 3, 'humiliating': 2, 'hopeful': 5, 'reopens': 1, 'toback': 2, 'partner-wife': 1, 'tutee': 1, 'instantly': 16, 'subjects': 10, 'nervously': 6, 'vail': 1, 'losers': 5, 'copy': 21, 'november': 6, 'complicated': 31, 'cringe': 3, 'recommended=85=85but': 1, 'sons': 11, 'jacobi': 3, 'distance': 13, 'admittedly': 16, "'smoothly'": 1, 'ymca': 1, 'birkenau': 1, 'pan-and-scanned-out': 1, "stargher's": 1, 'deconstruction': 2, "d'or": 2, 'contrived': 22, 'compound': 3, 'wiez': 1, 'crooks-posed-as-photographers': 1, 'work-loving': 1, 'eccentricities': 2, 'spew': 2, 'cryogenically': 2, 'smiling': 8, 'produces': 5, "howard's": 2, 'pete': 7, 'categories': 8, 'liking': 10, 'computer-generated': 17, 'inquisition': 2, 'lunchmeat': 1, 'path': 27, 'arnold': 11, 'notre': 5, 'self-fulfilling': 1, 'electrically': 1, 'action/adventure': 3, "70's": 7, 'sturdy': 3, 'harsh-seeming': 1, 'donato': 1, '`never': 1, 'leno': 1, 'coldest': 1, "couple's": 3, 'relayed': 1, 'well-written': 9, 'on-court': 1, 'follies': 1, 'brainiac': 2, 'glitz': 1, 'petrice': 1, 'ability': 63, 'traditionally': 2, 'collectively': 1, 'silk': 1, 'jeanine': 1, 'consumer': 2, 'leichtling': 1, 'mastering': 1, 'fee': 3, 'nails': 8, 'harkens': 2, 'tug': 2, '28th': 1, 'rebounding': 1, 'attracting': 4, 'tirade': 1, 'phallic': 1, 'represented': 9, 'hmmmmm': 1, 'longstreet': 1, 'outlined': 1, 'mets': 1, 'half-naked': 1, 'still-present': 1, 'hayek': 5, 'starsystem': 1, 'reputation': 23, 'saturn': 1, "video's": 1, 'totoro': 1, 't-shirts': 1, 'comte': 1, 'what-not': 1, 'tan': 4, 'finely-tuned': 1, 'absolved': 1, 'storm': 16, 'guerilla': 1, 'dami': 1, 'gazers': 1, 'slogan': 2, 'dissembles': 1, 'alfonso': 1, 'leaks': 1, 'clockwatchers': 1, 'recycles': 2, 'peebles': 1, 'neglecting': 1, 'lois': 2, 'foreground': 1, "ilona's": 1, "andreas'": 1, 'gratuities': 1, 'innumerable': 2, 'impressive-as-ever': 1, 'knightdom': 1, 'arrange': 2, 'humorously': 2, 'dauntingly': 1, 'outline': 6, 'human-factor': 1, 'hayashi': 2, "sv2's": 1, 'ronny': 1, 'barcelona': 1, 'directing-wise': 1, 'mankind': 14, 'skarsgard': 8, 'revisions': 1, 'characterized': 6, 'misadventures': 5, 'there=92s': 1, 'barroom': 1, 'immersive': 2, "`ronna'": 1, 'breathe': 8, 'bay': 13, 'summarized': 2, 'magali': 1, "serra's": 1, 'wonsuk': 1, 'targeting': 1, 'goldsman': 1, 'asked': 39, 'limb': 2, 'swarms': 4, 'steriotypically': 1, 'fishnets': 1, 'dictator_': 1, 'parts': 82, 'forwarned': 1, 'plexus': 1, 'meet': 98, 'waltzing': 1, 'tightens': 1, 'produce': 17, 'calming': 2, 'betrayed': 5, 'infuriated': 2, 'wives': 6, 'flutist': 2, 'swank': 1, 'undisclosed': 1, 'bombers': 1, 'disfigured': 3, 'smart/sassy': 1, 'uncorking': 1, 'hordes': 3, 'ruthlessness': 1, 'paulie': 4, 'substantive': 1, 'holland': 3, 'committee': 5, "chris'": 2, 'abundant': 2, 'preaches': 1, 'bloated': 6, 'pappy': 1, "takin'": 1, 're-edited': 1, 'genericized': 1, 'rookie': 9, 'microsoft': 1, 'entails': 2, 'hidding': 1, 'culled': 3, 'offing': 2, 'monday': 1, 'industry': 35, 'iago': 1, 'composers': 2, 'blends': 4, 'disapproving': 1, 'afterglow': 1, "wilde's": 1, 'daydreams': 2, 'firm': 17, 'raiders': 4, "portillo's": 1, 'midway': 4, 'marital': 2, 'grossly': 1, 'engage': 10, 'retaliating': 1, 'sommers': 1, 'partaking': 2, 'sorvino': 7, 'chart': 1, "attanasio's": 1, 'towns': 4, 'science-fiction/action': 1, 'oneliners': 1, 'graphic': 44, 'conditions': 12, 'vocal': 14, 'watercolors': 1, 'vinterberg': 1, 'convince': 22, 'screenwriters': 25, 'cusack': 13, 'alakina': 1, 'meteor': 6, 'experience': 117, 'reconst': 1, "west'": 1, 'mu-mu': 1, 'underused': 10, "johnny's": 1, 'bottin': 2, 'intentionally': 7, "'gi": 1, 'entitled': 12, 'studies': 10, 'yuk': 1, 'animate': 2, 'penny': 3, 'hair': 43, 'eponymous': 1, 'knocking': 4, 'doorstep': 2, 'magical': 26, 'gecko': 2, 'chute': 2, 'vibrate': 1, 'janney': 3, 'lindberg': 1, '2000': 12, 'prescence': 2, 'koch': 1, 'cheater': 1, 'regulations': 1, 'society': 69, 'crazy/beautiful': 1, 'remakes': 5, 'catering': 1, "diesel's": 1, 'dither': 1, 'attire': 4, 'andree': 1, 'reiterated': 1, 'mufasa': 1, 'prettiest': 1, 'two-dimensional': 2, 'benny': 4, 'relies': 21, 'charade': 2, '1847': 1, 'sadists': 1, "trunchbull's": 1, 'maxim': 1, 'boomers': 2, 'strictly': 12, 'dogme': 1, 'matchmaking': 1, 'obligingly': 2, 'minimalist': 3, 'belongs': 12, 'larger': 21, 'terrified': 4, 'guilderland': 3, 'lubricants': 1, 'non-traditional': 1, '[kjv]': 1, 'invades': 1, "cleese's": 1, 'horrifyingly': 2, 'hitchhike': 1, "'contact'": 1, "boss'": 1, 'goddamn': 2, 'sour': 3, 'burdened': 3, 'hotel': 27, 'life-threatening': 1, 'cons': 2, 'femme': 3, 'stillness': 1, '1554': 1, 'skateboarders': 1, 'awakenings': 3, 'patrol': 2, 'seduce': 8, 'throws': 17, "anney's": 1, 'mercifully': 3, 'wisecracking': 1, 'migrated': 1, 'datalink': 1, 'masturbation': 1, 'dour': 2, "youngest's": 1, 'near-constant': 1, '1965': 1, 'prize': 9, 're-watching': 1, 'stocks': 2, 'assumption': 1, 'calais': 1, 'starlets': 1, 'losses': 2, 'reigning': 4, 'barney-like': 1, 'emma': 12, 'innocence': 29, "diaz's": 2, 'time-titanic': 1, "seagal's": 1, 'jobs': 23, 'cocktail': 1, 'tossable': 1, 'corrupt': 19, 'legible': 1, 'vain': 6, 'garziano': 1, 'wellness': 1, 'shameless': 2, 'war-torn': 1, 'quicksilver': 1, 'seed': 1, 'spontaneity': 3, 'deceased': 4, 'prime': 26, "institute's": 2, 'high-cultured': 1, '26-year': 1, 'interviewers': 1, 'renee': 3, 'leppenraub': 1, 'jakob': 1, "michele's": 1, 'art-imitates-life': 1, "'artistic'": 1, 'overpaid': 1, 'touchstone': 2, 'pornographers': 3, 'classified': 3, '_matewan_': 1, 'roberts-whose': 1, 'decent': 52, 'wedge': 1, 'large-breasted': 1, 'beating': 23, 'robes': 2, 'tolerance': 4, 'mondays': 1, 'financed': 2, 'atlantis': 2, 'ever-infectious': 1, 'remembering': 6, "totty'": 1, 'dick_': 1, 'spats': 1, 'karate-kicking': 1, 'errand': 1, 'stewart--makes': 1, 'solace': 3, 'interplay': 4, 'tutelage': 1, 'valentine': 1, 'favorties': 1, 'pentecostal': 3, 'carlos': 2, 'selleck': 1, 'silhouettes': 2, 'best/worst': 1, 'fuller': 6, 'simmering': 1, 'infamy': 2, 'parameters': 1, 'probable': 2, "reese's": 5, 'interact': 12, 'ache': 1, 'embarked': 1, "kat's": 1, 'bible-reading': 1, 'parable': 3, 'lament': 1, 'ira': 2, 'graveyard': 1, 'creators': 5, 'prosecutor': 3, 'strippers': 1, 'excitable': 2, 'berry': 4, 'sacrificing': 7, '$23': 1, 'palisades': 1, 'life-form': 1, 'rewarded': 7, 'internet': 15, 'horseback': 3, 'admiral': 1, 'assasination': 2, 'grandmother': 8, 'impressively': 8, 'graying': 1, 'luciano': 1, 'bumper-sticker': 1, 'complications': 4, 'garth': 2, 'sensitivity': 3, 'hero--or': 1, 'pollock': 2, 'shifty': 5, 're-ignite': 1, 'optimism': 8, 'factors': 6, 'awesomeness': 1, 'animatronic': 3, 'slogans': 4, 'nearby': 21, 'behaviors': 1, 'purportedly': 1, 'song': 52, 'step-for-step': 1, 'sly-style': 1, 'passed': 20, 'assistant': 30, 'decision': 35, 'conti': 1, '_is_': 2, 'aficionado': 1, 'saxon': 1, 'putty': 1, 'tax': 3, 'global': 3, 'noblewoman': 1, 'devolved': 1, 'impertinent': 1, 'profound': 13, 'nigel': 11, 'isaacs': 1, 'push': 7, 'february': 8, 'years': 313, 'londoner': 1, 'godzilla': 9, 'problematically': 1, 'examined': 5, 'shaving': 1, 'aesthetic': 2, 'baz': 6, '41': 2, 'contracting': 1, 'man': 373, 'confinement': 2, 'carey': 1, 'well-made': 10, 'anthrax': 1, 't-name': 1, 'labute': 3, 'aspects': 45, 'creek': 7, 'wide-spread': 1, 'barbarians': 1, 'marky': 2, 'plume': 1, 'unsatisfying': 11, 'gain': 20, 'extreme': 39, "'chasing": 1, 'functionary': 1, 'studly': 1, "winters'": 1, 'cam': 1, 'presumed': 1, 'rueland': 1, 'intimidated': 1, 'toller': 4, "individual's": 2, 'humanistic': 2, 'roebuck': 1, 'discernable': 2, 'faints': 1, 'chen': 3, 'recreations': 1, 'brisk': 7, 'reeked': 1, 'gromit': 1, 'humour': 30, 'advice': 23, 'vengeful': 4, 'giamatti': 2, 'ruse': 2, 'glasgow': 1, 'peters': 1, 'rediscovered': 1, 'debutante': 1, 'zilch': 1, 'fastest': 3, 'crimson': 9, 'margins': 2, 'mao': 2, 'victoire': 1, 'streetwalker': 1, 'chagrin': 2, 'colleges': 1, 'nordoff-hall': 1, 'pleasurable': 1, 'caruso': 2, 'distinguished': 8, 'erosion': 1, 'ladybug': 2, 'sv2': 1, 'one-liner': 2, 'flora': 1, 'lawyer-in-training': 1, '25-year': 1, 'vocals': 4, 'honey': 6, 'anti-pill-popping-casual-tantric-sex-car-chase-attempted-murder': 1, 'wager': 3, 'disintegrate': 2, 'countrymen': 3, 'dreaming': 3, 'whalberg': 1, 'testicles': 1, 'nancy': 3, 'mission': 47, 'relative': 12, 'dads': 1, "merrill's": 1, 'codas': 1, 'burgon': 1, 'blast': 16, 'scalise': 1, 'inhabitants': 11, 'alessandro': 2, "he'll": 14, 'no-brainer': 4, 'enough-': 1, 'nuttiness': 1, 'king': 61, 'tykwer': 1, 'point-of-view': 7, "n'bushe": 1, 'winn': 1, "run'": 1, 'grand': 46, 'dewitt': 7, 'jinn': 6, "amidala's": 2, 'distinct': 10, 'snow-capped': 1, 'vicious': 16, 'zags': 1, 'hitch': 4, 'appreciated': 14, 'aguilar': 1, 'supposes': 1, 'purpose': 50, 'pansies': 1, 'shipped': 3, 'gnome': 1, 'commandeering': 1, 'mount': 4, 'past': 143, 'friendships': 5, 'adventurous': 7, 'understanding': 33, 'birdcage': 3, 'brashly': 1, 'candidate-director': 1, 'traits': 4, 'twenty-five': 2, 'berkeley': 2, "chuckie's": 1, 'scrutinization': 1, 'rejuvenating': 1, 'duplicate': 2, 'plot-driven': 1, 'expositions': 1, 'uninitiated': 2, 'misdeed': 1, 'ffing': 1, 'carly': 2, 'hustled': 1, 'scares': 11, '`we': 1, 'modernized': 2, 'rat-like': 1, 'outlining': 2, 'bright-hot': 1, 'swaggering': 1, 'sausalito': 1, 'hyperkinetic': 2, 'guinea': 1, 'whiner': 1, 'double-crosses': 5, 'quintessential': 5, 'exacting': 2, 'invigorated': 1, 'identity': 28, 'manager': 16, 'developments': 10, 'new-age': 1, 'rein': 1, 'rolling': 19, 'amadeus': 2, 'forty-eight': 1, 'prostitutes': 2, 'tenor': 2, 'mcgann': 1, 'forged': 2, 'premier': 3, 'tough-guys-who-rarely-': 1, 'blend': 17, 'commentators': 2, 'cliche-fest': 1, 'scene-stealer': 1, 'closed-off': 1, 'examine': 6, 'blurt': 1, 'lapaine': 1, 'hagen': 1, 'clearer': 3, "this'll": 1, 'vega': 8, 'sympathizers': 1, 'aquit': 1, 'obi': 2, 'fang-fest': 1, 'unsatisfied': 5, 'quasi-sadistic': 1, 'champ': 2, 'darling': 4, "lapaglia's": 1, "'so": 1, 'spine-tingling': 1, 'omission': 2, 'tournament': 1, 'layouts': 1, 'beliner': 1, 'enchanted': 5, 'vandalizing': 1, 'ezekiel': 1, 'divorcee': 1, 'vaporize': 1, 'reader': 5, 'gleam': 1, 'intervals': 1, 'panavision': 1, 'aki': 2, 'jewelry': 3, 'platoons': 1, 'retired': 9, 'furious': 11, "'fiona": 1, "`experience'": 1, 'writihing': 1, 'respectfully': 1, 'educational': 3, 'comprises': 1, 'whereabouts': 1, 'hegel': 1, 'left': 178, 'sneaky': 1, 'programmed': 1, 'palance': 1, 'thematic': 13, "beckinsale's": 1, 'paperwork': 1, 'ranchers': 1, 'lumbering': 2, 'waterworld': 6, 'writer-director-producer': 1, 'radiates': 1, 'flanked': 1, 'riggs': 3, 'scripts': 15, 'myers': 13, 'uphill': 1, 'self-exploration': 1, 'cranked': 1, 'faster-than': 1, 'scraping': 1, 'foolish': 6, 'nostaligism': 1, 'transforms': 7, '6/10': 7, 'latches': 1, "leon's": 1, 'equally': 57, 'heartstrings': 2, 'grievances': 2, "aliens'": 5, 'other-worldly': 1, 'ronna': 3, "sculptor's": 1, 'persson': 1, 'reoccurrence': 1, 'sparse': 2, 'docudramas': 1, 'watchers': 2, 'wastes': 3, 'borrows': 8, 'controlled': 16, 'repetitions': 2, "must've": 1, 'puffs': 1, 'slyly': 2, 'rameses': 1, "rendezvous'": 1, 'udall': 2, 'out-of-touch': 1, "greene's": 1, "willard's": 2, 'vernier': 1, 'rye': 1, 'resolves--surprisingly': 1, 'muff': 2, 'client': 10, 'president': 35, 'channing': 3, 'responsible': 31, 'bigot': 2, 'hugging': 2, 'hermosa': 1, 'veinotte': 1, 'beelzu-babe': 1, 'germane': 2, 'experimental': 4, 'belgians': 1, 'ya': 6, 'outback': 2, "newton-john's": 1, "campaign's": 1, 'sly': 17, 'taghmaoui': 1, 'rables': 2, 'winkless': 1, 'source': 33, 'billed': 4, 'osbourne': 1, 'multiplex': 5, 'schwarzbaum': 1, 'banks': 10, 'arriving': 11, 'whimsically': 1, 'worsened': 2, 'cinematically': 6, 'nagging': 1, 'sour-old-matriarch-from-hell': 1, 'chivalrously': 1, 'tightrope': 1, 'berserk': 1, 'admits': 15, 'questioned': 3, 'lighting': 17, 'stevie': 1, 'playboys': 1, 'preludes': 1, 'junior': 6, 'bared': 1, "majesty's": 1, 'faculty': 2, 'considerably': 12, 'mutiny': 1, 'marrakech': 1, 'flashier': 1, 'affiliations': 1, 'messengers': 1, 'zeroing': 2, 'yvonne': 1, 'homosexual': 10, 'sewell': 6, 'housekeeper': 2, 'sweet-eerie': 1, 'one-dimensional': 12, 'murdered': 19, 'serb': 1, "wrong--i'm": 1, 'snuggling': 1, 'transcends': 10, 'twisty': 3, 'william': 56, 'salesmen-talk': 1, 'inordinate': 1, 'predetermined': 1, 'affected': 12, 'colony': 7, 'sexuality': 19, "hurt's": 1, 'fund': 4, 'saturating': 1, 'acquit': 2, 'cassidy': 4, 'immeasurably': 3, 'penalties': 1, 'disability': 1, 'wearisome': 1, "'almost'": 1, 'fleshes': 1, "crew's": 1, 'graced': 3, 'wearing': 18, 'redistribution': 2, 'dialogue-those': 1, 'filmgoing': 1, 'bozo': 1, "fantasy's": 1, 'immigration': 1, "should've": 2, 'self-conscious': 2, 'factory-worker-turned-prostitute': 1, 'governments': 5, 'cronies': 7, 'hutt': 7, 'cornered': 2, "obi-wan's": 1, "lee's": 6, 'moritz': 1, 'resurrecting': 1, 'degraded': 1, 'comprising': 1, 'expressions': 13, 'd-day': 3, 'warda': 1, 'potemkin': 3, 'strutting': 3, 'corporation-funded': 1, 'interrelated': 1, 'plastic': 6, 'screwed-up': 1, 'haim': 1, 'fuels': 3, 'state-of-the-art': 2, 'escorted': 1, 'hornby': 1, "international's": 1, 'tale': 110, 'resist': 12, 'nakedly': 1, 'blooming': 1, 'not-inconsiderable': 1, 'sassy': 4, 'acceptable': 11, "gale's": 2, 'abound': 8, 'hogan': 4, 'snow-encrusted': 1, 'pg-13': 13, 'amplify': 1, 'reassured': 1, 'sundered': 1, 'wanda': 1, 'nobler': 1, 'two-fold': 1, 'penchant': 6, 'soft': 20, 'sardonic': 1, "paltrow's": 3, 'crime': 65, 't&a': 1, 'localized': 1, 'wraps': 6, 'sometimes-tedious': 1, 'xvi': 2, "'realistic'": 1, 'beatific': 1, 'ex-gangsta': 1, 'bess': 2, 'slow-witted': 1, 'mold': 4, 'computer-driven': 1, 'wrong-doing': 2, 'jada': 2, 'earpiece': 1, 'detriment': 1, 'album': 4, 'defecated': 1, 'loyalty': 13, 'gunpowder': 1, 'weaver': 9, 'proven': 10, 'wryly': 1, 'tickle': 1, 'divorce': 13, 'jean-pierre': 3, '.': 900, 'hard-edged': 2, 'wo': 2, 'winding': 1, 'mystique': 1, 'assume': 19, 'approaching': 6, 'recycled': 6, 'photographed': 6, 'howdy': 1, 'juncture': 2, 'castleton': 1, 'environment--': 1, 'frailty': 1, 'priest': 15, 'sigourney': 8, 'resevoir': 1, "'gothic'": 1, 'knee-jerk': 1, 'apparition-like': 1, 'chooses': 19, "lib'": 1, 'cinnamon': 1, 'phils': 1, 'newer': 6, 'paralleled': 1, '[sorry': 1, 'exciting': 64, 'wildlife': 2, 'zhou': 2, 'knights': 10, 'caberat': 1, 'contraceptive': 1, 'superfluous': 6, 'haddad': 2, 'capture': 24, 'kunz': 1, 'much-welcome': 1, 'interiors--in': 1, 'frock': 1, 'trot': 1, 'skyler': 1, 'baker': 16, "skipping'": 1, 'leftovers': 1, 'times': 227, 'adherents': 1, 'vivid': 12, 'morphin': 1, 'debbie': 5, "marlowe's": 1, 'theresa': 5, 'fonzie': 1, "shaud's": 1, 'exceedingly-professional': 1, 'starships': 1, 'fresher': 3, 'gimmicks': 1, 'stoker': 1, 'effortlessly': 19, 'heading': 14, "towne's": 2, 'oscar-nominated': 3, 'groceries': 2, '`loose': 1, 'needing': 4, 'nuggets': 2, 'dragged-out': 1, 'remi': 3, 'believes': 47, 'pine': 1, 'seized': 1, 'sant': 6, 'universe': 23, 'candles': 3, 'stepson': 1, 'recovering': 3, 'kio': 1, 'malfunctions': 1, 'sucess': 1, 'gen-xer': 1, 'so-named': 1, 'lends': 17, 'stimulates': 1, 'shue': 4, 'accepting': 12, 'caligula': 1, 'night-sky': 1, 'shigeta': 1, 'ejected': 1, 'conducting': 3, 'fleas': 1, 'barnyard': 1, 'suffrage': 1, 'kadosh': 1, 'points': 64, 'perpetual': 2, 'audre': 1, 'versatility': 1, 'recorder': 2, 'booming': 4, 'dalmatian': 1, "pescucci's": 1, 'instigated': 1, 'scowls': 3, 'space': 70, 'heroic': 14, 'top-40': 1, "patient's": 2, 'achieving': 6, 'medley': 1, 'mosaic': 2, 'horror-trilogy': 1, 'swearing': 5, "doesnt'": 1, 'thriving': 2, 'ramblings': 1, 'subplot': 29, 'bugs': 10, 'unsuitable': 2, 'bum': 5, 'places': 47, 'deep-seated': 1, 'paradise/prison': 1, 'fierce': 6, "ebouaney's": 1, 'korben': 1, 'everything-and-the-kitchen-sink': 1, 'lighten': 2, 'crutch-carrying': 1, 'news': 37, 'amid': 7, 'tori': 2, "verhoeven's": 3, "boorman's": 1, 'female': 43, 'enhancement': 1, 'lebanese-american': 1, 'sarandon': 4, 'contributions': 4, 'near-death': 1, 'ford': 24, 'banned': 3, 'barreness': 1, 'grateful': 2, 'self-image': 1, 'corrected': 1, 'pieh': 1, 'casualties': 7, 'maj': 2, 'interval': 1, 'jillian': 1, 'dinosaur-sized': 1, 'disembowelments': 3, 'forgetting': 4, 'hosted': 4, 'investing': 4, 'contacts': 6, 'flagging': 3, 'crinkled': 1, 'closeups': 1, 'till': 13, 'diseased': 1, 'cocoon': 3, 'sitcom-ish': 1, "qui-gon's": 2, 'self-respecting': 1, 'fingered': 1, "avary's": 1, 'self-assured': 4, 'helps': 47, 'clowning': 1, 'peyote': 2, 'chablis': 1, "henry's": 4, "wittliff's": 1, 'negligent': 1, 'sebulba': 1, 'piaf': 1, 'islandnet': 1, 'jews': 10, 'self-preserving': 1, 'incredulously': 1, 'staple': 7, 'guy-using': 1, 'commoner': 4, 'lamp': 1, 'gilliam-esque': 1, 'previews': 14, 'rocketeer': 1, 'pullam': 1, 'rear-ended': 1, 'flurry': 1, 'wardrobe': 6, 'reach': 39, 'atheists': 1, 'anarchy': 3, 'plaintive': 2, 'drastic': 5, 'throaty': 2, 'soundly': 1, "winfield's": 1, 'reeling': 1, 'switchblade': 1, 'cored': 1, 'stories': 66, 'understudy': 1, "o'connell": 3, 'bullseye': 4, 'co-executive': 2, 'walk': 43, 'trudge': 1, 'hopelessness': 3, 'mauds': 1, 'burtt': 1, 'flamm': 2, 'tian': 1, 'imprisonment': 2, 'pocket': 8, 'skulls': 2, "bowfinger's": 2, 'mathieu': 1, 'body': 97, 'insults': 4, 'testifying': 1, 'sues': 1, 'relocated': 2, 'down-on-his-luck': 1, 'bubbling': 2, "camel's": 1, 'invisible': 11, 'gratingly': 1, 'faithfulness': 2, 'bobbly': 1, 'reinvention': 1, 'globe': 10, 'jerked': 1, 'tourists': 4, "cholodenko's": 1, 'stomachs': 2, 'trysts': 1, 'developer': 2, '36th': 1, 'hillbilly': 3, 'saddles': 2, 'smooth': 19, 'remember': 80, 'shootout': 3, 'palentologist': 1, 'defiled': 2, 'kujan': 1, 'fo': 1, 'criticize': 2, 'assassinated': 1, 'third-person': 1, 'payback': 4, 'inclusions': 1, 'double': 21, 'eagerness': 2, 'maintenance': 2, 'defenses': 2, 'chandelier': 2, 'really-they': 1, 'buff': 5, 'eccentricity': 2, 'eights': 1, 'host': 16, 'parasites': 2, 'elton': 1, 'protein': 1, 'gargantuan': 1, 'hitmen': 4, 'arise': 9, 'behoove': 1, 'tea': 9, 'lombardo': 4, 'promising': 13, 'tarkofsky': 1, 'des': 1, 'liberally': 1, "lou's": 1, 'daft': 1, 'stern': 8, 'thoroughly-modern': 1, 'confesses': 5, 'blip': 1, 'shrek': 5, 'choreographer': 1, 'laws': 12, 'coachmen': 1, 'bold': 14, 'harrison': 14, 'rebellion': 3, 'carrere': 1, 'front-page': 1, 'multi-colored': 1, 'muncie': 1, 'dunno': 2, 'sear': 3, 'dell': 1, 'riviting': 1, 'marlowe': 3, 'incessantly': 8, 'metropolitan': 2, 'sparks': 15, 'carelesness': 1, 'crows': 1, 'mincemeat': 1, 'stiff': 9, 'locket': 1, "bankole'": 1, 'outlast': 1, 'stream': 4, 'photo': 9, 'bunton': 1, 'snowmen': 1, 'forceful': 5, 'increases': 4, "'f": 1, 'well-liked': 1, 'coupling': 1, 'international': 26, 'anchor': 7, 'sexton': 2, 'dissappears': 1, 'noir-': 1, '15-year': 1, "alien's": 1, 'perpetually': 13, 'child-friendly': 2, 'predictability': 8, 'hollywood-ian': 1, 'voluminous': 1, 'ida': 1, 'reynaldo': 1, 'escapism': 5, 'carefully': 24, 'probation': 2, 'lurid': 5, 'chemo-electric': 1, 'drumlin': 2, 'bones': 5, 'devout': 3, 'blumberg': 1, 'notice': 40, 'celebs': 1, 'power-hungry': 2, 'just-deceased': 1, 'unfrozen': 1, 'colours': 5, "o'brian": 1, 'garofalo': 4, "`padawan'": 1, 'fireflies': 1, 'officers': 10, 'day': 220, 'crustier': 1, 'ripper': 2, 'convulsions': 1, 'bonneville': 1, 'nearly-brain': 1, 'kettle': 1, 'endowment': 1, 'placid': 2, 'riot': 11, "'popcorn'": 1, 'paradox': 4, 'expected': 45, 'dodge': 3, 'itinerant': 1, "elliot's": 1, 'much-talked-about': 1, 'alfre': 2, 'defies': 6, 'episodic': 2, 'charges': 11, 'marisol': 1, 'chicken': 9, 'acknowledges': 3, 'ass-kicking': 1, 'irrelevant': 6, 'godfathers': 1, 'wished': 7, 'responsibilities': 5, 'flourish': 5, 'orderly': 2, 'fucked': 1, 'died': 35, 'substantial': 10, 'initech': 1, 'battleships': 1, 'nerve': 7, 'inheriting': 1, 'a-go-go': 1, 'commodities': 2, 'thorton': 1, 'aims': 6, 'imprecation': 1, 'decisively': 1, 'detectives': 11, 'computer-animated': 7, 'plunge': 3, 'lurch': 1, 'indelible': 6, 'monitor': 3, "farrelly's": 2, 'backstory': 3, 'reinforced': 1, 'glare': 1, 'tattooed': 1, '118': 1, 'whispering': 1, 'gradually': 26, 'hides': 10, 'butt': 7, 'weendigo': 1, 'ridiculed': 4, 'parsons': 2, 'mega': 4, 'husbands': 4, 'accustomed': 5, 'screamers': 2, 'tortoise-like': 1, "filmmakers'": 2, 'playtone': 1, 'apparent': 40, 'hypocritical': 3, 'geller': 1, 'comic-book': 5, 'imaginations': 3, 'balletic': 1, 'bodyguards': 1, '13-': 1, 'thenardiers': 1, 'botching': 2, 'shining': 17, 'bandages': 2, 'cohesive': 5, 'corporate': 9, 'proves': 70, 'confuse': 3, 'inhibition': 1, 'bankruptcy': 2, 'goofy': 30, 'ned': 7, "arnold's": 1, 'dilemnas': 1, "money's": 1, 'slim': 5, 'omelet': 1, 'mile-high': 1, 'end-of-the-world': 1, 'lorenzo': 1, 'widows': 2, 'carrey': 16, 'machinations': 5, 'oakland': 1, "father's": 26, 'loudmouth': 1, 'muppet': 2, 'celebrity-or': 1, 'ferretti': 1, 'bumpy': 2, 'zephram': 1, 'ashton': 1, 'websites': 1, "hawthorne's": 1, 'markedly': 2, 'boozed': 1, 'nudge': 1, 'undermining': 2, 'harrowingly': 1, 'shocks': 8, 'elders': 2, 'ferocious': 4, 'baldwin': 18, 'unarmed': 5, 'bowlers': 1, 'scarsely': 1, 'evokes': 8, 'resolutely': 3, 'redefines': 3, 'hattie': 3, 'airline': 6, "actor'": 1, 'redeeming': 5, 'rank': 8, 'christabella': 1, 'twilight': 8, 'digging': 2, 'distress': 3, 'lakeside': 1, 'zombie': 1, "llewelyn's": 1, 'investigate': 16, 'helen': 14, 'instruments': 3, 'armande': 1, 'locating': 2, 'gunslingers': 1, 'pursuits': 1, 'splendor': 3, 'craft': 13, 'hamper': 1, 'looses': 4, 'augment': 2, 'mccamus': 2, "`believers'": 1, 'iconography': 1, 'termites': 1, "airplane's": 1, "bull'": 1, 'grown': 21, '1999': 40, 'interrogator': 1, 'sisters': 13, 'cleaners': 1, 'summer-time': 1, 'axe-wielding': 1, 'reworkings': 1, "mcdowell's": 1, 'madonna-marrying': 1, 'avenues': 1, 'unreality': 3, 'beckwith': 1, '2010': 1, 'rosary': 1, 'fellows': 2, 'counts': 6, 'none-too-subtly': 1, 'memoir': 1, "suspects'": 2, '31-year-old': 1, 'pre-release': 1, 'incorporating': 6, 'promotes': 3, 'nita': 1, 'eighty': 1, 'ewan': 15, 'drunker': 1, 'serviceman': 1, 'cautious': 5, 'transgression': 1, "lars'": 1, 'thumping': 1, 'distraction': 11, 'carts': 1, 'stupor': 2, 'goal': 34, 'bratiness': 1, 'overheated': 2, 'chats': 2, 'coke': 4, 'laroque': 1, 'modesty': 2, 'illinois': 1, 'notable--and': 1, 'commanding': 25, 'phobia': 3, 'juror': 1, 'anew': 2, 'sullivans': 1, 'opera-type': 1, 'softness': 1, 'proved': 14, 'glacier': 1, 'pals': 9, 'bonitzer': 2, 'policy': 7, 'boastful': 1, 'fronted': 1, 'thirdly': 1, 'whimsical': 9, 'allowed': 28, 'artifice': 3, 'shagwell': 5, 'tuning-up': 1, 'no-thought': 1, 'stiller': 15, 'crossed': 7, 'flaw': 30, 'interstitial': 1, 'intoxicating': 1, 'weirder': 1, 'funniest': 32, 'adjust': 2, 'bald': 8, 'hovers': 2, 'exhileration': 1, 'graham': 17, 'unraveled': 2, 'get-ups': 1, 'dialogueless': 1, 'overdrawn': 1, 'gabe': 1, 'ucla': 1, 'challenged': 5, 'sydow': 2, 'clanking': 1, 'schreiber': 5, 'moriarty': 2, 'malko-visits': 1, 'vibe': 3, 'inquisitive': 2, 'looting': 2, 'azaria': 4, 'louie': 1, 'hung': 4, 'celebrities': 8, 'wisecracks': 6, "doll's": 1, 'revisitings': 1, 'terrorists': 8, 'strives': 4, 'welch': 2, 'skittish': 1, 'acknowledging': 4, 'rebuffed': 2, 'phillippe': 3, 'bottles': 1, 'incarcerated': 4, 'turn': 145, 'realistic': 72, 'corporation': 9, '=20': 3, 'pantolianto': 1, 'triangle': 11, 'flicker--a': 1, 'boning': 1, 'orginal': 1, 'lehman': 1, "frisch's": 1, 'capitalism': 2, 'liv': 5, 'jump-in-your-seat': 1, 'shirley': 2, 'ghoulishness': 2, 'locales': 5, 'nebulous': 2, 'level': 90, 'mismatched': 4, 'penal': 1, 'lanky': 1, 'guile': 1, 'grubby--but': 1, 'sorrowful': 1, 'cremet': 1, 'quick-fire': 1, 'risk-taking': 2, 'famke': 7, 'all-time': 9, 'bukater': 6, 'location': 20, 'jonesy': 1, 'devotees': 3, 'underplay': 1, 'momentum': 12, "granger's": 8, 'natassja': 1, 'heer': 1, 'downfalls': 2, "breckman's": 1, 'elie': 1, 'mot': 1, 'cusackian': 1, 'throwing': 20, 'cgi': 14, 'frenchie': 1, 'gavin': 1, 'arroway': 4, "rubin's": 1, 'zappa': 1, 'superstructures': 1, 'nicole': 9, 'neglected': 5, 'occurs': 22, 'beings': 22, 'scattered': 4, 'taxed': 1, 'boringly': 1, 'mixed-up': 1, "uhry's": 1, 'journeyed': 1, 'hooks': 6, 'staring': 8, 'sterile': 3, 'cross-cutting': 1, "bergman's": 2, 'speculate': 3, 'permeates': 2, 'jingle': 1, 'puccini': 1, 'sera': 1, 'skidded': 1, 'eschewed': 2, 'insulatory': 1, 'fizzles': 3, 'wouldn=92t': 1, 'snippets': 1, 'triggered': 2, 'cheesy': 19, 'frequent': 21, 'near-future': 1, 'fields': 7, 'forty-five': 4, 'raise': 25, 'manipulator': 3, 'chamber': 6, 'vegas': 19, 'commerce': 2, 'attempts': 67, 'foppish': 5, "newton's": 1, 'cathedrals': 1, 'bitten': 7, 'grosse': 6, 'burnings': 1, 'romanced': 1, 'failures': 6, "neighbor's": 1, 'interior': 5, 'pour': 4, 'bit': 199, 'pregnant': 17, 'tang': 1, 'reynaud': 1, 'smallish': 2, 'drama-its': 1, 'ostensibly': 4, 'corporations': 1, 'apted': 1, 'sensuality': 4, 'iron-fisted': 1, 'doting': 4, 'mentally-challenged': 1, 'strive': 6, 'self-contradicted': 1, 'fun-in-the-sun': 1, 'yom': 1, 'disconnects': 1, 'planets': 7, "'interview": 1, 'reporter': 17, "`x-files'": 1, 'walloping': 1, 'melancholy': 12, 'evolves': 4, 'quirkiness': 1, 'slave-owners': 1, 'ad2am': 2, 'swing': 4, 'varying': 8, 'mixing': 14, 'wigs': 2, 'unfounded': 1, 'recap': 3, 'lunatic': 4, "nations'": 1, 'rages': 1, 'bureaucratic': 1, 'sideburns': 1, 'kel': 1, 'concert': 12, 'roderick': 1, 'idealized': 4, 'blood': 67, 'genuine': 46, 'misogynist': 1, 'pseudo-climax': 1, 'concentrates': 10, 'mild': 9, 'lounging': 2, 'inmate': 2, 'bauer': 1, '$1': 5, 'highlighting': 3, 'big-haired': 1, 'joseph': 16, 'angst': 5, 'noticed': 20, 'inventive': 28, 'critic-mode': 1, 'loaf': 5, 'crispness': 1, 'cousin': 8, "sheedy's": 1, 'inhabits': 5, 'then-reigning': 1, 'diasappointing': 1, 'slammed': 1, 'gayle': 1, 'mark': 67, 'continuos': 1, "'79": 1, 'moot': 1, 'maka': 2, 'pods': 1, 'dweller': 1, 'wharton': 1, 'intense': 55, 'slouched': 1, 'charmed': 3, 'recall': 23, 'trapped': 20, 'half-way': 3, 'incomparable': 2, 'justification': 1, 'bout': 3, 'top-secret': 2, 'lowly': 2, 'eyes': 116, 'integral': 2, 'breen': 1, 'loft': 2, 'siren': 3, "singer's": 1, 'sung': 5, 'avowed': 1, 'decorates': 1, "horror's": 1, 'partygoer': 2, 'chopping': 1, 'emmanuel': 3, 'afraid': 28, 'passageways': 1, 'gator-wrestling': 1, 'linoleum': 1, 'humphrey': 3, 'uncle': 18, 'receptionist': 2, 'module': 1, 'acrimonious': 1, 'pauly': 2, 'odd-sounding': 1, "titta's": 1, 'defends': 4, "heinlein's": 1, 'expansionism': 1, 'barrels': 2, 'implode': 1, 'odious': 1, 'le': 3, 'plagiarisms': 1, 'naughtiness': 2, 'gertz': 1, 'illogically': 1, 'satisfying': 45, 'inconsiderate': 1, 'rob': 34, 'charisma': 23, 'overplaying': 1, 'spooky': 6, 'arm-in-arm': 1, 'spoiling': 4, 'chasing': 16, 'chance': 91, 'gold-digging': 1, 'schmaltz': 1, 'scholar': 4, 'hunches': 1, 'wreaked': 1, 'moving--usually': 1, 'syrupy': 2, 'star-crossed': 2, 'showdown': 16, 'walther': 1, 'footprints': 1, 'onlookers': 2, 'rumble': 7, 'snipes': 3, 'edifice': 1, 'comprehending': 1, 'boil': 2, 'seemlessly': 1, 'singing': 24, 'stunned': 9, 'prospects': 1, 'viscera': 2, 'scantily-clad': 1, 'endearingly': 3, 'oblige': 1, 'torture': 11, 'inspector': 17, 'fingernail': 2, 'hara': 1, 'pulses': 1, 'seventh': 5, 'writer-director-star': 2, 'sock': 2, 'squirrel': 5, 'armour': 2, 'enthusiasts': 4, 'videogames': 1, 'refreshing': 33, 'announcer': 3, 'baffled': 4, 'quintessentially': 1, 'unlikeable': 2, 'orange': 8, 'tripping': 2, 'out-of-step': 1, 'familiarize': 1, 'tailored': 1, 'precise': 9, 'fantastic': 45, 'slade': 2, "mozart's": 1, 'cinque': 5, 'strangelove': 3, 'verbally': 3, 'townsfolk': 4, '1969': 5, 'citizen': 12, 'olyphant': 4, '30%': 1, 'song-and-dance': 2, 'frankenstein-films': 1, 'harkened': 1, 'alluring': 3, 'timothy': 9, 'renews': 1, 'posit': 1, 'rainy': 2, 'archimedes': 1, 'over-long': 1, "o'connor": 6, 'gloom': 5, 'transitional': 1, 'vague': 12, 'duality': 2, 'fluctuations': 1, "prison's": 1, 'interupts': 1, 'assuaged': 1, 'peggy': 1, 'rams': 1, 'chereau': 1, 'tantamount': 2, 'half-witted': 1, 'anti-woman': 1, 'true-to-life': 2, 'cece': 1, 'thandie': 5, 'unfaithfulness': 1, 'solomon': 1, 'oscar-nominee': 1, 'transition': 9, 'bath': 5, 'bowed': 4, 'medium': 12, 'kinetic': 4, 'catalyze': 1, 'chore': 1, 'scores': 9, 'previewed': 1, 'darnedness': 1, '_real_': 1, 'relieving': 1, 'protection': 5, "floop's": 1, 'chewbacca': 6, 'flips': 6, 'payoff': 9, 'reportedly': 13, 'solidify': 1, 'adrenaline': 4, 'barracades': 1, "brandi's": 1, 'briesewitz': 1, 'established': 15, 'beret': 1, 'horseflies': 1, 'spewing': 3, 'wickedly': 5, 'newly-broken': 1, 'rosenbaum': 1, 'jetee': 1, 'denies': 5, 'tanked': 2, 'exclusion': 1, 'tide': 9, 'payload': 1, 'anti-appeal': 1, 'crackled': 1, 'cannes': 6, 'unsophisticated': 1, 'recording': 4, 'execution': 18, 'setting-intensified': 1, 'cleverness': 2, 'larter': 5, "autumn's": 1, 'aftertaste': 2, 'subdued': 8, 'suspiciously': 3, 'scandal': 5, 'lovesick': 2, 'october': 14, 'synchronised': 1, 'smuggles': 1, "mcgann's": 1, 'spaces': 9, '90%': 3, 'gonz': 1, 'reptilian': 3, 'goo': 1, 'neophyte': 2, 'short-circuits': 1, 'flamingo': 2, 'ratcheted': 1, 'sneak': 9, 'hollywood-ized': 1, 'handled': 21, "coffey's": 1, 'precedes': 4, 'dazed': 2, 'malleability': 1, 'ordered': 12, 'matter-of-factly': 2, 'theater-shaking': 1, 'aimlessly': 2, 'spared': 3, 'tradiational': 1, 'require': 13, 'ins': 1, 'plagues': 2, '[r]': 6, 'reruns': 3, 'snatch': 1, 'ga-ga': 1, 'blasted': 1, 'jury': 5, 'disadvantage': 1, 'defects': 2, 'exacly': 1, 'refuse': 5, 'quo': 4, 'delivering': 22, 'ideals': 11, 'tavington': 1, 'munching': 1, "chidduck's": 1, "rabbit'": 1, 'mercury': 6, 'cross-pollination': 1, 'critically': 13, 'dirty': 22, 'kershner': 1, 'expression': 18, 'reflection': 9, 'redcoat': 1, '1950': 1, 'sore': 3, 'varela': 1, 'kitchen': 8, "mom's": 3, 'marginalized': 1, 'goodman': 14, 'capitalize': 3, 'vinyard': 2, 'continent': 3, 'trap': 5, "june's": 1, 'time-conserving': 1, 'detect': 1, 'come-hither': 1, 'nc': 1, 'flaming': 4, 'ventura': 3, 'carnegie': 2, 'personification': 2, 'objection': 1, 'tentatively': 2, 'blessed': 7, 'rebelous': 1, 'swore': 3, 'corrects': 1, 'festive': 1, 'docter': 1, 'conceptually': 1, 'brave': 12, "president's": 6, 'wang': 4, 'itching': 2, 'greed': 16, 'pervasiveness': 1, 'mile': 11, 'jeunet': 2, 'misled': 1, 'abruptly--': 1, 'deer': 1, 'dien': 5, 'fetched': 2, 'sarah': 20, "abby's": 1, 'temptress': 2, 'slimeball': 1, 'discord': 1, 'jekyll': 2, 'sets': 120, 'masseur': 1, 'home/dance': 1, 'agamemnon': 1, 'savoy': 1, 'ranft': 2, 'genetic': 5, 'unhurried': 2, 'stipulated': 1, 'linus': 1, 'footsteps': 8, 'boothe': 2, 'hijack': 2, 'michaels': 2, 'mano-et-mano': 1, 'link-ups': 1, 'first-timers': 1, "supermarkets'": 1, 'wrinkles': 1, 'reprocessed': 1, 'papa': 1, 'horn': 1, 'recognizing': 2, 'accurately': 12, 'nail-biting': 1, 'jimi': 1, 'oracle': 1, 'communicating': 2, 'motss': 1, 'unconsidered': 1, 'avidly': 2, 'barbie': 3, 'industrial': 13, 'monotony': 1, 'arrow': 11, 'beekeeper': 1, 'featherweight': 1, "rota's": 1, 'tentative': 3, 'sammy': 1, 'force-loving': 1, 'hood\x12s': 1, 'dedicates': 3, 'vein': 4, 'daydream': 1, "castle's": 1, 's&m': 1, 'manifold': 2, 'undertsanding': 1, 'rafting': 1, 'sightings': 2, 'coherence': 3, 'ruby': 2, 'shih': 1, 'chronically': 1, 'overruns': 1, "mob's": 1, 'coincidence': 10, 'dude': 19, 'overturned': 3, 'straw': 2, 'unerving': 1, 'kathe': 1, "nygard's": 1, 'sweetness': 10, 'crack': 12, 'congratulations': 2, 'stylized': 11, 'tours-de-force': 1, 'grovel': 1, 'caton': 1, 'strong': 130, 'chest-': 1, 'obi-wan': 13, 'heterosexual': 5, 'form': 97, 'achievement': 23, 'mallory': 3, 'scripture': 1, "david's": 5, 'peagreen': 1, 'cosmonaut': 1, 'maples': 1, 'famous': 63, 'gorier': 1, 'connery': 5, 'excruciating': 1, 'babylon': 1, 'dolled': 1, 'exemplifying': 1, 'homey': 1, 'formulaisms': 1, 'freddy': 3, 'sonnenfield': 1, 'turgidson': 1, 'purchasing': 1, '105': 3, 'taj': 1, 'rocking': 4, 'flushing': 1, 'prints': 2, 'gladnick': 1, 'paup': 1, 'knot-tieing': 1, 'splattering': 1, 'pepe': 1, 'demonstrating': 4, 'lecherous': 1, 'downplays': 1, 'considers': 13, 'symmetry': 1, 'agee': 1, '107-year-old': 1, 'overdue': 3, 'slip-ups': 1, 'diet': 2, 'sayings': 1, 'philosopical': 1, 'partnerships': 1, 'frightening': 43, 'isabelle': 3, 'flaws': 70, 'harvard-educated': 1, 'pairing': 8, 'gates': 6, 'lyne': 1, "turturro's": 2, 'mild-mannered': 4, 'weaker': 5, 'boy-likes-girl': 1, 'subservient': 1, 'rebirth': 3, 'tranquility': 2, 'bisset': 1, 'experimenting': 4, 'negative': 18, 'trance': 4, 'switches': 6, 'mum': 2, 'freeway': 3, 'condition--non-studio': 1, 'burnout': 1, 'sufficiently': 3, 'gwen': 1, 'witnessed': 12, 'freshness': 4, 'disgustingly': 2, '_hustler_': 1, 'lauren/sylvia': 1, 'tostitos': 1, 'lemmy': 1, 'emmy': 3, 'mambos': 1, "husband's": 8, 'iraqi': 1, 'eeriness': 1, 'listener': 1, '1-on-1': 1, 'carol': 7, 'georgetown': 1, 'koans': 1, 'hard-ass': 1, "herzog's": 1, 'core': 29, 'cameraderie': 1, 'turnabout': 1, 'pie': 10, "deceiver's": 1, "shane's": 1, 'kerrigans': 1, 'tiresome': 8, 'voiced-over': 1, 'throat': 10, "d'oh": 1, 'producer': 40, 'ewwwws': 1, 'flanders': 2, "rosaleen's": 1, 'seeker': 2, 'silliness': 3, 'flake': 1, "rising's": 1, 'nine-month-old': 1, 'natalie': 18, 'multi-plot': 1, 'far-away': 1, 'structured': 8, 'complaints': 11, 'genie': 2, 'incest': 1, 'leland': 1, "forlani's": 1, 'goddam': 1, "grodin's": 1, "x'": 1, 'dianne': 2, 'ambiguous': 10, 'socialist': 1, 'bench': 1, 'shapes': 1, 'regroup': 2, 'annual': 10, 'margoyles': 2, 'solution': 11, 'diddly': 1, 'mouth': 39, 'engine': 4, 'acquiescence': 1, 'smugly': 1, 'pickup': 1, 'confined': 5, 'flexing': 1, 'filmmkaing': 1, 'schizophrenic': 5, 'decemeber': 1, 'warning': 25, '*': 85, '30-year-old': 1, 'moved-and': 1, 'promoted': 1, 'tiniest': 1, 'helicoptered': 1, 'flock': 7, 'carver': 8, 'monotonous': 5, 'charted': 1, 'expressionism': 1, 'guild': 1, 'impoverished': 3, 'blurry': 2, 'extremely': 123, 'bio-pics': 1, "horses'": 2, 'stamina': 5, 'ants': 5, 'repercussions': 3, 'lower-case': 1, 'rivers': 6, 'lifted': 7, 'jargon': 1, 'snagged': 1, 'powdered': 2, 'pointing': 4, 'hysterically': 5, "allen's": 8, 'white-fenced': 1, 'handstand': 1, 'maximal': 1, 'rudely': 1, 'terrorizes': 2, 'shae': 1, 'empires': 2, 'figuratively': 4, 'musician': 3, 'wrestle': 1, 'rooting': 16, 'circles': 6, 'julie': 10, 'arecibo': 1, 'possessing': 1, 'conspiracies': 2, 'non-stereotyped': 1, 'well-acted': 4, 'victory': 11, 'prowess': 4, 'high-concept': 3, 'dirtiness': 1, 'dougray': 1, 'face/off': 8, 'cowardice': 2, "year's-open": 1, 'upper-class': 1, 'idea-driven': 1, 'hit-': 1, 'hayseed': 1, 'five-year': 2, 'blown': 10, 'borrowed': 9, 'ribs': 1, 'hk': 2, 'convenience': 6, 'orbit': 2, "angela's": 1, 'dinsmoor': 2, 'christy': 1, 'knee': 2, 'completely': 148, 'perpetuate': 1, 'calf': 1, '|': 3, 'heightened': 7, 'hitchhikes': 1, 'squeamish': 5, "`dawson's": 1, 'propose': 1, 'music': 158, 'red': 55, 'camel': 1, 'spunky': 7, 'then-technological': 1, 'smart': 70, 'despicable': 5, 'slain': 1, 'pull': 37, 'summon': 2, 'crawling': 6, '1958': 3, 'contractor': 2, "shepard's": 1, 'volatile': 5, 'tiriel': 1, 'superstition': 1, 'coda': 2, 'relishes': 3, 'physicality': 1, 'gee': 4, '1990': 9, 'enjoy': 108, 'drank': 1, 're-connect': 1, 'ineffectual': 1, 'pakistan': 1, 'forecast': 2, 'lasts': 3, 'dorm': 1, 'freezing': 7, 'rented': 6, 'invincibility': 2, 'pie-making': 1, 'carolco': 1, "'rich'": 1, 'spaniards': 2, 'nit-picking': 1, 'adversarial': 3, 'defenders': 1, 'moon': 11, 'sex-scandal': 1, '_patlabor': 1, 'frustration': 8, 'tasty': 3, 'ultra-cool': 2, 'frighteners': 2, 'roman': 8, 'exaggeration': 5, 'yen-khe': 1, 'perceptible': 3, 'define': 8, 'farrell': 1, 'verdell': 1, 'lump': 2, 'pampered': 2, 'closest': 9, 'prematurely': 3, 'trampled': 1, 'fingal': 1, 'erect': 1, 'dating-and': 1, 'disturbing': 53, 'reinventing': 3, 'hans': 6, 'ultra-orthodox': 1, 'viewers': 74, 'tempting': 4, 'literal': 10, "penn's": 3, 'sprucing': 1, 'matchmaker': 4, 'mute': 7, 'rocks': 7, 'naifeh': 1, "shop's": 1, 'fur': 3, 'invigorating': 2, 'indignities': 1, 'genres': 10, "dictator's": 1, 'succeeded': 11, "bing's": 1, 'stowed': 1, 'opus': 3, 'gentleman': 6, 'fortunes': 1, 'intricately': 4, 'strengthening': 2, 'persuasion': 3, 'multi-dimensional': 6, 'godard': 3, 'hayley': 1, 'vacuum': 1, 'flashback': 18, 'tribulations': 7, 'located': 7, 'asian-american': 3, 'thrust': 12, 'skeksis': 1, 'far-off': 1, 'trademarks': 2, 'delve': 7, 'sucks': 1, 'fantasized': 1, 'ghostbusters': 5, 'stimulation': 2, 'psyches': 2, '-type': 1, 'manny': 2, 'beam': 3, 'pollak': 1, 'ontario': 1, 'jour': 1, 'wrecking': 4, 'dextrous': 1, 'lunes': 1, 'rosie': 7, 'lookalikes': 1, 'lennix': 1, 'belle': 4, 'hovering': 3, 'superficial': 11, 'wresting': 1, 'dealt': 9, 'exercising': 1, 'limits': 11, 'altar': 3, 'contingency': 1, 'misfit': 1, 'well-dressed': 1, "suggs'": 1, 'donations': 1, 'pigeon-holed': 1, 'townsperson': 1, 'harbors': 3, 'vilified': 1, 'beautiful': 126, 'overeager': 1, 'nightmares': 8, 'spinal': 1, 'sidekicks': 3, 'volleyball': 1, 'spiriting': 1, 'non-verbal': 1, 'framework': 7, '17': 15, 'reconcile': 1, "line's": 1, 'congratulate': 3, 'jen': 2, 'coffee': 12, "t'aime": 1, 'crescendo': 2, 'whatsoever': 8, 'money-making': 1, 'grumbles': 1, 'undertones': 6, 'wage': 4, "holnists'": 1, 'helgenberger': 2, 'doorway': 2, "bacon's": 5, 'rufus': 7, 'closely': 17, 'truck': 21, 'ballad': 6, "achin'": 1, 'embarassed': 1, 'cushing': 2, 'cargo': 2, 'sky_': 1, 'laugher': 1, "`kundun'": 1, 'sadness': 9, "frat's": 1, 'cohagen': 1, 'icky': 1, 'hanks/meg': 1, 'joking': 5, 'katrin': 1, 'tricky': 5, 'so-so': 4, 'madsen': 3, 'neumann': 1, 'revenge-driven': 1, 'dumped': 7, 'abstinence': 2, 'rewrites': 1, 'mis-adventure': 1, "palantine's": 1, 'dissects': 2, 'trend': 9, 'amusingly': 2, 'pertwee': 2, 'soundtrack': 67, 'randall': 5, 'tags': 5, "toy's": 1, "things'": 1, 'acrobats': 1, 'rollers': 1, "pantoliano's": 1, 'red-carpet': 1, 'not-so-obvious': 1, 'tizard': 1, 'messier': 1, 'entertainment': 98, 'bilal': 1, 'recon': 2, 'intoning': 1, 'architecture': 5, 'lorre': 2, 'byron': 2, 'intellectualized': 2, 'pinocchio': 4, 'hiver': 1, 'hustles': 1, 'thusly': 1, 'salsa': 1, 'thicker': 2, 'resurfaces': 2, 'designing': 3, 'average-joe': 2, 'burbon': 1, 'gauzy': 1, 'telecommunications': 1, 'affair': 35, "elizabeth's": 3, 'barbie-doll': 1, 'filmming': 1, '20th': 12, "weir's": 3, 'hip-hop': 4, 'teen-styled': 1, 'knew': 43, "'no": 1, 'jason': 23, 'tragedy': 36, 'install': 1, 'bound': 27, 'robespierre': 2, 'gladstone': 1, 'lists': 3, 'furball': 1, "taxpayer's": 1, 'taboos': 2, 'shakier': 1, 'gasmask': 1, 'accompaniment': 3, "amistad's": 1, '$400': 1, 'acrylic': 1, 'waste': 17, 'reflexively': 1, 'eases': 1, 'pointed': 16, 'video': 80, 'percentage': 4, 'betrayals': 1, '-jackson': 1, 'pleasures': 14, 'anymore': 27, 'fretful': 1, 'superweapon': 1, 'concurs': 1, 'bevy': 1, "attendant's": 1, 'sample': 3, 'rummages': 1, 'jodie': 8, 'xtc': 1, 'vegas-based': 1, 'patently': 2, 'stinky': 2, '18-24': 1, '#8': 1, 'clea': 1, 'honor': 24, 'intellectualizing': 1, 'ling-ching': 1, 'exchanges': 11, 'horor': 1, '40s': 1, 'brancato': 1, 'betrays': 1, 're-shot': 1, 'bunker': 2, 'manifesting': 2, 'lopped': 1, 'born': 45, 'odds': 16, "navaz's": 1, 'yorkers': 4, 'resource': 1, 'burkhart': 1, 'vent': 3, 'beetles': 3, 'gripped': 3, 'soldering': 1, 'filmography': 1, 'self-centered': 4, 'marginally': 5, 'advertisement': 2, 'unfurls': 1, 'mib': 5, 'affects': 7, 'preached': 1, 'pleading': 3, "rooker's": 2, 'courtyards': 1, 'war--that': 1, 'gorillas': 5, 'underestimate': 1, 'sona': 1, 'moaning': 2, 'fourteenth': 1, 'captive': 4, 'surrenders': 2, 'brought': 81, "fate'": 1, 'liking/disliking': 1, 'preparations': 4, 'sculptor': 3, 'movements': 17, 'writer': 64, 'living': 95, 'lionize': 1, 'knock': 11, 'basically': 68, 'prized': 1, "hackman's": 2, 'bbc': 2, 'johnathon': 1, 'middle-class': 3, 'bull-horn': 1, 'artist': 41, 'nieces': 1, 'cad': 2, 'unchecked': 2, 'sentences': 5, 'radek': 1, '`varsity': 1, 'one-night': 1, 'provincial': 1, 'bolton': 1, 'glassy': 1, 'sibling': 2, 'craziness': 2, 'psychopathic': 1, 'answers': 32, 'webb': 1, 'marybeth': 1, "stan's": 1, 'danish': 4, 'thrugh': 1, 'portentous': 2, 'exclude': 1, 'stricken': 1, "stricter's": 1, 'unnerving': 3, 'red-eyed': 1, "skinheads'": 1, 'unnerved': 2, 'morgan': 16, 'compassion': 10, 'scenery': 15, 'erratic': 2, 'winged': 1, 'fledged': 1, 'succession': 2, 'affliction': 3, 'years-gone-by': 1, 'blunt': 6, 'billions': 1, 'scriptwriters': 1, 'faux-sympathy': 1, 'propensity': 3, 'benign': 3, 'palette': 6, 'steadi-cam': 1, "'philosophy": 1, "bethany's": 2, 'direct-to-video': 3, 'collectible': 2, 'couple': 147, 'table': 22, 'line-up': 3, 'bille': 3, 'weddell': 1, "ball's": 1, 'one-day-a-week': 1, 'true-life': 2, 'scissor-happy': 1, 'grand-daddy': 2, 'pseudo-playmate': 1, 'mgm': 2, 'underlings': 1, 'gospel': 2, 'taxing': 4, 'long-term': 2, 'heroics': 1, 'rarity': 5, 'four-star': 7, 'moron': 3, 'three-pronged': 1, 'wordly': 1, 'saves': 15, 'retained': 1, 'perversion': 2, 'yuck': 2, 'didi': 1, 'closese': 1, 'soon-to-be-ruler': 1, 'dunks': 1, 'analogous': 1, "stayin'": 1, 'rabbi': 2, 'beatnik': 1, 'world': 313, 'malfunctioning': 1, 'tran': 1, 'dependent': 5, 'kwan': 1, 'gonna': 19, '7': 24, 'well-earned': 1, 'stays': 22, 'males': 6, 'awe-inspiring': 6, "'til": 1, 'drain': 1, 'greatest': 68, 'tension-building': 1, 'winston': 1, 'capitulate': 1, 'int': 1, 'confer': 1, 'mayor': 6, 'reborn': 3, 'strove': 2, 'loop': 1, "sonny's": 5, 'separatist': 1, 'handily': 1, 'trite': 14, '`clerks': 1, 'apologize': 3, 'outrunning': 1, 'medal': 2, 'spiced': 1, 'blond': 8, 'anonymously': 3, 'elias': 6, 'existence': 31, 'nightly': 2, 'unlawful': 2, "grisham's": 5, 'scarlett': 3, 'showbusiness': 1, 'perkily': 1, 'filmed': 55, 'omnipresent': 1, 'participated': 1, 'academic': 6, 'date-night': 1, 'high-stakes': 3, 'fuddy-duddy': 1, 'denied': 10, 'karl': 2, 'hamill': 5, 'ebouaney': 2, 'entrepreneur': 2, 'drugged-out': 2, 'failing': 5, 'towne': 1, 'grudgingly': 3, 'tape-to-film': 1, 'co-mingling': 1, "malkovich's": 3, 'prospect': 2, 'frowns': 1, 'zoom-out': 1, 'clenched': 1, 'cristal': 1, 'rehash': 4, 'foray': 6, 'not-so-wise': 1, 'casanova--what': 1, 'nourishing': 1, 'donna': 4, 'imprisoned': 12, 'songcatcher': 1, 'janis': 1, 'strides': 3, 'videocassette': 3, 'overlong': 7, 'spearing': 1, 'soho': 1, 'coliseum': 3, 'race--starting': 1, 'noted': 11, 'avoiding': 10, 'leary-sandra': 1, 'preserved': 2, 'manslaughter': 1, 'cutter': 3, 'poled': 1, 'multimillion-dollar': 1, "seth's": 3, "`ev'": 1, 'mendes': 1, 'resent': 4, 'cinderella': 1, 'virginia': 3, 'imitation': 5, 'curly-haired': 1, 'overkill': 4, 'punched': 2, 'ultimate': 34, 'good-not-great': 1, "head'": 1, 'sizeable': 1, 'mime': 2, 'rent': 21, 'archival': 1, 'half-wit': 1, 'adrenalin': 1, 'tunes': 22, 'mirren': 3, 'bottoms': 1, 'hurry': 3, 'son': 106, 'neptune': 2, 'extinct': 1, 'bethlehem': 2, 'weaves': 8, 'patterson': 3, "arroway's": 1, 'dietrich': 1, 'caleb': 1, 'pepto-bismol': 1, 'cleave': 1, 'viewpoints': 1, 'smith': 37, 'kaurism': 1, 'death-as-pitt': 1, 'wakefield': 1, 'interacting': 2, 'specifications': 2, 'liability': 1, 'filmgoer': 1, 'freelance': 1, 'mini-masterpieces': 1, 'numb': 1, 'magic-realism': 1, 'predilection': 2, 'subs': 1, 'pumped': 3, 'scholarly': 2, 'incorrect': 2, "earth's": 3, 'cosby': 1, 'restricts': 1, 'tinted': 3, 'disconcerting': 3, 'mobbed': 1, 'cite': 1, 'broker': 3, 'additional': 8, 'acquitted': 1, 'rambo': 1, 'vivica': 1, 'caped': 2, 'shuffle': 1, 'bashed': 2, 'ashamed': 1, 'verbatim': 1, 'dare': 16, 'jobless': 1, 'nephews': 1, 'bodacious': 1, 'all-around': 2, 'explicit': 12, 'quinn': 5, 'spain': 9, 'activist': 3, 'embarrassingly': 3, 'brahma': 1, "horrocks'": 1, 'machina': 1, '_does_': 1, 'atmostpheric': 1, 'stan': 5, 'nfl': 2, "brean's": 1, 'noun': 1, 'membership': 1, 'erotically': 1, 'deplore': 1, 'simplest': 3, 'rivka': 1, "segonzac's": 1, 'seinfeld': 8, 'genome': 1, 'arbuthnot': 1, 'installation': 1, 'compadre': 1, 'tropics': 1, 'randle': 1, 'rhyme': 1, 'viking/bowling': 1, '101': 2, 'estrogenic': 1, 'signed': 6, 'hid': 1, 'traveled': 3, 'olga': 1, "kiki's": 1, 'carpark': 1, 'scruffy': 1, 'gondola-trekked': 1, 'unadulterated': 1, 'shagadelic': 1, 'watchdog': 1, 'solos': 2, 'woo-styled': 1, 'sensei': 1, 'facade': 8, 'robbery': 12, 'patsy': 1, 'messiest': 1, 'fullest': 6, 'jamileh': 1, 'miss': 47, 'delightful': 32, 'mcgoohan': 2, '97-minutes': 1, 'sidesteps': 1, 'yeoh': 6, 'measles': 1, 'raul': 1, 'armando': 1, 'passion': 31, 'indoctrinated': 2, 'shipping': 2, 'exposure': 3, 'wide-eyed': 7, 'elmore': 9, 'heard': 71, 'gunfight': 2, 'campy': 11, 'vista': 1, "snowball's": 1, 'mimmo': 1, 'ambitious': 19, "nomi's": 1, 'cetera': 1, 'saints': 1, 'fanaticism': 1, 'shower': 8, 'mobsters': 7, '70mm': 2, 'counterexample': 1, 'scowl': 1, 'wendell': 1, 'amphetamines': 1, 'tristine': 1, 'altman': 9, 'coffers': 1, 'her--in': 1, 'deliriously': 2, 'scratching': 1, 'brother': 75, 'advertise': 1, 'backup': 1, 'grodin': 1, 'limitations': 4, 'theatens': 1, 'tex-mex': 1, 'dolores': 4, 'telescopes': 1, 'brawley': 1, 'skirting': 1, 'reports': 13, 'shrine': 1, 'contest': 11, 'buddies': 18, 'beguiling': 4, 'synch': 1, 'shwarzenegger-like': 1, 'strip-act': 1, 'buckets': 1, 'baroque': 1, 'un-detailed': 1, 'jean-jacques': 1, 'wan': 3, "jolie's": 1, 'painstaking': 1, 'bamboo': 2, 'insufferable': 1, "o'brady": 1, 'malko': 1, 'sabbatical': 1, 'monsters': 19, 'forecasts': 2, 'harrold': 1, 'exemplify': 1, 'mandible': 1, 'wonderful--i': 1, 'comfortable': 16, "cassandra's": 1, 'lobbyists': 1, 'sauce--the': 1, 'pittsburgh': 1, 'unheard': 6, 'bathed': 2, "'titanic'": 1, 'microwave': 2, 'wandering': 11, 'bluntman': 1, 'motherly': 2, 'mano': 1, 'drooling': 1, 'filters': 3, 'phelps': 2, 'fetish': 3, 'sling': 3, 'volker': 1, 'streeming': 1, 'unwinding': 1, 'essay': 1, 'modus': 1, 'pirate': 3, 'sin': 13, 'ferdinand': 1, 'well-balanced': 1, 'guinness': 5, 'unfinished': 5, 'debatable': 2, 'screenplays': 4, 'screen-viewer': 1, 'suave': 10, 'crewman': 1, "cameron's": 16, 'prospective': 5, 'roots': 14, 'annette': 11, 'impoverishment': 1, 'loathe': 5, 'palmer': 4, 'bunyan': 1, 'angelica': 1, 'awol': 2, 'labour': 3, 'plucks': 1, "randy's": 1, 'advertiser': 1, 'performances-': 1, 'cater': 1, 'sigmund': 1, 'ocp': 1, 'posts': 1, 'tyrannical': 4, 'horrible': 26, "'see": 1, 'thereof': 8, "vault's": 1, 'frantically': 7, 'trents': 1, 'hinges': 3, 'bride-to-be': 1, 'fujimoto': 1, 'sex/masturbation': 1, 'articulated': 1, 'tax-auditor': 1, 'countryside': 9, 'wachowski': 3, 'backwoods': 6, 'stogie': 1, "lloyd's": 3, 'wimpy': 1, 'precursor': 2, 'penetrating': 1, 'cheap-laugh': 1, 'posessing': 1, "farquaad's": 2, 'idiosyncracy': 1, "wife's": 12, 'layers': 5, 'loops': 3, 'devastating': 12, 'relaxing': 2, 'lustful': 2, 'craving': 2, 'dante': 2, 'gees': 2, 'requires': 26, 'crunchem': 1, "funeral'": 1, 'wednesday': 4, 'pathetic': 28, 'monterey': 1, 'bubble': 6, 'caffrey': 1, 'wiseguys': 3, 'lifeforce': 1, 'crushing': 1, 'chan': 17, 'culturally': 1, 'faldwell': 1, 'virulent': 1, 'grisham': 5, 'torturing': 3, 'soars': 4, 'always-grisly': 1, 'mean-spiritedness': 1, 'daria': 1, 'overreacts': 1, 'token': 13, 'insanely': 5, 'marisa': 3, 'brothel': 1, 'jackson/chris': 1, "gray's": 1, 'focusing': 4, 'impulsively': 1, 'imelda': 1, "spielberg's": 17, 'increments': 1, 'ethel': 3, 'proverb': 2, 'meddled': 1, 'guessing': 10, 'wyatt': 1, 'dealing': 28, 'valour': 1, 'disguises': 5, 'technicolor': 6, 'church-the': 1, "lithgow's": 1, 'superstar': 7, 'specializes': 4, '?': 493, 'messiah': 5, 'literalized': 1, 'uncanny': 9, 'muhammad': 1, 'compromised': 3, 'idella': 1, 'kathie': 1, 'ebeneezer': 1, 'retelling': 1, 'smokes': 3, 'refusing': 11, 'assholes': 1, 'enlightening': 3, 'caretaker': 2, 'addled': 1, 'incentive': 1, 'surprises': 36, 'supporter': 1, 'graffiti': 4, 'doorman': 1, 'shoo-be-doo': 1, 'incriminator': 1, 'stores': 8, 'winslett': 2, 'conducts': 2, '`reindeer': 1, 'thora': 3, 'brutal': 26, 'pitfalls': 1, 'awe': 7, 'underpaying': 1, 'backhanded': 1, 'dud': 2, 'blane': 1, 'tribesmen': 1, 'oscar-worthy': 5, 'crucial': 22, 'stereotypically': 1, 'atlantic': 13, 'gusty': 1, 'mai': 2, 'affection': 24, "what'sgoingonaroundhere": 1, 'damn': 31, 'connundrum': 1, 'circuit': 4, 'evicted': 1, "fox's": 4, 'prodigal': 1, 'transposed': 1, 'consisted': 3, 'pale': 8, 'hippies': 2, 'hardcover': 1, 'ingenious': 10, 'artificial': 16, 'profits': 2, 'crusher': 1, 'unbalanced': 2, 'marion': 2, 'humvee': 1, 'tollbooth': 1, 'insanity': 11, '[': 1, 'signing': 2, "adams'": 1, 'produced': 51, 'informative': 4, 'coach': 10, 'adriana': 1, 'berkley': 1, 'tackle': 6, 'overlooked': 14, 'neidermeyer': 1, 'thing-': 1, 'defecting': 1, 'deathcamps': 1, 'german': 30, 'coltrane': 2, 'faith-healing': 1, 'rectitude': 1, 'shots': 74, 'playes': 2, 'rapist': 1, 'crown': 5, "shot's": 2, 'houses': 8, 'tiptoeing': 1, 'queue': 1, 'enemy': 21, 'tow': 5, 'permission': 3, 'await': 7, 'dalton': 3, 'snickered': 1, 'upholds': 1, 'opium': 1, 'randolph': 1, 'desk': 5, 'caricaturist': 2, 'lez': 1, 'jokey': 1, 'floated': 1, 'key': 61, 'cleans': 1, 'organizations': 3, 'pawnbroker': 1, 'soothingly': 1, 'commune': 4, '-set': 1, 'ridiculously': 6, 'shell-game': 1, 'vindictive': 1, 'rapidly': 10, 'variant': 2, 'atreus': 1, 'homoerotic': 4, 'pressured': 2, 'director/writer': 7, 'suffice': 16, 'trounced': 1, 'deceive': 1, 'admiring': 1, 'criticizes': 2, 'swordfishing': 1, 'flown': 3, 'manifest': 2, 'puzo': 2, 'grays': 1, 'duong': 1, 'rugged': 4, 'castle': 17, 'rescues': 5, 'lyle': 1, 'hundred': 18, 'humanist': 2, 'mandel': 2, 'haig': 1, 'sex-maniac': 1, "'shagging": 1, 'parent': 15, '6': 17, 'ambitions': 10, "shelly's": 1, 'sweep': 7, 'discouragingly': 1, 'inflection': 3, 'culture-specific': 1, 'deal': 102, 'engrossing': 15, 'spellbound': 1, 'confides': 1, 'distinguishes': 3, 'photographs': 7, 'approximate': 1, 'hypnotherapist': 1, 'dakota': 1, 'exhausted': 5, 'nicest': 2, 'wizard': 7, 'displayed': 8, 'nerves': 4, 'geekish': 1, 'leap': 7, 'knuckle': 2, 'ranking': 1, 'over-stylized': 1, 'hanako': 1, 'eartha': 1, 'pained': 5, 'lumbergh': 1, 'big-name': 2, 'timeless': 10, 'campaign': 19, 'assumes': 10, 'relinquishes': 1, 'idea': 117, "cliff's": 1, 'drug/rave': 1, '45-year-old': 1, 'spotted': 3, 'lands': 14, 'initial': 29, 'backlash': 2, 'movie-within-a-movie': 1, 'walters': 3, 'soak': 3, 'trunks': 2, 'underwear': 1, 'warming': 1, 'justified': 6, 'things': 279, "'smocky'": 1, 'macpherson': 1, 'fascism': 3, "ifans'": 1, 'haunting': 30, 'purrs': 1, "pam's": 1, 'dries': 1, 'mixes': 8, 'endings': 11, 'finicial': 1, 'creates': 52, 'self-reference': 2, 'surrounds': 6, 'proving': 14, 'novicorp': 1, 'habitat': 2, "bitchin'": 1, 'mod': 3, 'roundly': 1, 'feel-good': 18, 'viceversa': 1, 'disbelief': 19, 'department': 26, 'monica': 2, 'middleton': 1, 'quanq': 1, 'poker': 13, 'collaborate': 3, 'aft': 1, 'derides': 1, 'bobby': 15, 'noir-ish': 2, 'titanic-buff': 1, 'mewes': 4, 'monarchy': 1, 'languidly': 1, 'raindrops': 1, 'monsieur': 1, 'martial': 19, 'number': 86, 'andrew': 23, 'months': 40, 'purists': 1, 'ceased': 2, 'dinah': 1, 'trudging': 1, 'vagrants': 1, 'foggy': 2, 'breathed': 1, "italy's": 1, 'practically': 22, 'emissary': 3, 'derived': 5, 'duties': 7, 'obssessively': 1, 'urgently': 1, "figgis'": 1, 'trails': 1, 'jean-paul': 1, 'lampoon': 1, 'robe': 3, 'nabs': 1, 'chatty': 1, 'southerners': 1, 'clayton': 8, "stephane's": 1, 'needles': 3, 'satire': 30, 'tatyana': 1, 'heading--but': 1, 'moniters': 1, 'drang': 1, 'volatility': 1, 'reject': 1, 'green': 38, 'geraldo': 1, 'underplayed': 2, "bathgate's": 1, 'braga': 1, 'castaways': 1, 'phenomenal': 11, "'helen'": 1, 'denizen': 1, 'bland': 15, 'insight': 21, 'guest-host': 1, "preacher's": 1, 'once-blossoming': 1, 'prop': 1, '1977': 10, 'fruitition': 1, 'jealous': 12, 'smarts': 1, 'mind-reading': 2, 'nose': 11, 'gooding': 6, 'overly-dramatic': 2, 'vincennes': 1, 'repetitive': 7, 'governor': 5, 'ditto': 2, 'crawford': 3, 'plea': 1, 'oscar-consideration': 1, 'convention': 6, 'old-self': 1, 'vaudeville': 1, 'millar': 1, 'tireless': 1, 'picasso': 1, 'chuck': 6, 'addresses': 13, "biziou's": 1, 'bacteria': 1, 'employment': 2, 'wolfe': 1, 'resourceful': 3, 'addicts': 1, 'tantrum': 2, 'splits': 3, 'getup': 1, 'dominating': 3, 'rushmore': 7, 'cody': 2, 'carpenter': 9, 'maxi': 1, 'blackburn': 1, 'tourism': 1, 'furrow': 1, 'benigni': 4, 'neice': 1, 'irving': 5, 'cabby': 1, 'drones': 1, 'coraghessan': 1, 'army/navy/air': 1, 'lovely': 29, 'goldie': 2, 'cotterill': 1, 'treats': 27, 'collage': 1, 'front-and-center': 1, 'ravaging': 1, 'wowed': 1, 'chafing': 1, 'with--gasp--a': 1, 'crazies': 1, 'x-mas': 1, 'conquer': 2, 'kat': 2, 'pills': 2, 'must-sees': 1, 'pop-in': 1, 'nichols': 3, 'breakfast': 8, "beloved's": 1, "'steptoe": 1, 'wit': 37, 'juxtapose': 1, 'gazillions': 1, 'richly': 5, "smilla's": 1, 'swastica': 1, 'retrospect': 4, 'serum': 1, 'border-jumpers': 1, 'insecure': 7, 'unveiled': 2, 'exceedingly': 9, 'trimming': 1, 'phoebe': 3, 'hoffman': 13, 'migrate': 1, 'presiding': 1, 'persistently': 1, 'jane-': 1, 'dietl': 1, "gugino's": 1, 'crests': 1, 'lecture': 2, 'kline': 2, 'worthless': 4, 'abductions': 1, 'etched': 2, 'jake': 23, '30-minute': 2, 'acclaimed': 20, 'case': 140, 'worse': 62, 'boundless': 1, 'contention': 1, 'elicit': 4, 'out-of-shape': 1, 'henchman': 3, 'hallways': 1, 'pal': 10, 'cartoonishly': 1, 'interference': 2, 'rape': 12, 'self-destructive': 4, 'obscurity': 6, 'fernando': 4, 'passages': 1, "beaumarchais's": 1, 'dog-bone': 1, 'discourteous': 1, 'roar': 2, 'silently': 2, 'rant': 5, 'gadgets': 9, 'nookey': 1, 'affleck-damon': 1, 'precisely': 17, 'two-headed': 2, '150': 3, 'life-or-death': 1, 'high-roller': 1, 'cultured': 1, 'gleaned': 1, 'pre-existing': 1, ']': 7, 'half-sister': 1, 'unannounced': 2, 'invitation': 3, 'exhaustion': 1, 'onward': 3, 'timed': 1, 'mishap': 3, 'probing': 2, 'labyrinth': 4, "o'donnell": 10, 'split-screen': 1, 'violence/gore': 2, 'acquits': 2, 'jacket': 7, 'improvements': 3, 'nearest': 5, 'spying': 2, 'insightful': 18, 'foots': 1, 'politically-charged': 1, 'macnamara': 1, 'curry-spiced': 1, 'sheet': 2, 'fans': 72, 'customary': 4, 'perennially': 1, 'caveman': 1, 'adjective': 5, "judd's": 1, 'crying': 12, 'insane': 14, 'chipperly': 1, 'boob': 1, 'inspiration': 15, 'gambling': 13, 'hiberli': 1, 'confiscate': 1, 'timberland': 1, 'small': 179, 'farther': 11, 'acrobat': 1, '20': 25, 'voicing': 2, 'chlo': 1, 'affords': 1, 'scrapbook': 1, 'alacrity': 1, 'treetop': 1, 'eqsuisite': 1, 'adventurer': 3, 'spread': 10, 'nocturnal': 2, 'protestor-marching': 1, 'phallus': 1, 'littlest': 3, 'noah': 6, 'copulate': 1, 'vhf': 1, 'morbid': 3, 'sito': 1, 'condemning': 3, 'graves': 2, 'maniacial': 1, 'schoolmarm': 1, 'grows': 37, 'differentiates': 1, 'exploration': 14, 'excellently': 1, 'generalizations': 1, 'shears': 2, 'ridiculing': 1, 'rivera': 2, 'uproarious': 3, 'self-consciously': 2, 'palm': 3, 'adaptation--in': 1, 'dividing': 2, "`sophisticated'": 1, 'donate': 1, 'respectable': 14, 'hidden': 40, 'webber': 2, 'trashy--well': 1, 'prepare': 6, 'heinous': 3, 'bowie-like': 1, 'israel': 1, "dick's": 1, 'landou': 1, 'hook': 18, 'ashbrook': 1, 'softener': 1, 'lyrical': 5, 'cases': 17, 'antz': 7, 'desolation': 2, 'dinner-table': 1, "riggs'": 1, 'body-themed': 1, 'tommorow': 1, 'soong': 1, 'slapped': 4, 'conflict': 35, 'confidence-lacking': 1, 'passengers': 8, 'displays': 26, 'expulsion': 1, 'shape-shift': 1, 'chambers': 4, 'races': 8, 'editorial': 3, 'novels': 11, 'sell-out': 1, "paul's": 3, 'penleric': 1, 'creeping': 2, 'involved': 105, 'storyline': 44, 'feathered': 1, 'distinctly': 3, '`d': 1, 'hot-blooded': 2, "sense's": 1, 'evades': 1, 'objecting': 1, 'undaunted': 1, 'jack-off': 1, 'noticable': 2, 'tolerate': 3, 'dudley': 2, 'off-camera': 1, '$16': 1, 'manages': 108, 'imported': 2, 'interestingly': 6, 'ceti': 1, 'multi-purpose': 1, 'trivia': 6, "barbara's": 1, 'turned': 81, 'seductive': 6, 'maiden': 8, 'everything--it': 1, 'uncreative': 1, 'good': 537, 'channeled': 1, 'jingo-all-the-way': 1, 'procedure': 5, 'rigorous': 1, 'loser': 12, 'mundy': 1, 'comer': 1, 'informs': 7, 'translates': 4, "mouse's": 1, 'babyzilla': 2, ')': 852, 'oddities': 3, 'super-impaling': 1, 'maroon': 2, 'less-than-savoury': 1, 'majorino': 1, 'burned': 4, 'puzzling': 3, 'all-too-willing': 1, 'fold': 2, 'flare': 2, 'finch': 4, 'purity': 5, 'tribute': 15, 'accountable': 1, 'reiser': 2, 'moonlit': 1, 'hitches': 1, 'pseudo-': 2, 'slipping': 2, 'curtly': 1, 'grisly-faced': 1, 'catfights': 1, 'eventuality': 1, 'wuthering': 1, 'florist': 2, 'dapper': 1, 'alain': 3, 'opportunistic': 3, 'questionable': 10, 'solemnly': 1, 'immeadiately': 3, 'renamed': 2, 'instigates': 2, 'opportunity': 50, 'clinches': 1, 'downes': 1, 'calmer': 1, 'unattractive': 2, 'nightclub': 8, "movie's": 79, 'erupt': 1, 'decision--irving': 1, 'sol': 1, 'mind-bender': 1, 'dress': 18, 'smoked': 1, 'bonet': 1, 'fei-hong': 3, 'karras': 1, '777-film': 1, 'finzi': 1, 'paladin': 1, 'sobieski': 3, 'epics': 6, 'attractiveness': 1, 'hay-soos': 1, 'organizing': 5, 'parting': 2, 'broadcast': 16, 'pub': 4, 'remains': 67, 'unlike': 88, 'roxanne': 1, 'camerawork': 9, 'identity--struggling': 1, 'reload': 1, 'nuances': 10, 'boy-meets-girl': 1, 'trigger': 3, '_would_': 1, 'jack-in-the-box': 1, 'weight': 24, 'cottage': 2, '28': 3, 'dissect': 1, 'projectionist': 1, '_titus_': 1, 'preceeding': 1, 'shtick': 5, 'nomination': 38, 'megalopolis': 1, 'content': 32, 'prequels': 1, 'full-frontal': 2, 'obdurate': 1, 'unease': 2, 'suggs': 1, 'grouped': 1, "20's-early": 1, 'guarded': 2, 'tournaments': 1, 'inaction': 1, 'assists': 2, 'flim': 1, 'anij': 1, 'galoshes': 1, 'excrutiatingly': 1, 'persecution': 3, 'abuses': 1, 'intending': 1, 'behind-the-': 1, 'churning': 3, 'spoofs': 6, 'coos': 2, 'bullies': 4, 'gels': 1, 'connelly': 4, 'refuge': 7, 'insure': 3, 'beacham': 1, 'eloquently': 1, 'brisville': 1, 'middleage': 1, "furst's": 2, 'astronomer/writer': 1, 'softens': 3, "rand's": 1, 'suffocates': 1, 'assemblage': 1, 'addicted': 2, 'ressurection': 1, 'fan-base': 1, "janssen's": 1, 'gwyneth': 8, 'anne': 16, 'decimated': 2, 'syd': 1, 'assembled': 12, 'equalizers': 1, 'bowie': 5, 'overgrown': 1, 'springs': 4, "rules'": 2, 'paradoxically': 2, 'majority': 20, 'jackie': 35, 'simpleton': 2, 'nuttiest': 1, 'polack': 1, 'blackman': 2, "fidelity''": 1, 'incense': 1, 'not-too-friendly': 1, 'hospitals': 2, 'mouthing': 2, 'eminently': 3, 'struggling': 20, 'accordance': 2, 'bottlecaps': 1, 'comical': 18, 'digital': 19, 'award': 33, 'taunting': 1, 'skewers': 2, 'thinker': 1, 'over-eager': 1, 'blood-soaked': 1, 'boone-type': 1, 'noticeably': 5, 'accentuating': 1, 'crumb': 2, 'pimpernel': 1, 'kidnapper': 1, 'extremity': 1, 'resistant': 1, 'spielberg-inspired': 1, 'goodness': 11, 'mechanism': 3, 'gremlins': 1, 'ph': 2, 'dostoyevski': 1, 'clements': 1, "curly's": 1, 'wholly': 10, 'stunt': 13, 'break-down': 1, 'kasinsky': 1, "derek's": 4, 'garnered': 4, 'lacrosse': 1, "club's": 3, 'dilapidated': 2, 'waited': 8, 'impresario': 1, 'recognition': 13, 'histrionics': 1, "tv's": 18, 'aerosmith': 1, 'philadelphia-area': 1, 'hottie': 1, 'gobbling': 1, 'tidal': 1, 'ritzy': 1, 'mph': 1, 'transformed': 9, 'great': 369, 'sworn': 2, 'good-natured': 8, 'melrose': 3, 'raped': 10, 'victoria': 4, 'o-dog': 1, 'debilitates': 1, 'vanishing': 4, 'incorporation': 1, 'regard': 17, 'directive': 1, 'duelling': 1, 'copland': 1, 'mindfuck': 2, 'butt-head': 1, 'generosity': 2, 'kids': 103, 'convincing': 64, 'suspense-filled': 1, 'gawain': 1, 'chinatown': 7, 'roommate': 10, 'surprisingly-resilient': 1, 'primrose': 1, 'mickey': 10, 'background': 61, 'godawa': 1, 'director': 333, 'broadly-observed': 1, 'poorly': 11, 'baboon': 1, '-yeager': 1, 'elfman': 10, 'fizzgig': 1, 'shrug': 4, 'intelligentsia': 1, 'tantor': 3, 'ahab': 1, 'successful': 82, 'modernity': 3, 'manipulation': 14, 'unforgettable': 16, "`theory'": 1, 'gitai': 1, 'victimize': 1, 'ents': 1, 'slept': 2, 'unzips': 1, 'sells': 5, 'bikini-clad': 1, 'kid': 64, 'suffer': 22, 'care-free': 2, 'breakups': 1, 'puppet': 6, 'eye-catching': 6, 'villain': 57, 'sampling': 3, "levy's": 1, 'connor': 3, "younger's": 1, 'liu': 1, 'squash': 2, 'corvino': 1, 'roxy': 1, 'sandy': 4, 'vanishes': 2, 'wildness': 1, "rose's": 3, "hitchcock's": 6, 'blackmails': 1, 'habitual': 1, 'insatiable': 2, 'affraid': 2, 'resteraunt': 1, 'mockery': 7, 'battle-damaged': 1, "batman's": 1, "sandler's": 3, 'reliant': 3, 'disney-esque': 2, "gitai's": 1, "uncut's": 1, 'fu': 10, 'impressiveness': 1, 'large': 76, 'partly': 13, 'beek': 2, 'assigned': 21, 'flubber': 1, 'self-produced': 1, 'commitment': 4, 'z-grade': 1, 'shagging': 1, 'understatement': 9, 'grimaces': 1, 'heck': 14, 'analysts': 1, 'rumours': 4, 'thoughtful': 15, 'monger': 1, 'console': 2, 'alien3': 1, 'jail': 30, 'compliment': 8, 'bleakly': 1, 'carville': 2, 'story': 498, 'pod': 11, 'scene': 363, 'decently': 2, 'archaic': 1, 'immediatly': 1, 'rollerblades': 1, 'mistake': 26, "panavision's": 1, 'eszterhas': 1, 'exqueeze': 1, 'ravishing': 1, 'helgeland': 1, 'douchebag': 1, 'showpiece': 1, 'cram': 3, 'dainty': 2, 'unforgiveably': 1, 'vanities': 1, 'comprehend': 6, 'patronizing': 1, 'whirls': 1, 'allegra': 1, 'read': 67, 'transgressions': 2, 'expects': 9, 'showmanship': 1, 'modeling': 3, 'dragonlike': 1, 'burley': 1, 'children': 92, 'escalates': 4, '84': 1, 'capitalizes': 3, '1948': 1, 'terminator': 16, 'coping': 3, 'boo': 1, 'exaggerating': 1, 'jenna': 3, 'once-removed': 1, 'splitting': 5, 'boost': 7, 'spilled': 6, 'intuits': 1, 'fourth': 17, 'coaching': 1, 'thickened': 2, 'tykes': 1, 'approachment': 1, 'well-selected': 1, "secret--he's": 1, "armageddon's": 2, 'd-reper': 1, 'morpheus': 5, 'salesmen': 4, 'non-teen': 1, 'schmaltzy': 4, 'kissner': 2, 'levitt': 1, 'headless': 2, 'giveaways': 2, 'suspicion': 7, 'blew': 6, "gilliam's": 3, 'spiritually': 4, '-reviewed': 1, 'possibly': 52, 'grammer': 2, 'fulfills': 1, 'degeneres': 2, 'respectful': 4, 'quickly-paced': 1, 'kidnapping': 14, 'landlords': 1, 'careens': 1, 'coalwood': 2, "guest's": 1, 'concocted': 4, 'send': 29, 'authorizes': 1, 'becky': 1, '300': 4, 'terrorized': 6, '25': 13, 'second-tier': 1, '`i': 3, 'aline': 1, 'adhered': 1, 'drugs': 27, 'jawbreaker': 1, 'crevillen': 1, 'attraction': 19, 'highest-grossing': 1, 'yuji': 1, 'violinist': 1, 'red-headed': 1, "kasdan's": 1, 'non-issue': 1, 'parker': 12, 'undertaken': 1, 'disastrously': 3, 'venerable': 5, 'moviereviews': 1, 'baptist': 2, 'lightyear': 4, 'heavier': 3, 'padre': 1, 'outrageousness': 3, 'sinker': 1, 'chazz': 4, 'good-spirited': 2, 'motionlessly': 1, 'honeys': 1, 'worlds': 14, 'bach': 3, 'aiello': 1, 'sir': 21, "renard's": 1, 'examples': 12, 'backing': 7, 'presidents': 4, 'philosophical': 12, "africa's": 1, 'rampages': 2, 'convicts': 1, 'turner': 5, 'resists': 3, 'handguns': 2, 'japanese-specific': 1, 'herky': 1, 'religions': 2, "function--smith's": 1, 'equally-innovative': 1, 'pry': 1, 'cast': 263, 'cuff': 1, 'fairy': 18, 'options': 2, 'avoids': 17, 'getaway': 3, "et's": 1, "princess'": 2, 'premieres': 1, 'toucha': 1, 'shorts': 3, 'constricted': 2, "redgrave's": 1, 'addiction': 10, 'anal': 2, 'boast': 5, 'goldsmith': 2, '1988': 8, 'flesh': 24, 'uncooperative': 2, 'difference': 40, 'manual': 2, 'bounteous': 1, 'unconscionable': 1, 'title': 112, 'blethyn': 1, 'eliza': 1, 'poon': 2, 'open-ended': 2, 'scowling': 1, 'speeds': 1, 'serve': 35, 'rosanna': 2, 'juries': 2, 'book/film': 1, 'xv': 1, 'allure': 3, 'sucker-punch': 1, 'tiered': 1, 'langella': 1, 'acknowledged': 3, 'adventures': 23, 'assassins': 2, 'ashes': 1, "flanagan's": 1, 'murawski': 1, 'mudslides': 1, 'intentioned': 1, "cloud's": 1, 'threats': 5, 'gladiator-like': 1, 'corrupting': 2, 'properties': 3, 'sethe': 2, 'kirby': 1, 'absence': 9, 'vile': 1, 'maternity': 1, 'tests': 8, 'fullscreen': 1, 'greek': 2, 'nicely-titled': 1, 'weds': 1, 'flemish': 2, 'warrants': 4, 'composite': 1, 'heat': 13, 'shorthand': 1, 'audience--in': 1, '15-24': 1, 'morva': 1, 'margaret': 5, 'devarim/yom': 1, 'corsucant': 1, 'disappear': 13, "leila's": 1, 'designed': 15, 'warship': 2, 'huggable': 1, 'djimon': 7, 'trustworthiness': 1, 'mantle': 1, 'leplastrier': 1, 'motherf': 1, 'violated': 1, 'earthward': 1, 'acceptance': 10, 'scaled-down': 1, 'woodman': 1, 'profane-filled': 1, 'cynicism': 8, 'tomboyish': 1, 'relunctently': 1, 'heart-strings': 1, 'ris': 2, 'winthrop': 1, "irvin's": 2, 'launch': 8, 'unassailable': 1, 'bolster': 3, 'skimp': 2, 'visceral': 7, '200': 4, 'learning': 24, 'readings': 4, 'hairstylist': 1, 'prefacing': 1, 'helmer': 7, 'fossils': 1, 'chewing': 3, 'helplessly': 1, 'secluding': 1, "bambi's": 1, 'pictorial': 1, 'cauterizes': 1, 'presume': 2, 'sewer': 2, 'yu': 2, 'tested': 5, 'followups': 1, 'kasi': 1, 'mishaps': 3, 'tighten': 2, 'adrien': 1, "lansbury's--she": 1, 'scare-thrillers': 1, 'rural': 11, 'film-music': 1, "'secret": 1, 'twitching': 1, "other's": 8, 'mini-series': 2, "screendom's": 1, 'cube': 2, 'joe': 64, 'colors': 29, 'action/': 1, 'polished': 6, "allison's": 2, 'clique': 4, 'free-thinking': 1, 'ephrons': 1, 'eyebrows': 2, 'mindel': 2, 'badges': 1, 'late-mother': 1, 'operator': 1, 'androgonys': 1, 'assortment': 7, 'aid': 18, 'ageing': 2, 'mugging': 3, 'non-fans': 6, 'newly-born': 1, 'long-overdue': 1, 'kwietniowski': 1, 'underplaying': 1, 'conflicts': 12, 'hawks/cary': 1, 'recluse': 3, "yoda's": 2, '30-something': 1, '130': 2, 'leftist': 2, 'box-office': 11, 'anticlimactic': 3, 'inaccuracies': 1, 'darkest': 7, "freud's": 1, 'tomahawk': 1, 'sweetly': 4, 'philosophers': 1, 'situations': 58, "bulow's": 1, 'shoo-in': 3, 'pieced': 1, 'invests': 4, 'nigra': 1, 'lure': 2, 'bedside': 1, 'urgent': 1, 'dragon': 18, 'blandness': 2, 'packer': 1, 'henn': 1, 'shrewd': 6, 'crucible': 1, 'embeth': 5, 'goer': 1, 'diamond': 11, 'smog': 1, 'morally': 5, 'gravely': 3, 'drag': 19, "fletcher's": 1, 'digressions': 1, 'fairer': 1, 'payer': 1, 'languages': 2, 'jugs': 1, 'rococo': 1, 'sandoval': 1, 'psychology': 2, 'kill': 86, 'queen': 47, 'crewmen': 1, 'morrow': 6, 'tub': 3, "experience--you'll": 1, 'regretfully': 1, 'teacher': 42, 'brown': 33, 'godless': 2, 'trifle': 2, 'lattitude': 1, 'brow': 3, 'non-union': 1, 'mcalisters': 1, 'existent': 1, 'horror--less': 1, 'cabaret': 2, 'light-hearted': 10, 'slaves': 10, 'majesty': 1, 'coattails': 2, 'yen': 3, 'exclamation': 3, 'butterfly': 4, 'vincenzo': 1, 'hypnotist': 1, 'taglines': 1, 'statesman-like': 1, 'leads': 108, 'oath': 1, 'shakesperean': 1, 'participants': 3, 'humane': 7, 'beg': 1, 'congenial': 2, 'wreaking': 2, 'airplanes': 1, 'lifting': 3, 'capone': 2, 'confidence': 15, 'animators': 13, 'bottoming': 1, 'miserable': 8, '1830s': 1, 'time-machine': 1, 'sensationalistic': 1, 'haircut': 2, 'competing': 4, 'rituals': 5, 'bleaches': 1, 'post': 9, 'relay': 1, 'expressed': 12, 'movie_': 1, 'sherri': 1, 'presidential': 7, "bubbles'": 1, 'bridges': 13, 'pollini': 2, 'chides': 1, 'employ': 11, 'diversity': 3, 'britisher': 1, 'drawn': 34, 'archive': 1, 'pincus': 1, 'gigantic': 10, 'tandon': 1, 'mocking': 5, 'craftsmen': 2, 'ends': 104, 'cheek--and': 1, 'tammy': 2, 'brokedown': 1, 'overly-religious': 1, 'tim': 42, 'disapproved': 1, 'gene': 15, 'youth-restoring': 1, 'artistry': 3, 'welfare': 1, 'modine': 1, 'horizontal-moving': 1, 'demonstrates': 21, 'compensates': 2, 'bride': 17, 'traci': 1, 'curses': 4, "jake's": 1, 'thugs': 11, 'actress': 70, 'haze': 1, 'icebergs': 1, 'hathaway': 1, 'nicer': 1, 'christoff': 2, "silver's": 1, 'charities': 1, 'discreet': 1, 'ugliest': 1, 'looker': 1, 'hmm': 3, 'purely': 15, 'velvet': 9, 'father-in-law': 3, 'landis': 1, 'hero-wanderer': 1, 'faster': 6, 'think--if': 1, 'senator': 12, 'ice-blue': 1, 'kinky': 2, 'passion-denying': 1, 'palmetto': 1, 'carpet-pissers': 1, 'archer': 2, 'grant-type': 1, 'i-the': 1, 'marienthal': 1, 'recollections': 3, 'bremner': 1, 'emptiness': 5, "brando's": 2, 'saucers': 2, 'remorseful': 1, "brady's": 2, 'unblinking': 2, 'misguided': 9, 'yard': 10, 'mimic': 3, 'subcontinent': 1, 'longtime': 10, 'spoiler': 8, 'invoking': 1, 'gradual': 6, 'gregarious': 1, 'stears': 1, '_la': 1, '112': 1, 'dinos': 1, 'back--turn': 1, 'humidity': 3, 'affect': 12, 'ians': 1, "humankind's": 1, 'alien': 63, 'near-revolutionary': 1, 'angeles': 19, 'salmon': 1, "'ll": 1, 'subtitle-phobes': 1, 'autopilot': 1, 'giggle': 5, '20-foot': 1, 'fall': 97, 'polemic': 1, 'streets': 37, '-moderator]': 1, 'countercultural': 1, 'crumbling': 2, 'imagination': 38, 'scraped': 2, 'heart-wrenchingly': 1, 'rival': 27, "'hidden": 1, 'docile': 1, 'pier': 1, 'concerned': 35, "'mallrats'": 1, 'renton': 3, 'vibrant': 12, 'glenross': 1, 'stationary': 1, "bugs'": 2, 'romances': 4, 'undertake': 1, 'systems--it': 1, 'yammering': 1, 'self-healing': 1, 'well-deserved': 4, 'manager--and': 1, 'discernible': 3, 'schwarz': 1, 'neck-brace': 1, 'multitude': 6, 'hapless': 11, 'haunts': 1, 'trimmed': 5, 'informing': 1, 'superheroes': 5, 'wise': 25, 'intermingling': 1, 'heart-pounding': 3, 'clint': 11, 'ousted': 1, 'schoolchildren': 1, 'sal': 2, 'well-created': 1, 'cook': 8, 'narration': 28, 'catered': 1, 'crumble': 1, 'most-eagerly': 1, "vader's": 2, 'wait': 41, 'atf': 3, 'opined': 1, 'travelers': 3, 'sands': 2, 'exiled': 5, 'stink': 1, 'yolande': 1, 'supermodel': 2, 'off-putting': 2, 'spacecraft': 7, 'shack': 1, 'contenders': 2, 'moronism': 1, 'jackhammer': 1, 'sing-song': 2, 'nanook': 1, 'wisened': 1, 'fingers': 11, 'paralyzed': 5, 'excluding': 1, 'spark': 13, 'bruin': 1, 'followers': 7, 'liverpool': 1, 'psyched': 1, 'lung': 2, 'dusting': 1, 'contradiction': 4, 'indoors': 2, 'dressed': 12, 'gore': 38, 'swintons': 1, 'leon': 2, 'describe': 29, 'blake': 9, 'terrors': 1, 'zwick': 5, "'best": 1, 'ingeniousness': 1, 'schema': 1, 'bearable': 3, 'cigarette': 3, 'closeup': 1, 'purposefully': 2, "samurai's": 1, 'mink': 1, 'kaminski': 1, 'dance': 40, 'passenger': 8, 'dosage': 3, 'mcdonald': 4, 'phobias': 1, '$40': 3, 'pizza': 4, 'clicked': 1, 'once-promising': 1, 'official': 7, 'hass': 1, 'wahoo': 1, 'renshaw': 1, 'claiming': 9, 'darkened': 3, 'daemon': 1, 'pascal': 3, "lab's": 1, 'shrewish': 1, 'lancaster': 1, 'tabio': 1, 'starters': 4, 'upstate': 3, 'horseshit': 1, 'wackiness': 1, 'verdant': 1, 'mullen': 2, "62's": 1, 'exiting': 3, 'tod': 3, 'near-certain': 1, "carlyle's": 2, 'low': 33, 'bash': 2, 'med': 1, 'glitzy': 1, 'recieved': 2, 'wired': 1, 'inclusion': 5, 'dornacker': 1, 'center-stage': 1, 'bartender': 6, 'newfound': 5, 'franco': 2, "fiona's": 2, 'bell': 10, 'camille': 7, 'oedipal': 2, 'sioux': 1, 'carded': 1, 'termination': 1, 'period-piece': 1, '_will_': 1, 'tricks': 22, 'well-polished': 1, 'comrade': 2, 'strengths': 14, 'pretext': 1, 'education': 8, 'back-stabs': 1, '_fifty_': 1, 'fetishes': 1, 'fi/comedy': 1, '150th': 1, 'stroehmann': 1, 'here-dna': 1, 'parrot': 1, 'pervert': 1, 'storyteller': 5, 'bonus': 10, 'labors--including': 1, '-so': 1, 'consolidate': 1, 'catching': 7, "annoying'": 1, "'o'": 1, 'uninspired': 6, 'cs': 1, 'amputated': 1, 'furst': 2, 'douriff': 2, 'paralysed': 3, 'irene': 5, 'earning': 5, 'conan': 1, 'glory--starring': 1, 'electroshock': 1, 'consisting': 3, 'eno': 1, 'unfunny': 13, 'masterful': 13, 'factories': 1, 'sacha': 1, '1954': 2, 'boundaries': 9, 'boardroom': 2, 'touchy': 1, 'ravera': 1, 'hassles': 1, 'accidents': 5, 'relished': 1, 'haute': 1, 'memorably': 4, 'radiantly': 3, 'oceans': 2, 'polito': 1, 'come--bringing': 1, 'merton': 1, 'chuckle-worthy': 1, 'flames': 5, 'betters': 2, 'formula-wise': 1, 'abrasive': 5, 'silhouetted': 1, 'keri': 1, 'sixteenth': 1, 'palmer-ellie': 1, 'matty': 1, 'captain': 34, 'intentions': 20, 'us$150': 1, "hannigan's": 1, 'cleaves': 1, "bickle's": 1, 'full-circle': 1, 'over-the-top': 15, 'samuel': 14, 'paradigms': 1, "'it's": 1, 'munundated': 1, 'set-bound': 1, 'thinnest': 2, "mcconaughey's": 2, "australian's": 1, 'degrades': 1, 'ashau': 1, 'windswept': 1, 'pointy-eared': 1, 'angry': 30, "tyler's": 1, 'stereo-type': 1, 'ulysses': 1, 'najimy': 1, 'stabbed': 2, 'levitch': 1, 'bloods': 1, 'france': 15, 'briefly': 23, 'invasion': 13, 'tenants': 2, 'crewmembers': 3, 'worrying': 1, 'dough': 4, 'transcend': 7, 'sepia': 1, "alexander's": 2, 'lonergan': 1, 'shootouts': 2, '$100m': 1, "1940's": 1, 'solo-flight': 1, 'lady': 32, 'accelerated': 1, 'cradles': 1, 'sheikh': 1, 'ex-friend': 1, '30-second': 3, 'mtv-style': 1, 'ballroom': 5, 'hot-as-hell': 1, 'chair': 18, "althea's": 1, 'worms': 1, "'eat": 1, "night's": 1, 'cruelly': 4, 'uniformly': 9, 'atomizers': 1, 'illustrious': 2, "squad'": 1, 'appalled': 1, 'soran': 1, 'sleepless': 6, 'vii': 1, 'reflecting': 6, "lerner's": 1, 'shines': 23, 'squeeze': 5, 'emission': 1, 'endangered': 2, 'terrestrials': 1, "lion's": 4, 'simplistic': 8, 'high-fives': 1, 'savant': 3, 'hipster': 3, "kilmer's": 2, "`roswell'": 1, 'inconceivable': 1, 'r-rated': 7, 'a-bomb': 2, 'freely': 8, 'mlakovich': 1, 'eternal': 6, 'factual': 2, '4/10': 4, 'wether': 3, 'julius': 1, 'hype': 18, 'updated': 4, 'attired': 1, "devil's": 3, 'gleiberman': 2, 'gung-ho': 3, 'men': 163, 'mins': 2, 'hiding': 16, 'aughra': 1, 'three-year': 4, 'unshaven': 3, 'bonifant': 1, 'glaringly': 2, 'neo': 6, 'hokey': 9, 'christof': 5, 'hurley': 5, '152': 1, 'soon-to-be-published': 1, 'unfolding': 1, 'onset': 1, 'capturing': 8, 'hamburgers': 1, 'anchoring': 1, 'meddling': 1, "jr's": 1, 'bloodthirsty': 3, 'elsewhere--their': 1, "francie's": 1, 'startrek': 1, 'infests': 2, 'mentioned': 36, 'quantity': 3, 'installing': 1, 'interrupts': 2, 'anti-environmentalist': 1, 'marshal': 3, 'orwell': 1, 'botanical': 1, 'nathan': 3, 'stubby': 1, 'blocked': 2, 'jude': 7, 'fruit': 2, 'visitors--friends': 1, 'graduate': 9, "gallo's": 1, 'ob-cpo': 1, 'legions': 2, 'adair': 1, 'beggars': 1, 'semi-circle': 1, 'wary': 7, 'lever': 1, 'unrequisite': 1, 'thickly': 2, 'goldblum': 3, 'illiterate': 3, 'ribald': 2, 'bookshelves': 1, 'rotterdam': 2, 'cotton': 7, 'whitewater': 1, 'donnell': 1, 'narrow-minded': 4, 'counter-attack': 1, 'drip': 1, 'homicide': 5, 'technician': 2, 'drawings': 1, 'outraged': 2, 'groundwork': 1, 'largest': 2, 'sincerely': 3, 'owners': 6, 'staunton': 1, "television's": 3, 'edifying': 1, 'deviant': 1, 'scamper': 2, 'accentuate': 3, 'roomate': 1, 'mnemonic': 1, 'oceanic': 1, 'series': 131, 'mandated': 1, 'champagne': 3, "unabomber's": 1, 'intangible': 2, 'lefty': 1, 'deference': 1, 'magnate': 1, 'look-alike': 2, 'claiborne': 1, 'surrealism': 4, 'photos': 4, 'harford': 1, 'harming': 2, 'tusse': 1, "dad's": 2, 'bust': 8, 'self-fulfillment': 2, 'moor': 2, 'glistening': 4, 'cecilia': 1, 'harder': 15, 'cures': 2, '_least_': 1, 'rms': 1, 'mortician': 4, 'buildings': 8, 'basque': 1, 'abducted': 3, 'brimming': 2, 'reprise': 1, 'boxing': 14, 'summed': 4, 'resort': 13, 'suzie': 4, 'incapable': 6, 'slice-of-life': 1, 'carter': 25, 'predatory': 3, 'convenient': 10, 'prevent': 19, "shackleton's": 1, 'unpredictable': 18, "jim's": 2, 'ridge': 1, 'profession': 18, 'hunting': 12, 'homies': 1, 'speaker': 2, 'sleeps': 5, 'hairy': 3, 'slandering': 1, 'rabbinical': 1, 'cristof': 1, 'thrive': 1, 'agonizing': 2, 'doom-and-gloom': 1, 'lingo': 1, 'setback': 2, 'conquered': 1, 'fondness': 8, 'weights': 1, 'oblique': 2, 'huge': 62, 'marriages': 1, "son'": 1, 'enraptured': 4, "plot's": 5, "sutherland's": 1, '51-year-old': 1, 'debating': 2, 'blather': 1, 'grey-skinned': 1, "directors'": 2, 'airs': 1, 'smugglers': 2, 'mantlepiece': 1, 'idosyncrasies': 1, 'sluggish': 4, 'dafoe': 4, '56': 1, 'six-string': 1, 'fa': 4, 'surprising': 55, 'pagan': 1, 'natured': 4, 'booger': 1, 'judas': 2, 'nostromo': 2, 'killings': 4, 'fegan': 1, 'picking': 13, 'kiddie-friendly': 1, 'grooms': 1, 'coupled': 7, 'screenplay': 99, 'reb': 1, 'quashing': 1, 'egpyt': 1, 'referred': 9, 'kid-in': 1, "hayes'": 1, 'whew': 3, 'master': 50, 'self-aware': 1, '109': 1, 'spaceships': 2, "d'angelo": 2, 'somesuch': 1, 'reevaluation': 1, 'artillery': 1, 'thoughtless': 1, 'unbridled': 3, 'jamusch': 1, 'singled': 1, 'desperado': 4, 'bangs': 1, 'longed': 1, 'larry': 20, 'roberto': 4, 'foursome': 1, 're-experienced': 1, 'suggests': 27, 'truffaut': 1, "darby's": 2, 'narrowly': 7, 'twain': 1, 'potente': 1, 'staunch': 1, 'jane': 29, 'grimy': 1, 'mercenaries': 1, 'bardsley': 1, 'slayer': 4, 'mika': 1, 'patrons': 2, 'juice': 1, 'booster': 1, "'dating": 1, 'reception': 5, 'brilliancy': 1, '_today_': 1, "heart'": 2, 'hi-': 1, 'mostow': 3, "pie's": 1, 'regulars': 5, 'blackmailing': 3, 'before--round': 1, 'lunches': 1, 'plan': 57, 'pomade': 1, 'skylines': 1, 'hoodlums': 1, 'herb': 1, '&#137': 1, 'biggs': 3, 'clark': 3, 'affable': 8, 'step-in': 1, 'magherini': 1, 'self-destruction': 1, 'flickers': 1, 'boards': 5, 'hairstyles': 3, 'euro-trash': 1, 'chemically-induced': 1, 'ahead': 32, "hasidic's": 1, 'lightheaded': 1, 'replaceable': 1, 'bela': 2, 'montana': 9, 'vibrantly-acted': 1, 'norton': 13, 'much_': 1, 'lovelace': 1, 'simm': 1, 'junction': 2, 'slick': 25, 'predestined': 1, 'expressionistically': 1, '94': 2, 'mugatu': 1, 'vadis': 1, 'mess': 27, 'lyon': 1, 'funnest': 1, 'fleeting': 2, 'picard': 4, 'construction': 16, 'mutants': 1, '`no': 1, 'voice-overs': 2, 'brassy': 2, "coppola's": 7, 'tours': 3, 'mentions': 7, 'they-might-be-caught': 1, '17th': 2, "how's": 1, 'comic-book-gone-feature-film': 1, 'pesky': 3, 'leaping': 2, 'juggling': 3, 'wonderland': 4, 'bala': 1, 'cartlidge': 1, 'self-destructs': 3, 'bird': 9, 'pronunciation': 1, 'a+': 2, 'winsomeness': 1, 'daphne': 6, 'voids': 1, "china's": 3, 'defend': 9, 'sunrise': 4, 'neurological': 1, 'nazareth': 1, 'balloons': 1, "simpsons'": 1, 'enlargement': 1, 'toothpaste': 1, 'drivers': 2, 'express': 16, 'unmentioned': 1, 'homemade': 1, 'vowing': 2, 'osgood': 1, 'preperations': 1, 'deciding': 3, 'snobbery': 2, 'victims': 29, 'battling': 8, 'glam-rock': 2, 'australian': 14, 'finely-written': 1, 'belt': 3, 'searles': 1, 'lorde': 1, 'geyser': 1, 'otherness': 1, 'operating': 5, 'mesmerizing': 17, "1600's": 1, 'madrid': 1, '360': 1, 'unifying': 2, 'leafy': 1, 'locals': 6, 'blazing': 2, 'under-written': 1, "coppolas'": 1, 'low-angle': 1, 'energized': 2, 'treatment': 23, 'investigation': 15, 'projectors': 2, 'fletcher': 3, 'slippery': 1, "suspiria's": 1, 'fruity': 1, 'strongly--but': 1, 'venture': 15, 'espouses': 1, 'central_': 2, 'gloriously': 6, 'incorporate-actors-into-existing-film-footage': 1, 'all-day': 1, 'croatians': 1, 'tempered': 2, 'screamer': 1, 'intimidating': 5, 'washes': 2, 'mathilde': 2, 'relling': 1, 'separated': 13, 'miguel': 3, 'lucilla': 1, 'macabre': 6, 'celebrity': 10, 'harassing': 1, 'cahiers': 1, 'blatty': 1, 'sporadically': 2, 'superbly': 13, "christine's": 1, 'purgatory': 3, 'gross': 13, 'bar': 26, 'twang': 1, "chan's": 5, 'kin': 1, 'words--': 1, 'weirdly': 4, 'eggnog': 1, 'sandwiched': 1, 'survivors': 17, 'obscenities': 4, 'ever--but': 1, 'shortened': 2, 'lou': 7, 'restraint': 14, 'fawcett-style': 1, 'quentin': 16, 'duke': 6, 'shatner': 2, 'ascaride': 1, 'town': 117, 'counterculture': 1, 'outdated': 1, 'fully-developed': 2, 'self-deprecation': 1, 'checks': 3, 'theory': 15, 'planted': 1, 'settle': 13, 'ack': 1, 'macdowell': 5, 'hartman': 1, 'co-written': 6, '#9': 1, 'spacey': 15, 'swords': 1, 'outings': 1, 're-living': 1, 'chainsaw': 5, 'consumers': 2, 'decker': 1, "ellie's": 2, '3rd': 4, 'shoulder': 9, 'adore': 4, "'snoopy'": 1, 'ever-attentive': 1, 'rating=': 4, 'recaptures': 1, 'discards': 2, 'fights--especially': 1, 'un-dynamic': 1, 'indictment': 1, 'safeties': 1, '90': 15, 'uncut': 8, "ribisi's": 1, 'cutesy': 4, 'scuzzy': 1, 'street-like': 1, 'rabbitish': 1, 'extort': 2, 'halves': 1, 'piggy': 3, 'unassuming': 7, 'free-spirited': 3, 'confection': 3, "gayle's": 1, "powers'": 3, 'patriarch': 5, 'alluringly': 1, 'na-ture': 1, 'intersect': 2, 'fiorentino': 5, 'cats': 5, 'tch': 1, 'solaris': 2, 'shane': 3, 'laziness': 2, 'capita': 1, "shostakovich's": 1, 'h}': 1, 'isabel': 1, 'highways': 1, 'creatively': 3, 'lept': 1, 'marni': 1, 'periphery': 4, '1964': 1, 'photocopying': 1, 'b-movies': 3, 'bendrix': 1, 'lawyerly': 1, 'non-sexual-yet-slightly-homoerotic': 1, 'mandrian': 1, 'indistinguishable': 6, 'mcshane': 2, 'cheezy': 3, "scene's": 2, 'mcgovern': 1, 'sunshine': 4, 'govworks': 1, 'tyrell': 1, 'unclaimed': 1, 'roshomon': 1, 'unflagging': 1, 'hyperbolic': 2, 'talking-directly-into-the-camera-schtick': 1, 'korshunov': 1, 'fraction': 1, 'good-but-not-great': 1, 'panders': 4, 'joanna': 2, 'afire': 1, 'ayers': 1, 'infirm': 2, 'bibile': 1, 'before--almost': 1, '-lite': 1, 'agnostic': 1, "kane's": 1, 'smart-mouthed': 2, 'bickford': 1, 'fillings': 1, 'gameport': 1, 'murdoch': 4, 'motive': 9, 'florida': 13, '99': 3, 'availability': 2, 'greenfingers': 1, 'intervention': 6, 'undecided': 2, 'sorossy': 1, 'enveloping': 2, 'molestation': 3, 'attachs': 1, 'judgement': 9, 'machine': 33, 'scapegoat': 1, 'handgun': 2, 'oblivious': 9, 'unerotic': 1, 'dolefully': 1, 'diatribe': 4, 'condominas': 1, 'batter': 1, 'crony': 3, 'meetings': 4, 'schubert': 1, 'institute': 3, "rudy's": 2, 'brion': 1, 'engrossingly': 1, 'go-round': 1, 'bedroom': 14, 'mastermind': 6, 'hicks': 3, 'umpteen': 1, 'achieved': 19, 'penalty': 2, 'crow': 12, 'ancient': 20, 'asthmatic': 1, 'claws': 3, "mid-50's": 1, '1912': 7, 'double-cross': 2, 'reporters': 4, "college's": 1, 'breathes': 2, 'entir': 1, 'pleasantly': 8, 'anyway--yep': 1, 'overcomes': 3, 'discharging': 1, 'idols': 1, 'prices': 3, 'agrees': 33, 'madigan': 2, '`bringing': 1, 'petri': 1, 'foregoing': 1, 'troma': 1, 'tubercular': 1, 'called': 116, 'conscription': 1, 'composer/lyricist': 1, 'yong': 1, 'disheartened': 1, "team's": 2, 'unavailable': 1, 'punish': 2, 'felons': 1, 'rattigan': 1, 'calhoun': 1, 'once-great': 1, 'forcibly': 2, "spader's": 1, 'maureen': 1, 'post-snl': 1, 'keys': 4, 'retorts': 1, 'blackwell': 1, 'cynically': 1, 'sinful': 4, 'hacked': 3, 'develop': 31, '1998s': 1, 'hoist': 1, 'successive': 1, 'cynic': 1, 'adaptable': 1, 'orange@centuryinter': 1, 'registering': 1, 'mc-ubiquitous': 1, 'similarly': 10, 'bucking': 1, 'niro': 18, 'spends': 35, 'arrietty': 1, 'cuckolded': 1, 'doll-sized': 1, 'porn-star': 1, 'good-hearted': 3, 'midst': 14, 'offend': 4, 'bailed': 1, 'inability': 11, 'issues-namely': 1, 'ceases': 1, 'feds': 6, 'illustration': 1, 'solo': 7, 'cheesiness': 1, 'vh1/vogue': 1, 'reckless': 4, 'hugeness': 1, 'jitters': 1, 'struts': 1, 'dark-': 1, 're-drawn': 1, 'looney': 2, 'bill': 71, 'approaches': 21, "beatles'": 2, 'intial': 1, 'correspondence': 2, 'incorporate': 4, 'supersede': 1, 'parisians': 1, 'adored': 2, 'predicament': 8, 'rongguang': 1, "strocker's": 1, 'snobbish': 3, 'slaver': 1, 'maguire': 11, 'beah': 1, 'perceives': 1, 'psychopath': 7, 'posess': 1, 'da-be-dah': 1, 'upending': 1, 'commitments': 2, 'messiness': 1, 'thompson': 6, 'bosses': 3, 'hermit': 2, "1960's": 9, 'spacious': 4, 'cooperation': 2, 'lancelot': 1, 'supports': 9, 'sedated': 2, 'headline': 1, 'weekly': 10, 'benton': 1, 'deteriorates': 1, 'story-writing': 1, '1888': 2, 'mastrantonio': 2, 'unquestioned': 1, 'first-time': 9, 'pescara': 1, 'aumont': 1, 'honcho': 1, 'drixenol': 1, "singles'": 1, "`there's": 2, 'exuberhant': 1, 'call': 92, 'suspicious': 19, 'mcsorley': 1, 'motivations': 13, 'masks': 1, 'drugged': 1, 'drawl': 2, '`urban': 2, 'figuring': 1, 'frakes': 2, 'postmodern': 3, "tamora's": 1, 'symbiosis': 3, 'needed': 39, 'attracts': 5, 'theorems': 1, 'palate': 1, 'dallas': 3, 'anti-violent': 1, 'laude': 1, 'presidency': 3, 'swagger': 1, 'farfetched': 1, 'brenda': 2, "ulee's": 1, "mcpherson's": 1, "mother-in-law's": 2, 'blinking': 2, 'bujold': 2, 'alfred': 11, 'present': 83, 'mcclaine': 1, 'theatres': 4, 'pocahontas': 3, 'fire-breathing': 1, 'east/west': 1, 'dungeon': 1, 'navaz': 1, 'audacity': 8, 'popularity': 13, 'pinkett': 2, 'liven': 1, 'gut-busting': 1, "'characters'": 1, 'smuggle': 2, 'stillman': 4, 'desi': 1, 'headed': 12, 'hunky': 3, 'burkettsville': 1, 'lotte': 2, 'johanssen': 1, 'biting': 8, 'meditation': 4, 'hassled': 1, 'fast-moving': 1, 'destruction': 20, 'sculpted': 1, 'koenig': 1, 'intent': 8, 'year': 216, 'assures': 3, 'transylvanians': 1, 'jackson/robert': 1, 'lookout': 2, 'interrogation': 3, 'clinical': 1, 'two-timing': 3, 'assuredly': 2, 'squeaky': 2, 'pre-juliani': 1, 'roddy': 1, 'fonda': 13, 'pall': 1, 'alma': 1, 'arena': 4, 'aspiring': 5, "colors'": 1, 'problem': 114, 'misconception': 1, 'weisz': 3, 'segue': 1, 'reminisce': 1, 'monumental': 1, 'tear-jerkers': 1, 'zeal': 2, 'fruits': 1, 'shy': 15, 'aformentioned': 1, 'open-and-shut': 1, 'swedish': 1, 'corridor': 1, 'affectation': 1, 'appetizing': 2, 'lender': 1, 'choosing': 5, 'pop-culturally': 1, 'perils': 5, 'grittier': 1, 'oppritunites': 1, 'manic-depressive': 1, 'alar': 1, 'trepidation': 3, 'valentina': 1, 'mimi': 5, 'urinals': 1, 'basking': 1, "park's": 1, "rasputin's": 1, 'betsy': 1, 'denouncing': 1, 'irreparable': 3, 'assembly': 1, 'intimidation': 1, 'drubbing': 1, 'extent': 11, 'noises': 1, 'loquacious': 2, 'feud': 1, 'jolt-inducers': 1, "book's": 4, 'canoeing': 1, "daring's": 1, 'emissaries': 1, 'disrupts': 2, 'havoc': 11, 'six-year-old': 1, '29-': 1, 'doug': 8, 'published': 3, 'helms': 1, 'little-boy': 1, 'binds': 2, 'downgrade': 1, 'repressed': 1, 'progresses': 27, 'visualized': 1, 'assaulted': 1, 'pretzels': 1, 'partridge': 1, "crunchings'": 1, 'fusing': 1, 'submerge': 1, 'nicely': 55, 'crimes': 17, 'minister': 6, 'decides': 82, 'grants': 2, 'problematic': 2, 'beaten': 11, 'distasteful': 1, 'maximillian': 2, 'residence': 4, 'painless': 1, 'wannabes': 4, 'lineman': 1, 'nodded': 1, 'crops': 2, 'and/or': 8, 'dexterous': 1, 'movie--jackie': 1, 'mescaline': 1, 'barber': 7, 'admitted': 4, 'endeavours': 1, 'baron': 2, 'haters': 1, "ba'ku": 2, 'devotedly': 1, 'present-day': 4, 'bubblegum': 1, 'interstellar': 2, 'biologist': 1, 'retrospection': 1, 'natives': 3, 'discover': 46, 'guise': 4, 'processes': 2, 'foreshadows': 2, 'withstand': 3, 'amidst': 12, "c-for-charlie's": 1, 'vows': 7, 'alanis': 3, "fall's": 1, 'dunaway': 1, 'stumbles': 14, "after'": 1, "julie's": 1, "mckellan's": 2, 'treasured': 3, 'disc': 7, 'postman': 3, 'banalities': 1, 'hypnotised': 1, 'self-advancement': 1, 'rubbing': 2, 'little-seen': 1, 'wet': 7, 'heralding': 1, 'tastes': 4, "winter's": 1, 'snowbell': 1, 'underside': 1, 'duvall': 13, 'touchingly': 3, 'cage': 14, 'forboding': 1, 'yea': 1, 'toughs': 1, 'inaugurate': 1, 'report': 11, 'linney': 6, 'vessel': 9, 'infects': 1, 'make-up': 9, 'launching': 4, '_cliffhanger_': 1, 'snubbed': 5, 'unerringly': 2, 'advisor': 9, 'reteaming': 2, 'sentencing': 1, 'shapechanging': 1, 'prejudices': 2, 'hood': 12, 'eroding': 1, 'samir': 1, 'rumba': 1, 'self-possessed': 1, 'partake': 3, 'marceau': 3, 'stoner': 1, 'tradition': 34, 'hollywood': 152, 'chess-like': 1, 'hyberboreans': 1, 'flights': 3, 'excepting': 3, 'befriending': 2, 'eat': 22, 'sneaking': 5, 'extra': 33, 'advisers': 2, 'arcadia': 1, 'jovial': 7, 'poking': 4, 'arm': 15, 'ubiquitous': 5, 'grand-scale': 1, 'farrah': 5, 'ferocity': 2, 'bluescreen': 3, '#2': 6, 'artiness': 1, 'gargoyles': 1, 'mogul': 2, 'relinquished': 1, 'descends': 6, "'dantes": 1, 'soundtrack--an': 1, 'experiments': 5, 'revolting': 3, 'resplendent': 3, 'jack': 58, '`hopelessly': 1, 'unscathed': 4, '1979': 9, 'chad': 6, 'tensions': 5, 'venner': 1, 'crosby': 1, 'seminary': 1, 'hermits': 1, 'dracula': 7, 'corridors': 4, 'sculpture': 1, 'lighter': 8, 'undefined': 2, 'fantine': 3, 'chauvenist': 1, 'horrifies': 1, '_too_': 1, 'suspecting': 1, 'motif': 4, 'unbusy': 1, 'sausages': 1, 'unfamiliar': 10, 'blockade': 4, 'compensated': 2, 'suzanne': 1, 'patriots': 1, 'mechanic': 3, 'muslim': 1, 'quandary': 2, "austen's": 2, 'city-bred': 1, 'ministry': 2, 'movie-star': 1, 'yonezo': 1, 'invaded': 7, 'confusion': 17, 'sputnik': 5, 'unkind': 1, 'watered': 1, 'pizzazz': 1, 'unconnected': 1, 'beavis': 6, 'genders': 2, "atlanta's": 1, 'distorts': 1, 'rails': 2, 'kringle': 1, 'correct': 18, "daylight'": 1, 'uma': 12, 'deputy': 5, 'witchcraft': 2, 'novikov&#137': 6, 'boorem': 2, 'relishing': 2, 'armstrong': 1, 'latex': 1, '16-year-old': 2, 'dread': 12, 'kasdan': 4, 'horror-lab': 1, 'bait': 2, 'passive': 6, 'celebrated': 5, 'highlight': 19, 'shunted': 1, 'buffoonish': 3, 'fluidly': 1, 'unbelief': 1, 'cells': 5, 'well-advised': 1, 'argues': 7, 'dumbest': 2, 'heroically': 1, "co's": 1, 'readouts': 1, 'exile': 2, 'spoke': 5, 'ensures': 3, 'sven': 1, 'break-up': 1, 'safecracker': 2, 'end--in': 1, 'cults': 1, 'shabby': 3, 'garner': 7, 'teen-dominated': 1, "childs'": 1, 'mtv-generation': 1, 'marvelling': 1, 'imitating': 1, 'cyclic': 1, 'naturalistically': 1, 'pouty': 3, 'impromptu': 5, 'wwi': 1, "river's": 1, 'canoes': 1, 'teddy': 5, 'bresslaw': 2, 'seing': 3, 'restraining': 1, 'emphasize': 3, 'creed': 3, 'alread': 1, 'low-key': 13, 'jilted': 2, '8-year-old': 1, 'civilian': 2, 'epic--handsome': 1, "puzo's": 1, 'roscoe': 1, 'scribe': 2, 'crashes': 9, 'guelph': 1, 'expectations': 34, 'bumblingly': 1, 'unbearable': 2, 'inky-blue': 1, 'girls': 38, 'thrived': 2, 'adult': 52, 'weena': 1, 'inspirational': 8, 'venomous': 1, 'claps': 1, 'all-in-all': 1, 'surpassing': 2, 'kret': 1, 'informant': 1, 'don': 27, 'solider': 2, 'cineplexes': 1, 'oppressive': 9, 'cover-ups': 1, 'auteur': 7, 'lighthearted': 16, 'reinvents': 1, "story's": 11, 'cancer': 1, 'northward': 1, 'schiffer': 1, '_dragon': 1, 'sprees': 1, 'introductory': 2, 'coalesce': 1, 'yellowish': 1, 'debut--stars': 1, 'insect': 1, 'slowed': 2, 'mora': 1, "neuwirth's": 1, 'listens': 6, 'water-tight': 1, 'motivation': 8, 'spiteful': 4, 'lasers': 1, 'greenery': 1, 'protracted': 2, 'sk': 1, 'goodnight_': 1, 'toting': 2, 'nullified': 1, 'snippet': 1, 'adorned': 1, "stephens'": 1, 'downsides': 1, 'wayside': 2, 'relationships': 39, 'face-to-face': 4, 'bierko': 1, 'tangle': 1, 'pacha': 1, 'malcovich': 1, 'tones': 12, 'legwork': 1, 'collects': 3, 'elizabeth': 27, 'returned': 21, 'you=92ll': 1, 'quincey': 1, 'weighs': 2, 'chew': 6, 'dearly': 3, 'escorting': 2, 'fever': 5, 'uninhabited': 1, 'sovereign': 1, 'sit': 49, 'lifts': 1, 'making': 224, 'lamanna': 1, 'off-balance': 2, 'maron': 1, "resurrection's": 1, 'frustrations': 3, 'faded': 3, 'crookier': 1, 'cue': 7, 'thirty-year-old': 1, 'silent': 25, 'embittered': 3, 'enclosed': 5, 'harboring': 2, 'conn': 1, 'payne': 3, 'jean-hugues': 2, 'interpret': 3, 'stalag': 1, 'curdled': 2, "queen's": 2, 'rewound': 1, 'eighteen-wheeler': 1, "mehrjui's": 1, 'tracked': 4, 'second-story': 1, 'three-star': 1, 'emerich': 1, 're-takes': 1, 'gershon': 3, '81': 2, 'upbeats': 1, 'companies/families': 1, 'law': 56, 'cheaply': 2, 'joy-provider': 1, 'koolaid': 1, 'sheep': 5, 'depiction': 16, 'jamaica': 2, 'opportunist': 2, 'position': 36, 'denis': 8, 'espoused': 1, 'massive': 14, 'playful': 9, 'railings': 1, 'stomper': 1, 'davies': 3, 'tot-friendly': 1, "itami's": 1, 'ariel': 2, 'dotted': 1, 'better-than-usual': 1, 'prefecture': 1, 'fabrice': 1, 'control-freak': 1, 'enrols': 1, 'penance': 2, 'holier-than-thou': 1, 'friendliness': 1, 'exercises': 3, 'benigni+s': 1, 'wheat--all': 1, 'stake': 7, 'hybrid': 5, 'breasts': 7, 'decrease': 1, 'one-time': 1, 'netherworld': 2, "'il": 1, "1965's": 1, 'pole': 4, 'valerie': 2, 'sub-leaders': 1, 'efficient': 7, 'inundation': 1, 'vhs': 4, 'hamster': 1, 'scolding': 1, 'battlefields': 1, 'plunges': 1, 'yglesias': 2, "schoolteacher's": 1, 'encyclopedia': 1, 'courtesan': 2, 'unctuous': 2, 'repetition': 3, 'mortgages': 1, "d'onofrio": 5, 'immortalized': 1, 'waterlogged': 2, "tourette's": 1, 'reindeer': 1, "breen's": 1, 'tokyo': 1, 'son+s': 1, 'out--an': 1, 'mononoke': 1, 'mixer': 1, 'eventual': 12, 'pearce': 4, 'refers': 5, "race's": 2, 'pint': 2, 'first-person': 1, 'baseness': 1, 'packing': 2, 'cock-eyed': 1, 'bleak': 26, 'resources': 9, 'has-been': 1, 'spent': 56, "revival's": 1, 'bimbo': 1, 'horsemen': 1, 'messily': 1, 'deploying': 2, 'ahem': 1, 'steele': 2, 'communities': 1, 'coins': 1, "korngold's": 1, 'screwing': 2, 'detested': 1, 'beekeeping': 1, 'off-target': 1, 'hobby': 3, 'maccormack': 1, 'decor': 1, 'degrees': 7, 'contact--refers': 1, 'pile-ups': 2, 'stimulate': 2, 'purchases': 4, "ullmann's": 1, 'monster': 30, 'long-suffering': 5, 'afford': 6, 'drawing': 20, 'theatre': 18, 'nuseric': 1, 'goldfinger': 2, 'min': 1, 'anxiously': 4, 'rug': 6, 'hounsou': 7, 'dimesional': 1, 'rebuked': 1, 'disrepair': 1, 'manchu': 1, 'select': 10, '143': 1, 'pathos': 4, 'cinemascope': 1, 'mamas': 1, "arm's": 4, 'loathed': 2, 'traumnovelle': 1, 'jing': 1, 'unfaithful': 5, 'overlord': 1, 'supermarkets': 1, 'norfolk': 1, 'white-owned': 1, 'relied': 3, '1914': 2, 'viewer-': 1, 'alligator': 1, 'careless': 1, 'novelists': 2, 'spoilers': 19, 'smary': 1, 'ovular': 1, 'majoring': 1, 'thermians': 1, 'subvert': 1, 'homophobia': 1, 'revision': 1, 'slaves/steerage': 1, "rueff's": 1, '`austin': 1, 'honeymoon': 6, 'cross-promotion': 1, 'rio+s': 1, 'hun': 3, 'gutierrez': 1, 'weakens': 1, 'whisk': 1, "zinnemman's": 1, '167': 1, 'stocked': 2, 'stoppard': 2, 'quick': 50, 'sparring': 2, "reagan's": 2, 'slumped': 1, 'leia': 6, 'executed': 14, 'dickinson': 1, 'immaculate': 3, 'wasteland': 3, 'road': 41, "original's": 3, 'struthers': 1, 'outbursts': 7, 'building-up': 1, 'gypsy': 1, 'legends': 6, 'seventy-five': 1, "valjean's": 1, 'ensuing': 4, 'goings': 3, 'shirt': 14, 'mildew-like': 1, 'cellars': 1, 'ladden': 1, 'tough-as-nails': 1, 'er': 10, 'skeptical': 11, 'fairfolk': 1, 'uncountable': 1, 'evan': 1, 'equations': 1, 'transpired': 3, 'franchise': 17, 'hustler': 6, 'pipelines': 1, 'cookies': 2, '\\': 4, 'brean': 1, 'nurse': 16, 'hatami': 1, 'transcendental': 3, 'political': 62, 'company': 71, 'glorious': 9, 'touted': 2, 'tyrany': 1, 'immoralistic': 1, 'match': 41, 'boyfriend': 28, 'instinctively': 1, 'ticks': 2, 'fairness': 1, 'pastels': 1, 'sweater': 1, 'succumbed': 4, 'elevator': 8, 'crook': 4, 'depression': 14, 'reunion': 6, 'ghouslish': 1, 'youths': 4, 'million': 64, 'spielbergs': 1, 'sun-': 1, 'jamey': 3, 'toughened': 1, 'fairs': 2, 'gals': 1, 'disney-ized': 2, "nora's": 1, 'giant': 45, 'inkpot': 2, 'palma': 5, 'retread': 6, 'tante': 1, 'saint': 5, 'scooped': 2, 'couldn': 1, "chaplin's": 1, 'ore': 1, 'rains': 1, 'disappointments': 2, "setting's": 1, 'earlygoings': 1, 'surfacing': 1, "'buy'": 1, 'academy': 46, 'lures': 3, 'barnes': 5, 'trinca': 1, 'rightfully': 6, 'barely': 37, 'taymor': 1, 'awakens': 3, 'dodgy': 1, "movie'": 1, 'scented': 1, 'dresser': 1, 'tear': 10, 'accost': 1, "pickin's": 1, "dora's": 1, 'generates': 7, "smuntz's": 1, 'gangbusters': 1, 'top-level': 1, 'equates': 2, 'iconic': 3, 'shuts': 1, 'synonym': 1, "toys'": 1, 'reservations': 3, 'burger': 5, "viper's": 1, 'touchy-feely': 2, 'disagrees': 2, 'condescend': 2, 'yaz': 1, 'practitioner': 2, 'simpletons': 1, 'cares': 22, 'typically': 13, 'longsword-': 1, 'deepest': 4, 'interactions': 10, 'corkscrews': 1, 'centuries': 5, 'prisoner/former': 1, 'meg': 13, 'speak': 39, 'lolipop': 1, 'brunette': 2, 'shadowing': 2, 'marcy': 3, 'poster': 9, 'somber': 7, 'uncomfortably': 3, 'bookstores': 2, 'steen': 1, 'co-producing': 1, "2000's": 1, 'chang': 4, 'line': 141, 'vh-1': 1, 'seeping': 1, 'face-off': 2, 'livelier': 4, 'lebanon': 1, 'yards': 2, 'gruelling': 1, 'deceitful': 3, 'overlook': 2, 'single-handedly': 5, 'philips': 1, 'real-life': 21, 'brazil': 6, 'payed': 1, '$25': 2, 'yorick': 1, "braddock's": 1, 'pangs': 1, 'jules': 10, 'cheeks': 3, 'enhances': 4, 'twin': 11, "needn't": 3, 'kotto': 2, 'heartbreaking': 9, 'thieves': 4, 'holloway': 1, "deliveree's": 1, 'maestro': 3, 'sean': 32, 'stitch': 1, 'sputtering': 2, 'outtakes': 8, 'hints': 13, 'frisky': 1, 'nyah': 1, 'targeted': 2, 'rattling': 2, 'believabilty': 1, 'vanessa': 6, "manager's": 1, "actress's": 3, 'night-time': 1, 'baseball': 10, 'avid': 4, 'saturated': 1, 'well-defined': 2, 'deconstruct': 1, '`like': 1, 'annoyingly': 1, 'kidding': 1, 'quinland': 1, 'mischievousness': 1, 'mueller-stahl': 2, "heaven's": 1, 'ill-equipped': 1, 'imagery/cel': 1, 'treating': 5, 'apathetic': 1, 'mechanisms': 2, 'hendrix': 3, 'manned': 1, "mother's": 21, 'stripe': 2, 'bounced': 2, 'bateman': 1, 'luminous': 4, 'empirical': 1, 'muth': 1, 'fiennes': 15, "pacino's": 3, 'dargus': 2, 'embarks': 4, 'flow': 13, 'smartly-scripted': 1, 'basset': 1, 'callous': 4, 'disapprobation': 1, 'havisham': 1, 'self-proclaimed': 3, 'uh': 3, "romy's": 1, 'ali': 9, 'tirades': 2, 'correctness': 3, 'zuehlke': 1, 'goode': 1, 'earnestness': 1, 'basic-': 1, 'west': 26, 'imploded': 1, 'x-rated': 1, 'wilted': 1, 'floor': 26, 'whorehouse': 1, 'zealot': 1, 'cried': 2, 'receipts': 4, "magazine's": 1, 'tasted': 2, 'astin': 2, "gay's": 1, 'termed': 1, "barbieri's": 1, 'beheaded': 1, 'egyptian': 4, 'folles': 1, 'vicellous': 1, 'dancefloor': 1, 'enterprises': 2, 'headstrong': 3, 'xavier': 1, 'abject': 1, 'relatibility': 1, 'fascists': 2, 'peaceniks': 1, 'alternating': 3, 'chorus': 1, 'ex-boss': 1, 'tso': 1, 'kim': 10, 'stares': 5, 'sprinkles': 1, 'ex-champion': 1, 'marked': 10, "rohmer's": 2, 'pasta': 1, 'rigorously': 1, 'maeda': 1, "fallen's": 1, "murderer-on-the-loose'": 1, 'collins': 7, 'period': 65, 'dismissive': 1, 'ilsa': 1, 'identified': 2, 'disallows': 1, 'cyberpunk': 1, 'waist': 3, 'greeted': 8, 'colonists': 1, 'ethos': 1, 'thacker': 2, 'dopple': 1, 'ballyhoo': 2, "hero's": 2, "its'": 2, 'zooms': 4, 'wise-cracking': 5, 'buries': 1, 'oblivion': 2, 'vanity': 8, 'boston': 11, 'consoling': 1, 'ex-pat': 1, "'gift'": 1, 'palminteri': 4, 'misinterprets': 1, 'gorman': 1, 'deathbed': 4, 'beaulieu': 2, 'selflessly': 2, "carolyn's": 1, 'weil': 1, 'recognizably': 1, "mccourt's": 1, 'we=92ve': 1, 'broached': 2, 'mccullah': 2, 'replicate': 1, "august's": 2, 'hard-pressed': 2, 'three-dimensionality': 1, '`patch': 1, 'fran': 2, 'dueling': 4, 'gobs': 2, 'mcferran': 1, "each's": 2, 'yellow': 7, 'cover-up': 3, 'horner': 8, 'gigi': 1, 'drummond': 1, 'showing': 62, 'delicately': 3, 'daddy': 9, 'internationally-acclaimed': 1, 'obviousness': 1, 'leaps': 13, 'mid-1800s': 1, 'desired': 10, 'buddy-action': 1, 'aniston': 3, 'extracted': 3, 'redux': 1, 'big': 263, 'wealthiest': 1, 'tumultuosness': 1, 'godfather-esque': 1, 'buster': 1, "wood'": 1, 'pelicans': 1, 'cheri': 1, 'indians': 5, 'speculative': 1, 'uninformed': 1, 'ambiguously': 2, 'goyer': 1, 'chinese': 29, 'fulfilling': 5, 'steeped': 4, 'slay': 2, 'zidler': 1, "jennifer's": 2, 'houseguest': 1, 'ambassador': 3, 'bawling': 1, 'aloof': 3, 'sprites': 1, 'greeting': 1, 'makers': 18, 'capabilities': 2, 'gantry': 1, 'bicep': 1, 'befuddled': 2, 'portion': 9, "apostle's": 1, 'terrorfying': 1, 'stringent': 1, 'indulgence': 2, 'clean': 25, 'documentary-like': 1, 'vow': 2, 'alibis': 1, 'blankness': 2, 'characted': 1, 'abounding': 1, "ophelia's": 1, 'dropping': 7, 'davison': 2, 'ilona': 1, 'chariots': 1, 'wreckage': 3, "haunting'": 1, 'reloading': 1, 'memoirs': 1, 'resentment': 5, 'scholarship': 1, "it'": 1, 'tiffani-amber': 1, 'university': 11, 'wizards': 1, 'journeys': 5, 'tristen': 1, 'palladium': 1, 'memories-': 1, 'exteriors': 3, 'realist': 1, 'phew': 1, 'heald': 1, 'williams': 49, 'effectful': 1, 'judd': 5, 'masayuki': 1, 'jams': 1, 'destroyed': 26, 'omniscient': 2, 'entertainingly': 1, 'persuading': 2, 'eckhart': 4, 'sci-fi/thriller': 1, 'rethought': 1, 'vaginal': 2, 'seraphs': 1, 'nile': 1, 'conscientious': 2, "blethyn's": 1, 'redheaded': 1, 'lively': 14, 'palaces': 1, 'dazzled': 4, "messenger'": 1, 'outrage': 5, "lesnie's": 1, 'angered': 3, 'follow': 84, 'surrealistic': 9, 'voting': 1, 'turbulence': 1, 'shredded': 2, "'96": 1, 'multihued': 1, "ozon's": 1, "bulworth's": 3, 'battiston': 1, 'fat': 19, 'obstacles': 6, 'khondji': 1, 'thirst': 4, 'faultless': 1, 'fleeing': 7, 'tilted': 1, 'gras': 1, 'basis': 22, 'simon': 17, 'assured': 17, 'rushed': 11, 'off-kilter': 8, 'diggler': 3, 'spite': 15, 'punisher': 1, 'player': 34, 'soliloquy': 2, 'acquire': 1, 'pubs': 2, 'niggles': 1, 'horde': 2, '1973': 10, 'delusions': 1, 'flatliners': 1, 'perform': 25, 'thirty-year': 1, 'every-man': 1, 'meantime': 7, 'sean/will': 1, 'landed': 7, 'brigette': 1, 'misdirection': 1, 'mainstream': 20, 'penitentiaries': 1, 'mommies': 1, 'physician': 3, 'populace': 2, 'ghostly': 4, 'breathable': 1, 'bava': 1, 'chipmunks': 1, 'kindred': 3, 'swan': 3, 'feminine': 1, 'irish-sounding': 1, 'language': 61, 'batinkoff': 1, 'understandably': 7, '`macabre': 1, 'records': 7, 'showgirl': 1, 'zophres': 1, 'dish': 4, 'needfully': 1, 'jess': 1, 'sarner': 1, 'jordon': 1, 'arab-american': 2, 'shandling': 1, 'weddings': 5, 'fatal': 10, '--crooked': 1, 'mindlessness': 1, 'dora': 4, 'falcon': 3, '1305': 1, 'egoyan': 5, 'operandi': 1, '`raging': 1, 'hookers': 1, 'hyde': 4, 'tossing': 8, "chen's": 1, '1966': 1, 'fedoore': 1, 'well-meant': 1, 'unsubtle': 3, 'delievered': 1, 'shire': 1, 'yesteryear': 2, 'tightened': 1, 'sweeten': 1, 'temp': 1, 'reappear': 1, 'chemistry': 65, 'zany': 7, "lang's": 5, 'bodyguard': 7, 'intensely': 8, 'numbing': 1, 'blocking': 2, "shandling's": 1, 'reworked': 1, 'subsumed': 1, 'radar': 1, 'carefully-stacked': 1, 'awed': 1, "malcovich'": 1, 'dims': 1, 'bobbitt': 1, 'ex-fighter-pilot': 1, "'matron's'": 1, 'events': 86, 'threw': 3, 'knotts': 1, 'attain': 1, "fred's": 1, "creek'": 1, "linklater's": 1, "thornton's": 2, 'cecil': 4, 'all-together': 1, 'benevolent': 5, 'deconstructing': 1, '`john': 1, 'exquisitely': 3, 'theodore': 3, 'oil-infested': 1, 'dissolve': 2, 'marriage': 33, 'daytime': 3, 'warn': 4, 'wash': 3, 'house': 121, 'focal': 3, 'agnostics': 1, 'rounded': 4, 'downbeat': 1, 'watchdogs': 1, 'full-scale': 1, 'actor-turned-director': 1, 'numerous': 45, 'repetitious': 1, 'dislike': 9, 'clocktower': 1, 'holodeck': 1, 'standards': 17, 'espionage': 1, 'accountability': 1, 'nature': 84, 'flatulent': 1, 'downhill': 4, 'dreamquest': 1, "commodus'": 1, 'country': 69, 'whitney': 3, 'evening': 19, 'colin': 6, 'homely': 1, 'mardi': 1, 'erudite': 1, 'doubled': 1, 'waltz': 1, 'goofier': 1, "ants'": 1, "glass's": 1, 'multi-billionare': 1, "webster's": 1, 'continual': 2, 'dines': 1, 'govenment': 1, 'tarlov': 1, 'fiction-esque': 2, "make's": 1, 'templeton': 2, 'steadfast': 2, 'noodle': 1, 'vh1': 1, 'born-again': 1, "westerner's": 1, 'lackies': 1, 'talk': 77, 'holidays': 3, 'zanin': 1, 'stott': 2, 'ear': 9, 'loner': 3, 'hammering': 1, 'nods': 5, 'taped': 3, "mpaa's": 2, 'dakinah': 1, 'conciliatory': 1, "'60s": 7, 'dallben': 1, 'sensations': 3, 'mounts': 1, '5-5-1': 1, 'protestant': 4, 'dealers': 5, 'visionary': 3, 'u2': 1, 'billionaire': 2, "theatre's": 1, 'deftly': 7, 'taut': 3, 'katy': 1, 'shit-kicking': 1, 'often-criticized': 1, 'saracastic': 1, 'preflight': 1, 'pilgrimage': 2, 'consumption': 2, 'romeo': 10, 'resurrection': 6, "lynch's": 5, 'full-time': 1, 'manipulate': 4, 'protect-earth-from-destruction': 1, 'differs': 3, 'westworld': 1, 'menagerie': 3, "yesterday's": 2, 'judith': 2, 'finklestein': 1, 'liners': 2, 'foreboding': 8, 'binoche': 1, 'von': 8, 'spawn': 8, 'juxtaposes': 1, 'specialize': 1, 'converge': 1, 'god-given': 1, "gilbert's": 1, 'fro': 2, 'evils': 3, 'sugarcoat': 1, 'titties': 1, 'oh-negg-in': 1, 'father-and-son': 1, 'slot': 1, 'worldly-wise': 1, 'druggie': 2, 'comeback': 8, 'untrue': 1, 'imbued': 1, 'burner': 1, 'alleged': 6, 'dionne': 1, 'tape': 16, 'gaines': 1, 'identifying': 4, 'extinguishing': 1, 'shard': 1, 'anglican': 1, 'wilhelm': 1, 'lucio': 1, 'debt': 16, 'entertainers': 1, 'blueprints': 2, 'allison': 5, 'mabel': 1, 'snag': 1, 'paola': 2, 'permits': 3, 'living-and-breathing': 1, 'pharmacist': 1, '-winning': 1, 'catalysts': 1, 'fiji': 1, 'spielbergian': 1, 'bouajila': 1, 'hugs': 1, '40-year-old': 1, 'poignance': 3, 'spirited': 11, 'teary': 1, 'putting': 26, 'cauldron': 5, 'frizzy': 1, 'rest': 157, 'peace': 21, 'show_': 1, 'kant': 1, 'alec': 15, 'forbidding': 1, 'expresses': 4, 'promiscuity': 1, 'lovemaker': 1, 'nora': 7, 'brotherhood': 2, 'cross-bow': 1, 'chicks': 4, 'preventing': 2, 'pseudo-maid': 1, "'companion'": 1, 'saved': 17, 'half-expected': 1, "gere's": 1, 'unequivocally': 1, 'salesman': 10, 'smidgen': 2, 'bennet': 1, 'katherine': 2, 'yachting': 1, "guy's": 9, 'good-': 1, 'wholeheartedly': 4, 'mega-bookstores': 1, 'butch': 5, 'transforming': 5, 'semi-slum': 1, 'sci-fi/kung-fu/shoot-em-up': 1, 'meticulously': 2, 'respect': 37, 'faked': 4, 'environmentalist': 1, 'memories': 26, "macy's": 1, 'impossibilty': 1, 'gyllenhaal': 3, 'beth': 3, 'encouraged': 2, 'dillane': 2, 'involuntary': 2, 'bog': 2, 'slaughtered': 2, 'determine': 14, 'complaint': 13, 'sermonize': 1, 'idiosyncratic': 5, 'murdering': 6, 'fiqure': 1, 'exorcist-rip': 1, 'body-strewn': 1, 'maze': 3, "sigel's": 1, 'edouard': 1, 'suburbia': 2, 'tenacity': 1, 'aggressive': 10, 'resonance': 11, 'damme': 6, 'low-lit': 1, 'dobbs': 1, 'scribbling': 1, 'drafting': 1, 'better-suited': 1, 'appreciation': 7, 'jean': 8, 'piddling': 1, "1988's": 1, 'eleven-year-old': 1, 'magisterial': 1, 'ambassadors': 1, 'innovative-never': 1, 'brightly': 6, 'phantoms': 1, 'annabella': 2, 'anti': 1, 'psychic': 8, 'flirtation': 1, 'timeframe': 1, 'viterelli': 2, 'mushy': 1, "hamlet's": 2, 'kennesaw': 1, "rainmaker'": 1, 'quick-flash': 1, "woods'": 2, 'violence': 104, 'correctly': 5, 'somalia': 1, 'distinction': 6, 'self-composure': 1, 'overlap': 1, 'ridden': 1, 'giddy': 5, 'jorgen': 1, 'reliable': 7, 'prophesy': 1, 'scorpions': 1, "plummer's": 1, 'meatloaf': 4, 'doo-wop': 1, 'merits': 11, 'middair': 1, 'pisken': 1, "kubrick's": 8, 'transcendent': 2, "1950's": 8, 'enormous': 17, 'gambler': 1, 'hann-byrd': 1, 'holographic': 1, 'thankless': 6, '$20': 3, '`out': 1, 'bowman': 1, 'strategically': 5, 'plausibility': 4, 'duplictious': 1, 'lookalike': 1, 'lip-synched': 2, "tassel's": 1, 'yun': 1, 'arditi': 1, 'crawl': 6, 'nod': 10, 'complain': 13, 'scriptwriter': 1, 'sensibilities': 9, 'serenely': 1, "romulus'": 1, 'missiles': 2, 'bauble': 1, 'carrie': 12, 'irma': 1, 'balcony': 3, 'one-sided': 1, '19th-century': 2, 'butchered': 3, 'gawk': 1, "ireland's": 1, 'detrimental': 1, 'downy': 1, 'vosloo': 2, 'mole': 1, 'hatchets': 1, 'fired': 11, 'inevitable': 35, 'bartellemeo': 1, 'thrill': 11, 'mountain': 17, '1981': 5, 'stabbin-a-plenty': 1, 'idelogy': 1, 'embark': 4, 'post-vietnam': 1, 'authentically': 1, 'unthinkable': 1, 'subtley': 1, 'x-men': 1, 'malnourished': 1, 'whodunnit': 1, 'gnosis': 1, 'masturbator': 1, 'impaled': 2, 'self-hypnosis': 1, 'ian': 20, 'lyn': 1, 'mccarthy': 1, 'kasa': 2, 'bladerunner/robocop': 1, 'close-ups': 12, 'head-to-head': 2, 'first-ever': 1, 'gediman': 1, 'hooted': 1, 'papers': 4, 'sports': 12, 'spells': 2, "leonard's": 4, 'avocations': 1, 'chips': 5, 'wallowing': 1, 'wrestler': 1, 'unspectacular': 3, 'supply': 7, 'network-affiliate': 1, 'ruination': 1, 'explicating': 1, 'newly': 15, 'hobbits': 1, 'stifling': 1, 'brigand': 1, 'conception': 2, 'pseudo-suspenser': 1, "matt's": 1, 'bustling': 4, 'holy': 14, '1800s': 1, 'honest-to-goodness': 1, 'brrrrr': 1, 'astor': 1, 'self-reliance': 1, 'immeadiate': 1, 'latitude': 1, 'squares': 2, 'same-named': 1, 'ass': 18, 'posh': 2, 'defiantly': 4, 'don+t': 2, 'finely-crafted': 2, 'rza': 1, 'resulting': 12, 'sentiment': 8, 'propel': 2, 'suo': 1, 'destroying': 9, 'loves': 45, 'dust': 10, 'phyllis': 1, 'squealing': 2, 'sympathise': 1, 'aroway': 1, 'connections': 7, 'likable': 35, 'camera': 111, 'deathly-ill': 1, 'alicia': 2, 'unwed': 2, 'shunned': 3, 'accomplice': 4, 'arguably': 16, 'katharine': 1, 'pulp': 42, 'cherbourg': 1, 'play': 152, 'sub-genre': 4, 'ranch': 3, 'verbal': 13, 'foundation': 9, 'beggining': 1, 'ticking': 1, "richards'": 1, 'perfectly-assembled': 1, 'writing/directing': 1, 'sequence--scored': 1, 'faction': 1, 'plimpton': 1, 'sarge': 1, 'sneezing': 1, 'leisure': 3, 'lite-rock': 1, 'bataillon': 1, 'whisked': 2, 'oscillation': 1, 'veritable': 5, 'wallop': 2, 'junkies': 2, 'shoplifting': 2, "andersen's": 1, 'superstardom': 1, 'simultaneously': 20, 'forefathers': 1, 'teams': 11, '2029': 1, 'proceeder': 1, 'parodies': 7, 'lane': 8, 'sewers': 1, 'roseanne': 1, 'stayed': 6, 'detatched': 1, "nicole's": 2, 'bernie': 2, 'gullette': 1, 'marionettes': 1, 'misadventure': 1, 'orlando': 2, 'pornography-related': 1, 'acheives': 1, 'wissner': 1, 'schroeder': 3, 'gadget': 1, 'presuming': 1, 'requirement': 2, 'viv': 1, 'fantasies': 18, 'shattered': 10, 'low-rent': 1, 'hayakawa': 2, 'smiley': 1, 'olympics': 2, 'showers': 5, 'restrain': 1, 'schumann': 1, 'sorcerer': 1, 'ears': 7, 'cary': 5, 'bathroom': 16, 'elected': 2, 'soul-collecting': 1, 'entire': 162, 'renna': 1, 'talents': 31, 'jugular': 1, 'cobra': 3, 'clever-ing': 1, 'well-traveled': 1, 'micro-organisms': 1, 'shaved': 3, 'farmboy': 1, 'include': 38, 'cds': 1, 'verve': 1, 'stuyvesant': 1, 'sacked': 1, 'rattle': 1, 'ewwww': 1, 'elle': 2, 'timon': 1, 'stadium': 4, 'relegated': 7, 'tactic': 2, "_daylight_'s": 1, 'laundry': 7, 'nagged': 1, 'irishfolk': 1, 'comatose': 4, "co-worker's": 1, 'barbara': 11, 'elwes': 2, 'spout': 1, 'maintains': 20, 'bomb': 13, 'pimps': 1, 'baranski': 2, 'discarged': 1, 'permutations': 1, 'uptight': 7, 'yeager': 1, 'destroys': 9, 'singapore': 2, 'beiderman': 1, 'star-struck': 1, 'unreasonable': 2, 'rolf': 1, 'must-see': 7, 'incapacitate': 1, 'verisimilitude': 3, 'guerrillas': 1, 'resolution': 23, 'rigmarole': 1, 'learns': 53, 'authorities': 15, 'illegal': 16, 'gruff-talking': 1, 'diminishing': 1, 'wilcock': 3, 'homophobe': 4, 'labelled': 2, 'semi-official': 1, 'near-masterpiece': 1, 'masses': 5, 'maude': 5, 'parks': 4, 'intrude': 1, 'fumes': 1, 'thing': 241, 'offset': 2, 'mena': 1, 'cars': 17, 'stardust': 2, 'emasculation': 1, 'cont': 1, 'reconstruction': 2, 'grim': 20, 'detector': 1, 'powerlessness': 1, 'self-contained': 1, 'pakistani': 1, 'narrow-mindedness': 1, 'infact': 1, 'well-endowed': 1, 'career-minded': 1, 'societal': 5, 'masked': 2, 'banker': 5, 'barton': 3, 'slightly': 74, 'schizophrenia': 1, 'on-track': 1, 'clunkiness': 1, 'shuddered': 1, '_very_': 1, "vianne's": 1, 'trippy': 1, 'tenderly': 2, 'illuminati': 1, 'kauffman': 1, 'woo-fest': 1, 'estate': 8, 'drama': 116, 'nsa': 3, 'teleplay': 1, 'submission': 1, 'age': 89, 'watcher': 1, 'oozing': 5, 'atypical': 4, 'dunst': 3, 'romp': 9, 'staged': 14, 'manip': 1, 'enamoured': 1, 'startling-but-effective': 1, 'tullymore': 2, 'vintage': 8, 'compardre': 1, 'strange': 76, 'limitless': 1, 'hospital': 28, "bennet's": 1, 'cal': 7, 'barletta': 1, 'leather': 4, "wenders'": 2, 'foul': 6, 'posey': 4, 'reciting': 3, 'entertains': 3, "l'atalante": 1, 'premiere': 10, 'quietly': 12, 'hypnotize': 2, 'gravedigger': 1, 'brute': 3, 'gold-starved': 1, 'monologues': 2, "groom's": 1, "amy's": 2, 'utilising': 1, 'stylings': 1, 'detachment': 3, 'bike': 5, 'easter': 3, 'bloodsucker--and': 1, 'grissom': 1, '70': 3, 'droll': 6, 'accountants': 1, 'oliver': 27, 'obliterate': 1, 'depends': 9, 'mitch': 2, "guzman's": 1, 'thespian': 2, 'chilly': 1, 'one-up': 1, 'triumvirate': 2, 'studying': 7, 'grungy-': 1, 'experiences': 30, 'dusty': 3, '00': 2, "crystals'": 1, 'zeljko': 1, 'confrontational': 1, 'assignment': 5, 'illusion': 2, "cynthia's": 2, 'muffin': 1, "'greatest": 1, 'judge-jury-and': 1, 'teenage-targeted': 1, '`light\x12': 1, 'raping': 3, 'die-hard': 5, 'bleibtreu': 1, 'semi-brainless': 1, 'revolver': 1, 'nailed': 2, 'marooned': 1, 'cumberland': 1, "udderley's": 1, 'rafts': 1, 'ummm': 1, 'raskolnikov-like': 1, 'sentiments': 5, 'difficult': 86, 'footnotes': 1, 'farscial': 1, 'devine': 4, 'magically': 3, "seahaven's": 1, 'deities': 1, 'slapstick-fu': 1, 'scariest--thing': 1, 'disorientation': 1, 'united': 40, 'frugal': 1, 'endure': 5, 'revolve': 5, 'amazement': 3, 'disposal': 5, 'jibe': 1, 'cultural': 17, 'poof': 1, 'saskatoon': 1, 'mostly-straightforward': 1, "'judge'": 1, "pushkin's": 1, 'webpages': 1, 'stand': 55, 'diary': 4, 'proning': 1, 'smartness': 1, 'ties': 13, 'gala': 1, 'feathers': 2, 'plank': 2, 'growl': 1, 'evita': 2, 'ingolstadt': 1, 'awaited': 3, 'descriptive': 1, 'richardson': 9, 'writer/director/comedian': 1, 'nab': 2, 'side-plots': 1, 'attributable': 1, 'children-only': 1, 'joy': 37, '-awarded': 2, 'coarseness': 1, 'posture': 1, 'presses': 1, 'stamped': 1, 'penetrate': 1, 'confetti': 1, 'atwood': 2, 'illegally': 3, 'media-saturated': 1, 'paunchy': 1, 'envision': 2, 'brie': 1, 'repressiveness': 1, 'transported': 7, "naturalist's": 1, 'growth': 9, 'tour-de-force': 3, "chocolat's": 1, 'condensed': 1, 'participation': 4, 'seaport': 1, 'flowed': 1, 'dork': 1, 'oriential': 1, "life'": 2, 'lift-off': 1, 'incestuous': 2, 'laughton': 2, 'alchemy': 1, 'even-odd': 1, 'heckerling': 1, 'oversexed': 3, 'race-motivated': 1, 'party-goers': 1, 'processed': 1, 'lapping': 1, '1935': 2, 'masterpiece': 59, 'pseudonymous': 1, 'b-films': 1, 'droopy-eyed': 1, 'investigative': 1, 'ebony': 1, 'river': 15, 'stung': 1, 'abilities': 15, 'blair': 13, 'chances': 10, 'ii': 49, 'slice': 3, 'distinquished': 1, 're-appearance': 1, 'strangest': 2, 'counterfeiter': 1, 'tequila': 2, 'steadily': 4, 'ninja': 3, 'one-note': 2, 'won': 42, 'suicides': 1, 'quash': 1, 'magic_': 1, 'dave': 14, "kret's": 1, 'aptitude': 3, 'adversaries': 1, 'multiplicity': 2, '19-year-old': 1, 'klitorsky': 1, 'dussolier': 1, 'applauding': 1, 'toupee-sporting': 1, 'soderburgh': 1, 'cheapjack': 1, 'then-sagging': 1, 'transport': 10, "o'neil": 3, 'lechery': 1, 'weinberger': 1, 'action-movie': 1, 'model': 26, 'paints': 8, 'veered': 1, 'claimants': 1, 'bullet': 12, 'werewolves': 2, 'benito': 2, 'editing': 38, 'special': 176, 'gramercy': 1, 'monkeys': 8, 'stirling': 2, 'd-fens': 1, 'heed': 2, 'thelma': 1, 'ivey': 1, 'gazes': 4, "ghost's": 2, 'doogie': 4, "'40s": 3, 'status': 32, 'procedural': 1, 'rotates': 1, 'unwillingness': 1, 'croatia': 1, "morpheus's": 1, 'artfully': 2, 'uninteresting': 10, 'deathless': 1, 'back-story': 2, 'waiter': 7, 'excitedly': 1, 'shoulders': 8, 'scrap': 2, 'gum': 1, 'crucifix': 1, 'workmen': 2, 'felon': 1, 'fundraisers': 1, 'peoples': 2, 'zapped': 1, "patlabor's": 1, 'post-movement': 1, 'mid': 6, 'forest': 17, 'unarguably': 1, 'lightning-fast': 1, 'survival-of-the-fittest': 1, 'ethereal': 5, 'detachable': 1, 'egg': 7, 'documentaries': 5, 'tamora': 1, "welsh's": 1, 'volunteers': 2, 'savviness': 1, 'millionaires': 2, 'unapologetic': 1, 'dwelling': 3, 'madam': 1, 'dumbing-down': 1, 'cheerleading': 3, 'colored': 3, 'boy--called': 1, 'strike': 27, 'azure': 1, 'extension': 1, 'fractured': 2, 'abounded': 1, 'claustrophobia': 2, 'egocentric': 1, "o-dog's": 1, 'sage': 3, "mary's": 3, 'recount': 2, 'enteratining': 1, 'guilt-ridden': 1, 'bentley': 3, 'second-in-command': 1, 'baadasssss': 1, 'high-priced': 2, 'white-hot': 1, 'co-screenwriter': 4, 'animator': 2, 'melding': 1, "'pleasantville'": 1, 'population': 11, "amazin'": 1, 'subtitle': 1, 'age-type': 1, 'shore': 6, 'wint': 2, 'neighboring': 4, 'cardinal': 3, 'bueller_': 1, 'pulling': 15, 'preachings': 1, 'taco': 2, 'dent': 2, 'periodically': 6, 'seattle': 9, 'scrambling': 1, 'wittier': 1, 'boots': 3, 'greenaway': 1, 'decreasing': 2, 'perspective': 34, 'rhodes': 2, 'anly': 1, 'undone': 1, 'product': 31, 'hilary': 3, 'drifted': 1, 'kieslowski': 3, 'terminus': 1, 'adrian': 4, 'unlimited': 1, 'sympathiser': 1, 'subdues': 1, 'stereo': 8, 'graceless': 1, 'dashboard': 1, 'communicate': 12, 'answering': 1, 'dostoevsky': 2, 'nosedives': 1, 'doubted': 6, 'us-they': 1, 'disemboweling': 1, 'strumming': 1, 'unconvincing': 5, 'harbinger': 2, 'lemme': 1, 'menage-a-trois': 2, 'kinkier': 1, 'injustices': 1, 'aulon': 1, 'millie': 1, 'powerfully': 6, 'inhabiting': 2, "patti's": 1, 'idly': 1, 'praying': 2, 'duds': 5, 'flawed': 29, 'wilfrid': 1, 'flies': 11, 'fealty': 1, "voight's": 3, 'environment': 19, 'freedom-fighters': 1, 'concoct': 3, 'schweig': 1, 'padding': 1, 'entrails': 1, '_into_': 1, "dawson's": 5, 'mistaking': 3, 'lingered': 1, 'humphry': 1, 'outinen': 1, 'oavs': 1, 'vacant': 5, 'ant-like': 1, 'schedule': 4, 'halen': 2, 'screenwriter': 46, "hamilton's": 1, 'killing': 43, 'verdict': 5, 'announce': 4, 'barbaric': 1, 'sweet': 63, 'langlet': 1, 'destructions': 1, 'wet-behind-the-ears': 1, 'norris': 2, 'shell_': 1, "month's": 2, 'discharge': 1, 'buffalo-hunting': 1, 'zoolander': 2, 'rational': 10, 'indicating': 2, 'debonair': 1, 'casper': 8, 'architectural': 1, 'privacy': 3, 'opinions': 7, 'overabundance': 1, "cannon'": 1, 'disqualified': 1, "'normal'": 3, 'counter': 7, 'daddies': 1, "joker's": 1, 'marg': 2, 'ironically-showmanship': 1, '1/25th': 2, 'floor_': 1, 'brining': 1, 'hated': 13, 'shoals': 1, 'humongous': 1, 'serial': 26, 'morita': 4, 'knight': 18, "astronauts'": 1, "hopkins'": 2, 'particulars': 5, 'alida': 2, 'buds': 1, 'mumbo': 1, 'unexperienced': 1, 'pornogrpahy': 1, 'charity': 2, 'clarity': 7, 'crisp': 18, 'legged': 1, 'babyface': 1, 'hallstrom': 2, 'jungle': 10, 'virgin': 10, 'made--and': 1, 'dunne': 2, 'rollerblading': 1, 'soprana-like': 1, 'budget': 45, '`hello': 2, 'payroll': 2, 'valek': 2, 'directors': 47, 'flashlight': 1, "ladies'": 1, 'winslet': 10, 'inferiority': 1, 'reclusion': 1, 'clarence': 2, 'intrusive': 5, 'schools': 5, 'miscast': 2, '[his]': 1, 'rebecca': 7, 'porno': 5, 'refining': 1, 'nearly-perfect': 1, 'paled': 2, 'machinery': 4, 'brosnon': 1, 'eye-catcher': 1, 'motorized': 1, 'legit': 1, 'bicentennial': 1, 'indianapolis': 2, 'real': 269, 'zookeeper': 1, 'vorzon': 1, 'upkeep': 1, 'informer': 1, 'acknowledge': 4, 'unwittingly': 10, 'generating': 4, 'lotta': 1, 'gloppy': 1, 'symbolism': 7, 'grandeur': 4, 'mutton-chop': 1, 'beleivable': 1, 'slow-moving': 3, 'fatter': 1, 'voodoo-like': 1, 'releases': 13, 'brimstone': 1, 'spiner': 3, 'politically': 8, 'harmonica': 1, 'bouncer': 1, 'pinup': 1, 'bristling': 1, 'groomed': 1, 'gypsies': 1, 'knockoff': 2, 'gellar': 4, "'master'": 1, 'mournfulness': 1, 'unnatural': 1, 'urgayle': 1, 'jawed': 1, 'guises': 1, 'marina': 4, 'newly-freed': 1, "'straight'": 1, 'otis': 1, 'sociology': 1, 'mercenary': 4, 'someway': 1, 'accomodates': 1, 'big-time': 5, 'dilbert': 1, 'thorny': 2, 'aspires': 2, "hill's": 3, 'rises': 19, 'paris': 18, 'crumpled': 1, 'joadson': 2, "humanity's": 2, 'nerve-wracking': 1, 'zap': 1, 'soap-opera': 1, 'sitcom': 11, 'walsh': 9, 'befitting': 1, 'whisperer': 4, 'dimensionality': 1, '_basquiat_': 1, '1945': 2, 'overlaps': 2, 'mawkishness': 2, 'unaffecting': 1, 'usda': 1, 'operation': 16, 'equine': 1, 'catapults': 1, 'figaro': 1, 'overblow': 1, 'half-million': 1, 'immin': 1, 'absurdly': 2, 'influential': 6, 'musters': 1, 'eastern': 5, 'otto': 2, 'human': 155, 'cartoonish': 13, 'blechhhs': 1, 'silences': 1, 'method': 15, 'voiced': 17, 'similarity': 2, 'radiation': 2, 'targets': 6, 're-assigned': 1, "springer's": 1, 'artful': 2, "python's": 2, 'mummy': 2, 'yugoslavia': 3, 'removed': 15, 'greater': 36, 'comment': 8, 'spiderman': 1, 'clarke': 2, 'lucy': 6, 'helplessness': 1, 'taye': 3, 'riddled': 5, '--as': 1, 'farcical': 2, 'voice': 82, 'shoring': 1, 'jungles': 2, 'visibly': 1, 'fantasizes': 3, 'intolerant': 1, 'interview': 20, "'out-of-character'": 1, 'fi': 1, "writers'": 2, 'glass': 21, 'outweighs': 1, 'rudimentary': 3, 'dissipated': 1, 'pomp': 2, 'innappropriate': 1, 'genial': 1, 'inflicting': 1, 'apocalyptic': 2, 'stagnate': 1, 'chandler': 1, '`white': 1, "entertainment's": 1, 'photogenic': 1, 'twister': 7, 'scotland': 5, 'quickie': 6, 'keeslar': 1, 'undies': 2, 'acid-eaten': 1, 'acclaim': 8, 'operatives': 1, 'schoolboy': 2, 'astrology': 1, 'eastern-sounding': 1, 'kissed': 4, 'jarvis': 1, 'trains': 5, 'prompted': 2, 'inclement': 1, 'schnitzer': 1, 'polls': 3, 'othello': 1, 'dignity': 18, 'engrossed': 2, 'hanks/ryan': 1, 'wichita': 1, 'space-ships': 1, 'over-the-hill': 1, 'biographies': 1, 'fitting': 14, 'executioners': 2, 'toiled': 1, 'stun': 2, 'stitching': 1, 'no-holds-barred': 2, 'purported': 1, "alda's": 1, "spouse's": 1, 'flopped': 2, 'plates': 2, 'norwood': 1, 'details': 65, 'tipping': 3, 'loopiness': 1, 'twenty-six': 1, 'bubbles': 2, 'carbon': 1, 'contemplative': 3, 'sidney': 19, '_practical': 1, 'whacking': 1, 'well-intentioned': 1, 'playacting': 1, 'philadelphia': 3, 'looted': 1, 'eileen': 3, 'tingwell': 1, 'trekkers': 1, 'disobeys': 1, 'calahan': 1, 'burbank': 8, 'greenbury': 1, 'manic': 10, 'quynh': 1, 'smeared': 3, 'fanciful': 3, 'inept': 7, 'minus': 2, 'cashier': 2, 'mckidd': 1, 'shame': 43, 'jekyll/hyde': 1, 'to-hell-with-the-law': 1, 'who--gasp--is': 1, 'ethnocentric': 1, 'ping': 3, 'semi-lighting': 1, 'jokes': 57, 'predicts': 1, 'pre-intellectualizing': 1, 'awaits': 3, 'stray': 6, 'highlighted': 2, 'collated': 1, 'paprika': 1, 'heralds': 1, "'rekall'": 1, 'feuding': 3, 'grabs': 10, 'aix': 1, 'extravaganzas': 2, 'refinery': 1, 'ann': 8, 'unexpectedly': 14, 'visualised': 1, 'boasting': 4, 'speculation': 2, 'sociological': 2, 'excesses': 4, 'ear-to-ear': 1, 'despised': 1, 'unholy': 2, 'pitfall': 1, 'explanations': 4, 'connect-the-dots': 1, 'humanizes': 1, 'rennie': 2, 'prevalent': 7, '[there': 1, 'smirkingly': 1, 'battle': 73, "captain's": 2, 'morales': 1, 'prefer': 11, 'nutcases': 1, 'tumbleweed-like': 1, 'thinking': 57, 'proficient': 2, 'resisting': 2, "mchugh's": 1, 'kid-friendly': 1, 'crude': 11, 'filmmakers': 52, 'lunching': 1, "copolla's": 1, 'vestments': 1, 'length': 26, 'genvieve': 1, 'criminally': 5, "bueller's": 3, 'inconspicuously': 1, 'cantankerous': 1, "helfgott's": 1, 'hotpants': 1, 'humorist': 1, 'specifically': 21, 'enact': 1, 'blindness': 1, 'subtitling': 1, 'ex-model': 1, 'visited': 4, 'bailing': 1, 'credibility-in': 1, 'jam': 5, "there've": 1, "barr's": 1, 'ovens': 2, 'advantage': 18, 'non-colored': 1, 'winningly': 1, 'ye': 1, 'disserve': 1, 'collapsing': 2, 'decisions': 18, 'disappearing': 3, '700': 2, 'uncompleted': 1, 'africa': 12, "delilah's": 1, 'cork': 1, 'absense': 1, 'sixty': 4, 'anarchic': 2, 'prosperous': 3, 'preface': 2, 'eliott': 1, 'nasty-tempered': 1, 'own--fall': 1, 'meyers': 5, 'why---unless': 1, 'young-david': 1, 'inanimate': 1, 'temping': 1, 'glorifying': 2, 'occurrences': 3, 'authoritatively': 2, 'rosselini': 1, 'foundations': 1, '1984': 10, 'evolve': 2, 'hot-shot': 3, 'duckling': 1, 'quirks': 8, "clinton's": 4, 'lifted--unauthorized--from': 1, 'surveilance': 1, 'tailor-made': 1, 'calls': 35, 'separate': 22, 'miracles': 2, 'cave': 7, "nello's": 1, 'nut-cake': 1, 'eligibility': 1, 'a&m': 1, 'vulture-like': 1, 'intervene': 3, 'rag': 2, 'asssss': 1, 'quire': 1, 'extinguisher': 1, 'by-the-end': 1, 'never-seen': 1, 'meanie': 1, 'premonition': 3, 'zaentz': 1, 'reinforcing': 2, 'detonation': 2, 'brass': 1, 'belushi': 2, 'present-time': 1, 'everyweek': 1, "eastwood's": 1, 'wretched': 7, 'comic-panel': 1, 'bijou': 1, 'portly': 3, 'sidesplitting': 1, 'crazy': 31, 'gaultier': 1, 'immigrant': 5, 'expense': 8, "ned's": 1, 'elmo': 1, 'falk': 2, 'disguised': 6, 'enactment': 1, "junger's": 1, 'image--that': 1, 'herring': 2, 'weddding': 1, 'whimpering': 1, 'thewlis': 1, 'bearing': 4, 'bravado': 3, 'midpoint': 2, 'forays': 1, 'consultant': 1, 'late-year': 1, 'impenetrable': 1, "cuaron's": 1, 'insincerely': 1, "'patient'": 1, 'reverend': 7, 'klieg': 1, 'one-persons': 1, 'hike': 1, 'midsummer': 1, 'abiility': 1, 'marius': 2, 'possesses': 14, '$6': 3, 'nude': 6, 'henry': 25, 'vichy': 1, 'emotionally-crippled': 1, 'skinning': 1, 'sessions': 9, 'iroquois': 1, 'myth': 8, 'sharply': 5, 'recollect': 1, 'zones': 1, 'sermon': 1, 'teenage-david': 1, 'heft': 3, 'slam-dunked': 1, 'pontificating': 1, "baker's": 1, 'sicily': 1, 'well-choreographed': 1, 'moran': 1, 'big-to-do': 1, 'scheduling': 1, 'barbatus': 1, 'lauri': 1, 'sawa': 4, 'mcgregor': 14, 'bezucha': 1, 'tonino': 1, 'gallo': 1, 'keeper': 1, 'ex-girlfriend': 6, 'supergenius': 1, 'short-term': 2, 'inconsequential': 6, 'stripper': 6, "mikey's": 1, 'rea': 5, 'winnfield': 2, 'heady': 4, 'ryan-starrer': 1, 'expertise': 2, 'sagan': 4, 'mcinnerny': 1, 'vampyre': 1, 'prejudiced': 1, 'emotions': 54, 'tornado': 1, "producers'": 1, 'herrings': 3, 'steed': 1, 'honours': 2, 'sixty-ish': 1, 'ryan': 43, 'recalling': 2, 'partakes': 2, 'hadnut': 1, 'prickly': 1, 'stanleyville': 1, 'technologies': 1, 'indignantly': 1, 'kiddie-oriented': 1, 'posession': 2, 'danner': 1, 'sort-of': 2, 'gentleness': 1, '35': 9, 'frights': 1, 'gifts': 5, 'criteria': 3, 'introduced': 48, 'intrigue': 10, 'creep': 6, 'rosemary': 3, 'tracker': 1, 'tamiyo': 2, 'poets': 1, 'threat': 32, 'multi-dimensions': 1, 'erase': 4, 'plucky': 3, 'gaudy': 3, 'comic': 125, 'creatures': 24, 'english-': 1, 'sharp-witted': 1, 'boxoffice': 1, "alain's": 1, 'toshiyuki': 1, 'brake': 1, 'colorless': 2, 'possessions': 4, 'raps': 1, 'sans': 1, 'relating': 2, 'cybill': 1, 'cerebellum': 1, 'thankful': 4, 'newsgroup': 2, 'ether-addicted': 1, 'coma': 3, 'all-encompassing': 1, 'pressed': 4, 'aided': 14, 'einstein': 6, 'bother': 9, 'nzonzi': 2, 'offending': 2, 'caught-': 1, 'altercation': 1, 'atmospheric': 6, 'beaters': 1, "impact's": 1, 'proclaim': 5, 'check': 25, 'dement': 1, 'overplay': 1, 'reserves': 1, 'blasts': 3, 'indecent': 3, 'vignettes-styled': 1, 'metropolis': 9, 'scatter-brained': 1, "cusack's": 2, 'italoamerican': 1, 'mouth-watering': 1, 'fincher': 7, 'motiveless': 1, 'eaten': 3, "eilonwy's": 1, 'specifics': 3, 'intricacies': 3, 'storyboard': 1, 'sarajevo': 3, 'departments': 4, 'recommendable': 3, 'poem': 7, 'krzysztof': 2, 'preferences': 2, 'additionally': 6, 'howie': 1, 'phonograph': 2, 'financing': 1, 'gigolo': 3, 'outclass': 1, 'commenting': 2, 'textures': 5, 'reported': 4, 'wide': 25, 'aka': 5, 'valley': 4, 'procuring': 1, 'infidelities': 1, 'fonder': 1, 'vinie': 1, 'gaskell': 1, 'stirrup': 1, 'suggestively': 2, 'intelligently-constructed': 1, 'refugee': 3, 'station': 32, 'restraints': 1, 'builds': 20, 'infectious': 10, 'sugar-coated': 1, 'one-two': 2, 'introductions': 2, 'appropriately': 26, 'sessue': 2, 'shimmering': 3, 'edina': 1, 'spanned': 2, 'science-a': 1, 'vlad': 1, 'ancestors': 2, '$10': 9, 'risking': 3, 'taboo': 3, 'certificate': 3, 'mirth': 1, 'email': 4, 'brothers': 51, 'disreputable': 1, 'barry': 22, 'flamenco': 1, 'luketic': 1, 're-created': 2, 'disclose': 1, 'respectively-this': 1, 'skilful': 1, 'catch-22': 2, 'luppi': 1, "era's": 1, 'dalmatians': 1, 'pickpocket': 1, 'productions': 16, 'arouse': 2, 'multiracial': 1, '1970': 4, "hartley's": 1, 'mcconaughey': 15, "'lives'": 1, 'delirium': 1, 'cornucopia': 1, 'copeland': 1, 'waylaid': 1, 'collapsed': 1, 'challenging': 19, 'chapters': 4, 'bewildered': 4, 'his/her': 3, 'jorden': 1, 'infusion': 1, 'tokens': 2, 'attempted': 14, 'marijuana': 3, 'stagy': 2, 'frankiln': 1, 'mppa': 1, 'caf=8a': 1, 'ricci': 7, 'imperial': 9, 'incorrigible': 1, 'vignette': 3, "dreamworks's": 1, 'parlor': 4, 'fades': 5, 'itself-a': 1, 'conquers': 2, 'frighteningly': 3, "sun's": 1, 'yasmeen': 1, 'machine-like': 1, 'business-minded': 1, 'loathsome': 4, 'underdog': 2, 'courier': 6, 'reprimand': 1, 'prey': 4, 'pachanga': 1, 'long-haired': 2, 'aim': 7, 'cutting': 18, "dinosaurs'": 1, 'observe-as': 1, 'jewel': 3, 'induction': 1, 'foreshadowing': 6, 'profundis': 1, 'easygoing': 3, 'environments': 2, 'repel': 1, "harker's": 1, 'study': 41, 'pathological': 1, 'macy': 9, 'reclamation': 1, "`candyman'": 1, 'squabble': 1, 'snarling': 1, 'pleasureless': 1, 'shaft': 6, 'roles': 119, 'michele': 3, 'duffel': 1, 'ballerina': 2, 'fibber': 1, 'streams': 1, 'time-traveling': 2, 'cornfield': 1, "marius's": 1, 'illicit': 4, 'nikos': 2, 'faithful': 15, 'infectuous': 1, 'conditions--sonorous': 1, 'latent': 1, 'unrated': 1, 'dingo': 1, 'zigzagged': 1, 'davidovich': 1, 'wing': 6, 'disturbs': 5, 'verhoeven': 8, 'ditties': 2, 'schwartzman': 1, 'fully': 55, 'terminals': 1, 'antarctica': 2, 'rehabilitating': 1, 'kurtz': 2, 'religios': 1, 'seemingly-strange': 1, 'admirals': 1, "`knowledge'": 1, 'childlike': 3, 'standoff': 2, 'hockey-masked': 1, 'expanding': 3, 'kubrick': 12, 'martineau': 1, 'cheers': 4, 'kemler': 1, 'grape': 4, 'disilusioned': 1, 'spectacular': 44, 'jenette': 1, '05': 1, 'scurrying': 1, 'dark-side': 1, "cruise's": 2, 'cams': 1, 'foods': 2, 'metaphorical': 2, 'busybody': 1, 'tiff': 2, 'sort': 96, 'secondarily': 1, 'sara': 5, '`entertainment': 1, "marie's": 2, 'zip': 2, 'decade--': 1, 'czech': 3, 'hunk': 2, 'dissatisfying': 1, 'munch-esque': 1, 'electricity': 2, 'keller': 1, 'battled': 1, 'thirty-five': 1, 'pigeonholing': 2, 'lusty': 4, 'irreverence': 2, 'avant': 1, 'soon-tek': 4, 'osment': 9, 'sciorra': 2, '39': 1, 'dropped': 17, 'cg': 2, 'untamed': 1, 'formula': 33, 'arbitrarily': 1, 'cultures': 4, 'juxtaposition': 1, 'astonishment': 2, 'artificially': 2, 'grouchiness': 1, 'patterns': 9, "ripper's": 1, 'website': 1, "griffith's": 1, 'nicoletta': 3, "nature's": 1, 'punctiation': 1, 'exploitative': 3, 'toronto': 10, 'castro': 4, 'eye': 60, "something's": 1, 'sacrilegious': 3, 'c-word': 1, 'mood': 51, 'anti-semite': 2, 'dynamic': 13, 'scanning': 1, 'fraud': 4, 'evil': 120, 'asserting': 1, 'incorpates': 1, 'resonate': 3, 'guilfoyle': 1, 'humbler': 1, 'journal': 2, 'm&m': 1, 'clown': 10, 'compliments': 3, 'instant': 12, 'heroines--ariel': 1, 'reasons': 64, 'hungus': 1, 'soft-spoken': 4, 'giger': 1, 'radiating': 1, 'occurred': 11, "ordell's": 5, "japan's": 1, 'eighteenth': 1, 'nam': 1, 'porn': 9, 'nightspot': 1, 'hesitance': 3, "castor's": 1, '_lone': 1, 'finchers': 1, 'sentimentally': 1, 'snorri': 1, 'scum-sucking': 2, 'eisely': 1, 'gould': 3, 'subscribers': 1, 'ham-handed': 1, '`final': 1, 'assassinate': 5, 'divides': 1, 'elephant': 8, '216-digit': 1, 'sherman': 1, "riddick's": 1, 'jonas^os': 1, 'parties': 10, 'insidious': 1, 'influenced': 14, 'rhythms': 5, 'tension--as': 1, 'pei': 2, 'spills': 4, 'intriguing': 42, 'neccessary': 1, 'happiness': 17, 'parodyish': 1, 'packaging': 4, 'goldmine': 2, 'analyzing': 2, 'person': 130, 'city': 116, 'misses': 11, 'morsel': 3, 'the-top': 1, 'optimum': 1, 'toole': 1, 'continued': 9, 'two-room': 2, 'ap2': 2, 'scenes--the': 1, 'oozed': 1, 'mcguire': 4, 'splendidly': 2, 'authored': 1, '`ed': 1, 'myths': 1, 'aids': 9, 'utterances': 1, 'actors': 232, 'span': 10, 'delroy': 5, 'director-writer': 1, 'ilah': 1, 'shelf': 3, 'mean-spirited': 5, 'stiffest': 1, 'ilk': 1, 'identities': 2, 'scientific': 5, 'scooby': 1, 'religion': 24, 'rocher': 1, 'tragi-comic': 1, 'pre-aids-scare': 1, 'jittering': 1, 'prolific': 2, 'teri': 4, 'shohan': 1, 'meaty': 5, 'rooftop': 1, 'persistence': 4, "feelin'": 1, 'besson': 2, "election's": 1, 'paltrow': 8, 'characteristically': 1, 'mine': 14, 'lower-middle': 1, 'desperation': 13, "cinema's": 2, 'majestic': 2, 'paglia': 1, '`aliens': 1, 'agenda': 10, 'oversees': 1, 'slave': 17, "`design'": 1, 'dustin': 7, 'purple': 6, 'simpler': 3, 'cuong': 1, 'halls': 5, 'sauce': 2, 'rhett': 1, 'nest': 3, 'falling': 34, 'chicago': 23, 'posed': 3, 'attempt': 70, "shannon's": 1, 'fine': 113, 'audacious--and': 1, 'colleagues': 10, 'presides': 1, 'respected': 12, 'rats': 6, 'tropps': 1, 'manuel': 1, 'kenobi': 12, "dvd's": 1, 'moroccan': 2, 'potatoes': 1, 'dialog': 12, 'sellers': 2, 'asshole': 2, 'gave': 62, 'junk': 8, 'arresting': 4, 'fleming': 3, 'closer': 22, 'teasing': 1, 'nerdiest': 1, 'set': 224, 'allegedly': 4, 'inject': 3, 'heighten': 2, 'legitimate': 7, '600': 2, 'engulfs': 1, 'mermaid': 6, 'super-secret': 1, 'ingmar': 3, 'disney': 38, 'pre-marital': 2, 'burroughs': 3, '--------------------------------------------------------------': 1, 'bedazzled': 1, 'consumerism': 1, 'jettison': 3, 'thw': 1, "trek's": 1, "today's": 24, 'directly': 24, 'clamor': 1, 'bronx': 7, 'overseen': 1, 'hues': 4, 'impossibility': 1, 'nonstop': 1, 'adultery': 5, 'script': 182, 'propriety': 2, 'scratchy': 1, 'discussing': 13, "andy's": 3, 'dimension': 13, 'holes': 20, 'goodies': 1, 'gaining': 11, 'higher': 27, 'cryptic': 3, 'no-show': 1, 'costs': 10, 'platonic': 1, 'liberated': 6, 'forgettable': 11, 'inadequacies': 2, 'representations': 1, 'offshoots': 1, 'chocolat': 2, 'public-domain': 1, 'writer/director': 28, 'let-downs': 1, 'woodard': 2, '25-and-under': 1, 'sierra': 2, 'rubber': 3, 'daugher': 1, 'twenty-year-old': 1, '1st': 3, 'facets': 2, 'duped': 2, 'lawn': 2, 'guitar-playing': 1, 'short-film': 1, 'divulge': 4, 'hungarian': 2, 'junkyard': 1, 'reddick': 4, 'greased': 2, 'screwer': 1, 'remarking': 2, 'cancellation': 1, 'roundtree': 1, 'neutral': 4, 'lightly': 9, 'lamberto': 1, 'dyer': 2, 'clarice': 1, 'barren': 6, 'visiting': 13, 'snazzy': 1, "lumumba's": 2, 'amity': 1, 'nuke': 1, 'rasping': 1, 'clipped': 2, 'matilda': 2, 'gen-xers': 1, '`oh': 1, 'tagged': 4, 'expressionless': 3, "`batman'": 1, 'leder': 2, 'endeavor': 4, "wood's": 1, 'publique': 1, 'incoherent': 1, '`snake': 1, '_matrix_': 1, 'sociopath': 1, 'speed': 23, 'increase': 8, 'villiany': 1, 'snobbishness': 1, "evans'": 2, 'flaunted': 1, 'doll': 6, 'benefactors': 1, 'splashed': 1, 'jocks': 3, 'theatrics': 7, 'inherited': 3, 'upbringing': 2, 'referenced': 3, 'hypnosis': 2, 'speaking': 39, 'clan': 7, 'riley': 6, 'cogent': 1, 'labor': 14, 'stating': 5, 'neat': 11, 'down-right': 2, 'tonya': 1, 'proxy': 4, 'freedom': 28, 'admit': 52, 'collide': 6, 'oliveira': 1, "goblin's": 1, 'resurrect': 3, 'lounge': 11, 'ultraconfident': 1, "martin's": 5, 'sooty': 1, 'vis': 1, 'prosper': 2, 'incognito': 2, '40-student': 1, 'awkward': 23, 'encourage': 5, 'beams': 2, 'sela': 1, 'grandmothers': 1, 'toxic': 3, 'bassett': 4, 'ode': 4, 'jeter': 3, 'marks': 14, 'paying': 18, 'abandons': 5, 'observer': 5, 'advises': 2, 'breaking': 29, 'amusement': 8, 'segregation': 1, "booker's": 1, 'thoroughbreds': 1, 'tracheotomy': 1, 'sommerset': 1, 'lensing': 2, 'monastery': 1, 'hand-feed': 1, 'imr': 1, 'barf': 2, 'raunch': 3, 'amends': 1, 'wife-husband': 1, 'traitorous': 1, 'vault': 4, 'modernist': 1, 'lag': 4, 'upside': 6, 'gotcha': 5, 'trench-coats': 1, 'fozzie': 1, 'hanger-on': 2, 'edits': 4, 'fizzle': 1, 'quintana': 2, 'portuguese': 1, 'pup': 2, 'running/jumping': 1, 'larded': 1, 'shawn': 5, 'suplee': 1, 'month': 14, 'thubten': 1, 'disciples': 2, 'soooo': 1, 'choses': 4, 'deserves': 72, 'woo': 11, 'tantrums': 4, 'gloomily': 1, "star'": 1, 'vogue': 2, 'grandma': 5, 'burying': 2, 'boy-and-dog': 1, 'repulsing': 1, '7/10': 16, 'satine': 1, 'warren': 7, 'analyses': 2, 'cosette-marius': 1, 'equipment': 10, 'depressed': 14, 'stravinsky': 1, 'niccol': 8, "taguchi's": 1, "thompson's": 1, 'emigrants': 1, "ellis'": 1, 'lewton': 3, 'punished': 2, 'privileged': 4, 'experimentation': 1, 'renewed': 2, 'pondered': 2, 'inferences': 1, "leigh's": 2, 'dynamites': 1, 'productive': 2, 'flame': 6, 'germans': 4, 'ogre': 5, 'find--she': 1, "crystal's": 3, '---': 2, 'mutates': 1, 'tingles': 1, 'washer/dryer': 1, "winner's": 1, 'braugher': 3, 'temper': 11, 'skin-tight': 2, 'outlook': 7, 'selecting': 2, 'innocent-looking': 1, '2065': 1, 'self-imposed': 3, 'eleven': 4, 'absinthe': 1, 'sharpest': 2, 'dialect': 2, 'retires': 1, 'casablanca': 5, 'inclosed': 1, "beethoven's": 2, 'bright-coloured': 1, 'redone': 1, 'that\x12s': 2, '`pig': 1, 'olsen': 2, 'work--sometimes': 1, 'swipe': 4, 'duane': 5, 'educated': 3, 'boxers': 2, 'shelby': 1, 'beaudene': 1, 'knockdown': 1, 'piano': 12, "rhames'": 1, 'obligations': 2, 'feasting': 1, 'charnel': 2, 'innocuous': 4, 'lemon': 1, 'photographer': 7, 'lee': 67, 'wallow': 1, 'universal': 25, 'basketballs': 2, 'leelee': 2, 'unabashedly': 1, 'technique': 16, "driver's": 1, 'venice': 8, 'hazing': 1, 'misplaced': 6, 'serious-minded': 1, 'crackles': 1, 'tattoo': 2, 'scope': 8, 'furnishings': 3, 'ribisi': 5, 'laughed': 17, 'thickens': 3, 'secretly': 26, '_highly_': 1, 'overblown': 6, "boyz'n'hood": 1, 'brandi': 1, 'michell': 1, 'second-string': 1, 'headache': 2, 'archetype': 1, "blake's": 1, 'blush': 4, 'comeuppance': 3, 'ordeals': 1, 'shady': 7, 'davis/harrison': 1, 'foreign-language': 1, 'motorcylce': 1, 'cruel': 19, 'luhrman': 1, 'chagrinned': 1, 'outskirts': 1, 'marin': 3, 'lines': 99, 'morsels': 1, 'retreated': 1, 'hawthorne': 9, 'sgt': 6, 'visionaries': 1, 'stack': 2, 'deduce': 1, 'enlistment': 1, 'rum': 5, 'harden': 2, 'reassures': 3, 'supplies': 7, 'oranges': 1, 'unengaging': 3, 'branch': 5, 'effectively': 30, 'lipstick': 3, 'gutch': 1, '1951': 1, 'auto': 3, 'spins': 6, 'ghastly': 6, 'exotic': 11, 'ashby': 1, 'jena': 2, 'untangle': 1, 'compile': 1, 'brooks': 15, 'transplanted': 1, 'hedonistic': 1, 'capra-esque': 1, 'beheadings': 2, 'investigated': 3, 'klingman': 1, 'lived-in': 1, 'smother': 1, 'israeli': 1, 'beautiful--in': 1, 'labored': 2, 'high-level': 2, 'progeny': 1, "deputy's": 1, 'arch-nemesis': 1, 'reads': 13, 'cesspool': 1, 'component': 3, 'science-heavy': 1, 'cinema': 80, 'trivia--co-stars': 1, 'color': 49, "cal's": 3, 'artifact': 1, 'hair-raising': 2, 'jack-styled': 1, 'inquired': 2, 'patterned': 1, 'walkie': 1, 'hoax': 4, 'listeners': 2, 'better-known': 1, 'louiso': 1, 'gas': 16, "blessed's": 1, "murron's": 1, 'gleason': 1, 'yankees': 1, "anij's": 1, 'parsley': 1, 'misunderstanding': 3, 'mafia': 10, 'moves': 60, 'innovation': 2, 'introduction': 13, 'circulated': 2, 'aesthetically': 2, 'midget': 1, 'kicked': 9, 'beliefs-eloquently': 1, 'introverted': 2, 'prisoners-of-war': 1, 'cameron-regular': 1, 'raining': 4, 'evidently': 5, 'out-of-character': 1, 'chinese-american': 2, 'reagan-era': 1, 'undoubtedly': 9, "jip's": 1, 'prolonged': 1, 'enright': 1, "dorothy's": 1, 'describes': 15, 'input': 3, 'extraction': 1, 'banquet': 3, 'tschombe': 1, 'lapses': 5, 'jailed': 2, 'lobs': 1, 'carpings': 1, 'maggies': 1, 'elderly': 17, 'overlays': 1, 'vaild': 1, 'all-star': 2, 'grave': 11, "women's": 3, 'forgivable': 4, 'perilously': 1, 'seduction': 9, 'anarchist': 1, 'dumbfounded': 1, 'shrew': 3, 'marketers': 1, 'exploded': 2, 'vader': 13, 'replacing': 2, 'tripp': 1, 'abdul-jabbar': 1, 'atrophy': 1, "wall'": 1, 'bounding': 1, 'deceptively': 2, 'disingenuous': 2, 'awe-filled': 1, 'wonka': 3, '-worthy': 1, 'indolent': 1, 'nebbish': 1, 'hit-and-miss': 2, 'patlabor': 1, 'stealthy': 1, 'whippersnappers': 1, 'stressed-out': 3, "african's": 1, 'medicinal': 1, 'patachou': 1, 'freshman': 2, 'fairytale-like': 1, 'reluctance': 6, 'geographical': 2, 'thousands': 18, 'relocating': 2, "coach's": 1, 'aged': 7, 'iris': 3, 'navy': 7, 'counterparts': 8, 'hebrew': 2, 'sunflowers': 1, 'attention': 119, 'sold': 18, 'sitch': 1, 'thirtysomething': 2, 'capoeira': 1, 'befriend': 3, 'pllllleeeeease': 1, 'ratzenberger': 3, 'wrap': 7, 'joshua': 1, 'unflinching': 3, 'skerrit': 1, 'saul': 1, 'respects': 11, 'escapist': 8, 'disposed': 1, 'corpse': 7, 'investments': 1, 'newton-john': 3, '_do_': 1, 'earthly': 3, 'six-pack': 1, 'woman': 152, 'winds': 14, "opus'": 1, 'brinks': 1, 'firearms': 3, 'arguing': 9, 'bravura': 5, 'canceled': 2, 'straight-faced': 3, 'power--to': 1, "cliches'": 1, 'johnny': 19, 'hayes': 2, "yeoman's": 1, 'peasants': 4, 'seep': 3, '`eat': 1, 'concerto': 1, 'insult': 7, "morrison's": 3, 'interments': 1, 'clapping': 1, 'pool': 14, 'clinics': 2, 'pounds': 6, 'joyous': 2, 'sustenance': 3, 'filmmaking': 42, 'shift': 6, 'situated': 1, "bresson's": 1, 'teamed': 6, 'fruit-cake': 1, 'goldstein': 1, 'mettle': 2, 'archbishop': 2, 'godfather': 16, 'namuth': 1, 'commended': 5, 'ball-bouncing': 1, 'unlicensed': 1, 'seedy': 11, 'sci-fi/horror': 2, 'grooving': 2, 'statues': 1, 'previous': 81, 'odette': 1, 'flavour': 3, 'sonar': 1, 'procrastination': 1, 'self-esteem': 2, 'bizarro': 2, 'familial': 4, 'sound-truck': 1, 'reagan': 4, 'mental-patient': 1, 'tampering': 2, 'quicktime': 1, 'andreas': 2, 'sesame': 1, 'ceos': 1, 'overseas': 1, "creators'": 2, 'tough-talking': 1, 'esteemed': 1, 'prevents': 8, 'everyman': 10, 'damnation': 2, 'dudeness': 1, 'destination': 15, 'pikers': 1, 'sellars': 2, 'earthy': 3, 'malcolm': 7, 'guest-quarter': 1, 'monkey': 9, 'dr': 55, 'hannon': 1, 'realize': 79, 'everytime': 4, 'suspense': 50, 'vinyl': 2, 'warnings': 8, "schindler's": 14, 'lordship': 1, 'afforded': 2, 'angers': 2, 'it]': 1, 'kosteas': 2, 'unearths': 2, "anti-hero's": 1, 'upping': 1, 'tick': 2, 'pharmaceutical': 1, 'infallible': 2, 'acception': 1, 'treacly': 2, 'massacres': 1, 'off-handed': 1, 'skill': 17, 'cuisine': 1, 'bitchie': 1, 'butterworth': 4, "don't-hate-me-for-being-a-simpleton": 1, 'lyndon': 2, 'shiny': 2, 'malt': 2, 'typing': 1, 'league': 13, 'fex': 1, '$80': 1, 'perf': 2, 'brew': 4, 'section': 15, 'golden': 27, 'policies': 1, 'gretel': 1, 'palpatine': 1, 'liveliness': 1, 'part-human': 1, 'hurt': 34, 'hedwig': 2, 'danielle': 2, 'pronounce': 1, 'surveying': 3, 'berkeley-esque': 1, 'overused': 5, 'seance': 1, 'treatments': 2, 'hasty': 1, 'cementing': 1, 'tromping': 2, 'ex-slave': 1, 'agreed': 7, 'dolls': 4, '25-year-old': 1, 'starving': 3, 'mouth-opening': 1, 'ever-likeable': 1, 'primitive': 8, 'protesting': 1, 'named': 141, 'tempts': 1, 'mixed-coffin': 1, 'occurring': 4, 'persuasive': 2, 'faux-pas': 1, 'conducive': 1, 'squat': 1, 're-converge': 1, 'examination': 8, 'hawkins': 2, 'cleopatra': 2, "mulan's": 7, 'father+s': 1, 'sway': 4, 'young': 261, 'mirrors': 4, 'rapist-killer': 1, 'regarded': 7, 'snow': 17, 'subjugate': 1, 'portents': 1, 'constant': 38, "genre's": 2, 'expended': 1, "poledouris'": 1, 'worrisome': 2, 'substantially': 3, 'acheivement': 1, 'batcave': 1, 'minors': 1, 'gershwin': 1, 'heartwrenching': 2, 'hero': 82, 'chubby': 7, '9-year-old': 1, 'implodes': 2, "'little": 1, "felix's": 2, 'engulfing': 1, 'observance': 2, 'destroy': 36, 'sportsmanship': 1, 'appallingly': 1, 'qualifying': 1, 'gods': 3, 'cruise': 16, 'contractual': 1, 'wound': 9, 'co-producers': 1, 'amazingly': 16, 'revelled': 1, "show's": 16, 'pamela': 5, "capone's": 1, 'chemicals': 1, 'poison': 4, 'arachnids': 2, 'rodriguez': 7, 'tidy': 6, 'burping': 1, 'corruptive': 1, 'eroded': 1, 'exstensive': 1, 'sugiyama': 2, 'partners': 8, 'outsmarting': 2, 'fist': 2, 'leguiziamo': 1, 'selects': 2, 'he/she': 2, 'bespectacled': 2, 'dazzles': 2, 'theatrically': 5, 'of=': 1, 'igor': 2, 'enabled': 1, 'warrirors': 1, "meredith's": 1, 'limo': 1, 'shelley': 2, 'flashbacks': 26, 're-creating': 1, 'research': 18, 'benefits': 10, 'prodigy': 4, "ihmoetep's": 1, 'apprehensions': 2, 'first-rate': 12, 'overacts': 2, 'thrown': 41, 'smithee': 1, 'batterings': 1, 'unoriginality': 1, 'kicker': 1, 'king-inspired': 1, 'disk': 2, 'pilgrim': 2, "stretch's": 1, 'robberies': 4, 'two-and-a-': 1, 'mel': 18, 'misinterpreted': 1, 'finace': 1, 'candour': 1, 'simplify': 1, "'friend'": 1, 'wicked': 12, 'gordon': 7, 'eugenio': 1, 'plummet': 1, 'ready-made': 1, 'miscommunicated': 1, "'lake": 1, 'cohort': 2, 'aidan': 1, 'projects': 20, 'choreographing': 1, 'mystery-horror': 1, 'brandon': 4, 'ca_': 1, "zwick's": 3, "al'": 1, 're-examine': 2, 'pigeons': 1, "tampopo's": 1, 'surrendered': 2, 'sofia': 1, 'olenska': 1, 'philanthropist': 1, '$60': 4, 'racially-motivated': 1, 'scruches': 1, 'tricia': 1, 'taxi': 13, 'seth': 11, 'muldoon': 2, 'breakup': 4, 'arguments': 8, 'r&r': 1, 'out-of-this-world': 1, 'liased': 1, "`thrillers'": 1, 'broad': 18, 'insincere': 1, "fincher's": 5, 'announcing': 2, 'tended': 5, 'stuart': 11, 'sole': 18, 'muted': 4, 'contend': 2, 'substandard': 1, 'straightforward': 22, 'redman@bvoice': 1, 'detection': 1, 'mortgage': 1, 'robber': 5, 'uninterest': 1, 'smart-assed': 1, 'needful': 1, 'discourage': 1, 'sham': 6, 'wine': 2, 'yee': 1, 'radiate': 2, 'gough': 3, 'fewer': 7, 'bitingly': 2, 'supervisor': 4, 'kimba': 1, 'one-upped': 1, 'hands-washing': 1, 'doctrines': 2, 'alienating': 3, 'vampire-films': 1, 'austin': 15, 'bravo': 1, 'effects-laden': 3, 'solidifying': 1, 'implement': 1, 'tore': 2, 'flexes': 1, 'cowboy=': 1, 'dvds': 4, 'memento': 2, 'hard-working': 5, 'script-writers': 1, 'detonate': 1, 'efforts': 29, 'installments': 8, 'withheld': 1, 'non-clinton': 1, 'sympathies': 1, 'winfrey': 2, 'kelsey': 4, 'groaned': 1, 'beautifully': 36, 'self-trance': 1, 'charlton': 2, 'warped': 9, 'acquaintances': 2, 'barrymore': 13, 'recoiling': 1, 'dragons': 2, 'lawyer': 42, 'voice-changer': 1, '_angel': 1, 'cocker': 1, 'hair-obsessed': 1, 'doli': 1, 'freakish': 3, 'duchovny': 6, '8-lane': 1, 'christianity': 2, 'unreliable': 3, 'steroids': 1, 'replace': 4, 'theorist': 1, 'occurances': 1, "marker's": 1, 'broader': 3, 'rate': 29, 'resound': 1, 'non-david': 1, 'magazine': 19, 'rawhide': 1, 'completly': 1, 'layer': 6, 'hick': 1, 'improv': 1, 'linz': 2, 'abortions': 1, 'clutching': 2, "barnett's": 1, '37': 2, 'jinx': 1, 'bunny': 8, "harm's": 2, 'funkiness': 1, 'porch': 2, 'idolising': 1, 'bloke': 1, 'fenster': 2, 'kisses': 5, 'greatness': 20, 'pseudo-sexy': 1, 'tumble': 1, 'spoilsport': 1, 'abeyance': 1, 'predominantly': 3, 'defeat': 12, 'larina': 1, 'seafood': 1, "sarandon's": 1, 'reserving': 2, 'tyrant': 1, 'injection': 2, 'anthony': 30, 'euro-vamps': 1, 'low-hanging': 1, 'simulation': 2, 'favorable': 3, 'fishtank': 1, 'interesting': 206, 'loved': 56, 'inflicted': 4, 'buzz': 12, 'tangerine': 1, 'broadcaster': 1, 'resign': 1, 'competitions': 1, 'irish-italian': 1, 'heart-pull': 1, 'sinks': 7, 'beastie': 1, 'yearbook': 2, 'cigarettes': 2, 'bathrobes': 1, 'bullying': 2, '8/10': 15, '//www': 4, 'ecologist': 1, "maunau's": 1, 'varies': 3, 'dredd': 2, 'katrina': 3, 'maxwell': 1, 'grossest': 1, 'greyer': 1, 'francesco': 1, 'fantsy': 1, 'opening-night': 1, 'remedy': 2, 'pastoral': 1, 'rohmer': 2, 'recouperating': 1, 'digitized': 1, 'foams': 1, ';': 358, "juvenile's": 1, 'hooch': 1, 'squabbling': 1, 'romanticism': 2, 'treated': 21, 'timing-oriented': 1, 'hat': 11, 'speech': 40, 'occupation': 4, 'harkonnen': 1, 'crouches': 1, 'ten-minute': 1, '2015': 1, 'bounds': 3, 'lining': 2, 'nuanced': 2, "`that's": 1, 'cooperated': 1, 'drowned': 2, 'columbia': 5, 'derailed': 1, 'moronic': 8, 'reporting': 3, 'paraphrase': 3, 'cusp': 1, 'out-maneuvers': 1, 'smash-hit': 1, "seven's": 1, 'insisted': 2, 'stevenson': 1, 'spaceballs': 2, 'mst3k': 2, 'intellectually-challenged': 1, 'down-but-not-out': 1, 'prodigies': 2, 'attaining': 1, 'heartache': 2, "luedeke's": 1, 'underpopulated': 1, "d'arc": 1, "group's": 5, 'co-produced': 1, 'precedents': 1, 'scalding': 1, 'annoying': 52, 'haired': 1, 'dashing': 3, 'caucasian': 2, 'gong': 2, 'donatelli': 1, 'me-had': 1, "foreigners'": 1, 'brush': 3, "doesn's": 1, 'inarguably': 1, 'cliche': 14, 'faceless': 3, 'mcgill': 3, 'night-journey': 1, 'impressionable': 2, 'uncomplicated': 2, 'oscar-less': 1, 'trelkins': 1, 'robotic': 2, 'army': 39, 'resonates': 2, 'kinkiness': 1, 'patchy': 1, 'abusive': 8, 'college': 38, 'terrifed': 1, 'halt': 6, 'attending': 5, 'aww': 2, 'hour-and-a-half': 1, 'quoc': 1, 'justice': 37, 'darnell': 1, 'troop': 1, 'hollander': 1, 'vince': 11, 'needlessly': 4, 'releasing': 4, 'kafkaism': 1, 'judiciary': 1, 'peering': 1, 'dismal': 9, 'titanic-type': 1, "'craziness'": 1, 'thanksgiving': 2, 'nickname': 5, 'carping': 1, 'positively': 5, "morris's": 1, 'effects': 155, 'casinos': 2, 'accompany': 7, 'guerrilla': 2, "bruce's": 1, 'les': 9, 'absorbed': 6, 'oversee': 2, 'van': 48, 'nguyen': 1, 'mythology': 7, 'keenly': 2, 'glimmer': 3, 'pre-': 3, 'three-dimensional': 9, 'revolutionized': 1, 'renewing': 1, 'relive': 1, 'hottest': 1, 'widow': 6, 'co-writing': 2, 'sixth': 14, 'cuteness': 4, 'empathy': 6, 'chases': 17, 'nights': 30, 'jousting': 2, 'super-vampire': 1, 'slavomir': 1, 'waitering': 1, 'expenses': 3, 'cassini': 1, 'transmission': 3, 'steers': 3, 'downer': 1, "marco's": 1, 'misdemeanors': 1, 'jade': 2, 'schwartzeneggar': 1, "gettin'": 1, 'pedophilia': 2, 'g-rated': 1, 'long-time': 4, 'steinberg': 1, "cheadle's": 1, 'ultra-eccentric': 1, 'smalltime': 1, "father-in-law's": 1, 'grasshopper': 1, "columbia's": 1, 'mcbeal': 1, 'claire': 14, 'violations': 1, 'hewitt': 2, 'leoni': 3, 'toeing': 1, "shelley's": 1, 'stockard': 3, 'american/spanish': 1, 'searching': 26, 'joker-smile': 1, 'governs': 1, 'loyalist': 1, 'robots': 10, 'squealed': 1, 'peep': 2, 'idyllic': 7, 'venom': 1, 'occuring': 2, 'bombshell': 3, 'alexi-malle': 1, 'buck-toothed': 1, 'climbing': 1, 'creepily': 1, "o'halloran": 1, 'focussed': 1, 'murkily': 1, 'rainstorm': 4, 'leboswki': 1, 'polarizes': 1, 'low-brow': 1, 'recognize': 24, 'opponents': 5, 'cartman': 1, 'virginal': 3, 'ione': 1, 'rereading': 1, 'hilariously': 8, 'eachother': 7, 'molded': 4, 'amarcord': 1, 'presumedly': 1, 'under-developed': 1, 'hip': 18, 'spurts': 1, 'leaving': 86, 'simple': 130, 'ready': 50, 'falls': 83, 'judges': 7, 'outdid': 1, 'baseman': 1, "carrey's": 8, "wahlberg's": 3, 'domain': 5, 'scold': 2, 'teeter-totter': 1, 'staples': 2, 'hispanic': 2, 'rebellions': 1, 'intrusion': 2, 'anxiety': 5, 'wry': 5, 'heart-warming': 4, 'daper': 1, '_am_': 1, 'millieu': 1, 'not-so-good': 1, 'reasonable': 8, 'betterment': 1, "fiction's": 1, "cup'a'coffee": 1, 'desertion': 1, 'wrenching': 2, 'academia': 2, 'shake': 3, 'tourfilm': 1, 'type-casted': 1, 'rhinoceros': 1, "necklace's": 1, "dorn's": 1, '_the_lion_king': 1, 'notched': 1, 'premium': 2, 'fly-guy': 1, 'terk': 4, 'lock': 4, 'flaxen-haired': 1, 'unchained': 1, 'proceedings': 19, 'alison': 3, 'nutcracker': 1, "tolkien's": 1, 'bustle': 1, 'researching': 1, 'biker': 3, 'movingly': 1, '1939': 2, 'bannen': 4, 'desklamps': 1, 'high-flying': 1, 'viewing': 50, 'reappeared': 1, 'belief': 25, 'zangaro': 1, 'fused': 3, 'providing': 22, 'julliard-trained': 1, 'overdosing': 1, 'gere': 4, 'interpersonal': 1, 'endless': 18, 'prefers': 6, 'ratchets': 1, 'adult-oriented': 2, 'destructiveness': 2, 'surgery': 4, 'cole-his': 1, 'letters': 9, "alice's": 3, 'makeshift': 1, 'cruz': 3, 'heavy-metal': 1, 'gwtw': 1, 'threatens': 17, 'jumanji': 1, 'keach': 2, 'jock': 9, 'eluded': 1, 'illustrate': 2, 'non-formula': 1, 'story-telling': 1, 'faintly': 1, 'mullen=92s': 1, 'scoff': 2, 'program': 20, 'run-of-the-mill': 3, 'basil': 3, 'humpback': 1, 'hickam': 3, '42': 4, 'meanings': 6, 'invents': 3, 'pickett': 1, 'teetering': 2, 'starlight': 1, 'cosmos': 1, 'involving': 97, 'panicking': 3, 'slightly-less': 1, "clark's": 1, 'corman': 1, 'horseman': 1, 'clancy': 1, 'gattaca': 11, 'dromey': 1, '$200': 6, 'combs': 3, 'destroyers': 2, 'yarn': 5, 'shaves': 2, 'buena': 1, 'frigid': 2, 'korshonov': 1, 'torah': 2, 'actioner': 2, "calloway's": 1, 'thwarted': 1, 'ulation': 1, 'tooth': 3, 'devoid': 6, 'goblin': 2, 'rejoining': 1, 'bigalow': 2, 'snoozing': 1, 'part-time': 3, 'disheveled': 1, 'vetern': 1, 'pieces': 42, 'panamanian': 1, 'nightclubs': 1, 'stereotyped': 1, 'rereleased': 3, 'moralist': 1, 'partying': 1, 'peforming': 1, 'mcgreggor': 1, 'taels': 1, 'indecisive': 2, 'scratched': 3, 'cardiac': 1, 'stout': 1, 'regretting': 1, 'antiseptic': 1, 'beaumont': 1, 'infra-red': 1, 'award{symbol': 1, 'unpopular': 1, 'postlethwaite': 1, 'role-model': 1, '_that_': 2, 'off-broadway': 3, 'watanabe': 3, 'minions': 2, 'ukraine': 1, 'carter--a': 1, 'alcoholic': 6, '!': 256, 'lawsuits': 1, 'sloshed': 2, 'hugh': 8, 'rosalba': 2, '1996': 20, 'head-bopping': 1, 'banisters': 1, "'tune": 1, 'kendrick': 1, 'detracted': 3, 'ditching': 2, 'redeemed': 2, 'confiscates': 1, 'distributed': 4, 'cumming': 2, 'riotous': 3, '`scientifically': 1, 'pop-pop': 1, 'defiance': 2, 'par': 17, 'make-shift': 1, 'condom': 3, 'essays': 2, 'undergoes': 1, '1953': 2, 'cheerful': 9, 'items': 5, 'cautionary': 4, "tucker's": 4, 'dressing': 9, 'alcohol-based': 1, 'latch': 2, 'catholic': 10, 'likability': 1, "veronica's": 1, 'shakespearian': 1, 'lesser': 20, 'seemingly-perfect': 1, 'errant': 3, 'stomach': 9, 'irrational': 1, 'farbissina': 2, 'weary': 9, 'rivals': 5, 'weighty': 3, 'construed': 4, 'coup': 2, 'rade': 2, "sean's": 2, 'strolls': 1, 'tagging': 3, 'sickly': 1, "watson's": 3, 'overdoes': 1, 'substituting': 1, 'caf': 1, 'tibetan': 3, 'confessionals': 1, '80s': 11, 'cringe-worhy': 1, 'sits': 10, 'haphazard': 1, 'club': 34, 'wiping': 1, 'flops': 1, 'half-baked': 1, 'speechless': 1, 'twins': 6, 'envelope': 4, 'correlated': 1, 'repressive': 2, 'clutterbuck': 1, 'tells': 110, 'impresses': 2, 'hand-fed': 1, 'nicknamed': 8, 'commotion': 1, 'sews': 1, 'farrelly-funny': 1, 'vivian': 1, 'punch-outs': 1, 'decided': 38, 'unambitious': 1, 'toughness': 1, 'blatantly': 6, 'hefty': 9, 'stylistic': 12, 'prof': 2, 'strands': 4, 'twohy': 3, 'flushes': 1, 'disturbingly': 4, 'hubley': 2, 'frenchman': 2, 'bear-like': 1, 'slope': 2, "frankie's": 1, 'paxton': 14, "kevin's": 3, 'breaths': 1, 'half-cynical': 1, 'sage-like': 1, 'athletic': 5, 'martian': 1, 'then-footage': 1, 'evolving': 2, "winslet's": 1, 'disturbance': 1, 'time-travel': 2, 'unforeseen': 2, 'invigorate': 1, 'trey': 3, 'deserved': 22, 'stop': 94, 'ritchie': 3, 'smokescreen': 1, 'hilt': 1, "survivors'": 1, 'promise': 22, 'dahlgren': 1, 'stuggles': 1, 'fagina': 1, 'inspire': 2, 'gigs': 4, "liotta's": 1, 'chauffeur': 4, 'despise': 3, 'sena': 1, 'disappears': 11, 'trick': 18, 'insider': 1, 'mathematical': 5, 'belong': 12, 'agreeable': 7, 'completed': 9, '`toon': 1, 'dialects': 1, 'apparition': 2, "peak'": 1, "'rewind'": 1, 'combined': 22, 'bratty': 1, 'fight': 98, 'doctor/patient': 1, "rivka's": 1, 'audiotapes': 1, 'mentor': 9, "moriceau's": 1, 'film--twice': 1, 'wig': 4, 'antagonistic': 1, 'rail': 2, 'fatale': 2, 'pack': 13, 'documentarian': 1, 'filipino': 1, 'springsteen': 1, 'mysterious': 57, 'existance': 1, 'alumni': 1, 'partner': 29, 'friedkin': 2, "issue's": 1, 'childbirth': 1, 'lined': 1, 'comepete': 1, 'substitute': 5, 'gellies': 1, 'bygone': 1, 'subjected': 12, 'secession': 1, 'polynesian': 1, 'pencilled': 1, 'cheeze': 1, 'rounder': 1, "government's": 1, 'gore-fest': 1, 'angus': 2, "miramax's": 1, "vincenzo's": 1, 'tusken': 1, 'well-told': 2, 'classical': 9, 'transparently': 1, 'wiseguy': 5, 'egar': 1, 'formed': 6, 'cues': 5, 'nut-biting': 1, 'farewell': 1, 'first-class': 2, 'precious': 13, 'capitalist': 2, 'wellville': 1, 'concussion': 1, 'viable': 2, 'fiery': 6, 'flounder': 4, 'tumbleweeds': 1, 'pretentiousness': 4, 'promenade': 1, 'half-gods': 1, '----': 1, 'karla': 1, 'intends': 4, 'offed': 2, 'ultra-naturalistic': 1, 'negatives': 3, 'cop-killers': 1, 'doctors': 6, 'decaying': 4, "get's": 2, "enright's": 1, 'collaborators': 4, 'neutrality': 1, 'immensely': 17, 'man--the': 1, 'niceness': 1, 'senge': 1, 'melissa': 2, "soderbergh's": 3, 'gallery': 7, 'multiplying': 1, 'unwanted': 3, 'boasts': 17, 'steps': 23, 'prophesized': 2, 'stifle': 1, 'curmudgeon': 3, 'bondian': 1, 'mother': 139, 'ito': 1, '1938': 1, 'facile': 1, 'serpent': 1, "lubezki's": 1, 'flowing': 9, 'ensnare': 1, "rocky's": 2, 'ideology': 10, 'georgia': 9, 'magnolia': 3, 'overbearing': 12, 'galactica': 1, 'attacker': 3, 'untalented': 2, 'padawan': 1, 'sub-genres': 2, 'anette': 1, "wayne's": 3, 'feore': 1, 'mend': 2, "witch'": 1, 'excellence': 3, '1272': 1, 'vacances': 1, "cappie's": 1, 'door': 40, 'premiered': 3, 'prob-lem': 1, 'wool': 2, "series's": 1, 'coups': 1, 'audacious': 5, 'peoples--or': 1, 'basement': 9, 'spinetinglingly': 1, 'callum': 2, 'dumping': 8, 'authorties': 1, 'categorization': 1, 'adjustments': 1, 'worker/janitor': 1, 'broodwarriors': 1, 'lava': 2, 'asserts': 1, 'pains': 4, 'cold-blooded': 2, 'grouchland': 1, 'puppeteer': 2, 'so-called': 8, 'outstanding': 57, 'dom': 2, 'myriad': 5, 'hand-in-hand': 4, 'arlo': 2, 'herrman': 1, "`la-dee-dah'": 1, 'choices': 28, 'palestinian': 1, 'reels': 1, 'clintonesqe': 1, "forrester's": 1, 'acerbic': 4, 'herlihy': 2, 'engross': 1, 'graduates': 2, 'strangely': 21, 'electronic-synthesized': 1, 'double-barreled': 1, 'update': 4, 'roselyn': 1, 'colonel': 16, 'hilfiger': 1, 'plotwise': 1, "creator's": 1, 'tactics': 6, 'choreographed': 14, 'gallons': 1, 'straighten': 1, 'sexually': 10, 'miramax': 5, 'josephine': 2, "marcy's": 1, 'humming': 3, 'counterprogramming': 1, 'slow-paced': 3, 'warts-and-all': 1, 'resigned': 3, 'farmhouse': 1, 'churchgoing': 1, 'chris': 52, 'bombings': 3, 'olympic': 1, 'sloppy': 5, 'midnight': 18, 'kindness': 6, 'polly': 1, 'loudmouthed': 1, 'fledgling': 3, 'incident': 19, 'skyrocketed': 1, 'feature--': 1, 'shan-yu': 2, 'half-hour': 8, 'incomprehensible': 3, 'marvellous': 2, 'earth-orbiting': 1, 'plight': 13, "pearce's": 1, 'voyeurs': 2, 'accept': 36, "'key'": 1, 'sci-fi': 38, 'co-owns': 1, 'dicaprip': 1, 'terrarium': 1, "character's": 45, "'till": 2, 'picked': 23, 'encompassed': 1, 'market': 20, '61': 1, 'alotta': 1, 'scramble': 1, 'overkilling': 1, 'backroads': 1, 'bale': 3, 'sidelines': 3, 'kudos': 16, 'mcgehee': 3, "'cripple'": 1, 'converging': 1, 'send-up': 3, "disaster's": 1, 'hates': 9, 'odessa': 3, 'generalization': 1, 'restless': 10, 'ex-lover': 1, 'b-movie': 3, 'actors/directors/producers': 1, 'magdelene': 1, 'rack': 1, 'stretch': 14, 'kkk': 1, 'break-out': 1, 'cracked': 5, 'spacemusic': 1, 'delusion': 1, 'motivational': 2, 'thankfully': 46, 'cabinet': 3, 'fiance': 4, 'matriarch': 1, 'adult-helfgott': 1, '_daylight_': 1, 'hypnotized': 4, 'kidnaping': 1, 'all--except': 1, 'tarnish': 1, 'rollickingly': 2, 'unnerrving': 1, '_cliffhanger': 1, 'lude': 1, 'viper': 1, 'happily': 12, 'lowlife': 2, "rob's": 1, '14-year': 2, 'jean-baptiste': 1, 'coal-miners': 1, 'interactive': 3, 'unassociated': 1, 'testosterone': 4, 'wanderlusting': 1, 'burp': 1, 'gangstar': 1, 'sweaters': 1, 'en-route': 1, 'heathers': 2, 'chase--but': 1, 'cyborsuit': 1, 'ballet-': 1, 'moviemakers': 1, 'absorption': 2, 'ufo-researching': 1, 'geoffrey': 14, 'watchful': 1, 'spontaneous': 7, "ray's": 1, 'forrest': 12, 'comissioned': 1, 'e-mail': 7, 'gable': 2, 'destabilizing': 1, 'evaluating': 1, 'praised': 5, 'countess': 1, 'stretches': 10, 'americas': 2, 'tailor': 1, 'mindy': 2, 'ot': 1, 'shoot-out': 1, 'dessert': 2, 'moore': 21, 'awfulness': 3, 'sweatshirt': 1, 'skywalker': 12, 'conventionally': 1, 'thunderstorms': 1, 'stopping': 1, 'professionals': 3, 'address': 6, 'implanted': 2, 'instills': 3, 'toughest': 4, "whale's": 3, 'albino': 1, 'predominately': 2, "o'connor's": 2, 'lovestruck': 2, 'gordie': 1, 'anaconda': 3, 'bottom-of-the-drug-food-chain': 1, 'mystics': 1, 'sometimes-blaring': 1, "browne's": 1, 'gilliam': 4, 'dreamt': 1, 'judgments': 1, 'assailants': 1, 'sagas': 1, 'trumbull': 1, "sugiyama's": 1, 'general': 80, 'sunlit': 1, 'always-reliable': 2, 'chaos': 21, 'momma': 2, 'taguchi': 1, 'judicial': 1, 'soundbite': 1, 'transports': 6, 'correcting': 2, 'delayed': 4, 'andromeda': 3, 'revealed': 28, 'glamorize': 1, 'phantom': 17, 'noteriaty': 1, 'east': 12, 'potatohead': 1, 'inviting': 9, 'helped': 26, 'ytv': 1, 'writhing': 2, 'dwells': 2, 'fend': 5, 'renowed': 1, 'deepa': 1, 'accent': 31, 'concussions': 1, 'miep': 1, 'bumblebee': 1, 'co-directed': 1, 'acts': 27, 'tall': 9, 'courteney': 3, 'clapped': 1, 'janusz': 2, 'weathers': 6, '$10000': 1, 'scissorhands': 5, 'message': 60, 'prisoner': 20, 'prozac': 1, 'slacker': 3, 'well-chosen': 1, 'initiation': 2, 'boats': 7, "patrick's": 1, 'unsparing': 2, 'chans': 1, 'warm-up': 1, 'ringer': 1, 'prot': 2, 'endearing': 30, 'self-replicating': 1, 'disloyal': 1, 'evenings': 3, 'self-preservation': 1, 'platform/prison': 1, 'ferociously': 5, 'cherished': 4, 'scorn': 1, 'careers': 6, 'deny': 7, 'cheer': 12, 'injustice': 8, '`long-distance': 1, 'hideko': 1, 'hamburger': 1, 'analee': 1, 'non-memorable': 1, 'nominally': 1, 'imhotep': 1, 'potato': 5, 'zest': 2, 'velma': 1, 'huckster': 1, 'omen': 2, 'wasted--and': 1, 'plumb': 1, 'abundantly': 3, 'yellows': 1, "welles'": 1, 'colleague': 11, "'n'": 1, 'glover': 7, 'judge': 29, 'twentysomething': 2, 'renaissance': 3, 'waters': 13, 'fragmentary': 1, 'org': 1, 'bracket': 1, 'grotesquely': 1, 'rigg': 1, 'generation-x': 1, 'perseverance': 4, 'routinely': 3, 'garage': 8, 'sf': 6, 'kicks': 10, 'encourages': 5, "slim's": 1, 'warriors': 7, 'aloud': 2, 'unwilling': 4, 'middle-man': 1, 'hobbyist': 1, 'commendable': 11, 'looked': 35, 'gothic/electronic': 1, 'satellite-controlled': 1, 'criticisms': 8, 'seagal': 1, 'film-the': 1, 'refused': 10, 'wall-to-wall': 5, 'sunglasses': 7, 'tug-of-war': 1, 'coney': 1, 'glamorized': 1, 'repayment': 1, 'portinari': 1, 'meant': 24, 'animation': 47, 'dwarves': 3, 'preposterous': 4, 'perpetuates': 1, 'affectations': 2, 'dated': 10, 'romulus': 1, 'commandments': 1, 'rivaled': 1, 'unger': 3, 'lying': 12, 'procession': 1, 'detonating': 2, 'accomplishes': 8, "rumpo's": 1, "intro'd": 1, 'frail': 2, 'jee': 1, 'inexpensive': 1, 'hutchinson': 1, 'dilutes': 2, 'adopt': 6, 'recreating': 7, 'warrior': 8, 'co-stars': 11, "campion's": 1, '--quite': 1, 'anouk': 1, 'womanizer': 3, 'film-maker': 4, 'rajot': 1, 'outdo': 3, 'sabor': 2, 'antagonizes': 1, 'hard': 169, 'castles': 4, 'rescored': 1, 'conveniently': 5, 'insidiously': 1, "emperor's": 2, 'narrator': 16, 'aroused': 2, 'skye': 1, 'fresh-faced': 2, 'apple-cheeked': 1, 'fueled': 3, "`impulse'": 1, 'post-modern': 2, 'plotline': 8, 'enriched': 1, "jaoui's": 1, 'militant': 1, "incomplete-it's": 1, 'nominations': 9, 'harker': 1, 'pokes': 5, 'bai': 1, 'croaker': 1, 'low-grade': 1, 'ole': 2, 'equidistant': 1, 'lukewarm': 3, 'slinks': 1, 'hopefuls': 1, 'symbolized': 4, 'gustav': 1, 'mckinley': 1, 'contemplates': 2, 'rollergirl': 2, 'soaked': 2, 'gouda': 1, 'clouded': 1, 'regicide': 1, 'poachers': 1, 'cartoony': 3, "clutterbuck's": 1, 'omaha': 4, 'skecthes': 1, 'keaton': 14, 'covers': 12, 'points-of-': 1, 'symptoms': 4, 'sleeper': 7, 'seat-handles': 1, 'putrefaction': 1, 'razzle-dazzle': 1, 'piss': 1, 'pact': 6, 'bummer': 1, 'super-mom': 1, 'glamorising': 1, 'invited': 11, '-genre': 1, 'mug': 1, 'moise': 2, 'flimsy': 4, 'neill--better': 1, 'liev': 6, 'beholder': 1, "'driven'": 1, 'dubbing': 6, 'entirly': 1, 'deviousness': 1, 'chimp': 1, "mobster's": 2, 'karate-chopping': 1, 'waiting': 40, 'pro-soldier': 1, 'gibb': 1, 'penner': 1, 'horny': 6, 'decade--the': 1, 'things-gone-awry': 1, 'bases': 2, "mid-80's": 1, 'issues': 52, "'haunting'": 1, 'irritatingly': 1, 'log': 2, 'ushered': 2, 'centre': 5, 'klugman': 1, 'protector': 2, '-esque': 2, 'staccatto': 1, 'tip': 7, 'whisper': 3, 'laforge': 1, 'share': 64, 'hideout': 1, 'bickering': 7, 'ravages': 3, 'riled': 1, 'underlining': 2, 'modernisation': 1, 'debra': 1, 'macivor': 1, 'inventor': 1, 'shaven': 1, 'retracing': 1, 'niche': 3, 'sadistic': 10, 'lenient': 1, '#5': 1, 'christie': 2, 'greggory': 1, 'embellish': 3, 'farrelly': 4, 'obliviousness': 1, 'dwellers': 1, 'craggy': 1, "losin'": 1, 'outlaw': 3, 'deadline': 2, 'unable': 29, 'infatuated': 6, 'kindergarten': 1, 'villiage': 1, 'drug': 46, 'hamunaptra': 1, 'aura': 6, 'hunts': 3, 'az--this': 1, 'tortures': 1, 'evangelical': 1, 'tinge': 3, 'movie-going': 1, 'residuals': 1, 'erotic': 13, 'loony': 1, 'preparation': 4, 'acting': 229, 'rejects': 4, 'banyon': 1, 'potts': 1, 'unsurpassed': 1, 'overpass': 1, "braun's": 1, 'clearing': 2, 'inscrutable': 1, 'docked': 2, 'unweaves': 1, 'peacably': 1, 'vie': 1, "anaconda's": 1, 'whacko': 1, 'expands': 2, 'movers': 1, 'multifaceted': 4, 'chow': 6, 'isolation': 10, "idiots'": 1, 'milestone': 5, 'willy': 4, 'diminished': 1, 'prodigious': 1, 'vincent/jerome': 2, 'sense': 202, 'pumpkin': 2, 'values': 28, 'restoration': 3, 'sharks': 3, 'outlet': 2, 'do--sing': 1, 'fright': 3, 'wistful': 2, 'failed': 32, 'screeching': 1, 'famished': 1, 'reduce': 3, 'well-functioning': 1, 'identifiers': 1, 'aprhodite': 1, 'endearment': 1, 'steddy': 1, 'self-referential': 4, 'unfolds': 16, 'connote': 2, "sayles'": 1, 'inanity': 1, 'hes': 1, 'copied': 3, 'flocking': 1, 'suggesting': 3, 'plop': 1, 'draped': 1, "rapist's": 1, 'passionately-': 1, 'combatants': 3, 'interfere': 1, 'corpulent': 1, 'gena': 2, '47-year': 1, '`rocky': 1, 'grenier': 1, 'goals--it': 1, 'moody': 14, "seattle's": 1, 'infidelity': 5, 'cup': 11, '55': 1, 'perk': 1, 'lillard': 2, 'segments': 14, 'douglas': 18, 'balto': 1, 'myrick': 1, 'corral': 1, 'b-grade': 1, 'stir': 10, 'put': 158, 'rejection': 4, 'heap': 7, 'return': 64, 'substances': 2, 'ricki': 1, 'harve': 1, 'barenboim': 1, 'hotels': 2, 'shocker': 6, 'gallagher': 3, 'jove': 1, 'estella': 3, 'hard-hitting': 3, 'genitals': 1, 'coating': 2, 'depp': 9, 'memorias': 1, 'perfectly': 99, 'electrocuting': 1, "deaky'": 1, 'high-schoolers': 1, 'agile': 1, 'scrolls': 1, 'modestly': 2, 'caress': 1, '102': 2, 'director/actor/co-writer': 1, 'wrencher': 1, 'poisonous': 2, 'sheikhi': 1, "pitt's": 2, 'lasseter': 1, 'tackling': 1, 'tabloid-fodder': 1, 'kati': 1, 'sacrament': 1, 'vinci': 1, "teachers'": 1, 'neighbourhood': 2, 'built-up': 1, 'toy': 22, '3-4': 1, 'editor': 20, 'groans': 2, '1922': 1, 'matador': 1, '70%': 2, 'tango-dancing': 1, "town's": 8, 'instructor': 3, 'molinaro': 1, 'adorably': 1, 'hampers': 1, 'rush': 30, 'amphibian': 1, 'travelogue': 2, 'dominic': 2, 'pedophilic': 1, 'cloaking': 1, 'untimely': 2, 'archibald': 1, 'torrid': 1, 'madeiros': 1, 'gestures': 7, 'capping': 1, 'kruger': 2, 'hunch': 1, 'prelude': 2, '`dharma': 1, 'self-propelled': 1, 'workaday': 2, 'baltus': 1, 'fun-to-watch': 1, 'promotions': 1, 'missteps': 12, 'aliases': 2, '2': 86, 'spares': 5, 'selected': 6, 'foredoomed': 1, 'disputes': 4, 'tatoo': 1, 'wondering': 26, 'copy-machine': 1, 'ceiling': 2, 'dripping': 1, 'skinheads': 3, 'jaw': 2, 'seductress': 1, 'mis-': 1, 'nowak': 1, 'bisexual': 2, 'bundles': 1, 'embry': 1, 'the-truth-at-all-costs': 1, 'neary': 1, 'shoveller': 1, "man'": 1, 'spud': 2, 'confrontation': 22, 'minefield': 1, 'freaked': 2, 'move': 66, 'sensational': 7, 'freaking': 2, 'glades': 1, 'socks': 4, '1920s-60s': 1, 'condolences': 1, 'descended': 1, 'trashing': 3, 'duty': 11, 'anti-phantom': 1, 'tragedies': 6, 'agricultural': 1, 'deeds': 9, 'cry': 17, 'jean-claude': 6, 'healy-louie': 1, "year's": 44, 'sound': 92, 'stupid/fun': 1, 'molina': 4, 'florida-set': 1, 'umpteenth': 1, 'cyanide': 1, 'instrumental': 2, 'encountered': 5, 'ailment': 1, 'magnificient': 1, 'magistrate': 1, 'romanticized-hemingway': 1, 'sink': 11, 'bully': 4, 'womanly': 1, 'roaches': 1, 'endangering': 2, 'addams': 3, 'seana': 1, 'unusually': 7, 'natural-born': 1, 'overrides': 1, 'showtime': 1, 'candyman': 2, 'conceal': 2, 'naming': 2, 'fallacy': 3, 'crams': 1, 'burly': 1, 'intermittent': 2, 'guitar': 4, 'volumes': 2, 'ashley': 7, 'landlady': 3, "writer's": 3, 'peckinpah': 3, 'attractive': 34, 'shadow': 16, 'doesnut': 1, 'panoramas': 1, 'hinged': 2, 'sun': 11, 'zen': 2, 'bayou_': 1, "mckellen's": 1, 'bloodlust': 1, "romero's": 1, 'bonding': 3, "convicts'": 1, 'archery': 1, 'attractions': 3, 'indestructible': 1, 'pan-slavic': 1, 'remembers': 4, 'flute': 2, 'suspect': 33, 'hooves': 2, 'qui-gon': 9, "mctiernan's": 2, 'indisputable': 1, 'maddalena': 1, 'skipping': 3, 'anticipates': 1, 'clearwater': 1, 'moses': 2, 'stuck': 31, 'watto': 1, 'quake': 1, 'transpire': 2, 'school': 111, 'natasha': 11, 'wilderness': 4, 'purple-sequined': 1, 'wields': 2, 'offense': 2, 'eskow': 1, 'greenhouse': 1, 'restaurant': 16, "'miming'": 1, 'townspeople': 9, 'positive': 22, 'incompetents': 1, 'rotate': 1, 'well-executed': 1, 'back': 341, 'beloveds': 1, 'outwitted': 1, 'renenged': 1, "material's": 1, '44-year': 1, 'luther': 4, 'cowardly': 6, 'dot': 2, "now-there's": 1, 'heartbreaks': 1, 'rhonda': 1, 'railly': 1, 'note-perfect': 1, 'budding': 5, 'ado': 1, 'disgusted': 7, 'detracts': 4, 'iceberg': 7, 'lover/prot': 1, 'forgiven': 5, 'jungian': 1, 'layoff': 3, 'polyester': 1, 'fulfils': 1, 'lenders': 1, 'species': 16, 'bureaucracy': 2, 'prose': 2, 'trickery': 2, 'gangland': 2, 'right-hand': 4, 'strasser': 1, 'reference': 18, 'mandala': 2, 'astonished': 3, 'bundle': 1, 'outgoing': 5, 'titillating': 4, 'busta': 1, 'departed': 2, 'privilege': 2, "nation's": 2, 'mutes': 1, 'bleeds': 2, 'however--the': 1, 'rex': 3, 'freshly': 1, 'theatrical': 21, "malkovich'": 2, 'explained': 20, 'matches': 14, 'dweeb': 1, 'mimieux': 1, '100+': 1, 'dialouge': 1, 'uneducated': 3, 'whipped': 1, 'coding': 1, 'megalomaniac': 4, 'chums': 5, "tati's": 1, 'idolized': 1, 'upscale': 2, 'blending': 4, 're-hashing': 1, 'tamer': 1, 'tombstone': 1, 'associating': 1, 'blessings': 1, "odile's": 1, "parton's": 1, 'beat': 23, 'kar-wei': 1, 'shamelessly': 5, 'imaginative': 22, 'rhys': 4, 'ecological': 1, 'relentlessly': 5, 'wylde': 1, 'critter': 3, 'avengers': 1, 'implausibility': 1, 'yielded': 2, 'post-forensic': 1, "zero's": 2, 'sexist': 2, 'showbiz': 1, 'inexplicable': 5, 'triumphant': 6, "holding's": 3, 'sensitive': 15, 'bertram': 1, "'everything": 1, 'hypernaturally': 1, 'yon': 1, 'above-ground': 1, 'jinks': 1, 'reclaiming': 1, 'pilson': 1, 'eigeman': 1, 'concrete': 3, 'hysterical': 15, 'overcoming': 2, '1/8': 1, 'danvers': 2, 'alot': 6, 'yeah': 22, "campbell's": 3, 'co-star': 10, 'stresses': 1, 'guggenheim': 2, 'arab': 4, 'tenderness': 6, 'chappy': 1, 'gladiator': 4, 'symphony': 5, 'rejuvenated': 1, 'handsome': 22, 'scheming': 5, 'obscenity': 2, 'begins': 182, 'implicitly': 1, 'edgy': 11, 'hostage-holding': 1, 'writer/director/actor': 1, 'hardship': 1, 'bombing': 2, 'exposition': 7, 'scoured': 1, 'exploitation': 5, 'submits': 1, 'lusts': 2, '-and': 1, "phillippe's": 1, 'puppets': 1, 'icing': 3, 'dream-': 1, 'bret': 1, 'fabrications': 1, 'captained': 1, 'yarns': 1, 'paddy': 1, 'aped': 1, 'resigns': 1, 'chevalier': 1, 'patially': 1, 'ratty': 1, 'embarass': 1, 'vukovich': 1, '98': 2, 'comedic-romance': 1, '1773': 2, 'rina': 1, 'feasible': 2, 'socially': 8, 'christopher': 30, 'wrestling': 2, 'sickening': 2, 'mast': 1, 'menaced': 1, 'greenlit': 1, 'uncommonly': 2, 'suverov': 1, "juliet's": 2, 'post-': 1, 'parody': 27, 'persona': 23, 'painfully': 11, 'legend': 28, 'cops-on-the-trail-of-serial-killer': 1, "tandy's": 1, 'blanco': 1, 'colonization': 3, "tibet's": 1, 'react': 13, 'character-oriented': 1, '911': 2, 'rightful': 4, 'hackneyed': 7, 'charact': 1, 'capitol': 1, 'applicant': 1, 'psychos': 1, 'weariness': 2, 'fully-realized': 2, 'pseudo-morbidity': 1, 'courthouse': 1, 'groom': 6, "coalwood's": 1, 'shelter': 3, 'driods': 1, 'blame': 22, "jerk'": 1, 'chun': 1, 'logo': 3, '1991': 11, 'obscene': 2, 'mobutu': 2, 'battleship': 3, 'hershman': 1, 'grossed': 3, 'travolta': 20, 'ambulance': 4, 'earthlings': 3, 'swarthy': 1, 'non-drinking': 1, 'favorably': 1, 'conners': 1, "se'": 1, 'types': 20, 'greene': 3, 'thoughtfulness': 2, 'flatly-written': 1, 'saunders': 1, 'individualism': 3, 'whooshed': 1, 'overstuffed': 1, 'rain': 14, 'drunkenness': 1, 'process': 53, 'private-eye': 1, 'smart-aleck': 1, 'carrot': 4, "prince'": 1, 'fembots': 1, 'clue': 13, 'candidate': 4, "spencer's": 1, 'keel': 1, 'drug-induced': 2, 'lawyer/fixer': 1, "'nikita'": 1, 'unforgivable': 2, 'signatures': 1, 'writings': 1, 'frenzy': 6, 'unspoken': 8, 'prosecution': 3, 'kinda': 9, 'live-in': 3, 'is--horror': 1, 'navigating': 2, 'razor-sharp': 1, 'conjugated': 1, 'jocular': 1, 'assessment': 4, "de'ath": 1, 'kiyoshi': 1, 'crooked': 5, 'mann': 8, 'boatloads': 1, 'believable': 62, 'lot': 229, 're-titled': 1, 'jiff': 2, 'arising': 1, 'jumping': 9, 'wash-outs': 1, 'automobiles': 2, 'expurgating': 1, 'sketchy': 3, 'cleaner': 2, "`n'": 1, 'silky': 2, 'disastrous': 7, 'well-timed': 4, 'hallucinations': 5, "o'": 6, "pertwee's": 1, "kuwait's": 1, 'beleives': 1, 'driveway': 1, 'unsurprising': 1, 'snooty': 1, 'giant-screened': 1, 'misirables': 1, 'shepard': 1, 'derelict': 3, 'anglo-saxon': 1, 'gregory': 10, 'disposable': 4, 'schrieber': 1, 'assination': 1, 'milton': 4, 'licence': 1, 'wordless': 2, 'dilapidating': 1, 'overemphasizing': 1, 'pursues': 2, 'neutral-coloured': 1, "duvall's": 5, 'one-hour': 1, 'rulers': 4, 'dream': 66, 'panning': 1, 'drug-ravaged': 1, 'effeminate': 2, 'themed': 1, 'implant': 3, 'tara': 5, 'concludes': 6, 'fundamentally': 3, 'coterie': 1, 'deem': 3, 'raved': 1, 'aberration': 1, 'famously': 1, 'power': 102, 'ultimately': 62, 'straying': 2, 'daredevil': 1, 'rehearsed': 2, 'noticably': 2, 'wild-idea': 1, 'syringe': 1, 'despair': 15, 'http': 6, 'faustian': 1, 'roughest': 1, 'haygarth': 1, 'chronicle': 1, 'missouri': 1, 'frightfulness': 1, 'brilliantly-construed': 1, 'damsel-in-distress': 1, 'specially': 1, 'retroactive': 1, 'rites-of-passage': 1, 'bewildering': 3, '(': 852, 'newsweek': 4, 'photograph': 2, 'murderer': 20, 'assasinated': 1, 'veterans': 3, 'izzard': 1, 'konner': 1, 'muscular': 4, 'dali': 1, 'dickens': 5, 'spastic-but-friendly': 1, 'credits': 75, 'access': 7, 'unhappily': 1, 'outfits': 6, 'feel': 178, 'interpreting': 1, 'benefit': 9, "60's": 3, 'fry': 5, 'spry': 1, 'for--cooking': 1, 'expatriate': 2, 'village': 16, 'wholesome': 7, 'bowden': 3, 'stud': 4, 'kickass': 1, 'newsradio': 1, 'gleeson': 3, 'backdrop': 20, 'undergo': 2, 'reincarnation': 1, 'thuggish': 3, 'witherspoon': 11, 'skg': 2, 'pleasing': 9, 'remarked': 3, '20somethings': 1, 'unwinable': 1, 'deviance': 1, 'daly': 2, 'sheets': 3, 'galloping': 1, 'scratch': 7, 'rehabilitation': 1, 'javier': 1, 'nasties': 1, 'elm': 4, 'bagpipe': 1, 'harmonious': 1, 'immediacy': 1, "jeremy's": 1, 'mercy': 4, 'hurdle': 2, 'embodied': 3, 'leopard': 2, 'healy': 2, 'supernatural-entity-falls-in-love-with-girl': 1, 'modelled': 1, "bassinger's": 1, "madisons'": 1, 'strengthens': 2, 'ramis': 4, 'urinates': 1, 'wilkinson': 7, 'aboriginals': 1, 'near-uproar': 1, 'kate': 22, 'symptomatic': 2, 'midler': 1, 'laugh': 65, 'thought': 121, 'innuendoes': 1, 'masturbates': 1, 'cryo-freeze': 1, 'pervasive': 2, 'compulsory': 3, 'regulation': 1, '1989': 6, 'let-down': 1, 'nino': 3, 'pre-feminist': 1, 'mask': 14, 'kimble': 1, 'captured': 34, 'huerta': 1, 'route': 17, 'frost': 4, 'slutty': 3, 'orientation': 3, 'thomas': 29, 'regretful': 2, 'hastily': 2, 'rear': 3, 'kristen': 5, "bureau's": 1, 'bothered': 2, 'strays': 4, 'adelaide': 1, 'angrily': 1, "mile'": 1, 'barking': 2, 'tentacles': 2, "evelyn's": 1, 'anjelica': 2, 'sammo': 1, "walsh's": 1, 'top-billed': 2, 'jocky': 1, 'empathic': 1, '`mrs': 1, 'maligned': 1, 'epperson': 1, 'gamble': 4, 'interogation': 1, 'precepts': 1, 'convolutedly': 1, 'karyo': 1, 'tedious': 8, 'wallace': 9, 'yulin': 1, 'portrayal': 55, 'understates': 1, 'lughnasa': 2, 'predicated': 1, 'warns': 15, 'student/teacher': 1, 'photographer-': 1, 'laboured': 1, 'adefarasin': 2, 'babs': 2, 'churn': 3, 'effectiveness': 5, 'injuries': 1, 'semi-autobiographical': 1, 'printed': 3, 'managing': 3, 'inundated': 4, 'organize': 1, 'hound-dog': 1, 'fact--that': 1, 'chose': 11, 'varsity': 5, 'well-lit': 1, '1993': 18, 'wrench': 1, "people's": 18, "woolf's": 1, 'eyre': 1, 'who-cares': 1, 'folded': 1, 'sales': 5, 'injects': 2, 'neverending': 1, "'beep'": 1, 'demy': 1, 'plaudits': 1, 'gasped': 1, 'promises': 15, 'onanism': 1, 'concentrations': 1, "louisiana's": 1, 'bitchy': 1, 'favour': 7, 'glamor': 1, 'well-equipped': 1, 'moralizing': 3, 'dahl': 4, 'camerons': 1, 'vigo': 1, 'jolt': 2, 'fickle': 2, 'joints': 2, 'botch': 2, 'bouncing': 2, 'heartless': 6, 'hitchhiking': 1, 'nailing': 2, 'emira': 2, 'unspecified': 2, 'megaglomaniac': 1, 'hopping': 2, 'malachy': 1, 'rothhaar': 1, 'eventful': 2, "fiance'": 1, 'annie': 14, 'hadden': 3, 'pests': 2, 'bodies': 24, 'benning': 1, 'isaac': 2, 'bean': 4, 'firebird': 2, 'airport': 13, 'liquor': 6, 'pike': 2, 'slanted': 1, 'ambition': 10, 're-tried': 1, "'sunk'": 1, 'held': 26, 'movie': 656, 'unceremoniously': 4, 'chimes': 1, 'unscary': 1, '800-mile': 1, 'joaquin': 4, 'warner-brothers-coyote-fall-off-the-cliff': 1, "jenny's": 1, 'tapes': 5, 'growing': 23, 'edge': 51, 'mamet': 6, 'sergio': 4, 'writing': 66, 'engages': 9, 'serenity': 1, 'actuality': 5, 'nurtures': 3, 'cochran': 1, 'aircraft': 3, "creature's": 1, 'buying': 15, "boxer's": 1, 'motherland--one': 1, 'recruits': 8, 'bind': 4, 'reverence': 1, 'create': 84, 'petrovsky': 1, 'kang': 1, "legend'": 2, 'infant': 3, 'minorly': 1, 'tv2': 2, 'night-club': 1, 'wingman': 1, 'description': 14, "she'd": 6, '1946': 1, 'spaceflight': 1, "'nuff": 2, 'slap': 7, 'addendum': 1, 'platitudes': 1, '-': 227, 'project': 44, 'zucker/abrahams/zucker': 1, 'jerk': 10, 'birth': 18, 'campus': 7, 'gang': 30, '`enemy': 1, 'lifelike': 1, 'soy': 1, 'act': 103, 'marking': 1, 'coli': 1, 'booted': 1, 'adhesive': 1, 'shrugging': 1, 'zuko': 2, 'fc': 1, 'death-row': 1, 'alc': 1, 'well-developed': 4, 'staggers': 2, 'deceptions': 2, 'prizes': 2, 'intrinsic': 1, 'cosmic': 2, 'boggles': 1, 'assets': 2, 'spokesman': 2, 'hamlet': 8, '101-year-old': 1, 'contract': 11, 'bogmen': 1, 'cookie-cutter': 2, 'emotions--awww': 1, 'moonshine': 1, 'gripes': 1, 'roache': 1, 'snodgress': 1, 'ignore': 11, 'buildup': 5, 'emissions': 1, "greenwald's": 1, 'brosse': 1, 'askew': 6, "murderer's": 1, 'damon': 26, 'often-told': 1, 'decadence': 2, 'grizzly': 1, 'reshooting': 1, "alzheimer's": 2, 'vivien': 1, 'emphasizes': 4, 'supplicate': 1, 'carnage': 3, 'nixon': 6, 'ham': 2, 'complicating': 7, "'copter": 1, 'disapproval': 1, 'feels': 92, 'battered': 4, 'jeroen': 1, 'oded': 1, 'tired': 23, 'enfield': 1, 'inferno': 2, 'bilingual': 1, 'isaacman': 2, 'agency': 8, 'leone': 3, 'drifts': 6, 'hooded': 3, 'blessing': 4, "graham's": 4, 'pile-up': 1, 'actresses': 20, "proyas'": 1, 'thrives': 2, 'terrifically': 5, 'unknowns': 4, 'sacrifices': 3, "palma's": 2, 'mature': 31, 'tyson': 1, 'rhythmless': 1, 'fortress': 4, 'causing': 8, 'not-so-practical': 1, 'prowse': 2, 'speakeasy-type': 1, 'anthropologist': 1, 'croc': 1, 'paradigm': 2, "frickin'": 1, 'subjectivity': 1, 'super': 14, 'lanes': 1, 'immunity': 1, 'vieluf': 2, 'careful': 11, 'delivered': 16, 'jolts': 2, 'advised': 4, 'mid-west': 1, 'fiesta': 1, 'cowrote': 1, 'kung-fu': 5, 'cites': 1, 'jehan': 1, 'self-pity': 3, 'kensington': 1, 'interrogating': 1, 'perjuring': 1, 'hermetic': 1, '34th': 1, 'assure': 6, 'biopic': 4, 'proposition': 3, 'harris': 30, 'ship': 43, 'experess': 1, 'surrender': 1, 'gangbangers': 1, 'sending': 6, 'riffing': 1, 'soon-to-be': 1, 'satisfyingly': 3, 'kidnapped': 16, "church's": 1, 'belone': 1, 'tip-toe': 1, 'vegetarianism': 1, 'masterwork': 2, 'patients': 9, 'rhythm': 5, 'hobbling': 1, 'sorrow': 7, 'dispare': 1, 'prayed': 1, 'regimes': 3, 'illena': 1, 'buy': 47, 'gills': 1, 'kaminsky': 1, 'gritty': 15, 'elaborate': 24, 'cici': 1, 'constituency': 1, 'skims': 1, 'cranky': 3, 'furs': 1, 'heywood': 1, 'footages': 1, 'rong': 1, "body's": 3, 'intelligent': 88, 'pea': 2, 'weave': 4, "festival's": 1, 'zemekis': 1, 'trait': 5, 'faint': 2, "bastard'": 1, 'balliol': 1, '11-year-old': 2, 'granted': 22, 'satirizing': 1, 'remarkable': 42, 'mussenden': 1, 'scot': 1, 'retell': 1, 'banter': 12, 'pedal': 1, 'post-torture': 1, 'raised': 30, 'australia': 8, 'realise': 4, 'theories': 6, 'means': 84, 'hailed': 8, 'foes': 2, 'contemptuous': 2, 'reminscing': 1, 'hats': 6, "time's": 1, '15-16': 1, 'uncharismatic': 1, 'dreariness': 1, 'emmylou': 1, 'liberation': 4, 'mangled': 1, 'by-passes': 2, 'guardians': 4, 'votes': 1, 'teaser': 1, 'urbaniak': 1, 'nominated': 15, 'hut': 1, 'border': 4, 'hannibal': 1, 'pretty': 147, 'kilborn': 1, 'stupidest': 1, 'ecker': 1, 'multi-level-marketing': 1, '21a': 1, 'firth': 1, 'half-full': 2, 'brim': 2, 'permanetly': 1, 'coufax': 1, 'unimaginatively': 1, 'legion': 1, 'commercial': 20, 'collect': 10, 'flashing': 5, 'ad-libs': 2, 'explores': 19, 'straight-to-the-cutout-bin': 1, 'demand': 6, 'downs': 6, 'exclusively': 10, 'whistles': 1, 'tampons': 1, 'city-street': 1, "weaving's": 1, "'n": 5, 'speer': 1, 'chain-smoking': 1, 'advances': 9, 'epiphanies': 1, 'prwhen': 1, "swinton's": 1, 'ascend': 2, 'carried': 23, 'wheel': 8, 'mm': 3, 'annoyance': 1, 'gilmore': 6, 'helmer/scripter': 1, 'insistence': 5, 'absurd': 15, 'ballet': 5, 'avi': 1, 'fictional': 15, 'morquio': 1, 'parrish': 1, 'publishes': 1, 'throw': 34, 'enhance': 9, 'carlin': 4, 'exterminate': 3, 'intertwining': 2, 'mourning': 2, 'functions': 7, 'invariably': 2, '1970s': 16, 'futile': 2, 'wasted': 14, 'suggestive': 2, 'devolve': 1, 'swelling': 1, 'merriment': 1, 'trash-talking': 1, 'proposing': 3, 'sophomore': 5, 'philadephia': 1, 'obnoxiously': 2, 'vigilantes': 3, 'ritualistic': 2, 'echoes': 13, 'lent': 3, 'shaggy': 1, 'cocktails': 1, 'barely-teen': 1, 'keen': 11, 'bushido': 1, 'horrors': 10, 'contantly': 1, 'fiercely': 1, 'divinely': 1, 'hulk': 2, "o'bannon": 1, 'sullivan': 5, 'linked': 5, 'scorching': 2, "sarossy's": 2, 'endured': 7, 'portia': 1, 'godfrey': 1, 'hide': 9, 'oops': 1, 'prestigious': 4, 'contribution': 6, 'retail': 1, 'orient': 1, 'doodyish': 1, 'trippier': 1, 'birdie': 1, 'thug': 5, 'methods': 16, 'beaumarchais': 1, 'denise': 11, 'stop-motion': 1, 'family-style': 1, '53': 2, 'princes': 1, 'apes': 5, 'clone': 8, 'transferred': 3, 'mailer': 1, 'displaced': 1, 'lifestyles': 3, 'federation': 11, 'publishing': 3, 'shooting': 35, 'ed': 33, 'combination': 30, 'rockwell': 2, 'tsi': 1, 'surpasses': 3, 'midriff': 1, 'pale-faced': 2, 'uncharted': 1, 'poodle': 1, 'vortex': 2, 'insists': 12, 'horrifying': 16, 'revelation-heavy': 1, 'heartbroken': 1, 'colonial': 3, 'corruption': 13, 'rising': 19, 'scientology': 1, 'kittens': 1, 'up-and-coming': 3, 'prototypical': 2, 'props': 5, 'slapdash': 2, 'alleviate': 1, 'maintaining': 14, 'barlow': 1, 'samantha/charly': 1, 'movies/tv': 1, 'refuses': 31, 'dogs-style': 1, 'proxima': 1, 'giuseppe': 4, 'distinctively': 3, 'pragmatic': 3, 'lurking': 6, 'alex': 24, 'fanatical': 4, "huston's": 2, 'diagnosed': 2, '#7': 1, 'co-pilot': 1, 'comparatively': 1, 'trash-talks': 1, 'wrong-doings': 1, 'max': 22, 'balance': 21, 'harsher': 2, 'voicework': 1, 'machinegun': 1, 'tati': 1, 'drew': 18, 'toddler': 1, 'gynecologist': 2, 'transformation': 17, 'contradictory': 2, 'forte': 1, 'medical': 16, 'relations': 6, 'experts': 2, 'replaced': 20, 'devilish': 7, 'finesse': 2, 'unarguable': 1, 'entrance': 12, 'seahaven': 4, 'feature': 92, 'triumph': 23, 'vices': 2, 'adroitly': 5, 'perceive': 4, 'demise': 16, 'cagily': 1, 'dedicate': 1, 'stride': 7, 'covering': 15, 'plummer': 4, 'motives': 19, 'troops': 10, 'forbade': 1, 'didya': 1, 'replete': 6, 'hai': 1, 'fit': 35, 'longs': 4, 'sidewalks': 1, 'job': 233, 'vidal': 1, 'babies': 4, 'angelic-appearing': 1, "earth'": 1, 'atkinson': 2, 'ricky': 7, 'roll': 29, 'baptism': 1, '_the_broadway_musical_': 1, 'gun-dealer': 1, 'kindergartners': 1, 'non-linear': 6, 'spider-man': 1, "harry's": 4, 'skull-faces': 1, 'albums': 2, 'blackening-heart': 1, 'dammit': 1, 'enjoys': 11, 'amendment': 2, 'detroit': 7, 'vehicle': 16, 'appearing': 14, "stepahne's": 1, 'oversized': 3, 'montero': 1, "florida's": 2, 'entrusting': 1, 'opened': 23, 'self-loathing': 2, 'expect': 88, 'dwarf': 1, 'encapsulates': 2, 'appliances': 2, 'scattering': 1, 'haunt': 10, 'albanian': 1, 'forms': 20, 'snorts': 1, 'sleeves': 1, 'eisner': 2, 'encounters': 24, 'producers': 27, 'psyche': 8, 'meany': 1, "owen's": 2, 'high-pitched': 3, 'freud': 2, '90s': 12, 'styled': 1, 'humanity': 33, 'bootleg': 2, 'discarding': 1, '2001': 19, 'afoot': 2, "maguire's": 1, 'high-strung': 4, 'commend': 2, 'impervious': 1, 'iowa': 5, 'savagery': 1, 'interrupt': 4, 'occupied': 3, 'dollars': 29, 'fabrizio': 1, 'grad': 1, "man's": 31, 'einstein-level': 1, 'british': 57, 'weakening': 1, "'if": 1, 'ms': 21, 'bandanna': 1, 'apprehensively': 1, 'induced': 4, 'eloquent': 1, 'devotion': 2, 'ironies': 2, 'pertains': 2, 'bambi': 1, 'exaggerated': 10, 'candid': 2, 'dig': 6, 'bastad': 1, 'dissapointment': 1, 'brings': 110, 'assembling': 2, 'reccurs': 1, 'bulletproof': 1, 'diverse': 13, 'close-knit': 3, 'low-budget': 4, "everyone's": 11, "minute's": 1, 'perched': 1, 'sliding': 8, 'endoskeleton': 1, 'echelons': 2, 'get-go': 2, 'bohemia': 1, 'prison': 53, 'valkenburgh': 1, 'meaney': 2, 'shout': 6, 'romanticized': 2, 'convictions': 5, 'brushing': 1, 'sinch': 1, 'individual-underdog': 1, 'indicting': 1, 'mary': 37, 'bare-footed': 1, 'cruising': 1, 'hellbent': 1, 'dogs': 23, 'shirtless': 1, 'advertised': 4, 'moonlighted': 1, "author's": 4, 'post-apocalypse': 1, 'nonentity': 1, 'demonstrate': 12, 'sheepish': 1, 'capt': 5, 'gretchen': 6, 'moulin': 1, 'tribune': 1, 'wisely': 19, 'impish': 3, 'broadsword': 1, "vincent's": 4, 'confidential': 8, 'robbie': 11, 'creaky': 5, 'peaks': 5, 'duration': 6, 'prosperity': 2, 'employed': 5, 'witnesses': 9, 'finger': 5, 'interpretation': 18, 'glorified': 7, 'unconventional': 7, 'applicable': 2, "night'": 1, 'post-world': 1, 'laughing': 43, "`lola'": 1, 'discontented': 1, 'manifested': 4, 'shaken': 4, 'dwarfs': 1, 'transylvania': 3, 'movie-making': 3, 'odds-and-ends': 1, 'launches': 3, 'stems': 3, '$7000': 1, 'privy': 4, 'huff': 1, 'high-profile': 10, 'couch': 8, 'epilogue': 8, 'guilt': 20, 'unwieldy': 1, 'josu': 1, 'boring': 39, 'prescott': 6, 'dramatics': 3, 'internalized': 1, 'writerly': 1, 'glengary': 1, 'trough': 1, 'peaceful': 17, 'outwardly': 3, 'century': 48, 'courteous': 1, 'glad': 20, 'wolf-beiderman': 1, 'construct': 3, 'shootings': 4, 'arrest': 8, 'aides': 1, 'golfer': 1, 'judging': 11, 'belted': 1, 'greasy': 2, 'embrace': 7, 'inactive': 1, 'arkham': 1, 'imbue': 2, 'risk': 27, 'au/~koukoula/': 1, 'unmistakably': 2, 'tirelessly': 1, 'surehanded': 1, 'finn': 4, 'timeline': 3, 'dissatisfied': 1, 'captivity': 3, 'frontiers': 1, 'neurotically-charged': 1, 'hard-edge': 1, 'eloquence': 2, 'bagged': 1, 'exterior': 6, 'victim': 42, "public's": 1, "detroit's": 1, 'devereaux': 3, 'virtual': 8, 'portable': 1, 'meager': 1, 'expertly': 8, 'glimpse': 12, 'bees': 1, 'south': 33, 'impalement': 1, 'precluded': 1, 'camelia': 1, 'revitalizing': 3, 'forebodings': 1, "smiths'": 1, 'appealed': 1, 'diffused': 1, 'singleton': 3, 'geographic': 1, "nikki's": 1, 'actualisation': 1, 'evoking': 1, 'rupaul': 1, 'succinct': 1, 'szubanski': 1, 'gist': 2, "gershwin's": 2, "joss's": 1, 'blanks': 3, 'triple-crosses': 1, 'farming': 2, 'fooled': 5, 'chidduck': 1, 'ultra-low': 1, 'reply': 4, 'philistine': 1, 'gassner': 2, 'cherokee': 2, 'accuse': 1, 'cerebrally': 1, 'exceeds': 4, 'peeled': 2, 'figures': 30, 'astronomy': 2, 'nipple': 1, 'stripped': 9, "odin's": 2, 'menges': 1, 'less-than-stellar': 1, 'dominant': 1, 'structure-wise': 1, 'inadvertently': 3, 'octopus': 1, 'assorted': 4, 'obtrusive': 1, '95': 4, 'leeanne': 1, 'roadrunner': 1, 'iced': 2, "cheese'": 1, "gordo's": 1, 'hughes': 10, 'traffic': 9, 'threesome': 1, 'terrifying': 19, 'mediums': 4, 'com/hollywood/academy/8034/': 1, 'orsen': 1, 'natural': 57, 'hancock': 1, 'taggart': 1, 'activated': 1, "''its": 1, 'kurt': 5, 'braun': 1, 'molecules': 1, 'jamaican': 5, 'hambling': 1, "donnie's": 1, 'corleones': 1, 'movie-lover': 1, 'ages': 21, 'glean': 1, "news'": 1, 'worth': 131, 'cromwell': 9, "paradine's": 1, 'finale': 37, 'halloween': 12, "glass'": 1, 'hollywood-conventional': 1, 'feat': 11, 'tenets': 1, 'dollop': 1, 'organic': 2, 'upper': 24, 'screw': 9, "minds'": 1, 'temporary': 2, 'drone': 2, 'personnel': 1, 'scandal-ridden': 1, 'well-worn': 1, 'brazillian': 1, 'emotionally-scarred': 1, 'klendathu': 1, "whol'o'": 1, 'lagoon': 1, 'bombshells': 1, 'mad': 22, 'fischer': 3, "music'": 1, 'numbskulls': 1, 'compendium': 1, "marsellus'": 1, 'lax': 2, 'affectionately': 3, 'distaste': 1, 'carolyn': 2, 'snidley': 1, 'inually': 1, 'restrictions': 1, 'colloquialisms': 1, 'debacle': 4, 'point-blank': 1, 'distinguishable': 1, 'delivers': 65, 'eiffel': 1, 'lip-sync': 1, 'extremists': 1, 'chabert': 1, 'diverted': 1, 'dinosaurs': 4, 'relatives': 4, 'close-minded': 1, '5-year': 1, 'full': 131, 'interrupted': 8, 'protectors': 4, 'bats': 2, 'ramses': 1, 'capra': 6, 'deterioration': 2, 'ruin': 14, 'centerfold': 1, 'seduces': 3, 'upside-down': 1, 'visit': 35, 'fables': 2, "grossy'": 1, 'parched': 1, 'ethically': 1, 'tucker': 15, 'flower-shop': 1, 'fish': 18, 'mensch': 1, 'unnoticed': 4, 'navigated': 1, "if'": 1, "jay's": 2, 'skillfully': 12, 'discount': 4, 'liza': 1, 'liddle': 1, 'violins': 2, 'nuns': 1, 'newtons': 1, 'horrendously': 1, 'dwarfed': 1, 'althea': 2, 'godfather-trilogy': 1, 'yahoo': 1, "ringwald's": 1, 'doddering': 1, 'uttered': 7, 'typify': 1, 'su': 1, 'tinges': 1, 'fundraiser': 1, 'visually': 51, "pipeline's": 1, 'sex-driven': 1, 'tapestry': 2, 'tzi': 1, 'extra-marital': 1, "peter's": 2, 'roads': 5, 'matthieu': 1, 'turbo': 1, 'salerno': 1, 'batallion': 1, 'fouls': 1, 'shone': 1, 'daylight': 8, 'reid': 8, 'protagonist': 32, 'thuroughly': 1, 'electric': 8, 'bourgeois': 2, "melvon's": 1, 'dogtags': 1, 'cursed': 3, 'addictively': 1, 'providence': 2, 'threesomes': 1, 'dogmatic': 2, 'plots': 14, 'brett': 5, 'slow-burn': 1, 'mastery': 7, 'tree--a': 1, 'entirety': 7, 'frivolous': 3, 'seng': 1, 'bittersweet': 4, 'sublime': 4, 'educes': 1, 'empty-headed': 1, "meyers'": 1, 'deliciously': 16, 'regiment': 2, 'varden': 1, "grant's": 3, 'all-purple': 1, 'gets-off': 1, 'protestations': 1, 'heretofore': 3, 'paraphrased': 1, 'takeoff': 2, 'steely': 1, 'renounced': 1, 'nutsy': 1, 'cuban': 5, 'romantically': 5, 'loads': 7, 'jip': 1, 'casted': 1, 'damnedest': 1, 'deniro': 8, 'economized': 1, 'officer': 45, 'fisburne': 1, 'boisterous': 1, 'january': 11, "starbuck's": 1, "jo's": 2, 'sigh': 5, 'primary': 20, 'fulfill': 14, 'to-hell-with-the-system': 1, 'delilah': 1, 'clyde': 5, 'posters': 6, 'polishing': 1, 'character-developing': 1, 'bresson': 1, 'pin-up': 1, 'stratum': 2, 'sanctioned': 2, 'plans': 38, 'sweet-faced': 1, 'well-being': 1, 'masterminded': 1, 'muriel': 1, 'straight-ahead': 1, 'entrusted': 2, 'caravaggio': 1, '1940s': 3, 'strung': 3, 'delicacies': 2, 'long-lost': 1, 'predictions': 3, "wharton's": 1, 'mythological': 2, 'mystifying': 1, 'horrocks': 2, 'borrowings': 1, 'tots': 1, 'handheld': 3, '-looking': 1, 'warned': 7, 'crab': 1, 'cities': 14, 'expunging': 1, 'svengalian': 1, 'lawrence': 16, 'revolves': 22, "zallian's": 1, 'villainess': 1, 'retrospective': 3, 'heroin': 7, 'survives': 8, 'fx': 2, 'hound': 4, 'pecker': 1, 'triumphing': 1, 'apricots': 1, 'derivative': 5, 'droid': 2, 'particular-': 1, 'appalachian': 1, 'obnoxious': 17, 'finest': 42, 'exchanging': 3, 'culkin': 4, 'giardello': 1, 'holly': 4, 'truce': 2, 'guernica': 1, 'newsletter': 4, 'clay': 2, 'springy': 1, 'struggles': 19, 'deck': 8, 'promote': 3, 'abberline': 1, 'clad': 3, 'fucked-up': 1, 'feisty': 4, 'tremors': 1, 'closemindedness': 1, 'escapes': 17, 'admirably': 12, 're-telling': 1, 'claw': 1, 'wright': 9, 'moat': 1, 'fade': 4, 'teague': 1, 'detained': 1, 'photojournalist': 1, 'down-trodden': 1, 'oldman': 4, 'burton': 15, 'physical': 43, 'terrorism': 5, "haskin's": 1, 'dons': 3, 'sciences': 3, 'plausible': 19, 'inherently': 8, 'kinds': 17, 'acquaintance': 2, 'seminars': 1, 'observing': 4, 'soften': 2, 'opportunities': 12, 'memory-diminishing': 1, "spadowski's": 1, 'recommending': 5, 'coarse': 1, 'squabbles': 5, 'advanced': 11, "bus'": 2, 'reign': 7, 'utilizes': 3, 'misunderstood': 5, 'fronts': 3, 'rossi': 2, 'catastrophe': 2, 'buckle': 1, 'silberg': 1, 'tautness': 1, 'theoreticians': 1, 'overplays': 1, 'cow': 2, 'reunited': 4, 'real-time': 5, 'insiders': 1, 'grown-ups--this': 1, 'procreating': 1, 'stomp': 1, 'debated': 2, 'eldest': 6, 'thick-witted': 1, "'off'": 1, "willis'": 6, 'shadows': 13, 'fly': 26, 'boredom': 9, 'remind': 11, 'spotlight': 14, 'silverware': 1, 'pessimistic': 5, 'mound': 2, 'spoken': 24, 'enlightenment': 2, 'comedically': 2, 'whirlwind': 2, 'lagravenese': 2, 'isolated': 14, 'view': 78, 'spawns': 1, 'championships': 1, 'rachel': 9, 'madman-cum-detective': 1, 'exhilaration': 1, 'splendid': 7, 'freeman--is': 1, 'ex-fianc': 1, 'laughers': 1, 'reyes': 1, 'rationalize': 1, 'stoned': 3, 'godawful': 1, 'statements': 10, 'brakes': 2, 'post-post-feminist': 1, 'recommendation': 6, '========': 1, 'cheech': 3, "fuentes'": 1, 'remove': 9, 'nuttgens': 2, "baldwin's": 4, 'subtler': 3, 'donohue': 1, 'piles': 1, 'pollster': 1, 'not-bad': 1, 'cuba': 11, 'consul': 1, 'upped': 1, 'bannister': 1, 'expedition': 6, 'erased': 4, 've': 9, 'evoke': 6, 'autocrat': 1, 'skyscraper': 2, 'cui': 1, 'intention': 10, 'accounted': 1, 'bottom': 27, 'pink': 9, 'wittiest': 1, 'petty': 10, 'molten': 2, 'shyamalan': 6, 'yapping': 1, "resnais's": 1, 'hairdo': 2, 'trojan': 1, "ricci's": 2, 'impress': 16, 'concentration': 11, 'malroux': 1, 'mackey': 2, 'susie': 1, "frank's": 6, 'tempers': 1, 'scalpel': 1, 'odin': 2, 'harrisburg': 1, 'berates': 1, 'chutzpah': 1, 'child-hating': 1, 'fuck': 4, 'disrupting': 1, 'intercepted': 2, 'elastica': 1, 'subpar': 1, 'marie': 6, 'landscape': 14, 'adam': 14, 'rescuing': 5, 'sevigny': 4, "oprah's": 1, 'abides': 1, 'nationals': 1, 'emotionless': 5, "louise'": 1, 'spirituality': 6, 'rudi': 1, 'forerunner': 1, 'retinal': 1, 'salted': 1, "danson's": 1, 'charly': 1, 'celebrates': 3, 'wielded': 1, 'print': 9, 'up-and-comers': 1, 'pop-star': 1, 'revelations': 10, 'keeken': 1, 'spit-like': 1, "szabo's": 1, 'mindstretching': 1, 'collaboration': 7, 'pot': 11, 'agonies': 1, 'does--and': 1, 'equivolence': 1, 'wino': 2, 'quagmire': 2, 'funner': 1, 'middling': 2, 'lifeforms': 1, 'heath': 2, 'slovenly': 1, 'braveheart-dosage': 1, "mafia's": 1, 'clutches': 9, 'slut': 1, 'filmic': 2, 'gar': 1, 'impersonator': 1, 'adrift': 2, 'phrase': 12, 'leery': 1, 'hen-wen': 1, 'contestant': 2, 'superhuman': 6, 'pellets': 1, 'jumbo': 1, 'dany': 1, 'pointe': 4, 'modern': 63, 'gazzara': 2, "verbal's": 1, 'traumatised': 2, 'kingston': 1, 'inportant': 1, 'avary': 1, 'partnered': 1, 'cheerleaders': 3, 'lisbon': 1, 'sci': 2, 'undulating': 3, 'meteoric': 2, 'melodrama': 12, 'wacky': 11, 'mooch': 1, 'instincts': 8, 'women': 96, 'finkbine': 1, 'lizard': 2, 'idea--but': 1, 'entertaning': 1, 'establishing': 12, 'gaps': 5, 'daze': 1, 'hacking': 2, 'version': 105, 'notable': 22, 'prominently': 2, 'rapper/composer': 1, 'pseudo-asian': 1, 'skaro': 1, 'bewilderment': 3, 'incorrectly': 1, 'sometimes-odious': 1, 'atomic': 1, 'haliwell': 1, 'ists': 1, 'arrived': 10, 'humanitarian': 2, 'sixties': 4, 'enigmatic': 9, "jonas's": 1, 'poise': 3, 'romantic-comedy': 1, 'spaghetti': 1, 'lantos': 1, "gibson's": 8, 'hijinks': 1, 'fragile': 6, 'forgetable': 1, 'fought': 10, 'holnists': 1, 'lawns': 1, 'elevating': 1, '`rip-off': 1, 'phone-pitch': 1, 'worthwhile': 13, 'played': 248, 'wittiness': 1, 'tormenters': 1, 'starphoenix': 1, 'conservative': 9, 'fled': 2, 'sunbathing': 1, 'givya': 1, "margaret's": 3, '2058': 1, 'spaced-out': 1, 'shiavelli': 1, 'exasperated': 3, "beresford's": 1, 'auto-arousal': 1, 'macleane': 1, 'layoffs': 1, 'featherstone': 3, 'yip': 1, 'supplement': 3, 'duc': 1, 'pond-': 1, 'appointment': 2, 'orchard': 1, 'penned': 11, 'bishop': 4, 'commandeer': 2, 'harwich': 1, 'masculine': 2, 'exfoliating': 1, "reed's": 1, 'outdone': 1, 'hoods': 2, 'contradict': 1, 'workshop': 2, 'popping': 9, 'inhaling': 2, 'donat': 3, 'sympathized': 1, 'puppeteering': 1, 'narrative': 32, 'atrocious': 2, 'lts': 1, 'rewarding': 10, 'messages': 12, 'air-headed': 1, 'courtland': 1, 'icy': 6, 'phenomenon': 12, 'thigh': 2, 'quick-edit': 1, 'views': 26, 'wink': 5, 'earnest': 7, '`you': 1, 'surging': 1, 'yamazaki': 1, 'necklace': 3, 'comments': 23, 'stranger': 16, 'skies': 4, 'dolby': 4, 'marseilles': 1, 'hesitate': 5, 'kraup': 1, 'added': 46, 'hill--if': 1, 'aires': 1, "nosferatu's": 1, "tibbs'": 1, 'wincing': 1, 'non-existent': 2, 'mosaffa': 1, "hilary's": 1, 'cross-dressing': 2, 'bubby': 1, 'sheffield': 1, 'thomsen': 1, 'energetically': 1, 'revell': 1, 'badalucco': 1, 'devlish': 1, 'digger': 2, 'privates': 1, 'truest': 5, 'semi-dramatic': 1, 'not-to-be-underestimated': 1, 'sweden': 2, 'chronicling': 1, 'preponderence': 1, 'revenge': 37, 'full-condescension': 1, 'maximes': 1, 'frightfest': 1, 'injecting': 1, '_wag': 1, 'tuan': 1, 'kraft': 2, 'jodi': 1, "george's": 2, 'omitting': 1, 'chilling': 19, 'sweathog': 1, 'drinks': 7, 'scotch': 1, '`dark': 1, 'secretary': 7, 'cellular': 5, "boyle's": 3, 'maturation': 1, 'mother-in-law': 3, 'meyer': 7, 'dukas': 1, 'anti-heroes': 1, 'vampira': 1, 'delgado': 1, 'crux': 4, 'elegant': 9, 'urging': 4, 'vaughn': 7, 'confirmation': 3, 'death--our': 1, 'adds': 57, 'controls': 3, 'gretta': 1, 'regrets': 1, 'dispose': 2, 'lyonne': 1, 'nestled': 1, 'mcdiarmid': 1, 'believing': 13, 'cinematographers': 1, 'dodo': 2, 'hey-dey': 1, 'topps': 1, 'tent': 6, 'pissed': 3, 'complained': 2, 'unwritten': 4, 'gut': 5, 'ensue': 6, 'even-numbered': 1, 'anticlimaxic': 1, "nights'": 1, "confidential''": 1, 'fill': 30, 'llama-herder': 1, 'ex-agent': 1, 'word/jazz': 1, 'proliferation': 2, 'bullion': 1, 'checking': 6, 'sweetheart': 7, 'supposedly': 25, 'rebuild': 1, 'deceiver': 1, 'machette': 1, 'picturing': 1, 'gags': 23, "betsy's": 1, 'comically': 3, 'inquires': 3, 'awoke': 3, 'watergate': 1, 'demeanour': 1, '`sea': 1, 'ninety': 5, 'dekay': 1, 'titans': 1, 'predict': 10, 'hydraulic': 1, 'yields': 6, 'eye-filling': 1, 'rainforest': 1, 'french-set': 1, 'trek-fan-only': 1, 'half-blurred': 1, 'casualness': 1, 'overcasts': 1, "girls'": 2, 'diminutive': 4, 'saddest': 1, 'suitor': 1, 'accessible': 10, 'reed': 8, 'virtuoso': 5, 'shoot-ups': 1, 'hjejle': 1, 'throwback': 4, 'detach': 1, 'unleashed': 1, 'bowfinger': 6, 'relocate': 3, 'averse': 1, 'weigh': 4, 'pot-induced': 1, 'kooky': 4, 'disposition': 2, 'supreme-bozo': 1, 'bloody--film': 1, 'victimized': 1, 'advancing': 3, 'hateful': 2, 'busy': 14, "moses's": 1, 'tyrone': 1, 'cookie': 3, 'hockey': 1, "ted's": 4, 'purist': 1, 'forbid': 1, 'noblest': 2, 'berridge': 1, 'typed': 3, 'glassworks': 1, 'chat': 3, 'digestive': 1, 'platinum-blond': 1, 'issue': 20, 'connect': 7, 'film-noir': 3, 'westernized': 1, 'sorely': 3, 'sharing': 5, 'rappaport': 3, 'ivan': 4, 'piers': 1, 'lynching': 2, 'belgian-owned': 1, 'ratings': 13, 'autobiography': 4, 'counselor': 10, 'payments': 2, 'dominates': 3, '98%': 1, 'phrases': 9, 'feuer': 1, 'audrey': 2, 'shag': 1, 'bullshit': 2, 'candidness': 1, "broadway's": 1, 'goldthwait': 1, 'crowbar': 1, 'unafraid': 3, 'rebound': 2, 'three-': 1, 'rewrite': 3, 'giallo': 1, 'beverages': 1, 'begining': 1, 'pile': 6, 'lolita': 3, 'drinking': 14, 'actions': 41, 'engaged': 23, 'terrorizing': 4, 'economics': 1, 'advocate': 4, 'violent': 54, 'dozing': 1, 'fringes': 1, 'stellan': 12, 'afflicted': 2, 'story--': 1, 'woody': 31, 'evidenced': 2, 'clashes': 4, 'haggard': 1, 'phones': 5, '1955': 1, 'rebel': 9, 'adlai': 1, "brown's": 1, 'film-within-a-film': 1, 'coke-addicted': 1, 'slaughterhouse-five': 1, 'trumpeting': 1, 'consequence': 6, 'oil': 7, 'conclusion--the': 1, 'action/suspense': 3, 'corporal': 1, 'ghosts': 14, 'widescale': 1, "picard's": 1, 'adolph': 1, 'appropriateness': 1, 'illnesses': 1, 'obituaries': 1, 'deride': 1, 'coming': 105, 'matron': 3, 'era': 46, 'dragging': 5, 'snow-shovel': 1, 'pates': 1, 'hotbed': 1, 'assuming': 6, 'marry': 23, 'mouse': 13, 'duck': 7, 'coolest': 2, 'strasberg': 1, "narrator's": 1, 'scent': 4, 'depalma': 2, 'scuba': 1, 'shenanigans': 2, 'egyptologist': 1, 'fair': 37, 'mid-east': 1, 'evaluation': 2, '1500s': 1, 'sample-mad': 1, 'creation': 16, 'actionfest': 1, 'murderous': 8, 't-1000': 1, 'episodes--patlabor': 1, 'humerous': 1, 'frankenstein': 9, 'awake': 8, 'hyena': 1, 'ludicrously': 1, 'takeover': 1, 'sip': 1, 'dummies': 1, 'dials': 1, 'pupil': 5, 'mesmerized': 2, 'murderers': 5, 'bizaare': 2, 'noni': 1, 'vivaldi': 1, "amadala's": 1, 'late': 86, 'vest': 1, 'commissioning': 1, 'nhu': 1, 'genetics': 2, 'translating': 4, 'event': 36, 'contributes': 12, 'arts': 22, 'smuntz': 1, 'supportive': 3, "bargain'": 1, 'stagey': 1, 'pimples': 1, 'sharpster': 1, 'costumed': 1, 'cooly': 1, 'dil': 1, '13th': 5, 'one-liners': 26, 'homages': 1, 'mortally': 1, 'referee': 2, 'documents': 1, 'sutton': 1, 'overlapping': 3, "culkin's": 1, 'yikes': 1, 'crusty': 1, 'manfully': 1, 'adjustment': 1, "cookie's": 1, 'sze-man': 1, 'below-average': 1, 'whomever': 4, 'fbi': 28, 'fortysomething': 1, '_shaft_': 1, 'jiji': 1, 'vividly': 7, 'sisco': 3, 'reprising': 4, "britain's": 1, 'supercop': 6, 'cohesiveness': 1, 'prince': 17, 'furnace': 1, 'humid': 1, 'clarkson': 1, 'galore': 10, 'horrendous': 3, 'unrealistic': 7, 'sighs': 1, 'isis': 1, 'bunch\x12': 1, 'forests': 4, "states'": 2, 'journey': 44, 'laboratory': 3, "elfman's": 3, 'conceit': 4, 'leans': 2, 'stages': 13, 'well-placed': 1, 'seymour': 5, 'astonishingly': 7, 'spreads': 7, 'halliwell': 1, 'clubbing': 1, 'multinational': 1, 'jazzy': 2, 'paired': 3, 'soulmate': 1, 'out-of-their-mind': 1, 'peach-colored': 1, 'bookend': 3, 'ironed': 1, 'scriptural': 1, 'songs': 39, 'kenny': 6, 'disillusioned': 4, 'enlist': 5, 'tripplehorn': 2, 'awaken': 1, 'team-up': 1, 'eager': 26, 'lapaglia': 1, 'inadequate': 1, 'overtime': 5, 'extensive': 10, 'shalhoub': 8, 'cheap-jack': 1, 'summation': 1, 'personal': 87, 'ticker': 1, "1995's": 6, 'screws': 5, 'amber': 3, "frances'": 2, 'fast': 51, 'matured': 2, 'trouble': 63, 'required': 23, 'unimaginable': 3, 'whales/triangle': 1, 'blowing': 13, 'emailing': 1, 'horror-mystery': 1, '1298': 1, 're-shoot': 3, 'extravagant': 3, 'subliminally': 1, 'disposing': 1, 'princess/ogre': 1, 'cramp': 1, 'particulary': 1, '12-year-old': 3, 'voltage': 1, 'rednecks': 3, "judge's": 1, 'markets': 1, 'liked-': 1, 'chant': 2, 'eight-track': 1, 'hiccup': 1, 'confuses': 1, 'oriental': 3, 'republicans': 1, 'sadism': 1, 'rave': 5, 'scored': 10, 'musketeers': 3, 'glitteratti': 1, 'side-tracks': 1, 'seat-of-the-pants': 1, 'phenomena': 9, 'alleviates': 1, 'desolate': 5, 'damning': 2, 'be-wigged': 1, 'untranslated': 1, 'amuse': 4, 'nelligan': 1, 'uniforms': 4, 'requiring': 3, 'nun': 1, 'filming': 18, 'evident': 13, 'vultures': 2, 'child-like': 3, 'hygienists': 1, 'houston': 2, 'cross-over': 1, 'screenwriting': 4, 'tycoon': 5, 'hit-list': 1, 'knack': 12, 'playboy': 7, 'encyclopedias': 1, 'wanderlust': 1, 'hero-': 1, 'unclothed': 1, 'fawcett': 4, 'explodes': 6, 'sundry': 1, 'bag': 26, 'saint-saens': 2, 'rya': 2, 'preliminary': 1, 'pestering': 2, 'fresa': 1, 'hypochondriac': 3, 'nenette': 1, 'seperate': 4, 'skilfull': 1, 'wishes': 18, 'connoisseurs': 3, 'stalingrad': 1, "mazzello's": 1, 'psychologists': 2, 'ransom': 12, "douglas'": 2, 'jutting': 1, 'mention': 44, 'letter_': 1, 'autopsy': 2, 'anger': 35, 'jeremy': 12, 'detractions': 1, 'center': 36, '_life': 2, 'narrators': 1, 'junkets': 1, 'philippe': 4, 'strand': 2, 'franks': 1, 'delves': 5, 'transitions': 3, 'dreams': 47, 'lucasfilm': 1, 'straight': 54, 'brit': 4, 'resilience': 1, 'disarming': 3, 'inbred': 1, 'dopey': 5, 'booked': 1, 'gibson': 14, 'in-between': 4, 'accompanied': 11, 'astray': 2, 'nope': 3, 'b----': 1, 'unattainable': 3, 'virtues': 13, 'yankee': 3, 'dirt-poor': 1, 'pent': 1, "auteur's": 1, 'rifles': 2, 'sheperd': 1, 'irritated': 2, 'battlefield': 6, 'overachievement': 1, 'perplexed': 3, "anthony's": 1, 'lhassa': 1, 'duddy': 1, 'compass': 1, 'nonense': 1, "stoker's": 3, 'mellow': 1, 'frank-n-furter': 1, 'patootie': 1, 'centres': 3, 'gradeschooler': 1, 'ra': 2, 'forthright': 1, 'witness': 30, 'advocated': 1, 'doggy': 1, 'revert': 1, 'smirkiest': 1, 'chloe': 3, 'austen': 1, 'working': 88, 'listless': 1, 'mass-market': 1, 'obtained': 2, 'thespians': 2, 'occupy': 1, 'momentarily': 1, 'quit': 10, 'god-forsaken': 1, 'kids-and': 1, '--this': 1, 'over-familiarization': 1, 'serra': 1, 'gamorreans': 1, 'unconsoling': 1, 'newt': 5, 'doyle': 1, 'wipes': 1, 'high-visibility': 1, 'crates': 1, 'appended': 1, 'permanently': 4, 'plummeting': 1, 'obsessed': 31, 'drug-running': 1, 'zippy': 3, 'companies': 6, "swingin'": 1, 'hospitality': 1, 'unofficial': 1, 'slasher-comedy': 1, 'dangers': 7, 'cheated': 9, 'st': 8, 'creepiest': 1, 'snowstorm': 1, 'rhetorically': 1, "steenburgen's": 1, 'unparalleled': 1, 'preclude': 1, 'browne': 1, 'eagerly': 3, 'jimmie': 2, 'homeland': 3, 'bible-thumper': 1, 'bat': 10, '122': 1, 'forrester': 2, 'evacuate': 4, 'waitresses': 3, 'niece': 4, 'alter': 9, 'nook': 1, "paxton's": 3, 'vessey': 1, 'american-chinese': 1, 'unduly': 2, 'black-and-white': 7, 'true-story': 1, 'swept': 6, 'adults': 38, 'colonized': 1, 'addressing': 2, 'probably-': 1, 'explorations': 2, 'umm': 1, 'mistake-free': 1, 'processing': 1, 'buscemi': 12, 'perverted': 3, 'decayed': 1, 'farmers': 3, 'confusing': 32, 'riddick': 2, 'spliced': 1, 'oswald': 1, 'jaye': 1, 'sexy': 23, 'field': 31, 'loftus': 1, 'yo-yo': 2, 'orser': 1, 'bros': 8, 'disowned': 1, 'tragically': 1, 'fulfilment': 1, 'penultimate': 3, 'asides': 4, 'cynics': 4, 'ouch--and': 1, 'residing': 1, 'krasner': 1, 'architect': 3, 'righteous': 2, 'albania': 1, 'terrestrial': 1, 'ntsc': 1, 'backbone': 6, 'almost-cameo': 1, 'considerations': 2, 'ahmad': 1, 'safe-keeping': 1, 'butte': 1, 'absolutes': 1, 'self-absorption': 1, 'dory': 1, 'unaffected': 1, 'staff': 14, 'brilliant': 99, 'cocky': 10, 'featuring': 40, 'abe': 1, 'seashell': 1, 'amply': 1, 'ounces': 1, 'inhibit': 1, 'presenting': 9, "critics'": 2, "bunny's": 3, 'housekeeping': 1, 'distinguishing': 2, 'favreau': 4, "soldier's": 2, 'elektra': 1, 'carvey': 2, 'gold-embroidery': 1, '21st': 4, 'barzoon': 1, 'doubles': 3, '1940': 3, 'territory': 14, 'cutout': 1, 'distracted': 9, 'ratio': 13, "burgess'": 1, 'initiated': 1, 'armies': 7, 'joust': 1, "malory's": 1, 'campbell': 24, "she's": 115, '$21': 2, 'rashomon': 1, 'affectionate': 5, 'clear-eyed': 1, 'outdoorsman': 1, 'errors': 2, 'materialistic': 6, 'drescher': 1, 'viscously': 1, 'mol': 5, 'obvious-looking': 1, 'harald': 1, 'bigscreen': 1, 'shucking': 1, 'gears': 6, 'fall-off': 1, 'subsequently': 6, 'interest': 87, 'honored': 1, 'celebrating': 8, 'clip': 4, 'marie-therese': 1, 'eyelashes': 3, 'wrecked': 3, 'gimp': 1, 'worf': 3, 'self-serving': 2, 'sirocco': 1, 'advocating': 2, 'infinite': 3, 'galactic': 4, 'intelligence': 47, 'catalog': 1, 'enacts': 1, 'wai': 3, 'substance': 24, 'students': 31, 'bitter': 26, 'tovoli': 1, 'dang': 2, 'narcissistically': 1, 'continents': 1, 'blanchette': 1, 'murch': 1, 'pants': 12, 'truthfulness': 1, "seinfeld's": 1, 'uncritical': 1, 'creeps': 10, 'crippled': 7, 'well-rounded': 4, 'absoloute': 1, 'enterprise': 6, 'brutish': 2, 'suicide': 24, 'strains': 7, 'realistically-written': 1, 'chyron': 1, 'minimal': 10, 'anarchists': 2, "coil's": 1, 'all-too-realistic': 1, 'wow-inspiring': 1, 'inconsistent': 5, 'rewriting': 2, 'underutilized': 1, "hoffman's": 3, 'gamut': 5, 'avenging': 1, 'pot-smoking': 1, 'ken-and-barbie': 1, 'lofty': 9, 'target': 38, 'blindfolded': 1, 'fast-paced': 11, 'difficulties': 6, 'notch': 15, "washington's": 5, 'strip': 12, 'iran': 1, 'cliffs': 2, 'immediately': 78, 'priveleged': 1, 'goddess': 1, 'zucker': 3, "cohen's": 1, 'lack': 87, 'internetters': 1, 'yugoslavian': 1, 'grammy': 1, 'tabloid': 5, 'instinct': 11, 'bodied': 1, 'discoverers': 1, 'viruses': 1, 'hammy': 5, 'implications': 6, 'side': 143, 'mantra-like': 1, 'spot-on': 1, 'beck': 4, 'wordsmith': 1, 'eclectic': 11, 'gi': 2, 'incarnation': 6, 'psychist': 1, 'remained': 9, 'fortunate': 6, 'c3po': 2, 'sprung': 2, 'bureaucrat': 1, 'wining': 1, 'note': 100, 'insurrection': 3, "neary's": 1, "1992's": 3, 'phoenix': 4, 'appealing': 36, 'england': 28, 'faith-': 1, "1994's": 4, 'slicked': 1, 'one-armed': 1, 'furiously': 1, 'happen': 78, 'awakened': 5, 'sizzling': 1, 'boddy': 1, 'countrysides': 1, 'charismatic': 19, 'rows': 2, 'inextricably': 1, 'reclining': 1, 'once-meek': 1, 'lifelessness': 1, "deming's": 1, 'mathematician': 2, 'distributing': 2, 'alexandria': 1, 'three-way': 2, 'hairstyle': 2, 'dj': 4, 'seaman': 1, 'sleeper/cult': 1, 'vera': 3, 'northeast': 2, 'dow': 1, 'turtle': 1, 'brannagh': 1, 'mortar': 2, 'wayward': 2, 'islamic': 2, 'televisions': 6, 'against-all-': 1, "cy's": 1, 'prior': 22, 'jumbled': 2, 'recklessly': 2, 'mea': 1, 'neurologist': 1, 'addict': 3, 'vivacious': 2, 'clever': 79, 'vi': 1, "1980's": 2, 'oncoming': 2, 'luis': 6, 'howard_': 1, 'individually': 1, 'bang': 3, 'challenges': 8, 'proximo': 2, 'silents': 1, 'nitpickers': 1, "'nam-inspired": 1, 'wars': 47, 'blade': 16, "rachmaninov's": 1, 'reverts': 2, "mama's": 1, 'personalized': 2, 'hopelessly': 10, "pistol's": 1, 'absolutist': 1, 'shores': 2, 'poombah': 1, 'wily-ness': 1, 'livens': 1, 'curtolo': 1, "scott's": 5, 'gleaming': 2, 'machiavellian': 2, 'industries': 3, 'hoodwink': 1, 'new--a': 1, 'gentler': 1, 'uninvited': 1, 'jonathan': 27, 'overtones': 7, 'eliminates': 2, 'scarring': 1, 'harding': 1, 'kolya': 1, 'negatively': 1, 'wise-': 1, 'all-grown-up': 1, 'party/rave': 1, 'rome': 7, 'minkoff': 1, 'monoko': 1, 'one-room': 3, 'yanks': 1, 'soil': 7, 'pee-wee': 2, 'rocked': 1, 'sufficient': 2, 'escaping': 13, 'hallmark': 3, 'catherine': 18, 'castor': 1, 'drool': 1, 'lending': 1, 'liscinski': 1, 'rugby': 1, 'professorship': 1, 'writing-his': 1, 'provokes': 3, 'comic-geeks': 1, 'mono': 3, 'mexican-american': 1, '/': 9, 'crossan': 1, '73': 2, 'precision': 7, 'purposeful': 5, 'bragg': 1, 'gashes': 1, 'quasi-campy': 1, 'tv-series': 1, 'top-of-the-line': 1, 'bottom-of-the-barrel': 3, 'pfieffer': 1, 'tootsie': 1, 'cherry': 6, 'stony': 1, 'dwindling': 2, 'grisly': 7, 'leprosy-stricken': 1, 'wartime': 4, 'tread': 5, 'curiousity': 1, 'synchronous': 2, 'eriksson': 1, 'compilation': 3, 'sunday': 9, 'stand-up': 1, 'rita': 6, 'shoddy': 1, 'sympathizer': 1, 'switched': 4, 'flores': 3, "whisperer's": 1, 'volcanoes': 1, 'codification': 1, 'nationally': 1, 'endorsing': 1, 'linking': 1, 'aahs': 1, 'anti-terrorism': 1, 'likeness': 1, 're-doing': 1, 'crue': 1, 'medium]': 1, 'receiving': 17, 'simulate': 1, 'relegating': 1, 'esoteric': 1, 'resembling': 6, '12-hour': 1, 'poked': 1, "'landworld": 1, 'dumb': 30, 'lighweight': 1, 'birthday': 13, 'zhivago': 1, 'underneath-': 1, "krippendorf's": 1, 'necronomicon': 1, 'gritted': 1, 'self-destructiveness': 2, 'brighter': 1, 'plug': 5, "bird's": 2, "cake'": 1, 'bugger': 1, 'pygmalion': 1, 'daswon': 1, 'effete': 1, 'basinger': 2, 'heal': 2, 'brat': 3, '_patlabor_': 1, 'johner': 1, 'ringing': 3, 'instructed': 3, 'worker': 21, 'superimposed': 2, 'touched': 10, 'checkout': 2, 'fated': 1, 'francie': 2, 'non-triple-x': 1, 'dethrones': 1, 'cycles': 1, 'oneself': 4, 'psycho': 16, 'thoughtfully': 2, 'retro-futuristic': 1, 'fellas': 2, 'reflect': 13, 'fe': 1, 'drawers': 2, 'one-joke': 3, 'excercise': 1, "_william_shakespeare's_romeo_+_juliet_": 1, 'cy': 1, 'parolees': 1, '60+': 1, 'anomaly': 1, 'frightens': 3, 'beethoven': 1, 'smooths': 2, 'deepening': 2, 'photography': 26, 'examining': 3, 'shagged': 3, 'tarnished': 1, 'comet': 3, 'quart': 1, 'quibble': 5, 'enormously': 6, 'vietnam': 14, 'optometrist': 1, 'theirselves': 1, 'baltimore': 1, 'chief-of-stafff': 1, "mickey's": 2, 'accommodating': 2, 'self-': 3, "'halloween'": 1, 'previously': 26, 'gifted': 8, 'shreds': 2, 'hijacker': 1, "taylor's": 2, 'she+s': 1, 'wanna': 5, 'burma': 4, 'eccentric': 30, 'evangelist': 1, 'dysfuntional': 1, 'pomegranate': 1, 'roberts': 13, 'hold': 57, 'darker': 15, 'battering': 1, 'neck-deep': 2, 'fortinbras': 1, 'sandra': 13, 'morse': 6, 'buren': 4, 'related': 18, 'ascribe': 1, '60s': 4, 'golden-age': 1, 'assault': 12, 'unrelenting': 2, 'radio/tv/film': 1, 'gordan': 1, 'over-whelming': 1, 'traveling': 12, 'poorer': 1, 'hostages': 3, 'unsuccessful': 4, 'processor': 1, 'half-assed': 1, 'proletarian': 1, 'record': 29, 'imaginable': 4, 'backed': 7, "spoon's": 1, 'patting': 1, "wachowski's": 1, 'directorial': 24, 'overshadow': 3, 'tagalong': 2, 'hbo': 5, 'ii-era': 1, "jamaica's": 1, "miracle'": 1, 'horizontally': 1, 'liman': 3, 'neutralize': 1, 'klingons': 1, 'decreed': 2, 'bizzare': 1, "dude's": 4, 'depressing': 16, 'elisa': 1, 'grass': 5, 'pseudo-intellectuals': 1, 'quivering': 1, 'evangelicals': 1, 'lindo': 5, 'transcending': 1, 'weirdoes': 1, 'fallen': 23, 'healed': 2, 'richman': 1, 'malone': 2, 'explored': 21, 'dobson': 1, 'eventually': 107, 'bonfire': 1, 'disappointment': 19, 'keegan': 3, 'backslidden': 1, 'clothed': 2, 'nifty': 13, 'figured': 12, 'squalid': 2, 'pondering': 4, 'duncan': 5, 'outburst': 1, 'limestone': 1, 'squishy': 1, 'abounds': 3, 'brightest': 4, 'hunky-dory': 1, 'man-hunting': 1, 'instructors': 1, 'topping': 6, 'betray': 4, 'owen': 11, 'superhero': 7, 'disjointment': 1, 'infection': 1, 'siege': 9, 'protected': 1, 'racy': 1, 'kent': 1, 'whiny': 6, 'infected': 4, 'unexplored': 3, 'solves': 2, 'bomber': 1, 'bookending': 1, 'arab-speaking': 1, 'duo': 11, 'cockroach-like': 1, 'sectors': 1, 'sixteenth-century': 1, 'waver': 1, 'fielder': 1, 'improving': 2, 'surf': 3, 'provision': 2, 'contrasting': 5, 'provoked': 2, 'subscribed': 2, 'non-offensive': 1, 'depth': 67, "data's": 2, 'setting': 66, 'caustic': 2, 'techno-ish': 1, 'behave': 2, 'dissected': 1, 'basket': 1, 'dramatism': 1, "'you're": 1, 'rhames': 7, 'borne': 1, 'riots': 2, 'revoltingly': 1, 'crushes': 1, 'suggestion': 5, 'ratttz': 1, 'lightening': 2, 'cunning': 7, 'demolish': 1, 'urine-stained': 1, 'uncannily': 2, 'indulgent': 3, 'normal': 57, 'hui': 1, 'inform': 2, 'opera': 30, 'dot-com': 1, 'confirmed': 6, 'tongue': 9, 'tommy': 21, 'declare': 4, 'whores': 1, 'hurlyburly': 1, 'cutedom': 1, 'triggers': 3, 'relentless': 5, 'standard': 41, 'bandit': 3, 'pare': 1, 'consultation': 1, 'punishing': 3, 'reduced': 13, 'grasp': 15, 'shoot-up': 1, 'self-staged': 1, 'ratted': 1, 'publicity': 11, 'murray': 16, 'bring': 99, '$91': 1, 'harland': 1, 'universality': 1, 'dating': 17, 'holstering': 1, 'handicap': 1, 'reubens': 2, 'pressures': 6, 'deviate': 2, 'frat': 2, "thomas's": 1, "onegin's": 1, 'exalted': 1, 'flanagan': 2, 'infancy': 1, 'shoveling': 1, 'classrooms': 1, 'mediterranean': 1, 'hotep': 1, 'gap': 4, 'modicum': 1, 'rolle': 1, 'scotty': 4, "jack's": 9, 'lay': 11, "wayland's": 1, 'espousing': 2, 'cloud': 6, 'treasure-seeking': 1, 'copping-out': 1, 'coordinated': 1, 'thermia': 1, 'bower': 1, 'skylar': 3, 'topsy-turvey': 1, 'subjecting': 3, "danny's": 3, 'life-saver': 1, 'fading': 6, 'vexing': 1, 'unsatisfactory': 1, 'undeniably': 7, 'piet': 1, 'ers': 1, 'robards': 2, 'soviets': 3, '16': 11, 'bred': 1, 'spectrum': 2, 'alliance': 7, 'twinky': 1, 'half-decade': 1, 'democratic': 6, "characters'": 25, 're-interpretation': 1, 'oft-dismissed': 1, 'ja': 1, 'reside': 4, 'cheeky': 5, 'inhibitions': 3, 'indifferent': 3, 'context': 15, 'lineage': 1, 'documentary': 24, 'guttenberg': 1, 'east-west': 1, 'money': 150, 'guttural': 1, 'planning-to-retire': 1, 'internally': 1, 'buckney': 1, 'neighborhoods': 1, 'achieves': 14, 'rekindle': 5, 'converted': 2, 'zippel': 3, 'fungus': 1, 'desirable': 1, 'roof': 6, 'costuming': 8, 'caliber': 11, 'maneuvers': 4, 'virus': 7, 'symbolize': 2, 'enterprise--commander': 1, "planet's": 2, 'veronicca': 1, 'marla': 1, 'pulitzer': 1, 'cleaning': 10, 'lacked': 3, 'hypocrisy': 9, 'thematically': 7, 'lupone': 1, "hollywood's": 20, 'masahitko': 1, 'jazz': 13, 'reece': 1, 'polite': 5, "stillman's": 1, 'dining': 2, 'bollocks': 1, 'ruritanian': 1, "sethe's": 1, 'miles': 25, 'gos': 1, 'crawly': 1, 'kudrow': 5, 'dimensional': 8, 'pretentious': 13, 'dickens-era': 1, 'solar': 4, 'foot-massage': 1, '+': 4, "parr's": 1, 'runner_': 1, 'aimed': 13, 'brooklyn': 6, 'drives': 26, 'broad-based': 1, 'proclaiming': 3, 'macfadyen': 2, 'toss': 3, "o'brien": 1, 'shortest': 2, "else's": 6, 'peacemaker': 3, 'securing': 1, 'sid': 9, 'jersey': 8, 'arrises': 1, 'tier': 1, 'extraordinarily': 8, 'fielding': 1, 'excerpt': 5, 'personalizes': 1, 'hoping': 30, 'passports': 2, 'comprise': 1, 'ruined': 12, 'meyjes': 1, 'brody': 2, 'georges': 1, 'carrie-ann': 1, 'flamed': 1, 'warmth': 11, 'shards': 1, 'snl': 1, 'acknowledgment': 1, 'assist': 4, 'better--among': 1, 'mode': 8, 'eschews': 5, "afi's": 1, 'signifying': 1, 'stunner': 2, 'nyman': 1, 'incinerating': 1, 'auction': 1, 'thrash': 1, 'interviewees': 1, 'love-triangle': 1, 'terror': 22, 'horns': 1, 'outlawed': 1, 'gads': 1, 'shakespearean': 4, 'underscore': 1, 'messed': 5, 'wannabe': 6, 'double-edged': 1, 'thieveing': 1, 'audaciously': 1, "arthur's": 4, "'f'": 1, 'freshest': 2, "siam's": 1, 'tupac': 1, 'roach': 2, 'sided': 1, 'wags': 3, 'alvarado': 2, 'nolot': 1, 'victorian': 3, 'sayles': 2, 'self-ridicule': 1, 'ethereally': 1, 'derivitive': 1, 'boogie': 16, 'hutton': 1, 'off-screen': 7, 'teen-accessible': 1, 'rating': 59, 'dean': 15, 'plumet': 1, "strangers'": 1, 'now-outdated': 1, 'shapeshifting': 1, 'harsh': 18, '206': 1, "curry's": 1, 'vegans': 1, 'hissed': 1, 'talkative': 4, "gingrich's": 1, "troy's": 1, 'financially': 8, 'renault': 1, 'dismissing': 2, 'corey': 1, 'pricelessly': 1, 'revered': 1, 'florence': 2, 're-powering': 1, 'selma': 1, 'shred': 4, 'screens': 10, 'dubs': 1, 'brock': 4, 'zit': 1, 'forehead': 3, 'pebbles': 2, 'cringed': 1, 'swim': 7, 'shrinking': 1, 'serves': 32, 'emil': 1, 'edwards': 7, 'tactically': 1, 'fatalistic': 1, 'busting': 1, 'modest': 7, '1971': 7, 'meditative': 2, 'co-starred': 1, 'cybernetic': 1, 'dwell': 2, 'newport': 1, 'salem': 1, 'square': 11, 'tastefully': 3, "helen's": 1, 'summary': 6, 'eddie': 36, 'heals': 2, 'slots': 1, 'tied': 10, "1976's": 1, 'excessive': 18, 'bigger': 26, 'vigil': 1, 'rapacious': 1, 't-shirt': 4, 'fishnet-stocking': 1, 'over-achiever': 1, 'quasi-nutritional': 1, 'indignant': 2, 'acidic': 1, 'americanised': 1, 'portman': 15, 'maaia': 1, 'conscious': 2, "bug's": 9, 'lovingly': 13, 'accomplished': 25, 'gabriela': 1, 'rose': 21, 'mantra': 4, 'toons': 1, 'unpleasant': 11, 'doffs': 1, 'oscar-bait': 1, 'classification': 4, 'biederman': 1, 'lambs': 7, 'now-deceased': 1, '000': 44, 'inauspicious': 1, 'plato': 2, 'typewriter': 4, 'out-of-work': 2, "'being": 3, 'finger-pointing': 1, 'fairly': 65, 'bigwigs': 2, 'carson': 1, 'dazzlingly': 1, 'knocks': 8, 'suck': 7, 'asylum': 5, 'edge--had': 1, 'consist': 6, "kirshner's": 1, 'dragon-protected': 1, 'entice': 1, 'geese': 2, 'red-tinted': 1, 'individuals': 15, 'elves': 1, 'tendency': 10, 'aplomb': 4, 'pseudo-lyrical': 1, 'accused': 16, 'demme': 3, 'longing': 5, 'mohr': 6, 'deming': 1, 'swastikas': 1, 'afterlife': 3, 'pre-paid': 1, 'complacent': 2, 'imperiously': 1, 'debts': 4, 'dad': 33, 'dubious': 5, 'stephn': 1, 'reform': 1, "d'oeuvre": 1, 'bravery': 2, 'relatable': 2, 'race': 45, 'undercurrents': 2, 'frau': 3, 'useless': 10, 'singh': 1, 'exuberant': 1, 'decades': 31, 'pimple': 3, 'grief-stricken': 1, 'bribe': 2, 'doomsaying': 1, 'plot-line': 1, 'dehaven': 1, 'jacks': 1, 'beijing': 1, 'divorcing': 2, 'reccomended': 1, 'soiled': 2, "ex-wife's": 1, 'textbook': 5, 'criticizing': 3, 'pandering': 4, 'tramp': 4, 'breaks': 41, 'clumsy': 13, 'facility': 7, 'hubbub': 1, 'pretense': 4, 'arielle': 1, 'kennedys': 1, 'sport': 4, "petit's": 1, '1960s': 16, 'lensky': 1, 'macchio': 1, 'solidarity': 2, 'dan': 19, 'stereotypes': 10, 'alternatively': 1, "'blair": 1, 'beatings': 2, '100%': 6, 'third-class-party': 1, 'shirts': 3, "daylight's": 1, 'beach': 30, "'snow": 1, 'reveal': 37, "`faith'": 1, 'arabian': 1, 'highs': 5, 'aware': 31, 'comparison': 33, 'oscar-caliber': 2, 'oldish': 1, 'kahn': 4, 'symbol': 13, 'fionnula': 1, 'ammunition': 1, 'quarters': 4, '939-2': 1, 'inner-city': 2, 'excuses': 1, 'thieve': 1, 'expressing': 7, 'high-speed': 1, 'dorsey': 1, 'monet': 1, "grasshopper's": 1, 'loopy': 4, 'otter': 1, 'effects-heavy': 1, 'wares': 1, "vinegar's": 1, "terminator's": 1, 'hustling': 2, 'huns': 7, 'virtue': 4, 'greaser': 2, 'conceived': 7, 'uplifts': 1, 'warner': 12, 'ordell': 6, 'narrating': 3, 'originals': 2, 'finance': 2, "u2's": 1, 'suprising': 3, 'runneth': 1, "bailey's": 1, 'hell-bent': 2, 'always-on-the-move': 1, 'helium': 1, "base's": 1, 'meticulous': 5, 'fogs': 1, 'susan': 21, 'sidebars': 1, 'top-flight': 2, "kong's": 2, 'star-making': 4, 'recipe': 3, 'curveball': 1, 'haven': 3, 'steering': 3, 'canada': 3, 'gulp': 2, "'rest'": 1, 'grifters': 3, 'wesley': 3, 'morphings': 1, 'stephen': 45, 'glow': 5, 'acid': 6, 'comfort': 12, 'willis': 21, 'pridefully': 1, 'routines': 3, 'akiko': 1, 'concurrently': 1, 'walt': 6, 'demoted': 2, 'unwavering': 5, 'shrunken': 1, 'gangster': 21, 'amercian': 1, 'o-negative': 1, 'suckered': 3, 'jinks/': 1, 'fisher': 9, 'detmer': 2, 'holnist': 1, 'progress': 14, 'ventura-ish': 1, 'links': 5, "patrice's": 1, 'ackland': 1, 'toiling': 3, 'vicky': 1, 'pseudo-religion': 1, 'dismembered': 1, 'flowers--tear': 1, 'churchgoers': 1, 'noboyd': 1, 'characterizes': 1, 'goriest': 2, 'hellraiser': 2, 'bombs': 6, 'russian': 20, 'stereotypical': 15, 'tate': 5, 'recognizes': 12, 'animus': 1, 'indeed-we': 1, 'masse': 3, 'uncovered': 4, 'fargo': 21, "existentialists'": 1, 'break': 77, 'guys': 86, "stewart's": 2, 'indelibly': 1, 'spirit-dead': 1, 'affirmative': 2, 'aria': 1, 'nyc': 6, 'impetus': 2, 'set-pieces': 2, 'depend': 4, 'cuddly': 4, 'cappers': 1, "jewison's": 2, 'schlockfest': 1, "kazantzakis'": 2, 'teen': 36, 'retells': 2, 'over-simplifies': 1, 'illustrated': 7, 'nominee': 5, 'bailey': 4, 'overflows': 1, 'realy': 1, 'coolidge': 4, 'believers': 1, "victoria's": 1, "world'": 2, 'infiltrating': 1, 'tracy': 8, 'pursuit': 16, 'oh-so-adorable': 1, "'freaky": 1, 'tenet': 1, 'employer': 8, 'shun': 1, 'diagnosis': 1, 'politics': 24, 'stretched': 3, 'glances': 5, 'smells': 2, "momma's": 1, 'twisters': 2, 'newly-acquired': 1, 'paid': 18, 'milking': 1, 'col': 4, 'trio': 25, 'fortitude': 2, 'stringing': 2, 'encapsulated': 1, 'motions': 6, 'cloaks': 1, 'shattering': 2, 'withdrawal': 2, 'matters': 37, 'pd': 1, 'listen': 21, 'pro-porn': 1, 'blah': 3, 'crammed': 1, 'kelly': 21, 'reflections': 5, 'jeffries': 1, "lucas's": 1, 'spielberg': 21, 'validation': 2, 'temperamental': 1, 'compositions': 1, "colonist's": 1, '23': 3, 'chumming': 1, 'mazzello': 2, 'helmed': 5, 'vantage': 1, 'c-3po': 3, 'gratuitious': 1, "emmerich's": 1, 'estefan': 1, 'renders': 3, "hustler's": 1, 'drive-in': 2, 'conspiracy': 14, 'worsening': 1, 'mascot': 1, 'bulworth': 3, 'strapped': 4, 'salma': 5, 'kaczynski': 1, 'governmental': 2, 'non-comedic': 1, 'honda': 1, 'impartially': 1, 'bowl': 7, 'single-mother': 1, 'bed': 23, 'metcalf': 2, 'pessimism': 2, 'wren': 1, 'giving': 78, '_star': 1, 'detail': 50, '--slacker': 1, 'hogarth': 1, "what're": 1, 'soul': 41, 'implying': 2, "pi's": 1, 'drunken-style': 1, 'organization': 11, 'dictates': 1, 'entrances': 2, 'restrict': 1, 'campaigner': 1, 'elspeth': 1, 'purging': 1, 'tomas': 1, 'quizzical': 1, 'ends--freud': 1, 'warrant': 2, 'finished': 12, 'recycling': 1, 'lamas': 2, 'usage': 5, "son'a": 2, 'hushed': 1, 'rejoice': 4, 'feverishly': 1, 'liar': 5, 'latin': 5, 'marridge': 1, "bride's": 1, 'crave': 1, 'francisco-based': 1, "willard'": 1, "jarvis'": 1, 'petaluma': 1, 'brains': 15, 'poolside': 2, 'license': 3, 'texas': 16, "mars'": 1, 'attack': 33, 'high-grossing': 1, 'cliffhanger': 5, 'backpedal': 2, 'sobbing': 6, 'trajectory': 3, 'toned-down': 3, 'pollutes': 1, 'loudly': 2, 'falter': 6, 'novak': 3, 'love-triangle/revenge': 1, 'unsecure': 1, 'neuroses': 2, 'patrols': 2, 'claim': 14, 'bullied': 1, "'cinderella'": 1, 'mat': 1, 'perpetrator': 2, 'menacing': 12, 'montenegro': 1, 'unsheathes': 1, 'dislocation': 1, 'broadcasting': 1, "hot'": 1, 'revelatory': 1, 'violet': 2, 'stones': 3, 'hugo': 8, 'ordinaryness': 1, 'shortcoming': 1, 'pornographic': 3, 'pan-and-scanned': 2, 'revival': 7, 'dunked': 1, 'strawberry': 2, 'scandals': 2, 'coastal': 2, 'beatty': 4, 'pyrotechnics': 2, '_come_': 1, 'melting': 3, 'racially': 2, 'wilder': 6, 'faltered': 1, "eno's": 1, 'suit': 28, 'marveled': 1, 'conscript': 1, 'fork': 1, 'flinch': 1, 'differing': 1, 'ade': 2, 'best-selling': 3, 'hi-lo': 1, 'blech': 1, 'nice-looking': 1, 'colm': 2, "'70s": 6, "sagan's": 1, 'flush': 1, 'miami': 10, 'grosses': 1, 'fearlessly': 1, 'gunned': 1, 'passport': 1, 'writer/director/producer': 3, 'strikes': 26, 'reward': 9, 'dialogue-': 1, 'chariot': 5, 'rested': 1, 'campfire': 1, 'poignantly': 1, 'riker': 3, 'primarily': 18, 'reservation': 1, 'spar': 1, 'compare': 20, "eyes'": 1, 'feel-the': 1, 'horrific': 15, 'karaoke': 2, 'tunie': 1, 'jot': 1, 'indiana]': 1, 'ritualistically': 1, 'potbelly': 1, 'injected': 3, 'isaiah': 1, 'nazi-collaborating': 1, 'enlists': 7, 'hippie': 4, "anything's": 1, 'abhorrence': 1, 'awesomely': 1, 'casey': 4, 'chip': 8, 'stationmaster': 1, "tarzan's": 5, 'engines': 2, 'flash-back': 1, "film's": 203, 'counteract': 1, 'uninvolving': 3, 'geisha': 1, 'semblance': 3, '103': 1, 'seducer': 1, 'offensive': 16, 'comic-relief': 2, 'rowdy': 2, "might've": 1, 'stone-ish': 1, 'jerri': 1, 'cattily': 1, 'caprio': 1, 'riff': 4, 'unresolved': 4, 'bashing': 4, 'cannibal': 2, 'repairman': 2, "smart-alec's": 1, 'honors': 4, '3': 40, 'mckee': 2, "roman's": 1, 'aunt': 6, 'crafted': 22, 'fly-by': 1, 'revivals': 1, 'foreign-exchange': 1, 're-teaming': 1, '10-year': 1, 'posted': 1, 'sorts': 22, 'clunky': 1, 'seemd': 1, 'wreak': 5, 'effectual': 1, 'pushing': 11, 'sporadic': 1, 'grown-ups': 2, 'jones': 44, 'expressionistic': 1, 'sea-bound': 1, 'katt': 3, 'whips': 1, 'joss': 6, 'formation': 4, 'militaristic': 1, 'sinners': 2, "shoudln't": 1, 'kleinfeld': 1, 'commandos': 2, 'borg': 1, 'susceptible': 1, 'panama': 1, 'implementation': 1, 'taunts': 4, "crow's": 1, 'improbable': 4, 'goldwyn': 6, 'pernilla': 1, 'interrelate': 1, 'cake': 5, 'controversies': 3, 'honesty': 15, 'halfheartedly': 3, 'nunn': 1, 'undocumented': 1, 'murmuring': 1, 'schlock': 4, 'doses': 6, 'extensions': 1, "ogre's": 2, 'rousing': 14, 'intentional': 5, 'noirish': 3, 'crimes-gone-wrong': 1, 'maladjustment': 1, 'rollicking': 3, 'lettuce': 1, 'grant': 19, 'unboarded': 1, 'movie-of-the-week': 1, 'platoon': 7, 'notably': 20, 'waterfront': 1, 'wane': 2, 'dissing': 1, 'doobie': 1, 'compensate': 1, 'union': 10, "craven's": 1, 'internal': 12, 'admitting': 5, 'repeat': 7, "scorsese's": 10, 'entertaining': 136, 'efficiency': 4, 'revolucionario': 1, 'chastity': 1, 'flash-bang': 1, 'cross-examining': 1, 'sonny': 5, 'subjective': 6, 'personally': 23, 'sing': 22, 'runners': 1, 'unfeeling': 2, 'ex-manager': 1, 'conscripted': 3, 'forever': 43, 'permeating': 2, 'palatable': 5, 'phone-assault': 1, 'occur': 34, 'information--something': 1, 'de': 66, 'rocky': 23, 'girl-crazy': 1, 'sweetest': 4, "mcteer's": 1, 'dissappointed': 1, 'toulouse-lautrec': 1, 'raymond': 4, 'marcellus': 2, 'retreat': 4, 'tough-guy': 2, 'headey': 1, '`hang': 1, "myer's": 1, 'hotshot': 6, 'paolo': 1, 'brazen': 1, 'effervescently': 1, 'marches': 5, 'movie--special': 1, 'meals': 4, 'hazy': 3, 'cutest': 1, 'games': 20, 'cyberspace': 1, 'wanking': 1, "d'aragon": 1, 'appreciative': 1, 'restore': 9, 'hiller': 1, 'humble': 6, 'bigfoot': 1, "henriksen's": 1, 'tell-it-like-it-is': 1, 'reflects': 11, 'munster': 1, 'periods': 6, "hank's": 3, 'mesh': 2, 'hands-down': 2, 'richards': 13, 'credible': 18, "zhuangzhuang's": 1, 'authenticity': 8, 'accuracy': 9, 'old-hollywood': 1, 'heated': 3, 'mayhem': 10, 'alphonsia': 1, 'stalwart': 1, 'angel': 14, 'intolerable': 2, 'emotional': 104, 'innocent': 50, 'caught': 63, "wagner's": 2, '10-week': 1, 'lien': 2, "mind's": 2, 'bursts': 2, 'rizzo': 2, 'elicitor': 1, 'humorless': 3, 'neighbors': 11, 'film-making': 7, 'com/~corona/films/details/sw4': 1, 'hein': 1, 'scenes': 354, 'destructive': 6, 'omens': 1, 'inquiries': 1, 'monuments': 1, 'necessarily': 29, 'ballads': 3, 'innate': 5, 'zimbabwe': 1, 'injure': 1, 'alfredo': 1, 'corn': 3, 'numbered': 3, 'risible': 1, 'roast': 2, 'warm-heartedness': 1, 'hays': 1, 'clients': 8, 'noir': 19, 'on-set': 1, 'skimpy': 1, "pee-wee's": 1, 'unsure': 8, '[note': 2, 'overdramaticizes': 1, 'neatly': 10, 'rogues': 1, 'underpinnings': 4, 'tie': 8, 'yuks': 2, 'twenty-year': 1, 'raid': 2, "5'9": 1, 'trekkie': 1, 'parallels': 13, 'anticipated': 11, 'gambon': 1, 'unpaved': 1, 'would-be': 13, 'alpine': 1, 'decaprio': 1, 'geri': 2, 'adolescent': 6, 'exasperation': 1, 'gymnastics': 1, 'currents': 2, 'law-suit': 1, 'doss': 2, 'enlivening': 1, 'dickson': 1, 'garcia': 3, 'shorter': 4, 'baddeley': 1, 'counterbalanced': 1, 'lawyers': 8, 'in-side': 1, 'bluto': 1, 'innocents': 3, 'flighty': 1, 'deformed': 1, 'necessitates': 1, 'benefactor': 5, "laughton's": 1, 'ryanson': 1, 'mistreated': 1, 'culminates': 1, "garth's": 1, 'pratt': 2, 'publisher': 5, 'stinker': 1, 'magnum': 2, 'mentality': 7, 'well-to-do': 4, 'patriot': 5, 'dicing': 1, "karla's": 1, '180': 6, 'upheld': 1, "`beautiful'": 1, 'cute': 44, 'imbecilic': 1, 'symbolically': 3, "murnau's": 2, "browning's": 2, 'screentime': 2, 'herbert': 2, 'branagh': 7, 'delegate': 1, 'loot': 3, 'macaulay': 2, 'michigan': 1, "ru'afro": 1, 'joins': 17, 'obsolete': 3, 'starring-liam': 1, 'critiques': 1, 'shown--like': 1, 'partition': 1, 'again--these': 1, 'wishing': 3, 'tactile': 1, 'laurence': 8, 'hooker': 8, "mallory's": 3, 'deals': 34, 'come-back': 1, 'weeks': 36, 'proclamations': 1, 'walken': 5, 'apparel': 1, "michelle's": 1, 'drug-dealers': 1, 'voil': 1, 'ti': 1, 'summers': 1, 'verber': 1, 'pray': 2, 'kinship': 4, 'zipper': 3, 'ulee': 1, 'jaw-dropping': 5, 'dreamy': 4, 'syllable': 1, 'shyer': 1, 'perfume': 1, 'dario': 4, 'yielding': 1, 'minimizing': 1, 'head-spinning': 1, 'cockroaches': 1, 'enforcing': 1, 'dismayingly': 1, 'interned': 1, 'idealistic': 7, "rosalba's": 1, 'gotta': 8, 'paneled': 1, 'texan': 2, 'bleach': 1, "sorcerer's": 2, 'befalls': 1, 'invaders': 5, 'shrink': 7, 'amorality': 1, 'nunez': 1, 'postulate': 1, 'mcmanus': 2, 'cryogenic': 3, 'competency': 1, 'reproduction': 2, 'punches': 4, 'hairnets': 1, 'navigate': 5, 'pixote': 1, 'motivate': 2, '1950s': 7, 'brims': 1, 'aquamarine': 1, 'interconnectedness': 1, 'registers': 2, 'gosh': 4, 'exposed': 11, "sound-we're": 1, "anakin's": 2, 'wizzard': 1, 'convert': 3, 'cackles': 1, 'ringleader': 1, 'eye-opening': 1, 'seann': 4, 'anney': 1, 'cozy': 1, 'calgary': 1, 'babble': 2, 'cremation': 2, 'non-fictional': 1, 'die': 70, 'interaction': 17, 'overdone': 10, 'safer': 1, 'comforts': 9, 'anti-family': 1, 'coin': 6, 'archie': 2, 'disfiguring': 1, "ramsey's": 2, "carla's": 1, 'profit': 13, 'offices': 2, "`promoted'": 1, 'quatto': 1, 'barkin': 1, 'traders': 2, 'annabeth': 1, 'correlates': 1, "venice's": 2, 'coldly': 2, 'sur-prise': 1, 'fiancee': 2, 'tortured': 10, "gridlock'd": 1, 'spend': 58, 'demeaning': 2, 'zallian': 1, 'phrasing': 1, 'emotion': 36, 'sucker': 4, 'homestead': 1, 'installment': 19, 'unnecessary': 24, 'impersonal': 3, 'reversed': 4, 'brizzi': 1, 'hammer': 3, 'fireball': 2, 'takei': 2, 'underwood': 1, 'shrimp': 1, 'amended': 1, 'anguish': 7, "week's": 1, 'siskel': 3, 'intestines': 2, 'hybridization': 1, 'forster': 7, 'gaming': 2, 'renew': 1, 'airplane': 16, 'jailbreak': 1, 'lovejoy': 1, 'adjoining': 2, 'shrimp-scarfers': 1, 'restlessly': 1, 'arrests': 3, 'misconstrued': 1, 'herbie': 3, 'doc': 4, 'prejudge': 1, 'landings': 1, 'whelmed': 1, 'backyard': 1, 'christmas': 19, 'paths': 10, 'marched': 1, 'appears': 82, 'concentrated': 2, 'jogs': 2, 'contribute': 11, '1998': 44, 'black': 129, 'hyberboreans--we': 1, 'pinching': 1, 'jesus': 16, 'short-lasting': 1, 'ill-luck': 1, 'sahara': 3, 'aurally': 1, 'rang': 1, 'solved': 6, 'sanctity': 1, 'tampopo': 2, 'cameo': 33, 'carry': 50, 'okeday': 1, 'aristocrat': 2, '56-year': 1, 'inherit': 1, 'envisions': 2, 'merciful': 1, 'rally': 8, 'radical': 10, 'hug': 2, "granddaughters'": 1, 'spiel': 1, 'tormey': 1, 'motorcycle': 4, 'demands': 21, 'unhappy': 16, 'make-over': 2, 'bolts': 2, '200-page': 1, 'arduous': 1, 'incredibility': 1, 'movies--the': 1, 'overwhelm': 3, 'will/skylar': 1, 'commodus': 3, 'empathetic': 3, 'pawns': 1, 'ninjas': 2, 'banal': 6, 'beginning--while': 1, 'swoop': 1, 'skates': 2, 'biehn': 3, 'unprepared': 2, "louie's": 1, 'ibanez': 2, 'washed': 6, 'blackboards': 1, 'chuckling': 2, 'implied': 5, 'floods': 1, 'attitudes': 9, 'overseeing': 2, 'drenched': 3, 'allied': 2, 'walls': 15, 'hudson': 1, 'not-so-typical': 1, 'declarations': 2, 'rental': 9, 'pi': 1, 'swigert': 1, 'unimpeachable': 1, 'adapted': 24, 'tour': 18, 'offenders': 1, 'death-obssessed': 1, "warhol's": 1, 'salinger-ism': 1, 'rio': 1, 'exacerbated': 1, 'harry': 17, 'mirror': 16, 'yesterday': 3, 'then-unknown': 2, 'jackpot': 1, 'dynasties': 1, 'bruklin': 1, 'hit': 98, 'debuted': 1, "season's": 1, 'observant': 3, 'cain': 2, 'smearing': 1, 'ungar': 1, 'turns': 162, 'trucks': 1, 'panhood': 1, 'slavers': 2, 'unhappiness': 3, "parent's": 6, 'revels': 4, 'singularly': 1, 'b-': 2, 'pied': 2, 'howler': 1, 'energetic': 26, 'hibernated': 1, 'fontaine': 1, 'grieco': 1, 'blindsided': 2, 'sexually-frustrated': 1, "falk's": 1, 'ditzy': 6, 'bellboy': 1, 'anti-matter': 1, 'scoring': 2, 'prone': 4, 'binge': 2, 'feverish': 1, 'berdan': 1, 'dish-molded': 1, 'cornball': 3, 'non-religious': 1, 'huiessan': 1, 'casually': 5, 'croupier': 1, 'bargains': 2, 'locks': 1, 'robertson': 1, 'thumb': 1, 'redeem': 5, 'fitzswalter': 1, 'deus': 1, 'duet': 2, 'oral': 5, 'oh-so-nice': 1, 'deserts': 1, 'kingdom': 9, 'kinsella': 1, 'unconventionally': 1, 'locusts': 2, 'tops': 7, 'tv': 83, 'san': 12, 'tweaked': 1, 'ineffable': 1, 'hoover': 1, 'pose': 8, 'rip': 6, 'hatred': 12, 'successes': 6, 'catchphrases': 2, 'fugitives': 1, 'wormwoods': 1, 'quarry': 1, 'pants-wetting': 1, 'cranking': 1, 'pens': 1, 'follower': 1, 'sophistication': 5, 'acrimony': 1, 'upstages': 2, 'solid': 98, 'stomach/lack': 1, 'dormant': 4, 'drinker': 1, 'involves': 23, 'conspiring': 1, "`tuning'": 1, 'malintentioned': 1, 'shackles': 1, 'amos': 2, 'spilling': 2, 'indignity': 2, 'stonecutters': 1, 'desire': 36, 'menken/david': 1, '[1': 1, 'barrage': 3, 'mama': 2, 'sword-wielding': 1, 'jhvh': 1, 'operated': 2, 'hire': 12, 'prisons': 1, 'signifcance': 1, 'huston': 3, 'lashing': 1, 'pickings': 1, 'repair': 3, 'vary': 4, 'doe-eyed': 1, 'stooge': 1, 'reclessly': 1, 'degradation': 2, 'moviemaker': 1, 'junichiro': 1, 'barmaid': 1, 'straight-forward': 3, 'breast': 4, 'smorgasbord': 1, 'fitzgerald': 2, 're-delivering': 1, 'inspires': 10, 'mcdowell': 2, 'loose-ends': 1, 'tinseltown': 2, 'leathery': 1, 'walnuts': 1, 'vulgarity': 4, 'nah': 1, 'hear': 52, 'customer': 8, 'rumbles': 1, 'chronicled': 3, 'geneticist': 1, 'makings': 1, 'evolved': 2, 'barrel': 2, 'blessedly': 1, 'fundamental': 5, 'marxist': 1, 'tomei': 3, 'perpretrators': 1, 'feminism': 1, 'discredited': 1, 'unfair': 10, 'quotas': 1, 'persuade': 5, 'tin': 8, 'substitutes': 1, 'armpit': 3, 'crowd': 31, 'hampton': 1, 'vistas': 2, 'blurs': 1, 'brightened': 1, 'laughable': 14, 'good-looking': 8, 'assertive': 4, 'tenko': 1, 'ments': 1, "1987's": 1, 'vastly': 7, 'pew': 1, 'disorganization': 1, 'exhibits': 8, 'freak': 11, 'unconditionally': 1, 'jonas': 3, 'ladies': 10, 'leeloo': 1, 'girl': 124, 'trash': 15, 'radio': 35, 'ariane': 1, 'derring-do': 1, 'chien-po': 1, 'anonymity': 2, 'prudish': 1, 'sex': 106, 'willa': 1, 'bayou': 2, 'wade': 3, 'mil': 1, 'charms': 7, 'comedy': 202, 'tables': 9, 'fulcrum': 1, 'ligaments': 1, 'briers': 3, 'resorting': 5, 'ingrid': 2, 'outer': 12, 'glut': 1, 'dialogues': 4, 'bands': 2, 'working-class': 4, 'ebb': 1, 'licia': 2, 'consult': 2, 'eastwick': 2, "parks'": 1, 'out-of-balance': 1, 'hideously': 3, 'delacroix': 2, 'anastasia': 4, 'truthfully': 4, 'holt': 1, 'charlatans': 1, 'pledges': 2, 'refer': 7, 'limon': 1, 'marble': 1, 'mychael': 3, 'free-lance': 1, 'darting': 1, 'phases': 1, 'superior': 30, 'nervousness': 3, 'nefarious': 5, 'lovett': 4, 'channel': 9, 'nbc': 2, 'opts': 8, 'pennies': 1, 'cheekbones': 1, 'pugnacious': 1, 'butts': 1, 'fei-hung': 1, 'discotheque': 1, 'puns': 3, 'americans': 27, 'commie': 1, 'trickiest': 1, 'arsed': 1, 'gender-bending': 1, "boy's": 6, '86': 1, 'unorthodox': 6, 'marksman': 1, 'bail': 9, 'undermines': 2, 'fault': 24, 'pleasantries': 1, 'richard': 47, 'karaszewski': 2, 'roughly': 8, 'cole': 9, 'concealed': 4, 'blow': 18, 'rube': 2, 'grow': 36, 'stardom': 11, 'wanders': 9, 'treasures': 4, 'computerized': 1, 'skillful': 9, 'spill': 2, 'chap': 1, 'interacts': 1, 'youthful': 4, 'graciela': 1, 'brokers': 1, 'vocation': 2, 'slow-mo': 1, 'folklore': 4, 'affair-a-week': 1, 'indiana': 9, 'driving': 28, 'sunrises': 2, 'bong': 1, 'snipers': 1, "merlin's": 1, 'soccer': 2, 'cows': 1, 'shelly': 2, 'imitate': 4, 'impose': 1, 'dandy': 5, 'drizzling': 1, 'aboard': 16, 'brosnan': 9, 'mythical': 9, "o'neal": 2, 'opting': 2, 'labors': 1, 'tania': 1, 'honed': 3, 'semi-introspective': 1, 'action': 215, 'toneless': 1, 'mowing': 1, 'kala': 4, 'wows': 3, '1972': 6, 'shrouded': 2, 'shapely': 1, "apartment's": 1, 'pioneer': 3, 'wore': 5, "buck's": 1, 'moons': 1, 'lowdown': 1, 'ferguson': 1, 'prototype': 1, "donkey's": 1, 'ernest': 7, 'war': 100, 'lawsuit': 3, 'kotter': 2, "novel's": 1, 'fuentes': 1, 'caricature': 12, 'undetectable': 1, 'peculiar': 9, 'satisfies': 3, 'spews': 1, 'cafe': 4, 'blood-spattered': 1, 'cable': 14, 'repression': 2, 'shor': 2, 'physically': 18, 'strategy': 4, 'beals': 1, 'ate': 2, "rice's": 1, "virgil's": 1, 'titled': 10, '`tarzan': 1, 'nivola': 1, 'queasy': 3, "memphis'": 1, 'westerners': 2, 'glimmers': 1, 'chairman': 4, 'werewolf': 2, 'vice-versa': 1, 'buyer': 1, 'non-actors': 2, 'cloth': 2, 'calorie-free--so': 1, 'transvestite': 5, 'stickers': 1, 'self-involved': 1, 'shite': 1, 'developing': 15, 'saddened': 4, 'redundant': 6, 'anakin': 9, 'fortune': 20, 'interpreter': 1, 'mullan': 1, 'make': 444, 'bartkowiak': 1, 'commander': 4, 'overact': 2, 'circular': 2, "caveman's": 1, 'baad': 1, 'adian': 1, 'healing': 5, 'melaine': 1, 'uncommon': 3, 'knob': 1, 'twitch': 2, 'urbanites': 1, 'millius': 1, 'pain': 45, "sawa's": 1, 'nealon': 1, 'middle-of-the-road': 1, 'sprinkle': 2, 'lieutenants': 1, 'ex-porn': 1, 'trashcan': 1, 'commited': 1, 'polarization': 1, 'grip': 11, 'peaceably': 1, 'sleepy': 6, 'peel': 1, '-october': 1, 'preppie': 2, "stanton's": 1, 'fiction': 73, 'negotiable': 1, 'returns': 45, 'dreck': 1, 'assassination': 5, 'janitor': 8, 'bloodshed': 3, 'throats': 3, 'liberates': 1, "gerry's": 2, 'unrest': 1, 'martians': 4, 'murron': 1, 'loonier': 1, 'historic': 5, 'jaunt': 2, 'airborne': 1, 'expending': 1, 'mike': 31, 'gotham': 2, 'basics': 4, 'admire': 16, 'acuity': 1, 'kermit': 1, 'ufos': 1, 'damage': 8, 'laments': 2, 'demurely': 1, '50': 15, 'swann': 1, 'scatological': 1, 'bushes': 3, 'kirkeby': 1, 'heaven': 19, 'sarossy': 3, 'singer/lover': 1, 'casino': 13, 'happiest': 1, 'night': 143, 'phillipe': 1, 'ally': 7, 'proper': 24, 'vocally': 1, 'stirs': 3, 'suits': 14, 'booth': 3, 'lazer': 1, 'impactful': 1, 'critical': 30, 'hodge': 1, 'career-threatening': 1, 'gold--when': 1, 'schmaltziness': 1, 'action-thriller': 6, 'whovians': 1, 'worrall': 1, 'dissuasive': 1, 'testament': 16, 'sober': 5, 'garbed': 1, 'obsessional': 1, 'ventilation': 1, 'shekhar': 1, 'singling': 1, 'oftentimes': 2, 'beaufoy': 1, 'mysteriously': 8, 'models': 8, 'virgil': 3, 'anand': 2, 'acme': 1, 'fussy': 1, 'propaganda': 7, 'mclachlan-aided': 1, "ranger's": 1, 'improvisation': 2, 'thriller': 73, 'twinkle-toed': 1, 'counsel': 2, 'puncturing': 1, 'confirm': 2, "wen's": 1, 'judgements': 1, 'miseries': 2, 'disembodied': 3, 'casting': 56, 'director/co-writer': 1, 'employers': 1, 'confide': 1, 'courts': 6, 'suvari': 1, "mitch's": 1, 'albert': 19, 'tint': 2, 'poitier': 3, 'comedies': 32, 'sub-themes': 1, 'trains/rocky': 1, 'mid-teenage': 1, 'attacked': 14, 'metaphysics': 1, 'khanjian': 1, 'anti-fascist': 1, 'melts': 4, 'siblings': 3, 'patrasche': 1, 'medieval': 7, "grady's": 2, 'anton': 6, 'talented': 56, 'prim': 2, 'hanif': 1, 'we+ve': 1, 'excerpts': 2, 'organa': 1, 'preternatural': 1, 'newbigin': 1, 'underscores': 2, 'bogart': 3, 'colqhoun': 1, 'embroiled': 2, "reynolds'": 1, 'chocolate': 9, 'dreamworld': 1, 'best-friend': 2, 'morte': 1, 'trapping': 1, 'neurotic': 13, "dar's": 1, 'trotted': 2, 'revisionist': 2, '-films': 1, "parents'": 2, "nicholas'": 1, 'gatherings': 3, 'sandals': 1, 'eszterhaz': 1, 'quieter': 4, 'consumatory': 1, 'arms': 19, 'score-writer': 1, 'twenty-somethings': 2, 'backstabbers': 1, 'enlivened': 1, 'blaxploitation-era': 1, '99-cent': 1, 'assailing': 1, 'grossing': 6, 'exchanged': 2, "'you": 1, 'enables': 3, 'esther': 2, 'schlesinger': 2, 'elections': 2, 'politician': 7, 'lesra': 1, 'raiment': 1, 'clare': 1, 'badly': 13, "'either": 1, 'exaggerate': 3, 'audience-pleaser': 1, 'hipper': 1, 'enlisted': 2, 'unveils': 2, 'playing': 135, '2470': 1, 'vondie': 1, 'coloring': 1, "dogma's": 2, 'cowboy': 14, 'self-satisfaction': 1, 'three-hour-tear-fest': 1, 'fucking': 4, 'dial': 2, 'rewind': 4, 'museum': 12, 'buddy/weight-lifter': 1, 'apologetic': 1, 'hawn': 1, 'estranged': 10, 'bondsman': 4, '135': 1, '1941': 2, 'toddlers': 1, 'entrace': 1, 'loud': 26, 'wounded': 14, 'handphone': 1, 'rozencrantz': 1, 'weaving': 7, 'abc': 2, "charlie'": 1, 'glendale': 1, 'slayings': 1, 'melody': 4, 'declining': 2, 'imposed': 1, 'spreading': 3, 'ease': 21, 'wars_': 1, '12-part': 1, 'maryann': 1, 'leaves': 88, 'ah': 2, 'lustre': 1, 'shauny': 1, 'reuben': 1, 'technological': 7, 'plainly': 2, '107': 1, 'trace': 9, 'boom': 2, 'ah-ha': 1, 'throes': 1, 'deadly': 25, 'fondest': 1, 'heather': 13, 'employs': 11, 'ten': 50, 'supererogatory': 1, 'cosmetics': 1, 'yossef': 1, 'dotty': 2, 'race-against-time': 1, 'luchini': 1, 'ejecting': 1, 'tales': 15, 'junk-fest': 1, "olga's": 1, 'unappreciated': 1, 'educate': 2, 'intercut': 5, 'mtv-type': 1, 'countenance': 2, 'ciello': 1, "jeunet's": 2, 'mentioning': 10, 'worrys': 1, 'plead': 1, 'lifeless': 6, 'unbelievably': 4, 'creation-': 1, 'um-teenth': 1, 'encompassing': 2, 'terrence': 4, 'hitchhiker': 2, 'bilingually-subtitled': 1, 'presaged': 1, 'soundstage': 1, 'reinvigorating': 1, 'dune': 2, 'quaint': 9, 'slows': 8, 'ketchup': 1, '[g]': 1, 'crucifixtion': 1, 'insurmountable': 3, 'induces': 2, 'dues': 1, 'hop': 8, 'maria': 5, 'dubuque': 1, 'odile': 1, 'persia': 1, 'sleepwalking': 3, 're-introduced': 2, 'pretensions': 1, 'cryptically': 1, 'stockwell': 3, 'kenner': 1, 'megabytes': 1, 'hurriedly': 1, 'short-cut': 1, 'keogh': 1, 'sweeping': 9, "forman's": 1, 'media': 32, 'leuchter': 1, 'low-to-medium-budget': 1, 'colourful': 7, 'drove': 3, 'femme-butch-femme': 1, 'delivers--meaningful': 1, '`when': 1, 'irrelevent': 1, 'acted': 28, 'telekinetic': 1, 'muttered': 2, 'megahit': 1, 'leeper': 1, 'disgusting': 7, 'covenant': 1, 'ominously': 3, 'begbie': 2, 'mill': 8, 'ringwald': 2, 'flamboyantly': 3, 'lengths': 11, 'lfe': 1, 'shops': 3, 'sweating': 1, 'executive': 20, 'nielsen': 5, 'hu': 1, 'frustrates': 2, 'dissenting': 1, 'one-of-a-kind': 3, 'watson': 12, 'torrance': 1, 'mutation': 3, 'hard-nosed': 5, 'walled': 2, 'non-committal': 1, 'ridgemont': 1, 'compton': 1, 'fucks': 1, 'unravel': 7, 'internalizing': 1, 'novel--but': 1, 'agent': 52, 'achieveing': 1, 'horizontal': 1, 'drowning': 4, 'corny': 13, 'tenaciously': 1, 'spits': 5, 'keanu': 11, 'pair': 31, 'obese': 1, 'dussander-much': 1, 'promised': 12, 'paranoiac': 1, 'skellington': 1, 'locations': 10, 'explosion': 12, 'kimmy': 1, 'afternoon': 7, 'role': 297, 'fletcher--channel': 1, 'sadly': 26, 'rambling': 7, 'ramifications': 3, 'cowards': 1, 'aristocracy': 5, 'harlin': 3, 'tomorrow': 10, 'zdenek': 1, 'flirt': 1, 'inconvenience': 1, 'pearly': 1, 'strikingly': 7, 'nasa': 2, "monkey's": 1, 'galaxies': 1, 'woven': 2, "billy's": 3, 'softened': 1, 'mechanically-challenged': 1, 'goetz': 1, 'catharsis': 3, 'retort': 1, 'endurance': 2, 'legs': 20, 'conversing': 1, 'salieri': 1, 'cello': 3, 'disneyfied': 2, 'sharp': 35, 'mant': 1, 'salminen': 1, 'projecting': 2, 'triple': 2, 'sunbathes': 1, 'identically-wrapped': 1, 'coldness': 2, 'examines': 6, 'gibson=92s': 1, "stanley's": 1, 'schoolers': 3, 'chain': 21, 'ventures': 5, 'schticks': 1, 'overhead': 3, 'aquarium': 3, 'miscast--as': 1, 'grassroots': 1, 'forensic': 1, 'victor': 9, 'line-ups': 1, 'financier': 2, 'reacts': 3, 'cockroach': 1, 'even-handed': 1, 'yawn': 1, 'flash-card': 1, 'situation': 78, 'auditor': 1, 'queens': 2, 'luxuriantly': 1, 'non-disney': 1, 'actor': 191, "hanks's": 1, 'fanatics': 4, 'beasties': 1, 'booths': 1, 'deluding': 1, "'powdered": 1, 'good-bye': 2, 'supervising': 1, 'cahoots': 1, 'pseudo-science': 1, 'abrupt': 4, 'scouting': 1, 'post-secondary': 1, "tulips'": 1, 'date': 55, 'vaults': 2, "cage's": 3, 'remembered': 28, 'squalor': 3, 'boxes': 2, 'terri': 1, 'bedridden': 1, 'stabilize': 1, 'peers': 9, "michael's": 3, 'co-conspirator': 1, 'role-playing': 2, 'foster': 13, 'implies': 5, 'constellations': 2, 'fuck-you': 1, "80's": 12, 'extorts': 1, 'electrifying': 3, 'blythe': 1, 'shipwreck': 2, 'telling': 57, 'important': 98, 'disconcerted': 2, "1986's": 1, 'mental': 24, 'quest': 28, "rickman's": 1, 'paramedic': 4, "'crazy'": 1, 'indulges': 3, 'cufflinks': 1, 'grader': 1, 'doodles': 1, 'credibly': 1, 'paragraphs': 2, 'matrons': 1, "hugo's": 5, 'kronk': 1, 'balls': 8, 'julia': 24, 'erik': 1, '17-year-old': 1, 'zahn': 6, 'rehearsals': 1, '30th': 1, 'sand': 8, 'whodunit': 3, 'celebration': 9, 'defeaning': 1, 'spall': 2, '$15': 1, 'level-headed': 2, 'junger': 1, 'angelina': 2, 'girlfriend': 71, 'invention': 5, 'worldwide': 6, '`true': 1, 'waterfall': 1, 'brightness': 2, 'dim': 3, 'doubly': 1, 'unions': 1, 'reissued': 1, 'cullen': 1, 'lodge': 1, 'guenveur': 1, "brook's": 1, 'character-defining': 2, 'stephan': 2, 'amour/last': 1, 'epstein': 1, 'howard': 16, 'fellow': 50, 'admiration': 7, 'berendt': 1, 'yao': 2, 'gelfling': 1, 'cell': 19, 'readily': 2, 'concocts': 3, 'charm': 47, 'tsk': 1, 'ling': 3, 'knockout': 3, "hulot's": 1, 'intimacy': 3, 'deejay': 1, 'richest': 3, 'wallets': 1, 'usurp': 1, 'gorey': 1, 'coincide': 1, 'wrapper': 1, 'indulge': 5, 'plane': 24, 'cambell': 1, 'travel': 34, 'locking': 4, 'placeholders': 1, 'zoe': 4, 'freaks': 6, 'abstraction': 1, 'rogue': 8, 'depictions': 3, 'royal': 13, "fans'": 1, '_amadeus_': 1, "stahl's": 1, 'underbids': 1, 'floop': 1, 'lumpy': 1, 'revamped': 2, 'sycophant': 1, 'overshadowed': 9, 'motivated': 2, 'jarrett': 1, 'frighten--two': 1, 'swaying': 4, 'homeless': 4, 'dangerously': 2, 'hip-talking': 1, 'naive': 23, 'financially-strapped': 1, 'russian-german': 1, 'trepidations': 1, "e's": 1, 'seizes': 2, 'mosques': 1, 'slip': 16, 'persist': 1, 'possibilities': 10, 'oz': 9, 'dismay': 6, 'swanbeck': 1, "studio's": 3, 'hushes': 1, 'punchlines': 2, 'loves/hates': 1, "anyone's": 6, 'battlestar': 1, 'kirsten': 5, 'replaces': 4, 'fathom': 2, 'claude': 5, 'kitt': 1, 'guitarist': 2, 'nathaniel': 1, 'affability': 1, 'antique': 2, 'impaired': 1, 'pupils': 1, 'skaarsgard': 1, 'scenarios': 3, 'nervous': 20, 'mishandled': 2, 'beachfront': 1, 'percussion-heavy': 1, 'eleveate': 1, 'organizer': 2, 'pick': 32, 'elaine': 4, 'pop-up': 1, 'khanijian': 1, 'politicians': 7, 'balding': 1, 'yawns': 1, 'risque': 3, 'guts': 14, 'scooby-doo': 1, 'drilled': 1, 'storytellers': 1, 'sizable': 1, "hughes'": 1, 'mirrored': 4, 'ruckus': 1, 'housing': 3, 'diaz': 14, 'stick': 23, 'subterfuge': 2, 'time-': 1, 'sought-after': 2, 'climb': 3, 'astute': 4, "mcglory's": 1, 'aversion': 2, 'conglomerate': 1, 'richness': 4, 'widower': 3, 'enthrall': 1, 'vianne': 1, 'schramm': 1, 'forgotten': 29, 'larch': 1, 'accessibility': 2, 'highly-stylized': 1, 'wallet': 5, "cornwallis'": 1, "'hokey'": 1, 'wonie': 1, 'differentiation': 1, 'nanni': 1, 'strewn': 2, 'vance': 2, 'chimera': 1, 'drunken': 13, 'darkly': 8, 'leila': 1, 'likewise': 9, 'eavesdropping': 1, 'courageous': 4, 'bridge': 14, 'tertiary': 1, 'mia': 4, 'briskly': 1, 'continuity': 3, 'affleck': 20, 'predicaments': 3, 'request': 5, "`gangsta'": 1, 'existed': 8, 'nicolet': 4, 'kerri': 1, 'bee': 3, 'masterpieces': 3, 'gruff': 6, 'loading': 4, 'mousehunt': 1, 'fog': 3, 'speedy': 3, 'charlie': 12, '_fantastic_': 1, 'dern': 5, 'affiliate': 2, 'tolan': 1, 'ozzy': 1, 'flirtatious': 1, 'poker-playing': 2, 'arthouse': 2, 'spaceship': 8, 'darren': 1, "affleck's": 1, 'pines': 6, 'restricting': 1, 'all-or-nothing': 1, 'olivia': 9, 'concoction': 4, 'trailers': 20, "jude's": 1, "'80s": 5, 'miserly': 1, 'sate': 2, 'unlikable': 3, 'brigantine': 1, 'scrambles': 1, 'estimate--but': 1, 'incubator': 1, 'lifer': 1, 'lennie': 2, 'brooke': 1, 'levity': 6, 'ohh': 1, 'smuggled': 3, 'mines': 1, "us's": 1, 'bruckheimer': 1, 'murdered--forced': 1, 'inedible': 1, 'playground': 1, 'divorced': 6, 'indie': 2, 'manhattan': 13, 'messes': 1, '50s': 7, 'silberling': 2, "fuller's": 2, '19-months': 1, 'futility': 2, 'action-comedy': 2, 'utility': 1, 'cheered': 3, 'compared': 47, 'chunk': 5, 'kidnap': 6, 'chalk': 1, 'reflective': 1, 'shampoo-like': 1, 'subsidiary': 2, 'recruit': 5, 'regional': 1, 'garments': 2, 'bone-rattling': 1, 'criminal': 36, 'model-citizens': 1, 'mass-entertainment': 1, 'cinema-wise': 1, 'chiming': 1, 'gays': 1, "coens'": 4, 'mannerisms': 11, 'aiming': 4, 'multiplies': 1, 'respighi': 1, 'arrive': 19, 'swindled': 1, 'interlinked': 1, "bob's": 5, 'toni': 7, 'ross': 10, '[pg]': 1, 'started': 39, 'drowns': 3, 'three-digit': 1, 'russell': 28, 'newell': 1, 'isabella': 7, 'maneating': 1, "'stealth": 1, 'lair': 4, 'suburban': 11, 'brautigan': 2, 'underwhelmed': 1, 'turkish': 2, 'flashdance': 1, 'devastated': 2, 'claimed': 7, 'avocation': 1, 'centrestage': 1, 'kramer': 1, 'settlers': 2, 'hostage': 10, 'wicca': 1, 'schlumpy': 1, 'revel': 1, '24th-century': 1, 'cult-status': 1, 'conformist': 1, 'stempel': 1, "tykwer's": 2, 'jealously': 1, 'mice': 8, "teri's": 1, 'safety': 17, 'vampires': 9, 'pig-keeper': 1, "harlin's": 1, 'history': 102, 'implausible': 3, 'attracted': 17, 'sky': 29, 'look-alikes': 1, 'ivy': 1, 'cradle': 2, 'id': 1, 'preservation': 1, 'geez': 3, 'descending': 1, 'stone-faced': 1, 'wintery': 1, 'farina': 5, 'peer': 3, 'patronized': 1, 'startled': 5, 'agreeably': 1, 'deactivating': 1, 'boon': 1, 'restaurants': 4, 'hanson': 3, 'redding': 1, 'lodged': 1, 'b-production': 1, 'comfortably': 5, 'lorraine': 4, 'unraveling': 1, 'mayfield-directed': 1, 'slavery': 6, 'already-beautiful': 1, 'listening': 21, 'maven': 1, 'last-ditch': 2, 'attains': 2, 'astonishing': 20, 'mamoru': 1, 'chock': 2, 'admonition': 2, 'nonetheless': 31, 'structuring': 1, 'dies': 44, 'squadron': 3, 'buffalo': 5, "'titus'": 1, 'propositioned': 1, 'far-right': 1, 'manouvering': 1, 'states': 46, 'true': 180, 'picture]': 3, 'novice': 2, 'technicians': 2, 'volkswagen': 1, 'zeke': 1, '21': 8, 'domination': 2, 'shui': 1, 'pinto': 1, 'scrubs': 1, 'act/speak': 1, 'caddyshack': 1, 'expert': 14, 'millionare': 1, "walken's": 1, 'eerily': 6, 'unimpressed': 2, 'sharon': 11, 'gratification': 1, 'climax': 51, 'parallelism': 1, 'veidt': 1, 'vomiting': 1, 'donation': 1, 'sufi': 1, 'court-room': 1, 'hutter': 2, 'reestablishing': 1, 'beginning': 109, 'proclivities': 1, 'blown-out': 1, 'giggling': 1, 'prisoners': 16, 'strathairn': 3, 'wonderment': 2, 'embodiment': 1, 'lotto': 2, 'goro': 2, 'aviator': 2, "o'clock": 1, 'novelistic': 2, 'accents': 7, 'holdover': 2, 'aggrieved': 1, 'mishmash': 1, 'naught': 1, 'pasadena': 1, 'delegates': 1, 'ellen': 13, 'cannibalism': 3, 'depart': 1, 'chad-ness': 1, 'hill': 23, 'lucrative': 7, 'plights': 1, 'wells': 2, 'action-packed': 5, "movies'": 1, 'anvil': 1, 'stucker': 1, 'dalloway': 1, 'assumptions': 2, 'pierce': 12, 'contrasted': 4, 'streamline': 1, 'byplay': 1, 'rand': 1, 'unreleased': 1, 'diehard': 2, 'accommodates': 1, 'rodents': 1, 'duffle': 1, 'pope': 1, 'dolce': 1, 'enjoyment': 22, 'merged': 3, "kristen's": 1, 'wife': 186, 'unequivocal': 1, 'kellogg': 1, 'paves': 1, 'infinitum': 1, 'pasties': 1, 'philosophizing': 2, 'extracurricular': 2, 'satisfied': 12, 'overwrought': 1, 'steaming': 1, 'showed': 22, 'strident': 1, 'off-track': 1, 'reaching': 10, 'videotapes': 2, 'directed--': 1, 'cross': 26, 'robbers': 5, 'off-the-cuff': 1, 'measures': 5, 'inn': 1, 'inventiveness': 3, 'sudden': 22, 'war-': 1, 'observation': 9, 'activists': 3, 'painter': 4, "/let's-strip-down-the-sport-to-its-bones": 1, 'already-existing': 1, 'refrigerator': 2, "sweetback's": 1, 'monologue': 12, 'granger': 8, 'satyrical': 1, "sluizer's": 1, 'yim': 2, 'ludwig': 1, 'atheism': 4, 'sat': 7, 'rollergirls': 1, "'chateau": 1, 'bought': 19, 'optimistically': 1, 'inside': 73, 'suspension': 5, 'illicitly': 1, 'parodied': 2, 'amiss': 2, 'frustrated': 14, 'corruptor': 1, 'leit': 1, 'circumstance': 9, 'atkine': 1, 'fink': 3, 'refugees': 1, 'nfeatured': 1, "shang's": 1, 'impetuous': 1, 'sure-fire': 1, 'friction': 1, 'hour': 88, 'qui-': 1, 'chaotic': 5, "tiffany's": 1, 'lansbury': 1, 'beginners': 1, 'conventional': 26, 'beleive': 1, "contact's": 1, 'curtiz': 1, 'recreated': 3, 'excusable': 1, 'headlines': 5, 'spy': 12, 'ferry': 1, 'marty': 4, 'chews': 2, 'contemplate': 2, 'exceptional': 23, 'crooning': 2, 'quaternary': 1, 'surrendering': 1, 'bacri': 1, 'melvin': 3, 'caro': 1, 'seminar': 1, "zemeckis's": 1, 'benicio': 5, 'shoe': 4, 'domicile': 1, 'entry': 13, "marceau's": 1, 'wrestled': 2, 'marketable': 2, 'urbano': 1, 'ownership': 4, 'intermediary': 1, 'state-sanctioned': 1, '[pg-13]': 6, 'saws': 1, 'broken': 27, 'butoy': 1, 'wrong-way': 1, '`big': 1, 'debased': 1, 'brother-in-law': 1, 'deported': 1, 'paranormal': 2, 'earliest': 3, 'trooper': 5, 'fleshing': 4, 'simplicity': 9, "bentley's": 1, 'imagines': 3, 'bizarre': 39, 'trannsferred': 1, 'phil': 12, 'tithe': 1, "gloria's": 1, 'petition': 1, 'steer': 4, 'matt': 40, 'adaptations': 9, 'popcorn': 11, 'impression': 31, 'ragtag': 2, 'comedy/romance': 1, 'milks': 1, 'best-looking': 1, 'judicious': 1, 'underling': 1, 'scientologists': 1, 'apparitions': 2, 'frees': 1, 'taylor': 13, "family's": 11, 'visitors': 2, 'hubbard': 4, 'head-strong': 1, 'yvette': 2, 'zone': 8, 'cronenberg': 6, 'bitching': 3, 'lasse': 3, 'incongruous': 2, 'essences': 1, 'epitome': 6, 'lavishness': 1, 'orion': 2, 'anti-theft': 1, 'territories': 1, 'plethora': 2, 'generation': 31, 'questions': 73, 'ultra-violence': 1, 'rooster': 1, 'wrongdoings': 1, 'barks': 3, 'appeals': 7, 'animals': 12, 'michelle\x12s': 1, 'atrophied': 1, 'attended': 8, 'walks': 24, 'dowd': 1, 'henson': 4, 'peanuts': 1, "jury's": 1, 'furthers': 3, 'clout': 3, 'alert': 4, 'totemic': 1, 'flippers': 1, '14': 10, 'action-adventure': 1, 'laying': 3, 'entanglements': 2, 'wrapping': 2, 'reverse': 5, 'lasted': 2, 'fell': 15, 'swift': 1, "dinsmoor's": 1, 'reilly': 7, 'zodiac': 1, 'vcr': 5, 'bowls': 2, 'horror/comedies': 1, 'trendy': 2, 'shocked': 17, 'ambiance': 2, 'thesis': 4, 'viking': 1, 'overloaded': 2, 'chord': 3, 'saga': 13, "storm's": 1, 'travelled': 1, "liman's": 1, 'generously': 1, 'impressions': 3, 'accidentally': 16, 'cornflake': 1, 'craftily': 1, 'window-shopping': 1, "burton's": 10, 'semi-darkness': 1, 'suffered': 16, 'grudge': 3, 'gentle': 10, 'rehashed': 1, 'fact': 265, 'conventionalism': 1, 'tightly-wound': 1, 'catty': 1, 'smashingly': 1, 'non-fiction': 1, 'empowerment': 1, 'antagonist': 5, 'cellist': 2, "'sex": 1, 'development': 42, 'eighties': 5, 'much-needed': 2, 'scotia': 1, 'illusions': 5, 'unbeaten': 1, 'maxime': 1, 'beloved': 23, "herman's": 1, 'swooping': 1, 'heinlein': 3, 'split': 19, 'unintentional': 2, 'unflashy': 1, 'love-dependent': 1, 'sternly': 1, 'hunchback': 6, 'bridal': 1, 'begs': 7, 'ignoring': 2, 'tyler': 7, 'shrills': 1, 'massaging': 2, 'engrosses': 1, 'geographically': 1, 'grinds': 2, 'goose': 2, 'stella': 1, "nietzsche's": 1, 'helper': 2, 'salads': 1, 'perfectionist': 1, 'demon': 7, 'individuality': 2, 'squaddie': 1, 'doesn': 2, 'daughter-in-law': 1, 'uninhabitable': 1, 'south--forever': 1, 'geologist': 1, 'terms': 48, 'bragging': 1, 'regis': 1, 'burns': 9, 'substitution': 1, 'granny': 3, 'world--what': 1, 'decapitations': 1, 'scots': 2, 'stuff--except': 1, 'perspectives': 2, 'boardwalk': 1, "hours'": 4, 'throbbing': 1, 'motta': 1, '1937': 1, "walt's": 2, 'ingredients': 17, 'dictatorship': 1, 'specialised': 1, 'ciaran': 2, 'sake': 14, "'dogma'": 1, 'banderas': 4, "loki's": 1, 'swiss': 2, 'hot': 40, 'towers': 2, 'gathered': 3, "clancy's": 1, 'seediness': 1, 'cocooned': 1, 'deportee': 1, 'briggs': 1, 'mutinies': 1, 'nowheresville': 1, 'helecopters': 1, 'manufacture': 1, 'lsd': 1, 'matarazzo': 1, 'counterbalance': 2, 'portal': 5, 'cardshark': 2, 'prix': 1, '4': 21, 'tuxedos': 1, 'specter': 1, '44': 1, 'withdrew': 1, 'shopgirl': 1, 'tings': 1, 'timelessness': 1, 'adopted': 10, 'seduced': 4, 'contracts': 3, 'lohan': 1, 'includes': 41, 'stridency': 1, 'slasher': 16, "'93": 1, 'crotchety': 1, 'denzel': 11, 'educator': 2, 'distilling': 1, 'dazzling': 18, 'severity': 2, 'browning': 4, 'waded': 1, 'slow-motion': 7, "dallwitz's": 1, 'shogun': 1, 'slater': 3, 'lesbianism': 3, 'swear': 6, 'wordy': 2, 'highly': 70, "sctv's": 1, 'surfing': 2, 'thespian-at-large': 1, 'dummy': 2, 'gilles': 1, 'parfitt': 1, 'emphasis': 14, 'supplant': 1, 'tears': 25, 'muses': 2, 'stay': 74, 'sfpd': 1, 'cracraft': 1, 'cinematography': 77, 'junket': 1, "'90's": 1, 'wanted': 62, 'boo-hoo': 1, 'caption': 2, 'well-stocked': 1, 'snowbound': 1, 'das': 5, 'contempt': 10, 'centerpoint': 1, 'kind-hearted': 6, 'ratso': 1, 'guillotine': 2, 'broods': 2, 'rouen': 1, 'diversion': 3, 'insipid': 2, 'dissuade': 2, 'deschanel': 1, 'princeton': 3, '34-degree': 1, 'lazy': 11, "rick's": 2, 'profundities': 1, "bond's": 1, 'freedoms': 1, 'spoofy': 1, "mission'": 1, 'lew': 1, 'heavies': 1, 'approximately': 6, 'autonomy': 1, 'sheldon': 1, 'drawer': 1, 'misfire': 1, 'obsesses': 3, 'curse': 4, 'ensemble': 33, 'lovably': 2, 'dissimilar': 3, 'busload': 2, 'zaire': 1, "traders'": 1, 'lindas': 1, 'orgiastic': 1, 'blended': 4, 'cabin': 9, 'picturesque': 3, 'scheduled': 4, '48': 2, 'fill-in-the-blank-american-comedy': 1, 'chants': 2, 'dominique': 4, 'writing/directing/': 1, 'defied': 1, 'lon': 1, 'sync': 5, 'computer-enhanced': 2, 'fiefdoms': 1, 'block': 23, 'stupidity': 3, 'lieu': 1, 'kgb': 4, 'combating': 1, 'penthouse': 2, 'donald': 11, 'onassis': 1, 'ingenuous': 2, 'cheerfulness': 1, 'caution': 4, 'perfected': 3, 'pinon': 1, 'sophie': 5, 'clubs': 6, 'crummy': 1, 'reaper': 4, 'duels': 2, 'faithfully': 3, "austin's": 1, 'invincible': 6, "hartnett's": 1, 'theron': 4, '160': 2, "zoolander's": 2, 'bread': 8, 'blindly': 2, 'blankets': 1, 'amaze': 4, 'freaky': 2, 'nominal': 2, 'dave-good': 1, 'copycats': 2, 'films': 411, 'recurs': 1, 'solitude': 6, 'exacts': 2, 'sandrine': 1, 'not-quite-subtle': 1, 'murder-mystery': 2, 'heroes': 33, 'merciless': 4, 'ignites': 1, 'kerrigan': 1, 'vienna': 1, 'gazarra': 2, 'sequel/spinoff': 1, 'thursday': 1, 'distressed': 3, 'stunted': 1, 'synergy': 1, 'understood': 11, 'mystic': 4, '$280': 1, 'brewing': 6, 'all-consuming': 1, 'unexplainable': 1, 'haywire': 3, 'prohibition': 2, 'darkness': 22, 'telss': 1, 'rumor': 5, 'disdainful': 1, 'speaks': 29, 'fury-161': 1, 'fogy': 1, 'unpredictability': 4, 'madeline': 5, 'torpedoes': 1, 'meteorologist': 1, 'bink': 1, 'snobby': 2, 'professions': 1, 'foregrounded': 1, "giger's": 1, 'donager': 1, 'bard': 5, 'crispin': 1, '[katarina]': 1, 'savior': 3, 'judgment': 13, 'ninth': 3, 'composition': 5, 'nymphet': 1, 'eloi': 2, 'horror-action': 1, 'beliavable': 1, "walter's": 2, 'tyranny': 2, 'brutality': 11, 're-unite': 1, 'moneymakers': 1, 'semi-noir': 1, 'impossible': 52, 'yawk': 1, 'devises': 3, 'odds-maker': 1, 'tanktops': 1, 'sleigh': 1, 'tasha': 1, 'scientist': 23, "wilder's": 2, 'tastelessness': 1, 'whaley': 1, "gwtw's": 1, 'guillermo': 1, 'nearly-farsical': 1, 'miranda': 3, 'evaluate': 3, "geno's": 1, 'expatriates': 1, 'ozon': 1, 'lightweight': 3, 'paraphernalia': 2, 'banjo': 1, 'ratchet': 2, 'gambols': 1, 'enchantment': 1, 'keeley': 1, 'modes': 3, 'rated': 58, 'loutish': 1, 'nymphomaniac': 1, 'damned': 10, 'lester': 7, 'coastline': 1, 'dana': 3, 'livingston': 3, 'focused': 22, 'gungans': 2, 'sprocket': 1, 'airtime': 1, 'ichor': 1, 'stated': 8, 'balk': 1, 'stored': 2, 'thunderlips': 1, 'zanetti': 1, 'demarco': 1, 'behemoth': 2, "artist's": 3, 'rarely': 50, 'faggot': 1, "roy's": 1, 'unintentionally': 8, "kennesaw's": 1, 'maniacal': 3, 'outing': 12, 'nitpicked': 1, 'disdain': 2, 'braxton': 1, 'title--first': 1, 'valjean': 3, 'seemed--though': 1, 'slight': 17, 'foodmart': 1, 'laden': 3, "squaresoft's": 1, 'conventionality': 1, 'weak': 31, 'fireballs': 2, 'nuts': 2, 'smokey': 1, "logan's": 1, 'belgian': 2, 'egregious': 1, 'companionship': 2, "game'": 1, 'nickelodeon': 2, 'executing': 1, 'purer': 1, 'checking-in': 1, 'demesne': 1, 'grips': 4, 'papaya': 1, 'peter': 85, 'cicero': 1, "garafolo's": 1, 'schemes': 7, 'fighters': 6, 'accumulation': 1, 'bloodier': 1, 'smarmy': 4, 'servo': 2, 'absurdist': 2, 'akiva': 1, 'party-girl': 1, 'shoestring': 2, 'overlight': 1, 'beneficial': 1, 'bigger-than-life': 1, 'unnoticeably': 1, '`american': 2, 'interests': 11, 'munundating': 1, "israeli's": 1, 'incredibly': 51, 'corpses': 5, 'release': 87, 'rushbrook': 1, 'eliminated': 3, 'guard': 26, 'disclaimer': 4, 'sheerly': 1, 'exterminator': 2, 'time-consuming': 1, 'smitten': 8, 'felines': 1, 'pubbing': 1, 'axiomatic': 1, 'penniless': 3, 'alexa': 1, 'abolitionists': 4, 'familar': 1, 'goers': 5, 'shields': 2, 'embedded': 5, 'rubin-vega': 3, 'vomitted': 1, 'office': 64, 'multi-talented': 1, 'preview': 10, 'central': 51, 'cracking': 3, 'poverty': 6, 'strained': 7, "leia's": 1, '`not': 1, 'bungle': 1, 'count--at': 1, 'hymns': 1, 'extras': 10, 'regains': 1, 'darryl': 1, 'take-it-or-leave-it': 1, "'help": 1, "barry's": 1, 'steve': 47, 'co-starring': 1, 'shrugs': 1, 'pixar': 5, 'exercise': 19, 'costumer': 1, 'utah': 4, 'narratives': 3, 'overtly': 2, 'secluded': 2, 'spewed': 2, 'posing': 5, 'peck': 4, 'andrei': 1, 'inmates': 5, 'verne': 2, 'insisting': 2, 'pekurny': 1, 'shanta': 1, 'one-upmanship': 1, 'weatherman': 3, 'arthur': 7, 'snap': 6, 'meatiest': 3, "'invaders'": 1, 'christian': 21, 'rounds': 4, '3-d': 2, "1996's": 12, 'stuff': 69, 'kei-ying': 3, 'fishing': 2, "element's": 1, "one's": 31, 'choir': 1, 'fearful': 6, 'miracle': 5, 'award-worthy': 2, 'planetary': 3, 'disquieting': 3, 'sweetback': 1, 'carmel': 1, 'lampooning': 3, 'translators': 1, 'prehistoric': 2, 'heavens': 1, 'notches': 3, 'kindest': 1, 'chase-riboud': 1, 'streak': 8, 'yeshiva': 1, 'braschi': 3, 'hitting': 9, "tings'": 1, 'hutchison': 1, 'gestapo': 1, "ross'": 3, 'gevedon': 1, 'follow-up': 10, 'uprisings': 1, 'amberlike': 1, 'android-wishing-to-be-human': 1, 'bronson': 1, 'beta': 1, 'defected': 1, 'dizzying': 7, 'retarded': 2, 'idziak': 2, 'vocabulary': 4, 'chicanery--runneth': 1, 'live': 131, 'vibrance': 2, 'symbolism--in': 1, 'garlic-filled': 1, 'incompleteness': 1, 'graduated': 7, 'oreo-munching': 1, 'emergence': 2, 'exacty': 1, 'mercutio': 1, 'relish': 9, 'anecdote': 2, 'wich': 1, 'euphoric': 2, 'dolly': 4, 'tics': 4, 'six-month': 1, 'rosette': 1, 'adverse': 2, "blade's": 3, 'bastille': 2, 'rebelling': 1, 'stone': 36, 'hose': 1, 'positions': 6, 'community': 33, 'exist': 33, 'armand': 1, 'married': 68, 'dreadful': 7, 'reich': 2, 'politely': 2, 'horseplay': 1, 'ineptitude': 1, 'flat': 34, 'overheard': 1, 'raided': 1, 'ascetic': 1, 'magruder': 2, 'aspirations': 7, 'comedy-thriller': 2, 'loomed': 1, 'extramarital': 1, 'band': 38, 'stalked': 5, 'effected': 4, 'orphange': 1, 'hua-chi': 1, 'faux': 2, "heroes'": 1, 'mindful': 2, 'semi': 1, 'rocketry': 1, 'conversation': 28, 'narratively': 1, 'stormy': 6, "barrymore's": 1, "gnosis'": 1, 'soggy': 2, 'revitalized': 1, 'becase': 1, 'acne': 1, 'norman': 7, 'boatwright': 1, 'nazis': 8, 'immerses': 5, 'emptor': 1, 'unpolished': 1, 'waynes': 1, 'prompter': 1, "smith's": 9, 'packard': 1, 'wayland': 1, 'chick': 7, 'nihilistic': 5, 'borrowing': 3, 'abby': 5, 'sidetracked': 3, 'venna': 1, 'grasshoppers': 2, 'nemesis': 10, 'floats': 4, 'dangling': 2, 'com/~jimkendrick': 1, 'henrikson': 1, "kellogg's": 1, 'seated': 2, 'parkers': 1, 'fred': 12, 'convalescence': 1, 'lavish': 6, 'analyse': 1, 'funnier': 11, 'knish': 1, 'interface': 2, 'gloria': 11, 'incompetent': 4, 'stately': 1, 'beancurd': 1, 'wake': 11, 'overjoyed': 2, 'robert': 101, 'stripped-down': 1, 'gen-x': 2, 'anya': 1, 'staging': 5, 'talking': 59, 'anti-': 1, "flynn's": 1, 'displaying': 9, 'digesting': 1, 'finalist': 1, 'denver': 10, 'rhode': 1, 'floodwaters': 1, 'pursuer': 1, 'unfairly': 7, 'compromises': 2, 'insensitive': 2, 'snake': 7, 'jew': 5, 'planet': 48, "script's": 3, 'cast--most': 1, 'desparation': 1, 'knowing': 31, 'undressing': 1, 'landscapes': 5, 'matthau': 1, 'coverage': 5, 'schwartzenegger': 1, 'perversions': 1, 'hanging': 18, 'ethnic': 2, 'semi-smiliar': 1, 'wring': 1, 'skyline': 1, 'pacification': 1, 'goldman': 2, 'sith': 2, 'rome--but': 1, 'multiplexes': 1, 'shuttles': 1, 'cop': 38, 'thrilled': 11, 'mix': 37, 'formative': 1, 'couriers': 1, 'creeped': 1, 'unstimulated': 1, 'miraculously': 5, 'bracco': 1, 'dumbed': 1, 'problematical': 1, 'tamahori': 1, 'lamented': 1, 'austen-like': 1, 'html': 1, 'inspiring': 7, 'masquerade': 1, 'kiefer': 2, 'curious': 12, 'dissertations': 1, 'unenviable': 3, 'flavors': 1, "besson's": 4, 'bringing': 41, 'anti-movie': 1, "alan's": 1, 'exceptions': 15, 'asteroid': 3, 'council': 3, 'founded': 2, 'ultraviolet': 1, 'edit': 1, 'no-nudity': 1, 'god-like': 2, 'anticipation': 7, 'mechanical': 5, 'chic': 2, 'trials': 9, 'landmarks': 1, 'westerner': 1, 'persecuted': 1, 'egypt': 7, 'scariest': 5, 'rushing': 3, 'bruno': 5, 'similarly-structured': 1, 'sarcasm': 4, 'unimitible': 1, 'sunk': 9, 'spindly': 1, 'babysitting': 1, 'workplaces': 2, 'curves': 2, 'hockney': 1, 'equate': 2, 'kirk': 7, 'alyssa': 1, 'unintelligent': 1, 'barb': 1, 'costar': 3, 'saddening': 3, 'whizzing': 1, 'retentive': 1, 'exemplary': 2, 'hinting': 4, 'unmistakable': 2, 'vertigo': 4, 'resubmit': 1, 'infuse': 3, 'absorbs': 1, "protagonist's": 1, 'preys': 1, 'analogy': 5, 'chalks': 1, 'actualizing': 1, 'pretends': 7, 'strokes': 4, 'deja': 3, 'heche': 5, 'naturalism': 3, '_are_': 1, 'polonius': 1, 'clifton': 1, 'outcasts': 2, 'blackness': 3, 'satisfaction': 7, 'redford': 6, 'solitary': 6, 'consume': 3, 'knocked': 5, 'comics': 3, "club'": 2, 'masterpeice': 2, 'physique': 1, 'temple': 13, 'realises': 2, 'rafael': 3, "breakin'": 1, 'voyage': 9, 'unrepentant': 1, 'smashes': 4, 'kishikawa': 1, 'wide-angle': 1, 'kareem': 1, '_happen_': 1, 'perturbed': 1, 'buehler': 1, 'ralph': 17, 'watches': 16, 'machine--a': 1, 'cursing': 3, 'hardships': 3, 'stopper': 1, 'harassed': 3, 'hopeless': 3, 'incompletely': 1, 'offerings': 2, 'improbably': 2, 'videos': 12, 'aboriginal': 1, 'lucid': 2, "vaughan's": 1, 'liberals': 1, 'mcelhone': 3, 'recovered': 3, 'organised': 2, 'on-': 1, 'soavi': 1, 'outsmarted': 1, 'roommates': 6, 'cerebral': 6, 'pentameter': 1, 'flashed': 1, 'hostelries': 1, 'interviewer': 5, "broderick's": 3, 'diva': 1, 'demornay-like': 1, 'invest': 2, 'flynt': 7, 'surrounded': 20, 'out-of-place': 2, 'curdling': 1, 'impacted': 1, 'no-brainers': 2, 'und': 1, 'pressure': 8, 'scoundrel': 3, 'mox': 1, 'voight': 7, 'ruffle': 1, 'consistently': 22, 'interviewed': 1, 'patiently': 4, "`i'll": 1, 'atop': 5, 'notwithstanding': 6, 'closeness': 1, 'screem': 1, 'cohen': 5, 'realities': 8, 'porcine': 1, 'award-winning': 9, 'savor': 5, 'tendancy': 1, 'sharman': 1, 'violets': 1, 'maternal': 3, 'ruses': 1, 'wwii': 6, 'entertain': 28, '26th/1998': 1, 'net': 10, 'hum': 2, 'sticks': 14, 'stranded': 10, 'deprive': 1, 'cross-country': 2, 'gunshot': 1, "hanako's": 1, 'ostensible': 3, 'ballinagra': 1, 'guide': 10, 'writer/performer': 1, 'fatuous': 1, 'high-powered': 1, 'badge': 1, 'fate': 37, 'perception': 9, 'supermasochist': 1, 'splicing': 2, 'zemeckis': 3, 'inspite': 1, 'progressed': 4, 'involvement': 12, 'downsized': 1, 'regularly-updated': 1, 'outcast': 9, 'talent': 69, '1899': 1, 'newland': 1, 'rasaan': 1, 'lived': 20, 'due': 95, 'dared': 4, "stein's": 1, 'definetly': 1, 'expanded': 3, 'ghibli': 1, 'zipped': 1, 'emcee': 1, 'bennington': 1, 'offhand': 1, 'light-saber': 2, 'underground': 14, 'signaling': 1, 'shoes': 13, 'clerical': 2, "mm's": 1, 'gwythants': 1, 'cent': 2, 'fugly': 1, '$4': 1, 'begging': 4, 'critic': 20, 'botched': 7, 'gizmos': 1, 'telegraphs': 1, 'delineated': 2, 'searing': 1, 'spinning': 7, 'finally': 141, 'less-successful': 1, 'ngoc': 1, 'rotund': 2, "`darn'": 1, 'clubber': 2, 'cb': 1, 'six-year': 1, 'whedon': 2, 'lynn': 5, "coladas'": 1, 'robbed': 5, 'lifeboats': 4, 'butler': 9, 'outgrosses': 1, 'supporting': 132, 'showings': 3, 'goldenthal': 2, 'improvised': 2, 'fifteen': 22, "abc's": 1, 'meld': 1, 'allegory': 2, 'snub': 2, 'epps': 2, 'underwritten': 5, 'car/who': 1, 'surprisingly': 98, "nepotist's": 1, 'wind-bag': 1, 'fragmented': 1, "qt's": 1, 'secretive': 3, 'maul': 4, 'creepiness': 2, 'slightest': 11, '32': 1, 'rearranged': 1, 'starlet': 3, 'kinear': 1, "`goodfellas'": 1, 'tandy': 1, 'mould': 1, 'henstridge': 4, 'overrun': 2, 'donny': 7, 'heavy-handed': 9, 'dreary': 6, 'celia': 1, 'elementary': 3, 'nonexistent': 2, 'instict': 1, "fosse's": 1, 'patron': 2, 'realisticaly': 1, 'tinting': 1, 'mad-scientist-in-bavarian-castle': 1, 'meatier': 5, 'professes': 1, 'movement': 24, 'peasant': 3, 'glazed': 2, 'put-downs': 1, 'unapologetically': 2, 'star-studded': 2, 'evangelista': 1, 'voyeur-next-door': 1, 'spikes': 1, "taupin's": 1, 'birdy': 1, 'blown-up': 1, 'up-and-up': 2, 'exhilarating': 8, 'pg': 21, 'excitement': 25, "menace's": 1, 'lessons': 9, 'unfulfilled': 1, 'shawshank': 6, 'stacey': 1, 'mulan--get': 1, 'comparing': 8, 'nauseating': 4, 'stabbing': 2, 'qualifies': 5, 'stirring': 7, 'cheap': 37, 'eclipse': 1, 'varied': 4, 'second-fiddle': 1, 'sniper': 3, 'travel-bookstore': 1, 'redman': 7, "gallagher's": 1, 'backpedaling': 1, 'charts': 4, "kyle's": 1, 'guarding': 1, 'enzyme': 1, 'gauthier': 1, 'damned-if-you-do': 1, 'argued': 8, 'drift': 4, "naboo's": 2, 'mulqueen': 1, 'distraught': 6, 'uncovers': 5, 'mouths': 7, 'rattner': 1, 'fare': 29, 'spheeris': 1, 'dane': 1, 'flammable': 1, 'earthquake': 4, 'masterfully': 9, 'valiant': 3, 'nerdy': 2, 'fingertips': 2, "lancelot's": 1, 'mcgillis': 1, 'mcdonough': 1, 'harmed': 2, 'inhumanity': 1, 'slumber': 1, 'breezes': 1, 'classmates': 4, 'verona': 3, 'mediator': 1, 'ever-popular': 1, 'swamps': 1, "shue's": 1, 'kidnaps': 1, 'competition': 19, 'willful': 1, 'widowed': 4, 'fraser': 5, 'stations': 1, 'peacekeeper': 1, 'fearless': 3, 'retro': 3, 'curtain': 5, 'letterbox': 1, 'movies--everything': 1, "south's": 1, 'cloning': 1, 'allocate': 1, '$30': 2, 'reversalf': 1, 'suburbs': 1, 'losing': 31, 'duh': 2, 'unconnectedness': 1, 'misogynistic': 1, 'psych-tests': 1, 'federal': 14, 'co-wrote': 6, 'demonstration': 4, 'sytable': 1, 'cough': 2, 'argo': 1, 'pageant': 1, "makin'": 1, 'camp-like': 1, 'charlotte': 6, 'fling': 2, 'algar': 3, 'moral': 40, 'hollywoodised': 1, 'savage': 7, 'chapel': 3, 'digitally': 4, 'sophisticated': 19, 'cultish': 1, 'unobtrusive': 2, 'stapleton': 3, 'this-is-what-we-think-kids-want-to-hear': 1, 'super-tech': 1, 'paroled': 3, 'storagehouse': 1, 'lives': 128, 'dementia': 1, 'teenage': 31, 'advisors': 2, 'necessities': 1, 'self-rearranging': 1, 'vanish': 3, 'genuinely': 25, 'anti-war': 6, 'exceptionally': 9, 'minbo': 1, 'slinging': 1, 'baked': 2, 'riverboat': 1, 'appealingly-chirpy': 1, 'studios': 19, 'brent': 5, "waves's": 1, 'generous': 4, 'washington-supplied': 1, 'bestial': 1, 'outdoor': 6, 'zhang': 3, 're-mastered': 2, 'sonnets': 1, 'schnook': 1, 'unconventionality': 1, 'whitmore': 1, 'enrages': 1, 'million+': 1, 'establishes': 6, 'brannon': 2, 'hourglass': 1, '"': 695, 'questioning': 12, 'pass': 35, 'reading': 40, 'frenetically': 1, 'obsess': 1, 'father-son': 3, '_poltergeist_': 1, 'q-man': 1, 'klutz': 1, 'limbaugh': 1, 'divisible': 1, 'ambiguity': 7, "ravel's": 1, "accident's": 1, 'idiotic': 4, 'proudest': 1, 'distant': 20, 'manage': 33, "denuto's": 1, 'familiar': 83, "boolie's": 1, 'press--to': 1, 'sincerity': 11, 'save': 79, 'alice': 13, 'frenzied': 4, 'dramatization': 2, 'mise-en-scene': 3, 'arm-wrestling': 2, '60': 4, "weekly'": 1, 'whip': 2, 'fool-proof': 1, 'openness': 1, 'andrzej': 1, 'mckenna': 2, 'preteen': 2, 'expire': 1, 'thx': 2, 'nutty': 3, '12-year': 1, 'hand-delivered': 1, 'cinematographer': 38, 'egg-eyes': 1, '1/3': 1, 'interim': 1, 'climaxes': 5, "leiter's": 1, "kick-the-aliens'-asses": 1, 'writer-director': 6, 'clownish': 1, 'proffered': 1, 'exposes': 1, 'moments': 163, 'fairy-like': 1, 'sequel': 43, 'rides': 10, 'picture/love': 1, "intelligence's": 1, 'furthering': 1, 'wesleyan': 1, 'busts': 1, 'unworthy': 1, 'forbids': 5, 'grady': 2, 'wall': 23, 'cautiously': 1, 'driscoll': 2, 'facilitate': 2, 'falsehood': 2, 'bishops': 1, 'piecing': 1, 'impassioned': 5, 'loveable': 3, 'backfires': 3, 'footage': 40, 'stunts': 19, 'francois': 3, 'announcement': 3, 'omnipotent': 2, 'polarize': 1, 'barges': 1, "mongkut's": 1, 'struggled': 2, 'avenue': 1, 'mj': 1, 'appellation': 1, 'zoologist': 1, 'nether': 1, 'catches': 13, 'roughneck': 1, 'contemplation': 1, 'rebhorn': 2, 'lip-stick': 1, 'discern': 2, 'finely': 4, 'impressionist': 1, 'shield': 9, '70-something': 1, 'steals': 38, 'glossed': 1, 'portrayals': 7, 'eckstrom': 1, 'enlisting': 2, 'maura': 2, 'gambit': 1, 'rebels': 8, 'respectably': 1, 'rioters': 1, '#1': 3, 'cribbing': 1, 'hand-in-marriage': 1, 'mews': 1, 'babe': 15, 'owns': 16, 'loss': 24, '70s': 9, 'lenny': 3, "member's": 1, 'sherbedgia': 1, 'jointly': 1, 'scribbles': 1, 'alarming': 1, 'calcium': 1, 'inset': 1, 'dimwit': 3, 'darabont': 2, 'sparkles': 3, 'reactionary': 1, '1947': 5, 'senior': 13, 'odyssey': 12, 'expansive': 3, 'solvent': 1, 'gallop': 1, 'mounting': 3, 'panic': 13, 'learn': 76, 'carnal': 2, 'majors': 2, 'exquisite': 13, 'bedtime': 1, "wit's": 1, 'restoring': 4, 'distanced': 2, 'sullen': 4, 'clones': 1, 'refrains': 2, 'serenade': 1, 'box-sets': 1, 'stanislas': 1, 'rantings': 1, 'pleaser': 2, 'chapped': 1, 'iq': 4, 'entrapment': 1, 'muscles': 2, 'fucnniest': 1, 'simmrin': 1, "romulus's": 1, 'larenz': 2, 'deduction': 1, 'sickened': 2, 'schulman': 2, 'treehouse': 2, 'ambient': 2, 'farce': 11, 'shakur': 1, "zorro's": 1, 'jelly': 2, 'cemented': 1, 'bryan': 3, 'klein': 9, 'kennedy': 14, 'badness': 1, 'gall': 1, 'cinematographr': 1, "5'2": 1, "8's": 1, 'vincent': 22, 'stargher': 1, 'phony': 12, 'bloodbath': 2, 'holds-barred': 1, 'bachmann': 1, 'puerto': 5, 'remote-controlled': 1, 'toils': 1, 'cretaceous': 1, 'desperatly': 1, 'perused': 1, 'don\x12t': 1, 'hussein': 1, 'keyboardist': 1, 'rodman': 3, 'cider': 3, 'twenty-pound': 1, 'hoyts': 2, 'horror/sci-fi': 1, 'flipping': 1, "ratzo's": 1, 'clifford': 1, "butt'": 1, "folks'": 1, 'noble': 13, 'tahoe': 3, 'occupational-hazard': 1, 'feats': 3, 'stance': 3, 'continue': 26, 'skeletal': 1, 'hinckley': 1, "portman's": 2, 'starred': 12, 'childishness': 1, 'bleeth': 1, 'agents': 19, 'comforting': 7, 'tuneful': 1, 'shiftlessness': 1, 'riveting': 18, 'bet': 26, 'bong-hitting': 1, 'verging': 1, 'bourgeoisie': 2, 'tropical': 3, 'lighters': 1, 'set-up': 17, 'holding': 22, 'debates': 1, 'tonight': 5, 'cabinets': 2, 're-made': 1, 'animations': 2, 'unfulfilling': 1, 'reworking': 2, 'grieve': 1, 'non-': 2, 'falstaff': 1, 'drafted': 3, 'grubby': 2, 'effects-loaded': 1, 'erie': 2, 'auteil': 1, "friends'": 1, 'tellingly': 2, 'depressingly': 1, '_titanic_': 1, 'geared': 6, 'season': 33, 'secret': 70, 'manhatten': 1, 'winsome': 3, 'workaholic': 2, 'seamy': 2, 'flavor': 5, 'frantic': 8, 'meets': 93, 'prompts': 3, 'hard-driven': 1, 'nashville': 1, 'carradine': 1, 'letterboxed': 1, "'no'": 1, 'salutory': 1, 'chicanery': 1, 'norma': 3, 'warms': 1, "maclachlan's": 1, 'mcmullen-style': 1, '2040': 1, 'broadway': 10, "enterprise's": 2, 'firstly': 6, 'unzipped': 1, 'burnell': 3, 'overcame': 2, 'novikov': 1, 'funk': 2, 'refinement': 2, "lucas'": 4, 'bemoans': 1, 'mad-libs': 1, 'less-manic': 1, 'vonnegut': 1, 'shows': 185, "garbageman's": 1, 'valentin': 1, 'fixture': 2, 'deed': 4, 'tawdry': 3, 'fleshed': 4, 'frieberg': 1, 'blocks': 5, 'plumped': 1, 'relationship': 149, 'tried-and-tested': 1, 'advance': 9, 'repugnant': 3, 'starting': 22, 'goatee': 3, 'remixed': 1, 'owe': 2, '26min': 1, 'mouthed': 2, 'mustache': 1, 'fighter': 10, 'george': 71, 'hooper': 1, 'detached': 11, 'channels': 3, 'bulk': 10, 'areas': 8, '9': 15, 'supercilious': 3, 'zestful': 1, 'lone': 19, 'none-too-pleased': 1, 'mchugh': 1, 'vocalist': 1, "m's": 1, 'chosen': 32, 'accounts': 3, 'idiosyncrasies': 2, "payne's": 2, 'abbotts': 1, 'crisis': 21, 'trends': 5, 'district': 7, 'pinkie': 1, 'nietzsche': 3, 'erbe': 2, 'profile': 6, 'nandita': 1, 'top-notch': 16, 'synonymous': 2, 'modulated': 1, 'indifference': 4, "clarity's": 1, 'loosing': 2, 'separates': 4, 'cnn': 1, 'chest': 7, 'predestination': 1, 'aerial': 3, 'ekberg': 1, 'materialize': 1, 'aisles': 1, "tommy's": 3, 'participates': 2, 'lumumba': 2, 'proceed': 13, 'portions': 16, 'winters': 2, 'flick': 70, 'free-wheeling': 1, 'lovey-dovey': 1, "d-reper's": 1, 'surely': 34, 'penguin': 1, 'plague-riddled': 1, 'gardener': 4, 'mannered': 2, 'huy': 1, 'carrie-anne': 4, 'chicago-bound': 1, "france's": 2, 'reflected': 5, 'gloat': 1, 'mind--charlie': 1, 'rigid': 4, "malka's": 1, 'karenina': 1, 'mock': 4, 'unfeasible': 1, 'boney': 1, 'not-so-subtle': 1, 'josh': 10, 'carry-on': 2, 'warring': 1, 'hampshire': 2, 'harvey': 12, 'bargain': 5, 'heel': 1, '_casablanca_': 1, 'demanding': 14, 'off-chance': 1, 'impulsiveness': 1, 'trial': 19, 'flatuence-that': 1, 'nears': 4, 'badly-dubbed': 1, "monica's": 1, 'button-downed': 1, '$50': 5, 'hypocrites': 1, 'unavoidable': 4, 'androids': 1, 'loudness': 1, '108': 1, 'heralded': 1, 'stickler': 1, 'peices': 1, 'soft-eyed': 1, 'venturing': 3, 'dim-wittedness': 1, "wars'": 1, 'clerk': 5, '`fight': 2, 'lynchings': 1, "progress'": 1, 'hoser': 1, 'game-plan': 1, 'reaches': 17, 'kincaid': 1, 'distractedness': 1, 'misprinted': 1, 'respect-me-because-my-father-was-a-good-man': 1, 'bahamas': 2, "hall's": 2, 'non-threatening': 1, 'branched': 1, 'newcomer': 19, 'outhouse': 1, 'centered': 5, 'awk': 1, 'unsuccessfully': 4, 'sharper': 3, 'reitman': 2, 'cowgirl': 3, '1975': 6, 'diminish': 1, ':': 545, 'slimy': 9, 'sully': 1, 'preyed': 1, 'maneuver': 1, 'manhunt': 1, 'b-horror': 1, 'hesitant': 3, 'utmost': 1, 'street-wise': 1, 'oprah': 3, "potter's": 1, 'nikki': 2, 'suddenly': 55, "waller's": 1, 'unnecessarily': 7, 'morlocks': 1, 'york': 77, 'filler': 1, 'earth-shattering': 1, 'i=': 1, 'glide': 2, 'software': 1, 'prudent': 1, 'irritating': 15, 'outshine': 3, 'angles': 12, 'gesture': 6, 'envigorating': 1, 'garp': 2, 'garafolo': 3, 'matheson': 6, 'jim': 56, 'theater': 89, 'faint-hearted': 1, 'evenly': 3, 'favorites': 8, 'flooded': 2, 'maverick': 3, 'gestate': 1, 'moment': 116, 'joel': 19, 'stronger': 22, 'unmatched': 4, 'teen-targeted': 1, '`who': 1, 'mina': 1, '`faggot': 1, 'grieving': 6, 'trumps': 1, 'paladru': 1, 'credits-': 1, 'heavily': 16, 'maclaine': 1, 'fence': 6, 'caesar': 1, 'toga': 1, 'shoreline': 2, 'empathize': 6, 'solver': 1, 'octogenarian': 1, 'titanium': 1, 'extremly': 1, 'milieu': 3, 'gratuity': 1, 'big-league': 1, 'tatopolous': 1, 'prague': 1, 'seminal': 2, "benson's": 1, 'garfalo': 1, 'arbitrary': 3, 'ensure': 6, 'structures': 1, 'teeth': 14, 'americanized': 2, 'satellites': 1, 'mars': 14, 'ivana': 2, 'rewarding--a': 1, 'rumpled': 1, 'naked': 15, 'beauty': 57, 'nephew': 4, 'pleasantville-one': 1, 'plucked': 2, 'ambivalent': 2, 'wag': 4, 'nabbed': 1, 'appalling': 6, 'jeered': 1, 'theaters': 45, 'desperate': 38, 'deapens': 1, "braxton's": 1, 'willpower': 2, 'mckellan': 4, "trainee's": 1, 'foreseen': 1, 'bright': 35, 'rene': 4, 'realm': 14, "skipper's": 1, "factory's": 1, 'qualified': 4, "epstein's": 1, 'constraints': 1, 'hole': 14, 'melinda': 2, 'entity': 5, 'lan': 1, 'remote': 13, 'veteran': 38, 'moralistic': 2, 'hired': 18, 'awesome': 20, 'colisseum': 1, 'thrusting': 1, 'lisa': 15, 'proof': 17, 'jurassic': 21, 'underlying': 13, 'gardening': 3, "soldiers'": 2, 'oldest': 5, 'harlem': 4, 'brainstorm': 1, "hedwig's": 2, "burwell's": 1, 'jester': 2, "hawke's": 1, 'tainted': 1, 'yugoslavians': 1, 'horror': 101, 'bare-knuckle': 1, 'loyalties': 2, 'kint': 1, 'boinked': 1, 'frame-of-mind': 1, 'equality': 1, 'ernst': 1, 'ponderous': 1, 'man-portable': 1, 'corman-wannabe': 1, "army's": 1, 'candle-lighting': 1, 'drug-stoked': 1, 'whore': 2, 'hold-ups': 1, 'bible-quoting': 1, 'abuser': 1, 'funny': 206, 'bly': 1, 'flaky': 1, 'bellbottoms': 1, 'well-produced': 1, 'frozen': 8, 'molly': 8, 'stacks': 3, 'neighbor': 18, "nyah's": 1, 'hawtrey': 5, 'pollsters': 1, 're-evaluate': 1, '1968': 3, 'fearfully': 1, 'lad': 8, "cinque's": 3, 'caerthan': 3, 'talmud': 1, 'pitiful': 3, 'materialist': 1, 'monroe': 4, 'gymnasium': 1, 'fennymann': 1, 'crushed': 6, 'inherent': 12, "whitaker's": 1, 'pumps': 2, 'trees': 15, 'porcelain': 3, 'tunes--and': 1, 'bidet': 1, 'outnumber': 2, 'therapist': 6, 'deleting': 1, 'inventie': 1, 'salinger': 1, 'sniveling': 1, 'chappelle': 1, 'wily': 3, 'sonnenfeld': 4, 'latest': 70, 'floating': 12, 'impunity': 1, 'badly-written': 1, "kar-wei's": 1, 'unviable': 1, 'perez': 3, 'presences': 1, 'unlocked': 1, 'buggers': 1, 'displeasing': 1, 'scandalous': 2, "foley's": 1, 'never-ending': 1, 'arkin': 3, 'flesh-eating': 1, 'www': 4, "gast's": 1, 'bank': 20, 'lucas': 24, 'calling': 14, 'prequel': 4, 'running': 112, 'believeable': 3, 'alan': 32, 'literate': 4, 'harron': 1, 'lorna': 1, 'head-on': 3, 'hormone': 1, 'clubish': 1, 'paybacks': 1, 'reversing': 1, 'skinny': 3, 'yared': 1, 'summations': 1, 'unsuspecting': 4, 'eviction': 2, 'disasters': 6, 'minimums': 1, 'exchange': 18, 'bulletins': 1, 'rotting': 1, 'fictions': 1, 'announces': 9, 'competitors': 2, 'super-agent': 1, 'krige': 1, 'chronological': 2, 'devlin': 1, 'displacement': 1, 'visualizing': 1, 'repeatedly': 11, 'camping': 2, 'levy': 4, 'trusty': 4, "steiger's": 1, 'chaplin+s': 1, "darlene's": 1, 'd2': 1, 'chamberlain': 1, 'regaining': 2, "house'": 1, 'tenure': 1, '129': 2, 'ill-natured': 1, 'implicit': 1, 'haney': 1, 'finishing': 6, 'uniquely': 4, 'elvira': 1, "tv'": 1, 'afeminite': 1, 'fleder': 1, 'occupations': 1, 'evasions': 1, 'thanklessly': 1, 'stephens': 4, 'matrimony': 1, 'vinnie': 1, 'belongings': 3, 'hauser': 1, 'remaking': 1, 'stellar': 11, 'threatened': 9, 'beckon': 1, 'grunt': 2, 'voe': 1, 'fanged': 1, 'bare': 12, 'classify': 2, 'stylistically': 4, 'scare-fest': 1, 'underseen': 1, 'cultist': 1, "_eve's": 1, 'brushed': 3, 'karloff': 2, 'servant': 1, 'splein': 1, 'recognizable': 8, 'nesmith': 1, 're-released': 5, 'mousetrap': 1, 'pacing': 25, 'wildfire': 1, 'material': 75, 'tingle': 2, 'settling': 2, 'damaging': 1, 'hasidim': 1, 'landscaping': 1, 'hedaya': 5, 'shine': 21, 'blaine': 4, 'muscle-bound': 1, 'choose': 18, 'gig': 7, 'holst': 1, 'trainee': 2, 'evans': 5, 'brutalized': 1, "maxine's": 1, 'vulnerabilities': 1, 'hummable': 3, 'frightful': 2, 'cocoon-type': 1, 'unassertive': 1, 'sturm': 1, 'narcotic': 1, 'thumps': 1, 'suitors': 4, 'visible': 7, 'coppola': 8, 'lauded': 3, 'spoofery': 1, 'crewmates': 1, 'empath/psychic': 1, 'pepperidge': 1, 'witty': 51, 'lewis': 12, 'performig': 1, 'gasoline': 1, 'composer-lyricist': 1, 'savvy': 7, 'tongues': 3, 'kross': 1, 'pee': 2, 'oppressed': 3, 'trainspotting': 7, 'side-splitting': 4, '`go': 1, 'foe': 2, 'winners': 6, 'compelling': 47, 'wealthy': 23, "stallone's": 3, 'bidding': 2, 'convicted': 15, 'techs': 2, 'tilly': 1, 'relieve': 2, 'starship': 15, 'kiss': 22, 'animted': 1, 'choppily': 1, 'girlfriends': 4, "'hard": 1, 'shopping': 6, 'alligators': 2, 'revulsion': 2, 'declan': 1, 'flaunt': 3, 'filial': 2, "rosmary's": 1, 'obscure': 8, 'asbestos': 2, 'sublte': 1, 'brittle': 2, 'evangelizing': 1, 'branches': 3, 'happened': 67, 'brother/manager': 1, 'biblically': 2, 'whoopi': 4, 'committed': 20, 'malloy': 1, 'maxine': 3, "building's": 1, 'pendel': 1, 'plush': 1, 'ocious': 1, 'ample': 3, 'migraine-inducing': 1, "stone's": 7, 'sheds': 4, '1976': 2, 'hashish': 1, "jacob's": 3, 'stylist': 1, 'jurisdiction': 2, 'cop/new': 1, 'obtuse': 3, 'totalitarian': 3, 'dina': 4, 'holders': 1, 'raking': 2, '_shine_': 1, 'no-frills': 1, 'carlyle': 7, 'romance': 73, 'feasibility': 1, 'rothchild': 1, 'gloss': 4, 'newly-jarred': 1, 'woolsley': 1, 'franklin': 2, '77-year-old': 1, 'tremendous': 19, 'outpouring': 1, 'screen': 234, "asia's": 1, "firm's": 1, 'unchallenged': 1, 'depicted': 19, 'delay': 3, 'solidly': 1, 'buys': 7, 'booze': 3, 'flirting': 8, "star's": 4, 'majpr': 1, "satellite's": 1, 'haynes': 1, 'yanked': 1, 'swastika': 3, 'inflections': 1, 'comebacks': 1, 'backs': 4, 'marvelous': 23, 'boastfully': 1, 'euliss': 2, 'giles': 3, 'baileys': 1, 'eng': 1, 'muppets': 3, 't-bird': 1, 'disturb': 3, 'purged': 1, 'hardy': 1, 'francis': 16, 'adhere': 1, 'cramped': 3, 'koepp': 4, 'irvin': 4, 'inexperience': 3, 'neumeier': 1, 'out-number': 1, 'independence': 22, 'exploring': 16, 'cineplex': 1, 'contacted': 2, 'constructed': 11, 'delaware': 1, 'wee': 1, '`last': 1, "richardson's": 3, 'seven-year-olds': 1, "`ute'": 1, '1997': 39, 'opposed': 15, 'advancement': 2, '======': 1, 'supersedes': 2, 'payoffs': 2, 'desperados': 1, "shindler's": 1, 'emily': 10, 'rediscovering': 1, "`disneyfied'": 1, "mast's": 1, 'nastassja': 2, 'geocities': 2, 'uncertain': 10, 'endlessly': 12, 'platform': 6, 'pretentions': 2, 'ar': 1, 'release--and': 1, 'expelling': 1, 'spirit': 45, 'freeman': 17, 'punks': 1, 'bonnie': 6, 'slippery-smooth': 1, 'it+s': 2, 'flung': 2, 'allusions': 2, "scream's": 3, 'revolutionary': 11, 'lomax': 1, 'jealousy': 10, 'psychology-researcher': 1, 'buzzes': 1, 'deserted': 8, "maggie's": 2, 'spunk': 5, 'caters': 2, 'sopranos': 1, 'distorted': 5, 'meri': 1, 'pensive': 1, 'wasting': 1, 'eschewing': 3, 'avalon': 1, 'frain': 1, "'fear": 1, 'coast': 13, 'superiority': 2, 'human-like': 2, 'tanks': 1, 'attributed': 6, 'gorgeous': 27, 'amoral': 6, 'robocop': 6, 'torontonian': 1, 'melora': 3, 'sing-a-longs': 1, 'kiddies': 1, 'prompting': 2, 'hour-long': 1, 'co-ed': 2, 'self-absorbed': 3, 'intrepid': 1, 'timeliness': 1, 'collision': 4, 'utlizes': 1, 'slaughtering': 3, 'hoke': 1, 'ordinary--yet': 1, 'tyke': 1, "damned-if-you-don't": 1, 'watchmen': 1, '8mm': 2, 'pads': 1, 'jan': 11, 'veterinarian': 2, 'unprofessional': 1, 'schneider': 4, 'ordinarily': 2, 'existenz': 1, 'native': 17, 'vehicles': 6, 'whoa': 2, 'last-minute': 2, 're-evaluating': 1, "`stud'": 1, 'ron': 19, 'flashes': 6, 'answer': 38, 'loggia': 1, 'gale': 5, 'work': 332, 'threefold': 1, "amy'": 1, 'unhinged': 4, 'overdose': 4, 'hoskins': 2, 'lectures': 3, 'iranian': 1, 'emanates': 1, 'bandstand': 2, 'much-prized': 2, 'prepared': 23, 'surreal': 21, 'lore': 11, 'vigau': 1, 'reclusive': 1, 'forced': 99, 'earnestly': 2, 'cheating': 14, 'hegwig': 1, 'computer-assisted': 2, 'conductor': 3, 'one-ness': 1, 'clack': 1, 'schumacher': 1, 'boat': 24, 'ex-boyfriend': 3, 'departure': 16, 'trucker': 1, 'offered': 10, 'brennan': 1, 'turmoil': 15, 'angels': 15, 'bombast': 2, 'tassel': 1, 'clandestinely': 1, '&': 62, "j's": 1, 'sticking': 8, "kleiser's": 1, 'gerrald': 1, 'ons': 1, 'consciously': 2, 'dumpster': 1, 'prayer': 4, 'fete': 1, 'group': 125, 'scott': 47, 'catcher': 1, 'homecoming': 2, 'luxurious': 5, 'portrays': 28, 'clacks': 1, 'rh2': 1, 'clancy-esque': 1, 'jettisoned': 6, 'gadfly': 1, 'tone': 59, "song'": 2, 'magic': 41, 'tweedy': 1, 'blows': 12, 'dogged': 2, 'stall': 3, 'di': 1, 'morissey': 1, 'copious': 2, 'reprieve': 1, 'pig-like': 1, 'ehren': 1, 'weeping': 2, 'intrigued': 8, 'alleyways': 1, 'aids-cautionary': 1, 'orchestrated': 3, 'abider': 1, '1974': 5, 'overwhelming': 12, 'sailing': 3, 'flakes': 1, 'foregone': 1, "daughter's": 4, 'abductees': 1, 'concern': 9, 'intricate': 15, 'teacher-student': 1, 'donne': 1, 'tray': 2, 'disco-era': 1, 'mansell': 1, 'ornery': 1, 'findings': 1, 'built': 31, 'zeta-jones': 2, 'vixen': 1, 'delete': 1, "dosen't": 3, 'hairspray': 1, 'motherlode': 1, 'phifer': 3, 'longevity': 2, 'distinctions': 1, 'oxford': 1, 'appearance': 50, 'resolving': 1, 'leaders': 11, 'blinding': 1, 'editions': 1, 'criticise': 1, "hoyts'": 1, "charlies'": 1, 'elliptical': 1, "condon's": 1, 'pitched': 2, 'mclachlan': 2, 'jeffery': 1, 'co-operation': 1, 'homosexuals': 3, 'bottle': 7, 'academy-beloved': 1, 'mindsets': 1, 'caraboo': 1, 'ditzism': 1, 'mid-film': 1, 'wolfgang': 4, 'techno': 5, 'commandant': 2, 'hirings': 1, 'todd': 10, "leadership'": 1, "harwood's": 1, 'wardens': 1, 'monk-like': 1, 'prepares': 6, 'bucket': 2, 'imperfections': 2, 'compromising': 4, 'boucing': 1, 'waking': 9, 'simplying': 1, 'imaganitive': 1, 'cronenbergto': 1, 'subtitles': 11, 'star-69': 1, 'monks': 2, 'elevated': 5, 'grander': 2, 'film-makers': 1, 'mesmerising': 1, 'masking': 3, "kurosawa's": 4, 'half-hearted': 2, 'mascara': 1, 'dearest': 1, 'liner': 6, '[in': 1, 'biblical': 2, "days'": 2, 'lured': 3, 'itty': 1, 'yavin': 1, 'lawman': 1, 'specific': 12, 'motor': 3, 'distractions': 5, 'existing': 3, 'appetite': 2, 'soviet': 8, "ritchies's": 1, 'paddle': 1, '||': 1, 'tens': 1, 'disneyland': 3, 'titantic': 1, 'ravel': 1, 'stalker': 2, 'outsmarts': 1, 'error': 5, 'git': 1, 'existential': 5, 'unstable]': 1, 'joker': 5, 'surviving': 12, "publisher's": 1, 'take-off': 2, 'embracing': 3, 'lem': 2, '1983': 4, "hildyard's": 1, 'all-terrain': 1, 'alvin': 1, 'back-seat': 1, 'malt-liquor': 1, 'materials': 2, 'spotty': 1, 'glides': 2, 'bounty': 1, 'airy': 2, 'liotta': 3, 'associate': 10, 'shoved': 3, 'categorized': 1, 'rehabilitated': 1, 'pau': 1, 'rediscovers': 1, 'monotone': 1, 'interweaves': 2, 'epic-wannabe': 1, "callin'": 1, 'confidently': 5, 'guards': 14, 'designers': 2, 'coveted': 1, 'inch': 6, 'preordained': 1, 'defying': 2, 'financiers': 1, 'medical-gross-outs': 1, 'gen': 2, 'determined': 39, 'kick': 14, 'orchestral': 1, 'rat-infested': 1, 'heebie-jeebies': 2, 'seeds': 1, 'possibilites': 1, 'freeing': 4, 'sparkle': 3, 'twitches': 1, 'resurgence': 1, 'hollow': 9, 'nay': 1, 'occupying': 2, '29': 3, 'blanchett': 4, 'curing': 1, 'amenities': 1, 'typified': 1, 'illuminates': 1, '[theater': 6, 'til': 1, 'pm': 1, 'shortcomings': 12, 'sydney': 3, 'tendencies': 7, "claire's": 3, 'walking': 36, 'gulf': 6, 'speakers': 1, 'casuality': 1, 'saps': 1, 'drained': 4, 'pfcs': 1, 'qualify': 4, 'supplied': 2, 'jemmons': 2, 'overpraise': 1, 'horniness': 1, 'smugness': 1, 'transporters': 1, 'lull': 1, 'psycholically': 1, 'spherical': 1, 'beverly': 10, 'ingenius': 1, 'narrated': 5, 'impressive': 94, 'integrate': 3, 'torn': 19, 'notices': 11, 'flees': 8, 'bumstead': 2, 'ciggy': 1, "zwigoff's": 1, 'funding': 8, 'darn': 12, '45': 3, 'apparently': 66, 'crusader': 2, 'hollers': 1, 'phillips': 4, 'copulating': 1, 'cameos': 17, 'syndrome': 5, 'dorn': 3, 'survivor': 9, 'dreaded': 9, 'bowels': 3, 'independents': 1, 'neighbour': 1, 'boarded': 2, 'valid': 7, 'christened': 1, 'morph': 1, 'richter': 1, 'awards': 21, 'jarring': 13, 'poetical': 1, 'interchangeable': 1, 'soldiers': 28, 'excepts': 1, 'expressively': 1, 'freemasons': 1, 'renewal': 2, 'uncharacteristic': 2, 'humanism': 1, 'scum': 10, 'drastically': 3, 'functional': 2, 'waves': 18, 'tips': 5, 'profitable': 6, 'erstwhile': 1, 'near-final': 1, 'amen': 1, 'assassin': 8, '8-years': 1, 'squandered': 1, 'fuse': 3, 'culinary': 2, 'bloody-red': 1, 'osmond': 4, 'rile': 1, 'spots': 22, "dicaprio's": 1, 'weanies': 1, 'videocameras': 1, 'sulaco': 1, 'corporeal': 1, 'amplifying': 1, 'challenge--not': 1, 'stunningly': 5, 'inter-racial': 1, "blanchett's": 1, 'confessions': 3, 'promiss': 1, 'unreal': 4, 'foreseeable': 1, 'ivanek': 1, 'fond': 11, 'skills': 32, "_beloved_'s": 1, 'oed': 1, 'pretenses': 3, 'adaptions': 1, 'covered': 20, 'half-human': 2, 'thundering': 2, 'science': 53, 'reinterpretation': 1, 'administrative': 1, "'five": 1, 'fish-out-of-water': 1, 'traumatizes': 1, "l'argent": 1, 'volcano': 4, 'browns': 1, 'ben': 56, 'misstep': 3, 'gloating': 1, 'injured': 5, 'leonardo': 13, 'rota': 1, '`bring': 1, 'lidner': 1, 'wish-fulfillment': 1, 'racism': 11, 'maclean': 3, 'prosky': 1, 'half-grabbing': 1, 'puddle': 1, 'mockumentary': 3, 'argue': 16, 'kriss': 1, 'non-related': 1, "o'donnell's": 2, 'begrudgingly': 3, 'ghallager': 1, 'giorgio': 5, 'creativity': 9, 'dayton': 1, 'corporals': 1, 'arab-americans': 1, 'bungee-jumping': 1, 'hsu': 1, 'dorrance': 1, 'disconnecting': 1, 'characters': 424, "pyle'": 1, 'tempo': 3, 'blind-side': 1, 'director/executive': 1, 'obstructions': 1, 'breezy': 5, 'recently-widowed': 1, 'hilarity': 8, 'ufo-related': 1, 'ex-lawman': 1, 'exhaling': 1, 'shrift': 2, 'rejuvenates': 1, 'officials': 10, 'ingeniously': 2, 'chatter': 3, 'copyrighted': 1, '1963': 4, 'cop-gone-bad': 1, 'duloc': 1, 'story-wise': 2, 'flirtations': 2, '`': 1, 'merchandising': 6, 'suppression': 1, 'whitaker': 3, 'face-first': 1, '712': 1, 'imprison': 1, '750': 1, 'elbows': 1, 'stallone': 11, 'awhile': 7, 'submerged': 4, 'rubbish': 2, 'transpires': 4, 'height': 7, 'herzog': 2, 'bikers': 1, 'blusterous': 1, 'snatched': 1, 'horatio': 2, 'harmful': 2, 'mtv': 13, 'hagerty': 1, 'contact': 35, 'panes': 1, 'fabulous': 13, "'blew": 1, 'conveniences': 1, 'uneven': 13, "schiff's": 1, 'undermine': 2, 'coy': 1, 'saigon': 2, 'smack': 1, 'hieroglyphics': 1, 'germinate': 1, 'harmony': 5, 'sister-in-law': 2, 'jacques': 7, 'orchestra': 5, 'boundary': 2, 'choral': 1, 'attractive-seeming': 1, 'riotously': 3, 'maddening': 1, 'miscalculation': 1, 'homemaker': 1, 'apologizing': 2, 'sex-pervert': 1, 'power-oriented': 1, 'asleep': 7, 'cyril': 1, 'buenos': 1, 'contributed': 5, 'long-standing': 1, '_mafia_': 1, 'interfered': 1, 'unsinkable': 3, 'cope': 14, 'fantasizing': 2, 'uncompromisingly': 1, 'fireworks': 2, 'drawn-out': 1, 'batfans': 1, 'snakes': 2, "grim's": 1, 'cliches': 31, 'non-stop': 12, 'invocation': 1, "boss's": 1, "monkey'": 1, 'exotically': 1, 'retooling': 1, 'littered': 6, 'sucessful': 1, 'unseasonably': 1, 'ever-enjoyable': 1, 'exhibiting': 1, 'dispatches': 1, 'unbearably': 3, 'win-at-all-costs': 1, 'miyake': 1, 'exception': 54, 'dimwitted': 1, "wonders'": 1, 'brainy': 3, 'wargames': 1, 'gawky': 1, 'soulless': 5, 'immersion': 1, 'wrapped': 18, 'levied': 1, 'hellhole': 1, 'fulci': 1, 'irate': 1, 'willard': 4, "'hope": 1, 'dancing': 18, 'shut': 14, 'continuation': 1, 'partnership': 2, 'hos': 1, 'guido+s': 1, 'high-rise': 1, 'lit': 9, 'do--this': 1, 'depict': 10, 'vegetables--often': 1, 'khanh': 1, 'screen--the': 1, 'hypnotic': 5, 'well-thought': 2, '6th': 1, 'laughter': 17, 'menu': 2, 'bettered': 1, 'distributors': 3, 'ceo': 1, 'fascination': 16, 'vaccaro': 1, 'services': 5, 'meddle': 1, 'murrieta': 1, 'metaphysical': 5, 'gab': 1, '#3': 1, 'broadly': 5, 'foley': 7, 'extinguished': 1, 'lia': 1, 'reignited': 1, 'corinthians': 1, 'seclusion': 1, 'naval': 12, 'well-meaning': 5, 'tangible': 2, 'ditched': 2, 'undo': 3, 'falwell': 1, 'stench': 1, 'historians': 1, 'spiritual': 19, 'china': 21, 'greenstreet': 1, 'honour': 5, 'schtick': 6, 'luster': 2, 'redneck': 3, 'applaud': 6, 'reproduce': 1, 'pedantic': 2, 'canon': 3, 'well-secured': 1, 'jamie': 14, 'sauna': 2, 'girl-power': 1, 'marmet': 1, 'thompson-based': 1, 'risky': 12, 'tool': 12, 'outcomes': 1, 'not-': 1, 'library': 7, 'electrify': 1, 'circled': 1, 'windsor': 7, 'characterize': 1, 'starved': 1, 'ephesians': 1, 'necessity': 6, 'unconscious': 3, 'seriousness': 10, "pixar's": 5, 'conciousness': 1, "university's": 2, 'droves': 1, 'stratham': 1, 'all-too': 1, 'high-octane': 2, 'socially-acceptable': 1, 'lowest': 4, 'landing': 9, 'hyman': 1, 'caroline': 1, 'turnpike': 2, 'franz': 5, 'oceanscape': 1, 'obssession': 1, "allie's": 1, '40': 15, 'obliges': 1, 'medak': 1, "kaye's": 2, 'russel': 1, 'ingred': 1, 'farley': 1, 'foreigners': 4, 'created': 77, 'holliday': 1, 'narrates': 9, 'loving': 38, 'co-director': 3, 'fast-talking': 7, "on's": 1, "nadia's": 1, 'notoriously': 5, 'unpretentious': 3, 'barriers': 3, 'smaller': 18, 'continues': 48, 'cloudy': 1, 'canderday': 1, 'roundup': 2, 'much-discussed': 1, 'regent': 1, 'combed': 1, 'badmouth': 1, 'orleans': 4, 'stump': 1, 'texts': 1, 'treetops': 1, 'carl': 13, 'driver': 40, 'half-expecting': 1, 'moderate': 4, 'terrance': 2, 'adversely': 1, 'ruffini': 1, '12th': 1, 'kitty': 2, 'linger': 9, 'what-': 1, 'commodity': 5, 'edvard': 1, "kurtz's": 1, 'owned': 14, 'intermission': 2, 'cream': 6, 'hot-and-cold': 1, 'arranging': 2, 'pay-off': 2, 'locker': 4, 'foresee': 2, 'minstrels': 1, 'vuh': 1, 'agitated': 1, 'capitalized': 1, "pirate's": 2, 'resounding': 1, 'figurative': 4, 'stopped': 11, 'moonlight': 1, 'single-minded': 1, 'endeavored': 1, 'daughters': 6, 'resolved': 7, 'fictitious': 3, 'insult-throwing': 1, 'wonderful': 112, 'equals': 5, 'game-like': 1, '151': 1, 'candor': 1, 'mcteer': 1, 'coeur': 2, 'horses': 7, 'tsang': 1, 'guided': 6, 'romantic-mistaken': 1, 'auteuil': 3, 'metatron': 3, 'surrealist': 1, 'rampage': 1, 'preferred': 3, 'feet': 32, 'jessica': 12, 'goth-looking': 1, 'mantis': 1, 'aspect': 66, 'lars': 3, 'savages': 1, 'observed': 8, 'oscar-nomination-worthy': 1, 'television-addicted': 1, 'flee': 3, 'pbs': 2, 'on-again-off-again': 1, 'outfield': 1, 'dominate': 6, 'prosecuting': 1, 'behaviour': 10, 'sulking': 1, 'bells': 3, 'innovative': 13, 'merit': 10, 'fluke': 1, 'bleeding': 4, 'warburton': 1, 'seat': 33, 'noctis': 1, 'impossibly': 5, 'ruins': 7, 'wiring': 1, 'licks': 1, 'denmark': 2, "saunder's": 1, 'inclined': 5, 'bums': 3, 'unstoppable': 2, 'embarrassment': 3, 'downplaying': 1, 'practiced': 1, 'transmits': 1, 'persuasively': 1, 'helpful': 5, 'ongoing': 10, 'karey': 1, 'solve': 12, 'likened': 2, 'disturbances': 1, 'leg': 10, 'animated': 47, 'wiper': 1, 'seti': 4, 'stirs-he': 1, 'shovels': 1, 'scattershot': 1, 'footloose': 2, 'identical': 6, 'kansas': 1, 'unsung': 1, 'religious': 26, 'us$100': 2, 'mcfly': 1, 'amistad': 6, 'insincerity': 1, 'patriotism': 5, 'reached': 16, 'verbose': 1, 'ingratiating': 1, 'kravitz': 1, 'bad-ass': 1, 'laudible': 1, 'boutte': 1, 'experienced': 16, 'inarguable': 1, 'ledge': 1, 'caused': 14, 'griebe': 1, 'journals': 1, 'outpourings': 1, 'brilliance': 14, 'mutha--shut': 1, 'draft': 3, 'ann-moss': 1, 'captions': 1, 'longish': 1, 'renegade': 6, 'psycho-sexual': 1, 'recast': 1, 'etymology': 1, 'adolescents': 1, 'team': 80, 'viciously': 6, 'pell': 1, 'dose': 10, 'costars': 1, 'jackson': 28, 'adjani': 1, "first's": 1, 'brownie': 1, 'problems': 101, "game's": 2, 'eva': 1, 'tomb': 1, 'english-language': 1, 'baptizes': 1, 'mistakes': 10, 'iridescent': 2, 'improper': 2, 'dharma': 1, 'shockingly': 6, 'superlative': 1, 'sub-plot': 4, 'zealously': 1, 'boyce': 2, 'mayhew': 3, 'warhead': 1, 'cement': 2, "faith'": 1, "protesters'": 1, 'bucketsful': 1, 'cobble': 1, 'publicly': 1, 'silenced': 1, 'purple_': 1, 'mojo': 5, 'scene--is': 1, 'damsel': 1, 'ephron': 3, 'floom': 1, 'avert': 1, 'suppoosed': 1, 'disco': 9, "it's-a-spielberg-film": 1, 'emrboidered': 1, 'elexa': 1, 'unmemorable': 1, 'crowded': 7, 'complaining': 9, 'kara': 2, 'remain': 38, 'bulls-eye': 1, 'maintained': 4, 'mister': 1, 'dusk': 8, 'cashing': 2, "beek's": 1, 'sentimentality': 5, 'talkie': 1, 'pate': 1, 'replacement': 13, 'oakley': 2, 'nolan': 1, 'star_': 1, 'diplomat': 1, "hot-fudge-rockin'": 1, 'accusing': 1, 'coe': 1, 'dislikes': 1, 'godly': 2, 'campion': 2, 'renown': 1, 'pill-popping': 3, 'ex-baseball': 1, 'romanticizing': 1, 'ditch': 2, 'troy': 3, 'kristy': 2, 'percent': 6, 'grunts': 2, 'malice': 1, 's&l': 1, 'works': 169, 'seeking': 20, 'openly-homosexual': 1, '289': 1, 'breathy': 1, '36': 3, '1792': 1, 'splash': 7, 'mariachi--director': 1, 'granddaughter': 1, 'princess': 20, 'clearillusions': 1, 'secondary': 10, '[animated': 1, 'film-': 4, 'world-reknowned': 1, 'wonderfully': 57, 'bliss': 3, "_shaft_'s": 1, 'hoopla': 3, 'accident': 41, 'cluttered': 4, 'business': 77, 'hynek': 1, 'state': 72, 'dressmaker': 2, 'motel': 5, 'moff': 1, 'militants': 1, 'downloaded': 1, 'upwardly': 1, 'teachings': 2, 'discouraged': 2, 'lieutenant': 10, 'charming': 48, 'geek': 10, 'revelry': 1, 'enable': 4, 'pilot': 18, 'politcally': 1, 'donning': 2, 'mania': 3, 'decide': 52, 'mistrustful': 1, 'elise': 8, 'hitherto': 1, 'unauthorized': 1, 'jimkendrick@bigfoot': 1, 'envy': 8, 'cracks': 6, 'sadist': 2, 'astoundingly': 1, 'pours': 2, 'sepia-toned': 2, 'charlize': 4, 'self-acknowledgment': 1, 'prominence': 2, 'drummer': 5, 'cum': 2, 'approached': 10, 'telepathically': 2, "yankovic's": 1, 'denial': 10, 'stress-inducing': 1, 'distrustful': 3, 'phonation': 1, 'one-third': 1, 'conservatism': 1, 'piety': 1, 'white-haired': 1, "partygoers'": 1, 'goldberg': 8, 'bladders': 1, 'regales': 1, "vitti's": 2, "grier's": 3, 'attorney': 21, 'videodrome': 1, 'intersection': 2, 'tending': 1, 'clause': 3, 'delmar': 1, 'secrets': 14, 'sentient': 1, 'constrast': 1, 'steel': 13, 'lucidity': 2, 'jakoby': 1, 'icons': 8, 'fable': 15, 'migration': 1, 'protective': 8, 'claudius': 2, 'postcard': 2, 'beaten-up': 1, 'pattern-like': 1, "ames'": 1, 'educating': 1, 'philander': 1, 'decades-spanning': 1, 'registration': 1, 'manifestation': 1, 'blume': 1, 'quantities': 1, 'movies': 339, 'funeral': 12, 'hansel': 4, 'non-human': 2, 'militia': 2, 'catchy': 14, "harris'": 1, 'magnus': 1, 'well-suited': 1, 'lollipop': 1, 'get-out': 2, 'effective--contribution': 1, 'spot': 35, 'mitigated': 1, 'heedful': 1, 'black-clad': 2, 'supremacist': 1, 'paper': 16, 'personified': 3, 'turntables': 1, 'highly-energetic': 1, 'vulcan-like': 1, 'boy-o': 1, 'sanders': 2, "'freedom'": 1, "fonda's": 1, 'confessor': 1, 'london': 28, 'banking': 3, 'mulan': 10, 'street-smart': 1, 'alcohol': 2, 'idiot': 5, 'downward': 2, 'analogue': 1, 'reconciliation': 2, 'evolution': 5, 'nations-type': 1, 'guideline': 1, 'decline': 3, 'hardware': 1, 'deflower': 1, 'sponsors': 2, 'regret': 10, "schreck's": 1, 'researches': 1, 'aghast': 1, 'seville': 1, 'uproariously': 4, 'schindler': 2, 'award-calibre': 3, 'hang': 22, 'glassy-eyed': 1, 'adventure/drama]': 1, 'totally': 42, 'backdown': 1, 'drilling': 1, 'murder': 83, 'oscar-nomination': 1, 'giggs': 1, 'devised': 5, 'super-computer': 1, 'edtv': 3, 'shaker': 1, 'guffaws': 1, 'warfare': 4, 'entrancing': 2, 'improvisations': 1, 'one-eyed': 1, 'psychologically': 5, "branagh's": 6, "murdoch's": 1, 'chronic': 2, 'representation': 4, 'spineless': 1, 'world-': 1, 'european': 11, 'saintly': 1, 'daisy': 5, 'gym': 5, 'camera-time': 1, 'dim-witted': 7, 'failure': 19, "mcelhone's": 1, 'meditate': 2, 'collected': 6, 'getz': 1, '``high': 1, 'speeches': 8, 'stolen': 31, 'outcome-there': 1, 'interiors': 5, 'watering': 1, 'bullets': 15, 'practice--margaret': 1, 'seek': 16, 'appreciable': 1, 'brady': 4, 'zed': 1, '2nd': 4, 'tawdriness': 1, 'hears': 16, 'sword-fight': 1, 'off-key': 1, "model's": 1, 'cries': 3, '_and_': 1, 'cates': 2, 'stuffed': 7, 'between-the-line': 1, 'hearings': 2, 'august': 13, "maurice's": 2, "pariah's": 1, 'drags': 19, 'matsumoto': 1, 'cop-out': 1, 'harper': 4, 'stimulating': 8, 'all-stops-out': 1, 'powder': 3, 'repertoire': 3, 'sabian': 1, 'persian': 1, 'audience-pleasing': 1, 'rackwitz': 2, 'galahad': 1, 'aryans': 1, 'danger': 26, 'rolex': 1, 'pre-conceptions': 1, 'senate': 4, 'plugged': 1, 'arch': 2, 'companions': 6, 'training': 19, 'drifting': 2, 'unchangeable': 2, 'boyish': 2, "computer's": 1, 'older': 43, 'single-mindedly': 1, 'exams': 2, 'finalized': 2, 'prohibits': 1, 'herc': 1, 'anderson': 20, 'blaster': 1, 'recommended': 10, 'ex-assistant': 1, 'toying': 2, 'kozmo': 1, 'investment': 5, 'chucky': 2, 'liberating': 4, 'hatosy': 1, 'admission': 8, 'heaven-sent': 1, 'edwin': 1, 'matinee': 6, 'missions': 4, 'action/animation': 1, 'making-of': 1, 'ghetto': 5, 'transcripts': 1, 'pillaged': 1, 'documentation': 3, 'goggles': 1, 'bees--both': 1, 'plumbing-supply': 1, 'homogeneity': 1, 'herald': 1, 'cattle': 2, 'sam': 40, 'gunfights': 2, 'facial': 12, "everything's": 2, 'vatican': 1, 'thwart': 2, 'brain-damage': 1, 'glowers': 1, "inmate's": 1, 'cantakerous': 1, 'affordable': 2, 'warwick': 1, 'pippa': 1, 'interminably': 1, 'rainmaker': 3, "princess's": 1, 'unwelcomed': 1, 'overcome': 23, 'contemplating': 3, 'debussy': 1, 'painted': 7, 'tongue-in-cheek': 4, '2-disc': 1, 'slang': 7, 'expectedly': 1, 'soup': 4, "polanski's": 1, 'tamara': 3, 'teeter': 1, "judith's": 1, "giles'": 1, 'enliven': 1, 'acumen': 1, 'colleen': 3, 'hoggett': 1, 'eye-candy': 3, 'dogmatism': 1, 'abundence': 1, 'pad': 4, 'john': 226, 'hartley-ian': 1, 'entices': 2, 'caper-gone-awry': 1, 'trademark': 14, "anne's": 1, 'adoration': 2, 'switching': 5, 'fantasy': 37, 'blues': 12, 'trumpet': 1, 'chapter': 8, 'wheezy': 1, "'don't": 1, 'sends': 26, 'boss': 37, 'animal-loving': 1, 'adequately': 2, 'reverses': 1, 'nations': 2, 'spencer': 6, 'determinedly': 1, 'unpromisingly': 1, "gosse's": 1, 'tolerating': 1, 'testosterone-driven': 1, '4th': 3, 'heavy-handedness': 1, 'der': 2, 'negotiator': 2, 'buried': 9, 'blur': 2, 'plot': 319, 'kretschmann': 1, "jedis'": 1, 'shafts': 1, 'fruitless': 2, 'meloncholy': 1, 'rough-and-tumble': 1, "instructor's": 1, 'ihmoetep': 1, 'scathing': 6, 'darby': 3, 'mystical': 12, 'solely': 13, 'golden-era': 1, 'skepticism': 2, 'prom': 9, 'rendered': 9, 'chu': 1, 'ward': 11, 'squib': 1, 'ged': 1, "disney's": 22, 'conclude': 3, 'partially-realized': 1, 'irreplaceable': 1, 'pouncing': 1, 'tyrannic': 1, 'martyrs': 1, 'ring': 16, 'procedures': 1, 'elude': 4, 'responsibility': 11, 'labels': 1, 'pinnochio': 1, 'rarest': 2, 'insights': 3, 'stewardess': 2, 'sorta-redux': 1, 'europe': 11, 'tinier': 2, 'plot-wise': 3, 'liaisons': 1, 'importantly': 24, 'teller': 3, 'nass': 2, 'ramble': 1, 'precocious': 4, 'deliberate': 10, 'one-by-one': 2, 'terminally': 6, 'nature-loving': 1, 'pairs': 2, 'strangeness': 2, 'ringmaster': 1, "54's": 1, 'dimly': 1, 'columbo': 2, 'sordid': 3, 'summarizing': 2, 'mobster': 8, 'tend': 26, 'boingo': 1, 'age-old': 1, "'volcano'": 1, 'obeidient': 1, 'artillary': 1, 'letter': 12, 'raison': 2, 'demystification': 1, 'commands': 9, 'griffin': 3, 'supplier': 1, 'adventure': 39, 'blakely': 1, 'courtship': 4, 'tossed': 8, 'maynard': 1, 'is-': 1, 'comedians': 2, 'transatlantic': 1, 'ethics': 6, '24th': 1, "myers'": 2, 'crash': 24, 'viewings': 7, 'tuition': 1, 'toned-downed': 1, "kelly's": 4, 'documentarians': 1, 'all-american': 2, 'whit': 2, 'yearn': 2, 'menial': 4, 'self-acceptance': 2, 'movie-goers': 1, 'betwixt': 1, 'not-totally-great': 1, "guiness'": 1, 'almanac': 1, 'reagents': 1, 'slimming': 1, 'horfield': 1, 'achieve': 16, "houston's": 1, 'steaks': 1, 'pusateri': 1, "hunt's": 1, 'roxbury': 2, 'dissapointed': 2, 'homily': 1, 'cowers': 1, 'dome': 1, 'carefree': 2, 'disastorous': 1, 'strickler': 1, 'moisten': 1, 'allegorical': 1, 'reassume': 1, 'attitude': 51, 'mr': 68, 'newman': 14, 'military': 25, 'garners': 2, 'lance': 10, 'dinky': 2, "price's": 2, 'dottie': 1, 'thrusts': 2, 'ministers': 1, 'strips': 5, 'stereotype': 9, 'missile': 4, 'objects': 15, 'buffy': 3, 'colorado': 1, 'down-to-earth': 9, 'obliteration': 1, 'fasting': 1, 'hardford': 1, 'enshrining': 1, 'signal': 3, "mills'": 1, 'fiorina': 2, 'overworking': 1, 'crafting': 6, 'breckman': 1, 'symbolizes': 4, 'snugly': 2, 'swallows': 1, "eliot's": 1, 'ornamental': 1, 'rail-thin': 1, 'censoring': 1, 'ghettos': 1, 'shakespeare-obsessed': 1, 'darthur': 1, '11/20/97': 1, 'invoke': 1, 'duveyrier': 1, 'nero': 2, 'yeller': 2, 'underway': 4, '14th': 4, 'over-effective': 1, 'charged': 16, "brothers'": 7, 'heartfelt': 14, 'sleek': 2, 'ounce': 2, '$29': 1, '40+': 1, 'lightest': 1, 'array': 9, 'flavorful': 1, 'outsiders': 6, 'davis': 10, 'button': 2, "respighi's": 2, 'slabs': 1, 'defined': 9, 'investigating': 10, 'obscured': 3, 'small-budget': 1, 'schlub': 1, 'impatience': 1, 'insubstantial': 3, 'lodging': 2, 'screenplay--with': 1, 'hemingway': 1, 'nicholson': 13, 'recants': 1, 'willem': 3, 'figure': 63, 'difficulty': 4, 'thereof-threatens': 1, 'jailterm': 1, 'philosophy': 21, 'touch-up': 1, 'graces': 3, 'professor-student': 1, 'circulation': 1, 'norms': 2, "webber's": 1, 'carved': 1, 'brian': 26, "beau's": 2, "bethlehem's": 1, 'incarceration': 1, 'taran': 1, 'swimmers': 1, 'ingrained': 3, 'not-so-rightfully': 1, 'instances': 9, 'copycat': 3, '1980s': 10, '--the': 7, 'pans': 3, 'sitcoms': 5, 'danilov': 1, 'miniature': 3, 'misses/catches': 1, 'co-counsel': 1, 'display': 20, 'gentlewoman': 2, 'habits': 4, 'co-': 2, 'favors': 3, "cronenberg's": 2, 'wyoming': 1, 'slimey': 1, 'unfathomably': 1, 'cash': 29, 'budge': 1, 'catacombs': 1, 'befall': 4, 'coaxes': 2, 'chaplin': 2, 'profoundly': 4, 'barbed': 1, 'storylines': 8, 'infinitely': 3, "al's": 4, 'reminds': 18, 'eliminate': 6, 'savaged': 1, 'conor': 1, 'theology': 2, 'imbecile': 1, 'reminicent': 1, 'nagle': 1, 'colosseum': 1, 'bazelli': 1, "trial's": 1, 'slaps': 1, 'grittiest': 1, 'redcoats': 1, 'inning': 1, 'pairings': 2, 'dun': 1, 'salesmanship': 1, 'salt': 4, 'nosferatu': 4, 'sack': 2, 'galvanizing': 1, 'neon-lit': 1, 'face-lift': 1, 'skis': 1, 'rests': 1, 'paved': 1, 'rigueur': 1, 'introduce': 10, 'weed': 1, 'throwaway': 8, 'grasps': 1, "titanic's": 4, 'alleviated': 1, 'gutsy': 3, 'uppers': 1, 'authoritarian': 4, 'weirded': 1, 'taps': 3, 'japanese': 23, 'heigh': 1, 'not-in-a-hurry': 1, 'crime-foiling': 1, 'pitch-perfect': 1, 'a-list': 1, 'contorted': 1, 'incarnations--graphic': 1, 'idealistically': 1, 'consuming': 2, 'boosts': 1, 'determining': 1, '1934': 1, 'base': 18, 'reversals': 2, 'ire': 1, "'when": 1, 'foreshadow': 2, 'chapman': 3, 'backstage': 1, 'beliefs': 16, '_little': 1, 'orefice': 1, 'pharaoh': 2, 'transfusions': 1, 'full-size': 1, 'aladdin': 6, 'alabama': 1, 'slew': 3, 'best-seller': 6, 'aswell': 3, 'pander': 1, 'kusatsu': 1, 'metaphors': 4, 'aftereffects': 1, 'black-market': 1, 'hyperactive': 8, 'carmine': 1, 'trini': 2, "rea's": 1, 'esmerelda': 1, 'jabba': 8, 'sincere': 19, 'babysit': 1, '-courtesy': 1, 'obsessive/compulsive': 1, 'rogers': 2, 'ratings-a-plenty': 1, 'anchor/correspondant': 1, 'cements': 3, 'darlene': 2, 'ponder': 10, 'mirthless': 1, 'recreate': 6, 'awakening': 3, 'preceded': 6, 'amusing': 51, 'deviation': 1, 'hulot': 1, 'classes': 9, 'explaining': 18, 'seaside': 1, 'computer': 55, 'mandibles': 1, 'iron-clad': 1, 'bouts': 4, 'moi': 1, 'ills': 3, 'enchanting': 7, 'ripple': 1, 'boatload': 1, 'surpassed': 3, 'line--it': 1, 'discipline': 5, 'unveil': 1, 'rahad': 1, 'better-made': 1, 'violin': 4, 'alejandro': 2, 'triptych': 1, 'multidimensionality': 1, 'bunching': 1, 'passenger-side': 1, 'leer': 2, "riff-raff's": 1, 'cooling': 1, "`atmosphere'": 1, 'gloucester': 1, 'aykroyd': 3, 'hits': 52, 'part-way': 1, 'clair': 2, 'leeched': 1, 'centerpiece': 5, 'stylishly': 6, 'wispy': 1, 'carvings': 1, 'nouri': 1, 'judaism': 1, 'remembers--and': 1, "murphy's": 7, 'hundreds': 20, '16mm': 4, 'venora': 2, 'attributes': 3, 'stroke': 10, 'frollo': 1, 'dilemma': 11, 'mae': 2, 'sherry': 2, 'virginity': 6, 'hills': 14, 'arabian-themed': 1, 'popul': 1, 'describing': 7, 'cabinet-level': 1, 'regeneration': 2, 'mesmerizes': 1, 'maximilian': 1, 'reese': 16, 'tearjerkers': 1, 'all-ages': 1, 'mercilessly': 4, 'successfuly': 1, 'outstandingly': 1, 'unlucky': 2, 'trumpet-playing': 1, 'oingo': 1, 'microcosm': 2, 'petula': 1, "lamont's": 1, 'ipkiss': 1, 'emphatically': 1, 'r&b': 1, "society's": 2, 'hitchcock': 16, 'squad': 5, 'misperception': 1, 'disgruntled': 1, 'lola': 2, '43': 1, 'reddish': 1, 'tiredness': 2, 'gently': 6, 'article': 9, "domingo's": 1, 'sirens': 2, 'affairs': 9, 'neeson': 14, 'ramp': 1, "will's": 6, "carver's": 3, 'disassociates': 1, 'golden-boy': 1, 'givers': 1, 'ailing': 3, 'irs': 1, 'studi': 1, 'life-ruining': 1, 'respite': 1, 'two-bit': 3, 'needle-jabbing': 1, 'bull': 3, 'fifties': 3, 'grammar': 2, 'superficially': 6, '1962': 6, 'itus': 1, 'ordeal': 4, 'nonprofessional': 1, 'seeks': 9, 'sooooo': 2, 'hesitates': 1, 'potshots': 1, 'spanking': 2, 'u-turn': 1, 'rancher': 1, 'subordination': 1, 'overflying': 1, 'fanfare': 4, 'costly': 2, 'carriages': 1, 'sympathetic': 39, "peoples'": 2, 'pitied': 1, 'strength': 44, "ocean's": 1, "laden's": 1, "jabba's": 1, "gavin's": 1, 'reincarnated': 1, 'pin': 3, 'fantastically': 6, 'supervision': 1, 'negotiations': 6, 'performances--always': 1, 'leak': 3, 'infamously': 1, 'goldfish': 1, 'toned': 6, 'frumpy': 1, 'guaranteed': 9, 'romy': 2, 'spoon': 2, 'tm': 1, 'light-mood/serious-mood': 1, 'ominous': 17, 'pais': 1, 'problem-solving': 1, 'deemed': 8, 'chess': 4, 'hackers': 1, 'incurring': 1, 'kindly': 10, 'two-hour': 4, 'soaking': 1, 'coincidentally': 10, '`blade': 1, 'lots': 51, 'fuel': 5, 'clientele': 1, 'detective': 44, 'sector': 2, 'rectify': 2, '[warning': 1, 'five-minute': 1, 'spiers': 1, 'underestimated': 2, 'confers': 2, 'slasher-movie': 1, 'single-mindedness': 1, 'film--on': 1, 'villagers': 2, 'reappears': 3, 'tic': 1, 'interferes': 1, 'hairs': 1, "lemmon's": 1, 'tiegs': 1, 'components': 5, 'sooner': 6, 'priced': 1, 'skull': 2, 'surfer': 6, 'increasingly': 18, 'taint': 1, '`groundhog': 1, 'pop-broadway': 1, 'remake': 20, 'helens': 1, 'nc-17-rated': 1, 'weakness': 9, 'assembles': 3, 'feeble-minded': 1, 'maglietta': 2, 'bogged': 3, 'flatmate': 1, 'overplayed': 2, 'teenaged': 2, 'penchent': 1, 'mid-air': 1, 'timid': 6, 'breeth': 1, "god's": 8, 'firecrackers': 1, 'old-fashioned': 13, 'accomplishing': 1, 'licentious': 1, "spielberg-katzenberg-geffen's": 1, 'gloves': 3, "ephrons'": 1, 'modulation': 1, 'data': 5, 'dea': 2, 'invading': 8, 'embassy': 1, 'fretting': 1, 'laughs': 81, 'lv-426': 1, 'assassin/exterminator': 1, 'expansiveness': 2, 'short-cropped': 1, 'roxburgh': 1, 'height-challenged': 1, 'shindig': 1, 'glance': 17, 'kurosawa': 2, 'irradiation': 1, 'contrastive': 1, 'powerless': 4, 'conjures': 5, 'pysche': 1, 'frothy': 4, 'caricatured': 1, 'steel-gray': 1, 'wanting': 15, 'ziyi': 1, 'sandler': 4, 'lions': 3, 'we+re': 1, 'anti-life': 1, 'gon': 1, '_election': 1, 'television': 66, 'dispair': 1, 'terrorize': 3, '\x05': 1, 'dupe': 1, 'staggering': 9, 'ecstasy': 1, 'devices': 8, 'sea': 24, "fbi's": 1, 'taraji': 1, 'exaggerates': 1, 'qui': 1, 'click': 5, 'clash': 10, 'brauva': 1, 'immense': 8, 'greedy': 11, 'burt': 6, 'wildly': 18, 'raising': 14, 'topsy-turvy': 1, "hicks'": 1, 'yuppie': 2, 'forlani': 2, 'comically-named': 1, 'onscreen': 7, 'wanna-be': 1, 'deepens': 2, 'invective': 1, 'basketball': 8, 'originated': 1, "user's": 1, 'ferraris': 1, 'staros': 1, 'manni': 3, 'manson': 2, 'hopper': 7, 'courtroom': 22, 'theremin-driven': 1, 'gooey': 1, 'document': 7, 'unabashed': 1, 'handbook': 1, '-to': 1, 'shrinks': 2, 'nipples': 1, 'behaves': 4, 'bodily': 6, 'tank': 5, 'sentimental': 23, 'materialized': 1, 'experiment': 11, 'oversimplified': 2, 'anime': 3, 'leonor': 1, 'collette': 6, 'topicality': 1, 'realizes': 41, 'incessant': 4, 'ex-convict': 2, 'nikita': 1, 'ex-stripper': 1, 'open': 62, "tracy's": 2, 'matching': 2, 'dc': 3, 'inspired': 47, 'attentiveness': 1, 'programming': 5, 'oaf': 2, 'westlake': 2, 'visitor': 3, "audience's": 16, 'yuen': 2, 'altered': 9, 'glam': 2, 'stokely': 1, 'firepower': 4, 'tweaks': 1, 'ireland': 6, 'brando': 6, 'frustrating': 7, 'information': 37, 'defining': 5, 'goateed': 1, 'format': 12, 'execs': 1, 'remodeled': 2, 'spoilers]': 1, 'accepted': 11, 'tutors': 1, 'faculties': 1, 'drillers': 3, 'doer': 1, 'candy': 13, 'giancarlo': 1, 'metaphorically': 2, 'developed': 38, 'to-die-for': 1, 'topics': 5, 'non-pc': 1, 'hoot': 11, 'shorty': 8, 'deep-voiced': 1, 'nationwide': 2, 'counterfeiters': 1, 'bea': 1, 'demonstrated': 6, 'midknight': 1, 'albany': 1, 'mississippi': 3, 'ceremonial': 1, 'outspoken': 2, 'factory': 11, "heer's": 1, 'louuuudd': 2, 'portraying': 13, 'roaring': 2, 'spurting': 1, 'implausibilities': 2, 'cult-like': 1, 'y2k': 1, 'weller': 2, 'micheal': 1, "havisham's": 1, 'births': 1, 'weak-hearted': 1, 'semi-wily': 1, 'opinion': 48, 'oscar-winning': 6, "skg's": 1, 'mob': 21, 'debuts': 2, 'ooooh': 1, 'bette': 2, 'boathouse': 2, 'multi-billion': 1, 'malka': 1, 'live-action': 6, "decade's": 3, 'successor': 6, 'heroines': 3, 'devastingly': 1, 'doren': 2, 'lotsa': 1, 'dissapointing': 2, "collins'": 1, 'slinky': 3, 'rifle-shot': 1, 'fanaro': 1, 'cheese': 5, 'old-school': 2, 'youull': 1, 'expressionist': 1, 'squirm': 2, 'waterworks': 1, "ronnie's": 1, 'effects-bound': 1, 'tasteless': 6, "bickford's": 1, 'haise': 1, 'mumbling': 2, 'plotted': 3, 'athletes': 2, "'69": 1, 'nearing': 3, 'revived': 4, 'savoring': 5, 'contained': 13, 'corwin': 1, 'undifferentiated': 1, 'animal-hater': 1, 'accountant': 8, "floats'": 1, 'kiberlain': 1, 'ardent': 2, "rome's": 1, 'puff': 1, 'businessmen': 2, 'teeters': 2, 'hobbs': 2, 'boosterism': 1, 'traffiking': 1, 'truer': 4, 'nello': 1, 'riches': 2, 'derives': 5, 'prepostrous': 1, 'ogle': 1, 'stature': 5, "actor's": 12, 'california': 24, 'mikey': 1, 'hitman': 4, 'counterpoint': 6, 'kerchak': 3, 'habit': 7, 'bluegrass': 1, 'replied': 2, 'hallmarks': 3, 'orphan': 5, 'pierre-loup': 1, 'caste': 2, 'novelist': 3, "'the": 13, 'mall': 7, '16th': 2, 'avoidance': 1, 'foam': 1, 'stats': 1, 'sailboat': 4, "buehler's": 1, 'objectivity': 2, 'bits': 18, 'humanizing': 1, 'magnuson': 1, 'household': 15, 'postapocalyptic': 1, 'hypocrite': 1, 'week': 28, 'tri-annual': 1, "bottin's": 1, 'random': 7, "bianca's": 2, 'showering': 3, 'misleading': 6, 'rudner': 1, 'mountain-side': 1, 'schreber': 2, "lorenzo's": 1, 'pedigree': 1, 'pulverizes': 1, 'roundhouse': 1, 'flash': 24, 'riegert': 1, 'raging': 9, 'frankie': 3, 'picks': 19, 'waikiki': 1, 'accolades': 8, 'thearapeutic': 1, 'squirm-inducing': 2, 'terrible': 25, 'oldies': 1, 'sil': 2, 'leave': 104, 'removing': 4, 'italy': 6, "mcfarlane's": 1, 'switzerland': 1, 'attic': 4, 'abbots': 1, 'term': 18, 'aspired': 1, 'woman-of-the-wild': 1, 'guidance': 11, 'overshadows': 1, 'precarious': 3, 'roberta': 3, "others'": 4, 'fantasia': 2, 'tender': 16, 'spoiled': 4, 'writer/producer': 1, 'mastubatory': 1, 'ramius': 1, 'courage': 23, 'size': 21, 'thread': 4, 'commutes': 1, 'rv': 1, 'tad-too-long': 1, 'lara': 2, 'back-door': 1, 'easily-readable': 1, 'pod-race': 1, 'vandernoot': 1, 'influence': 23, 'quicksand': 1, 'bond': 35, 'wanker': 1, 'interlude': 4, 'properly': 13, '3/10': 2, 'zagged': 1, '600-pound': 1, 'shove': 1, 'uninhibited': 2, 'miserables': 3, 'vinicius': 1, 'pursuers': 5, "postman's": 1, 'fruition': 1, 'tacky': 3, 'bigotry': 1, 'fills': 12, "yimou's": 1, 'unoriginal': 2, 'prospected': 1, 'deductions': 1, 'apprehended': 2, 'service': 22, 'troublesome': 3, 'northern': 4, 'jean-marc': 1, 'laszlo': 1, "america'": 1, 'wilkes': 2, 'olin': 1, 'polluted': 1, 'recommend': 43, 'pina': 1, 'spiritualist': 1, 'amyls': 1, 'zombie-like': 2, 'chopsocky': 1, 'callously': 1, 'unearthed': 1, 'hurts': 2, "keith's": 1, 'dynamics': 6, 'twenties': 8, 'aptly': 5, 'acclimatize': 1, 'off-beat': 5, 'plantation': 5, 'platter': 1, 'tantric': 2, 'rice': 9, 'surfeit': 1, 'mapplethorpe': 1, 'receptions': 2, 'planes': 4, 'anglade': 2, 'stupendously': 1, "hung's": 1, 'stash': 3, 'phenomenally': 1, 'slickers': 2, 'appreciating': 3, 'startling': 17, "joey's": 1, 'ryan/hanks': 1, 'reptiles': 1, 'grim-looking': 1, 'fray': 3, 'frothing': 1, 'preempt': 1, 'marries': 4, 'appointed': 3, 'post-death': 1, 'safari': 1, 'appliance': 1, 'upwardly-mobile': 1, 'quasi-popular': 1, 'class': 73, "beth's": 1, 'realism--but': 1, 'badlands': 1, 'bathtub': 6, 'traditional': 37, 'all-powerful': 4, 'orr': 1, 'celebrate': 8, 'treehorn': 3, 'wind-tunnel': 1, 'adheres': 3, 'preconceptions': 2, 'deaf': 2, "farm's": 1, 'successfully': 31, 'complementing': 4, 'hotheaded': 1, 'un-disneyish': 1, 'pop': 45, 'kay': 6, 'resolves': 5, 'lau': 1, 'urinating': 1, 'garnish': 1, 'nary': 3, 'little-to-no': 1, 'reassure': 1, 'lifeline': 1, 'militarism': 1, 'freighter': 2, '90210': 5, 'moonshiner': 2, "roberta's": 2, 'scoundrels': 1, 'ala': 4, 'maturity': 11, 'sorcery': 1, 'bass-pounding': 1, 'edward': 36, 'chaplin-inspired': 1, 'graciously': 4, 'conjunction': 2, 'pursue': 9, 'titillation': 3, 'ruler': 6, 'self-made': 1, 'guadalcanal': 1, 'vastness': 1, 'romping': 1, 'late-entry': 1, 'glum': 2, 'smuggler': 2, 'needle': 4, 'speeding': 8, 'commiserate': 1, 'cutbacks': 1, 'gangsters': 10, '35-years-old': 1, 'oddball': 4, 'onboard': 3, 'canaan': 1, 'sails': 1, 'toaster': 2, 'mysterioso': 1, 'pregnancy': 2, 'committing': 8, 'patch': 4, 'leung': 1, 'keane': 1, 'commensurate': 1, "'robocop": 1, 'abuzz': 1, 'promoting': 3, 'sounds': 45, 'barbet': 1, 'elegantly': 6, 'gugino': 2, 'gratuitous': 13, 'interrogated': 3, 'generate': 6, '1799': 1, 'trademarked': 1, 'affirm': 1, 'swamped': 1, 'sobriety': 1, 'sparkling': 5, 'knocked-out': 1, 'twofold': 1, 'thiessen': 1, 'enthused': 2, 'string': 12, 'fearsomely': 1, 'eyes-': 1, 'trench': 4, 'delineation': 1, "jarman's": 1, 'irwin': 1, 'stumbled': 3, 'reminiscence': 2, 'wilson': 12, '1794': 1, ',': 899, 'bazaar': 1, 'tsugawa': 1, 'museums': 1, 'catalyst': 6, 'willie': 4, 'coverup': 1, 'unsurprised': 1, "capra's": 3, "frank-n-furter's": 1, 'leto': 1, 'crop': 4, 'depersonalized': 1, 'latched': 1, 'sullenly': 1, 'arrogance': 6, 'integration': 2, 'noon': 1, 'stand-outs': 1, 'insulted': 2, 'graveyards': 2, 'raucous': 5, "eisenstein's": 1, 'asian': 11, 'singer/guitar': 1, "chippendale's": 1, 'cappie': 1, 'show-stopper': 1, 'handicapped': 4, 'bisexuality': 1, 'nicolas': 9, 'sony/loews': 3, 'irritation': 3, "school's": 4, 'grueling': 1, 'recurring': 5, 'occasions': 10, '1994': 18, 'jewish': 16, 'non-judgemental': 1, 'tie-ins': 4, 'moreau': 3, 'taxes': 4, 'lifelong': 6, 'belfast': 2, 'stuffy': 1, 'wire-fu': 1, 'executive-produced': 1, 'farmer-wizard': 1, 'parkes': 1, 'minded': 2, 'convey': 17, 'salability': 1, 'thirteeth': 1, "farrellys'": 1, '139': 1, 'sawalha': 1, 'goads': 1, 'bunch': 44, 'burned-out': 1, 'soothe': 2, 'pound': 4, 'watermelons': 1, 'theses': 1, '1583': 1, 'hosts': 3, 'eilonwy': 1, 'day-to-day': 4, 'valli': 2, 'enumerates': 1, 'encroachment': 1, 'beleiveable': 1, 'candlelit': 1, 'mothers': 3, "there'd": 1, 'pond': 4, "cole's": 4, 'whale': 4, 'ex-': 1, "children's": 28, 'well-drawn': 3, 'geena': 2, 'tee-shirts': 2, 'hotchner': 1, 'psychologist': 9, 'alumnus': 2, 'spring': 14, 'diana': 1, 'rethinking': 2, 'blares': 1, 'fella': 2, 'forks': 1, 'panicked': 1, 'genital': 1, 'gorham': 1, 'sail': 6, 'crashing': 7, 'hate': 49, 'hard-to-get': 1, 'stigma': 1, 'riveted': 1, 'regalia': 1, 'hendel': 1, 'ufo': 1, 'contentious': 1, 'heists': 2, 'cups': 1, 'crichton': 2, 'start': 109, 'chill': 6, 'fashionable': 4, 'periodical': 1, "2's": 2, 'deacon': 2, 'shakespearean-trained': 1, 'barns': 1, 'citing': 1, "pollock's": 1, 'reoccurring': 1, '1793': 1, 'ingenue': 1, 'toosie': 1, "hedges'": 1, 'issuing': 4, 'propelled': 7, 'refrigerates': 1, 'snowy': 3, "grandmother's": 1, 'willingly': 1, 'rapier-9mm': 1, 'titta': 1, "whedon's": 1, 'disasterously': 1, 'vubu': 2, 'insouciance': 1, 'gun-toting': 4, "evening's": 1, 'cops': 23, '`primal': 1, 'splotty': 1, 'eradicate': 1, 'caffeine': 1, 'smashing': 4, "marcellus'": 1, 'earlier': 68, 'vita': 1, 'couplet': 1, 'aftermaths': 1, 'disbelieving': 1, 'enjoyably': 4, 'samaritan': 2, 'viggo': 2, 'northwestern': 2, 'mainland': 1, 'stroller': 1, 'founders': 1, 'facilitates': 1, 'bickle': 3, 'cubs': 1, "would've": 9, 'brundage': 1, 'rubens': 1, 'civility': 2, '88': 2, 'stomps': 1, "kafka's": 1, 'underdrawn': 1, 'gratuitously': 1, 'loyal': 24, 'ebola': 1, 'sprinkling': 1, 'squeezing': 1, 'heartwarming': 4, "suong's": 1, '500': 3, 'yeeeeah': 1, 'moved': 26, 'procreate': 2, 'blockbuster': 25, 'morass': 2, 'swiftness': 2, 'embezzling': 1, '000-a-week': 1, 'goofing': 1, 'wind-up': 1, 'tapert': 1, 'shark': 5, 'blatant': 13, 'start--pollock': 1, "deniro's": 1, 'reared': 2, 'collectors': 2, 'trekkies': 1, 'pinky': 2, 'looters': 2, 'whichever': 1, 'archenemies': 1, 'big-screen': 9, 'grandison': 1, 'orlock': 2, 'plops': 1, 'sealed': 2, 'tierney': 2, 'foot': 21, 'author': 22, 'mulling': 1, 'overthrow': 2, 'characters--in': 1, 'potential': 47, 'possess': 9, 'traumatic': 5, 'sorcha': 1, 'dmitri': 2, 'row': 10, 'superseventies': 1, 'inquisitiveness': 2, 'stark': 17, 'stinks': 2, 'maintain': 20, 'hermit-like': 1, 'louis': 10, 'score--i': 1, 'yun-fat': 2, 'textual': 2, 'alone-until': 1, 'wreaks': 3, 'not-really-all-that-funny': 1, 'misfortune': 1, 'corps': 3, 'cloaked': 2, 'naoto': 2, 'observers': 2, 'heresy': 1, 'stephanie': 1, 'redefined': 2, 'neophytes': 2, 'kellerman': 1, 'liberal': 10, 'neill': 4, 'disappointing': 24, 'ping-bin': 1, "doctor's": 3, 'arranges': 3, 'character-driven': 2, 'husband': 80, 'tenny': 1, 'desert': 26, 'broderick': 8, "ripley's": 2, 'india': 5, 'alters': 3, 'seasonal': 1, 'kimberly': 3, 'sub-plots': 3, 'jabs': 8, 'imparted': 2, 'rollercoaster': 4, 'early': 104, 'sad': 42, 'philipe': 1, '93': 1, 'edition': 14, 'calm': 19, 'rooker': 3, 'forlorn': 2, 'employees': 7, 'assurance': 4, 'millions': 18, 'behalf': 7, 'gerard': 5, 'musicals': 8, "hoke's": 1, 'scam-artist': 1, 'management': 5, 'musicologist': 1, 'forman': 5, 'dreamworks': 13, 'montague': 2, 'illuminate': 2, "weaver's": 1, 'abduction': 3, 'darth': 13, 'youngest': 3, 'detraction': 1, 'floored': 2, 'imam': 1, "luchini's": 1, 'staking': 2, 'bacon': 10, 'determines': 1, 'traps': 5, 'batch': 3, "who've": 5, 'slickly': 1, 'wondrous': 10, 'canned': 1, 'sketchiness': 1, 'web-based': 1, 'subtitled': 9, 'snaps': 5, "gambler's": 1, 'thornton': 8, 'grazia': 1, 'reappearing': 2, "filmmaker's": 4, 'comparable': 6, 'naif': 1, 'kazdan': 1, 'coals': 2, 'cheery': 4, 'bullhorn': 1, 'temptation': 10, 'characterizations': 10, 'people-less': 1, 'guitry': 1, 'secure': 6, 'boarding': 8, 'gumption': 2, 'trivial': 2, 'hail': 5, 'day-lewis': 2, 'stepmother': 2, 'arcade': 1, 'inventively': 1, 'lowell': 2, 'in-laws': 1, 'nighthawks': 1, 'endures': 1, 'hockley': 7, 'soon-to-be-divorced': 1, 'people': 395, 'countermanded': 1, 'screaming': 14, 'schiff': 1, 'mcgowan': 1, 'business-like': 1, 'passionately': 1, 'esmeralda': 1, 'blames': 2, 'inflict': 1, 'culture-clash': 1, 'sue': 9, 'forges': 2, "travolta's": 6, 'lie': 24, "steig's": 1, 'warm-hearted': 2, 'variety': 27, 'coercion': 2, 'madcap': 2, 'baddies': 3, 'hounded': 2, 'preference': 2, 'pies': 1, 'coffins': 2, 'akira': 3, 'artifacts': 4, 'smash': 9, 'edge-of-your-seat': 1, 'selflessness': 1, "rubell's": 1, 'aren': 1, 'subplots': 17, 'pornstar': 1, 'maniacally': 1, 'delia': 1, "_shaft's_big_score": 1, 'transference': 1, 'bugsy': 1, 'excruciatingly': 1, "wells'": 1, 'phoniness': 1, "malick's": 2, 'sponge': 1, 'lillian': 3, 'guinee': 1, 'guildenstern': 2, 'chatters': 1, 'lengthen': 1, 'ethnically': 1, 'faucet': 1, 'wealth': 14, 'reconnaissance': 1, 'females': 5, 'caring': 20, 'denying': 8, '`patty': 1, "flynt's": 2, 'so-bright': 1, 'caretakers': 1, 'mate': 9, 'hershey': 2, "lambeau's": 3, 'true-man': 1, 'boop': 1, 'leering': 3, 'deuce': 2, 'divers': 1, 't-rex': 2, 'dimly-lit': 2, 'eleventh': 1, 'practicing': 2, 'well-spent': 1, 'enhancer': 1, 'wild': 52, 'lasting': 13, "ruby's": 1, 'ankles': 1, 'thing--the': 2, 'clive': 3, 'initiatives': 2, 'wrack': 1, 'peculiarities': 1, 'utilize': 8, 'vapor': 1, 'lando': 1, 'janssens': 1, 'brrrrrrrrr': 1, 'outside-in': 1, 'auditorium': 1, 'triumphantly': 2, 'tremor': 1, 'shmaltzy': 1, 'nicola': 1, 'casanova': 2, 'radiant': 7, 'horned': 2, '_american_psycho_': 1, "kuzco's": 1, 'competitor': 3, 'physiology': 1, 'dis-organized': 1, "hedaya's": 1, 'perky': 6, 'dead-end': 4, 'double-deals': 1, 'marshall': 9, 'teen-speak': 1, "joe's": 5, 'partially': 4, 'outright': 4, 'alive': 50, 'sire': 1, 'facehugger': 1, "davis'": 1, 'individual': 33, 'mini': 3, 'starve': 2, 'hartley': 3, 'explain': 36, 'pursued': 5, 'sanitary': 1, 'blink': 3, 'colour': 2, 'weinstein': 2, 'travelling': 7, 'crawls': 4, 'locate': 6, 'oozes': 5, 'disgust': 6, 'beleaguered': 2, 'arenas': 1, 'skarsg': 2, 'sfxfest': 1, "teacher's": 1, 'elixir': 1, 'signficance': 1, 'motto': 1, 'tongue-tied': 1, 'thrillers': 17, 'criticised': 1, 'orphaned': 3, 'eduardo': 1, '$500': 5, 'crannies': 1, 'introspective': 8, 'ingested': 1, "luke's": 3, 'olive': 2, 'santo': 1, 'grounding': 1, 'suger-coated': 1, 'hirschfeld': 2, 'locus': 1, 'danes': 7, 'steakley': 1, 'mongkut': 1, 'pensively': 1, 'unspool': 1, 'film-hannibal': 1, 'bubbly': 3, 'cogs': 1, 'salvador': 1, 'royces': 1, 'selfishly': 1, 'welland': 1, 'detour': 6, 'vengeance': 7, 'blackmail': 5, 'harvard': 8, 'degeneration': 1, 'trimmings': 1, 'dombasle': 1, 'trips': 5, "boy'": 1, 'turks': 2, 'rewatchable': 1, 'highway': 5, "everett's": 2, 'shannon': 5, 'lund': 1, "taster's": 1, 'millenium': 4, 'policemen': 1, 'trashed': 1, 'rubble': 2, 'morality': 12, 'encompasses': 4, 'groan': 2, 'depicting': 10, 'equal-rights': 1, 'graziella': 1, 'libidinous': 1, 'kampf': 1, 'hive': 1, 'caffeinated': 1, 'snot': 1, 'borderline': 1, 'bothers': 5, 'french': 37, 'kick-ass': 3, 'flying': 31, 'armada': 3, 'punchline': 3, 'overflowing': 5, 'balanced': 5, 'preserve': 3, 'muffled': 1, 'flicker': 1, 'gladly': 4, 'red-haired': 1, 'catastrophic': 4, 'indulging': 1, 'napalm': 1, 'teenagers': 28, 'serialized': 1, 'peppered': 2, 'ulcer': 1, 'earned': 12, 'laurie': 8, 'melee': 1, 'thirty-eight': 1, "mib's": 1, 'lectured': 1, 'customs': 8, 'newton': 10, 'stirred': 2, 'pervaded': 1, 'off-limits': 1, 'exemplified': 3, 'celebrity-hood': 1, 'mistletoe': 1, 'gurgi': 1, 'matthew': 40, 'gear': 6, 'mom': 16, 'backstabbing': 1, 'highbrow': 1, 'billing': 7, '_beloved_': 1, "schwarzenegger's": 2, 'janitorial': 1, 'africans': 4, 'hudgeons': 1, 'musical': 54, "dobson's": 1, 'schamus': 1, 'dano': 1, 'psychobabbler': 1, 'fanatic': 5, 'healthy': 10, 'celebrity-directed': 1, 'kidnaper': 1, 'creating': 47, 'ideas': 48, 'zukovsky': 1, 'calvin': 2, 'hall': 26, 'buzzing': 2, 'combines': 14, 'aunjanue': 2, 'vacuous': 4, 'dynamism': 1, 'weakened': 1, 'directs': 22, "boyfriend's": 2, 'sights': 11, 'second-favorite': 1, 'mafioso': 4, 'umptenth': 1, 'striptease': 2, 'lulls': 1, 'cabo': 1, 'bebe': 2, 'disguising': 2, 'crowns': 1, '8-ball': 1, 'escalate': 2, "davidtz's": 1, 'filing': 1, 'programme': 1, 'unborn': 1, 'deliver': 41, 'troupe': 5, 'decipher': 2, 'beneath': 13, 'vishnu': 1, 'vj': 1, 'rockets': 3, 'cinematically-savvy': 1, 'likelihood': 3, 'formulaism': 1, 'fetishist': 1, 'tristar': 2, 'owed': 2, "white's": 3, "burnhams's": 1, 'danging': 1, 'characterization': 33, 'kassovitz': 1, 'mutilating': 2, 'typecasting': 2, 'second-grade': 1, 'rekall': 1, 'chef': 4, 'give': 185, 'audience-friendly': 1, "channing's": 2, 'mcdermott': 6, 'marlon': 7, 'rossum': 1, 'characterizing': 2, 'seer': 1, 'snappiness': 1, '99%': 1, '1862': 1, 'matched': 6, 'fountain': 4, "mile's": 1, 'results': 38, 'sublimely': 1, 'foul-tempered': 1, '--so': 1, 'resemblance': 6, 'suspenseful': 28, 'publicity-seeking': 1, 'antagonist--unstoppable': 1, 'gleeful': 3, 'smarter': 7, 'snowball': 3, 'two-million': 1, 'soulful': 2, 'defecation': 1, 'engineer': 2, 'neo-gothic': 1, 'complete': 71, 'bing': 1, 'scrawny': 2, 'without--shock': 1, '_blade': 1, 'com/hollywood/bungalow/4960': 1, 'berardinelli': 2, 'hydrophobia': 1, 'usa': 6, 'donella': 3, 'accidental': 6, 'missing': 47, 'masseuse': 3, 'caterpillar': 2, 'lord': 31, 'reachable': 1, 'lugosi': 1, 'income': 4, 'dead': 120, "avi's": 1, 'overexposed': 2, 'elitist': 1, 'realms': 2, 'most-famous': 1, 'announced': 4, 'self-sustaining': 1, 'chisselled': 1, 'repellent': 3, 'royalists': 1, 'nuance': 3, 'nine-tenths': 1, "hai's": 1, 'debate': 19, 'hime': 1, 'executioner': 1, 'conclusion': 59, 'fun-filled': 1, 'half-scared': 1, 'contented': 2, "craig's": 1, 'apology': 1, 'oilrig': 1, 'innundated': 1, 'gaelic': 2, 'geeks': 1, 'blas': 1, 'arrogant': 17, 'three-week': 2, 'trainer': 3, 'occasional': 26, 'triva': 1, 'postage': 2, 'stood': 9, 'burning-bush': 1, "`r'": 1, 'smirked': 1, 'deeper': 27, 'colorful': 28, 'unabated': 2, 'slump': 1, 'decency': 5, "ain't-i-sexy": 1, 'label': 5, "karen's": 3, 'zant': 1, 'preposterousness': 1, 'compact': 3, 'restlesness': 1, 'predecessor': 21, '60-something': 1, 'seats': 10, 'angst-filled': 1, 'decreased': 1, 'wont': 2, 'perfunctory': 3, 'defeats': 5, 'forementioned': 1, "documentary's": 1, 'villainous': 5, 'knives': 1, 'repay': 1, 'cathartic': 3, 'facher': 1, 'hong': 22, 'prevented': 3, 'inches': 1, 'professionally': 2, 'single': 88, 'shoelaces': 1, 'treading': 1, 'lynched': 3, 'math': 7, 'ball': 22, 'indian': 12, 'egotistical': 3, 'jeff': 28, 'execute': 9, 'tighter': 4, 'rotten': 4, 'cigar': 1, 'pleasent': 1, 'christ-like': 2, 'hazel': 1, '$14': 1, 'commensurately': 1, 'conspicuous': 1, 'story_': 1, 'behaving': 2, 'institutes': 1, 'hippos': 1, 'coolness': 4, 'melancholic': 2, 'marshals': 2, 'striking': 21, 'larger-issue': 1, 'yoda': 6, 'anway': 1, '80%': 1, 'testimonials': 1, 'matrix': 15, 'safely': 6, 'selena': 3, 'practical-joking': 1, 'mixed': 20, 'slices': 1, 'fake-looking': 1, 'somewhat-childish': 1, 'bitty': 1, '-spouting': 1, 'resuscitation': 1, 'dear': 8, 'requirements': 2, 'thirty-': 1, 'dutiful': 1, 'illeana': 1, 'nudie': 1, 'parade': 7, 'less-than-ideal': 1, 'rebelliousness': 2, 'infantry': 6, "bard's": 2, 'tempestuous': 1, 'warton': 1, 'shanghaied': 1, "ironside's": 1, 'succeeding': 3, 'blunder': 1, 'balloon': 1, 'chaser': 2, 'conversational': 1, 'purge': 1, 'regularly': 6, 'bunnies': 2, 'procol': 1, 'racial': 6, 'wittily': 1, 'tucked': 2, 'custom': 3, 'bless': 1, 'befriended': 6, "poirot-delpech's": 1, 'collaborator': 1, 'obstacle': 8, "louisa's": 1, 'flip': 5, 'raja': 1, "lampoon's": 1, 'talia': 1, 'you-at': 1, 'yanking': 1, 'undoing': 3, 'dzundza': 1, 'bellylaughs': 1, 'first-semester': 1, 'e-block': 1, 'windshield': 2, 'frewer': 1, 'pantomime': 1, 'completion': 4, 'lurk': 1, 'furtwangler': 1, 'boiler': 5, 'docket': 1, 'trading': 5, 'gateway': 1, 'professor/archeologist': 1, 'connick': 1, 'osnard': 1, 'register': 2, "stifler's": 1, 'ruled': 9, 'awestruck': 2, 'double-duty': 1, 'coinage': 1, 'virile': 1, 'gaze': 5, 'coen': 11, 'maker': 6, '`special\x12': 1, '44-year-old': 1, 'elmer': 2, 'lackeys': 1, 'tennant': 1, 'high-tech': 7, 'ghettoized': 1, 'filled': 66, 'enhanced': 8, 'saint-pierre': 1, 'mantegna': 2, 'multi-personality': 1, "picture's": 3, 'impeccably': 2, 'sect': 1, 'bus': 18, 'plastered': 1, 'calvert': 1, 'toppling': 1, 'sleezebag': 1, 'tappan': 2, 'sex-filled': 1, 'proportionately': 1, 'wheelchair-stricken': 1, 'two-timed': 1, 'foghorn': 1, 'wobbles': 1, 'gwynne': 1, 'toro': 6, 'promiscuous': 4, '3-dimensional': 1, 'suviving': 1, 'starling': 2, '80': 3, 'misogyny': 1, 'snicker-worthy': 1, '7th': 2, 'outrun': 1, 'spades': 5, 'browsed': 1, 'long': 264, 'tapped': 2, 're-popular': 1, 'penn': 12, 'ex-mining/maximum': 1, 'orgasm': 4, 'guides': 3, 'proposed': 1, 'degree': 15, 'puritan': 1, 'plan-': 1, 'duchesse': 1, 'closure': 10, 'primordial': 1, 'gains': 10, 'galaxy': 14, 'fiery-haired': 1, '`but': 1, 'underperforms': 1, 'relents': 1, "o'brien's": 1, '1839': 2, 'interactivity': 1, 'rabbit': 8, 'menace--labor': 1, "winterbottom's": 1, 'dive': 5, 'and-miss': 1, 'aristocrats': 2, 'technology': 33, 'cindy': 2, 'writes': 16, 'competed': 2, 'combat': 14, 'punctuated': 3, 'waits': 6, 'hangar': 1, 'shoot-outs': 3, 'existences': 1, 'be-littles': 1, 'phi': 1, 'astaire': 1, 'lovable': 20, 'certianly': 1, 'adjectives': 2, 'quibbling': 1, 'repeats': 3, 'fanny': 1, 'irish': 15, 'obligation': 1, 'generated': 17, 'presnell': 1, 'tuck': 1, '_in': 1, 'only-children-has': 1, 'frolic': 3, 'raspy': 1, 'longer-than-necessary': 1, 'rejecting': 1, 'breeze--or': 1, "doesn'": 1, 'stead': 1, 'aggression': 4, 'fetishization': 1, "dealers'": 1, 'rossio': 2, 'imposing': 9, 'audition': 2, 'behavior': 23, 'picky': 3, 'lea': 4, "o'daniel": 1, "edition'": 1, 'outsider': 14, 'gristly': 1, 'li': 4, 'smell': 5, 'circa': 2, 'adversity': 2, '-sound': 1, 'presided': 2, 'hazardus': 1, 'unrestrained': 6, 'complex': 77, "popitti's": 1, 'over-the-edge': 2, '$8': 3, "`marketing'": 1, 'tomatoes': 2, 'briscoe': 2, 'marriageable': 1, 'defendant': 1, 'ensuring': 3, 'picture-perfect': 1, 'lacks': 32, 'seasons': 8, 'wafflemovies': 1, "gridlock'd's": 1, 'sci-fi/action/comedy': 1, 'prophesies': 2, 'rewritten': 1, 'hoary': 2, 'boorish': 1, 'vitriol': 1, 'bates': 11, 'laziest': 2, 'little-known': 3, 'coaches': 2, '100': 21, 'current-films': 1, 'trendsetters': 1, 'extends': 2, 'toffler': 1, 'lulu': 1, "grabthar's": 1, 'handle': 18, 'non-supermodel': 1, 'gambled': 1, 'commercialist': 1, 'aforementioned': 23, 'baby-boom': 1, 'raft': 2, 'disengaged': 1, 'eastwood': 7, 'coined': 2, 'underline': 1, 'postal': 5, 'post-watergate': 2, 'hunt': 19, 'magnitude': 4, 'olympian': 1, 'capital': 6, "you'": 1, 'trilogy': 19, 'drink': 12, 'nacho': 1, 'college-town': 1, 'jukebox': 1, 'sailor': 1, 'keely': 1, 'bastardised': 1, 'unharmed': 1, 'invent': 1, "`stigmata'": 1, 'internalize': 1, 'peat': 1, 'weak-willed': 1, 'year-old': 4, 'cambodian': 1, 'life-like': 1, 'mozart': 4, 'exploratory': 1, 'garofolo': 1, 'fiasco': 2, 'troi': 4, 'armageddon': 14, 'signature': 4, 'agitation': 1, 'deployment': 1, 'closet': 5, 'dry': 20, 'ji': 1, 'lifestyle': 16, 'chords': 3, 'peddling': 2, 'barrister': 1, 'marcia': 4, '27%': 1, 'simpsons': 3, 'automatic': 3, 'approve': 3, 'track': 36, 'athlete': 5, 'double-whammy': 1, 'fart': 4, 'meshes': 2, 'unbiased': 1, 'stranding': 2, 'rules': 29, 'strife': 2, 'diction': 1, 'fully-ripened': 1, 'sci-fi/action': 1, 'ruddy': 1, 'lark': 2, 'albarn': 1, 'well-cast': 2, 'blondes': 1, 'identifiable': 5, 'literature': 9, 'condemns': 1, 'un-idolizable': 1, 'code': 8, 'ushers': 1, 'constitutes': 3, 'non-glamorous': 1, 'repays': 1, 'xeroxes': 1, 'fabled': 1, "'women's": 1, 'profiteers': 1, 'leeway': 1, 'sales-pitch': 1, 'requests': 5, 'parental': 1, 'garde': 1, 'otherworldly': 2, 'kwai': 4, 'faith': 31, 'emits': 1, 'unmitigated': 1, 'deliveries': 1, 'politician/developer': 1, 'jetliners': 1, 'undergone': 2, 'arsenal': 5, 'reynolds': 7, 'guilty': 17, 'replayed': 1, 'ultra-cheesy': 1, 'parked': 1, 'pounding': 3, 'upholding': 1, 'hog-nosed': 1, 'driven': 26, 'synopsis': 27, 'studio-released': 1, 'cooler': 3, 'conceive': 6, 'independant': 1, 'gathers': 3, 'ghoulish': 2, 'lift': 8, 'tzudiker': 1, 'hysteria': 7, 'girl-friend': 1, 'undermined': 3, 'nutcase': 1, 'overwhelms': 1, 'pause': 4, 'reduces': 4, 'attainable': 2, 'tarts': 1, 'arch-enemy': 1, '60-some': 1, 'innuendo': 2, 'bart': 2, "wanna-be's": 1, 'simultaneous': 1, 'renny': 3, 'politically-correct': 1, 'rip-offs': 2, 'trauma': 3, 'nil': 2, 'attends': 4, 'baby': 36, 'wah': 1, 'region': 1, 'miscarries': 1, 'time-honored': 2, 'guerra': 1, 'generations': 15, 'definitly': 1, 'pugilistic': 1, 'science-fiction': 7, 'preserving': 1, 'gazillionaire': 1, "visnjic's": 1, 'payment': 2, 'professor': 20, 'hackman': 8, "boyd's": 1, 'bud': 13, 'satirist': 1, 'controversial': 24, 'visits': 15, 'biased': 6, 'lobster': 1, 'resultant': 2, "ru'afo": 1, 'el': 5, 'virtuous': 1, 'compassionate': 5, 'aquatica': 1, 'unremarkable': 4, 'sylvie': 1, 'constantly': 68, 'resurrects': 1, 'depart-': 1, 'english-dubbed': 1, 'books': 36, "joe'": 2, 'passionless': 1, 'litters': 1, 'sugary': 1, 'garwood': 1, 'posses': 1, 'slum': 1, 'sane': 3, 'scary-looking': 1, 'dinners': 1, 'obliterated': 1, 'post-cannibal': 1, 'ex-lax': 1, 'jonn': 1, "`seven'": 1, 'ruminations': 2, 'functioning': 3, 'outdoors': 4, "victim's": 2, 'honorably': 1, 'revolted': 1, 'actual': 55, '$2': 4, 'african-americans': 2, 'apprentice': 8, 'well-known': 12, 'stem': 1, 'busey': 4, 'deviates': 1, 'smelled': 1, 'reality': 74, "snl's": 1, 'archetypal': 4, 'massages': 1, 'heiress': 2, 'thievery': 1, "question's": 1, '_quite_': 1, 'sinking': 16, 'best-known': 2, 'taught': 12, 'perplexing': 3, 'impacting': 2, "mcnaughton's": 2, 'introducing': 10, 'paltry': 1, 'kalifornia': 2, 'vin': 6, 'gu': 1, 'punctuate': 3, 'purse': 1, 'rehabilitation--he': 1, 'grains': 3, 'prowling': 1, "williams'": 11, 'color-saturated': 1, 'unmasking': 1, 'rousselot': 1, 'largeur': 1, 'marian': 1, 'precipitous': 1, 'rockhound': 2, 'blind': 21, 'adversary': 3, '16x9': 3, 'lapoirie': 1, 'fireman': 2, 'vergil': 1, 'lemay': 1, 'attenborough': 5, 'heritage': 7, 'sabotages': 1, 'stowe': 1, 'pepsi': 1, 'anti-voyeuristic': 1, 'decapitate': 2, 'documented': 2, 'contents': 2, 'curry': 5, 'amy': 23, 'demo': 2, 'unsocial': 1, 'representatives': 2, 'discovering': 15, 'goregeous': 1, 'comply': 4, 'kyzynski-style': 1, 'trousers': 1, 'tying': 3, 'response': 15, 'biographical': 3, 'hawkes': 1, 'griff': 1, 'feldman': 2, "o'shea": 2, 'hierarchy': 2, 'despises': 2, 'disarmingly': 2, "`spiritual'": 1, 'regan': 1, 'defense': 23, 'fidelity': 5, 'adopts': 8, 'fisherman': 2, 'mucked': 1, 'airbrushed': 1, 'supplementing': 1, 'keener': 5, 'showcase': 15, 'grating': 3, 'playfully': 1, 'nonplused': 1, 'breakout': 2, 'presuite': 1, '4-inch': 1, 'misusing': 1, 'coruscant': 3, 'amiable': 3, '_film': 2, 'improves': 4, 'stock': 21, 'communists': 3, 'misfortunes': 2, 'lug-head': 1, 'tragicomic': 1, 'game': 94, 'harshly': 2, 'sign': 32, 'poisoned': 1, 'scrubbing': 1, 'spritely': 1, 'wormhole': 2, 'wagon': 2, 'unclear': 5, 'grief': 7, 'tit': 1, 'boham': 1, 'century-old': 2, 'ruth': 6, 'angelic': 2, "tunney's": 1, 'wooing': 1, 'banjo-picking': 1, 'michelle': 15, 'flynn': 6, 'notorious': 9, 'searches': 9, 'muse': 6, 'unforgiving': 2, 'screening': 15, 'russ': 2, 'chatting': 2, '#@%$^': 1, 'crazed': 14, 'devour': 1, 'styles': 9, '161': 1, 'intimate': 12, 'catch': 30, 'neurotic-looking': 1, 'critics': 55, 'adolescence': 2, 'surgeon': 6, "'bend": 1, 'docudrama': 3, 'guido': 3, 'tuttle': 1, "camera's": 1, 'atone': 1, 'screams': 12, "stoppard's": 1, 'classism': 1, 'elder': 5, 'dairy': 1, 'defuse': 1, 'legally': 5, 'maxx': 1, 'fest': 2, 'ostentatious': 1, 'startle': 1, 'cockburn': 1, 'lopsided': 1, 'unfold': 12, 'self-references': 1, 'morante': 1, 'williamson': 11, 'younger': 49, 'persistent': 1, 'blank': 15, 'clayon': 1, 'sexual': 62, 'divine': 7, '$85': 1, 'keyser': 3, 'glasnost': 1, 'mystery': 58, 'prestige': 3, 'patrick': 32, 'saville': 1, 'akin': 7, 'mortizen': 1, 'droids': 4, 'headaches': 1, 'weirdness': 6, "'normal": 1, 'souvenir': 1, 'robust': 6, 'hangers': 1, 'change': 106, 'unified': 1, 'american/italian': 1, 'race-related': 1, 'chores': 1, 'sewing': 2, 'pits': 1, 'admirable': 14, 'poorly-employed': 1, 'full-thx': 1, 'lunch': 3, 'spitting': 3, 'echelon': 3, '_pollock_': 1, 'hefti': 1, 'shortage': 2, 'wide-screen': 1, 'devotes': 5, 'curtin': 1, 'haddonfield': 1, 'mysteries': 2, 'olsson': 1, 'lhermite': 1, "west's": 1, 'pistol': 2, 'up-front': 3, 'broncobuster': 1, 'funky': 5, 'asks': 62, 'malkovich': 11, 'overexposure': 1, 'corners': 3, 'entering': 13, 'file': 3, 'cancelled': 2, 'ash': 3, 'paintings': 4, 'pretend': 6, "christoff's": 1, 'palestinians': 1, "mussolini's": 1, 'ghostface': 1, 'carnival': 4, 'mer': 1, 'distribution': 8, 'tabloids': 1, 'denounce': 1, 'ly': 1, 'innately': 1, 'farren': 1, "cartman's": 1, 'bianca': 2, 'weird': 30, 'adeptly': 1, 'freezer': 1, 'puzzled': 5, 'populating': 1, "burbank's": 2, 'hangs': 8, 'butter': 5, 'osmosis': 1, 'adorning': 1, 'opinionated': 1, 'guardrail': 1, "christ's": 3, 'places--': 1, '65th': 1, 'litz': 1, 'commonplace': 7, 'massachusetts': 5, 'harshest': 2, 'directon': 1, 'rat': 8, 'co-venture': 1, 'regime': 4, 'like_blade': 1, 'snooping': 1, "virgin's": 1, 'gregson': 2, 'dethroned': 1, 'tuna': 1, 'brewster': 1, 'inhabitant': 1, 'erm': 1, 'searcing': 1, 'minnesota': 2, "apes'": 1, 'gary': 21, 'sum': 12, 'hyde-pierce': 2, 'enchantingly': 1, 'yakusho': 2, 'leghorn': 1, 'focker': 1, 'unit': 5, 'russians': 9, 'minutes]': 1, 'a-plenty': 1, 'amadala': 1, 'mates': 2, 'malibu': 1, 'syndicate': 2, 'housewives': 1, "a-changin'": 1, 'stacy': 4, 'wax': 3, 'compressed': 2, 'zorro': 2, "sisters'": 1, 'propulsion': 1, 'clicks': 4, 'sucking': 1, 'gourmet': 2, 'showgirls': 6, 'comrades': 3, 'flutter': 1, 'atlanta': 2, 'gaping': 3, 'venom8@hotmail': 1, 'appeasement': 1, 'communications': 4, 'unglamorous': 1, 'myserious': 1, 'railway': 3, 'tiles': 1, 'shotgun': 7, 'tableware': 1, 'shakespeare': 17, 'bookie': 1, 'lange': 2, 'rerelease': 2, 'guilty-pleasure': 1, 'tom': 88, 'competently': 2, 'type': 63, 'reviews': 36, 'hacker': 3, 'disappearances': 1, 'admitedly': 1, 'grier': 9, 'sight-seeing': 1, 'graphics': 5, 'begun': 13, 'easiest': 4, 'peanut': 3, 'grove': 1, 'proportions': 11, 'irks': 1, 'macneill': 2, 'sobol': 1, 'cutting-and-pasting': 1, 'mormon': 1, "motta's": 1, 'bow': 6, 'aggravating': 1, 'monsoon': 1, 'vineyard': 1, 'honest': 39, 'operate': 5, 'uncontrollably': 4, 'tiering': 3, 'morris': 2, 'ebert-written': 1, 'dustbuster': 1, 'male/female': 1, 'surreptitiously': 1, 'advent': 4, 'viperous': 1, 'shatter': 2, '12': 19, 'hostess': 1, 'complains': 3, 'heart_': 1, 'shags': 1, 'flood': 3, 'kilgore': 2, 'anus': 1, 'displeasure': 1, 'snow-globe': 1, 'linebackers': 1, 'ducks': 4, 'deaden': 1, 'outset': 7, 'progenitor': 2, 'stringfield': 1, 'crescendos': 2, 'dart': 1, 'lib': 2, 'koslova': 1, 'arlington': 1, 'steep': 1, 'creepier': 1, 'lowered': 2, "ten'": 1, 'well-planned': 1, 'finney': 1, 'buketer': 1, 'landsbury': 1, "`no-name'": 1, 'buttons': 3, 'cartoon-like': 2, 'suffocating': 4, 'professors': 1, 'picaresque': 1, "rat's": 2, 'lake-type': 1, 'tac': 1, 'bird-loving': 1, "syd's": 1, 'jared': 1, 'fondacairo': 1, 'organized': 14, 'over-night': 1, 'juggle': 1, 'hitching': 1, 'institution': 6, 'devote': 2, 'yep': 2, 'rifling': 1, 'fox': 30, 'systems': 6, 'indies': 1, 'dreamed': 5, 'two-and-a-half': 1, 'michael': 112, 'antenna': 2, 'un-flashy': 1, 'chiefly': 4, '1943': 3, 'bulimia': 1, 'anti-climactic': 1, 'encino': 2, 'masculinity': 2, 'confrontations': 5, 'upstairs': 4, 'aplenty': 1, 'clicking': 1, 'jacqueline': 6, 'witzky': 1, 'gail': 5, "black's": 2, 'sentence': 13, 'wessell': 2, 'spade': 2, 'singable': 1, 'berated': 1, 'ang': 3, 'salo': 1, 'guaspari': 2, 'pas': 2, 'large-assed': 1, 'spottiswoode': 1, 'jessie': 6, 'studious': 1, 'affiliated': 1, 'roving': 2, 'avoided': 9, 'noodles': 1, 'dudes': 3, 'calvinist': 1, "japan'": 1, 'whites': 5, "uncle's": 3, 'cane': 2, 'measure': 11, 'schwartz': 2, 'frasier': 2, 'starbucks': 2, 'stavros': 1, 'mathematics': 4, 'estuff': 1, 'changed': 35, 'classics': 22, 'nightmare': 25, 'potrayal': 1, 'rewards': 5, "rocco's": 1, 'sailors': 3, 'cystic': 1, 'wills': 1, 'conway': 2, 'magenta': 1, 'baby-faced': 1, 'dreamscape': 1, "hannah's": 1, 'sirtis': 1, 'rain-soaked': 1, 'crossover': 2, 'mcallister': 1, 'donned': 1, 'telly': 1, 'monomaniacal': 1, 'audiences': 71, 'anna': 17, 'action-': 1, 'romantic': 73, 'humphries': 1, 'acquired': 5, 'tubes': 1, 'sears': 1, 'all-white': 2, 'ridding': 3, 'siunin': 1, 'censorship': 2, 'dissolves': 3, 'princesses': 1, 'suppressing': 2, 'blue-collar': 2, 'bravely': 4, 'congressman': 2, 'ex-rolling': 1, 'pyramids': 1, 'death': 145, 'portillo': 1, 'wlodkowski': 1, 'squiggly': 1, 'atmospherically': 3, 'sexpot': 3, "ford's": 4, 'build': 31, "high'": 1, 'warlord': 2, 'walsingham': 1, 'apprenticeship': 1, 'meeting': 32, 'withdrawn': 3, 'heels': 3, 'deathly': 1, 'rougly': 1, 'zooming': 2, 'complexities': 4, "lovett's": 1, '`saving': 1, 'reaction': 26, 'hamilton': 5, 'vigilante': 3, 'walkway': 1, 'royalist': 1, 'unforced': 2, 'spicy': 2, 'credit': 63, 'pierces': 1, 'judy': 4, "lolita's": 1, 'edelman': 1, 'burwell': 1, 'transmitions': 1, "'77": 1, 'logically': 1, 'near-sweetness': 1, 'enthusiast': 2, 'manga': 2, 'misunderstand': 1, "kieslowski's": 1, 'multiple': 25, "1971's": 1, 'paws': 1, 'anti-hero': 1, 'scalawags': 1, 'videotape': 11, 'ghouls': 1, 'floors': 2, 'batting': 1, 'ee': 1, 'cigar-smoking': 1, 'narcolepsy': 1, 'blandest': 1, 'burning': 15, 'hickey': 1, 'spans': 5, 'absent': 10, 'offends': 1, 'ottis': 1, 'priority': 1, 'totatly': 1, 'cheryl': 1, 'connection': 31, 'spare': 14, 'loki': 2, 'tantalizingly': 1, 'essential': 21, 'extended': 19, 'unopposed': 1, 'morals': 14, 'disheartening': 2, 'nerve-shattering': 1, 'david': 136, "o'harra": 2, '_people_': 1, 'snatches': 3, 'loveliest': 1, 'salon': 2, "costner's": 3, 'love-in': 1, 'heart-felt': 1, 'engineered': 6, 'aerospace': 1, 'snicker': 1, '`run': 1, 'savings': 4, 'flashiest': 1, 'reissue': 3, 'skin': 18, 'offbeat': 18, 'banners': 1, 'ivory': 2, 'frequents': 2, 'relevance': 2, 'recently': 59, 'average': 48, "harford's": 1, 'dixon': 2, 'op': 1, 'shots--and': 1, 'smart-alecky': 1, 'singer': 31, 'synchronicity': 2, 'mutual': 8, "'vampires'": 1, 'indy': 3, 'gies': 1, 'ambers': 1, 'canadian': 13, 'spock-like': 1, 'schaech': 1, "dreamworks'": 4, 'spirits': 6, "forsyth's": 1, 'sandbox': 1, "damme's": 2, 'u-boat': 1, 'conflicting': 4, 'taylor-thomas': 1, 'joyful': 3, 'graininess': 1, 'bills': 8, 'basic': 48, 'decked': 4, 'lauren': 3, 'lagging': 1, 'idiots': 3, 'gereally': 1, 'dialogue': 148, 'gregg': 1, 'empire': 18, 'piper': 3, 'conspire': 1, 'ingenuity': 3, 'dictatorships': 1, 'daunting': 2, 'clump': 1, 'beristain': 1, "1590's": 1, 'moreso': 1, 'subverted': 1, 'machinists': 1, 'bereavement': 1, 'senile': 1, "lookin'": 1, 'perrineau': 2, 'deltas': 1, 'darkman': 1, 'out--last': 1, 'defeated': 2, 'duran': 1, 'hinging': 1, 'scarface': 4, 'herky-jerky': 1, 'yelps': 1, 'mobile': 7, 'favourite': 6, 'terribly': 14, 'absolution': 1, 'dovetail': 1, 'overracting': 1, 'tardis': 1, 'nondescript': 2, 'fabulously': 2, 'overload': 3, 'summa': 1, 'hors': 1, 'memorial': 2, 'exalts': 1, 'angering': 1, '-4': 7, 'mcdormand': 6, 'donaldson': 2, 'hokum': 1, "`work'": 1, 'attached': 8, 'charmless': 1, 'behaved': 1, 'pointer': 2, 'culls': 1, 'installed': 1, 'amazed': 12, 'reluctantly': 19, 'failings': 2, "boys'": 2, 'chiseled': 2, 'bowler': 7, 'lemmons': 1, 'repeated': 10, 'senses': 21, 'mui': 1, 'skew': 1, 'washing': 1, 'strangled': 2, '17-day': 1, 'hyper': 4, 'underwater': 11, 'represents': 16, 'tranquil': 4, 'gunfire': 4, 'bail-bondsman': 1, 'no-': 1, 'wrong-side-of-the-tracks': 1, "nugent's": 1, 'cool-as-hell': 2, 'darius': 1, 'buyers': 2, 'copacabana': 1, 'buggs': 1, 'krabb': 1, '15-million-dollar': 1, 'puppy': 6, 'whacked-out': 2, '11': 9, 'unenjoyable': 1, 'domino': 1, "jessie's": 1, 'stagner': 1, 'pileggi': 1, 'tasks': 7, 'scraps': 1, "corleone's": 1, "`go'": 1, 'schmucks': 1, 'demonstrably': 2, "hernandez's": 1, 'embodying': 1, 'spontaneously': 3, "cartoonish-barlow's": 1, 'vast': 19, 'commuter': 1, 'facinelli': 1, 'incites': 1, '25-years': 1, 'morocco': 3, 'bareikis': 1, "rampling's": 1, 'project--sweet': 1, 'humbled': 2, 'cared': 9, 'supburb': 1, 'straight-laced': 1, 'scars': 7, "'yawn'": 1, 'genuis': 1, 'extraterrestrial': 5, 'delving': 1, 'swirls': 1, 'separately': 6, 'resignedly': 1, 'yasuoka': 1, "vail's": 1, 'non-exploitative': 1, "gershon's": 2, 'confident': 18, 'immortality': 1, 'jitterish': 1, 'placing': 3, "patrik's": 1, 'potent': 11, 'more--all': 1, 'groove': 2, 'bug': 9, 'untraditional': 1, 'ashmore': 1, 'eastward': 1, 'sketch': 5, 'phenomenal--observe': 1, 'sunlight': 4, 'timing': 18, "russian's": 1, 'operas': 2, 'plot-': 1, 'over-protective': 1, 're-release': 4, 'myagi': 1, 'thunderous': 1, 'arsinee': 2, "emma's": 1, 'parent-koening': 1, 'speedboat': 1, 'doomed': 16, 'nsync': 1, 'hiring': 2, 'elk': 2, 'costa': 1, 'cab': 10, 'map': 6, 'continuous': 1, 'analyst': 3, 'cremated': 1, 'dedicated': 16, 'dropout': 3, 'cherry-red': 1, 'agent-in-charge': 1, 'special-effects': 2, 'kiddie': 4, 'balthazar': 1, 'barbs': 1, 'ships': 13, 'slapstick': 27, 'cruise/paul': 1, 'wacked-out': 1, 'integrity': 9, 'grab': 8, 'radioing': 1, 'gulped': 1, 'double-teamed': 1, "depp's": 3, 'defenseless': 1, 'undead': 3, 'acrobatics': 1, 'contrivances': 4, 'painting': 19, 'underachieving': 1, 'kyra': 1, '`p-p-p-p-please': 1, 'pollack': 4, 'dissipate': 1, 'simply': 142, 'kiki': 1, 'relax': 5, 'bradshaw': 1, 'renfro': 3, 'moretti': 1, 'henricksen': 1, 'ether': 1, 'conaway': 3, "wolf-beiderman's": 1, 'seagulls': 1, 'soldier': 17, 'poe': 1, 'interviewing': 2, 'hope/crosby': 1, 'cumbersome': 1, 'supervisors': 4, 'harmlessly': 1, "frankenstein's": 1, 'lima': 2, "hurd's": 1, 'summaries': 1, 'entities': 3, 'negotiate': 7, 'order': 116, 'grown-up': 6, 'pages': 10, 'imho': 2, 'magazines': 4, 'hauer': 1, 'gem': 25, 'grizzled': 3, 'beer': 13, 'portrait': 24, 'curtail': 1, 'annoy': 2, 'own--are': 1, 'eye-level': 1, 'juggernaut': 1, 'tolerates': 1, 'subtlety': 19, 'exaggeratedly': 1, 'solidifies': 2, 'impolite': 1, 'misty': 1, 'hang-out': 1, 'cubism': 1, 'oft': 1, 'joisey': 1, 'invlove': 1, 'online': 8, 'original': 196, 'rowan': 2, 'crowing': 1, 'arlene': 2, "'82": 1, 'roller': 5, 'feathery': 1, 'dispatching': 1, 'aural': 4, 'rosio': 1, 'spice': 7, 'bella': 2, 'upsets': 2, 'bonkers': 2, 'promotional': 5, "'em": 6, 'drops': 11, 'symbolizing': 2, 'slips': 16, "bloomin'": 1, 'action-craving': 1, 'mail': 9, 'cliquey': 1, 'nugent': 2, "nixon's": 1, 'skip': 7, 'magda': 2, 'telephones': 1, 'sitting': 35, "quaid's": 1, 'multi-layered': 1, 'goodwin': 1, 'baluyev': 1, 'current': 35, 'stock-shoot': 1, 'awful': 19, 'pinnacles': 1, 'shelmikedmu': 1, 'resurrected': 3, 'guesses': 2, 'complacency': 2, 'maximum': 8, 'hymn': 2, 'recovery': 6, 'often-invisible': 1, 'manifestations': 2, '5/10': 15, 'globes': 1, 'botches': 1, 'semi-star-making': 1, 'no-holds-': 1, 'swastika-wearing': 2, 'archetypes': 5, "critic's": 6, 'doppling': 1, 'window': 29, 'luxury': 3, 'hydra': 1, 'nastiness': 2, 'laughless': 1, 'rangers': 2, 'rocker': 1, 'unbelieveably': 1, 'valleys': 2, 'self-congratulatory': 3, 'gautier': 1, 'runny': 1, 'felton': 1, 'turning': 33, 'inscribed': 1, 'ninjaman': 1, 'mistakingly': 1, 'fed': 11, 'cut-out': 2, 'yellow-greens': 1, 'full-speed': 1, 'speculations': 1, 'adorable': 10, 'mutilation': 2, "kinsella's": 1, 'noting': 10, 'ferris': 7, "kerrigan's": 1, 'bouyant': 1, 'amnesia': 3, 'swashbuckler': 1, 'removal': 6, 'one-year': 1, 'ejection': 1, 'retro-garbo': 1, 'keats': 1, "1970's": 6, 'comprehension': 4, 'shifting': 7, 'supreme': 8, 'commented': 1, 'family': 196, 'round': 15, 'trenches': 1, 'sobchak': 1, 'two-year-old': 1, 'quinn-robin': 1, 'straits': 1, 'retire': 6, 'pistols': 3, 'travis': 6, 'proclaims': 3, 'unconditional': 2, 'much-admired': 1, 'close-up': 3, 'idioms': 1, 'belts': 2, 'prayers': 4, 'smoothly': 9, 'aram': 1, 'germany': 9, 'segal': 1, 'derek': 11, 'stability': 6, 'casserole': 1, 'hairline': 1, 'storybook': 1, 'pricks': 1, 'stinkers': 1, 'scorsese': 17, 'mesmerize': 1, 'canvas': 5, 'derbies': 1, 'matchmakers': 1, 'life-affirming': 2, 'resents': 2, 'narrations': 1, "tate's": 1, "hollow'": 1, 'mikhail': 1, 'disciple': 4, 'consumed': 4, 'cloying': 3, 'metallic': 1, 'suspect\x05': 1, 'fuzzy': 5, 'crucified': 1, 'imponderable': 1, 'inhering': 1, 'breathtakingly': 3, 'nuptials': 1, 'slime': 2, 'dousing': 1, 'afterward': 8, 'enforcement': 5, 'internationally': 3, 'stealer': 6, 'raving': 7, 'la': 44, 'avoid': 38, '`strange': 1, 'mugs': 1, 'bat-like': 1, 'aimless': 2, 'hating': 2, 'humiliate': 1, 'ogre-witch': 1, 'definitions': 2, 'compete': 6, 'eli': 1, 'well-done': 6, 'oversimplification': 1, 'loni': 1, 'take-no-crap': 1, 'girlfriend/boyfriend': 1, 'cheats': 3, 'firefighter': 2, 'businesswomen': 1, 'succumb': 3, 'televised': 5, 'preferring': 3, 'accomplish': 7, 'amalgamations': 1, 'wear': 18, 'peeking': 1, 'devoured': 1, 'canals': 1, 'generalize': 1, 'category': 10, '18-plus': 1, 'cafes': 1, 'tepid': 2, 'hampered': 5, 'white': 94, 'milked': 3, "couples'": 1, 'regular': 27, 'fragments': 3, 'cartridges': 1, 'surround': 8, 'mesa': 1, 'engaging': 37, 'endowing': 1, "king's": 8, 'patriarchal': 3, 'upsurge': 1, 'revealing': 22, 'outed': 1, 'occured': 3, 'mind-moving': 1, 'relying': 4, 'mining': 4, 'intern': 2, 'jenny': 5, 'tissues': 1, 'protgaonist': 1, 'landmark': 5, 'dworkin': 1, 'interracial': 3, "cleon's": 1, 'coyle': 1, 'interminable': 2, 'python': 4, 'swain': 2, 'improved': 13, 'australian/belgian': 1, 'commemorating': 1, 'battles': 23, 'banquets': 1, 'baring': 1, 'entangled': 1, 'haughty': 1, 'skerritt': 2, 'poverty-stricken': 1, 'marijunana': 1, 'kit': 7, 'on-the-spot': 1, 'chairs': 2, 'veneer': 3, 'kingdome': 1, 'concluding': 2, 'pesticides': 1, 'trekker': 1, 'anti-charm': 1, 'comedies-': 1, 'amnesiac': 2, 'grading': 1, 'impersonation': 4, 'utterly': 32, 'urged': 1, 'mancina': 1, 'sickest': 1, 'orbiting': 1, 'escalated': 1, 'sticky': 2, 'enforces': 1, 'puppy-dog': 2, 'unrelentless': 1, 'sheppard': 1, 'deserting': 1, 'sedgwick': 1, 'formulas': 2, 'viet': 1, 'camelot': 2, 'reactor': 1, 'merrill': 3, 'matin': 1, 'tux': 1, 'boxer': 9, 'luke': 14, 'placating': 1, 'met': 25, 'paycheck': 2, 'carcass': 1, 'whiff': 1, 'iliff': 1, 'in-court': 1, 'self-confident': 3, 'farquaad': 5, 'norville': 3, "hubbard's": 1, 'exits': 2, 'telescope': 2, 'fiscally': 1, '-brand': 1, 'excess': 7, 'jeb': 1, 'sequitur': 1, 'disconnected': 1, 'exprience': 1, 'del': 8, 'texas/mexico': 1, 'griffith': 8, 'hooray': 1, 'galpal': 1, 'costanza': 2, 'rabid': 1, 'vicariously': 1, 'son-in-law': 1, 'tenth': 3, 'tautou': 1, 'coraci': 1, 'mills': 6, 'medicine': 3, 'planner': 1, 'smoking': 13, "activist's": 1, 'nastier': 1, 'develops': 36, 'tense': 28, 'organizes': 1, 'freeze': 6, "inkpot's": 3, 'owens': 2, 'nabokov': 1, 'guaranteeing': 1, 'property': 8, 'blossoms': 5, 'rejoiced': 1, 'brokerage': 1, 'drawback': 6, 'craftsmanship': 5, "archer's": 2, 'tenuous': 2, 'ill-conceived': 2, 'illusory': 1, 'dinosaur-like': 1, 'ursula': 1, 'joint-smoking': 1, 'cure': 10, 'burnham': 2, 'sweathogs': 1, 'crudely': 1, 'loath': 1, 'pic': 4, 'nerds': 3, 'initiates': 1, 'deniro-like': 1, '26': 3, 'slackers': 3, 'deposit': 2, 'plagiarism': 1, 'adviser': 3, 'teachers': 1, 'caste-like': 1, 'spoof': 11, 'salvage': 5, 'hope': 96, '1995': 24, 'crosses': 10, "tweedy's": 1, 'counting': 4, 'parodying': 3, 'emergency': 6, 'spell': 11, 'barksdale': 1, 'depression-era': 1, 'downey': 7, 'tarkovskian': 1, 'drop-out': 1, 'now-famous': 3, 'shearim': 1, 'strangles': 1, 'immune': 4, 'shreck': 1, 'wiped': 2, 'limousine': 1, 'ill--we': 1, 'giggles': 2, 'forgive': 10, 'confess': 1, 'movie-goer': 2, 'ticket': 22, "talkin'": 1, 'fin': 1, '--a': 2, 'uncontrollable': 4, 'prosecuted': 1, 'inexperienced': 1, 'statement-making': 1, 'shanghai': 3, 'razor': 1, 'low-level': 3, 'pesci': 5, 'naboo': 6, 'moviemaking': 7, 'versions': 13, '1/20th': 1, 'allegiances': 2, 'all-but-illegible': 1, 'hallway': 1, 'powers': 29, 'quota': 1, "demented's": 1, 'sneaks': 9, 'valet': 1, '9/10': 8, 'ferrer': 2, 'touch-a': 1, 'falsetto': 1, 'fundamentals': 3, 'jam-packed': 2, 'arranged': 3, 'twelve': 8, 'dumber': 7, 'culpability': 2, 'measured': 3, 'buddy': 23, 'auriga': 1, 'abandonment': 2, 'oklahoma': 1, 'admirer': 1, 'computech': 1, 'iii': 5, 'pushes': 10, 'watched': 33, 'quoting': 3, 'filby': 1, 'luckily': 32, 'toll': 6, 'deeply': 32, 'muscat': 1, 'no-name': 2, 'reintroduced': 1, 'cambodia': 1, '25-cent': 1, 'retains': 5, 'inner-friend': 1, 'pierre-augustin': 1, 'misrepresentation': 1, 'embody': 4, 'co-creator': 1, 'robinson': 3, 'crowning': 1, 'felonies': 1, 'jibberish': 1, 'alone--his': 1, 'rick': 7, 'rickman': 5, 'owner--veteran': 1, '`for': 1, "kids'": 6, 'accordion': 2, 'stunning': 51, 'aimee': 2, 'piled': 1, 'ignores': 4, 'hasan': 1, 'mistress': 6, 'swampy': 1, 'bob': 36, 'indie-film': 1, "ez's": 1, 'hiv': 2, 'influences': 4, 'foolishly': 2, 'spur': 3, 'humbert': 2, 'unhip': 1, 'brill': 2, 'terry': 20, 'nobles': 1, 'carrot-colored': 1, 'nuez': 1, 'longshanks': 1, 'unoffensive': 1, 'straightforwardness': 1, 'scottish': 10, 'mexico': 14, "90's": 11, 'koop': 1, 'duking': 1, 'dying': 33, 'mysticism': 2, 'forcefully': 3, 'waved': 1, '1992': 12, 'idol': 3, 'command': 26, 'masterpiece-he': 1, 'enhancing': 2, 'ensues': 17, 'adandon': 1, 'nudity': 22, 'soundtracks': 1, 'inter-galactic': 1, 'rendering': 2, 'meek': 3, "cuba's": 1, 'backdraft': 3, 'spelled': 5, 'artsy': 6, 'benson': 4, 'sanchez': 1, 'tyrese': 1, 'self-done': 1, 'cinematography--': 1, '`jackie': 1, 'villalobos': 1, 'rouge': 1, 'routes': 3, 'yin': 1, 'forsyth': 2, 'echoing': 2, 'dine': 1, 'begrudging': 1, 'bias': 2, 'awareness': 3, 'upright': 1, 'life': 428, 'banjos': 1, 'evne': 1, 'delegation': 2, 'machismo': 4, "roberts'": 3, 'collection': 24, "cliche'd": 1, 'two-century': 1, '+4': 7, 'dipping': 1, 'banditos': 1, 'ill-tempered': 1, 'lethargic': 1, 'knox': 2, 'long-hampered': 1, 'envied': 2, 'well-spoken': 1, "frears'": 1, 'sledgehammer': 2, '`enough': 1, "edgecomb's": 1, 'grilling': 1, "parker's": 2, 'tumultuous': 5, 'slave-ship': 1, 'system': 51, 'maurice': 9, 'inferior': 10, "jane's": 1, 'kong-hollywood': 1, 'documenting': 2, 'davin': 1, 'references': 27, 'scripting': 2, 'stoppidge': 1, 'upcoming': 10, 'ploy': 3, 'macguffin': 2, 'stressed': 1, 'subdesarrollo--': 1, 'extensively': 1, 'humility': 1, 'resistance': 7, 'agendas': 1, 'blemishes': 1, 'cheaper': 3, 'demons': 8, 'royce': 1, 'chronology': 1, 'orators': 1, 'kroon': 1, "'tradition'": 1, 'hackett': 2, 'option': 3, 'grail': 3, 'dispensing': 1, 'rickles': 3, 'carreer': 1, 'far-out': 1, 'reflexes': 1, 'devito': 11, 'unimportant': 1, '$70': 2, 'coming-of-age': 4, 'teaches': 10, "johnson's": 1, 'dieppe': 1, 'duplicitous': 1, 'wind': 24, 'fide': 1, 'pidgeon': 3, 'margot': 1, 'mistrust': 1, 'flamboyant': 3, 'candidates': 3, "lewis'": 1, 'vips': 1, 'moloney': 1, 'menage': 2, 'duel': 4, 'identikit': 1, 'shortcuts': 1, 'gentlemen': 3, 'spam': 2, "eye's": 1, 'chummy': 1, 'small-town': 1, 'caged': 1, 'opulent': 1, 'film--one': 1, "rep'ing": 1, 'mo': 2, 'bogs': 2, 'hiroshima': 2, 'capacity': 7, '$5': 4, "connery's": 1, 'well-designed': 1, "associates'": 1, 'copes': 1, 'faking': 4, 'comprehensive': 1, "shark's": 1, 'rochelle': 1, 'atrocity': 1, 'showstopping': 2, 'edgar': 6, 'cd-rom': 1, 'mucho': 3, 'doubtlessly': 2, 're-start': 1, 'humans': 37, 'coat': 8, 'congregation': 5, 'purchased': 2, 'discrepancies': 1, 'remington': 2, 'lestat': 2, 'dian': 1, 'ev': 1, 'sidekick': 23, 'juba': 1, 'heads': 43, 'overreaching': 1, 'strict': 14, 'johansson': 2, 'reeks': 2, 'gamblers': 3, "kidnappers'": 1, 'heil': 1, 'deliberately': 9, 'financially-oriented': 1, 'sctv': 3, 'robs': 6, 'scence': 1, 'cosmopolitan': 1, 'tilvern': 1, 'pre-1960s': 1, 'qualms': 4, 'tributes': 1, 'resolve': 8, 'average-sized': 1, 'stable': 9, 'makeup': 14, 'rudd': 1, 'inexorably': 4, 'glossing': 1, 'bitterness': 6, 'holistic': 1, 'presence': 72, 'true-love': 1, 'exploding': 10, 'fertile': 2, 'decter': 1, 'opponent': 5, 'claims': 34, 'kidnappers': 6, 'hurried': 2, 'slower': 7, 'cyborg': 3, 'bernhard': 1, 'violently--that': 1, 'patrice': 1, 'excellent': 103, 'hooking': 3, 'dispelled': 1, 'cackling': 2, 'flailing': 2, 'mileage': 2, 'bemoaning': 1, 'hate-mail': 1, 'feudal': 3, 'letter-perfect': 1, 'minds': 26, 'hard-to-categorize': 1, 'eastwood-directed': 1, "'i've": 1, 'mason': 2, 'column': 7, 'five-second': 1, 'whitechapel': 1, "europe's": 1, 'hustle': 1, "york's": 4, 'inhumanly': 1, 'inspect': 1, "lugosi's": 1, 'brackett': 1, 'curtains': 1, 'adaptation': 38, 'trunchbull': 1, 'adams': 17, 'teased': 3, 'self-mocking': 1, 'camembert': 1, 'declaring': 1, 'free-form': 1, 'ass-map': 1, "is'": 1, 'primer': 1, 'dilemmas': 4, "authorities'": 1, 'subtely': 1, 'mayer-goodman': 1, "'star": 1, 'suspense/horror': 1, 'rallied': 1, 'bravest': 2, 'startlingly': 5, 'performer': 11, 'burlesque': 1, 'happier': 3, 'distracting': 22, 'dramatically': 6, '35th': 1, 'pep': 5, 'external': 4, 'wells=92': 1, 'lackadaisical': 1, 'devastate': 1, "placid'": 1, 'chrichton': 1, 'mckellen': 2, "hobbes'": 1, 'ky': 1, 'brooding': 6, "ship's": 7, 'wages': 2, 'horribly': 13, 'damper': 2, 'dealer': 17, 'exploit': 8, 'dutton': 2, 'dreyfus': 2, 'lily': 3, 'cage-world': 1, 'revisiting': 4, 'undergoing': 1, 'gunmen': 1, 'philosophies': 3, "fate's": 1, 'behind-the-scenes': 3, 'resuscitate': 1, 'lovelorn': 1, 'distinctive': 9, 'episode': 38, 'northwest': 3, 'validity': 1, 'dishwasher': 2, 'alea': 1, 'decrees': 2, 'chili': 1, 'nameless': 4, 'titan': 1, 'finish': 18, 'pettiness': 1, 'annoyed': 7, 'lightning': 5, 'pales': 3, 'toby': 3, 'astronauts': 3, 'crafted-not': 1, 'hannah': 7, 'features': 82, 'mooney': 1, "lebowski's": 4, '175': 1, 'mortal': 9, 'harebrained': 1, "duke's": 2, 'slashing': 2, 'files': 3, 'supremely': 2, "illiterate'": 1, 'normalcy': 2, "alex's": 4, 'countdown': 1, 'scene--even': 1, 'roman-chariot': 1, 'starkly': 1, 'refreshingly': 12, 'alterations': 2, "'roles'": 1, 'newbie': 1, 'mist': 3, 'sweeps': 4, 'stewart': 18, 'benignly': 1, 'buddhist': 1, 'morning': 25, 'motherland': 1, 'declared': 5, 'included': 32, 'retain': 4, 'dual': 3, 'occult': 1, 'psitive': 1, 'hue': 1, 'mommas': 1, 'phillip': 8, 'tak': 1, 'concentrating': 3, 'perfection': 27, 'gobbles': 1, 'rolls': 6, 'poet': 4, 'conviction': 13, 'fulfillment': 6, "tarantino's": 10, 'kosovo': 1, 'native-born': 1, 'broadcasted': 1, 'germ': 1, 'obtaining': 3, 'delirious': 3, 'masters': 9, 'blurred': 3, '357': 1, 'needy': 2, 'injections': 1, 'roars': 2, 'know-it-all': 1, 'fellinian': 1, 'mcquarrie': 2, 'pyramid': 1, 'banality': 1, 'aunts': 1, 'alternate': 12, 'bandmate': 1, 'plenty': 55, 'marvin': 3, 'administered': 1, "jackie's": 4, 'ken': 8, 'airplane-type': 1, 'morphine': 1, 'planned': 12, 'blueblood': 1, 'stare': 3, 'civilization': 10, 'targetted': 1, 'sabotage': 7, 'bloomington': 2, 'purposely': 1, 'breathlessly': 3, 'harbor': 4, 'salami': 1, 'alternately': 2, 'tell-all/show-all': 1, 'emotive': 1, 'conversely': 2, 'impersonators': 1, 'nails-tough': 1, 'food': 33, 'spyglass': 2, 'hrundi': 1, 'professionalism': 1, 'mammoth': 1, 'alexandra': 4, 'plot-relevant': 1, 'oddest': 2, 'hines': 1, 'columbine': 1, 'eternally': 1, 'adores': 1, 'grand-daughter': 1, 'upbeat': 7, 'font': 1, 'cease': 3, 'rig': 3, 'progression': 5, 'over-explain': 1, 'measly': 1, 'grotesque': 6, 'friedrich': 1, 'talky': 4, 'iconoclast': 1, 'randal': 1, 'robbery--he': 1, 'authors': 4, 'impulse': 2, 'greener--and': 1, 'cricket': 5, 'bent': 6, 'babboons': 1, 'pussyfoot': 1, 'emshwiller': 1, 'salacious': 1, 'gowns\x05\x05euuugh': 1, '_unbreakable_': 1, 'reggae': 2, 'broadbent': 3, 'btw': 1, 'adepts': 1, 'is]': 1, 'loathes': 1, 'wolves': 3, 'ran': 13, 'fernanda': 1, 'blockbusters': 5, 'latifah': 2, 'stairs': 4, "'dante's": 1, 'goeth': 1, 'schrader': 4, "mcgregor's": 2, 'eight-hundred': 1, 'garrison': 1, 'quills': 1, 're-wire': 1, 'commanded': 2, 'climactic': 26, 'intrigues': 2, 'editors': 2, "woo's": 4, 'actors-': 1, 'levar': 1, "zurg's": 1, 'risqu': 1, 'herded': 1, 'scully': 6, 'bloom': 2, 'scarcely': 1, '---has': 1, "muriel's": 3, 'cuddy': 1, 'decrepit': 3, 'beard': 2, 'mediocre': 24, 'pluck': 1, 'hayworth': 1, 'cheapskate': 1, 'eurocentrism': 1, 'pure': 47, 'pommfrit': 1, 'phone': 35, '110': 2, 'frances': 13, 'shostakovich': 1, 'hull': 1, 'serling': 2, 'sppedboat': 1, 'subscribe': 2, 'rule': 20, 'increased': 2, 'commando': 2, 'epitomizes': 1, 'felicity': 5, 'goony': 1, 'web': 9, 'homeworld': 1, 'platt': 7, 'carla': 3, 'overpopulated': 1, 'rejoicing': 1, 'manufacturer': 2, 'sprite': 2, 'worn': 10, 'watery': 3, 'emporer': 1, 'breakdown': 12, 'explains': 26, 'devastation': 7, 'bets': 2, 'career-defining': 1, 'adoption': 5, 'syndrome-like': 1, 'deakins': 4, 'boos': 1, 'screed': 1, 'bleach-blond': 1, 'pets': 2, 'lowe': 5, 'prostitution': 2, 'weitzman': 1, 'suitable': 13, 'joke': 33, 'postponement': 1, 'monroth': 1, 'sinister': 22, 'fatales': 1, "carter's": 6, 'lucinda': 1, '22': 7, 'uptown': 1, 'soupy': 1, "labyrinth's": 1, 'melodic': 1, "i-don't-know-when": 1, 'wallows': 1, 'couture': 1, 'winbush': 1, 'wilde': 3, 'hearts': 16, 'overwritten': 3, 'civilisation': 2, 'notting': 3, 'gator': 1, 'drug-addicted': 2, 'talk-show': 3, 'masterfully-done': 1, 'non-roles': 1, 'kirshner': 2, 'realism': 23, 'logical': 6, 'gabriella': 3, 'conniptions': 1, 'aristocratic': 3, 'kingsley': 2, 'lacombe': 1, 'predator': 1, 'observations': 14, 'shrugged': 1, 'hebrews': 1, "leoni's": 1, 'catatonic': 3, 'kaftan': 1, 'tightest': 1, 'skarsgaard': 1, 'polymorphously': 1, 'aspire': 4, 'slack': 2, 'flares': 1, 'lili': 2, 'foremost': 12, 'nine-year-old': 2, 'strong-willed': 2, 'words': 88, 'remand': 1, 'odd': 46, 'seemingly-harmless': 1, "'toy": 1, "destination'": 1, 'chilled': 1, 'superiors': 5, 'apothecary': 1, 'bartok': 1, 'amounts': 10, 'katie': 4, 'fists': 2, 'anabasis': 1, 'crocheting': 1, 'wynn': 1, 'apartment': 45, 'compounded': 1, 'unsettled': 1, 'career': 86, 'isaach': 1, 'hostility': 1, 'ironic': 24, 'brambell': 1, 'fichtner': 2, "actors'": 2, 'higher-ups': 1, 'result': 97, 'palantine': 1, 'boxed': 1, "swope's": 1, 'dug': 6, 'systematically': 4, 'turkeys': 1, 'stormare': 4, 'coward': 1, 'pleases': 2, 'believably': 5, 'stitches': 5, 'set-piece': 2, 'filthy': 2, 'assisting': 1, 'aggrivate': 1, 'disappointingly': 3, "see's": 1, 'hard-luck': 1, 'floorboards': 2, 'ww2': 3, 'main': 154, 'long--so': 1, 'snorted': 1, 'participate': 3, 'chirping': 1, 'encounter': 25, 'parenthetically': 1, 'miniseries': 1, 'maduro': 1, 'stretching': 3, 'loathing': 4, 'breath-taking': 1, 'harsh-reality': 1, 'bloopers/flubs': 1, 'double-natured': 1, 'self-discovery': 2, 'naturally': 30, "someone's": 8, 'deja-vu': 1, 'swicord': 1, 'kafka': 3, '`idle': 1, 'well-paced': 5, 'pouts': 1, 'lucky': 28, 'wiser': 2, 'landwalker': 1, 'handed': 9, 'detritus': 1, 'lebanese': 1, 'locksley': 1, 'applications': 2, "mehta's": 1, 'admires': 3, 'fearing': 8, 'dazzle': 2, 'aberdeen': 1, 'footnote': 1, 'excursions': 2, '_the': 5, 'tougher': 6, 'slothful': 1, 'goth-girl': 1, 'vaus': 1, 'bluntness': 2, 'marketplace': 2, 'anticipating': 5, 'burden': 5, 'vigilant': 1, 'zigs': 1, 'out-going': 1, "ol'": 12, 'arc': 8, 'occasion': 14, 'voyager': 3, 'timbre': 1, 'snoop': 1, "dolores'": 1, 'rounders': 3, 'worried': 11, "pidgeon's": 1, 'hormones': 4, 'high-spirited': 2, 'pang': 1, 'preachers': 1, 'unleash': 1, 't-birds': 1, 'arian-looking': 1, "kingpin's": 1, 'apostle': 6, 'images': 58, 'roald': 1, 'jubilee': 1, 'lightness': 1, 'hearted': 4, 'espadrilles': 1, 'impregnated': 1, 'menendez': 1, 'vicki': 2, 'gnarled': 2, 'september': 7, 'love': 312, 'riff-raff': 2, 'harmonies': 1, 'kind': 185, 'outspoken--perhaps': 1, 'supress': 1, 'elgin': 1, 'priestly': 1, 'nen': 1, 'incidentally': 9, 'state-run': 1, 'berlin': 6, 'creditors': 1, 'predicate': 1, 'cucumber': 1, 'intertwine': 1, 'uluru': 1, 'bend': 5, 'humpalot': 2, 'reserved': 10, 'unwarranted': 2, 'director/producer': 1, "z's": 2, 'burglars': 1, 'legitimize': 1, 'dramatic': 89, 'academe': 1, 'departing': 2, 'mustafa': 1, 'evars': 1, "homer's": 3, 'multi-line': 1, 'mainstay': 1, 'profanities': 1, 'connected': 11, 'pummeling': 1, 'bogeyman': 1, 'harmon': 1, 'recreation': 8, 'infused': 1, 'nothingness': 1, '802': 1, 'victorian-era': 1, 'undercurrent': 6, 'car': 88, 'ck': 1, 'barcode': 1, 'spousal': 1, 'retention': 1, 'risked': 1, 'aurelien': 1, 'maryland': 2, 'meteors': 1, 'braddock': 1, 'miyamoto': 2, 'rubin': 3, 'intimately': 2, 'sorted': 1, 'spatial': 1, 'backward': 2, 'skeet': 1, '63': 1, '=': 5, "world's": 17, "house's": 2, 'origins': 7, 'engulf': 2, 'prophesied': 2, 'circumstances': 28, 'alright': 5, 'sabara': 1, 'hallstr': 1, 'technicality': 2, 'indigenous': 1, 'supply-trading': 1, 'import': 8, 'pertaining': 3, 'evacuation': 1, 'slice-and-dice': 3, 'poltergeist': 3, 'pronouncements': 2, 'bewonderment': 1, 'worries': 1, 'courtrooms': 1, 'cult': 32, 'skirmishes': 1, 'placer': 1, 'skins': 1, 'landau': 4, 'canadians': 3, 'familiarity': 5, 'chopsticks': 2, 'federico': 2, 'lets': 36, 'awarding': 1, "beings'": 1, 'oscars': 14, 'spies': 7, 'things/sprites': 1, 'mimicking': 1, 'nail-bitingly': 1, 'boorman': 1, 'firestorm': 1, "toll's": 1, 'proficiency': 2, 'gained': 11, 'hoped': 16, 'kingpin': 7, 'retrieval': 1, 'sad-looking': 1, 'nascar': 1, 'apoplectic': 1, 'effect': 72, "'independent": 1, 'good-looker': 1, 'terminated': 1, 'dire': 6, 'nadia': 1, 'applause': 5, 'wow': 8, "coen's": 6, 'naysayers': 1, 'horsing': 1, 'self-worth': 3, 'drive-by': 1, 'lice': 1, 'public': 41, 'witch-wannabe': 1, 'tolkin': 1, 'simpithize': 1, 'financial': 12, 'puerile': 1, 'commercially': 4, 'democracy': 3, "design'": 1, 'reviewer': 9, 'mattered': 3, 'trip': 44, 'saucy': 2, 'ichabod': 3, 'inescapable--and': 1, 'advantages': 4, 'iraq': 1, 'reminded': 27, 'ashlee': 1, 'diesel': 6, 'full-well': 1, 'dares': 6, 'leguizamo': 3, 'light-heartedness': 1, "labor's": 1, "robots'": 1, 'aug': 1, 'assess': 2, 'versatile': 7, 'morons': 1, 'spider': 4, "schoolboy's": 1, 'sculptures': 1, 'responded': 2, 'provoke': 3, 'apartments': 3, 'runied': 1, 'veins': 2, 'mistfortune': 1, 'rawness': 1, 'conference': 4, 'supertechnology': 1, 'meaningless': 4, 'guru': 3, 'healings': 1, 'grounds': 6, 'smattering': 1, "mizrahi's": 1, 'bountiful': 1, 'reacting': 3, 'assimilate': 1, "eddie/dirk's": 1, 'frears': 1, 'allie': 1, 'strategist': 1, 'hunter': 17, 'tracks': 20, 'breathnach': 1, 'mummies': 2, 'creator': 13, 'du': 5, 'just-another-striptease': 1, 'inhabit': 7, 'oneders': 1, 'gun-runner': 1, 'russia': 3, 'ardently': 1, '$3': 1, 'humourous': 1, 'snapped': 1, 'boyz': 1, 'noise': 10, 'succeed': 22, 'stevens': 2, 'jolly': 4, 'assimilation': 1, 'scriptures': 2, 'lt': 7, 'businesswoman': 1, 'sidious': 1, 'coal': 5, 'recess': 2, 'dream-like': 3, 'knees': 2, 'wessex': 1, 'hack': 7, "'holocaust": 1, 'well-educated': 2, 'stops': 33, 'economical': 3, 'dope-dealing': 1, 'auntie': 1, 'hallucination': 2, 'encouragement': 2, 'pornography': 4, 'brotherly': 1, 'etchings': 1, 'reveals': 24, 'antagonists': 2, 'reminiscient': 3, 'signals': 4, 'sacred': 6, 'spin': 15, 'up-beat': 1, 'lenses': 1, 'epsode': 1, "either's": 1, 'wonder-bred': 1, 'shilling': 1, 'slated': 1, 'hallucinogenic': 2, 'sores': 2, 'consensus': 1, "`lower-class'": 1, 'anniversary': 8, 'disciplined': 3, 'balancing': 11, 'servicewear': 1, 'sleeve': 6, 'amanda': 8, 'forget': 55, 'fight/rocky': 1, 'pushy': 3, 'tremendously': 6, '[good]': 1, 'snapshots': 3, 'revenue': 2, 'kings': 7, 'kidman': 5, 'klingon': 3, 'chaney': 1, 'fashion': 43, 'reno': 4, 'index': 1, 'loverboy': 1, 'royalty': 2, 'nutmeg': 1, 'gentry': 1, 'owner-turned-painter': 1, '`opium': 1, 'timo': 1, 'depletes': 1, 'biggerstaff': 1, 'reticent': 2, 'hundred-year-old': 1, 'reducing': 1, 'cyncial': 1, 'immersing': 3, '$27': 2, 'acid-trip': 1, 'entitlement': 1, 'historically': 7, 'vaughns': 1, 'arrival': 20, 'flicks': 34, 'presentations': 1, 'rochefort': 1, 'employee': 12, 'frailties': 1, 'quote': 9, 'wonders': 23, 'probability': 1, 'matter-their': 1, 'freudian': 2, 'cling': 1, 'stoltz': 2, 'pneumonia': 1, 'post-war': 3, 'neigbors': 1, 'pleasance': 1, 'gleefully': 8, 'storage': 4, 'yorke': 1, 'bid': 2, 'skateboarding': 3, 'ironside': 2, 'organ': 1, 'librarian': 4, 'bra': 1, 'weir': 8, 'caller': 2, 'psychedelic': 4, 'diabolical': 8, 'nuclear': 21, 'corresponds': 2, 'trollop': 1, 'dozens': 12, 'underworld': 10, 'negotiators': 1, 'ramen': 1, 'prydain': 1, 'arrivals': 2, 'supporters': 4, 'depressive': 1, "castro's": 1, 'coke-addled': 1, 'trundle': 1, 'steerage': 3, 'assaulting': 3, 'caine': 3, "january's": 1, 'sloth': 1, 'leblanc': 1, 'commentary': 24, 'pixie': 1, 'curtis': 13, 'juxtaposing': 2, 'stripclub': 1, 'daily': 10, 'chuckie': 4, 'amerocentric': 1, 'easy': 96, 'bloody': 28, 'saturday': 14, 'pigs': 3, 'rants': 4, 'giovanni': 6, "kid's": 7, 'spree': 4, 'big-budget': 10, 'diguise': 1, 'galas': 1, 'surname': 1, 'plagerism': 1, 'joined': 11, 'counted': 5, 'abroad': 4, 'mingle': 1, 'lo': 4, 'anti-aircraft': 1, 'shortly': 24, 'co-dependency': 1, 'bluntly': 2, 'land': 43, 'fix': 8, 'resume': 3, 'usurped': 1, 'irishman': 2, 'filtered': 1, 'sites': 4, 'restored': 7, 'delicatessen': 2, 'issued': 1, 'career-topping': 1, 'gingerbread': 5, 'abuse': 11, 'hatch': 2, "rush's": 1, 'unrequited': 5, 'conveying': 7, 'confidant': 1, 'weakly': 1, 'hypnotism': 2, "hoskins'": 1, 'hitler': 2, 'obsession': 15, 'num-num': 1, 'poetic': 11, 'bass': 3, 'fantasy/allegory': 1, 'ramsey': 4, 'girlfight': 1, 'query': 1, 'bostock': 1, 'solon': 1, 'racers': 2, 'simple-minded': 1, 'limp': 3, 'regions': 1, 'batman': 19, 'augusts': 1, "'good": 2, 'vainly': 1, 'james': 147, 'vanished': 1, 'scots-irish': 1, 'sherbet': 1, 'crowe': 9, 'gangsta': 5, 'similar': 95, '19th': 11, 'tires': 2, 'it=92s': 1, 'manhole': 1, 'impulses': 5, 'healers': 1, "egoyan's": 4, 'troubling': 3, 'wangenhein': 1, 'paragraph': 7, 'oomph': 1, 'dvd': 18, 'unsuspenseful': 1, 'devane': 2, "ronna's": 1, 'workers': 14, 'sculptor/russian': 1, '_': 2, 'dennison': 1, 'alas': 11, "chereau's": 1, '460': 1, 'appeased': 1, 'answered': 7, 'strategize': 2, 'mara': 1, 'western': 19, 'homerian': 1, 'twist': 32, 'illuminated': 1, '/10': 1, 'coenesque': 1, "bromides'": 1, 'unemployed': 8, 'ceaseless': 1, 'teen-comedy': 1, 'duetting': 1, 'mandy': 3, 'right-wing': 1, "jordan's": 8, 'mordred': 1, 'gole': 1, 'maltese': 1, 'volcanic': 1, 'action--': 1, 'conveys': 12, 'blotted': 1, 'interpol': 1, 'russian-sounding': 1, 'arch-rival': 1, 'revolving': 3, "cmaeron's": 1, 'burn': 4, 'dicaprio': 11, 'echo': 4, 'dugan': 1, 'silvery': 1, 'man=92s': 1, 'heartily': 1, 'final': 158, 'outrageous': 19, 'dunsworth': 1, 'african-american': 9, 'congolese': 2, "finn's": 2, 'schoonmaker': 1, "son's": 13, 'mcnaughton': 5, 'budget-wise': 1, 'trailing': 3, "aronofsky's": 1, 'one-horse': 1, 'ripped': 9, 'cradling': 1, 'cruiser': 1, 'anatomy': 6, 'suspense--and': 1, "barth's": 1, 'halloweentown': 1, 'showpieces': 1, 'mcguigan': 1, 'rosalind': 1, 'stallion': 1, 'dismembering': 1, 'mindset': 2, 'goggins': 1, 'robbing': 2, 'triad': 1, 'not-guilty': 2, 'dariush': 1, 'pet': 10, 'rehearsing': 1, 'bargaining': 1, 'overrated': 6, 'preforming': 1, 'continuing': 7, 'da': 2, 'piquant': 1, 'kathryn': 3, 'butt-kicking': 1, 'highly-populated': 1, 'opposition': 9, 'heston': 4, 'half-starved': 1, "ganz's": 1, 'tidbits': 3, 'dialectic': 1, 'comet-bombing': 1, 'proposes': 6, "pilots'": 1, 'conceivable': 2, 'semi-': 1, 'pisses': 1, 'lulling': 1, 'identify': 28, 'laugh-out-loud': 5, 'feral': 1, "singleton's": 1, 'diving': 6, 'diego': 3, 'skits': 3, 'explosions': 23, 'ambiguities': 2, 'signs': 13, 'infertility': 1, 'sidekick/sidekicks': 1, 'nucci': 2, 'done-that': 1, 'greenwood': 6, 'pankow': 1, 'enemies': 11, 'joys': 5, 'nineteen-eighties': 1, 'leaned': 1, 'bb': 1, 'misspelled': 1, 'caravan': 1, "sidney's": 4, 'doc--in': 1, 'confections': 1, 'ho-hum': 3, 'forego': 1, 'infestation': 1, 'braniff': 1, 'locked': 12, 'pups': 1, 'unparalled': 1, 'standout': 15, 'glowing': 9, 'fun-making': 1, 'afield': 1, "communication'": 1, 'herbal': 1, 'generic': 14, 'proceeds': 12, 'tilda': 3, 'co-writers': 3, 'soon-to-be-marines': 1, 'uncles': 1, 'made-up': 1, 'dispatch': 2, 'helicopter': 5, 'garbage': 5, 'mails': 1, 'baffles': 2, 'shivers': 1, 'lonely': 36, 'enforcer': 1, 'beasts': 4, 'heavily-bespectacled': 1, 'assisted': 8, 'decisive': 4, 'terence': 4, 'raoul': 2, 'tsutomu': 1, 'production': 126, 'proponents': 1, 'freedman': 1, 'gunplay': 8, 'serbedzija': 1, 'hires': 15, 'tree-surfs': 1, 'satiate': 1, 'deservedly': 4, 'responses': 5, 'genesis': 1, 'completing': 4, 'sought': 6, 'mediocrity': 1, 'u-boats': 1, 'sequels': 24, 'surmise': 2, 'opaque': 1, 'pops': 10, 'diamonds': 4, '`keeping': 1, 'gay': 30, 'middle-aged': 8, 'opposites': 2, 'substantiated': 1, 'noticeable': 12, 'fellatio': 1, 'y2g': 1, 'affirming': 1, 'chum': 1, 'executions': 2, 'meredith': 1, 'spawned': 8, 'conditioned': 1, 'maybury': 1, 'festering': 1, 'boogieman': 1, 'underwent': 1, 'kerry': 3, 'squeezed': 2, 'searchlight': 1, 'albertson': 2, 'henslowe': 1, 'tv-movie': 2, 'overboard': 10, 'banana': 4, 'horrors--bad': 1, 'kuo': 1, 'ruining': 6, 'shaun': 2, 'conversion': 2, 'child-genius': 1, 'blue-eyed': 1, 'populus': 1, 'crowdpleasing': 1, 'mcjob': 1, 'flashback-type': 1, 'peach': 3, 'rarer': 1, 'unearth': 2, 'mifune': 1, 'claustrophobic': 11, 'incandescent': 1, 'precinct': 2, 'legendary': 21, 'surface': 30, 'ulrich': 1, 'explicitly': 7, 'foul-mouthed': 7, 'charge': 19, 'engagement': 4, 'sanity': 9, 'rushes': 6, 'stressful': 3, 'beef': 1, '[beware': 1, 'shades': 5, 'testing': 4, 'braeden': 1, 'always-interesting': 1, 'hint': 17, 'gratifying': 1, 'donal': 1, 'cooney': 1, 'silver-bladed': 1, 'lid': 2, 'bleached': 2, 'granddaddy': 1, 'futuristic': 13, 'polarizing': 1, 'mini-van': 2, '5th': 4, 'conniving': 4, "ship'": 1, '27': 2, 'topical': 1, 'tranisition': 1, 'hot-tempered': 1, 'coincided': 1, 'loud-mouth': 1, "auteuil's": 1, 'film': 798, "beal's": 1, 'dinosaur': 3, 'persay': 2, 'comic-slapstick': 1, 'appetites': 3, 'betting': 3, 'sony': 1, 'inkling': 2, "gabriela's": 1, 'vibrates': 1, 'metro': 1, "elijah's": 1, 'knowingly': 2, "'shoot-em-up'": 1, "hogarth's": 1, 'environs': 1, 'spasmodic': 1, 'kitten': 1, 'neighing': 1, 'applies': 2, 'courtesans': 1, 'campell': 1, "weddings'": 2, 'qe2': 1, 'disappearance': 3, 'pertain': 1, 'panorama': 2, '50-foot': 1, 'earth-hangs-in-the-balance': 1, 'sodomized': 1, 'bracken': 1, 'linguists': 1, 'heavily-armed': 1, 'folks': 20, 'summarize': 1, "crap'": 1, 'feature-length': 5, 'overstay': 1, 'continuously': 9, 'raspiness': 1, 'mislead': 1, 'purposes': 7, 'stampede': 4, 'pious': 1, 'schillings': 1, 'soldini': 2, 'ceilings': 1, 'positioned': 2, 'brits': 3, 'power-trip': 1, 'sang': 2, 'place': 235, 'gloomy': 9, 'lama': 3, 'chutes': 1, 'furlong': 3, 'conrad': 4, 'application': 2, 'lydia': 3, 'squashed': 2, 'happenings': 3, 'detailing': 5, 'run-ins': 1, "argento's": 3, 'egon': 1, 'fletch': 1, 'shallowly': 1, 'cute-ified': 1, 'fine--': 1, 'epos': 1, 'suitcase': 1, "rebel's": 1, "rodriguez's": 1, 'kathleen': 7, 'fiesty': 3, 'millionaire': 19, 'jesse': 4, 'dussollier': 1, 'sounded': 5, 'synthesis': 1, 'vulgar': 4, 'plagued': 6, "norristown's": 1, "norton's": 7, 'planners': 1, 'unpleasantness': 1, 'made': 329, 'guffaw': 1, 'uneasieness': 1, 'goofiness': 1, 'neville': 1, 'packers': 1, 'reminiscing': 2, 'deftness': 1, 'fateful': 8, 'oddly': 28, 'farting': 1, 'disracting': 1, 'con': 12, 'versa': 4, 'warmly': 1, 'jiving': 1, 'bestowed': 2, 'fervent': 2, 'intellectual': 20, 'exclaim': 1, 'hypsy': 1, 'sims': 5, 'unwisely': 1, 'val': 6, 'snatchers': 2, 'storywise': 1, 'lloyd': 18, 'ophelia': 2, 'transit-oriented': 1, 'feed': 6, 'breakdances': 1, 'coto': 1, 'canyon': 1, 'so--there': 1, 'loophole': 1, 'godsend': 1, 'kill-or-be-killed': 1, 'teenager': 13, 'drugstore': 3, 'draw': 35, 'quaid': 9, 'scaffolding': 1, 'dreamy-eyed': 1, 'applied': 6, "woody's": 5, "mailer's": 1, 'hoyle': 1, 'afri-can-american': 1, "1800's": 1, 'theme': 61, 'sheer': 35, 'churches': 4, 'garish': 2, 'wrath': 4, 'kaczkowski': 1, 'exclaims': 3, 'superb': 62, 'lunacy': 3, 'essentially': 29, 'buffoons': 3, 'permitted': 2, 'primate': 1, 'baggage': 2, 'severities': 1, 'object': 16, 'explode': 5, 'seemingly': 49, 'complainer': 1, 'soprano': 2, 'poke': 4, 'bugged': 2, 'discoveries': 4, 'well-realized': 1, "wincott's": 1, 'hesitation': 6, 'reprises': 5, 'blond-haired': 1, 'standouts': 6, 'engineering': 1, 'profiling': 1, 'chat-up': 1, 'hard-fought': 1, 'excused': 5, 'whopping': 1, 'filmdom': 1, 'teen-age': 2, 'jewison': 2, 'tearing': 1, '20-year': 2, 'unbeknownst': 6, 'programmer': 2, 'spoil': 18, "dead-guy-seems-to-have-come-back-to-life-but-then-we-find-out-it's-only-a-dream": 1, 'cloned': 2, 'terminology': 1, 'ballstein': 1, 'raises': 16, 'premise': 69, 'slicing': 1, 'maimed': 1, 'personas': 2, 'surmises': 1, 'multi-millionaire': 1, 'harrington': 2, 'vines': 1, '1978': 5, 'engagements': 1, 'self-effacing': 1, 'murders': 21, 'flag': 6, 'computers': 10, 'zoo-like': 1, 'washington': 23, 'stanwyck-gloria': 1, 'beefed': 1, 'show': 198, 'drumbeat': 1, 'breathtaking': 33, 'bo': 3, 'activites': 1, "addam's": 1, 'morlocs': 1, 'obsessive-compulsive': 2, 'equalizes': 1, 'armin': 3, 'indescribable': 2, 'worthy': 43, 'relevant': 6, 'honestly': 13, 'aphrodite': 4, "generation's": 1, 'stoicism': 2, 'groups': 15, 'slashed': 2, "'hospital": 1, 'plentiful': 5, "lesra's": 1, 'silence': 15, 'indefinantly': 1, 'zealots': 1, 'ladders': 1, 'reeking': 1, "laurie's": 1, 'dreads': 1, 'rumored': 2, "irving's": 4, 'organ-playing': 1, 'naps': 1, 'earth-like': 1, 'associates': 4, 'real-estate': 2, 'bookends': 2, 'enticing': 1, 'tribal': 2, 'concentrate': 7, 'wheels': 7, 'alteration': 4, 'rendition': 9, 'memphis': 1, 'non-trekkies': 1, 'digs': 5, "garcia's": 1, 'worldly': 1, 'dishes': 4, 'ride': 64, 'roulette': 1, 'webcams': 1, 'neighbours': 1, 'swells': 2, 'petulant': 1, 'dawes': 1, 'part-owner': 1, "money'": 1, "jesus'": 1, 'permissiveness': 1, 'gordo': 2, 'pill': 1, 'intensity': 38, 'finds': 156, 'deadbeat': 1, 'shudder': 1, 'walhberg': 1, 'frenchmen': 1, 'rivalry': 3, 'proportion': 2, 'wayne': 22, 'fiddles': 1, 'pressing': 2, 'epilepsy': 1, 'manhood': 2, 'inadequacy': 3, "goodman's": 2, 'beckoning': 2, 'non-supernatural': 1, '`after': 1, 'horror-comedy': 1, 'delhem': 1, "julia's": 1, "peebles'": 1, 'hosting': 1, 'longingly': 1, 'towering': 4, "films'": 4, 'elements': 95, 'lean': 13, "tribune'": 1, 'blaring': 2, 'debris': 2, 'over-used': 1, 'self-deprecating': 1, 'sister': 54, 'sverak': 1, "lambs'": 1, 'mcfadden': 1, 'complicate': 3, 'maclachlan': 2, 'jumps': 15, 'dimitri': 1, "station's": 1, 'sign-up': 1, 'motorhead': 1, 'voice-over': 15, 'octane': 1, 'forward': 32, 'unneeded': 2, 'layered': 9, 'widely': 5, 'toes': 4, 'mist-shrouded': 1, 'coffey': 3, 'appeal': 38, 'radius': 2, 'orator': 1, 'luggage': 1, 'mansley': 1, 'bouquet': 1, 'ultra-secret': 1, 'delights': 11, 'raimi': 5, 'greener': 2, 'addition': 45, 'complexity': 14, 'railing': 1, 'wife-to-be': 3, 'enters': 22, 'unquestionably': 5, 'humdrum': 1, 'hygiene': 3, '_arrrgh_': 1, 'nu': 3, 'polygamy': 1, 'salute': 1, "predecessor's": 2, 'fishburne': 11, 'booboos': 1, 'designs': 7, 'levien': 1, 'gazillion': 1, 'gamely': 2, 'becker': 2, 'swope': 1, '27/98': 1, 'descriptions': 3, 'nurture': 1, 'recalls': 9, 'spiders': 2, 'klux': 1, 'dignitary': 1, 'surroundings': 11, 'backdrops': 2, "witches'": 1, 'dokken': 1, 'featurette': 2, 'aloise': 1, 'count': 31, 'hurricane': 8, 'throwaways': 1, 'sacks': 1, 'nielson': 2, 'tiring': 1, 'bookstore': 7, 'tightness': 1, 'aldous': 1, 'ice': 30, 'newsreel': 2, 'man-made': 2, 'worldview': 1, 'realizing': 16, 'alluded': 1, 'inadvertly': 1, 'in-jokes': 3, 'languishing': 1, 'faux-bloopers': 1, 'choked': 4, "past's": 1, 'wide-release': 1, 'secrecy': 3, 'reshoots': 2, 'hard-boiled': 5, "magnolia's": 2, 'condemnation': 4, 'free-talking': 1, 'aviation': 1, 'tony': 42, 'methodical': 7, 'incidents': 7, 'horizons': 2, 'animal': 19, 'padua': 2, "harold's": 1, 'lurks': 1, 'autobiographical': 7, 'poorly-written': 1, 'nevada': 5, 'generational': 1, 'quay': 1, 'coaxed': 1, 'gusto': 2, 'spiral': 2, 'zips': 1, 'implants': 3, 'nelson': 7, 'twisting': 3, 'mutant': 4, 'errs': 1, 'dennehy': 1, 'prostitute': 10, 'capulet': 2, 'sinatra': 1, "'mulan": 1, "fear'": 1, 'walk-ons': 1, 'unstable': 6, 'womanizing': 3, 'gruesomely': 2, 'homes': 9, 'convoluted': 9, 'included--wishing': 1, 'utopian': 1, 'dentist': 2, 'helm': 6, 'bumped': 2, 'clamoring': 1, 'feeding': 2, '//leopard': 1, 'polygram': 1, 'sonorra': 1, 'sloppiness': 1, 'statue': 6, 'ill': 11, 'wardrobes': 1, 'it--people': 1, 'interviews': 17, 'extinction': 2, 'inserted': 6, "hoggett's": 1, 'dino': 3, 'artists': 15, 'loners': 2, "scholl's": 1, 'amused': 7, 'captor': 1, 'discusses': 1, "lord's": 1, "law's": 2, 'uncredited': 6, 'fail': 25, 'physicfirst': 1, 'contender': 6, 'guzman': 2, 'progressing': 1, 'differ': 3, 'leader': 49, "'grease'": 1, 'laughably': 2, 'pony': 1, 'small-screen': 1, '_ferris': 1, 'entered': 10, 'illogical': 1, 'smuggling': 5, 'moviegoer': 2, 'languid': 2, 'pseudo-world': 1, 'visual': 89, 'reacted': 1, 'u-571': 3, 'lion-hearted': 1, 'anthem': 1, 'rahul': 1, 'reparations': 1, 'ceaselessly': 2, 'privelege': 1, 'menace': 31, 'intolerance': 4, 'praises': 1, 'undertone': 1, 'patton': 5, 'one-man/woman': 1, "burnham's": 1, 'moe': 1, 'sugarry': 1, 'suspects': 34, 'predictable': 48, 'ny152': 1, 'pities': 1, 'awaiting': 12, 'immoral': 3, 'rhyitm': 1, 'miscasting': 1, 'gaga': 1, 'sings': 12, 'az': 1, 'insufficient': 1, 'vampiric': 1, 'senators': 1, 'orders': 20, "warriors'": 1, '-like': 4, 'thai': 1, 'parallel': 9, 'contradictions': 3, 'sizemore': 3, 'cartwright': 1, 'someday': 5, 'informants': 2, 'concocting': 2, 'managers': 4, 'ressurect': 1, 'klan': 1, 'middleweight': 2, 'quiz': 5, 'aesthetics': 2, 'resentful': 2, 'geek/god': 1, 'agreement': 5, 'visnjic': 3, 'heightens': 3, 'derive': 2, 'goings-on': 4, 're-meet': 1, 'loewi': 1, 'arrives': 36, 'delicate': 9, 'pleads': 6, 'berlinger': 1, 'halfway': 18, 'settles': 8, 'unshrouded': 1, 'schoolchild': 2, 'wondered': 7, 'bite': 6, 'doubts': 12, 'march': 7, 'i=92ve': 1, 'performences': 1, 'starring': 68, 'backfire': 1, 'revolution': 12, 'dillon': 11, 'subculture': 2, 'eerie': 21, "'great": 1, 'primaries': 1, 'simpson': 3, 'miller': 21, 'rid': 11, 'powerful': 91, "'leon'": 1, "pupil's": 1, 'remar': 1, 'veracity': 1, 'stand-out': 8, 'self-assuredness': 2, 'heimlich': 1, 'pyre': 1, 'crowened': 1, 'koran': 1, 'reluctant': 19, 'pugnaciousness': 1, 'ku': 1, 'computer-': 1, 'alberta': 5, 'catholics': 4, "bowden's": 1, 'home': 194, "old's": 1, 'stoic': 6, 'characteristic': 9, 'plugs': 4, 'hallucinatory': 2, 'tswsm': 1, "woman'--female": 1, 'shock-factor': 1, 'researcher': 2, 'bleachers': 1, 'earle': 1, 'disdains': 1, 'bluish': 2, 'evade': 3, 'earp': 2, 'hinwood': 1, 'retrospectives': 1, 'topped': 1, 'stricter': 1, 'emerges': 10, 'de-glamorizes': 1, '449-3411': 1, 'slob': 1, 'supposed': 67, 'superceded': 1, 'thinly': 4, 'lovemaking': 1, 'string-laden': 1, 'sinuous': 2, 'flik': 1, 'loaded': 9, 'nathanson': 1, 'altering': 2, 'marsellus': 1, 'donner': 1, 'embellished': 1, 'pesimism': 1, 'levine': 1, 'ranging': 12, 'mutated': 1, 'hanks': 22, 'clears': 3, 'stringy': 1, "elgar's": 1, 'tormented': 5, 'appearence': 1, 'flooding': 3, 'flings': 1, 'tariq': 1, 'convincingly': 14, 'mcpherson': 1, 'foiling': 1, 'unextraordinary': 1, 'all-male': 1, 'nosebleeding': 1, 'sojourn': 1, 'auteurs': 1, "camembert's": 1, 'bankable': 4, 'imports': 1, 'dismisses': 2, 'lays': 8, 'unseen': 14, 'surrandon': 1, 'uphold': 2, 'corleone': 3, 'garafalo': 1, 'firing': 5, 'felt': 81, 'ark': 6, 'reigns': 4, 'naomi': 3, 'harrowing': 14, 'amon': 1, 'r2d2': 1, 'henderson': 2, 'retiring': 3, 'hyped': 3, 're-born': 1, 'traditions': 3, "noah's": 2, 're-enters': 1, 'fluids': 3, 'entourage': 1, "water's": 1, 'schoolteacher': 2, 'backgrounds': 7, 'naidu': 1, 'lerner': 3, 'two-way': 1, 'pro': 6, 'sex-even': 1, 'prospector': 3, 'carjacking': 1, 'always-phenomenal': 1, 'chuckles': 6, 'turbulent': 2, 'werner': 3, 'statuette': 1, 'ulu': 1, 'suing': 2, 'offstage': 1, 'grande': 1, 'gleisner': 1, 'vertical': 4, 'hideous': 10, 'poor-rich': 1, 'warden': 4, 'fong': 2, 'winks': 1, 'derail': 1, 'diane': 7, 'ewoks': 1, 'assemble': 1, 'cleverly': 11, 'director-cum-thespian': 1, 'tel': 1, 'near-impossibility': 1, 'baboons': 2, 'varney': 3, 'telegraphing': 1, 'telepathic': 1, '78': 1, 'messinger': 1, 'special-effect': 2, 'beirut': 1, 'painful': 16, 'canal': 2, 'happily-ever-after': 1, 'mirror-panel': 1, 'smashed': 2, 'quality': 80, 'identification': 4, 'vancouver': 3, 'dread-factor': 1, 'recruitment': 2, 'self-important': 1, 'inspection': 1, 'increasing': 6, 'vodka': 1, "denver's": 2, "edward's": 1, 'blamed': 3, 'frontal': 1, 'checked': 1, 'well-bared': 1, 'decieving': 1, 'swindler': 1, 'hubert': 1, 'usher': 1, 'conspiratorial': 1, 'talks': 20, 'oregon': 1, 'calisthenics': 1, 'lomper': 1, 'plaza': 1, 'over-hyped': 1, 'murnau': 1, 'rhapsody': 2, 'businesses': 2, 'blanketing': 1, 'jaynes': 1, 'aftermath': 3, 'ideological': 4, "clooney's": 3, 'sutherland': 5, 'cityscapes': 1, 'unexciting': 2, 'dickerson': 1, 'daring': 11, 'rock/warner': 1, 'aisle': 3, 'loose-cannon': 1, 'cinemas': 2, 'asinine': 1, 'outstretched': 1, 'transexual': 1, 'carve': 1, 'playbook': 1, 'geologic': 1, 'tzipporah': 2, 'gara': 3, 'scrappy': 1, 'fulfilled': 4, 'oscarworthy': 1, 'wreck': 12, 'hypothesis': 2, 'paint': 13, 'unleashing': 1, 'about-face': 1, 'choppy': 5, 'eclipsed': 1, 'carriage': 2, 'psychoanalysts': 1, 'cemetery': 1, 'organisms': 2, 'rite': 3, 'titanic': 22, 'lb': 1, 'steady-date': 1, "o'conor": 1, 'napkin': 1, 'uplifted': 1, 'fire': 58, 'skyscrapers': 5, 'spouting': 5, 'ante': 3, 'wave': 13, 'shared': 15, 'atrocities': 6, 'laserman': 1, 'tranquilizers': 1, 'uniqe': 1, 'verge': 7, "o'neill": 2, 'kari': 1, 'exhilerating': 1, 'stepdaughter': 1, '`gary': 1, "kapur's": 1, 'pervious': 1, 'humiliation': 5, "reeve's": 1, 'sox': 1, 'poetry': 12, 'strategies': 1, 'beau': 8, 'lansquenet': 1, 'rearing': 1, 'screamed': 1, 'sympathize': 17, 'streetcorner': 1, "lama's": 2, 'cuss': 1, 'braveheart': 15, 'cocks': 1, 'girl-on-girl': 1, 'zinger': 1, 'widescreen': 8, "kerdelhue's": 1, 'hartnett': 3, 'intuition': 1, 're-create': 1, 'well-built': 1, 'jedi-size': 1, 'mccalister': 1, 'pitching': 1, 'inhibits': 1, 'conveyed': 7, 'thick': 11, 'instrument': 3, 'vu': 3, 'collar': 5, "lindo's": 1, 'classic': 128, 'funds': 4, 'pacifier': 1, 'seniors': 2, 'valmont': 1, 'schlondorff': 1, 'wafer-thin': 1, 'ajay': 1, 'unforgiven': 2, 'montage-backed': 1, 'exit': 6, "gattaca's": 1, 'ouch': 1, 'problem--clone': 1, 'teach': 13, 'stong-willed': 1, 'photographic': 3, 'rehab': 2, 'orgazmo': 1, 'whetted': 1, 'wyman': 1, 'overplotting': 1, 'bozzio': 1, 'drunk': 13, 'stressing': 1, 'mindlessly': 1, 'sniping': 1, 'abolitionist': 1, 'fates': 3, 'resulted': 3, 'vigor': 3, 'county': 4, 'learned': 30, 'owner': 34, "freeman's": 4, 'rylance': 1, 'coworkers': 2, 'contrary': 15, 'suspenders': 2, 'ignorance': 5, 'bombard': 1, 'incarnations': 2, 'framed': 15, 'uproar': 3, '67': 1, 'tarzan': 9, 'snubs': 1, 'tantalizing': 1, "mckean's": 1, 'zemecki': 1, 'egan': 3, 'life--after': 1, 'undisputed': 1, 'operative': 6, 'panthers': 1, "1997's": 9, 'eligible': 3, 'prided': 1, 'impending': 9, 'nova': 2, 'brigante': 2, 'bluster': 1, "hospital's": 1, 'trolls': 1, 'two-hander': 1, "idol's": 2, 'valkyries': 1, 'survived': 11, 'high-energy': 3, 'dissatisfaction': 4, 'woman/': 1, 'strongman': 1, 'government': 52, 'character': 469, 'fop': 1, 'jacob': 5, 'actualization': 1, 'videotaping': 1, 'coven': 1, 'giosu=e9': 1, 'allowances': 1, 'winnings': 1, 'bored': 27, 'heretical': 2, 'balances': 3, 'unemployment': 2, 'guterman': 1, 'reds': 3, 'boone': 2, 'ving': 8, 'elaborately': 1, 'sportscasters': 1, 'mansfield': 1, 'imperfect': 2, 'contracted': 2, 'dajani': 1, 'implication': 3, 'exudes': 6, 'cairo': 1, 'countless': 18, 'colossal': 2, 'self-chiding': 1, 'hindu': 2, 'annaud': 1, 'ultramaterialistic': 1, 'stendahl': 1, '1961': 1, 'detestable': 2, 'compartments': 1, 'engagingly': 1, 'lottery': 5, 'levels-as': 1, 'shebang': 1, 'mini-me': 2, 'bulow': 1, 'jeopardise': 1, 'agencies': 1, 'crs': 1, 'crunching': 1, 'unquestioning': 1, 'nihilism': 1, 'been-there': 1, 'carnality': 1, 'omegas': 1, 'prinze': 2, 'celibacy': 4, 'lobsters': 1, "hawks'": 2, 'wondrously': 3, 'ha': 4, 'adulthood': 5, 'yar': 1, 'divined': 1, 'paradise': 15, 'dymanite': 1, 'ramboesque': 1, 'legitimacy': 2, '______': 1, 'slamming': 2, 'anachronisms': 1, 'laborious': 1, "grosbard's": 1, 'umbrellas': 1, 'villains': 16, 'footing': 6, 'deceiving': 2, "eisner's": 1, 'frequency': 5, 'cleese': 3, 'tiger': 5, "`8mm'": 1, 'amazing': 79, 'pagans': 1, 'sugiyana': 1, 'melted': 1, 'conceptions': 1, 'typical': 64, 'traverse': 1, 'loomis': 2, 'businessman': 14, 'madeliene': 1, 'blurts': 1, 'emo': 2, 'carrying': 18, 'alarm': 4, 'acquainted': 1, 'irrepressible': 2, 'pick-up': 2, 'breed': 6, 'surprised': 51, 'jive-talking': 1, 'hog': 1, "dean's": 3, 'override': 1, 'sparticus': 1, 'cherie': 1, 'daytrippers': 1, 'sunny': 7, 'abolish': 2, 'pheiffer': 1, 'chuckle': 3, '1932': 4, 'slurs': 2, 'undesirable': 1, 'ever-growing': 3, 'impersonations': 1, 'terrific-': 1, 'filter': 3, 'lead': 107, 'downright': 16, 'psychedelia': 1, 'henning': 1, 'forecaster': 1, 'shown': 54, 'eamonn': 2, 'gomorrah': 1, 'anxieties': 1, 'pa': 1, 'recieves': 2, 'seawall': 1, 'flashy-new-thing': 1, "film'": 1, 'comedic': 28, 'weirdo': 2, 'tripe': 2, 'cheadle': 7, 'delusional': 2, 'naivete': 1, 'goofups': 1, 'hitchcockian': 4, 'blanket': 4, 'enwrapped': 1, 'soared': 3, 'portals': 1, 'plotlines': 3, 'vendor': 2, 'torch': 3, 'juliet': 11, 'katarina': 1, 'lords': 4, 'columbus': 2, 'progressively': 3, 'cigars': 1, 'fun': 200, 'fitts': 2, 'embroidered': 1, "douglas's": 2, 'jokingly': 2, 'ecstatic': 4, 'deceptive': 2, 'rec': 2, 'smirks': 1, 'shading': 2, 'understandingly': 1, "collector's": 5, 'balsan': 1, 'wo-ping': 1, '_melvin': 1, 'weighted': 1, 'javert': 4, 'buttercup': 1, 'requiem': 2, 'winde': 1, 'translate': 5, 'hyper-colorized': 1, '18th': 4, 'refreshing--and': 1, 'hercules': 7, 'straining': 1, 'parcel': 1, 'interrotron': 1, 'shell-shocked': 1, 'popularizer': 1, 'glebas': 1, 'justify': 6, 'april': 7, 'permeated': 4, 'communist': 6, 're-releases': 2, 'justin': 2, 'alienate': 4, 'joplin': 1, 'pitch': 8, 'dion-esque': 1, '1908': 1, 'beetle': 2, 'diatribes': 1, 'honey-soaked': 1, 'terse': 1, 'flexibly': 1, 'boot': 11, 'cubicle-filled': 1, '8%': 1, "animators'": 1, 'reenactments': 1, 'eisenberg': 1, 'steamy': 2, 'squemish': 1, 'barstool': 1, 'linear': 5, 'darwinism-': 1, 'high': 162, 'gawkers': 1, 'sick': 25, 'technical': 46, 'underestimates': 1, 'hiv-positive': 1, 'plath': 1, 'jenkins': 3, 'undie': 1, 'wide-open': 2, "raimi's": 1, 'iv': 2, 're-introduce': 1, 'baffle': 1, 'headphones': 3, 'stodge': 1, 'knutt': 1, "stendhal's": 1, 'flows': 6, 'mexican': 2, 'glenn': 20, 'misgivings': 1, 'unwitting': 2, 'noel': 1, '`batman': 1, 'chunks': 1, 'suspense/science': 1, 'organizers': 1, "stiller's": 4, 'insinuated': 1, 'chapin': 1, 'kills': 25, '14-year-old': 4, 'guiding': 2, "keaton's": 2, 'tape-worshipping': 1, 'chugging': 1, 'schematics': 1, 'helluva': 2, 'grumpy': 6, 'exhibit': 4, 'cherish': 1, 'exert': 1, 'muddled': 4, 'glickman': 1, 'lingering': 6, 'wcw': 1, 'tutus': 1, 'control': 55, 'philandering': 4, 'ex-preacher': 1, 'sleeze': 1, '310': 1, 'topic': 8, 'condescending': 6, 'pindar': 1, 'squeal': 6, 'well-crafted': 7, 'priceless': 10, 'lip': 6, 'mario': 4, 'brash': 8, 'yaakov': 1, 'broadcasts': 3, 'hospitalized': 2, 'exploited': 4, 'penuries': 1, 'gimmicky': 4, 'jean-luc': 3, 'exact': 23, 'brushes': 1, "'cause": 2, 'servants': 5, "bostwick's": 1, 'vaguely': 9, 'disjointed': 7, 'abusing': 1, 'apprehension': 2, 'creative': 35, 'thailand': 2, 'jeopardy': 2, 'doted': 2, 'borderline-psychotic': 1, 'billington': 3, 'reality-based': 2, '8th': 1, 'blue': 45, 'rooms': 17, 'nerd': 5, 'obligatory': 20, 'filmcritic': 1, "disco's": 1, 'flitting': 2, 'tough': 45, 'dominican': 1, 'slugs': 1, 'cigarrettes': 1, 'nasty': 31, 'one-man': 2, 'incisive': 2, 'sendoff': 1, 'alienates': 1, 'scale': 21, 'three-act': 1, 'in-valids': 1, 'products': 3, 'allegations': 2, 'trier': 1, 'cyclops': 1, 'addy': 3, 'gillian': 6, 'cohorts': 3, 'de-mystify': 1, 'across--the': 1, 'overpraising': 1, 'panther': 2, 'earthquakes': 1, 'uplifting': 20, 'protagonists': 18, 'motor-mouthed': 1, 'garber': 1, 'liars': 1, 'mushu': 6, 'cleon': 1, 'jedi': 17, 'tome': 1, 'strangers': 11, 'woodstock': 1, "'or": 1, 'vampirism': 1, 'ryder': 8, 'apologetically': 1, 'decoding': 1, 'rampling': 2, 'tobey': 9, 'cubans': 1, 'afore-mentioned': 2, 'altar-boy': 1, 'disappointed': 32, 'fuss': 4, 'caraciture': 1, 'inevitably': 14, 'recurrent': 1, 'suspicions': 3, 'garlic': 3, '25th': 1, 'aired': 2, 'disrespectful': 2, 'burgles': 1, 'playwright': 6, 'terrace': 2, "'90s": 9, 'matewan': 1, 'jolie': 2, 'subsidiaries': 1, 'velociraptor': 1, 'jelly-donut': 1, 'wings': 16, "doubt's": 1, 'goof-ups': 1, 'obtains': 2, 'flourishing': 3, 'travels': 16, 'odalisque': 1, "there'll": 1, 'sandstorm': 1, 'thrills': 22, 'x-files': 13, 'pointless': 15, 'doo': 2, 'montages': 2, '_two_': 1, 'subordinates': 3, 'scrutinized': 1, 'infiltrate': 2, 'protestors': 1, 'mourn': 2, 'fairman': 1, 'emerged': 3, 'short': 115, 'high-definition': 1, 'blueprint': 2, 'diner': 12, 'cavalry': 2, 'countryman': 2, 'wipers': 1, 'jo': 8, 'brain': 24, 'isolationism': 1, 'alexander': 13, 'cathy': 1, 'magnified': 2, 'spatulas': 1, 'poseidon': 3, 'humanoid': 2, 'cheat': 6, "driver'": 1, 'rampant': 7, 're-election': 1, '15-year-old': 1, "schmoe's": 1, 'potboiler': 1, 'non-professional': 1, 'ming': 1, "benigni's": 1, "anton's": 1, '$100': 5, 'fictious': 1, 'prophecies': 1, 'crotch': 3, 'recollection': 2, 'spectators': 3, 'unfortunates': 2, 'dance/song': 1, 'illegitimate': 2, 'insignificant': 6, 'conformity': 4, 'interpreted': 3, "thailand's": 1, 'abode': 2, 'principals': 6, 'wading': 2, 'tugs': 2, 'incorporated': 1, 'darrow': 1, 'subordinate': 1, 'moms': 2, 'periodic': 2, 'vaughan': 1, 'tag': 9, 'brasco': 3, 'break-dance': 1, 'sub-par': 2, 'male': 43, 'standard-issue': 1, 'bicker': 1, "steven's": 2, 'college-dormitory': 1, 'bargained': 1, 'maddeningly': 3, 'birds': 9, 'scoop': 1, 'bigots': 1, 'arrangement': 3, 'clocks': 6, 'icecream': 1, 'groupies': 1, 'napoleon': 1, 'bellies': 1, 'tunney': 1, 'occassionally': 1, 'lies': 61, 'breasted': 1, 'delicately-narrated': 1, 'tear-jerker': 1, 'hectic': 4, 'brick': 2, "renfro's": 1, 'good-guy': 1, 'specimens': 1, 'serene': 2, 'sparky': 1, 'adolescent-minded': 1, 'cliff': 7, 'mercedes': 1, 'marines': 1, 'wormwood': 1, 'faber': 1, 'up-to-date': 1, 'neo-nazi-dom': 1, 'rourke': 2, 'dizzy': 5, '======================': 1, 'meditating': 1, 'dubbed': 10, 'dorff': 4, 'jewels': 2, 'unconsummated': 1, 'fleshed-out': 1, 'skids': 1, 'pen': 2, 'tenacious': 1, 'kong/hollywood': 1, 'lam': 3, 'suited': 9, 'viscous': 1, 'gold': 28, 'prevail': 2, 'brawn': 1, "sellars'": 1, 'gus': 8, 'broom': 1, 'thought-provoking': 19, 'slackness': 1, '2013': 2, 'eggar': 1, '____________________________________________': 1, 'innocently': 2, 'coz': 1, 'vitality': 3, 'swingers': 7, 'arrested': 11, 'defending': 7, "nick's": 2, 'merge': 1, 'complimentary': 1, 'risks': 9, 'movie-': 2, 'contributing': 2, 'costing': 3, 'rummaging': 1, 'containers': 1, 'huey': 1, 'elliott': 6, 'objections': 4, 'cu-sack': 1, 'llama': 1, 'glued': 2, 'vassili': 1, 'aquits': 1, 'impeccable': 4, 'katsulas': 1, "bale's": 1, 'session': 9, "diego's": 1, 'dictator': 1, "1984's": 1, 'ex-patient': 2, 'popular-but-slow': 1, 'first-string': 1, "wang's": 3, 'sarcastic': 6, 'nbk': 1, 'rhythmic': 2, 'expand': 5, "crowe's": 1, 'daresay': 1, 'weekends': 1, 'investigators': 4, 'rich': 88, 'kaplan': 1, 'neo-noir': 2, 'stings': 2, 'edginess': 3, 'slipped': 4, 'indubitably': 1, 'cajun': 1, 'jumped': 2, 'attract': 7, 'stumble': 6, 'mutually': 1, 'rider': 2, 'life-altering': 1, 'policeman': 4, 'overly-dark': 1, 'calder': 1, 'pasts': 1, 'yitzhak': 1, 'directing': 53, 'cinematic': 56, 'tame': 4, 'rocket': 5, 'fantasy--one': 1, 'alleys': 3, 'ground-breaking': 3, 'ugly-duckling': 1, 'nobuko': 2, 'father/david': 1, 'tons': 11, 'embarrasses': 1, 'debut': 45, 'braces': 1, 'effect-dependent': 1, 'bigglesworth': 1, 'specialist': 4, 'baylor': 2, "lawyer's": 2, 'adopting': 2, 'smolder': 1, "1979's": 1, 'keeve': 1, 'muddy': 2, 'ritual': 3, 'longstanding': 1, 'card': 12, 'connects': 2, 'co-worker': 14, "ming's": 1, 'covetous': 1, 'lapd': 2, 'thud': 1, 'roofs': 1, 'knife': 11, 'gift-bearer': 1, 'burgeoning': 2, 'rightly': 2, 'schweethaat': 1, 'missive': 1, 'headcount': 1, 'long-winded': 2, 'jedda': 1, 'immature': 4, 'pyle': 1, 'registered': 2, 'quenches': 1, 'one-scene': 1, 'pcu': 1, 'runway': 1, 'drug-addiction': 1, 'serialised': 1, 'noisy': 3, 'ticoton': 1, 'gray': 4, 'zachary': 2, 'allah': 2, 'apply': 5, 'realisticly': 1, 'divide': 2, 'resurfacing': 1, 'di$ney': 1, '170': 1, 'illustrating': 7, 'burkhard': 1, 'katanga': 2, 'close': 95, 'itself--box': 1, 'bitch': 9, 'meryl': 9, 'meudon': 1, 'kenneth': 13, 'plunkett': 1, "greg's": 2, 'setpieces': 1, '17-year': 1, 'hardback': 1, 'thrill-ride': 3, 'zundel': 1, 'eightly': 1, 'behold': 17, 'handful': 24, 'saddam': 3, 'out-right': 1, 'boyhood--and': 1, 'top-down': 1, 'capsule': 13, 'heirloom': 1, 'dried-up': 2, 'juliette': 6, 'voters': 1, 'liz': 1, 'networks': 2, 'raj': 1, 'sap': 2, 'tricked': 1, 'temporal': 1, 'co-write': 2, 'transylvanian': 1, 'troyer': 2, "moroder's": 1, 'yeomen': 1, 'bad-boy': 1, 'graceful': 3, 'connives': 1, 'ably': 7, 'list': 54, 'nolte': 5, 'impacts': 1, 'not-so-secretly': 1, 'congregations': 1, 'supremacy': 2, "todd's": 2, 'norm': 7, 'conversation-heavy': 1, 'aggressively': 2, 'grinning': 3, "ed's": 1, 'reproach': 2, "busey's": 1, 'irresistible': 6, 'primal': 9, 'suns': 2, 'blistering': 1, 'ways': 84, 'rooted': 4, 'ex-wife': 4, 'hunger': 5, 'invaluable': 1, 'x-rating': 1, 'eyeful': 1, 'gallantry': 1, 'lemonade': 1, 'valuable': 9, 'jackal': 2, 'bystander': 3, 'cringing': 1, 'deficiencies': 1, 'farquaads': 1, 'irreverent': 1, 'stubborn': 6, 'skilled': 7, 'reclaims--her': 1, 'propagating': 1, 'write': 27, 'louise': 2, 'referring': 8, 'awkwardly': 3, '_election_': 1, 'drawbacks': 1, 'suitably': 11, 'uncovering': 2, 'afterschool': 1, 'stumbling': 7, 'swimmer': 2, 'heterosexuals': 1, "shaft's": 1, 'catalyzes': 1, 'orson': 2, 'choke': 1, "recipient's": 1, 'victories': 5, 'subversive': 3, 'statement': 22, 'garfield': 2, 'fiel': 1, 'maine': 3, 'self-taught': 1, 'realization': 13, 'philisophical': 1, 'episodes': 19, 'keitel': 7, 'pounder': 1, 'pride': 18, 'jovovich': 2, 'decreases': 1, "williamson's": 3, 'buffet': 1, 'declines': 2, 'succumbs': 4, 'ludicrous': 3, 'gag': 13, 'bandwagon': 1, 'palliative': 1, 'icelandic': 1, 'preach': 3, 'pirahna': 1, 'voluntary': 3, 'trek': 25, 'comers': 1, 'stomach-turning': 1, 'worry': 17, 'long-awaited': 2, 'manor': 1, 'falsely': 4, 'presumes': 1, 'populated': 4, 'mighty': 16, 'shell': 6, 'swashbuckling': 4, 'bankers': 1, 'disabled': 3, 'naturalistic': 3, 'oddballs': 4, 'symbolises': 1, 'meandering': 4, 'playfulness': 2, 'abyss': 10, 'geordi': 1, 'carter--whose': 1, 'ww': 1, 'reveling': 1, 'herd': 1, 'racist': 13, 'gauge': 10, "mathilda's": 1, 'strengthen': 1, 'fun-spoiling': 1, 'soap': 14, 'moscow': 2, 'deliverance': 2, 'emerge': 9, 'unexpected': 34, 'pfeiffer': 6, 'calculated': 5, 'neck': 13, 'goons': 5, 'ulterior': 3, 'sparked': 6, 'stationed': 2, 'more-than-welcome': 1, 'weakest': 12, 'terrorist': 13, 'telegraphed': 1, 'llosa': 1, 'wisconsin': 1, 'astounded': 1, 'amounted': 1, 'broadens': 1, 'lap': 2, "d'etat": 1, "series'": 3, "estella's": 2, "sherlock's": 1, 'anti-military': 1, 'whitman': 1, 'edgecomb': 2, 'utilised': 1, 'cynical': 19, 'priivate': 1, 'prep': 3, "sam's": 3, 'moss': 9, 'siegel': 3, 'chimps': 1, 'rydell': 2, 'elgar': 1, 'tacked-on': 1, 'neve': 15, 'money-grubbing': 1, 'southampton': 1, 'static': 4, 'stale': 9, 'fool': 19, 'abba': 1, "anybody's": 2, 'bikini': 3, 'shaky': 5, 'imprint': 1, 'remark': 1, 'narcotics': 4, 'gritty--make': 1, 'disrespecting': 1, 'jensen': 2, 'bitterly': 2, 'one-upping': 2, 'parliamentary': 1, 'adulterous': 3, "arnie's": 1, 'fraternizing': 1, 'limited': 46, 'objectively': 1, 'young-lovers-on-the-road': 1, "mugatu's": 1, '144': 1, 'clips': 7, 'multi-million': 2, 'ambrose': 1, 'stars': 144, 'reprimands': 1, 'blondie': 1, 'south-winds': 1, 'idolize': 1, 'facing': 12, 'hank': 9, 'girds': 1, "vampire's": 1, 'predetermine': 1, 'anti-vampire': 1, 'kowtowing': 1, 'mainstays': 1, 'clues': 17, 'critique': 26, 'shelved': 4, "maid's": 1, 'trendiness': 1, 'infer': 1, 'aggravate': 1, 'dabbles': 1, 'morgue': 3, 'eh': 3, 'gal': 5, 'capably': 1, 'lessen': 2, '1960': 4, 'provide': 60, 'rays': 2, 'talmudic': 1, 'authorship': 1, 'messenger': 5, 'marvel': 10, "stevenson's": 1, 'exterminating': 1, 'lyrics': 8, 'national': 25, "mike's": 6, 'bummed': 1, 'punishment': 11, 'dame': 8, 'jarmusch': 2, 'perpetual-motion': 1, 'grouchy': 2, 'twenty': 39, 'prick': 1, 'suburb': 1, 'punctuating': 1, 'positioning': 1, 'obligated': 2, 'professed': 1, 'four-some': 1, 'blathers': 1, 'rustic': 3, 'annoyances': 1, 'unlistenable': 1, 'clothing': 14, 'resumes': 3, 'piscapo': 1, 'stalking': 6, 'rasp': 1, 'morton': 2, 'hugely': 8, '_great_': 1, 'crickey': 1, 'silva': 1, 'genre-crossing': 1, 'villages': 2, 'excel': 3, 'investigates': 2, 'grownups': 1, "giacomo's": 1, 'reserve': 3, 'twenty-eight': 1, 'ringo': 3, 'pre-production': 2, 'cream-filled': 1, 'rod': 7, 'ads': 9, 'mulder': 6, 'potenza': 1, 'plays': 266, 'feelgood': 1, 'garr': 1, 'psych': 2, 'imagined': 8, 'abundance': 8, 'graduation': 5, 'reliance': 3, 'esquire': 1, 'shoots': 18, 'godley': 1, 'acting-related': 1, 'arthurian': 1, 'stacked': 1, 'gayness': 1, 'g-men': 2, 'toast': 4, 'haley': 9, 'improvise': 3, 'wrongfully': 1, 'persists': 1, 'actors/actresses': 1, 'ming-': 1, 'hurting': 2, 'b-action': 1, 'relics': 1, 'isolate': 1, 'counterfeit': 2, 'occupies': 3, 'bury': 3, 'socialite': 3, 'well-populated': 1, 'loaned': 1, 'phase': 5, 'mantel': 1, 'propelling': 1, 'deprivation': 1, 'easton': 1, 'link': 9, 'zombies': 2, 'condescension': 1, 'festivals': 1, 'thirty': 18, 'reprehensible': 1, 'permanent': 3, 'wierd': 1, 'broach': 2, "colqhoun's": 1, 'heavy': 41, 'incongruity': 1, 'one-on-one': 2, 'encoding': 1, 'tennessee': 2, 'goran': 2, 'kanoby': 1, 'shaking': 4, 'jar': 7, 'brashness': 1, 'observes': 10, 'mucus': 2, 'quell': 2, 'father': 163, 'stabs': 2, 'courtesy': 12, 'manmade': 1, 'blowhard': 1, 'hernandez': 2, 'impotent': 1, 'shock': 28, 'child-support': 1, 'kirkland': 1, 'quilty': 1, '--travel': 1, 'feared': 5, 'richer': 6, 'indication': 6, 'toontown': 1, 'us$130': 1, 'consulted': 1, 'newest': 14, 'commentaries': 2, 'wilds': 1, 'hsaio': 1, 'isolationist': 1, 'justly': 2, "jokes'": 1, "friend's": 8, "cop's": 3, "negin's": 1, 'eccleston': 4, "hitler's": 2, 'skirmish': 1, 'uniform': 9, 'overly': 27, 'rely': 18, 'libra': 1, 'near-fully': 1, 'upholstered': 1, 'noggin': 1, '701': 1, 'voyeurism': 2, 'equaling': 2, 'lion': 6, 'nobleman': 2, 'three-plus': 1, "kurtz'": 1, 'time-to-time': 1, 'butthead': 5, 'sequences': 94, 'defect': 3, 'letterman': 1, 'crossing': 12, 'plants': 1, "streep's": 2, 'tahiti': 1, 'condition': 19, 'hallie': 3, 'inexcusable': 2, 'unprecedented': 2, 'praising': 2, 'nov': 1, 'conspires': 2, 'stumped': 2, 'britain': 6, 'alike': 14, 'diggs': 3, 'million-plus': 2, 'moderately': 3, 'real-': 1, 'hyper-kinetic': 1, 'credentials': 4, 'copies': 2, 'gruesome': 12, 'gabriel': 7, 'ideologies': 3, 'skeleton': 1, 'steven': 38, 'camouflaged': 1, 'rogue-ish': 1, 'craven': 11, 'diverge': 1, 'burst': 15, 'entries': 4, 'contaminated': 2, 'allen': 37, 'ifans': 1, 'boils': 8, 'witches': 5, 'justifying': 1, 'shifted': 3, 'worn-out': 1, 'integrated': 2, 'bends': 1, 'hold-your-breath': 1, 'roam': 3, 'likeablity': 1, 'dale': 6, 'rep': 3, "epp's": 1, 'trophy': 5, 'fizzled': 1, 'tribe': 5, 'drive': 30, 'sexism': 1, 'prance': 1, 'sergeant': 4, 'stemple': 1, "nicolet's": 1, "'92/early": 1, 'bedrooms': 1, 'kissing': 4, 'telekinetically': 1, 'responsive': 1, 'jump': 25, 'edd': 1, 'kane': 8, 'clooney': 12, 'mythic': 2, 'sheltering': 1, 'quirk': 1, 'consumes': 1, 'cunningham': 1, 'separating': 3, 'sight': 44, "shan-yu's": 1, 'norway': 2, 'transmitting': 1, 'outperform': 1, 'orgy': 7, 'wendy': 2, 'hawke': 7, 'tackled': 3, 'mortality': 5, 'super-judgmental': 1, 'sally': 8, 'grills': 1, 'sappy': 14, 'shocking': 33, 'round-up': 1, 'yuppy': 1, "lola's": 1, 'residential': 2, 'three-hour': 3, 'american-release': 1, 'legacy': 4, 'palace': 6, 'robbins': 4, 'cerebrality': 1, "'matrix'": 1, 'sleazy': 18, 'germphobe': 1, 'chicago-sun': 1, 'seventeen': 4, 'ben-hur': 1, 'poured': 2, 'brenneman': 3, 'bewitches': 1, 'c3p0': 1, 'crush': 12, 'wimp': 2, 'brush-up': 1, 'dequina': 1, 'insulting': 2, 'ex-ira': 1, 'pivotal': 20, 'consideration': 8, 'jethro': 1, 'gel': 3, 'letscher': 1, '30': 26, 'replenishing': 1, 'glorify': 3, 'account': 16, 'robot': 12, 'voizin': 1, 'harasses': 2, 'crusade': 3, 'kyle': 5, 'mcflurry': 1, 'writer-performers': 1, 'abu': 1, 'boys': 40, 'narrow': 6, 'hades': 2, 'veronique': 1, '`come': 1, 'definite': 18, "patric's": 1, 'spicebus': 1, 'strongest': 16, 'bracing': 1, 'vice': 8, 'leary': 6, 'peopled': 1, 'dilbert-esque': 1, 'clockwork': 5, 'patriarchy': 1, 'old-money': 1, "annie's": 4, 'fundamentalists': 2, 'andy': 18, 'fable-like': 1, 'face-hugger': 1, 'scenario': 16, 'glaring': 8, 'conservatives': 1, 'instantaneously': 3, 'tapping': 1, '[the': 2, 'taxation': 2, 'oval': 1, 'spider-scorpion-crab': 1, 'slums': 3, "singin'": 1, 'bowen': 3, 'tearfully': 1, 'canny': 1, 'insecurity': 3, 'keenan': 1, 'calibre': 4, 'yum': 1, 'clashed': 1, 'vocalized': 1, 'madly': 6, 'infatuation': 1, 'walked': 8, 'golds': 1, 'lowbrow': 2, 'showcases': 4, 'figurines': 1, 'stentorian': 1, 'omega': 1, 'masochistic': 1, 'alternates': 2, 'agape': 1, 'reality-warping': 1, 'reminding': 4, "sheen's": 4, 'carcasses': 1, 'representative': 4, 'bowling': 8, 'mix-and-match': 1, 'well-used': 2, "miller's": 6, 'regrettably': 1, 'tunes--but': 1, 'brosh': 1, 'stab': 8, 'deux': 2, 'myrlie': 1, 'radiance': 1, 'redemption': 14, 'peak': 12, 'lilith': 1, 'capes': 1, 'winter': 11, 'dingy': 1, 'rhymes': 3, 'sorta': 5, 'alliances': 1, 'polo/ralph': 1, 'trust': 31, '18': 10, 'edmund': 2, 'eagle': 1, 'coats': 5, 'quickly--yes': 1, 're-creates': 1, 'drips': 3, 'evocative': 4, 'folly': 1, 'sitting-ducks': 1, "'what": 2, 'nonchalantly': 1, 'deserving': 8, 'uniqueness': 4, "camille's": 1, 'often-incongruous': 1, 'encore': 1, 'nonjudgemental': 1, 'sensation': 7, 'site': 14, 'spin-off': 1, "skeptic's": 1, 'solving': 5, 'colanised': 1, 'nihgt': 1, 'emulate': 3, 'curve': 1, 'jr': 26, 'accusation': 3, 'dominated': 4, 'lusting': 3, 'suicidal': 13, 'half-': 1, 'tolerable': 3, 'gilbert': 5, 'guinevere': 3, 'tennis': 1, 'forwards': 1, "juni's": 1, "han's": 1, 'tormentor': 1, 'dussander': 1, 'moral-christian': 1, 'maurice-jones': 1, 'eating': 15, 'janeane': 7, 'strain': 6, 'lobby': 4, 'slime-splattering': 1, 'frog': 1, '165': 1, 'morosely': 1, 'absurdities': 2, 'limbs': 5, 'pledge': 2, 'slender': 1, 'underdeveloped': 8, 'goulash': 1, 'hurtling': 1, 'killers-with-hearts': 1, "lam's": 2, 'enrich': 1, 'unchanged': 3, 'fleet': 3, 'tonal': 1, 'judgeship': 1, 'tormenting': 2, 'began': 31, 'watershed': 1, 'lovell': 1, 'happening': 30, 'repulsion': 3, 'operators': 1, 'mcclain': 1, "musn't": 1, 'stoolie': 1, 'facts': 21, "kaufman's": 1, 'las': 14, 'overstatement': 1, 'jay': 21, 'non-cgi': 2, 'uta': 1, 'un-sanitary': 1, 'oppose': 2, 'winslet-dicaprio': 1, '96-minute': 1, 'foreknowledge': 1, 'arizona': 8, 'overpowered': 2, 'quitting': 2, 'convinced': 27, 'groovy': 2, 'beautiful_': 2, 'upwards': 1, 'approach': 58, 'hands': 60, 'anemic': 1, 'ted': 17, 'cutting-edge': 1, 'fixes': 2, 'salonga': 4, 'jobeth': 1, 'eyepopping': 1, 'apocalypse': 7, "brooks'": 2, 'hallucinogen-fueled': 1, 'fission': 2, 'formidably': 1, "pair's": 1, 'locates': 1, 'witt': 1, 'unbeknown': 1, 'stuck-up': 1, 'handles': 19, 'itami': 2, 'bear': 19, 'campfest': 1, 'orphans': 1, '92': 1, 'gory': 20, 'gnawing': 1, 'horror-drama': 2, 'probes': 1, 'lynchian': 1, 'hot-air': 1, 'ames': 2, 'ernie': 3, 'mere': 26, 'philanderer': 1, 'discoverer': 1, 'assumed': 2, 'ex-pug': 1, 'escalating': 4, 'changwei': 1, 'leasure': 2, "'did": 1, "womens'": 2, 'minghella': 1, 'brogue': 3, 'expressive': 5, 'superbly-realized': 2, 'hacks': 1, 'tautly': 2, 'kaye': 6, 'stopover': 1, 'corky': 1, 'mid-1500s': 1, 'testicle': 1, 'elevates': 1, 'borgnine': 2, 'poems': 1, 'tandem': 1, "'la": 1, 'artisan': 2, 'estates': 1, 'user-friendly': 2, 'fiddle': 1, 'orgies': 3, 'scatting': 1, 'stamper': 3, 'reasoning': 5, 'aristo': 1, "hallstrom's": 1, 'depravity': 5, 'ottman': 1, "picasso's": 1, 'agnes': 1, 'brandishing': 2, 'seething': 1, 'waystation': 1, 'spouse': 4, "fingal's": 1, "connor's": 1, 'sketchbook': 1, 'paradine': 1, 'unkrich': 1, 'nykvist': 1, 'santiago': 1, 'pam': 11, 'wipe': 4, 'baily': 1, 'glamour': 4, 'rife': 3, '`nightmare': 1, 'stole': 9, 'non-questioning': 1, 'hard-knock': 1, 'paneling': 1, 'scarred': 2, 'surreptitious': 1, 'youngster': 2, 'disappoint': 6, 're-hab': 1, 'super-bookstores': 1, 'missy': 1, 'weaknesses': 11, 'culprit': 2, 'capano': 1, 'mie': 1, 'middlemen': 1, 'line-': 5, 'accosted': 1, 'breathless': 3, 'prescribed': 2, 'vat': 1, 'arnie': 3, 'noiresque': 1, 'bombay': 1, 'pudgy': 1, 'yells': 4, 'companionability': 1, "fuckin'": 1, 'catatonia': 1, 'recruiting': 2, 'danced': 1, 'patric': 2, 'sensory': 3, '10-year-old': 2, 'mindless': 9, 'benefited': 4, 'forgiveness': 5, 'ribbing': 1, 'manageable': 1, 'slaying': 2, 'dabbling': 1, 'publish': 1, 'impact': 50, 'nan': 1, '300+': 1, 'wed': 5, 'kevlar': 1, 'enjoyable': 80, 'columnist': 1, 'tabloid-ish': 1, 'activities': 12, 'notoriety': 7, 'un-dicaprio-esque': 1, '16-17': 1, 'prostration': 1, 'self-consciousness': 3, 'memory': 33, "'realism'": 1, 'residents': 11, "summer'": 1, 'hummm': 2, 'reinvent': 2, 'synergism': 1, 'believer': 4, 'sociopathic': 2, 'inserts': 3, "fanny's": 1, 'ellie': 4, 'retaliation': 3, 'father/son': 1, 'weepy': 3, 'treasure-trove': 1, 'enjoying': 13, 'stork': 1, 'balrogs': 1, 'doe': 3, 'discovers': 53, 'paranoia': 12, "doom's": 1, 'chock-full': 1, 'elizabethian': 1, 'walter': 15, 'compatriots': 1, 'cpr': 1, "depalma's": 1, 'guns': 28, 'exhausting': 6, 'well-constructed': 2, 'attained': 4, 'storms': 3, 'armor': 11, 'reliving': 1, 'hone': 2, 'declaration': 2, 'mapped': 1, 'inventors': 1, 'live-action-disney': 1, 'scarier': 7, 'muttering': 2, 'infidelitous': 1, 'physicals': 1, 'discomfort': 7, 'everlasting': 1, 'minimum': 8, 'blaming': 1, 'invite': 1, 'renting': 2, 'hynes': 1, 'inexplicably': 4, 'cleared': 2, 'dwight': 1, 'rag-tag': 3, 'accommodations': 1, 'side-splittingly': 1, 'unawareness': 1, 'strings': 11, 'dismaying': 1, 'showy': 6, 'motherfucker': 1, 'scream-esque': 1, 'overtakes': 2, 'circumcision': 1, 'minding': 1, 'casts': 12, 'calculating': 3, 'unjust': 2, 'mediciney': 1, 'unsympathetic': 5, 'guessed': 5, 'asset': 3, "john's": 7, "ramis's": 1, 'howard=92s': 1, 'ever-present': 6, 'velocity': 2, 'dykstra': 2, 'eduard': 2, "coyle's": 1, 'concurrent': 1, 'unpredictably': 1, 'bolt': 2, 'outmatches': 1, 'messy': 4, 'stupid': 34, 'degenerate': 1, 'twenty-one': 1, 'synthesize': 1, 'gothic': 16, 'inopportune': 1, 'sermonizing': 1, 'blacks': 7, 'shadings': 1, 'livestock': 1, 'mon': 1, 'commits': 9, 'korean': 3, 'cult-favorite': 1, 'no-dialog': 1, 'sexualized': 1, 'forums': 1, 'cat': 17, 'benji-like': 1, 'sentenced': 5, "'isolated'": 1, 'en': 10, 'over-weight': 1, 'groom-wannabe': 1, 'eyewitness': 1, 'gaz': 3, 'slavehood': 1, 'laptop': 1, 're-imagining': 1, 'juxtaposed': 2, 'daniel': 18, 'evidencing': 1, "prisoner's": 3, 'imaginatively': 2, 'maniac': 2, 'screw-ups': 1, 'criminals': 9, 'woefully': 2, 'loan': 2, "'fright": 1, 'bureaucrats': 2, 'calif': 1, 'binks': 8, 'guy': 173, 'pow': 2, 'tarsem': 1, "kennedy's": 1, 'sylvester': 8, "death's": 1, 'hennessey': 1, 'redefinition': 1, 'inoffensive': 2, "montague's": 2, 'bricks': 1, 'pulpit': 1, 'soft-focus': 1, 'consists': 20, 'unabomber': 1, 'utilizing': 10, 'neo-nazis': 1, 'honorary': 2, "villain's": 2, 'extrapolation': 1, '_titus_andronicus_': 1, 'lab': 5, 'fugitive': 8, 'halle': 4, 'contemptible': 1, 'koji': 2, "narrative's": 1, 'washed-up': 2, 'dick': 15, 'spurgeon': 2, 'heslop': 1, 'universally': 3, 'whiplash': 2, 'bullet-riddled': 1, 'relentlessness': 1, 'resemblances': 1, 'setup': 22, 'oscar-': 1, 'packages': 3, 'gimmick': 6, 'shooters': 1, 'sensual': 6, 'talons': 1, 'vulcan': 1, 'hammered': 2, 'imf': 1, "high-society's": 1, "close's": 2, 'cryo-tubes': 1, 'rundown': 1, 'stuffing': 1, 'fifty': 5, '0': 9, "'plague'": 1, "mckellar's": 1, "frequency's": 1, 'bombed': 4, 'spinster': 1, 'cooking': 5, 'grandparents': 2, 'youth': 24, 'couples': 8, 'generally': 43, "orlock's": 2, 'unique': 60, "be-ribboned'": 1, 'monty': 10, 'exploits': 8, 'disliked': 6, 'porter': 5, 'captures': 28, 'vacationing': 1, "jones'": 6, 'exclusive': 7, 'escort': 3, 'ugh': 2, 'lousy': 2, "lucy's": 1, 'dash': 4, 'debauchery': 2, "`unmemorable'": 1, 'democrat': 2, 'voyages': 1, 'emphasized': 3, 'birdsall': 1, 'zookeepers': 1, 'repartee': 2, 'mathilda': 2, 'kinda-bitchy-in-a-reserved-way': 1, 'je': 1, 'undercover': 6, 'dougherty': 1, 'kleiser': 1, 'constitutionality': 1, 'flexibility': 3, 'predominant': 2, 'deprived': 5, 'amphetamine': 1, 'transistion': 2, 'fastidious': 1, 'scrawling': 1, 'nicely--': 1, 'invented': 8, 'trilogies': 1, 'sugar': 5, 'pinnacle': 6, 'biggest-breaking': 1, 'scout': 1, 'murtaugh': 2, 'ouija': 2, 'shaw': 6, 'prompt': 2, 'elitists': 1, 'whirlpool': 1, 'bellhop': 1, 'seventies': 6, 'walker': 6, 'expressly': 2, 'spelling': 5, 'defy': 2, "ferrara's": 1, 'b+': 7, 'alyson': 1, "chad'z": 1, "she'll": 10, 'intresting': 1, 'stomping': 2, 'satirical': 15, 'foretaste': 1, 'playhouse': 1, 'nobly': 1, 'laid-back': 1, 'additions': 1, 'america': 82, 'gracefully': 3, 'doctrine': 3, 'hardcore': 8, 'irregular': 1, 'incorporates': 7, 'horse': 18, 'surpass': 6, 'storytelling': 28, 'justifies': 3, 'eternity': 5, 'madison': 7, 'crowds': 6, 'widespread': 5, 'viewer': 86, 'tantoo': 1, 'anita': 3, 'rapture': 1, 'eastwood--': 1, 'onslaught': 3, 'twenty-foot-tall': 1, 'heaped': 1, 'sensationalism': 2, 'wong': 16, 'reza': 1, 'humiliates': 2, 'discard': 1, 'nail': 5, 'oshii': 1, "'ol": 1, 'vantages': 1, 'edna': 1, 'congo': 4, 'demanded': 2, "crisis'": 1, 'rejected': 9, 'smartmouths': 1, "'real'": 1, 'preminger': 1, 'matter-of-fact': 6, 'iron': 11, 'solemn': 4, 'intersplices': 1, 'wakes': 13, 'survival': 7, 'ex-drug': 1, 'thumbs': 3, 'pryce': 4, 'complimented': 1, 'watkin': 1, '$35': 1, 'unintelligible': 1, 'divert': 1, 'dispute': 4, "studios'": 2, 'christina': 8, 'historical': 28, 'independent': 25, 'glitches': 1, 'boy': 110, 'operatic': 3, 'already-chaotic': 1, 'mccourt': 1, "`swingers']": 1, 'hanna': 1, 'intruders': 2, 'tillman': 1, 'murderer-in-training': 1, 'resignation': 2, 'criticism': 15, 'calmly': 3, 'self-control': 1, 'uncompromising': 8, 'mizrahi': 1, 'underlined': 3, 'goldbergesque': 1, 'overview': 1, '12-fingered': 1, "fellini's": 1, 'witzy': 1, 'ill-fated': 5, 'over-medicated': 1, 'breakthrough': 9, 'sultry': 7, 'garland': 1, 'insides': 2, "o'sullivan": 3, "'": 47, 'barred': 4, 'wetmore': 3, 'bandied': 1, 'inefficiencies': 1, 'captivating': 14, 'self-doubt': 1, 'worm': 4, 'analyzation': 1, 'ropes': 2, 'actively': 5, 'mahal': 2, 'opens': 87, 'convent': 1, 'preexisting': 1, 'robin': 39, 'bin': 2, 'ia': 1, 'seductively': 1, 'determination': 8, 'offering': 21, 'hinders': 1, 'kilmer': 3, 'dignities': 1, 'turf': 2, 'fflewdurr': 2, "beatty's": 2, 'powell': 1, 'limbo': 6, 'depardieu': 2, 'late-night': 2, 'desires': 15, 'concluded': 2, 'plumber': 3, 'isles': 2, 'worshiper': 1, "'slap": 1, 'fiction/social': 1, 'fighting': 41, 'powers-of-10': 1, 'agony': 1, 'swears': 2, 'buddhism': 2, 'unlocks': 1, 'staining': 2, "oscar's": 1, 'in-joke': 3, 'filer': 1, 'faster-than-light': 1, 'discouraging': 2, 'hesitantly': 2, 'cottwell': 1, "kimba's": 1, 'receipt': 1, 'riffini': 1, 'briefed': 1, "ahmad's": 1, 'moods': 3, 'listed': 2, 'norbu': 1, 'wrist': 1, 'significantly': 5, 'abusers': 2, 'neal': 4, 'custody': 8, 'charmingly': 5, "fort's": 1, 'division': 4, 'systematic': 1, 'perch': 1, 'warm-and-fuzzy': 2, 'speared': 1, 'stowaway': 2, 'famed': 8, 'organisation': 1, 'to-be-weds': 1, "mark's": 1, 'installs': 1, 'player/musician/composer': 1, 'vore': 1, 'parisian': 1, 'deadening': 1, 'principle': 8, 'sweet-natured': 1, 'out-of-control': 1, "'50s": 5, 'chillingly': 4, 'rethink': 1, 'thurman': 12, 'restart': 1, 'worst': 40, 'disregard': 2, 'over-stylize': 1, 'enlightens': 1, 'ensued': 2, 'recommendations': 1, 'vans': 2, 'hmmmm': 1, 'ravished': 1, 'spectacle': 15, 'wire': 2, 'induce': 1, 'kitschy': 1, 'murky': 4, 'bullshitting': 1, 'enthusiastic': 8, 'apollo': 8, 'well-voiced': 1, 'microscope': 1, 'grandfather': 7, 'denounced': 1, "mankind's": 3, 'one-hit': 1, 'interested': 46, "lean's": 4, 'ad': 13, 'hatcher': 2, 'world-weary': 3, "evil's": 2, "1972's": 1, 'incarnate': 2, 'us$68': 1, 'seasoned': 8, 'screwed': 5, 'hauls': 1, 'lebowski': 7, 'proto-humans': 1, 'nazism': 1, 'ansell': 2, 'lead-in': 1, 'admittance': 4, 'sources': 6, 'burnt': 2, 'amazon': 1, 'presently': 2, 'ringers': 1, 'tackles': 11, 'god-fearing-ultra-serious-anti-racist-black-man-of-power': 1, "his`plump'": 1, '1/2': 12, 'branded': 2, 'accomplishment': 13, 'turn-off': 3, 'molding': 1, 'blackmailer': 3, 'fascinated': 14, 'stanley': 15, 'guiler': 1, 'adult-david': 1, 'bulleye': 1, "simon's": 7, 'dynasty': 1, 'winkler': 3, 'one-syllable': 1, 'pump': 2, 'academics': 1, 'ferrell': 4, 'cheek': 6, "go's": 2, 'codgers': 3, 'billboard': 2, 'anti-apartheid': 1, "fitzgerald's": 1, 'leslie': 3, 'geeky': 1, 'rapport': 3, 'genre': 102, 'atmosphere': 67, 'malls': 2, "don'": 1, 'evoked': 4, 'dawson': 8, 'justifiable': 2, 'performence': 1, 'synchronized': 2, 'family/friends': 1, 'obradors': 1, 'splatter': 2, 'instance': 31, 'origin': 2, 'destiny': 20, 'exuberance': 1, 'rosencrantz': 1, 'flapping': 1, 'stein': 2, 'degenerated': 1, "'twister'": 1, 'tickling': 1, '21st-century': 1, 'brilliantly': 31, "tillman's": 1, 'imax': 2, 'clean-cut': 3, 'bearded': 2, 'cannon': 2, 'rosenthal': 2, "anna's": 4, 'darcy': 1, 'swinging': 7, 'watch': 225, 'scoffed': 1, 'uncouth': 1, 'ussr': 2, 'ambulance-chasing': 1, 'spectacularly': 2, 'carrier': 2, 'vladimir': 2, 'flawlessly-acted': 1, 'beetlejuice': 1, 'sommeliers': 1, 'uncertainty': 6, 'gourmets': 1, 'pescucci': 2, 'graduation-specifically': 1, 'odd-numbered': 1, 'cortez': 2, 'nia': 3, 'less-experienced': 1, 'units': 1, 'regain': 6, 'hauntingly': 1, "child's": 10, 'distrust': 6, 'reproduced': 1, 'sizes': 3, 'fatherless': 1, "nicholson's": 6, 'flawless': 28, 'enter': 32, 'equated': 1, 'puncture': 2, 'ex-spook': 1, 'allowing': 24, 'featured': 14, 'cute-as-a-button': 1, 'nightmarish': 6, 'meanders': 2, 'pan-and-scan': 1, 'pantoliano': 5, 'spoon-fed': 2, 'parodic': 1, 'bregman': 1, 'followup': 1, 'thinks': 56, 'gift': 21, 'll': 3, 'ominpotence': 1, 'meting': 1, 'saving': 38, 'tempest': 1, 'version--who': 1, 'dialogue-driven': 1, 'unbreakable': 4, "conclusion's": 1, 'bribery-based': 1, "film--fei-hong's": 1, 'grandiose': 5, 'submerging': 1, 'expections': 1, 'barn-burner': 1, 'anthesis': 1, 'rapids': 1, 'lacking': 19, 'venable': 1, '54': 1, 'dumps': 3, 'free-agent': 1, 'overhyped': 1, 'darned': 3, 'perilous': 1, 'coherent': 8, 'shade': 3, 'hur': 2, 'montage': 11, 'ix-i-i': 1, 'kidnaped': 4, 'segues': 6, 'potatoe': 1, "mclachlan's": 1, 'descent': 6, 'cardinals': 1, 'morrison': 6, 'chased': 11, 'psychological': 29}}, {'maneuvering', 'provocative', 'newcomers', 'synth-heavy', 'white-knuckle', "laugh'", 'punisher', 'lout', 'slides', 'morte', 'violate', 'dampened', 'dinner', "director's", 'alps', 'outstripped', 'ark-full', 'motion', 'beckel', 'in-depth', "olga's", '57', 'demoliton', 'homer+s', 'kaufman', 'detractors', 're-establish', 'review', 'adherents', 'aging', 'perkus', 'larsen', 'ceremonies', 'princesses', 'decorated', 'amusements', 'tried-and-true', '2050', 'experimenters', 'laser', 'scared', 'lurches', 'packed', 'outdone', '1/50th', 'super-charged', 'stronghold', 'trunk', 'cityscape', 'creedence', "hatcher's", 'letting', 'wrong', 'braintree', 'prank', 'lang', 'reconsider', 'onatopp', 'kev', 'materialism', "trapper's", 'demolishes', 'skit-length', 'institutionalized', 'rossellinia', 'competitive', 'valor', 'blunders', 'truism', 'quicker', 'warmed', 'tsai', 'swagger', "tom's", 'ledger', 'kombat', 'courtroom-certainly', 'hilarious', 'mind-meld', 'daryl', 'sub-titles', 'cincinnati', 'hazardous', "william's", 'confirming', 'boomy', 'forcing', 'mysterious-boyfriend', 'muster', 'fx--such', 'hillarous', 'scumball', 'critic-type', 'aardman', 'summer', 'upper-crust', 'strategic', 'mongolian', "hilly's", 'mosey', "fraser's", 'depths', 'pulled', 'six-stories', 'exacerbates', 'fancy', 'satan', 'horne', 'dinsdale', 'question', 'agonisingly', 'marveling', 'non-violent', 'yucky', 'talkshow', 'blaxploitation', 'drop-off', 'kessler', 'pussy', 'intelligently', 'unsworth', 'intellectually', "photographer's", 'exorcist', 'girl/boy', 'interligator', 'drainage', 'donovan', 'mayan', 'adrianne', 'danny', 'godfried', 'in-your-ear', 'breckin', 'over-used', 'uplifiting', 'withing', 'abandoned', 'severed', 'cyber-travelling', 'largely', 'benz', 'superstars', 'appaling', 'modernize', 'god-fearing-ultra-serious-anti-racist-black-man-of-power', 'story--or', 'greg', "dash'", 'perdy', 'nick', 'navarro', 'oil-driller', 'snappy', 'co-workers', "hutter's", 'tornadoes', "sheriff's", 'unknown', 'brothers-style', 'maidservant', "teachers'", 'intellectuals', 'latura', 'favorite', 'ex-con', 'thief', 'proficient', "'big", 'irrelevent', "jinn's", 'provoked', 'macks', 'shacking', 'alien-like', 'avant-garde', 'begets', 'marcus', "niccol's", 'milla', 'no-good', 'lulled', 'feeds', 'motorists', 'priestess', 'un-disneyish', 'psychotics', 'untapped', 'sami', 'krippendorf', 'sebastiano', "gilbert's", 'blades', 'out-', 'trumpeted', 'metaphor', 'improvement', 'lifeboat', 'cuff', 'alley', '_daylight_', 'castigates', 'nitwits', 'thirties', 'teleprompted', 'wrongs', 'wolf-beiderman', "floom's", "doin'", 'montreal', '-all', 'waterbeds', 'ambience', 'sanxay', 'scream-mask', 'usual', 'chivalry', 'krueger', 'barrier', 'inevitability', 'directionless', 'carmen', 'deceived', 'hudreds', 'infuriating', 'aamir', 'mildly', 'doped-out-eye-view', 'fury', "christopher's", '_vampires_', 'revelation--when', 'intensifies', 'enlistee', 'skull', 'junkie', 'outbreak', 'sleepy-voiced', 'train', 'herek', 'alcott', 'barondes', 'sell', 'second-rate', 'break-down', "adult-helfgott's", 'rap', 'reactions', 'cymbals', 'envious', 'six-hundred-years-old', 'good-buy', 'dysfuntion', 'salad', 'lennox', 'cell-mates', 'transfer', 'kemler', 'ze', 'dragonheart', 'ripe', 'unusable', 'dragged', 'vernacular', 'topless', 'tremble', 'represent', 'insult', 'improvisational', 'ghost-like', 'redoing', 'photographers', 'suspiria', 'gayheart', 'immaculately-white', 'tuned', 'prospers', 'slaughter', 'mccall', 'non-questioning', 'finishes', "'matrix", 'septic', "scientist's", 'veering', 'heart', 'roasting', 'wachowskis', "bubby's", 'twelve-year-old', 'mix-ups', 'rancorously', '_murder_', 'short-lived', 'edna', 'genius', 'groan-inducing', 'coolio', 'rack', '13-20', 'taking', 'grey', 'crust', 'frederick', 'circa-10th', 'sinewy', 'volume', 'streep', "worker's", 'rotate', 'join', 'costume', 'ignoramus', '10/10', 'heenan', 'barage', 'automaton', 'dictionary', 'choices--married', 'slade/wild', 'pitches', 'watching', 'ambivalence', 'un-detailed', 'kennedyesque', 'unqualified', 'menancing', 'room', 'effective', '$7', 'hierarchies', 'oj', 'outs', 'msnbc', 'yojimbo', '600-pound', 'consort', 'reflex', 'big-be', 'outta', 'judged', 'censors', 'hinds', 'guitars', 'nation-state', "act's", 'accelerates', 'metered', 'cleverness', 'cheering', "nasa's", 'found', 'reiter', 'twentieth-century', 'doors', 'dress-up', 'force', 'attempting', 'enamor', 'ethic', 'pout', "farley's", 'tre', 'urge', 'viii', "stamp's", 'eleven-year-old', 'millisecond', 'dancer', 'wildflowers', 'overcoats', 'daqughter', "element'", 'excursion', 'grazes', 'fixed', "donner's", 'decorum', "lib'", '33', 'occupants', 'once-breeding', 'weakling', 'louisiana', 'pooh-bah', 'offer', 'net-surfing', 'jalla', 'sensibility', "jeopardy'", "kate's", 'travellers', 'sympathizes', 'fear--the', 'ballistic', 'divining', 'lowlifes', 'madman', 'shooters', 'hoosiers', 'imaging', 'racing', 'animosity', "real-men'", 'cleanest', 'celia', 'inaccurately', 'unknowing', 'sparking', 'fervor', 'gavras', 'corsucant', 'instalment', 'vaulerbility', "'snuffed'", 'anatomical', 'pillage', 'all-important', 'seagal/mickey', 'amfibium', 'bay-style', "lawerence's", 'katzenberg', 'mansions', "standefer's", 'jordans', 'suggestiveness', "car's", 'revolutionaries', "ones'", 'pikul', 'geriatric', 'branded', 'extravaganza', 'articulate', 'lambasted', "mobutu's", 'headlong', 'plusses', 'copying', '`cheer', 'copasetic', 'affections', 'unexplained', 'cellar', '1980', 'cloud-choked', 'cid', 'berkoff', 'led', 'real-world', 'honest-politician-rare', 'avery', "moore's", 'minisub', 'illiteracy', 'belinda', 'jacobs', 'durning', 'lucked', "helgeland's", 'zingers', 'commonly', 'stage', 'eight-year-old', 'bening', 'plastics', 'tools', 'hillbillies', 'scheme', 'square-jawed', 'spoliers', 'dar', 'holzbog', 'weber', 'ponders', 'relates', 'life-saving', 'armrest', 'beads', 'weep', 'asuma', 'craves', 'engravings', 'fundamentalist', 'metzler', "'daddy", 'pervaded', 'carlton', 'split-screens', 'insectopia', 'at-odds', "haynes'", 'amateurishly', 'distract', 'factor', '1791', 'mancini', 'church-', 'grenades', "`snatch-the-paycheck-and-run'", "woodward'", 'chaykin', 'unappetizing', 'zellweger', 'negligible', 'terraformed', '_not_', 'divines', '13-year-old', 'singular', "labute's", 'tong', "minutes'", 'lips', 'predicted', 'mahurin', 'free', 'underage', 'yet-to-be-released', "nerds'", 'fragment', 'rohde', 'jonze', "'more", 'radiator', 'leni', 'chon', 'burgess', 'jittery', 'prequels', 'film-within-the-film', 'woolly', 'dutifully', 'electronically', "'matrix'", 'splashes', 'steel-factories', 'asbury', '100+', 'erric', 'gingrich', "`it's", 'superdemon', 'torrent', 'diametrically', "d'or", "jimmy's", 'intercourse', 'morose', 'stalingrad', "waxman's", 'foothold', "eye-full's", 'sumo', 'courageously', 'fast-emptying', 'fatherly', 'semi-developed', 'operations', 'therefor', 'acutely', 'next-door', 'writer/first-time', 'sanctimonious', 'carving', 'romero', 'unispiring', 'friend-friend', 'blotter', 'dispense', 'nihilist', 'amidala', 'charles', 'accountants', 'financier', 'search', 'hee-haw', 'modifier', 'two-thousand', 'ostott', "luis'", 'errupts', 'rock-solid', 'nicolodi', 'gushing', 'autumn', 'dismays', 'attendant', "leuchter's", 'mcglory', 'lengthens', 'hyperjump', 'frenetic', "evars'", 'gallifrey', 'reville', 'tolkien', 'infantile', 'elaborating', 'banishment', 'hermaphrodite', 'repeating', 'interrupting', "country's", 'inducing', 'classroom', 'guffman', 'alternate-1999', 'ruiz', 'goo-iffy', 'holed', 'chickens', 'restricted', 'superficiality', 'healings', 'un-noticeable', "wong's", 'snitches', "makers'", 'self-referencing', "goyer's", 'overexplain', 'diminuitive', 'ratner', "caulder's", 'homer', 'tempered', 'recourse', 'smartest', "d'oevre", 'klass', 'honorable', 'clich', 'rifkin', 'unemotional', 'marco', 'emotionally-charged', 'admirers', "crow'", 'corruptor', 'radioing', 'michelangelo', 'forge', 'uneventful', 'laced', 'cliffhanger', 'subconscious', 'epigraphs', 'manufactured', "neuve'", 'computer-aided', 'disatisfaction', 'calitri', 'explanation', 'erica', 'malaise', 'tornado-caused', 'illness--which', 'mortensen', 'actress-type', 'brood', 'sledgehammers', 'explicate', 'humankind', 'fairy-tale-in-the-', 'point', 'instant-gratification', 'old-school', "syndrome's", 'a-', 'aneeka', 'workings', 'gaffe', 'rallying', 'whiskey', 'overzealous', 'sensitivities', 'unveiling', 'sterling', 'hasidic', 'dense', 'patented', 'briefcase', 'flop', 'dromey', 'concurrent', "tavoularis'", 'rates', 'anti-semitic', 'kinski', 'milbauer', "queens'", 'piven', 'bumps', 'leone\x12s', "hunting'", 'gait', 'dives', 'garden', 'whispers', 'paquin', '_brazil_', 'neill--better', 'sheik', 'theatrics', 'carlito', '31', 'invested', 'millennium', 'ramboids', 'ho', 'debacles', 'creams', 'fbi/atf', 'mated', 'k9', 'aisling', 'jeffrey', 'sakes', 'ofcs', '`macabre', 'snarf', 'focker', 'kitschiest', 'dispensed', 'fights', 'sarone', 'niceties', 'corridor/tunnel/airduct', 'noxzema', 'strenuous', 'ocean-going', 'uhry', 'cyborg-ninja', 'kreskin', 'alum', 'recover', 'snob', 'responds', 'fleabag', 'hand-held', 'horror-fans', 'abraham', 'skyjacked', 'earth-', 'emptying', 'folk', 'larger-than-life', 'oddity', '1885', 'quarantined', 'ooooooo', 'muscle', 'exiles', 'deranged', 'juke', "reiner's", '-jack', 'tight', 'glory', 'melodramatic', 'virtually', 'vaudevillian', 'extra-strength', "birdsall's", 'vexatious', 'dzundza', 'orginality', 'backstreet', 'simulating', 'quiet', 'embarrass', 'amphetimenes', 'temporarily', 'ex-new', 'impetus', 'multimillion-dollar', 'toback', 'mother-', 'ipo', 'inversion', "howard's", 'people--would', 'badalucco', 'mark-paul', 'fahey', 'immortalised', 'half-naked', "dillon's", 'unscathed', 'mets-orioles', "sv2's", 'two-day', 'monitored', 'best-supporting', 'vignettes', 'passing', "action'", 'wayans', 'uninspiring', 'slobbering', 'nut', 'guelph', 'pappy', 'relate', 'angst-ridden', "interrupted'", 'revolutions', 'armed', 'anti-classics', "kivilo's", 'byrnes', 'film--is', 'zinnia', 'feigns', 'hen', 'interfering', "dante's", 'two-dimensional', 'napalm', 'delpy', 'mccoy', 'hyper-drive', 'untold', 'stashed', 'losses', "farmer's", 'beans', 'tapeheads', 'walcott', 'trophies', 'maudlin', 'deter', "pariah's", 'carey', 'vision', 'crew', 'watchable', 'strippers', 'homosexuality', 'stubbornly', 'truant', '_is_', 'mckellar', 'bullet-cam', "keeble's", 'esque', 'caricatures', 'manhoff', 'orca', 'bloodletting', 'guinea', 'macarena', 'type-casting', 'flesh-and-blood', 'reagan-loving', 'springer', 'aquit', 'troubles', 'k-obsessed', 'lillian', 'transitioning', 'off-again', 'effort', 'vuh', 'vomits', 'large-scale', 'fecal', 'fairytale', 'footing', 'cocooned', 'honing', 'xenophobe', 'charme', 'matter', 'well-formed', 'sex-starved', 'crotch-biting', 'titmalau', "matrix'", 'skeet', 'strove', 'sugarish', 'likes', 'self-conscious', 'blundered', 'moritz', 'kraftwerk', 'charades', 'snow-covered', 'cheaply-made', 'clerks', 'untouchables', 'distasteful', 'settings', 'misunderstandings', 'dna', 'groceries', 'instruct', 'visas', 'impatient', '+blah', 'crudeness', 'scripted', 'flintstones', 'jacobim', 'proyas', 'japan', "'round", 'half-hokey', 'outpost', 'benches', 'inane', "wittliff's", "executives'", 'excreting', "london's", 'stomachs', 'colorfully', 'brauva', 'humiliated', 'benichou', 'eights', 'descendant', 'agreeing', 'bloodthirstiness', 'clapton', 'families', 'incoming', 'uhhm', 'petersen', 'spurgin', 'romano', 'lori', 'weendigo', 'bodyguards', 'unaware', 'bankruptcy', 'sharp-edged', 'written', '5', 'increasingly-flustered', "_dawson's_creek_", 'interrogator', 'malevolent', 'unreality', 'favoring', 'indiana]', 'transgression', 'jawas', 'limit', 'cod', 'voila', 'bonds', 'directional', 'injury', 'paternal', 'requisite', 'pint-size', 'foundered', 'kicking', 'deflated', 'debuting', 'promisingly', 'coen-heads', 'goodfellas', 'corroding', 'subscription', 'lisping', 'franks', 'killed', "neighbors'", 'promotion', 'barbie-doll', 'biggest', 'maleness', 'manipulator', 'maginnis', 'gearing', '_looks_', "sadie's", 'wisened', 'waging', 'bureaucratic', 'androgony', 'fluctuates', 'fukienese', 'highest', 'commit', 'reunions', 'redefining', 'astounded', 'well-staged', 'practices', 'seemlessly', 'mournfulness', 'udesky', 'pickets', 'banner', '`if', 'familiarize', "'hot", 'volcanos', "prison's", 'levites', 'catholicism', 'directions', 'franka', 'dragonflies', 'harlequin', 'zoom', 'objectionable', 'yeas', 'prophecy', 'immobilise', 'defects', 'disliking', '_don+t', 'comedian', 'policemen', 'excels', 'amsterdam', 'long-running', 'antagonism', 'minorities', 'boy-next-door', 'merman', 'decease', 'intents', 'reprocessed', 'lassic', 'sittings', 'budweiser', 'homage', "_shaft_'s", 'sins', 'mentors', 'belonging', 'westerns', 'homey', "devito's", 'natascha', '`2001', 'damsels', 'woodies', 'undervalued', 'lynch', 'ridiculous', 'contests', 'reviewers', 'trashy', 'weaker', 'angel-related', 'unrepentantly', 'burnout', 'cockburn', 'hasten', '1916', 'prosaic', 'blubbering', 'mouseketeer', 'griffiths', 'stillman', 'irresistable', 'unfriendly', 'fizzling', 'rights', 'grammatical', 'poslethwaite', "sant's", 'slain', 'whiz', 'reunite', "mirkin's", 'architecturally-unsound', "logan's", "food's", 'kowalski', 'crappy', 'limbo', 'schreck', 'man-meets-alien', 'single-handed', 'shiro', 'raids', 'relentlessy', 'he/she/they', 'yuppies', 'them--', 'constable', 'bilal', 'terrifyingly', 'terl', 'divulges', 'un-funniest', 'glasses', 'unidentifiable', 'exec', "composer's", 'throne', 'grissom', "dracula's", 'renegades', "_schindler's", 'artist', 'beantown', 'crenna', 'cooled', 'codes', "stricter's", 'comprised', 'porns', 'greystoke', 'silver', 'reccomending', 'flic', 'cattaneo', 'bubble-headed', 'lodgerley', 'screenings', 'mcclane', 'keyboard', 'cares/if', 'bow-wow', 'too-good-to-be-true', '_a_night_at_the_roxbury_', '8-year', 'nimble', 'lebanese-born', 'gangstas', '_does_', "cinderella's", 'significance', '--michael', 'merchant', 'closes', 'strip-act', 'timely', 'rutger', 'trappings', 'horoscopes', 'polson', "d'onofrio's", 'joan', 'fencing', 'ranting', "slashers'", 'monstrous', 'juices', 'tedium', "alone'", 'nashel', 'vikings', "pocum's", 'dirk', 'risk-adverse', "'shemp'", 'delany', "ricky's", 'cannell', 'thora', 'unfortunatly', 'timm', 'missed', 'hurls', 'fortresses', 'rapist', 'lunghi', 'fun-loving', "god'", 'protigi', 'tolerated', 'professional', 'inconvenient', 'wally', 'mushroom-cloud', 'portorican', 'doo-doo', 'theresas', 'westridge', 'retorts', 'swallowed', 'purists', 'monarchy', 'leiter', "jfk's", 'shave', 'uhh', 'magician', "anderson's", 'savannah', 'happy', 'picasso', 'fallible', 'roam', 'monk', 'peek', 'well-the', 'musicians', 'dietl', 'pleasant', 'first-time', 'venetian-looking', 'extents', 'trinidad', 'fiona', 'achievements', 'outsmart', "rameses'", 'schemer', 'copulate', 'nether-realm', 'projectioner', 'epidemic', 'pinter-esque', "nights'", '24-hour', 'lahore', 'wetsuits', 'belive', 'statuesque', 'ghost-lady', 'forefront', 'embarrassing', 'steroid', 'decipherer', 'sustain', "fat's", "pyle's", 'spectacles', 'loneliness', 'arye', 'mohiam', 'non-ape-speaking', 'mass', 'helpless', 'respective', 'boone-type', 'fleetingly', 'gremlins', 're-writes', 'flinstone', 'emoting', "blossom's", 'spurted', 'cachet', 'nm', 'sextet', 'seedier', 'unchristian', "brynner's", 'composed', 'escaped', 'trap-ish', 'demonic', 'underperforming', 'invade', 'fusion', 'bystanders', 'inner-workings', 'inadvertenty', 'pretty-boy', 'trotted', 'swindlers', 'venerated', 'commandments', 'earthworm', 'shea', 'llamas', "nobody's", 'cheif', '1959', 'acute', 'evil-eyed', 'joes', "'rest'", 'even-', 'recieve', "vic's", 'relation', 'grades', 'smiles', "nossiter's", 'german-', 'skanky', 'mackenzie', 'polarized', 'secluding', 'rowell', 'farmer', 'rigeur', 'center-fielder', 'lashings', 'milquetoast', 'nationalism', 'bordertown', 'syrup', 'heart-breaking', 'kite', 'ganz', 'bothering', 'accentuates', 'keith', 'bottoming', 'mistreats', 'pasty', 'perry', 'teaming', "silver's", 'kristin', 'monks', 'catastrophe', 'indiscretion', 'gif', "period's", 'intensive', 'requesting', 'lawerence', "rohmer's", "`design'", 'awry', 'reprisal', 'protect', 'cherubic', 'leon', 'unfailingly', 'not-so-darkest', 'leather-clad', 'holmes', 'riders', 'vice-president', 'tank-', 'baird', 'hass', 'petersburg', 'catwalks', 'profusely', 'rainman', 'conducted', 'masterful', 'thai-born', 'machete', 'atkins', "1998's", 'heed', 'accelerated', 'repaint', 'authority', 'incredible', "gedrick's", "tidyman's", 'out-and-out', 'recounting', 'doubt', 'slam-bang', 'schizopolis', 'five-second', 'psychlos', 'costners', 'tigers', 'theorists', 'madyline', 'hurricanes', 'opening', 'buildings', 'redirecting', 'cookies', 'malicious', 'cloud-backed', 'saito', 'upgrades', 'disown', 'mecca', 'recycle', 'drawn', 'exert', 'venting', 'ocmic', '_twice_', 'lightheaded', 'weasel', 'out-of-water', 'excited', 'marmet', 'salesperson', 'tabasco', 'finely-written', 'souffle', 'refined', 'mahoney', 'originally', 'detailed', 'mathilde', 'lucilla', 'bar', 'survivors', 'magnet', 'quentin', 'aurora', 'russia', 'chainsaw', 'ambushes', 'bumbling', 'ginseng', '$5000', 'stong-willed', 'hi-fi', 'flight', 'bugs--how', 'sundance', 'cavorting', 'negligence', 'smart-mouthed', 'plunder', 'monosyllabic', 'turn-of-the-century', 'dodges', 'pepto', 'ariyan', 'spoon-fed-entertainment', 'late-father', 'brained', 'dismiss', 'b-rated', 'highest-ranking', 'fifths', 'shameful', 'hindrance', 'camaraderie', 'idea-driven', 'ironically', 'deemphasize', 'quick-tempered', 'alaska', "croft's", 'rancid', 'lookout', 'remorse', 'kilo', "namesake's", 'forgets', 'graced', 'selection', 'embraced', "nelkan's", 'fullfill', 'yea', 'dumbass', 'menzies', 'eight-year', 'gish', 'muslims', "dead'", 'hortense', 'bluescreen', 'devise', 'com/~canran', 'revolting', 'epigrams', 'doubtless', 'paranoid', 'disillusionment', 'ignorant', 'devestating', 'sitcom-airiness', 'caper-esque', 'klumps', 'ed-209', 'batteries', 'belgium', 'essence', 'insinuated', 'soderbergh', 'drab', 'anti-slavery', 'water-tight', 'religiously', 'tagline', 'chords', 'experiencing', 'flannery', 'splish', 'rosy-cheeked', "mehrjui's", 'humbert', "schoolteacher's", 'incronguously', 'stoners', "friedkin's", 'unacceptable', 'altough', 'legends', 'silvie', 'phlegmming', 'magnificently', 'depression', "mikey's", 'lures', 'festival', 'waitress', 'zestful', 'vicinity', 'third-class', 'provided', 'top-level', 'catchiness', 'shouts', 'commercials', 'gripping', 'domingo', 'conceal', 'spending', 'sputtering', 'beats', 'frisky', 'amigos', 'hurray', 'withholds', 'diverts', 'generalized', 'severe', 'do--become', 'rattlesnakes', 'flatten', 'bumper', 'zuehlke', 'pedestrian', 'stagnant', 'avaricious', 'alternating', 'full-blown', 'transform', 'spector', "butt'", 'total', 'ut', 'bio-dome', 'paddock', 'pris', "apostle's", 'competence', 'snatchers', 'entertainingly', "sean's", 'irreparable', 'sci-fi/thriller', 'paralized', 'pertinent', 'aptly-titled', 'intros', 'multiplots', 'lanky', 'keeping', 'helping', 'mindlessness', 'choice', 'explosive', 'spa', 'panteliano', 'technically', 'unprotected', "webster's", 'dian', 'bartleby', 'nykvist', 'overlooking', 'preflight', '999', 'dropouts', 'relaxation', 'trilian', 'yokel', 'mish-mash', 'rko', 'non-british', 'whitworth', 'groundhog', 'steaminess', 'uncomfortable', 'better-suited', 'two-minute', 'eatery', "lumet's", 'freedom--in', 'leo', 'alluringly', 'roses', 'motifs', 'tubercular', 'hann-byrd', 'cloke', 'curt', 'schwimmer', 'hypothetical', 'breakage', 'apon', 'brightly-colored', 'kapur', 'visions', 'undersea', 'morass', 'schellhardt', "drugs'", 'stormtroopers', 'chung', 'passage', 'prophetic', 'aces', 'seamlessly', 'un-flossed', "truman's", 'small-scale', "kimmy's", 'mach', 'potion', 'comicbook', 'suckers', 'defendants', 'hypersleep', 'autumnal', 'spate', 'needless', 'uneasiness', 'fleugel', 'eighteen-wheeler', 'two-hour-plus', 'categorizing', 'sixteen', 'desintegration', 'microscopic', 'greenlighted', 'innuendos', 'nailed', 'outrageously', "'hollow", 'teen-flicks', 'evita', 'break-danced', 'jackson/travolta', 'authoritarianism', "'maybe", 'valek', 'chronicles', 'chasers', 'italian', 'kosher', 'orwellian', 'marshmallows', "`demons'", 'cult-classics', 'idle', 'rahul', 'prosky', 'looming', 'detachable', 'pricking', 'friedman', 'transfers', 'perspective', 'impregnation', 'mountainsides', 'archaic', "`tarzan'", 'gladiators', 'audio-recorded', 'downside', "girlfriend's", 'porn-actress-turned-porn-producer', 'automatically', 'prepubescent', 'perkuny', 'hemingwayesque', '81-minute', 'dreadfully', 'liners', 'brake', 'jenson', 'coincides', 'false', 'sneakers', 'liberty', 'mcmillan', 'banished', 'nordoff-hall', 'litten', 'coal-mining', 'cruelty', 'brimstone', 'genes', 'imagination--and', 'fifty-three', 'jonathon', 're-assigned', 'thrilling', 'laugh-filled', 'hohh', 'port-o-potties', 'patricia', 'recommends', 'edison', 'eric', 'blackmailed', 'kiernan', 'ryuichi', 'trogolodistes', 'prejudice', 'fierstein', 'mehki', 'unforgettable--perhaps', 'sexually-ripe', 'delight', 'plummets', 'gerry', 'delectable', 'drill', "branaugh's", 'emmet', 'magritte', 'kurring', 'space-faring', 'attention-grabbing', 'possibility', "sinise's", 'meanie', 'belays', 'asssss', 'anticipate', 'introduces', 'vegetables', 'melodramtic', 'aussie', 'koren', 'luxuries', 'step', 'battery-charging', 'idealism', 'stuttering', 'backside', 'all-encompassing', "danna's", 'confronted', '153', 'chainsmoking', 'uttering', 'instructions', 'cards', 'speedily', 'sitations', 'touching', 'seattle-based', 'misconceived', 'joanne', "vacano's", 'sexier', '48th', 'alamo', 'brable', '4/23/99', 'skewered', 'offended', 'copy', 'ogled', 'disorder', 'gurgi', 'captivated', 'potty-mouth', 'summarizes', 'donuts', 'silly', "tracks'", 'cheaply', 'rooftop', 'leibowitz', 'skitters', 'contemporizing', 'patrolled', 'sphere', 'scariest-looking', 'reason', "person's", 'pining', 'southern', 'duped', 'genzel', 'over-night', 'personalities', 'out-of-the-way', 'nosedive', "bank's", 'port', 'dench', 'cameraman', 'prentice', 'tacked', "bats'", 'cheerfully', 'departs', 'duane', "tatyana's", "salonga's", 'magwitch', 'front', "mikhail's", 'yell', 'kuzco', 'levels', 'montoya', "suspense'", 'dancers', 'inputs', 'pov', 'holm', 'shoot', 'bankrupt', 'paralyzingly', 'sections', 'angle', 'plastic', 'deep', 'brauvara', 'zar', "singleton's", 'seep', 'whales', 'ikea', 'fools', 'recovers', 'supergirl', "'funny", 'previous', 'personality-impaired', "schutze's", 'indeterminate', "creators'", 'tech', 'eye-turned', 'pikers', 'corrects', 'groundlings', 'hard-to-find', 'days', 'that--despite', 'kosteas', 'area', 'schintzius', 'eye-opener', 'cuisine', 'ranges', 'kikujiro', "1989's", 'primitive', 'occurring', 'comedy--and', 'testify', 'previewed', 'resuce', 'subtly', 'exstensive', 'ged', "who're", 'conspicuously', 'thecatrical', 'middle-man', 'finace', 'mi2', 'tricia', 'rispoli', 'walkin', 'clam', 'enojyed', 'gaetan', 'diggers', 'love-', 'doli', 'pineapple', 'hand-drawn', 'casino/hotel', 'esposito', 'tienne', 'pulleys', 'clippings', 'nuanced', 'perreault', 'rancor', 'chivalrous', 'vote', "luedeke's", 'shpadoinkle', 'kentucky', 'enlivens', 'snuffing', 'half', 'titanic-type', 'smacked', 'winston', 'tweeder', 'marred', 'sextette', 'fiascoes', 'sakaguchi', 'african', 'dispenses', 'ice-cream', 'sighing', 'indefatigable', 'dystopic', 'courting', 'vampire', 'analyzed', 'comedy-thriller', "3's", 'emperor', 'gossip', 'al', 'motherload', 'rhinoceros', 'mistakenly', 'factions', "tolkien's", 'mountains', 'well-placed', 'after-effects', 'chops', 'rusty', 'huggan', "officer's", 'thy', 'beast', 'unrealistic-looking', 'anxious', 'sprints', 'reiner', "1990's", 'carter--a', 'alan', 'understand', 'detracted', 'pauline', 'rade', 'bread-and-butter', 'innovations', 'infrequent', 'dozen', 'tunnel', "hodge's", 'smartly', 'genre-parody', 'unremittingly', 'pun', 'fagina', 'railroads', 'loggins', '3d', 'glaze', 'boatman', 'epic', 'gellies', 'lector', 'existentialist', "peet's", 'future', 'murderer-in-training', 'bungler', 'posits', 'poorly-done', 'raves', 'fennyman', 'conceivably', 'wonderous', 'awsome', '_death', 'six-foot', 'braided', 'regent', 'storyboards', 'molesting', 'cholodenko', 'groundbreaking', 'one-line-at-a-time', 'ex-love', 'taunt', "fries'", 'blurb', 'composing', 'nutrition', 'abortion', "april's", 'film-going', 'grab-bag', 'utter', 'faults', 'cassel', 'diarist', "taxi's", 'full-page', 'smirk', 'rothwell', 'potentialities', 'wonderbra', 'ski', 'irons', 'conclusions', 'plumb', 'timoney', 'alienated', 'dies', 'refused', 'siding', 'caan', 'guess', 'aliens-as-human', "travis'", 'ladybugs', 'insidiously', 'pokes', 'vampire-fest', 'reaffirming', 'mounted', 'non-event', 'nom', 'shutting', 'skecthes', 'summing', 'trim', 'k-filled', 'blackout', 'chomped', "f'words", 'multi-faceted', 'croft', 'caveat', 'gerald', 'paternalistic', '65', 'blue-ish', 'monstrously', 'common', 'bros', 'over-zealous', 'ontop', 'man-god', 'stamp', 'inventing', "forster's", 'shed', 'fully-armed', 'far-fetched', 'mirkine', 'molinaro', 'downtrodden', 'chunnel', "down's", 'collars', 'trigger-happy', 'rainbow', 'sensational', "'quack", 'definition', 'tuxedo', 'unsentimental', 'beckinsale', 'clashed', 'coifed', 'chains', 'slug', 'theatre-shaking', 'misfits', 'whistle', 'natasha', 'shelmickedmu', "knowin'", 'undoubtful', "`horror'", 'amyls', '28th', 'praise', 'contagion', 'peril', 'yak-fest', 'trevor', 'concise', 'uneducated', 'mind-blowing', 'lumber', 'magnifying', 'informercial', 'roadblock', 'zeffirelli', 'puppets', 'backer', 'bright-eyed', 'harvard-schooled', 'accumulated', 'whisking', 'hessian', 'uncommonly', 'disaffected', "chareau's", 'absorbant', 'cola', 'win', 'hundred-million-dollar', 'battleship', 'moving', 'quiclky', "bogart's", 'sitch', 'war-crazy', 'attend', 'contractually-obligated', 'newsreporter', 'sideshow', 'gregorio', 'mastered', 'mikael', 'honoring', 'bagpipe', "employer's", 'hastily', 'brethren', 'sums', 'sandworms', 'publishers', 'pitillo', 'memorizing', 'manufacturing', 'entourage', 'rooftops', 'breadwinner', 'radha', 'competiton', 'infuses', 'ended', 'nightingale', 'slanted', 'yuelin', 'stirrings', 'flanks', 'writing', 'screenplay-and', 'psychobabble', 'tatooine', 'fascistic', 'pick-ups', 'pulsing', 'lapse', 'thawed', 'half-buttoned', 'frolicking', 'immeasurable', 'motormouth', 'rigour', 'cici', 'feelings', 'yoshi', 'postino', 'cringe-inducing', 'papas', "mtv'ish", 'occurrence', 'fort', 'unpopular', 'takeshi', 'scavengers', 'removes', 'repetitiveness', 'zorg', 'morquio', 'scratches', 'sequal', '15th', 'expound', 'preachy', 'cog', 'zeus', 'spiritless', 'adultrous', 'cuckoo-bird', 'attacking', 'titilation', 'crusaders', 'risqueness--mostly', 'reap', "50's", 'painstakingly', 'emmannuelle', 'nonsense', 'specfically', 'standing', 'raheem', 'crafty', 'cold', 'subsist', 'latter-day', 'breathing', 'serviceable', 'blender', 'carr', 'brand', 'juvenile', 'full-grown', 'recklessness', 'hormonally', 'soused', 'weighed', 'nigeria', "gordo's", 'roth', 'cretin', 'lemmon', 'reassure', 'clear', 'riding', 'drama-packed', 'gaffer', 'seder', 'smoker', 'danforth', 'synopses', 'transformers', 'barring', 'mujibur', 'masterminded', 'cora', 'honey-bunny', 'juggles', 'meadows', 'love-it-or-hate-it', 'quasi-artistic', 'casting-wise', 'abberline', 'coppolas', 'serpico', 'flatulence', 'leftover', 'scotch-induced', 'roosevelt', 'grieco-esque', 'cutthroat', 'privelege', 'weaponesque', 'performed', "poem's", 'chutzpah', 'spine-chilling', 'books-about-movies', "jock's", 'hiave', 'wielded', 'spoofery', 'long-dead', 'vehemently', 'jumbo', 'arouses', 'applauded', 'double-sided', "gaz's", "verbal's", 'lawman', 'bert', 'hijinks', 'extremist', 'christ', 'procured', 'distinguish', 'evacuated', 'chief', 'adultery', 'multimillionaire', 'toothy', 'lifts', 'colliding', 'rochester', 'fooling', 'disintegration', 'maturation', 'bantering', 'shutterbug', 'newly-reunited', 'magnifcent', 'connect', 'pileup', 'eliminating', 'float', "firm'", 'marry', 'un-hip', 'scuba', 'non-specificity', 'trusted', "m&m's", 'demented', "40's", '1956', 'semi-successful', 'lazily', 'revives', 'robby', 'croatian', 'prays', 'henriksen', 'levied', 'arachnid-like', 'jack/rose', 'establishment', 'overscored', 'moustaches', 'tender--de', 'all-seeing', 'picket', 'daviau', 'flattened', 'doofus', 'stakeout', 'accolade', 'boasted', 'cabbie', "auteur's", 'glue', "claudia's", 'unquestionable', 'optic', 'pimply', 'high-visibility', 'spacecraft', "jean's", 'rescuers', 'blawp', 'sheryl', 'angela', 'tediousness', "carol's", 'railed', 'eighteen', 'ubiquitous', "woman's", "90s'", 'manuscript', "`fish-out-of-water'", 'reformer', 'once-meek', 'norse', 'hairstyle', 'surgical', 'horndogs', 'sizzles', '90-plus', 'maelstrom', 'consciouness', 'bonham', 'riddler', 'mcgraw', "fa's", 'beckham', 'local', 'waltzes', 'merry', 'jubilantly', 'near-revolutionary', 'rozema', 'blossom', 'replies', "sly's", 'congress', 'exposing', 'effete', 'marlo', 'valencia', 'antagonizing', 'contagious', 'cy', "'special", 'ascribe', 'somersets', 'moores', 'fallen', 'tondo', 'outpace', 'scumbag', 'neo-noirs', 'nebraska', 'resorts', 'voiceover', 'whoop', 'doffs', 'dams', 'skinnier', 'bothersome', 'stirringly', 'spear', 'villain/hero', 'feeble', 'charitable', 'huntz', 'portobello', 'scientists', 'worship', 'sarita', 'chumming-up', 'undistinguished', 'cursory', 'disoriented', 'unwelcome', 'on-screen', 'enhancers', '_waiting_to_exhale_', 'mancuso', 'coaster', 'meaner', 'alyssa', 'clouds', 'extraneous', 'egomania', 'flirts', "troy's", 're-powering', 'pre-launch', 'catania', 'oversaturated', "goldberg's", 'sways', 'pithy', 'primeval', 'feingold', 'seemingly-endless', 'three-minute', 'dehaven', 'striker', 'fathers', "ebouaney's", 'scarce', 'softly-toned', 'build-up', 'ballentine', 'activity', 'unnerve', "andrew's", 'bondsmen', 'quotes', 'medgar', 'gd', 'lindsay', 'churchgoers', 'desensitized', 'swanson', 'childish', 'wolf', 'dichotomy', 'secondarily', 'horror-movie', 'clubbed', 'analytical', 'mid-teen', '23', 'gratuitious', 'pissing', 'gump', 'renounce', 'royalties', 'nowadays', "adjuster'", 'true--a', 'co-producer', 'odder', 'staterooms', 'jump-roping', 'contemporary', 'flush', 'ballot', 'slinking', 'idiocy', 'glove', 'abhorrence', "'here'", 'whilst', "bunk-you're", 'lesseps', 'sigh-inducing', 'feminist', 'fairfax', 'winking', 'nunn', 'rallies', 'budgets', 'ryan/andy', 'someodd', 'mordantly', 'cassie', 'compounding', 'noticing', 'jefferson', 'gord', 'bribes', 'tell-it-like-it-is', 'redeems', 'accuracy', 'fraught', '1', 'sorrier', 'dopy', 'superflous', 'neighborhood', 'bmw', 'pheromones', 'gunk', 'plauged', "calitri's", 'big-shot', 'time', 'nationality', 'promotions', 'brendan', 'inky', 'belonged', 'goals', 'on-form', 'one-hundred', 'ex-hubby', 'vacillating', 'motorcycle', 'fudd', 'pearl', 'surrogate', 'macfadyen', "fiance's", 'performance', 'linklater', "taiwan's", 'subconsciously', 'shallowness', "isn''t", 'meat', 'headlines', 'casually', 'goodnight', "mi2's", 'graves', 'generics', '_moby', 'mudhole', 'collaborated', 'mama', 'veer', "draven's", 'cuz', 'twenty-year', 'updating', 'marker', 'jingles', "drivin'", 'comedy', 'bigwig', 'alek', 'psychosis', 'vacancy', 'phases', "he'd", 'discotheque', 'collective', 'copolla', 'beaches', 'affair-a-week', "amblyn's", 'reeves', 'omar', 'whippersnappers', 'tarantino', 'sexes', 'deceit', 'julian', "dunaway's", 'songwriter', 'medium]', 'strozzi', 'd-grade', 'mccoll', 'mchugh', 'venier', 'creator-owned', 'sticker', 'uninfected', 'viola', 'association', 'contemporaries', 'compulsive', 'gobbledygook', 'sup-plots', 'inaccurate', 'equipped', 'ensnare', 'directoral', 'murmur', 'batarangs', "jerry's", 'arms', 'jails', 'intrinsics', 'softy', 'hee', 'progressive', 'bilk', 'spreading', 'divison', 'cox', 'conventions', 'super-orgasmic', 'yossef', 'egotism', "jedi's", 'extend', 'picture', 'mtv-type', 'inception', 'amateur', 'newspaper', 'hurriedly', "review's", 'nihilists', 'underlines', 'grace', 'cia', 'christiane', 'jabbering', 'elusive', 'jawa', 'onlooker', 'ref', '`music', 'pressure-cooker', "millie's", "jury's", 'abruptly', 'bourke', 'discussion', 'co-exist', 'prydain', 'portrtayal', 'whiz-bang', 'amount', 'implies', 'publication', 'annihilation', 'evelyn', 'favor', 'tangled', 'valenti', 'naive', 'eisenhower', 'freddie', 'dismay', 'mosques', 'resident', 'claude', "ho's", 'henrikson', 'alarmed', 'congratuations', 'lockup', 'yawns', 'nastier', 'abril', 'come_', 'voyeuristic', 'paulina', 'koteas', "`padawan'", 'felix', 'midgett', 'sure-deal', 'interconnectedness', 'disappeared', 'dogsit', 'groan-worthy', 'olivia', 'lovitz', 'damages', 'talke', 'futility', 'catholic-rattling', 'easier', '121', 'conceiving', 'conformist', 'cronenberg-gore', 'emotionally', 'oasis', 'yaps', 'dey', 'winona', 'centenial', 'martyr', 'structuring', 'santa', 'not-so-glorious', 'closed-in', 'along--and', 'chingmy', 'juzo', 'mcclaine', 'tinam', "ken's", 'roams', 'conscience', 'mabe', 'finely-realized', 'opposite', 'schmoe', 'ex-prison', 'pledged', 'wednesdays', 'kissed_', 'snoots', 'domicile', '_no', 'reawaken', 'scholarships', 'fonz', 'sulfuric', 'analytic', 'buns', 'azazel', 'minion', 'wandered', 'mundane', 'listeners', 'antonia', 'hellish', "oates'", 'sagebrush', 'hopes', 'exited', 'bakker', 'credo', 'focuses', 'beset', 'depression-era', 'scientests', 'bestselling', "hrundi's", "summer's", 'ermey', 'remastered', 'temerity', "skeet's", 'element', '`fastest', 'ex-porn-', 'onshore', 'roelfs', 'child', 'lisp', 'credulity', 'ellis', 'numbers', 'arliss', 'austrian', 'toughs', 'horror-action', 'escapades', 'inserting', 'harold', "ed's", 'ichor', 'ii_--it', 'weapon', '+3', 'franchises', 'motherf', 'icon', 'conventionality', "century's", 'ex-olympic', "garafolo's", 'sanches', 'outwitting', 'woke', 'uninterested', 'gook', 'deviating', 'revisited', 'shadowed', "gerry's", 'suite', 'whorehouses', 'completist', "beth's", 'malevolence', 'cowrites', 'mccann', 'demille', 'endangers', 'vessels', 'mindful', 'transmitter', 'leash', 'waynes', 'pulsates', 'crouching', 'sometimes-understated', 'succumbing', 'analyse', 'quarterback', 'touch', 'renard', "varley's", 'techniques', 'medic', 'elena', 'workplaces', 'funniest--and', '[1996]', "dream-rosaleen's", 'bananas', 'poop', 'self-indulgence', 'upset', 'coote', "'predator'", 'july', 'self-named', "mtv's", 'vig', 'concepts', 'stoops', 'korner', 'begging', 'tomlin', 'greats', 'casket', 'ad-libbing', '2023', 'loom', 'taplitz', "munchkin's", 'lower', 'confessing', 'loftier', 'humbly', 'fries', 'musetta', 'defended', 'anti-social', 'deluded', 'honey-mustard', 'adoptive', 'elects', 'composer', 'hand-delivered', 'october', 'affinity', 'blissfully', 'buttocks', 'semi-excusable', 'murdoch-esque', 'inventions', 'schwartznager', 'tortoise', 'mega-budgeted', 'sleepwalk', 'renshaw', 'bannister', 'midgets', 'bathing', 'aquire', 'drain', "everybody's", 'truths', 'dime', 'falstaff', 'geoff', "atlantic's", 'visage', "angel's", 'unexceptional', 'embodies', 'idiosyncrasies', 'laconic', 'solutions', "dracula'", 'lancing', 'straps', 'captivate', 'hinder', 'bye', 'errol', 'shagless', 'unavoidable', 'collapsing', "aloise's", 'schnazzy', 'mistakenly-suspected', 'forty', 'launched', 'spinella', '2024', 'creepy', 'significant', 'tip', 'ducts', 'rumpo', 'everett', 'rene', 'in-cluding', 'gems', 'rips', "zurg's", 'all--and', 'penetration', 'feeling', 'ago', 'pitiful', 'psycho-in-the-rain', 'nell', "gould's", 'sculptress', 'd2', 'empty', 'giacomo', 'preacher', 'respond', 'decapitated', 'mousetrap', 'mulroney', 'wham', 'author=hicks', 'accumulate', 'corks', 'ineptness', 'stylist', 'cruttwell', '77-year-old', 'cuarsn', 'goodnatured', 'dilorenzo', 'blemish-free', 'microphage', 'accuses', 'supersedes', 'cleanse', 'petrie', 'enthusiastically', 'harker', 'sufi', 'buxom', "deane's", 'platform-soled', 'titles', 'aggravation', 'dashed', 'wrought', 'labrynthine', 'licked', 'masquerade', 'reconsiders', 'overacting', 'earl', 'reprieve', 'herve', 'breeds', 'barrels', "harwood's", 'short-hand', 'dilbert-esque', 'calendar', 'skirts', 'shortcut', 'textured', 'amazes', 'after-school', 'amply-sized', 'researched', 'wry', "arkin's", 'thought--will', 'rat-infested', 'palladino', 'stepford', "james'", 'humanism', 'link-between', 'deserts', "sanders'", 'brassieres', 'unknowingly', 'jive', 'maclean', 'ever-changing', 'sappiness', 'befriends', 'hilarity', 'dorothy', 'steak', 'dismayed', 'ability', 'illness', 'jedi', 'philip', 'funari', 'holley', 'samurai', 'continuation', 'tai', 'definitive', 'undoubtly', 'russo', 'stoop', 'cameras', 'imperious', '13', 'owing', 'cutouts', 'withdrawing', 'mansion', 'broad-stroked', 'pretending', 'steal', 'versus', 'gatsby', 'devoted', 'institutions', 'culmination', 'kravitz', 'herrington', 'foresight', 'suffering', 'hairpiece', 'channeling', 'blotteth', '155', 'divergent---', 'characterize', 'buddie', 'long-vanished', 'poisons', 'hatched', 'cracks', 'grounded', 'void', 'violator', 'taciturn', 'one-legged', 'diaper', 'now-ritual', 'hyped-up', 'nurses', 'roughed', 'rained', 'well-suited', 'hum-vee', "g's", 'perused', 'turtles', 'dim-witted', 'perceptively', 'executes', 'programs', 'phenominal', "'handling'", 'unavailable', 'prohibits', 'dogmatism', 'insinuations', 'hanged', 'mightiest', 'schrager', 'cartoons-', 'biotech', 'pleased', 'imply', 'ramble', 'multiplied', 'didactic', 'sox', 'trinca', 'deneuve', 'overstated', 'attenuated', 'reviewing', 'curiosity', "ephron's", 'vivre', 'erich', 'completely-is', 'martial-arts', 'self-awareness', 'coffin', 'openly-homosexual', "'blabbed", 'robb', 'close-to-death', 'xiong', 'stridently', 'objective', 'favors', 'eleanor', 'conduct', 'stooges', 'rong', 'double-crossed', 'tower', "bach's", 'coughs', 'martha', 'jabba', 'chiouaoua', "austen's", 'fowler', 'southport', 'bearse', 'bligh', 'torturous', '__________________________________________________________', 'phillippe', 'cache', 'surrounding', 'studi', 'leonard', 'grandpa', 'zenith', 'poignancy', 'luftwaffe', 'pees-ahn', 'maximus', 'lead-in', "cummin'", 'edged', 'pie-selling', 'caraccio', 'schramm', 'exuding', 'cuts', 'easily-corrupted', 'un-technical', "best'", 'mightily', 'half-win', 'kahuna', 'unwind', 'struggle', 'lugia', "rydain's", 'helper-', 'manipulated', '&nbsp', 'vaporize', 'below-ground', "fisherman's", 'handcuffs', 'expressionist', 'grime', 'wresting', 'stendhal', 'davidtz', 'discussions', 'nonfans', '16th', 'immerse', 'tosses', 'mona', 'iggy-esuqe', 'frankie', "fears'", 'judicious', 'unleashes', 'here-dna', 'differently', "ferrera's", "nancy's", 'soar', 'uninhibited', 'assante', 'elizabethan', 'cary-hiroyuki', 'machines', 'stupendously', 'teapot', 'madmen', 'mowed', 'verify', 'poll', 'cheerleader', 'ruthlessness', 'rapid-fire', 'dionna', 'voices', 'legree', 'masquerading', 'trivialized', 'conjunction', 'passionate', 'chester', 'enthused', 'item', 'stand-outs', 'sermon', 'patent', 'acid-trip', 'faylen', '24-year-old', 'wire-fu', 'minded', 'astronomer', 'proletarian', 'imagine', 'crashing', 'jax', 'goths', 'cents', 'small-to-big-screen', 'nuclear-weapon', 'convergence', 'duplicity', "susan's", "spacey's", 'newly-gestated', 'coordinator', 'luscious', 'crashed', 'india', "wah's", 'tenny', 'los', 'bellow', "beresford's", 'superman', 'art-form', 'yawning', 'explorers', 'bischoff', 'lingers', 'waittress', 'sweeney', 'crooks', 'surveillance', "boss'", 'groundlessness', 'scientologist', 'depsite', 'sniff', 'lack-of-speed', 'plying', "arc's", 'lyons', 'youngsters', 'thirty-eight', '`mystery', 'materialistic', 'berzerk', 'lap--a', "capshaw's", 'sprockets', 'freshest', 'changing-of-the-seasons', 'safe', 'patriotic', 'crewmates', 'quinlan', 'suspenseful', 'hackford', 'emote', 'sprinter', 'intently', 'melodies', 'seats', 'boyhood', 'alferd', 'repay', 'expletitive', 'screwy', "hell's", 'ex-round', "c'est", 'priests', 'wittily', 'whazoo', 'jemima', 'pleasure', 'redundantly', 'polish', 'orth', 'gaze', 'submarine', 'high-tech', 'elloquently', 'broyles', 'wrap-up', 'kaisa', 'sicken', 'spades', 'ultra-huge', 'strenuously', 'genevi', 'existences', 'tempted', 'hammerhead', 'ultra-hip', "raining--you'd", '8', 'presided', 'outcome', 'earful', 'limp-wristed', 'automatic', 'run', 'stranding', 'belongings', 'theater-goers', 'dirtied', 'snigger', 'merian', 'rough-yet-debonair', 'digest', 'upholding', 'absorbing', 'schwarzenegger', 'vito', "musketeer's", 'fobbed', 'ritter', 'generations', 'lightsaber', 'berenger', 'gorillas/live', 'escapee', 'dignam', 'one-down', 'kyzynski-style', 'shyer', 'fassbinder', 'pixie-faced', 'amplified', 'december', 'negating', 'winningham', 'championship', 'swinging-', 'tattoos', 'spritely', 'reviewed', 'x-', "scorces'", 'cleaned-up', 'dollies', 'graft', 'primae', 'breathers', 'comely', 'feast', 'headache-inducing', 'bereft', 'loxley', 'brining', 'desaster', 'symbols', 'sleaze', 'kusakari', 'compatible', 'withdrawal', 'phenomenon-', 'cousins', "liman's", 'anus', 'startup', 'wobbly', 'squeak', 'stripping', 'roger', 'organized', 'unfortunate', 'crunchily', 'brassed', 'off-the-wall', 'dependant', 'wills', "1969's", 'encouraging', 'fishes', 'barta', 'chisolm', 'jerome', 'engineered', 'tabbed', 'metres', 'roux', 'posturing', 'gutted', 'sale', 'attachment', 'antichrist', 'low-flying', 'jab', 'ethical', 'tidied', 'repent', 'provocation', 'kieran', 'whoopee', 'vulnerable', "experimentation's", 'unisol', 'christmastown', 'elijah', 'shephard', "erin's", 'zakk', 'chihuahua', 'unnamed', 'selling', 'attacks', 'grew', 'athlete-actors', 'grotesqe', 'coordinates', 'self-centeredness', 'slap-stick', 'connecting', 'relax', 'simplistically', 'haunted', 'banged', '35mm', 'entendres', 'crowing', 'pilion', 'supermarket', 'reconstructive', 'leadership', 'minefield', 'grade', 'gingerly', 'phnah', 'stupid-', 'shot', 'mutilation', 'spin-doctoring', 'bakkers', "all's", 'mccormack', 'falters', 'solidly', 'businesswomen', 'blatherings', 'aronov', 'domineering', 'killer', 'prison-bound', "'bluer'", 'bulging', 'pirhanna', 'puppy-dog', 'ramp', 'locally', 'top', "mantegna's", 'meaning', 'bracket', 'pawn', 'tafkap', 'illusory', 'former-angel', 'love-hewitt', 'penmen', 'elitism', 'pint-sized', 'ventriloquist', 'unashamed', 'electronics', "shakespeare's", 'doesn\x12t', 'oogie', 'ill--we', 'aissa', "comet's", 'polley', 'drunkenness', 'confronts', 'carrey-ish', 'frauds', 'iii', 'newswoman', 'glossy', 'thursday', 'snack', 'co-creator', 'underplays', 'louison', 'fluff', 'hat-trick', 'recommended=85=85but', 'chabat', 'wahlberg-former', 'metal', 'paymer', 'remarry', 'impersonating', 'bras', 'julianna', 'performs', 'imminent', 'critically-savaged', 'cheaper', "lucinda's", 'neighbouring', 'dispensing', "esquire's", 'easy-to-', 'rapes', 'disappoints', 'rigmarole', 'rock-em', 'kung', 'baroque', 'ding', 'rikiya', 'divesting', 'deprivation/appreciation', 'postulate', 'r-rating', 'orton', 'cyborg', 'spacing', 'dominance', 'free-form', 'quizzing', 'snotty', 'wastelands', 'u-haul', "frasier's", 'go-cart', 'donnie', 'wood', 'red/blue', 'tension-filled', 'chili', 'serenaded', 'rifle-ly', 'germania', 'yoo', 'slashers', 'decade', 'wyat', 'frankly', 'pauses', 'submarines', 'limelight', 'rama', 'reworks', '3/4', 'vincenzo', 'unpromising', 'phone', 'bio-lab', 'realistically', 'enlistees', 'trusting', "weyl's", "jip's", 'straight-forwardly', 'leaking', 'wynn', "tupac's", 'outraging', 'marrieds', 'fax', 'homogenized', 'creeper', 'actress-turned-comedy', 'aberdeen', 'down-and-out', 'inspections', "pidgeon's", 'hearted', "labors'", 'elgin', 'vanquish', 'helming', 'babbling', 'indie-rock', 'drug-addled', 'pedersen', '24', 'swiftly', 'alter-egos', 'palme', 'inhabit', 'subtleties', "theron's", 'con-air', 'hyperspeed', 'slog', 'hauled', 'fight/rocky', 'smoke', 'index', 'shooter', 'showcased', 'landlord', 'decoys', 'hardened', 'caller', 'ascension', '1919', 'hungry', 'supporters', 'trundle', 'marx', 'vocabularies', 'inconveniences', 'hums', "'barntill", 'party', 'knowledgeable', 'gangsta', 'treatise', 'bladed', 'growls', "30's", 'psychopaths', 'common-', 'tightly', 'um', 'stag', 'teen-comedy', "kimball's", '`take', 'low-life', 'conceivable', 'dispirited', "schwimmer's", 'morissette', 'varieties', 'manifesto', 'urine', 'y2g', 'zaftig', 'brutally', 'plant', 'foul-mouthed', 'punching', 'mandala', 'half-vampire', 'disaster', 'mobutu', 'baio', 'sodomized', 'embraces', "crap'", 'sebastian', 'notice-rather', 'garry', 'towers', 'bilingual', 'stakes', 'loophole', 'wrongheaded', 'down--way', 'stoicism', 'locale', 'sordidness', 'shits', '[insert', 'massacering', 'permissiveness', 'selick', 'wald', 'repelled', 'aoki', 'furrowed', 'aboslutely', 'contacting', 'yzma', 'interpretations', 'huckster', 'magoo-style', 'tad', 'lilting', 'tentatively', 'faux-bloopers', 'longings', 'one-notedom', 'outgunned', "spain's", "doer'", 'subdued-and', 'run-down', 'loners', 'ustinov', 'darlanne', '_so', 'beastiality', 'vitriol', 'cold-cocks', 'bible', 'sitter', 'woburn', 'raquel', 'effaced', 'computer-', 'spiked', 'sprightly', 'stylish', 'play-by-', "jerusalem's", 'weiss', 'pictures', 'maddy', "--it's", 'descend', 'schoolteacher', 'attentions', 'non-animated', 'festivities', 'marketed', 'whiney', 'throttle', 'organisms', 'messinger', 'unworkable', 'scam', 'glave', 'flex', 'multi-character', 'underneath', 'reacquaints', 'videotaping', 'nitpicks', 'harried', 'ultramaterialistic', 'resentful', 'inuits', 'impregnable', 'independance', 'troublemaker', 'sunny', 'inferring', 'priesthood', 'freed', '`sand', "'92/early", 'wando', 'wooden', 'dion-esque', 'suspected', 'hiv-positive', 'unrecognizable', 'legal', 'coburn', 'bull-headed', 'ghidrah', 'moffat', 'home-shopping', 'cracklings', 'image--that', 'mario', 'sequence', 'clashing', "remington's", 'wizened', "d'amato", 'lorne', 'penile', 'dairies', 'double-exposure', 'mousy', 'receive', 'typecasted', 'billion', 'mountain/hill', 'overtake', 'natacha', 'protestors', 'consummate', 'brain', 'full-price', 'cheeseball', 'strasser', 'prescripted', 'slawomir', 'planting', 'burke', '2013', 'womanfriend', "`bats'", 'osborne', 'ferries', 'dreyfuss', "counselor's", 'bouncy', "hurley's", 'scare', 'nauseum', 'depresses', 'voters', 'confirms', "todd's", 'technologically', 'co-existed', 'adhere', 'in-valid', 'reforms', 'semblence', 'originality', 'rune-like', 'bancroft', 'impulsive', 'revelation', 'boozed-out', 'souls', 'drawbridge', 'helicoptered', "shalhoub's", 'bunyans', 'appearances', 'ever-appealing', 'flyboys', "tanner's", "birdy's", 'quintet--', 'villian', 'embarrassed', "sound-we're", "cod's", 'non-classics', "cardinal's", 'vicent', 'masterminds', 'anti-terrorism', 'spit', '_saturday_night_live_', 'mischievous', 'labs', "jarmusch's", 'frontier', "verlaine's", 'supporter', 'hunkish', 'sharp-eyed', 'intact', 'rockies', 'toughly', "ben's", "nookey's", 'newmar', 'struck', 'masochistic', 'covering-up', 'mineo', 'tucci', 'smallest', 'incorporate-actors-into-existing-film-footage', 'marching', 'kink', 'sleight-of-hand', 'plods', '28up', 'batters', 'hole--symbol', "costanza's", 'sunhill', 'gripe', 'con-', "snake's", 'wigand', 'wagered', 'pouring', 'swamp', 'overextension', 'veloz', "womens'", "channel'", 'ghost', 'poems', 'howser', '_ghost', 'artisan', 'android', "mary'", 'ton', 'thivisol', 'emerging', 'teasers', 'arid', "empire's", 'ex-alchoholic', "brother's", 'compatriots', 'narcoleptic', 'juicy', 'subtle', 'labelling', 'viewpoint', 'patches', 'url', 'momentous', 'shadings', 'discarded', 'to--again', 'amblyn', 'nazi', 'meanspirited', 'pooh', 'flaquer', 'introversion', 'salvation', 'gun', 'hodge-podge', 'loren', "congo's", 'pratfalls', "1991's", 'gwar', 'apathy', 'variation', 'shakycam', '+2', 'wrongly', 'humiliates', "1930's", 'survive', 'fawcett', 'musings', 'cronenberg-esque', 'divert', 'carhart', 'hal', 'perfect', 'scarlet', "daisy's", 'over-', 'signaled', 'stowaway', 'hindus', 'ringwood', 'fending', 'farfetched', 'michel', 'attuned', 'souring', 'melanie', "mulder's", 'vaguest', 'restores', 'hippest', 'crew-cut', 'projection', 'mouthpieces', 'ferreted', 'reappearance', 'pillow', 'entire--and', "1985's", "skywalker's", 'followup', 'saddam\x12s', 'walkouts', 'profane', 'stunts/special', 'generously', 'coconut', 'football', 'moore-esque', 'trillions', 'tragic', 'giosue', 'timex', 'incubus', 'joystick', 'winterbottom', 'dogg', 'inflate', 'loses', 'gedrick', 'continental', 'populate', 'chirped', 'migrates', 'hijacker', 'today', 'embroils', 'smug', 'mumbled', 'crocs', 'josie', 'miramax/constellation/ugc/hatchette', 'simplifies', 'copping', 'owes', 'meteorite', 'cranium', 'rantings', 'organism', 'parrots', 'ultimo', 'mush', 'commissioned', 'movie\x05', 'uncaring', 'first-hand', 'conglomeration', 'screwball', 'overeacting', 'coincidences', 'city-devastation', 'commence', 'foolishness', 'satisfy', 'takes', 'fodder', 'jansen', 'picture]', 'masochist', 'infuriatingly', 'andersons', 'dates', 'lin', 'yimou', 'twists', 'riveted', 'perceiving', 'rick/ilsa', 'top-rated', 'blue-screen', 'scholars', 'weeps', 'translation', '95-minute', 'tampered', 'psychodrama', 'quint', "sciorra's", 'stepped', 'resoundingly', 'time-constrained', 'incidental', "nolan's", 'writers', 'pillsbury', 'keating', 'on-board', 'emmanuelle', 'privileges', 'quartered', 'davidson', 'ingredient', 'not-so-obvious', 'unspectacular', 'billy', 'distortion', 'affay', "badalamenti's", 'palmieri', 'overlaid', 'served', 'user', 'handcuffed', 'deleted', 'receptacle', 'ceasefire', 'humor', 'gasp', 'runnings', 'inflation', 'wes', 'relief', 'trinity', 'uneasily', 'exaggerated--this', 'strauss', 'sourabaya', 'recipes', 'ousting', "halloween's", 'perceived', 'cellophane', 're-energize', 'smacks', 'rienzo', 'claus', 'piling', 'gobbling', 'platform/prison', 'herman', 'anchors', 'life-forms', 'mailing', 'unrelated', 'autopilot', 'disintegrating', 'sinclair', 'rogue-ish', '[mike', 'chalderns', 'transparent', 'maddened', 'condemn', 'abortive', 'boyfriend/cop/partner', 'seamless', 'hunka', 'letdown', 'farrelly-funny', 'opposes', 'bandits', 'ex-assistant', 'sporting', "lorenzo's", 'gang-bullseye', 'xeroxed', 'theologians', "`face/off'", 'burnell', 'dewey', "high'", 'infurating', 'sitcom-esque', 'undetected', 'gaffes', 'obssessed', 'guardia', 'confronting', 'ancestor', 'gentleman-at-large', 'tropic', 'inclination', "detective's", 'twenty-two', 'accomplishments', 'barrie', 'oaf', "kaaya's", 'bore', 'enraged', 'all-round', 'jackman', 'action/sci-fi', 'schmoozes', 'friends', 'multi-', "shyamalan's", 'pedigree', 'undeniable', 'ever-contrary', 'offensiveness', 'winn', 'divisible', 'killers', 'citizens', 'loudness', 'movie/road', 'samples', 'auster', 'asians', 'life-sized', 'adaption', 'sandler-annoying', 'mccleod', 'denouement', 'bombarded', 'kickoff', 'goody-two-shoes', 'dostoevski', 'mckean', 'jeep', 'blasting', 'unlovable', 'brighten', 'jurisdiction', 'pirates', 'dispatched', 'dreamt', 'boardwalk', 'inteligence', 'ninety-seven', 'liquid', 'believed', 'co-stars', "cup's", 'afro-americans', 'tibbs', 'nation', 'arrrghhh', 'hallucinates', 'sow', 'lovers', 'mini', 'journalist', 'quibbles', 'holes', 'off-limits', 'leland', 'unhealthy', "dafoe's", 'dalliances', 'thief-the', 'worse--stick', 'sinead', "paulie's", 'deft', 'dogs-could', 'tail', 'sinise', 'filling', 'full-length', 'wider', 'encased', 'christine', 'glamorous', 'berkley-like', 'cleave', 'poignant', 'big-breasted', 'childhood', 'flashy', 'severely', 'oppression', 'ineffective', 'pivot', 'petter', "regan's", 'revised', 'blanc', 'triumphs', 'investigator', 'chameleonic', 'fits', 'jolted', 'ranted', 'bawls', "'film", 'traces', 'merges', 'depletion', 'understands', '20-something', 'kingpins', "odyssey'", 'grasping', 'helium', 'jonah', 'prophets', 'afloat', 'liam', 'quincy', 'facto', 'barkeep', 'frank', 'rearrange', 'crud', 'lowest-common-denominator', "white'", "li's", 'multi-form', 'mid-70s', 'weaponry', 'hyper-tense', 'golly', 'priam', 'byways', 'threateningly', 'mctiernan', 'begbee', 'tshombe', 'sneers', 'fetching', 'thunder', '000-foot', 'cadre', 'blitzstein', "spawn's", 'glitter', 'prosthetic', 'plate', 'professors', 'breakneck', "luhrmann's", 'tribunal', 'ballisitic', 'ii--the', 'womanhood', 'charizard', 'observe', 'soaks', 'sheltered', 'betty', 'promenade', 'kelley', 'proverbial', 'stiles', 'depthless', 'uniqe', 'permanence', 'cokehead', 'pulse-emitting', 'brownlee', 'unintended', 'puppeteering', 'seen-it-all', 'efects', "supernatural-entity-takes-over-boy's-body", "gilligan's", 'boyle', 'manners', 'soho', "bishop's", 'mocked', 'sleeping', 'paste', 'selfish', '_last', 'tambor', 'sketches', 'network-affiliate', 'director-george', 'nausuem', 'hard-to-believe', 'denominator', 'favourites', 'exude', 'technology--the', 'rescue', 'holiday', 'heartwarming-nuzzly-follow-your-dreams', 'type-casted', 'sliders', 'suit-and-tie', 'zimmer', 'honeymooning', "jia-li's", 'bejesus', 'icelandic', "day's", 'usaf', 'acrobatic', 'bedpost', 'harrassing', 'film--', 'delores', 'danson', 'unjustifyably', 'casual', 'bested', "newt's", 'dermot', 'salary', 'sex-for-pay', 'denounces', 'decks', 'manicured', 'rebelled', 'unites', 'quotation', 'handing', 'on-the-job', 'shoveller', 'discloses', "amazing's", 'commotion', 'coworker', 'lengths-no', 'squirmy', 'over-wrought', 'julianne', 'freewill', 'gabfest', 'step-away', '#6', 'hughton', 'crystal', '66', 'dyed', 'bont', 'edith', 'over-processed', 'ziembicki', 'theme/theory', 'pikser', 'butcher', 'vandalism', 'penetrated', 'corner', 'thomson', 'roshomon-like', 'didn\x12t', "schiff's", 'clinging', 'obstetrician', 'floyd', 'recipient', 'tinder', 'two-by-four', 'decapitates', "larter's", 'placements', 'cadaverous', 'mykelti', 'manuals', "yang's", 'catagory', 'kidnapped--she', 'potency', 'kafkaesque', 'guest', 'equaled', 'atcheson', 'amok', 'neil', 'accusations', 'aviators', 'dagger-edged', 'bloody--film', 'disarm', 'turnaround', 'elliot', 'igniting', 'tbwp', 'recent', 'tee-shirt', 'urges', 'carters', 'morale', '1987', 'garnered', 'divergent', 'chockfull', 'love-struck', 'complemented', 'leading', 'zane', 'director/actor', 'shoot-first-then-shoot-some-more', 'low-powered', '$12', "street'", 'princess-guarding', 'lesson', 'plain-cheese', 'voodoo', 'judi', 'show-stopping', 'feign', 'detractions', 'sherriff', 'body-strewn', 'surge', 'ronnie', 'hart', 'doubly-cast', 'ugliness', 'drive-ins', 'detests', 'manipulative', 'feurerstein', 'luketic', 'digitalized', "payback's", 'worships', '_breakfast_', 'jerry', "'original'", 'short-cropped', 'striptease-an', 'inconsistently', 'assitance', 'forgot', 'easy-going', 'billionth', 'parsee', 'bags', 'camera-work', 'fake', 'karnstein', 'presumed-but-never-proved', 'broke', "marcy's", 'obvious', 'once--and', "theater's", 'protests', 'show-yet', 'bakersfield', 'eclipsed', 'plumbing', 'portrayed', 'formulates', '58', 'coleman', 'insurance', 'bop', "client's", 'paus', "roger's", 'heartthrob', 'squirtle', '70s/early', 'assistants', 'indie-minded', 'kulich', 'lust', 'perps', 'aggravatingly', 'tickets', 'kuosmanen', '20s', "hopper's", 'non-matinee', 'expunging', 'misbegotten', 'skeletal', 'absorb', 'eisley', 'nobler', "'fiona", 'pitted', 'lazard', 'aplenty', 'iii-woefully', 'snowdad', 'delivery', 'daleks', 'err', '939-2', 'consistent', 'amateurish', 'epicenter', "so-generic-and-predictable-it's-beyond-ridiculous", 'plusse', 'bereaved', 'quarter-century', 'swoon', '`as', 'ciphers', 'hyperdrive', 'thousand', 'flat-out', 'bussing', 'pointy-breasted', 'constrained', 'ebonics', "'unusual", 'gisbourne', 'reeled', 'lukas', 'wahlberg', 'hime', 'strip-mining', "bobby's", 'percy', 'co-writer', "stern's", 'attendees', 'eyebrow', 'dark', 'freakiest', "'poking", 'irritates', 'damme/wesley', 'repackaging', 'quirky', 'stunt-butts', 'humvee', 'noodle-headed', 'heroine', 'sword', 'easily', "nun's", 'soze', 'knot', 'immortality', 'anthology', 'swinton', 'foreigner', 'liza', 'gardner', 'ambiguousness', 'ensnared', 'lesbian', 'actor/waiter', 'dignified', 'top-flight', 'afa', 'attentive', 'sized', 'immortalizing', 'radicals', 'ridicule', 'tatoulis', 'sense--and', 'bastardizing', 'coulda', 'sub-mediocrity', 'creepy-looking', 'compunction', 'madame', 'disclosing', "brooks's", 'choosing', 'adjusting', 'one-dimensionally', 'animal-men', 'gazing', 'mariner', 'slow-starting', 'bark', '_scream_2_', 'countries', 'notes', 'flippant', 'brunt', 'luthor', 'livelihoods', 'crowd-pleasing', 'pisspoor', 'tutee', 'subjects', 'nervously', 'straightman', 'robicheaux', 'incorporation', 'november', 'complicated', 'collosal', 'cringe', 'thou', 'jacobi', 'admittedly', 'streams', 'bossman', 'simmer', 'identital', 'ymca', 'pan-and-scanned-out', 'deconstruction', 'contrived', 'work-loving', 'eccentricities', 'cryogenically', 'smiling', 'produces', 'stewing', 'computer-generated', 'razzie', 'happy-for-all-parties', 'arnold', 'botched', 'bluntness', 'aretha', 'sexed', 'soul-searching', 'harsh-seeming', 'stephane', 'donato', 'mini-sub', 'leno', 'uber-rich', 'highly-anticipated', 'petrice', 'traditionally', 'jeanine', 'consumer', 'greets', 'fee', 'misunderstand', 'mastering', 'henchlady', 'attracting', 'ogden', 'most-powerful', 'badham', 'recites', 'hmmmmm', 'cross-cuts', 'outlined', 'mets', 'reputation', 'saturn', 'big-undersea-serpent', 'hair-removal', "video's", 'blunted', 't-shirts', 'sourpuss', 'finely-tuned', 'storm', "raimi's", 'seperation', 'chemistry', 'clockwatchers', 'bonnier', "breadwinner's", 'peebles', 'lois', 'policewomen', 'diametric', "andreas'", 'gratuities', 'impressive-as-ever', 'andrzej', 'knightdom', 'sp', 'humorously', 'one-tenth', 'lackadasically', 'outline', 'birnbaum', 'hayashi', 'un-sanitary', 'reductionism', 'mankind', 'sprayed', 'traceable', 'barroom', 'breathe', 'bay', 'grittier', 'optimum', 'magali', 'wonsuk', 'goldsman', 'asked', 'faced', 'limb', 'ekland', 'olek', 'fishnets', 'dictator_', 'waltzing', 'abdomen', 'father-daughter', 'betrayed', 'infuriated', 'milk', 'wives', 'swank', 'bombers', 'disfigured', 'mekhi', 'babysitter', 'smart/sassy', 'uncorking', 'hordes', 'paulie', 'shoals', 'eruption/fire/earthquake/explosion/tsunami/tornado/meteorite', 'annabel', 'holland', 'looping', "chris'", 'abundant', 'separation', "takin'", 'consistency', 'genericized', 'danny--here', 'extract', 'entails', 'hidding', 'labor-inducing', 'composers', 'ounce', 'vallick', 'beyer', 'afterglow', 'eponine', 'posturings', "'acting'", 'cheekily', 'pop-up', 'royally', 'marital', 'warm-n-fuzzy', 'engage', 'collections', "to'", 'sommers', 'sorvino', 'hayek', 'stream-of-consciousness', 'conversation-heavy', 'bostonians', 'chart', "attanasio's", 'towns', 'blow-ups', 'graphic', 'conditions', 'spiritedly', 'ingrains', 'cannibalize', 'watercolors', 'vinterberg', 'hannon', 'bodes', "dome'", 'alakina', "west'", 'bottin', 'intentionally', 'studies', 'yuk', 'hardest-hitting', 'linc', 'animate', 'andree', 'knocking', 'ennui', 'magical', 'kirshner', 'gecko', 'gamorreans', 'vibrate', 'calais', '2000', 'prescence', 'war-torn', 'cheater', 'ulcers', 'concoctions', 'soundman', 'attire', 'mufasa', 'prettiest', 'benny', 'passenger-side', '1847', 'step-for-step', 'sly-style', 'polo', 'strictly', 'dogme', 'matchmaking', 'larger', '30ish', 'non-traditional', '[kjv]', "young's", 'hitchhike', 'goddamn', 'kyra', 'burdened', 'inimitable', 'cakes', 'cons', 'hamming', 'eilbacher', '1554', 'patrol', 'golino', 'throws', 'tongue-and-cheek', "anney's", 'mercifully', 'wisecracking', 'datalink', "youngest's", 'near-constant', 'prize', 'liao', 'awakener', "guy'", 're-watching', 'assumption', 'preludes', "'plague'", 'innocence', "diaz's", 'jane', 'supreme-bozo', 'tossable', "schlondorff's", 'legible', 'jb', 'vain', 'wellness', 'quicksilver', 'spontaneity', 'bustle', 'sometimes-tedious', "whistler's", '26-year', 'interiors--in', "bimbo's", 'jakob', 'leftovers', '_the_last_days_of_disco_', 'overpaid', 'touchstone', 'decent', 'born-again', 'widmark', "1972's", 'earthworms', 'robes', 'hiss', 'mondays', "videotape's", 'atlantis', 'remembering', 'pastel-colored', 'dick_', 'karate-kicking', 'purports', 'grandest', 'valentine', 'drastic', 'favorties', 'armful', 'fuller', 'trysts', 'probable', 'host', 'interact', 'crank', "hoover's", 'embarked', 'bible-reading', 'parable', 'lament', 'ira', 'creators', 'prosecutor', 'incredulity', 'excitable', 'berry', 'sacrificing', 'rewarded', 'flourish', 'admiral', 'assasination', 'luciano', 'complications', 'hero--or', 'jo-jo', 'movies--like', 'hauff', "timer'", 'optimism', 'singer/alcoholic', 'animatronic', "aiello's", 'paraglider', 'empathise', 'purportedly', "wcw's", 'revelation/twist', 'passed', 'bios', 'tuning-up', 'moronic-looking', 'class-a', 'municipality', 'aficionado', 'smothered', 'spoon_', 'putty', 'stylewise', 'proximity', 'global', 'saxon', 'misfired', 'noblewoman', "y'all", 'profound', 'liane', 'isaacs', 'push', 'londoner', 'godzilla', 'self-contradicted', 'walloping', 'contracting', "cortino's", "ambrose's", 'confinement', 'well-made', 'crops', 'anthrax', "lords'", "titta's", 'le', 'shoes', 'marky', 'plume', 'extreme', 'rumble', 'comprehending', "'chasing", 'studly', "winters'", 'cam', 'intimidated', 'song-and-dance', 'westerner-in-peril', 'roebuck', 'chen', 'reeked', 'vengeful', 'giamatti', 'mismash', 'debutante', 'right--frighteningly', 'dupes', 'much-publicized', 'conceptually', 'multimillion', 'crimson', 'mao', 'abruptly--', 'streetwalker', 'bradford', 'pleasurable', 'caruso', 'isn\x12t', 'one-liner', 'sylvie', 'centennial', 'subconscience', 'lawyer-in-training', '25-year', 'lecherous', 'disintegrate', 'countrymen', 'gates', 'cullen', 'whalberg', 'testicles', 'brooms', 'mission', 'kick-off', 'relative', 'nancy', 'dads', 'burgon', 'excommunicated', 'pennsylvania', 'scalise', 'alessandro', "he'll", 'no-brainer', 'depraved', 'enough-', 'king', 'tykwer', 'point-of-view', 'suburbia', "hittin'", "n'bushe", "run'", 'grand', 'non-stereotyped', 'slayers', 'dewitt', 'christy', 'snow-capped', 'vicious', 'merhi', 'hitch', 'aguilar', 'lawns', 'purpose', 'weissmuller', 'uncomplicated', '_mortal', 'friendships', 'opus', 'brashly', 'godard', '`something', 'traits', 'twenty-five', "chuckie's", 'ontario', 'highwaymen', 'duplicate', 'a-trois', 'plot-driven', 'appalls', 'uninitiated', 'anabelle', 'misdeed', 'ffing', 'possesion', 'out-of-print', 'swaggering', 'bright-hot', 'sausalito', 'double-crosses', 'toghether', 'soons', 'hala', 'exacting', 'elbow', 'manager', 'rein', 'groaning', 'forty-eight', 'prostitutes', 'tenor', 'forged', 'premier', 'commentators', 'scene-stealer', 'closed-off', 'examine', 'drug-crazed', 'compacts', "this'll", 'vega', 'sympathizers', 'obi', 'fang-fest', 'cat-and-mouse', 'participated', "lapaglia's", 'intuitive', 'omission', 'fops', '\x14', 'enchanted', 'vandalizing', 'noted', '50-year', 'illusionists', 'reader', 'it92s', 'panavision', 'retired', 'quirkyness', "perkins'", 'plaything', 'writihing', "keitel's", 'not-so-brilliantly', 'educational', 'respectfully', 'rusnak', 'whereabouts', 'diligence', 'thematic', 'elmore', 'paperwork', 'lumbering', 'waterworld', 'chain', 'flanked', 'radford', 'suckered', 'scripts', 'buddies', 'myers', 'uphill', 'sauce--the', 'mcclements', 'transforms', '6/10', 'ocp', 'latches', 'equally', '20-year', 'nannies', 'other-worldly', 'verse', 'hyper-capitalism', 'docudramas', 'wastes', 'repetitions', 'puffs', 'name&quot', 'goliath', 'addled', 'slyly', "fasano's", 'crusher', "greene's", 'berkley', 'rye', 'resolves--surprisingly', 'menage-', 'expressionless', 'way-too-convenient', 'rushing', 'responsible', 'haig', 'beelzu-babe', 'dumped', 'belgians', 'ya', 'outback', "campaign's", 'rables', 'prositute', 'lampoon', 'reject', 'multiplex', 'aborted', 'queued', 'skeeter', 'schwarzbaum', 'overtly', 'cinematically', 'nagging', 'vaudeville', 'chivalrously', 'huffs', 'questioned', 'stevie', 'splenetik', 'playboys', 'bared', 'seige', 'considerably', 'mutiny', 'hurl', 'likens', 'sewell', 'smallish', 'housekeeper', 'sweet-eerie', 'voicing', 'murdered', 'serb', 'nutter', 'snuggling', 'rivera', 'inordinate', 'fund', 'saturating', 'acquit', 'cassidy', 'penalties', 'stupidly', 'burned', 'flare', 'fleshes', 'wearing', "'we", 'filmgoing', "scaramanga's", 'whiz-quiz-kid', 'infiltrated', "fantasy's", 'immigration', 'emigrating', 'governments', 'traumatized', 'bullies', 'cornered', 'imperioli', 'hutt', 'well-run', 'comprising', 'warda', 'strutting', 'hottie', 'interrelated', 'haim', 'wistfully', 'husbands/boyfriends', 'etienne', 'challenged', 'pritchett', 'shrug', 'restauranteur', 'productivity', 'hornby', 'not-inconsiderable', 'strung-out', 'acceptable', 'quasi-incestuous', 'abound', 'harmful', 'sundered', 'bacall', 'wanda', 'approachment', 'two-fold', 'soft', '300', '15-foot', "paltrow's", 'jawbreaker', 'sardonic', 'localized', 'bushy', 'premieres', 'marsden', 'beatific', 'bess', 'allure', 'computer-driven', 'tediously', 'jada', 'greek', 'detriment', 'audience--in', 'defecated', 'designed', 'loyalty', 'gunpowder', 'leplastrier', 'weaver', 'baby-faced', 'tickle', 'revving', 'jean-pierre', 'wo', 'mystique', 'assume', 'brown', '`white', 'juncture', 'castleton', 'priest', "'gothic'", 'nonsensical', "'script'", 'apparition-like', "clooney's", 'phils', 'newer', 'paralleled', 'exciting', 'zhou', 'caberat', 'crews', 'contraceptive', 'reynaud', 'superfluous', 'capture', 'boner', 'kunz', 'nauseous', 'trot', 'skyler', "skipping'", 'vivid', 'debbie', 'cult-ish', "marlowe's", "shaud's", 'dullest', 'mckinney', 'starships', 'rogaine-nightmare', 'fresher', 'gimmicks', 'stoker', "towne's", 'semi-lighting', '`loose', 'needing', 'dragged-out', 'believes', "friedman's", 'sant', 'universe', '$100m', 'candles', 'stepson', 'kio', 'meng-hua', '12-minute', 'gen-xer', 'hospitable', 'lends', 'night-sky', 'shigeta', 'conducting', 'sabotaging', 'barnyard', 'kadosh', 'points', 'look-alike', 'wexler', 'recorder', 'booming', 'deveu', 'achieving', 'ghandi', 'mountainous', 'grooms', 'royalists', "doesnt'", '127', 'unproven', 'bum', 'deep-seated', 'paradise/prison', 'fierce', 'voice-overs', "federation's", 'news', 'amid', 'jimkendrick@bigfoot', "verhoeven's", 'lebanese-american', 'side-effect', 'contributions', "keith's", 'schwartzeneggar', 'banned', 'petievich', "korda's", 'horta', 'dinosaur-sized', 'resource-strapped', 'hierarchy', 'show-and-tell', 'disembowelments', 'highways', 'till', 'diseased', 'infirm', 'semi-serious', 'high-wire', "qui-gon's", "avary's", 'self-assured', 'clowning', 'breathes', 'piaf', 'jews', 'staple', 'lamp', 'rocketeer', 'scales', 'tbn', 'big-shots', 'benton', 'disjointment', 'atheists', 'unpaid', 'needed', 'arthritis', 'drains', 'plaintive', 'throaty', 'soundly', 'obstetrics', 'understudy', 'zeal', 'botulism', 'alcatraz', 'strap', 'dead-in-the-water', 'mathieu', 'interstellar', 'sues', 'relocated', 'bubbling', 'invisible', 'gratingly', 'inclement', 'philips', 'kissy-face', 'bobbly', 'glob', '36th', 'contracts', 'saddles', 'remember', 'shootout', 'palentologist', 'kujan', 'payback', 'maintenance', 'latex', 'sniggering', "sciora's", 'eccentricity', 'vagrant', 'parasites', 'protein', 'participant', "buzzcocks'", 'strapless', 'indiglo', 'tea', 'mandella', 'conjunctions', 'daft', 'storm-swept', 'shrek', 'choreographer', 'timecop', 'bold', 'gigabytes', 'rebellion', 'handphones', "1965's", 'multi-colored', 'dell', 'riviting', 'incessantly', 'sparks', 'white-owned', 'near-impossible', 'cordless', 'pre-fabricated', 'straightened', 'subvert', 'hun', 'lamest', 'stream', 'bunton', 'plow', 'snowmen', "'f", 'brean', 'international', 'immoralistic', 'child-friendly', 'predictability', 'hollywood-ian', 'speak', 'escapism', '___', 'probation', 'lurid', 'payed', 'chemo-electric', 'bones', 'devout', 'goon', 'celebs', 'mayersberg', 'just-deceased', 'garofalo', 'complexly', 'fireflies', 'crustier', 'convulsions', 'gutlessness', "headley's", 'kettle', 'rehearsel', 'placid', 'steeped', "'popcorn'", "elliot's", 'much-talked-about', '607', "rule'", 'defies', 'charges', 'marisol', 'acknowledges', 'meowth', 'orderly', 'died', 'substantial', 'nerve', 'inheriting', 'recoup', 'subsumed', 'a-go-go', 'commodities', 'aims', 'decisively', 'computer-animated', 'indelible', 'monitor', 'backstory', 'surplus', 'visionary', 'duked', 'tattooed', '118', 'whispering', 'untrue', 'hides', 'lassie', 'smooch', 'saved', 'lighweight', 'husbands', 'flexible', 'playtone', 'apparent', 'overreacts', "sigel's", 'amercian', 'imaginations', 'balletic', 'vci', 'botching', 'unfilmable', 'ex-beau', 'guileless', 'over-extended', "love'", 'ned', "money's", 'slim', 'head-to-head', 'arbitrarily-titled', 'omelet', "disaster's", '_soldier_', 'widows', 'carrey', 'blindly', 'machinations', 'unthrilling', 'tv-conversions', 'mid-august', 'celebrity-or', 'ferretti', 'edict', 'flower-arranging--in', 'zephram', 'mid-week', "hawthorne's", 'markedly', 'boozed', 'mic', 'undermining', 'handsomely', 'shocks', 'elders', 'humpback', 'offset', 'barton', 'staged', 'ayers', 'airline', 'sperm', "actor'", 'redeeming', 'rank', 'startling-but-effective', 'styling', 'moranis', 'easily-angered', 'workmen', "llewelyn's", 'helen', 'instruments', 'keg', 'disagreeable', 'bruise', 'statistics', 'hamper', "chocolat's", 'iconography', 'termites', 'stung', 'weinberger', 'sisters', 'gramercy', 'procedural', 'axe-wielding', 'wherewithal', 'seen--little', 'sook-yin', 'rimmer', 'drifted', 'hefner', 'pseudo-father', 'fellows', 'slathering', 'counts', 'lingered', 'botched-robbery', 'fun--oh', 'nita', 'eighty', 'sand-twister', 'ewan', 'raeeyain', "ashley's", 'paled', 'serviceman', "lars'", 'thumping', 'goal', 'bratiness', 'colo', 'notable--and', 'guiler', 'sullivans', 'wuhrer', 'johns', 'bonitzer', 'operatives', 'fronted', 'whimsical', 'allowed', 'bubbles', 'shagwell', 'squall', "'exciting'", 'billboard', 'actualy', "'10", 'flaw', 'intoxicating', 'jam', 'bald', 'unraveled', 'get-ups', 'worm-eaten', 'overdrawn', 'nut-cake', 'sydow', 'clanking', 'schreiber', 'housemaid', 'falk', 'only--reason', 'mojorino', 'malko-visits', 'azaria', 'chong', 'wisecracks', 'inconsequential', "doll's", 'barbecue', 'cul-de-sac', 'demoralised', 'welch', 'rebuffed', '000-acre', 'bottles', 'incarcerated', 'realistic', 'corporation', 'plausible', 'girlish', '=20', 'pantolianto', 'mouth-watering', 'flicker--a', 'ioan', "beach--yang's", 'astonishing', 'jump-in-your-seat', 'shirley', 'lowering', '1896', 'fewest', 'level', 'mismatched', 'penal', 'guile', 'grubby--but', 'quick-fire', 'foo', 'coarsely', 'risk-taking', 'finals', 'all-time', 'location', 'jonesy', 'heer', "teen's", 'ry', 'nightspot', 'frenchie', 'jason', 'arroway', 'zappa', 'nicole', 'occurs', 'sales-pitch', 'mixed-up', 'golf', 'extravertedness', 'hooks', 'journeyed', 'staring', 'consigned', 'gosselaar', 'cross-cutting', 'speculate', 'jingle', 'skidded', 'wouldn=92t', "samantha's", 'triggered', 'slicker', "ac/dc's", 'frequent', 'near-future', 'fields', 'forty-five', 'amity', 'raise', 'shock-therapy', 'speaking', 'katana', "newton's", 'cathedrals', 'chest-beating', 'bitten', 'grosse', 'eiffel', 'i-2', "neighbor's", 'pour', 'bit', "robin'", 'vogue', 'crimefighting', 'tang', "loser's", '`homage', 'drama-its', 'ostensibly', 'selecting', 'out-smarted', 'sensuality', 'doting', 'lizzy', 'hairdos', 'flute-and-wind', 'disconnects', 'onstage', 'evolves', 'klingman', 'slave-owners', 'mixing', 'desparate', 'lunatic', 'woodward', 'sideburns', 'concert', 'farts', 'roderick', 'idealized', 'banquet', 'interferred', 'avrech', 'abdul-jabbar', 'stitches', 'peons', 'sagemiller', 'angst', 'inventive', 'cousin', "sheedy's", 'geological', 'gayle', 'mark', 'sultans', 'named', 'once-removed', 'intense', "tennant's", 'recall', 'half-way', 'snipe', 'integral', 'conceited', 'himalayas', 'breen', 'siren', 'two-and-a-', 'sung', 'condescend', 'decorates', 'afraid', 'gator-wrestling', 'humphrey', 'acrimonious', "hyams's", 'defends', 'expansionism', 'replace', 'plagiarisms', 'illogically', 'pg-movie', 'satisfying', 'rob', 'reminiscing', 'chance', 'scholar', 'star-crossed', 'showdown', 'walther', 'footprints', 'onlookers', 'lives--allows', 'snipes', 'edifice', 'three-dimensional', 'boil', 'shrillness', 'ultra-eccentric', 'fourth', 'scantily-clad', 'spoonfuls', 'torture', 'inspector', 'backlit', 'fingernail', 'hara', 'nullifies', 'pulses', "o'conor", 'man--namely', 'writer-director-star', 'squirrel', '_john', 'armour', 'enthusiasts', 'refreshing', 'impregnated', 'teetering', 'unlikeable', 'orange', 'scratched', 'antiseptic', 'fast-forward', 'laissez-faire', 'slade', 'butter', 'strangelove', '_____', 'townsfolk', 'citizen', 'olyphant', 'french-canadian', '30%', 'frankenstein-films', 'harkened', 'vets', 'realised', "o'connor", 'transitional', 'vague', 'filmy', 'interupts', 'heavily-hyped', 'rams', 'devious', 'bronson', 'anti-woman', 'cece', '@#%', 'thandie', 'exclaimed', 'solomon', 'keenen', 'unfaithfulness', 'oscar-nominee', 'before---and', 'uninterestingly', 'scores', 'darnedness', '_real_', 'relieving', 'protection', 'flips', 'dwi', 'solidify', 'ventura-style', 'though--wahlberg', 'spewing', 'wickedly', 'newly-broken', 'attackers', 'jetee', 'gangstar', 'thirty-foot', 'aramis', 'denies', 'roy', 'duchovny', 'tide', 'idea--a', 'exclusion', 'salted', 'cannes', 'judgments', 'mercaneries', 'sketching', 'andromeda', 'larter', 'aftertaste', 'bondage', 'subdued', "patrick's", 'synchronised', 'smuggles', 'bitch-queen', '$172', "slow-mo's", 'aggrieved', 'clarion', 'inundates', 'goo', 'neophyte', 'flamingo', 'elspeth', 'sneak', 'dilutes', 'dazed', 'lukewarm', 'ordered', 'aimlessly', 'require', 'ins', '-genre', "kick's", 'meaninglessness', 'disadvantage', 'refuse', 'quo', 'delivering', 'tavington', 'munching', 'ravenscroft', 'ex-partner', 'mercury', 'cross-pollination', 'unfolds', 'dirty', 'hardware', 'aprhodite', 'expression', "tatopoulos'", 'varela', 'goodman', 'clarinet-playing', 'nail-biting', 'trap', 'come-hither', 'nc', 'frighten--two', 'ventura', 'brinkley', 'carnegie', "man'", 'objection', 'heartland', 'blessed', 'rebelous', 'festive', 'work--while', "president's", 'anthology-style', 'pervasiveness', 'jeunet', 'misled', 'dryburgh', 'slimeball', 'sets', 'colombian', 'home/dance', 'savoy', 'sanitary', 'judicial', 'expands', 'unhurried', 'several-hundred', 'hijack', 'mano-et-mano', 'first-timers', "supermarkets'", 'percussive', "diego's", 'swarthy', 'communicating', 'lessening', 'massuh', 'appelation', 'mop', 'dissipates', 'barbie', 're-titled', 'highness', 'monotony', 'beekeeper', 'snuffed', 'featherweight', 'doppelganger', 'coterie', 'cordial', 'force-loving', 'hood\x12s', 'dedicates', 'vein', 'daydream', 'indulgently', 'manifold', 'coherence', 'ruby', 'open-armed', 'shih', 'chronically', 'harmonious', 'dude', 'straw', 'unerving', 'kathe', "nygard's", 'sweetness', 'crack', 'congratulations', 'bubba', 'sasha', 'stylized', 'sammo', 'grovel', '`mrs', '3-hour', 'purses', "dierdre's", 'heterosexual', 'achievement', 'mallory', "david's", 'peagreen', 'movietone', 'cosmonaut', "o'shae's", 'gorier', 'connery', 'penetrating', 'babylon', 'dolled', 'exemplifying', 'professory', 'freddy', 'minorly', 'sonnenfield', 'adhesive', 'rocking', 'soothsayer', 'paup', 'pepe', 'kensington', 'overdue', 'sayings', 'regret--the', 'danger/more', 'frightening', 'flaws', 'harvard-educated', 'unreachable', 'pairing', 'realise', "'girl", 'lyne', 'jody', 'knife-wielding', 'mild-mannered', 'bisset', 'experimenting', 'trance', "pimp's", 'princes', 'condition--non-studio', 'sufficiently', 'gwen', "programme's", 'sadomasochists', '#7', 'balance', '117', 'tostitos', 'mambos', "husband's", 'babies', 'hard-ass', 'core', 'regression', 'primed', 'cameraderie', "deceiver's", '4/29/00', 'tiresome', 'voiced-over', 'landon', 'cavern', 'producer', 'ewwwws', 'excavating', 'flanders', "rosaleen's", 'silliness', 'bizarrely', 'muffle', 'structured', 'no-one', 'complaints', 'flamboyance', "forlani's", 'goddam', 'lusted', "grodin's", 'vanilla', 'socialist', 'margoyles', 'solution', 'appropriately', 'dramatize', 'pickup', 'bases', 'confined', 'phenomenas', 'warning', 'newtons', 'moved-and', 'losely', 'tzi', "zellweger's", 'proxies', 'paraphrasing', 'monotonous', 'jip', 'gent', 'string-based', 'delicacies', 'schneid', 'scream3', 'amputation', "riss'", 'pteranodons', 'overdirecting', 'journalistic', 'powdered', 'hendricks', 'rioters', 'soften', "allen's", 'felatio', 'white-fenced', 'empires', 'linn', 'rooting', 'circles', 'julie', 'beckons', 'arecibo', 'possessing', 'conspiracies', 'victory', 'fs', 'high-concept', 'albinos', 'cowardice', "year's-open", 'pre-schoolers', 'blown', 'borrowed', 'ribs', 'hk', 'rafts', 'orbit', 'pollution', 'croaker', 'tryst', 'completely', 'humor-free', 'world-class', 'hitchhikes', 'squeamish', 'millions-esque', 'propose', 'jet-powered', 'red', 'jagged', 'then-technological', 'emerald', 'chopin', 'cassettes', 'pull', 'coo', 'summon', 'contractor', 'pissed', 'topps', 'vllainous', 'tiriel', 'rebuild', 'coda', 'relishes', 'gee', 'globel', 'enjoy', "girls'", '1990', 're-connect', 'freezing', 'carolco', 'tank/prison', 'moon', 'z-movies', 'raffo', '_patlabor', 'shalit', 'ultra-cool', 'roman', 'define', 'abating', 'smuntz', 'verdell', 'lump', 'pampered', 'adjustment', 'prematurely', 'trampled', 'rents', 'dating-and', 'disturbing', 'hans', 'shalhoub', 'llama-herder', 'tempting', 'fowl', 'tongue-tied', 'matchmaker', 'musketeers', 'sadism', '_still_', "shop's", 'fur', 'crevasse', 'dead-set', 'populist', 'fondle', 'indignities', "bing's", 'heidi', "herrmann's", 'head-chopping', 'gentleman', 'strengthening', 'persuasion', 'vacuum', 'flashback', 'jovi', 'grimace', 'rebuts', 'set-pieces--', 'thrust', 'skeksis', 'trademarks', 'sucks', 'oddly-timed', 'ghostbusters', "1977's", 'manny', 'eyelashes', 'beam', 'unforgivably', 'wrecking', 'dextrous', 'mopes', 'rosie', 'nodding', 'belle', 'hovering', 'pap', 'meditations', "lambert's", 'brief-clad', 'dealt', 'overpowers', 'misfit', 'townsperson', "mama's", 'beautiful', 'nosing', 'nightmares', 'afro', 'retailers', 'framework', 'loudest', 'ex-army', "t'aime", 'grumbles', 'wage', 'cutherton-smyth', 'urban-legendary', "holnists'", 'defeating', 'rufus', 'closely', '65th', "achin'", 'prwhen', 'cushing', 'cargo', 'sky_', 'grass', "`showgirls'", 'ill-synched', 'mini-me', 'sadness', "lovett's", 'cohagen', 'icky', 'plantations', 'hanks/meg', 'revenge-driven', 'salvageable', 'abstinence', 'mis-adventure', "palantine's", 'dissects', 'trend', 'soundtrack', 'lure', "pantoliano's", 'vampires_', 'leaguers', 'entertainment', 'recon', 'intoning', 'knicks', 'disadvantageous', 'byron', 'intellectualized', 'hutton', 'disconnect', 'resurfaces', 'laugh-o-meter', 'vi', 'gauzy', 'affair', 'send-off', '20th', 'beautifully', "weir's", 'campiness', 'hip-hop', '_21_jump_street_', '000', 'bound', 'lists', "taxpayer's", 'shakier', 'profiler', 'gasmask', '$400', 'reflexively', 'pointed', 'video', "'hooked'", 'inner-city', 'narrating', 'takeoff', '-jackson', 'anymore', 'superweapon', "vikings'", "gellar's", 'sample', 'go-around', 'hitters', 'rummages', "attendant's", 'vegas-based', 'incarnations', '18-24', '#8', 'exchanges', 'scan', '40s', 'blankety-blank', 'brancato', 're-shot', 'lopped', 'odds', "navaz's", 'yorkers', 'ends--freud', 'resource', 'vent', 'jeni', 'beetles', 'cut-outs', 'gripped', 'filmography', 'left-field', 'dislocation', 'knicked', 'silberling', 'affects', "rooker's", 'divorcee', 'maggots', 'sona', 'uninvolving', "ashton's", "fate'", 'preparations', "ne'er-do-well", 'sculptor', 'writer', 'knock', 'basically', 'fortune-telling', 'christmas-themed', 'johnathon', 'big-underwater-snake', 'ventured', 'middle-class', 'bull-horn', 'wunderkind', 'omens', 'telemovies', 'sibling', 'craziness', 'yuks', '1952', 'webb', 'dickson', 'virgin/whore', 'tackyana', 'innocents', 'souped', 'screentime', "villard's", 'valentines', 'obsolete', 'unnerving', 'unnerved', 'compassion', 'wail', 'scenery', 'erratic', 'yielding', 'inklings', 'idelogy', 'glasgow', 'at--unaccountably', 'billions', 'trivialize', 'cut-from-nc17', 'sustainably', 'traders', 'steadi-cam', 'taghmaoui', 'direct-to-video', 'table', "ball's", 'whelmed', 'one-day-a-week', 'reinserted', 'presenter', 'harvie', 'underlings', 'capote', 'welcoming', 'long-term', 'rarity', 'moron', 'gerrard', 'vr', 'jokes--otherwise', 'sexually-frustrated', 'retained', 'perversion', 'yuck', 'didi', 'closese', "carrie's", 'beatnik', 'world', 'non-believers', 'pickings', 'kwan', 'gonna', 'well-earned', 'greatest', 'rapidity', 'blofeld', 'conspiracy/martial-arts', 'confer', 'mayor', 'bad-enough-to-be-good', 'spungen', 'blinded', 'cycling', 'handily', 'apologize', 'spiced', 'blond', 'twirl', 'anonymously', "donkey's", 'blood-spattered', 'showbusiness', "black'", 'specliast', 'academic', 'mind--', 'date-night', 'hot-shot-young-no-experience-never-killed-a-man', 'fuddy-duddy', 'hamill', 'sidetracks', 'entrepreneur', 'courted', 'drugged-out', 'scatological', "farrelly's", 'critical', 'co-mingling', 'zoom-out', 'cristal', 'rehash', 'foray', 'donna', 'noraruth@aol', 'songcatcher', 'janis', 'videocassette', 'implicit', 'spearing', 'race--starting', 'croc-hunting', 'preserved', 'manslaughter', 'apologetic', "seth's", "`ev'", 'technological', 'waaaaay', 'ciello', 'ultimate', 'unauthentic', 'rent', 'adrenalin', 'tunes', 'mirren', 'ringwald', 'hacked-up', 'bottoms', 'internalizing', 'r&r', 'patterson', 'f--k', 'caleb', 'kaurism', 'snickered', 'variant', 'interacting', 'pyramids', 'wakefield', "michael's", 'liability', 'numb', "dorothy's", 'pumped', 'character-defining', 'restricts', 'disconcerting', 'mobbed', 'additional', 'twenty-cuts-per-minute', 'add-ins', 'incipient', 'club-hopping', 'pick', 'dare', 'harbours', 'conglomerate', "channing's", 'adlib', 'accessibility', 'gravely', 'brahma', 'bestow', 'dalliance', "brean's", 'noun', 'bruckheimer', 'egotist', 'membership', 'deplore', 'genome', 'arbuthnot', 'pooches', 'installation', 'no-no', 'chiming', 'tropics', 'webster', 'on-line', 'parka', 'contentedly', '101', 'estrogenic', 'bene', 'traveled', 'olga', "cliche'", 'turtletaub', 'strategically-placed', 'watchdog', 'expert', 'solos', 'woo-styled', 'sensei', 'ripoff', 'messiest', 'fullest', 'jamileh', 'miss', 'delightful', 'overwrought', 'mcgoohan', '97-minutes', 'eyes-', 'raul', 'care-free', 'shipping', 'exposure', 'workhorse', 'beginners', 'froehlich', 'gunfight', 'campy', "'sci-fi'", 'nada', 'saints', 'hayden', 'mobsters', '70mm', 'amphetamines', 'lasted', 'tristine', 'altman', 'shocked', 'empowerment', 'akroyd', 'unintentional', 'flyover', 'telescopes', 'brawley', 'reports', 'ingredients', 'beguiling', 'synch', 'withdrew', 'dejectedly', 'jean-jacques', 'wan', 'economy', 'bamboo', 'respectively-this', 'insufferable', 'idiotically', 'malko', 'sabbatical', 'appeased', 'monsters', 'harrold', 'down-trodden', 'wonderful--i', "cassandra's", 'lobbyists', 'pittsburgh', 'similarly-themed', 'munches', 'microwave', 'wandering', 'madeline', 'mega-producer', 'motherly', 'bluntman', 'drooling', 'skin-diving', 'volker', 'unwinding', 'one-week-old-blue-cheese-smelling', 'pirate', 'seemed--though', 'well-balanced', 'unfinished', 'malik', 'screen-viewer', 'accumulation', 'crewman', 'fx-heavy', 'annette', 'impoverishment', 'awol', 'labour', "universal's", "randy's", "major'", 'bringer', 'kewpie', 'matty', 'in-crowd', 'horrible', 'tyrannical', 'movie-bedpost', 'trents', 'pongo', 'outdid', 'unarguable', 'fujimoto', 'announce', 'articulated', 'swapping', 'unmet', 'structural', 'pay-per-view', 'shipboard', 'roz', 'precursor', 'damed', 'slips', 'cheap-laugh', 'tone-perfect', 'anya', 'posessing', "farquaad's", 'mutilating', "wife's", 'sufferings', 'layers', 'creeped', 'cheapest', 'danica', 'suppressed', 'wednesday', 'focusing', 'lifeforce', 'crushing', 'homing', 'badge', 'insanely', 'keough', 'brothel', 'recuperate', 'heyst', "gray's", 'seuss', 'increments', 'allen-', 'proverb', '_remains', 'dealing', 'lessons', 'church-the', 'mosquito', 'specializes', '?', 'serling-ish', 'idella', 'kathie', 'refusing', "turner's", 'fims', 'surprises', 'doorman', 'shoo-be-doo', 'routine', 'conducts', "'mr", 'awe', "`darlene's", 'pimps', 'snively', 'underpaying', 'sherbedgia', 'backhanded', 'blane', 'oscar-worthy', 'crucial', 'mai', 'reelection', "'kong", 'damn', 'connundrum', 'skeptic', 'pale', 'cabinets', 'leprechaun', 'ingenious', 'unbalanced', "seattle'", 'marion', 'anticipants', 'lovebirds', 'tollbooth', 'signing', 'prostration', 'junger', 'oohs', 'brightens', 'tackle', 'busty', 'neidermeyer', 'thing-', 'deathcamps', 'button-downed', 'accentuating', 'shots', 'kincaid', 'moron-proof', 'hagerty', 'niggling', 'boosts', 'handsome-looking', 'await', 'gobs', 'credits-', 'dano', 'opium', 'desk', 'upholds', 'injection', 'jokey', 'key', 'passer', 'organizations', 'pawnbroker', 'ridiculously', 'anser', 'shell-game', 'gazed', 'pressured', 'director/writer', 'suffice', 'abnormally', 'trip', "lil'", 'manifest', 'rugged', 'whisperer_', 'rescues', 'genre--_i_know_what_you_did_last_summer_', 'achieved', 'formidable', 'humanist', 'antonioni', 'doesn9t', 'sweep', 'pall', 'culture-specific', 'distinguishes', 'approximate', 'whoopi', 'displayed', 'ranking', 'buys', 'hanako', 'shimmering', 'over-stylized', "niece's", 'pained', 'ravages', 'lumbergh', 'big-name', 'timeless', 'endlessly', 'relinquishes', 'fathered', 'spotted', 'ordinarily', 'backlash', 'one-dimension', 'charlatans', 'soak', 'trunks', 'culture-whiz', 'jacksonville', 'bigfoot-sized', 'clancy-esque', 'squarish', 'haunting', 'mcdowall', "pam's", 'radioactive', 'endings', 'guest-starred', 'unmade', 'creates', 'recounts', 'self-reference', 'columbo', 'resemblance', 'inexcusable', 'habitat', 'mod', 'feel-good', 'viceversa', 'disbelief', 'department', 'middleton', 'alvin', 'derides', 'noir-ish', 'languidly', 'monsieur', 'jemmons', 'andrew', 'months', 'comer', "2'8''", 'ceased', 'foggy', 'practically', 'scum', 'emissary', "mann's", 'obssessively', 'kernel', 'capshaw', "casino's", 'broughton', "avengers'", "booze-drinkin'", "[spawn's]", 'perches', 'prizefighted', 'chatty', "stephane's", 'chatter', 'satire', 'tatyana', 'heading--but', 'volatility', "that'", 'tarp', "bathgate's", 'castaways', 'phenomenal', 'denizen', 'bland', 'oblige', "preacher's", 'prop', 'fruitition', 'stench', 'smarts', 'mind-reading', 'nose', "cane's", 'thompson-based', 'superweapons', 'echoes', 'lowest', "`accidentally'", 'ditto', 'launcher', 'crawford', 'plea', 'oscar-consideration', 'convention', 'old-self', 'iq', 'keital', 'employment', 'innocuously', 'dagger', 'observing', 'tantrum', 'buck-naked', 'splits', 'knowles', "'amusing", 'dominating', 'rushmore', "suvari's", 'carpenter', 'tourism', 'furrow', 'benigni', 'diplomat', "butler's", 'coraghessan', 'army/navy/air', 'lovely', 'cotterill', 'front-and-center', 'sunglass-wearing', 'wowed', 'bleed', 'factual', 'kat', 'captains', 'co-production', 'must-sees', 'pop-in', 'tempe', '6-year-old', "beloved's", 'dzunza', "'steptoe", 'lhermite', 'retreats', 'juxtapose', 'thomspons', 'gazillions', 'gritted', 'horror/sci-fi', 'serum', 'insecure', 'unveiled', 'phoebe', 'practice--margaret', 'migrate', 'presiding', 'promptly', "gugino's", 'lorraine', 'lecture', 'crests', 'abductions', 'etched', 'predicable', 'jake', 'worse', 'boundless', 'elicit', 'nobel', "drag-it's", 'pal', 'dependable', 'hallways', 'cute--try', 'fernando', 'pastiches', 'barest', 'age-old', 'dog-bone', 'estrangement', 'filmmkaing', 'precisely', 'steaks', '150', 'enshrining', 'ornamental', 'cultured', 'offensively', 'silverware', 'pre-existing', "hamlet's", 'unannounced', 'invitation', 'bicycle', 'timed', 'fiber', 'redoubtable', 'probing', 'labyrinth', "o'donnell", 'chia', 'acquits', 'jacket', 'nearest', 'foots', 'ranged', 'kusatsu', "lange's", "--there's", "judd's", 'chipperly', 'overdoser', 'boob', 'inspiration', 'gambling', 'wises', 'timberland', 'farther', 'treetop', 'tagawa', 'swastica', 'spread', 'sleep-walking', 'littlest', 'morbid', 'smarter', 'condemning', 'schoolmarm', 'dampens', 'excellently', 'generalizations', 'pseudonyms', 'devices', 'mulqueen', 'theremin-driven', 'palm', 'tease-a-thon', 'malebolgia', 'lenz', 'donate', 'hidden', 'webber', 'trashy--well', 'bowie-like', 'israel', 'boathouse', 'hook', "men's", 'landou', 'lyrical', 'dissapointing', 'accountant', 'businessmen', 'sunbathes', "returns'", "riggs'", "phillip's", 'trysha', 'soong', 'slapped', 'conflict', 'confidence-lacking', 'prequel', 'chambers', 'races', 'novels', 'movingly', 'creeping', 'acclimatize', 'distinctly', 'objecting', 'exorcised', 'jack-off', 'scoundrels', 'tolerate', 'toe-stub', 'manages', 'patch', 'interestingly', 'ceti', 'multi-purpose', 'trivia', 'turned', 'seductive', 'good', 'channeled', 'pokemon', 'olsson', 'mundy', 'informs', "there'd", "scinto's", 'babyzilla', 'uh-uh', 'auto-arousal', 'puzzling', 'fold', "'vanity", 'ebola', 'goads', 'finch', 'purity', 'tribute', 'miscegenation', 'grisly-faced', 'hard-earned', 'catfights', 'eventuality', 'ultraconservative', 'questionable', 'scene-chewing', 'renamed', 'instigates', 'self-destructed', 'intruding', "movie's", 'inhalants', 'smoked', 'bonet', 'fei-hong', 'betrayals', 'now-escaped', 'paladin', 'sobieski', 'thiefs', 'epics', 'artistry', 'attractiveness', 'organizing', 'gun-runner', 'disgust', 'egos', 'croissant', 'remains', 'flashcards', 'unlike', 'nuances', 'boy-meets-girl', 'sirens', '_would_', 'beattie', 'monahan', 'projectionist', 'nomination', 'megalopolis', 'content', 'lulls', 'obdurate', 'fetishist', 'unease', 'action-comedy', "mile's", 'inaction', 'defoliates', 'assists', '40+', 'persecution', 'intending', 'behind-the-', 'coos', 'gels', 'connelly', 'enchilada', 'cadillac-cruising', 'borrr-ring', "rand's", 'suffocates', 'assemblage', 'ressurection', 'gwyneth', 'trask', 'gains', "betty's", 'paradoxically', 'jackie', 'simpleton', "mccoy's", 'incense', 'hospitals', 'struggling', 'accordance', 'cambodian', 'comical', 'digital', 'armageddon', 'kwouk', 'taunting', 'brother/manager', 'blood-soaked', 'ultra-serious', 'kidnapper', 'extremity', 'goodness', 'cop-corruption', 'ph', 'clements', 'nourishing', 'fashioning', 'bides', 'kasinsky', 'gymnasts', "derek's", 'dilapidated', 'backwords', 'waited', 'largeur', 'gators', 'waterboy', 'commendably', 'histrionics', "tv's", 'tidal', 'ritzy', 'detatched', 'transformed', 'great', 'sworn', '#@%$^', 'good-natured', 'transvestitism', 'raped', 'victoria', 'legally', 'civilian', 'winsome', "mandoki's", 'debilitates', 'persistent', 'vanishing', "carroll's", 'regard', 'duelling', "west's", 'butt-head', 'generosity', 'kids', 'writer/director/hack', 'roommate', 'mickey', 'hyde-pierce', 'broadly-observed', 'hold-up', 'poorly', 'communications', 'fizzgig', 'intelligentsia', 'tantor', 'modernity', 'manipulation', 'shatter', 'gitai', 'victimize', 'slept', 'unzips', 'ballyhooed', 'coroner', 'forgetful', 'ultraconfident', 'goldberg-type', 'eye-catching', 'villain', 'sampling', 'quotas', "levy's", 'donned', 'anthesis', 'parrot', "younger's", 'liu', 'squash', 'roxy', 'sandy', 'wildness', 'sanguine', 'average', 'pig-headed', "hitchcock's", "rose's", 'predetermined', 'insatiable', 'ex-rolling', 'mockery', 'reliant', 'senile', 'disney-esque', 'fu', 'impressiveness', 'budget', 'councilmembers', 'commitment', 'z-grade', 'seizure', 'grimaces', 'analysts', 'map', 'rumours', "vanities'", 'apfelschnitt', 'console', 'alien3', 'bleakly', "henstridge's", 'example--but', "`dragonheart'", 'carville', 'pod', 'scene', 'decently', 'immediatly', 'improvises', 'game-experience', 'schooler', 'enticed', 'rollerblades', "panavision's", 'eszterhas', 'ravishing', 'barcode', 'misogyny', 'chit', 'douchebag', 'comprehend', 'weld', 'fresh-from-jersey', 'consumed', 'hodges', 'mumbo-jumbo', 'exterminating', 'transgressions', 'expects', 'children', '84', 'milked', 'capitalizes', 'terminator', 'boo', 'hand-jobs', 'splitting', 'up-and-comer', 'coaching', 'singed', 'thickened', "secret--he's", "armageddon's", "fichtner's", 'salesmen', 'nerds', 'crudely', 'non-teen', 'schmaltzy', 'emergency', 'giveaways', 'suspicion', 'rape/murder', "gilliam's", 'potboilers', 'spiritually', 'valet', 'venkman', 'embarassing', 'grammer', 'straws', 'kidnapping', 'chameleon', 'careens', 'male/female', 'pagans', "guest's", 'enhancing', 'concocted', 'send', 'belloq', 'becky', 'authorizes', 'terrorized', "scattershot'", 'newark', '`enough', "off'", 'adhered', 'deathly-ill', 'raider', 'secretaries', 'vips', 'fulcrum', "kasdan's", 'non-issue', 'parker', 'undertaken', 'moseys', 'disastrously', 'purchased', 'venerable', 'stallone/stone', 'baptist', 'moviereviews', 'lightyear', 'feeble-minded', 'stable', 'heavier', 'outrageousness', 'padre', 'chazz', 'worlds', 'bach', 'aiello', "maureen's", 'sir', "recall'", 'qu', "africa's", "meyers'", 'golfballs', 'religions', "function--smith's", 'equally-innovative', 'saharan', 'getaway', "et's", 'constricted', 'petite', 'thighs', "redgrave's", 'anal', 'boast', 'gunman', 'impulse', 'manual', 'commanded', 'blethyn', "'taken", 'dotson', 'profs', 'eliza', 'accord', 'speeds', 'rosanna', 'juries', 'sucker-punch', 'acknowledged', 'adventures', "flanagan's", 'stamping', "tristen's", 'slack', 'unreceptive', 'corrupting', 'properties', 'post-shower', 'sethe', 'absence', 'dinner-table', 'mufti', 'fullscreen', "emma's", 'roald', 'weds', 'shorthand', 'covertly', 'luke-skywalker-like', '15-24', 'admirer', 'greenblatt', 'neckbraces', 'mantle', 'snarl', 'ogres', 'acceptance', 'looooot', 'enmeshed', 'relunctently', 'winthrop', "irvin's", 'launch', 'back-street', 'skimp', 'viscous', 'juergen', '200', 'readings', 'improved', 'hairstylist', 'prefacing', "either's", 'chewing', 'yu', 'followups', 'mishaps', 'tighten', 'sherbet', 'mullally', 'polished', "allison's", 'free-thinking', 'inuit', 'heartily', 'badges', 'mindel', 'androgonys', 'assortment', 'butt-kicking', "ganz's", "camembert's", 'mugging', 'challenging', 'powerhouses', 'newly-born', 'explosions', 'kwietniowski', 'conflicts', "cruncher'", 'joys', 'travesty', 'consumated', 'shalhoub---but', 'mails', '30-something', 'once-simple', 'leftist', 'box-office', 'arranging', 'pearls', 'gritty--make', "freud's", 'surface', 'electrician', 'afred', '27', 'invests', 'nigra', 'victoriously', 'neighing', 'shrewd', 'crucible', 'embeth', 'replica', 'goer', 'diamond', 'smog', 'morally', 'show-unrelated', 'drag', 'digressions', 'payer', 'investing', 'rococo', 'hands-why', 're-creates', 'buffoons', 'psychology', 'queen', 'pop', "experience--you'll", 'regretfully', 'supportive', 'godless', 'rudolph', 'trifle', 'processed', 'lime-colored', 'slashed', 'existent', 'nntp-posting-host', 'cabaret', 'light-hearted', 'slaves', 'shudder', 'butchers', 'permeating', 'hypnotist', 'taglines', 'leads', 'oath', 'salute', 'participants', "petersen's", 'beg', 'shoddier', 'congenial', '_fear_and_loathing_in_las_vegas_', 'fabiani', 'capone', 'confidence', 'animal', 'miserable', '1830s', 'time-machine', 'convoluted', "ferrell's", 'polygram', 'post', 'irreversible', 'illogical', 'disable', 'kitten', "'women's", "bubbles'", 'aircraft', "space'", 'booger', "valerie's", 'ryu', 'spider-scorpion-crab', 'craftsmen', 'ends', 'dawkins', 'brokedown', 'disapproved', 'youth-restoring', 'ginty', 'bilking', 'modine', 'horizontal-moving', 'directed--', 'demonstrates', 'cobblers', 'bride', 'compensates', 'curses', 'fong', 'mouthful', 'thugs', 'icebergs', 'nicer', 'christoff', 'hmm', 'purely', "lambs-he's", 'father-in-law', '`gary', 'co-scripted', 'superglued', 'senator', 'kinky', "nobody'll", 'sired', 'archer', 'carpet-pissers', 'showpieces', 'abolitionist', 'ex-junkie', 'marienthal', 'recollections', 'wallpaper', 'bladerunner/robocop', 'max-sized', "brady's", 'sub-', 'hindu', 'yard', 'prinze', 'gradual', 'gregarious', 'laborious', "theresa's", 'stears', '_la', 'dinos', 'capitalise', 'madeliene', 'back--turn', 'jive-talking', 'frosting', 'welder', 'four-minute', "humankind's", 'comedic', "holm's", 'angeles', "'ll", 'ecstatic', '20-foot', 'fall', 'polemic', 'bitchiest', 'howl', 'countercultural', 'scraped', 'pursuit', "'hidden", 'pier', 'unanimous', 'concerned', 'stereoscopic', 'renton', 'chapin', 'glenross', 'distributor', 'romances', 'idolizes', "leonardo's", 'sliced-off', 'yammering', 'doted', 'manager--and', 'uplifting', 'glitzy', 'trimmed', 'superheroes', 'wise', 'fairman', 'ousted', 'schoolchildren', 'sal', 'well-created', 'entertainers', 'junk-bonds', 'utterly-pointless', 'diago', 'narration', 'lasser', 'clocks', 'catered', 'sparky', 'most-eagerly', "alexander's", 'opined', 'travelers', 'stink', 'supermodel', 'galapagos', 'off-putting', 'specifications', 'resigns', 'bartusiak', 'shack', 'impostor', 'nanook', 'architectural', 'personality', 'close', 'oppressing', 'bruin', "'lost", 'networks', 'liverpool', 'sap', 'conejo', 'indoors', 'dressed', 'swintons', "fulci'", 'buildings--a', 'zwick', 'ingeniousness', 'bearable', 'closeup', 'ho-hummer', "samurai's", 'dosage', 'mcdonald', 'phobias', 'pizza', 'ramshackle', "dauphin's", "space's", "`v'", 'composure', 'daemon', 'psychically', 'shrewish', 'starters', 'substory', 'horseshit', 'mullen', 'tod', 'near-certain', "carlyle's", 'titillatingly', 'enviromentally', 'med', 'un-funny', 'recieved', 'toontown', 'center-stage', 'dornacker', 'incriminator', 'bartender', 'newfound', 'transcended', "fiona's", 'never-before-seen', 'camille', 'oedipal', 'optioned', 'harrier', 'imogen', 'period-piece', 'all-but-screaming', 'tricks', 'kinka', 'arabella', 'comrade', 'cesarean', 'pretext', 'genre-shifting', 'loni', 'bewitches', 'pimp', 'jethro', '1600', "patric's", 'leary', 'bonus', 'agape', 'amputated', 'furst', 'douriff', 'irene', 'earning', 'conan', 'miffed', 'kevil', "skeptic's", 'electroshock', 'caned', 'gidget', 'eno', 'unfunny', 'tyra', 'boundaries', 'booby', 'boardroom', 'searched', 'ravera', 'accidents', 'apocalypse', 'hopefulness', 'relished', 'bear', 'upping', 'academy-beloved', 'kilt', 'radiantly', 'chuckle-worthy', 'abrasive', 'ritualistic', 'frills', 'us$150', 'inexplicably', 'cleaves', "bickle's", 'fiberglass', 'touting', 'mindnumbingly', 'cheerily', 'multi-week', "mcconaughey's", 'degrades', 'receiver', 'elevators', 'stereo-type', 'ulysses', 'tenants', 'transcend', 'lonergan', 'shootouts', 'ever-infectious', 'picnic', 'sheikh', 'eastwood--', 'discard', 'mtv-style', 'converts', 'ballroom', 'hot-as-hell', 'chair', '$35', "'eat", 'cruelly', 'tag-line', 'atomizers', 'garland', 'appalled', 'sleepless', 'dignities', 'shines', 'squeeze', 'emission', 'murky', 'inconceivable', 'rambunctous', 'cadavers', 'hitwoman', '4/10', "haskin's", 'geeky', 'paratroop', 'opulence', "devil's", 'attired', 'laxative', 'super-adorable', 'gung-ho', 'men', 'mins', 'three-year', 'nightmarish', 'bonifant', 'glaringly', 'hokey', 'soon-to-be-published', 'hamburgers', 'anchoring', 'meddling', "jr's", 'elsewhere--their', 'swiping', "francie's", 'infests', 'mentioned', 'flowers', 'interrupts', 'marshal', "cindy's", 'straitlaced', 'botanical', 'nathan', 'stubby', 'jude', 'graduate', 'teaser', 'legions', 'beggars', 'rears', 'lever', 'formally', 'ribald', 'rotterdam', 'arraki', 'wazzoo', 'narrow-minded', 'donnell', 'counter-attack', 'homicide', 'drawings', 'laughfests', "brosnan's", "sober'", 'half-paid', 'deviant', 'id-fueled', 'accentuate', 'stubbornness', 'series', 'mandated', 'hanky', 'low-tech', 'deference', 'chomp', 'predates', 'camoes', 'surrealism', 'photos', "headmaster's", 'lila', 'harming', "'80's", 'ankh', "'showgirls'", 'manmade', 'reprise', 'boxing', 'polar', 'keiko', 'resort', 'slice-of-life', 'bordering', 'i+ll', 'abouts', 'unpredictable', 'crowds', 'speaker', 'gradient', 'sleeps', 'hairy', 'dozier', 'mca', "senator's", 'ultra-right', 'doom-and-gloom', 'opportunies', 'setback', 'conquered', 'oblique', 'agree', "plot's", 'vibrations', 'gatlin', 'debating', 'blather', 'character-the', 'proclamations', 'grey-skinned', 'pummel', "directors'", 'damnit', 'smugglers', 'suzy', 'characers', 'earshot', '56', 'fa', "mcmahon's", 'surprising', 'craftiness', 'judas', 'nostromo', 'killings', 'down-and-dirty', 'kiddie-friendly', "gee's", 'coupled', 'rastafarian', 'referred', 'zabriskie', 'apollonia', 'self-aware', '109', 'somesuch', '_offscreen_', 'sympathies', 'thoughtless', 'jamusch', 'roberto', "beart's", 'suggests', 're-experienced', 'depress', 'yee', 'capulet', 'bursting', 'grimy', 'towners', 'manson-type', 'mika', 'juice', 'booster', "'dating", 'sewed', 'reception', 'mostow', "'cheezy'", 'regulars', 'blackmailing', 'before--round', 'look-at-me', 'plan', 'pomade', 'hoodlums', '&#137', "moment's", 'clark', 'affable', 'evades', 'self-destruction', 'boards', 'chemically-induced', 'ahead', 'holder', 'doormen', "play's", 'bela', 'montana', 'replaceable', 'norton', 'vibrantly-acted', 'lovelace', "rocque's", 'challenger', '94', 'fricking', 'rocketman', 'excusing', "labour's", 'vadis', 'feudal', 'lyon', '_before_', 'picard', 'construction', 'gardens', 'character-controlled', 'tours', 'mentions', 'they-might-be-caught', "times'", 'comic-book-gone-feature-film', 'leaping', 'divorces', 'limping', 'self-destructs', 'falsies', 'winsomeness', 'sunrise', 'neurological', 'taller', "simpsons'", 'benj', 'drivers', "'usual'", 'homemade', 'osgood', 'snobbery', 'victims', 'cinemax', 'glam-rock', 'australian', 'belt', 'hagakure', 'swayed', 'lorde', 'geyser', 'holier', 'mesmerizing', 'madrid', 'unifying', 'leafy', 'locals', 'blazing', 'preserves', 'under-written', 'boston-based', 'energized', 'treatment', 'nuptial', 'projectors', 'slippery', 'saloon', 'venture', 'boaz', 'conflicted', 'gloriously', 'treasured', 'central_', 'rocco', 'croatians', 'arguable', 'intimidating', 'unprintable', '_dirty_work_', 'fireplace', 'scary', 'miguel', 'adamson', 'muriel', 'buisness', 'harassing', 'blatty', 'cahiers', 'sporadically', 'jah', 'superbly', 'erosion', 'twang', 'matilda', 'unbroken', 'obscenities', '60-something', 'restraint', 'gluttony', 'duke', 'shatner', 'kirstin', 'counterculture', 'fatality', 'ack', 'hartman', 'co-written', '#9', 'relaxing', 'fluidly', "ellie's", '3rd', 'iguana', 'full-feature', "assante's", "'snoopy'", 'un-dynamic', 'safeties', '1869', 'uncut', 'trumping', 'pierced', 'panties', 'rabbitish', 'tight-lipped', 'halves', 'unassuming', '97%', "gayle's", 'patriarch', 'intellect', 'brown-it', 'cats', 'houser', 'two-gunned', 'swashbuckers', 'capita', 'isabel', 'avenged', 'creatively', 'dead-ends', 'teammates', 'flattered', 'roundish', 'venetian', 'coughed', 'device', 'mcshane', 'nusevic', 'lightness', 'slithery', 'not-for-mail', 'paperthin', 'tyrell', 'unclaimed', 'kitchy', 'roshomon', 'hazel', 'hyperbolic', 'talking-directly-into-the-camera-schtick', 'hui-kung', "delpy's", 'ufo-ology', 'bahia', 'condor', 'joanna', 'panders', 'good-but-not-great', "mancini's", 'super-agent', 'before--almost', 'pickens', 'graded', 'bickford', 'fillings', 'trade-in', 'lez', 'sorossy', 'enveloping', 'scapegoat', 'imcomprehinsable', 'unequipped', 'sank', 'sub-par', 'meetings', 'mini-elwood', 'dead-on', 'paralleling', 'institute', "'moving'", 'brion', 'engrossingly', 'go-round', 'mastermind', 'umpteen', 'borrowings', 'ancient', 'trailer', 'shock', 'huntingburg', 'kalecki', '1912', 'pleasantly', 'elks', 'overcomes', 'idols', 'prancer', 'light', 'oppertunity', '`bringing', 'lamppost', 'obeying', 'molasses', 'hydrogen', 'conscription', 'yong', 'firorina', "team's", 're-mixing', 'disheartened', 'calhoun', 'aristo', 'once-great', "hearts'", 'forcibly', 'un-involving', 'blackwell', 'hacked', '1998s', 'oppressively', 'successive', 'capped', 'orange@centuryinter', 'mc-ubiquitous', 'pf', 'bucking', 'spends', 'arrietty', 'futile-attempt', 'cuckolded', 'marshmallow', "mm's", 'katrin', 'cornerstone', 'offend', 'wisdom', 'solo', 'run-in', 'ultra-expensive', 'dark-', 're-drawn', "cgi's", 'undertook', "beatles'", 'glenwood', 'hardness', 'parisians', 'lowlives', "strocker's", 'snobbish', 'slaver', 'regretted', 'perceives', 'commitments', "badgers'", '_here_', 'hermit', "1960's", 'spacious', 'classier', 'sedated', 'drix', 'headline', 'library', 'weekly', 'story-writing', 'uh-huh', "shrek's", 'pescara', 'drixenol', "singles'", 'leavins', "`there's", 'nekhorvich', 'call', 'once-sharp', 'sendup', 'mcsorley', "aaliyah's", 'brynner', 'drugged', 'drawl', 'administered', '`urban', "diner'", '_monster_movie_', 'postmodern', 'winnebago', 'toyko', 'agonizing', 'palate', 'that--energy', 'dallas', 'laude', 'poehler', 'brenda', "ulee's", 'blinking', 'alfred', 'theatres', 'pocahontas', 'soto', 'east/west', 'dungeon', 'olde', '1hr', 'audacity', 'popularity', "ovation'", 'liven', 'gut-busting', "iliff's", 'tell-all/show-all', 'headed', "cox's", 'iii--though', 'biting', 'revitalise', 'fast-moving', "chicago's", 'destruction', 'year', 'deepened', 'pre-pubescent', 'self-enlightenment', 'cinematically-savvy', 'clinical', 'two-timing', 'assuredly', 'roddy', 'chancellor', 'charleston', 'fruits', 'script-writer', 'lap', "australia's", 'swedish', 'open-and-shut', 'premeditated', 'pop-culturally', 'denouncing', 'teresa', 'perks', 'intimidation', 'drubbing', 'extent', 'antiheroes', 'loquacious', 'jolt-inducers', "book's", 'canoeing', 'disrupts', 'crowd-drawer', 'little-boy', 'binds', 'orgasms', 'pretzels', 'out-of-shape', "crunchings'", 'fusing', 'outweigh', 'submerge', 'exist', 'crimes', 'grants', 'beaten', 'hanoi', 'ohlmyer', 'thigh-high', 'still-lively', 'dummy', 'vallejo', 'lineman', 'drum', 'catalyzes', 'barber', 'admitted', 'baron', 'visualize', 'penalosa', 'devotedly', 'present-day', 'biologist', 'kar-wai', 'retrospection', 'natives', 'manse', 'guise', 'obscenely', 'imrie', 'amidst', 'alanis', "fall's", "worralls'", 'long-missing', 'disc', 'postman', 'banalities', 'hypnotised', 'rubbing', 'little-seen', 'soft-pedaled', 'underweight', 'heralding', "winter's", 'theater--and', 'recapture', 'duvall', 'touchingly', 'forboding', 'ex-brooklynite', 'report', 'inaugurate', 'infects', 'jeweller', 'hub', 'then-colleagues', 'unerringly', 'advisor', 'reteaming', 'nine-feet', 'prejudices', 'rimbaud', 'silvers', 'shudder-inducing', 'cilauro', 'partake', 'tradition', 'jaw-', 'hollywood', 'mini-series', 'what--', 'two--then', 'flea', 'chess-like', 'befriending', 'ruffians', 'sneaking', 'arcadia', 'jovial', 'verges', 'grand-scale', 'velocity', 'lakes', 'understated', 'soundtrack--an', 'experiments', 'rattle', 'hi-larious', 'dracula', 'corridors', 'a--', 'larson', 'edmonds', 'lighter', 'undefined', 'horrifies', '15-week', "bowie's", 'suspecting', 'motif', 'unbusy', 'multitudes', 'unfamiliar', 'mysogny', 'pyroclastic', 'hatosy', 'patriots', 'zesty', 'carefree', 'jackets', 'movie-star', 'ove', "desi's", 'undefeatable', 'sputnik', 'unkind', 'watered', 'power-mad', 'distorts', 'kringle', 'lapsing', 'uma', 'novikov&#137', 'boorem', 'comic-book', 'armstrong', '16-year-old', 'nolstalgic', 'calculations', 'horror-lab', 'bait', 'highlight', 'shunted', 'simpatico', 'unbelief', "losers'", 'lotsa', 'etiquette', 'heroically', 'readouts', 'speed-reading', 'exile', 'ensures', 'ebay', 'sven', 'fidget', "brown's", 'slates', 'safecracker', 'non-acceptance', 'cults', 'garner', "kelley's", 'naturalistically', 'pouty', 'eraserhead', 'counterfeit-looking', 'collectible', 'teddy', 'bresslaw', 'seing', 'romanian', 'restraining', 'emphasize', 'cabel', "'video", 'creed', 'stealing', 'squishing', 'alread', 'snatching', 'split-second', 'epic--handsome', 'roscoe', 'expectations', 'intensively', 'bumblingly', 'star-torturer', 'boolie', 'wizzard', 'weena', 'inspirational', 'claps', 'kret', 'aimless', 'asthma', 'witzy', 'environmental', 'colleague', 'auteur', 'blimp', 'lighthearted', 'lite-rock', 'deserve', "story's", 'retake', 'cancer', 'psychiatric', 'schiffer', '_dragon', 'introductory', 'coalesce', 'tics', 'yellowish', '-her', "2000'", 'mora', 'bastion', 'listens', 'lasers', 'sk', 'toting', 'nullified', 'adorned', '$2', "stephens'", 'movies--a', 'com/moviefan983/moviereviewcentral', 'che-kirk', 'tangle', 'strapping', 'returned', 'randy', 'outshine', 'weighs', 'forbidden', 'jnr', 'dearly', 'you=92ll', 'fever', 'quincey', 'headhunter', 'uncrowned', 'stanwyck-gloria', 'brainiac', 'choderlos', 'frustrations', 'faded', 'domestically', 'cue', 'harboring', 'payne', 'stalag', 'curdled', 'va-va-va-voom', 'second-story', 'three-star', 'firm', 'permissive', 'upbeats', 'law', 'shortens', "rubik's", 'koolaid', 'depiction', 'opportunist', 'denis', 'massive', 'boing', 'outcome-there', 'life-threatening', 'repairs', 'davies', 'hath', 'ariel', 'inflames', 'seagull', 'friendliness', 'exercises', 'high-cultured', 'benigni+s', 'wheat--all', 'stake', 'baldly', 'hybrid', 'loves/hates', 'netherworld', 'shyster', "'il", 'shale', 'shifty', 'pole', 'repulsion', 'blackening-heart', 'paleontology', 'riotous', 'valerie', 'birthed', 'inundation', 'euclidean', 'vhs', 'hamster', 'ardor', 'aesthetic', 'battlefields', 'mower', 'yglesias', 'burglary', 'encyclopedia', 'writers--', 'unctuous', 'motown', '10s', 'reindeer', 'tokyo', 'son+s', 'out--an', 'mixer', "hackman's", 'prokofiev', "amidala's", 'hurrah', 'bleak', 'resources', 'has-been', 'spent', 'toning', 'tabio', 'contrastingly', 'immortals', 'deploying', 'screwing', 'grievances', "hope's", 'hobby', 'wavers', 'negotiator', 'percolating', 'decor', 'degrees', 'applegate', 'controlled', 'pile-ups', 'stimulate', 'purchases', 'marrakech', 'mazar', 'monster', 'streeming', 'afford', 'drawing', 'voluntarily', 'theatre', 'pacific', 'degraded', 'anxiously', 'hounsou', 'charasmatic', 'manchu', 'select', 'hard-edged', 'jing', 'unfaithful', 'we\x12ll', 'norfolk', 'relied', 'viewer-', 'remi', 'spoilers', 'ovular', "baigelman's", 'dill', 'revision', 'homophobia', '`austin', 'honeymoon', 'cross-promotion', 'broom', 'bumblings', 'steer', "north's", 'quick', 'sparring', 'deferrential', 'criticize', 'fiercer', "halicki's", 'four--credited', 'bathes', "original's", 'struthers', 'van-damme', "'deep", 'snaggleteeth', 'goings', 'shirt', 'program', 'ass-kicking', 'skeptical', 'uncountable', 'accustomed', 'transpired', 'franchise', 'hustler', 'peeping', 'dunces', 'arts-type', 'uuuhhmmm', 'company', 'downstairs', 'poirrier-wallace', 'match', '1999', 'recognizeable', 'boyfriend', 'tow-truck', 'jaffa', 'fairness', 'sweater', 'succumbed', 'youths', 'elaborated', 'waddling', 'toughened', 'jarrell', 'nebulous', 'tusks', 'collateral', 'hoisted', "var's", 'inkpot', 'saint', 'scooped', "'98", "chaplin's", 'ore', 'rains', 'disappointments', 'misogynist', 'unstoppably', 'surfacing', "'buy'", 'moot', 'charmed', 'heavily-accented', 'tip-toe', 'inches-from-death', 'dresser', 'tear', 'streetfighter', 'accost', 'teamwork', "dora's", 'obsesses', "smuntz's", 'incriminates', 'billowing', 'equates', 'iconic', 'shuts', 'amadala', "sunday'", 'begining', 'theater-shaking', 'simpletons', 'typically', 'reflection', 'longsword-', 'chun', 'corkscrews', 'centuries', 'meg', 'lolipop', 'shadowing', 'marcy', 'pharmacological', 'sharp', 'steen', 'saddled', "2000's", 'famous', 'chang', 'shot-for-shot', 'splattering', 'lebanon', 'yards', 'hubby', 'real-life', 'brazil', '_hustler_', 'divvy', 'skinner', 'yorick', 'jules', 'cheeks', 'enhances', 'yer', 'far-away', "needn't", 'shipmate', 'kotto', 'heartbreaking', 'smugly', 'preppies', 'holloway', 'bio-pics', 'trashiness', 'sean', 'outtakes', 'dirtiness', 'woodbine', 'mistimed', 'cavemen', 'peet', 'spattering', 'rattling', 'tuesday', "manager's", 'night-time', 'baseball', 'frustration', 'farrell', 'clan', 'deconstruct', 'well-defined', 'squemish', 'quinland', 'mischievousness', '-style', 'jour', 'ratzenberger', 'apathetic', 'greenwich', "mother's", 'fiennes', 'tags', 'callous', "'surprise'", 'nests', 'acrylic', 'uh', "romy's", 'tirades', 'correctness', 'ling-ching', 'torments', 'neatness', 'west', "banderas's", 'x-rated', 'knockin', 'wilted', 'floor', 'cad', 'tasted', 'astin', "gay's", "barbieri's", 'studs', 'enterprises', 'dancefloor', '5-million', 'loop', 'fascists', 'stylishness', 'stalk-and-slash', 'chorus', 'peeling', "enright's", 'frowns', 'kim', 'low-caliber', 'stares', 'tso', 'semi-subplot', 'poled', 'pasta', 'rigorously', 'cliche-ridden', 'jose', 'collins', 'period', 'dismissive', 'ilsa', 'howewer', 'identified', 'cyberpunk', 'waist', 'superhero/action', 'colonists', 'ethos', 'robbery', 'rose', 'dopple', 'doodles', 'oblivion', 'vanity', 'partner-in-crime', 'boston', 'consoling', "'gift'", 'hard-as-nails', 'trades', 'palminteri', 'gorman', 'liable', 'selflessly', "mccourt's", 'we=92ve', 'why---unless', 'whoops', 'hard-pressed', 'requires', 'transvestite/medium', 'dueling', 'mcferran', "each's", 'yellow', 'horner', 'delicately', 'daddy', '`reindeer', 'leaps', 'aniston', 'hi-def', 'redux', 'hommage', 'saw--at', 'typos', 'buster', 'westerner', "'shagging", 'auctioned', 'ensuring', 'hypnotherapist', 'callow', 'slay', 'purrs', 'zidler', 'houseguest', 'tiles', 'sprites', 'gantry', 'befuddled', 'tenement', 'terrorfying', 'indulgence', 'clean', 'abounding', 'dropping', 'goldie', 'ex-cop', 'jane-', 'memoirs', 'resentment', 'contention', '--but', 'punched-up', 'programmers', 'facetiousness', 'split-screen', 'realist', 'phew', 'heald', 'small', 'version--who', 'effectful', 'phallus', 'blatantly-do', 'destroyed', 'omniscient', '8-minute', 'adaptation--in', 'low-budget', 'eckhart', "blethyn's", 'combine', 'follow', 'clinches', 'voting', "'96", 'multihued', 'paddling', "ozon's", 'battiston', 'fat', 'thirst', 'faultless', "20's-early", 'fleeing', 'tilted', 'basis', 'simon', 'rushed', 'livingrooms', 'player', 'acquire', 'cultist', 'niggles', '1973', 'delusions', 'kayley', 'perform', 'chapelle', 'meantime', 'sean/will', 'landed', "banderas'", 'misdirection', 'penitentiaries', 'alias', 'poppins', 'phoniest', 'assigned', 'old-sitcoms-to-movie', 'kindred', 'swan', 'skimps', 'feminine', 'irish-sounding', 'records', 'poorly-motivated', 'casualty', 'head-start', 'zophres', 'ex-cellent', 'jess', 'sarner', 'jordon', 'well-selected', 'shandling', 'fatal', "randle's", '--crooked', 'herky', 'falcon', 'egoyan', 'muddle', 'operandi', 'hookers', 'hyde', 'tossing', 'ignite', 'tests', 'yesteryear', 'destitute', 'tightened', "life'", 'temp', 'power-oriented', 'zany', 'bumfuck', 'adrien', 'bodyguard', 'intensely', "other's", 'supersecret', 'blocking', 'numbing', 'goals--it', 'awed', 'dims', 'bobbitt', "fletcher's", "yakin's", 'insuring', "fred's", "hart's", 'benevolent', 'gigantic', 'grunting', '`john', 'marriage', 'think--if', 'wash', 'legitimate', 'arena', 'focal', 'mccallum', 'glitz', 'racks', 'rounded', 'grungiest', 'agnostics', 'schematics', 'holodeck', 'marshalls', 'knothole', 'dreamquest', 'albania', 'dusting', 'evening', "haven'", 'guilty', 'yakov', "ants'", '`let', 'govenment', 'well-polished', "make's", "annoying'", 'templeton', 'steadfast', 'noodle', 'em', "adele's", 'full-circle', 'stott', 'ear', 'nods', 'differnt', 'languishing', 'dakinah', 'mumps', 'ever-smoldering', "coleman's", '5-5-1', 'pre-occupied', 'deftly', 'often-criticized', 'saracastic', 'pilgrimage', 'housewives', 'resurrection', '51-year-old', "lynch's", 'differs', 'syllables', 'budgie', 'limey', 'juxtaposes', 'converge', 'magherini', 'daredevils', 'mugatu', "1997's", 'pronunciation', '$45', 'imbued', 'burner', 'alleged', 'quoc', 'l-u-v', "suspiria's", 'identifying', 'wilhelm', 'allison', 'trashiest', 'mabel', 'brenner', 'paola', 'permits', 'living-and-breathing', 'pharmacist', 'spielbergian', 'bouajila', 'stroehmann', 'hugs', '40-year-old', 'poignance', 'spirited', 'teary', 'cauldron', 'pakistan--they', 'frizzy', 'kant', 'depravation', 'nora', 'brotherhood', 'preventing', "'companion'", 'troma', 'half-expected', "gere's", 'salesman', 'consecutive', 'inability', 'note-for-note', "guy's", 'good-', 'butch', 'headings', 'return--', 'sci-fi/kung-fu/shoot-em-up', 'gallup', 'impossibilty', 'gyllenhaal', 'beth', 'pointy', 'double-entendres', 'dillane', 'bog', 'slaughtered', 'involuntary', 'complaint', 'jackson/robert', 'sermonize', 'idiosyncratic', 'murdering', 'wristwatch', 'mimi', 'feud', 'maze', 'partridge', 'tenacity', 'brawls', 'resonance', 'page-turner', 'dobbs', 'scribbling', 'marcelli', 'gto', 'jean', 'only--and', 'chestnuts', 'ambassadors', 'plotholes', 'psychic', 'unrightfully', 'sliced', "beatty's", 'flirtation', 'viterelli', 'mushy', 'kasdan', 'timeframe', 'kennesaw', 'teen-dominated', 'fisherman-victim', 'somalia', 'kurusawa', 'pushups', 'self-composure', 'benson', 'overlap', 'repulse', "loggia's", 'scorpions', "chucky's", 'critically-acclaimed', 'winches', 'middair', "kubrick's", 'gambler', 'vexatiousness', 'holographic', 'thankless', 'judah', 'realising', '`out', 'bowman', "lawman's", 'lip-synched', "tassel's", 'yun', 'arditi', 'nod', "owner's", 'un-led-up', 'carrie', 'down-on-her-luck', 'bondish', 'gypsy', 'one-sided', 'boulle', 'indifferently', 'detrimental', 'vosloo', 'hatchets', 'inevitable', 'bartellemeo', 'stabbin-a-plenty', 'unthinkable', 'droning', 'adorn', 'masturbator', 'dummies', '`eye', 'cell-phone-guy', '_saving', 'kasa', "bosses'", 'preciously', 'sports', 'spells', 'pericles', 'avocations', 'speculative', 'hobbits', 'brigand', 'conception', '200+', 'bustling', 'quarrelling', 'wreckage', '1800s', 'brrrrr', 'astor', 'judd', 'inebriation', 'immeadiate', 'lively', 'squares', 'ass', 'posh', 'vainly', 'don+t', 'finely-crafted', 'obviously-inserted', 'rza', 'resulting', 'sentiment', 'propel', 'loves', 'batinkoff', 'phyllis', 'squealing', 'rugrats', 'connections', 'ivins', 'twitch', 'accomplice', 'town', 'sub-genre', 'multi-billionare', 'rubbery', "richards'", 'faction', 'sneezing', 'leisure', 'belittling', "finale's", 'whisked', 'oscillation', 'votes', 'early-warning', 'junkies', 'plod', 'unused', 'simultaneously', 'deluge', 'sisterhood', 'forefathers', 'devouring', 'parodies', '`dead', 'un-noteworthy', "sonia's", 'missiles', 'gullette', 'marionettes', 'iron-on', 'misadventure', 'pornography-related', 'subtley', 'true-crime', 'viv', 'shattered', 'oteri', 'high-power', 'overly-sensitive', 'smiley', 'hayakawa', 'alerted', 'showers', 'restrain', 'lex', "'kidnapped'", 'sorcerer', 'cary', "nicole's", 'low-rent', 'elected', 'soul-collecting', 'entire', 'obliterating', 'camo', 'renna', 'farmboy', 'crippling', 'cds', 'verve', 'estate', 'ewwww', 'stadium', 'relegated', 'orgasmically', 'nagged', 'comatose', "co-worker's", 'elwes', 'spout', 'quasi-utopian', '_2001_', 'bomb', 'racoon', 'permutations', 'uptight', 'yeager', 'destroys', 'singapore', 'beiderman', 'not-so-bright', 'rolf', 'verisimilitude', 'guerrillas', 'resolution', 'model', 'embezzlement', 'authorities', 'illegal', 'diminishing', 'wilcock', 'dwelling', 'homophobe', 'animator', 'maude', 'intrude', 'fumes', "evans'", 'stardust', 'cont', 'foreseen', 'pakula', 'powerlessness', 'advancements', 'basquiat', 'infact', 'career-minded', 'ironically-showmanship', 'societal', 'commencement', 'sacked', 'slightly', 'porthos', 'on-track', 'shuddered', 'slow-moving', 'bonfire', 'inarguably', 'three-quarters', 'umpire', 'selections', 'woo-fest', 'submission', 'human', 'age', 'aft', 'funneled', 'breaths', 'sorna', 'chandler', 'goth-style', "'men", 'enamoured', 'polls', 'strange', 'bickerman', 'hospital', 'norwood', 'sunlit', "simmons'", "'partner'", 'ping', 'entertains', 'cuckor', 'gravedigger', 'brute', 'gold-starved', 'beliner', "groom's", 'stylings', 'would-be-ahabs', 'detachment', 'duckling', 'nintendo', 'semi-naked', 'oliver', 'intervene', 'depends', 'mitch', 'present-time', "hudson's", 'impenetrable', 'one-up', 'slightly-neurotic', "crystals'", 'assignment', "cynthia's", 'samurai_', 'sharp-witted', 'judge-jury-and', 'raping', "pericles'", 'die-hard', "cabot's", 'semi-brainless', 'greys', 'cumberland', 'polar-opposite', 'caterer', 'raskolnikov-like', 'sentiments', 'difficult', 'testosterone', 'splat', 'magically', 'skyrocket', 'environments', 'disposal', 'ditties', 'childlike', 'squander', 'poof', "'judge'", "pushkin's", 'stand', 'soon-tek', 'diary', 'smartness', 'ties', 'growl', 'bodhi', 'ingolstadt', 'descriptive', 'sheree', 'last-place', 'writer/director/comedian', "pongo's", 'side-plots', 'twentysometyhings', 'joy', 'decay', 'coarseness', 'posture', 'langton', 'presses', 'stamped', 'illegally', 'transported', 'script', 'serina', 'condensed', 'participation', 'tents', "'con", '24/12/99', 'thames', 'seaport', 'incestuous', 'laughton', '1935', 'thade', 'jobson', 'droopy-eyed', 'abilities', 'ii', 'mintoro', 'distinquished', 'swipe', 'piousness', 'everglades', 'steadily', 'feasting', 'won', 'perpetuation', 'quash', 'magic_', 'hazing', "kret's", 'michell', 'overstaying', 'orchestrating', '19-year-old', 'applauding', 'soderburgh', 'cheapjack', 'then-sagging', 'outfits--and', 'transport', 'cannom', "o'neil", 'vomit', 'action-movie', 'thorn', 'bullet', 'carpings', 'werewolves', 'laps', 'monkeys', 'stirling', 'keanur', 'd-fens', 'ancient-sounding', 'ren', 'serious-sounding', "ghost's", 'doogie', "'40s", 'plot', 'superstructures', 'rotates', 'splices', "violator's", 'artfully', 'uninteresting', 'waiter', 'scrap', 'shiny', 'healed', 'physics-', 'spat', 'light-plane', 'zapped', "patlabor's", 'post-movement', 'gunmen', 'cow-moo', 'comaraderie', 'unarguably', "'little", 'documentaries', 'unapologetic', 'madam', 'kicker', '_last_', 'hiatus', 'joseph', 'claustrophobia', 'teaspoons', 'sage', 'enteratining', 'suaku', "`surprise'", 'baadasssss', 'high-priced', 'dorm', 'co-screenwriter', 'melding', 'population', 'subtitle', 'exoticism', 'cardinal', 'shore', 'wint', 'bueller_', 'uninterrupted', 'wittier', 'bounds', 'contacted', 'rapaport', 'kieslowski', 'hollander', "pandora's", 'communicate', 'doubted', "successor's", 'disemboweling', 'strumming', 'unconvincing', 'menage-a-trois', 'kinkier', 'injustices', 'millie', 'wilfrid', 'meanings', 'fealty', "voight's", 'goofiest', 'pieces', 'quarrels', 'slushee', '_into_', 'defiance', 'mistaking', 'loud-mouthed', 'appreciates', 'outinen', 'vigorous', 'slurring', 'sickly', 'ant-like', 'screenwriter', "hamilton's", 'clutterbuck', 'dispondent', 'verdict', 'martyr-figure', 'sweet', 'langlet', 'destructions', 'mans', 'wet-behind-the-ears', 'disturbance', 'inspire', 'theological', 'insider', 'buffalo-hunting', 'keeble', 'fatale', 'indicating', "matty's", 'donne', 'opinions', 'sarajevo-like', "cannon'", 'disqualified', 'carousing', 'counter', "'normal'", "joker's", 'marg', "lubezki's", '1/25th', 'floor_', 'daddies', 'hated', 'morita', 'knight', 'throughout-erase', 'edell', "then-he's", 'violation', 'encyclopedias', 'pornogrpahy', 'charity', "schwartzenegger's", 'babs', 'virgin', 'blaze', 'expletive', 'rollerblading', 'soprana-like', 'en-route', 'payroll', 'gagging', 'winslet', 'professionals', 'inferiority', 'hotties', 'clarence', 'transports', 'schools', 'callousness', '[his]', 'rebecca', 'motorized', 'indianapolis', 'sf', 'informer', "waterworld's", 'symbolism', 'superscript', 'grandeur', 'mutton-chop', 'carelessly', 'fueled', 'head-shaving', 'voodoo-like', 'politically', 'groomed', 'klugman', 'wise-ass', 'requested', 'unnatural', 'kindergarten', 'marina', "'straight'", "danza's", 'crapped', 'sociology', 'susses', 'accomodates', 'estimate', 'aspires', "humanity's", 'sitcom', 'walsh', 'befitting', '_basquiat_', '1945', 'overlaps', 'nowak', 'mawkishness', 'narcissistic', 'bramon', 'immin', 'musters', 'otto', 'blechhhs', 'fight-picking', 'ha', 'roster', 'voiced', 'similarity', 'procures', 'megalomaniac', 'targets', 'artful', "python's", 'mummy', 'removed', 'automats', 'greater', 'comment', 'infomercial', 'lucy', '--as', 'hackneyed', 'gawk', 'visibly', 'fantasizes', 'heyday', 'travolta', 'intolerant', 'interview', 'thoughtfulness', 'fi', "writers'", 'ravens', 'designated', 'shampoo', 'political-', '_fisherman', 'stagnate', "entertainment's", 'subliminal', 'adroit', 'salkind', 'undies', 'acid-eaten', '500k', 'astrology', 'dwindled', "bureau's", 'trains', 'schnitzer', 'stoli', 'naoki', 'othello', "good'", 'araki', 'apaches', 'over-the-hill', 'executioners', 'stitching', 'no-holds-barred', 'contingency', "alda's", 'mt', 'tipping', 'pc', 'darkness_', 'sidney', 'pile-up', 'commanders', 'gossamer-thin', 'well-intentioned', 'playacting', 'lanes', 'looted', 'trekkers', '40mins', 'fanciful', 'prodded', 'vice-presidential', 'undisturbed', 'mckidd', 'hmos', 'shame', 'faux-mysticism', 'rot', 'jekyll/hyde', 'to-hell-with-the-law', 'scuffle', 'who--gasp--is', 'mcbride', 'pathetically', 'twinkling', 'validates', 'highlighted', 'tomato', 'terminator-like', 'heralds', 'feuding', 'grabs', 'mustang', 'bushido', "latter's", 'extravaganzas', 'refinery', 'rammed', 'ann', 'cagney', 'unexpectedly', 'boasting', 'despised', 'unholy', 'squirt', 'undemanding', 'humanizes', 'cagily', 'lauper', 'fit', 'plan-', 'mirror-imaged', '[there', 'massage-therapist', "chiba's", 'battle', "captain's", 'freud', 'prospect', 'prefer', 'video-game', 'thinking', 'resisting', "mchugh's", 'kid-friendly', 'crude', 'strobe', 'valkenburgh', 'nobodies', 'genvieve', "bueller's", 'impediment', 'squiggly', 'loud-mouth', 'modernisation', 'third-person', 'member-gone-bad', "helfgott's", 'cantankerous', 'hawk', 'humorist', 'specifically', 'visited', "'bout", '>', "there've", 'advantage', 'break-up', 'ye', 'pronto', 'disappearing', 'super-intelligent', 'snidley', '700', 'uncompleted', '11/20/97', 'unspeakably', 'prissy', 'olden', 'aperitif', 'sixty', 'preface', 'nasty-tempered', 'own--fall', 'turbo', '80-year', 'temping', 'authoritatively', 'rosselini', 'undisciplined', 'hot-shot', 'hil-ari', 'premise--kafka', 'cd', 'culkin', 'surveilance', 'lifted--unauthorized--from', 'tailor-made', 'funnery', 'miracles', 'mercury-encrypted', 'dangerously', 'unsurprising', 'quire', 'extinguisher', 'shyamalan', 'wretched', 'impregnating', 'bijou', 'sidesplitting', 'crazy', 'immigrant', "ned's", 'contestant', 'thereus', 'avital', "'uncle", 'enactment', "junger's", 'herring', 'enigmatic', 'bravado', "inspector's", 'forays', 'jacki', 'late-year', 'insincerely', 'pent', "'patient'", 'reverend', 'klieg', 'american-', 'dissect', 'abiility', 'marius', 'skinning', 'myth', 'faux-psycho', 'minks', 'dabney', 'slam-dunked', 'cubicled', 'sicily', 'actions', 'barbatus', 'lauri', 'occaisional', 'poland', 'retrieved', 'sawa', 'mcgregor', 'dragging', 'bezucha', 'disobey', 'superfreak', '1500s', 'keeper', 'ex-girlfriend', 'short-term', 'stripper', '90-tooth', '//filmfreakcentral', 'documents', 'winnfield', 'expertise', 'sadsack', 'emotions', 'tornado', 'out-of-their-mind', 'ol', 'kar-wei', 'recalling', 'glitteratti', 'change-of-pace', 'prickly', 'untranslated', 'hadnut', 'indignantly', 'sort-of', 'psychologists', '35', 'confiding', 'gifts', 'saves', 'introduced', 'intrigue', "anthony's", 'rosemary', 'simcha', 'tamiyo', 'erase', 'plucky', 'hospitality', 'drug-running', 'creatures', 'english-', 'craze', 'flat-top', "charm's", 'countermanded', 'tragically', 'robe', 'infantrymen', 'yore', 'sans', 'relating', '75%', 'raps', '25%', "burgess'", 'thankful', '_leave', 'things--a', 'cerebellum', 'derring-do', 'einstein', 'creeps', 'caught-', 'grant', 'argonauticus', 'header', 'implications', 'check', 'bureaucrat', "topol's", '`a', 'scatter-brained', 'addict', 'fincher', 'storyboard', 'thanking', "hurricane's", 'departments', 'feline', 'recommendable', 'shiftlessness', 'preferences', 'additionally', 'drier', 'outclass', 'wide', 'animalities', 'funded', 'procuring', 'infidelities', 'resevoir', 'dog-loving', 'suggestively', 'buren', 'neck-deep', 'gordan', 'refugee', "what'sgoingonaroundhere", 'put-upon', 'dunwich', "'three's", 'sessue', 'instructors', 'edina', 'quick-e', 'spanned', 'vlad', 'risking', 'modeling', 'inscrutable', 'davidovich', 'salvati', 'skilful', 'catch-22', 'productions', 'ex-newspaper', '1970', 'geezers', 'mcconaughey', 'delirium', 'cornucopia', 'waylaid', 'collapsed', 'chapters', 'uuuhmm', 'shakespearean', 'jammed', 'jorden', 'abundence', 'tokens', 'mppa', 'horror-trilogy', 'whould', 'imperial', 'incorrigible', "`what's", 'parlor', 'acidic', 'frighteningly', 'outlanders', "sun's", 'yasmeen', 'loathsome', 'underdog', 'demme', 'reprimand', 'swastikas', 'heart-broken', 'converter', 'deterioration', 'aim', 'cutting', 'pachanga', 'thieve', 'originals', 'macy', 'reclamation', 'ego-driven', "`candyman'", 'eyed', 'fibber', 'fund-raiser', 'time-traveling', 'cornfield', "marius's", 'illicit', "delia's", 'opted', 'latent', 'dingo', 'semi-robed', 'biel', 'purging', 'disturbs', 'verhoeven', 'fully', 'texas', 'antarctica', 'rehabilitating', 'kurtz', 'religios', 'admirals', "`knowledge'", 'expanding', 'martineau', 'nazi-collaborating', 'grape', 'disilusioned', 'gapes', 'foods', 'metaphorical', 'busybody', 'tiff', 'sort', 'un-interrupted', 'spores', 'zip', 'decade--', 'czech', 'functioned', 'paolo', 'thirty-five', 'osment', 'sciorra', 'unmarried', 'cg', "houdini's", 'arbitrarily', 'patterns', 'bigshot', 'castro', 'tarts', 'amendments', 'eye', 'tangents', 'tv-movie-ish', 'anti-semite', 'mixon', 'fraud', 'rejoin', 'romanovs', 'incorpates', 'asserting', 'humbler', 'nooooo', 'renew', 'heroines--ariel', 'soft-spoken', 'giger', 'joke---that', 'drug-dealing', "japan's", 'eighteenth', 'porn', 'hesitance', "warhol's", '_lone', "partners'", "robin's", 'finchers', 'massironi', "soran's", 'snorri', 'subscribers', 'ham-handed', 'assassinate', 'elephant', 'play-write', '216-digit', "riddick's", 'jonas^os', 'vastly', 'fitted', 'rhythms', 'symbolised', 'otherwise--but', "patillo's", 'pei', 'spills', 'channel', 'happiness', 'wanders', 'vocation', 'misses', 'leeringly', 'sunrises', 'specials', 'ophiophile', '$115', 'ap2', '30', 'amboy', 'oozed', 'splendidly', 'authored', 'kapner', 'aids', "chase's", 'actors', 'resound', 'delroy', 'ilah', 'stiffest', 'sober', 'religion', 'reflective', 'campus', 'sea-', 'teri', 'shohan', 'breach', 'sapling', 'persistence', 'trapping', 'besson', "election's", 'paltrow', 'salwen', 'self-satisfaction', 'majestic', 'paglia', '`aliens', '107', 'dustin', 'wheezing', 'purple', 'addictive', 'junk-fest', 'beheadings', 'sauce', 'rhett', 'nest', 'executive', 'respected', 'prince-like', 'kenobi', 'tenaciously', 'souped-up', 'potatoes', 'dialog', 'asshole', 'junk', 'arresting', 'discharging', 'closer', "finch's", 'nerdiest', 'set', 'voted', 'inject', "figgis'", 'sketchbook', 'hap', '600', 'friedberg', 'profundity', 'super-secret', 'pre-marital', 'erik', 'brightness', 'everclear', 'bedazzled', 'hippyish', 'goldberg/ted', "brook's", 'jettison', 'thw', "trek's", 'directly', 'locking', 'hues', 'warranted', 'teases', 'eh-dan', 'maximum-security', 'impaired', 'snapshot', 'dimension', 'goodies', 'vance', 'higher', 'drawbridges', 'costs', 'calculatedly', 'big-town', 'miserly', 'chocolat', 'writer/director', 'bumming', 'semi-bright', 'sierra', 'enervation', '1st', 'mass-entertainment', 'divulge', 'blotted', 'hungarian', 'junkyard', 'kick--willis', 'ivy', 'blairwitch', 'cancellation', 'movies--', "'titus'", 'dyer', 'clarice', 'barren', 'snazzy', 'prisoners', "mid-50's", 'nuke', 'action-packed', 'rasping', 'gen-xers', 're-visited', 'coinciding', "deadpan'", 'other--', 'angie', 'benefactors', 'jocks', 'referenced', 'monastery', 'generation', 'labor', 'stating', 'down-right', 'williamson-esque', 'freedom', 'admit', '1949', "goldblum's", "goblin's", "martin's", 'sooty', 'vis', 'ills', 'incognito', 'scots', 'prosper', 'awkward', 'road-comedy', 'beams', 'specter', "bookseller's", '7=good', 'grandmothers', 'sabotages', 'bassett', 'jeter', 'marks', 'breaking', 'unbearable', 'ensemble', 'amusement', "booker's", 'tracheotomy', 'cady', 'sommerset', 'lensing', "hughley's", 'hand-feed', 'hooters', 'barf', 'amends', 'wife-husband', 'bonk', "'in", 'gotcha', 'trench-coats', 'angeles/las', 'fozzie', "boyz'n'hood", 'coifs', 'quintana', 'stagelights', 'ant-eaters', 'shawn', 'month', 'disciples', 'soooo', 'familar', '40-plus', 'fated', 'grandma', 'burying', 'featherbrained', 'heisting', 'equipment', "there're", "taguchi's", 'emigrants', 'experimentation', 'spacek', "leigh's", 'germans', 'ogre', 'mutates', 'ascends', 'cattily', 'skin-tight', 'hat-wearing', 'poirot', 'outlook', 'barbeque', '2065', 'eleven', 'fast-food', 'exceptions', 'retires', 'casablanca', "power's", 'patrolman', 'encountered', 'exemplary', 'naturalism', 'olsen', 'work--sometimes', "`diddly'", 'intro', 'harassed', 'beaudene', "terl's", "rhames'", 'obligations', 'flashed', 'impacted', 'photographer', 'lee', 'wallow', 'reproduces', 'personal--but', 'universal', 'closeness', 'basketballs', 'unabashedly', 'technique', "driver's", 'inspite', 'expressions', 'misplaced', 'porch', '`review', 'serious-minded', 'furnishings', 'christening', 'lycanthropy', "schreiber's", 'ribisi', 'omnivorous', 'scroll', 'sleeves', 'movement', "'face", 'second-string', 'headache', 'archetype', 'humourless', "blake's", 'comeuppance', 'erratically', 'non-colored', 'toe', 'cruel', 'misogynistic', 'exhilaration', 'outskirts', 'marin', 'lines', 'morsels', 'end-of-the-world', 'observational', 'advisors', 'nerve-jangling', 'stack', 'visionaries', 'caton', 'establishes', 'rum', 'reassures', "ravel's", 'unengaging', 'branch', 'pimples', 'gutch', 'bellyaching', 'pacify', 'spins', 'ghastly', 'compile', 'smidgen', "diedre's", 'uninviting', 'sparkles', 'capra-esque', 'investigated', 'archbishop', 'heroics', 'israeli', 'half-wit', 'cuong', 'bombarding', 'progeny', "deputy's", 'carwash', 'reads', 'arch-nemesis', 'science-heavy', 'trivia--co-stars', 'color', 'dodging', 'inquired', 'patterned', 'walkie', '779', 'hoax', 'date-unsound', 'thick-bladed', 'shill', "blessed's", "anij's", 'misunderstanding', 'mafia', 'innovation', 'flatuence-that', 'introduction', 'kicked', 'introverted', 'prisoners-of-war', 'third-rate', 'evidently', 'out-of-character', 'whiteness', 'periscope', '`pig', 'undoubtedly', 'pleasantville-one', 'townfolk', 'describes', 'extraction', 'jailed', 'corman-wannabe', 'lobs', "plane's", 'maggies', 'elderly', '2654', 'grave', 'unimpressive', "women's", 'perilously', 'anarchist', 'shrew', 'exploded', 'stephens', 'atrophy', '-worthy', 'indolent', 'viciousness', 'nebbish', 'hit-and-miss', 'patlabor', 'cherry-on-the-sundae', 'medicinal', 'freshman', 'relocating', "asia's", 'iris', 'counterparts', 'hebrew', 'noiresque', 'attention', 'thirtysomething', 'pllllleeeeease', 'wrap', 'race-against-time', 'respects', 'escapist', 'kiddies', 'watchmen', 'investments', 'hichock', '_do_', 'cede', "swope's", "herc's", 'woman', 'winds', "opus'", 'firearms', 'coutroom', 'pondered', 'work-a-day', 'unanticipated', 'hayes', 'swindles', 'objectifying', 'anti-male', 'pool', 'inconsistency', 'clinics', 'star-69', 'joyous', 'shift', "bresson's", 'teamed', 'overracting', 'booty', 'mettle', 'delaney', 'godfather', 'namuth', 'by-the-number', 'turboman', "troyer's", 'integrate', 'torn-up', 'ass-map', 'uhhhm', 'banshee', "rodgers'", 'nuzzles', 'procrastination', 'sonar', 'self-esteem', 'familial', 'compiled', 'tampering', "'anywhere", "o'donnell's", 'sesame', 'ceos', 'andreas', 'tough-talking', 'esteemed', 'sabbato', 'moss-covered', 'damnation', 'dudeness', 'matronly', 'destination', 'sellars', 'earthy', 'malcolm', 'thurmand', 'monkey', "out'", 'machineguns', 'suspense', 'vegetables--often', 'vinyl', 'engenders', 'slothful', 'lordship', 'angers', 'bio-genetics', 'unearths', 'tick', "ustinov's", 'infallible', 'treacly', 'tailed', 'ruffini', 'apparatuses', 'figurative', 'bitchie', 'intonation', 'lyndon', 'league', '$80', 'leg', 'perf', 'im', 'maugham', 'dipped', 'policies', 'pell', 'falwells', 'palpatine', 'sables', 'hedwig', 'surveying', 'alana', '1792', 'overused', 'business', 'hasty', 'cementing', 'tromping', 'agreed', 'never-heard-of', '25-year-old', 'relevent', 'rushon', 'videoshelves', 'ever-likeable', 'mouth-opening', 'tempts', 'upham', 'tetsuo-the', 're-issued', 'banking', 'counterfeiter', 'cleopatra', "december's", 'young', 'mirrors', "'f'", 'regarded', 'smirkingly', 'portents', "genre's", 'beaudine', 'worrisome', 'meteorological', 'substantially', 'batcave', "shaiman's", 'cyan', 'heartwrenching', 'hero', "felix's", 'export', 'observance', 'destroy', 'cruise', 'revelled', 'shadow', "capone's", 'rodriguez', 'hesitates', 'corruptive', 'administrator', 'sugiyama', 'outsmarting', 'fist', 'jascha', 'he/she', 'bille', 'rail-thin', "norman's", 'theatrically', 'igor', 'outsiders', 'shelley', 're-creating', 'squared', 'prodigy', 'peron', 'apologizes', 'apprehensions', 'first-rate', 'overacts', 'timid', 'waylon', "thief'", 'disk', 'self-advancement', '[car', 'purged', 'candour', 'symbolist', 'remembers--and', "'lake", 'brandon', 'chirping', 'setups', 'ca_', 'medicore', 'pigeons', "geronimo's", "jabba's", '747s', 'dialogues', 'stranded', 'searing', 'one-hour', 'taxi', 'seth', 'teenaged', 'mindy', 'drifter-cum-postman', 'out-of-this-world', "`thrillers'", 'broad', 'snook', 'ecstasy', "fincher's", 'announcing', 'tended', 'on-again', 'planchet', 'rottingham', 'redman@bvoice', 'robber', 'browbeats', 'candy', 'sham', 'jean-luke', 'radiate', 'fewer', 'supervisor', 'kimba', 'austin', 'madison', 'effects-laden', 'implement', 'postapocalyptic', 'tatopoulos', "'judas'", 'flexes', 'script-writers', 'efforts', 'installments', 'withheld', 'southwest', "conscience'", 'soundmix', "'his'", 'kelsey', 'self-mocking', 'perth', 'acquaintances', 'isla', 'recoiling', 'dragons', 'lawyer', 'voice-changer', '_angel', 'farm/estate', 'cocker', 'hair-obsessed', 'thiessen', 'unfortunatetely', 'theorist', 'broader', 'echo', 'college', 'shawnee', 'rawhide', 'non-david', 'to-the-point', 'candlelit', 'spring', 'francie', 'jinx', "harm's", 'pity', 'funkiness', 'hendel', 'hunky-dory', "stacey's", 'son--is', "aalyah's", 'eradicate', 'abeyance', 'predominantly', "ferrara's", 'polishing', "sarandon's", 'anthony', 'favorable', 'interesting', 'thigh', "smarter-than-you'd", 'erection', 'en', 'nik-nak', 'unfathomable', "donny's", 'competitions', 'vest', 'jeans', 'heart-pull', 'yearbook', 'bathrobes', 'harford', 'ecologist', 'sunderland', 'varies', 'dredd', 'maxwell', 'grossest', 'remedy', 'divers', 'digitized', 'squabbling', 'robertses', 'treated', 'gogolack', 'scalp', 'speech', 'crouches', 'misconduct', 'four-', 'uncharacteristically', '2015', 'acting/english', 'equal-rights', 'columbia', 'moronic', 'psychobabbler', "'total", 'exceptionlly', "starbuck's", 'stevenson', 'owed', 'mst3k', 'down-but-not-out', 'prodigies', 'actor-wise', 'attaining', 'defecation', 'dead', "d'arc", 'knives', 'gong', 'donatelli', 'self-titled', 'mcgill', 'nudie', '_halloween_', 'crunch', 'caro/jean-pierre', 'mcgaw', 'trauma', 'resonates', 'kinkiness', 'winless', 'halt', 'swatch', 'who\x12s', 'justice', 'vince', 'needlessly', 'releasing', 'kafkaism', 'demolition', 'pilgrim', 'title-card', 'adversity', 'cross-burners', 'positively', 'gobbledegook', 'beater', 'casinos', 'life-like', "bruce's", 'ajax', 'froth', 'puke', 'otherworldly', 'missus', 'glimmer', 'dewayne', "suong's", 'relive', 'widow', 'nil', "kingsley's", 'pugilistic', 'swims', 'slavomir', 'expenses', 'sane', 'cassini', 'tuxedo-clad', "marco's", 'pedophilia', 'mushroom', 'g-rated', 'trifecta', 'super-encrypted', 'execept', "cotton's", "cheadle's", 're-recorded', "plenty's", "columbia's", 'mcbeal', 'cummings', 'violations', 'genre-crossing', 'leoni', "gaynor's", 'lawernce', 'stockard', "shelley's", "`enriched'", 'american/spanish', 'tainsy', 'searching', 'governs', 'robots', 'squealed', 'idyllic', "mystery's", "godard's", 'himselfs', "o'halloran", 'focussed', 'baggy', 'leboswki', "horner's", 'low-brow', 'flabbergasted', 'beefcake', 'recognize', 'stripes', 'cartman', 'virginal', 'ione', 'compressed', 'hip', 'choosen', 'spurts', 'flunked', 'leaving', 'pebble', 'simple', 'ready', "bianca's", 'deems', "carrey's", 'basing', 'teeter-totter', 'staples', '[in', 'fiddling', 'lacey', 'mugged', 'anxiety', 'fours', 'nihilism', '_am_', 'rougly', 'who-and', "cup'a'coffee", "penny's", 'desertion', 'wrenching', 'shake', 'wattage', 'premium', "feel'", 'barbarino', 'terk', 'lock', 'machinists', 'unchained', 'proceedings', 'bloc', 'immigrated', 'nutcracker', 'desklamps', 'spaghetti-strapped', 'fused', 'julliard-trained', 'more--all', 'ashmore', '_knock_off_', 'endless', 'people-less', 'ratchets', 'adult-oriented', 'surgery', 'bergl', 'letters', 'gwtw', 'rejuvenates', 'computerized', 'jumanji', 'whimsy', 'rootless', 'jock', 'illustrate', 'eluded', 'dempsey', 'sender', 'story-telling', 'cohesion', 'faintly', 'mullen=92s', 'scoff', "scenario'", 'encouraged', "critic's", 'basil', 'putzulu', 'run-of-the-mill', 'felton', '42', 'anti-depressant', 'invents', 'pickett', 'mesmerize', 'contraption', 'involving', 'panicking', "clark's", 'corman', 'three-day', 'gattaca', "warner's", '$200', 'underdone', 'swain', 'shaye', 'shaves', 'utterly', 'korshonov', 'torah', 'actioner', 'deserting', "calloway's", 'tooth', 'goblin', 'rabid', 'bigalow', 'sundae', 'ill-conceived', 'journalist-turned-screenwriter', 'stereotyped', 'b&w--everything', 'robed', 'drop-out', 'limousine', 'peforming', 'lip-synchs', 'cardiac', 'regretting', '234', 'beaumont', 'infra-red', 'expressionists', 'award{symbol', 'schlubby', 'off-broadway', 'watanabe', 'inter-galactic', 'ukraine', '!', 'sloshed', "'trainspotting'", 'kati', 'slurping', 'lee-zerd', 'banisters', "'tune", 'niko', 'home-schools', 'cumming', 'spam', 'pop-pop', 'subgenre', 'ponytails', 'ev', 'undergoes', 'epa', 'lifetimes', '1953', "jon's", 'cheerful', 'items', 'bemoaning', "tucker's", 'dressing', 'latch', 'motorhead', 'shakespearian', 'lesser', 'mottled', 'mothballs', 'vc', 'irrational', 'briggs', 'easygoing', 'spouting', 'construed', 'coup', 'pulling', 'rollins', 'short-i', 'overdoes', 'caf', 'tibetan', '80s', 'sits', 'mammals', 'haphazard', 'barney', 'club', 'wiping', 'reggae', 'speechless', 'correlated', 'repressive', 'canny', 'tells', 'biloxi', 'nicknamed', 'hand-fed', 'sews', 'punch-outs', 'spleen', 'handiwork', 'toughness', 'blatantly', 'hefty', 'plotlessness', 'mega-movie', 'hubley', '$140', 'slope', 'paxton', "kevin's", 'swicord', 'savagely', 'woo', 'athletic', 'martian', 'tougher', 'evolving', "winslet's", "'gomer", 'time-travel', 'sneak-outs', 'unforeseen', 'invigorate', 'device--you', 'civilised', 'match-ups', 'promise', "survivors'", 'dahlgren', "macdonald's", 'stuggles', 'below-average', '20-minute', 'jung', 'gigs', 'braschi', 'chauffeur', 'despise', 'multidimensional', 'sena', 'disappears', 'twinkies', 'cosette-marius', 'mathematical', 'dire', '`toon', 'dialects', 'apparition', "peak'", 'combined', 'out-perform', 'versatile', 'doctor/patient', "rivka's", 'mentor', 'drive-thru', 'wig', 'antagonistic', 'filipino', 'springsteen', 'mysterious', 'alumni', 'partner', 'friedkin', 'revelation-heavy', 'shortsighted', 'curtis', 'childbirth', 'romanticising', 'beautiful_', 'cole-his', 'comepete', 'disburse', 'knock-knock', 'coyote', 'rorschach', 'loopiness', 'cleaners', 'rochefort', 'angus', "miramax's", 'wangenhein', 'well-told', 'finds', 'tentacled', 'transparently', 'wiseguy', 'equalizers', 'cues', 'new-kid-on-the-block', 'abbreviated', "cmaeron's", 'first-class', 'stallion', 'robbing', 'capitalist', 'da', 'elevated', 'perkins', 'wiretaps', 'feral', 'misspelled', 'pretentiousness', 'tudeski', 'tell-tale', 'karla', 'uncles', 'doctors', 'jaunt', 'collaborators', 'greeks', 'mini-van', 'unwanted', 'steps', 'anti-aircraft', 'furlong', 'tarantino-ish', "rebel's", 'ito', "norton's", 'goofiness', 'flowing', 're-occurring', 'serpent', 'intellectual', 'moniker', 'ideology', 'drugstore', 'aliens/titanic', 'object', 'ever-reliable', 'fire--', 'padawan', 'well-constructed', 'anette', 'honestly', 'aphrodite', 'prob-lem', 'expressionistic', 'earth-like', 'coups', 'audacious', 'basement', 'fancy_', 'authorties', 'adjustments', 'spiceworld', 'grouchland', 'frightfulness', 'so-called', "zeta-jones's", 'outstanding', 'hand-in-hand', "`la-dee-dah'", 'choices', 'palestinian', 'multi-personality', 'acerbic', 'true-life', 'strangely', 'electronic-synthesized', 'eminem', 'colonel', 'roselyn', 'hilfiger', "creator's", 'choreographed', 'straighten', 'sexually', 'humanities', 'slow-paced', 'not-too-bright', 'warts-and-all', 'halfway', 'buds', 'olympic', 'fledgling', 'gown', 'augustus', 'skyrocketed', 'atkine', 'do-whatever-it-takes', 'half-hour', 'shan-yu', 'earth-orbiting', "'key'", 'sci-fi', 'co-owns', 'dicaprip', 'devils', "'till", 'exploiting', 'picked', 'submit', 'encompassed', 'tarantino-look', "antonioni's", "capote's", 'gross-wise', 'goalie-mask-wearing', 'kudos', 'poetry', 'converging', 'hates', 'abrams', "freeman's", 'contractions', 'assail', 'encountering', 'cracked', 'dajani', 'spacemusic', 'motivational', 'shebang', 'thankfully', 'fast-talker', 'fiance', 'heckerling', 'matriarch', 'sugiyana', 'clavell', 'unnerrving', '_cliffhanger', 'dumfounding', 'lude', "rob's", 'facinelli', 'coal-miners', 'quasi-fascist', 'gyrating', 'heathers', 'hawaiian', 'thespian-at-large', 'ballet-', 'moviemakers', 'wcw', 'ufo-researching', 'geoffrey', 'priceless', "ray's", 'odereick', "lenz's", 'praised', 'droves', 'stretches', 'evaluating', 'countess', 'ot', 'aired', 'moore', 'awfulness', 'conventionally', 'trickster', 'thunderstorms', 'scrutinized', 'implanted', 'overextended', 'jo', 'toughest', 'albino', 'predominately', 'dance/song', "o'connor's", 'lovestruck', 'anaconda', 'gilliam', 'kong/hollywood', 'weird-looking', 'trumbull', 'jocularly', 'always-reliable', 'burtt', 'running', "rudd's", 'penance', 'gut-wrenching', 'revealed', 'headcount', 'phantom', 'dwells', 'grinning', 'survey', 'deepa', 'accent', 'concussions', 'miep', "donnie's", 'tear-jerking', 'preempt', 'wookie', 'weathers', 'colonize', 'cried', 'scissorhands', 'fun-spoiling', 'neck', 'faulkner', 'budding', 'initiation', 'unsparing', "arnie's", 'warm-up', 'prepped', 'abraded', 'ringer', 'prot', 'manchurian', 'disloyal', 'busts', 'wards', 'evenings', 'bummed', 'ferociously', 'mulder', 'scorn', 'topnotch', 'counterfeit', 'gregson', 'hideko', 'gunpoint', 'nominally', 'lancelot', 'zest', 'velma', 'thrill-a-minute', 'abundantly', 'cocoons', 'yellows', "streep's", 'metaphasic', 'waters', 'action-film', "t&c's", 'copperfield', 'grotesquely', 'generation-x', 'baluyev', 'routinely', 'guestbook', 'emanating', 'kicks', "slim's", 'aloud', 'unwilling', 'realism--in', 'tailing', 'looked', 'satellite-controlled', 'sinks', 'cheapie', 'seagal', 'mix-and-match', 'lower-case', 'sunglasses', 'tug-of-war', 'feminism', 'vindicated', 'repayment', 'portinari', 'meant', 'spin-off', 'dated', 'romulus', 'rivaled', 'recommending', 'hoo', 'lying', 'approach', 'procession', 'proclaiming', 'frail', 'hero-wanderer', 'adopt', 'warrior', 'banish', 'adjacent', 'film-maker', 'rajot', 'sub-leaders', 'memory', "o'quinn", 'sabor', 'hard', "emperor's", 'narrator', 'vh-1', 'aroused', 'fresh-faced', 'apple-cheeked', 'stern-', 'post-modern', 'truncated', "sakamoto's", 'plotline', "welles'", 'militant', "o'day", 'bai', 'low-grade', 'hopefuls', 'tammi', 'mckinley', 'contemplates', 'relentlessness', "traders'", 'squeal', 'rollergirl', 'clouded', 'rationalization', 'poachers', 'uncommercial', 'cartoony', "clutterbuck's", 'keaton', 'indicting', 'symptoms', 'sleeper', 'seat-handles', 'putrefaction', 'razzle-dazzle', 'bummer', 'super-mom', 'glamorising', 'mug', 'dollop', 'moise', 'liev', 'beholder', 'entirly', 'deviousness', 'powell', 'sleepers', 'concluded', 'karate-chopping', "mobster's", 'penner', 'heigl', 'impossibilities', 'riegert', 'crestfallen', 'out-terrorize', 'log', 'ushered', 'kidlets', '-esque', 'misanthropy', 'staccatto', 'whisper', 'guards', 'leslie', 'engaged', 'underlining', 'holidaygoers', 'respect-me-because-my-father-was-a-good-man', 'niche', 'sadistic', 'free-agent', 'greggory', 'slow-going', 'craggy', 'outworld', 'outlaw', 'deadline', "losin'", 'lags', 'unable', 'forgiving', 'roldan', 'benjamin', 'aura', "july's", 'inflated', 'tinge', 'movie-going', 'residuals', '12-', 'potts', 'greener--and', 'docked', "iguana's", 'unweaves', 'whacko', "idiots'", 'milestone', 'grapples', 'diminished', 'charisma-free', 'vincent/jerome', 'bashing', 'widest', 'outlet', 'interacted', 'do--sing', 'fright', 'screeching', 'voice-over', 'pentathol', 'reduce', 'identifiers', 'endearment', 'steddy', 'rework', 'baddie', 'fingernails', 'inanity', 'hensley', 'copied', 'flocking', 'suggesting', 'quantities', 'draped', 'campers', "rapist's", 'arbiters', 'interfere', 'healer', 'clinton', 'gena', 'deviously', "ryder's", 'grenier', 'moody', 'jolting', 'infidelity', 'cup', "seattle's", "autumn's", 'douglas', 'snl', 'impressionable', "penelope--it's", 'yank', 'five-year-old', 'stir', 'put', 'flash-backs', 'rejection', 'return', "'fuck'", 'hendren', '$11', "shopkeeper's", 'barenboim', "gittin'", 'remembrance', 'shocker', 'terminal', 'helpfully', 'jove', 'hard-hitting', 'coating', 'wendell', 'memorias', 'perfectly', 'mispaired', 'electrocuting', 'wiseass', 'agile', 'director/actor/co-writer', 'wrencher', 'poisonous', 'sheikhi', 'lasseter', 'tackling', 'tabloid-fodder', 'sacrament', "'comedy", 'vinci', 'built-up', 'toy', '3-4', 'editor', 'groans', '6%', '70%', 'tango-dancing', "town's", 'pouty-lips', 'instructor', 'ribbons', 'hampers', 'amphibian', 'travelogue', 'dominic', 'ornate', 'prefontaine', 'cloaking', 'archibald', 'madeiros', 'gestures', 'hunch', 'schwarzneggar', 'workaday', 'fun-to-watch', 'poacher', 'sartain', 'trick', 'landord', 'spares', 'irrevocably', 'bulb', 'foredoomed', 'self-depreciating', 'updates', 'perceptive', 'tatoo', 'alums', 'ceiling', "fighter's", 'jaw', 'half-truth-telling', 'sparing', 'bisexual', '19-year', 'cheapened', 'embry', 'movie--', 'thing__about', 'spud', 'confrontation', "`starship'", 'trashing', 'fremen', 'tragedies', 'cry', 'naturalistic', 'repectively', 'jean-claude', "year's", 'appetizers', 'florida-set', 'umpteenth', 'tautly', 'sindel', 'instrumental', 'ailment', 'zelllweger', 'magistrate', 'romanticized-hemingway', 'sink', 'bully', 'womanly', 'roaches', 'endangering', 'unusually', 'overrides', 'naming', "scagnetti's", 'swings', 'burly', 'intermittent', 'guitar', 'resignedly', 'nausea', 'volumes', 'ashley', 'landlady', "writer's", 'caps', 'attractive', 'doesnut', 'overwhelmed', 'sun', 'splashed', 'faulty', 'bungled', 'attractions', 'pan-slavic', 'remembers', 'flute', 'indisputable', 'moses', 'stuck', 'leaden', 'quake', 'overachieving', "pignon's", 'transpire', 'school', 'participate', 'brightened', "think's", 'purple-sequined', "'miming'", 'costumes', 'senitmental', 'positive', 'too-cheesy-to-be-accidental', 'back', 'outwitted', 'noxious', 'confessional', "material's", '44-year', 'disney-animation', 'agonizingly', 'cowardly', "now-there's", 'rhonda', 'requisitive', 'railly', 'note-perfect', 'rebuttal', 'ado', 'mimics', 'lover/prot', 'tap', 'invulnerable', 'filler', 'disco-', 'todde', 'layoff', 'fringe', 'seam', 'trickery', 'right-hand', 'paget', 'turner-owned', 'astonished', 'titillating', 'busta', 'departed', "nation's", 'rex', 'theatrical', 'matches', 'dialouge', 'midget', 'whipped', 'tarheels', 'chums', 'republics', 'propell', "`pig-keeper'", 'clocking', 're-hashing', 'determinedly', 'ab', 'beat', 'shamelessly', 'covert', 'ecological', 'critter', 'yielded', 'vinnie', 'showbiz', 'sensitive', 'aficionados', 'way-too-glossy', 'lungs', 'reclaiming', 'pilson', 'brazilian', 'concrete', 'rood', 'else--and', 'overcoming', "breen's", 'santoro', 'co-star', 'landslide', 'scudding', 'guggenheim', 'arab', 'tenderness', 'gladiator', 'symphony', 'platinum-blond', 'implicitly', 'edgy', 'yaakov', 'hardship', 'scoured', "rubick's", 'korea', 'innocuous-enough-on-the-surface', 'lusts', '-and', 'dream-', 'bret', 'fabrications', 'captained', '`dad', 'aped', 'chevalier', 'embarass', 'fab', 'compass--her', 'above-mentioned', 'socially', 'kitana', 'persona', 'painfully', 'legend', "tandy's", 'blanco', "hughes's", 'hair-brained', 'trekker', 'short-', 'character-oriented', "`boob'", 'drastically', 'one-and-only', 'charact', 'applicant', 'psychos', 'pseudo-morbidity', 'courthouse', 'groom', 'puzzle', 'blame', 'logo', 'brainchild', '-so', 'launchers', 'earthlings', 'ambulance', 'super-intelligence', "se'", 'greene', 'also-ran', "sausages'", 'rain', 'toilets', 'mine-cart', 'pulitzer-prize', "underwood's", 'smart-aleck', 'spiraling', 'fembots', 'pesence-challenged', 'candidate', 'bangkok', 'geographic', 'lawyer/fixer', 'unforgivable', "existentialists'", 'portos', 'frenzy', 'unspoken', 'prosecution', 'burrow', 'ambiance', 'live-in', 'navigating', 'bacchanal', 'conjugated', 'jocular', 'assessment', 'lhermitte', 'shiver', 'kiyoshi', 'believable', 'pegged', 'nicknames', 'reccomend', 'sketchy', 'cerrano', 'rubber-faced', 'expurgating', 'inhospitable', 'silky', 'mone', "`n'", 'stingy', 'headful', "pertwee's", 'snooty', 'badies', 'giant-screened', 'shepard', 'anglo-saxon', 'gregory', 'punishment', 'disposable', 'schrieber', 'charlize--surprise', 'aback', 'inflating', 'cringes', 'highly-publicized', 'dilapidating', 'heisted', 'embrassment', 'neutral-coloured', 'dickie', 'rulers', 'dream', 'panning', 'hops', 'loretta', 'legislation', "`final'", 'implant', 'tara', 'etch', 'avika', 'raved', 'aberration', 'daredevil', 'whisk', 'despair', 'http', 'chronicle', 'wronged', 'retroactive', 'half-empty', '(', 'razor', 'newsweek', 'flatlining', 'murderer', 'stabile', 'muscular', 'dali', 'friskiness', 'outfits', 'feel', 'beresford', 'odoriferous', "60's", 'fry', 'hehehe', 'makeovers', 'expatriate', 'village', 'eisenberg', 'stud', 'kickass', 'gleeson', 'undergo', 'reincarnation', 'thuggish', 'witherspoon', 'skg', 'pleasing', 'ridulous', 'unwinable', 'daly', 'immorality', 'scratch', 'uli', 'immediacy', 'mercy', 'esteem', 'embodied', 'supernatural-entity-falls-in-love-with-girl', 'mtv-influenced', "bassinger's", 'immobile', 'near-uproar', 'scions', 'kate', 'symptomatic', 'laugh', 'password', 'thought', 'masturbates', 'innuendoes', 'compulsory', 'reconcile', '1989', 'cued', 'nino', 'misconceptions', 'pre-feminist', "chow's", 'huerta', 'champs', 'mysterians', 'route', 'thomas', 'regretful', 'punishments', 'rear', 'rin-tin-tin', 'rapped', 'angrily', 'certifiable', 'empathic', 'dre', 'epperson', 'nts', 'precepts', 'sdds', 'tedious', 'wallace', 'understates', 'simpering', 'predicated', 'photographer-', 'actor/magician', 'rename', 'adefarasin', 'appropriateness', 'semi-religious', 'effectiveness', 'alienbusting', 'printed', 'hound-dog', 'chance--unless', 'pilgrims', '1993', 'chop-chopped', 'wrench', 'there--two', 'hoodoo', 'digest/gumpian', 'ice-bound', "wendy's", 'brambell', "woolf's", 'prison-matron', 'neverending', 'unworthy', 'demy', 'scour', 'onanism', 'favour', 'suitcases', 'moralizing', 'camerons', 'vigo', 'fickle', '92', 'bouncing', "morritz's", 'copyright', 'hitchhiking', 'hurricaine', 'faculty', 'emira', 'newly-restored', "whispery-voice'd", 'unspecified', "forever'", 'forgives', 'expressway', 'pests', 'benning', "epps'", 'isaac', 'bean', "philip's", 'barnfield', 'muslim--are', 'liquor', 'pike', 'firebird', 'unown', 're-tried', 'held', 'buddha', 'eamonn', 'ball-peen', 'prolong', 'unscary', 'warner-brothers-coyote-fall-off-the-cliff', "fleder's", 'sergio', 'engages', 'hullo', 'writer/producer', 'audi', 'nurtures', 'cochran', "creature's", 'buying', 'farces', 'send-ups', 'bind', 'reverence', 'chicken-bashing', 'create', 'high-heaven', 'beaker', 'cirque', "legend'", 'renovated', 'camouflage', 'tv2', 'sotomejor', 'wingman', 'description', 'knifepoint', "_polish_wedding_'s", '1946', 'smears', 'addendum', 'platitudes', '-', 'zucker/abrahams/zucker', 'viewfinder', 'birth', 'lifelike', 'traveller', 'act', 'coli', 'zuko', 'fc', 'emilio', 'alc', 'bane', 'well-developed', 'deceptions', 'intrinsic', 'cosmic', 'assets', 'explosives', 'fine-', 'afar', 'contract', 'bogmen', 'emotions--awww', 'gripes', "eve's", 'snodgress', 'emissions', "greenwald's", 'super-cop', 'askew', 'damon', 'often-told', 'ibenez', 'grizzly', 'reshooting', "alzheimer's", 'girth', "children-it's", 'nixon', 'polynice', 'ex-athlete', 'sequence/flashback', 'limousines', 'oded', 'tired', 'expendable', 'inferno', 'welles', 'drifts', 'tiger/land', 'blessing', 'athos', "graham's", 'actresses', 'drape-hanging', "proyas'", 'terrifically', 'unknowns', 'sacrifices', '$50000', 'bounderby', 'mature', '+s', 'half-expect', 'fortress', 'causing', "altman's", 'not-so-practical', 'prowse', 'vorld', 'hand-to-hand', 'super', 'ami', 'vieluf', 'delivered', "harding's", 'fornication', 'advised', 'mid-west', 'remarrying', 'fiesta', 'kung-fu', 'jehan', 'perjuring', 'amalgam', 'insinuates', 'proposition', 'accomplishes', 'harris', 'telegram', "hershman's", 'trek-fan-only', 'sending', "church's", 'unoriginality', 'vegetarianism', 'masterwork', 'rhythm', 'banishes', 'hobbling', 'blazingly', 'tis', 'dispare', 'prayed', 'illena', 'gills', 'gritty', 'obscurities', 'boiling', 'director/producer', 'wasps', 'male-ego-stroking', 'pea', 'weave', 'cranks', 'creaking', 'faint', 'takeover', 'granted', 'satirizing', 'kull', "heche's", 'longish', 'scot', 'sideline', 'georgie', 'raised', 'mekum', 'super-hero', 'slipups', 'disparate', 'hailed', 'foes', 'contemptuous', 'reminscing', 'hats', '15-16', 'kinks', "fahey's", 'liberation', 'recommitted', 'latrobe', 'logs', 'guardians', 'machinas', 'nominated', 'weakness', 'border', 'pre-school', 'french-accented', 'fashions', 'multi-level-marketing', 'hubbub', 'firth', "couple's", 'brim', 'ad-libs', 'interchange', 'lukanic', 'downs', 'fatalities', 'shoestring', 'tampons', 'geometry', "weaving's", "'n", 'super-rich', "'hit", 'dishonourable', 'knockers', "institution's", 'gilmore', 'novo', 'helmer/scripter', 'insistence', 'oceanographic', "paris'", 'tylenol', 'twiggish', 'revolver', 'enhance', 'exterminate', '80-minute', 'bellowing', 'mickelberry', 'invariably', '1970s', 'kellner', 'infamy', 'myra', 'merriment', 'trash-talking', 'sophomore', 'truely', 'vigilantes', 'lent', 'societies', 'cruisers', 'cocktails', 'spouts', 'peters', 'keen', 'horrors', 'fiercely', 'divinely', "o'bannon", 'sullivan', 'scorching', 'portia', 'madeleine', 'contribution', 'doodyish', 'birdie', 'beaumarchais', 'stop-motion', 'family-style', 'shread', 'transferred', 'mailer', 'displaced', 'lifestyles', 'federation', 'wanton', 'shooting', 'publishing', 'ed', "aliens'", 'watchers', 'rockwell', 'hustled', 'midriff', 'grief', 'umbrella', 'uncharted', 'pale-faced', 'vortex', 'insists', 'joely', 'kidvid', 'up-and-coming', 'red-handed', 'studdly', 'marijuana-laced', 'maintaining', 'goofy/embarrassing', 'samantha/charly', 'movies/tv', 'merged', 'proxima', 'giuseppe', 'distinctively', 'insulated', 'lurking', 'alex', 'pragmatic', 'dietz', "huston's", 'wrong-doings', 'profitably', 'max', 'effortlessly', 'harsher', "hunter's", 'electrocuted', 'machinegun', 'ejected', 'toddler', 'instigated', 'serous', 'relations', 'experts', 'devilish', 'finesse', 'entrance', 'seahaven', 'pew', 'feature', 'triumph', 'schl', 'vices', 'self-appointed', 'adroitly', 'baotian', 'perceive', 'demise', 'sidious', 'motives', 'wendkos', 'kennel', 'front-page', 'longs', 'dropper', "secret'", 'job', 'anchor', 'amicable', "earth'", 'swindled', 'fully-clothed', 'atkinson', 'baptism', '4am', 'imprecation', 'offspring', 'cigarette-smoking', 'enjoys', 'tinkering', "vicky's", 'detroit', 'vehicle', 'scientists\x12', "stepahne's", 'opened', 'self-loathing', 'christabella', 'nuez', 'prue', 'counsellor', 'appliances', 'haunt', 'vanquished', 'forms', 'encounters', 'psyche', "owen's", 'researchers', '90s', 'exhileration', 'afoot', 'molecular', 'commend', 'occupied', 'dollars', 'fabrizio', 'speeder', 'amalgamation', 'grad', 'douse', 'fizzles', 'induces', 'british', 'einstein-level', 'frampton', 'lorre', 'pertains', 'candid', 'dig', 'then-reigning', 'assembling', 'exploitative', 'de-virginize', 'cheesecake', 'scatter', 'implode', 'inconsiderate', 'raunchy', "'seven", 'prison', 'shout', 'verbally', 'way-back', 'convictions', 'bamboozled', 'machine-gun', 'bare-footed', 'hellbent', 'dogs', 'barstool', "author's", 'presumable', 'tortuous', "1978's", 'recording', 'post-apocalypse', 'short-circuits', 'hum-drum', 'capt', 'candlelight', 'gretchen', 'wisely', 'broadsword', "vincent's", 'confidential', 'shellulite', 'robbie', 'duration', 'prosperity', 'employed', 'fondly', 'finger', 'interpretation', 'unconventional', 'copier', 'laughing', "`lola'", 'rear-ended', "hark's", 'shaken', 'transylvania', 'movie-making', 'odds-and-ends', 'stems', "landis'", 'summit', 'huff', 'privy', '$7000', 'josu', 'nine-month-old', 'gassed', 'nincompoops', 'prescott', 'writerly', 'trough', 'outwardly', 'century', "horses'", 'courteous', '`princess', 'shootings', 'judging', 'belted', 'greasy', 'audacious--and', 'inactive', "bird's-eye-view", '1991-france', '|', 'prioritized', 'imbue', 'sprite', 'fingal', 'manifesting', 'energize', 'timeline', 'dissatisfied', 'resigned', 'captivity', 'frontiers', 'eloquence', 'devereaux', 'virtual', "mackintosh's", 'meager', 'glimpse', 'bees', 'south', 'censorship', 'revitalizing', "smiths'", 'appealed', 'resigning', 'diffused', 'singleton', "nikki's", 'hou', 'patently', 'problem-why', 'tigerland', 'flashiness', 'gist', "gershwin's", 'provincial', 'chidduck', 'ultra-low', 'reply', 'gassner', 'palette', 'tormey', 'peeled', 'figures', 'governing', 'astronomy', 'menges', 'biebe', 'doherty', 'decriminalization', 'dominant', 'blucher', 'octopus', '95', "cheese'", 'hughes', 'fretful', 'terrifying', 'chained', 'zillion', 'com/hollywood/academy/8034/', 'death-as-pitt', 'replicas', 'orsen', 'hancock', "''its", 'scruffy', 'hambling', 'corleones', 'ages', 'sanitation', 'worth', 'cromwell', 'tex-mex', 'finale', 'sketch-comedy', 'disproportioned', 'upper', 'indecipherable', 'screw', 'romania', 'drone', 'personnel', 'well-worn', 'stupid-ass', 'vacationers', 'compendium', "marsellus'", 'lax', "mayor's", 'affectionately', 'netting', 'carolyn', 'coughing', 'debacle', 'adriana', 'telecast', 'vindictive', 'song', 'full', 'flash-forwards', 'protectors', 'capra', 'centerfold', 'seduces', 'upside-down', 'justified', 'francesca', "grossy'", 'fish', 'stuffs', 'dissection', 'liveliness', 'difference-', 'violins', 'nuns', 'horrendously', 'neice', 'yahoo', 'chafing', 'joke--a', 'uttered', 'su', 'parasail', 'fundraiser', 'visually', '4-year-old', 'discourteous', 'sex-driven', 'tapestry', 'extra-marital', "`rumble'", 'exhaustion', 'roads', "peter's", 'batallion', "'speed'", 'reid', 'protagonist', 'bourgeois', "melvon's", 'dogtags', "'rich'", 'threesomes', 'dogmatic', 'brett', 'dormies', 'slow-burn', 'mastery', 'tree--a', 'entirety', 'seng', 'sublime', 'siblings', 'chainsaw-massacre-type', 'varden', "grant's", 'brace', "'wooden", 'paraphrased', 'nutsy', 'loads', 'officer', "jo's", 'spielberg-inspired', 'fulfill', 'to-hell-with-the-system', 'lacing', 'clyde', 'foils', 'bresson', 'clean-up', 'dostoevsky', '1940s', 'salvadorian', 'predictions', 'moustache', 'nonreligious', 'pleasure-seeking', 'mythological', 'mystifying', 'horrocks', 'handheld', 'mobs', 'warned', 'crab', "'crazy'", 'sliver', 'svengalian', 'revolves', 'heroin', 'survives', 'fx', 'blackboard', 'shorts', 'triumphing', 'derivative', 'open-ended', 'particular-', 'obnoxious', 'vile', 'finest', 'probide', 'giardello', 'dolph', "twit'", 'guernica', 'clay', 'springy', 'clad', '137', 'tomahawk', 'tremors', 'brightly-lit', 'closemindedness', 'fantasyland', 'escapes', 'parallel', 'admirably', 'everywoman', 'statesman-like', 'burton', 'expressed', 'dons', "carolco's", 'haze', 'incongruities', 'longstreet', 'memory-diminishing', "forsythe's", 'invoking', "spadowski's", 'squabbles', 'neck-brace', "bus'", 'reign', 'cronenberg--is', 'utilizes', 'sunshiny', 'rossi', 'psychlo', 'real-time', 'expired', 'thrillingly', 'isle', 'bash', 'wired', 'debated', 'shadows', 'tearfully', 'boredom', '150th', 'vibrating', 'back-stabs', 'remind', 'pervert', 'silhouetted', 'inquisitive', 'volvo', 'enlightenment', 'comedically', 'isolated', 'view', 'set-pieces', 'jaunts', 'spawns', 'madman-cum-detective', 'fastidious', 'ex-ranger', 'juxtaposition', "tingle's", 'blam', 'stoned', 'copout', 'conceptualization', "'enough", 'hayman', 'pseudoerotic', 'cheech', 'unforgiveable', "jim's", 'nuttgens', "baldwin's", 'blush', "son'", 'donohue', 'not-bad', 'flamenco', 'upped', 'carves', 'hormonal', 'soft-', 'autocrat', 'justifiably', 'intention', 'posterior', 'can-can', 'bottom', 'unwillingness', "1968's", 'kiddie-porn', 'squinting', '-rydain', 'wittiest', 'tentacle', 'trojan', 'palaces', 'malroux', 'overconfident', 'mackey', 'doggy-style', 'tempers', 'odin', 'berates', 'child-hating', 'extort', 'made-for-disney', 'facilitates', "lanier's", 'adam', 'sevigny', 'abides', 'nationals', "louise'", 'shrunk', 'forerunner', 'undiscerning', 'nazi-charged', "danson's", 'celebrates', 'foyle', 'yancy', 'spit-like', 'mindstretching', 'pot', 'agonies', 'equivolence', 'abominable', 'symbiosis', 'funner', 'rammed-down-your-throat', 'heath', 'slovenly', 'airplane-style', 'braveheart-dosage', "amazin'", 'clutches', 'memphis', 'weisz', 'impersonator', 'comedienne', 'hana', 'six-year-old', 'wild-eyed', 'superhuman', 'pellets', 'gazzara', 'avary', 'partnered', 'cheerleaders', 'sci', 'undulating', 'meteoric', 'women', "jjaks'", 'gargoyles', 'gaps', 'uber', 'mostly-misfired', 'notable', 'rails', 'raines', 'xena', 'rapper/composer', 'cells', 'bewilderment', 'lux', 'skaro', 'car/who', 'atomic', 'haliwell', 'disposed', 'arrived', "luger's", 'reprint', "jonas's", 'pretty-in-pink', 'spaghetti', 'lantos', "gibson's", 'constrictor', 'fragile', '`rip-off', 'worthwhile', 'cotner', 'wittiness', 'co-ordinator', 'trader', 'ravaging', 'givya', '2058', 'spaced-out', "esposito's", 'yip', 'hulot', 'supplement', 'pond-', 'penned', 'genuineness', 'bishop', 'hang-ups', 'completists', 'fairfolk', "bogie's", 'exfoliating', 'hoods', 'junky', 'donat', 'lts', 'rewarding', 'messages', 'icy', 'mid-1960s', 'phenomenon', 'views', 'earnest', 'yamazaki', 'bateer', 'comments', 'stranger', "heaven's", 'manned', 'marseilles', 'hesitate', 'doppelgangers', 'added', "nosferatu's", 'stieger', "tibbs'", 'wincing', 'dodgers', 'cross-dressing', 'revell', 'documentary-like', 'voodoo-inspired', 'chronicling', 'reverting', 'oafish', 'preponderence', 'czechoslovakia', 'full-condescension', 'maximes', 'overlit', 'tuan', 'kraft', 'jodi', 'chilling', 'cellular', 'mother-in-law', 'dukas', 'vampira', 'crux', 'downbeat', 'elegant', 'vaughn', 'urging', 'zachary', 'death--our', 'glycerine', 'mcdiarmid', 'serpents', 'dodo', 'hey-dey', 'desi', 'finklestein', 'teeny-bopper', 'mused', 'non-spoiler', 'gut', 'even-numbered', 'anticlimaxic', "confidential''", 'york', 'peace', 'fill', 'fight-scenes', 'trash-talks', 'word/jazz', '104-minute', 'checking', 'supposedly', 'deceiver', 'auctioneer', 'machette', 'gags', 'awoke', 'high-caliber', 'midsummer', 'demeanour', 'titans', 'predict', 'bauble', 'eye-filling', "sinyor's", 'rainforest', 'slithering', 'overcasts', 'earthward', 'diminutive', 'saddest', 'war_', 'suitor', 'accessible', 'foundation', 'plimpton', 'virtuoso', 'reed', 'grueling', 'shoplifting', "can'", 'relocate', 'fantasies', 'weigh', 'villard', 'well-traveled', 'terrible-occasionally', 'sit-com', 'double-dealing', 'advancing', '128', 'deciphered', 'busy', 'masked', 'tyrone', 'cookie', 'unsound', 'hockey', 'vintage', 'purist', 'monstrosity', 'confrontational', 'chat', 'film-noir', "n'", 'westernized', "udderley's", 'sorely', 'relatibility', 'blemishes', 'ivan', 'lynching', 'vulture-like', 'penetrate', 'payments', 'dominates', 'au', 'phrases', 'streetwalkers', '98%', 'indiscernable', 'password-protected', 'tequila', 'silly-sounding', 'default', 'candidness', 'goldthwait', 'crowbar', 'claimants', 'dominions', 'rewrite', 'giallo', 'thin-looking', "'hilarious'", 'lolita', 'volunteers', 'airwaves', 'abounded', 'terrorizing', 'violent', 'thrills', 'periodically', 'boots', 'stellan', 'youthful', 'story--', 'woody', 'clashes', 'phones', 'kristel', '1955', 'adlai', 'slaughterhouse-five', 'conclusion--the', "picard's", 'illnesses', 'coming', 'matron', 'potty-humor', 'assuming', 'ex-girlfriends', 'mouse', 'strasberg', 'pathetically-written', 'compel', 'depalma', 'egyptologist', 'fair', 'mid-east', 'details', 'creation', 'obliviousness', 't-1000', 'frankenstein', 'stray', 'godzila', 'mesmerized', 'straights', 'bizaare', 'ovens', "amadala's", 'uber-buff', 'calls', 'event', 'arts', 'evacuates', 'murder-', 'cooly', 'dil', '13th', 'homages', 'referee', 'ppk', 'boobies', 'herrings', 'overlapping', "culkin's", 'manfully', 'boxoffice', 'climate-controlling', 'howitt', "brenda's", 'fortysomething', 'reprising', 'prince', 'ancestors', 'furnace', 'galore', 'tweed', 'nymphomaniac', 'isis', 'forests', 'journey', 'laboratory', "elfman's", 'delinquents', 'kubrick', 'side-tracks', 'seymour', "fargo's", 'clubbing', 'multinational', 'jazzy', 'bookend', 'ironed', "palma's", 'prevention', 'tripplehorn', 'team-up', 'eager', 'mean-spirited', 'scientific', 'supposedly-intellectual', 'extensive', 'inedible', 'summation', 'deep-voiced', "bostwick's", 'colorblind', "jones's", 'ooooooooh', "frances'", 'sverak', 'scaffold-like', 'cast-member', 'required', 'trouble', 'blowing', 'amc', 'horror-mystery', 'emailing', 'lightly', 'subliminally', 'ex-special', 'carreer', 'hypnosis', "judge's", 'cogent', 'feces', 'hiccup', 'observer', '6th', 'rave', 'cadets', 'sputter', 'desolate', 'damning', 'alleviates', 'terrio', 'electro-magnetic', 'nelligan', 'requiring', 'nun', 'filming', 'gun-crazy', 'ha-ha', 'vultures', 'hygienists', 'nitrite', 'screenwriting', 'knack', 'jack-styled', 'leghorn', 'explodes', 'scans', 'saint-saens', 'rya', 'preliminary', 'vaild', 'round-robin', 'pestering', 'fresa', 'hypochondriac', 'nenette', 'fourth-graders', 'skilfull', 'capoeira', "richelieu's", 'fiend', 'fanatasies', 'ransom', 'letter_', 'flustered', 'anger', 'jeremy', 'center', 'mile-a-minute', '_life', 'narrators', 'brave', 'strand', 'delves', 'figuring', 'dreams', 'golden', 'straight', 'semester', 'disarming', 'dopey', 'demonstrable', 'watto', 'blowout', 'in-between', 'gershwin', "ming's", 'dirt-poor', 'irritated', 'battlefield', 'demeans', 'overachievement', 'smurfs', 'waturu', 'surrendered', 'lhassa', 'compass', 'sole', 'nonense', "stoker's", 'mellow', 'frank-n-furter', 'patootie', 'winfrey', 'witness', 'barrymore', 'doggy', 'advocated', 'revert', 'hick', 'chloe', 'lead-faced', 'zelig', 'simulation', 'buzz', 'obtained', 'occupy', 'long-long', 'traced', 'god-forsaken', 'kids-and', 'over-familiarization', 'newt', 'attending', 'crates', 'appended', 'permanently', 'obsessed', 'les', 'companies', "swingin'", 'sulka', 'dmx', 'transmission', 'slasher-comedy', 'dangers', 'cheated', 'st', 'reminisces', 'creepiest', 'snowstorm', 'rhetorically', "steenburgen's", 'browne', 'eagerly', 'jimmie', 'rebellions', 'bat', '122', 'evacuate', 'waitresses', "cbs's", 'flawlessness', 'niece', 'overdosing', 'american-chinese', 'unduly', 'starlight', 'colonized', 'addressing', "network's", 'vargas', 'wessell', 'indecisive', 'postlethwaite', 'confusing', 'riddick', 'oswald', 'sexy', 'no-dialog', 'impresses', 'fulfilment', 'railings', 'killer-taunting-his-victim-on-the-phone', 'buggery', 'krasner', 'hutchinson', "pistols'", 'ntsc', 'pack', 'absolutes', 'self-absorption', 'dory', 'unaffected', 'faux-cute', 'advertisements', 'abe', 'seashell', 'ounces', 'relatively-brief', "critics'", 'falling-out', "bunny's", 'favreau', "soldier's", 'gold-embroidery', 'polly', '1940', 'marvellous', 'leguizamo', 'distracted', 'hard-he', 'sustained', 'easy-bake', 'cutout', "malory's", 'campbell', "she's", 'affectionate', 'subtexts', 'clear-eyed', 'sh--', 'opener', 'shoot-out', 'obvious-looking', 'go-carts', 'harald', 'bigscreen', 'shucking', 'fall-off', 'subsequently', 'honored', '$10000', 'celebrating', 'marie-therese', 'wrecked', 'worf', 'sirocco', 'expression-challenged', 'galactic', 'intelligence', 'lomper', 'wai', 'substance', 'students', 'bitter', 'point--and', 'recreating', 'dang', 'flashdancer', 'masterson', 'narcissistically', 'tovoli', 'sfxfest', 'truthfulness', "seinfeld's", 'amounting', 'crippled', 'russkies', 'waiting', 'enterprise', 'melodramas', 'suicide', 'spy-', 'absoloute', 'laforge', 'riled', "coil's", 'all-too-realistic', 'rewriting', 'dole', 'smokers', 'swap', 'gamut', 'avenging', 'ken-and-barbie', 'unintelligble', "sayles'", 'blindfolded', 'fast-paced', 'difficulties', 'suave', 'strip', 'iran', 'cliffs', '_four_', 'glitch', 'priveleged', 'zucker', "cohen's", 'lack', 'goofing', 'nasal', 'grammy', 'half-baked', 'missteps', 'time-filler', 'postmodernism', "officer'", 'side', 'ketchum', 'spot-on', 'founding', 'bonding', 'eclectic', 'segal-like', 'film-making', 'fortunate', 'renenged', 'sprung', 'c3po', 'wining', 'insurrection', 'goofy-looking', 'quasi-ominously', "1992's", 'appealing', 'snippet', "1994's", '106', 'bat-like', "campbell's", 'sizzling', 'boddy', 'message-id', 'countrysides', 'inextricably', "deming's", 'three-way', 'dj', 'seaman', 'cautiously', 'vera', 'turtle', 'brannagh', 'mortar', 'wayward', 'islamic', 'televisions', 'post-snl', 'jumbled', 'recklessly', 'mea', 'neurologist', 'intones', 'super-smart', 'photograph', 'oncoming', 'recognise', 'hourly', 'challenges', 'proximo', 'silents', 'nitpickers', 'wars', 'blade', 'adelaide', 'animus', 'name-brand', 'personalized', 'semi-autobiographical', 'absolutist', 'averting', 'wily-ness', 'curtolo', 'botch', 'low-down', 'machiavellian', 'dog-half', 'jonathan', 'overtones', 'spence', 'roxane', 'scarring', 'eliminates', "o'barr", 'harding', 'kolya', 'brosse', 'buddy-type', 'rome', 'minkoff', 'once-revered', 'one-room', 'pee-wee', 'rocked', 'escaping', 'catherine', 'kidnapped', 'drool', 'lending', 'swayze', 'sandbox', 'liscinski', 'super-light', 'writing-his', 'possum', 'rugby', 'comic-geeks', 'decoy', 'overpopulation', 'mexican-american', '73', 'precision', 'stupidest', 'partygoers', 'quasi-campy', 'bottom-of-the-barrel', 'absurd', 'stony', 'dwindling', 'grisly', 'jawed', 'shaggy', 'tread', 'contantly', 'at-a-glance', 'synchronous', 'curiousity', 'heartbroken', 'shoddy', 'sympathizer', 'codification', 'nationally', 'endorsing', 'linking', "bachelor'", 'aahs', 'likeness', 'receiving', 'vidal', 'dumb', 'birthday', 'state-of-the-art', 'iowa', 'self-destructiveness', 'bandanna', 'lapping', 'plug', 'volley', "cake'", 'pygmalion', 'tub', 'daswon', 'basinger', 'heal', '_patlabor_', 'outpourings', 'worker', 'checkout', 'non-triple-x', 'forebodings', 'retro-futuristic', "mainstream's", 'fe', 'triple-crosses', 'one-joke', "rock'n'roll", 'excercise', "_william_shakespeare's_romeo_+_juliet_", '@', 'beethoven', 'donaldson', 'smooths', 'deepening', 'photography', 'shagged', 'claudio', 'burrito', "february's", 'quibble', 'baltimore', 'fables', 'self-', "'halloween'", 'wanna', 'cashier', 'dysfuntional', 'gets-off', 'cuban', 'hold', 'scarborough', 'fortinbras', 'strung', 'morse', '60s', 'golden-age', 'assault', 'classless', 'traveling', 'poorer', 'scotty', 'soft-porn', 'unsuccessful', 'edmund', 'hostages', 'processor', 'scuzzy-looking', 'imaginable', 'sunning', "wachowski's", 'directorial', 'overshadow', 'increasingly-', 'tagalong', 'edie', 'off-balance', 'liman', 'insipid', "1982's", 'depressing', 'lena', 'elisa', 'overlooks', 'snuggly', 'pseudo-intellectuals', 'lindo', 'malone', 'romantic-comedy', "air'", 'dobson', 'backslidden', 'blood-and-gore', "schneider's", 'commandeer', 'pondering', 'outburst', 'limestone', 'aires', 'squishy', 'massacres', 'brightest', 'sheffield', '_but', '16-year-olds', 'topping', 'wagon-train', 'guys/bad', 'protected', 'tearing', 'unexplored', 'solves', 'ex-secretary', 'duo', 'cockroach-like', 'sectors', 'sixteenth-century', 'fielder', 'country-western', 'spurned-psycho-lover-gets-her-revenge', 'provision', 'occupies', 'contrasting', 'subscribed', 'non-offensive', "spazio'", 'dissected', 'basket', 'weeds', 'pulpy', 'dramatism', 'peach-colored', 'borne', 'riots', 'crushes', 'grossness', 'synthesized', 'mailed-in', 'ball-picking', 'urine-stained', 'indulgent', 'normal', 'uncannily', 'inform', 'dot-com', 'confirmed', 'tongue', 'tommy', 'mayer', 'cutedom', 'triggers', 'relentless', 'standard', 'bandit', 'pare', 'consultation', 'dishonest', 'action-comedies', 'reduced', 'irina', '_and_', 'forrester', 'shoot-up', 'self-staged', 'murray', 'perverted', 'explorations', 'universality', 'eastwood-directed', 'holstering', 'dishonor', 'reubens', 'deviate', 'techno-ish', 'frat', 'something-or-the-other', "onegin's", 'draping', "'exorcised'", 'owens', 'infancy', 'classrooms', "nympho's", 'hotep', 'immediately', 'goddess', 'studying-abroad', 'discoverers', 'enmity', 'cloud', 'copping-out', 'subjecting', "danny's", 'howard_', 'undeniably', 'for--in', 'piet', 'drawing-room', 'farms', '35-year-old', "characters'", 'oft-dismissed', "'landworld", 'reside', 'cheeky', 'bugger', 'poet/songwriter/', 'documentary', "spike's", 'herzfeld', 'east-west', 'money', 'guttural', 'de-', 'boy-meets-fish-and-saves-environment', 'buckney', 'achieves', 'rekindle', 'wierdness', 'converted', 'zippel', 'keegan', 'desirable', 'roof', 'costuming', 'caliber', 'grungy', 'however--these', 'symbolize', 'grown-worthy', "planet's", '-rated', 'suggestion', 'cleaning', 'instinctive', 'harland', 'thematically', 'lupone', 'dismemberments', 'masahitko', 'eccentrics', 'vanish', "sethe's", 'crawly', 'kudrow', 'on-edge', 'minimally', 'pretentious', 'solar', 'minstrel', 'broad-based', 'ndorff', 'assasin', 'runner_', 'aimed', 'drives', 'toss', 'jersey', 't-shirt', 'arrises', 'elizondo', 'extraordinarily', 'tier', '_blade_', 'hoping', 'passports', 'firearm', 'ruined', 'brody', 'georges', 'penleric', 'flamed', 'grief-stricken', 'vicarious', '90-minute', 'mode', "afi's", "daylight's", 'stunner', 'nyman', 'immaculate', 'auction', 'incinerating', 'interviewees', 'spruce', 'hinky', 'terror', 'horns', 'love-triangle', 'outlawed', 'gangster', 'messed', 'wannabe', 'sight-gag', 'audaciously', 'tupac', 'roach', 'sided', 'wags', 'shun', 'well-groomed', 'sayles', 'voltage', 'bulworth', 'boogie', "eilonwy's", 'golfing', 'teen-accessible', 'rating', 'dean', 'plumet', "strangers'", 'now-outdated', 'then-wants-to-pay-for-sex-', 'harsh', '157', 'hissed', 'talkative', 'shot---as', 'pricelessly', 'raccoons', 'shred', 'screens', 'bazooms', 'dubs', 'zit', 'forehead', 'ex-gangbanger', 'rollicking', 'plows', 'cringed', 'shrinking', 'barely-funny', 'edwards', 'tactically', 'busting', 'german/west', 'newport', "gregio's", 'square', 'heals', 'sexist', "blondie's", 'vigil', 'rapacious', 'fishnet-stocking', 'over-achiever', 'iguanas', 'indignant', 'briesewitz', 'americanised', 'portman', 'publisher', 'khanjian', 'gabriela', 'bootlegging', 'unpleasant', 'goo-goo', 'biederman', 'self-irony', 'mid-1980s', 'plato', 'night-sticking', 'out-of-work', "'being", 'hybridization', 'spy/romance', 'rationale', 'carson', 'knocks', 'suck', 'edge--had', 'dimwit-with-a-shady-past', 'geese', 'individuals', 'conscience-stricken', 'tendency', 'elves', 'heavies', '_entertainment_weekly_', 'accused', 'deming', 'live_', 'pre-paid', 'imperiously', 'debts', 'dad', 'stephn', 'relatable', 'undercurrents', 'decades', 'outre', 'representatives', 'true-story', 'bribe', 'plot-line', 'diedre', "ex-wife's", 'textbook', "jaws'", 'sitcomish', 'clumsy', 'prevails', 'impose', 'disobeying', 'labors', 'sport', "petit's", 'ninth-hand', 'solidarity', "`faith'", 'brushstrokes', '2000-pound', 'third-class-party', 'hadley', 'beach', 'garbed', 'ten-minutes', 'reveal', 'arabian', 'twinkle-toed', 'aware', 'comparison', 'judgements', '[', 'longshanks', 'rockefeller', 'oldish', 'symbol', 'eyeball', 'dreamworld', 'expressing', 'multiples', 'transpose', 'dorsey', 'loopy', 'humored', 'effects-heavy', "terminator's", 'brawn', 'huns', 'lustre', 'conceived', 'man-animal', "witherspoon's", "callaway's", 'finance', 'bikini-', "u2's", 'suprising', 'do-gooders', 'runneth', "bailey's", 'fogs', 'sidebars', "kong's", 'recipe', 'curveball', 'steering', 'canada', 'morphings', 'scouting', 'levritt', 'akiko', 'cufflinks', "matthau's", 'unwavering', 'contort', 'o-negative', 'peaches', 'playstation', 'fisher', 'detmer', "'summer", 'slathers', 'jinks/', 'ventura-ish', 'uncooperative', "patrice's", 'ackland', 'toiling', 'vicky', 'often-adapted', 'dismembered', 'flowers--tear', 'pseudo-religion', 'nathaniel', 'by-the-books', 'goriest', 'hellraiser', 'pallisades', 'russian', 'tate', 'masse', 'uncovered', 'fargo', "chompin'", 'break', 'bee', 'foaming-mouth', 'loading', 'ball-bouncing', 'dune-esque', 'indelibly', 'deedle', 'affirmative', 'nyc', "us's", 'balloons', 'schlockfest', 'gwennie', 'nominee', 'realy', 'tramps', 'tracy', 'employer', 'heat-16', 'diagnosis', 'outages', 'lotto', 'stretched', 'politics', 'chokes', 'odious', "momma's", 'power-mongering', 'fortitude', 'unsatisying', 'off-track', 'personification', 'isolationist', 'shattering', 'pd', 'listen', 'suspension', 'pro-porn', 'trenchcoat', 'jeffries', 'spielberg', 'validation', 'chumming', 'mazzello', 'eaters', 'c-3po', "emmerich's", 'saunters', 'betraying', 'renders', 'estefan', 'drive-in', 'conspiracy', 'mascot', 'insubstantial', 'strapped', 'salma', 'hitmen--that', 'honda', "schrader's", 'bed', 'timmonds', 'metcalf', 'billiard', 'baby-boomers', 'giving', 'professional/leon', 'detail', 'midland', '_star', "'dogma'", 'evolve-they', 'hogarth', 'soul', 'moviegoer', 'restrict', 'entrances', 'campaigner', "rick's", 'tomas', 'warrant', 'finished', 'combating', 'ingenuous', 'annie', "son'a", 'hushed', 'lamas', 'rejoice', 'commercialize', 'trout', 'liar', 'latin', 'marridge', 'undo', 'crave', 'anti-fans', 'petaluma', 'poolside', 'lease', 'license', 'schrader', 'exasperating', "mars'", 'attack', 'nearly-farsical', 'high-grossing', 'keeley', 'holt', 'ozon', 'backpedal', 'rarely', 'trajectory', 'destructive', "belly'", 'sobbing', 'patrols', 'bahns', "'cinderella'", 'mat', 'synthesier', 'menacing', 'high-ranking', 'exerts', 'returnee', "hot'", 'hugo', 'ordinaryness', 'less-arrogant-than-usual', 'shortcoming', 'wich', 'pornographic', 'reich', 'carlo', 'coastal', 'beatty', 'deer', 'photo-copied', '_come_', 'racially', 'faltered', 'blech', "alma's", 'nice-looking', 'kiefer', 'trois', "sagan's", 'asteroid', 'blatancy', 'miami', 'grosses', 'landmarks', 'fearlessly', 'gunned', 'reward', 'strokes', '_scream_', 'infrared', "blair's", "pipstone's", 'feel-the', 'delinquent', 'sharman', 'tissue', 'wrists', 'curveballs', 'jot', 'hash', 'injected', 'enlists', "benson's", "'psycho'", '`jeez', 'casey', 'chip', 'psychotherapy', 'flash-back', 'kfc', 'counteract', 'semblance', 'fedex', '103', 'lameness', 'seducer', 'offensive', 'aa-meetings', 'vehicular', 'stations', 'top-billers', 'falsetto', 'cannibal', 'repairman', 'non-sensical', 'gravitates', 'aunt', '`ends', 'by-standers', 'preteen', 'supplementing', "reviewer's", 'clownish', 'wreak', 'airs', "`praise'", 'enlisting', 'rochon', 'film--and', 'katt', 'mics', 'formation', "eliot's", 'sinners', 'bumpkin', 'keyboardist', 'borg', 'panama', 'taunts', 'chucklesome', 'coached', 'workaholic', 'cake', 'halfheartedly', 'sinyor', 'undocumented', 'doses', 'could--run', "ogre's", 'movie-of-the-week', "kersey's", 'notably', 'beetle', 'butte', 'dissing', 'compensate', 'union', 'hexagons', 'internal', 'agent/assassin', 'entertaining', "mistress'", 'figs', 'sprinklers', 'cowgirl', 'chastity', 'flash-bang', 'cross-examining', 'sonny', 'excluding', 'personally', 'joel', 'connors', 'runners', 'dane', 'palatable', 'occur', 'legends--those', 'cgi-enhanced', 'rocky', 'ishibashi', 'girl-crazy', 'oldest', 'purproses', "peploe's", 'sweetest', "mcteer's", 'homies', 'toulouse-lautrec', 'raymond', 'grungy-', 'retreat', 'tough-guy', 'mamet-style', '1968', 'confidante', 'salinger', 'marches', "'should", '129', 'appreciative', 'hiller', "i'm-having-a-mid-life-crisis", 'will--mere', 'bigfoot', 'double-bill', 'pave', 'santaro', 'munster', "hank's", 'richards', 'credible', 'authenticity', 'old-hollywood', 'mayhem', 'roan', 'smokealot', 'lien', 'bursts', 'hot-shots', 'billionaire', 'elicitor', 'hein', 'pads', 'har', 'necessarily', 'ballads', 'innate', 'zimbabwe', 'remote-controlled', 'alfredo', 'ons', "edgar's", 'risible', 'clients', 'noir', 'on-set', "pee-wee's", 'unsure', '[note', 'neatly', 'underpinnings', 'raid', "5'9", 'parallels', 'anticipated', "satellite's", 'gambon', 'unpaved', 'would-be', 'all-terrain', 'rediscovers', 'adolescent', 'fight--', 'squeaker', 'shorter', 'rhino', 'baddeley', 'covered', 'lawyers', 'kumble', 'multiply', 'deformed', 'necessitates', 'fabulous', 'skinny-dip', "laughton's", 'intrusions', 'keister', 'miscalculation', "sunhill's", 'stinker', 'vidnovic', 'fig', 'ceo', "browning's", 'branagh', 'delegate', 'not-', 'loot', 'macaulay', 'joins', 'partition', 'twofold', 'wishing', 'tactile', 'laurence', 'woman-child', "mallory's", 'pg-13', 'deals', 'preferred', 'mirth', 'stolz', "feeder's", 'summers', 'verber', 'fissure', 'ulee', 'jaw-dropping', 'dreamy', 'syllable', 'dario', 'minimizing', 'head-spinning', 'welshman', 'capitalizing', 'idealistic', 'inseminates', 'headphones/tiara', 'outward', 'fratboy', 'cadaver', '10=a', 'bleach', 'befalls', 'invaders', 'maniacle', 'nunez', 'reproduction', 'pixote', 'spineless', 'aquamarine', 'pieter', 'exposed', 'pre-production', "anakin's", 'convert', 'cackles', 'matsumoto', 'eye-opening', 'rodent', 'anney', 'calgary', 'babble', 'cremation', 'non-fictional', "incomplete-it's", 'cantakerous', '1871', 'traversing', 'comforts', 'anti-family', 'coin', 'discharge', "wainwright's", "carla's", 'profit', 'offices', "`promoted'", "exec's", "serra's", 'miner', 'liaisons', "venice's", 'brother-brother', 'zoom-ins', 'brought', 'emotion', 'sucker', 'defuse', "mickey's", 'brizzi', 'hammer', 'sandar', 'takei', "casey's", 'underwood', '1898', 'aquanet', 'needling', 'amended', 'anguish', 'urgayle', 'cockamamie', 'siskel', '_reach_the_rock_', 'splice', 'gaming', 'heigh', 'airplane', 'jailbreak', 'lovejoy', 'adjoining', 'misconstrued', 'ponder', 'guerrilla', 'classes', 'prejudge', 'backyard', 'chins', 'marched', 'jogs', '1998', 'black', 'objectified', 'melodic', "'i'm", 'pinching', 'reddish', 'ill-luck', 'sahara', 'aurally', 'itus', 'solved', 'sooooo', 'multi', 'crisply', 'face-painting', 'sanctity', 'cameo', 'filmakers', '56-year', 'clientele', 'kai', 'slasher-movie', 'sillier', 'breeth', 'slo-mo', 'pre-credits', 'karma', 'dispair', 'undiscovered', 'incredibility', 'voyeuristically', 'prairie', 'overwhelm', 'will/skylar', 'commodus', 'litters', 'beginning--while', 'terra-forming', 'biehn', "havisham's", "louie's", 'lowbrow', 'implied', "leguizamo's", 'attitudes', 'floods', 'drenched', 'rifle-shot', 'she-ra', 'mish-mosh', 'hudson', 'not-so-typical', 'harrowing', 'menacingly', 'pi', 'swigert', 'adapted', 'bicker-', 'offenders', "childhood's", 'death-obssessed', 'salinger-ism', 'suck-ups', 'harry', 'mirror', 'precarious', 'dynasties', 'bruklin', 'hit', 'debuted', 'cain', 'sweeten', 'puddy', 'kelsch', 'slavers', 'unhappiness', "parent's", 'militiamen', 'quasi-popular', 'badlands', 'pied', 'howler', 'neo-fascist', 'hibernated', 'vivian', 'blindsided', 'ditzy', 'bellboy', 'anti-matter', 'scoring', 'prone', 'kalman', 'dish-molded', 'entitle', 'quashed', 'huiessan', 'croupier', 'draw', 'redeem', 'day-to-day', "measurement's", 'fitzswalter', 'duet', 'oh-so-nice', '1793', 'prima', 'locusts', 'tops', 'tv', 'san', 'ineffable', "kafka's", 'collectors', 'fugitives', 'moist', 'pens', 'mistreated', 'follower', 'acrimony', 'exonerated', 'upstages', 'dormant', 'involves', 'windswept', 'spirals', 'conspiring', 'malintentioned', 'gascogne', 'vipers', 'amos', 'spilling', 'desire', "1590's", '[1', 'warm-hearted', 'sword-wielding', 'signifcance', 'repair', 'cohort', 'doe-eyed', 'huevelman', 'brrrrrrrrr', 'junichiro', 'squid', 'straight-forward', "joe's", 'smorgasbord', 'fitzgerald', 'inspires', 'loose-ends', 'orphaned', 'mandel', 'nah', 'run-through', 'duval', 'yuor', 'chronicled', 'barrel', 'blessedly', 'refusal', 'ultra-religious', 'newton', "'unhip'", 'plotless', 'unfair', "_don't_", 'tilts', 'tin', 'authoritive', 'shrubs', 'substitutes', 'whirs', 'hampton', 'vistas', 'mommy', 'blurs', 'chef', 'rejuvenate', 'laughable', 'good-looking', 'tenko', 'kirby', "avi's", 'shrimp-scarfers', 'mouth-to-mouth', 'freak', 'indefinitely', 'ladies', 'leeloo', 'trash', 'radio', 'daytime', 'bennett', 'anonymity', 'slow-dancing', 'sex', 'willa', 'wade', 'charms', 'camera', 'pseudo-feminist', 'ligaments', 'scolds', 'briers', 'resorting', 'pantomime', 'pornos', 'glengarry', 'angelo', 'flocked', "hitch's", '_red', 'saint-pierre', 'suppress', 'ebb', 'consult', 'licia', "sobel's", 'anastasia', 'truthfully', 'oh-so-wise', 'pledges', 'shamed', 'limon', "m'kay", 'nervousness', 'rehearses', 'hick-boy', 'spurning', 'nefarious', 'goldin', 'wrestlers', 'opts', 'pugnacious', 'butts', 'fei-hung', 'americans', 'tut', 'politician/developer', '86', 'marksman', 'bail', 'undermines', "wanna-be's", 'pleasantries', 'richard', "chuck's", 'karaszewski', 'on-target', 'heiress', 'grow', '_andre_', 'hogs', 'skillful', 'confucius', 'graciela', 'moptop', 'slow-mo', '4-inch', 'folklore', 'indiana', 'hitter', 'soccer', 'shelly', 'dandy', 'chores', 'mythical', 'tania', 'honed', '`there', 'action', 'vickie', "herbert's", 'toneless', 'guardrail', '1972', 'strongpoint', 'half-mast', 'shapely', 'wore', "buck's", 'lowdown', 'ferguson', 'girlfriend/moll', 'paces', 'dead-weight', 'mint', "novel's", 'viperous', 'caricature', 'targeted', 'peculiar', 'satisfies', 'spews', 'cafe', 'uninsightful', 'cable', 'lombard', "ten'", '_shaft_in_africa_', 'bewitching', "virgil's", '`tarzan', 'nivola', 'queasy', 'inundated', 'westerners', 'glimmers', 'chairman', 'cloth', "elena's", 'cheryl', 'calorie-free--so', 'frumpy', 'embarrsingly', 'mongo', 'self-involved', 'saddened', 'anakin', 'homogenize', 'mullan', 'make', 'hardball', 'commander', 'adian', 'a-team', 'culls', 'melaine', 'knob', 'pain', 'potheads', 'scraps', 'sprinkle', 'lieutenants', 'ex-porn', 'doomed', 'grip', 'sleepy', 'costa', 'hairless', 'peel', '-october', "stanton's", 'fiction', 'negotiable', 'moretti', "wolf-beiderman's", 'janitor', 'bloodshed', 'throats', 'thirsty', 'liberates', "macy's", 'me-he', 'loonier', 'expending', 'unbelieveably', "'controversial'", "kerrigan's", 'admire', 'misgivings', 'plot-related', 'bushes', 'heaven', 'sarossy', 'ally', 'proper', "wolf's", 'vocally', "secretary's", 'skull-faces', 'lazer', 'action-music-video', 'action-thriller', 'whovians', 'thulin', 'f-word', 'wright-penn', 'minimalistic', 'ventilation', 'shekhar', 'obsessional', 'spraying', 'oftentimes', 'roundabout', 'slamming', 'virgil', 'anand', 'acme', 'propaganda', 'factly', 'counsel', 'puncturing', 'confirm', 'althea', 'amazonas', 'disembodied', 'surly', 'situated', 'director/co-writer', 'confide', 'spillane', 'suvari', 'masterpiece-he', 'three-piece', 'spelled', 'tint', 'poitier', 'nomadic', 'maudit', 'trains/rocky', 'subdesarrollo--', 'nonmembers', 'attacked', 'goop', 'deathless', 'melts', 'picaresque', 'embarrassments', 'medieval', 'alluded', 'mo', 'talented', 'patrasche', 'prim', 'atrocity', 'hiroshima', 'hanif', 'we+ve', 'organa', 'scence', 'newbigin', 'exterminated', 'thrillerism', '[he', 'justin', "lugosi's", 'chocolate', 'neurotic', 'ventricle', "dar's", 'resuscitate', 'gatherings', 'sandals', 'fuhrer', 'download', "tarantino's", 'score-writer', 'twenty-somethings', 'bare-chested', 'enlivened', 'overpower', '99-cent', 'spyglass', 'story--it', 'softball', 'mercer', 'enables', "'you", 'cricket', 'elections', '`demons', 'audience-pleaser', 'bowel', "rock's", 'vondie', "wigand's", 'fucking', 'museum', 'hang-dog', 'buddy/weight-lifter', 'interruptus', 'estranged', 'undercuts', 'toddlers', 'parenthetically', 'loud', 'wounded', 'handphone', 'bloopers/flubs', 'rozencrantz', 'entrace', 'weaving', 'abc', 'melody', 'arian-looking', 'imposed', 'wars_', 'leaves', 'stolen-from-no-less-than-three-movies', 'blander-than-bland', "bronson's", 'fortune-hunter', 'trace', 'ah-ha', 'throes', 'super-hyped', 'emitting', 'employs', 'ten', 'hoobler', 'supererogatory', "hogget's", 'derailing', 'luchini', 'reiterate', 'mummies', 'jimmi', 'then-diana', 'etchings', 'unappreciated', "abigail's", "biel's", 'countenance', 'heroize', "jeunet's", "'secret'", '2020', 'post-war', 'lifeless', '300-plus-years', "authors-i've", 'terrence', 'hitchhiker', "thulin's", 'prematurely--not', 'bilingually-subtitled', 'soundstage', 'dune', 'time-space', 'ketchup', '[g]', 'system', 'godmother', 'odile', 'keogh', 'sweeping', 'advises', 'leuchter', 'colourful', 'cloths', 'enforcer', 'femme-butch-femme', '`when', 'mother-who-loves-you', 'paradoxes', 'telekinetic', 'disgusting', '80-foot', 'over-acts', '_his_', 'ominously', 'hurtling', 'shops', 'regurgitating', 'character', 'one-of-a-kind', 'watson', 'mutation', 'walled', 'ridgemont', 'fucks', 'unravel', 'achieveing', 'horizontal', 'deloreans', 'spits', 'keanu', 'sufis', 'pair', 'nicky', 'frenchmen', 'afternoon', 'role', 'sadly', 'mansley', 'starkness', 'mi', 'lowenstein', 'aristocracy', 'harlin', 'leticia', "warriors'", 'fyfe', 'undoings', 'figueras', 'woven', "hoggett's", 'mcclurg', 'double-teamed', 'legs', 'conversing', 'inter-woven', 'identically-wrapped', 'coldness', 'examines', "stanley's", 'schoolers', 'ventures', 'bode', 'aquarium', 'grassroots', 'panel-sized', 'co-hosting', "deliveree's", 'brown-haired', 'cockroach', 'even-handed', 'stringy', 'davidovitch', "'powdered", 'post-secondary', 'moodier', 'drug-related', 'lacklustre', 'cro-magnon', 'psychoanalysts', 'not-so-rightfully', 'refrigerates', 'squalor', 'reacquaint', 'role-playing', 'problem--clone', 'hardy-har-har', 'constellations', 'fuck-you', 'rosewood', 'electrifying', 'similarly-named', 'kickboxer', 'black-wannabe', 'paramedic', 'yo', 'indulges', "sexton's", 'credibly', 'paragraphs', 'matrons', 'egar', "hugo's", '30th', 'whodunit', 'celebration', 'howls', "consumer's", 'girlfriend', 'guiding', 'worldwide', '`true', 'locklear', 'dim', 'reality-based', 'doubly', "disco's", 'reissued', 'hails', 'reinspired', 'molester', 'under-fire', 'howard', 'admiration', 'velociraptor', 'six-million', 'yao', 'gelfling', "'dude'", 'charm', 'ming', 'ling', 'spatulas', 'knockout', 'trots', 'heeere', 'wallets', 'usurp', 'gloomily', 'coincide', 'plane', 'travel', 'oh-so-cleverly', 'zoe', 'placeholders', 'anti-violent', 'royal', "fans'", 'floop', 'self-described', 'lumpy', 'double-jeopardy', 'overshadowed', 'sidewinder', 'burkhard', 'swaying', 'saddam', 'homeless', 'affirmations', 'hip-talking', 'temporal', 'financially-strapped', 'androginous', 'siskel/ebert', 'seizes', 'slip', 'kubrickesque', 'oz', 'animatronics', "studio's", 'punchlines', "anyone's", 'government-controlled', 'replaces', 'stationed', 'kitt', 'guitarist', 'pussy/carlos', 'pupils', 'tersely', 'nervous', "vampire's", 'capably', 'eleveate', 'abyssinian', 'organizer', "d'artangnan", 'heavily-sponsored', 'risque', 'gayness', 'scooby-doo', 'nyu', 'hand-job', 'mirrored', 'housing', 'diaz', 'firefighters', 'time-', 'digestible', 'climb', "mcglory's", 'richness', 'enthrall', 'unbilled', 'larch', 'afoul', 'highly-stylized', 'wallet', 'yasmin', 'angstdom', "'hokey'", 'barn-burner', 'nanni', '57th', "k-she's-supposed-to-be", 'italicize', 'courageous', 'bridge', 'deserter', 'regurgitated', 'continuity', 'deflate', 'accusation', 'benign', "`gangsta'", 'existed', 'killers-with-hearts', 'kerri', 'began', 'masterpieces', 'gruff', 'fog', 'tritt', 'charlie', 'rudiments', '_fantastic_', 'dern', 'vela', 'yells', 'avoidance', 'spaceship', 'arthouse', 'mutter', 'sluts', "affleck's", 'restricting', 'olmstead', 'concoction', 'trailers', 'storms', 'goodall', "dino's", 'sate', 'scummy', 'estimate--but', 'houseman', 'ohh', 'smuggled', 'slaughters', 'murdered--forced', 'buoyant', 'indie', 'manhattan', 'in-game', 'aborginal', 'cheered', 'compared', 'chunk', 'mantlepiece', 'shampoo-like', 'subsidiary', 'winsomely', 'angelina', 'perfects', 'bone-rattling', 'criminal', 'glitches', "coens'", 'aiming', 'multiplies', 'respighi', 'arrive', 'chelsom', 'dukakis', 'gopher', 'ex-fiance', 'drowns', 'kitschy', '1/2', 'lair', 'suburban', 'winkler', 'recylcled', 'devastated', 'claimed', 'phantastic', "tillman's", 'kramer', 'multi-plot', "_scream_'s", 'binary', "`tremors'", 'hectic-but-overall-uneventful', 'stempel', 'psycho-playing', 'buchman', 'safety', 'darwanism', 'clubish', 'unexploited', 'implausible', 'super-soldier', "aim's", "'clean", 'cradle', 'id', "`ultra-stylish'", 'preservation', 'farina', '_october', 'patronized', 'startled', 'fingertips', 'agreeably', "morris's", 'actress/singer', 'redding', 'lodged', "reeves'", 'annex', 'unraveling', 'janet', 'greece', 'slavery', 'listening', 'ex-cons', 'mamoru', 'chock', 'admonition', "quack'", 'squadron', 'buffalo', '10th', 'readied', 'states', 'true', 'mopey', 'volkswagen', '21', 'sophomoric', 'unpopped', 'scrubs', 'murderess', 'followings', 'millionare', 'slickster', 'climax', 'deflecting', 'donation', 'forebearing', 'fret', 'hutter', 'proclivities', 'proprietor', 'strathairn', 'embodiment', 'pounce', 'goro', 'homilies', 'registering', "o'clock", 'aviator', "bond's", 'accents', 'pasted', 'drunker', 'naught', "dugan's", 'timer', 'free-associating', 'handfuls', 'chad-ness', 'resides', 'mortimer', 'wells', 'anvil', 'dalloway', 'pierce', 'sassiness', "england'", 'rusted-out', 'self-importance', 'accommodates', 'dolce', 'comeback', '_breakfast_of_champions_', 'infinitum', 'downshifts', 'extracurricular', 'satisfied', 'hassidic', 'steaming', 'negatively', 'indomitable', '_wartime', 'strident', 'harum', 'clooney', 'videotapes', 'pennebaker', 'cross', 'robbers', 'cusp', "lopez's", 'itchy', 'over-effective', 'uplift', 'dispite', '1-dimensional', "'true", "clyde'", 'twenty-four', 'activists', 'au/~koukoula/', '1-', 'granger', 'hook-hand', 'mutations', 'nooks', 'downcast', 'ludwig', 'robotics', 'rollergirls', "'chateau", 'bought', "goldman's", 'inside', 'optimistically', 'illicitly', 'amiss', 'nfeatured', "shang's", 'basically-big-roach-type-bug', 'impetuous', 'sure-fire', 'friction', 'gia', 'chaotic', "tiffany's", 'qui-', 'lansbury', 'conventional', 'curtiz', 'forming', 'even-tempered', 'contemplate', 'scrambled', 'dribble', 'aquatic', 'flourishes', 'caro', 'shoe', "marceau's", 'ritz', 'far--arguably', '[pg-13]', 'broken', 'brother-in-law', 'attended', 'deported', 'paranormal', 'earliest', 'trooper', 'taxed', 'simplicity', 'punishable', 'bizarre', 'bostwick', 'screened', 'not-too-distant', 'underling', "boneheaded'", 'scientologists', "clayton's", "life's", "family's", 'pitch-black', 'pudding', 'yvette', 'mustafa', 'zone', 'xenia', 'lasse', 'incongruous', 'essences', 'epitome', 'zi-yi', 't2', 'sould', 'plethora', 'coca-cola', 'rooster', 'vansihes', 'appeals', 'barks', 'referencing', 'newsmagazine', 'walks', 'ostertag', 'dowd', 'peanuts', 'in-your-face', 'rowlf', "fernando's", 'reverse', 'expel', 'spock', 'vcr', 'campery', 'bowls', 'hf', 'cho', 'viking', 'scantily', 'saga', "storm's", "jason's", 'accidentally', 'endora', "`mission'", 'lundy', "burton's", 'floats_', 'distancing', 'ashau', 'scenes--grocery', 'squish', 'fact', 'conventionalism', 'divinity', 'cking', 'pruit', 'antagonist', "'sex", 'famously', 'plot-by-numbers', 'scotia', "herman's", 'unflashy', 'love-dependent', 'beastmaster', 'snap-brim', 'ignoring', 'massaging', 'engrosses', 'grinds', '15', 'stella', 'scrapped', 'helper', 'salads', 'rainfall', 'harvests', 'lobotomy', 'wormer', 'daughter-in-law', '800', "uhry's", 'waft', 'regis', 'contortions', 'world--what', 'decapitations', 'olympia', 'intersecting', '1937', "walt's", 'conqueror', 'ciaran', 'boobs', 'out-of-step', 'hot', 'stipe', 'elmaloglou', 'tolkan', 'rocks-actually', 'portal', "'r'", 'angelic--in', '4', 'shopgirl', 'hind-ends', 'stoic', 'timelessness', 'one-two-punch', 'seduced', 'genxers', 'stridency', 'stepsister', 'slasher', 'denzel', "russel's", 'educator', 'distilling', 'severity', 'browning', 'videophones', 'shogun', 'slater', 'lesbianism', "dallwitz's", 'swear', 'creepily', "sctv's", 'surfing', 'dallben', 'gilles', 'emphasis', 'amrriage', 'tears', 'affront', 'muses', 'cinematography', "'90's", 'slattery', "early-'90s", 'possessive', 'contempt', 'centerpoint', 'diffuse', 'kind-hearted', 'ridiculity', 'diversion', 'part-time', 'gazarra', 'hole\x14', 'mind-boggeling', '34-degree', 'lazy', 'balasko', 'freedoms', 'spoofy', 'approximately', '`taxi', 'kennif', 'sheldon', 'dissimilar', 'busload', 'stirrup', 'zaire', 'blended', 'cabin', 'soporific', 'movement--without', 'chants', 'dominique', '_really_', 'lon', 'incompetence', 'stupidity', 'kgb', 'horror/schlock', 'penthouse', 'cheerfulness', 'caution', "`zulu'", 'perfected', 'pinon', 'sophie', 'crummy', 'reaper', "fleischer's", 'faithfully', 'danna', 'tuptim', 'tia', 'suppose', "zoolander's", 'amaze', 'cast--', 'freaky', 'nominal', 'movie-ness', 'copycats', 'ups', 'redd', 'in-', 'heroes', 'ignites', "ghoul's", 'pixie-hairdo', 'synergy', 'understood', 'moviestar', '\x13ripley', 'all-consuming', "lynn's", 'unexplainable', 'darkness', 'nicol', 'demi-god', 'chortle', 'disdainful', 'speaks', 'fury-161', 'torpedoes', 'proverbs', 'snobby', 'foregrounded', 'baby-sitting', 'coquette', 'donager', 'bard', "don'tcha", 'savior', "squad's", 'composition', 'tyranny', 'brutality', 're-unite', 'cock-eyed', 'semi-noir', 'hottest', 'peculiarly', 'tanktops', 'sleigh', 'tastelessness', "hoblit's", 'guillermo', 'throw-away', 'kilrathi', "geno's", 'expatriates', 'obeys', 'banjo', 'replacements', 'enchantment', 'sadie', 'station-giveaway', 'chimp', 'rated', 'skateboard-related', 'damned', 'seducing', 'livingston', 'focused', 'gungans', 'airtime', 'thunderlips', 'zanetti', 'biner', 'savages', "artist's", 'aligned', 'trustworthiness', 'delicious', 'involuntarily', 'maniacal', 'outing', "'well", 'amish-mocking', 'disdain', 'nitpicked', 'beanfield', 'slight', 'drugging', 'reprehensible', "squaresoft's", 'fireballs', 'smokey', 'belgian', 'con-man', 'nickelodeon', 'executing', 'purer', 'roadster', 'character--a', 'grips', 'ex-co-', 'filmgoers', 'schemes', 'designers', 'perlman', 'auschwitz', 'absurdist', 'beguilingly', 'beneficial', 'whimper', 'pork', 'all-around', 'intruder', "israeli's", 'standby', 'munundating', 'rushbrook', 'carrol', 'smitten', 'axiomatic', 'foreign-guy-who-mispronounces-english', 'penniless', 'conisderation', 'shields', 'intereference', "toback's", 'liased', "l'atalante", 'half-assedness', 'hsaio', 'central', 'cracking', 'poverty', 'bungle', 'count--at', 'darryl', 'take-it-or-leave-it', "barry's", 'together--feuding', 'flamethrowers', 'shrugs', 'adorableness', 'beckon', 'pixar', 'turteltaub', 'unfazed', 'narratives', 'generically', 'insisting', 'things--start', 'strayed', 'weatherman', 'meatiest', 'christian', '3-d', "herskovitz's", 'antiquated', 'stuff', 'utopia', 'kei-ying', 'man-whore', 'fishing', 'cornball', 'tape-worshipping', "one's", 'retaliating', 'award-worthy', 'pseudo-documentary', 'translators', 'definable', 'lyric', 'harelik', 'yeshiva', "tings'", "ross'", 'gevedon', 'follow-up', 'backtracks', "wayland's", 'uprisings', 'beta', 'mailman', "'double", 'retarded', 'idziak', 'whine', 'hacky', 'hounding', "ling's", 'symbolism--in', 'incompleteness', 'felliniesque', 'oreo-munching', 'ucla', 'moment-for-moment', 'exacty', 'mercutio', 'bouchez', 'hauteurs', 'relish', 'euphoric', "leder's", 'tippi', 'unreasonably', 'adapters', "blade's", 'positions', 'placate', 'dreadful', 'fosdick', 'horseplay', 'five-finger', 'superintendent', 'full-force', 'extramarital', 'band', 'effected', 'hua-chi', 'faux', "heroes'", 'unlistenable', 'stormy', "gnosis'", 'soggy', 'revitalized', 'outlines', 'boatwright', "boat's", 'immerses', 'suddeth', 'motorboat', 'steenburgen', 'prompter', 'packard', 'bastard', 'chick', 'wayland', 'borrowing', 'grasshoppers', 'nemesis', '`they', "kellogg's", 'seated', 'parkers', 'fred', 'convalescence', 'overly-officious', 'baseless', 'interface', 'gloria', 'raccoon', 'man-in-black', 'deane', 'overjoyed', 'delude', 'stripped-down', 'mystically', 'phylida', 'has-beens', "gabriel's", 'gadget', 'insensitive', 'fern', "columbus'", 'themself', "kinnear's", 'perversions', 'hanging', 'ethnic', 'shelling', 'pilkington', 'pacification', "stephie's", 'goldman', 'overhaul', "courtney's", 'sith', 'contentment', 'catalysts', 'unstimulated', 'lapsed', 'bracco', 'dumbed', 'boy-gets-killed-in-horrible-accident', 'foot', "dien's", 'sapping', 'dissertations', 'zakharov', 'grabbing', "besson's", 'bringing', 'margerithe', 'council', 'adores', 'edit', 'no-nudity', 'rowe', 'mechanical', 'trials', 'babe-a-licious', 'scariest', 'soon-to-be-committed', 'bruno', 'possiblities', 'thumb-sucking', 'unimitible', 'sunk', 'morant', 'spindly', 'miller-ey', 'curves', "knockin'", 'barb', 'costar', 'saddening', 'whizzing', 'unmistakable', 'geopolitics', 'analogy', 'gwythants', 'deja', 'voyages', 'pete', 'outcasts', 'blackness', 'satisfaction', 'redford', 'consume', 'knocked', 'comics', 'edition', 'preceeded', 'flunky', 'temple', 'rahman', 'link-ups', 'voyage', 'mcmullen', "freddie's", 'wide-angle', 'phrygia', 'kareem', '_happen_', 'flutist', 'buehler', 'ralph', 'starlets', 'machine--a', 'hopeless', '1908', 'gaghan', 'offerings', 'videos', 'aboriginal', 'guilderland', "vaughan's", 'terrrible', 'character-', 'brazenly', 'dry-cleaned', 'percussion-heavy', 'on-', "michele's", 'roommates', 'four-armed', 'prances', 'interviewer', "broderick's", 'invest', 'molesters', 'flynt', 'nearby', 'surrounded', 'interred', 'curdling', 'forfeited', 'no-brainers', 'smirking', 'scoundrel', 'reins', 'fahdlan', '_require_', "`i'll", 'spy-girl-for-the-bad-guy', 'realities', 'laurene', 'award-winning', 'savor', 'merchant-ivory', 'violets', 'positives', 'unsupportive', 'wwii', 'glazed', "1988's", '26th/1998', 'deprive', "hanako's", 'rousingly', "silverstone's", 'writer/performer', 'whore-chasing', 'fatuous', "willard's", 'mtcts2', 'fate', 'veinotte', 'splicing', 'progressed', 'involvement', 'downsized', 'outcast', 'foreclosure', 'attracted', 'dorky-looking', '1899', 'newland', 'dared', "stein's", 'ghibli', 'fuels', 'zipped', 'scanner', 'reassured', 'downpour', 'wilma', 'chooses', 'with--which', 'unmotivated', 'delineated', 'finally', 'vy', 'clubber', 'cb', 'six-year', 'whedon', 'robbed', 'butler', 'flagging', 'showings', 'puff-piece', 'goldenthal', "abc's", 'megabytes', 'bilko', 'snub', "bowfinger's", '113', 'smooth', "qt's", 'creepiness', 'piller', 'starlet', 'chump', 'tandy', 'perpetually', 'notice', 'overrun', 'ardent', 'sadler', 'instict', 'patron', 'realisticaly', 'ridiculed', 'mad-scientist-in-bavarian-castle', 'meatier', "demented's", 'star-studded', "someone's-in-the-house", 'evangelista', "kaisa's", 'holdup', 'movie-plot-ridiculity', 'birdy', 'exhilarating', 'pg', 'hedley', '2010', 'restatement', 'maids', 'shawshank', 'stabbing', 'anti-gravity', 'cheap', 'sniper', 'brain-child', 'travel-bookstore', "kyle's", 'enzyme', 'gauthier', 'yearns', 'drift', 'distraught', 'uncovers', 'loki', 'romanced', 'sprite--this', 'mouths', 'fare', 'spheeris', 'earthquake', 'masterfully', 'mcgillis', 'harmed', 'landowners', 'breezes', 'classmates', 'korsmo', 'intimidates', 'alphonsia', 'swamps', "shue's", 'kidnaps', 'curtain', 'archimedes', 'mineral', 'nemeses--shaken', "1987's", 'reversalf', 'losing', 'transmit', 'psych-tests', 'reptilian', 'federal', 'co-wrote', 'goodhearted', 'matter-of-factly', 'demonstration', 'guiteau', 'sytable', "coffey's", 'contradicting', "makin'", 'pageant', 'charlotte', 'turf', 'fling', 'algar', 'moral', 'fetched', 'sophisticated', 'stapleton', 'this-is-what-we-think-kids-want-to-hear', '_48_hrs', 'sightings', 'teenage', 'nelken', 'beal', 'circling', 'seriously--god', "waves's", 'implanting', 'bandaged', 'stagy', 'zhang', 'genie', 'sonnets', 'schnook', 'freight', 'handstand', 'brannon', 'ever-partying', 'huddle', "project's", 'questioning', "rodman's", 'rheinhold', 'flash-forward', 'assumptions', "'ghostbusters'", 'father-son', 'q-man', 'chaired', 'limbaugh', 'idiotic', 'familiar', 'save', 'thyme', 'alice', 'frenzied', 'hacker', 'mise-en-scene', 'arm-wrestling', '60', 'thx', 'cinematographer', 'interim', 'furball', 'writer-director', "protagonists'", 'moments', 'fairy-like', 'sequel', 'spherical', 'picture/love', 'furthering', 'mustered', "'philosophy", 'facilitate', 'falsehood', 'bishops', 'impassioned', 'firelight', 'footage', 'uninvolivng', 'omnipotent', 'polarize', 'barges', 'perkily', 'clenched', 'worrall', "producers'", 'nether', 'roughneck', 'good-not-great', 'rebhorn', 'lip-stick', 'finely', 'shield', '70-something', 'valued', 'harrigan', 'vivica', 'pantolinao', 'nastiest', "horrocks'", 'lindy', 'klebold-harris', 'escobar', 'imgen', 'mews', 'babe', 'axel', 'owns', '70s', "member's", 'jointly', "host's", 'rebbe', 'reactionary', '1947', 'dangereuses', 'expansive', 'queries', 'solvent', 'performances-', 'mounting', 'carnal', 'majors', 'exquisite', 'bedtime', 'virulent', 'sullen', "financee's", 'stanislas', 'clinched', 'pleaser', 'entrapment', 'muscles', 'fucnniest', "romulus's", 'sooo', 'joe-wreaking', 'briar', 'ambient', 'randolph', 'shakur', 'rapidly', 'cemented', 'bryan', 'gall', 'unexistent', "8's", 'vincent', 'stargher', 'phony', 'tensy', 'walters', 'rudner', 'scrawled', 'toils', 'cretaceous', 'deguerin', "zoom-in's", 'raindrops', 'don\x12t', 'hussein', 'single-digit', 'twenty-pound', 'flipping', "ratzo's", 'clifford', "modine's", 'noble', '`captain', 'tahoe', 'off-post', 'shies', 'feats', 'stance', 'continue', "portman's", 'starred', 'puking', 'childishness', 'trying-to-be-hip', 'anders', 'front-line', 'tuneful', 'slickest', 'untidy', 'bong-hitting', 'bourgeoisie', 'tropical', 'set-up', 'debates', 'groupie', 'example-', 'conspirator', 'hope-horror', 'flambeeing', 'non-', 'pheneomena', 'slammed', "friends'", 'tellingly', 'tri-star', 'woeful', 'depressingly', 'heaping', 'geared', 'season', '_titanic_', 'secret', 'globus', 'manhatten', 'flavor', 'frantic', 'prompts', 'hard-driven', 'salutory', 'chicanery', 'mcmullen-style', 'caldicott', '2040', 'broadway', 'resistant', 'berkeley--er', "enterprise's", 'doubtfire', 'flatulating', 'desiring', 'overcame', 'funk', "lucas'", 'bemoans', 'snuggles', "'leaving", 'less-manic', "garbageman's", 'englund', 'sound-bite', 'tawdry', 'blocks', 'knits', 'relationship', "fleming's", 'starting', 'boost', 'eye_', 'beamed', 'massacusetts', '26min', 'mouthed', 'mustache', 'moonlighting', 'stifled', 'co-owner', 'detached', 'channels', 'sanderson', 'areas', 'toho', 'supercilious', 'vocalist', 'step-father', 'accounts', 'keystone', 'abbotts', 'crisis', 'much-needed', 'unassailable', 'sentencing', 'pictorial', 'f-18', 'nandita', 'bureaucrats', 'top-notch', 'publically', 'perfectionist', 'lawerences', "clarity's", 'separates', 'draconian', 'aerial', 'ekberg', 'foxx', "tommy's", 'portions', 'free-wheeling', 'surely', 'helpings', 'penguin', 'renound', 'plague-riddled', 'gardener', "'perfect'", 'humidity', 'chicago-bound', "france's", 'gloat', 'mind--charlie', 'rigid', 'overburdened', "malka's", 'b-----d', 'hissing', 'carry-on', 'distended', 'heel', '_casablanca_', 'off-chance', 'trial', "mao's", 'nears', 'arm-chair', 'cute-ified', 'head-scratcher', 'memorably', 'cruella', 'polito', "disgusting-for-disgustingness'-sake", 'scene-by-scene', 'intentions', 'androids', '108', 'heralded', 'dim-wittedness', 'vip', 'half-machine', 'tangling', "wars'", 'dice', 'clerk', "progress'", 'game-plan', 'misprinted', 'branched', 'newcomer', 'centered', 'awk', 'all-purple', 'reitman', '1975', 'cristof', "'lucky'", ':', 'slimy', "piller's", 'preyed', 'inhereit', 'manhunt', 'distressingly', 'hesitant', 'utmost', "potter's", 'nikki', "waller's", 'pruitt', 'morlocks', 'eight-', 'prudent', 'irritating', 'gesture', 'side-effects', 'envigorating', 'garafolo', '360', 'jim', 'analect', "gadget's", 'fletcher', 'faint-hearted', 'boni', 'flooded', 'maverick', 'gestate', 'moment', 'post-w', 'koven', 'ever--but', 'stronger', "adam's", 'ritually', 'decker', '`who', '`faggot', 'paladru', 'non-sexual-yet-slightly-homoerotic', 'fence', 'indistinguishable', 'caesar', 'shoreline', '99', 'empathize', 'solver', 'gratuity', 'big-league', 'series--was', 'seminal', 'arbitrary', 'ensure', "spader's", 'structures', 'teeth', 'tugboat', 'buketer', 'ivana', "arroway's", 'rumpled', 'naked', 'ambivalent', 'nabbed', "gio's", 'theaters', 'desperate', 'smart-alec', 'bright', 'seething', "cabbie's", "skipper's", 'qualified', 'melinda', 'entity', 'lan', 'veteran', 'moralistic', 'colisseum', 'melted', 'lisa', 'thrusting', 'proof', 'jurassic', 'fish-', 'near-sighted', 'make-up', "burwell's", "hawke's", 'bedded', 'yugoslavians', 'horror', 'loyalties', 'equality', 'vilmos', 'ponderous', 'man-portable', 'candle-lighting', 'whore', 'bible-quoting', 'flaky', 'bellbottoms', 'well-produced', 'lip-reading', "tyson's", 'stacks', 'neighbor', "nyah's", 'endorsement', 'pollsters', 'emphatically', 'concussion', 'fearfully', 'lad', 'dither', 'caerthan', 'poppa', 'thirty-year-old', 'mentos', 'braga', 'gershon', 'inherent', 'porcelain', 'one-time', 'tunes--and', 'efficient', 'therapist', 'outnumber', 'scolding', 'deleting', 'wily', 'latest', 'ahem', "korngold's", "kar-wei's", 'unlocked', 'buggers', 'displeasing', 'peoples', 'slimeballs', 'mumble', 'analyzes', 'flesh-eating', 'trawls', 'kasalivich', 'calling', 'lister', 'not-so-stunning', 'head-on', 'hormone', 'reversing', 'yared', 'unsuspecting', 'eviction', 'lettering', 'movie]', 'disasters', 'rotting', 'bulls-eye', 'announces', 'royce', 'competitors', '$90', 'oversexed', 'saturated', 'checkbook', 'chaplin+s', 'stripe', 'tenure', 'ladles', 'ill-natured', 'finishing', 'elvira', "tv'", 'afeminite', "company'", 'occupations', 'extracted', 'mid-1800s', 'pelicans', 'stellar', 'threatened', 'goluboff', 'midshipman', 'sneaked', 'voe', 'fanged', 'redheaded', 'randian', 'stylistically', 'scare-fest', "_eve's", 'mcconnell', 'pacing', 'tingle', 'settling', 'oracle', 'one-man', 'shine', 'blaine', 'gig', 'balki', 'prohibition-era', "kattan's", 'aww', 'menagerie', 'hummable', 'unassertive', 'sturm', 'narcotic', 'nuptials--to', 'visible', 'empath/psychic', "leone's", 'composer-lyricist', 'phantoms', 'hasta', 'color-by-numbers', 'pee', 'trainspotting', 'dinah', 'thong', 'compelling', 'bidding', 'ruination', 'hindi', 'techs', 'starship', 'kiss', "ebert's", 'wavering', 'honest-to-goodness', 'animted', 'choppily', "folk's", 'senatorial', 'superstardom', 'ern', 'flaunt', 'obscure', 'talents', 'evangelizing', 'happened', "nicky's", 'biblically', 'committed', 'manhole', 'malloy', 'labelled', 'grim', 'pendel', 'timing-oriented', 'ocious', 'estuff', 'lamarr', 'sheds', 'obtuse', 'dina', 'delro', 'holders', '_shine_', 'no-frills', "'greatest", 'ummm', 'feasibility', 'gloss', 'woolsley', 'boozing', 'supplant', 'tremendous', 'outpouring', 'imhotep', 'stratford', 'hobo', 'unchallenged', 'depicted', 'delay', 'sloppiest', "star's", 'majpr', 'bakshi', 'swastika', 'inflections', 'excitedly', 'marvelous', 'boastfully', 'zero-gravity', 'giles', 'baileys', "'30s-style", 'film--topping', 'francis', 'cramped', 'koepp', 'irvin', 'inexperience', '1709', 'cineplex', 'essay', 'alida', 'ahh', 'unexperienced', "`ute'", '1997', 'opposed', '`hello', '======', 'sprucing', 'soft-tossed', 'luger', 'desperados', "shindler's", 'emily', 'unwittingly', 'rediscovering', 'gloppy', 'align', "mast's", "`disneyfied'", 'uncertain', 'infer-red', 'platform', 'pretentions', 'ar', 'release--and', 'spirit', 'puppies', 'bonnie', 'frankenstein-style', 'overdo', 'it+s', 'allusions', 'carribean', 'doomsday', 'flummoxing', 'jealousy', 'buzzes', 'spunk', 'victim--her', 'lite', 'sopranos', 'distorted', 'curmudgeonly', 'avalon', 'soured', 'salt', '_practical', 'coast', 'superiority', 'jericho', 'puny', 'melora', 'prompting', 'timeliness', 'collision', 'cork', 'whitey', 'non-satirical', 'agent', 'tyke', 'ordinary--yet', "damned-if-you-don't", 'jan', 'veterinarian', 'unprofessional', 'schneider', 'weddding', 'vehicles', 'last-minute', 'suggestions', 'mazursky', 'three-year-old', 'tally', 'work', 'thermians', "amy'", 'tome', 'exaggeratedly', 'hoskins', 'hotpants', 'pressed', 'emanates', 'bandstand', 'reserves', 'prepared', 'surreal', 'grant/matthew', 'lore', 'reclusive', 'inveterate', '_huge_', 'intelligently-constructed', 'cheating', 'parochial', 'boat', 'departure', 'redirection', 'bombast', 'install', 'countdown', 'millionaires', 'clandestinely', 'scaled-down', 'sticking', 'gerrald', 'prayer', 'patched', 'not-so-talented', 'imelda', "menace's", 'handcuff', 'catcher', 'homecoming', 'lusty', 'luxurious', 'nicoletta', 'rh2', 'guilfoyle', 'jettisoned', 'redemptive', 'better---so', 'gould', 'blows', 'ly', 'cleansed', 'tweedy', 'lala', 'di', 'uuuuuuggggggglllllllyyyyy', 'wyle', 'ehren', 'glanced', 'scooby', 'weeping', 'aids-cautionary', 'overwhelming', 'sailing', 'strums', "daughter's", 'intricate', 'teacher-student', 'tray', 'jeffery', 'findings', 'built', 'zeta-jones', 'vixen', "'having", 'screen--the', 'snarky', '`oh', 'professions', 'oxford', 'proxy', 'leaders', 'kinder', 'pinkerton', 'criticise', 'lightsome', 'spun', 'editions', 'elliptical', "condon's", 'pitched', 'mclachlan', 'effects-extravaganza', 'mindsets', 'wolfgang', '_i_know', "leadership'", 'absinthe', 'supress', 'noooo', 'monk-like', 'bucket', 'imperfections', 'simplying', 'subtitles', 'dans', 'crotchety', 'mulholland', "kurosawa's", 'half-hearted', 'cornfields', 'masking', 'better-known', 'yavin', 'motor', 'distractions', 'recovery', 'crackle', 'soviet', 'paddle', '||', 'tens', 'methodologies', 'disneyland', 'titantic', '`eat', 'sympathized', 'git', 'burnt-out', 'existential', 'unstable]', 'plagues', 'surviving', "araki's", 'fighter-bombers', 'embracing', 'lem', 'slapped-together', "hildyard's", 'good-and', 'spotty', 'airy', 'associate', 'categorized', 'rehabilitated', 'sportsmanship', 'ledoyen', 'biography', 'monotone', 'mallet', 'interweaves', 'skirt-chasing', 'linkage', 'twelfth', 'inch', 'preordained', 'projections', 'mystery-horror', 'gen', 'determined', 'olenska', 'bulgaria', 'dental', 'heebie-jeebies', 'seeds', 'possibilites', 'twitches', 'entrepeneurs', 'resurgence', 'sydni', 'occupying', 'curing', '//www', '[theater', 'til', 'pastoral', 'undercut', 'wench', "neil's", 'sydney', 'hat', 'shortcomings', 'pm', 'reel-to-reel', '2/6/99', 'donors', 'speakers', 'film-music', '@#$^%', 'supplied', 'fehmiu', 'opar', 'transporters', 'dominio', 'grate', 'lull', "trilogy's", 'psycholically', 'transplanted', 'counterargument', 'narrated', 'missable', "wahlberg's", 'jackass', 'you+ve', 'betterment', 'oh-so-carefully', "dorn's", 'socialite', 'easily-readable', 'crusader', 'hollers', 'slays', 'cleverest', 'dorn', '2176', 'thisclosefromdeath', 'dreaded', 'independents', 'newsreels', 'boarded', '101%', 'stemmed', 'jarring', 'interchangeable', 'reboux', 'confessionals', 'excepts', 'tiranny', 'tv-idea', 'freemasons', 'renewal', 'doves', 'waves', 'tips', 'profitable', 'erstwhile', 'know-how', 'scissorshands', 'fuse', 'rounder', 'pikachu', 'precious', 'osmond', 'spots', "terribly-'90s", "eszterhas'", 'bondian', 'ulimate', 'crimefighters', 'pains', 'skills', 'master', 'knocked-out', 'satirized', 'loudmouthed', 'feature--', 'heart-felt', 'corporals', 'dead-zone', 'manip', 'reinterpretation', 'speilberg', "'five", 'traumatizes', 'volcano', 'hypnotized', 'browns', 'ben', "letterman's", 'gloating', 'glumly', 'breaches', 'leonardo', 'rota', 'wish-fulfillment', 'east', 'blob', 'bar/whorehouse', 'monster-instigated', 'argue', 'ethnocentric', 'kriss', 'non-related', 'begrudgingly', 'ghallager', 'giorgio', 'creativity', 'shovel', 'wondrous', 'tarantino-esque', 'hsu', 'bungee-jumping', 'dorrance', 'dimwit', "associates'", 'obscuring', '`mission', 'obstructions', 'eliot', 'recently-widowed', 'faggots', 'exhaling', 'ingeniously', 'copyrighted', 'cop-gone-bad', 'ibn', 'duloc', 'cuffs', 'flirtations', "`messiah'", 'merchandising', '59', 'whitaker', 'az--this', '712', 'imprison', 'overpass', 'siouxsie', 'shearer', 'awhile', 'sharks', 'cheesiest', "o'brady", "virgin's", 'rubbish', 'height', 'herzog', 'over-written', 'mtv', "'blew", 'uneven', 'shatters', 'culture', '1600s', 'coy', 'saigon', 'antony', 'germinate', 'harmony', 'sister-in-law', 'cyanide', 'boundary', 'choral', 'ussr', 'maddening', 'woman--another', "orion's", 'cyril', 'buenos', 'elasped', '_mafia_', 'unsinkable', 'cope', 'fantasizing', "parton's", 'fireworks', 'drawn-out', 'kaela', "'ghetto'", 'snakes', 'but-is-shot', 'accuser', 'triumphant', 'invocation', "monkey'", 'hc', 'littered', 'sucessful', 'plaster', 'unbearably', "`armageddon'", 'miyake', "'nikita'", 'brainy', 'wargames', 'gawky', 'soulless', 'strangling', 'hellhole', 'fulci', 'insistent', "duvall's", 'willard', 'dancing', 'shut', 'umbrellas', 'partnership', 'roughest', 'starves', 'lit', 'depict', 'stained-glass', 'ribbon', 'coherently', 'hypnotic', 'rutheford', 'rehabilitation', 'kimble', "'nother", 'gab', 'tentacles', 'broadly', 'vendetta', 'high-class', 'aztec', 'convolutedly', 'actor-turned-directors', 'ex-football', 'corinthians', 'well-meaning', "criminals'", 'stare', 'gimme', 'double-spaced', 'perusing', 'evers', 'china', 'greenstreet', 'honour', 'abortions', 'luster', 'redneck', 'canon', 'drive', 'jamie', 'roache', 'sauna', 'risky', 'outcomes', 'misspelling', 'electrify', 'windsor', 'action-crime', "cool's", "`battlefield'", 'rhetoric-spouting', 'necessity', "pixar's", 'conciousness', 'stratham', 'all-too', 'mentor-turned-evil', 'socially-acceptable', 'landing', 'quick-draw', 'speirs', 'oceanscape', "allie's", 'fetishized', '40', 'obliges', 'russel', 'commercial', 'foreigners', 'holliday', 'narrates', 'ascend', 'fast-talking', "on's", 'partner-wife', 'back-and-forth', 'noises', 'unpretentious', 'continues', 'orient', 'roundup', "bauchau's", 'much-discussed', 'combed', 'orleans', 'vota', 'unauthorised', 'stockpiling', 'medical', 'moderate', 'terrance', 'linger', 'what-', 'expect', 'tabloids', 'thrift', 'foresee', 'capitalized', 'reccurs', 'miasma', 'single-minded', 'advertised', 'counter-culture', 'insult-throwing', 'wonderful', 'ethnicities', 'internalized', '151', 'mcteer', 'horses', 'guided', 'auteuil', 'stripped', 'rampage', 'enticing', 'jessica', 'goth-looking', 'aspect', 'movies--the', 'oscar-nomination-worthy', 'flee', 'pbs', 'on-again-off-again', 'outfield', 'dominate', 'prosecuting', 'skillfully', 'moxxon', 'miniseries', 'innovative', 'bleeding', 'seat', 'incapable', 'ruins', 'licks', 'criticising', "saunder's", 'inclined', 'bums', 'unstoppable', 'plans', 'vaccine', 'downplaying', 'transmits', 'boggles', 'ruses', 'helpful', 'lovey', 'pecker', 'karey', 'likened', 'disturbances', 'underbelly', 'animated', 're-telling', 'wiper', 'scattershot', 'footloose', 'colon', 'us$100', 'mcfly', 'patriotism', 'junkie-i', 'expedition', 'bad-ass', 'inarguable', 'bertolini', 'gooden', 'griebe', 'retinal', 'atoll', 'captions', 'over-the-', 'madden', 'adolescents', 'jordanna', 'viciously', 'enlighten', 'dose', 'jackson', 'adjani', 'retrograding', "first's", 'brownie', 'hush-hush', 'elevating', 'tormenters', "game's", 'eva', 'watered-', 'horseracing', 'baptizes', 'mistakes', 'sweden', 'superlative', 'embarrased', "faith'", 'scariness', 'fugitive-like', 'cobble', 'picturing', 'self-control', 'scattering', 'purple_', 'mojo', 'scene--is', 'potbelly', 'ephron', 'avert', 'suppoosed', 'emrboidered', 'elexa', 'unmemorable', 'kara', 'remain', 'demographics', 'maintained', 'dozing', 'cashing', 'traumnovelle', 'us$68', 'raver', 'hotbed', 'character--but', 'replacement', 'duck', 'oakley', 'hitchhikers', 'nolan', 'accusing', 'taco', 'cohesiveness', 'rake', 'thailand', 'nationalist', 'romanticizing', 'antagonists-as-protagonists', 'kristy', 'percent', 'grunts', 'talon', 'unfrozen', "madonna's", 'works', 'seeking', '289', 'markets', '36', 'splash', 'canoe', 'granddaughter', 'mariachi--director', 'secondary', 'world-reknowned', 'wishes', 'hoopla', 'heading', 'sheperd', 'state', 'dressmaker', 'motel', 'atrociously', 'militants', 'upwardly', 'teachings', 'over-indulgence', 'fistfight', 'geek', 'dredging', 'enable', 'asides', 'whatnot', 'politcally', 'mistrustful', 'stew', 'unauthorized', 'envy', 'depaul', 'sadist', 'mis-guided', 'murch', 'sepia-toned', 'charlize', "francis's", 'self-acknowledgment', 'drummer', 'fine-looking', 'seltzer', 'denial', 'stress-inducing', 'distrustful', 'white-haired', 'goldberg', 'stranglers', 'tending', 'thunk', 'clause', 'delmar', 'sentient', 'lucidity', 'jakoby', 'icons', '60+', 'postcard', 'enormously', 'puppetry', 'umu', 'educating', "ames'", 'remorse--the', 'manifestation', 'crest', "spoon's", 'flounders', 'blume', 'movies', 'funeral', 'wisecrack', "ayla's", 'eminently', 'catchy', 'contentiousness', "harris'", 'ing', 'lollipop', 'get-out', 'effective--contribution', 'pro-soldier', 'ari', 'mitigated', 'revoltingly', 'black-clad', 'grunt-like', 'paper', 'diluted', 'london', 'mulan', '_what', 'analogue', 'reconciliation', 'decline', 'deflower', 'regret', "schreck's", 'researches', 'aghast', 'reenlists', 'rican', 'coens', 'shards', 'glassy-eyed', 'adventure/drama]', 'totally', 'backdown', 'drilling', 'oscar-nomination', 'super-computer', 'mede', 'searchers', 'sunburned', 'shaker', 'warfare', '-helmer', 'psychologically', "branagh's", "murdoch's", 'representation', 'entice', 'beast-people', "mandel's", 'insipidly', 'inspection', 'de-clawed', 'collected', '``high', 'speeches', 'stolen', 'watering', 'over-decorated', "arquette's", 'seek', 'appreciable', 'brady', 'links', '2nd', 'wise-cracks', 'stereotypical', 'hears', "model's", "endo's", 'between-the-line', "maurice's", "snail's", 'drags', 'all-stops-out', 'mogul/psychotic', 'persian', 'negates', 'rackwitz', 'bungles', 'aryans', 'seldon', 'pups', 'pre-conceptions', 'plugged', "dylan's", 'wiseass-is', 'atreides', 'training', 'drifting', 'unchangeable', 'boyish', "computer's", 'williamson--to', 'single-mindedly', 'riker', 'rested', 'finalized', 'caprio', 'recommended', 'mind-numbingly', 'kozmo', 'toying', 'romanctic', 'warning--spontaneously', 'feather-hatted', 'hot-wires', "craven's", 'action/animation', 'ghetto', 'pillaged', 'sweat-inducing', 'homogeneity', 'sam', 'facial', 'hays', '==========================', 'brilliantly-construed', "inmate's", 'interminably', 'overcome', 'contemplating', 'kathyrn', 'violates', '1950s', 'boy-loses-girl', 'enliven', 'acumen', 'locket', 'hatchett', 'archie', 'john', 'hartley-ian', 'adoration', 'fantasy', 'witzky', 'loincloth', 'wheezy', 'sends', "predecessors'", 'reverses', 'nations', 'unimpeachable', 'testosterone-driven', '4th', 'yatf', "joel's", 'der', 'blur', 'hasidics', 'fruitless', 'ingenue', "instructor's", 'scathing', 'darby', 'solely', 'golden-era', 'prom', 'rendered', 'ward', "richie's", 'pouncing', '_all_', 'procedures', 'labels', 'pinnochio', 'certified', 'rarest', 'insights', 'title', 'yellowstone', 'europe', 'tinier', 'undetectable', 'beautifully-rendered', 'teller', 'nass', 'sex-scandal', 'custodians', 'bluescreening', 'strangeness', 'polarization', 'sordid', 'reused', 'mobster', 'emperors', 'tend', 'stirs', 'obeidient', 'artillary', 'letter', 'steakley', 'commands', 'griffin', 'supplier', 'embroiled', 'blakely', 'courtship', 'tossed', 'reshoot', 'is-', 'comedians', 'raucously', 'ethics', '24th', 'exaggerate', 'huckleberry', 'glen', 'viewings', 'tuition', "dogma's", 'documentarians', 'whit', 'yearn', 'shauny', 'well-supported', 'betwixt', "caprenter's", 'always-', 'almanac', 'reagents', 'slimming', 'horfield', "houston's", "hunt's", 'roxbury', 'short-cut', 'homily', 'boogieman', 'reek', 'strickler', 'allegorical', 'indulged', 'mr', 'newman', 'gibson=92s', 'garners', 'lundgren', 'witch-related', 'thrusts', 'strips', 'missile', 'objects', 'knifed', 'obliteration', 'trusts', 'collides', "game'", 'dope-smoking', 'epstein', 'fiorina', 'crafting', 'incongrous', 'symbolizes', 'snugly', 'bawitdaba', 'rediculous', 'censoring', 'darthur', 'dushku', 'khanijian', 'recut', 'charged', "brothers'", 'wonie', 'chimera', '$29', 'briskly', 'carin', 'too-forgiving', "respighi's", 'brooke', 'doa', 'obscured', 'small-budget', 'schlub', 'recants', 'jailterm', 'philosophy', 'roaring', 'touch-up', 'circulation', 'norms', "webber's", 'truck-chase', 'brian', 'super-policeman', "beau's", 'reputedly', 'frey', 'incarceration', 'taran', 'swimmers', 'collate', 'instances', 'copycat', 'nozaki', 'ex-cia', 'inventory', '1980s', 'gregor', 'miniature', 'misses/catches', 'gentlewoman', "talkin'", 'co-', 'awwwww', 'unfathomably', 'sequel-crazy', 'cash', 'catacombs', 'anti-theft', 'questions', 'coaxes', 'infinitely', "al's", "'snuff'", 'reminds', 'savaged', 'non-thrilling', 'conor', 'theology', 'sweats', 'slaps', 'self-parodizing', '_cliffhanger_', 'licking', 'redcoats', 'pairings', 'teleconferencing', 'sentinels', 'galvanizing', 'unnecessarily--the', 'nineties', 'face-lift', 'rests', 'profundities', 'introduce', 'throwaway', 'childers', 'gutsy', 'personage', 'tunneled', 'limburgher', 'hospitalised', 'authoritarian', 'weirded', 'taps', 'japanese', 'scrubbed', 'judgment', 'cydonia', 'contorted', 'base', 'reversals', "'when", 'venna', 'beliefs', '_little', 'duveyrier', 'cut-and-paste', 'howington', 'gnaw', 'alabama', 'leeched', '`not', 'metaphors', 'hyperactive', 'spectaculars', 'trini', 'bashfully', 'stooped', 'nordern', 'esmerelda', 'powaqqatsi', '-courtesy', 'obsessive/compulsive', 'rogers', 'ratings-a-plenty', 'anchor/correspondant', 'horrifying', 'cements', 'mirthless', 'recreate', 'awakening', 'kissinger', 'sinner', 'knish', 'computer', 'iron-clad', 'bruskotter', 'trotting', 'semi-smiliar', 'enchanting', 'boatload', 'unveil', 'alejandro', 'bunching', 'proctor', 'diabolique', "riff-raff's", 'cooling', 'aykroyd', 'part-way', 'clair', 'stylishly', 'nouri', 'judaism', "murphy's", 'hundreds', '16mm', 'venora', 'clerical', 'attributes', 'fegan', 'butchery', 'stroke', 'dilemma', 'tasting', 'bowery', 'gawking', 'virginity', 'describing', 'mesmerizes', 'reese', 'tearjerkers', 'sillas', 'welcomes', 'oingo', "lancelot's", 'petula', "lamont's", 'diversions', 'alpine', 'fearless', 'ipkiss', "wyatt's", "society's", 'hitchcock', 'squad', 'disgruntled', 'hustle', 'lola', 're-mastered', 'article', "domingo's", 'razor-sharp', "carver's", 'disassociates', 'fugly', 'golden-boy', 'egg-eyes', 'ailing', 'pre-med', 'underline', 'life-ruining', 'respite', 'needle-jabbing', 'bull', 'hard-unearned', 'overestimated', 'superficially', 'ordeal', 'wilmington-as-detroit', 'potshots', "disappoint'", 'u-turn', 'krupa', 'rancher', 'overflying', 'costly', "peoples'", 'tarsem', 'remixed', 'zealously', 'hooper', 'guaranteed', 'engelbrecht', 'spoon', 'tm', 'fondled', 'ominous', 'pais', 'problem-solving', 'chess', 'hackers', 'two-hour', 'motorbike', 'com/~jimkendrick', 'lynchings', 'lots', 'unsuccessfully', 'detective', 'flawlessly', 'sector', '[warning', 'toolbox', 'single-mindedness', 'film--on', 'tic', 'side-kick', 'hairs', "lemmon's", 'sooner', 'wag', 'heartstrings', 'priced', "fern's", 'deapens', 'pop-broadway', 'remake', 'assembles', 'not--graphics', 'sniveling', 'bogged', 'human-animal', 'overplayed', '10-year-olds', 'literate', "spielberg-katzenberg-geffen's", 'tripper', "bill's", "ephrons'", 'time-frame', 'dea', 'hasidim', '5671', 'fretting', 'crapper', 'lv-426', 'sangfroid', 'squinty-eyed', 'assassin/exterminator', 'literacy', 'glance', 'kurosawa', 'irradiation', 'contrastive', 'conjures', 'pysche', 'wanting', 'lingering', 'instinctively', 'lions', 'we+re', 'ziyi', 'smirked', 'eighth-assedness', 'backs', '_election', 'television', 'terrorize', '\x05', 'on-key', 'potrayed', '8%', 'sea', 'clairvoyant', 'stoddard', 'word--of', 'taraji', 'hypothalamii', 'fishtank', 'hegwig', 'immense', 'consciously', 'wildly', 'raising', 'burt', 'topsy-turvy', "hicks'", 'clacks', 'yuppie', 'forlani', "'ghetto", 'trumpeter', 'comically-named', 'again-except', 'onscreen', 'morissey', 'noteriaty', 'weather', 'alleyways', 'invective', 'torpor', 'staros', 'plucks', 'manni', 'manson', 'non-horror', 'courtroom', 'gooey', 'handbook', 'corn', 'tank', 'penny-dreadful', 'experiment', 'negligees', 'rankin', 'open', 'dc', 'pretenses', 'uganda', 'programming', 'get-up', "magoo'", 'graynamore', "audience's", 'yuen', 'glam', 'foul', 'nazareth', 'tweaks', 'ireland', 'frustrating', 'information', 'defining', 'spoilers]', 'accepted', "ship'", 'giancarlo', 'kits', 'metaphorically', 'judson', 'pseudo-sensitive', 'hoot', 'reproduce', 'shorty', 'nationwide', 'counterfeiters', 'bea', 'demonstrated', 'medak', 'farley', 'ceremonial', 'loving', 'interchangable', 'churning', 'sometimes-sick', 'louuuudd', 'portraying', 'outspoken', 'implausibilities', 'hot-and-cold', 'weller', 'births', 'counselors', 'opinion', 'inborn', 'mob', 'innundated', 'successor', 'heroines', 'youull', 'squirm', 'equivalents', 'revelry', "sinclair's", 'tasteless', 'god-given', "'69", 'nearing', 'revived', 'savoring', 'edging', 'exhilarated', 'corwin', 'undifferentiated', 'evolution', 'trachtenberg', "rome's", 'perfomances', 'traffiking', 'nello', 'riches', 'spinosauraus', 'derives', 're-animator', "rd's", 'mcpherson', 'stature', "actor's", 'mikey', 'counterpoint', 'untouched', 'replied', 'hallmarks', 'orphan', 'pierre-loup', 'caste', "'the", 'dreyfus', 'foam', 'stats', 'decemeber', "issue's", "buehler's", 'bits', 'hardheaded', 'magnuson', 'household', 'resolute', 'hypocrite', "bottin's", 'random', 'adventure', 'showering', 'i\x12m', 'cyr', 'flash', 'lithium', 'invoke', 'raging', 'genitalia', 'evelyne', "landon's", 'thearapeutic', 'squirm-inducing', 'terrible', 'leave', 'wyoming', 'removing', "mcfarlane's", "pill-poppin'", 'terribly-plotted', 'term', 'aspired', "others'", 'fantasia', 'backstage', 'spoiled', 'best-seller', 'ramius', 'size', 'thread', "ray'", 'moi', 'ooh', '45-minute', 'lara', 'jargon', 'influence', 'bond', '47', 'wanker', 'tiredness', 'mentorship', '114', 'fruition', 'fills', 'catwoman', 'unoriginal', 'fondness', 'troublesome', 'habanera', 'jean-marc', 'radmar', 'northern', "rosenberg's", 'olin', 'greedy', 'castle', 'zombie-like', 'callously', 'hurts', 'sync', 'reassessed', 'aptly', 'maxims', 'plantation', 'platter', 'anytime', 'transit', 'rice', 'surfeit', 'mapplethorpe', 'tantric', 'factory', 'receptions', "hung's", 'intercuts', 'phenomenally', 'slickers', 'ryan/hanks', 'grim-looking', 'frothing', 'marries', 'grazia', 'novelist', 'audience-unfriendly', 'appliance', 'jogging', 'bidets', '-with-a-passion', 'abort', 'realism--but', "1950s'", 'all-powerful', 'zagged', 'treehorn', 'wind-tunnel', 'adheres', 'oh-negg-in', 'quatermass', 'preconceptions', 'symbolise', 'deaf', 'complementing', 'brain-damage', 'resolves', 'garnish', 'lau', 'nary', "'robocop", 'talkfest', '90210', 'moonshiner', "roberta's", 'ala', "'i", 'sorcery', 'rebs', 'chaplin-inspired', 'mechanic', 'graciously', 'hotchner', 'titillation', 'drama/comedy', 'late-entry', 'glum', 'posit', 'speeding', 'commiserate', 'gangsters', 'ps', 'oddball', '35-years-old', 'edel', 'mysterioso', 'observers', 'ping-bin', 'jabs', 'commensurate', 'abuzz', 'promoting', 'sounds', 'curmudgeons', 'elegantly', 'gugino', 'lewinsky', 'sobriety', 'co-penned', 'achoo', 'pig-keeper', 'string', '460', 'beginnings', "jarman's", 'irwin', 'lando', 'wilson', ',', '1942/1986', 'bazaar', 'museums', 'figurine', 'coverup', 'unsurprised', "capra's", "frank-n-furter's", "dietz's", 'leto', 'crop', 'depersonalized', 'saviors', 'sullenly', 'splendour', 'integration', 'raucous', 'mafioso', 'asian', 'singer/guitar', 'funny--it', 'cappie', 'android-wishing-to-be-human', 'irritation', 'recurring', 'whiffleball', 'disection', 'jewish', 'chuckled', "'innocent'", 'tie-ins', 'bursted', 'occasional', 'lifelong', 'abject', 'roaming', "merlin's", 'executive-produced', 'forementioned', 'softening', 'double-natured', 'painted--you', 'egotistical', "newspaper's", 'salability', "farrellys'", '139', 'soothe', '1583', 'paramour', 'flipped', 'mothers', 'menace--labor', 'stammering', 'geena', 'tee-shirts', 'frolic', 'gibbs', 'psychologist', 'alumnus', '\x13earth', 'fella', 'forks', 'stupidness', 'hate', 'internalize', 'regalia', 'heists', 'stupefyingly', 'crichton', 'start', 'chill', 'fashionable', "2's", 'deacon', 'pause', 'shakespearean-trained', 'chain-snatchers', 'stick-to-what-you-do-best', 'assailant', 'churchill', 'willingly', 'disenfranchised', 'titta', 'fanaticism', 'mangle', "visconti's", 'trousers', 'vubu', 'insouciance', 'none-too-obvious', "evening's", 'unresolved', 'hones', 'pg-rated', 'caffeine', 'wormhole', 'vita', 'aftermaths', "hoyts'", 'disbelieving', 'ryan_', 'viggo', 'northwestern', 'stroller', 'shortage', 'perfecting', 'crystal/hugh', 'brundage', 'rubens', 'civility', 'it--she', 'freezer', 'underdrawn', 'gratuitously', 'loyal', 'uno', 'squeezing', 'revelatory', 'self-defeating', 'yeeeeah', 'embezzling', 'non-kosher', "motta's", '000-a-week', 'cohn', 'emulating', 'blatant', 'quebec', 'start--pollock', "deniro's", 'moms', 'archenemies', 'mathematics', 'tierney', 'overthrow', 'vigilante', 'nine-year', 'multiple', 'lobbying', 'stark', 'maintain', 'score--i', 'textual', 'maryam', 'wreaks', 'indeed-we', 'mind-numbing', 'heresy', 'stephanie', 'redefined', 'admiting', 'kellerman', 'transpiring', 'liberal', "doctor's", 'broderick', "_it's", 'extraterrestrial', 'seasonal', 'trods', 'timing', 'injures', 'imparted', 'early', 'bartkiwiak', '93', 'calm', 'seagulls', 'rooker', 'magazines', 'assurance', 'behalf', "man-animals'", 'gerard', '`like', 'bayouesque', "quaid's", 'musicologist', 'abduction', 'skateboarding', 'youngest', 'rocker', 'motorcycles', 'floored', 'detraction', "luchini's", 'bacon', 'traps', "o'reilly", 'patriarchy', 'pows', 'slickly', 'canned', 'breathtakingly', 'multi-millionaire', 'thornton', 'twinge', 'reappearing', 'luck', 'endowing', 'kazdan', 'human-like', 'coals', 'bullhorn', 'chainsaws', 'corrupter', 'characterizations', 'guitry', 'boarding', 'gumption', 'fiscally', 'screenwriting-is-hell', 'day-lewis', 'stepmother', 'carted', 'lowell', 'in-laws', 'inventively', 'nighthawks', 'stephie', 'innumerable', 'leyland', 'blames', 'pretzel', 'culture-clash', 'sluizer', 'stormtrooper', 'inflict', 'defenders/video', "travolta's", 'lie', 'figueroa', "steig's", 'edifying', 'reissues', 'hounded', 'preference', 'principal', 'dying', 'rover', 'gramophone', "rubell's", 'upright', 'aren', 'subplots', 'envied', 'xxv', 'maniacally', 'gorey', 'bugsy', "wells'", 'sponge', 'guildenstern', "gershon's", "gunton's", 'ethnically', 'children-only', 'girls', "fuentes'", '3654', 'cosmopolitan', 'females', 'caring', 'denying', "ross's", 'caretakers', 'dilemmas', "lambeau's", 'clare', 'emotion-', 'deuce', 'practicing', 'well-spent', 'enhancer', "ruby's", 'ankles', 'gobbles', 'fumbles', 'clive', 'initiatives', 'vapor', 'rangoon', 'professionalism', 'fo', 'gignac', 'pausing', 'nicola', 'radiant', 'ashe', "kuzco's", 'competitor', 'overstays', "buscemi's", 'deus', '2002', 'marshall', 'mumford', "connor's", 'teen-speak', 'director-writer', 'whitfeld', 'individual', 'starve', 'it--people', 'excursions', 'colour', 'travelling', "dolores'", 'locate', 'beleaguered', 'arenas', "nyc's", 'skarsg', 'scurries', 'tolling', '$500', 'strolling', 'introspective', 'altman-esque', 'skins', 'olive', 'grounding', 'santo', 'hirschfeld', 'locus', 'dickens-era', 'darrell', 'royces', 'selfishly', 'detour', 'vengeance', 'quietly', 'blackmail', 'trimmings', "boy'", 'detonator', 'shannon', 'royalty', 'reticent', 'lotte', 'trashed', 'rubble', 'morality', 'pleasance', 'encompasses', 'depicting', 'graziella', 'commentary', 'surname', 'snot', 'borderline', 'bothers', 'resume', 'french', "beast's", 'kick-ass', 'punchline', 'lithuanian', '1914', 'muffled', 'flicker', "lucus'", 'catastrophic', 'maltese', 'disproves', 'vision-impaired', 'ulcer', 'proposes', 'melee', 'nigh-invulnerable', 'customs', 'dispatch', 'stirred', 'terence', 'proponents', 'exemplified', 'crouched', 'fetishistic', 'matthew', 'gear', 'backstabbing', 'punts', 'billing', 'dub', "schwarzenegger's", '_beloved_', 'janitorial', "it'", 'infection', 'schamus', "dobson's", 'healthy', 'inkling', 'celebrity-directed', 'kidnaper', 'dimitri', 'laserdiscs', 'implemented', 'zukovsky', 'brits', 'hall', 'gloomy', 'twist--people', 'buzzing', 'busted', 'combines', 'aunjanue', 'con', "boyfriend's", 'second-favorite', 'striptease', 'cabo', 'bebe', 'binding', 'crowns', '8-ball', 'escalate', 'celebrity-type', 'poor-rich', "davidtz's", 'programme', 'deliver', '4=disappointing', "karla's", 'decipher', 'vj', 'likelihood', 'baileygaites', 'tristar', 'horror-comedy', "burnhams's", 'miscue', 'second-grade', 'playoffs', 'goddard', 'rossum', 'characterizing', 'snappiness', 'stutter', 'spiral', '99%', 'medfield', '1862', 'womanizing', 'wide-screen', '18-year-old', 'proposterous', 'sublimely', 'foul-tempered', 'adagio', 'nihgt', 'snorting', 'gleeful', 'two-million', 'soulful', 'engineer', 'neo-gothic', 'complete', 'bing', 'scrawny', 'without--shock', 'com/hollywood/bungalow/4960', 'hydrophobia', "bowden's", 'donella', 'accidental', 'poorly-staged', 'nation-wide', 'lord', 'caterpillar', 'masseuse', 'dismisses', 'lugosi', 'income', 'michele', 'reachable', 'realms', 'unity', 'self-sustaining', 'assemble', 'nine-tenths', '1944', 'debate', 'blue-screens', 'flatliners', 'executioner', 'hallahan', 'fun-filled', 'mosh', 'oilrig', 'geeks', 'arrogant', 'three-week', 'plot-', 'vu', 'postage', 'nominatored', 'stood', "a-ha's", 'montage-backed', 'slump', "ain't-i-sexy", 'label', 'fop', 'zant', 'compact', 'impassive', 'boone', 'predecessor', 'then-unknown', 'angst-filled', "documentary's", 'lehmann', 'childs', 'villainous', "eisner's", 'chop-job', 'vampire$', 'hong', 'inches', 'virtual-reality', 'professionally', 'single', 'toned-down', 'treading', 'amourous', 'math', 'melbourne', 'indian', 'cobo', 'execute', 'tighter', 'punctuates', 'rotten', 'pleasent', '$14', 'commensurately', 'conspicuous', 'hippos', 'melancholic', 'marshals', 'larger-issue', 'nerd', 'ever-', 'anway', '80%', 'matrix', 'safely', 'urgencies', 'selena', 'cleon', 'practical-joking', 'mixed', 'continous', 'on-par', 'bitty', '-spouting', 'requirements', "anton's", 'gussied', 'traditionalist', 'parade', 'less-than-ideal', 'infantry', 'shanghaied', 'semi-mystical', 'leered', 'succeeding', 'chaser', 'purge', 'xxx', 'befriended', 'seismic', 'gabe', 'obstacle', "ape's", 'iago-like', 'ontkean', 'yanking', 'undoing', 'first-semester', 'arabe', 'kamikaze', 'hemp', 'gallantry', 'textbooks', 'pointers', 'docket', 'professor/archeologist', 'thirty-minute', 'register', 'ruled', 'double-duty', 'fatigued', 'moscow', 'dulles', 'coen', 'sofa', 'maker', 'stigmata', '44-year-old', 'elmer', 'lackeys', 'filled', 'enhanced', 'lowest-brow', 'mantegna', "picture's", 'sect', 'calvert', 'toppling', 'sex-filled', 'ad-lib', 'two-timed', 'louder', 'sideways', 'promiscuous', 'suviving', '80', 'praising', 'penn', 'diverge', 'ex-mining/maximum', 'hilarities', 'primordial', 'galaxy', '`but', 'interactivity', 'mathematician', 'aristocrats', 'cindy', 'constance', 'hangar', 'pre-frontal', 'be-littles', 'foul-ups', 'out-of-nowhere', 'harpoons', 'quibbling', 'fanny', 'ditches', 'obligation', 'generated', 'lesbo', 'presnell', 'tuck', 'barreness', '_in', 'only-children-has', 'mathew', 'rejecting', "doesn'", 'aggression', 're-hab', 'picky', 'companionability', 'counterprogramming', 'peacefully', "edition'", 'hazardus', 'shon', 'unrestrained', "popitti's", 'teen-agers', '$8', 'over-the-edge', "`marketing'", 'ingrown', 'chow-yun', 'picture-perfect', 'cult-favorite', "horta's", "hurlyburly's", 'sci-fi/action/comedy', 'rewritten', 'consists', 'boorish', 'bates', 'laziest', '100', 'sillyness', "long's", 'extends', 'toffler', '9-year-old', 'engross', "grabthar's", "obi-wan's", 'neon-lit', 'dew', 'mentionable', 'hunt', 'heavy-handedly', 'capital', "you'", 'boy', 'vampires', 'drink', 'trilogy', 'gratifying', 'jukebox', 'subsided', 'college-town', 'glenne', 'garrison', 'sailor', 'convent', 'fumble', '61', 'garofolo', 'vining', 'signature', 'over-stylize', 'wroth', 'drug-abuser', 'closet', 'dry', 'somewhat-childish', 'lifestyle', "time'", 'poolboy', 'seriousness', 'peddling', "magoo's", 'simpsons', 'shirts', 'track', 'mind-bogglingly', 'innocuous', 'diction', 'fully-ripened', 'ruddy', 'lark', 'albarn', "jasper's", 'blondes', 'identifiable', 'literature', 'condemns', 'code', 'reoccurrence', 'constitutes', 'xeroxes', 'yucked', 'profiteers', 'funnel', 'leeway', "mandingo's", 'parental', 'garde', 'kwai', 'emits', 'chattering', 'deliveries', 'jetliners', 'greased-up', 'astound', 'clitoris', 'replayed', '`beverly', 'parked', 'devastatingly', 'ultra-cheesy', 'synopsis', 'rescored', 'haise', 'model-friends', 'gathers', 'ghoulish', 'biases', 'artificiality', 'girl-friend', 'overwhelms', 'mayfield', 'reduces', 'attainable', '60-some', 'simultaneous', "replacements'", 'renny', 'politically-correct', 'concealing', 'frederic', 'jjaks', 'ethan', 'baby', 'mook', 'cop/fbi', 'vapid', 'dawn', 'guerra', 'talal', 'definitly', 'one-character-decides-not-to-return', 'cross-cultural', 'grappling', 'satirist', 'controversial', 'visits', 'biased', 'own--are', 'voyeur', 'el', 'fears', 'mired', 'unremarkable', 'constantly', 'depart-', 'books', 'screaming', 'garwood', 'already-thin', 'slum', 'scary-looking', 'wizardry', 'meeks', 'frankenheimer', 'woud', 'wagner', "`seven'", 'functioning', 'outdoors', "victim's", 'cabana', 'unimaginatve', "dudley's", 'revolted', "mazzello's", 'holism', 'apprentice', 'reverberates', 'flour', 'well-known', "berkley's", 'tie-up', 'sex-romp', "snl's", 'virtuous', 'massages', "'penalty'", 'thievery', 'suspensful', 'sexiness', 'taught', 'best-known', 'impacting', 'famished', 'introducing', 'punk-in-the-making', 'alien-possessed', 'vin', 'punctuate', 'purse', 'grains', 'cutting-edge', 'color-saturated', 'unmasking', 'rousselot', 'marian', 'precipitous', 'blind', 'killer-like', 'adversary', 'squint', 'vergil', 'noses', 'classy', 'underdogs', 'dom', 'stowe', "saddam's", 'decapitate', 'actosta', 'deep-core', 'blackmailer', 'demo', 'unparallelled', 'vichy', 'goregeous', 'comply', "priest's", 'film-schoolish', 'three-', 'pitch', 'griff', 'torro', 'feldman', 'private', 'bears_', 'despises', 'disarmingly', "`spiritual'", 'defense', 'student', 'lustily', 'adopts', 'fisherman', 'llewelyn', 'airbrushed', 'showcase', 'breakout', 'caldwell', "finney's", 'coruscant', 'damn-near', 'amiable', 'bar-owner', 'communists', 'misfortunes', 'lug-head', 'hingle', 'harshly', 'moustached', 'asp', 'poisoned', 'wagon', 'unclear', 'mrs', 'angelic', "perdy's", 'wooing', 'cyndi', 'extremel', 'michelle', 'johnnie', 'notorious', 'dalai', 'beaters', 'sighting', 'far-from-overpowering', '_titus_andronicus_', 'outfit', 'intimate', 'neurotic-looking', 'critics', 'docudrama', 'velvet', "camera's", 're-appear', "stoppard's", 'smelling', 'classism', 'once-linked', 'irritable', 'elder', 'dairy', 'pace', 'maxx', 'fest', 'yogi', 'ostentatious', 'lopsided', 'unfold', "leo's", 'follies', 'blank', 'merpeople', 'sorderbergh', 'radically', 'conduit', 'keyser', '`biggest', 'prestige', 'robertson', "novalee's", 'droids', "suspects'", 'souvenir', 'monarch', 'salvadoran', "audiences'", "beat'em", 'race-related', 'thirty-something', 'pits', 'soc', 'bump', 'murilo', 'machine-like', 'spitting', '_pollock_', 'gehrig', 'championing', 'heretic', 'curtin', 'lol', 'rounds', 'rewatch', "caesella's", 'mysteries', 'backus', 'haddonfield', 'niagra', 'therapy', 'up-front', 'broncobuster', 'funky', 'asks', 'overexposure', 'entering', 'file', 'ash', 'pretend', "christoff's", 'palestinians', "mussolini's", 'distribution', 'denounce', 'countered', '-2', 'casseus', 'sneak-ins', 'farren', "cartman's", 'bianca', 'weird', 'un-virginal', 'mini-skirt', "burbank's", 'hangs', 'osmosis', 'hips', 'ceremony', 'opinionated', 'persnickety', 'adorning', 'belated', 'flaus', 'months--due', 'litz', 'massachusetts', 'directon', 'lefty', 'a&m', 'like_blade', 'sagging', 'situational', 'overachieve', 'dethroned', "skulls'", 'erm', 'enchantingly', 'yakusho', 'unit', 'logy', 'mates', 'chittering', 'syndicate', 'iben', "a-changin'", 'uprising', "sisters'", 'propulsion', 'tavern', 'gourmet', 'jeopardise', 'comrades', 'zena', 'cockpit', 'venom8@hotmail', 'myserious', 'hero/antihero', 'layout', "version's", 'lange', 'tease', 'bologna', 'tom', 'type', 'assist', 'signified', 'disappearances', "renault's", 'half-brother', 'angular', 'peanut', 'grove', '$@^@', 'macneill', 'sobol', 'cutting-and-pasting', "regiment's", 'bow', 'aggravating', 'honest', 'operate', 'uncontrollably', 'trouble-making', 'morris', "gary's", 'ebert-written', 'darken', 'pedicure', 'advent', "tango's", 'gatins', 'bedtimes', 'complains', 'shags', 'looms', 'displeasure', 'linebackers', 'poseurs', 'coloreds', 'dope', "'everything", 'headlined', 'steep', 'canary', 'flubber-powered', 'haysbert', 'landsbury', "`no-name'", '2099', "rat's", 'him/herself', 'lake-type', 'logic', 'tonal', 'bird-loving', 'extraordinaire', 'jared', 'quick-edit', 'juggle', 'teed', 'half-bad', 'amis', 'two-and-a-half', 'moralize', 'nightie', "'whatever'", 'defusing', 'overnight', 'face-off', 'jerks', 'jacqueline', 'visceral', 'hollan', 'quayle-ish', 'spade', 'birch', 'salo', 'spottiswoode', 'pentagon', 'shao', 'studious', 'roving', 'avoided', 'likeability', 'calvinist', "japan'", "uncle's", 'measure', 'schwartz', 'jousting', 'starbucks', 'stavros', 'improperly', 'nightmare', 'commonplace', 'sailors', 'cystic', 'magenta', 'unlock', 'dreamscape', 'conciliatory', 'funnyman', 'hey', 'mcallister', 'telly', "news'", 'concorde', 'anna', 'humphries', 'acquired', 'tubes', 'settlement', 'dissolves', "pucci's", 'ressurect', 'bravely', 'swindler', 'tyrany', 'death', 'carol+ghostbusters=scrooged', 'atmospherically', 'sexpot', "ford's", 'warlord', 'correcting', 'chuckie', 'bumble', 'second-to-last', 'heels', "sister's", 'zooming', 'hairpin', 'complexities', 'glengary', 'silverman', 'triplets', 'royalist', 'aaliyah', 'inexorable', 'diplomatic', 'stalkings', 'jeremiah', 'journalists', "meerson's", 'transmitions', 'manga', "1971's", 'paws', 'anti-hero', 'scalawags', 'cigar-smoking', 'blandest', 'burning', "alicia's", 'spans', 'crazed-looking', 'connection', 'spare', 'reginald', 'extended', 'david', "o'harra", 'brennan', 'withholding', "'cute", '`run', 'savings', 'irrefutable', 'banners', 'frequents', 'relevance', 'dixon', 'toque', 'op', 'bitchy', 'clemente', "depardieu's", 'synchronicity', "'vampires'", 'land-once', 'ambers', 'canadian', 'rifle/camera', '#$%', 'schaech', 'popped', 'u-boat', 'conflicting', 'graininess', 'sluggishly', 'bills', 'basic', 'imaginatively', 'decked', 'lauren', 'lagging', 'branaugh', 'hay-soos', 'gregg', 'ingenuity', 'chenoa', 'terrifies', 'figgis', 'clump', 'blanketing', 'ravishingly', 'moreso', 'booker', 'bereavement', 'perrineau', 'direcing', 'legal-drug', 'felony-studded', 'gather', 'herky-jerky', 'mobile', 'dovetail', 'tardis', "i's", "yost's", 'health', 'memorial', '-4', 'modernized', 'sensing', 'prostheses', 'smirky', 'hokum', "`work'", 'boy-and-his-dog', 'charmless', 'demotion', 'installed', "boys'", 'merienbad', 'opportune', 'rewards', 'chiseled', 'evens', "fbi's", 'washing', 'duarte', 'finger-pointing', 'crucifix-weapon', 'fore', 'wrong-side-of-the-tracks', 'mare', 'buggs', 'krabb', 'puppy', 'whacked-out', '11', 'unenjoyable', "jessie's", '2-d', 'pristine', 'prevue', 'pileggi', "`go'", 'secondhand', 'demonstrably', "hernandez's", 'spontaneously', 'embodying', 'vast', 'doff', 'morocco', "rampling's", 'landowner', 'project--sweet', 'supburb', 'straight-laced', 'scars', "peel's", "vail's", 'populous', 'mckenney', 'placing', 'parent', 'plot-holes', 'meaning-free', 'bug', 'guzzling', 'sunlight', '500-like', 'maps', 'scrabble', "clinton's", 'myagi', "green's", 'parent-koening', 'speedboat', 'durham', 'nsync', 'tangiential', 'agent-in-charge', 'seague', 'slapstick', 'grab', 'slater/johnny', 'rev', 'posse', 'undead', 'acrobatics', 'contrivances', 'underachieving', 'pollack', 'kiki', 'renfro', 'founder', 're-made', 'tour-', 'poe', 'interviewing', 'mountaintops', 'lima', 'characterisation', 'summaries', 'made-up', 'entities', 'beast-men', 'grown-up', 'pages', 'imho', 'hauer', 'grizzled', "why'd", 'portrait', 'juggernaut', 'tolerates', 'subtlety', 'jump-out-at-you-from-the-dark', 'insomniac', 'cubism', 'donkey', 'furnishing', 'online', 'accurately', 'lovingly', 'arlene', "'82", 'dispatching', 'digestion', 'rosio', 'spice', 'skif', 'upsets', 'bonkers', "allayah's", "'em", 'symbolizing', 'modeled', "bloomin'", 'mail', 'buggy', 'nugent', "nixon's", 'multi-layered', 'goodwin', 'sinister-sounding', 'current', '`the', 'boring/confusing', 'pinnacles', 'resurrected', 'guesses', 'screendom', 'confront', 'complacency', 'maximum', 'greenlit', "gulliver's", 'consanguinity', 'often-invisible', 'manifestations', 'terrain', 'globes', 'extentions', 'border-jumpers', 'archetypes', 'faiths', 'specialized', 'echoed', 'doppling', 'nastiness', 'laughless', 'dynamite', 'rangers', 'finite', 'valleys', 'rehashing', 'litefoot', 'engrossment', 'salle', 'kirsebom', 'yellow-greens', 'overprotective', 'full-speed', 'speculations', 'noting', 'ooze', 'amnesia', "hippo's", 'retro-garbo', 'keats', 'hilarous', 'chilled', 'commented', 'family', 'quinn-robin', 'pistols', 'proclaims', 'pammy', 'unconditional', 'close-up', 'arthurian', 'idioms', 'smoothly', 'premature', 'aram', 'germany', 'segal', 'gallant', 'storybook', 'ferrets', 'pricks', 'stinkers', 'scorsese', 'canvas', "'evolution", "'new'", 'shallows', 'matchmakers', 'quick-cut', 'art-house', 'biz', "schaech's", 'cloying', 'fuzzy', 'sattler', 'notre', 'inhering', 'sequelitis', 'phallic', 'lot--bored', 'raving', 'stealer', 'avoid', "`secret'", 'laughlin', 'compete', 'committee', 'honchos', 'take-no-crap', 'succumb', 'accomplish', 'peeking', 'resilient', 'generalize', 'category', 'lamely', 'tepid', 'hampered', 'boyfriend/soon-to-be', 'white', 'cafes', "couples'", 'regular', 'surround', 'currency', 'engaging', 'telekinesis', 'halting', 'patriarchal', 'upsurge', 'revealing', '`have', 'coke', 'relying', 'carlos', 'intern', 'jenny', 'balm', "sweethearts'", '1799', 'protgaonist', 'dworkin', 'life-form', 'man--the', 'coyle', "'minor-regulars'", 'interminable', 'python', 'centaur', 'banquets', 'newborn', 'dilema', 'scare-thrillers', 'on-the-spot', 'chairs', 'barr', 'concluding', '`drive', 'grading', 'urged', 'mancina', 'scrutinizing', 'orbiting', "'so", 'unrelentless', "beckinsale's", 'munchausen', 'camelot', 'reactor', 'lingerie', 'boxer', 'placating', 'met', 'carcass', 'whiff', "newton-john's", 'iliff', 'in-court', 'guest-host', 'one-dimensional', 'sublte', 'exits', 'telescope', 'prisoner/former', '-brand', 'imperialist', 'friedklin', 'fleshing', 'disconnected', 'tale', 'del', 'texas/mexico', 'hooray', 'home-cooked', 'vicariously', 'goody-goody', 'tenth', 'woodenly', 'envoke', '`what', 'cinnamon', 'mills', 'coraci', 'medicine', 'smoking', 'frock', 'serpentine', 'whitfield', 'organizes', 'freeze', 'confusingly', "inkpot's", 'roro', 'blossoms', 'rejoiced', 'brokerage', 'tenuous', 'sebulba', 'cure', 'bullseye', "moriceau's", 'ime', 'post-_scream_', "'a", '26', 'romantic', 'walton', 'slackers', 'oboe', 'plagiarism', 'coachmen', 'refunds', 'introductions', 'whoop-dee-doo', '1995', "sterling's", 'whoo', 'crosses', 'sergio-leone-', 'furnished', "mca's", 'shearim', 'shreck', 'hisses', 'ballard', 'vaporized', 'ticket', '--a', "bull'", 'inexperienced', 'statement-making', 'journalism', 'clapet-the', "religion's", 'berger', 'all-but-illegible', 'sneaks', 'hung', '9/10', 'secures', '_some_', 'jam-packed', 'twelve', 'dumber', 'jitters', 'culpability', 'beings', 'jellyfish', 'measured', 'abandonment', 'sera', 'alfie', 'despondent', 'computech', 'pushes', 'watched', 'burnings', 'swartzenegger', 'luckily', 'toll', 'expensive-', 'deeply', 'muscat', 'retains', 'inner-friend', 'counsels', 'misrepresentation', 'embody', 'robinson', 'padded', 'jibberish', 'owner--veteran', '`for', 'streetcar', 'accordion', 'stunning', 'aimee', 'ignores', 'hasan', 'indie-film', "ez's", 'celestrial', "morse's", 'spur', 'foolishly', 'brill', 'nobles', 'wielding', 'yelled', 'scottish', 'straightforwardness', "chidduck's", 'mexico', "90's", 'koop', 'duking', 'mysticism', 'rodmilla', 'waved', '1992', 'novalee', 'ensues', 'torrential', 'duetting', 'teeming', 'soundtracks', 'rendering', 'puberty', 'consulting', 'twinky', 'artsy', 'max-like', 'hollywoodized', 'cinematography--', 'dishing', 'greyhound', 'yin', 'disgustingly', 'glider', 's-t-r-e-t-c-h', 'arouse', 'carol', 'absconded', 'dine', 'bias', 'forsyth', 'begrudging', "rising's", 'banjos', 'evne', 'delegation', 'hyberboreans--we', 'collection', 'whisker', 'snagged', '+4', 'banditos', 'snorks', 'lilianna', "frears'", "firm's", 'gypsy-like', 'ineffectual', 'grilling', "parker's", 'maurice', 'inferior', 'invigorating', 'acres', 'documenting', 're-christening', 'stressed', 'non-verbal', "line's", 'humility', 'body-builders', 'chronology', 'extracts', 'orators', "'tradition'", 'option', 'grail', 'reflexes', 'exports', 'advertisement', 'coming-of-age', "johnson's", 'trendily', 'wind', 'fide', 'pidgeon', 'red-eyed', 'candidates', 'fledged', 'moloney', 'wynter', 'identikit', 'shortcuts', '500th', "eye's", 'chummy', 'savant', 'caged', 'opulent', 'familiar--one', "unger's", "rep'ing", 'bogs', 'gentry', '$5', 'wonka', 'copes', 'faking', 'comprehensive', "shark's", 'drug-stoked', 'indoctrinated', 'coat', 'congregation', "jackal's", 'remington', 'billowy', 'life-and-death', 'sidekick', 'takings--and', 'juba', 'overreaching', 'korda', 'high-quality', 'johansson', 'cringe-', 'gamblers', 'wimpy', 'span', 'deliberately', 'financially-oriented', 'rudd', 'sctv', 'takaaki', 'tilvern', 'average-sized', 'half-convincing', 'inexorably', 'intaking', 'bitterness', 'atlantic', 'true-love', 'exploding', 'britney', "stigmata's", 'sciorra+s', 'bernhard', 'heavy-rotation', "canines'", 'excellent', 'fool-proof', 'hate-mail', 'expelled', 'reinforcement', 'schandling', 'column', 'narcolepsy', 'proving', "york's", 'underscoring', 'inhumanly', 'brackett', 'wwii-era', 'curtains', 'regurgitate', 'boosterism', 'bacteria', 'rookies', "bean's", "authorities'", 'subtely', 'zeltser', 'suspense/horror', 'interference', 'rape', 'pursing', 'logistics', 'burlesque', 'external', 'strangulation', 'wells=92', 'improvements', 'devastate', 'sheet', 'mckellen', 'bloodstains', 'confiscate', 'brooding', "ship's", 'fists', 'horribly', 'damper', 'exploit', 'respectable', "corky's", 'lily', 'revisiting', 'undergoing', 'bedfellows', 'humors', 'philosophies', 'lovelorn', 'slightly-less', 'northwest', 'dishwasher', 'alea', 'decrees', 'calmer', 'conked', 'roxanne', 'finish', 'pettiness', 'megan', "milton's", 'toby', 'excrutiatingly', 'features', '_somewhere_', "lebowski's", '175', 'rosario', 'supremely', "illiterate'", 'normalcy', "alex's", 'scene--even', 'urinal', 'starkly', 'cooder', 'refreshingly', 'nightwolf', 'parasitic', 'holgado', 'mist', 'undeserving', 'stewart', 'corvino', 'benignly', 'morning', "damone's", 'included', 'dual', 'psitive', 'vanities', 'hue', 'deceptive', 'rolls', 'conviction', 'kosovo', 'fairy', 'blurred', 'addiction', 'bounteous', 'roars', 'xv', 'tiered', 'mcquarrie', 'riiiiight', "jackie's", 'ken', 'weisberg', 'polaroid', 'self-professed', 'bloomington', 'breathlessly', 'salami', 'hard-to-decipher', 'languages', "duchovny's", 'emotive', 'impersonators', 'conversely', 'food', 'nails-tough', 'greaser', 'loco', 'exclamation', 'torsos', 'mammoth', 'alexandra', 'miscarrying', 'upbeat', "cross's", 'palitaliano', 'cease', 'nico', 'grotesque', 'friedrich', 'salmon', 'pansies', 'bosomy', 'randal', 'gilsig', 'gebrecht', 'schmaltzfest', 'authors', 'sequel-the', 'scoopfuls', 'bent', 'vies', 'emshwiller', 'gowns\x05\x05euuugh', 'beeper', 'cigarette', 'chiefly', 'occaisionally', '_unbreakable_', 'ire', 'loathes', 'crusades', 'blockbusters', "'dante's", 'oedipus', 'merging', 're-wire', 'climactic', 'actors-', 'scully', 'psychoanalyze', 'scarcely', 'bernsen', 'cuddy', 'it_', 'beard', 'mediocre', 'falzone', 'cowled', 'here-to-there', 'eurocentrism', 'pommfrit', '110', 'frances', 'illiterate', 'hull', 'shostakovich', 'new--other', 'editted', 'ups-and-downs', 'rule', 'increased', 'abducted', 'goony', 'lingo', 'huge', 'carla', 'bespectacled', 'rejoicing', 'over-alled', 'larry', 'emporer', 'watery', 'slot', 'brilliancy', 'devastation', 'career-defining', 'syndrome-like', "hasidic's", 'bleach-blond', 'glimpsing', 'lowe', 'prostitution', 'suitable', 'joke', 'soupy', "labyrinth's", 'bootsie', 'carpet', 'wallows', 'winbush', 'wilde', 'lysterine', 'drug-addicted', '1961', 'circumventing', 'kingsley', 'lacombe', "lumumba's", 'summertime', "leoni's", 'knepper', 'drug-addict', 'skarsgaard', 'hicks', 'aspire', 'ably', 'strong-willed', 'words', 'odd', 'doctorate', 'one-man/woman', 'projectile', 'katie', 'garp', 'theorems', 'anabasis', 'crocheting', "mother-in-law's", 'spurned', 'compounded', "code'", 'unsettled', 'badgers', "'95s", 'isaach', 'somerset', 'fichtner', 'vadar', 'higher-ups', 'result', 'monkey-like', 'boxed', 'redheads', 'wrights', 'stormare', 'eraser', 'believably', 'rut', 'filthy', 'longest', 'disappointingly', 'main', 'encounter', 'prix', 'psyched', 'dolphins', 'stretching', 'loathing', 'harsh-reality', 'naturally', "someone's", 'deja-vu', 'one-if', 'he-man', 'witchcraft', 'lewd', 'lucky', 'deedles', 'wiser', 'landwalker', 'arquettes', 'detritus', 'locksley', 'applications', "mehta's", 'sprees', 'handbasket', 'dazzle', 'escorting', 'mtv-land', 'lipsynching', 'goth-girl', 'vaus', 'sub-road', 'marketplace', 'anticipating', "bogg's", 'burden', "itami's", 'apace', 'occasion', 'repetition', 'voyager', 'bickers', 'pecan', 'timbre', 't-birds', 'illuminata', 'fraidy-cat', 'binoculars', 'apostle', 'jubilee', 'impregnate', 'pedestal', 'daylights', 'riff-raff', 'courtoom', 'kind', 'metamorphoses', 'arkansas', 'disruptions', 'state-run', 'waded', 'creditors', 'berlin', 'touchy-feely', 'practitioner', 'intertwine', 'unwarranted', "z's", 'madio', 'dramatic', 'mueller-stahl', 'academe', 'departing', 'empirical', 'evars', 'smartly-scripted', 'coma', 'connected', 'ex-boss', 'figureheads', 'bogeyman', 'at--revenge', 'recreation', 'infused', 'victorian-era', 'jimmies', 'car', 'gigi', 'spousal', 'aurelien', 'pangs', 'rubin', 'intimately', 'sellers', 'backward', '63', '=', 'thrice', 'engulf', 'prophesied', 'overalls', 'alright', 'hallstr', 'technicality', 'supply-trading', 'import', 'rafe', 'pertaining', 'evacuation', 'slice-and-dice', 'no-thought', 'poltergeist', 'pronouncements', 'landau', 'erudite', 'tarlov', 'brats', 'chopsticks', 'lets', 'coeds', 'spies', "chan'", 'boy--called', 'marathon', 'nail-bitingly', "toll's", 'gained', 'hoped', 'petra', 'guys/girls', 'magoo', "'independent", 'poll-takers', 'guano', 'beymer', 'ingen', 'iwo', 'horsing', 'director/screenwriter', 'self-worth', 'drive-by', 'public', 'thrashing', 'serenely', 'commercially', 'thrill', 'ichabod', 'bone-headed', 'diesel', 'full-well', 'surfaces', 'assess', 'morons', 'spider', 'margiulles', "schoolboy's", 'responded', 'provoke', 'apartments', 'hughley', 'shortages', 'mistfortune', 'rawness', 'conference', 'maintains', 'star-struck', 'meaningless', 'supertechnology', 'exterminators', 'guru', 'pissant', 'me-had', 'smattering', 'lucent', 'bountiful', 'reacting', 'assimilate', 'mite', 'unaffecting', 'solipsism', 'teleplay', 'breakdown--that', 'allie', 'hunter', 'heaving', 'breathnach', 'creator', 'millers', 'triumvirate', 'zeljko', 'ardently', '$3', 'humourous', 'deities', 'noise', 'succeed', 'lt', 'paunchy', 'pungent', 'ashore', 'coal', 'recess', 'actualisation', 'hack', 'command-line', 'driods', "'holocaust", 'hav', 'stops', 'economical', 'dope-dealing', 'auntie', 'hallucination', 'pornography', 'murron', "spade's", 'pharmaceuticals', 'antagonists', 'phoned-in', "rumor's", 'reminiscient', 'dashboard', 'spin', 'lenses', 'epsode', 'wonder-bred', 'equaling', 'slated', 'schweig', "herzfeld's", 'hallucinogenic', 'sores', 'anniversary', 'amanda', 'porno', 'forget', 'character-based', 'pushy', 'tremendously', 'frayn', '[good]', 'kings', 'kidman', 'chaney', 'francine', 'fashion', "sammy's", 'rises', 'slasher-flick', 'nicolo', "90210'", 'architecture', 'loverboy', 'yoram', 'owner-turned-painter', 'marsters', 'depletes', 'rollin', 'hundred-year-old', 'cyncial', 'immersing', 'marv', 'entitlement', 'prompted', 'vaughns', 'arrival', 'menstruating', 'presentations', "degeneres'", 'quote', 'wonders', 'probability', 'matter-their', "disagree'", 'stoltz', 'pneumonia', 'jaw-droppingly', 'gleefully', 'storage', "o'keefe", 'bid', '`humans', 'organ', 'credibility-in', 'nuclear', 'corresponds', 'underworld', 'ramen', 'eligibility', 'maclaine', 'sitters', 'anthropology', 'overly-chatty', 'caine', 'klutzes', 'skiing', 'stripclub', 'daily', 'sagan', 'amerocentric', 'saturday', 'pigs', 'spree', 'plagerism', 'counted', 'proclaim', '$20-million', 'usurped', 'restored', 'pennies', 'issued', 'career-topping', 'on-the-sidelines', 'hatch', 'grins', 'disintigrated', 'unrequited', 'conveying', 'remedial', 'weepie-wannabe', 'obsession', 'overdubbed', 'poetic', 'battled', 'bass', 'almasy', 'fantasy/allegory', "griffith's", "monkey's", "castro's", 'torturously', 'washed-up', 'regions', 'biology', 'scum-sucking', 'gusty', 'beta-testing-cum-teaser', 'half-human/half-alien', 'scots-irish', 'sleaziness', 'crowe', 'person', 'neccessary', "heston's", 'court-ordered', 'pleased--in', 'tires', 'it=92s', 'charm-meter', 'healers', 'dryland', 'breached', "egoyan's", 'fine', 'unsuspenseful', '_', "chereau's", 'mara', 'homerian', 'cellulars', 'betsy', "bromides'", 'volcanic', 'action--', 'accidentlly', 'warmer', 'lagpacan', "sue's", 'not-totally-great', 'dicaprio', 'fouled', 'final', 'nauseam', 'outrageous', 'african-american', 'congolese', 'dunsworth', 'hackles', 'hamburg', 'budget-wise', 'trailing', 'faired', "aronofsky's", 'upstage', 'anatomy', 'h20', "barth's", 'halloweentown', 'italy', "hemming's", 'goggins', 'triad', 'hinders', 'bargaining', 'overrated', 'continuing', 'kathryn', 'burlier', 'heston', 'half-starved', 'tidbits', 'semi-', 'pisses', 'lulling', 'hanger-on', 'bubblebath', 'diving', 'suave-yet-unsuave', 'allyson', 'nucci', 'done-that', 'enemies', 'exponential', 'nineteen-eighties', 'micromanaged', 'made-people', 'leaned', "sidney's", 'doc--in', 'funhouse', 'smithee', 'broeck', 'infestation', 'braniff', 'locked', "barr's", 'afield', 'hoodlum', 'racially-motivated', 'generic', 'insincere', 'co-writers', 'gough', 'stipend', 'outgrown', 'drawls', 'garbage', 'not-so-innocent', 'bunny', 'assisted', 'decisive', 'bullying', 'inish', 'hires', 'responses', 'genesis', 'completing', 'mediocrity', 'surmise', 'opaque', 'pops', 'diamonds', "matalin's", 'gay', 'won92t', 'opposites', 'substantiated', 'noticeable', 'nguyen', 'downtime', 'granite', '43-foot-long', 'meredith', 'executions', 'maybury', 'underwent', 'powerhouse', 'albertson', 'searchlight', 'minute-and', 'full-to-bursting', 'tv-movie', 'providing', 'threatens', 'banana', "auteuil's", 'hickam', 'shaun', 'child-genius', 'crowdpleasing', 'mcjob', 'peach', 'bluesman', 'unearth', 'claustrophobic', 'chuckle-out-loud', 'carlisle', 'incandescent', 'ulrich', 'explicitly', 'charge', 'envelope', 'engagement', 'only-in-the-movies', 'rushes', 'awe-filled', 'beef', "frankie's", 'zealot', 'always-interesting', 'crazymadinlove', 'silver-bladed', 'audiotapes', 'granddaddy', 'futuristic', '5th', 'tranisition', 'immensely', 'gotham', 'barracking', 'saffron', 'coincided', "beal's", 'door', 'persay', 'subs', 'myriad', 'neighbour-socking', "gabriela's", 'vibrates', "heather's", "'shoot-em-up'", "hogarth's", 'environs', 'spasmodic', 'deformation', 'lightening', 'applies', 'courtesans', 'ex-lover', 'wearer', 'disappearance', 'near-unintelligble', 'blustered', '50-foot', 'panorama', 'earth-hangs-in-the-balance', 'bracken', 'competitiveness', 'co-defendant', 'kidnaping', 'linguists', 'heavily-armed', 'agnieszka', 'feature-length', 'overstay', 'continuously', 'raspiness', 'roche', 'assailants', 'big-city', 'purposes', 'stampede', 'he/she/it', 'ceilings', 'rulebook', 'lama', 'winces', 'chutes', 'near-riot', 'recommendation', 'commendable', 'martinis--and', 'kerrigans', 'run-ins', 're-election', 'fine--', 'epos', 'fiesty', 'millionaire', 'dussollier', 'taxis', 'sounded', 'plagued', 'scar', 'new-improved', 'planners', 'unpleasantness', 'beelzebub', 'guffaw', 'stepchildren', 'neville', 'sweets', 'penitentiary', 'fateful', 'oddly', 'hes', 'warmly', 'lillard', 'jiving', 'fervent', 'trod', "batologist's", 'stupor', 'milagro', 'storywise', 'lloyd', 'prurient', 'himbos', 'transit-oriented', 'feed', 'breakdances', 'coto', 'babysitting', 'anti-glory', 'pureed', 'kahlberg', '_the_', 'dreamy-eyed', 'applied', 'cbs', 'addams', "woody's", 'hoyle', 'buddy-buddy', "1800's", "romero's", 'churches', 'wrath', 'exclaims', 'well-executed', 'essentially', 'primate', 'ticked', 'chernobyl', 'bleeds', 'complainer', 'soprano', 'sabihy', 'poke', 'bugged', 'well-realized', 'blond-haired', 'potters', 'standouts', 'comedic-romance', 'engineering', 'brokering', 'hard-fought', 'lavatorial', "tibet's", 'whopping', 'filmdom', 'dissipating', "dead-guy-seems-to-have-come-back-to-life-but-then-we-find-out-it's-only-a-dream", 'harmonizing', 'raises', 'bavarian', 'surmises', 'harrington', "tsui's", 'engagements', 'self-effacing', 'murders', 'computers', 'washington', 'show', 'drumbeat', 'breathtaking', 'bo', 'activites', 'armin', 'indescribable', 'eagles', 'groups', "'hospital", "lesra's", 'silence', 'indefinantly', 'altruist', 'course-a', 'smolder', 'reminscent', "laurie's", 'helmut', 'rumored', "irving's", 'organ-playing', 'naps', 'jarmusch', 'mamet', 'concentrate', 'neverland', 'wheels', 'alteration', 'smut', 'doohan', 'digs', 'performace', 'leeches', "garcia's", 'exacted', 'worldly', 'ride', 'near-indestructible', 'supplicate', 'macho-misfits', 'dawes', 'stewards', "jesus'", 'grapple', 'gordo', 'pill', "frickin'", 'intensity', 'walhberg', 'wayne', 'buy', 'ayla', 'pressing', 'coughlan', 'kaminsky', 'epilepsy', 'banter', 'beckoning', 'cranberries', '`after', 'emmylou', "peebles'", 'hosting', 'claustral', 'towering', 'longingly', 'wheel', 'bench', 'parrish', 're-visit', 'ana', 'sister', 'mcfadden', 'complicate', 'skirmishes', 'widely', '_wayyyyy_', 'toes', 'mist-shrouded', 'coffey', 'charleville', 'boy-drinks-entire-bottle-of-shampoo-and-may-or-may-not-get-girl-back', 'bouquet', 'snorts', 'delights', 'raimi', "kieslowski's", 'discarding', 'addition', 'ottman', 'complexity', 'bastad', 'wife-to-be', 'gnosis', 'enters', 'unquestionably', 'first-ever', 'humdrum', '_arrrgh_', 'moonlighted', 'polygamy', "predecessor's", 'booboos', 'friendliest', 'descriptions', 'also-rans', 'klux', 'cancerogenic', "witches'", 'flabby', 'fluidity', 'succinct', 'axed', 'blanks', 'dokken', 'aloise', 'hedonism', 'sacks', 'nielson', 'bookstore', 'aldous', 'man-made', 'obssessive', "arlene's", "whol'o'", 'inadvertly', 'in-jokes', 'newly-formed', "past's", 'wide-release', 'reshoots', 'tale--the', "magnolia's", 'condemnation', 'free-talking', 'aviation', 'dunois', '04', 'methodical', 'padua', 'lurks', 'subkoff', 'poorly-written', "wc's", 'nevada', 'al-l-l-l-l-l-l-l-l-l-lrighty', 're-adaptation', 'coaxed', "kirshner's", 'sherilyn', 'percentages', 'twisting', 'prostitute', 'jermaine', 'sinatra', "fear'", 'homes', 'highpoint', 'included--wishing', 'utopian', 'helm', 'bumped', 'clamoring', '//leopard', 'babe-magnet', 'sloppiness', 'grisham-like', 'ill', 'traci', 'fulfils', "marty's", 'dino', 'artists', 'shirishama', "scholl's", 'amused', 'captor', 'discusses', "lord's", "law's", 'contender', 'pavement', 'evince', 'leader', "'grease'", 'treks', 'third-billing', 'winger', "issac's", 'entered', 'alchemy', 'traitor', 'mindset', 'tac', 'reprised', '170', 'visual', 'reacted', 'reparations', 'french-set', 'intolerance', 'praises', 'phrasing-too', 'hjejle', 'hulking', 'bale', 'bio-mechanical', 'predictable', 'issue', 'balaban', 'womanized', 'ope', 'awaiting', "'number", 'pities', 'az', '`i', 'senators', 'semi-stall', 'thai', 'cherry-colored', 'irritate', 'cartwright', 'someday', 'tattoo', 'meschach', 'managers', 'english-language', 'klan', 'middleweight', 'quiz', 'translucent', 'zoot', 'agreement', 'geek/god', 'visnjic', 'goings-on', "`what-ever-happened-to'", 'loewi', 'broder', 'princess/ogre', 'pleads', 'wondered', 'bite', 'doubts', 'sundry', 'trafficking', 'fatass', 'sodomy', 'gibson', "'great", 'simpson', 'shay', 'powerful', 'make-shift', "'leon'", '133', "pupil's", "charlotte's", 'exclusive', 'self-assuredness', 'unskilled', 'holograms', 'janice', 'happily', 'buscemi', 'freezes', 'reluctant', 'crowened', 'entrepreneurial', 'pelvic', 'ku', 'nehru', 'and--', 'lebrock', "'most", 'alberta', 'home', "old's", 'rashomon', 'hallucinatory', 'filth', 'strains', 'researcher', 'earle', 'disdains', 'evade', "falzone's", 'topped', 'retrospectives', 'perado', 'wedding-needing', 'emerges', 'de-glamorizes', 'supposed', 'bleeth', "amy's", 'string-laden', 'flik', 'favours', 'nathanson', 'levine', 'offbeat', 'flores', "elgar's", 'flooding', 'flings', 'sprouse', 'all-male', 'auteurs', 'bankable', 'validating', "nazi's", 'surrandon', 'ovitz', 'uphold', 'garafalo', 'firing', 'naomi', 'goodly', 'amon', 'r2d2', 'noni', 'points-of-', 'rip', 'retiring', 'chipmunk', 're-born', 'traditions', "water's", 'tunney', 'graceless', 'eagerly-awaited', 'naidu', 'emax', 'apothecary', 'chuckles', 'turbulent', 'statuette', 'ulu', 'grande', 'vertical', 'demonstrators', 'winks', 'oracles', 'diane', 'meyjes', 'valour', 'cleverly', 'academics', 'plumber-turned-private', 'buddy-cop', 'lavishly-decorated', 'sprinting', 'rote', 'tel', 'terel', 'baboons', 'telepathic', '78', 'beirut', 'painful', 'typewriter', 'blamed', 'positing', 'f-u-n', 'ville', 'hubert', 'talks', 'uhm', 'anatomizes', 'calisthenics', 'tautness', 'rhapsody', 'businesses', 'aftermath', "wexler's", 'cityscapes', 'unexciting', "'cept", 'daring', 'zamora', 'stock-in-trade', 'fedoore', 'spicoli', 'ushers', 'glances', 'carve', 'pepsi', 'kelly', 'scrappy', 'gara', 'fulfilled', 'oscarworthy', "hustler's", 'corresponded', 'regails', 'about-face', 'twitchy', 'marching/singing', 'claim', 'softhearted', 'fire', 'oh-so-ironic', 'skyscrapers', "perry's", 'hargrove', 'kari', 'well-plotted', 'overdress', 'sporadic', 'treasure-seeking', "crow's", 'strategies', 'ever-enjoyable', 'rearing', 'sharkskin', 'unboarded', 'thaddeus', "lama's", 'cuss', 'lettuce', 'girl-on-girl', 'widescreen', 'hartnett', 're-create', 'jedi-size', 'mccalister', 'excitment', 'meditate', 'foregin', 'collar', 'pacifier', 'bufford', 'undoes', 'leguizimo', 'godzillas', 'flop-house', 'cliche-spewing', 'reimagines', 'lace', 'wafer-thin', 'mentality', 'rehab', 'orgazmo', 'whetted', 'wyman', 'bozzio', 'evactuate', 'drunk', 'stressing', 'notables', 'topol', 'vigor', 'lumbers', '11th-hour', 'conquistador', 'woof', 'rylance', 'tanks', 'coworkers', 'uproar', 'argonautica', '67', 'zemecki', 'life--after', 'undisputed', 'eligible', 'brigante', 'bluster', "hospital's", 'smearing', 'shwarzenegger', "idol's", "falk's", 'two-hander', 'valkyries', 'high-energy', 'thumb', 'imparts', 'jacob', 'shackles', 'unendurably', 'degradation', 'giosu=e9', 'heretical', 'furrier', 'scaramanga', 'sinking', 'nigga', 'overworked', 'reds', 'blooded', 'manero', "'tremors", 'non-webb', 'elaborately', 'sportscasters', 'mansfield', 'outshines', 'contracted', 'countless', 'greenfingers', 'self-chiding', 'novotny', 'detestable', 'lottery', 'gargling', 'bulow', 'agencies', 'endowed', 'baad', 'omegas', 'tainer', 'celibacy', 'installations', 'singling', 'divined', 'verbalized', 'casting', 'dismally', 'computer-masked', 'coax', 'dymanite', 'ramboesque', 'barnett', 'anti-fascist', 'smugness', 'anachronisms', 'preternatural', 'spooking', "grosbard's", 'fetishizes', 'amazing', "`8mm'", 'typical', 'traverse', 'semi-clothed', 'businessman', 'loomis', 'nerdies', 'modulated', 'emo', 'radiation-rich', 'ski-doos', 'recently-deceased', 'acquainted', 'irrepressible', 'breed', 'upmost', 'lanced', 'override', 'sparticus', 'wacked-out', 'daytrippers', "proctor's", 'outperform', 'pheiffer', 'slurs', 'deluding', 'mcracken', 'filter', 'hard-core', 'remnants', 'medically', 'forecaster', 'shown', 'gomorrah', 'magnetism', "`cops'", "teresa's", 'woolworth', '11-story', 'flashy-new-thing', 'readily', 'cheadle', 'witless', 'khachaturian', 'naivete', 'midly', 'hitchcockian', 'blanket', 'enwrapped', 'rekindling', 'juliet', 'self-parody', "bethany's", 'progressively', "rhino's", 'fun', 'fitts', 'vianne', 'jokingly', 'smirks', 'shading', 'understandingly', "collector's", 'balsan', "florida's", 'sexiest', '_melvin', 'javert', "fuller's", "reporter's", 'refreshing--and', 'hercules', '[pg]', 'parcel', 'shell-shocked', 'macomber', 'hostage', 'popularizer', 'glebas', 'window-shopping', 'april', 'forgetting', 'communist', 're-releases', 'alienate', 'flexibly', 'cubicle-filled', 'parenting', "animators'", 'depart', 'linear', 'immersion', 'gawkers', 'technical', 'discontents', 'underestimates', 'jenkins', '`unsung', 'iv', 'mccardie', 'stodge', "stendhal's", 'flows', 'voluptuous', 'warheads', 'noel', 'adept', 'chunks', 'kills', '14-year-old', "keaton's", '89', 'sternly', "lundgren's", 're-living', 'grumpy', 'advancement', 'specialised', 'glickman', 'realizations', 'tutus', 'philandering', "'93", '310', 'topic', 'condescending', 'gouda', 'commericials', 'demarco', 'brash', 'maeda', '48', "et'", 'stampeding', 'scorecard', 'jean-luc', 'scowl', 'monthly', 'keyhole', 'vaguely', 'disjointed', 'curled', 'barter', 'jeopardy', 'borderline-psychotic', 'delightfully', 'meddled', 'distinct', '8th', 'blue', 'arch-enemy', 'rooms', 'gambols', 'bushel', 'loveless', 'obligatory', 'buddy/cop/drug/sexy-witness', 'flitting', 'tough', "christina's", 'cigarrettes', 'incisive', 'sendoff', 'zombie-stomping', 'three-act', "winston's", 'in-valids', 'allegations', 'extras', 'cyclops', 'prefered', 'gillian', 'tie-in', 'overpraising', 'turi', 'mushu', 'red-lit', 'woodstock', 'ryder', 'acne', "rex's", 'rampling', 'tobey', 'brs', 'jiggling', 'dalmantions', 'fuss', 'wake', 'inevitably', 'caraciture', 'suspicions', 'ground--all', '25th', 'inspiring', 'curious', "'90s", 'ultraviolet', 'subsidiaries', 'duh', 'reverts', 'goof-ups', 'obtains', 'travels', 'lick', 'odalisque', 'luchino', 'x-files', "'baby", 'out-of-place', 'montages', 'frown', 'truckers', 'subordinates', 'short', 'high-definition', 'guide', 'countryman', 'hurry-up-and-wait', 'bootcamp', 'telegraphs', 'cheat', "schmoe's", 'potboiler', 'knockoffs', "fosse's", "benigni's", "her's", 'betcha', 'crotch', 'qualifies', 'unfortunates', 'interpreted', 'principals', 'awash', 'darrow', 'subordinate', '21-year-old', 'post-cold', 'self-rearranging', 'tag', 'anti-war', 'bestial', 'flick-', 'standard-issue', 'bargained', 'sub-snl', 'whip', '1/3', 'adrien/marc', 'arrangement', 'mulcahy', 'icecream', 'napoleon', 'lies', 'breasted', 'god-damns', 'jameses', 'overscore', 'raptor', 'clones', 'flabbergastingly', 'mercedes', "'terrore", 'faber', 'up-to-date', 'vantages', 'neo-nazi-dom', '======================', 'meditating', 'fleshed-out', 'tenacious', 'pumpkins', 'lam', '--it', "sellars'", 'gus', 'thought-provoking', 'slackness', 'eggar', '____________________________________________', 'innocently', 'vitality', 'bartenders', 'arrested', 'hots', 'high-budget', 'headquarters', 'risks', 'movie-', 'rummaging', 'tv-star-to-film', 'containers', 'huey', 'across--the', 'incurably', 're-capture', 'cu-sack', 'glued', 'hoser', 'impeccable', 'katsulas', 'glide', 'dictator', 'popular-but-slow', 'sarcastic', 'nbk', 'erotica', 'americanized', "crowe's", 'daresay', "agree'", 'investigators', 'inseperable', "cassavettes'", 'rebelling-against-daddy', 'writer/creator', 'jumped', 'stumble', 'life-altering', 'badly-written', 'overly-dark', 'cell-phones', 'calder', 'pasts', 'directing', 'cinematic', 'disastrous', 'visitors', 'dash', 'amphetamine', 'fantasy--one', 'meet-cute', 'alleys', 'classify', 'nobuko', 'servant', 'father/david', 'kerrigan', 'muscle-bound', 'moore\x12s', 'allergy', 'baylor', 'beachwalkers', "health's", "1949's", 'adopting', "'hard", 'thirty-six', 'demeaned', 'card', 'co-worker', 'migraine-inducing', 'migrating', 'mcbainbridge', 'thud', 'batgirl', 'roofs', 'knife', "sims'", "families'", 'aa', 'wink-wink', 'theatrics-', 'molds', 'burgeoning', 'schweethaat', 'gift-bearer', 'missive', 'poopies', 'refrained', 'immature', 'cross-cut', "leiter's", 'jedda', 'pcu', 'runway', 'drug-addiction', 'noisy', 'ticoton', 'gray', 'allah', 'oh-so', '1903', 'deus-ex-machina', 'computer-assisted', 'realisticly', 'turmoil', 'resurfacing', 'tassel', 'midair', 'itself--box', 'plunkett', 'pitting', 'hazards', 'setpieces', 'zundel', 'bottle', 'handful', 'todd', 'heirloom', 'dried-up', 'juliette', 'blitzed', "ritchies's", 'upstaging', 'tricked', 'cemetery', 'short-changed', 'kick', 'transylvanian', 'herger', 'connives', 'headlining', 'nolte', 'ever-versatile', 'misuses', 'impacts', "`braveheart'", 'resistible', 'aggressively', 'blacked', 'reproach', 'ways', 'shrine', '70m', 'reconstructed', 'conveniences', 'bah', 'toninho', 'jackal', 'clap', 'cringing', 'cliches', 'deficiencies', 'naaaaah', 'exhibiting', 'propagating', 'bettered', '_election_', 'xfx', 'suitably', 'afterschool', '1940s-style', 'swimmer', 'incantations', 'orson', "gosselaar's", "so-corny-it's-funny", 'statement', 'garfield', 'slam-dunks', 'romantic-mistaken', 'self-taught', 'realization', 'shipwrecked', 'sulking', 'pounder', 'jovovich', 'imitates', 'unsung', '========================', "buddy's", 'orgasmic', 'palliative', 'soles', 'voluntary', 'trek', 'comers', 'bradley', 'stomach-turning', 'strategist', 'campion', "episode's", 'shell', 'fakery', 'oddballs', 'doused', 'symbolises', "genuises'", 'playfulness', 'wickedness', 'desouza', 'geordi', 'carter--whose', 'ww', 'reveling', 'mariah', 'racist', 'herd', 'norsemen', 'joie', "mathilda's", 'self-mutilation', 'booby-trapped', 'deliverance', 'emerge', 'co-eds', 'downward', 'calculated', 'horn-dogs', 'goons', 'ulterior', '-based', 'pfeiffer', 'sparked', 'weakest', 'camera-time', 'horsemen', 'spelenetik', 'gordy', 'broadens', "rafe's", "up's", 'arty', 'goggles', 'out--last', 'dreadlocks', 'utilised', 'pad', 'priivate', 'prep', 'switching', 'siegel', "bluebirds'", "gandolfini's", 'sulk', 'neve', 'southampton', 'klaus', 'stale', "goldsmith's", 'della', 'batmans', 'prison-turned', 'batterings', 'shaky', "guiness'", 'richman', 'narcotics', 'disrespecting', 'jensen', 'bitterly', 'dork-chick', 'one-upping', 'objectively', 'multi-million', 'ambrose', "bethlehem's", 'reprimands', 'blondie', 'enviable', 'south-winds', 'now-uma', 'marketwise', 'shriker', 'kowtowing', 'withhold', 'clues', 'barnacles', "maid's", "even's", 'trendiness', 'infer', "can't-miss", 'aggravate', 'dabbles', 'gal', '1960', 'provide', 'discipline', 'violin', 'publications', 'talmudic', 'authorship', 'centerpiece', 'participating', 'carbon-copy', 'gently', 'rooming', 'perpetual-motion', 'twenty', 'prick', 'with]', 'jackee', 'over-played', 'sprouted', 'mimms', 'resumes', 'muresan', '8/10', 'her/and', 'hugely', 'pigeonholed', 'morton', 'rasp', 'mid-air', 'simplified', 'crickey', 'excel', 'silva', 'investigates', 'sandler', 'twenty-eight', 'ringo', 'killlers', 'ads', 'potenza', 'garr', 'bodily', 'imagined', 'psych', 'ips', 'graduation', 'reliance', 'shoots', 'completes', 'hard-luck', 'g-men', 'wrongfully', 'storyboard-to-scene', 'cheesy-looking', 'ming-', 'plaguing', 'b-action', 'wc', 'isolate', 'validity', 'snake', 'creativeness', 'easton', 'mementos', 'thirty', 'impales', 'permanent', 'wierd', 'moresco', 'incongruity', 'traditional', 'stacking', 'one-on-one', 'encoding', 'tennessee', 'nadja', 'goran', 'militarism', 'edward', 'observes', 'director/writer/cinematographer/editor', 'quell', 'father', "gottfried's", 'rehabilitates', 'mucus', 'courtesy', 'blowhard', 'languid-how', 'bisexuality', 'child-support', 'kirkland', 'farmer-wizard', 'richer', 'indication', 'yale', 'semi-nude', 'deconstructed', "children's", 'whines', 'fur-coated', "jokes'", "hedges'", 'eccleston', 'egan', 'switch', 'uniform', 'overly', 'tapert', 'braced', 'upholstered', '701', "kurtz'", 'time-to-time', '64', 'butthead', 'stand-in', 'crossing', 'unobtrusive', 'condition', 'factory-worker-turned-prostitute', 'concur', 'nov', 'partially-realized', 'hailing', 'diggs', 'million-plus', 'moderately', 'pornstar', 'real-', 'credentials', 'copies', 'gabriel', 'camouflaged', 'abhorrent', 'confiscates', 'sabrina', 'regaled', 'burst', 'entries', 'marcie', "x-files's", 'contaminated', 'motto', 'ifans', 'americanizing', 'curse', 'dale', 'emblems', 'trophy', 'smothering', 'mimicing', "nicolet's", 'kissing', 'telekinetically', "fishburne's", 'separating', 'sight', 'lamps', 'norway', "shan-yu's", 'intermittently', 'orgy', 'hawke', 'mortality', 'super-judgmental', 'sally', 'ball', 'shocking', "lola's", 'warton', 'rehearsals', 'sleazy', 'germphobe', 'ben-hur', 'c3p0', 'wimp', 'brush-up', 'carbon', 'cut-rate', 'blowjob', 'dequina', 'fiery-haired', 'tasked', 'burned-out', 'ex-ira', 'consideration', 'blinding', 'gel', 'waster', 'rio+s', 'glorify', 'robot', 'tennille', 'voizin', 'crusade', 'kyle', 'mcflurry', 'abu', 'exploratory', '`come', 'definite', 'meshes', 'strongest', 'pouting', 'bracing', 'trainer', "passenger's", 'peopled', 'old-money', 'fundamentalists', 'fable-like', 'lindinger', 'face-hugger', 'miscarries', 'conservatives', 'instantaneously', '[the', 'oval', 'haphazardly', 'slums', "singin'", 'goner', "folks'", 'paltry', 'keenan', 'worth--le', 'madly', 'fire-taming', 'walked', 'showcases', 'figurines', 'alternates', 'reminding', 'anti-climatic', "sheen's", 'bowling', "kruger's", 'wehrmacht', 'paradoxical', 'well-used', "miller's", 'regrettably', 'tunes--but', 'brosh', "'normal", 'shana', 'ex-ballet', 'redemption', "_knock_off_'s", 'puzzled', 'dingy', 'sawyer', 'rhymes', '18', 'honsou', 'russians', 'eagle', 'impatiently', 'atlanta', 'quickly--yes', '777-film', 'evocative', 'bookie', 'lachman', '15-year-old', 'nonchalantly', 'uniqueness', 'suspending', 'site', 'dialogue-laden', 'gospel', 'emulate', 'projector', 'jr', 'reknown', 'pre-conceived', 'lusting', 'potrayal', 'gilbert', 'guinevere', 'spanking', "juni's", "han's", 'tormentor', 'saget', 'moral-christian', 'single-elimination', 'semi-likable', 'ghouls', 'tight-budget', 'mid-1970s', 'burkittsville', 'trivia--joel', 'breasts', 'pendleton', 'morosely', '165', 'absurdities', 'limbs', 'goulash', 'titanic-like', 'dissolved', 're-enters', 'typhoon', 'not-really-all-that-funny', 'judgeship', 'summa', 'dread-factor', '17-day', 'operators', "musn't", 'facts', 'las', 'belabors', 'jay', 'mish-mashed', 'uta', 'girard', 'winslet-dicaprio', 'overbaked', 'eludes', 'convinced', 'groovy', 'hands', 'plaudits', 'ted', 'fwahahahahahahahaha', 'fixes', "untouchables'", 'satisfactorily', 'maestro', 'locates', 'weekened', 'groucho', 'gory', 'gnawing', 'horror-drama', 'pooch', "veronica's", 'hot-air', 'elegaic', 'philanderer', 'alacrity', 'ex-pug', 'ordeals', 'hacks', 'superbly-realized', 'maccormack', 'eszterhaz', 'corky', 'disenchanted', 'farquaad', 'borgnine', 'squader', 'tandem', "'la", 'pumped-up', 'fiddle', 'props-strategically-positioned-between-naked-actors-and-camera', 'user-friendly', 'acquiescence', 'scatting', "hallstrom's", 'depravity', 'doobie', 'jetsons-like', 'agnes', 'shrugged', 'ollie', "houser's", 'spouse', 'hallway', "angels'", 'normal-sized', 'shit-terpiece', 'croon', 'unkrich', 'rick', 'wipe', 'bueller', 'baily', 'glamour', 'rife', 'ugly-ass', 'hard-knock', 'paneling', 'surreptitious', 'youngster', 'lethargic', 'super-bookstores', 'suspeneful', 'direct-to-cable', 'traction', 'mie', 'breathless', 'arnie', 'thanksgiving', 'low-cut', 'pudgy', 'isiah', 'freeze-dried', 'unempathetic', 'patric', 'flees', 'sensory', "reeve's", 'forgiveness', 'dispelled', 'dabbling', 'publish', 'underplay', 'impact', 'physicality', 'rickety', 'governed', 'shelton', 'enjoyable', 'columnist', 'notoriety', '16-17', 'self-consciousness', 'tak', "'realism'", "summer'", 'hummm', 'bungee', 'synergism', 'believer', 'inserts', 'eternally', 'akin', 'weepy', 'treasure-trove', "muriel's", 'doe', 'discovers', 'far-too-common', 'paranoia', 'piss', "doom's", 'pasadena', 'cpr', 'guns', 'attained', 'tot-friendly', 'board', 'declaration', 'live-action-disney', 'scarier', 'muttering', 'vertigo', 'infidelitous', 'discomfort', 'everlasting', 'minimum', 'blaming', 'pre-arranged', 'hynes', 'dwight', 'rag-tag', 'diplomats', 'unawareness', 'strings', 'dismaying', 'two-hours-and-then-some', 'spoiler-filled', 'showy', 'minding', 'inveigles', 'mediciney', 'ribboners', 'unsympathetic', 'masculine', 'guessed', '`battlefield', 'asset', "ramis's", 'howard=92s', 'andrews', 'eduard', 'transposing', "coyle's", 'unpredictably', 'bolt', 'degenerate', 'synthesize', 'blacks', 'barkeeps', 'commits', 'flicks', 'observatory', '$27', 'cpl', 'forums', 'cat', 'bewonderment', 'benji-like', 'groom-wannabe', 'gaz', 'contradict', 'slavehood', 'laptop', 'costumer', 'daniel', 'spiritual', "prisoner's", 'nado', 'laces', 'episodes--patlabor', 'fortenberry', 'burr', 'opposition', '--are', 'hedgetrimmers', 'pow', 'hardback', 'sylvester', "death's", 'redefinition', 'inoffensive', 'but-you-knew-along', 'big-busted', "montague's", 'pulpit', 'soft-focus', 'early-19th', 'unaccomplished', 'utilizing', 'neo-nazis', 'lab', 'fugitive', 'swim', 'contemptible', 'spurgeon', 'universally', 'whiplash', "nothin'", 'resemblances', 'setup', 'packages', 'flutters', 'cast', "high-society's", 'stuffing', 'fifty', '0', 'plentiful', "mckellar's", 'seams', 'shoelaces', 'cooking', 'generally', 'sponsor', "orlock's", 'appeal', 'tattered', 'monty', 'disliked', 'ultra-secret', 'captures', "jones'", 'becker', 'escort', 'rodman', 'ice', "`unmemorable'", 'predicts', 'zookeepers', 'nonplussed', 'always-woozy', 'deprived', 'transistion', 'lint', '60%', 'scrawling', 'invented', 'lauter', 'trilogies', 'sugar', 'sonic', 'se', 'plugs', 'discouraging', 'tswsm', 'whirlpool', 'bellhop', 'seventies', 'walker', 'expressly', 'spelling', "mcdonald's", 'thriving', 'interogation', 'alyson', 'fireside', 'cutie', 'satirical', 'nobly', 'laid-back', 'quality', 'vancouver', 'self-important', 'hardcore', 'skateboarder', 'doctrine', 'expand', 'silicone', 'gbn', 'surpass', 'skilled', 'fuelled', 'widespread', 'tantoo', 'anita', 'reza', 'verhoven', 'nail', 'non-threatening', 'ship/ocean', 'nova', "'ol", 'alexis', 'partyers', 'off-the-rack', 'rejected', 'molasses-slow', 'ranked', '_wishmaster_', 'matter-of-fact', 'snake-oil', 'smartmouths', 'preminger', 'cracker', 'closed', 'wakes', 'solemn', 'intersplices', 'thumbs', 'pryce', 'complimented', 'pick-up', 'solondz', 'watkin', 'unintelligible', 'envies', 'historical', "simpson's", 'already-chaotic', 'glorification', 'mccourt', "`swingers']", 'chase/goldie', 'intruders', 'tillman', 'resignation', 'calmly', 'dov', 'overview', '_polish_wedding_', 'creative', 'sultry', "'", 'nasty', "inadequate'", 'wetmore', 'captivating', 'self-doubt', 'worm', 'incomprehensibly', 'ropes', 'disappointed', 'wyatts', '30m', 'ia', 'melodramatic--but', 'seductively', 'ferrari', 'offering', 'kong-based', 'portait', 'gassing', 'late-night', 'riddles', 'desires', 'airliner', 'yucks', "'slap", "means'", 'meanness', 'fiction/social', "welle's", 'cotton-mouthed', 'buddhism', 'filer', 'sudddenly', 'only--attempt', 'ritalin', "kimba's", 'horndog', 'receipt', 'riffini', 'briefed', 'three-person', 'significantly', 'abusers', 'wallenberg', 'neal', 'charmingly', 'division', 'systematic', 'maryann', 'warm-and-fuzzy', "stiles'", 'to-be-weds', 'player/musician/composer', 'scoop', 'principle', 'all-business', 'sweet-natured', 'out-of-control', 'chillingly', 'close-to-awful', 'restart', 'stalking', 'expire', 'enlightens', 'ensued', 'hmmmm', 'unenthusiastic', 'ruffian', 'malarkey', 'mantel', 'clutch', 'bullshitting', 'enthusiastic', 'apollo', 'sickeningly', 'nobleman', 'one-hit', 'interested', 'hatcher', 'artifacts', 'alike', 'world-weary', 'witches', 'rep', 'seasoned', "delilah's", 'fizzled', 'cisco', 'proto-humans', 'residential', 'jurrasic', 'pivotal', 'techie', 'sources', 'vivona', 'amazon', 'burnt', 'storefront', 'peeks', 'ringers', 'mtv-', 'molding', 'fascinated', 'non-communicative', 'distractedness', 'ferrell', 'unforgivingly', "go's", 'independece', "fitzgerald's", 'mere', 'rapport', 'atmosphere', 'legless', 'thenardiers', 'evoked', 'dawson', 'justifiable', 'obradors', 'instance', 'centerpieces', 'destiny', "jet's", 'rosencrantz', 'flapping', 'stefanson', 'gawain', 'degenerated', "'twister'", 'retaliation', 'brilliantly', 'imax', "jericho's", 'platonic', 'watch', 'scoffed', 'uncouth', "shuckin'-and-jivin'", 'flawlessly-acted', 'pescucci', 'buffet', 'nia', 'perking', 'shoud', 'regain', 'units', 'hauntingly', "child's", 'distrust', 'sizes', 'bride-hopeful', 'flawless', 'post-credit', 'puncture', 'equated', 'utters', 'featured', 'cute-as-a-button', 'spoon-fed', 'ex-drug', 'scornful', 'spaniard', 'plot-twist', 'll', 'ominpotence', 'meting', 'tempest', 'miracurously', 'dialogue-driven', 'flavoring', 'commerical', "crawford's", 'pharmaceutical', "film--fei-hong's", 'grandiose', 'camcorder', 'rapids', 'senstivie', 'clouseau', 'overhyped', 'darned', 'hur', 'montage', 'ix-i-i', 'gourmets', 'segues', 'less-experienced', 'potatoe', 'crawled', 'cardinals', 'morrison', 'dank', 'precariously', 'seas', 'concierge', 'vegetarian', 'forensics', 'bergman', 'dolittle', "i'm-whining-because-i-can't-get-a-girl-i-want", 'winifred', 'goth', 'unabashed', '2=awful', "kai's", 'cuddles', 'clothes', 'impregnates', 'zero-sum', "panther's", "colony's", 'down-played', 'loosely', 'fall-out', 'smokejumpers', 'microbombs', 'egg-selling', 'typically-stimulating', "gurgi's", 'illustrates', "`r'", 'origami', 'mir', 'baseketball', 'abetting', '$110', 'testings', 'manderley', 'infallibility', 'centering', 'transistor', 'fianc', 'steadiocam', 'stapled', 'crises', 'protects', 'achingly', 'hummana-hummana-hummana', 'clock', '`pulp', 'lining', 'iggy', 'violin-maker', 'scatology', 'boyishly', 'human/alien', 'mural', 'cleavage', 'saccharine', 'joon', 'doonesbury', 'mouth-breathing', 'mattes--leave', 'resolutely', 'helfgott', 'tv-conversion-movies', 'soiree', 'crackdown', 'radios', 'grin', 'capper', 'prelude', 'gracie', 'clear-thinking', 'nerve-addled', 'imagining', 'producer/writer', 'pratfall', 'uglies', 'stress', 'pat', 'entrenched', "pickin's", 'do-it-yourselfer', '`call', 'linden', 'ent', '`one', 'guarantee', 'mischief', 'mcgrath', 'sus-pects', 'stultifying', 'clever-way', 'characteristics', 'fluffy', 'gauntlet', 'castilian', 'susanne', 'offence', "lovers'", 'skewer', 'reformed', 'improvized', 'helena', 'collector', 'make-out', 'moland', 'blissful', 'low-traffic', 'siva', 'neo-realism', 'wen', 'renaldi', 'christians', 'amphibious', 'forego', 'marrow', 'mar', 'domestic', 'characterless', 'johnston', 'klingons', 'enjoyment', 'extinction', 'erin', 'paige', 'merry-go-round', 'sufism', 'ki', 'reservoir', 'oppressors', 'sure-handed', 'assent', 'coping', 'too-obvious', 'solvable', 'tights', 'eighth', 'tinged', 'civilized', 'ca', 'kristofferson', 'andrea', 'dismemberment', "f$&#in'", "capulet's", 'unsavory', 'snowman', 'elina', "greg'", "'60s/'70s", 'scoffs', 'taxi-cab', 'lieh', 'bonked', 'luc', 'takers', 'male-oriented', 'diddley', 'fritz', 'lobbyist', 'voids', 'salesmen-talk', 'undesirable', "source'", 'intend', 'invasive', 'tv-star', '`death', 'andie', "butthead's", "'do", 'brad', 'conversations', 'skewed', 'captivatingly', 'claudia', 'shelve', 'loose', 'eliciting', "jill's", 'courtney', 'antarctic', 'cong', 'kathy', 'milius', 'jfk', 'wordplay', 'tuning', 'ty', 'unsettling', 'establish', 'contractual', 'cantonese', 'by-the-book', 'dvd-rom', 'obviousness', 'lobotomizing', 'dramas', 'compelled', 'jailbait', 'family-friendly', 'haggardly', 'cold-hearted', 'brittany', 'equip', 'parkes', 'asparagus', 'bullock', 'railroad', 'careless', 'plunging', 'electronic', 'hooked', 'crumbs', 'leonetti', 'ticky', 'sprinkled', 'bull-crap', "enclave's", 'brags', 'renier', 'risen', 'managed', 'mass-market', 'excluded', 'sauntering', 'supervise', 'yelchin', 'endeavors', 'range', 'hickey', "winterbottom's", 'sanitises', 'tardio', 'dissapointly', 'satellite', 'toucher-uppers', 'back-up', 'anecdotal', 'ava', "miner's", 'entranced', 'afterword', "die'", 'non-goateed', 'unnoticeable', 'confines', 'pics', 'zen-like', 'floundering', 'aborbed', 'itself--what', 'bedford', '_dog', 'swordsman', 'silverado', 'springboard', 'often-moving', 'gardening', 'pictured', "malkovich'", 'deterent', "binoche's", '_roxbury_', 'arthur', 'reckoned', 'imitate', 'mcnamara', 'culminate', "quinn's", 'neo-reality', "mgm's", 'sorority', 'spake', "mccabe's", 'c-3p0', 'witnessing', 'songwriting', 'tarveling', 'commission', 'petals', 're-make', 'permission', 'stammers', "mugatu's", 'colleges', 'chi', 'authorial', 'roland', 'clueless', 'wilbur', 'subway', 'economic', 'syd', 'nomi', 'defines', 'wiest', 'creations', 'surprise', 'shadowy', "ishioka's", 'e-dreams', 'katarina', 'argento', 'ace', 'anticlimax', 'book', 'reingold', 'donated', "giant'", 'cleverly-constructed', '-ridden', "a'", 'go-go', '19', 'pelvis', 'tearjerker', "schindler's", 'protesters', 'major', "1900's", 'high-school', 'preston', 'parliamentary', 'viewed', 'hinterlands', 'nen', 'clapet', 'players', "that'll", 'hugged', 'someplace', '1941', 're-write', 'serial-killer', 'advising', "spall's", 'detox', 'wim', 'infamous', 'frau', 'proning', 'arquette', 'gobbledy-gook', 'rationality', 'doubting', 'persuades', 'wedlock', 'wrongdoing', 'flair', "whedon's", 'karras', 'birkin', 'undress', 'partitions', 'ig', 'minnie', 'thom', 'fairy-tale', 'outruns', 'romanticise', 'humorous', 'pilots', 'declaring', 'gbatokai', 'rock', 'counseling', "another's", 'horror-loving', 'unpublished', 'lovemaker', 'action-verb', 'meat-hooks', "america's", 'metamorphosis', 'sightless', 'electrocution', 'sickens', 'day-style', 'sikh', 'treads', '37th', "male's", 'ultra-modern', 'lovestory', 'juni', 'rosen', 'coolly', 'father-knows-best', 'claymation', "foster's", 'airheads', 'disguise', 'based', 'exists', 'buckles', 'insects', 'herding', 'dewy-eyed', 'villainy', 'supported', 'birdwatcher', 'trademarked', 'nostril', '2400', 'steamier', 'envisioned', 'amateurs', "mel's", 'klutzy', 'ony', 'potted', "-it's", "robbie's", 'intergalactic', 'receptive', 'uvula', 'gedde', 'dourif', 'friend', 'uv', 'lensed', 'obsessions', 'should--and', 'cape', 'osmet', 'mith', 'straddle', 'urgent', 'palmer-ellie', 'separated-at-births', 'slop', 'labour-intensive', 'obsenities', '[see', 'marrying', "o'hara", 'connecticut', 'expose', 'ridings', "faculty'", 'baa-ram-ewe', 'fairytail', 'lunges', 'world-renowned', 'tiny', 'cheetah', 'three-time', 'consciousness', 'criticized', 'julia-louis', 'guests', 'saxophonist', 'co-operating', "malcolm's", 'tracer', '_boom_', 'abused', 'difilippo', 'religiously-oriented', 'set-ups', 'hopkins', 'b-monster', 'takenaka', 'undeserved', 'unformal', 'redgrave', 'battler', "'kung", 'fevered', 'mysterty', 'isley', 'hawks/cary', 'a-quarter', 'congregate', 'stomach/lack', 'liddy', 'non-cynics', 'steadicam', 'rags-to-riches', 'less-than-original', 'tough-nosed', 'still-operational', 'heavenly', "furlong's", "company's", 'doom', "briggs'", "deschanel's", 'smidgeon', 'bethany', 'wiccan', 'articles', 'arachnid-type', 'monetary', 'snorted', 'collegiate', 'vaccinated', 'deep-space', 'gallows', 'ear-piercing', 'cinematogrophy', 'ridley', "minute's", "villains'", 'ziggy', '_onegin_', 'almod', 'cocaine', 'caron', 'nicjolson', 'pronounced', 'massacre', 'fistful', 'enjoyed', 'stalks', 'frontman', 'grabbed', 'bathtub', 'enduring', 'watered-down', 'rude', 'hard-nosed', 'meathook', 'pre-fab', 'dispassionate', 'tiffani-amber', 'gathering', 'custom-tailored', "rachel's", 'biological', 'i-know-what-you-like', 'arrows', 'clear-cut', 'men-only', 'discrimination', 'busby', 'fabio', "cheatin'", "brother'", 'eater', "could've", 'headliner', 'alda', "swingers'", '1986', "nolte's", "original'", 'quotable', 'transsexual', '1138', 'stubble', '4/4', 'ny', 'flurry', 'grinder', "alessa's", 'yipes', '52', 'guardian', 'initials', 'police-station', 'dumbo', 'prerequisite', 'switchback', 'recur', 'spotless', 'swimming', 'disintegrated', 'weirdest', 'asses', "railroad's", 'rethinks', 'sleepwalks', 'green-light', 'rigormortis', 'a-hole', 'spielberg\x12s', 'antwerp', 'excuse', 'grainy', 'stavro', 'jiang', '5000', "eachother's", 'damaged', 'lighted', 'ocean', 'mit', 'dishearteningly', 'nsa', 'runs', 'drug-lord', 'pollo', 'hipsters', 'besieged', 'madonna', '97', 'presents', 'travers', 'magnificent', 'to-hell-with-morality', 'ornament', '85', 'morphs', 'nostalgic', 'fervour', 'beart', "room's", 'glare', 'violently', 'spoiled-rich-girl', 'oscar', 'incited', 'epinions', 'deadeningly', 'elvis', 'direct', 'maddeningly-long', 'unredeemable', 'meter', 'flatly', 'red-and-white', "fingernail'", 'continually', 'lifelessly', 'cker', 'load', 'vanna', 'oed', 'revolt', 'highschool', 'soft-core', "conrad's", 'story-line', 'beaming', "peck's", 'frazzled', 'portuguese', 'reined', 'sappiest', 'passable', 'thirds', 'disowned', 'pathologically', 'integrating', 'aggressivelly', 'ballhaus', 'leukemia', 'sw', 'convinces', '`sleepy', '_the_quest_', "powell's", 'unflattering', 'albeit', 'shape', 'mud', 'dourdan', 'subtheme', 'daughter', 'film-writing', 'sanitized', 'turnstyle', 'easy-listening', 'cantarini', 'shrill', 'nice-guy', 'cellmate', 'kris', 'lied', 'housebound', 'action/thriller', 'skipper', 'predictament', 'fibrosis', 'shemp', 'laugh-free', 'malnutrition', 'all-too-brief', '`yeah', 'butt-numbing', 'page', "goldsman's", 'newbies', "dicillo's", 'koppelman', 'lengthy', 'editorializing', "haddaway's", 'glimpsed', 'ses', 'deanna', 'delight--', 'analysis', 'corman-grade', 'sympathy', "gus'", 'tag-team', 'steiger', 'unidentified', 'compromise', 'predecessors', 'qualities', 'poisoning', 'possiblity', 'linda', 'bubble-gum', 'niflheim', 'ex-president', 'arte', 'abigail', 'civilians', '>from', 'sentimentalized', 'malay', 'euphoria', 'algeria', 'threatening', 'seargent', 'kleenex', 'egomaniacal', 'poor', 'twirling', 'bendix', 'milo', 'slap-sticky', 'adage', 'guarantees', 'gast', 'dreading', 'uppity', 'naughty', 'arises', "floop's", 'cru', 'tracking', 'leverage-using', 'fear', 'cathrine', "perpetrator's", 'ripped-off', 'kureishi', 'loaf-sung', 'repulsive', 'pistone', 'glee', 'goofily', 'toys', 'panicky', 'ax', "'don't", 'curl', "murderer'", 'controlling', 'cancels', 'kong-style', 'performances', 'safina', 'quirkily', 'plains', 'corp', 'newsman', 'interspersed', 'hanly', 'year--every', 'freedom-fighters', 'motley', 'grump', 'madness', "bates'", 'police', 'bogus', 'building', 'heist', 'open-minded', 'gate', 'havilland', 'cockney', 'signify', 'catty', 'devolves', 'persuaded', 'dangerous', 'ade', "'animal'", 'semi-tear', 'slugfest', 'ultra', 'ex-circus', 'rotunno', 'pucker', 'portraits', 'proffessor', 'migraines', 'sickness', 'aried', 'ink', 'fandom', 'lorry', 'kell', 'eats', 'thirteenth', 'psychotically', 'dee', 'naturalness', 'gaunt', 'cyrus', 'numeric', "didn't-we-dress-funny-back-then", 'blacker', 'unbuttoning', 'exceed', 'nemeses', 'necking', 'sidewalk', 'third-string', 'dirt', "dumas'", 'handy', 'you\x12re', "'carry", 'hanksian', 'preceding', 'freeze-frame', 'vehemence', 'hironobu', 'cans', 'summations', 'deaths', 'santiago', 'gerbil', 'langenkamp', 'chabert', 'dysfunctional', 'beeyatch', 'carpets', 'spookiness', 'trump', 'skinhead-dom', 'goodbye', 'no-budget', "nielsen's", 'pipes', 'preaching', 'province', 'foreman', 'sturdy', 'self-fulfilling', 'yitzak', 'scrapping', 'tango', '\x16', 'accompanies', 'frustrate', 'gibson-inspired', 'standup', 'sub-culture', 'treason', 'panoramic', 'purveyor', 'predicting', 'eases', 'ordinary', 'jessalyn', 'sold', 'arm-mounted', "1983's", 'permit', 'plexus', 'culp', 'revive', 'modify', 'culled', 'zeta', 'wink-a', 'contrast', 'war-era', 'lift-off', "poseidon's", 'released', 'concept', 'cameo-esque', 'awakenings', 'kang-sheng', 'manly', "`simpsons'", 'tran', 'malaysia', 'politicos', 'wedge', 'enigma', 'ratso', 'heavyweight', 'novelisation', 'tearful', 'matte', 'guiltily', 'visualization', 'sylvia', '41', 'paradice', '1/4', 'mega-city', 'quintessential', 'piloted', 'seize', 'customers', 'cameron', 'once-profitable', 'intervals', 'complicity', 'numero', 'guang', 'matriarchal', 'narrations', "donaldson's", 'adoring', 'out-of-touch', 'beatings', 'assiduously', 'glutton', 'erupts', 'prudish', 'studied', 'candle', 'fences', 'novelization', '[expectations', 'amplify', 'deposits', 'kinnear', 'anti-intruder', 'corrupted', 'outdoes', 'winning', 'extravagance', '[sorry', 'strived', 'realized', 'undeterred', 'mixture', 'kettler', 'danza', "slade's", 'earn', 'amelie', 'clunker', 'ez', 'malfunctions', 'chekhov', 'surfboards', 'disapproves', 'tenfold', 'lighten', 'wabbit', 'crinkled', 'mongrel', 'assembly-line', 'self-confidence', 'definately', 'aiding', 'choreograph', 'smuggle', 'defiled', 'one_', 'elfont', 'timberland-dweller', 'silverstone', 'morphin', 'relic', "vierny's", 'gots', 'meatballs', 'arctic', 'endemic', 'lionizing', 'curvier', 'standoffish', 'rebellious', 'krikes', 'imagery', "fugees'", 'thierry', 'shaolin', 'undressing', 'cybil', 'cinematical', 'admired', "thing's", 'astronaut', 'hucksters', 'henri', 'france/usa', 'worralls', 'offers', 'dialogueless', 'incredulous', 'spaced', 'candies', 'extending', 'genre-last', 'elie', 'sacrifice', 'sanctuary', 'jonnie', 'irrationality', 'potholes', 'suschitzky', 'stahl', 'bimbos', 'wharton', 'strongly', 'utterances', 'odd-sounding', 'floris', 'neuwirth', 'baghdad', 'kegger', 'brest', 'long-suppressed', 'reel', 'half-dozen', 'huxley', 'ideal', 'intended', 'knock-off', 'quarter', 're-examine', 'delorenzo', 'devotee', "glamorise'", 'tiepin', 'horseflies', 'crackled', 'pseudonymous', 'church', 'lizards', 'wits', 'deception', 'carcasses', 'absolutely', 'caan/bruce', 'googly', 'undertsanding', 'ex-universal', 'chemisty', 'chest-', 'maples', 'kevin', 'creepers', 'symmetry', 'dichotomous', 'mean-looking', "kimble's", 'studio-imposed', 'crackerjack', 'chauffeuring', 'biker-chick', 'woman-abusing', 'murphy', 'unromantic', 'lincoln', 'impoverished', 'deficit', 'pointing', 'cross-dresser', 'worth-paying-to-see', "`scream'", 'paper-thin', 'lasts', 'greatly', 'accomplished-but-stiff', 'independent-minded', 'tiresomely', 'mythos', 'flaunts', 'olds', 'white-man-black-man', 'frightfully', 'chelcie', 'dreams/fantasies', 'jeannie', 'extra-terrestrial', "bale's", "fei-hong's", 'slumping', 'laugh-inducting', 'urgency', 'rise', 'bloodfilled', 'laxatives', 'ack-acking', 'horor', 'dion', 'cycle', 'malick', 'fallow', 'aryan', 'radek', 'line-up', 'couldn', 'acrobats', 'pseudo-playmate', 'rehearsal', "guys'", 'fiftieth', 'garters', 'mob-style', 'bumpkinisms', 'initially', 'denied', "screenplay's", 'things--tea', '5%', 'fiction-type', 'shaffer', 'archival', 'countryish', 'representing', 'fifteen-minute', 'reminiscent', 'pranks', 'wires', 'monitoring', 'blackly', 'vaulerble', 'artwork', 'barcalow', 'mistreatment', 'fascist', 'francoise', 'remakes-cum-bastardizations', 'skirt', 'hammers', 'hetero', 'grocery', 'blandy', 'coordinators', 'scream', 'ten-plus', "beings'", 'gravestown', 'attribute', 'moderation', 'unrelentingly', 'perky/cute', 'shadix', "finder's", 'touring', "armand's", 'clinic', 'bochner', 'dog_', "`she's", 'titty', '91-minute', 'compromised', 'optimistic', "'97", 'trans', "oldman's", 'ratzo', 'futuristic-looking', 'motown-singing', 'floppy', 'cardboard', 'commune', 'criticizes', 'pansexual', "god's", 'generals', 'eartha', 'demographic', 'imitations', 'unperturbed', 'dries', 'gravity', 'glamourous', 'lutz', 'enticingly', 'off-base', 'fees', 'purchase', 'savour', "harold's", 'thang', 're-inforced', 'jealous', 'litmus', 'anatomically', 'tireless', 'denton', 'oversleeps', 'drones', 'bonanza', 'verlaine', 'bidder', 'churned', 'spying', 'runaway', 'demonically', 'passover', 'bloodied', 'cheng', 'sunsets', 'acrobat', 'gradisca', 'anakins', 'writeup', 'zack', 'pedro', 'bridging', 'soppy', 'recreational', 'storyline', 'elevate', 'above-average', 'pompous', 'ginger', 'anally', 'novelty', 'bask', 'unattractive', 'typecast', 'ad-libbed', 'vatican-sponsored', 'bloodflow', 'instigate', 'midwestern', 'maury', 'suffer', "somebody's", "1999's", 'taiwan', 'hooligans', 'mitchum', 'logjams', 'hams', 'nice', 'mediating', 'member', 'writers/directors', "harkonnen's", 'swell', 'marine', 'blackmails', 'huison', 'hours', 'peppy', 'pony', 'reclusion', 'jar-jar', 'sharpening', 'hangers', 'sallah', 'flickering', 'mid-life', 'root', 'heartwarmers', 'levitt', 'companion', 'prestigious', 'deadpan', 'uk', 'urkel', "ann's", 'truman', 'red-headed', "mamet's", 'destinies', 'horizon', 'go-between', 'dismissed', 'tsui', 'talker', 'wasn\x12t', 'knell', 'gentile', 'lucille', 'stupids', 'schmoozy', 'hospitalizes', 'built-in', 'situations', 'phobic', 'gainfully', 'handy-dandy', 'cages', 'retribution/fight', 'butterfly', 'made--and', '_many_', 'cassandra', 'cool--and', 'assination', 'ruggedly', 'twisted', 'crate', "`plump'", 'overacted', "who'd", '000--paltry', 'presentation', 'stationary', '1942', 'apparantly', 'evaporates', "chelsom's", "guinness's", 'gaerity', 'patience', 'orphanage', 'veronica', 'over/underwritten', 'chopped', "tamora's", 'bullet-shaped', 'recognized', 'bloodsucking', 'unconsciously', 'partisanship', 'well-kept', 'maggie', "wilson's", 'stargate', 'clarification', 'adequate', 'concede', 'warfield', "lion's", 'oscar-winner', 'constantine', 'prophet', 'sakari', 'unmistakeable', 'asexual', 'clich-heckerling', 'shiva', 'peachy', "all-in-a-day's-work", '1/2th', 'park', 'artillery', 'immortal', 'reversal', 'yeller-style', "goro's", 'warp', 'consumption-crazy', "master's", 'kamikazes', 'differentiated', 'typicalness', 'so--there', 'jaded', "dragonheart's", 'mining/slave', 'hole-in-the-wall', 'mess', "end'", 'mac', 'blunts', 'annoys', 'camp', "brazil's", 'ding-dong', 'stephanopolus-style', 'battling', "`x-files'", 'by-the-numbers', 'desmond', 'mildred', 'gums', 'matzo', 'innovate', 'proud', 'threaten', "it's-a-holocaust-film", 'jonny', 'scripters', 'housewife', 'portland', 'indictment', 'hybridizes', 'waif', 'beware', 'bendrix', 'lugubriously', 'hooey', '_scarface_', 'motive', 'crackling', 'attachs', 'degenerating', 'hunters', 'grindhouse', 'smokejumper', 'placards', 'mitchell', 'maneuver', 'manolo', 'films-run', 'crafts', 'weekend', 'secular', 'efficiently', 'messiness', 'powered', 'lavender', 'man-ethnicity', 'sheila', 'trustworthy', 'aumont', 'clothesline', 'ultraman', 'fire-breathing', 'windows', 'dateline', "hirst's", 'hi-jinks', 'pre-juliani', 'brain-over-brawn', 'longbaugh', 'marilyn', 'crusading', 'remade', 'tit-shot', 'wannabes', 'writer/director/star', 'specializing', 'withstand', 'meticulousness', 'satisfactory', 'alongside', 'anomaly', 'tipped', 'richmond', 'hyberboreans', 'advisers', 'skips', 'torching', 'test', 'patois', 'chop-socky', 'lindenlaub', 'elisabeth', 'cocaine-i', 'mankiewicz', 'wounding', 'spoke', 'unneeded', 'plague', 'edification', "short's", 'anti-gay', 'rosenberg', 'carolina', 'i-think-i-actually-have-', 'licensed', 'greenery', 'grungers', 'foil', 'eugene', 'crookier', 'enclosed', 'audience', 'ladder', 'spook', 'trailed', 'water', 'complexion', 'hard-rock', 'sting', 'air', 'r2-d2', 'windmill', 'boris', "players'", 'self-help', 'cataloguing', 'idolizing', 'hairdresser', "rueff's", 'jaw-dropper', 'gasps', 'reassess', 'wild-and-crazy', 'reclaim', 'rimini', "chad's", 'pinup', 'prizefighter', 'overdirected', 'bigelow', 'baby-sitter', 'monosyllables', 'gun-', 'refreshed', 'quits', 'barbarity', 'bat-loathing', 'pitcher', 'laid', 'fully-animated', 'veil', 'uncomfortably', 'moira', 'spacewalk', 'unison', 'ancy', 'kidding', 'mpaa', 'checkmate', 'tone/mood', 'iranians', 'hyper-silly', 'review-', 'eve', 'shouldn+t', 'black-tie', 'replaying', 'glib', 'antonio', 'skunk', 'fascinating', 'keyzer', 'three-dimensionality', 'recognizably', 'ho-sun', 'sects', 'cylinders', 'sarris', 'legionnaire', 'motorist', 'henchmen', 'pursuing', 'bicep', 'rosaleen', 'revisit', 'man-hating', 'memories-', 'tues', 'connotations', 'rage', 'conscientious', "messenger'", 'filtered-light', "dealers'", 'protestants', 'assured', 'expositions', 'rooney', 'sex-film', 'sustaining', 'perverse', 'intuitively', 'spader', 'jerky', 'congregations', 'take-no-prisoners', 'doubtful', 'heros', 'threw', 'not-so-engrossing', 'out-thinking', "grace's", 'exquisitely', 'western-like', 'slide', 'fiction-esque', 'vh1', 'chrysler', 'rowlands', "jeffrey's", 'moviegoers', 'perfectly-groomed', 'instructional', 'blueprints', 'libby', 'coalition', 'heartstopping', 'karate', 'skewering', 'lenser', 'ugly', 'paradiso', 'ineptitude', 'bad', 'you=92re', 'legends--is', 'bounces', "three's", 'hooted', 'pseudo-suspenser', 'appendages', 'self-righteous', 'mandatory', 'saddens', 'filmmaker', 'pulp', 'whic', 'ronald', 'transfixed', 'presuming', 'unprecedented', 'scenery-chewing', 'over-directing', "rockin'", 'cars', 'spotting', 'edges', 'estranging', '123', 'suggest', 'romp', 'auxiliary', 'phyllida', 'eternity', "dixon's", 'barletta', 'landfill', 'executives', 'flashily', 'seagrave', 'circulate', "series'", 'united', 'sleuthing', 'incriminating', 'saskatoon', 'dreamer', 'formulaic', '1967', 'lazy-as-always', 'fellowship', 'lamentable', 'fracture', 'francisco', 'ikwydls', 'suicides', 'klitorsky', 'instill', 'foghorn', '`mr', 'profess', 'romeo+juliet', 'housed', 'favored', 'fundraisers', 'contenda', 'writing/directing', 'movies--this', 'cleaned', 'semi-official', 'worked', 'us-mexico', 'excelled', 'harbinger', 'awoken', 'milano', 'pr', "salsa'd", 'flits', "city'", 'mixes', 'tree', "baby's", "astronauts'", 'blonde', 'linder', 'shell_', 'implores', 'wagnerian', "crackin'", 'agutter', 'intriguingly', "burrough's", 'generating', 'fatter', 'scatalogical', 'hypothetically', 'cut', 'hardest', 'weslely', 'usda', 'buliwyf', 'equine', 'borrow', 'focus', 'dullness', 'shoring', 'heh', 'ahmed', 'eastern-sounding', 'checkpoint', 'deplorable', 'wichita', 'heights', 'by-play', 'fad', 'favorably', 'vowed', 'college-set', 'irregulars', 'beds', 'unfilmable--the', 'xref', 'jimmy', 'no-goodniks', 'credited', "up'", 'terrify', 'cheesily', 'thriller/suspense', 'changed', 'rag', 'chucky', "stravinsky's", 'lestercorp', 'maliciously', 'aw', 'lams', 'minutiae', 'midsection', 'panther', 'rico', 'parillaud', 'yacht', 'dollar', "alain's", 'townie', 'meir', 'ant', 'ex-presidents', 'affluent', 'adafarasin', 'ether-addicted', 'holden', 'goldenberg', 'drug-dealers', 'troopers', 'valley', 'wh', 'punch', 'multiracial', "boulle's", 'depite', 'keel', 'accompanying', 'throw-together', 'investor', 'translated', 'collapses', "cruise's", 'kirstie', 'rand', 'humanly', "marie's", 'zoo', 'cost', "most'", 'isaacman', 'apple', 'a-flailing', 'unmagical', 'rathbone', 'supernatural', 'industrialist', 'munchie', "paramount's", 'messageable', "aurelius'", 'linearity', 'vamp', 'carat', 'curiously', 'friendship', 'sporty', 'update', 'smits', 'stands', 'balderdash', 'pryor', 'bunnies', 'often-acclaimed', 'supplying', 'press', 'roadblocks', 'discussing', 'dedication', 'reinvented', 'driller', 'geneva', 'oscar-contender', 'dennis', 'screwer', 'nighshift', 'telephone', 'high-camp', 'whack', 'crock', 'unheralded', 'erupting', 'back-to-', 'hunted', 'antithesis', 'eggo', 'fires', 'waxes', 'charging', 'pros', 'no-hoper', 'boy-and-dog', 'perversely', 'cooked', 'lambeau', 'talbot', 'recruiter', 'semi-predictable', 'whatever-the-f', 'persons', 'leigh', "puppeteer's", 'peterson', '51', 'inseparable', 'contingent', "speilberg's", 'johnson', 'holnist', 'sexualize', 'doltish', 'enamored', 'electro-green', 'parsley', '`where', 'civil', 'lightheartedness', 'considerable', 'transposes', 'vanzant', 'patachou', 'metamorphosizes', 'told', 'sustains', 'japanimation', 'winner', 'leaning', 'plexiglas', 'tv-movie-of-the-week', 'arabs', 'complementary', 'quicktime', 'astoundingly', "renee's", 'gorefest', 'galileo', 'twig-sculptures', '-drenched', 'would-be-girlfriend', 'seinfeld', 'demeanor', 'angsty', "show's", 'downers', '95%', 'fess', 'clowns', 'jennifer', 'schwarzeneggar', 'sonnenfelds', 'piss-poor', 'producing', 'petstore', 'emmerich', 'juan', 'upteenth', 'dweebish', 'shifts', 'cliched', 'pseudo-sexy', 'damian', 'hard-sell', 'swallow', "bilal's", 'breillat', 'advisable', 'levis', 'haired', 'breech', 'night-journey', 'herniated', "'clerks'", 'laundry-impaired', 'slow', 'blemheim', '1930s', 'passions', "bernstein's", 'disconcertingly', 'misdemeanors', '1928', 'sequence--scored', 'decadent', 'slooooow', 'climbing', 'exasperation', 'meander', 'intertwined', 'asexually', 'insinuate', 'colt', 'equal', 'shindig', 'misery', 'pay', 'variable', 'family-run', 'obscura', 'mugshots', 'neumann', 'cs', 'slowly', 'slo', 'elicits', 'pastel', 'angrier', 'redecorating', 'broadly-drawn', 're-cuts', 'unanswerable', 'modern-day', 'weasted', 'conspirators', 'rough', "romeo's", 'crs', 'eduardo', "f-18's", '23-year-old', 'controversy', 'hiroshi', "zeffirelli's", 'artifically', 'overwritten', 'vasquez', 'wince', 'connects', 'borders', 'backwards', "ulysses'", "ryan's", 'zelweggar', 'importance', 'opposing', 'turgeon', 'unanswered', "presidents'", 'pseudo', 'scrooge', "general's", 'negativity', '--you', 'island', 'recorded', 'magdelene', 'cabinet', 'masturbatory', 'won=92t', 'rumoured', "'authenticity'", 'wanderlusting', 'graeme', 'drop', "dobie's", '1990s', 'blueprint', 'destabilizing', 'addressed', 'examinations', 'compares', '82-minutes', 'laserdisc', 'regains', 'spaceport', 'farm', 'touches', "geoffrey's", "talon's", 'libido', 'shoulder-length', 'ellicited', 'lifesaving', 'org', 'unger', 'tasteful', 'succeeds', 'avenge', 'sigel', 'craftors', 'wrote', 'unending', 'horny', 'pant-wetter', "salt's", 'lucifer', 'pulls', 'well-noted', 'assigns', 'barely-teen', 'breath', 'mcginley', 'schomberg', 'adornments', 'picketers', 'interrogation', 'republican', 'self-referential', 'mst3ked', 'bernard', 'sleezo', 'relinquished', 'tumbleweeds', "seriously-it's", 'faulted', 'harve', "nice-guys-who-don't-deserve-to-be-in-prison", 'immigrants', 'workplace', 'vegetative', 'self-references', 'netted', 'protecting', 'wheaton', 'henpecked', "hbo's", 'silkwood', 'eruption', 'aux', 'wowzers', 'straight-arrow', 'authentic', 'hell', 'blowed', 'hummingbird', 'gross-out', 'bonafide', 'beloveds', 'alderaan', 'f/x', "korben's", 'suspend', 'sojourn', 'in-jokey', 'improbabilities', 'shaping', 'barn', 'hypernaturally', 'interrogate', '_54_--so', 'bombing', 'hamm', 'na', "girl's", 'ratty', 'monogamy', 'feasible', 'sickening', 'pimple-faced', '`that', 'algae', 'packaged', 'post-revolutionary', 'buffoon', 'ruthless', 'smoothes', 'isaak', 'complicates', 'pastry', "guggenheim's", 'paul', 'boatloads', 'ogre-witch', 'idealist', 'finding', 'thurral', 'samoan', 'entertained', 'upkeep', 'diesl', 'non-sexual', 'nominees', 'docks', 'simian', 'receives', 'internment', 'regulation', 'psychiatrist', 'slutty', 'miriam', 'his/her', 'uberhack', 'protege', 'moat', 'dorky', 'dam', 'four-letter', "carpenter's", 'mostly-unrelated-storywise', "max's", 'matthews', 'paramount', 'geronimo', 'witted', "'act'", 'chan-film', 'decapitation', 'longshank', 'gangster-flick', 'wad', 'overcast', 'automated', 'destined', 'unite', 'marko', 'feng', 'un-idolizable', "prince's", 'well-connected', 'temperature', 'walk-on', 'budgeted', 'unsurprisingly', 'nympho', 'catatonically', 'preoccupation', 'circus', 'hilariosly', 'shells', 'direction', 'accelerating', 'kidnapers', 'subject', 'murkily', 'relocates', 'thurgood', 'innit', 'minority', 'roundtree', 'bridget', 'legion', 'dove', 'anti-shark', 'monopoly', 'reemerging', 'digress', "four's", 'snobs', 'overt', 'mira', 'put--it', 'matted', 'chopra', "sarossy's", 'hide', 'scrutinize', 'stocky', 'caressing', 'mekanek', 'keyed', 'ex-flame', 'mind-fuck', 'frightened', 'bruce', 'voicework', 'teens', 'gidley', 'fact--that', 'romper', 'cripple', 'sneaky-smart', "cinematographer's", 'valentine_', 'harridans', 'councilmen', 'foxy', 'dwarf', 'sacrificial', 'rambles', 'co-script', "uecker's", 'misha', 'champion', 'burnett', 'brothels', "mia's", 'sethna', 'best--an', 'bagged', 'runner', 'cacophonous', 'pleasantville', "crazy's", 'text', 'yagher', 'hematologist', 'hallstrom', 'mostel', 'apt', 'rosy', '2d', 'decomposing', 'sunrisesucks', "human's", "f'gawds", 'aggrivate', 'bassan', 'mind', 'temporary', 'titus', 'goofball', 'scathingly', '1900s', 'remaining', 'locale--to', 'eensy', 'love-interest', 'disgrace', 'leatherface', 'planning', 'ego', "gates'", 'squalid', 'eye-popping', "dreyer's", 'slouched', 'election', 'bows', 'ferret', 'cate', "schumacher's", 'nicotine', 'xin-xin', 'smithees', 'urinate', "'long-lost'", 'themes', 'edited', 'sub-standardly', 'preppy', 'burial', 'straight-ahead', 'pvt', 'cleophus', 'owning', 'b-actress', 'buckle', 'box', 'confused', 'degenerative', "heroine's", 'statements', 're-evaluated', 'folding', 'manipulations', 'helene', 'kietal', 'piracy', 'ingesting', 'curb', 'screamingly', 'elastica', 'm-16', 'fawning', 'jon', 'aide', 'micelli', 'mechanically', 'taste', 'hacking', 'artist-directors', 'anxiety-ridden', "revolution's", 'tough-guys-who-rarely-', 'dionysius', 'resultant', 'earp-style', 'swamped', 'terrific', "parson's", 'manh', 'bumbles', 'well-off', "hanks'", 'joker-smile', 'cured', 'sachs', 'digger', 'jerusalem', 'truth', 'sea-sickness', 'constitute', 'deficiency', "libby's", 'bello', 'rounding', "hotel's", 'compensation', '`any', 'unintelligently', "maximus'", 'detach', "lucasfilm's", 'rais', 'anecdotes', 'janssen', 'garbageman', 'equations', 'pickpocket', 'afflicted', '`everyone', 'goofy-but-loveable', 'purest', 'ill-marketed', 'openly', 'laertes', 'intoxicated', 'release--john', 'organs', 'ear-plugs', 'mocks', 'paz', 'welsh', 'brainless', "britain's", 'happiest', 'extraordinary', 'misuse', 'mistaken', 'dues', 'restrained', 'vista', '1298', 'less-than-lackluster', 'hiker', 'baths', 'security', 'amuse', 'sequoia', 'virgins', 'blindingly', 'classmate', 'thompsons', 'degenerates', 'nosebleeds', 'ra', 'ranks', 'momsen', 'enthralled', '10', 'revengeful', 'literally', 'petrified', "'s", 'pattern', 'glue-like', 'dubarry', 'seats]', 'cassanova-ness', 'courtland', 'sherlockian', 'doozy', 'majestically', 'soda', 'reusing', 'flanery', 'mehrjui', 'handling', 'dispel', 'baffling', 'northeast', 'eroticism', 'cords', 'suture', 'bohemian', 'fall-down', 'argumentative', "love's", 'pacino', 'summons', 'richie', '_gattaca_', "ali's", 'cuckold', "abraham's", 'groundbreakingly', 'discussed', 'frisson', 'citywide', 'delaughter', 'bolek', 'constructive', 'consul', "hogan's", 'beesley', 'cartoon', "music's", 'cornwall', 'revenues', 'larceny', 'andrei', 'dershowitz', 'graphically', 'escape', 'robotically', "chestnut's", 'fishy', 'quaternary', 'lessened', 'brainwash', 'frames', 'evidence', 'marvelously', 'unloaded', "shelia's", 'haas', 'roller-coaster', 'historian', 'approved', 'understandable', 'slowest', 'nicholas', 'rubs', 'lizard-stomps-manhattan', 'veronicca', "needy'", 'undresses', 'possessed', 'vital', 'ethereally', 'mos', 'robbi', 'competent', 'alter-ego', 'dwell', 'oh-so-fabulous', 'longer', 'compress', 'humanity--is', 'upstaged', 'dylan', 'kihlstedt', 'emphasizing', 'slam', 'husband-to-be', "socialites'", 'don92t', 'higgins', 'unimaginative', 'depp-type', 'ro', 'mid-eighties', 'monet', 'hustling', 'hit-list', 'processors', 'bojan', 'fourteen', 'undeterring', 'country-club', 'garrett', 'principles', 'charleton', 'geezer', 'gotti', "world'", 'reticence', 'waddle', 'romanticizes', 'nba', 'movie-maker', 'khan', 'penalty', 'racer', 'islam', 'kaczynski', 'mccartney', 'heroine--gets', 'premises', 'francisco-based', 'uhf', 'climate', 'alt-rock', 'surprisinly', 'titillate', 'tobolowsky', 'walnuts--except', 'artistical', 'considered', 'granddaughters', 'usually-splendid', 'endor', 'estevez', 'ragsdale', 'bloodsuckers', "'attila'", "brill's", 'now-tired', 'apperance', "strangelove's", 'birthdays', 'egotisitical', "darren's", 'siamese', 'lurve', 'hands-down', 'enquirer', 'gut-wrenchingly', "wender's", 'pulsating', '18+', 'schwinn', "era's", 'overdramaticizes', "meir's", '_rushmore_', 'burstyn', 'law-suit', 'atheist', 'buddying', 'grater', 'mccabe', 'desensitization', 're-open', '17%', "bachelor's", 'dredges', 'boating', 'sumptuous', "raison-d'etre", 'divisive', 'motivate', 'rodney', 'gibbons', 'hearing', '115', 'pays', 'law-enforcement', 'restlessly', 'spoofing', 'masterminding', 'accurate', 'nosy', 'starved', 'kennedys', 'sheen', "band's", 'outlandish', "shows'", 'jars', 'fluctuations', 'furter', 'meaningful', "approach'", 'bon', 'gonzo', 'thumps', 'daniels', 'buffed-up', 'psychotic', 'board-room', 'pre-sold', 'pleas', 'admittingly', 'becase', 'unconditionally', 'chatterbox', 'stillborn', 'patti', 'output', 'image', 'mychael', 'nbc', 'farrellys', 'revel', 'inelegantly', 'top-selling', 'didn', 'savini', 'incompatible', 'battle-worn', 'cynthia', 'nutcases', 'treasury', 'idisyncratic', 'shelves', 'semi-remake', 'assaults', 'posthumous', 'interpreter', 'toilet', 'technobabble', 'middle', 'fancies', 'nc-17', 'macho', 'middle-of-the-road', 'shackleton', 'urbanites', 'commited', 'vertiginous', 'standpoint', 'bartholomew', 'mainframe', 'post-party', "professor's", 'shorten', 'antidote', 'rerun', 'ripping', "5'10", 'faye', 'gilliard', "dog's", 'vitamin', 'martin', 'destroying', 'shimmying', 'blokes', 'social', 'pitt', 'naivet', 'two-thirds', 'shannen', 'practice', 'mcniell', 'elimination', "corpses'", '_dead_', 'well-deserving', "movies-i've", 'trent', 'middlemen', 'word--it', 'unpure', 'flamboyantly', 'girl-poor', 'action/crime', 'resonation', '`apollo', '[on', 'koopa', 'made-for-tv', 'planetarium', 'insular', 'roundhouse', 'line-ups', 'masterless', 'constitution', 'mozambique', 'seldom', 'huh', 'emblazoned', 'celluloid', 'friday', 'tomb', 'forewarned', 'shakes', 'exotica', 'spiffy', 'quantum', 'spilt', 'semen', 'benecio', 'uecker', "ld's", 'turkey', 'dramaturgical', 'successors', 'energy', 'predicaments', "sand's", "francis'", 'shackled', 'mutha', 'weren=92t', 'sprang', 'success', 'lennie', 'waddington', 'repulsed', 'curtis-hall', 'gonzales', 'x-ray', 'meekness', 'maneating', 'extraterrestrials', 'snipping', 'vitti', 'effectual', 'lost', 'maven', 'remainder', 'laudable', 'knock-out', 'lionized', 'slams', 'c-note', 'distracts', '`down', 'nibble', 'captives', 'bratty', 'north', 'rodents', 'obstructing', 'cemetary', 'near-ruin', "'star'", 'stivaletti', 'thrillride', 'sprawling', 'near-eden-like', 'firmly', 'workmates', 'insecurities', 'furry', 'kong', 'spike', 'tithe', "burden's", 'aliens', 'classifies', 'gila', 'earth', 'sneer', 'south--forever', 'spielbergization', 'pig', 'chaps', 'expediency', 'seediness', 'unappealing', 'costner', 'negotiating', 'lights', 'hawking', 'shoot-em', 'gielgud', 'devil', '_dead_man_on_campus_', 'deliberation', 'mumbles', 'bloodstream', 'deschanel', 'believability', "charles'", 'pulse', 'defied', "that'd", 'as--of', 'ever-wise', 'heart-wrenching', 'sketched', 'shrieking', 'pushkin', "lem's", 'ladyship', 'un-creative', 'meal', '_the_lion_king', 'perpetuated', 'moneymakers', 'sprinkler', 'coastline', 'near-', 'bohemians', 'yowza', 'subjugate', "kennesaw's", 'hearty', 'interludes', 'eclairs', "`family'", 'cooper', 'corpses', 'alternative', "lester's", 'choices-', 'silvio', "'driven'", 'depending', 'draws', "d'artagnon", 'maid', 'mill-types', "'invaders'", 'humanness', 'sadomasochism', 'suggested', 'motta', '22-5', 'chase', 'macaw', 'salesmanship', 'housewarming', 'returning', 'harmless', 'emergence', 'recommeded', 'bastille', 'penciller', 'delta', "brewster's", 'underappreciated', 'scrape', 'nicolette', 'non-literal', 'pushed', 'multiplexes', 'skyline', 'ohio', 'buton', 'tommorrow', "'meaningful'", 'lanai', 'viewpoints', 'serving', 'equate', 'rogues', 'resubmit', 'mismatched-buddy', 'bibb', 'realises', 'counterpart', 'nitro', 'bongos', 'doctor', 'valeria', 'electrical', "beaver_'s", 'studio-financed', 'disciplined', 'outlands', 'plunged', 'therapists', 'foisted', 'herculean', 'mba', '131', 'hum', 'hecklers', 'sic', 'bemused', 'drown', 'aromas', 'enraging', 'expanded', 'emcee', 'ness', 'rasaan', 'jewel-thief', 'melange', 'ironsides', 'self-defense', '[read', 'similarities', 'lamenting', 'soaring', 'yearnings', 'woken', 'condone', 'fillmore', 'unamusing', 'podrace', 'members', 'penis', "o'donaghue", 'riverboat', 'dumbed-down', 'trade', 'task', 'niebaum', 'inscrutability', "soze's", 'advertising', 'simplify', 'pit', 'cover', 'sissy', 'mironoff', 'meritorious', 'it-', 'pratically', 'firmer', "animal's", 'absolut', 'coke-head', 'farce', 'wringing', 'fahrenheit', 'scrutiny', "russell's", 'airplay', 'yevgeny', 'discuss', 'jelly', 'yakking', 'fidel', 'dowdy', 'huggers', "'70's", 'not-quite-half-baked', 'agony', 'cardshark', 'recommened', 'dulloc', 'scenic', 'fame', 'holocaust', 'staccato', '`roswell', 'getting-even', 'grain', 'automobile', 'anthropologists', 'geiger', 'affecting', 'profanity', 'xander', 'disgraced', '`fight', 'equation', 'newspapers', 'spiritualistic', 'flinging', 'compulsion', 'poles', 'evenly', 'luhrmann', 'councils', 'caught-with-his-pants-down', "`closure'", 'jug', 'pastures', 'burne', 'personality-challenged', 'foch', 'cowboys', 'cretins', 'sameness', 'bystander-extras', 'ernst', 'hawaii', 'dungeons', "anorexic's", 'failings', 'halperin', "foley's", "michell's", 'reconnect', 'drop-dead', 'snowfall', 'hassled', 'replenish', 'miscalculated', 'herskovitz', 'chamberlain', 'spera', 'partment', 'ex-minister', 'ass-cracks', 'enterprising', 'concerns', "'what's", 'heartbreak', 'style', 'shinohara', 'sterotypically', 'good-horror', 'investigations', 'creeps-and', 'chalkboard', "heyst's", 'disagree', 'azrael', 'cries', 'bantu', "merton's", 'coasts', 'eng', 'arden', 'ending', 'raiden', "denzel's", 'berserk', 'geocities', 'freeman', 'shreds', 'meri', 'willingness', 'schmidt', 'rip-off', 'arizonians', 'recount', 'unsupervised', 'orthodox', 'existenz', 'distilled', 'affirmation', 'crocodile', 'merge', 'lobotomized', "shaw's", "fiennes'", 'groveling', 'locane', 'squirming', 'ditzism', 'teen-styled', 'framing', 'waking', 'pencil', 'hemispheres', 'haul', 'platinum', 'chappy', "spillane's", 'bounty', 'alienation', 'epic-wannabe', '29', 'guiness', 'insubordinate', 'marquee', 'face', 'rigors', 'beverly', 'bush', 'overarching', 'choreography', 'sheridan', 'appeared', 'sykes', 'critic-proof', 'kiffer', '--', 'near-final', 'combo', 'recognisable', '=====================', 'horse-training', 'grouch', 'converges', 'chills', 'morricone', 'hollywoods', 'horatio', "marcus's", "train's", 'well-publicised', 'glorifies', 'whitlow', 'predictably', 'collegue', 'preconceived', 'schuemacher', 'logan', 'swanky', 'wildest', 'apprehensive', 'precedence', 'overuse', 'incomplete', 'defenceless', 'falcone', 'building-up', 'bokeem', 'centers', 'olivier', 'fresh', 'ingred', "morricone's", "malone's", 'old-as-time', "wallace's", 'candor', 'affect', "1995's", 'defender', 'dung', 'beeline', 'crap', 'pioneers', "spears'", 'complement', 'lows', 'nobility', 'hero/heroine', 'panned', 'resnais', 'northmen', 'transplant', 'deaf-mute', 'outtake', 'windbag', 'twiddling', 'over-70', 'cort', 'sees', 'murtaugh', 'buffs', 'amarillo', '250', 'highly-placed', 'minor', 'names', 'caliber--and', 'registration', 'trigger', 'insert', 'joey', 'trivialities', 'elongated', 'things--nielsen-boosting', 'zimmely', 'shop', '`double', "harrelson's", 'possession', 'nypd', 'pseudo-intellectual', "`friends'", 'presented', 'nailbiter', 'treat', 'goods', 'mornay', 'testimony', 'jojo', 'oskar', 'teammate', 'khanna', "flick's", 'hammett', 'leisurely', 'co-dependency', 'mother-of-all-storms', 'honking', 'toad', 'anti-nukes', 'info', "54's", 'enlightened', 'weitz', 'dissapointed', 'lackluster', 'corporate', 'multi-national', 'joining', 'kurtwood', 'variations', 'mintues', 'bossy', 'bettany', 'chin', 'nagle', '54th', 'communication', 'treville', 'cmdr', 'defensive', 'bookshop', 'gravy', 'double-crossing', 'unwise', 'sweetback', 'iambic', 'performing', 'celestial', 'supple', 'score', 'turturro', 'hardworking', 'yosarian', 'enthusiasm', 'dresses', '\x13if', 'chris+', "sewell's", 'self-respect', 'krubavitch', 'gungan', "conservatives'", 'unusual', 'microcosm', '96', 'misperception', '1987/88', 'synonyms', 'tolkin', 'veins', 'exhumed', 'yelling', 'undiscernable', 'sudan', 'lame', 'verification', 'operates', 'betrayal', 'depicts', 'liaison', 'ann-margret', '`groundhog', '`thelma', 'bloodless', 'marc', 'questioners', 'achilles', "bumpy's", "swartzenegger's", '-to', 'paychecks', '[superior', 'full-on', 'sodium', 'delighted', 'rumors', 'ex-convict', 'declares', 'mack', 'visitor', 'brando', 'cancel', 'kingdome', 'contestants', 'stored', 'confession', 'migrant', 'antisocial', 'cleaner/slave', "inventor's", 'plotted', "windsor's", 'necks', 'puff', 'laila', "'isolated'", 'desperatly', "vampire'", "murray's", 'onw', 'bram', 'wears', 'oscar-hungry', 'bedhopping', "lomax's", 'n-word', 'assert', 'overabundant', 'achievable', 'firings', "d'etre", "sargent's", 'pianist', 'peace-loving', 'triangles', '1985', 'pointlessly', 'expectant', 'macalaester', 'taxes', 'infrequently', 'duplicitous', 'suspended', 'exam', 'agreements', 'wheelchair-bound', 'rethinking', 'overpraised', "hell/i'll", 'comedy-drama', 'texture', 'storyboarded', 'world-wide', 'mainland', 'swiftness', 'ultra-nationalist', 'big-screen', 'preservation--', 'unscrupulous', 'rooms-into', 'verna', "lara's", 'gregorian', 'sleep', 'finer', 'hook-laden', 'resurrect', 'labyrinthine', "'rumble", '2/10', "day'", 'remarkably', 'atom', 'anguished', 'tamala', "twins'", 'passionately', 'momentary', 'rasputin', 'greta', 'thor', "comedian's", 'ripley', 'workweek', 'reforming', 'kirkpatrick', 'won\x12t', 'priestley', 'discounted', 'stained', 'pencil-thin', 'journeying', 'demonstrations', 'tatum', 'talent--samuel', 'feebly', "mignatti's", 'projected', 'dentures', 'glitters', 'tilt-o-whirl', 'quickly', 'sucked', 'hensons', 'laurie', 'musker', 'serialized', 'morphed', 'bankroll', '666', 'musical', 'culminating', 'writing-henry', 'chevy', 'lumet', 'cleary', 'beauties', 'marlon', 'nth', 'ming-na', 'talked', 'decision--irving', 'hoblit', 'repellent', 'resonated', 'rewatched', 'elicited', 'cap', 'defeats', 'afterthought', 'moreu', 'cigar', 'unspeakable', 'trampy', 'paced', "casper's", 'woodsboro', 'overcoat', 'markings', 'colanised', 'kilometres', 'scold', 'ennio', 'new-found', 'world-view', 'hark', 'drippy', "metoclorian'", 'fingerprints', "scumball's", 'axe', 'lehman', '1839', 'deflects', 'barge', 'drop-offs', 'lacks', 'capable', 'uneasy', 'ashby', 'tune', 'gabrielle', 'contributors', 'mozart', 'suction', 'pan', 'abrahams', 'well-', 'ji', "santa's", 'double-whammy', 'prescient', 'anti-rich', 'perceptions', 'hours--or', 'trish', "demille's", 'nothingness', "fun'", 'double-pronged', 'well-oiled', 'prouder', "'strangers'", 'magnetic', "jody's", 'mobilizes', 'gazillionaire', 'full-fledged', 'camps', 'fortunately', 'contrasts', 'dinners', 'brolin', 'elegance', 'us-they', 'leonardi', 'lemay', 'movingly--into', 'concubines', 'forces', 'influx', 'vine', 'mucked', 'builder', 'bone', 'twosomes', 'staying', 'iranian', 'brancia', 'parole', "piano's", 'fares', 'computer-generate', 'tante', 'confederate', 'monomanical', 'scott-thomas', 'relieves', "pullman's", 'triton', 'operated', 'adherence', 'genteel', 'teriffic', "menges'", 'unbelievable', 'x-rating', 'preconception', 'ducks', 'remotely', 'thought-transference', 'juxtaposing', 'improve', 'gail', 'pocohontas', 'terrier', 'proclaimed', 'santanico', 'submitted', 'anh', 'god', "toys'", 'awarded', 'informed', 'carousel', 'quynh', 'segment', 'end', 'crane', 'spokesperson', 'comedy/mystery', 'cognac', 'comparitive', 'candy-coated', 'ole', 'chief-of-stafff', 'mischievously', 'passangers', 'cringing-', 'obtain', 'darius', 'utilized', '5-cent', 'moby', 'millennia', 'promoter', '25-years', 'first-tier', 'break-up/make-up', 'non-exploitative', 'hensleigh', 'homcoming', "patrik's", 'grudges', 'drug-kingpins', 'soothing', 'lethal', 'bascially', 'snails', "raymond's", 'ex-husband', 'disturbed', 'sanitarium', 'lost--literally--on', 'raleigh', 'agitate', 'fatigues', 'plently', 'matter-of-factness', 'responding', 'high-calorie', 'greener', 'shoulda-been-nominated', 'shayne', 'bewitched', 'delicacy', 'pistella', 'gizzards', 'raimi/rob', 'jet', 'goat', 'uh-oh', 'debilitating', 'personable', 'edmunds', 'labeled', '50%', 'scene-stealing', 'night-vision', 'australian/belgian', 'fan', 'pinchot', 'movie/tv', 'treacherous', "roddenberry's", 'furrows', 'angst-horror', 'leaf', "insurrection's", 'adviser', 'artsier', 'sleazeball', "sculptor's", "'debate'", 'steam', 'bars', 'roswell', 'mutilated', 'irony', "catch's", 'forcefully', 'half-hidden', 'lipnicki', 'matching', 'mentoring', 'machinist', 'kaczkowski', 'haste', 'well-spoken', 'threads', 'out-takes', "phil's", "flanders'", 'uzi-firing', 'fryer', 'condemned', 'rectified', 'cottrell', 'menage', 'koran', 'congratulates', 'littering', 'not-so-cheap', 'abetted', "lombardo's", "mozart's", 'extortion', 'dictated', 'renowned', 'qualms', "hooker's", 'norrington', 'actualization', 'flailing', 'post-apocalyptic', 'whitechapel', 'pefect', 'pidgeonhole', 'disciple', 'virtuosity', 'lunkheads', 'altogether', 'iniquity', 'kickback', "stupids'", 'retribution', 'recruited', '$130', 'fifteen', 'waterway', 'plotting', "duke's", 'galahad', 'resembles', 'tinker', 'word', 'american-ized', 'deadbeat', "nelson's", '91', 'alienating', 'perishes', "dane's", 'discovered', 'freebie', 'commercial-like', 'byrne', 'goeth', 'clipboard', 'bloom', "matilda's", 'pitchfork', 'insoluble', 'pantomine', 'norma', 'commando', 'notion', 'explains', 'zoom-outs', 'jasmine', 'network', 'revs', 'prospecting', "'flood'", 'thirteen', "'toy", 'acore', 'marketing', 'business-owner', 'oscar--pitt', 'mechanized', 'freshen', 'merchandise', 'bucks', 'castor', 'highlights', 'pummeling', 'rare', 'vestern', 'correctional', "'dark", 'microchip', 'solicit', 'karaoke-based', 'family--father', 'preferably', 'kira', 'flap', 'kerr', "oz's", 'marsh', "raja's", 'depended', 'fashioned', 'pseudo-asian', 'probes', 'faithless', 'jewelry-sporting', '`chef', 'counterattack', 'frailties', 'cake-walk', 'ironside', "e'", 'twister-occurrences', 'lover', "kid's", 'belittle', 'rail-laying', 'diguise', 'species--a', 'ella', 'spars', 'prominent', 'tee', 'torry', 'abysmally', 'smelly', 'maladjusted', 'oomph', '$250', 'shriek', '`dumb', "assignment'", 'raw', "'green", "madman's", 'pent-up', 'puritanical', 'american', 'pankow', 'striving', 'caravan', 'popeye', 'inhabited', 'celine', 'multi-ethnic/multi-national', 'squaring', "kessler's", 'find', 'jailbird', 'lawford', 'coffers', 'rehabilitate', 'combining', 'gays', 'inauspiciously', 'product-of-choice', 'ex-stripper', 'assignments', 'asia', "boolie's", 'lollies', 'pierre', 'holds', 'mick', 'enveloped', "seconds'", 'lodger', 'caviezel', 'explore', 'suitcase', 'awesomely', 'approval', 'pretender', 'diarrhea', 'lash', "byrne's", "mailer's", 'souza', 'superb', 'conjure', 'explorer/mariner', 'explode', 'quoted', 'heston/gardner/bujold', 'dry-witted', 'monoliths', 'faces', 'destabilize', 'transformation--is', 'sfx', "dragon's", 'excite', 'auriol', 'flatter', 'moisture', 'woods', 'ende', "ivy's", 'boarder', 'martyrs', 'spartacus', 'k-red', 'narrow', 'gases', 'miraculous', 'stuntman', 'distances', 'greens', 'quay', "patti's", 'film-class', "swift's", 'hodgepodge', 'rawls', 'mayo', 'nachos', 'unconsidered', 'tumbles', 'clunes', 'subsequent', 'epiphany', 'miscasting', 'orders', 'out-performs', 'well-paid', 'degrading', 'boyd', 'recharge', 'unshrouded', 'visuals', 'holiness', 'this-obviously-cost-a-lot-so-you-know-', 'hitchhiked', 'showcasing', 'rupert', 'tumbling', 'pre-teen', 'climax--which', 'canal', 'trained', 'suffuse', 'hobbes', 'jaynes', 'mis', 'pores', 'lithgow', 'involve', 'uplifted', '1000', 'cocks', 'onwards', 'comraderie', 'mist-esque', 'charbanic', 'jaoui', 'contraception', 'ground', 'codas', 'cinephiles', 'clara', 'turn-on', 'duquette', 'ecclesiastes', 'wander', 'anonymous', 'been-there', 'scampering', 'downtown', 'pansy', 'cast--goran', 'ta', 'hecuba', 'singer_', 'choo', 'carrying', 'allan', "games'", 'henning', 'walk-off', 'scintillating', 'tgif', 'faggot', 'desperately', "mitchell's", 'embroidered', 'container', 'skipped', 'hyper-color', 'probgram', 'nirvana', 'climbs', 'cartoons', 'reenactments', 'reminder', 'gunton', "damon's", 'torches', 'asap', 'riddle', 'brandy', 'enslaved', "basinger's", '_anything_', 'half-second', 'flipper', 'springtime', 'alienates', 'swish-swish-zzzzzzz', 'bites', 'ridiculousness', 'telecommunicative', 'amyl', 're-unification', 'bender', 'exercising', 'tobacco', 'hand', 'garber', 'mcmahon', 'later-in-the-year', 'settled', "die's", "geyser's", 'truckload', 'upstart', 'cheapens', 'devon', 'taming', 'aol', 'fluent', 'session', 'twentieth', 'dried', 'coed', 'engulfed', 'radiated', 'treachery', 'bigglesworth', 'articulately', 'registered', 'stepfather', 'oleynik', 'entomologist', 'extracting', 'deviants', 'menno', 'patronize', 'proportionately', 'deteriorate', 'babes', 'smokescreen', 'amassed', 'ibsen-esque', 'defanged', "producer's", 'pride', '`dr', 'preschoolers', 'belligerent', 'strong-worded', 'funerals', 'jigsaw', 'darwin', 'boldly', 'database', 'nanny', 'wincott', 'charmer', "griswold's", 'stagnated', 'lessen', 'gina', 'dealings', 'piercing', 'retro-style', 'esquire', 'clintonesqe', 'hannibal', 'valletta', 'bistro', 'sides', 'stabs', 'rope', 'expecting', 'zamm', 'film-goers', 'laura', 'ideologies', "milano's", 'regimen', 'academy-award', 'donovan--it', 'sheltering', 'errands', 'pardon', 'self-indulgent', 'anti-gun', 'remarks', 'overeager', 'victimizer', 'free-spirit', 'whining', 'shares', 'negotiation', 'writer-performers', 'story-driven', "annie's", 'resembled', 'elite', 'director/editor', 'buckman', 'prefixed', 'grossie', 'makes', 'capes', 'preening', "mornay's", 'deborah', 'nuisance', 'grosser', 'gandolfini', 'dramatized', 'contemptuously', 'fiendish', '_dragonheart_', 'radius', 'orphans', 'mating', 'randomly', '--gluttony', 'shrills', "rabin's", 'stain', 'curly', 'no-nonsense', "minister's", 'pothead', 'striding', 'capability', 'yawk', 'rudy', 'slaying', 'one-way', 'bootsy', 'artistic', 'silhouette', 'memo', 'totoro', 'var', 'simulated', 'accountability', 'hefnerism', 'apparatus', "granny's", 'incumbent', 'hard-to-swallow', 'republic', 'waving', 'humor-wise', 'disinterested', 'unabomber', "niro's", 'hogget', 'triggering', 'fay', "'00s", 'accepts', 'bombed', 'bemusing', 'portray', '#15', 'subverts', 'md', 'antipathetic', "ethan's", "'war", 'ambush', "hands'", 'craig', 'lopez', 'regrettable', 'paean', 'temperatures', 'occasionaly', 'violating', 'hanky-panky', '`munchings', 'goldbergesque', 'drivel', 'englishman', "brando's", 'arija', 'pipeline', 'engineers', "`georgia'", 'challenge', 'still-burning', 'retrieve', 'niftiest', 'swipes', 'sitcom-style', 'microscope', 'lambert', "o'callaghan", 'inadvertent', '``the', 'tatooed', 'arabia', 'sonnenberg', 'codgers', 'cropped', 'iffy', 'outlaws', 'woodwinds', 'beetlejuice', 'whistling', 'desiree', 'pornographer', 'leathers', 'pan-and-scan', 'discerning', 'recaptured', 'timelines', 'keynote', 'goldeneye', 'warehouse', 'differences', '43', "zaire'", 'imposes', 'dystopian', 'sherlock', 'quips', 'welcomed', 'ignorantly', 'insulin', 'better-than-usual', "viewer's", 'genetically', 'attilla-looking', 'tnt', 'negotiates', 'different--black', 'arrgh', 'friendly', 'unexpressed', 'equivalent', "'feel", 'schmuck', 'alevey', 'classifying', 'lindsey', '_two_', 'pitchers', 'doped-up', 'self-exploration', 'mercies', 'scant', "pie'", 'brag', 'abandon', "jackson's", 'june', 'bending', 'well-received', 'stimulates', 'seeps', 'larisa', 'eden', 'orbach', 'artistically', 'babaloo', 'figaro', 'earns', 'restrictive', 'winslow', 'memorabilia', 'digit', 'deck', 'weinstein', 'vibrancy', 'doctored', 'cinematography/art', 'dianne', 'matrix_', "jar's", 'tightest', 'brace-sporting', 'beefy', 'vulnerability', 'readers', 'video-age', '-give', 'sniffing', 'andersen', 'modernizing', 'vying', 'attach', 'fridge', 'frequently', 'faster-than-gravity', 'icily', 'obscures', 'jaws', 'istvan', 'regressive', "dickinson's", 'harm', 'convict', 'mid-level', 'jee-zus', 'generational', 'tire', "months'", 'unjustifiable', 'augmented', 'nineteenth', 'kidney', 'perfs', 'seconds', 'thrills-a-minute', 'blouse', 'still-heavily-accented', 'ideologically', 'underrated', 'wheelchair', 'horks', "liberty's", 'uncredited--can', 'unjustly', '`basic', 'tab', 'tourist', 'kundun', "o'toole", 'jetpacks', 'janey', 'ravenous', 'post-salary', 'ur-reality', 'characteristically', 'seventeenth', 'unconvincingly', 'absurdity', 'se7en', 'sorceror', 'sing-along', 'flamingos', 'harrelson', 'wakeup', 'scheider', 'veiled', 'terrestrials', 'parking', 'fumbling', "wives'", 'enslave', 'disrespect', 'dah-dum', "hayek's", 'crib', 'reon', 'rescued', 'jail', 'zigged', 'expectation', 'promesse_', 'cash-milking', 'delved', 'poses', 'filtered', 'aaaahhhs', '_lot_', 'scouts', 'thoughout', 'aloofness', 'dippe', 'kamen', 'theorizing', 'summoning', "crazy'", 'doesn=92t', 'de-glamed', 'kaleidoscopic', 'punk', 'dorado', 'cauterize', 'han', 'thoughts', 'much-larger', 'chalk-full', 'imbues', 'grafitti', 'negin', 'louisa', "reynold's", 'carjackers', "damsel's", 'fellini', 'shakily', 'notifying', 'colosseum', 'letterboxing', "donager's", 'vocalists', 'judgmental', 'crystallized', 'is--gasp', 'grogan', 'packet', 'glares', 'discovery', "directorate's", 'firefighting', 'comprehensible', 'typing', 'profoundness', 'dysfunction', "'58", 'happy-go-lucky', 'occasionally', 'jewess', 'id4', 'contemptable', 'tart', 'outshining', 'letscher', 'lapdance', 'post-twin', 'baffels', 'lineup', "industry's", 'urban', 'skinhead', 'sweaty', 'romanov', 'normal-looking', 'potentially', 'foibles', 'disuse', 'acquisition', 'shoving', 'gulped', "hadden's", 'photosensitivity', 'prerelease', 'shaped', 'thing_', 'venue', 'add', 'watchability', 'hypodermic', '45%', 'edgefield', 'softly', 'price', 'women_', 'flint', 'piece', 'bleibtreu', 'peripheral', 'mind-boggling', 'render', 'effortless', 'rock-concert', 'rib-tickling', 'enterataining', 'long-abandoned', 'illustration', 'dog', 'mnc', 'stopper', 'chaste', "parillaud's", 'noteworthy', 'immersed', 'womantic', 'divisions', 'forehead-slapper', 'hideaway', 'part', 'sexual', 'steely-eyed', 'chasms', 'warm', 'gloucester', 'andre', 'thick-headed', 'pelts', "linda's", 'charecter', 'bel-air', 'plundering', 'tunnels', 'reassuring', 'scandalous', 'hag', 'gabby', 'undying', 'downfall', 'wins', 'cutlery-flinging', 'small-time', 'minute', 'poorly-integrated', 'tummy', 'methodically', 'roper', 'dump', 'fluid', 'shouting', 'literati', 'schoolmates', 'antoine', "huang's", 'pinter', 'posed', "duty'", 'denigrates', 'crusoe', 'testosterone-overdosed', 'emannuelle', 'overpowering', 'partial', 'empathizes', 'corniest', 'cornelius', 'giddiness', 'prosecutes', 'treasure', 'perpetrated', 'tanner', 'street', 'credibility', 'styrofoam', 'initiative', 'oven', 'ray', 'breakdancing', 'swat', 'manipulating', 'sniffs', 'frighten', 'quotient', 'renovation', 'backtrack', 'horrified', 'newsgroups', "robocop's", 'opt', 'well-structured', 'c3-po', 'joblo', 'apologies', 'speakeasy', 'renoir', 'vacation', 'flirtatiously', 'life_', 'accursed', 'read-along', "facility's", 'transitory', 'joint', 'tribes', 'support', 'oeuvre', 'made-for-video', 'encapsulated', 'smile', 'bolstered', 'inappropriate', 'best-executed', 'active', 'seasick', 'flattering', "constanza's", 'mothering', 'seale', 'armored', 'leapfrog', 'kato', 'sled', 'technicalities', 'deposited', 'structure', 'wordlessly', 'exorbitance', 'treaty', 'quests', 'gun-running', 'ignited', 'oxymoron', 'obsessive', 'diverting', 'down--to', "explorers'", 'rohypnol', "be-ribboned'", "party's", 'methodology', 'rip-roaring', 'resemble', 'artist-speak', 'smirnov', 'valentin', 'drunkard', 'buff-o', 'baddass', 'convicts', 'shallow', 'wales', 'preached', "daring's", 'skeletons', 'hesse', 'muggers', 'brink', 'this-could-be-you', 'long-', 'tibet', '--tough', 'family-oriented', '1957', 'cheung', "coufax's", 'nottice', "valentine's", 'ilm', 'scriptwriting', 'free-lance', 'clumsily', 'livewire', 'avant', 'homicidal', 'rock-musician-turned-pavement-artist', 'we-lei', 'astounding', 'placement', 'unanimously', 'unmoved', 'franzoni', 'gertrude', 'adapting', '`my', 'witch', '`ready', 'joyride', 'slurpy', 'tip-top', 'heir', 'provoking', 'acting/music/effects', 'mcqueen', 'over-use', "cat's", 'coerce', 'pullman', 'disaster-slash-action', 'impressed', 'bonehead', 'italian-like', 'wd-40', 'waspy', 'still-alive', 'yearning', "city's", 'bekim', 'eriq', 'rawhide/stand', 'everyday', "fishes's", 'wedding', '_very_small_', 'ducastel', 'half-heartedly', 'sadder', 'respectability', 'intertwines', 'underdevelopment', 'formality', 'stupefying', 'newlyweds', 'out-of-chronological-order', 'roughness', 'hypnotises', 'bookseller', 'inaudacious', 'torment', 'chequered', 'belies', 'puts', 'scrumptious', 'military/cia', 'ma', 'coordinated', 'grit', 'sheedy', "'humour'", 'emblematic', "camel's", 'heartening', 'greenpeace-friendly', 'audio', 'head', 'blossoming', 'dull', 'demoted', 'art', 'blithe', 'blandly', 'jost', "graynamore's", 'ferland', 'marginal', 'omnipresence', 'closing', 'argument', 'job-title', "'zoot'", 'calculation', 'pizazz', 'menstruation', 'flower', '62', 'sweethearts', 'arcane', 'alphabet', "hayes's", 'characters--', 'sharif', 'backwater', 'miata', 'crass', 'thrill-ride', 'panache', 'lives--for', 'disease', 'agatha', 'luncheon', 'allusion', 'mentally', 'down-at-heels', 'pual', "shouldn't-for", 'contend', 'womb', 'morphing', 'mold', 're-edit', 'packs', 'outpaced', 'tampon', 'moment--', 'howling', 'wounds', 'huddleston', 'uppedity', 'darndest', 'distractingly', 'offscreen', 'churns', 'rotary', 'connie', 'sterile', 'note-', 'rambunctious', 'arrangements', 'kattan', 'ramirez', "1993's", "bastard's", 'correlate', 'liberties', 'fact--but', '===========', 'broodwarrior', 'eldard', 'satirizes', 'mandingo', 'formal', 'well-conceived', 'popular', 'design', "script's", 'egyptians', "governer's", 'dreamers', 'disagreement', 'swing-dancing', "hodges'", 'trickier', "`hot'", 'victorious', 'passe', 'adapt', 'bachelor', 'climatic', 'mein', 'whipping', 'wasn+t', 'lend', '007', 'hinted', 'seven-year-old', 'kitty-faced', 'nakedly', 'waning', 'boyfriends', "emile's", 'rabies', 'nine-episode', 'yup', 're-enactment', 'vindication', 'petri', "grusin's", "thornton's", 'lifetime', 'rapid', 'taks', 'retirement', 'unwraps', "killer's", 'jao', 'physics', 'furniture', 'english', "deer-in-the-headlights'", 'identifies', 'quasi-evil', 'carries', 'prologue', 'grand-daughter', 'befouled', 'pignon', 'barclay', 'stepping', "kinski's", 'pip', "bateman's", 'pyrotechnic', 'excitiable', 'spanish', 'unwatchable', 'sandefur', 'darts', 'massage', '`wild', 'narc', 'niceness', 'quartet', "mulqueen's", 'changing', 'theft', 'borealis', 'hostile', 'vegetable', 'avail', 'carroll', 'baseman', "robertson's", 'circle', 'dammit', "insider's", 'denker', '_saved_by_the_bell_', 'moat-filled', 'antics', 'ike', 'off-shoot', 'schweppervescent', 'notions', "holland's", 'borderlines', "narrative's", 'high-gadget', 'ebert', 'translator', 'joe/josephine', 'adding', 'overcloud', 'taylor-gordon', 'nutshell', 'unbrewed', 'rewatchable', '--------------------------------------------------------------', 'analyze', "ryan'", 'instructs', 'jeesh', 'bonnefroy', 'getty', 'divided', 'sheriff', 'second-', 'pox', 'jets', 'humiliating', 'gunshots', 'hopeful', 'hazard', 'westworld', 'stargaze', 'reopens', 'instantly', 'beers', 'nutmeg', 'vail', 'losers', 'skittery', 'distance', 'mechanics', "'smoothly'", 'birkenau', "presence's", "stargher's", 'compound', 'crooks-posed-as-photographers', 'spew', 'categories', 'jordan', 'liking', 'drillers', 'inquisition', 'grandson', 'insubordination', 'lunchmeat', 'path', 'kaaya', 'electrically', "70's", '`never', 'relayed', 'well-written', 'on-court', "matchmaker's", 'nntphub', 'collectively', 'silk', 'leichtling', 'nails', 'nostalgia', 'freewheeling', '76', 'harkens', 'rebounding', 'blueblood', 'tirade', 'possessors', 'confounding', 'convulses', 'represented', 'pounding', 'still-present', 'flashbacks', 'stuyvesant', 'starsystem', 'commences', 'disagreed', 'comte', 'what-not', 'mini-plot', 'clown-for-hire', 'tan', 'layla', 'absolved', 'witch-doctor', 'slogan', 'dissembles', 'boon', 'leaks', 'recycles', 'occasionally-humorous', 'neglecting', 'romanceless', 'knucklehead', 'joyce', "ilona's", 'absences', 'arrange', 'crumb', 'finfer', 'solmenly', 'dauntingly', 'human-factor', 'ronny', 'skarsgard', 'back-to-back', "dumas's", 'misadventures', 'characterized', 'headdresses', 'there=92s', 'immersive', "`ronna'", 'comediennes', 'summarized', 'schoolwork', 'preclude', 'leers', 'steriotypically', 'parts', 'presumed', 'forwarned', 'indecisiveness', 'meet', 'best-', 'toothpick', 'produce', 'tracing', 'tightens', 'samantha', 'sympathise', 'rapier', 'undisclosed', 'epectations', 'substantive', 'revisits', 'preaches', 'bloated', 'reminders', 'streetwise', 're-edited', 'rookie', '3465', "treill's", 'offing', 'monday', 'dierdre', 'industry', 'disapproving', 'abel', "wilde's", 'daydreams', 'brightening', 'sweepstakes', 'sons', 'raiders', "helena's", 'midway', "portillo's", 'grossly', 'receptionists', "mcgregor's", 'partaking', 'migrated', 'c-', 'serena', 'science-fiction/action', 'targeting', 'vocal', 'convince', 'cusack', 'irishman', 'experience', 'meteor', 'reconst', 'mu-mu', 'spiked-haired', "johnny's", "'gi", 'entitled', 'chute', 'packers', 'penny', 'hair', 'eponymous', 'catering', 'doorstep', 'bs', 'janney', 'lindberg', 'emma', 'koch', 'regulations', 'society', 'crazy/beautiful', 'remakes', 'spats', 'best/worst', 'sewage', 'reiterated', 'internet', 'amor', 'bumper-sticker', 'relies', "norris'", 'kalvert', 'pollock', 'charade', 'sadists', "trunchbull's", 'boomers', 'obligingly', 'minimalist', 'over-earnestness', 'belongs', 'terrified', 'rueland', 'dailies', "cleese's", 'horrifyingly', "'contact'", 'nuttiness', 'sour', "kudrow's", 'hotel', 'humor--as', 'femme', 'stillness', "june's", 'ezekiel', "bizet's", 'masturbation', '15-year', 'dour', 'transylvanians', 'spastic', 'kama', 'stocks', '`plot', 'reigning', 'mascara', 'time-titanic', "seagal's", 'necesary', 'corrupt', 'wearisome', 'immeasurably', 'breakaway', 'garziano', 'carrefour', "ru'afro", "price's", 'unenergetically', 'seed', 'deceased', 'prime', 'hex', "institute's", 'scrooged', 'renee', 'stalk', 'leppenraub', "'artistic'", "walker's", 'law-abiding', 'exceedingly-professional', 'pornographers', 'enacts', 'classified', '_matewan_', 'dalmatian', 'large-breasted', 'beating', 'cast--all', 'tolerance', 'leather-wearing', 'crutch-carrying', 'century-fox', "totty'", 'errand', 'manslaughterer', 'solace', 'previews', 'tutelage', 'interplay', 'things', 'dilbert', 'pentecostal', 'selleck', 'streamline', 'simmering', "reese's", 'ache', 'bawdy', 'establishments', 'graveyard', 'disney-made', '$23', 'palisades', 'horseback', 'advise', 'impressively', 'graying', 'hypocritical', 'plausability', 'sensitivity', "jasmine's", 'factors', 'backpedaling', 'foundering', 'slogans', 'ex-friend', 'mawkish', 'chats', 'assistant', 'decision', 'conti', 'oral-sex/prostitution', 'gargoyles--perhaps', 'tax', 'unsigned', 'de-force', 'devolved', 'impertinent', 'nigel', 'puccini', 'february', 'yelps', 'problematically', 'examined', 'shaving', 'baz', 'man', 'binges', 't-name', 'labute', 'aspects', 'creek', 'barbarians', 'narrowing', 'spoiling', 'unsatisfying', 'brick', "viewers'", 'functionary', 'coaxing', 'mars/venus', 'issue--there', 'toller', 'groundwork', "individual's", 'humanistic', 'discernable', 'faints', 'cybill', 'invulnerability', 'road-trip', 'priorities', 'brisk', 'humour', 'advice', 'raged', 'hipness', 'ruse', 'rediscovered', 'impartial', 'zilch', 'unfreeze', 'fastest', 'thorougly', 'chagrin', 'dissipate', 'workaholics', 'distinguished', 'ladybug', 'overturned', 'sv2', 'honey', 'anti-pill-popping-casual-tantric-sex-car-chase-attempted-murder', 'wager', 'agee', 'knot-tieing', 'dreaming', "astronaut's", "merrill's", 'coerced', 'blast', 'hounds', 'inhabitants', 'unlucky', "julie's", 'fletcher--channel', 'charted', 'extremely', 'lulu', 'cher', 'pronounce', 'jinn', 'zags', 'appreciated', "you's", 'supposes', 'shipped', 'sneering', 'cruder', 'gnome', 'frighteners', 'commandeering', 'mount', 'past', 'understanding', 'birdcage', 'candidate-director', 'berkeley', 'scrutinization', 'rejuvenating', 'harbors', 'volleyball', 'money-making', 'carly', 'scares', '`we', 'tizard', 'outlining', 'hyperkinetic', 'whiner', 'listing', 'declined', 'shel', 'identity', 'developments', 'newer-model', 'rolling', 'amadeus', 'seals', 'mcgann', 'bbc', 'blend', 'interloper', 'cliche-fest', 'truthful', 'propensity', 'blurt', 'wal-mart', 'lapaine', 'unsatisfied', 'quasi-sadistic', 'champ', 'darling', 'karl', 'spine-tingling', 'tournament', 'gleam', 'staked', 'aki', 'h2k', 'platoons', 'furious', "sarah's", 'antionio', 'left', 'cop-turned-private', 'herz', 'forze', 'palance', 'bot', 'ranchers', 'actionless', 'her--in', 'limitations', 'mockingly', 'radiates', 'spell', 'leather-face', "valek's", 'riggs', 'painstaking', 'writer-director-producer', 'faster-than', 'facilitated', 'scraping', 'foolish', 'fish-and-game', 'squarely', 'debatable', "leon's", 'objected', 'ronna', 'persson', 'sparse', 'always-grisly', "rourke's", 'impulsively', 'borrows', "must've", 'rameses', 'come-on', '#05425', 'telemarketer', 'udall', "vault's", '87', 'vineyard', 'vernier', 'german', 'alludes', 'typewriters', "audrey's", 'graziano', 'muff', 'client', 'snots', 'president', 'floated', 'channing', 'type-cast', 'bigot', 'dazzlingly', 'additive', 'hermosa', 'campaign', 'experimental', "'fright", "'smocky'", 'utlizes', 'sly', 'blunderheaded', 'number', 'winkless', 'source', 'banks', 'vincennes', 'whimsically', 'worsened', 'red-carpet', 'sour-old-matriarch-from-hell', 'tightrope', 'admits', 'emile', 'lighting', 'allman', 'wring', 'junior', "majesty's", 'flashier', "'bad'", 'affiliations', 'ever-antsy', 'mishap', 'onward', 'zeroing', 'yvonne', 'homosexual', 'customary', 'manor', 'toff', "wrong--i'm", 'transcends', 'northward', 'william', 'booths', 'affected', 'body-themed', 'psychomagically', 'sexuality', "hurt's", 'ballooning', 'tear-jerkers', 'disability', 'megastar', "'almost'", "crew's", 'solemnly', 'roomm', 'redistribution', 'nightclub', "'twas", 'illinois', 'drector', 'alway', 'succulent', 'weight', "should've", "blockbuster'", 'cronies', 'dangle', 'wreaked', 'innovative-never', "lee's", 'grownup', 'ex-fan', 'model-wife', 'd-day', 'potemkin', 'pre-1960s', 'xtdl', 'corporation-funded', 'o-dog', "international's", 'resist', 'blooming', 'sassy', 'circuitry', 'hogan', 'howled', 'totopoulos', 'regrettably-neglected', 'exuberantly', 'encapsulates', 'out--', 'crime', 't&a', 'wraps', 'sequestered', 'accommodations', "satan's", '_home', 'xvi', 'near-by', 'biochemist', 'hispanic', 'slow-witted', 'pejorative', 'wrong-doing', 'earpiece', 'wryly', 'pseudo-depth', 'divorce', 'gruelling', 'annie/she+s', 'nouvelle', '.', 'contrivance', 'winding', 'packer', 'approaching', 'recycled', 'fairer', 'photographed', 'fizzle', 'howdy', 'mcguigan', 'half-man', 'environment--', 'frailty', 'sigourney', 'humane', 'knee-jerk', 'serialised', 'employ', 'fleischer', 'koontz', 'substituted', 'knights', 'grant-type', 'transferring', 'remorseful', 'crunchable', "gwen's", 'haddad', "lillard's", 'much-welcome', 'double-zero', 'ac/dc', 'baker', '5=average', 'times', 'dance', 'theresa', 'fonzie', '89-minutes', 'kurtzman', 'bell', 'planing', 'meathooks', 'nuggets', 'flames', 'vah-vah-voom', 'seized', 'new-self', 'stiff-as-a-board', 'candleshoe', 'recovering', 'forbes', 'king-type', 'so-named', 'unfolding', 'entrepenaur', 'accepting', 'shue', 'martial-arts-star', 'caligula', "proyas's", 'cleaver', 'steppes', 'drip', 'fleas', 'scamper', "pescucci's", 'scowls', 'space', 'heroic', "patient's", 'medley', 'mosaic', 'swearing', 'racquel', 'subplot', 'bugs', 'screenwriterese', 'unsuitable', 'places', 'seafaring', 'korben', 'horn-rimmed', 'toothpaste', 'tori', 'female', 'enhancement', 'sarandon', "zach's", 'trembling', 'near-death', 'ford', 'relling', "mchale's", 'well-modulated', 'corrected', 'pieh', 'virginie', 'oddness', 'maj', 'jillian', 'randi', 'futures', 'hosted', 'cocoon', 'sitcom-ish', 'self-respecting', 'fingered', 'helps', 'condominas', 'peyote', "henry's", 'negligent', 'islandnet', "knott's", 'cheap-looking', 'con-girl', 'disproving', 'self-preserving', 'bootstraps', 'reckless', 'commoner', 'gilliam-esque', '54', 'pullam', "'pod'", 'wardrobe', 'reach', 'danilov', 'simmons', 'unquestioned', 'present', "winfield's", 'reeling', 'switchblade', 'cored', 'stories', 'fill-in-the-blank-american-comedy', "o'connell", 'chinlund', 'co-executive', 'walk', 'trudge', "o'herlihy", 'hopelessness', 'flamm', 'tian', 'imprisonment', 'gordon-levitt', 'pocket', 'skulls', 'body', 'insults', 'testifying', 'oneida', 'down-on-his-luck', 'tomes', 'faithfulness', 'reinvention', 'globe', 'mcnugent', 'jerked', 'tourists', 'competed', 'developer', 'hillbilly', 'quantify', 'adele', "charlie's", 'assassinated', 'double', 'defenses', 'alexandre', 'chandelier', 'really-they', 'fernwell', 'elton', 'scribe', 'gargantuan', 'merrick', 'hitmen', 'inky-blue', 'edvard', 'arise', 'tucker', 'behoove', 'lombardo', 'promising', 'levinson/dustin', 'protracted', 'tarkofsky', 'liberally', "lou's", 'sovereign', 'stern', "schnitzler's", 'conn', 'confesses', 'rosebudd', 'berg', 'blip', 'lesbos', 'position', 'lose', 'laws', 'harrison', 'carrere', 'dunno', 'sear', 'swashing', 'character--', 'jillion', 'and--again--you', 'off-target', 'marlowe', 'metropolitan', 'carelesness', 'e1', 'mincemeat', 'crows', 'stiff', "bankole'", 'photo', 'slumped', 'forceful', 'increases', 'well-liked', 'sexton', 'blow-off', "alien's", 'voluminous', 'ida', 'reynaldo', 'running-like-the-wind-prey', 'enrolls', 'single-handedly', 'thieves', 'drumlin', "negin's", 'nyah', 'blumberg', 'microwaves', 'trillian', 'foul-humored', 'power-hungry', "instinct'", 'colours', "o'brian", 'split-level', 'out-doing', 'day', 'officers', 'ripper', 'bonneville', 'nearly-brain', 'rard', 'ambassador', 'riot', 'paradox', 'expected', 'dodge', 'itinerant', "friggin'", 'jams', 'alfre', 'episodic', 'chicken', 'irrelevant', 'godfathers', 'responsibilities', "ihmoetep's", 'dora', "cartoonish-barlow's", 'initech', 'progresses', 'skim', 'thorton', 'pacifistic', 'clocktower', 'goofier', 'detectives', 'plunge', 'lurch', "spacek's", 'reinforced', 'moments--prologues', 'cross-section', 'gradually', 'butt', 'swoons', 'mega', 'tortoise-like', "filmmakers'", 'geller', 'shining', 'bandages', 'cohesive', '$20', 'half-open', "'lingerie", 'proves', 'confuse', 'cop-buddy', 'inhibition', '19th-century', 'hrundi', 'dilemnas', 'wails', 'gediman', 'emitted', 'lorenzo', 'arguably', 'play', 'oakland', "father's", 'soft-boiled', 'muppet', 'absorbed', 'bumpy', 'fantastical', 'ashton', 'histories', 'bathroom', 'self-image', 'anne-marie', 'nudge', 'baldwin', 'unarmed', 'bowlers', 'scarsely', 'tuva', 'evokes', '_very_', 'well-explored', 'joyless', 'twilight', 'digging', 'distress', 'lakeside', 'zombie', 'investigate', 'locating', "edwards'", 'pretension', 'gunslingers', 'splendor', 'craft', 'looses', 'augment', "`believers'", 'mined', "airplane's", 'grown', 'summer-time', 'reworkings', 'grenade', 'tube', 'avenues', 'madonna-marrying', 'beckwith', 'rosary', 'concoct', 'neo-slasher', 'country-wide', 'privacy', '31-year-old', 'pre-release', 'incorporating', 'promotes', 'omit', 'minimums', 'rash', 'evasion', "leary's", 'cautious', 'looooooong', 'distraction', 'beleivable', 'carts', "dumber'", 'overheated', 'options', 'decapitating', 'modesty', 'commanding', 'phobia', 'juror', 'anew', 'opera-type', 'dissipated', 'proved', 'glacier', 'stygian', 'policy', 'spraypaints', 'boastful', "maugham's", 'artifice', 'rehashes', 'suburbs', 'sociological', 'stiller', 'crossed', 'interstitial', 'funniest', 'adjust', 'homely', 'hovers', 'graham', 'alliances', 'moriarty', 'vibe', 'louie', 'turgid', 'celebrities', 'acquires', '_exactly_', 'revisitings', 'terrorists', 'matter-of-', 'strives', 'skittish', 'acknowledging', 'multi-dimensions', 'turn', 'offending', 'altercation', 'triangle', 'poem', 'boning', 'orginal', 'liv', '_jumanji_', 'ghoulishness', 'locales', 'nauseatingly', 'cremet', 'hoodwink', 'faithful', 'famke', 'holler', "'matron'", 'devotees', 'momentum', "granger's", 'untamed', 'natassja', 'kotter', "breckman's", 'mot', "larson's", 'throwing', 'cgi', 'stairwells', 'cusackian', 'predated', "rubin's", 'neglected', 'goldmine', 'scattered', 'sibling-like', '_home_alone_', "rustler's", "bergman's", 'permeates', 'ninety-five', 'eschewed', 'insulatory', 'snippets', 'cheesy', 'chamber', 'vegas', 'lapses', 'commerce', 'attempts', 'uploading', 'foppish', '-school', 'interior', 'non-presence', 'pregnant', 'pantyhose', 'preschool', 'corporations', 'consummated', 'savory', 'iron-fisted', 'strive', 'fun-in-the-sun', 'yom', 'merest', 'planets', "'interview", 'davis/harrison', 'reporter', 'retreated', 'melancholy', 'quirkiness', 'sob', 'voiceovers', 'ad2am', 'swing', 'varying', 'aris', 'wigs', 'recap', "nations'", 'rages', 'kel', 'blood', 'genuine', 'forgivable', 'demises', 'pseudo-climax', 'mild', 'bi-sexual', 'lounging', 'inmate', '$1', 'highlighting', 'moguls', 'big-haired', 'maxim', 'snipped', 'critic-mode', 'loaf', 'crispness', 'defeatist', 'inhabits', 'believe--in', 'diasappointing', 'continuos', 'maka', 'dweller', 'sway', 'trapped', "hutton's", 'threeway', 'bout', 'eyes', 'shitheads', 'disguising', 'avowed', 'wwf', "horror's", 'partygoer', 'chopping', 'passageways', 'uncle', 'come-uppance', 'pauly', 'controversies', 'gander', '37', 'kisses', 'dictators', 'gertz', 'lesley', 'naughtiness', 'fantsy', 'charisma', 'overplaying', 'recouperating', 'spooky', "gardenia's", 'arm-in-arm', 'chasing', 'out-maneuvers', 'diffusing', 'schmaltz', 'cliche', 'syrupy', 'artist/action', "danes'", 'mini-plots', 'bewilder', 'singing', 'stunned', 'prospects', 'grasshopper', 'pukes', 'viscera', 'endearingly', 'non-existant', 'macnee', 'whoaaaaaa', "laura's", 'with--gasp--a', 'seventh', 'choudhury', 'sock', 'edge-of-your-seat', 'announcer', 'baffled', 'quintessentially', 'tripping', 'gleisner', 'tailored', 'precise', 'dave-good', 'fantastic', 'hugh', 'wendt', 'cinque', 'willson', '1969', 'geniuses', 'alluring', 'timothy', 'renews', 'ritchie', 'rainy', 'over-long', 'gloom', 'completed', 'duality', 'film--twice', "mathilde's", 'maclachlan', '132', 'peggy', 'assuaged', 'chereau', 'dab', 'half-witted', 'true-to-life', 'patsy', 'transition', 'bedrooms', 'bath', 'bowed', 'nostalgics', 'medium', 'kinetic', 'golan', 'miramax', 'backpacker', 'chewbacca', 'always-on-the-move', 'landry', 'payoff', 'reportedly', 'adrenaline', "brandi's", "countries'", 'beret', 'rosenbaum', 'agility', 'chase--but', 'forrest', 'baboon', 'tailor', 'shouted', 'payload', 'mystics', 'unsophisticated', 'execution', 'setting-intensified', 'writhing', 'sitcom-bred', 'roles--', 'suspiciously', 'scandal', 'lovesick', "mcgann's", 'spaces', 'beowolf', '90%', 'ratcheted', 'hollywood-ized', 'handled', 'anouk', 'record-breaking', 'precedes', 'meier', 'blankets', 'malleability', 'spared', 'tradiational', 'hiself', '[r]', 'reruns', 'irritatingly', 'snatch', 'bevery', 'share', 'ga-ga', 'jury', 'exacly', 'evangelical', 'ideals', 'sense', "rabbit'", 'critically', 'kershner', 'redcoat', '1950', 'sore', 'bauer', 'kitchen', "mom's", 'marginalized', 'capitalize', 'vinyard', 'obssess', 'continent', 'time-conserving', 'capping', 'detect', 'sock-em', 'chastising', 'occult', 'swore', 'mandrake', 'docter', 'wang', 'itching', 'greed', 'mile', 're-generating', 'sez', 'colombia', 'dien', 'sarah', "abby's", 'temptress', 'discord', 'jekyll', 'tombstone', 'masseur', 'murata', 'rock-opera-turned-major', 'agamemnon', 'ranft', 'genetic', 're-interpretation', 'stipulated', 'linus', 'footsteps', 'michaels', 'arse', 'wrinkles', 'papa', 'poison-', 'recognizing', 'funicello', 'jimi', 'lughead', 'female-fearing', 'motss', 'avidly', 'industrial', 'schellenberg', 'tentative', 'sammy', "castle's", 's&m', 'sprinklings', '20somethings', 'overruns', 'player-turned-owner', "mob's", '_snl_', 'nosed', 'midler', 'let-down', 'loans', 'computer-animation', 'tours-de-force', 'panting', 'strong', 'imitation', 'obi-wan', 'presaged', 'smary', 'extinguishing', 'i-do-my-job-whether-they-are-innocent-or-guilty', 'scripture', 'red-herrings', 'rubble-strewn', 'raptors', 'excruciating', 'munson', 'remix', 'ric', 'unattended', 'sab', 'turgidson', 'purchasing', 'reform', '105', 'super-agents', 'indecency', '101-year-old', 'futurama', 'taj', 'xing', 'flushing', "rambaldi's", 'prints', 'gladnick', 'melt', 'demonstrating', 'croc', 'builds', 'downplays', 'pounced', '107-year-old', 'slip-ups', 'philosopical', 'partnerships', '11-year-old', 'isabelle', 'hoenicker', 'schemers', 'burrows', "turturro's", 'straight-to-the-cutout-bin', 'boy-likes-girl', 'subservient', 'rebirth', 'tranquility', 'negative', 'oops', 'switches', 'mum', 'freeway', 'witnessed', 'evict', 'freshness', "babysitter's", 'refuting', 'gudin', 'emmy', 'forbade', "sugiyama's", 'eeriness', '1-on-1', 'georgetown', 'koans', 'prehensile', 'turnabout', 'pie', 'scene-and', 'throat', 'seeker', 'nic', 'flake', 'arching', 'impalement', 'incest', 'jejune', 'ytv', 'ambiguous', 'scriptiing', 'deficient', 'regroup', 'annual', 'mouth', 'engine', 'jolly', 'fischer', 'eery', 'negated', 'undertaking', 'flexing', 'schizophrenic', 'mystical--perhaps', '*', "jung's", 'promoted', 'back-stabber', 'flock', 'paraded', 'carver', 'yoda-esque', 'brushing', 'expressionism', 'guild', 'slash', "wharton's", '-looking', 'ants', 'speculator', 'rivers', 'droid', 'lifted', 'fave', 'hysterically', 'jai', 'chunky', 'insiders', 'rudely', 'terrorizes', 'shae', 'maximal', 'foiled', 'figuratively', '8=excellent', 'musician', 'wrestle', 'girl-type', 'prominant', 'well-acted', 'prowess', 'marie', 'spoons', 'face/off', 'upper-class', 'computer-creation', 'hit-', 'five-year', 'filmic', 'sadistically', 'faddish', "angela's", 'dinsmoor', 'implacable', 'knee', 'boulder', 'conglomerates--have', 'calf', 'heightened', "`dawson's", 'music', 'arrakis', 'spunky', 'camel', 'auto-pilot', 'smart', 'despicable', 'scotch', 'liken', 'crawling', "shepard's", 'volatile', 'ex-agent', 'superstition', 'proclivity', 'sword-play', 'drank', 'pakistan', 'forecast', 'rented', 'pie-making', 'adversarial', 'defenders', 'now-ralph', 'majorly', 'arcs', 'tasty', 'castaldi', 'exaggeration', 'perceptible', 'late', 'closest', 'erect', 'overtime', 'flashing', 'ultra-orthodox', 'viewers', 'literal', 'shoulders', "penn's", 'ravine', 'amateurish-foregrounds', 'mute', 'rocks', 'naifeh', 'hero-', 'paragons', 'genres', 'succeeded', 'stowed', 'memoir', 'unattainable', 'fortunes', 'fedoras', 'intricately', 'bribing', "bilko's", 'multi-dimensional', 'hayley', 'joe/chris', 'tribulations', 'located', 'asian-american', 'mistake-free', 'reteamed', 'far-off', 'bombastically', 'plent', 'delve', 'fantasized', 'homework', "korman's", 'screech', '-type', 'psyches', 'stimulation', 'outdoorsman', 're-runs', 'pollak', 'well-rounded', 'warbling', 'lookalikes', 'yau', 'viruses', 'vicars', 'excalibur', 'superficial', 'limits', 'altar', 'priscilla', 'donations', 'pseudo-hip', 'pigeon-holed', 'vilified', 'epic-sized', 'spinal', 'sidekicks', 'spiriting', 'snapping', '17', 'congratulate', 'crescendo', 'whatsoever', 'cycles', 'undertones', 'jobeth', 'drawers', 'helgenberger', 'doorway', "bacon's", 'altruistic', 'truck', 'ballad', 'guterman', 'embarassed', 'laugher', "`kundun'", 'slack-jawed', "frat's", 'vinny', 'joking', "zeik's", 'accentuated', 'tricky', "arraki's", 'so-so', 'madsen', 'amusingly', "schmidt's", 'pertwee', 'randall', "toy's", "things'", 'schulz', 'neighborhoods', 'messier', 'refund', 'plunked', 'chablis', 'better--among', 'pinocchio', 'hiver', 'hustles', 'stick-handles', 'thusly', 'thicker', 'designing', 'ramon', 'average-joe', 'wailing', 'telecommunications', 'co-starred', 'salem', "elizabeth's", 'washington--one', 'filmming', 'knew', "hilary's", "'no", 'tragedy', "'breaker'", 'aplomb', 'shapeless', 'gladstone', 'snake-gaiter', "amistad's", 'patchwork', 'waste', 'shady', '100%', 'schow', 'percentage', 'pleasures', 'endo', 'stephen', 'concurs', "tomorrow'", 'jodie', 'characterizes', 'xtc', 'indigestible', "'realistic'", 'overflows', 'clea', 'honor', 'oh-so-adorable', 'intellectualizing', 'scalvaging', 'under-developed', 'betrays', 'bunker', 'oise', 'governmental', 'boggs', 'born', 'real-estate', 'shove', 'burkhart', 'soldering', 'self-centered', 'marginally', 'unfurls', 'mib', 'arrrrrrrghhhh', 'pleading', 'courtyards', 'war--that', 'gorillas', 'underestimate', "pitillo's", 'moaning', 'fourteenth', 'captive', 'punt', 'surrenders', 'holdups', 'imo', 'movements', 'living', 'injections', 'lionize', 'prized', 'boosting', 'hudlin', "xiii's", 'improvised', 'peddle', 'auel', 'aaa', 'discourse', 'nieces', 'unchecked', 'sentences', 'low-quality', 'workman-like', '`varsity', 'refilmed', 'com/~corona/films/details/sw4', 'bolton', 'glassy', 'cio', 'answers', 'marybeth', 'fibers', 'danish', 'thrugh', 'portentous', "stan's", 'exclude', 'bristling', "ratt's", "skinheads'", 'kinship', 'winged', 'succession', 'puns', 'pregnancy', 'affliction', 'years-gone-by', 'blunt', 'scriptwriters', 'faux-sympathy', 'stubborness', 'annabeth', 'two-man', 'phrasing', 'couple', 'volcanologist', 'weddell', 'most-famous', 'rang', 'manhunter', 'grand-daddy', 'mgm', 'taxing', 'slurry', "priestley's", 'four-star', 'three-pronged', 'wordly', 'loews', 'non-religious', 'locks', 'laff-o-meter', 'soon-to-be-ruler', "stayin'", 'snags', 'malfunctioning', 'dependent', '7', 'stays', 'males', 'awe-inspiring', "'til", 'tension-building', 'ishtar', 'capitulate', '`nightmare', 'siam', 'slither', 'mid-film', "sonny's", 'trite', 'illustrator', 'proposals', 'medal', 'stomach-churning', 'elias', 'existence', 'kissy-faces', 'nightly', "grisham's", 'unlawful', 'scarlett', 'filmed', 'omnipresent', 'weebo', 'bundled', "britt's", 'vendela', 'dragon-like', 'anyhoo', "''", 'high-stakes', 'elastic', 'surfaced', 'ebouaney', 'bad-guys', 'failing', 'towne', 'grudgingly', 'tape-to-film', "malkovich's", 'stomped', 'albert', 'not-so-wise', 'casanova--what', 'intruded', 'imprisoned', 'novella', 'strides', 'overlong', 'disney', 'hallie', 'avoiding', 'leary-sandra', 'beaman', 'precipices', 'retina', 'tingwell', "hour's", 'mendes', 'resent', 'cinderella', 'virginia', 'curly-haired', 'overkill', "head'", 'mime', 'dubuque', 'sublimated', 'hurry', 'son', 'neptune', 'white-faced', 'good-work', 'terminators', 'bethlehem', 'weaves', 'dietrich', 'pepto-bismol', 'unwatchably', 'swiped', 'smith', 'good-bye', 'filmgoer', 'freelance', 'mini-masterpieces', 'julia', 'snell', '-clever', 'magic-realism', 'animality', 'scholarly', 'incorrect', "earth's", 'cosby', 'tinted', 'cite', 'humanoids', 'dmv', 'acquitted', 'rambo', 'revamped', 'broker', 'sycophant', 'shuffle', 'bashed', 'suspensor', 'ashamed', 'verbatim', 'henreid', 'jobless', 'bodacious', 'explicit', 'quinn', 'spain', 'activist', 'embarrassingly', 'machina', 'atmostpheric', 'hanania', 'nfl', 'ill-composed', 'erotically', 'thingie', 'simplest', 'rivka', 'time-defying', "segonzac's", 'compadre', 'russell', 'disengaging', 'randle', 'viking/bowling', 'vodka', 'wicca', 'signed', 'hid', 'closed-down', 'foggiest', 'ex-champion', 'carp', 'balinski', 'revere', 'estelle', 'vomiting', 'veidt', 'facade', 'byplay', 'duffle', 'chaja', 'enola', 'peng-jung', 'yeoh', 'measles', 'crybaby', 'passion', 'materializes', 'wide-eyed', 'heard', "snowball's", 'ambitious', "nomi's", 'kernels', 'shone', 'shower', 'counterexample', 'frann', "brilliance--baker's", 'deliriously', 'scratching', 'advertise', 'grodin', 'demon', 'dolores', 'skirting', 'contest', 'shwarzenegger-like', 'includes', 'buckets', 'chain-weed-smoking', 'interrogates', 'wanted', "jolie's", 'could/but', 'niro/james', 'commentator', "mission'", "deliver'", 'picturesque', 'col-', 'inconclusive', 'forecasts', 'his/her/their', 'ape]', 'exemplify', '160', 'bathed', "'titanic'", 'pleasingly', 'cristoffer', 'mano', 'filters', 'bay/jerry', 'scientist', 'phelps', 'sling', 'ratchet', "d'abo", 'modus', 'gyllenhall', 'ahern', 'sin', 'ferdinand', 'guinness', 'toadie', "cameron's", 'eliminated', 'overwhelmingly', 'roots', 'prospective', 'loathe', 'palmer', 'bunyan', 'angelica', 'show-stopper', 'vicar', 'muck', 'posts', "c-for-charlie's", "'see", 'mock-ups', 'shoveler', 'thereof', 'frantically', 'community', 'impostors', 'hinges', 'bride-to-be', 'flat', 'sex/masturbation', 'tax-auditor', 'nazis', 'countryside', 'wachowski', '49', 'stogie', 'ecl@mtcts1', "lloyd's", 'backwoods', 'screenplay---or', 'karaman', 'energies', 'idiosyncracy', 'loops', 'devastating', 'lustful', 'craving', 'dante', 'slimes', 'kirk', 'brie', 'gees', 'lathered', "funeral'", 'pathetic', 'serra', 'monterey', 'bubble', 'marla', 'wiseguys', "garret's", 'chan', 'culturally', 'bulbs', "rasslin'", 'grisham', 'torturing', 'soars', 'gmt', 'cavity', 'mean-spiritedness', 'megalomania', 'daria', 'token', "letter's", 'wracked', 'marisa', 'underground', 'consensual', 'jackson/chris', 'kilner', "spielberg's", 'ethel', 'guessing', 'disguises', 'technicolor', 'improv', "lithgow's", 'superstar', "naboo's", 'messiah', 'puked', 'indentity', 'uncanny', 'muhammad', "south's", 'mentally-disturbed', 'ebeneezer', 'retelling', 'smokes', 'genuinely', 'necessities', "stillman's", 'assholes', 'enlightening', "died'", 'caretaker', 'hobbs', 'necrophilia', 'dumber--and', 'theatens', 'broadside', "carlito's", 'graffiti', 'nerve-wracking', 'stores', 'winslett', 'ramblings', 'power-trip', 'matiko', 'brutal', 'stop-animation', "canine's", 'bobo', 'pitfalls', 'nadir', 'tribesmen', 'robo', 'stereotypically', 'binge-drinking', 'affection', 'cinematographr', 'trapeze', 're-cut', 'circuit', 'dehumanization', "breillat's", 'cozart', "fox's", 'evicted', 'prodigal', 'transposed', 'arguement', 'structurally', 'consisted', 'whobilation', 'hippies', 'hardcover', 'artificial', 'profits', 'eyeballs', "`daylight'", 'insanity', "adams'", 'produced', 'informative', 'coach', 'overlooked', 'noxzeema', 'dolls', 'coltrane', 'faith-healing', 'rectitude', 'tight-knit', 'anthropological', 'playes', 'hemingway', 'crown', 'street-wise', 'houses', 'seafood', "instinct's", 'queue', 'tiptoeing', "shot's", 'tow', 'misteps', 'dalton', 'deal', '1939', 'caricaturist', 'overhears', 'cleans', "movie-he's", 'soothingly', 'helicopters', '--made', '-set', "febre's", 'zephyr', 'atreus', 'homoerotic', 'accommodate', 'materialist', 'trounced', 'morneau', 'disbelieved', 'scams', 'deceive', 'admiring', 'flown', 'chronological', 'puzo', 'grays', 'duong', 'haughtiness', 'hundred', 'semi', 'sex-maniac', 'noir-', '6', 'ambitions', "change-i'll", "shelly's", 'discouragingly', 'evans', 'inflection', 'engrossing', 'spellbound', 'confides', '`go', 'migration', 'photographs', 'dakota', 'exhausted', 'unwillingly', 'mallrats', 'nicest', 'wizard', 'nerves', 'cop/new', 'geekish', 'leap', 'knuckle', "helgenberger's", 'one-named', 'exorcise', 'assumes', 'idea', "cliff's", 'magma', 'sympathetically', 'drug/rave', '[spoiler', '45-year-old', 'hush', 'lands', 'initial', 'slate', 'answer', 'movie-within-a-movie', 'hoisting', 'unhinged', 'constructs', 'group', 'fascism', "ifans'", 'duct', 'finicial', 'fractions', 'surrounds', 'fornicators', 'fire-ball', 'thesps', 'novicorp', 'roundly', "days'", 'vandercave', 'monica', "'never", 'quanq', 'poker', 'glides', 'half-black', 'collaborate', 'bobby', 'unofficial', 'self-reliance', 'titanic-buff', 'ingenius', 'reconciling', 'copulating', 'trudging', 'vagrants', 'breathed', "italy's", 'derived', 'hennings', 'duties', 'deprivation', 'dellaplane', 'else--i', 'trails', 'urgently', 'jean-paul', 'southerners', 'preparing', 'clayton', 'kuo', 'needles', '`', 'orangutan', 'churchman', 'moniters', 'drang', 'perpetual', 'geraldo', 'underplayed', 'nukes', "'helen'", 'wire-', 'insight', 'seclusion', 'historians', 'gooding', 'upheld', "organization's", 'repetitive', 'governor', 'stump', 'sales', 'multi-talented', 'demon-fighting', 'millar', 'chuck', 'merit', "biziou's", 'odette', 'wolfe', 'solve', 'resourceful', 'addicts', 'nahon', 'getup', "he'", 'cody', 'ever-affable', 'cleavon', 'maxi', 'irving', 'bakalian', 'cabby', 'woos', 'brandon-like', 'downloaded', 'collage', 'horseshoes', 'tarantino/robert', 'crazies', 'videodrome', 'reiser', 'conquer', 'x-mas', "gets'", 'leachman', 'pills', 'nichols', 'breakfast', 'highly-energetic', 'wit', 'kindhearted', 'richly', 'retrospect', 'belaboured', 'ppreciate', 'hoffman', 'well-incorporated', "daniels'", 'yerzy', 'ingest', 'kline', 'transcripts', 'penultimate', '30-minute', 'acclaimed', 'case', 'toenail', 'henchman', "guccione's", 'soon-to-be-classic', 'curry', 'cartoonishly', 'improbable', 'bran', 'importantly', 'reccemond', 'self-destructive', 'obscurity', 'omits', "beaumarchais's", 'roar', 'rant', 'gadgets', 'nookey', 'affleck-damon', 'two-headed', "judah's", 'life-or-death', 'high-roller', 'gleaned', ']', 'defined', 'half-sister', "kitano's", 'ingrained', "gwenie's", 'messias', 'violence/gore', "daniel's", 'sack', 'insightful', 'afi', 'macnamara', 'fans', 'perennially', 'sincere', 'adjective', 'caveman', 'crying', 'insane', 'none-too-successful', "marc's", 'nostalgically', '20', 'restroom', 'affords', 'eqsuisite', 'adventurer', 'subordination', 'nocturnal', 'carriages', 'protestor-marching', 'noah', 'uninsured', 'vhf', 'sito', 'tiegs', 'maniacial', 'bussed', 'grows', 'differentiates', 'exploration', 'shears', 'ridiculing', 'uproarious', 'self-consciously', 'dividing', "`sophisticated'", 'dir-ect-or', 'prepare', 'dross', 'heinous', 'weak-hearted', 'weary', "dick's", 'daffy', 'ashbrook', 'assume--everything', 'shadyac', 'unspools', 'cases', 'antz', 'desolation', '`hang', 'long-lived', 'pronouncing', 'palms', 'tricking', 'scorned', 'tommorow', 'liberate', 'passengers', '$1000', "dga's", 'displays', 'expulsion', 'shape-shift', 'bigotry', 'editorial', 'deductions', 'sell-out', "paul's", 'pittance', 'involved', 'stash', 'feathered', 'forthcoming', '`d', 'hot-blooded', "sense's", 'burg', 'undaunted', 'spectre', 'globe-trotting', 'noticable', "springer's", 'dudley', 'plumbing-supply', 'onboard', 'off-camera', '$16', 'nut-biting', 'imported', '92-minute', "barbara's", 'maiden', 'everything--it', 'jingo-all-the-way', 'procedure', 'rigorous', 'loser', 'shade', ')', 'oddities', 'super-impaling', 'contentious', 'journeyman', 'maroon', 'less-than-savoury', 'one-word', '50s/early', 'forger', 'serafine', 'bruised', 'hitches', 'pseudo-', 'true/three', 'slipping', 'curtly', 'dapper', 'alain', 'staking', 'opportunistic', 'immeadiately', 'loyalist', 'people', 'opportunity', 'crossbreed', 'downes', 'ordering', 'tornatore', 'semi-experienced', 'product-placement', 'erupt', 'arms-merchant-cum-safari-host-', 'guinee', 'sol', 'applicants', 'bitches', 'harbour', 'marsha', 'hartley', 'crawls', 'broadcast', 'incidently', 'parting', 'pub', 'camerawork', 'arnez', 'reload', 'tramell', 'two-tone', 'multi-tiered', 'cottage', '28', 'preceeding', 'shtick', 'discredit', 'carrie-ann', 'suggs', 'grouped', 'guarded', 'tournaments', 'flim', 'galoshes', "'94", 'abuses', 'spoofs', 'decreased', 'refuge', 'insure', "frankie'", 'beacham', 'brisville', "furst's", 'softens', '^', '`special\x12', 'addicted', 'anne', 'decimated', 'assembled', 'bowie', 'overgrown', 'drug-op', 'springs', 'dog-napping', 'adjectives', "rules'", 'majority', 'nuttiest', 'forty-nine', 'polack', 'nite', 'not-too-friendly', 'mouthing', 'magnitude', 'bottlecaps', 'hangover', "macnee's", 'cronfronting', 'skewers', 'thinker', 'over-eager', "red's", 'noticeably', "field'", 'pimpernel', 'sidequel', 'mechanism', 'nuzzling', 'tannen', 'slipup', 'dostoyevski', 'cheddar', "curly's", 'wholly', 'stunt', 'lacrosse', "club's", "williams'", 'addresses', 'heritage', 'recognition', 'impresario', 'aerosmith', 'mph', 'neo-feminist', 'melrose', 'pornography--ie', 'reassembled', 'seemingly-strange', "aniston's", 'mortizen', 'thwarts', 'directive', 'copland', 'mindfuck', 'woo-like', 'suspense-filled', 'self-sacrificing', 'hsiao-hsien', 'primrose', 'surprisingly-resilient', 'background', 'godawa', 'director', 'clicks', '-yeager', 'elfman', 'ahab', 'idosyncrasies', 'successful', 'unforgettable', 'dashiell', 'limb-twisting', 'some--not', 'badass', 'ents', 'sells', 'kid', '1-800-call-att', 'nurture', 'large-assed', 'breakups', 'puppet', 'connor', 'spetters', "lehmann's", 'ottis', 'vanishes', 'near-blind', 'affraid', 'resteraunt', 'taylor-thomas', 'kross', 'bumping', 'piper', 'maltin', "gitai's", 'large', "`fatty'", 'partly', 'beek', 'flubber', 'self-produced', 'shagging', 'runner-esque', 'heck', 'mckay', 'monger', 'apex', 'compliment', 'cutaway', "finnegan's", 'bedmates', 'grumpier', 'story', 'strindberg', 'impolite', 'put-on', "'er", 'mistake', "sarone's", 'exqueeze', 'hess', 'helgeland', 'comfort', 'chewed', 'showpiece', 'cram', 'dainty', 'unforgiveably', 'whirls', 'allegra', 'comedies-', 'read', 'slime', 'la', 'showmanship', 'humiliate', 'duels', 'burley', 'dragonlike', 'wannabe-zany', 'escalates', '1948', 'duets', 'mining', 'verne', 'jenna', 'spilled', "adjani's", 'socially/culturally/politically/globally', 'scribblings', 'tykes', 'giggled', 'crawl', '22-minute', 'morpheus', 'd-reper', 'joint-smoking', 'codename', 'kissner', 'headless', 'blew', 'fundamentals', 'possibly', '-reviewed', '$20s', 'fulfills', 'degeneres', 'respectful', 'quickly-paced', "fulci's", 'landlords', 'knockoff', 'coalwood', 'pockets', '802', 'beasley', 'defusal', 'crannies', 'physicals', 'unentertaining', 'crash]', 'fumblings', '25', 'agendas', 'drugs', 'whacks', 'attraction', 'casanova', '$70', 'clarify', 'violinist', 'yuji', 'gentlemen', 'highest-grossing', 'bordello', 'bikini-ready', 'sinker', 'good-spirited', 'motionlessly', 'honeys', 'minds', 'examples', 'backing', 'malcontents', 'philosophical', 'dealer', 'rampages', 'turner', 'resists', 'candyman', 'japanese-specific', 'coloured', 'roman-chariot', 'accelerator', 'avoids', 'tangerine', "princess'", '105-minute', 'bronze', 'toucha', 'sabotage', 'weirder', 'hairball', 'columbine', 'goldsmith', 'uninvolved', '1988', 'flesh', 'manipulates', 'difference', 'unconscionable', 'robinson-blackmore', 'reeking', 'poon', 'scowling', 'serve', 'epitomizes', 'book/film', 'deakins', 'langella', 'screw-up', 'assassins', 'murawski', 'mudslides', 'intentioned', "cloud's", 'threats', 'unfit', 'semi-saving', 'maternity', 'nicely-titled', 'caves', 'flemish', 'scorn-ful', 'warrants', 'composite', 'heat', 'opportunists', 'morva', 'margaret', 'devarim/yom', 'disappear', "leila's", 'warship', 'huggable', 'federico', 'djimon', 'woodman', 'pinpoint', 'saucy', 'profane-filled', 'cynicism', 'aug', 'tomboyish', 'danika', 'heart-strings', 'ris', 'kiberlain', 'oneders', 'se7en-ish', 'bolster', 'learning', 'encouragement', 'up-beat', 'helmer', 'misusing', 'acupuncture', 'fossils', 'helplessly', "bambi's", 'cauterizes', 'presume', 'sewer', 'freudian', 'tested', 'alien-rip-off', 'kasi', 'fastened', "lansbury's--she", 'zeist', 'rural', 'hip-hop-talking', "'secret", 'hagen', 'observant', 'lass', 'twitching', 'cube', 'joe', 'ping-pong', "madsen's", 'nauseated', 'thrillseekers', 'ephrons', "mcgowan's", 'late-mother', 'operator', 'aid', 'ageing', 'non-fans', 'disco-themed', 'long-overdue', 'cessation', 'underplaying', "yoda's", '130', 'sought', 'anticlimactic', 'darkest', 'chum', 'sweetly', 'philosophers', 'variants', 'shoo-in', 'appetizer', 'koufax', 'dragon', 'blandness', 'pertain', 'campell', 'henn', 'stringfield', 'carjacking', 'neck-', 'spoof', 'hannigan', 'afri-can-american', 'sandoval', 'kill', 'baggage', 'nursery', 'crewmen', 'teacher', 'coagulate', 'personas', "cube's", 'vines', 'stonily', 'brow', 'lattitude', 'mcalisters', 'non-union', 'horror--less', 'minor-league', 'non-trekkies', 'dirtier', 'coattails', "films'", 'mover-and-shaker', 'yen', '`opium', 'unitentionally', 'pitt-esque', 'stewart-esque', 'shakesperean', 'herbal', 'spiders', 'transportation', 'wreaking', 'lifting', 'wont', 'animators', 'sensationalistic', 'haircut', 'competing', 'rituals', 'bleaches', 'uncredited', 'relay', 'movie_', 'mind-bender', 'presidential', 'payday', 'noirs', 'bridges', 'pollini', 'lynskey', 'diversity', 'britisher', 'backfire', 'pincus', 'hive', 'washed-out', 'mocking', 'tandon', 'plot-driving', 'cheek--and', "megan's", 'overly-religious', 'tim', 'reelviews', 'gene', 'welfare', 'imports', 'pastorelli', 'puritan', 'dismember', "jake's", 'actress', 'recruitment', 'nixes', 'charities', 'looker', 'transexual', 'landis', 'faster', 'insolent', "kerdelhue's", 'rappers', 'passion-denying', 'overshadows', 'bremner', 'emptiness', 'scotsman', 'saucers', 'unblinking', 'misguided', 'mimic', 'subcontinent', 'longtime', 'spoiler', "darryl's", 'tiger', '112', 'ians', 'alien', 'portals', 'subtitle-phobes', 'giggle', 'longest-seeming', 'fancy-schmancy', 'interrotron', 'streets', '-moderator]', 'crumbling', 'imagination', 'heart-wrenchingly', 'rival', 'charter', 'downsizing', 'docile', 'rift', 'vibrant', 'manner', 'muddled', "bugs'", 'pantry', "'shouting", 'systems--it', 'undertake', 'self-healing', 'turrets', 'slugs', 'schwarz', 'multitude', 'hapless', 'madwoman', 'informing', 'intermingling', 'heart-pounding', 'clint', 'guy--a', 'illegitimate', 'lobotomise', "vader's", 'englishmen', 'atf', 'sands', 'exiled', 'defending', 'altmanesque', 'yolande', 'superglues', 'contenders', 'moronism', 'hoenig', 'jackhammer', 'sing-song', 'longstanding', 'lapd', 'incompetently', "'problem'", 'spark', 'liked-', "diesel's", 'followers', 'lung', 'fetal', 'contradiction', 'un-related', 'gore', 'describe', 'spooketeria', 'blake', 'terrors', 'schema', 'purposefully', 'mink', 'kaminski', 'passenger', 'pallbearer', 'whitman', '$40', 'clicked', 'once-promising', 'official', 'gruffudd', 'wahoo', 'claiming', 'darkened', 'young-lovers-on-the-road', 'facing', 'stu', 'pascal', "batter's", "lab's", 'playthings', 'upstate', 'wackiness', 'verdant', 'exiting', 'stone-ish', 'low', "'production'", 'teenagers--though', 'inclusion', 'grocers', 'newest', '--travel', 'skirmish', 'impressing', 'franco', 'sioux', 'carded', 'loan-shark', 'strengths', 'education', '_fifty_', 'fetishes', 'fi/comedy', 'ufos', 'man/bag', 'storyteller', 'illogic', 'labors--including', "virus'", 'catching', 'consolidate', 'uninspired', 'oooh', 'cinemtography', 'radiance', 'paralysed', 'offender', 'glory--starring', 'discoverer', 'consisting', 'downgrade', 'factories', 'chopper', 'tormenting', 'lovell', 'stuntwoman', "'t", 'darkman', 'minute-long', 'touchy', 'hassles', 'witt', 'haute', '710', 'oceans', 'come--bringing', 'merton', 'pre-credit', 'staunton', 'motaro', 'formula-wise', 'lackey', "titus'", 'keri', 'sixteenth', 'fanboy', 'captain', "hannigan's", 'nob', 'terrible--mr', 'dethroning', 'josiane', 'over-the-top', 'samuel', 'alfonso', 'romancing', 'paradigms', "'it's", "bont's", 'thinnest', 'set-bound', "australian's", 'post-break-up', 'neuromancer', 'pointy-eared', 'angry', "tyler's", 'week-old', 'najimy', 'e-mails', 'levitch', 'bloods', 'france', 'briefly', 'invasion', 'once-blossoming', 'crewmembers', 'worrying', 'evil-blackmailing', 'honkey', 'mireniamu', '_halloween', "superman's", 'sepia', 'solo-flight', 'lady', 'cradles', 'survival', 'caligari', 'hot-to-trot', 'aalyah', "night's", 'uniformly', 'criticism', 'perpetually-changing', 'illustrious', "squad'", 'soran', 'vii', 'reflecting', 'endangered', 'coldest', 'simplistic', "dellaplane's", "y'know", 'hipster', 'r-rated', 'a-bomb', 'walkers', 'freely', 'authoritarians', "louise's", 'mlakovich', 'perkiness', "simon's", 'eternal', 'post-marriage', 'wether', 'updated', "ollie's", 'gleiberman', 'laleh', 'hiding', 'aughra', 'unshaven', 'pantoliano', 'neo', 'christof', 'hurley', 'non-roles', 'disobedience', '152', 'onset', 'closets', 'bloodthirsty', 'dami', 'startrek', 'four-day', 'griswolds', "ii's", 'quantity', 'shelter/satanist', 'installing', 'anti-environmentalist', 'serendipity', "t'was", 'orwell', "62's", 'rubbell', 'blocked', "policeman's", 'visitors--friends', 'kube', 'ob-cpo', 'adair', "'funny'", 'semi-circle', 'wary', 'function', 'unrequisite', 'thickly', 'reality-warping', 'goldblum', 'accountable', 'bookshelves', 'clinically', 'pepper', 'rings', 'cotton', 'whitewater', 'ladyhawke', 'rebaptizes', 'technician', 'outraged', 'largest', 'sincerely', 'owners', 'slogs', 'preoccupied', 'roomate', 'mnemonic', 'crash-testing', 'half-stated', 'champagne', 'magnate', 'claiborne', 'tusse', "dad's", 'bust', 'self-fulfillment', 'moor', 'glistening', 'harder', 'babycakes', 'cures', 'rms', 'nitty-gritty', 'denouncement', 'basque', 'unmasked', 'brimming', 'teaching', 'summed', 'is/are', 'go-ahead', 'suzie', "`disappointing'", 'warchus', 'carter', 'bingo', 'predatory', 'convenient', 'prevent', "shackleton's", 'over-populated', "packin'", 'profession', 'ridge', 'hunting', 'riffs', 'slandering', 'rabbinical', 'biceps', 'thrive', 'minutes--', 'eggs', 'weights', 'clemens', 'marriages', 'enraptured', "sutherland's", 'obsessively', '10%', 'teen-targeted', 'sluggish', 'sobs', 'dafoe', 'handsomest', 'sprays', 'six-string', 'natured', 'bombastic', 'picking', 'tanning', "psychlos'", 'screenplay', 'reb', 'quashing', 'egpyt', 'kid-in', 'whew', 'clocked', 'protruding', "classmate's", 'cadence', "d'angelo", 'spaceships', 'reevaluation', 'unbridled', 'singled', 'desperado', 'bangs', 'schell', "darby's", 'twain', 'potente', 'staunch', 'mercenaries', "howie's", 'castration', 'bardsley', 'slayer', "frisch's", 'emery', 'patrons', 'basic--and', '_today_', "heart'", 'magua', 'spoils', "pie's", 'skylines', 'herb', 'flickers', 'relief--to', 'euro-trash', 'verite', 'much_', 'slickness', 'actor/director', 'slick', 'predestined', 'drek', 'expressionistically', 'barked', 'gothic/electronic', 'keeken', 'rosalind', 'un-watchable', 'fleeting', 'funnest', 'grieg', 'yankovich', 'awwww', 'mutants', 'uncompelling', "apple's", 'munchkin', "coppola's", 'brassy', '17th', "how's", 'pesky', 'mommas', 'juggling', 'good-luck', 'bala', 'cartlidge', 'bird', 'a+', 'daphne', 'karen', 'defend', "china's", 'airhead', 'express', 'ham-fisted', 'unmentioned', 'vowing', 'preperations', 'deciding', 'book-skimming', 'mispronouncing', 'gunfighter', 'discretion', 'deadlier', 'numbed', 'borrowers', 'otherness', 'operating', 'redhead', "1600's", "coppolas'", 'low-angle', '1692', 'investigation', 'weighing', "impossible'", 'strongly--but', 'stunk', 'sppedboat', 'all-day', 'silhouettes', 'relaxed', 'screamer', 'this--i', 'waldo', 'washes', 'separated', 'vomity', 'more-than-welcome', 'macabre', 'celebrity', 'drug-dealer', "jessica's", "christine's", 'purgatory', 'gross', '_hard_ware', "chan's", 'kin', 'weirdly', 'sandwiched', 'franciscan', 'avocation', 'package', 'lou', 'shortened', 'fawcett-style', 'fronting', 'dramedy', 'ascaride', 'outdated', 'behavious', 'self-deprecation', 'fully-developed', 'theory', 'superstitions', '3-5', 'settle', 'planted', 'macdowell', 'whoah', 'reintroduced', 'spacey', 'swords', 'outings', 'snuff', 'secluded', "hewitt's", 'crassly-calculated', 'consumers', 'stansfield', 'ricardo', 'shoulder', 'hammond', 'adore', 'ever-attentive', 'drifters', 'rating=', 'recaptures', 'discards', 'fights--especially', '90', 'airlock', 'titanics', "ribisi's", 'cutesy', 'street-like', 'piggy', 'penning', 'free-spirited', 'staggeringly', 'na-ture', 'overtaking', 'chestnut', 'intersect', 'fiorentino', 'pervious', 'tch', 'granting', "cover's", 'shane', 'laziness', 'solaris', 'h}', 'lept', 'marni', '1964', 'photocopying', 'b-movies', 'plummet', 'lawyerly', 'mandrian', 'vigau', 'cheezy', 'wayyyy', 'keywords', 'mcgovern', 'sunshine', "l'auto", 'govworks', '1700s', 'unflagging', 'tailspin', 'weapons', 'descendent', 'fraction', 'korshunov', 'shoot-em-up', 'spacesuit', 'bibile', "kane's", 'calvinistic', 'holocaust-ravaged', 'gameport', 'quasi-', 'murdoch', 'adventurous', 'florida', 'toyed', 'availability', 'intervention', 'undecided', 'measureable', 'jerod', 'molestation', 'cleavage-busting', 'judgement', 'machine', 'productive', 'oblivious', 'unerotic', 'dolefully', 'diatribe', 'batter', 'crony', 'schubert', 'celeste', 'rapper', 'bedroom', "ships'", 'crow', 'eqypt', 'lu', 'asthmatic', 'swinging', 'claws', 'flimsy', 'double-cross', 'reporters', 'soliloquies', 'masochism', 'nikos', "college's", 'entir', 'plotholia', 'minivan', 'prices', 'agrees', 'charnel', "duguay's", 'ape-like', 'cash--so', 'madigan', 'backers', 'strait-laced', 'called', 'lauren/sylvia', 'punish', 'felons', 'rattigan', 'women-only', 'hammerstein', 'maureen', 'backup', 'keys', 'cynically', 'waterfall', 'sinful', 'develop', 'hoist', 'in-place', 'cynic', "hubby's", 'adaptable', 'similarly', 'foreheads', 'niro', 'doll-sized', 'porn-star', 'good-hearted', 'bailed', 'issues-namely', 'ceases', 'reliving', 'feds', 'housewives--that', 'cheesiness', 'ditz', 'vh1/vogue', 'hugeness', 'odgen', 'bill', 'approaches', 'impressionist', 'mannequins', 'counter-productive', 'mergers', 'intial', 'pedophile', 'correspondence', 'incorporate', 'adored', 'predicament', 'rongguang', 'penney', 'double-lines', "louisa's", 'maguire', '_their_', 'villa', 'shalt', 'psychopath', 'nexus', 'scion', 'beah', 'posess', 'protocol', 'upending', 'assistance', 'cooperation', 'eiko', "snipes'", 'gil', 'flu', '1888', 'kirbys', "whut're", 'albums', 'naivety', 'mastrantonio', 'edson', "em'", 'isuro', 'exuberhant', 'suspicious', 'beavis', 'motivations', 'masks', 'quickening', 'xenophon', 'frakes', 'excuse-someone', 'outfitted', 'gps', 'attracts', 'hamburger', 're-team', 'presidency', "mcpherson's", 'stilted', 'sub-theme', 'bujold', 'macgowan', "debney's", 'navaz', 'well-paced', "'characters'", 'eroticize', 'hunky', 'burkettsville', 'seku', 'johanssen', 'meditation', 'sculpted', 'koenig', 'intent', 'assures', 'schooling', 'cherot', 'corvette', 'palpable', 'squeaky', "joshua's", 'fonda', 'alma', 'exclaiming', 'chechik', 'aspiring', 'post-production', 'problem', 'misconception', 'segue', 'reminisce', 'shy', 'aformentioned', 'corridor', 'appetizing', 'lender', 'non-nudity', 'temptations', 'perils', 'oppritunites', 'bloodlines', 'manic-depressive', 'valentina', 'mirroring', 'urinals', 'basking', "park's", "rasputin's", 'falseness', 'assembly', 'kickstarting', 'assasins', 'emissaries', 'miserables', 'havoc', '29-', 'published', 'stifles', 'videographer', 'repressed', 'assaulted', 'it--i', 'nicely', 'sounding', 'decides', 'minister', 'problematic', 'bellows', 'maximillian', 'residence', 'painless', 'nodded', 'dulled', 'and/or', 'movie--jackie', 'mescaline', 'white-bread', 'haters', 'allergic', "ba'ku", 'pseudo-saintliness', 'romps', 'x-philes', 'thoughtlessly', "monster's", 'digested', 'discover', 'processes', 'foreshadows', "hassler's", 'vows', 'stumbles', "`sabo'", 'wet', 'fehr', 'trail', 'tastes', "labor's", 'snowbell', 'underside', 'picket-fenced', 'cage', 'rhea', 'linney', 'vessel', 'abysmal', 'blackwolf', 'launching', 'break-into-the-building', 'checkers', 'snubbed', 'uncover', 'piglia', 'shapechanging', 'malcom', 'hood', 'physicist', 'over-done', 'eroding', 'rumba', 'high-technology', "spicer's", 'flights', 'excepting', 'sensitively', 'eat', 'extra', '_pick_chucky_up_', 'poking', 'arm', 'straight-to-video', 'ten-foot', 'farrah', 'ferocity', 'dalle', 'stanton', '#2', 'jen', 'artiness', 'mogul', 'pseudonym', 'descends', 'baywatch', 'resplendent', 'jack', '`hopelessly', "homage'", 'spray', '1979', 'chad', 'tensions', 'escher', 'nookie', 'venner', 'crosby', 'singles', 'seminary', 'tpm', 'hermits', 'card-flipping', 'sculpture', 'grrrl', 'fantine', 'chauvenist', 'sausages', 'fitzpatrick', 'quintano', 'blockade', "haunting's", 'suzanne', 'gore-drenched', 'compensated', 'sizemore', 'quandary', 'litter', 'city-bred', 'great-', 'ministry', 'yonezo', 'confusion', 'balk', 'graph', 'unconnected', 'milos', 'genders', "atlanta's", 'beaudoin', "daylight'", 'deputy', "ermey's", 'relishing', 'dread', 'hep', 'celebrated', 'tourniquet', 'spatial', 'regurgitates', 'buffoonish', 'jima', 'dilate', 'well-advised', 'ciro', 'dumbest', "co's", 'labeling', 'end--in', 'shabby', 'roadside', "childs'", 'manila', 'marvelling', 'cyclic', 'rigged', 'dorian', 'hoke', 'wwi', "river's", 'canoes', "film's", 'moods', 'thicket', 'corpse-rotting', 'romy', 'low-key', 'jilted', "blawp's", '8-year-old', 'rombi', 'perish', "puzo's", 'crashes', 'miserably', 'ex-', 'thrived', 'adult', 'dogma', 'thick-headedness', 'surpassing', 'informant', 'don', 'solider', 'cineplexes', 'oppressive', 'cover-ups', 'gun-and-plane', 'treats', 'reinvents', 'retched', 'slowed', 'fraker', "neuwirth's", "eflman's", 'spiteful', 'blacksmith', 'goodnight_', 'lame-brained', 'downsides', 'wayside', 'relationships', 'bierko', 'fails', 'pacha', 'malcovich', 'tones', 'legwork', 'collects', 'elizabeth', 'seeping', 'chew', 'serials', 'dupr', 'sit', 'making', 'sheep-pig', 'out-of-balance', 'lamanna', 'maron', "resurrection's", 'silent', 'embittered', 'pathetic-looking', 'tinto', 'interpret', 'hugging', 'pastiche', 'jean-hugues', 'rewound', "queen's", 'calming', 'tracked', 'chimpanzee', 're-takes', 'emerich', 'microsoft', '81', 'companies/families', 'schedules', 'joy-provider', 'sheep', 'stuntwork', 'jamaica', 'volunteer', 'espoused', 'playful', 'stomper', 'dotted', 'prehistory', 'prefecture', 'control-freak', 'enrols', 'west/cpw', 'naughton', 'canna', 'bandmate', 'mitsubishi', 'salsa', 'courtesan', 'wide-spread', 'phosphorus', 'under-age', 'americanization', "d'onofrio", 'waterlogged', "tourette's", 'mononoke', 'eventual', 'refers', "race's", 'first-person', 'packing', 'docu-style', 'screenshots', "cholodenko's", 'bimbo', 'layouts', "cooper's", 'messily', 'comprises', 'steele', 'sneaky', 'communities', 'coins', 'detested', 'squints', 'beekeeping', 'hungus', 'contact--refers', 'furred', "ullmann's", "verne's", 'long-suffering', 'julio', 'nuseric', 'expo', 'dimesional', 'rebuked', 'penchant', 'disrepair', 'pathos', 'cinemascope', 'album', 'mamas', "arm's", 'loathed', "gaby's", 'supermarkets', 'appropriately-named', 'alligator', 'novelists', 'banzai', 'bounce', 'versatility', 'majoring', 'slaves/steerage', 'proudly', 'f-ing', 'gutierrez', 'weakens', "zinnemman's", '167', 'disclosure', "naville's", 'stocked', 'milhoan', 'stoppard', "reagan's", "catherine's", 'leia', 'executed', 'dickinson', 'western/sci-fi', 'wasteland', 'posting', 'road', 'outlast', 'outbursts', 'headfirst', 'gio', "valjean's", 'ensuing', 'buckaroo', 'cellars', 'ladden', 'theroux', 'tough-as-nails', 'fucked', 'er', 'evan', 'folktale', 'pipelines', 'beat-a-dead-horse-into-glue', '\\', 'nurse', 'political', 'defences', 'glorious', 'redefines', 'harrowingly', 'armande', 'sculptor/russian', 'ticks', 'scrupulous', 'pastels', 'laroque', 'enlistment', 'elevator', 'pals', 'crook', 'stressful', 'reunion', 'swarmed', 'ghouslish', "beltrami's", 'nightlife', 'meddle', 'million', 'spielbergs', 'sun-', 'jamey', 'fairs', 'gals', "nora's", 'giant', 'poolman', 'palma', 'retread', "setting's", "vonnegut's", 'academy', 'barnes', 'incomparable', 'rightfully', 'barely', 'glimcher', 'aniston-as-love-interest', 'dodgy', 'taymor', "movie'", 'food-fate', '83', 'generates', 'cornell', 'steelworks', 'gangbusters', "kilmer's", 'synonym', 'reservations', 'burger', "viper's", 'disagrees', 'yaz', 'prig', 'cares', 'deepest', 'interactions', '230', "dawn't", 'insulators', 'dohlen', '_double_team_', 'brunette', 'targetted', 'girl/aspiring', 'poster', 'somber', 'forsythe', 'bookstores', 'co-producing', 'line', 'livelier', 'geared-down', 'deceitful', 'overlook', 'southerner', 'breaching', '$25', "juliet'", "braddock's", 'fright-free', 'paralells', '1923', 'stitch', 'hints', 'uneffective', 'believabilty', 'vanessa', 'nit-picking', 'fantasise', 'avid', 'impotence', 'rubrics', "dictator's", 'imagery/cel', 'treating', 'mechanisms', 'hendrix', 'well-dressed', 'bateman', 'luminous', 'muth', 'democrats', "pacino's", 'prying', "davidson's", 'relinquish', 'freedman', 'embarks', 'nearly-inconsequential', 'basset', 'disapprobation', 'havisham', 'self-proclaimed', 'illustrations', 'bevy', 'telltale', 'pilleggi', 'goode', 'earnestness', 'basic-', 'tear-jerker', 'porsche', 'scripter', 'imploded', 'whorehouse', 'receipts', 'stricken', "magazine's", 'morgan', 'snyder', 'glamor', 'termed', 'terse', 'audible', 'beheaded', 'egyptian', 'admirer--', 'folles', 'vicellous', 'headstrong', 'xavier', '`clerks', 'peaceniks', 'applying', 'dowling', 'sprinkles', 'marked', 'codger', "state's", 'cuckoos', 'tidbit', "fallen's", "murderer-on-the-loose'", 'donahue', 'disallows', 'nephews', 'bio', 'greeted', 'bribery', "runner's", 'dredge', 'thacker', 'ballyhoo', "hero's", "its'", 'zooms', 'wise-cracking', 'thorugh', 'buries', "13'", 'ex-pat', 'maniacs', 'unheard', 'racked', 'misinterprets', 'alarms--the', 'deathbed', 'screenplays', 'beaulieu', 'cradling', 'tho', 'mccullah', 'replicate', "clerk's", 'bernt', 'fran', '`patch', 'caffrey', 'bafta', 'cover-up', 'wyatt', 'drummond', 'showing', "moxxon's", 'svenwara', 'incentive', "ancestor's", 'co-horts', 'desired', '10-foot', 'buddy-action', 'reappeared', 'big', 'tumultuosness', 'enemy', "lefty's", 'godfather-esque', "wood'", 'hirsch', 'cheri', 'lope', 'uninformed', 'ambiguously', 'goyer', 'chinese', 'fabric', 'fulfilling', 'warming', "'79", "yimou's", 'bawling', 'aloof', 'greeting', 'makers', 'capabilities', '1977', 'portion', 'stringent', 'vow', 'alibis', 'blankness', 'jeff', "ophelia's", 'ilona', 'chariots', 'reloading', 'university', 'wizards', 'jeez', 'journeys', 'tristen', 'cannery', 'palladium', 'exteriors', 'williams', 'masayuki', 'gangly', 'mama]', 'persuading', 'softener', 'anti-life', 'vaginal', 'seraphs', 'nile', 'dazzled', 'outrage', 'faring', 'caper', 'angered', 'wuthering', 'liner/haunted', 'surrealistic', 'turbulence', 'shredded', 'putrid', "bulworth's", 'obstacles', 'khondji', 'gras', 'sheerly', '$4', 'slave', 'off-kilter', 'streamed', 'diggler', 'spite', 'cassavetes', 'soliloquy', 'pubs', '_more_', 'dahlings', 'suspenseless', 'director-cinematographer', 'convincing', 'chinatown', 'thirty-year', 'brigette', 'mainstream', 'bikini-clad', 'mommies', 'physician', 'thrax', 'populace', 'ghostly', 'schulman', "batman's", 'breathable', 'bava', 'adulterer', 'understatement', 'threshold', 'chipmunks', 'frieberg', 'language', 'understandably', 'genevieve', 'neckties', 'showgirl', 'sketchily', 'dish', 'formless', 'needfully', 'harvest_', 'ferocious', 'arab-american', 'crevillen', 'not--she', '1305', 'honours', '`raging', 'impart', 'well-meant', 'unsubtle', 'wandering-but-not-going-anywhere', 'delievered', 'shire', 'retaining', 'trainload', 'reappear', "lang's", 'old-style', 'colors', 'eyebrows', "shandling's", 'enlist', 'radar', 'inaccuracies', "malcovich'", 'tries--then', "protesters'", 'ex-fighter-pilot', 'events', 'knotts', 'attain', "creek'", "linklater's", 'all-together', 'archive', 'deconstructing', 'hammerbottom', 'theodore', 'oil-infested', 'dissolve', 'warn', 'curfew', 'house', "residents'", 'cara', 'watchdogs', 'libero', 'numerous', 'mcglone', 'monumental', 'dislike', 'repetitious', 'haunts', 'standards', 'wait', 'espionage', 'treill', 'flatulent', "commodus'", 'country', 'mardi', 'doubled', 'waltz', 'comprise', "glass's", 'continual', 'always-broke', 'dines', 'termination', "marshall's", 'hathaway', 'thwarting', 'homicides', 'sacha', "westerner's", 'svengali-like', 'lackies', 'holidays', 'zanin', 'dough', 'loner', 'hammering', 'taped', "lerner's", "'60s", 'capturing', 'mounts', 'self-replicating', "helen's", 'dealers', 'protestant', 'romantic-comedy-action', 'sub-standard', 'u2', "theatre's", "unabomber's", 'taut', 'katy', 'shit-kicking', 'cecilia', 'consumption', 'romeo', 'full-time', 'manipulate', '1865', 'titties', 'shucks', 'manhood', 'swishes', "yesterday's", 'judith', 'hope/crosby', 'wilkes', 'foreboding', 'binoche', 'von', 'spawn', 'lunches', 'specialize', 'overly-long', 'conehead', 'beacon', 'elegiac', 'evils', 'fro', 'sugarcoat', 'diddly', 'worldly-wise', 'druggie', 'rocketship', 'ghost--make', 'performaces', 'dionne', 'tape', 'gaines', 'shard', 'anglican', 'lucio', 'debt', 'snag', '-winning', 'relation-', 'putting', 'escape-as-quickly-as-you-can', 'afire', 'trolley', 'show_', 'prove', 'alec', 'expresses', 'standbys', 'forbidding', 'promiscuity', 'bullied', 'cross-bow', 'lame-ass', 'chicks', 'thoughtful', 'attendent', 'unequivocally', 'excites', 'bennet', 'katherine', 'yachting', 'self-possessed', 'wholeheartedly', 'britt', 'mega-bookstores', 'transforming', 'semi-slum', 'meticulously', 'respect', 'deteriorates', "`blissfully-confused'", 'faked', 'environmentalist', 'memories', 'sunset', 'slut', 'determine', 'fiqure', 'exorcist-rip', 'totatly', 'edouard', "detorri's", 'inhuman', 'aggressive', 'damme', 'reifsnyder', 'low-lit', 'confessor', 'drafting', 'dunaway', 'appreciation', 'gnarled', 'piddling', 'wiseacre', 'differentiate', 'chushingura', 'else--perfect', 'cafeteria', 'brightly', 's$19', "'thing", 'annabella', "reader's", 'anti', 'correct', 'entrepreneurship', 'passive', 'quick-flash', "woods'", 'violence', 'correctly', 'distinction', 'volunteering', 'venomous', 'giddy', 'reliable', 'prophesy', "plummer's", 'meatloaf', 'merits', 'disorganised', 'pisken', 'transcendent', "1950's", 'enormous', 'fabrice', 'salvador', 'endeavor', 'plunges', 'immortalized', 'strategically', 'plausibility', 'baseness', 'duplictious', 'husk', 'lookalike', 'reneges', '_clueless_', 'sivero', 'wasteful', 'mianders', 'complain', 'scriptwriter', 'sensibilities', 'beauteous', "romulus'", 'irma', 'balcony', 'mildew-like', 'downy', 'not-so-', 'fired', 'mountain', 'berliner', '1981', 'awakens', 'anguishes', 'old-', 'scented', 'embark', 'supervillain', 'authentically', 'post-vietnam', 'perfume-drenched', 'malnourished', 'whodunnit', 'impaled', 'twin', 'serbia', 'self-hypnosis', 'ian', 'herrman', 'lyn', 'unashamedly', 'mccarthy', "dern's", 'close-ups', 'battle-damaged', 'knuckleheaded', 'lil', 'preist', 'papers', "leonard's", 'chips', 'realists', 'wallowing', 'wrestler', 'sphinx', 'supply', "august's", 'rhetoric', 'explicating', 'newly', 'stifling', 'cucumber', 'same-named', "lesnie's", 'defiantly', 'analyzation', 'qualifying', 'set-piece', 'grunted', 'suo', 'aroway', 'likable', 'unwed', 'carefully-stacked', 'katharine', 'cecil', 'cherbourg', 'ranch', 'verbal', 'still-living', 'whitney', 'congressional', 'what+s', 'clumsiness', 'beggining', 'ticking', 'fontenot', 'perfectly-assembled', "tis'", 'children_', 'bataillon', 'veritable', 'wallop', 'copper', 'glenda', "lima's", 'teams', 'detracting', '2029', "rainmaker'", 'lane', 'sewers', 'highlander', 'stayed', 'fabled', 'bernie', 'ricochet', 'butchered', 'libraries', 'inhumanity', 'orlando', 'money-', 'acheives', 'schroeder', 'requirement', 'holy', 'makeover', 'rienfenstal', 'olympics', 'ears', '____', 'dullard', 'shaved', 'cobra', 'individually', "maker's", 'clever-ing', 'unreasonable', 'micro-organisms', 'sub-genres', 'near-masterpiece', 'kirkeby', 'timon', 'hypnotize', 'tactic', 'cutsie', 'laundry', "_daylight_'s", 'barbara', 'baranski', 'discarged', 'financed', "naturalist's", 'paragliding', 'must-see', 'justification', 'toupee-sporting', 'underlit', 'learns', 'gruff-talking', 'mid', 'mk2', 'infect', "o-dog's", 'parks', 'thing', 'decreasing', 'mena', 'praying', 'ruber', 'flies', 'detector', 'york_', 'reconstruction', 'club/act', 'self-contained', 'gutenberg', 'pakistani', 'suicide--bad', 'well-endowed', 'serial', 'banker', 'potty', 'loosest', 'clunkiness', "vianne's", 'trippy', 'tenderly', 'illuminati', 'borscht-belt', 'kauffman', 'drama', 'watcher', 'method', 'waxing', 'oozing', 'grandchild', 'fingers', 'atypical', 'dunst', 'innappropriate', 'one-act', "acquaintances-i'd", 'tullymore', 'compardre', 'quoi', 'vesco', 'cal', "bennet's", 'leather', 'inept', "wenders'", 'oneself', 'pre-intellectualizing', 'premiere', 'archeologist', 'elated', 'time-bomb', "copolla's", 'monologues', 'utilising', 'ex-gangsta', 'bike', 'easter', 'occurrences', 'bloodsucker--and', '70', 'droll', 'disinterest', 'obliterate', "guzman's", 'thespian', 'chilly', 'studying', 'experiences', "moreu's", 'dusty', '00', 'illusion', '`light\x12', 'motiveless', 'caverns', 'marooned', 'yemen', 'science-a', 'swann', 'footnotes', 'oceanic', 'farscial', 'devine', "seahaven's", 'slapstick-fu', 'disorientation', 'transporting', 'frugal', 'revolve', 'amazement', 'jibe', 'directed', 'cultural', '_there_', 'mostly-straightforward', 'sara', 'meany', 'webpages', 'pedal', 'punctiation', 'evil', 'feathers', 'plank', 'richardson', "cumming's", 'nab', 'attributable', 'the-top', 'toole', '-awarded', 'prolific', 'canning', 'class-conscious', 'goofs', 'atwood', 'ingmar', 'moroccan', 'media-saturated', 'identically', 'envision', 'novel--but', 'repressiveness', 'flowed', 'dork', 'acquittal', "firestorm's", 'even-odd', 'race-motivated', '$$$', 'party-goers', 'masterpiece', 'b-films', 'investigative', "_can't_", 'river', 'warren', 'blair', 'chances', 'renewed', 'punished', 'slice', 're-appearance', "shirase's", 'strangest', 'krumholtz', 'ninja', 'one-note', 'fabricated', 'boy-friend', 'dave', 'class-divided', 'aptitude', 'heslop', 'adversaries', 'multiplicity', 'supplies', 'dussolier', 'pig-like', 'attention-deficit-disorder', 'lugging', 'lechery', 'paints', '``insectopia', 'editing', 'belabored', 'heirs', 'vader', 'bewilderingly', 'benito', 'special', 'is]', 'thelma', 'ivey', 'gazes', 'concerto', 'innocent-at-heart', 'status', 'time-honored', 'croatia', "morpheus's", 'back-story', "bava's", 'mailheart', 'fashion-conscious', 'squelch', 'gum', 'crucifix', 'felon', 'sec', 'lugubrious', 'forest', 'funny', 'aaaaaaaaah', 'lightning-fast', 'ethereal', 'survival-of-the-fittest', 'amazingly', 'tamora', "welsh's", 'sil', 'dumbing-down', 'cheerleading', 'mel', 'indicator', 'colored', 'strike', 'azure', 'extension', 'fractured', 'alberto', 'egocentric', "mary's", 'midwife', 'guilt-ridden', 'bentley', 'second-in-command', 'cowboy=', 'detonate', 'white-hot', 'groaned', "'pleasantville'", 'occurances', 'trashily', 'dent', 'diversionary', 'seattle', 'greenaway', 'mashed', 'paraphrase', 'flyboy', "group's", 'rhodes', 'anly', 'product', 'hilary', "nail's", 'interwoven', 'lobotomies', 'terminus', 'arriving', 'unlimited', 'prediction', 'sympathiser', 'disrobe', 'answering', 'super-vampire', 'near-total', 'vents', 'peep', 'reinvigorated', 'aulon', 'powerfully', 'inhabiting', 'interpersonal', 'idly', 'duds', 'irked', 'flawed', 'earthling', 'padding', "stalked's", 'entrails', "dawson's", 'syncing', 'humphry', 'oavs', 'halen', 'lennix', 'stylistic', 'frenchman', 'barbaric', 'scintilla', 'half-cynical', 'norris', "month's", 'zoolander', "hayes'", 'jerk', 'debonair', 'casper', 'wellville', 'leitch', 'ultra-naturalistic', 'nevers', 'humongous', 'cold-blooded', 'bendel', "hopkins'", 'particulars', 'arch-rival', 'mumbo', 'self-treatment', 'overkilling', 'clarity', 'crisp', 'legged', 'babyface', 'jungle', 'malcolm-as-momma', 'dunne', 'sil--is', "'ugly's", 'directors', "ladies'", 'hotly-contested', 'concurrence', 'intrusive', "engagement'", 'miscast', 's/m', 'bumblebee', 'refining', 'nearly-perfect', "aykroyd's", 'machinery', 'eye-catcher', 'brosnon', 'bicentennial', 'centrifuge', "pope's", 'real', 'zookeeper', 'vorzon', 'mid-way', 'acknowledge', 'snoozer', 'coney', 'confining', 'inexpensive', 'cut-away', '_escape', "bison's", 'releases', 'administration', 'spiner', 'bouncer', 'gypsies', 'gellar', 'infiltrates', 'guises', "anaconda's", 'newly-freed', 'otis', 'ingerman', 'mercenary', 'heap', 'seventy-five', 'thorny', "hill's", 'scrolls', 'paris', 'klingon-looking', 'crumpled', 'zap', 'stunt-work', 'whisperer', 'dimensionality', 'wondering', 'sideswipes', 'bazooka', 'operation', 'skulking', 'catapults', 'dlose', 'overblow', 'absurdly', 'influential', 'eastern', 'cartoonish', 'overplotting', 'silences', 'approachable', 'radiation', "e's", 'renounces', 'yugoslavia', 'yeah', "recipient's", 'erected', 'clarke', 'helplessness', 'taye', 'farcical', 'voice', 'jungles', 'small-screen', 'celebrity-hood', 'co-directors', "owens'", "'out-of-character'", 'glass', 'drills', 'rudimentary', 'anchorman', 'pomp', 'outweighs', 'genial', 'inside-out', 'inflicting', 'apocalyptic', 'photogenic', 'turnarounds', 'scotland', 'quickie', 'keeslar', 'acclaim', 'bowden', 'wrong-perhaps', 'drowsy-voiced', 'nighttime', 'kissed', 'jarvis', "pistone's", "nitro'", 'carrey-annoying', 'dignity', 'engrossed', 'erickson', 'hanks/ryan', "kidnapper's", 'space-ships', 'biographies', 'fitting', 'scumbags', 'rowing', 'stun', 'purported', 'humanizing', 'flopped', 'plates', 'debris', 'abbott', 'hoppe', 'launderer', "'one", 'twenty-six', 'contemplative', 'gazon', 'philadelphia', "yourself'", 'eileen', 'disobeys', 'calahan', 'burbank', 'greenbury', 'smeared', 'potentially-interesting', 'charred', "planet'", 'jokes', 'bead-adorned', 'awaits', "vehicle's", 'dryer', 'drought', 'collated', 'paprika', 'gilligan', 'counterpoint--or', 'aix', 'rising', 'visualised', 'speculation', 'ear-to-ear', 'three-pitch', 'explanations', 'roberts--', 'pitfall', 'anacondas', 'rennie', '100m', 'prevalent', 'under-10', 'gadget-filled', "fitchner's", 'quickest', 'creationism', "'threatening", 'bears', 'filmmakers', 'cuckoo', 'vestments', 'length', 'criminally', 'one-stardom', 'inconspicuously', 'mcg', 'evolved', 'kieren', 'enact', 'blindness', 'subtitling', 'steam-controlled', 'hangout', "godzilla's", 'corbin', 'bailing', 'cables', 'pop-psychology', 'bauchau', 'ginter', 'winningly', 'disserve', 'coiffed', 'decisions', 'distaste', 'africa', 'ruin', 'anarchic', 'prosperous', 'meyers', 'young-david', 'bittersweet', 'glorifying', 'weaken', 'foundations', '1984', 'co-coaching', 'evolve', 'stratum', 'character-developing', 'ws', 'lawrence', 'wads', 'quirks', 'cosette', 'apricots', "jakob's", 'cave', "nello's", 'papaya', 'bugs-but', "fightin'", 'sumptuous-looking', 'spire', 'hero-less', 'premonition', 'zaentz', 'reinforcing', 'brass', 'gurian', "frank's", 'anti-hype', 'everyweek', 'recitals', "eastwood's", 'comic-panel', 'portly', 'bankability', 'expense', 'elmo', '1900', 'disguised', 'kane', 'addictions', 'enigmatical', 'bearing', 'blasphemy', 'midpoint', 'holnists', 'composer/lyricist', 'consultant', 'soleil', 'beasly', "cuaron's", 'ming-liang', 'energetically', 'one-persons', 'optimists', 'neuroses', 'hating', 'possesses', 'nude', 'eloquently', 'henry', 'nhl', '$6', 'wilt', 'sessions', 'sharply', 'zones', 'lascivious', 'teenage-david', 'heft', 'pontificating', 'oh-so-cool', "baker's", 'well-choreographed', 'shag', 'moran', 'big-to-do', 'scheduling', 'reptile', 'lemon', '10-minute', 'pfferpot', 'gallo', 'programmed', 'quaint', 'rea', 'tubing', 'heady', 'woe', 'mcinnerny', 'vampyre', 'prejudiced', 'super-senses', 'steed', 'sixty-ish', 'confuses', 'zimmerly', 'partakes', 'seat-of-the-pants', 'stanleyville', 'technologies', 'kiddie-oriented', 'posession', 'danner', 'gentleness', 'pre', 'frights', 'criteria', 'accessorize', 'careers', 'creep', 'curiosities', 'cheek', 'one-star', 'threat', 'poets', 'excrements', 'rebuilding', 'gaudy', 'comic', 'burn-out', 'advertiser', 'toshiyuki', 'colorless', 'inhibit', 'newsgroup', 'self-serving', 'aided', 'nzonzi', 'reworked', 'parents', 'atmospheric', 'tv-series', "impact's", 'jiro', 'dement', 'faith-', 'overplay', 'blasts', 'indecent', 'vignettes-styled', 'metropolis', "cusack's", 'italoamerican', 'bang', 'eaten', "prudential's", "franklin's", 'specifics', 'intricacies', 'mega-buck', 'smutty', 'sarajevo', 'begin', 'krzysztof', 'mono', 'howie', 'phonograph', 'gigolo', 'vamping', 'commenting', 'textures', 'reported', 'aka', 'belches', 'flaming', 'rascally', 'gaskell', 'bubby', 'vinie', 'fonder', 'restraints', 'semi-graphic', 'hbo', 'infectious', 'sugar-coated', 'one-two', 'quivering', 'smug-looking', 'seven-foot', '$10', 'booze-filled', 'taboo', 'certificate', 'dunks', 'brothers', 'love-smitten', 'disreputable', 'barry', 're-created', 'disclose', 'luppi', 'utopia-gone-awry', "hartley's", "'lives'", 'thrill-seeker', 'acknowledgment', 'dully', 'bewildered', 'facemask', "gerald's", 'infusion', 'attempted', 'marijuana', 're-teaming', 'shrieks', "gingrich's", 'frankiln', 'kaige', 'caf=8a', 'ricci', "cast's", 'summary', 'vignette', "dreamworks's", 'first-', 'fades', 'cleanup', 'conquers', 'itself-a', 'kaleidoscope', 'jell-o', 'deduce', 'business-minded', 'quart', 'courier', 'izabella', 'prey', 'regularity', 'long-haired', "dinosaurs'", 'dead/and', 'jewel', 'foreshadowing', 'repel', 'study', 'pathological', 'squabble', 'snarling', 'acid', 'shaft', 'processions', 'lumps', 'pleasureless', 'moderately-successful', 'roles', 'microchip-sized', 'infomercials', 'duffel', "stewart's", 'ballerina', "jewison's", "mosley's", 'reprintable', 'genieveve', 'compositions', 'conditions--sonorous', 'attar', 'unrated', "vinegar's", 'zigzagged', 'wing', 'recycling', 'schwartzman', 'well-done', 'terminals', 'stuff--except', 'out-of-it', 'super-mushy', 'foulmouthed', 'standoff', 'hockey-masked', 'hepburn', 'horrific', 'oahu', 'cheers', 'proft', 'jenette', '05', 'scurrying', 'cribbed', 'dark-side', "smart-alec's", 'cams', 'sealy', 'maddeningly', 'unschooled', 'sian', 'db', 'honesty', '`entertainment', 'murmuring', 'nealon', 'hunk', 'dissatisfying', 'phone-assault', 'electricity', 'pigeonholing', 'games', 'irreverence', 'angel', '39', 'airplane-type', 'dropped', 'injure', 'formula', 'grouchiness', "ripper's", 'website', '1=a', "nature's", 'by-the-end', 'toronto', 'eomann', 'embellished', "something's", 'sacrilegious', 'c-word', 'mood', 'elle', 'dynamic', 'scanning', 'cantrell', "becker's", 'revenge', 'rampaging', 'resonate', 'journal', 'fireball', 'clown', 'reasons', 'radiating', "granddaughters'", 'unhappy', 'occurred', 'roundtable', "ordell's", 'nam', 'eyeful', 'sentimentally', 'brocade-draped', 'annoyances', 'eisely', 'quarry', 'tsavo', 'workload', '`final', 'vary', 'hear', 'sherman', 'skater', "lazard's", 'canton', 'impending', 'sandwich', 'parties', 'influenced', 'phillipie', 'darting', 'tribeca', 'unaddressed', 'intriguing', 'plodding', 'packaging', 'parodyish', 'city', 'chanting', 'morsel', 'continued', 'two-room', 'physically', 'scenes--the', 'considers', 'werewolf', 'mcguire', 'stickers', '`ed', 'myths', 'escadapes', 'returns', 'myrtle', 'brooklyn-bound', 'bounding', 'ilk', 'identities', 'elation', 'dissuasive', "grinch's", "ranger's", 'tragi-comic', 'sheepishly', 'pre-aids-scare', 'jittering', 'alger', 'limo', "wen's", "grady's", 'bad-asses', "feelin'", "parents'", 'quieter', 'assailing', 'mine', 'miley', 'lower-middle', 'desperation', "cinema's", 'suites', 'agenda', 'oversees', 'sweetie', 'co-lead', 'simpler', 'male-bonding', 'halls', 'um-teenth', "jermaine's", 'sub-inspired', 'falling', 'chicago', 'low-to-medium-budget', 'attempt', 'frontals', 'scenery-', 'pronouns', "shannon's", 'sitcom-level', 'colleagues', 'presides', 'frustrates', 'rats', 'distrusts', 'tropps', "dvd's", 'zdenek', 'gave', 'cello', 'fleming', 'calloway', 'overhead', 'victor', 'allegedly', 'farrow', 'heighten', 'ramones', 'engulfs', 'mermaid', "rickman's", 'shoot-em-ups', 'burroughs', 'consumerism', 'remar', 'berendt', "today's", 'yawn-provoking', 'bronx', 'overseen', 'impossibility', 'propriety', 'gladiator-like', 'scratchy', "andy's", 'snail', 'tune-meister', 'gaining', 'espn', 'eavesdropping', 'cryptic', 'nicolet', 'liberated', 'darren', 'inadequacies', 'recite', 'offshoots', 'public-domain', "mandell's", 'growling', 'woodard', '25-and-under', 'utility', 'rubber', 'daugher', 'twenty-year-old', 'facets', 'lawn', 'short-film', 'correlates', 'reddick', 'greased', 'remarking', 'wintery', 'leonowens', 'nonetheless', 'neutral', '1913', 'lamberto', 'disbanded', 'domination', 'visiting', 'spotlights', 'diehard', 'clipped', 'monoko', "`batman'", 'leder', 'menahem', 'suceeds', 'pompeii', 'schuster', 'trigonometry', "wood's", 'directorship', 'publique', 'incoherent', '`snake', "o'shae", 'sociopath', 'speed', 'increase', 'villiany', 'snobbishness', 'flaunted', 'doll', 'inherited', 'keifer', 'upbringing', 'moview', '_too_', 'riley', 'gold-tinted', 'tonya', 'collide', 'oliveira', 'heinlein', 'lounge', 'beeing', 'denuto', 'inheritance', 'encourage', 'wrinkled', 'sela', 'toxic', 'turtleneck', 'ode', 'x-men', 'well-stocked', 'remeber', 'abandons', 'dissuade', 'zellwegger', 'regales', 'segregation', 'thoroughbreds', 'kashiwabara', 'huntington', 'imr', 'in-habit', 'raunch', 'sequel/spinoff', 'vault', 'endear', 'modernist', 'lag', 'upside', 'edits', 'hilly', 'christianson', 'pup', 'running/jumping', 'larded', 'suplee', 'unflushed', 'unblemished', 'tediously-and', 'thubten', 'disclaimer', 'choses', 'deserves', 'utah', 'repulsing', '7/10', 'satine', 'batarang', 'bikinis', 'illusions', 'analyses', 'depressed', 'stravinsky', 'niccol', 'hutchison', 'salty', "ellis'", 'lewton', 'dolly', 'privileged', 'inferences', 'dynamites', 'that+s', 'flame', 'stalkers', 'find--she', "crystal's", 'floats', 'lavish', 'tingles', "slugs'", 'washer/dryer', 'talking', "winner's", 'braugher', 'pursuer', 'innocent-looking', 'self-imposed', "reichle's", 'unruly', 'sharpest', 'anti-movie', 'dialect', 'mugger', 'sarcasm', '[sic]', 'inclosed', "beethoven's", 'bright-coloured', 'redone', 'inclusions', 'educated', 'bangy', 'ammunition', 'shelby', "hyams'", 'pat-on-the-back', 'colonels', 'knockdown', 'piano', 'across-the-board', 'patiently', 'hand-wringing', 'jill', 'leelee', 'wu', 'precedent', 'venice', 'crackles', '2007', 'ponderously', 'scope', 'dublin', 'laughed', 'fished', 'thickens', 'secretly', 'tinting', 'overblown', '_highly_', 'brandi', 'bellies', 'motorcylce', 'anti-terrorist', 'hillhouse', 'reborn', 'luhrman', 'force-feeding', 'chagrinned', 'four--yes', 'hawthorne', 'sgt', 'generous', 'harden', 'klutz', "target's", 'oranges', 'flanagan', 'effectively', 'dramatization', 'lipstick', 'mckenna', '1951', 'auto', 'exotic', 'jena', 'zoologist', 'untangle', 'respectably', 'hedonistic', 'odyssey', 'romping', 'lived-in', 'smother', "freddy's", 'consequence', 'box-sets', 'tilt', 'kennedy', 'beautiful--in', 'labored', 'cesspool', 'component', 'shaven', 'cinema', "cal's", 'artifact', 'hair-raising', 'firstly', "'stab", 'paint-by-numbers', 'louiso', 'gas', 'petersson', 'reprogramming', "murron's", 'yankees', 'alignment', 'withering', 'fondest', 'moves', 'circulated', 'beliefs-eloquently', 'teensy', '_hope', 'raining', 'cameron-regular', 'enright', 'frogs', 'input', 'harlem', 'bare-knuckle', 'tschombe', "porter's", 'overlays', 'all-star', 'oseransky', 'seduction', 'dumbfounded', 'replacing', 'tripp', 'behest', "wall'", 'invincibility', 'deceptively', 'nesmith', 'disingenuous', "gal's", 'spooked', 'stressed-out', "nichols'", 'fairytale-like', 'reluctance', 'geographical', 'thousands', 'carlyle', "coach's", 'aged', 'navy', 'sunflowers', 'jerkish', 'delaware', 'deserving', 'mtv2', 'befriend', 'singe', 'viewing', 'joshua', 'anarchy', 'unflinching', 'skerrit', 'saul', 'festively', 'ment', 'corpse', 'waxen', 'newton-john', 'earthly', 'enthralling', 'six-pack', 'screenwriterly', 'brinks', 'fork', 'bravura', 'canceled', 'straight-faced', 'power--to', "cliches'", 'johnny', 'lastly', "yeoman's", 'peasants', 'interments', 'clapping', 'grander', 'pounds', 'lured', 'one-night', 'sustenance', 'filmmaking', 'ravel', "macpherson's", 'goldstein', 'cocktail', 'horror/crime', 'commended', 'unlicensed', 'seedy', 'sci-fi/horror', 'grooving', 'statues', 'flavour', 'harumph', 'videocameras', 'bizarro', 'broiled', 'thundering', 'sound-truck', 'reagan', 'navigator', 'mental-patient', 'harmonica', 'overseas', 'jacott', 'dayton', 'prevents', 'spheres', 'everyman', "'modernization'", 'legitimately', "bennett's", 'emsemble', 'dr', 'salome', 'realize', 'bakula', 'everytime', 'guido+s', 'disperses', 'warnings', 'dim-bulb', "odette's", "antonio's", 'again-the', 'it]', "anti-hero's", "kaye's", 'acception', 'adopter', 'off-handed', 'skill', 'butterworth', 'wainwright', 'cinema--or', 'malt', 'fex', 'tycoons', 'brew', 'alcoholics', 'joann', 'section', 'cord', 'gretel', 'movie-you', 'part-human', 'hurt', 'danielle', 'poorly-developed', 'berkeley-esque', 'navigators', 'seance', 'treatments', "'romeo", 'charming', 'ex-slave', 'downpours', 'starving', 'benoni', 'fluish', 'protesting', 'vomit-inducing', 'fritter', 'mixed-coffin', "garfunkel's", 'dallied', 'persuasive', 'faux-pas', 'squat', "guaspari's", 'examination', 'hawkins', 'shmaltzy', "mulan's", 'gossipy', 'father+s', 'european', 'rapist-killer', 'snow', 'sequences-they', 'constant', 'expended', "poledouris'", 'minors', 'institutionalize', 'grease', 'chubby', 'implodes', 'engulfing', 'kids-intense', 'laughter-packed-moments', 'blues', 'appallingly', 'always-likable', 'gods', 'chu', 'wound', 'co-producers', 'eugenio', 'attractively', 'pamela', '-creator', 'nature-loving', 'plain', 'stamper', 'rifle', 'chemicals', 'poison', 'bull-heaed', 'tidy', 'dexterity', 'burping', "1963's", 'eroded', 'partners', 'colorado', 'selects', "'macho", 'visor', 'leguiziamo', 'dazzles', 'of=', 'enabled', 'frizzi', 'footprint', 'warrirors', 'mail-order', 'bonus--', 'dachshund', 'inter-racial', 'research', 'ablaze', '_american_beauty_', 'seana', 'benefits', 'overfills', 'thrown', 'pitchman', 'king-inspired', "stretch's", '1934', 'robberies', 'misinterpreted', "'friend'", 'wicked', 'gordon', 'ready-made', 'miscommunicated', 'suns', 'aidan', 'projects', 'protoplasmic', 'endorsements', 'choreographing', 'slung', '-lite', 'caitlyn', "zwick's", "al'", "tampopo's", 'eszterhas-ish', 'resting', "chimp's", 'sofia', "martha'", 'philanthropist', '$60', 'scruches', 'entailing', 'breakup', 'cdc', 'back-', 'qui', 'stuart', 'muted', 'absent-minded', 'thrower', 'release', 'sheeva', 'straightforward', 'substandard', 'detection', 'mortgage', 'uninterest', 'smart-assed', 'needful', 'discourage', 'through-the-sight', 'wine', 'overplotted', 'albany', 'bitingly', 'one-upped', 'hands-washing', 'contained', 'doctrines', "floats'", 'vampire-films', 'bravo', 'dvds', 'memento', 'hard-working', 'deteriorating', 'one-tone', 'bobcat', 'non-clinton', 'raffin', 'irons-like', 'signy', 'self-trance', 'charlton', 'one-', 'post-death', "foods'", 'freakish', "richard's", 'eleven-o-clock', '8-lane', 'christianity', 'unreliable', "'he", 'steroids', "marker's", 'rate', 'scruples', 'magazine', 'alar', 'completly', 'layer', 'linz', 'clutching', "barnett's", 'idolising', 'spoon-feeding', 'bloke', 'fenster', 'imbeciles', 'greatness', 'listened', 'tumble', 'spoilsport', 'memorable', 'underscored', "`roswell'", 'obsess', 'reserving', 'tyrant', 'euro-vamps', 'low-hanging', 'loved', 'inflicted', 'talent-impaired', 'broadcaster', 'comedown', 'irish-italian', "gambler's", 'near-pornographic', 'cigarettes', "kent's", "maunau's", 'facetious', "whatever's", 'smash', 'wesson', 'katrina', 'greyer', 'francesco', 'aluminium', 'opening-night', 'barfing', "it's-so-bad-it's-good", 'six-pack-wielding', 'rohmer', 'foams', ';', 'manicures', 'hooch', 'belittled', "juvenile's", 'romanticism', 'lucasfilm', 'occupation', 'harkonnen', 'heffron', 'reanimator', 'flying', "`that's", 'derailed', 'reporting', 'smash-hit', '189', "seven's", 'umptenth', 'sod', "page's", 'wile', 'heartache', 'underpopulated', 'nuance', 'co-produced', 'harp', 'scalding', 'annoying', 'dashing', 'caucasian', 'loin', "1400's", 'brandis', 'thirty-', 'formed', 'detract', 'raja', 'oscar-less', 'early-teen', 'indoor', 'robotic', 'army', "stifler's", '18-foot-high', 'patchy', 'abusive', 'terrifed', 'edt', 'hour-and-a-half', 'darnell', 'troop', 'annaud', 'action/adventure', 'self-reflexivity', 'judiciary', 'peering', 'dismal', 'cherish', "sweeney's", 'ironing', 'nickname', "'craziness'", 'effects', 'accompany', 'mano-e-mano', 'oversee', 'van', '_bad_', 'foregoing', 'mythology', 'cowboy/hick', 'keenly', 'independant', 'pre-', 'cancelled', 'undermined', 'revolutionized', 'renewing', 'man-eating', 'sixth', 'cuteness', 'empathy', 'chases', 'co-writing', 'preserving', 'semi-interesting', 'waitering', "joe'", 'film--especially', 'jeopardize', 'steers', 'downer', 'jade', 'ranger', 'worse-than-stereotyped', 'maw', "gettin'", "kip's", 'long-time', 'dislikable', 'steinberg', 'smalltime', "father-in-law's", "intentions'", 'hewitt', 'toeing', '2%', 'allegiances', 'amboys', 'marienbad_', 'videocasette', 'handwaving', 'venom', 'adaptation', 'bombshell', 'locomotive', 'occuring', 'electro-', 'buck-toothed', 'retreads', 'adepts', 'low-class', 'rainstorm', 'polarizes', 'regime', "catharine's", 'opponents', 'dynamo', 'rereading', 'hilariously', 'disaster-movie', 'graduating', 'eachother', 'molded', 'amarcord', 'watchabe', 'surreptitiously', 'progenitor', 'falls', 'repessed', 'year-long', 'judges', 'brand-new', 'domain', 'athletes', 'joyously', 'archeologist/grave-robber', 'vitarelli', 'heart-warming', 'bleakness', 'daper', 'not-so-good', 'reaction', 'reasonable', 'here]', "stavros'", 'positronic', 'academia', 'tourfilm', 'palpitation', "terry's", 'it--and', 'downturn', 'notched', 'fly-guy', 'flaxen-haired', 'alison', 'researching', 'biker', 'newly-pumped', 'cool-as-hell', 'bannen', 'copacabana', 'thirdly', 'high-flying', "tornado'", 'wallpaperer', 'belief', 'zangaro', 'gere', "kiki's", 'arsinee', 'prefers', 'second-run', 'continuous', 'dropout', 'destructiveness', "alice's", 'blues/rock', 'cruz', 'makeshift', 'heavy-metal', 'take-the-money-and-', 'cumbersome', 'keach', 'circa-1983', 'gulia', 'non-formula', 're-enact', 'stock-shoot', 'sheesh', "halperin's", 'equating', 'cosmos', 'firefighter', 'horseman', 'wear', 'clancy', 'alleviate', "king's", 'combs', 'damping', 'yanni', 'airing', 'frigid', 'nine-year-', 'retardation', 'enforces', 'buena', 'lunes', 'outgrossed', 'ulation', 'devoid', 'rejoining', 'snoozing', 'nabokov', 'lapage', 'spurned-psychos-getting-their-revenge', 'disheveled', 'panamanian', 'vetern', 'nightclubs', 'balliol', 'cushions', 'muldoon', 'codenamed', 'moralist', 'partying', 'gofer', 'listener', 'mcgreggor', 'taels', 'nishi', 'prosecuted', 'dangerfield', 'stout', 'double-chin', 'cambodia', 'piled', 'captivates', 'terry', "o'fallon", '_that_', 'minions', 'alcoholic', 'guesswork', 'lawsuits', 'zellwegar', 'mountainside', 'rosalba', '1996', 'kendrick', '555', 'ditching', "spices'", 'redeemed', '`scientifically', 'par', 'condom', 'many--alyson', 'essays', 'holistic', 'cautionary', 'bluth', 'alcohol-based', 'catholic', 'likability', 'distracting', 'woolrich', 'errant', 'stomach', 'seemingly-perfect', 'canine', 'farbissina', 'brussels', 'adolescence', 'rivals', 'alterations', 'sweeps', 'strolls', 'tagging', "watson's", 'nigh', 'substituting', 'emmett', 'cringe-worhy', 'ismael', 'flops', 'stairs', 'twins', 'plasma', 'hammiest', "knight's", 'vegas-style', 'out-of-context', 'decided', '22', 'unambitious', 'skogland', 'gabriella', 'un-produced', 'nine-year-old', 'strands', 'twohy', 'flushes', "x'", 'disturbingly', 'shapes', 'bear-like', 'particulary', 'dimming', 'sage-like', 'grandmother', 'then-footage', 'trey', 'deserved', 'stop', 'mccracken', 'detects', 'hilt', "xiong's", 'mud-wrestling', 'astronomer/writer', "liotta's", 'ruben', 'nascar', 'agreeable', 'inescapable--and', "'rewind'", 'fight', 'everton', 'thomspon', 'woulda', "_roxbury_'s", 'girlie', 'hunks', 'rail', 'existance', 'lengthen', '`little', 'lined', 'oil-guy-cum-astronaut', 'land', 'subjected', 'secession', 'much-decorated', 'cheeze', 'maul', "government's", 'gore-fest', 'i-iii', "vincenzo's", 'classical', 'right-wing', 'farewell', 'employee', 'helmet', 'viable', 'fiery', "what're", 'gold-diggers', 'flounder', 'retro-comedy', 'signifiers', 'half-gods', 'tilda', '----', 'intends', 'charismatic', 'over-18s', 'negatives', 'offed', 'cop-killers', 'decaying', 'festering', "get's", 'neutrality', 'senge', 'melissa', 'excising', "soderbergh's", 'humanize', 'gallery', 'multiplying', 'boasts', "mohr's", 'stifle', 'mother', '1938', 'facile', 'retching', 'sims', "rocky's", 'georgia', 'magnolia', 'kill-or-be-killed', 'shining/event', 'attacker', 'untalented', 'trinket', 'ard', 'mcgruder', 'feore', "witch'", 'excellence', 'mend', '1272', 'vacances', "cappie's", 'equalizes', 'premiered', 'annihilate', 'alterior', 'wool', "series's", 'peoples--or', 'narrowed', 'dumping', 'callum', 'best-ever', 'categorization', 'worker/janitor', 'squeezes', 'lava', 'luggage', 'asserts', 'puppeteer', 'whovier', 'man/half', 'worldview', 'reels', "forrester's", 'herlihy', 'preposterously', 'wardrobes', 'kayla', 'squads', 'plotwise', 'gallons', 'ny152', 'sings', 'cassavettes', 'humming', '_h20_', 'farmhouse', 'churchgoing', 'chris', 'bombings', 'heimlich', 'sloppy', 'midnight', 'kindness', 'incident', 'ballinagra', 'incomprehensible', "pearce's", 'voyeurs', "character's", 'creature', 'stints', 'market', 'fer', 'scramble', 'alotta', 'seatbelts', 'backroads', 'sidelines', 'hurling', 'exhilerating', 'points--just', 'mcgehee', 'send-up', 'odessa', 'generalization', 'restless', 'b-movie', 'actors/directors/producers', 'winnings', 'kkk', 'break-out', 'delusion', 'molest', 'adult-helfgott', 'tarnish', 'rollickingly', 'terrific-', 'viper', 'lowlife', '14-year', 'jean-baptiste', 'interactive', 'pantaeva', 'burp', 'sweaters', 'insecticide', 'undie', "sadist's", 'cyborsuit', "go-get-'em-tiger-we're-behind-you", 'absorption', 'welland', 'watchful', 'spontaneous', "chances'", 'comissioned', 'e-mail', 'gable', 'americas', 'dessert', "addam's", 'skywalker', "wayne's", 'before--sometimes', 'flourishing', 'pearlstein', 'provocations', 'address', 'verbinski', 'cathy', 'instills', "whale's", "driver'", 'uprorously', 'natty', 'bed-ridden', 'sometimes-blaring', "browne's", 'heym', 'sagas', 'sick-and-twisted', 'general', "1984's", 'film--one', 'chaos', 'momma', 'taguchi', 'soundbite', 'eight-and-a-half', 'delayed', 'glamorize', 'kenneth', 'swarms', 'inviting', 'helped', 'coition', 'supremacy', 'greet', 'contraptions', 'renowed', 'sleepwalking', 'lemonade', 'fflewdurr', "laden's", 'co-directed', 'stumbling', 'acts', 'tall', 'victories', 'clapped', 'janusz', 'message', 'shang', 'prisoner', 'prozac', 'well-chosen', 'telegraphed', 'sunken', 'boats', 'chans', 'tachometer', 'endearing', 'coupling', 'blathers', 'self-preservation', 'terminating', 'cherished', 'ewoks', 'lizard', 'presumedly', 'deny', 'cheer', 'injustice', '`long-distance', 'lashes', 'snarfed', 'non-memorable', 'omen', 'wasted--and', "`at&t'", '3-7', 'newly-founded', "'n'", 'twentysomething', 'renaissance', 'expectedly', 'fragmentary', 'edd', 'rigg', 'pretentiously', 'perseverance', 'garage', 'encourages', 'warriors', 'maturing', '$15', 'veronique', 'summarize', 'hobbyist', 'bowen', 'yum', 'omega', 'criticisms', 'film-the', 'wall-to-wall', 'glamorized', 'animation', 'dwarves', 'initializes', 'preposterous', 'curve', "handmaid's", 'perpetuates', 'affectations', 'bulleye', 'detonating', "intro'd", 'jee', 'escalating', "picasso's", "campion's", '--quite', 'half-star', 'womanizer', 'gold-digging', 'courses', 'toasts', 'all-girl', 'castles', 'conveniently', 'invite', 'hotter', 'placer', "`impulse'", 'motherfucker', 'truffaut', 'enriched', 'messy', 'nominations', 'antler', 'sexualized', "'dog-in-danger'", 'equidistant', 'loan', 'city-street', 'seeked', 'slinks', 'compulsively', 'bricks', 'gismos', 'middle-aged', 'symbolized', 'gustav', 'soaked', 'ex-government', 'bachar', 'regicide', 'omaha', 'lunching', 'covers', 'bubblegum-blond', 'laurel', "khondji's", 'pact', 'heroin-chic', 'invited', 'winstone', 'hip-to-only-themselves', 'christina', 'in-person', 'moribund', 'dubbing', 'opens', 'forgoes', 'gibb', "robbins'", 'decade--the', 'things-gone-awry', 'grievous', 'stills--heather', 'despondence', '20-', "mid-80's", 'issues', "'haunting'", 'plagiarized', 'centre', 'gen-y', 'protector', "karloff's", "clique's", 'pump', 'bickering', 'narrate', 'indien', 'debra', 'macivor', 'meanders', 'playmate', 'coarser', '#5', 'christie', 'embellish', 'farrelly', 'repress', 'dwellers', 'infatuated', 'swells', 'drug', 'davidzt', 'hamunaptra', 'hunts', 'tortures', 'erotic', 'threatned', 'loony', 'hoop', 'preparation', 'acting', 'rejects', 'aaron', 'petey', 'unsurpassed', 'banyon', '_pecker_', "braun's", 'clearing', 'peacably', 'vie', 'movers', 'multifaceted', 'chow', 'isolation', 'willy', 'pumpkin', 'values', 'restoration', 'disney-ized', 'failed', 'knowledge', 'spine', 'well-functioning', 'connote', 'bunz', 'mattress', 'plop', '28-year-old', 'kouf', 'combatants', 'feather', 'corpulent', '47-year', 'lambs_', '55', 'perk', 'segments', '[stick', "crock's", 'balto', 'myrick', 'corral', 'b-grade', 'upsetting', "'bicentennial", 'miscues', 'neat', 'diablo', 'mushrooms', 'substances', 'scavenge', 'ricki', 'hotels', 'gallagher', 'estella', 'overtaken', 'genitals', 'depp', 'star-power', "deaky'", 'high-schoolers', 'b/w', 'capriciously', 'flashes-sideways', 'modestly', 'caress', 'occupational-hazard', '102', "freakin'", "pitt's", 'hatches', 'hyperreal', 'neighbourhood', 'samir', 'brides', 'matador', '1922', 'climact', 'inventie', '_long_', 'fatally', 'tooling', "colonist's", 'poopie', 'adorably', 'fedora', 'rush', 'tvs', 'movie--special', 'jeanne', 'untimely', 'anthro-zoologists', 'torrid', 'runtime', "olds'", 'kruger', 'elna', 'displeased', 'self-propelled', 'baltus', 'piza', 'aliases', '2', 'mahem', 'howerd', 'technology-', 'selected', "suggs'", 'buckley', 'disputes', 'seductress', 'mis-', 'hometown', "1981's", 'bundles', 'cool', 'the-truth-at-all-costs', 'veered', 'neary', 'sever', 'veers', 'freaked', 'move', 'freaking', 'glades', 'socks', '1920s-60s', 'curry-spiced', 'descended', 'saboteur', 'duty', 'little--', 'anti-phantom', 'agricultural', 'deeds', 'sais', 'healy-louie', 'sound', 'prance', 'stupid/fun', 'molina', 'dargus', 'natural-born', 'clarisse', 'showtime', 'oooooo', "pharoah's", 'cinematographer-turned-director', 'fallacy', 'unrighteous', "grier's", 'peckinpah', 'techno-babble', 'drabby', 'panoramas', 'hinged', 'zen', 'bayou_', "mckellen's", "atkinson's", 'archery', 'indestructible', 'exorbitantly', 'suspect', 'troubled', 'once-ever-5000-years', 'qui-gon', 'rapists', 'williamson-less', "mctiernan's", 'maddalena', 'hooves', "d'oh", 'skipping', 'anticipates', 'clearwater', 'compensating', 'sailboat', 'trys', 'examiner', 'wilderness', 'paleontologist', 'offense', 'wields', 'eskow', 'greenhouse', 'restaurant', 'townspeople', 'father-and-son', 'sloppily', 'drying', "snipes's", 'luther', "elwood's", 'lawnmowers', 'dot', 'cooperative', 'disgusted', '10-day', 'detracts', 'iceberg', 'foolhardiness', 'forgiven', "manson's", 'lenders', 'species', 'dinky', 'bureaucracy', 'prose', 'gangland', 'posey', 'reference', 'galas', 'mishears', 'bundle', 'outgoing', 'feuds', 'privilege', 'six-astronaut', 'niger', 'incurring', 'mutes', 'however--the', 'explained', 'dweeb', "arnold's", 'mimieux', 'coding', 'sub-section', 'exclusivity', 'idolized', 'upscale', 'blending', 'associating', 'blessings', 'bernheim', 'knit', 'tina', 'imaginative', 'alcoholism', 'rhys', 'relentlessly', 'wylde', 'avengers', 'implausibility', 'post-forensic', 'tulips', 'brunet', "zero's", 'c-for-charlie', 'inexplicable', 'arraigned', "willis's", "holding's", 'jutting', 'credit--', 'bertram', 'stanze', 'yon', 'above-ground', 'jinks', 'hysterical', '1/8', 'character-setups', 'danvers', 'alot', "cryin'", 'stresses', "patch's", 'handsome', 'scheming', 'begins', 'hostage-holding', 'writer/director/actor', 'pro-slavery', 'exposition', "phillippe's", 'icing', 'dime-a-dozen', 'allegiance', 'paddy', 'patially', 'hrs', 'tamuera', '98', 'vukovich', 'revenge-for-hire', '1773', 'rina', 'talent-challenged', 'wrestling', 'esophagus', '1521', 'suverov', "juliet's", 'peyton', 'beastuality', 'post-', '3000-level', 'parody', 'cops-on-the-trail-of-serial-killer', 'plopped', 'colonization', 'react', '911', 'rightful', 'erroneous', 'capitol', 'weariness', 'fully-realized', 'overcrowded', 'haplessly', 'foreplay', "coalwood's", 'shelter', 'beret-wearing', 'reoccurring', "jerk'", 'columbian', '1991', 'obscene', 'hershman', 'heaps', 'understandeably', 'servo', 'non-drinking', 'conners', 'types', 'noxima', 'shuffled', 'flatly-written', 'saunders', 'kibbe', 'stylelessly', 'howlingly', 'individualism', 'whooshed', 'overstuffed', "porky's", 'process', 'private-eye', 'inexcusably', 'carrot', "prince'", 'lush', 'clue', 'schwartzenegger', "spencer's", 'captors', 'drug-induced', 'signatures', 'writings', 'kario', 'nine-film', '9mm', 'kinda', 'ottorino', "de'ath", 'crooked', 'mann', 'widens', 'lot', 'jiff', 'insomnia', 'pajamas', 'jumping', 'arising', 'wash-outs', 'automobiles', 'nc-17-rated', 'cleaner', 'auditions', 'well-timed', 'ruling', 'hallucinations', "o'", "kuwait's", 'beleives', 'driveway', 'aggie', 'derelict', 'travails', 'valderrama', 'stickell', 'milton', 'floorboard', 'licence', 'wordless', 'gyrations', 'pursues', "cuckoo's", 'themed', 'effeminate', 'concludes', 'rubell', 'fundamentally', 'degredation', 'uploaded', 'longshot', 'deem', 'power', 'ultimately', 'sweat', 'straying', 'rehearsed', 'noticably', 'mecilli', 'hala-russell', 'wild-idea', 'syringe', 'glimpses', 'imbuing', 'faustian', 'haygarth', 'singers---they', 'egg-headed', 'damsel-in-distress', 'rites-of-passage', 'bewildering', 'trainman', 'hindering', 'schematic', 'affirms', 'ham-radio', "blues'", 'assasinated', 'izzard', 'konner', 'geekiest', "butabis'", 'dickens', 'spastic-but-friendly', 'credits', 'yorker', 'unhappily', 'auspicious', "moresco's", 'officially', 'benefit', 'intersperesed', 'interpreting', 'spry', 'for--cooking', 'narrow-waisted', 'ploys', "legend's", 'above-the-title', 'rung', 'newsradio', 'backdrop', "ritter's", 'wafer', 'remarked', 'deviance', 'model-turned-actress', 'sheets', 'galloping', 'top-down', 'sucess', 'predators', 'javier', 'nasties', 'elm', "jeremy's", 'monumentally', 'hurdle', 'leopard', 'healy', 'screechy', "madisons'", 'strengthens', 'ramis', 'urinates', 'malfunction', 'wilkinson', 'aboriginals', "grandson's", 'cryo-freeze', 'topple', 'stickiest', 'mask', 'captured', 'frost', 'emmanuel', 'breuer', 'replicants', 'kristen', 'strays', "mile'", 'barking', 'barcelona', "walsh's", 'james-who', 'jocky', 'iniquitous', 'graveness', 'maligned', 'consoles', 'gamble', 'deciphering', 'bathrobe', 'portrayal', 'lughnasa', "butcher's", 'frame', 'warns', 'student/teacher', 'realisation', 'laboured', "feldman's", 'cop-hunting-killer', 'churn', 'injuries', 'managing', 'hippo', 'underseen', 'chose', 'varsity', 'plissken', 'well-lit', 'pubic', "people's", 'eyre', 'indistinct', 'folded', 'injects', "'beep'", 'gasped', 'promises', 'oumph', 'docu-drama', 'concentrations', 'film-school-grad', 'dahl', "bloomingdale's", 'nina', 'joints', 'distressing', 'heartless', 'belushi', 'backlot', 'nailing', 'hopping', 'malachy', 'rothhaar', 'eventful', 'hadden', 'jorgen', 'bodies', 'reliability', 'airport', 'ambition', 'fakes', '_jerry_maguire_', "'sunk'", 'cheeziness', 'movie', 'tolerant', 'slashfest', 'unceremoniously', 'chimes', '800-mile', 'joaquin', 'skit', 'under-appreciated', 'shallowly', "jenny's", 'tapes', 'thal', 'edge', 'growing', 'kien', 'employing', 'writer/director/co-star', 'damnedest', 'duplication', "mimura's", 'primo', "boxer's", 'motherland--one', 'recruits', 'exonerating', "`jaws'", 'petrovsky', 'kang', 'death-defying', 'epigraph', 'infant', 'pace--was', "she'd", 'forbid', 'spaceflight', "'nuff", 'slap', 'swarm', 'straightens', 'ius', "haunting'", 'project', 'gang', 'resuscitated', 'marking', 'booted', 'shrugging', 'death-row', 'innacurate', 'staggers', 'semi-articulate', 'prizes', 'breather', 'spokesman', 'retreating', 'hamlet', 'cookie-cutter', 'savviness', 'ignore', 'buildup', 'forties', 'wished', "murderer's", 'allies', "killer'", 'decadence', 'vivien', 'emphasizes', 'scrumptiously', 'carnage', 'chaplin', 'ham', 'complicating', "connie's", "'copter", 'crappiness', 'disapproval', 'feels', 'battered', 'jeroen', 'enfield', 'suarez', 'ammunitions', 'agency', 'new-born', 'leone', 'strikeout', 'yarns', 'thrives', 'pseudo-maid', 'boondocks', "biebe's", 'tyson', 'rhythmless', 'ape', 'hodgman', 'speakeasy-type', 'anthropologist', 'bulldozer', 'privately', 'paradigm', 'newly-created', 'overpriced', 'subjectivity', 'proceeder', 'inciting', 'two-foot', 'immunity', 'careful', 'jolts', 'ebony', 'cites', 'self-pity', 'hermetic', '34th', 'gm', 'assure', 'tack', 'biopic', 'steady', 'ship', 'surrender', 'gangbangers', 'vet', 'soon-to-be', 'satisfyingly', 'avenger', 'nickolas', 'belone', 'cower', 'patients', 'sorrow', "'batman", 'regimes', 'razzle', 'elaborate', 'big-headed', 'constituency', 'cranky', 'dalmations', 'furs', 'heywood', 'yamamoto', 'co-sex', 'rich-', 'bazookas', 'intelligent', 'prime-time', "festival's", 'yippee', 'trait', 'zemekis', 'sanctimony', "bastard'", 'missles', 'fanaro', 'remarkable', 'mussenden', 'retell', 'post-torture', '79', 'horse', 'wong', 'australia', 'theories', 'means', '11-year', 'disoreitating', 'keels', 'blue-toned', 'life-long', "time's", 'uncharismatic', 'dreariness', 'mangled', 'imbalances', 'by-passes', 'waller', 'wrinkle', 'cigars', 'snitch', 'pretty', 'ridiculousy', 'thermopolis', 'kutcher', 'animal-rights', 'erykah', 'rool', '21a', 'ecker', '1-900', 'half-full', 'hankie', 'permanetly', 'coufax', 'unimaginatively', 'warhead', 'indigestion', 'collect', 'repelling', 'cabalistic', 'glop', 'explores', 'foreground', 'demand', 'exclusively', 'directing-wise', 'whistles', 'showiest', 'iota', 'speer', 'chain-smoking', 'miniscule', 'advances', 'epiphanies', 'video-esque', "swinton's", 'carried', 'underused', 'begotten', 'mm', 'sonja', 'cappuccino', 'ballet', 'avi', 'fictional', 'kool', 'publishes', 'throw', 'barney-like', 'shameless', 'carlin', 'intertwining', 'mourning', 'unspool', 'functions', 'stewart--makes', 'aspirins', 'uncompromisingly', 'futile', 'wasted', 'nonexistant', "'geniuses'", 'suggestive', 'devolve', '---has', 'marcel', 'proposing', 'philadephia', 'earmarks', 'conversions', 'lockers', 'margins', 'flora', 'hulk', 'linked', 'endured', 'godfrey', 'retail', 'thug', 'methods', 'new-age', 'invigorated', 'busboy', 'clearer', '53', 'camped', 'clone', 'made-for-the-sci-fi-channel', 'animations', 'combination', '_this_', 'perlich', 'tsi', 'surpasses', "music'", 'poodle', 'infectuous', 'billed', 'miracuously', 'corruption', 'messengers', 'vests', 'prototypical', 'props', 'ambivlaent', 'unwinds', 'slapdash', 'film\x12s', 'screwed-up', 'colton', 'refuses', 'dogs-style', 'erasing', 'obesity', 'fanatical', 'co-pilot', 'rectangles', 'resurrecting', '31st', 'swans', 'tati', 'drew', 'suffrage', 'gynecologist', 'transformation', 'contradictory', 'forte', 'replaced', 'jurgen', 'casualties', 'blackburn', 'sacking', 'performig', 'sensually', 'proposes-demands-that', 'dedicate', "i'll-convince-people-to-kill-each-other", 'covering', 'plummer', 'buff', 'eagerness', 'troops', 'paraplegic', 'des', 'didya', 'replete', 'hai', 'shelf', 'rope-bridge', 'sidewalks', 'angelic-appearing', 'dastardly', 'roll', 'patillo', '_the_broadway_musical_', 'gun-dealer', 'kindergartners', 'non-linear', 'spider-man', "harry's", 'alexi-malle', 'lifeform', 'screamers', 'amendment', 'herman-make', 'appearing', 'websites', 'oversized', 'montero', 'entrusting', 'tenseness', 'mccamus', 'albanian', 'eisner', "'financial", 'disregards', 'commemorate', 'producers', 'jennings', 'high-pitched', 'humanity', 'looting', "workers'", '2001', "maguire's", 'graver', 'blitz', 'savagery', 'interrupt', 'turkies', 'karina', "man's", "headey's", "'if", 'ms', 'failures', '400', 'apprehensively', 'induced', 'prattle', 'devotion', 'ironies', 'unfounded', 'massey', 'bambi', 'exaggerated', 'dodie', 'dissapointment', 'brings', 'koster', 'pods', 'diverse', 'loft', 'gym', 'close-knit', "everyone's", 'perched', 'echelons', 'get-go', 'bohemia', 'wonderland', 'meaney', 'sponsorship', 'romanticized', 'sinch', 'calibrated', 'individual-underdog', 'mary', 'cruising', 'shirtless', 'mishmash', 'nonentity', 'demonstrate', 'sheepish', 'prophesized', 'moulin', 'tribune', 'impish', "convicts'", 'commemorating', 'creaky', 'peaks', 'horn', 'mettler', 'witnesses', 'outer-city', 'arrow', 'glorified', 'coincidence', 'applicable', "night'", 'audio/video', 'post-world', 'discontented', 'manifested', 'launches', 'high-profile', 'couch', 'drug-ravaged', 'epilogue', 'eightly', 'unwieldy', 'boring', 'dramatics', 'peaceful', 'biggerstaff', 'rattner', 'extra-curricular', 'glad', 'arrest', 'golfer', 'embrace', 'arkham', 'risk', 'deflowered', 'unmistakably', 'tirelessly', 'surehanded', 'finn', 'caravaggio', '10-year-', 'neurotically-charged', 'hard-edge', "coote's", 'gere\x12s', 'exterior', 'victim', "public's", 'joylessness', "detroit's", "gorton's", 'pulse-pounding', 'tectonic', 'psycho-in-love', 'fiancee', 'expertly', 'film-hannibal', 'rollers', 'precluded', 'camelia', 'burbon', 'chicago-connected', 'controllers', 'shopkeeper', 'evoking', 'rupaul', "joss's", 'farming', 'fooled', 'larroquette', 'cherokee', 'venality', 'loeb', 'accuse', 'exceeds', 'precedents', 'nail-biters', 'nipple', "odin's", 'less-than-stellar', 'separatist', 'inadvertently', 'slugging', 'filmation', 'obtrusive', 'leeanne', 'coliseum', 'iced', 'roadrunner', 'shrunken', 'circumcision-giving', 'traffic', 'threesome', 'mediums', 'natural', 'taggart', 'kurt', 'pounders', 'rhyme', 'braun', 'jamaican', 'molecules', 'movie-lover', 'sigmoid', 'glean', 'cum-islamic-missionary', "paradine's", 'halloween', "glass'", 'brite', 'feat', 'tenets', 'organic', 'mikel', 'fetish', 'febre', "know's", "minds'", 'specially', 'sigmund', 'scandal-ridden', 'brazillian', 'emotionally-scarred', 'klendathu', 'however---not', 'bombshells', 'mad', 'numbskulls', 'inually', 'restrictions', 'colloquialisms', 'point-blank', 'distinguishable', 'delivers', 'extremists', 'beckoned', 'dinosaurs', 'relatives', 'close-minded', "pepper's", '5-year', 'interrupted', 'bats', 'visit', 'parched', 'ethically', "bitchin'", 'flail', 'mensch', "shelton's", 'unnoticed', 'cylk', 'navigated', "jay's", "if'", 'nabs', 'discount', 'ecl@mtcts2', 'dubois', 'dolenz', "robbin's", 'liddle', 'laguna', 'dwarfed', 'as--', 'godfather-trilogy', 'mori', "ringwald's", "belushi's", 'doddering', 'moving--usually', 'trimming', 'noseworthy', 'typify', 'worthless', 'port-o-potty', 'tinges', 'unpardonable', 'mfm', 'matthieu', 'nosedives', 'daylight', 'thuroughly', 'electric', 'cole--', 'addictively', 'washout', 'providence', 'plots', 'educes', 'empty-headed', 'deliciously', 'all-too-willing', 'regiment', 'florist', 'protestations', 'heretofore', 'steely', 'renounced', 'level--it', 'romantically', 'substitute', 'casted', 'deniro', 'economized', 'pretends', 'fisburne', 'boisterous', 'january', 'sigh', 'primary', 'posters', 'somethine', 'pin-up', 'sanctioned', 'well-being', "`theory'", 'overworking', 'duguay', 'entrusted', 'mariachi', 'insistance', "sandler's", 'ten-minute', 'long-lost', 'role-reversal', 'tots', 'cities', "zallian's", 'retrospective', 'presidents', 'whittaker', 'hound', 'toenails', 'severs', 'half-grabbing', 'appalachian', 'exchanging', 'tinkertoy', 'aaaaaaaahhhh', 'elucidate', 'implausable', 'holly', 'newsletter', 'action/', 'struggles', 'promote', 'fucked-up', 'feisty', 'dundee', 'jugs', "whisperer's", 'abreast', 'claw', 'wright', "martha's", 'sockets', 'fade', 'detained', 'photojournalist', 'oldman', 'physical', 'chides', 'terrorism', 'level-headed', 'sciences', 'inherently', 'kinds', 'acquaintance', 'bollocks', 'seminars', 'opportunities', 'coarse', 'bedrock', "roberts's", 'sliminess', 'shin-high', 'advanced', 'cook', 'misunderstood', 'fronts', 'crumble', 'silberg', 'theoreticians', 'overplays', 'cow', 'reunited', "demme's", 'cloaked', 'grown-ups--this', 'procreating', 'stomp', 'eldest', 'leveling', "'off'", "willis'", 'rafting', 'fly', '1954', 'spotlight', 'pessimistic', 'semi-accomplished', 'mound', 'spoken', 'whirlwind', 'lagravenese', '\x13they', 'championships', '30-second', 'saturday-morning', 'splendid', 'freeman--is', 'ex-fianc', 'laughers', 'reyes', 'rationalize', 'godawful', 'brakes', 'post-post-feminist', '========', '_least_', 'remove', 'pratfalling', 'marceau', 'subtler', 'piles', 'cuba', 'leashes', 'foursome', 'erased', 've', 'gesserit', 'mili', 'biggs', 'evoke', 'skyscraper', 'junction', 'wider-open', 'accounted', 'animal-skin', 'pink', 'die', 'petty', 'molten', 'yapping', "resnais's", "chester's", "stu's", "ricci's", 'hairdo', 'impress', 'yulin', 'concentration', 'leathery', 'fruity', 'susie', 'dollops', "screendom's", 'scalpel', 'scariest--thing', 'harrisburg', 'fuck', 'scuzzy', 'intercepted', 'confection', 'subpar', 'attests', 'disrupting', 'landscape', 'paddles', 'rescuing', 'emotionless', 'spirituality', 'gross-outs', 'rudi', 'allegorically', 'ticking-clock', 'letup', 'bosley', 'grunge', 'charly', 'rambo-style', 'print', 'vic', 'up-and-comers', 'pop-star', 'revelations', 'mozzell', 'supersede', 'pabulum', "szabo's", 'collaboration', 'irresponsible', 'supports', 'does--and', 'gruel', 'impervious', 'quagmire', 'shit', 'middling', "koontz's", "mafia's", "counterpart's", 'adrift', "'rmd'", 'phrase', 'hen-wen', 'aussies', '--stiff', 'dany', "'traffic", 'modern', 'synthetic', 'pointe', 'bubblegum', 'traumatised', 'kingston', 'inportant', "after'", 'lisbon', "attorney's", 'emasculation', 'brims', 'melodrama', 'pronounces', 'wacky', 'mooch', 'instincts', 'co-conspirator', 'nescafe', 'finkbine', 'stand-ins', 'idea--but', 'oscar-nominated', 'establishing', "'dantes", 'entertaning', 'good/there+s', 'daze', 'roles--dedicated', 'version', 'big-', 'footsy', 'prominently', "spouse's", 'incorrectly', 'cereal', 'sometimes-odious', 'mtv-generation', 'postulated', 'impromptu', 'obstructive', 'ists', 'humanitarian', 'sixties', 'confederation', 'all-in-all', 'crispin', 'aline', 'forgetable', 'fought', 'scrounges', 'phone-pitch', 'played', 'pennyworth', 'starphoenix', 'mcdonaldburger', 'conservative', 'fled', "margaret's", '_full_house_', 'shiavelli', 'exasperated', "'o'", 'macleane', 'layoffs', 'featherstone', 'sex-kitten', 'gekko', 'appointment', 'man-animals', 'orchard', 'harwich', "reed's", 'popsicle', 'popping', 'inhaling', 'narrative', 'atrocious', "katana's", 'air-headed', 'claptrap', 'reentry', "cass's", "`pump-bitch'", 'wink', '`you', 'surging', 'necklace', 'skies', 'dolby', 'mother-daughter', 'kraup', 'five-hundred-pound', 'hill--if', 'craters', "shane's", 'actors---washington', 'non-existent', 'mosaffa', 'usurper', 'broached', 'thomsen', 'wealthiest', 'devlish', 'must-be-improvised', 'truest', 'semi-dramatic', 'not-to-be-underestimated', 'characted', 'scholarship', 'costing', 'ring-wearing', 'swelled', 'bluff', 'frightfest', 'injecting', '_wag', "george's", 'houseboat', 'abs', 'omitting', 'drinks', 'mexico--what', 'invokes', 'weddings', 'allred', '`dark', "boyle's", 'meyer', 'anti-heroes', 'margin', 'delgado', 'suprisingly', 'downhill', 'colin', 'confirmation', 'blucas', 'adds', 'controls', 'gretta', 'dispose', 'lyonne', "mpaa's", 'unlocks', 'believing', 'nestled', 'cinematographers', "hawn's", 'tent', 'prude', 'complained', 'unwritten', 'satanic', 'ensue', 'ill-equipped', 'unmenacing', 'sweetheart', 'mcarthur', 'ringed-planet', "betsy's", 'comically', 'ridden', 'watergate', '`sea', 'ninety', 'dekay', 'anti-military', 'hydraulic', 'yields', "ireland's", '53-year-old', 'meanies', 'half-blurred', 'casualness', 'days--as', 'octogenarian', 'shunned', 'stiletto', 'tannek', 'shoot-ups', 'throwback', 'bowfinger', 'roseanne', 'wissner', "amanda's", 'pot-induced', 'kooky', 'unconcious', 'much-loathed', 'ni-sen', 'neilsen', 'disposition', 'wit--', 'commissioner', 'crossroads', "loach's", 'rainier', 'victimized', 'hateful', "moses's", 'whirry', 'pitying', 'limitless', "ted's", 'reciting', 'noblest', '3=poor', 'typed', 'glassworks', "hopkins's", 'enabling', 'digestive', 'muffin', 'sharing', 'rappaport', 'vartan', 'deepak', 'potter', 'belgian-owned', 'ratings', 'autobiography', 'include', 'counselor', 'dial', 'oriential', 'repellant', 'harvest', 'audrey', 'ofa', 'bullshit', 'disconnecting', "broadway's", 'unafraid', 'faintest', 'beverages', 'defecting', 'fertilization', 'pile', 'drinking', 'crater', "android's", 'economics', 'advocate', 'age-type', 'soap-opera', 'fringes', 'subdues', 'jana', 'evidenced', 'haggard', 'rebel', 'film-within-a-film', 'coke-addicted', 'gheorghe', "don'", 'action/suspense', 'flashlight', 'almost-halfway-there', 'adolph', 'circled', 'branching', 'obituaries', 'deride', 'tombs', 'era', 'someway', 'snow-shovel', 'pates', 'macmurray', 'coolest', "narrator's", 'scent', 'yam', 'after-hours', 'viability', 'shenanigans', 'evaluation', "gale's", 'actionfest', 'manic', 'murderous', 'humerous', 'egg', 'apes', 'warhol', 'awake', 'stuff--an', 'ludicrously', 'nonchalant', 'sip', 'roof-top', 'messes', 'dials', 'pupil', 'murderers', 'ex-model', 'vivaldi', "nathan's", 'inanimate', 'commissioning', 'genetics', 'nhu', 'translating', 'contributes', 'tripped-out', 'corner\x12s', 'costumed', 'hike', 'gift', 'one-liners', 'mortally', 'supergenius', 'sutton', 'casting-against-type', 'yikes', 'crusty', 'undirected', 'sze-man', 'bother', 'whomever', 'fbi', '_shaft_', 'vividly', 'sisco', 'supercop', 'humid', 'clarkson', 'unrealistic', 'sighs', "brasco's", 'bunch\x12', "states'", '`bio', 'thwarted', 'richelieu', 'conceit', 'leans', 'stages', 'actor/director/producer', 'astonishingly', 'spreads', 'halliwell', 'munch-esque', 'paired', 'soulmate', 'm&m', 'scriptural', 'songs', 'kenny', 'disillusioned', 'wrenched', 'awaken', 'denizens', 'lapaglia', 'inadequate', 'reignited', 'ravell', 'cheap-jack', 'personal', 'ticker', 'manuel', 'screws', 'teasing', 'infinity', 'amber', 'pill-induced', "director'", 'fast', 'matured', 'exemplifies', 'canet', 'unimaginable', 'credence', 'representations', 'let-downs', 'guitar-playing', 're-shoot', "meredith's", '12-year-old', 'buckwheat', 'rednecks', 'rub', 'chant', 'eight-track', 'oriental', 'sherry', 'republicans', 'traitorous', 'scored', 'fixing', 'phenomena', 'ovation', 'master-slave', 'expirating', 'uniforms', 'idaho', 'blood-suckers', 'trucking', 'evident', 'mechanic/musician', 'secretary', 'child-like', 'houston', 'cross-over', 'first--and', 'playboy', 'wanderlust', 'unclothed', 'goddammit', 'lambskin', 'northam', 'seperate', 'listings', 'montmartre', 'apropos', 'branson', 'connoisseurs', "douglas'", 'ambitiously', 'mention', 'autopsy', 'junkets', 'philippe', 'high-fiving', "don't-hate-me-for-being-a-simpleton", 'transitions', 'satirize', 'allocated', "witness's", 'brit', 'resilience', 'inbred', 'thunderously', 'booked', 'analogous', 'accompanied', 'astray', 'nope', 'b----', 'evaporated', "lacrosse's", 'virtues', 'yankee', 'sepia-colored', 'extensions', 'perplexed', 'duddy', "sequel's", 'silliest', 'tore', 'centres', 'gradeschooler', 'beautician', 'forthright', 'austen', 'working', 'fiji', 'listless', 'wowing', 'lustfully', 'momentarily', 'quit', '--this', 'spaceballs', 'unconsoling', 'doyle', 'mohawk', 'nincompoop', 'plummeting', 'unparalleled', 'homeland', 'perennial', 'bible-thumper', 'alter', 'playing', "paxton's", 'vessey', 'elwood', 'black-and-white', 'swept', 'destroyers', 'probably-', 'umm', 'processing', 'decayed', 'farmers', 'spliced', 'carton', 'head-bopping', 'jaye', 'field', 'loftus', 'orser', 'wizard-of-oz-fashion', 'zach', 'incoherence', 'snidely', 'cynics', 'ouch--and', 'residing', 'architect', 'righteous', 'belong', 'terrestrial', 'backbone', 'steel-gray', 'considerations', 'almost-cameo', 'osemt', 'safe-keeping', 'staff', 'brilliant', 'featuring', 'amply', 'galactica', 'presenting', 'housekeeping', 'distinguishing', 'comprehendible', 'lagoon', 'spinetinglingly', 'carvey', '21st', 'barzoon', 'doubles', 'territory', 'accept', 'pettiford', 'ratio', 'terrarium', 'kale', 'armies', 'joust', "'cripple'", 'bandaras', '$21', 'all--except', 'autism', 'engagingly', 'errors', 'drescher', 'viscously', 'bottom-of-the-drug-food-chain', 'gears', 'pools', 'interest', 'razzmatazz', 'clip', 'swoosie', 'gimp', 'advocating', 'catalog', 'outdo', 'skye', 'blanchette', "jaoui's", 'disprove', 'pants', 'near-flawless', 'techno-thriller', 'uncritical', 'overly-familiar', 'realistically-written', 'chyron', 'minimal', 'inventor', 'purr', 'inconsistent', 'wow-inspiring', 'underutilized', "hoffman's", 'pot-smoking', 'good--but', 'lofty', 'email', 'target', 'notch', "washington's", '100-minute', 'greenway', 'happy--yet', 'internetters', 'yugoslavian', "verhoven's", 'tabloid', "i'm-going-to-stare-you-down", 'instinct', 'dripping', 'bodied', 'hammy', 'magnificient', 'mantra-like', 'telephone-touters', 'beck', 'wordsmith', 'gi', 'incarnation', 'psychist', 'administering', 'remained', 'thing--if', 'filimg', 'note', 'polyester', 'tuckers', "coladas'", 'ambiguities', 'england', "tati's", 'slicked', 'one-armed', 'two-year', 'dogstar', 'furiously', 'yorke', 'happen', 'awakened', "corleone's", 'rejuvenated', 'food-eating', 'lianna', 'reply-to', 'rows', 'reclining', 'lifelessness', 'distributing', 'alexandria', 'sleeper/cult', 'high-security', 'dow', "cy's", 'prior', 'vivacious', 'clever', "1980's", 'luis', 'lads', "gallo's", "macdowell's", 'hyper-erratic', 'groaner', 'flotsam', 'bothered', 'bomber', "rachmaninov's", "evil'", "'nam-inspired", 'lela', 'hopelessly', "pistol's", 'shores', 'poombah', 'who-cares', 'livens', "scott's", 'gleaming', 'industries', 'new--a', 'gentler', 'chuckle', 'uninvited', 'nudnik', 'cryo-tubes', 'moonshine', "'neck'", 'wise-', 'all-grown-up', 'endowment', 'party/rave', 'yanks', 'soil', 'funny-movies', 'sufficient', 'hallmark', 'experess', 'daggers', "chen's", "_double_team_'s", 'professorship', 'provokes', 'footages', 'guam', '/', 'crossan', 'urbaniak', 'purposeful', 'kilborn', 'bragg', 'gashes', 'kava', 'top-of-the-line', "elisabeth's", 'pfieffer', 'tootsie', 'cherry', 'penelope', 'afternoons', 'leprosy-stricken', 'wartime', 'audre', 'billie', 'compilation', 'sunday', 'soylent', 'rita', 'scientology', 'combing', 'switched', 'barlow', 'volcanoes', 'reheal', 'moo-moo', 're-doing', 'stride', 'crue', 'simulate', 'relegating', 'esoteric', 'resembling', 'puffing', 'ricky', 'guild-mandated', 'poked', '12-hour', 'marcelo', 'zhivago', 'urbanite', "krippendorf's", 'necronomicon', 'underneath-', 'brighter', "bird's", 'pontoon', 'yuckity-yuck', 'brat', 'freshly', 'ringing', 'johner', 'instructed', 'dwarfs', 'decorative', 'touched', 'superimposed', 'robert--or', "serina's", 'dethrones', 'armoire', 'psycho', 'thoughtfully', "gwaaaa's", 'fellas', 'undid', 'reflect', 'whim', 'cerebrally', 'activated', 'examining', 'swordfishing', 'vietnam', 'optometrist', 'garments', 'theirselves', 'ramses', 'lip-sync', 'accommodating', 'previously', 'gifted', "danielle's", 'davison', "taylor's", 'she+s', 'salerno', 'cursed', 'eccentric', 'burma', '40-student', 'evangelist', 'alight', 'pomegranate', 'roberts', 'films-', 'darker', 'sandra', 'arachnids', 'hooky', 'related', 'in-command', 'thier', 'sexploitation', 'unrelenting', 'radio/tv/film', 'over-whelming', "away's", 'shrewdly', "peacemaker'", 'record', 'unrealized', 'tug', 'backed', 'patting', 'sugarland', 'home-video', 'ii-era', "jamaica's", 'cui', "miracle'", 'horizontally', 'tits', 'neutralize', 'decreed', 'bizzare', "dude's", 'brennick', 'evangelicals', 'transcending', 'weirdoes', 'acerbity', 'explored', 'eventually', 'messing', 'disappointment', 'limited-circulation', 'balbricker', 'clothed', 'nifty', 'homefront', 'series-someone', 'figured', 'duncan', 'clutter', 'flip-of-the-coin', 'abounds', 'sleaze-fest', 'subtext', 'oft-played', 'paull', 'well-documented', 'uhhhhhm', 'cristofer', 'betray', 'superhero', 'cloris', "doyle's", 'siege', 'racy', 'kent', 'whiny', 'dumfounded', 'infected', 'yost', 'piers', 'arab-speaking', 'magisterial', 'waver', 'stritch', 'improving', 'jettisons', 'earnest-to-a-fault', 'sample-mad', 'less-successful', 'shambles', 'depth', "data's", 'setting', 'caustic', 'behave', 'levinson-directed', "'you're", 'rhames', 'nigger', 'att', 'splashing', 'ratttz', 'cunning', 'demolish', "doogie's", 'normandy', 'be-wigged', 'singlehandedly', 'renditions', 'hui', 'opera', 'self-talk', 'declare', 'skaarsgard', 'whores', '--dolls', 'laflour', "subway's", 'whittled', 'gester', 'minus', 'grasp', 'carrion', 'dougray', 'ratted', 'adults', 'publicity', 'bring', 'peels', '$91', 'yo-yo', 'dating', 'handicap', 'elektra', "thomas's", 'exalted', 'continents', 'shoveling', 'mediterranean', 'gap', "luc's", 'modicum', 'rolle', 'intellectually-challenged', "jack's", 'lay', 'espousing', 'phoenix', "neary's", 'thermia', 'bower', 'skylar', 'life-saver', 'fading', "kline's", 'deafened', 'vexing', 'unsatisfactory', 'miscarriage', 'ers', "caine's", 'robards', 'soviets', 'lear', '16', 'bred', 'spectrum', 'alliance', 'montgomery', 'democratic', 'half-decade', 'crowned', 'tracts', 'hideout', 'ja', 'cabbies', 'indifferent', 'context', 'inhibitions', 'stroking', 'guttenberg', 'dutton', 'behaviour', 'planning-to-retire', "star'", 'fungus', 're-popular', 'owen', 'maneuvers', 'scissor-happy', 'peephole', 'enterprise--commander', 'goddaughter', 'congresswoman', 'pulitzer', 'lacked', 'hypocrisy', "chambers'", 'libel', 'lionel', 'pressures', "hollywood's", 'latifa', "song'", 'suri', 'jazz', 'reece', 'polite', '102-minute', 'haystack', 'dining', 'livened', 'boiled', 'ruritanian', 'miles', 'persistently', 'gos', 'comebacks', 'dimensional', 'harshness', 'foot-massage', '+', "parr's", 'kneecap', 'securing', 'dialouge/short', 'brooklyn', 'faze', "o'brien", 'turned-on', "else's", 'shortest', 'peacemaker', 'sid', 'fielding', 'vander', 'excerpt', 'personalizes', 'bigwigs', 'cassette', 'spruced', 'eschews', 'sift', 'signifying', 'eriksson', 'thrash', 'willis', 'gads', 'walt', 'eigeman', 'underscore', 'double-edged', 'precautions', 'thieveing', "arthur's", "siam's", 'interest/chambermaid', 'caulder', "jam's", 'alvarado', 'nolot', 'victorian', 'stringing', 'rigs', 'fumbled', 'derivitive', 'clarified', "1973's", 'pringles', 'off-screen', 'quizzical', 'usage', 'shapeshifting', 'selfless', '206', 'horned', "curry's", 'vegans', 'dialogue-', 'financially', "eyes'", 'dismissing', 'corey', "madonna'", 'ailments', 'revered', 'character-heavy', 'miniatures', 'florence', 'selma', 'nirto-est', '1/10', 'brock', 'schlock', 'intentional', 'pebbles', 'serves', 'emil', 'fatalistic', 'modest', '1971', 'hazy', 'twentysomethings', 'cybernetic', 'tastefully', 'eddie', 'slots', 'tied', "1976's", 'excessive', 'warm-heartedness', 'bigger', 'decaprio', "zardoz's", 'honeywell', 'quasi-nutritional', 'caftan', 'conscious', "bug's", 'maaia', 'accomplished', "colors'", 'leagues', 'baser', 'cubs', 'mantra', 'oscar-bait', 'classification', 'lambs', 'now-deceased', 'spurred', 'word-of-mouth-proof', 'schumaccer', 'oddly--who', 'non-baseball', 'fairly', 'chuckster', 'asylum', 'bolts', '1/20th', 'consist', 'xusia', 'dragon-protected', 'absent-mindedness', '1925', 'tour', 'red-tinted', 'goetz', 'longing', 'afterlife', 'old-guy', 'cranking', 'mysore', 'dubious', "d'oeuvre", 'parachutes', 'race', 'mcdowell', 'useless', 'singh', 'exuberant', 'braindead', 'deprophesized', 'werewolfism', 'pimple', 'bludgeoning', 'doomsaying', 'proceeders', 'hara-kiri', 'beijing', 'divorcing', 'reccomended', 'overlord', 'soiled', 'checked', 'roughly', 'criticizing', 'pandering', 'tramp', 'breaks', 'chap', 'millionaire/crocodile', 'brunettes', 'rocque', 'facility', 'pretense', 'arielle', 'alone_', 'rastafarians', '1960s', 'trese', 'lensky', 'arsenic', "rice's", 'digesting', 'macchio', 'hardee-har-har', 'dan', 'stereotypes', 'alternatively', '-inspired', 'pluck', "'blair", 'jubilation', 'josef', 'predator--all', '3-year', 'resonating', 'highs', 'oscar-caliber', 'bosoms', 'printing', 'kahn', 'tree-covered', 'quarters', 'excuses', 'misfires', 'bastards', 'otter', 'wares', "comics'", '12-part', 'virtue', "five's", 'uplifts', 'daisies', 'leaked', 'mohr', "someone's-out-to-get-me", 'hell-bent', 'dolphin', "base's", 'meticulous', 'nielsen', 'misrepresenting', 'susan', 'ardour', 'flirt', 'haven', 'endurance', 'gulp', 'grifters', 'wesley', 'miscast--as', 'glow', 'cahoots', 'pridefully', 'routines', 'bedridden', 'concurrently', 'fittingly', 'progress', 'non-acting', 'founded', 'villainess', 'hushes', 'noboyd', 'bombs', 'forgotten', 'recognizes', 'tri-annual', 'adulation', 'guys', 'recombination', 'baring', 'spirit-dead', 'aria', 'depend', 'cuddly', 'cappers', '19-months', "smilla's", 'recruit', "kazantzakis'", 'teen', 'retells', 'over-simplifies', 'illustrated', 'hodge', 'bailey', 'coolidge', 'believers', "victoria's", 'infiltrating', 'pared', 'cormack', "'freaky", 'tenet', 'jaleel', 'guamo', 'smells', 'twisters', 'newly-acquired', 'paid', 'milking', "5'2", 'col', 'scraggly', 'alternated', 'authorized', 'matters', '38th', "sweetback's", 'blah', 'action-hero', 'crammed', 'reflections', "lucas's", "zemeckis's", 'astride', 'temperamental', '20thcentury', 'helmed', 'oscar-', 'margulies', 'ace-in-the-hole', "tim's", '_disturbing_behavior_', 'couple-ness', 'worsening', 'indicative', 'award-winner', 'non-comedic', 'jun', 'yessssss', 'bowl', 'chandler/ross', "2'", 'single-mother', 'pessimism', 'armi', '--slacker', 'hypothalamuses', 'implying', "pi's", 'organization', 'dictates', "pfieffer's", 'deflect', 'propellers', 'misadvertised', 're-claim', 'theron', 'feverishly', "bride's", 'used-up', 'atms', "willard'", "jarvis'", 'brains', '$75', 'foregone', 'bella', 'vantage', 'pollutes', 'loudly', 'falter', 'love-triangle/revenge', 'unsecure', 'bigger-than-life', 'korman', 'perpetrator', 'vomitted', 'strained', 'montenegro', 'essentials', 'peri', 'unsheathes', 'brain-zapped', "face'", 'gap_', 'broadcasting', 'violet', 'stones', 'criterion', 'talia', 'epilogues', "boorman's", 'revival', 'pan-and-scanned', 'dunked', 'strawberry', 'loomed', 'scandals', 'mol', 'sharpe', 'melting', 'wilder', "eno's", 'suit', 'marveled', 'conscript', 'flinch', 'droney', 'differing', 'best-selling', 'hi-lo', 'exhibited', 'colm', "'70s", 'assassinates', "alan's", "hoenicker's", 'hamstrung', 'gadgetry', 'passport', 'writer/director/producer', 'strikes', 'drearily', 'chariot', 'cellulite', 'campfire', 'poignantly', 'primarily', 'reservation', 'spar', 'compare', 'tunie', 'waaaayyyyyy', 'ritualistically', 'regularly-updated', 'sonatine', 'isaiah', 'falco', 'hippie', "anything's", 'tycoon/villain', 'stationmaster', 'engines', 'nauseating', 'geisha', 'comic-relief', 'dobie', 'rowdy', "might've", 'jerri', 'riff', '`strange', 'honors', 'anti-appeal', '3', 'ick', 'outdoor', 'urinary', "roman's", 'crafted', 'mckee', "sandra's", 'fly-by', 'revivals', 'foreign-exchange', 'proudest', '10-year', 'posted', 'sorts', 'clunky', 'seemd', 'loli', 'glossed', 'grown-ups', 'jones', '#1', 'sea-bound', 'inset', 'whips', 'kethcum', 'joss', 'over-emotes', 'militaristic', 'refrains', "shoudln't", "drew's", 'kleinfeld', 'quotations', 'commandos', 'electro-pop', 'susceptible', 'implementation', 'policed', 'goldwyn', 'pernilla', 'interrelate', 'levying', '007-esque', 'humanized', 'ways---and', 'rousing', 'noirish', 'maladjustment', 'platoon', 'reflected', 'waterfront', 'wane', 'admitting', 'repeat', "scorsese's", 'cinergi', 'laxity', 'bussotti', 'efficiency', 'revolucionario', 'kilronan', 'self-ridicule', 'subjective', "'serve", 'sing', 'unmatched', 'unfeeling', 'ex-manager', 'conscripted', 'non-cartoon', "azaria's", 'yourself-but', 'willpower', 'information--something', 'de', 'neglect', "bulow's", "cleon's", 'dissappointed', 'disasterous', 'marcellus', 'headey', "myer's", "'investigation'", 'hotshot', 'ferrara', 'brazen', 'effervescently', 'believeable', 'meals', 'oxide', 'cutest', 'bafflingly', 'cyberspace', 'wanking', "d'aragon", 'restore', 'humble', "henriksen's", 'reflects', "'snuff", 'brutalized', 'periods', "luigi's", 'mesh', 'measurements', 'blurting', 'palotti', "zhuangzhuang's", 'branches', 'heated', 'hashish', 'stalwart', 'intolerable', 'emotional', 'youngers', 'innocent', 'caught', 'ericson', "wagner's", 'disasterously', 'puttingly', "mind's", 'galeweathers', 'master-mind', 'rizzo', "alana's", 'probably-never-will-bes', 'smartass', 'neighbors', 'scenes', 'barely-in-control', 'inquiries', 'monuments', 'numbered', 'roast', 'scorese', 'skimpy', 'phifer', 'pogue', "carey's", 'sherri', 'robinson-like', 'tie', 'film-makers', 'itty', 'trekkie', 'take-off', 'materials', 'draven', 'geri', 'gymnastics', 'spears', 'currents', 'humoring', 'doss', 'enlivening', 'squandered', 'garcia', "dicaprio's", 'counterbalanced', 'in-side', 'ultra-conservative', 'bluto', 'flighty', "`cooks'", 'nubla', 'mustard', 'limps', 'trampoline', 'benefactor', 'ryanson', 'culminates', "garth's", 'pratt', 'module', 'jellylike', 'magnum', 'pourkarim', 'well-to-do', 'patriot', 'herbert', 'dicing', '180', "`beautiful'", 'cute', 'imbecilic', 'symbolically', "murnau's", 'pigeon', 'hanover', 'handgun', 'obscenity', 'michigan', "nadia's", 'starring-liam', 'texts', 'shown--like', 'commodity', 'again--these', 'excises', 'stopped', 'whitmore', 'brainard', 'hooker', 'tsang', 'serenity', 'come-back', 'weeks', 'metatron', 'walken', 'gang-bangers', 'apparel', "michelle's", 'voil', 'lavatory', 'ti', 'stirs-he', "shostakovich's", 'pray', 'hudsucker', 'zipper', 'demornay', 'city-boy', 'fullyloaded', 'perfume', 'ear-shattering', 'tumbled', 'enforcing', 'interned', 'sorrowful', "rosalba's", 'gotta', 'paneled', 'calrissian', 'texan', "sorcerer's", 'shrink', 'amorality', 'vcrs', 'humbug', 'sats', 'role-shifting', 'cryogenic', 'mcmanus', 'competency', 'punches', 'hairnets', 'navigate', 'chisholm', 'registers', 'gosh', 'red-hot', 'ringleader', 'seann', 'cozy', 'interaction', 'overdone', 'safer', 'disfiguring', "ramsey's", 'quatto', 'barkin', 'oversimplified', 'asteroids', '_never', 'sayinig', 'coldly', 'sur-prise', 'tortured', "gridlock'd", 'spend', 'demeaning', 'swordfish', 'zallian', 'commencing', 'ordinary--premise', 'homestead', 'installment', 'breckman', 'unnecessary', 'impersonal', 'reversed', 'good-heartedfrom', 'pot-obsessed', 'carved', 'non-story', "'that's", 'shrimp', 'proportionate', "week's", 'zaniness', 'regulate', 'weed', 'intestines', 'forster', 'uppers', 'idealistically', 'arrests', 'herbie', 'preceded', 'doc', 'bouts', 'landings', 'christmas', 'paths', 'appears', 'concentrated', 'contribute', 'introvert-boy-falls-for-extrovert-girl', 'jesus', 'short-lasting', 'writing-directing', 'feature--is', 'tampopo', 'carry', 'leak', 'aristocrat', '6=better', 'inherit', 'okeday', 'merciful', 'rally', 'envisions', 'candlewax', 'radical', 'hug', 'spiel', 'demands', 'make-over', 'tree-in', 'arduous', 'handlebar', 'slaughterhouse', 'westlake', 'empathetic', 'pawns', 'ninjas', 'banal', 'tutors', 'swoop', 'troweled', 'skates', 'unprepared', 'ibanez', 'washed', 'live-action', 'seidler', 'blackboards', 'overseeing', 'allied', 'walls', 'betti', 'gilfedder', 'declarations', 'rental', 'overcooked', 'lision', 'bus--but', 'rio', "rudolph's", 'exacerbated', 'yesterday', 'rv', 'drifter', '3/10', 'cliques', 'mcclain', "season's", "this'", 'ungar', 'turns', 'trucks', 'panhood', 'reptiles', 'revels', 'singularly', 'b-', 'energetic', 'little-to-no', 'grieco', 'nymphet', 'binge', 'affirm', 'feverish', 'jere', 'shrug-of-the-shoulders', 'arqua', 'bargains', 'disposes', 'seesaws', 'oral', 'cosmo', 'kingdom', 'unconventionally', 'stymied', '`real', 'hoover', 'pose', 'hatred', 'teachers', 'successes', 'looters', 'catchphrases', 'wormwoods', 'pants-wetting', 'sophistication', 'arranges', 'solid', 'drinker', 'scam-artist', 'forman', "`tuning'", 'sexbomb', 'autos', 'buy-more', 'indignity', 'cringeworthy', 'arcade', 'stonecutters', "eddie's", "'good'", 'carefully', 'barrage', 'tony', 'inanely', 'menken/david', 'jhvh', "belong'", 'hire', 'silly-looking', 'selflessness', 'virile', 'huston', 'lashing', 'stooge', 'redolent', 'induction', 'moviemaker', 'barmaid', 'breast', 're-delivering', 'walnuts', 'vulgarity', 'customer', 'rumbles', 'jumbles', 'geneticist', 'quickly-made', 'makings', 'brasco', "everett's", 'fundamental', 'marxist', 'tomei', 'perpretrators', 'discredited', 'persuade', 'shear', 'crowd', 'armpit', 'espouses', 'perfuctory', 'ments', 'scrambling', "england's", 'boggingly', 'exhibits', 'disorganization', 'jonas', 'girl', 'ariane', 'chien-po', 'half-drunk', 'cart', 'bayou', 'mil', 'indie-underground', 'tables', 'ingrid', 'outer', 'glut', 'osnard', 'shockwave', 'bands', 'working-class', 'hi-tech', 'hysterics', 'eastwick', "parks'", 'tempt', 'hideously', 'delacroix', 'and-miss', 'refer', "o'daniel", '--schumacher', 'marble', 'victimisation', 'superior', 'lovett', 'invent', 'garret', "'rules", 'commie', "'rome", 'arsed', 'trickiest', 'gender-bending', "boy's", 'unorthodox', 'fault', 'wah', 'elses', 'cole', 'concealed', 'blow', 'recreates', 'rube', 'stardom', 'souk', 'treasures', 'spill', 'interacts', 'brokers', 'big-underground-worm', 'driving', 'bong', 'snipers', 'tuttle', 'one-hundred-dollar', 'cows', 'drizzling', 'aboard', 'brosnan', 'dysart', "o'neal", 'opting', 'semi-introspective', 'populating', 'kala', 'take-off--', "manipulations'", 'wows', 'shrouded', 'break-in', 'pioneer', "apartment's", 'moons', 'gaping', 'prototype', 'ernest', 'war', 'easiest', 'lawsuit', 'monsoon', 'fuentes', 'hostess', '[her', 'creepier', 'buttons', "millionaire's", 'repression', 'majesty', 'shor', 'ne', 'strategy', 'exhale', 'beals', 'pas', 'entei', 'ate', 'titled', "rocco's", 'plumets', 'monomaniacal', "memphis'", 'cheapo', 'ridding', 'conducive', 'stopwatch', 'characters--in', 'game-pod', 'buyer', 'vice-versa', 'homeboys', 'pagniacci', 'non-actors', 'near-sweetness', '`rocky', 'transvestite', 'asserted', "microphone's", 'shite', 'developing', 'gazers', 'redundant', 'ezio', 'fortune', 'siberian', 'bartkowiak', 'overact', 'circular', 'permutation', 'hors', "caveman's", 'baigelman', 'healing', 'animals--', 'implicates', 'uncommon', 'rat-like', 'millius', "sawa's", 'relieved', "'yawn'", 'trashcan', 'hubris', "emily's", 'analyst', 'peaceably', 'mcneill', 'painting', 'preppie', 'implementing', 'stove', 'backpacks', 'assassination', 'invlove', 'unrest', 'martians', 'materialize', 'historic', 'collided', "huntingburg's", 'babbles--she', 'swastika-wearing', 'airborne', 'throghout', 'mike', 'basics', 'acuity', 'kermit', 'hasn', 'casserole', 'damage', 'routes', 'exercise', 'flow', 'laments', '50', 'mikhail', 'demurely', 'bison', 'singer/lover', 'mind-twisting', 'night', 'vote-pandering', 'phillipe', 'fragments', 'carrera', 'tissues', 'suits', 'flaccid', 'booth', 'impactful', 'career-threatening', 'gold--when', '18-wheeler', 'testament', '/cinematographer/editor', 'actor/stunt', 'beaufoy', 'mysteriously', 'full-frame', 'now-famous', 'whispered', 'gussies', 'fussy', 'summoned', 'mclachlan-aided', 'krajeski', 'improvisation', 'thriller', 'chimpanzees', 'miseries', 'pierre-augustin', 'unhip', 'rocher', 'courts', "mitch's", 'sacramento', 'headly', 'to-back', 'ice-t', 'comedies', 'sub-themes', 'mid-teenage', 'marie-sylvie', 'stateside', 'metaphysics', 'stint', 'anton', 'mortified', 'eyro', 'mademoiselle', 'excerpts', 'boomer', 'underscores', 'bogart', 'colqhoun', 'trappers', "reynolds'", 'best-friend', 'leviathan', 'hoffmann', 'dorks', 'revisionist', 'luridly', '-films', "nicholas'", 'astronauts', "'roles'", 'two-hours', 'consumatory', 'backstabbers', 'blaxploitation-era', 'grossing', 'exchanged', 'over-explain', 'esther', 'politician', 'scooping', 'lesra', 'risqu', 'raiment', 'cheapskate', "'either", 'offshoot', 'wistful', 'hipper', 'enlisted', 'coloring', 'cowboy', 'electromagnetic', 'three-hour-tear-fest', 'rewind', 'name--', 'unconsummated', 'bondsman', 'up-and-up', 'drowned', 'kafka', 'glendale', 'slayings', 'declining', 'boulevard', 'blackface', 'ease', 'outspoken--perhaps', 'reserved', 'inhabitant', 'ah', "sena's", 'nothing--least', 'reuben', 'plainly', 'heightening', 'boom', 'deadly', 'heather', 'zak', 'fully-', 'mismatch', 'cosmetics', 'save-the-earth', '2hr', 'ejecting', '=======================', 'tales', 'educate', 'intercut', 'prodigious', 'baking', 'mentioning', 'sockful', 'worrys', 'plead', 'unbelievably', 'antihero', 'creation-', 'conning', "fiction's", 'encompassing', 'boban', "'staple", 'slows', 'bile', 'crucifixtion', 'insurmountable', 'hop', 'maria', 'russian-sounding', '7-13', 'persia', 'excitements', 're-introduced', 'cryptically', 'stockwell', 'not-guilty', 'kenner', 'skits', 'infertility', "forman's", 'media', 'ho-hum', 'drove', 'pelt', 'delivers--meaningful', 'acted', 'there\x12s', 'muttered', 'megahit', 'leeper', 'populus', 'covenant', 'cooney', 'engine-boy', 'dished', 'begbie', 'mill', 'betting', "caulfield's", 'sweating', 'capabilties', 'deftness', 'hu', 'owe', 'dissenting', 'eminent', 'fraternity', 'quaid', 'torrance', 'non-committal', 'encompass', 'compton', 'drowning', 'heart-tugging', 'corny', 'ladders', 'obese', 'dussander-much', 'promised', 'deploy', 'skellington', 'locations', 'explosion', 'on-stage', 'kimmy', 'policia', 'self-deprecating', 'rambling', 'ramifications', 'cowards', 'tomorrow', 'adolf', "'rekall'", 'mother-to-be', 'inconvenience', 'pearly', 'strikingly', 'nasa', 'galaxies', 'feeding', 'evil-doer', "billy's", 'softened', "`i'm", 'mechanically-challenged', 'dixie', 'catharsis', 'differ', 'salieri', 'disneyfied', 'everyone--you', 'mant', 'salminen', 'projecting', 'triple', 'limited_', 'upstanding', 'insufficient', 'derive', 'schticks', 'hemisphere', 'forensic', 'sheffer', 'fluff-piece', 'reacts', 'superceded', 'fruit-cake', 'yawn', "freeze's", 'situation', 'flash-card', 'auditor', 'queens', 'wigger', 'luxuriantly', 'actor', 'fanatics', 'spouses', 'beasties', 'erotic-thriller-cinemax-style', 'overexaggerant', 'supervising', 'pseudo-science', 'abrupt', "silverstein's", 'date', 'vaults', 'remembered', 'stabilize', 'streetcorner', 'hmmm', 'stipulate', 'peers', "lindo's", 'pimping', 'goose', 'specific', 'rolled', 'glen/glenda', 'underhanded', "80's", "bernie's", 'professionals-miramax', 'triangular', 'blythe', 'extorts', 'unemployment', 'telling', 'shipwreck', 'important', 'over-dramatic', "1986's", 'mental', 'quest', 'joanou', 'clique', "strangelove'", 'grader', 'ape-trader', "'28", 'previously-used', 'ahmad', 'balls', 'beer-drinking', 'sub-titled', 'zahn', 'sand', 'kanoby', 'riffing', 'defeaning', 'spall', 'gales', 'changwei', "spanner's", 'unions', 'lodge', 'guenveur', 'preferable', 'stephan', 'amour/last', 'fellow', 'cell', 'concocts', "faltermeyer's", 'tsk', 'intimacy', 'deejay', 'richest', 'gonz', 'wrapper', 'indulge', 'exit', 'cambell', "packer's", 'freaks', 'abstraction', 'depictions', 'caddies', 'ex-patient', '_amadeus_', "stahl's", 'bludgeoned', 'reunites', 'motivated', 'jarrett', "moroder's", 'russian-german', 'trepidations', 'blistering', 'persist', 'possibilities', 'ozone', 'swanbeck', 'fabricate', 'butabi', 'battlestar', 'kirsten', 'libidos', 'fathom', 'structure-wise', 'decoration', 'kitano', 'affability', 'antique', 'sciora', 'fraternizing', 'unsubstantial', 'scenarios', 'mishandled', 'clamp', 'elaine', 'politicians', 'yuri', 'balding', 'grownups', 'guts', 'taiwanese', 'drilled', 'payrolls', 'sizable', "hughes'", 'circuits', 'storytellers', 'ruckus', 'stick', 'subterfuge', 'pre-teens', 'sought-after', 'astute', 'snider', 'fish-out-water', 'aversion', 'now-estranged', 'widower', 'topical', "cornwallis'", 'inroad', 'friggin-nobi', 'differentiation', 'strewn', 'oriented', 'drunken', 'darkly', 'maher', 'leila', 'fetch', 'tertiary', 'mia', 'allergies', 'laszlo', 'affleck', "jovivich's", 'tile', "shadyac's", 'mousehunt', 'heroism', 'privates', 'speedy', 'reasoning', 'hildago', 'affiliate', 'tolan', 'ozzy', "fingal's", "`flaunt'", 'unheard-of', 'compromises', 'flirtatious', 'poker-playing', 'badu', 'unstructured', 'anothergreat', 'pines', 'all-or-nothing', "jude's", "'80s", 'renting', 'cleared', 'bucketloads', 'unlikable', 'brigantine', 'scrambles', 'lifer', 'levity', 'over-weight', 'mines', 'sinfully', 'pointlessness', 'divorced', 'playground', 'tributary', 'saber', '50s', 'supernova', 'lynch-esque', 'vetter', '--much', 'attention-getter', 'crouse', 'kinda-bitchy-in-a-reserved-way', 'kidnap', "d'lyn", 'selective', "foreigners'", 'chalk', 'installs', 'one-piece', 'regional', 'katz', '1926', 'model-citizens', 'cinema-wise', 'tightfitting', 'spurs', 'mannerisms', 'smoggy', 'bin', "arc'", "nest'", 'interlinked', "bob's", "heinlein's", 'ross', 'thurman', 'started', 'three-digit', 'newell', 'isabella', 'ansell', "'stealth", 'brautigan', 'underwhelmed', 'swordplay', 'flashdance', 'turkish', 'tram', 'centrestage', 'settlers', 'garnering', 'schlumpy', 'conquering', 'snide', '24th-century', 'venable', 'slotted', 'jealously', 'mice', 'drunkenly', "harlin's", 'history', 'fergus', 'sky', "hemingway's", 'collapse', "nbc's", 'geez', 'descending', 'stone-faced', 'peer', 'ill-advised', 'next-to-next-of-kin', 'ponytail', 'deactivating', 'restaurants', 'hanson', 'b-production', 'tipsy', 'feedback', 'mesquida', 'already-beautiful', 'sobieski--watch', 'last-ditch', 'attains', 'terri', 'ice-cold', 'propositioned', 'topsy-turvey', 'manouvering', 'advil', 'self-surgery', '144', 'emerson', 'innovatively', 'jacks', 'novice', 'technicians', "warren's", 'zeke', 'shui', 'pinto', 'caddyshack', 'entangled', 'remodeled', "walken's", 'eerily', 'unimpressed', 'houyhnhnms', 'sharon', 'sit-coms', 'gratification', 'bladder', 'parallelism', 'mass-produced', 'fictionalized', 'trespassing', 'extols', "'stand", '_experience_', 'automatic-pilot', 'court-room', 'reestablishing', 'beginning', 'blown-out', 'giggling', 'schlictmann', 'wonderment', 'skateboard', 'visitation', '_will_', 'including', 'mutually', 'punk-noir', 'novelistic', 'ungainly', 'holdover', 'phone-sex', 'delegates', 'ellen', 'squished', 'cannibalism', 'virility', 'un-safe', 'hill', 'someting', 'lucrative', 'mule', 'plights', 'matalin', "movies'", 'tour-de-force', 'dharma', 'stucker', 'shielded', 'contrasted', 'ethnicity', 'nit-picky', 'berkowitz', 'unreleased', "thewlis'", 'pope', 'decidedly', 'creed-miles', 'craven-directed', 'wife', 'unequivocal', 'wife--and', 'kellogg', 'zippy', "kristen's", 'paves', 'pasties', 'philosophizing', 'alien/human', 'art-imitates-life', 'aurelius', 'showed', 'lifeforms', 'reef', 'reaching', 'off-the-cuff', 'measures', 'hulce', 'airplanes', 'inventiveness', 'eschew', 'sudden', 'war-', 'observation', 'widening', 'painter', 'excrucitating', 'sluttier', "/let's-strip-down-the-sport-to-its-bones", 'already-existing', 'nook', 'refrigerator', 'monologue', 'satyrical', "sluizer's", 'yim', 'lens', 'atheism', 'sat', 'brubaker', 'romane', 'legalized', 'repetitively', 'dutch', 'titular', 'tutor', 'parodied', "caan's", 'frustrated', 'ravich', 'leit', 'asner', 'circumstance', 'fink', 'handlers', 'fried', 'hour', 'beleive', 'brawl', "contact's", 'recreated', 'excusable', 'nitrous', 'whimpering', 'jerk-ules', 'spy', 'ferry', 'marty', 'surround-sound', 'iraqi', 'chews', '#', 'langer', 'exceptional', 'crooning', 'anti-apartheid', 'disclosed', 'prevalence', 'melvin', 'surrendering', 'seminar', 'bacri', 'benicio', 'ali', 'passes', 'entry', 'shepherd', 'wrestled', 'marketable', 'gar', 'violently--that', 'urbano', 'intermediary', 'state-sanctioned', 'saws', "chinatown's", 'smashed-up', 'butoy', 'graff', 'debased', '`big', 'peat', 'saim', 'rediscover', 'ying', "bentley's", 'imagines', 'pesudo-pseudo-pseudo-character', 'tactical', 'self-discoveries', 'trannsferred', 'phil', 'hide-out', 'petition', 'matt', 'adaptations', 'popcorn', "lawrence's", 'impression', 'ragtag', 'care', 'comedy/romance', 'milks', 'best-looking', 'hawn', 'apparitions', 'hitch-', 'taylor', "gloria's", 'frees', 'swope', 'hubbard', 'head-strong', 'demure', 'arsenio', 'pimlico', 'cronenberg', 'g-friend', 'bitching', 'commentaries', 'emotion-charged', 'orion', 'curvaceous', 'pulman', 'territories', 'nurturing', 'dunn', 'tele-marketer', 'wrongdoings', 'animals', 'michelle\x12s', 'atrophied', 'henson', 'scrap-pile', 'furthers', 'years', 'clout', 'alert', 'totemic', 'flippers', '14', 'second-chance', 'action-adventure', 'laying', 'entanglements', 'carefully-crafted', 'fell', 'swift', 'jurek', "dinsmoor's", 'reilly', 'zodiac', 'zegers', 'horror/comedies', 'trendy', 'griswold', 'thesis', 'cropduster', 'overloaded', 'chord', 'kremp', 'impressions', 'heart-sweeping', 'bitchiness', 'craftily', 'semi-darkness', 'suffered', 'grudge', 'gentle', 'rehashed', "minelli's", 'drug-infested', 'tightly-wound', 'benzali', 'reverential', 'non-fiction', 'smashingly', 'cellist', 'development', 'eighties', 'po', 'playback', 'unbeaten', 'maxime', 'beloved', 'bureau', 'swooping', 'retalliation', 'split', 'tumor', 'hunchback', 'college-bound', 'musically', 'tyler', 'lip-synch', 'crossgates', 'recreations', 'anyway--yep', 'constipated', 'geographically', 'sonia', "nietzsche's", 'individuality', 'streaking', 'evolutionary', 'squaddie', 'doesn', 'honourable', '_doctor_dolittle_', 'uninhabitable', 'geologist', 'terms', 'oversentimental', 'bragging', 'burns', 'self-destruct', 'substitution', 'granny', 'chemical', "hours'", 'throbbing', 'dictatorship', 'sake', 'scaring', 'intercom', 'banderas', 'symbolic', "loki's", 'disjointedness', 'swiss', 'mousse', 'gathered', "clancy's", "clang's", 'deportee', 'consequences', 'eradicated', 'mutinies', 'helecopters', 'manufacture', 'consenting', 'lsd', 'nowheresville', 'cherie', 'counterbalance', '-the', 'mace', 'tuxedos', '44', 'copulation', 'tings', 'adopted', 'kops', 'lohan', 'taboo-smashing', 'dazzling', 'schultz', 'jacqeline', 'slow-motion', 'woodside', 'police-woman', 'complacent', 'wordy', 'highly', 'parfitt', 'painkiller', 'truer', 'evaporating', 'stay', 'argentina', 'sfpd', 'gottfried', 'cracraft', 'superseventies', 'stevens', 'boo-hoo', 'loaned', 'woman-bashes', 'caption', 'scumbagginess', 'snowbound', 'das', 'copybook', 'girl-friends', 'guillotine', 'broods', 'rouen', 'princeton', 'lew', '---agent', "dwayne's", 'misfire', 'eyelids', 'enforce', 'lovably', 'lindas', 'scheduled', 'diagnosed', "`colorful'", '_holy_man_', 'writing/directing/', 'sh', 'computer-enhanced', 'fiefdoms', 'block', 'lieu', 'ordell', "heckerling's", 'donald', 't-rex-i', 'clubs', "[editor's", 'tight-jawed', "austin's", 'ewwwww', 'invincible', 'mart', 'bread', 'films', 'sacrificed', 'recurs', 'solitude', 'neglects', 'exacts', 'sandrine', 'murder-mystery', "fiance'", 'digestions', 'vienna', 'distressed', 'stunted', '$280', 'brewing', 'dateless', 'haywire', 'prohibition', 'telss', 'rumor', 'slaparound', 'tolls', 'fogy', 'unpredictability', 'meteorologist', 'bink', 'selfishness', 'bins', "giger's", 'atm', '[katarina]', 'ninth', 'short-on-substance', 'eloi', 'unobjective', 'beliavable', "walter's", "jeanne's", 'impossible', 'propels', 'devises', '50s-ish', 'odds-maker', 'tasha', 'smalltown', 'mania', 'whaley', "gwtw's", 'yuh', 'miranda', 'evaluate', 'tibetans', 'lightweight', 'octopussy', 'paraphernalia', 'heavy-on-fx', 'modes', "'no'", 'eaddress', 'lester', 'dana', 'half-a-million', 'stated', 'yawner', 'behemoth', 'arcand', "roy's", 'unintentionally', 'oberon', 'accidently', 'pig-pig-pig-pig-pig', 'stopping', 'braxton', 'title--first', 'careen', 'valjean', 'foodmart', 'grinding', 'weak', 'nuts', 'neo-nazi', 'ill-timed', 'companionship', 'holbrook', "violation'", 'that\x12s', 'checking-in', "bening's", 'demesne', 'flew', 'tricky--', 'peter', 'cicero', 'typist', 'fighters', 'bloodier', 'smarmy', '`this', "'basic", 'party-girl', 'overlight', 'obserable', 'interests', '`american', 'rifling', 'designer', 'incredibly', "melvin's", 'guard', "carl's", 'oneness', 'plush', 'hollywood-conventional', 'time-consuming', 'felines', 'pubbing', 'in-heat', 'alexa', 'abolitionists', 'goers', 'junket', 'scantly', 'rubin-vega', 'psychiatrists', 'office', 'obsequious', 'preview', "o'neill", 'annoyingly', "leia's", 'quimby', "kombat_'s", 'hymns', "`goodfellas'", "'help", 'steve', 'co-starring', 'undoubtably', 'boling', 'telefixated', 'above-par', 'posing', 'peck', 'spewed', 'inmates', 'pekurny', 'brusque', 'shanta', 'chauvinistic', 'snap', 'horrid', 'lassez', 'recipients', "1996's", '`hanging', 'downloading', 'choir', 'venereal', 'fearful', 'miracle', "horse's", 'planetary', 'disquieting', 'carmel', 'punks', 'lampooning', 'abstract', 'heavens', 'notches', 'prehistoric', 'chase-riboud', 'streak', 'hitting', 'parsons', '5-year-old', 'gestapo', "ghostface's", 'amberlike', 'burger-flipper', 'foreign', 'defected', 'dizzying', 'notebook', 'vocabulary', 'deutch', 'chicanery--runneth', 'live', 'vibrance', 'cornish', 'overweight', "student's", 'mutters', 'graduated', 'garlic-filled', 'blockubuster', '-my', 'anecdote', '1982', 'six-month', 'rosette', 'adverse', 'emotional-radiance', 'stone', 'naunton', 'hotel_', 'environmentally', 'armand', 'married', 'politely', 'worshippers', 'overheard', 'raided', 'ascetic', 'magruder', 'aspirations', 'spooks', "millius'", 'stalked', 'orphange', 'zealous', 'fun-yet-dumb', 'conversation', 'narratively', 'rocketry', 'racking', 'norman', 'mcmurray', 'pedagogical', 'emptor', 'unpolished', 'party-pooper', "smith's", '&quot', 'nihilistic', 'abby', 'sidetracked', 'ehrin', 'effect', 'dangling', 'store', 'comet', 'durable', 'funnier', 'invades', 'swede', 'upheaval', 'nits', 'incompetent', '106-minute', 'polti', 'scaling', 'cloony', 'stately', 'beancurd', "debbie's", 'robert', 'smoother', 'gen-x', 'staging', 'anti-', "flynn's", "tee--it's", 'donnas', 'displaying', 'denver', 'rhode', '21-year', 'floodwaters', 'ammo', 'dalmatians', 'unfairly', 'numbingly', 'jew', 'planet', 'eliott', 'cast--most', 'desparation', 'knowing', 'ungodly', 'landscapes', 'matthau', 'coverage', 'round-the-world', 'submits', 'lubricants', 'rome--but', 'half-interested', 'shuttles', 'cop', 'facepaint', 'thrilled', 'mix', 'formative', 'couriers', 'lemons', 'miraculously', 'hot-enough-to-melt-rubber', 'problematical', 'tamahori', 'received', 'lamented', 'supervised', 'austen-like', 'shaved-headed', 'prevert', 'unenviable', 'flavors', 'half-interesting', 'fotomat', 'mcbain', 'broth', 'pisani', 'god-like', 'anticipation', 'ninety-minute', 'spokeswoman', 'chic', 're-evaluating', 'persecuted', 'egypt', 'prochnow', 'truckloads', 'allotted', 'similarly-structured', 'laded', 'forever', 'gender', 'conjectures', 'beatrice', 'artificially', 'foodstuffs', 'hockney', 'allowances', 'unintelligent', 'pay-day', 'withers', 'neon', 'retentive', 'hinting', 'septien', 'infuse', 'absorbs', "protagonist's", 'extremes', 'preys', 'heche', '_are_', 'polonius', 'solitary', "club'", 'masterpeice', 'physique', 'rafael', "breakin'", 'comprehendably', 'unrepentant', 'smashes', 'kishikawa', 'namesakes', 'perturbed', 'blends', 'watches', 'cursing', 'hardships', 'improbably', 'ronin', 'lucid', 'cultures', 'mcelhone', 'skateboarders', "mellis's", 'recovered', 'heldenberger', 'jobs', 'organised', 'nesting', 'outsmarted', 'cerebral', 'roberts-whose', 'pentameter', 'kuei-mei', 'hostelries', 'kaftan', 'diva', 'businesslike', 'demornay-like', 'arguing', 'und', 'roritor', 'pressure', 'mox', 'voight', 'ruffle', 'consistently', 'interviewed', 'atop', 'notwithstanding', 'screem', 'cohen', 'outfielders', 'porcine', 'applesauce', 'tendancy', 'maternal', "ferguson's", 'thee', 'entertain', 'net', 'sticks', 'electronica', 'hegel', 'cross-country', 'great-looking', 'cranked', 'gunshot', 'ostensible', 'weathered', 'high-powered', 'handguns', 'perception', 'supermasochist', 'osbourne', "`total'", 'zemeckis', 'amoeba', 'cannibalistic', 'arbogast', 'talent', "krishnamma's", 'twisty', 'due', 'gojira', 'imparting', 'paratrooper', 'definetly', 'noho', 'moxin', 'offhand', "segal's", 'light-saber', 'signaling', 'mostly-silent', 'movie--a', 'cent', 'critic', 'gizmos', "ellin's", 'athleticism', 'kersey', 'spinning', 'lecturing', 'ngoc', "`darn'", 'angels-style', 'waikiki', 'lynn', 'court', 'lifeboats', "students'", 'interval', 'outgrosses', 'zsigmond', 'supporting', 'guy-using', 'true-to-heart', 'extricate', 'bails', 'meld', 'allegory', 'mauds', 'epps', 'underwritten', 'surprisingly', 'secure', 'wind-bag', 'fragmented', "nepotist's", 'autographs', 'secretive', 'slightest', '32', 'rearranged', 'muncie', 'crystal-clear', 'assness', 'laxative-induced', 'mould', 'henstridge', 'donny', 'heavy-handed', 'dreary', 'elementary', 'nonexistent', 'jugular', '13-', 'goofy', 'kohn', 'professes', 'reassume', 'peasant', 'loudmouth', 'ungloriously', 'unapologetically', 'hattie', "taupin's", 'barker', 'excitement', 'numbness', 'unfulfilled', "'how", 'flower-shop', 'stacey', 'comparing', 'fluttering', 'stirring', 'psychodramas', 'eclipse', 'varied', 'second-fiddle', 'altercations', 'tanaka', "gallagher's", 'reputable', 'charts', 'downfalls', 'gavin', 'guarding', 'passingly', '`total', 'damned-if-you-do', 'argued', '-kilter', 'crept', 'delapidated', 'flammable', 'patronizing', 'valiant', 'nerdy', 'parolees', 'preachers', 'top-secret', 'crippen', 'comaprison', "singer's", 'slumber', '51st', 'mcdonough', 'receptionist', 'verona', "6'1", 'trainer/consultant', 'mediator', 'unthinking', 'ever-popular', 'colony', 'heavy', 'competition', 'winds-up', 'widowed', 'fraser', "'tis", 'willful', 'fondacairo', 'videogames', 'peacekeeper', 'retro', 'niagara', 'letterbox', 'movies--everything', 'allocate', 'chore', 'marie-laure', 'acceleration', '$30', 'unconnectedness', 'pod-race', 'unexploitive', 'blasted', 'cough', 'camp-like', 'baxter', 'attatched', 'hollywoodised', 'savage', 'chapel', 'digitally', 'cultish', 'first-timer', 'rustin', 'paroled', 'storagehouse', "screen--arkin's", "rota's", 'lives', 'well-trained', 'cubes', 'formulaisms', 'exceptionally', 'minbo', 'baked', 'appealingly-chirpy', 'studios', 'lemmy', 'washington-supplied', 'newhart', 'hi-', 'natalie', 'skinheaded', 'unconventionality', 'tiniest', 'stamina', 'overload', 'enrages', 'million+', '"', 'hayseed', 'pass', 'seminude', 'reading', 'frenetically', '1958', 'stoner', '_poltergeist_', 'spaniards', 'orator', 'ambiguity', "accident's", 'distant', 'manage', 'hytner', 'fishermen', "denuto's", 'intellectualism', 'sputters', 'press--to', 'conquest', 'coffee', "weekly'", 'cujo', 'openness', 'nutty', '12-year', 'climaxes', 'non-monkey', 'overemotional', "kick-the-aliens'-asses", 'stinky', "montgomery's", 'proffered', 'exposes', 'crassness', 'rides', "intelligence's", 'wesleyan', 'forbids', 'grady', 'wall', 'driscoll', 'piecing', 'rabbi', 'loveable', 'backfires', 'rereleased', 'stunts', 'francois', 'outrunning', "mongkut's", 'struggled', 'avenue', 'mj', 'appellation', 'mistakingly', 'sensationalism', 'half-assed', 'catches', 'contemplation', 'stater', 'confusions', 'discern', 'steals', 'portrayals', 'eckstrom', "cliche'd", 'maura', 'gambit', 'burbano', "rimbaud's", 'rebels', 'gondola-trekked', 'dis', 'jackpot', 'cribbing', 'hand-in-marriage', 'armando', 'chcaracters', 'insinuating', 'brother', 'loss', 'lenny', 'comprehendable', 'scribbles', 'alarming', 'johto', 'mandible', 'calcium', 'darabont', 'taling', 'senior', 'first-run', 'gallop', 'panic', 'learn', 'halted', 'crunchem', "wit's", 'distanced', 'serenade', 'literalized', 'fueling', 'chapped', 'hush-hushes', 'simmrin', 'limply', 'larenz', 'fasttalking', "speechless's", 'deduction', 'well-shot', 'sickened', 'treehouse', "tarzan's", 'not-quite-kosher', "zorro's", 'klein', 'badness', 'astonishment', 'bloodbath', 'holds-barred', 'bachmann', 'puerto', 'medic-alert', 'masses', 'peek-a-boo', 'negate', 'shook', 'cider', 'weasely', "huff's", 'slave-talk', 'hinckley', 'silently', 'agents', "'thriller", 'comforting', 'hovertank', 'out-of-body', 'riveting', 'scrapbook', 'squid-like', 'bet', 'verging', 'actioners', 'lighters', 'holding', 'tonight', 'rockers', 'unfulfilling', 'reworking', 'grieve', 'drafted', 'translates', 'grubby', 'erie', 'moonlit', 'auteil', 'seamy', 'producer/lover', 'assimilating', 'matchsticks', 'meets', 'anij', 'revisions', 'whos', 'carradine', 'letterboxed', 'nashville', "janssen's", 'warms', "maclachlan's", 'blackman', "assistant's", "novelist's", 'bores', 'unzipped', 'novikov', 'refinement', 'retort', 'mad-libs', 'sags', 'vonnegut', 'shows', 'fixture', 'deed', 'fleshed', 'plumped', 'tried-and-tested', 'advance', 'in-concert', 'repugnant', 'goatee', 'bro', 'fighter', 'george', "renard's", 'material--a', 'bulk', '9', 'baseball-related', 'lone', 'none-too-pleased', 'hell-liner', "m's", "payne's", 'enrolling', 'trends', 'pinkie', 'nietzsche', 'erbe', 'yang', 'crutch', 'profile', 'synonymous', 'overwight', 'indifference', 'fearest', 'loosing', "matt's", 'assassin/operative', 'cnn', 'chest', 'pieced', 'predestination', 'infested', 'aisles', 'participates', '4-d', 'lumumba', 'proceed', 'flick', 'ape-man', 'lovey-dovey', 'immolates', "d-reper's", 'foretold', 'discreet', 'mannered', 'tex', 'ramona', 'huy', 'carrie-anne', 'co-writed', "'mallrats'", "tourist's", 'discernible', "animals'", 'swerve', 'unfeasible', 'boney', 'not-so-subtle', 'josh', 'warring', 'hampshire', 'harvey', 'bargain', "bunz's", 'hindsight', 'demanding', 'over-acting', 'leggings', 'schitck', 'snake-eyes', 'impulsiveness', 'carrier', 'badly-dubbed', '$50', 'hypocrites', 'doze', 'dalva', 'cry-baby', 'stickler', 'soft-eyed', 'venturing', 'bayard', 'worms', 'topples', '26th', 'julius', 'reaches', 'dud', "brenner's", 'trailer-a', 'finesses', 'lunatics', 'bahamas', "hall's", 'chatham', 'outhouse', 'jowls', 'sharper', 'diminish', 'residue', 'b-horror', "revival's", 'oprah', 'grenkowitz', 'longed', 'suddenly', 'equalizer', 'prancing', 'unnecessarily', 'disinvited', 'gaby', 'jiji', 'testy', 'shroud', 'save-the-world-from-aliens-or-environment-', 'earth-shattering', 'dosmo', 'i=', 'software', 'unpleasantly', 'iago', 'angles', 'sex-obsessed', 'matheson', 'theater', 'psychoanalyst', 'favorites', 'words--', 'eggnog', 'tenley', 'checks', 'ever-mutating', 'conjugal', 'mina', 'grieving', 'trumps', 'leick', 'toga', 'bidet', 'gooder', 'inauspicious', 'titanium', 'milieu', "rudy's", "spice's", 'tatopolous', 'prague', "'collection", 'satellites', 'mars', 'nephew', "commandant's", 'plucked', 'da-be-dah', 'bosses', 'man-hunting', 'appalling', 'jeered', "double-d's", 'rising-star', "braxton's", 'mckellan', 'realm', "factory's", "epstein's", 'constraints', 'hole', 'trepidation', 'remote', 'house-ghost', 'hired', 'awesome', 'doug', 'autonomy', 'dexterous', 'underlying', "soldiers'", 'brainstorm', "hedwig's", "gosse's", 'insisted', 'jester', 'badder', 'tainted', 'vail\x05where', 'gilpin', 'kint', 'boinked', 'frame-of-mind', 'car-chase', 'muslim', 'pizzazz', "army's", 'porn-film', 'argues', 'hold-ups', 'abuser', 'bly', 'abandoning', 'frozen', 'fairuza', 'cia-like', 'molly', 'ke-', 'run-shrieking-from-the-theater', 'hawtrey', 'super-tech', 're-evaluate', "cinque's", 'trager', 'lindner', 'talmud', 'monroe', 'crushed', "whitaker's", 'pumps', 'trees', 'holier-than-thou', 'post-psychedelia', 'pint', 'sonnenfeld', 'floating', 'impunity', 'unviable', 'perez', 'wimmin', 'stinking', 'gu', 'quirkier', "'ani'", 'never-ending', 'arkin', 'lost-ray', 'www', "gast's", 'bank', 'lucas', 'height-challenged', 'transcendental', 'touted', 'harron', 'lorna', 'broken-down', 'paybacks', 'skinny', 'binks', 'exchange', 'dubya', 'bulletins', 'fictions', 'obnoxiously', 'slowing', 'undeveloped', 'instability', 'devlin', 'krige', 'displacement', 'visualizing', 'repeatedly', 'camping', 'levy', 'trusty', "steiger's", "darlene's", 'belly-button', 'regaining', 'hsueh-wei', "house'", 'english/drama', 'resistibility', 'not-so-happily', 'haney', 'dull--', 'uniquely', 'fleder', 'jadzia', 'evasions', 'thanklessly', 'indians', 'pancakes', 'matrimony', 'doyle--at', 'hauser', 'remaking', 'uprooting', 'grunt', "'amateur'", 'oppisites', 'scandal-', 'bare', 'rethought', 'stinkiest', 'gold-toned', 'brushed', 'karloff', 'horde', 'stinkor', 'splein', 'recognizable', 're-released', 'sensuous', "bullock's", 'material', 'damaging', 'landscaping', 'hedaya', "this's", 'operatic', 'choose', 'full-scale', "avery's", 'virus-like', 'loafer', 'stepsons', 'holst', 'sensations', 'trainee', 'sutra', 'mains', "maxine's", 'migraine', 'imperative', 'vulnerabilities', 'frightful', 'cocoon-type', 'action-horror', 'fairies', 'coppola', 'lauded', 'natch', 'pepperidge', 'rascal', 'witty', 'lewis', 'gasoline', 'savvy', 'dream-boat', 'tongues', 'tarantula', 'oppressed', 'diaphragm', 'littleton', 'side-splitting', 'foe', 'winners', 'wealthy', "stallone's", 'convicted', 'tilly', 'relieve', 'unbeknown', 'latitude', 'publicist', 'girlfriends', 'shopping', 'alligators', 'declan', 'filial', "rosmary's", 'schumann', 'relationships-from-hell', 'brittle', 'oneliners', 'coitus', 'handedly', 'irishfolk', 'incapacitate', 'maxine', "building's", 'narrow-mindedness', 'ample', "stone's", "prostitute's", '1976', "jacob's", 'totalitarian', 'godforsaken', 'raking', 'teenage-targeted', 'romance', 'rothchild', 'newly-jarred', 'gala', 'franklin', "'jurassic", 'screen', 'booze', 'flirting', 'minutes', 'haynes', 'yanked', 'arent', 'euliss', 'linchpin', 'muppets', 'disturb', 'satiric', 'hardy', 'translations', 'stereo', 'neumeier', 'mcconaughay', 'out-number', 'independence', 'scanners', 'exploring', 'vacant', 'constructed', 'overabundance', 'wee', '`last', "richardson's", 'seven-year-olds', 'kurgan', 'payoffs', 'snort', 'matures', 'pzoniaks', 'nastassja', 'impeccably', 'big-time', 'joadson', 'expelling', 'betters', 'slippery-smooth', 'flung', "scream's", 'sinbad', 'revolutionary', 'lomax', 'psychology-researcher', 'deserted', "maggie's", 'fervently', "hoskins'", 'sipping', 'pensive', 'wasting', 'regina', 'eschewing', 'frain', "'fear", 'grade-school', "crichton's", 'attributed', 'gorgeous', 'amoral', 'robocop', 'typcast', 'torontonian', 'sing-a-longs', '_dead_man_', "lovin'", 'radiation-induced', 'co-ed', 'facet', 'self-absorbed', 'intrepid', 'slaughtering', 'incantation', 'rendezvous', "roro's", 'native', 'deaf/mute', "`stud'", 'ron', 'flashes', 'ryan-starrer', 'loggia', 'gale', 'prophecies', 'threefold', 'tracker', 'cavanaugh', 'possessions', 'overdose', "prentice's", 'lectures', 'much-prized', 'intangible', 'forced', 'earnestly', 'one-ness', 'clack', 'schumacher', 'ex-boyfriend', 'trucker', 'offered', 'angels', 'half-digested', 'observe-as', 'profundis', '&', 'caroline', "j's", "harker's", "kleiser's", 'dumpster', 'shoot-', 'cattrall', 'fete', 'scott', 'keller', 'portrays', 'stiers', "brainard's", 'ellsworth', 'compliments', 'copeland', 'gadfly', 'tone', 'magic', 'dogged', 'stall', 'tension--as', 'copious', 'conservationist', 'intrigued', 'orchestrated', 'abider', '1974', 'consultants', 'flakes', 'mortars', 'concern', 'clamor', 'nonstop', 'abductees', 'disco-era', 'forgettable', 'ornery', 'mansell', 'delete', "dosen't", 'hairspray', 'motherlode', 'longevity', 'unncessary', 'distinctions', 'appearance', 'munundated', 'curling', "charlies'", 'bruel', 'initiated', 'co-operation', 'homosexuals', "supernova's", 'caraboo', "myers's", "_54_'s", 'techno', 'commandant', 'temper', 'hirings', 'dragnet', 'wardens', 'prepares', 'bathe', 'kamil', 'compromising', 'boucing', 'regurgitation', 'non-nude', "one-he's", 'repulsively', 'cronenbergto', 'high-level', 'space-trucker-turned-impromptu-survivalist', 'mesmerising', 'hyper-colorized', 'gleason', 'dearest', 'liner', "dogs'", "dove's", 'biblical', 'pricey', "african's", 'existing', 'appetite', "molly's", '0/10', "morrison's", 'stalker', 'outsmarts', 'error', 'snooze', 'joker', "publisher's", 'artie', 'guest-quarter', 'over-active', 'nineteen', '1983', 'malt-liquor', 'liotta', 'acheivement', 'shoved', 'pau', "callin'", 'confidently', 'britton', 'loitered', 'exaggerating', 'grievously', 'defying', 'disorienting', 'medical-gross-outs', 'chambermaid', 'arguments', 'orchestral', '$19', 'mega-advertising', 'solidifying', '29-year-old', 'amphibian-based', 'freeing', 'sparkle', 'raj', 'hollow', 'nay', 'blanchett', "'blue", 'amenities', 'typified', 'illuminates', 'fasting', 'tendencies', 'moon-crawler', "claire's", 'walking', 'brush', 'gulf', 'orangey', 'swill', 'preachings', 'drained', 'pfcs', 'qualify', 'overpraise', 'nights', 'horniness', 'estrogen', "langenkamp's", 'torn', 'notices', "elliott's", 'presumption', 'bumstead', "zwigoff's", 'funding', 'darn', 'apparently', 'wrong--and', 'phillips', 'anti-sexual', 'cameos', 'syndrome', 'yarn', 'sieber', 'survivor', 'role-model', 'bowels', 'neighbour', 'valid', 'distributed', 'christened', 'morph', 'richter', 'awards', 'poetical', 'soldiers', 'expressively', 'flaps', 'stupider', 'uncharacteristic', 'functional', 'explitives', 'tight-fitting', 'iscove', 'amen', 'assassin', "'laughter'", 'documentarian', 'culinary', 'bloody-red', 'tusken', 'rile', 'weanies', 're-experiencing', 'corporeal', 'challenge--not', 'stunningly', 'herrier', "blanchett's", 'girlfriend/boyfriend', 'confessions', 'business-as-usual', 'promiss', 'unreal', 'foreseeable', 'ivanek', 'tactics', 'justifying', 'strangles', 'adaptions', 'half-human', 'oversight', 'fish-out-of-water', 'shuttle', "l'argent", 'self-vanity', 'misstep', 'unassociated', 'expunged', 'peppers', '`bring', 'lidner', 'racism', 'fastball', 'weekends', 'puddle', 'mockumentary', 'courteney', 'roared', 'slacker', 'recedes', 'faux-jingoistic', 'mane', 'isacsson', 'simple-minded', 'arab-americans', 'arabian-themed', 'characters', "pyle'", 'condoms', 'well-planned', 'blind-side', 'director/executive', 'breezy', 'ufo-related', 'ex-lawman', 'shrift', 'officials', 'gorefests', 'schlictman', 'connect-the-dots', 'lower-class', '1963', 'story-wise', 'suppression', 'face-first', '750', 'stallone', 'submerged', 'transpires', 'dejection', 'rug', 'snatched', 'contact', 'panes', 'diseases', 'undermine', 'pickles', 'big-guy', 'hieroglyphics', 'jacques', 'orchestra', 'nuff', 'immolation', 'attractive-seeming', 'riotously', "oteri's", 'incompetents', 'homemaker', 'apologizing', 'sex-pervert', 'embers', 'asleep', 'convention-breakers', 'pregnancy/child', 'contributed', 'long-standing', 'interfered', 'batfans', "grim's", 'cheesefest', 'non-stop', 'sonya', 'soon-to-be-notorious', "screenwriters'", 'exotically', 'family-man', 'retooling', 'mast', "jeweler's", 'unseasonably', 'harriet', 'dispatches', 'win-at-all-costs', 'authoress', 'exception', "wonders'", 'severance', 'wrapped', 'noxema', 'titanic-fever', 'irate', 'facelessness', "'hope", 'hos', 'hector', 'veterans', 'do--this', 'access', 'khanh', 'wholesome', 'well-thought', 'laughter', 'trucksploitation', 'distributors', 'jagger', 'modelled', 'services', 'tiring', 'metaphysical', 'neat-o', 'rourke/jean-claude', "sherman's", '#3', 'murrieta', 'orientation', 'kittens', 'mpd', 'extinguished', 'lia', 'francisco_', 'naval', 'jolt', 'tangible', 'conaway', 'ditched', 'falwell', 'actuality', 'amulet', 'jasper', 'schtick', 'commuter', 'applaud', 'gailard', 'pedantic', 'well-secured', 'santostefano', '83-minute', 'girl-power', "witness'", 'tool', 'ephesians', "darnell's", 'unconscious', 'guidelines', "university's", 'high-octane', 'hyman', 'turnpike', 'franz', 'eyesight', 'obssession', 'less-than-subtle', "evelyn's", 'created', "inmates'", 'yech', 'co-director', 'faldwell', 'legit', 'notoriously', 'klump', "alfred's", 'barriers', 'smaller', 'penzance', 'cloudy', 'canderday', 'ambling', 'badmouth', 'treetops', 'carl', 'driver', 'half-expecting', 'adversely', '12th', "kurtz's", 'owned', 'intermission', 'cream', 'fluttered', 'pay-off', 'locker', 'bootleg', 'high-strung', 'minstrels', 'agitated', 'pedophilic', "pirate's", 'resounding', 'moonlight', 'endeavored', 'daughters', 'resolved', 'dip', 'fictitious', 'huffing', 'equals', 'protester', 'game-like', 'pseudo-epic', 'awakes', 'coeur', 'dollhouse', 'assorted', 'feet', 'well-filmed', 'awkwardly', 'mantis', 'indios', '--and', 'lars', 'observed', '_gag', 'top-form', 'earth-normal', 'bells', 'opning', 'cruddy', 'fluke', 'beebe', 'warburton', 'whala', 'frivolous', 'noctis', 'impossibly', 'wiring', 'denmark', 'embarrassment', 'napkin', 'practiced', 'skinheads', 'persuasively', 'ongoing', 'seti', 'identical', 'kansas', 'religious', 'amistad', 'insincerity', 'reached', 'verbose', 'ingratiating', 'laudible', 'boutte', 'experienced', 'ledge', 'caused', 'journals', 'berman', 'brilliance', 'mutha--shut', 'draft', 'ann-moss', 'renegade', "mcculloch's", 'recast', 'etymology', 'team', 'costars', 'poise', 'peformances', 'problems', "pipeline's", 'skill--thus', 'manifests', 'bandied', 'painkillers', 'blot', 'iridescent', 'improper', 'shockingly', 'potions', 'futz', 'boyce', "tatopoulos's", 'mayhew', 'cement', '15-minutes', 'bucketsful', 'publicly', 'inquires', 'silenced', 'belching', 'veiny', 'damsel', 'averse', 'floom', 'disco', "it's-a-spielberg-film", 'berridge', 'crowded', 'feuer', 'rebound', 'mister', 'dusk', 'trumpeting', 'ghosts', "beek's", 'sentimentality', 'talkie', 'pate', 'marlboro', 'eulogy', 'star_', 'kilpatric', "hot-fudge-rockin'", 'leery', 'coe', 'dislikes', 'warmth', 'godly', 'renown', 'pill-popping', 'ex-baseball', 'ditch', 'troy', 'malice', 's&l', 'starman', 'parallax', 'powers-that-be', 'breathy', 'garlington', 'princess', 'clearillusions', "oprah's", '[animated', 'bag', 'film-', 'wonderfully', 'bliss', 'coure', 'equiped', 'roomie', 'accident', 'ashes', 'cluttered', 'hynek', 'loaded', 'discouraged', 'lieutenant', 'hitmen--cisco', 'pilot', 'post-watergate', 'donning', 'decide', 'elise', 'courtrooms', 'dauphin', 'hitherto', "henson's", 'pours', 'brutish', 'best-forgotten', 'discordant', 'prominence', 'revulsion', 'bio-ports', 'cum', 'approached', 'telepathically', "yankovic's", 'demonic--power', 'against-all-', 'phonation', 'conservatism', 'one-third', 'streisand', "partygoers'", 'humping', 'bladders', "vitti's", 'attorney', 'pelted', 'intersection', 'secrets', 'constrast', 'steel', 'spoonful', 'fable', 'protective', 'claudius', 'frightens', 'repetitiously', "governor's", 'beaten-up', 'disposing', 'pattern-like', 'moff', 'philander', 'decades-spanning', 'he-', 'money-wise', 'hansel', 'non-human', 'militia', 'magnus', 'surf', 'spot', 'prowls', 'heedful', 'hurlyburly', 'personified', 'turntables', 'punishing', 'vulcan-like', 'jollies', 'boy-o', 'sanders', 'scharzenegger', "fonda's", 'street-smart', 'consternation', 'alcohol', 'idiot', 'collectibles', 'levien', 'regenerated', 'nations-type', 'sponsors', 'seville', 'uproariously', 'schindler', 'all-out', 'animes', 'hang', 'award-calibre', 'murder', 'coddling', 'giggs', 'devised', 'laughs-to-jokes', 'edtv', 'hyams', 'renault', 'entrancing', 'improvisations', 'one-eyed', 'execrable', 'boo-fest', 'chronic', 'world-', 'talentless', "mancuso's", 'display--there', 'cronos', 'bravery', 'saintly', "dershowitz's", 'daisy', 'preachiness', 'failure', 'hooded', "mcelhone's", 'getz', 'fionnula', 'high-speed', "powers'", 'warner', 'interiors', 'bullets', 'star-making', 'wedged', 'zed', 'sword-fight', 'off-key', 'snore', 'there--during', 'stuffed', 'motions', 'hearings', 'august', 'cop-out', 'harper', 'stimulating', 'wren', 'unturned', 'powder', 'dusted', 'repertoire', 'audience-pleasing', 'novak', 'rolex', 'senate', 'arch', 'pyrotechnics', 'companions', 'springfield', 'older', 'exams', 'splattered', 'herc', 'anderson', 'blaster', 'drumroll', 'sugary', 'colored-glass', 'investment', 'pushing', "andersen's", 'hot-and-bothered', 'tactful', "mirren's", 'admission', 'heaven-sent', 'demotic', 'edwin', 'matinee', 'missions', "thompson's", 'making-of', 'beachfront', 'documentation', 'bees--both', '10-week', 'herald', 'cattle', 'gunfights', "everything's", 'vatican', 'thwart', 'dt', 'glowers', 'cleansing', 'affordable', 'warwick', 'pippa', 'rainmaker', "song's", "princess's", 'mayoral', 'unwelcomed', 'administrative', 'debussy', 'painted', 'tongue-in-cheek', '2-disc', 'slang', 'cockroaches', 'soup', "polanski's", 'tamara', "judith's", "giles'", 'full-of-tension', 'high-fives', 'colleen', 'hoggett', 'eye-candy', 'trashes', 'bullcrap', 'double-decker', 'entices', 'trademark', 'caper-gone-awry', "anne's", 'strangle', 'punctuating', 'trumpet', 'chapter', 'spiderman', 'scientifically', 'boss', 'animal-loving', 'adequately', 'ciggy', 'spencer', 'unpromisingly', 'tolerating', 'replay', 'shen', 'heavy-handedness', 'tennis', 'iconoclast', 'mey', 'buried', 'amish', 'self-admittedly', 'jadzia-bolek', 'kretschmann', 'shafts', 'reclessly', 'meloncholy', 'tinseltown', 'ihmoetep', 'mystical', 'skepticism', 'stokely', 'cartridges', 'comedy--it', 'squib', "disney's", 'conclude', 'irreplaceable', 'tyrannic', 'ring', 'elude', 'responsibility', '`dharma', 'windup', 'nitroglycerine', 'tagged', 'stewardess', 'sorta-redux', 're-invent', 'plot-wise', 'bludgeons', 'grossed', 'deliberate', 'santas', 'one-by-one', 'terminally', 'def', 'pairs', 'downed', 'pre-review', 'cabs', 'ringmaster', "lover's", 'dimly', 'summarizing', 'casino', 'catsuit-clad', 'boingo', "'volcano'", 'raison', 'demystification', 'unobserved', 'motivator', 'semi-intriguing', 'zardoz', 'maynard', 'amateurism', 'transatlantic', 'badly', 'schlesinger', "myers'", 'crash', "trooper's", 'toned-downed', "kelly's", 'conductor', 'minimize', 'all-american', "charlie'", 'menial', 'self-acceptance', 'noticed', 'movie-goers', 'dotty', 'achieve', 'blank-from-hell', 'pusateri', '_54_', 'cowers', 'dome', 'lfe', 'disastorous', 'belch', 'moisten', 'not-very-good', 'effervescent', 'easely', 'military', "kat's", 'lance', 'dottie', "tulips'", 'ministers', 'foster', 'stereotype', 'greenlight', 'disconcerted', 'jells', 'buffy', 'down-to-earth', 'half-speed', 'kronk', 'invention', 'hardford', 'signal', "mills'", 'merrier', 'underbids', 'swallows', 'eighies', 'burglar', 'ghettos', '2017', 'shakespeare-obsessed', 'indestructable', 'nero', 'yeller', 'underway', 'convulsing', '14th', 'kinsella', 'detonation', 'debut--stars', 'heartfelt', 'sleek', 'likewise', 'figeroa', 'twist-ties', 'angry-but-inept', 'array', 'flavorful', 'davis', 'button', 'slabs', 'triple-bladed', 'investigating', 'impatience', 'stagey', 'lodging', 'wasp', 'screenplay--with', 'outacts', 'reinvigorate', 'nicholson', 'willem', 'figure', 'difficulty', 'risa', 'mathou', 'look-alikes', 'thereof-threatens', 'graces', 'mayfield-directed', 'hollywoodization', "mckellan's", 'trivialised', '2-2', 'mommie', 'inn', '--the', 'pans', 'sitcoms', 'lawrence/fat', 'co-counsel', 'display', 'wrong-way', 'habits', 'milliken', "cronenberg's", 'slimey', 'befall', 'profoundly', 'barbed', 'storylines', 'cornflake', 'eliminate', 'dispels', 'disbeliefs', 'bridal', 'imbecile', '9=borderline', 'reminicent', 'perspectives', 'bazelli', "trial's", 'grittiest', 'inning', 'dun', 'vannesa', 'nosferatu', 'apple-cheek', 'skis', 'paved', 'rigueur', 'horror/comedy', 'grasps', "titanic's", 'grabbag', 'alleviated', 'sarge-type', 'perfect--you', 'weakening', 'not-in-a-hurry', 'mystic', 'crime-foiling', 'a-list', "jacobi's", 'eaton', 'dougnac', 'incarnations--graphic', 'consuming', 'determining', 'foreshadow', 'chapman', 'lowry', 'egregious', 'orefice', 'pharaoh', 'transfusions', 'full-size', 'exterminator', 'aladdin', 'slew', 'aswell', 'pander', 'near-blizzard', 'aftereffects', 'hendra', 'black-market', "joan's", 'right--', 'mandoki', 'kindest', "rea's", 'babysit', '_must_', 'rebelling', 'siegfried', 'gizmo', 'trivializes', "barrymore's", 'amusing', 'metaphor-heavy', 'deviation', "_beloved_'s", 'explaining', 'surley', 'mandibles', 'ripple', 'blue-screening', 'surpassed', 'line--it', 'harring', 'rahad', 'detected', 'several-scene', 'better-made', 'chalks', 'es', 'straits', 'leer', 'soavi', 'perpetuate', "`atmosphere'", 'hits', 'wispy', 'seawall', 'carvings', 'lived', 'bennington', 'harebrained', 'jungian', 'every-man', 'sadsacks', "birds'", 'mae', 'kinear', 'hills', 'popul', 'cabinet-level', 'regeneration', 'voyeur-next-door', 'all-ages', 'mercilessly', 'successfuly', 'horticulturist', 'sleepy-eyed', 'outstandingly', 'trumpet-playing', 'carte', 'dementia', 'affairs', 'neeson', 'mile-high', "will's", 'wheedon', 'irs', 'travelled', 'two-bit', 'announcement', "moreau's", 'tweaking', 'do-gooder', 'manservant', 'grammar', 'fifties', '1962', 'goonies', 'restoring', 'seeks', 'fanfare', "job-it's", 'sympathetic', 'pitied', 'carnality', 'strength', "ocean's", "gavin's", 'reincarnated', 'pin', 'fantastically', 'supervision', 'negotiations', 'shielding', 'soy', 'performances--always', 'infamously', 'goldfish', 'toned', 'loud-noise-jump-scare', "id's", 'light-mood/serious-mood', 'ply', 'non-discriminating', 'deemed', 'mock', '_babe_', 'kindly', "monica's", 'frosted', 'soaking', 'coincidentally', 'peices', '`blade', 'fuel', 'guilifoyle', 'sully', 'rectify', "gangster's", 'five-minute', 'spiers', 'underestimated', 'confers', 'villagers', 'reappears', 'levinson', 'components', 'beauty', 'surfer', 'increasingly', "trainee's", 'weirdos', 'helens', 'see-through', 'maglietta', 'flatmate', 'presences', 'grimly', 'divest', 'penchent', 'firecrackers', 'old-fashioned', 'dodger', 'accomplishing', 'licentious', 'hijinx', 'gloves', 'is--and', 'modulation', 'invoked', 'nap', 'invading', 'embassy', 'soul-baring', 'well-publicized', 'laughs', "palmer's", 'roxburgh', 'powerless', 'pyjamas', 'frothy', 'caricatured', 'amtrak', 'chiller', "dads'", 'softness', 'dupe', 'staggering', 'caters', 'giulianni', 'hour-long', 'parentless', 'revise', "shoot'em", 'lampooned', 'exaggerates', 'click', 'clash', 'seventy-eight', 'wanna-be', 'award', 'deepens', 'basketball', "user's", 'ferraris', 'misappropriating', 'imaganitive', 'hopper', "bloodsuckers'", 'document', 'arli$$', 'detonated', 'nipples', 'behaves', "africans'", 'garbled', 'sentimental', 'materialized', 'casuality', 'plump', 'anime', 'leonor', '45', 'collette', 'realizes', 'incessant', 'nikita', "tracy's", 'sulaco', 'inspired', 'fond', 'toiled', 'science', 'attentiveness', 'evidencing', 'pellegrino', 'rivalling', 'trackers', 'altered', 'assigning', 'firepower', 'guccione', 'dramatizes', 'goateed', 'format', 'passionately-', 'execs', 'faculties', 'doer', 'menu', 'developed', 'armani', 'fascination', 'procreation', 'astrophysicist', "hooper's", 'topics', 'to-die-for', 'ticketed', 'non-pc', 'electing', 'midknight', 'mississippi', 'stubborn', "heer's", 'watershed', 'kitty', 'spurting', 'decrease', 'cult-like', 'y2k', 'micheal', 'surrealist', 'semi-wily', 'oscar-winning', "skg's", 'debuts', 'vamires', 'ooooh', 'bette', 'shovels', 'multi-billion', 'malka', 'centuri', "decade's", 'devastingly', 'doren', "collins'", 'slinky', 'perrier', 'cheese', 'film--the', "ronnie's", 'effects-bound', "bickford's", 'crocodiles', 'mumbling', 'piety', 'animal-hater', 'assertive', 'stemming', 'teeters', 'tawdriness', 'prepostrous', 'ogle', 'sabian', 'amuck', 'california', 'hitman', 'liberating', 'kerchak', 'bluegrass', 'teeter', 'films=92', 'objectivity', 'compartment', 'rough-and-tumble', 'stragely', "lot's", 'schreber', 'pulverizes', 'dicillo', 'guard/chet', 'picks', 'accolades', 'rule-the-world', 'ice-blue', 'oldies', 'switzerland', 'attic', 'abbots', 'woman-of-the-wild', 'guidance', 'schlocky', 'didja', 'roberta', 'tender', 'mastubatory', 'ineptly', 'courage', 'commutes', 'stupefied', 'tad-too-long', 'triptych', 'jailer', 'lemme', 'maximilian', 'vandernoot', 'intuition', 'quicksand', 'valiantly', 'interlude', 'properly', 'givers', 'overhearing', "right'", 'vinicius', 'pursuers', 'tacky', 'levers', 'internet-like', 'apprehended', 'service', 'pick-me-up', 'toppled', 'data', 'reformers', "america'", 'polluted', 'recommend', 'pina', 'spiritualist', 'originated', 'chopsocky', 'unearthed', 'dynamics', 'twenties', 'topicality', 'off-beat', 'lucie', 'planes', 'anglade', '1933', "up-i've", 'cyber-courier', "joey's", 'waterworks', 'immuno-deficient', 'appointed', 'safari', 'jarringly', 'upwardly-mobile', 'class', 'back-door', 'orr', "_dragon_'s", 'celebrate', 'i-wanna-please-dada', "farm's", 'successfully', 'fray', 'appreciating', 'hotheaded', "00'", 'lotta', 'kay', 'cannibals', 'urinating', '$39', '1794', 'lifeline', 'freighter', 'maturity', 'cruises', 'pursue', 'ice-pick', 'bismol', 'cyberdog', 'ruler', 'self-made', 'guadalcanal', 'vastness', 'pretensions', 'smuggler', 'needle', 'enjoyably', 'exaggerations', 'multi-multi-plex', 'cutbacks', 'patinkin', 'subtract', 'canaan', 'sails', 'toaster', 'caviar', 'committing', 'leung', 'keane', "'home", 'barbet', 'snaps', 'refrain', 'gratuitous', 'super-duper', 'interrogated', 'generate', 'soeur', 'lasso', 'sparkling', 'fearsomely', 'half-assedly', 'trench', 'delineation', 'clayon', 'likeable', 'stumbled', 'reminiscence', 'dead-end', 'bedside', 'facehugger', 'tsugawa', 'catalyst', 'willie', 'dicks', 'ww2', 'latched', 'arrogance', 'lectured', 'noon', 'goodtimes', 'insulted', 'highbrow', 'graveyards', "eisenstein's", "chippendale's", 'handicapped', 'nicolas', 'sony/loews', "school's", "necklace's", 'lesbians', 'occasions', '1994', 'tomfoolery', 'diabetic', "grasshopper's", 'belfast', 'stuffy', 'decency', 'coddle', 'abomination', 'polarity', 'defiant', 'convey', 'whoville', 'thirteeth', 'chomps', "bard's", "later'", 'conversational', 'sawalha', 'bunch', "uncut's", 'pound', 'watermelons', 'hosts', 'eilonwy', 'ghettoized', 'nothiiiiiinggggggggggg', 'valli', 'enumerates', 'encroachment', 'beleiveable', 'heartbreaks', 'parador', 'pond', 'bless', 'hyperviolent', 'whale', "cole's", 'phi', 'poorly-paced', 'well-drawn', "'end", 'tact', 'diana', 'blares', 'panicked', 'genital', 'urn', 'gorham', 'thing__not', 'sail', 'superpower', 'baby-boom', 'hard-to-get', 'stigma', "salwen's", 'ufo', 'cups', 'periodical', 'barns', 'citing', 'giggly', 'draco', "pollock's", 'toosie', 'resurrects', "corman's", 'issuing', 'caulfield', 'propelled', 'snowy', "grandmother's", 'master-servant', 'couplehood', 'rapier-9mm', 'wreck', 'heartbreakers', 'dominatrix', 'gun-toting', 'grier-worshiping', 'cops', '`primal', 'splotty', 'factored', 'smashing', "marcellus'", 'earlier', 'detrimentally', 'samaritan', 'founders', 'chlorine', 'spurns', 'bickle', '`cruel', "would've", '88', "apes'", 'sprinkling', 'heartwarming', 'closeups', '500', 'guilty-pleasure', 'moved', 'incongruent', 'blockbuster', 'pinch', 'procreate', 'hardass', 'wind-up', 'chi-chi', 'shark', 'guy', 'reared', 'trekkies', 'pinky', 'ill-defined', 'whichever', 'noodles', 'grandison', 'orlock', 'plops', '_urban_legend_', "hannah's", 'author', 'mulling', 'potential', 'proliferation', 'possess', 'traumatic', 'sorcha', 'dmitri', 'row', 'tasks', 'top-line', 'inquisitiveness', 'giveaway', 'stinks', 'stand-by', 'bests', 'louis', 'giants', "there'll", 'hermit-like', 'yun-fat', 'ninth-consecutive', 'alone-until', 'hashbrown', 'misfortune', 'subverted', 'corps', 'maine', 'ryan-nicolas', 'int', 'naoto', 'neophytes', 'contextual', 'fistfights', 'neill', 'disappointing', "hyam's", 'husband', 'ideally', 'desert', 'bareikis', "ripley's", 'claire', 'separately', 'alters', 'kimberly', 'sub-plots', 'continuum', 'over-protective', "'comedy'", 'rollercoaster', 'thunderous', 'lapoirie', 'sad', 'philipe', 'incongruously', 'forlorn', 'employees', 'fouls', 'millions', 'laurentiis', 'all-of-a-sudden-superhuman', "hoke's", 'management', 'dreamworks', 'conferences', 'illuminate', "weaver's", 'montague', 'darth', 'son-in-law', 'determines', 'batch', "who've", 'yasuoka', 'sketchiness', 'web-based', 'subtitled', 'oversimplification', "filmmaker's", 'seasoning', 'comparable', 'naif', 'cheery', 'temptation', 'salvaged', 'self-reflection', 'salivate', 'backseat', 'trivial', 'hail', 'never-ceasing', 'reconciles', 'endures', 'hockley', 'scrolled', 'soon-to-be-divorced', 'speculated', 'schiff', 'mcgowan', 'business-like', 'jonn', 'chauffeur-driven', '`alex', 'sue', 'forges', 'variety', 'coercion', 'madcap', 'baddies', 'pies', 'coffins', 'akira', 'get-together', 'georgina', 'crotchety20', 'delia', "_shaft's_big_score", 'others--mostly', 'ploy', 'excruciatingly', 'transference', 'rainslickered', 'senseless', 'digresses', 'faucet', 'wealth', 'equitably', 'reconnaissance', 'kidnappers', '`patty', "flynt's", 'so-bright', 'mate', 'hershey', 'rallied', 'true-man', 'boop', 'leering', 'costello', 'issac', 't-rex', 'dimly-lit', 'eleventh', 'ham-on-wry', 'wild', 'mishandle', 'lasting', 'pos', 'thing--the', 'peculiarities', 'utilize', 'know-it-all', "'broads'", 'janssens', 'plot-relevant', 'outside-in', 'auditorium', 'triumphantly', 'tremor', 'financiers', 'philadelphia-area', '_american_psycho_', 'physiology', 'dis-organized', "hedaya's", 'perky', 'double-deals', 'thrilled--', 'nazi-style', 'partially', "thomas'", 'outright', 'alive', 'beeps', 'sire', 'crabs', 'almost-witty', "davis'", 'overuses', 'systematically', 'gush', 'explain', "hytner's", 'pursued', 'one-celled', 'blink', 'confetti', 'broads', 'oozes', "teacher's", 'ronkonkoma', 'elixir', 'signficance', 'haviland', 'thrillers', 'maryland', 'criticised', 'mel-brooks-produced', 'ingested', "luke's", 'passages', 'danes', 'arlo', 'mongkut', 'pensively', 'bubbly', 'cogs', 'ziegler', 'tracks', 'harvard', 'degeneration', 'smirk--regardless', 'situations--into', 'dombasle', '70ies', 'trips', 'turks', 'highway', 'lund', "taster's", 'millenium', 'librarian', 'groan', 'bouyant', 'smirkiest', 'libidinous', 'sixth-grade', 'kampf', 'caffeinated', 'overlay', 'armada', 'overflowing', 'balanced', 'preserve', 'gladly', 'red-haired', 'indulging', 'teenagers', 'peppered', 'vernon', 'earned', 'preforming', 'overly-dramatic', "'all", 'teck-oh', 'mistletoe', 'mom', 'self-indulged', "generations'", 'africans', 'near-crazy', "holmes'", 'hudgeons', 'esmeralda', 'fanatic', 'creating', 'ideas', 'finalist', 'villains---and', 'calvin', 'cutter', 'vacuous', 'dynamism', 'weakened', 'directs', 'lillianna', 'sights', 'singers', 'filing', 'unborn', 'beefed', 'troupe', 'injurious', "partner's", 'beneath', 'trelkins', 'vishnu', 'rockets', 'grimm', 'bookends', 'roulette', 'lefessier', "white's", 'squandering', "jimmie's", 'characterization', 'kassovitz', 'danging', 'typecasting', 'rekall', 'give', 'gaultier', 'audience-friendly', 'menaces', 'glover', 'seer', 'matched', 'fountain', 'results', 'crucifies', 'morritz', 'tigher', '--so', 'publicity-seeking', 'antagonist--unstoppable', 'moe', 'snowball', 'arrives', 'wielders', 'hazmat', '_blade', 'berardinelli', 'usa', 'missing', 'altering', 'overexposed', 'elitist', 'announced', 'prakazrel', 'wipes', 'chisselled', 'awkwardness', "hai's", 'over-hyped', 'cupcake', 'conclusion', 'dickerson', 'half-scared', 'convicting', 'contented', "craig's", 'apology', 'philly', "powder's", 'gaelic', 'blas', 'lansquenet', 'murder-and-cover-up', 'screeches', 'triva', 'well-built', 'burning-bush', 'deeper', 'colorful', 'unabated', 'bemusement', 'nursing', 'clung', "karen's", 'preposterousness', 'ruptured', 'balances', 'restlesness', 'stendahl', "dollah'", 'kwik-e-mart', 'perfunctory', 'simcoe', 'ngo', 'cathartic', 'facher', 'prolonged', "dean's", 'prevented', 'independently', 'torch', 'lynched', 'rose-tinted', 'sound-alike', 'winded', 'ladder-fight', 'christ-like', 'struts', "hammer's", 'story_', 'behaving', 'institutes', 'coolness', 'bookkeeper', 'homeys', 'striking', 'yoda', 'testimonials', 'slices', 'fake-looking', 'resuscitation', 're-focuses', 'saluting', 'brain-dead', 'dear', 'ascended', 'dutiful', "thailand's", 'rebelliousness', 'tempestuous', 'pen', 'suited', 'blunder', "ironside's", 'repercussions', 'balloon', 'berdan', 'regularly', 'second-tier', 'voices/accents', 'procol', 'racial', 'tucked', 'custom', "poirot-delpech's", 'collaborator', 'flip', 'contradicts', '2036', "lampoon's", 'apply', 'you-at', 'bellylaughs', 'e-block', 'windshield', 'frewer', 'completion', 'lurk', 'furtwangler', 'boiler', 'rewarding--a', 'trading', 'gateway', 'ludicrousness', 'connick', "'back", 'fuses', 'lamb', 'disabled', 'meandering', 'ownership', 'beatles', 'awestruck', 'sadoski', 'recollect', "vegas'", 'mortgages', 'grouper', 'underwhelming', 'tennant', 'star', 'draggy', 'suburb', '`enemy', 'bus', 'plastered', '2293', 'sleezebag', 'tappan', "asimov's", 'racket', 'wheelchair-stricken', 'wobbles', 'gwynne', 'danger', 'toro', 'provocatively', '3-dimensional', "garrett's", 'starling', 'snicker-worthy', '7th', 'outrun', 'long', 'skeleton', 'simm', 'orgasm', 'integrated', 'guides', 'proposed', 'degree', 'r&b', 'swelling', 'duchesse', 'closure', 'sabotaged', 'underperforms', 'handyman', 'tammy', 'relents', "o'brien's", 'rabbit', 'dive', 'technology', 'writes', 'combat', 'posessed', 'spatter', 'punctuated', 'waits', 'shoot-outs', 'unkempt', 'servicable', 'astaire', 'lovable', 'underdeveloped', 'certianly', 'repeats', "kaufman's", 'irish', 'foreknowledge', 'frankfurters', 'assumed', 'raspy', 'leasure', 'longer-than-necessary', 'breeze--or', 'stead', 'brandishing', 'fetishization', 'rossio', 'imposing', 'audition', 'behavior', 'lea', 'danced', 'emancipation', 'liquefied', 'wed', 'outsider', 'li', 'smell', 'circa', '-sound', "depalma's", 'broach', 'complex', "dunne's", 'tomatoes', 'briscoe', 'defendant', 'mon', 'sneeze', 'seasons', 'juxtaposed', 'wafflemovies', 'exley', "gridlock'd's", 'prophesies', 'hoary', 'eyewitness', 'little-known', 'coaches', 'current-films', 'grouse', "ahmad's", 'ugliest', "thade's", 'non-supermodel', 'biggest-breaking', 'commercialist', 'aforementioned', 'raft', 'disengaged', 'eastwood', "liz's", 'coined', 'postal', 'olympian', 'nacho', 'keely', 'bastardised', 'unharmed', "`stigmata'", 'vending', 'year-old', 'weak-willed', 'fiasco', 'troi', 'agitation', 'woo-type', 'deployment', 'tackles', 'barrister', 'marcia', 'conferring', '27%', 'reviled', 'approve', 'busying', 'athlete', "'net", 'fart', "berenger's", 'rules', 'szubanski', 'strife', 'sci-fi/action', 'well-cast', 'pasty-faced', 'phaedra', 'initiates', 'forgo', 'blips', 'merteuil', 'repays', 'straight-', 'airwolf', 'requests', 'side-show', "denny's", 'faith', 'unmitigated', 'viard', 'undergone', 'sportscaster', 'comparisons', 'arsenal', 'reynolds', 'kills--he', 'lesabre', 'ice-cube', 'yanne', 'hog-nosed', 'white-collar', 'driven', 'anti-christ', 'studio-released', 'reigned', 'cooler', 'conceive', 'poor-taste', 'lift', 'rent-a-wreck', 'tzudiker', "kim's", 'hysteria', 'rectangle', 'nutcase', 'multi-armed', 'innuendo', 'bart', "fussell's", 'vylette', 'practical', 'faa', 'rip-offs', 'marienbad/m', 'attends', 'region', 'science-fiction', 'pest', 'stagecoach', 'hoses', "visnjic's", 'payment', 'professor', 'hackman', "boyd's", 'bud', 'shou', 'makeup-laden', 'lobster', "ru'afo", 'demi', 'compassionate', 'aquatica', 'strode', 'jungle2jungle', 'english-dubbed', 'patient', 'lightest', 'passionless', 'shoplift', 'posses', 'takashi', 'eaisly', 'obliterated', 'post-cannibal', 'ex-lax', 'chortling', 'ruminations', 'automatons', 'palantine', 'honorably', 'actual', 'absolute', 'african-americans', 'stem', 'busey', 'deviates', 'smelled', 'neater', 'archetypal', 'quick-cuts', "question's", 'unseen', '_quite_', '143', 'perplexing', 'superbly-staged', "mcnaughton's", 'kalifornia', "'bend", 'clandestine', 'incapacitated', 'rehabilitation--he', 'sexless', 'transaction', 'prowling', 'skating', 'woo-ping', 'traveler', 'downing', 'sulu', 'bagpipes', 'rockhound', '16x9', 'fireman', 'open-heart', 'attenborough', 'torpedo-ed', 'anti-voyeuristic', 'documented', 'contents', 'setbacks', 'amy', 'godley', 'unsocial', 'suger-coated', 'discovering', 'tying', 'aesthetically', 'response', 'suffers', 'biographical', 'comanche', 'hawkes', 'suspensefully', "salomon's", "o'shea", "top's", 'uncommunicative', 'shuns', 'phoned', "borgnine's", 'regan', 'fidelity', 'baker-enhanced', 'keener', 'super-cypher', 'grating', 'doofy', "redford's", 'playfully', 'nonplused', 'spectacular', 'presuite', 'earlygoings', '_film', 'improves', 'stock', 'tragicomic', 'game', 'miko', 'sign', 'scrubbing', 'talk', 'torkel', "humans'", 'beatng', 'tit', 'boham', 'century-old', 'ruth', "tunney's", 'spontenaity', 'banjo-picking', 'enjoyability', 'flynn', 'searches', 'muse', 'unforgiving', 'screening', 'careening', 'russ', 'chatting', 'mortals', 'anytown', 'crazed', 'frasier', 'devour', 'styles', '161', 'larina', 'misjudgment', 'manicurist', 'catch', "`6'", 'mugshot', 'surgeon', 'kathleen', '`bonnie', 'thurl', 'shante', 'startle', 'bagger', 'morante', 'williamson', 'younger', 'slapping', "frame's", 'cooperate', "suzie's", 'traeger', 'divine', '$85', 'occasionally-witty', 'glasnost', 'mystery', 'hormonally-advantaged', 'patrick', 'saville', 'cheque', 'on-ship', 'gorilla', 'misirables', 'headaches', 'weirdness', 'robust', 'change', 'unified', 'american/italian', 'sewing', 'admirable', 'poorly-employed', 'full-thx', 'lunch', 'depressive', 'pop-culture', 'hefti', 'devotes', 'infuriates', 'wb', "beautiful'", 'pistol', 'anti-charm', 'malkovich', 'woozy', 'misused', 'paintings', 'flealands', 'studio', 'carnival', 'ghostface', 'mer', 'goodboy', 'innately', 'karin', 'shimono', 'adeptly', 'initiate', 'parameters', 'delectably', "prinze's", "christ's", 'places--', 'harshest', 'debut', 'rat', 'lise', 'co-venture', 'honks', 'childdhood', 'snooping', 'tuna', 'brewster', 'atone', 'searcing', 'minnesota', 'aguirresarobe', 'gary', "edison's", 'sum', 'minutes]', 'a-plenty', 'sub-human', 'malibu', 'impressive', 'stacy', '2am', 'wax', 'g-string', 'zorro', 'showgirls', 'none-too-subtle', 'flutter', 'characters--but', 'appeasement', "thurman's", 'railway', 'rambo-on-testosterone-therapy', 'shotgun', 'intimidate', 'shakespeare', 'tableware', 'rerelease', 'competently', 'reviews', 'train-', 'sometimes-funny', 'admitedly', 'grier', 'green', 'ishmael', 'audiotape', 'graphics', 'begun', 'sight-seeing', 'proportions', 'veritably', '-ed', 'irks', 'permeate', 'statesmen', 'mormon', 'oldham-designed', 'dog-eared', 'brouhaha', 'retrieving', 'tiering', 'veruca', 'dustbuster', 'incalculable', 'dangles', '12', 'heart_', '_dancing_', 'flood', 'kilgore', '>honk', 'intelligent--and', '_people_', 'neandrathal', 'snow-globe', 'crowchild', 'incompletely', 'deaden', 'outset', 'impartially', 'dress', 'crescendos', 'pandemonium', 'dart', 'lib', 'koslova', 'arlington', "_america's_funniest_home_videos_", 'lowered', 'fauna', 'finney', 'vapors', 'cartoon-like', 'suffocating', 'wrong--again', 'tussles', 'conceptual', 'hitching', 'institution', 'devote', 'yep', 'fox', 'systems', 'indies', 'sued', 'dreamed', 'michael', 'antenna', 'un-flashy', '1943', 'bulimia', 'anti-climactic', '`no', 'encino', 'masculinity', 'confrontations', 'acknowledgement', 'proven', 'upstairs', 'district', 'clicking', 'pumping', "black's", 'sentence', 'stock-aitken-waterman', 'singable', 'berated', 'ang', 'guaspari', 'cale', 'jessie', '9-year', 'affiliated', 'growth', 'cipher', 'dudes', 'consequential', 'whites', 'cane', 'wet-', 'starts', 'imbedded', 'petitioned', '75', "reno's", 'classics', 'peeing', 'excessively', 'elbows', 'collecting', 'conway', 'polarizing', 'wanna-see-how-fast-i-can-drives', 'sirtis', 'rain-soaked', 'extravagant', 'crossover', 'lonsdale', 'puppet-style', 'unfocused', 'audiences', 'action-', 'thin', 'sears', 'all-white', 'bandies', "`experience'", 'siunin', 'sabine', 'filleted', 'suppressing', 'blue-collar', 'congressman', 'deedee', 'belting', 'portillo', 'wlodkowski', 'contacts', 'build', 'foolery', 'walsingham', 'apprenticeship', 'meeting', 'withdrawn', 'deathly', '`saving', 'hamilton', 'unforced', 'credit', 'pierces', 'judy', "lolita's", 'edelman', 'burwell', 'grimacing', 'logically', 'enthusiast', 'videotape', 'wheeler', 'floors', 'batting', 'word-of-mouth', 'ee', 'scolex', 'kong-recipe', 'absent', 'offends', 'priority', 'laptops', 'poorly-shown', 'kidnapper/murderer', 'cogliostro', 'tantalizingly', 'coked-up', 'essential', 'bosom', "brolin's", 'belly', 'unopposed', 'morals', 'disheartening', 'autistic', 'nerve-shattering', 'moreau', 'palace', 'snatches', "3'", "costner's", 'salon', 'love-in', 'loveliest', 'aerospace', 'snicker', 'reissue', 'skin', 'nike', 'ivory', 'silverback', "perf's", 'recently', 'bungling', "harford's", 'shivery', 'face-to-face', 'expensive', 'reason--somehow', 'radition', 'singer', 'shots--and', 'smart-alecky', 'mutual', 'apprise', '_jerry_maguire_--will', 'indy', "breeder's", 'gies', 'liking/disliking', 'suchet', 'imaginary', "ways-i'll", 'spock-like', 'weil', "dreamworks'", 'meanest', 'spirits', "forsyth's", "damme's", 'watts', 'whole-sentence', 'joyful', 'obey', 'fish-eye', 'top-billed', 'sluttish', 'one-and-a-half', 'gristly', 'idiots', 'gereally', 'dialogue', "'95", 'empire', 'spectral', "hounsou's", 'conspire', 'impossible--witness', "franchise's", 'dictatorships', 'daunting', 'catalogue', 'beristain', 'afore', 'hotcakes', 'supervirus', '3-why', 'deltas', 'defeated', 'duran', 'hinging', 'exorcism', 'scarface', "'three", 'frollo', 'lake', 'terribly', 'absolution', 'mis-steps', 'favourite', "odds-and-ends'", 'nondescript', 'fabulously', 'diverted', 'raspy-voiced', 'mohamed', 're-imagining', 'prospected', 'exalts', 'angering', 'explorer', 'mcdormand', 'ahmet', "method's", '8-years', 'attached', 'behaved', 'frosty', 'pointer', 'amazed', 'reluctantly', 'pego', 'cavernous', 'bowler', 'lemmons', 'repeated', "`we're", 'senses', 'mui', 'skew', 'tamer', 'samba', 'den', 'strangled', 'wading', 'underwater', 'represents', 'hyper', 'tranquil', 'pricked', 'no-', 'bail-bondsman', "nugent's", 'flushed', 'buyers', '15-million-dollar', 'quadruple', 'themselves--they', 'goddamned', 'domino', 'stagner', 'schmucks', 'exception--is', 'rossellini', "tribune'", 'humbled', 'cared', 'buck', 'filmore', 'genuis', 'delving', 'swirls', 'confident', 'rioting', 'juicing', 'jitterish', 'potent', 'chuckling', 'groove', 'defacing', 'eastward', 'check-out', 'young-whippersnapper', "denise's", 'sketch', 'phenomenal--observe', "russian's", 'operas', 'cesar', 'hiring', 'danger/attempted', 'zeit--get', 'elk', 'unintrusive', 'cab', 'crime-gone-wrong', 'bloodline', 'tanker', "maybury's", 'cremated', 'dedicated', 'dances', 'cherry-red', 'special-effects', 'kiddie', 'balthazar', 'barbs', 'ships', 'cruise/paul', 'shadowloo', "neither's", 'integrity', 'inconsistencies', "continuity's", "depp's", 'defenseless', 'one-fourth', '`p-p-p-p-please', 'simply', 'decalogue', 'bradshaw', 'evasively', 'henricksen', 'influences', 'pay-per', 'soldier', 'originator', 'supervisors', "frankenstein's", 'beautifully-constructed', "hurd's", 'cowering', 'negotiate', 'order', "spring's", 'gem', 'assuring', 'beer', 'pursuits', 'curtail', 'annoy', 'eye-level', 'solidifies', 'misty', 'shipments', 'bedsprings', 'hang-out', 'oft', 'joisey', 'original', 'rowan', 'roller', 'feathery', "o'barr's", 'aural', 'promotional', 'drops', 'slow-fade', 'obliterates', 'benoit', 'action-craving', 'out-bland', 'cliquey', 'near-sequel', 'skip', 'magda', 'telephones', 'sitting', 'proposal', 'vols', 'awful', 'shelmikedmu', 'wynt', 'wiggles', 'neccesity', 'testimonies', 'seal', 'onegin', "delaurentiis'", 'hymn', '5/10', 'botches', 'randanzo', 'fast-motion', 'semi-star-making', 'no-holds-', 'literary', 'window', 'luxury', 'hydra', 'prosecute', 'reinforce', 'self-congratulatory', 'gautier', 'runny', 'bankers', '_loathe_', 'turning', 'motels', 'wrecks', 'ninjaman', 'fed', 'cut-out', 'others--sikh', 'inscribed', '3000', 'adorable', "kinsella's", 'ferris', 'swashbuckler', 'removal', 'one-year', 'dog-walking', "1970's", 'comprehension', 'neutron', 'shifting', 'supreme', 'round', 'trenches', 'sobchak', 'two-year-old', 'retire', 'defeat', 'travis', 'shoeless', 'belts', 'prayers', 'b-list', 'tension', 'derek', 'stability', 'fujioka', 'whacked', 'hairline', 'circulating', 'shagadellic', 'kindergartner', 'tantrums', 'sucking-out', 'life-affirming', 'resents', 'tulip', "tate's", "kidman's", 'yield', "hollow'", 'eyeroll', 'panaro', 'metallic', 'suspect\x05', 'smacking', 'crucified', 'purred', 'dusters', 'tarnished', 'imponderable', 'counter-terrorist', 'nuptials', "desouza's", 'afterward', 'dousing', 'enforcement', 'internationally', 'guerilla', 'mandingos', 'mugs', 'remarries', 'definitions', 'eli', '8mm', 'paw', 'much-ballyhooed', 'cheats', 'screenwriters', 'preferring', 'televised', 'amalgamations', 'canals', 'couteney', '18-plus', 'expansiveness', 'seduce', 'unsexy', 'rapture', '1-1', '1965', 'mesa', 'interviewers', 'indigenous', 'rifles', 'propositions', 'outed', 'occured', 'mowing', 'mind-moving', 're-ignite', "'psychologically", 'landmark', 'garth', 'interracial', 'awesomeness', 'nevermind', 'battles', 'lavishness', 'haughty', 'gain', 'skerritt', 'extinction-', "oregon's", 'poverty-stricken', 'kit', 'gromit', 'wands', 'marijunana', 'incite', 'veneer', '_heathers_', 'habit', 'anamoly', 'pesticides', 'impersonation', 'out-of-key', 'sickest', 'escalated', '20-30', 'sticky', 'mozell', 'jewelry', 'sheppard', 'sedgwick', 'formulas', 'viet', 'pleasured', 'nostaligism', 'merrill', 'matin', 'tux', "rendezvous'", 'derogatory', 'luke', 'close-cropped', "'old", 'cuban/miami', 'paycheck', 'skulduggery', 'self-confident', 'norville', "hubbard's", 'excess', 'jeb', 'accompaniment', 'sequitur', 'exprience', 'frying', 'spices', 'griffith', 'snow-encrusted', 'galpal', 'costanza', 'squeaky-clean', 'tinkling', 'tautou', 'fleshy', 'planner', "activist's", 'ping-ying', 'develops', 'tense', 'pine', 'property', 'pantucci', 'grateful', 'mechoso', 'craftsmanship', 'drawback', "archer's", 'dinosaur-like', 'ursula', 'burnham', 'sweathogs', "kay's", 'loath', 'pic', 'deniro-like', 'deposit', 'caste-like', 'salvage', 'hope', 'superviolent', 'dissappears', "tweedy's", 'counting', 'put-downs', 'parodying', "sphere's", 'barksdale', 'downey', 'open-mike', 'cuesta', 'immune', 'multidimensionality', 'wiped', 'giggles', 'forgive', 'confess', 'movie-goer', 'fin', 'uncontrollable', 'yakuza', "everyone's-going-to-go", "mcdowell's", 'hypnotically', 'shanghai', 'low-level', 'none-too-subtly', 'pesci', 'beat-up', 'moviemaking', '7-up', 'versions', 'impeachment', 'straight-out', 'powers', 'quota', 'excellent-but-so-', 'ferrer', 'capitalism', 'touch-a', 'bukater', 'arranged', 'namesake', 'buddy', 'auriga', 'oklahoma', 'refugees', 'quoting', 'mentally-challenged', 'filby', 'concentrates', "don't-mess-with-my-life-and-i-won't-mess-with-yours", 'no-name', '25-cent', 'lowly', 'linoleum', 'crowning', 'felonies', 'hunches', 'alone--his', 'rickman', "kids'", 'battleships', 'mistress', 'tantamount', 'swampy', 'bob', 'hiv', 'established', 'muddies', 'dragging/salting', 'tanked', 'cyberkillers', 'barracades', 'carrot-colored', 'oh-so-promising', 'unoffensive', 'boothe', 'idol', 'command', 'bloodlust', 'alpha', 'adandon', 'nudity', 'jumping-off', 'form', 'meek', "cuba's", 'mulan--get', 'marijo', 'trendsetters', 'un-nerving', 'sanchez', 'diet', 'tyrese', 'self-done', 'thorpe', 'call-waiting', 'villalobos', '`jackie', 'rouge', 'echoing', "herzog's", 'constructed-harry', 'life', 'professing', 'musketeer', '30-year-old', 'machismo', "roberts'", 'arisen', 'blurry', 'two-century', 'supermodel-level', 'dipping', 'ill-tempered', 'knox', 'long-hampered', 'neo-', 'anglo', 'stank', 'sledgehammer', "edgecomb's", "'99", 'tumultuous', 'reinventing', 'slave-ship', 'swimsuit', "jane's", 'davin', 'references', 'alessa', 'scripting', 'red-dyed', 'stoppidge', 'bullion', 'upcoming', 'macguffin', 'extensively', 'resistance', 'demons', 'rewrites', 'antsy', 'heartbeat', 'kroon', 'hackett', 'accomplices', 'all--but', 'rickles', 'far-out', 'devito', 'unimportant', 'teaches', 'dieppe', 'margot', 'mistrust', 'flamboyant', "lewis'", 'duel', 'battering', 'blanche', 'small-town', 'capacity', 'cesars', 'punched', 'sizeable', "connery's", 'well-designed', 'amundsen', 'hobel', 'dont', 'predilection', 'crossbows', 'stan', 'intelligible', 'rochelle', 'showstopping', 'edgar', 'cd-rom', 'unadulterated', 'mucho', 'doubtlessly', 'carpark', 'imdb', 'sidesteps', 'humans', 'timbers', 'cetera', 'discrepancies', "woman'", 'movie-make', 'lestat', 'day-timer', 'comfortable', 'heads', 'pagan', 'strict', 'reeks', "kidnappers'", 'heil', 'robs', 'overdrive', '1/1/1900', 'tributes', 'resolve', 'neanderthal', 'makeup', 'glossing', 'modernization', 'presence', 'fertile', 'decter', 'opponent', 'claims', 'pallid', 'hurried', 'slower', 'patrice', 'hooking', 'cackling', 'lyle', 'mileage', 'letter-perfect', 'hard-to-categorize', "'i've", 'macpherson', 'psycho-sexual', "europe's", 'transvestites', 'inspect', 'mewes', 'kritic', 'irrationalities', 'starpower', 'trunchbull', 'adams', 'teased', 'camembert', 'bumcheeks', 'workman', "is'", 'primer', 'bozo', 'exceedingly', "'star", 'pore', 'perfekt', 'startlingly', 'performer', 'bravest', 'browsed', 'happier', 'dramatically', 'newfoundland', 'pep', 'greer', '35th', 'lackadaisical', "placid'", 'chrichton', "hobbes'", 'ky', 'wages', 'tumbleweed-like', 'insect', '36-hour', 'impersonates', 'cage-world', 'station', 'mcfarlane', "fate's", 'behind-the-scenes', 'distinctive', 'episode', 'smack', 'mystery-turns', "mouse's", 'step-in', "sorvino's", 'nameless', 'cleveland', 'titan', 'identity--struggling', '_titus_', 'lightning', 'full-frontal', 'pales', 'crafted-not', 'hannah', 'mooney', 'middleage', 'mortal', 'jia-li', 'slashing', 'files', 'strom', 'newbie', 'buddhist', 'moldy', 'twosome', 'motherland', 'declared', 'retain', 'monument', 'pharmacy', "rochon's", 'statue', 'motivation', 'phillip', 'perfection', 'intuits', 'amnesiac', 'perch', 'poet', 'fulfillment', 'native-born', 'tempo', 'broadcasted', 'germ', 'garb', 'obtaining', 'delirious', 'masters', 'non-sequiturs', '357', 'needy', 'fellinian', 'convenience', 'pyramid', 'banality', 'timekiller', 'eeeewwwww', 'alternate', 'prostitues', 'twotg', 'plenty', 'violated', 'morphine', 'planned', 'wincer', "outworld's", 'civilization', 'masaya', 'recluse', 'reinvigorating', 'purposely', 'harbor', 'briefer', 'alternately', 'conned', 'morrow', 'fair/canticle', 'gavan', 'oddest', 'half-world', 'sterotypes', 'font', 'palmetto', 'rig', 'i-the', 'aldys', 'progression', 'windy', 'measly', "cisco's", 'talky', 'topheavy', 'stills', 'babboons', 'pussyfoot', 'dearth', 'salacious', 'broadbent', 'btw', 'lancaster', 'sex-', 'wolves', 'ran', 'fernanda', 'latifah', 'eight-hundred', 'quills', 'intrigues', 'editors', "woo's", 'levar', 'garfalo', 'herded', 'philosophically', 'decrepit', 'hype', 'agnostic', 'pnly', 'pure', 'fruit', 'construct', 'serling', 'techno-gothic', 're-think', 'subscribe', 'felicity', 'web', "bujold's", 'homeworld', 'platt', "d'artagnan", 'overpopulated', 'thriftily', 'manufacturer', 'worn', 'breakdown', 'abberation', 'bets', 'adoption', 'jump-start', 'hairstyles', 'tilting', 'boos', 'screed', 'pets', 'skyrocketing', 'weitzman', 'strychnine-laced', 'postponement', 'gq', 'sinister', 'fatales', 'monroth', "carter's", 'enlargement', 'lucinda', 'searles', 'uptown', "i-don't-know-when", 'couture', "v's", 'hearts', 'avatar', 'allotting', 'civilisation', 'hoyts', 'notting', 'lo', 'gator', 'creamy', 'talk-show', 'masterfully-done', 'realism', 'logical', '[to', 'thompson', 'incredulously', 'aristocratic', 'periphery', 'conniptions', 'predator', 'observations', 'comfortably', 'hebrews', 'catatonic', 'stud-reporter', 'levison', 'polymorphously', 'inscribe', 'flares', 'boringly', 'lili', 'foremost', 'remand', 'midst', 'tenant', 'seemingly-harmless', "destination'", 'catalyze', 'superiors', 'bartok', 'amounts', 'winwood', 'semi-automatic', 'apartment', 'paralyzed', 'pinkett', 'beards', 'career', 'reflectively', 'hostility', 'ironic', 'affectation', "actors'", 'ker', 'helms', 'dug', 'black-he', 'pleases', 'ex-spouse', 'coward', 'assisting', 'ascent', 'colonial', 'everything-and-the-kitchen-sink', "see's", 'critters', 'floorboards', 'lukemia', 'gaius', 'long--so', 'dismayingly', 'spurned-psycho-lover-who-wants-revenge', 'permanant', 'then-patrick', 'maduro', 'bathgate', 'breath-taking', 'self-discovery', 'invaded', 'post-chasing', '`idle', 'pouts', 'imitating', 'handed', 'resurface', 'battle--going', 'savoured', 'lebanese', 'admires', 'fearing', 'footnote', '_the', 'uninhabited', 'animatedly', 'vigilant', 'ilynea', 'zigs', 'out-going', "ol'", 'arc', "1940's", 'tapioca', 'attitude', 'pearce', 'worried', 'caped', 'hurtful', 'hormones', 'moth', 'high-spirited', 'rounders', 'pang', 'unleash', "kingpin's", 'retriever', 'images', 'espadrilles', 'menendez', 'supercode', 'vicki', 'hatami', 'september', 'love', 'harmonies', 'jazzed', 'scamp', 'priestly', 'deathblows', 'incidentally', 'predicate', 'brainerd', 'uluru', 'bend', 'humpalot', 'seldes', 'burglars', "actress's", 'legitimize', "cavanaugh's", "homer's", 'multi-line', "doens't", 'mainstay', 'profanities', '-1', 'harmon', 'undercurrent', "carolyn's", 'ck', 'velda', 'howlers', "jennifer's", 'risked', 'motorcyclists', 'meteors', 'braddock', 'miyamoto', "scene's", 'champions', 'sorted', 'cackle', "world's", "house's", 'origins', "candidate's", 'circumstances', 'sabara', 'wet-wired', 'underestimating', '1966', 're-converge', "'matron's'", 'worries', 'cult', 'actor-turned-director', 'nature', 'elodie', 'canadians', 'groping', 'familiarity', '#19', 'ape-creature', 'awarding', 'oscars', 'protect-earth-from-destruction', 'awa', 'mimicking', 'boorman', 'rest', 'firestorm', 'beryl', 'proficiency', 'anti-semitism', '-but', 'kingpin', 'dreck', 'sad-looking', 'retrieval', 'apoplectic', 'terminated', 'nadia', 'applause', 'vitro', "coen's", 'naysayers', 'doo-wop', 'lice', 'witch-wannabe', 'simpithize', "'bots", 'puerile', 'parted', 'neighboring', '_whole_', "longo's", 'slake', 'reviewer', 'mattered', 'honcho', 'democracy', "design'", 'helmets', 'advantages', 'iraq', 'dust', 'reminded', 'alicia', 'ashlee', 'sarge', 'dares', 'light-heartedness', "robots'", "fidelity''", 'sculptures', 'bulldog', 'rayden', 'runied', 'finnegan', 'effects-filled', 'grounds', "mizrahi's", 'schizophrenia', "eddie/dirk's", 'frears', 'curtailed', 'tcheky', 'du', 'snoop', 'just-another-striptease', 'snapped', 'boyz', "webb's", 'awaited', 'assimilation', 'scriptures', 'sampson', 'allergy-prone', 'horesback', 'dream-like', 'knees', 'no-show', 'wessex', 'coated', 'well-educated', 'moviegoing', 'brotherly', 'reveals', "lucy's", 'signals', 'sacred', 'hijinks-addled', 'undone', 'soldier-type', 'environment', 'shilling', '23rd', 'schedule', 'killing', 'orgiastic', "`lower-class'", 'balancing', 'bookish', 'servicewear', 'sleeve', 'merchants', 'gosnell', 'snapshots', 'athena', 'revenue', "'master'", 'klingon', 'pacula', 'leverage', "glazer's", 'sunnybrook', 'reno', 'half-million', 'twister', 'sex-talk', 'ego-withering', 'historically', 'kit-kat', 'whacking', 'barbell', 'suspense-thriller', 'cling', 'post-operative', 'extinct', 'neigbors', 'antennae-like', 'excesses', 'substitutions', 'senility', 'rotating', 'kickboxing', 'jack-in-the-box', 'evilo', 'bra', 'weir', 'far/strength', 'psychedelic', 'diabolical', 'sex-even', 'absense', 'trollop', 'dozens', 'negotiators', 'arrivals', "malick's", "canada's", 'coke-addled', 'steerage', 'it--red', 'assaulting', 'thewlis', "january's", 'poised', 'leblanc', 'sloth', 'emotionally-crippled', 'pixie', 'iroquois', 'tonino', 'forethought', 'thespians', 'easy', 'palatial', 'bloody', 'rants', 'giovanni', 'infertile', 'big-budget', "jedis'", 'joined', 'wildlife', 'abroad', 'mingle', '30-40', 'trickle', 'financing', 'shortly', 'sucking', 'bluntly', 'like-minded', 'fix', 'sites', 'delicatessen', 'gingerbread', 'abuse', "rush's", 'confidant', 'weakly', 'hypnotism', 'hitler', 'num-num', 'stomps', 'ramsey', 'strut', 'query', 'spotlighted', 'buzzwords', 'girlfight', 'heroin-addicted', 'bostock', 'instant', 'munro', 'limp', 'racers', 'solon', 'batman', "castor's", "'good", 'augusts', 'james', 'vanished', 'divides', 'insidious', 'internationally-acclaimed', 'similar', 'hose', '19th', 'meaty', 'impulses', 'paragraph', 'besvater', 'dvd', '20-year-old', 'mole', 'devane', "ronna's", 'workers', 'dennison', 'half-formed', 'alas', 'answered', 'strategize', 'copywriters', 'western', 'twist', 'illuminated', '/10', 'coenesque', 'valuables', '_matrix_', 'unemployed', 'ceaseless', 'mandy', 'mid-1500s', "jordan's", 'mordred', 'cicely', 'gole', 'velicorapters', 'paying', 'spectacular--almost', 'conveys', 'parachute', 'tiberius', 'interpol', 'revolving', 'burn', 'schoolboy', 'dugan', 'second-class', 'therapeutic', 'silvery', 'boxers', 'man=92s', "finn's", "son's", 'slammer', 'mcnaughton', 'foreign-language', 'oil', 'brooks', 'ripped', 'one-horse', 'lenient', 'cruiser', 'suspense--and', 'performers', 'dismembering', 'chinese-american', 'bridgette', 'dariush', 'pet', 'marketers', 'rehearsing', 'stealthy', 'pubescent', 'piquant', 'highly-populated', 'dialectic', 'comet-bombing', '1-2-', 'three-year-olds', 'heavily', "pilots'", 'dismantling', 'identify', 'brent', 'laugh-out-loud', 'diego', 'signs', 'kiss-ups', 'sidekick/sidekicks', 'drunken-style', 'iffiness', 'greenwood', 'whocares', 'bb', "yamamoto's", 'amplifying', 'confections', "channel's", 'grinch', 'unparalled', 'standout', 'glowing', "communication'", 'inlists', 'proceeds', 'hospitalized', 'html', 'soon-to-be-marines', 'helicopter', 'warped', 'baffles', 'whales/triangle', 'mini-wonderland', 'shivers', 'lonely', 'prisons', 'beasts', "'tough", 'heavily-bespectacled', 'schmaltziness', 'easy--look', 'tsutomu', 'resign', 'production', "'comical'", 'cupid', 'gunplay', 'serbedzija', 'cooperated', 'tree-surfs', 'satiate', 'deservedly', "doesn's", 'faceless', 'spatula', 'u-boats', 'sequels', 'carping', 'gland', '`keeping', "rem's", 'fellatio', 'fm', 'affirming', 'ryan', 'spawned', 'bigtime', 'conditioned', 'necroplasmic', 'kerry', 'squeezed', 'millieu', 'breeding', 'henslowe', 'overboard', 'endoskeleton', 'horrors--bad', 'ruining', 'conversion', 'arne', 'non-involving', 'blue-eyed', 'flashback-type', 'rarer', 'mifune', 'legendary', 'harries', 'sanity', 'over-conceived', 'shades', 'testing', 'braeden', 'hint', 'busses', 'reduncancy', 'donal', 'ex-journalist', 'lid', 'bleached', 'movie-induced', 'worthier', 'reccover', 'conniving', 'hot-tempered', 'film', 'overbearing', 'rodgers', 'comic-slapstick', 'appetites', 'broodwarriors', 'sony', 'rightist', 'graduates', 'metro', "elijah's", 'knowingly', 'behaviors', 'josephine', 'uncreative', "element's", "weddings'", 'qe2', 'stretch', 'vamps', 'reagan-era', 'gordie', 'ledges', 'mislead', "bros'", 'pious', 'schillings', 'returing', 'silverstein', 'skateboards', 'quirky-but-realistic', 'non-friendly', 'positioned', 'soldini', 'sang', 'place', 'potato', 'analee', 'veterinary', 'fixedly', 'conrad', 'application', 'lydia', 'squashed', 'happenings', 'detailing', 'martial', "argento's", 'egon', "rumpo's", 'fletch', 'sked', "u'tinni", "rodriguez's", 'laclos', 'jesse', 'synthesis', 'vulgar', "norristown's", 'workmanlike', 'made', 'uneasieness', 'farting', 'disracting', 'versa', 'bestowed', 'apprehend', 'woodland', 'hypsy', 'exclaim', 'too-over-the-top', 'unwisely', "boston's", 'val', 're-situating', 'ophelia', 'fluently', 'canyon', 'godsend', 'teenager', 'condolences', 'scaffolding', 'momumentally', 'beastie', 'cardboard-cutout', 'crams', "television's", 'tightly-edited', 'theme', 'sheer', 'garish', 'parlay', 'cilvilization', 'strictly-by-the-numbers', 'permitted', 'severities', 'startling', 'glancing', "rumble'", 'seemingly', 'discoveries', 'mad-dog', "wincott's", 'hesitation', '`sleepless', 'reprises', 'exploitation', 'persuasions', 'veranda', 'christopher', 'club-singer', 'menaced', 'chat-up', 'caribbean', 'excused', 'teen-age', 'unbeknownst', 'programmer', 'spoil', 'sprint', 'duplicates', 'captivation', 'terminology', 'cloned', 'quadrangle', 'ballstein', 'premise', 'slicing', 'maimed', 'robespierre', 'overemphasizing', '1978', 'unironic', 'flag', 'missouri', 'zoo-like', 'dwayne', 'e-mailed', 'corners', 'rehired', 'morlocs', 'obsessive-compulsive', '2018', 'pervasive', 'worthy', 'relevant', 'philosopher', 'warlock', '`mirabella', 'anjelica', 'kick-boxing', "generation's", 'zealots', 'sex-crazed', "louisiana's", 'wackier', 'dreads', 'tails', 'megaglomaniac', 'bygone', 'associates', 'tribal', 'rendition', 'night-club', 'pretended', 'skipp', 'dishes', 'webcams', 'neighbours', "rosemary's", 'petulant', 'chromium', 'part-owner', "money'", 'thorne-smith', 'spinoff', 'fair-skinned', 'cowrote', 'non-intrusive', 'rivalry', 'proportion', 'fiddles', 'no-where', 'skims', "body's", 'inadequacy', '`10', "goodman's", 'non-supernatural', 'delhem', "julia's", 'hut', 'parent-child', 'elements', 'lean', 'mconaughy', "'only", 'blaring', 'eye-popper', 'slug-like', "lambs'", 'denise', 'youth', 'trippier', 'jumps', 'apted', 'sign-up', 'comparatively', 'octane', 'forward', 'layered', 'shariff', 'styled', 'eloquent', 'railing', 'bulletproof', "region's", 'birdhouse', 'sliding', 'escorted', "primate's", 'hygiene', 'fakeouts', 'nu', 'fishburne', "actresses'", 'designs', 'gazillion', 'gamely', 'artsy-fartsy', 'aides', 'freshener', '27/98', 'recalls', 'veggie', 'dignitary', 'surroundings', 'backdrops', 'portable', 'anticlimatic', 'reset', 'featurette', 'count', 'hurricane', 'throwaways', 'thoroughly-modern', 'tightness', 'newsreel', 'visualized', 'realizing', 'choked', 'secrecy', 'min', 'hard-boiled', 'fan-base', 'incidents', 'chlo', 'horizons', 'eyeballed', 'autobiographical', 'delilah', 'sweet-faced', 'sub-plot', 'gusto', 'robyn', 'zips', 'immaturity', 'implants', 'truce', 'nelson', 'teague', 'mutant', 'errs', 'cyber-doctor', 'dennehy', 'cocky', "'mulan", 'thick-witted', 'almost-subliminal', 'walk-ons', 'unstable', 'theo', 'gruesomely', 'shagadelic', 'pollster', 'followup-to', 'dentist', 'hulke', 'harnessed', "alzhiemer's", 'sonorra', 'obstacles--such', 'sicker', 'upn', 'interviews', 'wino', 'constipation', 'inserted', 'wilmer', 'fail', 'physicfirst', 'sunbathing', 'guzman', 'progressing', 'duc', 'workshop', 'bounced', 'laughably', 'insultingly', 'smuggling', 'languid', 'revulsed', 'pseudo-world', 'sweathog', 'u-571', 'regrets', 'lion-hearted', 'marley', 'anthem', 'ceaselessly', 'producer-director', 'menace', 'sprocket', 'undertone', 'patton', 'enriching', "burnham's", 'sugarry', 'suspects', 'entry-level', 'grisoni', 'immoral', 'rhyitm', 'chugging', 'gaga', 'vampiric', 'widescale', 'surges', 'painterly', '-like', 'contradictions', 'hyena', 'simplicities', "bargain'", 'informants', 'jumpy', 'concocting', 'sharpster', 'horrendous', 'aesthetics', 'nubile', 'heightens', 'unbelieveable', 'premere', 're-meet', 'delicate', 'berlinger', 'victim/potential', 'settles', "mystery-girl-who's-no-real-mystery", 'schoolchild', 'perilous', 'tycoon', 'march', 'i=92ve', 'performences', 'starring', 'revolution', 'dillon', 'subculture', 'eerie', 'motorcade', 'primaries', 'miller', 'rid', 'car-sex', 'tantalizing', 'kinney', "`m2m'", 'veracity', 'stand-out', 'tangential', 'pyre', 'pugnaciousness', 'asswhole', '03', 'characteristic', "woman'--female", 'shock-factor', 'bleachers', 'anarchists', 'earp', 'un-dicaprio-esque', 'workable', 'hinwood', 'stricter', 'genovia', 'letter-opener', '449-3411', 'slob', 'thinly', 'lovemaking', 'sinuous', 'karan', 'teen-orientated', 'donner', 'sun-times', 'tote', 'marsellus', 'jett', 'pesimism', 'ranging', 'stand-up', 'mutated', 'narrow-sighted', 'hanks', 'clears', 'debello', 'tormented', 'appearence', 'tariq', 'convincingly', 'perimeter', 'cagey', 'foiling', 'bernstein', 'vocals', 'unextraordinary', 'nosebleeding', "toho's", 'organisations', 'assistsant', 'ogling', 'seekers', 'lays', 'caddy', 'corleone', 'felt', 'ark', 'shlock', 'reigns', 'bookending', 'plausibly', 'henderson', 'hyped', 'lunacy', "noah's", 'fluids', 'backgrounds', 'becomming', 'lerner', 'two-way', 'pro', 'prospector', 'always-phenomenal', 'lineage', 'internally', 'werner', 'blare', 'suing', 'offstage', 'ditty', 'dann', 'hideous', 'practise', 'warden', 'endanger', 'kong-hollywood', 'derail', 'lecter', 'victoire', 'director-cum-thespian', 'near-impossibility', 'ejogo', 'varney', 'telegraphing', 'devirginized', 'tweaked', 'special-effect', 'happily-ever-after', 'usefulness', 'mirror-panel', 'smashed', 'trivia--ironically', 'toons', 'infinite', 'organize', 'increasing', 'pseudo-lyrical', "edward's", 'frontal', 'well-bared', 'soul-transferring', 'decieving', 'usher', 'conspiratorial', 'oregon', 'retitle', 'punxsutawney', 'plaza', 'murnau', 'handles', 'idea---a', "frighteners'", 'ideological', 'parenthood', 'sutherland', 'riddled', 'orignal', 'rock/warner', 'jaunty', 'aisle', 'loose-cannon', 'cinemas', 'asinine', 'outstretched', 'tussle', 'trio', 'playbook', 'cloaks', 'asimov', 'geologic', 'tzipporah', 'hypothesis', 'paint', 'outposts', 'unleashing', 'choppy', '`parole', 'carriage', 'reeve', 'atrice', 'rite', 'titanic', 'lb', 'lawnmower', 'steady-date', 'c+', 'faltering', 'ante', 'shared', 'atrocities', 'laserman', 'tranquilizers', 'verge', 'skeletor', 'stepdaughter', "kapur's", 'humiliation', 'rocky-style', 'lankier', 'politically-charged', 'beau', 'crimes-gone-wrong', 'screamed', 'sympathize', 'braveheart', 'zinger', 'businesswoman', 'sweat-literally', '$31', 'pitching', 'stutters', 'conveyed', 'inhibits', 'mountain-climber', 'thick', 'humorless', 'instrument', 'majorino', 'classic', 'funds', 'diaries', 'eruting', 'seniors', 'moonraker', 'valmont', 'schlondorff', 'ajay', 'unforgiven', "gattaca's", 'ouch', 'teach', 'photographic', 'combative', 'critiques', "bush's", 'peerless', 'ever-bemused', 'sportscar', 'mindlessly', 'sniping', 'resulted', 'garlic', 'county', 'learned', 'kod', 'owner', 'killer/horror', 'contrary', 'mcknight', 'suspenders', 'ignorance', 'bombard', 'framed', 'kimsey', 'retracing', 'tarzan', 'snubs', "mckean's", 'paucity', 'operative', 'drafts', '[fill', '200-page', 'panthers', 'suit-', 'squeals', 'crewmate', 'trolls', 'survived', 'dissatisfaction', 'woman/', 'strongman', 'government', 'svelte', 'coven', 'bored', 'mini-mr', 'dimwitted', 'ving', 'harmlessly', 'imperfect', 'implication', 'exudes', 'much-admired', 'cairo', 'co-writer/director', 'dewy', 'compartments', 'levels-as', 'crunching', 'karyo', 'inappropriately', 'make-believe', 'unquestioning', 'puffy', 'lobsters', "hawks'", 'wondrously', 'adulthood', "mo'nique", 'models', 'yar', 'less-than-honorable', 'lilith', 'paradise', 'eastwood-esque', 'legitimacy', '______', 'deceiving', 'clude', 'sergeants', '`macho', 'frequency', '2470', 'cleese', 'rachel', 'glowering', '135', 'blurts', 'alarm', 'macbeth', 'james-younger', 'surprised', 'lengths', 'lonliness', 'hog', 'paranoiac', 'genuinely-funny', 'macht', 'abolish', '1932', 'non-disney', "hanks's", 'impersonations', 'ever-growing', 'lead', "cage's", 'downright', 'boxes', 'fanatically', 'psychedelia', "'snow", 'anxieties', 'pa', 'siff', 'dumas', 'petitions', "film'", 'weirdo', 'tripe', 'fun-making', 'delusional', 'rogue', 'goofups', 'joiner', 'soared', 'denueve', '103-minute', 'plotlines', 'vendor', 'lords', 'girlina', 'lina', 'columbus', "douglas's", 'rec', 'adrian', 'wo-ping', 'weighted', 'buttercup', 'requiem', 'winde', 'translate', '18th', 'toni', 'straining', 'neo-noir', 'justify', "tykwer's", "teri's", 'permeated', 'presumes', 'joplin', 'far-right', 'diatribes', 'honey-soaked', 'boot', 'meryl', 'steamy', 'promo', 'brattish', 'darwinism-', 'high', 'nervous-wreck', 'sick', "grandparents'", '_they', 'plath', 'wide-open', 'mournful', 'plight', 'guillaume', 'baffle', 'headphones', 'knutt', 'conceptions', 'mexican', 'glenn', 'unwitting', '126', 'speech-impaired', '`batman', 'ultra-violence', 'suspense/science', 'organizers', 'wrapping', "stiller's", 'timidity', 'jupiter', 'chortled', 'begs', 'aurore', 'helluva', 'magyuver', 'exhibit', 'shrinks', 'digimon', 'firefight', 'matarazzo', 'control', "ira's", 'ex-preacher', 'sleeze', 'naville', "odile's", 'winters', 'well-crafted', 'lip', 'drawer', 'underwear', 'broadcasts', 'exploited', 'penuries', 'gimmicky', 'exact', 'guilt', "hartnett's", "'cause", 'merciless', 'servants', 'abusing', 'apprehension', 'radioactivity', 'canister', 'billington', "wilder's", 'miss-directed', 'loutish', 'suspends', 'filmcritic', 'complication', 'laden', 'explantion', 'oderkerk', 'dominican', 'self-evidently', 'scale', 'embedded', 'products', 'trier', 'hiberli', 'addy', 'cohorts', 'one-upmanship', 'de-mystify', 'earthquakes', "althea's", 'protagonists', 'motor-mouthed', 'liars', 'strangers', "'or", 'pectorals', 'vampirism', 'apologetically', 'decoding', 'cubans', 'afore-mentioned', 'altar-boy', 'gullible', 'disrespectful', 'burgles', 'playwright', 'terrace', 'matewan', 'jolie', 'jelly-donut', 'wings', 'anti-establishment', "doubt's", 'actualizing', 'clifton', 'succinctly', 'indebted', 'sandstorm', 'liberals', 'initally', 'pointless', 'cholera', 'doo', 'infiltrate', 'mourn', 'chop', 'diner', 'cavalry', 'kitsch', 'wipers', 'is--horror', 'twenty-minute', 'rotund', 'isolationism', 'alexander', 'once-in-a-lifetime', 'illeana', 'magnified', 'poseidon', 'humanoid', 'sleeplessness', 'rampant', 'non-professional', 'emanated', 'spikes', '$100', 'waiter/struggling', 'fictious', 'boyum', 'recollection', 'epitaph', 'blown-up', 'redman', 'spectators', 'pears', 'conformity', 'abode', 'drums', 'looney', 'cloning', 'egg-shaped', 'tugs', 'incorporated', 'meditative', 'batologist', 'periodic', 'vaughan', 'julliana', 'slinging', 'break-dance', 'male', 'bicker', "steven's", 'college-dormitory', 'sincerity', 'birds', 'bigots', "beholder'", "d'artagnan's", 'engelman', 'absentee', 'groupies', 'occassionally', 'delicately-narrated', 'hectic', 'pontificate', "renfro's", 'good-guy', 'specimens', 'crabby', 'serene', 'adolescent-minded', 'cliff', 'marines', 'wormwood', 'blow-out', 'rourke', 'dizzy', 'madoka', 'dubbed', 'invisibility', 'dorff', 'jewels', 'home-made', 'effects-loaded', 'skids', 'gold', 'prevail', 'timo', 'daena', 'chosen', 'mistress--but', 'semler', 'issiah', 'wades', 'swingers', 'high-', 'top-40', 'untraditional', "nick's", 'gurgle', 'complimentary', 'karenina', 'contributing', 'objections', 'pre-war', 'llama', 'exhibition', 'vassili', 'allayah', 'aquits', 'food-fight', 'salesgirl', 'morales', 'first-string', "wang's", 'nearly-', 'extremly', 'rhythmic', 'rich', 'thumbing', 'kaplan', 'stings', 'edginess', 'slipped', 'indubitably', 'cajun', "clueless's", 'artemus', 'catharine', 'attract', 'gymnasium', 'fly-overs', 'rider', 'policeman', 'yitzhak', 'mason', 'tame', 'robbery--he', 'rocket', 'cameramen', 'ground-breaking', 'tons', 'embarrasses', 'wildfire', 'titshots', 'braces', 'effect-dependent', 'ghostbuster', 'specialist', 'suitors', "lawyer's", 'exploitatively', "1979's", 'keeve', 'asbestos', 'ritual', 'muddy', 'bass-pounding', 'bikers', 'covetous', 'rent-a-', 'curmudgeon', 'philistine', 'rightly', 'long-winded', 'credits--who', 'pyle', 'too-pleasant', 'quenches', 'one-scene', 'smiled', 'fluoro', 'illegally-acquired', 'whoa', 'analyzing', 'catastrophes', 'stifler', 're-start', 'divide', 'pry', 'di$ney', "chaja's", 'illustrating', 'man-whoring', 'katanga', 'bitch', 'stiffs', 'islands', 'meudon', "greg's", '17-year', 'brooder', 'resolving', 'controled', 'behold', '_have_', 'well-equipped', 'protest', 'out-right', 'boyhood--and', 'capsule', 'liz', 'orville', 'dialogue-those', 'coveted', 'co-write', 'troyer', 'yeomen', 'bad-boy', 'graceful', 'saps', 'list', 'ahhh', 'not-so-secretly', 'biohazard', 'rabidly', 'norm', 'injured', "busey's", 'irresistible', 'akiva', 'primal', 'rooted', 'hunger', 'blusterous', 'invaluable', "american'", 'valuable', 'bystander', 'prof', 'sacrilege', 'farquaads', 'irreverent', "boss's", "atrocity's", 'reclaims--her', 'fleiss', 'vaccaro', 'louise', 'referring', 'worthiness', 'foley', 'durden', 'drawbacks', 'unprofessionalism', 'uncovering', 'unveils', 'mortician', 'things/sprites', 'heterosexuals', "shaft's", 'choke', 'subversive', 'fiel', 'philisophical', 'episodes', 'television-addicted', 'toothless', 'inter-office', 'decreases', "williamson's", 'declines', 'succumbs', 'ludicrous', 'brinkford', 'gag', 'bandwagon', 'preach', 'pirahna', 'worry', 'complaining', 'long-awaited', 'couplet', 'populated', 'mighty', 'hillary', 'swashbuckling', 'chriqui', 'blatz', 'hen-pecked', 'abyss', 'corporal', 'gauge', 'ancient-swords', 'strengthen', 'soap', 'hines', 'supremacist', 'fingerprint', 'unexpected', 'taboos', 'isles', 'stabbed', 'salutes', 'precocious', 'complements', 'terrorist', 'eggert', 'wisconsin', 'llosa', 'monolith', 'amounted', 'sludge', "d'etat", 'retro-future', 'real-looking', 'good-idea-turned-bad', "estella's", "sherlock's", 'edgecomb', 'cynical', "sam's", 'collap-', 'moss', 'chimps', 'rydell', 'cecile', 'ejection', 'tacked-on', 'elgar', 'money-grubbing', 'godfather-films', 'static', 'fool', 'abba', "anybody's", 'bikini', 'fetale', 'overdramaticized', 'remark', 'ortega', 'imprint', 'blue-lighted', "2001's", 'adulterous', 'limited', 'mothers-in-law', 'nien-chen', 'clips', 'professor-student', 'stars', 'idolize', 'budge', 'hank', 'often-times', 'girds', 'predetermine', 'anti-vampire', 'mainstays', 'killathon', 'critique', 'shelved', 'pitch-perfect', 'day\x12s', 'bumpers', 'rational', 'endure', 'morgue', 'eh', "rock'", 'rays', '1997--the', 'messenger', 'marvel', 'weighty', 'gross-fest', "stevenson's", 'lyrics', 'mccole', 'national', "mike's", 'roused', 'dame', 'curtailing', 'tiffany', 'nonprofessional', 'positioning', 'obligated', 'professed', 'four-some', 'rustic', 'clothing', 'interferes', 'piscapo', 'taint', '_great_', 'coco', 'villages', "giacomo's", 'reserve', 'gon', 'cream-filled', 'rod', 'hard-', 'deflection', 'plays', 'feelgood', 'kelogg', 'abundance', 'down-at-the-heels', 'liebes', 'acting-related', 'stacked', 'toast', 'haley', 'salaries', 'improvise', 'title--it', 'compartmented', 'decrement-life-by-90-minutes', 'persists', 'actors/actresses', 'zaillian', 'hurting', 'relics', 'mall', 'compose', 'repute', 'week', 'bury', 'napolean', 'misleading', 'well-populated', 'phase', 'mountain-side', 'propelling', 'wets', 'link', 'cramp', 'zombies', "postman's", 'condescension', 'festivals', 'twenty-something', 'cadillac', 'defeaningly', 'funnymen', 'semi-thorough', "colqhoun's", 'habitual', 'shaking', 'play-by-play', 'jar', 'brashness', 'chanteuse', 'hernandez', 'recurrent', 'impotent', 'quilty', 'feared', 'imitators', 'argo', 'us$130', 'theses', 'consulted', 'wilds', 'justly', "friend's", "cop's", 'attendants', 'darlene', 'humiliations', "hitler's", "d'", 'rely', 'libra', 'near-fully', 'cluelessness', 'noggin', 'sealed', 'voyeurism', 'lion', 'cambridge', 'three-plus', 'character-driven', 'sequences', 'modelling', 'sundowns', 'farrel', 'letterman', 'defect', 'musicals', 'plants', 'mimmo', 'imam', 'tahiti', 'straightest', 'coupon', 'mohicans', 'conspires', 'stumped', 'britain', 'phoniness', 'half-thought-through', 'chatters', 'hyper-kinetic', 'gruesome', 'marino', 'extemely', 'mid-january', 'wrack', 'steven', 'craven', 'cheekbones', 'english-speaking', 'allen', 'boils', 'shifted', 'worn-out', 'bends', 'narrowly', 'hold-your-breath', 'likeablity', "epp's", "_mind's", 'tribe', 'finley', 'sexism', 'movieplex', 'sergeant', 'stemple', 'obituary', 'slogging', 'unleashed', "ex's", 'responsive', 'jump', 'mcdermott', 'mythic', 'pipe', 'longo', 'quirk', 'overestimates', 'consumes', 'cunningham', 'castrate', 'unneccesary', 'transmitting', 'rentals', "meyer's", 'wendy', 'tackled', 'disservice', 'co-protagonist', 'grills', 'sappy', 'round-up', 'yuppy', "cab's", 'three-hour', 'american-release', 'legacy', 'prided', 'robbins', 'cerebrality', 'chicago-sun', 'seventeen', 'coinage', 'chieftain', 'poured', 'brenneman', 'crush', 'derisive', 'disgraceful', 'nicholson-blown-down-the-street', 'tapped', 'insulting', 'pounded', 'replenishing', 'marriageable', 'account', 'harasses', 'handle', 'gambled', '17-year-old', 'dices', 'boys', 'orpheus', 'hades', 'spicebus', 'ragtop', 'yen-khe', 'vice', 'non-glamorous', 'clockwork', 'revisted', 'prunella', 'andy', "heffron's", 'scenario', 'glaring', 'pig-tails', 'tapping', 'taxation', 'dissolute', 'kingsly', 'not-quite-subtle', 'insecurity', 'superviser', 'calibre', 'unnoticeably', 'villains', 'marvin', 'vocalized', 'infatuation', 'teddies', 'golds', 'threated', 'stentorian', 'representative', 'screams', 'adeptness', 'guido', 'berkshire', 'stab', 'sept', 'deux', 'reality', 'myrlie', 'echelon', 'slumming', 'peak', 'winter', 'sorta', 'polo/ralph', 'trust', 'potatohead', 'coats', 'fatboy', 'saxophone', 'drips', 'kip', 'folly', 'sitting-ducks', 'unglamorous', "'what", "camille's", 'often-incongruous', 'encore', 'overbeck', 'nonjudgemental', 'sensation', 'never-seen', 'judge', 'solving', 'pencilled', 'dominated', 'suicidal', 'half-', 'tolerable', 'triple-digit', 'forwards', 'walkway', 'dussander', "'77", "when's", 'maurice-jones', 'eating', 'janeane', 'strain', 'lobby', 'slime-splattering', 'frog', 'flashiest', 'damp', 'ooooo', 'retention', 'pledge', 'slender', "lam's", 'enrich', 'unchanged', "lookin'", 'happening', 'gunfire', 'mastabatory', 'putzes', 'stoolie', 'overstatement', 'hm', 'non-cgi', 'incites', 'fennymann', 'oppose', 'gravedigging', 'stroll', '96-minute', 'arizona', 'overpowered', 're-release', 'quitting', 'back-seat', 'upwards', 'seller', 'ether', 'psychological-romance-thriller', 'anemic', 'karaoke', 'touts', 'firms', 'salonga', 'eyepopping', 'muddling', "brooks'", 'helpfulness', 'hallucinogen-fueled', 'fission', 'formidably', "pair's", 'stuck-up', 'itami', 'campfest', 'wiez', '---', 'lynchian', 'derbies', 'ames', 'ernie', 'to-swallow', "`elvis'", "'did", 'employers', 'devoured', 'minghella', 'brogue', 'expressive', "terel's", 'drunks', 'kaye', 'stopover', 'gesserits', 'testicle', 'antagonizes', 'sirajul', 'estates', 'guaranteeing', 'orgies', 'lightning-lit', 'tarkovskian', 'stagebound', 'naboo', 'waystation', 'paradine', 'seaside', 'pam', 'self-assurance', 'backdraft', 'stole', 'scarred', 'awareness', "hulot's", 'disappoint', 'missy', 'weaknesses', 'culprit', "cookie's", 'mah', 'capano', 'forsee', 'attendance', 'line-', 'accosted', 'prescribed', 'vat', 'mis-casting', 'yessuh', 'bombay', 'moping', 'impersonated', "fuckin'", 'janevski', 'catatonia', 'recruiting', '10-year-old', 'mindless', 'benefited', 'ribbing', 'manageable', "kayley's", 'not-too-precocious', 'spicy', 'nan', '300+', 'kevlar', 'insist', 'casts', 'tabloid-ish', 'activities', 'residents', 'aunts', 'sweatshirt', 'pseudo-legendary', 'fend', 'reinvent', 'callisto', 'sociopathic', "fanny's", 'ellie', 'reflexivity', 'father/son', 'enjoying', 'stork', 'hayworth', 'balrogs', 'connotes', "mouseketeer's", 'chock-full', 'lochlyn', 'elizabethian', 'santi', 'bypassed', 'walter', 'swindling', 'exhausting', 'beales', 'corralled', 'armor', 'hone', 'turkeys', 'mapped', 'inventors', 'millimeter', 'joke-machine', 'side-splittingly', 'censored', 'scream-esque', 'overtakes', 'circumcision', 'keitel', 'absurdism', 'calculating', 'unjust', 'good-looker', "john's", 'financial', 'cutlery', 'mk', 'ever-present', 'dykstra', "syd's", 'overexcited', 'lether', 'break-ups', 'pervades', 'outmatches', 'double-barreled', 'stupid', 'twenty-one', 'annoyed', 'gothic', 'inopportune', 'sermonizing', 'livestock', "devlin's", 'reducing', 'korean', "'freedom'", 'mull', 'sentenced', 'scofield', 'good-film-making', 'unbiased', 'overseer', 'here--', 'non-titillating', 'troubling', 'sound-wave', 'maniac', 'alba', 'screw-ups', 'criminals', 'woefully', 'schoonmaker', 'calif', '2000-the', "kennedy's", 'hennessey', 'unplugging', 'raoul', 'virgo', 'precinct', 'consonants', 'old-time', 'honorary', "villain's", "mib's", '[beware', 'extrapolation', 'lollipops', 'copy-machine', 'intrusion', 'dinosaur', 'halle', 'uncontrolled', 'koji', 'well-photographed', 'folks', 'dick', "massee's", 'bullet-riddled', 'gimmick', 'sensual', 'talons', 'vulcan', 'cult-status', 'hammered', 'imf', 'profiling', "close's", 'jewison', 'rundown', 'toto', 'rhapsodic', 'amorous', 'utterance', 'fleet', 'incubator', 't-bird', 'europop', 'diners', 'bearings', "frequency's", 'spinster', 'grandparents', 'couples', 'americana', "station's", 'unique', 'exploits', 'boucher', 'porter', 'vacationing', 'potty-mouthed', 'ugh', 'lousy', 'identification', 'pastier', 'tongue-lashing', 'qb', 'democrat', 'con-team', 'emphasized', 'debauchery', 'birdsall', 'villiage', 'repartee', 'mathilda', 'je', 'undercover', 'dougherty', 'kleiser', 'consummating', 'constitutionality', 'macdonald', 'flexibility', 'predominant', '_ferris', 'well-deserved', 'nicely--', "dow's", 'pinnacle', 'flaunting', 'scout', 'palpably', 'catholics', 'shaw', 'ouija', 'prompt', 'consensus', 'elitists', 'bluish', 'annoyance', 'cater', 'defy', 'salomon', 'b+', "chad'z", 'second-half', "she'll", 'gatewood', 'intresting', 'stomping', 'propoganda', 'foretaste', 'playhouse', 'additions', 'america', 'gracefully', "alcott's", "denver's", 'irregular', 'incorporates', 'fabricates', 'frixos', 'storytelling', 'god-daughter', 'justifies', 'ultimatum', 'finzi', 'viewer', 'gangs', 'wave', 'onslaught', 'twenty-foot-tall', 'heaped', "o'byrne", 'fates', 'slithers', 'oshii', 'swirling', 'mayer-goodman', 'endeavours', 'demanded', 'iliopulos', 'empath', "crisis'", 'colossal', "'real'", 'formulaism', 'iron', 'guideline', 'psychopathic', 'telegraph', 'dispute', "studios'", 'recieves', 'independent', 'colonialists', 'italians', 'hanna', 'superpowers', 're-introduce', 'goldfinger', 'uncompromising', 'mizrahi', 'underlined', '12-fingered', "fellini's", 'polynesian', 'pindar', 'ill-fated', 'over-medicated', 'brushes', 'breakthrough', "'best", 'insides', "o'sullivan", 'barred', 'inefficiencies', 'actively', 'onassis', 'mahal', 'preexisting', 'robin', 'wow', 'emerged', 'determination', 'kilmer', "d's", 'insignificant', 'youre', 'civic', 'depardieu', 'coz', 'plumber', 'elliott', 'worshiper', 'twinkle', 'gaul', 'fighting', 'ugly-duckling', 'swears', 'request', 'staining', 'in-joke', 'flogging', 'headset', 'faster-than-light', 'hesitantly', 'virus', 'maud', 'cottwell', 'tactfully', 'fangs', 'ex-wife', 'biggie', 'listed', 'norbu', 'write', 'wrist', 'custody', '===============', "fort's", 'falsely', 'glencoe', 'speared', 'famed', 'organisation', "mark's", 'vore', 'business-oriented', 'parisian', 'deadening', 'kewpies', "'50s", 'rethink', 'niall', 'hourglass', 'non-judgemental', 'spanner', 'grouchy', 'fontaine', 'worst', 'concentrating', 'disregard', 'recommendations', 'vans', 'porpoise', 'ravished', 'spectacle', 'frappacino', 'wire', 'induce', 'late-seventies', 'well-voiced', 'grandfather', 'denounced', "mankind's", 'guffaws', "lean's", 'ad', "evil's", 'incarnate', 'screwed', 'counfound', 'hauls', 'lebowski', 'nazism', 'afforded', 'admittance', 'presently', 'clarissa', 'separate', "his`plump'", 'wayne/batman', 'accomplishment', 'comet-disaster', "pbs'", 'stanley', 'oily', 'adult-david', 'one-syllable', 'pear', "garofalo's", 'genre', 'elevates', 'malls', 'germane', 'performence', 'synchronized', 'family/friends', 'splatter', 'origin', 'exuberance', 'grafted', 'stein', 'tickling', '21st-century', 'clean-cut', 'bearded', 'cannon', 'rosenthal', 'escapade', "anna's", 'darcy', "burns'", 'cobbles', 'ambulance-chasing', 'spectacularly', 'vladimir', 'newsleak', 'sommeliers', 'interrogating', 'uncertainty', 'graduation-specifically', 'odd-numbered', 'kodak', 'cortez', 'shacks', 'yakin', 'bellamy', 'reproduced', 'fatherless', 'peccadilloes', "nicholson's", 'enter', 'hassle', 'high-rise', 'allowing', 'invites', 'takehiro', "investigator's", 'coincidental', 's-l-o-w', 'agent/pop', 'congo', 'condon', 'parodic', 'bregman', 'thinks', "vincent/jerome's", 'cates', 'saving', 'unbreakable', 'afer', "conclusion's", 'bribery-based', 'powers-of-10', "oscar's", 'submerging', 'expections', "bellboy's", 'carmine', 'all-night', "gyllenhall's", 'slab', 'lacking', 'storh', 'alignments', 'dumps', 'chappelle', 'turn-off', 'hackwork', 'coherent', 'fenn', 'dynasty', 'act/speak', 'semi-offensive', 'yemeni', 'kidnaped', 'swoops', 'post-industrial', 'zombified', 'ex-spook', "mclachlan's", 'descent', 'cop-who-sees-ashley-fleeing-an-accident-scene-and-', 'visa', 'chased', 'revenge--and', 'psychological'}))


TRAIN
############################################################
OK	train({'neg': ['./data/imdb1/neg/cv000_29416.txt', './data/imdb1/neg/cv001_19502.txt'... 
	=> ({'neg': 900, 'pos': 900}, {'neg': {'\x14': 2, '\x13ripley': 1, '#': 1, '"': 697, '#$%': 1, '\x13if': 1, '\x13earth': 1, '!': 341, '\x13they': 1, '\x16': 2}, 'pos': {'"': 695, '#15': 1, '\x05': 1, '#5': 1, '#2': 6, '#6': 1, '#3': 1, '#7': 1, '#1': 3, '!': 256}}, {'maneuvering', 'lout', 'provocative', 'newcomers', 'synth-heavy', 'slides', 'morte', 'white-knuckle', "laugh'", 'punisher', '...'})
	== ({'neg': 900, 'pos': 900}, {'neg': {'\x14': 2, '\x13ripley': 1, '#': 1, '"': 697, '#$%': 1, '\x13if': 1, '\x13earth': 1, '!': 341, '\x13they': 1, '\x16': 2}, 'pos': {'"': 695, '#15': 1, '\x05': 1, '#5': 1, '#2': 6, '#6': 1, '#3': 1, '#7': 1, '#1': 3, '!': 256}}, {'maneuvering', 'lout', 'provocative', 'newcomers', 'synth-heavy', 'slides', 'morte', 'white-knuckle', "laugh'", 'punisher', '...'})



## 7.3 Klasifikacija

Napravi funkciju **classify()** po sljedećim ulaznim i izlaznim podacima
* ulaz:
    * **model** trenirani model kojeg vraća funkcija `train()`
    * **filename** datoteka koja se klasificira
    * **stopwords** stop-riječi
* izlaz:
    * **klasa**

Vjerojatnost dokumenta $d$ za klasu $c$ se računa po Naivnom Bayesu uz dodaj-1 izglađivanje

$$ P(d|c) = p(c) \prod_{w \in d}{p(w|c)}$$ gdje je

$$ P(w|c) = \frac{br(w,c) + 1}{br(c) + |V|}$$

Na kraju, odabrana klasa se traži po

$$ c_{max} = \text{argmax}_{c \in C} P(d | c)$$

Radi preciznosti, koristiti logaritamski prostor za izračun $P(d|c)$ odnosno

$$ log(P(d|c)) = log(p(c)) + \sum_{w \in d}{log(p(w|c))} $$


In [58]:
def classify(model, filename, stopwords):
    """
    ulaz:
    -train_model: trenirani model kojeg vraća funkcija build_train_model()

    izlaz:
    -megadoc_model: je rječnik čiji su ključevi klase, a vrijednosti megadokumenti kao unigrami
    nastali "spajanjem" svih unigrama iz treniranog modela za određenu klasu
    """
    prior, megadoc, vocabulary = model    
    words = read_document(filename, stopwords)
    
    prob = {}
    for c in prior:
        prob[c] = math.log(prior[c])
        for w in words:

            p = (megadoc[c].get(w, 0) + 1) / (sum(megadoc[c].values()) + len(vocabulary))
            prob[c] += math.log(p)
            
    return max(prob, key=prob.get)
        

testname(classify)
_ = test(classify, (model, datasets[1]["test"]["pos"][0], stopwords), "pos")
_ = test(classify, (model, datasets[1]["test"]["neg"][0], stopwords), "neg")



CLASSIFY
############################################################
OK	classify(({'neg': 900, 'pos': 900}, {'neg': {'\x14': 2, '\x13ripley': 1, '#': 1, '"': 69... 
	=> pos
	== pos

OK	classify(({'neg': 900, 'pos': 900}, {'neg': {'\x14': 2, '\x13ripley': 1, '#': 1, '"': 69... 
	=> neg
	== neg



## 7.4 Unakrsna validacija

Funkcija **cross_validate()** vrši unakrsnu validaciju.
Za svaki "savijutak" (engl. fold) će se trenirati klasifikator i testirati preciznost.

In [60]:
def cross_validate(has_stopwords, counts):
    """
    ulaz:
    -has_stopwords: ima ili nema stop riječi
    -counts: lista očekivanog broja točno klasificiranih dokumenata za
             svaku unakrsnu podjelu podataka

    Koraci:
    1. Pripremi unakrsnu validaciju
    2. Ako ima stop riječi onda ih učitaj
    3. Za svaku unakrsnu podjelu podataka:
       3.1. Treniraj klasifikator na podacima za trening
       3.2. Klasificiraj na podacima za testiranje
       3.3. Izračunaj točnost klasifikatora za unakrsnu podjelu
    4. Izračunaj ukupnu točnost klasifikatora
    """
    print('[INFO] Preparing datasets...')
    datasets = make_datasets(DATA_PATH)
    if has_stopwords:
        print('[INFO] Reading stopwords...')
        stopwords = read_stopwords(STOPWORD_FILENAME)
    else:
        print("[INFO] No stopwords...")
        stopwords = set()
    print('[INFO] Training and classifying...')

    total_good, total_all = 0, 0
    for fold, dataset in enumerate(datasets):
        print("[TRAIN] Fold", fold)
        trainset, testset = dataset['train'], dataset['test']
        
        model = train(trainset, stopwords)
        print("[EVALUATE] Fold", fold)
        

        # evaluating
        count_good, count_all = 0, 0
        for klass, filenames in testset.items():
            counter = len(filenames)
            print(" ", klass, end=" ")
            for filename in filenames:
                predicted_klass = classify(model, filename, stopwords)
                count_good += 1 if predicted_klass == klass else 0
                count_all += 1
                counter -= 1
                if counter % 10 == 0:
                    print(counter, end=" ")
            print()
        total_good += count_good
        total_all += count_all

        success = count_good == counts[fold]
        success_txt = ' OK ' if success else ' X  '

        print('[%s] Fold %d: %d/%d\t%.1f%%' % (success_txt, fold, count_good, count_all, count_good/count_all * 100))
        print()

    print('[INFO] Accuracy %d/%d\t%.1f%%' % (total_good, total_all, total_good/total_all*100))

cross_validate(False, [161,168,167,165,167,165,169,167,158,171])

cross_validate(True, [162,168,168,164,167,166,168,168,153,170])

[INFO] Preparing datasets...
[INFO] No stopwords...
[INFO] Training and classifying...
[TRAIN] Fold 0
[EVALUATING] Fold 0
  neg 90 

KeyboardInterrupt: 